In [1]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np
import re

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = pd.read_csv("urls_final_v4.csv", sep="|")
print(df.shape)
df["full_text"] = df.parallel_apply(lambda x: str(x.raw_str) + " " + str(x.desc) + " " + str(x.title), axis=1)
df.full_text = df.full_text.parallel_apply(lambda x: re.sub("nan", " ", x))
df.full_text = df.full_text.parallel_apply(lambda x: re.sub(" +", " ", x))
df.full_text = df.full_text.parallel_apply(lambda x: x.strip())
df = df[df.full_text != '']
print(df.shape)
df.head()

(166135, 4)


(130946, 5)


,url_host,raw_str,desc,title,full_text
0,adriver.ru,Adriver – Система управления и аудита интерне...,NaN,Adriver – Система управления и аудита интернет...,Adriver – Система управления и аудита интернет...
1,apple.com,Apple Apple Apple Store Maci Padi Phone Watch...,Discover the innovative world of Apple and sho...,Apple,Apple Apple Apple Store Maci Padi Phone Watch ...
2,avatars.mds.yandex.net,410 Gone 410 Gone nginx,NaN,410 Gone,410 Gone 410 Gone nginx 410 Gone
4,i.ytimg.com,����,NaN,NaN,����
5,icloud.com,i Cloud,"Sign in to i Cloud to access your photos, vide...",i Cloud,i Cloud Sign in to i Cloud to access your phot...


# TFIDF

In [3]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from simplemma import lemmatize
nltk.download('stopwords')
nltk.download('wordnet')

# Stopword removal, converting uppercase into lower case, and lemmatization
stopwords = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('russian')
lemmatizer = WordNetLemmatizer()

def clean(text):
    text = re.sub('[^a-zA-Zа-яА-ЯЁё]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatize(word, lang=('ru', 'en')) for word in text if not word in set(stopwords)]
    # print(text)
    text = ' '.join(text)
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df['full_text_clean'] = df.full_text.parallel_apply(clean)
df = df[df.full_text_clean != '']
df = df.reset_index(drop=True) 
print(df.shape)
df.head()

(130778, 6)


,url_host,raw_str,desc,title,full_text,full_text_clean
0,adriver.ru,Adriver – Система управления и аудита интерне...,NaN,Adriver – Система управления и аудита интернет...,Adriver – Система управления и аудита интернет...,adriver система управление аудит интернет рекл...
1,apple.com,Apple Apple Apple Store Maci Padi Phone Watch...,Discover the innovative world of Apple and sho...,Apple,Apple Apple Apple Store Maci Padi Phone Watch ...,apple apple apple store maci padi phone watch ...
2,avatars.mds.yandex.net,410 Gone 410 Gone nginx,NaN,410 Gone,410 Gone 410 Gone nginx 410 Gone,gan gan nginx gan
3,icloud.com,i Cloud,"Sign in to i Cloud to access your photos, vide...",i Cloud,i Cloud Sign in to i Cloud to access your phot...,cloud sign cloud access photo video document n...
4,m.avito.ru,Доступ ограничен: проблема с IP Доступ ограни...,NaN,Доступ ограничен: проблема с IP,Доступ ограничен: проблема с IP Доступ огранич...,доступ ограниченный проблема ip доступ огранич...


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=768)
vectors = vectorizer.fit_transform(df.full_text_clean.tolist())

In [32]:
print("n_samples: %d, n_features: %d" % vectors.shape)
# n_samples: 131984, n_features: 1845415

n_samples: 130778, n_features: 768


In [35]:
df = pd.concat([df.reset_index(drop=True), pd.DataFrame(vectors.todense(), columns=[f"tfidf_{i}" for i in range(768)])], axis=1)
df.head()

,url_host,raw_str,desc,title,full_text,full_text_clean,labse_0,labse_1,labse_2,labse_3,...,tfidf_758,tfidf_759,tfidf_760,tfidf_761,tfidf_762,tfidf_763,tfidf_764,tfidf_765,tfidf_766,tfidf_767
0,adriver.ru,Adriver – Система управления и аудита интерне...,NaN,Adriver – Система управления и аудита интернет...,Adriver – Система управления и аудита интернет...,adriver система управление аудит интернет рекл...,-0.065302,-0.009131,-0.005315,0.001041,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.071416,0.0,0.0
1,apple.com,Apple Apple Apple Store Maci Padi Phone Watch...,Discover the innovative world of Apple and sho...,Apple,Apple Apple Apple Store Maci Padi Phone Watch ...,apple apple apple store maci padi phone watch ...,-0.066337,0.015036,0.009291,-0.039725,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,avatars.mds.yandex.net,410 Gone 410 Gone nginx,NaN,410 Gone,410 Gone 410 Gone nginx 410 Gone,gan gan nginx gan,0.014214,-0.024987,-0.020434,0.031122,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
3,icloud.com,i Cloud,"Sign in to i Cloud to access your photos, vide...",i Cloud,i Cloud Sign in to i Cloud to access your phot...,cloud sign cloud access photo video document n...,-0.009940,-0.005152,-0.048339,-0.064440,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
4,m.avito.ru,Доступ ограничен: проблема с IP Доступ ограни...,NaN,Доступ ограничен: проблема с IP,Доступ ограничен: проблема с IP Доступ огранич...,доступ ограниченный проблема ip доступ огранич...,-0.067474,-0.045772,-0.023864,-0.038760,...,0.0,0.0,0.0,0.0,0.0,0.093088,0.0,0.000000,0.0,0.0


# FastText

In [40]:
import fasttext.util
import fasttext

fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('cc.en.300.bin')
fasttext.util.reduce_model(ft, 300)

In [41]:
def sen_fasttext(text):
    tokens = text.split()
    token_emb = np.mean([ft.get_word_vector(el) for el in tokens], axis=0)
    return pd.Series(token_emb)

In [43]:
df[[f"fasttext_{i}" for i in range(300)]] = df.full_text_clean.parallel_apply(sen_fasttext)
df.head()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_1546/2063623032.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f"fasttext_{i}" for i in range(300)]] = df.full_text_clean.parallel_apply(sen_fasttext)
/tmp/ipykernel_1546/2063623032.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[[f"fasttext_{i}" for i in range(300)]] = df.full_text_clean.parallel_apply(sen_fasttext)
/tmp/ipykernel_1546/2063623032.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  

,url_host,raw_str,desc,title,full_text,full_text_clean,labse_0,labse_1,labse_2,labse_3,...,fasttext_290,fasttext_291,fasttext_292,fasttext_293,fasttext_294,fasttext_295,fasttext_296,fasttext_297,fasttext_298,fasttext_299
0,adriver.ru,Adriver – Система управления и аудита интерне...,NaN,Adriver – Система управления и аудита интернет...,Adriver – Система управления и аудита интернет...,adriver система управление аудит интернет рекл...,-0.065302,-0.009131,-0.005315,0.001041,...,-0.016087,0.009356,-0.003855,0.001606,-0.023376,-0.000371,-0.016116,-0.003463,0.004774,0.006424
1,apple.com,Apple Apple Apple Store Maci Padi Phone Watch...,Discover the innovative world of Apple and sho...,Apple,Apple Apple Apple Store Maci Padi Phone Watch ...,apple apple apple store maci padi phone watch ...,-0.066337,0.015036,0.009291,-0.039725,...,0.004246,0.006491,-0.000368,-0.000908,-0.002780,0.028508,0.021720,0.014457,0.027702,-0.025579
2,avatars.mds.yandex.net,410 Gone 410 Gone nginx,NaN,410 Gone,410 Gone 410 Gone nginx 410 Gone,gan gan nginx gan,0.014214,-0.024987,-0.020434,0.031122,...,0.100044,0.142225,-0.027569,0.035003,0.001267,-0.095813,-0.080865,-0.068084,0.106518,-0.067822
3,icloud.com,i Cloud,"Sign in to i Cloud to access your photos, vide...",i Cloud,i Cloud Sign in to i Cloud to access your phot...,cloud sign cloud access photo video document n...,-0.009940,-0.005152,-0.048339,-0.064440,...,-0.011047,0.029113,-0.003276,-0.015779,-0.005441,0.029033,0.036012,0.006069,-0.012162,-0.003867
4,m.avito.ru,Доступ ограничен: проблема с IP Доступ ограни...,NaN,Доступ ограничен: проблема с IP,Доступ ограничен: проблема с IP Доступ огранич...,доступ ограниченный проблема ip доступ огранич...,-0.067474,-0.045772,-0.023864,-0.038760,...,-0.031588,-0.007591,-0.001082,-0.065627,0.056277,-0.017403,0.015719,-0.022057,-0.013534,0.014634


# transformers

In [39]:
# from sentence_transformers import SentenceTransformer

# sentences = ["This is an example sentence", "Each sentence is converted"]

# model = SentenceTransformer('sentence-transformers/LaBSE')
# embeddings = model.encode(sentences)
# print(embeddings)

In [5]:
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
model = model.cuda()

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
sentences = df.full_text.tolist()

In [7]:
bs = 256
embeddings = np.zeros(shape=(df.shape[0], 768))
with torch.no_grad():
    for i in tqdm(range(df.shape[0] // bs + 1), total=df.shape[0] // bs + 1):
        encoded_input = tokenizer(sentences[i*bs : min(df.shape[0], (i+1)*bs)], padding=True, truncation=True, max_length=512, return_tensors='pt')
        encoded_input = encoded_input.to("cuda:0")
        model_output = model(**encoded_input)
        bs_embeddings = model_output.pooler_output
        bs_embeddings = torch.nn.functional.normalize(bs_embeddings)
        bs_embeddings = bs_embeddings.to("cpu").detach().numpy()
        embeddings[i*bs : min(df.shape[0], (i+1)*bs)] = bs_embeddings

100%|█████████████████████████████████████████| 511/511 [14:49<00:00,  1.74s/it]


In [8]:
df = pd.concat([df, pd.DataFrame(embeddings, columns=[f"labse_{i}" for i in range(768)])], axis=1)
df.head()

,url_host,raw_str,desc,title,full_text,full_text_clean,labse_0,labse_1,labse_2,labse_3,...,labse_758,labse_759,labse_760,labse_761,labse_762,labse_763,labse_764,labse_765,labse_766,labse_767
0,adriver.ru,Adriver – Система управления и аудита интерне...,NaN,Adriver – Система управления и аудита интернет...,Adriver – Система управления и аудита интернет...,adriver система управление аудит интернет рекл...,-0.065302,-0.009131,-0.005315,0.001041,...,0.003185,-0.044516,0.039780,-0.055606,-0.016112,0.027759,-0.056224,0.021654,-0.018652,-0.059592
1,apple.com,Apple Apple Apple Store Maci Padi Phone Watch...,Discover the innovative world of Apple and sho...,Apple,Apple Apple Apple Store Maci Padi Phone Watch ...,apple apple apple store maci padi phone watch ...,-0.066337,0.015036,0.009291,-0.039725,...,-0.005772,-0.043329,-0.015615,0.009778,-0.014170,-0.011154,-0.060273,0.047447,-0.056488,-0.057837
2,avatars.mds.yandex.net,410 Gone 410 Gone nginx,NaN,410 Gone,410 Gone 410 Gone nginx 410 Gone,gan gan nginx gan,0.014214,-0.024987,-0.020434,0.031122,...,-0.052355,0.035958,-0.030161,-0.059215,-0.052603,0.009592,0.004936,-0.038002,-0.025528,-0.026502
3,icloud.com,i Cloud,"Sign in to i Cloud to access your photos, vide...",i Cloud,i Cloud Sign in to i Cloud to access your phot...,cloud sign cloud access photo video document n...,-0.009940,-0.005152,-0.048339,-0.064440,...,0.011751,0.026023,0.029160,-0.044582,-0.027387,0.005563,-0.065146,0.032249,-0.059411,-0.052451
4,m.avito.ru,Доступ ограничен: проблема с IP Доступ ограни...,NaN,Доступ ограничен: проблема с IP,Доступ ограничен: проблема с IP Доступ огранич...,доступ ограниченный проблема ip доступ огранич...,-0.067474,-0.045772,-0.023864,-0.038760,...,-0.046360,0.038973,-0.031076,0.042341,-0.030982,0.036858,0.003289,-0.009188,-0.030353,-0.066328


In [9]:
df.to_csv("urls_embs.csv", index=False, sep="|", errors='replace')

In [10]:
df = pd.read_csv("urls_embs.csv", sep="|")
df = df.drop(columns=['raw_str', 'desc', 'title', 'full_text', 'full_text_clean']) 
df.head()

,url_host,labse_0,labse_1,labse_2,labse_3,labse_4,labse_5,labse_6,labse_7,labse_8,...,labse_758,labse_759,labse_760,labse_761,labse_762,labse_763,labse_764,labse_765,labse_766,labse_767
0,adriver.ru,-0.065302,-0.009131,-0.005315,0.001041,-0.057083,0.017263,-0.005744,-0.024478,-0.019549,...,0.003185,-0.044516,0.039780,-0.055606,-0.016112,0.027759,-0.056224,0.021654,-0.018652,-0.059592
1,apple.com,-0.066337,0.015036,0.009291,-0.039725,0.002405,-0.017151,-0.032456,0.007454,-0.033175,...,-0.005772,-0.043329,-0.015615,0.009778,-0.014170,-0.011154,-0.060273,0.047447,-0.056488,-0.057837
2,avatars.mds.yandex.net,0.014214,-0.024987,-0.020434,0.031122,0.001695,-0.082969,0.005082,0.066988,-0.020748,...,-0.052355,0.035958,-0.030161,-0.059215,-0.052603,0.009592,0.004936,-0.038002,-0.025528,-0.026502
3,icloud.com,-0.009940,-0.005152,-0.048339,-0.064440,-0.037156,0.029459,-0.056131,-0.039256,-0.043743,...,0.011751,0.026023,0.029160,-0.044582,-0.027387,0.005563,-0.065146,0.032249,-0.059411,-0.052451
4,m.avito.ru,-0.067474,-0.045772,-0.023864,-0.038760,-0.032988,0.004582,-0.030629,-0.027126,-0.034709,...,-0.046360,0.038973,-0.031076,0.042341,-0.030982,0.036858,0.003289,-0.009188,-0.030353,-0.066328


In [11]:
urls = dict(zip(df.url_host.tolist(), df[list(df.columns)[1:]].to_numpy()))
print(len(urls))

130778


In [1]:
import pandas as pd
import pyarrow as pa

In [2]:
replace_urls = {
    'drive2.com': 'drive2.ru',
    'm.sport-express.ru': 'sport-express.ru',
    'auto.khb.acoola.ru': 'auto.acoola.ru',
    'auto.krasnoyarsk.acoola.ru': 'auto.acoola.ru',
    'auto.stavropol.acoola.ru': 'auto.acoola.ru',
    'animal.msk.acoola.ru': 'animal.acoola.ru',
    'realty.krasnodar.acoola.ru': 'realty.acoola.ru',
    'goods.msk.acoola.ru': 'goods.acoola.ru',
    'auto.orel.acoola.ru': 'auto.acoola.ru',
    'realty.spb.acoola.ru': 'realty.acoola.ru',
    'auto.ekburg.acoola.ru': 'auto.acoola.ru',
    'auto.novosibirsk.acoola.ru': 'auto.acoola.ru',
    'auto.simferopol.acoola.ru': 'auto.acoola.ru',
    'equipment.msk.acoola.ru': 'equipment.acoola.ru',
    'med.msk.acoola.ru': 'med.acoola.ru',
    'raw.msk.acoola.ru': 'raw.acoola.ru',
    'auto.voronezh.acoola.ru': 'auto.acoola.ru',
    'auto.tula.acoola.ru': 'auto.acoola.ru',
    'med.spb.acoola.ru': 'med.acoola.ru',
    'auto.irkutsk.acoola.ru': 'auto.acoola.ru',
    'auto.nnov.acoola.ru': 'auto.acoola.ru',
    'auto.kemerovo.acoola.ru': 'auto.acoola.ru',
    'auto.salekhard.acoola.ru': 'auto.acoola.ru',
    'job.spb.acoola.ru': 'job.acoola.ru',
    'auto.hm.acoola.ru': 'auto.acoola.ru',
    'auto.novgorod.acoola.ru': 'auto.acoola.ru',
    'auto.chelyabinsk.acoola.ru': 'auto.acoola.ru',
    'auto.krasnodar.acoola.ru': 'auto.acoola.ru',
    'animal.spb.acoola.ru': 'animal.acoola.ru',
    'auto.rostov.acoola.ru': 'auto.acoola.ru',
    'build.spb.acoola.ru': 'build.acoola.ru',
    'build.msk.acoola.ru': 'build.acoola.ru',
    'auto.spb.acoola.ru': 'auto.acoola.ru',
    'auto.msk.acoola.ru': 'auto.acoola.ru',
    'n5.kinosimka3.be': 'kinosimka.ru.net',
 'kinosimka.uproxy2.biz': 'kinosimka.ru.net',
 'm.kinosimka3.be': 'kinosimka.ru.net',
 'kinosimka.unblocked2.net': 'kinosimka.ru.net',
 'kinosimka.ru.net': 'kinosimka.ru.net',
 'kinosimka.123sumo.net': 'kinosimka.ru.net',
 'kinosimka.uproxy.co': 'kinosimka.ru.net',
 'film-kinosimka.ru.com': 'kinosimka.ru.net',
 'kinosimka.ru.com': 'kinosimka.ru.net',
    'domru.wi-fi.ru': 'domru.ru',
 'lkdomru.ru': 'domru.ru',
 'domru.ru': 'domru.ru',
 'lk-domru.ru': 'domru.ru',
 'omsk.domru.wi-fi.ru': 'domru.ru',
 'freedomrussia.org': 'domru.ru',
 'domru-cabinet.ru': 'domru.ru',
 'domru-info.ru': 'domru.ru',
 'domru.iagg.ru': 'domru.ru',
 'kingdomrushfrontiers.com': 'domru.ru',
 'domru-lk.ru': 'domru.ru',
    'cataloxy.ru': 'cataloxy.ru',
 'almaty.cataloxy-kz.ru': 'cataloxy.ru',
 'cataloxy-by.ru': 'cataloxy.ru',
 'minsk.cataloxy-by.ru': 'cataloxy.ru',
 'mogilev.cataloxy-by.ru': 'cataloxy.ru',
 'nursultan.cataloxy-kz.ru': 'cataloxy.ru',
    'hd-28.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'tv.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'lordfilm.gd': 'ttr8.lordfilm5.zone',
 'lf7.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'ru2.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'zr.lordfilm.fans': 'ttr8.lordfilm5.zone',
 'ak.lordfilms-s.tv': 'ttr8.lordfilm5.zone',
 'apilordfilms-s.tobaco.ws': 'ttr8.lordfilm5.zone',
 'ag-online.lordfilms-s.biz': 'ttr8.lordfilm5.zone',
 'lf5.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'lord.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'lordfilm0.com': 'ttr8.lordfilm5.zone',
 'lordfilm.frl': 'ttr8.lordfilm5.zone',
 'ad.lordfilms-s.tv': 'ttr8.lordfilm5.zone',
 'aj.lordfilms-s.tv': 'ttr8.lordfilm5.zone',
 'an.lordfilms-s.tv': 'ttr8.lordfilm5.zone',
 'lf4.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'lordfilm.pet': 'ttr8.lordfilm5.zone',
 'lordfilmx.biz': 'ttr8.lordfilm5.zone',
 '8k-2022.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm.fan': 'ttr8.lordfilm5.zone',
 'lordfilm6.rpltb.online': 'ttr8.lordfilm5.zone',
 'ko.lordfilm.frl': 'ttr8.lordfilm5.zone',
 'ao.lordfilms-s.tv': 'ttr8.lordfilm5.zone',
 'ru10.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'al-online.lordfilms-s.art': 'ttr8.lordfilm5.zone',
 'id22.lordfilm.codes': 'ttr8.lordfilm5.zone',
 'lordfilm.fans': 'ttr8.lordfilm5.zone',
 'lordfilm2.band': 'ttr8.lordfilm5.zone',
 'hdp.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'ac-online.lordfilms-s.art': 'ttr8.lordfilm5.zone',
 'bz.lordfilml.xyz': 'ttr8.lordfilm5.zone',
 'lord.lordfilm.cfd': 'ttr8.lordfilm5.zone',
 'lordfilm.gg': 'ttr8.lordfilm5.zone',
 'lv.lordfilm.lu': 'ttr8.lordfilm5.zone',
 '2022.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm-s.wiki': 'ttr8.lordfilm5.zone',
 'ws-100.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm2.rpltb.online': 'ttr8.lordfilm5.zone',
 'hd7.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'lord444.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm5.rpltb.online': 'ttr8.lordfilm5.zone',
 'lordfilm-2022.com': 'ttr8.lordfilm5.zone',
 'za.lordfilmax.net': 'ttr8.lordfilm5.zone',
 'lordfilms1.rpltb.online': 'ttr8.lordfilm5.zone',
 'lordfilms2.serialru.online': 'ttr8.lordfilm5.zone',
 'film.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'lordfilm-0.fun': 'ttr8.lordfilm5.zone',
 'lordfilmhd.tv': 'ttr8.lordfilm5.zone',
 'lf.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'sir.lordfilm.frl': 'ttr8.lordfilm5.zone',
 'hd-25.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'lf6.lordfilm.lu': 'ttr8.lordfilm5.zone',
 'we14.lordfilm-w.xyz': 'ttr8.lordfilm5.zone',
 'ava.lordfilmtv.cyou': 'ttr8.lordfilm5.zone',
 'id25.lordfilm.codes': 'ttr8.lordfilm5.zone',
 'film.2lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'id23.lordfilm.codes': 'ttr8.lordfilm5.zone',
 'lordfilm.ru.net': 'ttr8.lordfilm5.zone',
 'lordfilmtv.online': 'ttr8.lordfilm5.zone',
 'lord-44.lordfilm0.org': 'ttr8.lordfilm5.zone',
 'lordfilm-tv17.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'n6.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'lordfilmx-online.net': 'ttr8.lordfilm5.zone',
 'lordfilm2.site': 'ttr8.lordfilm5.zone',
 'q27.lordfilm.ltd': 'ttr8.lordfilm5.zone',
 'q26.lordfilm.ltd': 'ttr8.lordfilm5.zone',
 'q28.lordfilm.ltd': 'ttr8.lordfilm5.zone',
 'lordfilms.cfd': 'ttr8.lordfilm5.zone',
 'yes.lordfilm.frl': 'ttr8.lordfilm5.zone',
 'lordfilms3.serialru.online': 'ttr8.lordfilm5.zone',
 'lordfilm5.livecc.online': 'ttr8.lordfilm5.zone',
 '1lordfilm.livecc.online': 'ttr8.lordfilm5.zone',
 'hd-24.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'gg-8.lordfilm.green': 'ttr8.lordfilm5.zone',
 'lordfilms5.rpltb.online': 'ttr8.lordfilm5.zone',
 'lordfilms4.rpltb.online': 'ttr8.lordfilm5.zone',
 'hd-23.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'q29.lordfilm.ltd': 'ttr8.lordfilm5.zone',
 '2022-online.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm-s.buzz': 'ttr8.lordfilm5.zone',
 'op-7.lordfilm-s.buzz': 'ttr8.lordfilm5.zone',
 'us.lordfilmq.click': 'ttr8.lordfilm5.zone',
 '07.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'n28.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 'zoom2.lordfilm.codes': 'ttr8.lordfilm5.zone',
 'gg.lordfilm.frl': 'ttr8.lordfilm5.zone',
 't.lordfilms-filmy.online': 'ttr8.lordfilm5.zone',
 '30may.lordfilma.com': 'ttr8.lordfilm5.zone',
 'lordfilm7.serialru.online': 'ttr8.lordfilm5.zone',
 'lordfilms-1080.site': 'ttr8.lordfilm5.zone',
 'lordfilms.link': 'ttr8.lordfilm5.zone',
 'hd-29.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'izz1.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'hd-27.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'hd5.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'ko1.lordfilms.gift': 'ttr8.lordfilm5.zone',
 '42.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'lordfilm-smotret.one': 'ttr8.lordfilm5.zone',
 'lordfilm-tv14.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'lordfilm-tv15.lofilmss.zone': 'ttr8.lordfilm5.zone',
 '2022-rezka.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilms-web.site': 'ttr8.lordfilm5.zone',
 'waa3.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'da.lordfilmax.net': 'ttr8.lordfilm5.zone',
 '08.lordfilm0.net': 'ttr8.lordfilm5.zone',
 '99.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'goy-9.lordfilms.gift': 'ttr8.lordfilm5.zone',
 'utm.lordfilm9.vg': 'ttr8.lordfilm5.zone',
 'zaa4.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'lord.lordfilm0.org': 'ttr8.lordfilm5.zone',
 'hd29.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'lord-02.lordfilm0.org': 'ttr8.lordfilm5.zone',
 'lordfilms3.rpltb.online': 'ttr8.lordfilm5.zone',
 'hd-30.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'm18.lordfilms.fund': 'ttr8.lordfilm5.zone',
 'kva3.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'nmo4.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'hd-19.lordfilm-s.me': 'ttr8.lordfilm5.zone',
 'ks-303.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 're-5.lordfilm.green': 'ttr8.lordfilm5.zone',
 'lord-55.lordfilm0.org': 'ttr8.lordfilm5.zone',
 'id24.lordfilm.codes': 'ttr8.lordfilm5.zone',
 'lordfilm720.xyz': 'ttr8.lordfilm5.zone',
 '1.lordfilm720.xyz': 'ttr8.lordfilm5.zone',
 'ar-8.lordfilm.green': 'ttr8.lordfilm5.zone',
 'bb-5.lordfilm.green': 'ttr8.lordfilm5.zone',
 'lordfilm7.rpltb.online': 'ttr8.lordfilm5.zone',
 'oh-7.lordfilm.green': 'ttr8.lordfilm5.zone',
 'buf3.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'lordfilm7.kinows.online': 'ttr8.lordfilm5.zone',
 'nu2.lordfilms.video': 'ttr8.lordfilm5.zone',
 'z.lordfilms-film.online': 'ttr8.lordfilm5.zone',
 'gg-1.lordfilm.green': 'ttr8.lordfilm5.zone',
 'lordfilmhd.life': 'ttr8.lordfilm5.zone',
 'ru1.lordfilms.video': 'ttr8.lordfilm5.zone',
 'q22.lordfilm.ltd': 'ttr8.lordfilm5.zone',
 'lordfilm3.livesx.online': 'ttr8.lordfilm5.zone',
 'ass-1.lordfilm.green': 'ttr8.lordfilm5.zone',
 'n19.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 '17.lordfilm1.com': 'ttr8.lordfilm5.zone',
 'utm-01.lordfilm9.vg': 'ttr8.lordfilm5.zone',
 '22.lordfilmtv.me': 'ttr8.lordfilm5.zone',
 '44-2022.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm-2.net': 'ttr8.lordfilm5.zone',
 'po-19.lordfilm.green': 'ttr8.lordfilm5.zone',
 'hd33.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'aa.lordfilm.group': 'ttr8.lordfilm5.zone',
 'n7.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'lordfilm-tv23.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'uyt2.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'ru.lordfilms-1080.site': 'ttr8.lordfilm5.zone',
 'f-4.lordfilm-s.buzz': 'ttr8.lordfilm5.zone',
 '09.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'lordfilm-tv18.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'eos3.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'buu7.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'lordfilm-tv19.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'lordfilm-tv20.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'hd1.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'lordfilm1.livesx.online': 'ttr8.lordfilm5.zone',
 'we11.lordfilm-w.xyz': 'ttr8.lordfilm5.zone',
 '4k-2022.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'n.lordfilm-smotret.one': 'ttr8.lordfilm5.zone',
 'ftt-2.lordfilm.green': 'ttr8.lordfilm5.zone',
 '00.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'mss2.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'n45.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'ma-8.lordfilm.green': 'ttr8.lordfilm5.zone',
 'n35.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'nu5.lordfilms.video': 'ttr8.lordfilm5.zone',
 'lordfilmhd.zone': 'ttr8.lordfilm5.zone',
 'lordfilm-hd.lordfilm1.zone': 'ttr8.lordfilm5.zone',
 'bb-4.lordfilm.gifts': 'ttr8.lordfilm5.zone',
 'nu3.lordfilms.video': 'ttr8.lordfilm5.zone',
 'ru.lordfilms.video': 'ttr8.lordfilm5.zone',
 'lordfilm4.livesx.online': 'ttr8.lordfilm5.zone',
 'zz-2022.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'hd720-lord.lordfilms0.com': 'ttr8.lordfilm5.zone',
 'go.lordfilmhd.top': 'ttr8.lordfilm5.zone',
 'lordfilm.uzor.su': 'ttr8.lordfilm5.zone',
 'ru1.lordfilms7.cam': 'ttr8.lordfilm5.zone',
 'raw-8.lordfilms.gift': 'ttr8.lordfilm5.zone',
 'lordfilm1.rpltb.online': 'ttr8.lordfilm5.zone',
 'lordfilm.serialmy.online': 'ttr8.lordfilm5.zone',
 'lordfilm-tv21.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'kinogo-lordfilm.net': 'ttr8.lordfilm5.zone',
 'hdi.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'm19.lordfilms.fund': 'ttr8.lordfilm5.zone',
 'lordfilm.cc': 'ttr8.lordfilm5.zone',
 'lordfilm2.livesx.online': 'ttr8.lordfilm5.zone',
 'dfa.lordfilm2.site': 'ttr8.lordfilm5.zone',
 'y4.lordfilms.fund': 'ttr8.lordfilm5.zone',
 'lordfilms4.serialru.online': 'ttr8.lordfilm5.zone',
 'lord222.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'n8.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'lordfilm4.rpltb.online': 'ttr8.lordfilm5.zone',
 'i.lordfilms.biz': 'ttr8.lordfilm5.zone',
 'fs2.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'we12.lordfilm-w.xyz': 'ttr8.lordfilm5.zone',
 'n20.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 'm1.lordfilmz2.zone': 'ttr8.lordfilm5.zone',
 'lordfilm0.net': 'ttr8.lordfilm5.zone',
 'nu7.lordfilms7.cam': 'ttr8.lordfilm5.zone',
 'no-7.lordfilmzz.xyz': 'ttr8.lordfilm5.zone',
 'utm-03.lordfilm9.vg': 'ttr8.lordfilm5.zone',
 'hd-lordfilm.2020-film.org': 'ttr8.lordfilm5.zone',
 'n26.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 '2022-hd.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 'lordfilm-hd9s.lordfilm.link': 'ttr8.lordfilm5.zone',
 'ru11.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'z.lordfilmy-film.online': 'ttr8.lordfilm5.zone',
 '8k-2022.lordfilms0.com': 'ttr8.lordfilm5.zone',
 'n23.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 'n24.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 'n21.lordfilm-tv.cam': 'ttr8.lordfilm5.zone',
 '21.lordfilm1.com': 'ttr8.lordfilm5.zone',
 'lordfilm-tv16.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'urr5.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'n22.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'lord555.lordfilm0.vg': 'ttr8.lordfilm5.zone',
 '18.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'ob-5.lordfilms.gift': 'ttr8.lordfilm5.zone',
 'n9.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'ru3.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'hd28.3lordfilm-0.xyz': 'ttr8.lordfilm5.zone',
 'lordfilm6.serialru.online': 'ttr8.lordfilm5.zone',
 'lordfilm3.kinolink.info': 'ttr8.lordfilm5.zone',
 'tb.lordfilml.xyz': 'ttr8.lordfilm5.zone',
 'lordfilm-web.xyz': 'ttr8.lordfilm5.zone',
 'utm-05.lordfilm9.vg': 'ttr8.lordfilm5.zone',
 '4ru.lordfilm.house': 'ttr8.lordfilm5.zone',
 'n23.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'lordfilms-film.online': 'ttr8.lordfilm5.zone',
 'n34.lordfilm7.com': 'ttr8.lordfilm5.zone',
 'lordfilmz.fans': 'ttr8.lordfilm5.zone',
 'lordfilm-tv22.lofilmss.zone': 'ttr8.lordfilm5.zone',
 'lordfilm.gift': 'ttr8.lordfilm5.zone',
 'nu4.lordfilms.video': 'ttr8.lordfilm5.zone',
 'no3.lordfilmsbs.com': 'ttr8.lordfilm5.zone',
 '66.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'lordfilm4.serialru.online': 'ttr8.lordfilm5.zone',
 'ho-9.lordfilm-s.buzz': 'ttr8.lordfilm5.zone',
 'we-8.lordfilm-s.buzz': 'ttr8.lordfilm5.zone',
 'lordfilmy-film.online': 'ttr8.lordfilm5.zone',
 'hd.lordfilms.cloud': 'ttr8.lordfilm5.zone',
 '66-sd.lordfilm.group': 'ttr8.lordfilm5.zone',
 'nu.lordfilms24.com': 'ttr8.lordfilm5.zone',
 'n32.lordfilm7.com': 'ttr8.lordfilm5.zone',
 '88.lordfilm0.net': 'ttr8.lordfilm5.zone',
 'lordfilm-w.xyz': 'ttr8.lordfilm5.zone',
 'ttr8.lordfilm5.zone': 'ttr8.lordfilm5.zone',
 'my.lordfilms.name': 'ttr8.lordfilm5.zone',
    'lordserial.ru': 'lordserials.com',
 '2lordserials.xyz': 'lordserials.com',
 'lordserials.xyz': 'lordserials.com',
 'lordserial.one': 'lordserials.com',
 'lordserial.tk': 'lordserials.com',
 'lordserial.live': 'lordserials.com',
 'lordserial.ws': 'lordserials.com',
 'lordserial.site': 'lordserials.com',
 'lordserial.xyz': 'lordserials.com',
 '1-lordserials.xyz': 'lordserials.com',
 'lordserial.fan': 'lordserials.com',
 'lordserials.tv': 'lordserials.com',
 'lordserials.me': 'lordserials.com',
 'lordserials.com': 'lordserials.com',
 'lordserials.online': 'lordserials.com',
 'hdi.1lordserials.xyz': 'lordserials.com',
 'hd3.lordserial10.xyz': 'lordserials.com',
 'lordserials.biz': 'lordserials.com',
 's1.lordserial.click': 'lordserials.com',
 'hd.lordserials.one': 'lordserials.com',
 'lordserials.one': 'lordserials.com',
 'vlordserial.ru': 'lordserials.com',
 'lordserials.pw': 'lordserials.com',
 'hd4.lordserial10.xyz': 'lordserials.com',
 'lordserial2.ru': 'lordserials.com',
    'thealloha.club': 'alloha.tv',
 'streamalloha.live': 'alloha.tv',
 'alloha.tv': 'alloha.tv',
 'allohastream.com': 'alloha.tv',
 'allohalive.com': 'alloha.tv',
    '72.lordfilm.ru.net': 'lordfilm.ru.net',
 '52.lordfilm.ru.net': 'lordfilm.ru.net',
 '223.lordfilm.ru.net': 'lordfilm.ru.net',
    'dussh15.edu.murmansk.ru': 'edu.murmansk.ru',
 'edu.murmansk.ru': 'edu.murmansk.ru',
 'gimcro.edu.murmansk.ru': 'edu.murmansk.ru',
    'ts4.x1.international.travian.com': 'international.travian.com',
 'ts30.x3.international.travian.com': 'international.travian.com',
 'ts3.x1.international.travian.com': 'international.travian.com',
    'ivanovo.ac.ru': 'ivanovo.ac.ru',
 'ivsu.ivanovo.ac.ru': 'ivanovo.ac.ru',
 'com.ivanovo.ac.ru': 'ivanovo.ac.ru',
    '55.kinosimka.ru.net': 'kinosimka.ru.net',
 '19.kinosimka.ru.net': 'kinosimka.ru.net',
 '70.kinosimka.ru.net': 'kinosimka.ru.net',
    'love.mail.ru': 'love.mail.ru',
 'm.love.mail.ru': 'love.mail.ru',
 'znakomstva.love.mail.ru': 'love.mail.ru',
    'm.sabah.com.tr': 'sabah.com.tr',
 'arsiv.sabah.com.tr': 'sabah.com.tr',
 'sabah.com.tr': 'sabah.com.tr',
    'spsl.nsc.ru': 'spsl.nsc.ru',
 'webirbis.spsl.nsc.ru': 'spsl.nsc.ru',
 'irbiscorp.spsl.nsc.ru': 'spsl.nsc.ru',
    'adsl.kirov.ru': 'adsl.kirov.ru',
 'users.adsl.kirov.ru': 'adsl.kirov.ru',
 'ut-ext.adsl.kirov.ru': 'adsl.kirov.ru',
    'rayon.in.ua': 'rayon.in.ua',
 'kowel.rayon.in.ua': 'rayon.in.ua',
 'lutsk.rayon.in.ua': 'rayon.in.ua',
    'svadba.net.ru': 'svadba.net.ru',
 'm.svadba.net.ru': 'svadba.net.ru',
 'forum.svadba.net.ru': 'svadba.net.ru',
    'ska4at.film.ru.net': 'film.ru.net',
 'video.film.ru.net': 'film.ru.net',
 'newperehod.film.ru.net': 'film.ru.net',
 'anwap.film.ru.net': 'film.ru.net',
    'minsk.m.irr.by': 'm.irr.by',
 'm.irr.by': 'm.irr.by',
 'belarus.m.irr.by': 'm.irr.by',
 'grodno.m.irr.by': 'm.irr.by',
    'tm13.shkola.hc.ru': 'shkola.hc.ru',
 'm-school10.shkola.hc.ru': 'shkola.hc.ru',
 'dutovo-scool.shkola.hc.ru': 'shkola.hc.ru',
 'omsk145.shkola.hc.ru': 'shkola.hc.ru',
    'm.11st.co.kr': '11st.co.kr',
 'buy.m.11st.co.kr': '11st.co.kr',
 'shop.11st.co.kr': '11st.co.kr',
 'global.m.11st.co.kr': '11st.co.kr',
    'nmfo-vo.edu.rosminzdrav.ru': 'edu.rosminzdrav.ru',
 'edu.rosminzdrav.ru': 'edu.rosminzdrav.ru',
 'nmfo-spo.edu.rosminzdrav.ru': 'edu.rosminzdrav.ru',
 'a.edu.rosminzdrav.ru': 'edu.rosminzdrav.ru',
    'ou110.omsk.obr55.ru': 'omsk.obr55.ru',
 'ou109.omsk.obr55.ru': 'omsk.obr55.ru',
 'spootmmp.omsk.obr55.ru': 'omsk.obr55.ru',
 'ds127.omsk.obr55.ru': 'omsk.obr55.ru',
    '184.kinosimka.ru.com': 'kinosimka.ru.com',
 '40.kinosimka.ru.com': 'kinosimka.ru.com',
 '74.kinosimka.ru.com': 'kinosimka.ru.com',
 '122.kinosimka.ru.com': 'kinosimka.ru.com',
    'sdo.krsk.irgups.ru': 'krsk.irgups.ru',
 'dpo.krsk.irgups.ru': 'krsk.irgups.ru',
 'irbis.krsk.irgups.ru': 'krsk.irgups.ru',
 'sdo-ktgt.krsk.irgups.ru': 'krsk.irgups.ru',
 'sdo1.krsk.irgups.ru': 'krsk.irgups.ru',
    'ts3.x1.europe.travian.com': 'europe.travian.com',
 'toc.x2.europe.travian.com': 'europe.travian.com',
 'ts6.x1.europe.travian.com': 'europe.travian.com',
 'ts5.x1.europe.travian.com': 'europe.travian.com',
 'ts30.x3.europe.travian.com': 'europe.travian.com',
    'gorod.dp.ua': 'gorod.dp.ua',
 'mobile.gorod.dp.ua': 'gorod.dp.ua',
 'forum.gorod.dp.ua': 'gorod.dp.ua',
 'web.gorod.dp.ua': 'gorod.dp.ua',
 'm.gorod.dp.ua': 'gorod.dp.ua',
    '210.detsad.27.ru': 'detsad.27.ru',
 '107.detsad.27.ru': 'detsad.27.ru',
 '1.detsad.27.ru': 'detsad.27.ru',
 'rodnichok.detsad.27.ru': 'detsad.27.ru',
 '57.detsad.27.ru': 'detsad.27.ru',
    'lib.uniyar.ac.ru': 'uniyar.ac.ru',
 'ivt.uniyar.ac.ru': 'uniyar.ac.ru',
 'fiz.uniyar.ac.ru': 'uniyar.ac.ru',
 'hist.uniyar.ac.ru': 'uniyar.ac.ru',
 'uniyar.ac.ru': 'uniyar.ac.ru',
    'price.nw.rt.ru': 'nw.rt.ru',
 'bill.nw.rt.ru': 'nw.rt.ru',
 'lock.nw.rt.ru': 'nw.rt.ru',
 'index.nw.rt.ru': 'nw.rt.ru',
 'go.nw.rt.ru': 'nw.rt.ru',
    'russian.people.com.cn': 'people.com.cn',
 'people.com.cn': 'people.com.cn',
 'cpc.people.com.cn': 'people.com.cn',
 'liuyan.people.com.cn': 'people.com.cn',
 'search.people.com.cn': 'people.com.cn',
 'hlj.people.com.cn': 'people.com.cn',
    'kislovodsk.ncb.su': 'kislovodsk.ncb.su',
 'dimitrova.kislovodsk.ncb.su': 'kislovodsk.ncb.su',
 'uzbekistan.kislovodsk.ncb.su': 'kislovodsk.ncb.su',
 'solnechnyy.kislovodsk.ncb.su': 'kislovodsk.ncb.su',
 'celebnyy-narzan.kislovodsk.ncb.su': 'kislovodsk.ncb.su',
 'zarya.kislovodsk.ncb.su': 'kislovodsk.ncb.su',
    'online.sberbank.ru': 'online.sberbank.ru',
 'node3.online.sberbank.ru': 'online.sberbank.ru',
 'node1.online.sberbank.ru': 'online.sberbank.ru',
 'web6.online.sberbank.ru': 'online.sberbank.ru',
 'web5.online.sberbank.ru': 'online.sberbank.ru',
 'web1.online.sberbank.ru': 'online.sberbank.ru',
 'web4.online.sberbank.ru': 'online.sberbank.ru',
    'balashiha.omsu.inlite.ru': 'omsu.inlite.ru',
 'lubertsy.omsu.inlite.ru': 'omsu.inlite.ru',
 'mytischi.omsu.inlite.ru': 'omsu.inlite.ru',
 'ivanteevka.omsu.inlite.ru': 'omsu.inlite.ru',
 'istra.omsu.inlite.ru': 'omsu.inlite.ru',
 'bogorodsky.omsu.inlite.ru': 'omsu.inlite.ru',
 'orehovo.omsu.inlite.ru': 'omsu.inlite.ru',
    'ufa.b2c.perspekt174.ru': 'b2c.perspekt174.ru',
 'b2c.perspekt174.ru': 'b2c.perspekt174.ru',
 'novosibirsk.b2c.perspekt174.ru': 'b2c.perspekt174.ru',
 'moskva.b2c.perspekt174.ru': 'b2c.perspekt174.ru',
 'kazan.b2c.perspekt174.ru': 'b2c.perspekt174.ru',
 'miass.b2c.perspekt174.ru': 'b2c.perspekt174.ru',
 'ekaterinburg.b2c.perspekt174.ru': 'b2c.perspekt174.ru',
    'lentaru.media.eagleplatform.com': 'media.eagleplatform.com',
 'ramblernews.media.eagleplatform.com': 'media.eagleplatform.com',
 'gazeta.media.eagleplatform.com': 'media.eagleplatform.com',
 'ferra.media.eagleplatform.com': 'media.eagleplatform.com',
 'afisha.media.eagleplatform.com': 'media.eagleplatform.com',
 'quto.media.eagleplatform.com': 'media.eagleplatform.com',
 'wmj.media.eagleplatform.com': 'media.eagleplatform.com',
 'letidor.media.eagleplatform.com': 'media.eagleplatform.com',
    'usr.vspu.ac.ru': 'vspu.ac.ru',
 'umu.vspu.ac.ru': 'vspu.ac.ru',
 'fmf.vspu.ac.ru': 'vspu.ac.ru',
 'pk.vspu.ac.ru': 'vspu.ac.ru',
 'kultura-fiz.vspu.ac.ru': 'vspu.ac.ru',
 'scivspu.vspu.ac.ru': 'vspu.ac.ru',
 'portal.vspu.ac.ru': 'vspu.ac.ru',
 'vspu.ac.ru': 'vspu.ac.ru',
    'mg.gmarket.co.kr': 'gmarket.co.kr',
 'global.gmarket.co.kr': 'gmarket.co.kr',
 'mgmyg.gmarket.co.kr': 'gmarket.co.kr',
 'mitem.gmarket.co.kr': 'gmarket.co.kr',
 'item.gmarket.co.kr': 'gmarket.co.kr',
 'glrpp.gmarket.co.kr': 'gmarket.co.kr',
 'gmember.gmarket.co.kr': 'gmarket.co.kr',
 'browse.gmarket.co.kr': 'gmarket.co.kr',
 'gsearch.gmarket.co.kr': 'gmarket.co.kr',
 'minishop.gmarket.co.kr': 'gmarket.co.kr',
    's12.stc.all.kpcdn.net': 'all.kpcdn.net',
 's5.stc.all.kpcdn.net': 'all.kpcdn.net',
 's1.stc.all.kpcdn.net': 'all.kpcdn.net',
 's4.stc.all.kpcdn.net': 'all.kpcdn.net',
 's3.stc.all.kpcdn.net': 'all.kpcdn.net',
 's14.stc.all.kpcdn.net': 'all.kpcdn.net',
 's16.stc.all.kpcdn.net': 'all.kpcdn.net',
 's7.stc.all.kpcdn.net': 'all.kpcdn.net',
 's13.stc.all.kpcdn.net': 'all.kpcdn.net',
 's11.stc.all.kpcdn.net': 'all.kpcdn.net',
    'dou40.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou237.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou208.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou105.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou76.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou232.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou219.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou115.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou116.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou199.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
 'dou54.edu.sarkomobr.ru': 'edu.sarkomobr.ru',
    'gorod.tomsk.ru': 'gorod.tomsk.ru',
 'alter.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'admin.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'fenix.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'pantarei.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'lara.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'han.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'peresvet.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'anastasia.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'artem.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'otello.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'shrrr.gorod.tomsk.ru': 'gorod.tomsk.ru',
 'xomatelecom.gorod.tomsk.ru': 'gorod.tomsk.ru',
    'school58.edu.27.ru': 'edu.27.ru',
 'otkrytie-khv.edu.27.ru': 'edu.27.ru',
 'khb-lpoliteh.edu.27.ru': 'edu.27.ru',
 'kna-s4.edu.27.ru': 'edu.27.ru',
 'school56.edu.27.ru': 'edu.27.ru',
 'kna-g9.edu.27.ru': 'edu.27.ru',
 'kna-g45.edu.27.ru': 'edu.27.ru',
 'kschool3.edu.27.ru': 'edu.27.ru',
 'bikin6.edu.27.ru': 'edu.27.ru',
 'kna-s30.edu.27.ru': 'edu.27.ru',
 'vostochnoe.edu.27.ru': 'edu.27.ru',
 'mongokhto.edu.27.ru': 'edu.27.ru',
 'school77.edu.27.ru': 'edu.27.ru',
 'okt.edu.27.ru': 'edu.27.ru',
 'kna-s16.edu.27.ru': 'edu.27.ru',
 'khb-lvm.edu.27.ru': 'edu.27.ru',
 'tele.edu.27.ru': 'edu.27.ru',
 'internat9lazo.edu.27.ru': 'edu.27.ru',
 'school-ilinka.edu.27.ru': 'edu.27.ru',
 'uov.edu.27.ru': 'edu.27.ru',
 'vzm1.edu.27.ru': 'edu.27.ru',
    '22248-sem-zhiznei.kinogo.ru.net': 'kinogo.ru.net',
 '22039-reportazh.kinogo.ru.net': 'kinogo.ru.net',
 'videopleer.kinogo.ru.net': 'kinogo.ru.net',
 '22741-titanik.kinogo.ru.net': 'kinogo.ru.net',
 'multserialy.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-melodrama.kinogo.ru.net': 'kinogo.ru.net',
 'kino.kinogo.ru.net': 'kinogo.ru.net',
 'video-2019.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-kriminal.kinogo.ru.net': 'kinogo.ru.net',
 'multfilmy.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-detektiv.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-komediya.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-boevik.kinogo.ru.net': 'kinogo.ru.net',
 'video-2020.kinogo.ru.net': 'kinogo.ru.net',
 'serialy.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-fentezi.kinogo.ru.net': 'kinogo.ru.net',
 'kino-2020.kinogo.ru.net': 'kinogo.ru.net',
 'hd4.kinogo.ru.net': 'kinogo.ru.net',
 'anime.kinogo.ru.net': 'kinogo.ru.net',
 'animeserialy.kinogo.ru.net': 'kinogo.ru.net',
 'serialy-drama.kinogo.ru.net': 'kinogo.ru.net',
 'video.kinogo.ru.net': 'kinogo.ru.net',
 '24131-merlin.kinogo.ru.net': 'kinogo.ru.net',
 'hd1.kinogo.ru.net': 'kinogo.ru.net',
 '12206-plokhaya-sudya.kinogo.ru.net': 'kinogo.ru.net',
    'amp-championat-com.cdn.ampproject.org': 'amp.championat.com',
 'm-lenta-ru.cdn.ampproject.org': 'm.lenta.ru',
 'm-sport--express-ru.cdn.ampproject.org': 'm.sport-express.ru',
 'm-gazeta-ru.cdn.ampproject.org': 'm.gazeta.ru',
 'aif-ru.cdn.ampproject.org': 'aif.ru',
 'iz-ru.cdn.ampproject.org': 'iz.ru',
 'news-mail-ru.cdn.ampproject.org': 'news.mail.ru',
 'amp-eda-ru.cdn.ampproject.org': 'amp.eda.ru',
 '0-hi--tech-mail-ru-0.cdn.ampproject.org': '0.hi-tech.mail.ru.0',
 'secretmag-ru.cdn.ampproject.org': 'secretmag.ru',
 'news-rambler-ru.cdn.ampproject.org': 'news.rambler.ru',
 'letidor-ru.cdn.ampproject.org': 'letidor.ru',
 'auto-rambler-ru.cdn.ampproject.org': 'auto.rambler.ru',
 'tengrinews-kz.cdn.ampproject.org': 'tengrinews.kz',
 'woman-rambler-ru.cdn.ampproject.org': 'woman.rambler.ru',
 'daily-afisha-ru.cdn.ampproject.org': 'daily.afisha.ru',
 'moslenta-ru.cdn.ampproject.org': 'moslenta.ru',
 'amp-sport24-ru.cdn.ampproject.org': 'amp.sport24.ru',
 'blitz-plus.cdn.ampproject.org': 'blitz.plus',
 'vz-ru.cdn.ampproject.org': 'vz.ru',
 'lifehacker-ru.cdn.ampproject.org': 'lifehacker.ru',
 'sport-rambler-ru.cdn.ampproject.org': 'sport.rambler.ru',
 'sportmail-ru.cdn.ampproject.org': 'sportmail.ru',
 'radiokp-ru.cdn.ampproject.org': 'radiokp.ru',
 'teleprogramma-pro.cdn.ampproject.org': 'teleprogramma.pro',
 'finance-rambler-ru.cdn.ampproject.org': 'finance.rambler.ru',
 'amp-ren-tv.cdn.ampproject.org': 'amp.ren.tv',
 'quto-ru.cdn.ampproject.org': 'quto.ru',
 'crimea-mk-ru.cdn.ampproject.org': 'crimea.mk.ru',
 'smotrim-ru.cdn.ampproject.org': 'smotrim.ru',
 'horoscopes-rambler-ru.cdn.ampproject.org': 'horoscopes.rambler.ru',
 'pogoda-mail-ru.cdn.ampproject.org': 'pogoda.mail.ru',
 'amp-kp-ru.cdn.ampproject.org': 'amp.kp.ru',
 'amp-7days-ru.cdn.ampproject.org': 'amp.7days.ru',
 'lady-mail-ru.cdn.ampproject.org': 'lady.mail.ru',
 'texta--pesen-ru.cdn.ampproject.org': 'texta-pesen.ru',
 'motor-ru.cdn.ampproject.org': 'motor.ru',
 'sevastopol-su.cdn.ampproject.org': 'sevastopol.su',
 'auto-mail-ru.cdn.ampproject.org': 'auto.mail.ru',
 'spb-mk-ru.cdn.ampproject.org': 'spb.mk.ru',
 'm-tvzvezda-ru.cdn.ampproject.org': 'm.tvzvezda.ru',
 'weekend-rambler-ru.cdn.ampproject.org': 'weekend.rambler.ru',
 'travel-rambler-ru.cdn.ampproject.org': 'travel.rambler.ru',
 'deti-mail-ru.cdn.ampproject.org': 'deti.mail.ru',
 'amp-kuban-kp-ru.cdn.ampproject.org': 'amp.kuban.kp.ru',
 'kino-rambler-ru.cdn.ampproject.org': 'kino.rambler.ru',
 'dni-ru.cdn.ampproject.org': 'dni.ru',
 'baikal-mk-ru.cdn.ampproject.org': 'baikal.mk.ru',
 'kuban-mk-ru.cdn.ampproject.org': 'kuban.mk.ru',
 'vrn-mk-ru.cdn.ampproject.org': 'vrn.mk.ru',
 'hab-mk-ru.cdn.ampproject.org': 'hab.mk.ru',
 'lena--miro-ru.cdn.ampproject.org': 'lena-miro.ru',
 'karel-mk-ru.cdn.ampproject.org': 'karel.mk.ru',
 'health-mail-ru.cdn.ampproject.org': 'health.mail.ru',
 'utro-ru.cdn.ampproject.org': 'utro.ru',
 'day-ru.cdn.ampproject.org': 'day.ru',
 'vlad-mk-ru.cdn.ampproject.org': 'vlad.mk.ru',
 'amp-kem-kp-ru.cdn.ampproject.org': 'amp.kem.kp.ru',
 'nn-mk-ru.cdn.ampproject.org': 'nn.mk.ru',
 'doramax-ru.cdn.ampproject.org': 'doramax.ru',
 'blitz-center.cdn.ampproject.org': 'blitz.center',
 'eburg-mk-ru.cdn.ampproject.org': 'eburg.mk.ru',
 'gtrk-tv.cdn.ampproject.org': 'gtrk.tv',
 'wccftech-com.cdn.ampproject.org': 'wccftech.com',
 'irma--stream-ru.cdn.ampproject.org': 'irma-stream.ru',
 'dorams--new-ru.cdn.ampproject.org': 'dorams-new.ru',
 'olegmakarenko-ru.cdn.ampproject.org': 'olegmakarenko.ru',
 'doctor-rambler-ru.cdn.ampproject.org': 'doctor.rambler.ru',
 'fitnessbaza-com.cdn.ampproject.org': 'fitnessbaza.com',
 'sportkp-ru.cdn.ampproject.org': 'sportkp.ru',
 '0-ru--cats-livejournal-com-0.cdn.ampproject.org': '0.ru-cats.livejournal.com.0',
 'doraman-ru.cdn.ampproject.org': 'doraman.ru',
 'natural--cosmetology-ru.cdn.ampproject.org': 'natural-cosmetology.ru',
    'toroca.as.alloeclub.com': 'alloeclub.com',
 'polati.as.alloeclub.com': 'alloeclub.com',
 'larkin.as.alloeclub.com': 'alloeclub.com',
 'bear.as.alloeclub.com': 'alloeclub.com',
 'torgash.as.alloeclub.com': 'alloeclub.com',
 'arven.as.alloeclub.com': 'alloeclub.com',
 'legolas.as.alloeclub.com': 'alloeclub.com',
 'harmon.as.alloeclub.com': 'alloeclub.com',
 'beggins.as.alloeclub.com': 'alloeclub.com',
 'miyagi.as.alloeclub.com': 'alloeclub.com',
 'tories.as.alloeclub.com': 'alloeclub.com',
 'hvitserg.as.alloeclub.com': 'alloeclub.com',
 'hela.as.alloeclub.com': 'alloeclub.com',
 'horr.as.alloeclub.com': 'alloeclub.com',
 'ant-man.as.alloeclub.com': 'alloeclub.com',
 'abraham-erskine.as.alloeclub.com': 'alloeclub.com',
 'ollo.as.alloeclub.com': 'alloeclub.com',
 'tenth.as.alloeclub.com': 'alloeclub.com',
 'tormented.as.alloeclub.com': 'alloeclub.com',
 'aragorn.as.alloeclub.com': 'alloeclub.com',
 'torgsin.as.alloeclub.com': 'alloeclub.com',
 'polyana.as.alloeclub.com': 'alloeclub.com',
 'serseya.as.alloeclub.com': 'alloeclub.com',
 'torn.as.alloeclub.com': 'alloeclub.com',
 'teoden.as.alloeclub.com': 'alloeclub.com',
 'tormaski.as.alloeclub.com': 'alloeclub.com',
 'pollute.as.alloeclub.com': 'alloeclub.com',
 'tori.as.alloeclub.com': 'alloeclub.com',
 'polemic.as.alloeclub.com': 'alloeclub.com',
 'torch.as.alloeclub.com': 'alloeclub.com',
 'abner.as.alloeclub.com': 'alloeclub.com',
 'sign.as.alloeclub.com': 'alloeclub.com',
 'art.as.alloeclub.com': 'alloeclub.com',
 'morgan.as.alloeclub.com': 'alloeclub.com',
 'fork.as.alloeclub.com': 'alloeclub.com',
 'a-bomb.as.alloeclub.com': 'alloeclub.com',
 'torturer.as.alloeclub.com': 'alloeclub.com',
 'patriot.as.alloeclub.com': 'alloeclub.com',
 'bike.as.alloeclub.com': 'alloeclub.com',
 'polygamist.as.alloeclub.com': 'alloeclub.com',
 'a-bomba.as.alloeclub.com': 'alloeclub.com',
 'polonite.as.alloeclub.com': 'alloeclub.com',
 'team.as.alloeclub.com': 'alloeclub.com',
 'polynoy.as.alloeclub.com': 'alloeclub.com',
 'abigail-brand.as.alloeclub.com': 'alloeclub.com',
 'mcdonell.as.alloeclub.com': 'alloeclub.com',
 'henrygyrich.as.alloeclub.com': 'alloeclub.com',
 'torpedoist.as.alloeclub.com': 'alloeclub.com',
 'agent-venom.as.alloeclub.com': 'alloeclub.com',
 'devon.as.alloeclub.com': 'alloeclub.com',
 'etelstan.as.alloeclub.com': 'alloeclub.com',
 'tent.as.alloeclub.com': 'alloeclub.com',
 'trecall.as.alloeclub.com': 'alloeclub.com',
 'torchbearer.as.alloeclub.com': 'alloeclub.com',
 'terzet.as.alloeclub.com': 'alloeclub.com',
 'adam-wolt.as.alloeclub.com': 'alloeclub.com',
 'agamoto.as.alloeclub.com': 'alloeclub.com',
 'faramir.as.alloeclub.com': 'alloeclub.com',
 'cusik.as.alloeclub.com': 'alloeclub.com',
 'polyhedron.as.alloeclub.com': 'alloeclub.com',
 'abbis.as.alloeclub.com': 'alloeclub.com',
 'telescope.as.alloeclub.com': 'alloeclub.com',
 'telograyka.as.alloeclub.com': 'alloeclub.com',
 'hugin.as.alloeclub.com': 'alloeclub.com',
 'tortuous.as.alloeclub.com': 'alloeclub.com',
 'synthezoid.as.alloeclub.com': 'alloeclub.com',
 'agent-13.as.alloeclub.com': 'alloeclub.com',
 'teleology.as.alloeclub.com': 'alloeclub.com',
 'hoarfen.as.alloeclub.com': 'alloeclub.com',
 'tech.as.alloeclub.com': 'alloeclub.com',
 'ivar.as.alloeclub.com': 'alloeclub.com',
 'theft.as.alloeclub.com': 'alloeclub.com',
 'groot.as.alloeclub.com': 'alloeclub.com',
 'tree.as.alloeclub.com': 'alloeclub.com',
 'politicalscientist.as.alloeclub.com': 'alloeclub.com',
 'anaconda.as.alloeclub.com': 'alloeclub.com',
 'hogun.as.alloeclub.com': 'alloeclub.com',
    'lexa--tiger-livejournal-com.turbopages.org': 'lexa-tiger.livejournal.com',
 'aranger-livejournal-com.turbopages.org': 'aranger.livejournal.com',
 'kripto--bots-ru.turbopages.org': 'kripto-bots.ru',
 'gadalkataro-livejournal-com.turbopages.org': 'gadalkataro.livejournal.com',
 '17ur-livejournal-com.turbopages.org': '17ur.livejournal.com',
 'n102-ru.turbopages.org': 'n102.ru',
 'parxis-livejournal-com.turbopages.org': 'parxis.livejournal.com',
 'cambofake-livejournal-com.turbopages.org': 'cambofake.livejournal.com',
 'tomkad-livejournal-com.turbopages.org': 'tomkad.livejournal.com',
 'furmur-ru.turbopages.org': 'furmur.ru',
 'algrab-livejournal-com.turbopages.org': 'algrab.livejournal.com',
 'semiurg-livejournal-com.turbopages.org': 'semiurg.livejournal.com',
 'project--ancient-livejournal-com.turbopages.org': 'project-ancient.livejournal.com',
 'moneytimes-ru.turbopages.org': 'moneytimes.ru',
 'xapaycm-livejournal-com.turbopages.org': 'xapaycm.livejournal.com',
 'deti--sistem-livejournal-com.turbopages.org': 'deti-sistem.livejournal.com',
 'insanepasha-livejournal-com.turbopages.org': 'insanepasha.livejournal.com',
 'nilllok-livejournal-com.turbopages.org': 'nilllok.livejournal.com',
 'mixalich--spb-livejournal-com.turbopages.org': 'mixalich-spb.livejournal.com',
 'denis--galitsky-livejournal-com.turbopages.org': 'denis-galitsky.livejournal.com',
 'fotosergs-livejournal-com.turbopages.org': 'fotosergs.livejournal.com',
 'sendemail-livejournal-com.turbopages.org': 'sendemail.livejournal.com',
 'ilfasidoroff-livejournal-com.turbopages.org': 'ilfasidoroff.livejournal.com',
 'imnotsaint-livejournal-com.turbopages.org': 'imnotsaint.livejournal.com',
 'big--book--med-ru.turbopages.org': 'big-book-med.ru',
 'ak--12-livejournal-com.turbopages.org': 'ak-12.livejournal.com',
 'stootvet-ru.turbopages.org': 'stootvet.ru',
 'nefraziya-livejournal-com.turbopages.org': 'nefraziya.livejournal.com',
 'hvv-livejournal-com.turbopages.org': 'hvv.livejournal.com',
 'kotelnich-bezformata-com.turbopages.org': 'kotelnich.bezformata.com',
 'fish12a-livejournal-com.turbopages.org': 'fish12a.livejournal.com',
 'strana--sssr-livejournal-com.turbopages.org': 'strana-sssr.livejournal.com',
 'xn------5cdmcpdqgaddqd0cb3cqijwf9bzt.turbopages.org': 'xn---5cdmcpdqgaddqd0cb3cqijwf9bzt',
 'israil--95reg-livejournal-com.turbopages.org': 'israil-95reg.livejournal.com',
 '6auto-ru.turbopages.org': '6auto.ru',
 'bezpalov-livejournal-com.turbopages.org': 'bezpalov.livejournal.com',
 'cofe--v--noyabre-livejournal-com.turbopages.org': 'cofe-v-noyabre.livejournal.com',
 'xn----itbobnebfefihq4a.turbopages.org': 'xn--itbobnebfefihq4a',
 'xn----ctbjlbdhregeba1ajfrf8byn.turbopages.org': 'xn--ctbjlbdhregeba1ajfrf8byn',
 'xn----7sbfbnbvnd7ahdqiir4b7bc.turbopages.org': 'xn--7sbfbnbvnd7ahdqiir4b7bc',
 'asbmsk-ru.turbopages.org': 'asbmsk.ru',
 'yogadoma-net.turbopages.org': 'yogadoma.net',
 'wkusnoiprosto-ru.turbopages.org': 'wkusnoiprosto.ru',
 'kidslitera-ru.turbopages.org': 'kidslitera.ru',
 'poetry--six-livejournal-com.turbopages.org': 'poetry-six.livejournal.com',
 'in--moskow-livejournal-com.turbopages.org': 'in-moskow.livejournal.com',
 'cheshzhanna-livejournal-com.turbopages.org': 'cheshzhanna.livejournal.com',
 'antialle-livejournal-com.turbopages.org': 'antialle.livejournal.com',
 'koshki-site.turbopages.org': 'koshki.site',
 'bl--almalexia-livejournal-com.turbopages.org': 'bl-almalexia.livejournal.com',
 'rvs-livejournal-com.turbopages.org': 'rvs.livejournal.com',
 'lisenok-livejournal-com.turbopages.org': 'lisenok.livejournal.com',
 'phishka-ru.turbopages.org': 'phishka.ru',
 'panzer--bjorn-livejournal-com.turbopages.org': 'panzer-bjorn.livejournal.com',
 'grib69-ru.turbopages.org': 'grib69.ru',
 'aserty-livejournal-com.turbopages.org': 'aserty.livejournal.com',
 'dmrb-livejournal-com.turbopages.org': 'dmrb.livejournal.com',
 'gerat-livejournal-com.turbopages.org': 'gerat.livejournal.com',
 'algre-livejournal-com.turbopages.org': 'algre.livejournal.com',
 'gamebattlegrounds-ru.turbopages.org': 'gamebattlegrounds.ru',
 'dominicana--faq-livejournal-com.turbopages.org': 'dominicana-faq.livejournal.com',
 'bordur--007-livejournal-com.turbopages.org': 'bordur-007.livejournal.com',
 'anastasiasugar-livejournal-com.turbopages.org': 'anastasiasugar.livejournal.com',
 'parnik--teplitsa-ru.turbopages.org': 'parnik-teplitsa.ru',
 'makar--svirepii-livejournal-com.turbopages.org': 'makar-svirepii.livejournal.com',
 'bigturtle-ru.turbopages.org': 'bigturtle.ru',
 '2346894-livejournal-com.turbopages.org': '2346894.livejournal.com',
 'kot--ivan76-livejournal-com.turbopages.org': 'kot-ivan76.livejournal.com',
 'sasha--india-livejournal-com.turbopages.org': 'sasha-india.livejournal.com',
 'monozygotic--t-livejournal-com.turbopages.org': 'monozygotic-t.livejournal.com',
 'roslavl-bezformata-com.turbopages.org': 'roslavl.bezformata.com',
 'novouralsk-bezformata-com.turbopages.org': 'novouralsk.bezformata.com',
 'ural--news-net.turbopages.org': 'ural-news.net',
 'vpnki-ru.turbopages.org': 'vpnki.ru',
 'alex--anpilogov-livejournal-com.turbopages.org': 'alex-anpilogov.livejournal.com',
 'dapoh-livejournal-com.turbopages.org': 'dapoh.livejournal.com',
 'ru--knife-livejournal-com.turbopages.org': 'ru-knife.livejournal.com',
 'kykycapol-livejournal-com.turbopages.org': 'kykycapol.livejournal.com',
 'viva--vova-livejournal-com.turbopages.org': 'viva-vova.livejournal.com',
 'veseloeradio-ru.turbopages.org': 'veseloeradio.ru',
 'besshardwick-livejournal-com.turbopages.org': 'besshardwick.livejournal.com',
 'z--alexey-livejournal-com.turbopages.org': 'z-alexey.livejournal.com',
 'stakhanov-info.turbopages.org': 'stakhanov.info',
 'digitaluser-ru.turbopages.org': 'digitaluser.ru',
 'ant--63-livejournal-com.turbopages.org': 'ant-63.livejournal.com',
 'seakonst-livejournal-com.turbopages.org': 'seakonst.livejournal.com',
 'govorunman-livejournal-com.turbopages.org': 'govorunman.livejournal.com',
 'afisha--krasnodar-ru.turbopages.org': 'afisha-krasnodar.ru',
 'orfelius-livejournal-com.turbopages.org': 'orfelius.livejournal.com',
 'ezhick-livejournal-com.turbopages.org': 'ezhick.livejournal.com',
 'zakruti-com.turbopages.org': 'zakruti.com',
 'smart--pr-livejournal-com.turbopages.org': 'smart-pr.livejournal.com',
 'financedigest-livejournal-com.turbopages.org': 'financedigest.livejournal.com',
 'fortels-livejournal-com.turbopages.org': 'fortels.livejournal.com',
 'vityi--1976-livejournal-com.turbopages.org': 'vityi-1976.livejournal.com',
 'sanchess--city31-livejournal-com.turbopages.org': 'sanchess-city31.livejournal.com',
 'torontoru-livejournal-com.turbopages.org': 'torontoru.livejournal.com',
 'brandmonkey-ru.turbopages.org': 'brandmonkey.ru',
 'ru--babyhealth-ru.turbopages.org': 'ru-babyhealth.ru',
 'lmystra-livejournal-com.turbopages.org': 'lmystra.livejournal.com',
 'astromaya-ru.turbopages.org': 'astromaya.ru',
 'nikolay--siya-livejournal-com.turbopages.org': 'nikolay-siya.livejournal.com',
 'agent--40-livejournal-com.turbopages.org': 'agent-40.livejournal.com',
 'aspirin--for--all-livejournal-com.turbopages.org': 'aspirin-for-all.livejournal.com',
 'arbago-livejournal-com.turbopages.org': 'arbago.livejournal.com',
 'cc--t1-ru.turbopages.org': 'cc-t1.ru',
 'nashe--nasledie-livejournal-com.turbopages.org': 'nashe-nasledie.livejournal.com',
 'szhaman-livejournal-com.turbopages.org': 'szhaman.livejournal.com',
 'vihod--net-livejournal-com.turbopages.org': 'vihod-net.livejournal.com',
 'visit--volgograd-livejournal-com.turbopages.org': 'visit-volgograd.livejournal.com',
 'afinyanka-livejournal-com.turbopages.org': 'afinyanka.livejournal.com',
 'cantadora--1st-livejournal-com.turbopages.org': 'cantadora-1st.livejournal.com',
 'diana--225-livejournal-com.turbopages.org': 'diana-225.livejournal.com',
 'valentincehov-livejournal-com.turbopages.org': 'valentincehov.livejournal.com',
 'anton--montana-livejournal-com.turbopages.org': 'anton-montana.livejournal.com',
 'ecvator-livejournal-com.turbopages.org': 'ecvator.livejournal.com',
 'jurgen--m-livejournal-com.turbopages.org': 'jurgen-m.livejournal.com',
 'tatabubu-livejournal-com.turbopages.org': 'tatabubu.livejournal.com',
 'gonza03-livejournal-com.turbopages.org': 'gonza03.livejournal.com',
 'silver2013-livejournal-com.turbopages.org': 'silver2013.livejournal.com',
 'bgproject-livejournal-com.turbopages.org': 'bgproject.livejournal.com',
 'madam--shazly-livejournal-com.turbopages.org': 'madam-shazly.livejournal.com',
 'stsvv-livejournal-com.turbopages.org': 'stsvv.livejournal.com',
 'detilife-ru.turbopages.org': 'detilife.ru',
 'pryamaya--ru-livejournal-com.turbopages.org': 'pryamaya-ru.livejournal.com',
 'kruzilina-livejournal-com.turbopages.org': 'kruzilina.livejournal.com',
 'tambiev--magomed-livejournal-com.turbopages.org': 'tambiev-magomed.livejournal.com',
 'akievgalgei-livejournal-com.turbopages.org': 'akievgalgei.livejournal.com',
 'chergizov-livejournal-com.turbopages.org': 'chergizov.livejournal.com',
 'wilych96-livejournal-com.turbopages.org': 'wilych96.livejournal.com',
 'jet--avia--blog-livejournal-com.turbopages.org': 'jet-avia-blog.livejournal.com',
 'slavniy99-livejournal-com.turbopages.org': 'slavniy99.livejournal.com',
 'p2beep-livejournal-com.turbopages.org': 'p2beep.livejournal.com',
 'yk--news-kz.turbopages.org': 'yk-news.kz',
 'belnaviny-by.turbopages.org': 'belnaviny.by',
 'max--sky-livejournal-com.turbopages.org': 'max-sky.livejournal.com',
 'ru--fenomen-livejournal-com.turbopages.org': 'ru-fenomen.livejournal.com',
 'atom222-livejournal-com.turbopages.org': 'atom222.livejournal.com',
 'lilofeia-livejournal-com.turbopages.org': 'lilofeia.livejournal.com',
 'shevoxa-livejournal-com.turbopages.org': 'shevoxa.livejournal.com',
 'vlad--khr-livejournal-com.turbopages.org': 'vlad-khr.livejournal.com',
 'pianoatlas-livejournal-com.turbopages.org': 'pianoatlas.livejournal.com',
 'kunashir-livejournal-com.turbopages.org': 'kunashir.livejournal.com',
 'protvagrad-livejournal-com.turbopages.org': 'protvagrad.livejournal.com',
 'elesika73-livejournal-com.turbopages.org': 'elesika73.livejournal.com',
 'diana--janzen-livejournal-com.turbopages.org': 'diana-janzen.livejournal.com',
 'handemade--ilia-livejournal-com.turbopages.org': 'handemade-ilia.livejournal.com',
 'igorui-livejournal-com.turbopages.org': 'igorui.livejournal.com',
 'parus28-livejournal-com.turbopages.org': 'parus28.livejournal.com',
 'w--okkam-livejournal-com.turbopages.org': 'w-okkam.livejournal.com',
 'dimagurov-livejournal-com.turbopages.org': 'dimagurov.livejournal.com',
 'avis5-livejournal-com.turbopages.org': 'avis5.livejournal.com',
 'serjru-livejournal-com.turbopages.org': 'serjru.livejournal.com',
 'virlandii-ru.turbopages.org': 'virlandii.ru',
 'galea--galley-livejournal-com.turbopages.org': 'galea-galley.livejournal.com',
 'callmycow-livejournal-com.turbopages.org': 'callmycow.livejournal.com',
 'analizator--sam-livejournal-com.turbopages.org': 'analizator-sam.livejournal.com',
 'kokteyly-ru.turbopages.org': 'kokteyly.ru',
 'joki--linguarum-livejournal-com.turbopages.org': 'joki-linguarum.livejournal.com',
 'morowain-livejournal-com.turbopages.org': 'morowain.livejournal.com',
 'optica--raduga-ru.turbopages.org': 'optica-raduga.ru',
 'dopul-ru.turbopages.org': 'dopul.ru',
 'senbernar--ru-livejournal-com.turbopages.org': 'senbernar-ru.livejournal.com',
 'summer56-livejournal-com.turbopages.org': 'summer56.livejournal.com',
 'club--vesta-ru.turbopages.org': 'club-vesta.ru',
 'mrfilin-com.turbopages.org': 'mrfilin.com',
 'kriek-ru.turbopages.org': 'kriek.ru',
 'evgenuss-livejournal-com.turbopages.org': 'evgenuss.livejournal.com',
 'su--industria-livejournal-com.turbopages.org': 'su-industria.livejournal.com',
 '0lga--marple-livejournal-com.turbopages.org': '0lga-marple.livejournal.com',
 'us--retired-livejournal-com.turbopages.org': 'us-retired.livejournal.com',
 'montaj--decor-ru.turbopages.org': 'montaj-decor.ru',
 'dimashyshkin-livejournal-com.turbopages.org': 'dimashyshkin.livejournal.com',
 'vladimir--akinin-livejournal-com.turbopages.org': 'vladimir-akinin.livejournal.com',
 'humanstory-ru.turbopages.org': 'humanstory.ru',
 'alexey--klyuev-livejournal-com.turbopages.org': 'alexey-klyuev.livejournal.com',
 'v--kalashnikov-livejournal-com.turbopages.org': 'v-kalashnikov.livejournal.com',
 'perekrestokclub-ru.turbopages.org': 'perekrestokclub.ru',
 'madrib-livejournal-com.turbopages.org': 'madrib.livejournal.com',
 'zugumov--zaur-livejournal-com.turbopages.org': 'zugumov-zaur.livejournal.com',
 'kashira-bezformata-com.turbopages.org': 'kashira.bezformata.com',
 'lussien-livejournal-com.turbopages.org': 'lussien.livejournal.com',
 'maritebo-livejournal-com.turbopages.org': 'maritebo.livejournal.com',
 'babkin--k-livejournal-com.turbopages.org': 'babkin-k.livejournal.com',
 'mts--is-ru.turbopages.org': 'mts-is.ru',
 'initiativ--group-livejournal-com.turbopages.org': 'initiativ-group.livejournal.com',
 'akdgs62-ru.turbopages.org': 'akdgs62.ru',
 'stuzer-livejournal-com.turbopages.org': 'stuzer.livejournal.com',
 'ranja-livejournal-com.turbopages.org': 'ranja.livejournal.com',
 'shepelev-livejournal-com.turbopages.org': 'shepelev.livejournal.com',
 'alex--uuu-livejournal-com.turbopages.org': 'alex-uuu.livejournal.com',
 'tiberius66-livejournal-com.turbopages.org': 'tiberius66.livejournal.com',
 'abcdefgh-livejournal-com.turbopages.org': 'abcdefgh.livejournal.com',
 'smb-ixbt-com.turbopages.org': 'smb.ixbt.com',
 'doktorbel-livejournal-com.turbopages.org': 'doktorbel.livejournal.com',
 'peccator--sum-livejournal-com.turbopages.org': 'peccator-sum.livejournal.com',
 'inessaleb-livejournal-com.turbopages.org': 'inessaleb.livejournal.com',
 'saga--optika-ru.turbopages.org': 'saga-optika.ru',
 'shtem-ru.turbopages.org': 'shtem.ru',
 'el--info-ru.turbopages.org': 'el-info.ru',
 'zhivoi-livejournal-com.turbopages.org': 'zhivoi.livejournal.com',
 'turist--kavkaz-ru.turbopages.org': 'turist-kavkaz.ru',
 'ru--scorpio-livejournal-com.turbopages.org': 'ru-scorpio.livejournal.com',
 'magis--amica-livejournal-com.turbopages.org': 'magis-amica.livejournal.com',
 'remi--meisner-livejournal-com.turbopages.org': 'remi-meisner.livejournal.com',
 'dmsh92-livejournal-com.turbopages.org': 'dmsh92.livejournal.com',
 'ekonovalov-livejournal-com.turbopages.org': 'ekonovalov.livejournal.com',
 'kp-kz.turbopages.org': 'kp.kz',
 'sn--portal-ru.turbopages.org': 'sn-portal.ru',
 'zmikealex-livejournal-com.turbopages.org': 'zmikealex.livejournal.com',
 'wordpressmania-ru.turbopages.org': 'wordpressmania.ru',
 'hipilion-livejournal-com.turbopages.org': 'hipilion.livejournal.com',
 'pustota--2009-livejournal-com.turbopages.org': 'pustota-2009.livejournal.com',
 'greezley--boy-livejournal-com.turbopages.org': 'greezley-boy.livejournal.com',
 'labas-livejournal-com.turbopages.org': 'labas.livejournal.com',
 'nosh2-livejournal-com.turbopages.org': 'nosh2.livejournal.com',
 'turbonsk-ru.turbopages.org': 'turbonsk.ru',
 'saprpavel-livejournal-com.turbopages.org': 'saprpavel.livejournal.com',
 'razvivashka-site.turbopages.org': 'razvivashka.site',
 'geodzen-com.turbopages.org': 'geodzen.com',
 'n--gantievskaya-livejournal-com.turbopages.org': 'n-gantievskaya.livejournal.com',
 'pro--usadbi-livejournal-com.turbopages.org': 'pro-usadbi.livejournal.com',
 'maykop-big--book--med-ru.turbopages.org': 'maykop.big-book-med.ru',
 'browserinfo-ru.turbopages.org': 'browserinfo.ru',
 'repin-livejournal-com.turbopages.org': 'repin.livejournal.com',
 'taiyna-livejournal-com.turbopages.org': 'taiyna.livejournal.com',
 'mapapama-ru.turbopages.org': 'mapapama.ru',
 'denyaleto-livejournal-com.turbopages.org': 'denyaleto.livejournal.com',
 'ira--travel-livejournal-com.turbopages.org': 'ira-travel.livejournal.com',
 '5and8-livejournal-com.turbopages.org': '5and8.livejournal.com',
 'stalamamoyv35-ru.turbopages.org': 'stalamamoyv35.ru',
 'kopika-livejournal-com.turbopages.org': 'kopika.livejournal.com',
 'yakyaks-livejournal-com.turbopages.org': 'yakyaks.livejournal.com',
 'zetlex-net.turbopages.org': 'zetlex.net',
 'wildmale-livejournal-com.turbopages.org': 'wildmale.livejournal.com',
 'andruru-livejournal-com.turbopages.org': 'andruru.livejournal.com',
 '13vainamoinen-livejournal-com.turbopages.org': '13vainamoinen.livejournal.com',
 'xtreamfory-com.turbopages.org': 'xtreamfory.com',
 'irkuem-livejournal-com.turbopages.org': 'irkuem.livejournal.com',
 'vlad--gluh-livejournal-com.turbopages.org': 'vlad-gluh.livejournal.com',
 'liza--valieva-livejournal-com.turbopages.org': 'liza-valieva.livejournal.com',
 'stomaa-livejournal-com.turbopages.org': 'stomaa.livejournal.com',
 'humankind-livejournal-com.turbopages.org': 'humankind.livejournal.com',
 'pokodeksu-ru.turbopages.org': 'pokodeksu.ru',
 'retsept--salatov-livejournal-com.turbopages.org': 'retsept-salatov.livejournal.com',
 'pupunia93-livejournal-com.turbopages.org': 'pupunia93.livejournal.com',
 'ralphmirebs-livejournal-com.turbopages.org': 'ralphmirebs.livejournal.com',
 'sredaobitaniyatv-ru.turbopages.org': 'sredaobitaniyatv.ru',
 'uznayka-livejournal-com.turbopages.org': 'uznayka.livejournal.com',
 'new--finder-livejournal-com.turbopages.org': 'new-finder.livejournal.com',
 'dobriy--samarit-livejournal-com.turbopages.org': 'dobriy-samarit.livejournal.com',
 'zvonaryov-livejournal-com.turbopages.org': 'zvonaryov.livejournal.com',
 'stalker347-livejournal-com.turbopages.org': 'stalker347.livejournal.com',
 'mcensk-bezformata-com.turbopages.org': 'mcensk.bezformata.com',
 'samodelki--info-ru.turbopages.org': 'samodelki-info.ru',
 'vereteno--frigg-livejournal-com.turbopages.org': 'vereteno-frigg.livejournal.com',
 'veneya-livejournal-com.turbopages.org': 'veneya.livejournal.com',
 'option--systems-livejournal-com.turbopages.org': 'option-systems.livejournal.com',
 'popova--artclass-livejournal-com.turbopages.org': 'popova-artclass.livejournal.com',
 'svonb-livejournal-com.turbopages.org': 'svonb.livejournal.com',
 'inesa--baronessa-livejournal-com.turbopages.org': 'inesa-baronessa.livejournal.com',
 'stop3000-livejournal-com.turbopages.org': 'stop3000.livejournal.com',
 'blef--nasa-livejournal-com.turbopages.org': 'blef-nasa.livejournal.com',
 'alla--ta-livejournal-com.turbopages.org': 'alla-ta.livejournal.com',
 'edoni-livejournal-com.turbopages.org': 'edoni.livejournal.com',
 'travel--fanat-livejournal-com.turbopages.org': 'travel-fanat.livejournal.com',
 'pavelrudnev-livejournal-com.turbopages.org': 'pavelrudnev.livejournal.com',
 'gromobor-livejournal-com.turbopages.org': 'gromobor.livejournal.com',
 'common--man-livejournal-com.turbopages.org': 'common-man.livejournal.com',
 'fiteria-ru.turbopages.org': 'fiteria.ru',
 'ebookwriter-livejournal-com.turbopages.org': 'ebookwriter.livejournal.com',
 'vetryanka-net.turbopages.org': 'vetryanka.net',
 'tenbest-ru.turbopages.org': 'tenbest.ru',
 'hipot-livejournal-com.turbopages.org': 'hipot.livejournal.com',
 'eysmont14-livejournal-com.turbopages.org': 'eysmont14.livejournal.com',
 't--34--111-livejournal-com.turbopages.org': 't-34-111.livejournal.com',
 'olegshnyrev-livejournal-com.turbopages.org': 'olegshnyrev.livejournal.com',
 'yekaterinburg-big--book--med-ru.turbopages.org': 'yekaterinburg.big-book-med.ru',
 'ikonoplev-livejournal-com.turbopages.org': 'ikonoplev.livejournal.com',
 'makulovvladimir-livejournal-com.turbopages.org': 'makulovvladimir.livejournal.com',
 '1500py470-livejournal-com.turbopages.org': '1500py470.livejournal.com',
 'astrakhan-su.turbopages.org': 'astrakhan.su',
 'mila--lova-livejournal-com.turbopages.org': 'mila-lova.livejournal.com',
 'mihtimak-livejournal-com.turbopages.org': 'mihtimak.livejournal.com',
 'shipilevsky-livejournal-com.turbopages.org': 'shipilevsky.livejournal.com',
 'zelenstroy--biysk-ru.turbopages.org': 'zelenstroy-biysk.ru',
 'polet--64-livejournal-com.turbopages.org': 'polet-64.livejournal.com',
 'proftpd2000-livejournal-com.turbopages.org': 'proftpd2000.livejournal.com',
 'shejere-livejournal-com.turbopages.org': 'shejere.livejournal.com',
 'zabvolos-ru.turbopages.org': 'zabvolos.ru',
 'am--9-ru.turbopages.org': 'am-9.ru',
 'ryazan-big--book--med-ru.turbopages.org': 'ryazan.big-book-med.ru',
 'garden--vlad-livejournal-com.turbopages.org': 'garden-vlad.livejournal.com',
 'autocad--ru-livejournal-com.turbopages.org': 'autocad-ru.livejournal.com',
 'osbsd-com.turbopages.org': 'osbsd.com',
 'kovrov-bezformata-com.turbopages.org': 'kovrov.bezformata.com',
 'gorlouhonos-ru.turbopages.org': 'gorlouhonos.ru',
 'dream502-livejournal-com.turbopages.org': 'dream502.livejournal.com',
 'darwinson-livejournal-com.turbopages.org': 'darwinson.livejournal.com',
 'xn------7cdaby8bbnadzdffyhl2b.turbopages.org': 'xn---7cdaby8bbnadzdffyhl2b',
 'online--urozhai-ru.turbopages.org': 'online-urozhai.ru',
 'predeina--zaural-livejournal-com.turbopages.org': 'predeina-zaural.livejournal.com',
 'vis0tnik-livejournal-com.turbopages.org': 'vis0tnik.livejournal.com',
 'linalapina-livejournal-com.turbopages.org': 'linalapina.livejournal.com',
 'vocal--noty-ru.turbopages.org': 'vocal-noty.ru',
 'irinadvorkina-livejournal-com.turbopages.org': 'irinadvorkina.livejournal.com',
 'kirinushka-livejournal-com.turbopages.org': 'kirinushka.livejournal.com',
 'm--yu--sokolov-livejournal-com.turbopages.org': 'm-yu-sokolov.livejournal.com',
 'dolgih-livejournal-com.turbopages.org': 'dolgih.livejournal.com',
 'magas-bezformata-com.turbopages.org': 'magas.bezformata.com',
 'frayer--man-livejournal-com.turbopages.org': 'frayer-man.livejournal.com',
 'megazuz-livejournal-com.turbopages.org': 'megazuz.livejournal.com',
 'alezgo-com.turbopages.org': 'alezgo.com',
 'luckypit-livejournal-com.turbopages.org': 'luckypit.livejournal.com',
 'utro--vecher-livejournal-com.turbopages.org': 'utro-vecher.livejournal.com',
 'anikeev--yuriy-livejournal-com.turbopages.org': 'anikeev-yuriy.livejournal.com',
 'kavkaz--istina-livejournal-com.turbopages.org': 'kavkaz-istina.livejournal.com',
 'pomolvka-guru.turbopages.org': 'pomolvka.guru',
 'evaevg-livejournal-com.turbopages.org': 'evaevg.livejournal.com',
 'abiens--abi-livejournal-com.turbopages.org': 'abiens-abi.livejournal.com',
 'alexkostabrava-livejournal-com.turbopages.org': 'alexkostabrava.livejournal.com',
 'nc59-ru.turbopages.org': 'nc59.ru',
 'dipifr1968-livejournal-com.turbopages.org': 'dipifr1968.livejournal.com',
 'club--365-livejournal-com.turbopages.org': 'club-365.livejournal.com',
 'starovina-livejournal-com.turbopages.org': 'starovina.livejournal.com',
 'chel--week-ru.turbopages.org': 'chel-week.ru',
 'statistwot-ru.turbopages.org': 'statistwot.ru',
 'sarcasmi-livejournal-com.turbopages.org': 'sarcasmi.livejournal.com',
 'pandorabox--ru-livejournal-com.turbopages.org': 'pandorabox-ru.livejournal.com',
 'pbtest24-ru.turbopages.org': 'pbtest24.ru',
 'rita542-livejournal-com.turbopages.org': 'rita542.livejournal.com',
 'kirk--bv-livejournal-com.turbopages.org': 'kirk-bv.livejournal.com',
 'chebatkov-livejournal-com.turbopages.org': 'chebatkov.livejournal.com',
 'shusharmor-livejournal-com.turbopages.org': 'shusharmor.livejournal.com',
 'xn--1--elcklt7bvm.turbopages.org': 'xn-1-elcklt7bvm',
 'fintraining-livejournal-com.turbopages.org': 'fintraining.livejournal.com',
 'progrud-com.turbopages.org': 'progrud.com',
 'irinapravdina-livejournal-com.turbopages.org': 'irinapravdina.livejournal.com',
 'blankplus-ru.turbopages.org': 'blankplus.ru',
 'shaltay0boltay-livejournal-com.turbopages.org': 'shaltay0boltay.livejournal.com',
 'polydi-livejournal-com.turbopages.org': 'polydi.livejournal.com',
 'vvv2010-livejournal-com.turbopages.org': 'vvv2010.livejournal.com',
 'alla--pj-livejournal-com.turbopages.org': 'alla-pj.livejournal.com',
 'soskidkami-ru.turbopages.org': 'soskidkami.ru',
 '2lumpsofsugar-livejournal-com.turbopages.org': '2lumpsofsugar.livejournal.com',
 'consensus-livejournal-com.turbopages.org': 'consensus.livejournal.com',
 'fashion--daily-livejournal-com.turbopages.org': 'fashion-daily.livejournal.com',
 'tpakt-livejournal-com.turbopages.org': 'tpakt.livejournal.com',
 'moikesha-ru.turbopages.org': 'moikesha.ru',
 'v--doljenko-livejournal-com.turbopages.org': 'v-doljenko.livejournal.com',
 'skypaem-ru.turbopages.org': 'skypaem.ru',
 'terralia-livejournal-com.turbopages.org': 'terralia.livejournal.com',
 'meggisimpson-livejournal-com.turbopages.org': 'meggisimpson.livejournal.com',
 'moscow--gothica-livejournal-com.turbopages.org': 'moscow-gothica.livejournal.com',
 'p--tashka-livejournal-com.turbopages.org': 'p-tashka.livejournal.com',
 'varlamovru-livejournal-com.turbopages.org': 'varlamovru.livejournal.com',
 'apple--russia-livejournal-com.turbopages.org': 'apple-russia.livejournal.com',
 'shakherezada-livejournal-com.turbopages.org': 'shakherezada.livejournal.com',
 'viktorbobrov89-livejournal-com.turbopages.org': 'viktorbobrov89.livejournal.com',
 'triakomm-livejournal-com.turbopages.org': 'triakomm.livejournal.com',
 'pereval1959-livejournal-com.turbopages.org': 'pereval1959.livejournal.com',
 'gents--blog-ru.turbopages.org': 'gents-blog.ru',
 'il--ducess-livejournal-com.turbopages.org': 'il-ducess.livejournal.com',
 'castaneda-livejournal-com.turbopages.org': 'castaneda.livejournal.com',
 'pan--demetrius-livejournal-com.turbopages.org': 'pan-demetrius.livejournal.com',
 'dissomnia-livejournal-com.turbopages.org': 'dissomnia.livejournal.com',
 'anastastazia-livejournal-com.turbopages.org': 'anastastazia.livejournal.com',
 'val20101-livejournal-com.turbopages.org': 'val20101.livejournal.com',
 'inzoomnet-livejournal-com.turbopages.org': 'inzoomnet.livejournal.com',
 'i--burjjui-livejournal-com.turbopages.org': 'i-burjjui.livejournal.com',
 'liteday-ru.turbopages.org': 'liteday.ru',
 'armavest-ru.turbopages.org': 'armavest.ru',
 'gull--25-livejournal-com.turbopages.org': 'gull-25.livejournal.com',
 'vegchel-ru.turbopages.org': 'vegchel.ru',
 'agroplus-livejournal-com.turbopages.org': 'agroplus.livejournal.com',
 'nik--aleks-livejournal-com.turbopages.org': 'nik-aleks.livejournal.com',
 'slengi-ru.turbopages.org': 'slengi.ru',
 'belenky-livejournal-com.turbopages.org': 'belenky.livejournal.com',
 'apti1-livejournal-com.turbopages.org': 'apti1.livejournal.com',
 'gm--dar-livejournal-com.turbopages.org': 'gm-dar.livejournal.com',
 'pyx--pyx-livejournal-com.turbopages.org': 'pyx-pyx.livejournal.com',
 'shurik--m-livejournal-com.turbopages.org': 'shurik-m.livejournal.com',
 'lenstroydom-ru.turbopages.org': 'lenstroydom.ru',
 'yandexbro-ru.turbopages.org': 'yandexbro.ru',
 'elektrik--a-su.turbopages.org': 'elektrik-a.su',
 '150cc-ru.turbopages.org': '150cc.ru',
 'xn----7sbouddofcqgqc8a7ig.turbopages.org': 'xn--7sbouddofcqgqc8a7ig',
 'pokrovitelptiz-livejournal-com.turbopages.org': 'pokrovitelptiz.livejournal.com',
 'lekzet-livejournal-com.turbopages.org': 'lekzet.livejournal.com',
 'eprst2000-livejournal-com.turbopages.org': 'eprst2000.livejournal.com',
 'naritsyna-livejournal-com.turbopages.org': 'naritsyna.livejournal.com',
 'nikkori-livejournal-com.turbopages.org': 'nikkori.livejournal.com',
 'womenburg-ru.turbopages.org': 'womenburg.ru',
 'nikolaiskuratov-livejournal-com.turbopages.org': 'nikolaiskuratov.livejournal.com',
 'inmysight-ru.turbopages.org': 'inmysight.ru',
 'neftekamsk-big--book--med-ru.turbopages.org': 'neftekamsk.big-book-med.ru',
 'anti--fasmer-livejournal-com.turbopages.org': 'anti-fasmer.livejournal.com',
 'arm--world-ru.turbopages.org': 'arm-world.ru',
 '13kadr-livejournal-com.turbopages.org': '13kadr.livejournal.com',
 'smmguru-info.turbopages.org': 'smmguru.info',
 'tochka--cborki-livejournal-com.turbopages.org': 'tochka-cborki.livejournal.com',
 'chungro-livejournal-com.turbopages.org': 'chungro.livejournal.com',
 'max--a--lebedev-livejournal-com.turbopages.org': 'max-a-lebedev.livejournal.com',
 'tksmi-ru.turbopages.org': 'tksmi.ru',
 'sova--1304-livejournal-com.turbopages.org': 'sova-1304.livejournal.com',
 'kvn--quote-livejournal-com.turbopages.org': 'kvn-quote.livejournal.com',
 'stavropolskiy-com.turbopages.org': 'stavropolskiy.com',
 'bankesse-ru.turbopages.org': 'bankesse.ru',
 'diondion-livejournal-com.turbopages.org': 'diondion.livejournal.com',
 'cegob-livejournal-com.turbopages.org': 'cegob.livejournal.com',
 'verapapkova-livejournal-com.turbopages.org': 'verapapkova.livejournal.com',
 'rgva-livejournal-com.turbopages.org': 'rgva.livejournal.com',
 'litl--bro-livejournal-com.turbopages.org': 'litl-bro.livejournal.com',
 'lj--editors-livejournal-com.turbopages.org': 'lj-editors.livejournal.com',
 'dushegubov-livejournal-com.turbopages.org': 'dushegubov.livejournal.com',
 '2babochki-livejournal-com.turbopages.org': '2babochki.livejournal.com',
 'vdovenko-livejournal-com.turbopages.org': 'vdovenko.livejournal.com',
 'dtpnet-livejournal-com.turbopages.org': 'dtpnet.livejournal.com',
 'time--fixer-livejournal-com.turbopages.org': 'time-fixer.livejournal.com',
 'fototoka-livejournal-com.turbopages.org': 'fototoka.livejournal.com',
 'ru--fotoplenka-livejournal-com.turbopages.org': 'ru-fotoplenka.livejournal.com',
 'penstart-livejournal-com.turbopages.org': 'penstart.livejournal.com',
 'kinomaestro-livejournal-com.turbopages.org': 'kinomaestro.livejournal.com',
 'maxilla--k-livejournal-com.turbopages.org': 'maxilla-k.livejournal.com',
 'marina--art--wool-livejournal-com.turbopages.org': 'marina-art-wool.livejournal.com',
 'p--d--m-livejournal-com.turbopages.org': 'p-d-m.livejournal.com',
 'sorokovs-livejournal-com.turbopages.org': 'sorokovs.livejournal.com',
 'podpolkovnikvvs-livejournal-com.turbopages.org': 'podpolkovnikvvs.livejournal.com',
 'lyalechka-livejournal-com.turbopages.org': 'lyalechka.livejournal.com',
 'zurn-ru.turbopages.org': 'zurn.ru',
 'gukovfamily-livejournal-com.turbopages.org': 'gukovfamily.livejournal.com',
 'brpochep-livejournal-com.turbopages.org': 'brpochep.livejournal.com',
 'dachamir-ru.turbopages.org': 'dachamir.ru',
 'corrente--1-livejournal-com.turbopages.org': 'corrente-1.livejournal.com',
 'jozhik-livejournal-com.turbopages.org': 'jozhik.livejournal.com',
 'spatrol-livejournal-com.turbopages.org': 'spatrol.livejournal.com',
 'grunja-livejournal-com.turbopages.org': 'grunja.livejournal.com',
 'wokrug--sveta-livejournal-com.turbopages.org': 'wokrug-sveta.livejournal.com',
 'thaiprofi-ru.turbopages.org': 'thaiprofi.ru',
 'gribok24-ru.turbopages.org': 'gribok24.ru',
 'wild--cat-livejournal-com.turbopages.org': 'wild-cat.livejournal.com',
 'fontyler-livejournal-com.turbopages.org': 'fontyler.livejournal.com',
 'ryazan--news-net.turbopages.org': 'ryazan-news.net',
 'nik--bet--78-livejournal-com.turbopages.org': 'nik-bet-78.livejournal.com',
 'stancoy-livejournal-com.turbopages.org': 'stancoy.livejournal.com',
 'maminblog24-ru.turbopages.org': 'maminblog24.ru',
 'newstbilisi-info.turbopages.org': 'newstbilisi.info',
 'start--fit-ru.turbopages.org': 'start-fit.ru',
 'kakras-livejournal-com.turbopages.org': 'kakras.livejournal.com',
 'sochny--otdyh-livejournal-com.turbopages.org': 'sochny-otdyh.livejournal.com',
 'c0men-livejournal-com.turbopages.org': 'c0men.livejournal.com',
 'leovik33-livejournal-com.turbopages.org': 'leovik33.livejournal.com',
 'scrolllock-livejournal-com.turbopages.org': 'scrolllock.livejournal.com',
 'prolaw24-ru.turbopages.org': 'prolaw24.ru',
 'kinoloh-livejournal-com.turbopages.org': 'kinoloh.livejournal.com',
 'pashkov-livejournal-com.turbopages.org': 'pashkov.livejournal.com',
 'ru--wqc-livejournal-com.turbopages.org': 'ru-wqc.livejournal.com',
 'zip--kirov-ru.turbopages.org': 'zip-kirov.ru',
 'o--andrey-livejournal-com.turbopages.org': 'o-andrey.livejournal.com',
 'd--sanin-livejournal-com.turbopages.org': 'd-sanin.livejournal.com',
 'kovaleva-livejournal-com.turbopages.org': 'kovaleva.livejournal.com',
 'bolotov--bv-livejournal-com.turbopages.org': 'bolotov-bv.livejournal.com',
 'cleverperson-livejournal-com.turbopages.org': 'cleverperson.livejournal.com',
 'asgorenyuk-livejournal-com.turbopages.org': 'asgorenyuk.livejournal.com',
 'bortnik-livejournal-com.turbopages.org': 'bortnik.livejournal.com',
 'audio--hi--end-livejournal-com.turbopages.org': 'audio-hi-end.livejournal.com',
 'un--tizio-livejournal-com.turbopages.org': 'un-tizio.livejournal.com',
 'annabaskakova-livejournal-com.turbopages.org': 'annabaskakova.livejournal.com',
 'akurochkin-livejournal-com.turbopages.org': 'akurochkin.livejournal.com',
 'ametis--vedma-livejournal-com.turbopages.org': 'ametis-vedma.livejournal.com',
 'esteticanapa-ru.turbopages.org': 'esteticanapa.ru',
 'resheniya--sudov2-ru.turbopages.org': 'resheniya-sudov2.ru',
 'na--zapade-ru.turbopages.org': 'na-zapade.ru',
 'old--bottles-livejournal-com.turbopages.org': 'old-bottles.livejournal.com',
 'godovenok-livejournal-com.turbopages.org': 'godovenok.livejournal.com',
 'ykristianna-livejournal-com.turbopages.org': 'ykristianna.livejournal.com',
 'standartgost-ru.turbopages.org': 'standartgost.ru',
 'krisandr-livejournal-com.turbopages.org': 'krisandr.livejournal.com',
 'topnahabino-ru.turbopages.org': 'topnahabino.ru',
 'lesovedenie-livejournal-com.turbopages.org': 'lesovedenie.livejournal.com',
 'lena--sama7-livejournal-com.turbopages.org': 'lena-sama7.livejournal.com',
 'personjob-ru.turbopages.org': 'personjob.ru',
 'vasiaalexandrov-livejournal-com.turbopages.org': 'vasiaalexandrov.livejournal.com',
 'bolshe--ne--mogu-livejournal-com.turbopages.org': 'bolshe-ne-mogu.livejournal.com',
 'elena030672-livejournal-com.turbopages.org': 'elena030672.livejournal.com',
 'seringvar-livejournal-com.turbopages.org': 'seringvar.livejournal.com',
 'babypalace-ru.turbopages.org': 'babypalace.ru',
 'kubazoud-livejournal-com.turbopages.org': 'kubazoud.livejournal.com',
 'stalinist-livejournal-com.turbopages.org': 'stalinist.livejournal.com',
 'org--gorod--tomsk-livejournal-com.turbopages.org': 'org-gorod-tomsk.livejournal.com',
 'permyakovr-livejournal-com.turbopages.org': 'permyakovr.livejournal.com',
 'smiroslav-livejournal-com.turbopages.org': 'smiroslav.livejournal.com',
 'childguide-ru.turbopages.org': 'childguide.ru',
 'alexsrb-livejournal-com.turbopages.org': 'alexsrb.livejournal.com',
 'cskainfo-com.turbopages.org': 'cskainfo.com',
 'mopsia-livejournal-com.turbopages.org': 'mopsia.livejournal.com',
 'ozma--lipetsk-livejournal-com.turbopages.org': 'ozma-lipetsk.livejournal.com',
 'tikandelaki-livejournal-com.turbopages.org': 'tikandelaki.livejournal.com',
 'poigarmonika-ru.turbopages.org': 'poigarmonika.ru',
 'osipov-livejournal-com.turbopages.org': 'osipov.livejournal.com',
 'mart999-livejournal-com.turbopages.org': 'mart999.livejournal.com',
 'miele-livejournal-com.turbopages.org': 'miele.livejournal.com',
 '16vik09-livejournal-com.turbopages.org': '16vik09.livejournal.com',
 'audiomania-livejournal-com.turbopages.org': 'audiomania.livejournal.com',
 'zakir--5535-livejournal-com.turbopages.org': 'zakir-5535.livejournal.com',
 'maks--markoff-livejournal-com.turbopages.org': 'maks-markoff.livejournal.com',
 'kononenkome-livejournal-com.turbopages.org': 'kononenkome.livejournal.com',
 'devochki--help-ru.turbopages.org': 'devochki-help.ru',
 'yasko-livejournal-com.turbopages.org': 'yasko.livejournal.com',
 'ps2sega-ru.turbopages.org': 'ps2sega.ru',
 'citataua-livejournal-com.turbopages.org': 'citataua.livejournal.com',
 'selestafox-livejournal-com.turbopages.org': 'selestafox.livejournal.com',
 'sib--bastion-ru.turbopages.org': 'sib-bastion.ru',
 'wolf--kitses-livejournal-com.turbopages.org': 'wolf-kitses.livejournal.com',
 'topmagiya-ru.turbopages.org': 'topmagiya.ru',
 'velikijsultan-ru.turbopages.org': 'velikijsultan.ru',
 'techno--od-livejournal-com.turbopages.org': 'techno-od.livejournal.com',
 'boris--prok-livejournal-com.turbopages.org': 'boris-prok.livejournal.com',
 'miass-bezformata-com.turbopages.org': 'miass.bezformata.com',
 'remotehelper-ru.turbopages.org': 'remotehelper.ru',
 'actmc-ru.turbopages.org': 'actmc.ru',
 'imperium--ross-livejournal-com.turbopages.org': 'imperium-ross.livejournal.com',
 'nionila-livejournal-com.turbopages.org': 'nionila.livejournal.com',
 'daria--ma-livejournal-com.turbopages.org': 'daria-ma.livejournal.com',
 'sakuradveri-livejournal-com.turbopages.org': 'sakuradveri.livejournal.com',
 'portal69tver-livejournal-com.turbopages.org': 'portal69tver.livejournal.com',
 'scienceblogger-livejournal-com.turbopages.org': 'scienceblogger.livejournal.com',
 'novgorodstat-livejournal-com.turbopages.org': 'novgorodstat.livejournal.com',
 'driver--id-ru.turbopages.org': 'driver-id.ru',
 'autoradioservice-ru.turbopages.org': 'autoradioservice.ru',
 'dzeso-livejournal-com.turbopages.org': 'dzeso.livejournal.com',
 'strannik--severa-livejournal-com.turbopages.org': 'strannik-severa.livejournal.com',
 'sam--turizm-ru.turbopages.org': 'sam-turizm.ru',
 'rujournalist-livejournal-com.turbopages.org': 'rujournalist.livejournal.com',
 'xn------------zyhr0asf5bgbaghiwme0bugjeb1aqyz8i1a7c5a.turbopages.org': 'xn------zyhr0asf5bgbaghiwme0bugjeb1aqyz8i1a7c5a',
 'tomas--morr-livejournal-com.turbopages.org': 'tomas-morr.livejournal.com',
 'schadribicus-livejournal-com.turbopages.org': 'schadribicus.livejournal.com',
 'master7009-livejournal-com.turbopages.org': 'master7009.livejournal.com',
 'vse--obipoteke-ru.turbopages.org': 'vse-obipoteke.ru',
 'aminavra-livejournal-com.turbopages.org': 'aminavra.livejournal.com',
 'lennews-ru.turbopages.org': 'lennews.ru',
 'magistral77-livejournal-com.turbopages.org': 'magistral77.livejournal.com',
 'balberov-livejournal-com.turbopages.org': 'balberov.livejournal.com',
 'ermakovandrey-ru.turbopages.org': 'ermakovandrey.ru',
 'biletiki-livejournal-com.turbopages.org': 'biletiki.livejournal.com',
 'povar--v-ru.turbopages.org': 'povar-v.ru',
 'saidpvo-livejournal-com.turbopages.org': 'saidpvo.livejournal.com',
 'nutrisale-livejournal-com.turbopages.org': 'nutrisale.livejournal.com',
 'shonok-livejournal-com.turbopages.org': 'shonok.livejournal.com',
 'ivc8bit-ru.turbopages.org': 'ivc8bit.ru',
 'moscow-big--book--edu-ru.turbopages.org': 'moscow.big-book-edu.ru',
 'blearreaz0n-livejournal-com.turbopages.org': 'blearreaz0n.livejournal.com',
 'ilya--verhovsky-livejournal-com.turbopages.org': 'ilya-verhovsky.livejournal.com',
 'phalaris-livejournal-com.turbopages.org': 'phalaris.livejournal.com',
 'sigurthr--galdra-livejournal-com.turbopages.org': 'sigurthr-galdra.livejournal.com',
 'nooma--magia-livejournal-com.turbopages.org': 'nooma-magia.livejournal.com',
 'all--worlds-ru.turbopages.org': 'all-worlds.ru',
 'rudnik-mobi.turbopages.org': 'rudnik.mobi',
 'zeka--vasch-livejournal-com.turbopages.org': 'zeka-vasch.livejournal.com',
 'azlk--team-ru.turbopages.org': 'azlk-team.ru',
 'whiteobserver-livejournal-com.turbopages.org': 'whiteobserver.livejournal.com',
 '5tudent-livejournal-com.turbopages.org': '5tudent.livejournal.com',
 'ecosever-ru.turbopages.org': 'ecosever.ru',
 'grozniy-bezformata-com.turbopages.org': 'grozniy.bezformata.com',
 'madam57-livejournal-com.turbopages.org': 'madam57.livejournal.com',
 'plimpa-livejournal-com.turbopages.org': 'plimpa.livejournal.com',
 'shaman--karagay-livejournal-com.turbopages.org': 'shaman-karagay.livejournal.com',
 'dormidontesq-livejournal-com.turbopages.org': 'dormidontesq.livejournal.com',
 'palaman-livejournal-com.turbopages.org': 'palaman.livejournal.com',
 'velocyraptor-livejournal-com.turbopages.org': 'velocyraptor.livejournal.com',
 'karjalaskier-livejournal-com.turbopages.org': 'karjalaskier.livejournal.com',
 'sstass-livejournal-com.turbopages.org': 'sstass.livejournal.com',
 'raevskaya--o-livejournal-com.turbopages.org': 'raevskaya-o.livejournal.com',
 'zametkianny-livejournal-com.turbopages.org': 'zametkianny.livejournal.com',
 'reposted-livejournal-com.turbopages.org': 'reposted.livejournal.com',
 'specavto69-ru.turbopages.org': 'specavto69.ru',
 'rights--freedom-livejournal-com.turbopages.org': 'rights-freedom.livejournal.com',
 'karieva--raufa-livejournal-com.turbopages.org': 'karieva-raufa.livejournal.com',
 'odonta-org.turbopages.org': 'odonta.org',
 'cemetery--ru-livejournal-com.turbopages.org': 'cemetery-ru.livejournal.com',
 'ru--sovarch-livejournal-com.turbopages.org': 'ru-sovarch.livejournal.com',
 'bohemicus-livejournal-com.turbopages.org': 'bohemicus.livejournal.com',
 'sbolt-livejournal-com.turbopages.org': 'sbolt.livejournal.com',
 'etceterini-livejournal-com.turbopages.org': 'etceterini.livejournal.com',
 'irisss--k-livejournal-com.turbopages.org': 'irisss-k.livejournal.com',
 'kapuchin-livejournal-com.turbopages.org': 'kapuchin.livejournal.com',
 'buhlabaz-ru.turbopages.org': 'buhlabaz.ru',
 'anonssmi-ru.turbopages.org': 'anonssmi.ru',
 'irina--chaykina-livejournal-com.turbopages.org': 'irina-chaykina.livejournal.com',
 'dom--i--semya-com.turbopages.org': 'dom-i-semya.com',
 'klock-livejournal-com.turbopages.org': 'klock.livejournal.com',
 'testnavirus-com.turbopages.org': 'testnavirus.com',
 'can--motalka-livejournal-com.turbopages.org': 'can-motalka.livejournal.com',
 'billi--rubin-livejournal-com.turbopages.org': 'billi-rubin.livejournal.com',
 'kamendon-ru.turbopages.org': 'kamendon.ru',
 'cosmogenesis-livejournal-com.turbopages.org': 'cosmogenesis.livejournal.com',
 'truespec-livejournal-com.turbopages.org': 'truespec.livejournal.com',
 'viktor--te3-livejournal-com.turbopages.org': 'viktor-te3.livejournal.com',
 'filfak--agao-livejournal-com.turbopages.org': 'filfak-agao.livejournal.com',
 'nafanko-livejournal-com.turbopages.org': 'nafanko.livejournal.com',
 'turist--sa-livejournal-com.turbopages.org': 'turist-sa.livejournal.com',
 'ekzogamiya-livejournal-com.turbopages.org': 'ekzogamiya.livejournal.com',
 'pirog--life-livejournal-com.turbopages.org': 'pirog-life.livejournal.com',
 'mono--able-livejournal-com.turbopages.org': 'mono-able.livejournal.com',
 'azizam--iri-livejournal-com.turbopages.org': 'azizam-iri.livejournal.com',
 'rjev-bezformata-com.turbopages.org': 'rjev.bezformata.com',
 'heattreatment-ru.turbopages.org': 'heattreatment.ru',
 'storytelling--ru-livejournal-com.turbopages.org': 'storytelling-ru.livejournal.com',
 'kruzenwtern-livejournal-com.turbopages.org': 'kruzenwtern.livejournal.com',
 'anna--chertkova-livejournal-com.turbopages.org': 'anna-chertkova.livejournal.com',
 'nikab-livejournal-com.turbopages.org': 'nikab.livejournal.com',
 'intesense-ru.turbopages.org': 'intesense.ru',
 'belayakisa-livejournal-com.turbopages.org': 'belayakisa.livejournal.com',
 'gunesru-livejournal-com.turbopages.org': 'gunesru.livejournal.com',
 'api--x-livejournal-com.turbopages.org': 'api-x.livejournal.com',
 'bylkov-ru.turbopages.org': 'bylkov.ru',
 'cookery-guru.turbopages.org': 'cookery.guru',
 'ra3dhl-livejournal-com.turbopages.org': 'ra3dhl.livejournal.com',
 'niferon-livejournal-com.turbopages.org': 'niferon.livejournal.com',
 'lyces-livejournal-com.turbopages.org': 'lyces.livejournal.com',
 'xn----7sbcm2agk7avq.turbopages.org': 'xn--7sbcm2agk7avq',
 'razborka--moskva-livejournal-com.turbopages.org': 'razborka-moskva.livejournal.com',
 'gryadki-com.turbopages.org': 'gryadki.com',
 'avarskii-livejournal-com.turbopages.org': 'avarskii.livejournal.com',
 'china--phone-info.turbopages.org': 'china-phone.info',
 'p00h-livejournal-com.turbopages.org': 'p00h.livejournal.com',
 'a--shalimov-livejournal-com.turbopages.org': 'a-shalimov.livejournal.com',
 'sammy--belarus-livejournal-com.turbopages.org': 'sammy-belarus.livejournal.com',
 'mosoblprofgos-ru.turbopages.org': 'mosoblprofgos.ru',
 'stopobman777-livejournal-com.turbopages.org': 'stopobman777.livejournal.com',
 'ru52-livejournal-com.turbopages.org': 'ru52.livejournal.com',
 'udav102-livejournal-com.turbopages.org': 'udav102.livejournal.com',
 'blumavolfovna-livejournal-com.turbopages.org': 'blumavolfovna.livejournal.com',
 'missis--panda-livejournal-com.turbopages.org': 'missis-panda.livejournal.com',
 'sasha0404-livejournal-com.turbopages.org': 'sasha0404.livejournal.com',
 'instanation-ru.turbopages.org': 'instanation.ru',
 'fatima611-livejournal-com.turbopages.org': 'fatima611.livejournal.com',
 'na--shpilke-livejournal-com.turbopages.org': 'na-shpilke.livejournal.com',
 'daria--telezhko-livejournal-com.turbopages.org': 'daria-telezhko.livejournal.com',
 'tosainu-livejournal-com.turbopages.org': 'tosainu.livejournal.com',
 'oops-ru.turbopages.org': 'oops.ru',
 'interviwer-livejournal-com.turbopages.org': 'interviwer.livejournal.com',
 'skrust-livejournal-com.turbopages.org': 'skrust.livejournal.com',
 't2t-livejournal-com.turbopages.org': 't2t.livejournal.com',
 'zloy--hipster-livejournal-com.turbopages.org': 'zloy-hipster.livejournal.com',
 'avtonovosty1-ru.turbopages.org': 'avtonovosty1.ru',
 'trinity--med-ru.turbopages.org': 'trinity-med.ru',
 'samuray--074-livejournal-com.turbopages.org': 'samuray-074.livejournal.com',
 'apleg--ru-livejournal-com.turbopages.org': 'apleg-ru.livejournal.com',
 'nadiatanguy-livejournal-com.turbopages.org': 'nadiatanguy.livejournal.com',
 'viktor--aralov-livejournal-com.turbopages.org': 'viktor-aralov.livejournal.com',
 'nosh--i-livejournal-com.turbopages.org': 'nosh-i.livejournal.com',
 'feminism-livejournal-com.turbopages.org': 'feminism.livejournal.com',
 'orden--z--flaga-livejournal-com.turbopages.org': 'orden-z-flaga.livejournal.com',
 'good--wish-livejournal-com.turbopages.org': 'good-wish.livejournal.com',
 'alkoletchik-livejournal-com.turbopages.org': 'alkoletchik.livejournal.com',
 'vlad68borisov-livejournal-com.turbopages.org': 'vlad68borisov.livejournal.com',
 'tascha--cook-livejournal-com.turbopages.org': 'tascha-cook.livejournal.com',
 'lacosanostra007-livejournal-com.turbopages.org': 'lacosanostra007.livejournal.com',
 'dm--bondarenko-livejournal-com.turbopages.org': 'dm-bondarenko.livejournal.com',
 'ivanesergeev-livejournal-com.turbopages.org': 'ivanesergeev.livejournal.com',
 'leonidluchkin-livejournal-com.turbopages.org': 'leonidluchkin.livejournal.com',
 '000000000-livejournal-com.turbopages.org': '000000000.livejournal.com',
 'naturemed-ru.turbopages.org': 'naturemed.ru',
 'retromap-livejournal-com.turbopages.org': 'retromap.livejournal.com',
 'ershowg-livejournal-com.turbopages.org': 'ershowg.livejournal.com',
 'kosdor-livejournal-com.turbopages.org': 'kosdor.livejournal.com',
 'remstroy-pro.turbopages.org': 'remstroy.pro',
 'kolganov--su-livejournal-com.turbopages.org': 'kolganov-su.livejournal.com',
 'remont--main-ru.turbopages.org': 'remont-main.ru',
 'pv--nkvd-livejournal-com.turbopages.org': 'pv-nkvd.livejournal.com',
 'rx24-ru.turbopages.org': 'rx24.ru',
 'crimea--guru-ru.turbopages.org': 'crimea-guru.ru',
 'fotovmire-ru.turbopages.org': 'fotovmire.ru',
 'pan4en-livejournal-com.turbopages.org': 'pan4en.livejournal.com',
 'ruza-bezformata-com.turbopages.org': 'ruza.bezformata.com',
 'arabyaz-livejournal-com.turbopages.org': 'arabyaz.livejournal.com',
 'renatar-livejournal-com.turbopages.org': 'renatar.livejournal.com',
 'n--caleb-livejournal-com.turbopages.org': 'n-caleb.livejournal.com',
 'nahash-livejournal-com.turbopages.org': 'nahash.livejournal.com',
 'conversion-su.turbopages.org': 'conversion.su',
 'azov-donday-ru.turbopages.org': 'azov.donday.ru',
 'slo--vo-livejournal-com.turbopages.org': 'slo-vo.livejournal.com',
 'anonim--from--rus-livejournal-com.turbopages.org': 'anonim-from-rus.livejournal.com',
 'e--stroy-pro.turbopages.org': 'e-stroy.pro',
 'liubov--zla-livejournal-com.turbopages.org': 'liubov-zla.livejournal.com',
 'staraya--dacha-livejournal-com.turbopages.org': 'staraya-dacha.livejournal.com',
 'd--m--vestnik-livejournal-com.turbopages.org': 'd-m-vestnik.livejournal.com',
 'stabrk-livejournal-com.turbopages.org': 'stabrk.livejournal.com',
 'ttweak-livejournal-com.turbopages.org': 'ttweak.livejournal.com',
 'jmcreation-livejournal-com.turbopages.org': 'jmcreation.livejournal.com',
 'berdnik--stiv-livejournal-com.turbopages.org': 'berdnik-stiv.livejournal.com',
 'strou--sam-ru.turbopages.org': 'strou-sam.ru',
 'wkrymu-ru.turbopages.org': 'wkrymu.ru',
 'kreekate--01-livejournal-com.turbopages.org': 'kreekate-01.livejournal.com',
 'taffy729-livejournal-com.turbopages.org': 'taffy729.livejournal.com',
 'cafard-livejournal-com.turbopages.org': 'cafard.livejournal.com',
 'vasiliybalanyuk-livejournal-com.turbopages.org': 'vasiliybalanyuk.livejournal.com',
 'shpatak-livejournal-com.turbopages.org': 'shpatak.livejournal.com',
 'alex--stepanofff-livejournal-com.turbopages.org': 'alex-stepanofff.livejournal.com',
 'hanka--ordonowna-livejournal-com.turbopages.org': 'hanka-ordonowna.livejournal.com',
 'rus--turk-livejournal-com.turbopages.org': 'rus-turk.livejournal.com',
 'skreebl-livejournal-com.turbopages.org': 'skreebl.livejournal.com',
 'alex--averyanov-livejournal-com.turbopages.org': 'alex-averyanov.livejournal.com',
 'manage--it-livejournal-com.turbopages.org': 'manage-it.livejournal.com',
 'postremont-ru.turbopages.org': 'postremont.ru',
 'robloxgame-ru.turbopages.org': 'robloxgame.ru',
 'i--shalyt-livejournal-com.turbopages.org': 'i-shalyt.livejournal.com',
 'prorem0nt-ru.turbopages.org': 'prorem0nt.ru',
 'vetumtrud-livejournal-com.turbopages.org': 'vetumtrud.livejournal.com',
 'oxana--oxanovna-livejournal-com.turbopages.org': 'oxana-oxanovna.livejournal.com',
 'ru--chrysotile-livejournal-com.turbopages.org': 'ru-chrysotile.livejournal.com',
 'avrom-livejournal-com.turbopages.org': 'avrom.livejournal.com',
 'muza--130672-livejournal-com.turbopages.org': 'muza-130672.livejournal.com',
 'rst--163-livejournal-com.turbopages.org': 'rst-163.livejournal.com',
 'dinara--d--oro-livejournal-com.turbopages.org': 'dinara-d-oro.livejournal.com',
 'miroslavv-livejournal-com.turbopages.org': 'miroslavv.livejournal.com',
 'dulsinneya-livejournal-com.turbopages.org': 'dulsinneya.livejournal.com',
 'ortclinic-ru.turbopages.org': 'ortclinic.ru',
 'vetapeta-com.turbopages.org': 'vetapeta.com',
 'nauchitsya-ru.turbopages.org': 'nauchitsya.ru',
 'photophren-livejournal-com.turbopages.org': 'photophren.livejournal.com',
 'refsou-livejournal-com.turbopages.org': 'refsou.livejournal.com',
 'mirai8-livejournal-com.turbopages.org': 'mirai8.livejournal.com',
 'kitv-livejournal-com.turbopages.org': 'kitv.livejournal.com',
 'kalininskaya-bezformata-com.turbopages.org': 'kalininskaya.bezformata.com',
 'kulinarnyepraktiki-ru.turbopages.org': 'kulinarnyepraktiki.ru',
 'gruppman-livejournal-com.turbopages.org': 'gruppman.livejournal.com',
 'budidich-livejournal-com.turbopages.org': 'budidich.livejournal.com',
 'ilmusa-livejournal-com.turbopages.org': 'ilmusa.livejournal.com',
 'm--cardinal-livejournal-com.turbopages.org': 'm-cardinal.livejournal.com',
 'hcdinamo-by.turbopages.org': 'hcdinamo.by',
 'worldofwomen-club.turbopages.org': 'worldofwomen.club',
 'potsreotizm--new-livejournal-com.turbopages.org': 'potsreotizm-new.livejournal.com',
 'asteryaa-livejournal-com.turbopages.org': 'asteryaa.livejournal.com',
 'ploughlike--elk-livejournal-com.turbopages.org': 'ploughlike-elk.livejournal.com',
 'bala--bamba-livejournal-com.turbopages.org': 'bala-bamba.livejournal.com',
 'garkushev-livejournal-com.turbopages.org': 'garkushev.livejournal.com',
 'roman--i--darija-livejournal-com.turbopages.org': 'roman-i-darija.livejournal.com',
 'nowaste-ru.turbopages.org': 'nowaste.ru',
 'the--paper--plane-livejournal-com.turbopages.org': 'the-paper-plane.livejournal.com',
 'lana--prini-livejournal-com.turbopages.org': 'lana-prini.livejournal.com',
 'washerman-livejournal-com.turbopages.org': 'washerman.livejournal.com',
 'vobche-livejournal-com.turbopages.org': 'vobche.livejournal.com',
 'ingoodfit-ru.turbopages.org': 'ingoodfit.ru',
 'dolo--res-livejournal-com.turbopages.org': 'dolo-res.livejournal.com',
 'colormann-ru.turbopages.org': 'colormann.ru',
 'mioxa-livejournal-com.turbopages.org': 'mioxa.livejournal.com',
 'morseg-livejournal-com.turbopages.org': 'morseg.livejournal.com',
 'vasylysa78-livejournal-com.turbopages.org': 'vasylysa78.livejournal.com',
 'minus--procent-ru.turbopages.org': 'minus-procent.ru',
 'hyundaistarex-livejournal-com.turbopages.org': 'hyundaistarex.livejournal.com',
 'dzerginsk-bezformata-com.turbopages.org': 'dzerginsk.bezformata.com',
 'a0-livejournal-com.turbopages.org': 'a0.livejournal.com',
 'p--i--l--a--r-livejournal-com.turbopages.org': 'p-i-l-a-r.livejournal.com',
 'a--college-ru.turbopages.org': 'a-college.ru',
 'galkriv-livejournal-com.turbopages.org': 'galkriv.livejournal.com',
 'titanic15--04-livejournal-com.turbopages.org': 'titanic15-04.livejournal.com',
 'o--l--g--a--r--i-livejournal-com.turbopages.org': 'o-l-g-a-r-i.livejournal.com',
 'ivlae-livejournal-com.turbopages.org': 'ivlae.livejournal.com',
 'kingowatch-livejournal-com.turbopages.org': 'kingowatch.livejournal.com',
 'is--pain-livejournal-com.turbopages.org': 'is-pain.livejournal.com',
 'qsuxa--terkin-livejournal-com.turbopages.org': 'qsuxa-terkin.livejournal.com',
 'dr--jamais-livejournal-com.turbopages.org': 'dr-jamais.livejournal.com',
 'allfest-ru.turbopages.org': 'allfest.ru',
 'hurgada-in.turbopages.org': 'hurgada.in',
 'morozov66-livejournal-com.turbopages.org': 'morozov66.livejournal.com',
 'tuse4kakorzueva-livejournal-com.turbopages.org': 'tuse4kakorzueva.livejournal.com',
 'redwomen-livejournal-com.turbopages.org': 'redwomen.livejournal.com',
 'prosahar-livejournal-com.turbopages.org': 'prosahar.livejournal.com',
 'flashdecor-livejournal-com.turbopages.org': 'flashdecor.livejournal.com',
 'dachamaster-org.turbopages.org': 'dachamaster.org',
 'petr--kosyh-livejournal-com.turbopages.org': 'petr-kosyh.livejournal.com',
 'fish--hunter-livejournal-com.turbopages.org': 'fish-hunter.livejournal.com',
 'klimoff--den-livejournal-com.turbopages.org': 'klimoff-den.livejournal.com',
 'ege--rus-livejournal-com.turbopages.org': 'ege-rus.livejournal.com',
 'chekhova--olga-livejournal-com.turbopages.org': 'chekhova-olga.livejournal.com',
 'stroimzanovo-ru.turbopages.org': 'stroimzanovo.ru',
 'consolier-livejournal-com.turbopages.org': 'consolier.livejournal.com',
 'igorpodgorny-livejournal-com.turbopages.org': 'igorpodgorny.livejournal.com',
 'fir--vst-livejournal-com.turbopages.org': 'fir-vst.livejournal.com',
 'world--cuisine-livejournal-com.turbopages.org': 'world-cuisine.livejournal.com',
 'prokuhnya-com.turbopages.org': 'prokuhnya.com',
 'irilllonok-livejournal-com.turbopages.org': 'irilllonok.livejournal.com',
 'aboutcccp-livejournal-com.turbopages.org': 'aboutcccp.livejournal.com',
 'kuzmenov-ru.turbopages.org': 'kuzmenov.ru',
 'xn-----com-xof3aak1cl9b4b.turbopages.org': 'xn--.com.xof3aak1cl9b4b',
 'mypitbike-livejournal-com.turbopages.org': 'mypitbike.livejournal.com',
 'sasha--lotus-livejournal-com.turbopages.org': 'sasha-lotus.livejournal.com',
 'norma--migracia-ru.turbopages.org': 'norma-migracia.ru',
 'ru--photoshop-livejournal-com.turbopages.org': 'ru-photoshop.livejournal.com',
 'oboleznjah-ru.turbopages.org': 'oboleznjah.ru',
 'amfeiya-livejournal-com.turbopages.org': 'amfeiya.livejournal.com',
 'zashitimkorolev-livejournal-com.turbopages.org': 'zashitimkorolev.livejournal.com',
 'krasotkin--m-livejournal-com.turbopages.org': 'krasotkin-m.livejournal.com',
 'travelunlimited-ru.turbopages.org': 'travelunlimited.ru',
 'cerkved-livejournal-com.turbopages.org': 'cerkved.livejournal.com',
 'zed007-livejournal-com.turbopages.org': 'zed007.livejournal.com',
 'n--job-ru.turbopages.org': 'n-job.ru',
 'nicecook-ru.turbopages.org': 'nicecook.ru',
 'vassilio-livejournal-com.turbopages.org': 'vassilio.livejournal.com',
 'diflyon-livejournal-com.turbopages.org': 'diflyon.livejournal.com',
 'james--valone-livejournal-com.turbopages.org': 'james-valone.livejournal.com',
 'olt--z--s-livejournal-com.turbopages.org': 'olt-z-s.livejournal.com',
 'romapetrov-livejournal-com.turbopages.org': 'romapetrov.livejournal.com',
 'bey-livejournal-com.turbopages.org': 'bey.livejournal.com',
 'vasilalena-livejournal-com.turbopages.org': 'vasilalena.livejournal.com',
 'premier--sport-ru.turbopages.org': 'premier-sport.ru',
 'vadimus58-livejournal-com.turbopages.org': 'vadimus58.livejournal.com',
 'peter--kiele-livejournal-com.turbopages.org': 'peter-kiele.livejournal.com',
 'cherkessk-su.turbopages.org': 'cherkessk.su',
 'kamienec-livejournal-com.turbopages.org': 'kamienec.livejournal.com',
 'lota--live-livejournal-com.turbopages.org': 'lota-live.livejournal.com',
 'krasnodar-livejournal-com.turbopages.org': 'krasnodar.livejournal.com',
 'alivespace-ru.turbopages.org': 'alivespace.ru',
 'vitebsk-cc.turbopages.org': 'vitebsk.cc',
 '1001sposob-ru.turbopages.org': '1001sposob.ru',
 'bogdana--00-livejournal-com.turbopages.org': 'bogdana-00.livejournal.com',
 'vad--nes-livejournal-com.turbopages.org': 'vad-nes.livejournal.com',
 'morethantravel-livejournal-com.turbopages.org': 'morethantravel.livejournal.com',
 'dimkagrigoryev-livejournal-com.turbopages.org': 'dimkagrigoryev.livejournal.com',
 'ne--pokidai-livejournal-com.turbopages.org': 'ne-pokidai.livejournal.com',
 'dr--watsn-livejournal-com.turbopages.org': 'dr-watsn.livejournal.com',
 'strannik1990-livejournal-com.turbopages.org': 'strannik1990.livejournal.com',
 'deepshades85-livejournal-com.turbopages.org': 'deepshades85.livejournal.com',
 '2011dnevnoe-livejournal-com.turbopages.org': '2011dnevnoe.livejournal.com',
 'detskie--knigi-livejournal-com.turbopages.org': 'detskie-knigi.livejournal.com',
 'malferov-livejournal-com.turbopages.org': 'malferov.livejournal.com',
 'vkusnij--blog-ru.turbopages.org': 'vkusnij-blog.ru',
 'elven--tankmen-livejournal-com.turbopages.org': 'elven-tankmen.livejournal.com',
 'itb-spb-ru.turbopages.org': 'itb.spb.ru',
 'ymorno--ru-livejournal-com.turbopages.org': 'ymorno-ru.livejournal.com',
 'cpp2010-livejournal-com.turbopages.org': 'cpp2010.livejournal.com',
 'edgarin-livejournal-com.turbopages.org': 'edgarin.livejournal.com',
 'la--finezza-livejournal-com.turbopages.org': 'la-finezza.livejournal.com',
    'alexeyshornikov-livejournal-com.turbopages.org': 'alexeyshornikov.livejournal.com',
 'brat--luber-livejournal-com.turbopages.org': 'brat-luber.livejournal.com',
 'prom1-livejournal-com.turbopages.org': 'prom1.livejournal.com',
 'felix--edmund-livejournal-com.turbopages.org': 'felix-edmund.livejournal.com',
 'facilitatorman-livejournal-com.turbopages.org': 'facilitatorman.livejournal.com',
 'yegorka-livejournal-com.turbopages.org': 'yegorka.livejournal.com',
 'evgenij565-livejournal-com.turbopages.org': 'evgenij565.livejournal.com',
 'tanya--mass-livejournal-com.turbopages.org': 'tanya-mass.livejournal.com',
 'gunhatemachine-livejournal-com.turbopages.org': 'gunhatemachine.livejournal.com',
 'alex--dbc-livejournal-com.turbopages.org': 'alex-dbc.livejournal.com',
 'history--ru-livejournal-com.turbopages.org': 'history-ru.livejournal.com',
 'aegusev-livejournal-com.turbopages.org': 'aegusev.livejournal.com',
 'technolirik-livejournal-com.turbopages.org': 'technolirik.livejournal.com',
 'materinskiy--semeyniy--kapital-ru.turbopages.org': 'materinskiy-semeyniy-kapital.ru',
 'sushchikh-livejournal-com.turbopages.org': 'sushchikh.livejournal.com',
 'kotiko23-livejournal-com.turbopages.org': 'kotiko23.livejournal.com',
 'ultimaguardian-livejournal-com.turbopages.org': 'ultimaguardian.livejournal.com',
 'mjspace-livejournal-com.turbopages.org': 'mjspace.livejournal.com',
 'prokuratura--ru-livejournal-com.turbopages.org': 'prokuratura-ru.livejournal.com',
 'nekropoli-livejournal-com.turbopages.org': 'nekropoli.livejournal.com',
 'my--twilight--sun-livejournal-com.turbopages.org': 'my-twilight-sun.livejournal.com',
 'svojak55-livejournal-com.turbopages.org': 'svojak55.livejournal.com',
 'matchgirl--ru-livejournal-com.turbopages.org': 'matchgirl-ru.livejournal.com',
 'oshtorah-ru.turbopages.org': 'oshtorah.ru',
 'valentin--sky-livejournal-com.turbopages.org': 'valentin-sky.livejournal.com',
 'sergeygrnsky-livejournal-com.turbopages.org': 'sergeygrnsky.livejournal.com',
 'dostoevschina-livejournal-com.turbopages.org': 'dostoevschina.livejournal.com',
 'dr--corner-livejournal-com.turbopages.org': 'dr-corner.livejournal.com',
 'na6ludatelb-livejournal-com.turbopages.org': 'na6ludatelb.livejournal.com',
 'vovm-livejournal-com.turbopages.org': 'vovm.livejournal.com',
 'istinata-livejournal-com.turbopages.org': 'istinata.livejournal.com',
 'sv--minskaya-livejournal-com.turbopages.org': 'sv-minskaya.livejournal.com',
 'vls--smolich-livejournal-com.turbopages.org': 'vls-smolich.livejournal.com',
 'crazy--zoologist-livejournal-com.turbopages.org': 'crazy-zoologist.livejournal.com',
 'uurist-livejournal-com.turbopages.org': 'uurist.livejournal.com',
 'parkhomovg-livejournal-com.turbopages.org': 'parkhomovg.livejournal.com',
 'anonymus--74-livejournal-com.turbopages.org': 'anonymus-74.livejournal.com',
 'morky-livejournal-com.turbopages.org': 'morky.livejournal.com',
 'chapleeng-livejournal-com.turbopages.org': 'chapleeng.livejournal.com',
 'miccro-livejournal-com.turbopages.org': 'miccro.livejournal.com',
 'radioparty-ru.turbopages.org': 'radioparty.ru',
 'petrovvf-livejournal-com.turbopages.org': 'petrovvf.livejournal.com',
 'dizzokoo-livejournal-com.turbopages.org': 'dizzokoo.livejournal.com',
 'zakon-kz.turbopages.org': 'zakon.kz',
 'lifecoolinfo-ru.turbopages.org': 'lifecoolinfo.ru',
 'lifeservice-me.turbopages.org': 'lifeservice.me',
 'm--sch-livejournal-com.turbopages.org': 'm-sch.livejournal.com',
 'shiha--shiha-livejournal-com.turbopages.org': 'shiha-shiha.livejournal.com',
 'gobeats-ru.turbopages.org': 'gobeats.ru',
 'enasiwa-livejournal-com.turbopages.org': 'enasiwa.livejournal.com',
 'realistki-livejournal-com.turbopages.org': 'realistki.livejournal.com',
 'roman--carman-livejournal-com.turbopages.org': 'roman-carman.livejournal.com',
 'kupite--vorota-ru.turbopages.org': 'kupite-vorota.ru',
 'afportal-ru.turbopages.org': 'afportal.ru',
 'kardioportal-ru.turbopages.org': 'kardioportal.ru',
 'kontakt--karate-ru.turbopages.org': 'kontakt-karate.ru',
 '09lorry-livejournal-com.turbopages.org': '09lorry.livejournal.com',
 'kirlish-livejournal-com.turbopages.org': 'kirlish.livejournal.com',
 'kremlin--pu-livejournal-com.turbopages.org': 'kremlin-pu.livejournal.com',
 'volgograd--kredits-ru.turbopages.org': 'volgograd-kredits.ru',
 'beliy--tigr-livejournal-com.turbopages.org': 'beliy-tigr.livejournal.com',
 'ananda--sh-livejournal-com.turbopages.org': 'ananda-sh.livejournal.com',
 'lique--bez-livejournal-com.turbopages.org': 'lique-bez.livejournal.com',
 'knebeckaize-livejournal-com.turbopages.org': 'knebeckaize.livejournal.com',
 'koshek-ru.turbopages.org': 'koshek.ru',
 'plus--minuses-livejournal-com.turbopages.org': 'plus-minuses.livejournal.com',
 'shaverblade-ru.turbopages.org': 'shaverblade.ru',
 'katsaylidiru-livejournal-com.turbopages.org': 'katsaylidiru.livejournal.com',
 'ru--bezh-ru.turbopages.org': 'ru-bezh.ru',
 'lirina-livejournal-com.turbopages.org': 'lirina.livejournal.com',
 'valera061147-livejournal-com.turbopages.org': 'valera061147.livejournal.com',
 'orchidfan-ru.turbopages.org': 'orchidfan.ru',
 'tutskill-ru.turbopages.org': 'tutskill.ru',
 'superulov-com.turbopages.org': 'superulov.com',
 'anaxios-livejournal-com.turbopages.org': 'anaxios.livejournal.com',
 'kompas--m-livejournal-com.turbopages.org': 'kompas-m.livejournal.com',
 'ru--prokhanov-livejournal-com.turbopages.org': 'ru-prokhanov.livejournal.com',
 'sontucio-livejournal-com.turbopages.org': 'sontucio.livejournal.com',
 'kondrovo-bezformata-com.turbopages.org': 'kondrovo.bezformata.com',
 'sige--vic-livejournal-com.turbopages.org': 'sige-vic.livejournal.com',
 'jukovskiy-bezformata-com.turbopages.org': 'jukovskiy.bezformata.com',
 'mozgotron-livejournal-com.turbopages.org': 'mozgotron.livejournal.com',
 'kavolekat-livejournal-com.turbopages.org': 'kavolekat.livejournal.com',
 'mechta--ind--posh-livejournal-com.turbopages.org': 'mechta-ind-posh.livejournal.com',
 'toro35-livejournal-com.turbopages.org': 'toro35.livejournal.com',
 'dm--lihachev-livejournal-com.turbopages.org': 'dm-lihachev.livejournal.com',
 'geografishka-ru.turbopages.org': 'geografishka.ru',
 'mevamevo-livejournal-com.turbopages.org': 'mevamevo.livejournal.com',
 'klarissa45-livejournal-com.turbopages.org': 'klarissa45.livejournal.com',
 'slobod-ru.turbopages.org': 'slobod.ru',
 'splitpine-livejournal-com.turbopages.org': 'splitpine.livejournal.com',
 'romanas-livejournal-com.turbopages.org': 'romanas.livejournal.com',
 'maystre-livejournal-com.turbopages.org': 'maystre.livejournal.com',
 'heihoka-livejournal-com.turbopages.org': 'heihoka.livejournal.com',
 'znay--vce-livejournal-com.turbopages.org': 'znay-vce.livejournal.com',
 'pechora-bezformata-com.turbopages.org': 'pechora.bezformata.com',
 'al--firsov-livejournal-com.turbopages.org': 'al-firsov.livejournal.com',
 'zabyg17-livejournal-com.turbopages.org': 'zabyg17.livejournal.com',
 'foto--programmer-livejournal-com.turbopages.org': 'foto-programmer.livejournal.com',
 'rebel--rouge-livejournal-com.turbopages.org': 'rebel-rouge.livejournal.com',
 'evgris-livejournal-com.turbopages.org': 'evgris.livejournal.com',
 'masterotoplenie-ru.turbopages.org': 'masterotoplenie.ru',
 'pro--packaging-livejournal-com.turbopages.org': 'pro-packaging.livejournal.com',
 'toira-ru.turbopages.org': 'toira.ru',
 'stanislavgai-livejournal-com.turbopages.org': 'stanislavgai.livejournal.com',
 'xodorovsky-livejournal-com.turbopages.org': 'xodorovsky.livejournal.com',
 'humandesignlive-livejournal-com.turbopages.org': 'humandesignlive.livejournal.com',
 'bel--il-livejournal-com.turbopages.org': 'bel-il.livejournal.com',
 'iozhinsbajin-livejournal-com.turbopages.org': 'iozhinsbajin.livejournal.com',
 'seo--assistant-ru.turbopages.org': 'seo-assistant.ru',
 'sergeyvelo-livejournal-com.turbopages.org': 'sergeyvelo.livejournal.com',
 'dagliese581g-livejournal-com.turbopages.org': 'dagliese581g.livejournal.com',
 'dvernaya--moda-livejournal-com.turbopages.org': 'dvernaya-moda.livejournal.com',
 'borisoglebsk-riavrn-ru.turbopages.org': 'borisoglebsk.riavrn.ru',
 'kompromat--kuz-livejournal-com.turbopages.org': 'kompromat-kuz.livejournal.com',
 'belgorod-big--book--med-ru.turbopages.org': 'belgorod.big-book-med.ru',
 'proriv--gbi-ru.turbopages.org': 'proriv-gbi.ru',
 'invis--li-livejournal-com.turbopages.org': 'invis-li.livejournal.com',
 'rjadovoj--rus-livejournal-com.turbopages.org': 'rjadovoj-rus.livejournal.com',
 'sarycheva--s-livejournal-com.turbopages.org': 'sarycheva-s.livejournal.com',
 'grey--ultra-livejournal-com.turbopages.org': 'grey-ultra.livejournal.com',
 'antonsafronov-livejournal-com.turbopages.org': 'antonsafronov.livejournal.com',
 'top--mmorpg-ru.turbopages.org': 'top-mmorpg.ru',
 'dshalikov-livejournal-com.turbopages.org': 'dshalikov.livejournal.com',
 'domizi-ru.turbopages.org': 'domizi.ru',
 'souztrade-ru.turbopages.org': 'souztrade.ru',
 'vyazhy-ru.turbopages.org': 'vyazhy.ru',
 'znajiumej-ru.turbopages.org': 'znajiumej.ru',
 'qr--gorod-livejournal-com.turbopages.org': 'qr-gorod.livejournal.com',
 'omnesolum-livejournal-com.turbopages.org': 'omnesolum.livejournal.com',
 'seg--o-livejournal-com.turbopages.org': 'seg-o.livejournal.com',
 'ru--vegetarian-livejournal-com.turbopages.org': 'ru-vegetarian.livejournal.com',
 'animalsfriendly-livejournal-com.turbopages.org': 'animalsfriendly.livejournal.com',
 'vm--dacha-ru.turbopages.org': 'vm-dacha.ru',
 'erandl-livejournal-com.turbopages.org': 'erandl.livejournal.com',
 'system--ok-livejournal-com.turbopages.org': 'system-ok.livejournal.com',
 'dabat--3-livejournal-com.turbopages.org': 'dabat-3.livejournal.com',
 'doctor--stoletov-livejournal-com.turbopages.org': 'doctor-stoletov.livejournal.com',
 'enekrasova-livejournal-com.turbopages.org': 'enekrasova.livejournal.com',
 'andreysemenov-livejournal-com.turbopages.org': 'andreysemenov.livejournal.com',
 'yakimovmihail-livejournal-com.turbopages.org': 'yakimovmihail.livejournal.com',
 'np--ogdps-livejournal-com.turbopages.org': 'np-ogdps.livejournal.com',
 'one--in-livejournal-com.turbopages.org': 'one-in.livejournal.com',
 'gotovte--s--udovolstvyem-ru.turbopages.org': 'gotovte-s-udovolstvyem.ru',
 'vegastalker-livejournal-com.turbopages.org': 'vegastalker.livejournal.com',
 'ohotnik1975-livejournal-com.turbopages.org': 'ohotnik1975.livejournal.com',
 'mariro-livejournal-com.turbopages.org': 'mariro.livejournal.com',
 'oetar-livejournal-com.turbopages.org': 'oetar.livejournal.com',
 'goanoid-livejournal-com.turbopages.org': 'goanoid.livejournal.com',
 'grozny--ivan-livejournal-com.turbopages.org': 'grozny-ivan.livejournal.com',
 'blogirina-livejournal-com.turbopages.org': 'blogirina.livejournal.com',
 'zheniac-livejournal-com.turbopages.org': 'zheniac.livejournal.com',
 'lana--sv-livejournal-com.turbopages.org': 'lana-sv.livejournal.com',
 'olenenok--ovi-livejournal-com.turbopages.org': 'olenenok-ovi.livejournal.com',
 'umoristy-livejournal-com.turbopages.org': 'umoristy.livejournal.com',
 'vfigure-ru.turbopages.org': 'vfigure.ru',
 'svetdio-ru.turbopages.org': 'svetdio.ru',
 'nelidova--ng-livejournal-com.turbopages.org': 'nelidova-ng.livejournal.com',
 'lediglamur-ru.turbopages.org': 'lediglamur.ru',
 'olhimiya-livejournal-com.turbopages.org': 'olhimiya.livejournal.com',
 'torin--kr-livejournal-com.turbopages.org': 'torin-kr.livejournal.com',
 'klyaksina-livejournal-com.turbopages.org': 'klyaksina.livejournal.com',
 'sazhin64-livejournal-com.turbopages.org': 'sazhin64.livejournal.com',
 'letohin-livejournal-com.turbopages.org': 'letohin.livejournal.com',
 'sergeylarin-livejournal-com.turbopages.org': 'sergeylarin.livejournal.com',
 'doctor--tresh-livejournal-com.turbopages.org': 'doctor-tresh.livejournal.com',
 'holistic--way-livejournal-com.turbopages.org': 'holistic-way.livejournal.com',
 'purgine-livejournal-com.turbopages.org': 'purgine.livejournal.com',
 'surrealnay-livejournal-com.turbopages.org': 'surrealnay.livejournal.com',
 'dostavkaprom-ru.turbopages.org': 'dostavkaprom.ru',
 'detskij--ostrovok-ru.turbopages.org': 'detskij-ostrovok.ru',
 '0015auto-livejournal-com.turbopages.org': '0015auto.livejournal.com',
 'myearphones-ru.turbopages.org': 'myearphones.ru',
 'tamarapravdina-livejournal-com.turbopages.org': 'tamarapravdina.livejournal.com',
 'troe--vmeste-livejournal-com.turbopages.org': 'troe-vmeste.livejournal.com',
 'medglav-com.turbopages.org': 'medglav.com',
 'wetravelers-ru.turbopages.org': 'wetravelers.ru',
 'ru--bmo-livejournal-com.turbopages.org': 'ru-bmo.livejournal.com',
 'atroshchenko-ru.turbopages.org': 'atroshchenko.ru',
 'ekb-dixinews-ru.turbopages.org': 'ekb.dixinews.ru',
 'itshair-livejournal-com.turbopages.org': 'itshair.livejournal.com',
 'daemon--simplex-livejournal-com.turbopages.org': 'daemon-simplex.livejournal.com',
 'sky--corsair-livejournal-com.turbopages.org': 'sky-corsair.livejournal.com',
 'sverhrazum-livejournal-com.turbopages.org': 'sverhrazum.livejournal.com',
 'progmatikus-livejournal-com.turbopages.org': 'progmatikus.livejournal.com',
 'kirill--kaluga-livejournal-com.turbopages.org': 'kirill-kaluga.livejournal.com',
 'vkusnaykuxny-ru.turbopages.org': 'vkusnaykuxny.ru',
 'ekooborona-livejournal-com.turbopages.org': 'ekooborona.livejournal.com',
 'myfamilyblog-ru.turbopages.org': 'myfamilyblog.ru',
 'kolbasin-livejournal-com.turbopages.org': 'kolbasin.livejournal.com',
 'atroxx-livejournal-com.turbopages.org': 'atroxx.livejournal.com',
 'economlegko-ru.turbopages.org': 'economlegko.ru',
 'ruforces-livejournal-com.turbopages.org': 'ruforces.livejournal.com',
 'bibaliy17-livejournal-com.turbopages.org': 'bibaliy17.livejournal.com',
 'profi--biz-ru.turbopages.org': 'profi-biz.ru',
 'olegchagin-livejournal-com.turbopages.org': 'olegchagin.livejournal.com',
 'ksoftware-livejournal-com.turbopages.org': 'ksoftware.livejournal.com',
 'a4vlad-ru.turbopages.org': 'a4vlad.ru',
 'golitsina--n-livejournal-com.turbopages.org': 'golitsina-n.livejournal.com',
 'sevenevildevils-livejournal-com.turbopages.org': 'sevenevildevils.livejournal.com',
 'mister--pleskov-livejournal-com.turbopages.org': 'mister-pleskov.livejournal.com',
 'bescker-livejournal-com.turbopages.org': 'bescker.livejournal.com',
 'pap--a-livejournal-com.turbopages.org': 'pap-a.livejournal.com',
 'kaptbl-livejournal-com.turbopages.org': 'kaptbl.livejournal.com',
 'alex--new--york-livejournal-com.turbopages.org': 'alex-new-york.livejournal.com',
 'podmoskovye-livejournal-com.turbopages.org': 'podmoskovye.livejournal.com',
 'office--gemotest-ru.turbopages.org': 'office-gemotest.ru',
 'three--passions-livejournal-com.turbopages.org': 'three-passions.livejournal.com',
 'trubchyk-livejournal-com.turbopages.org': 'trubchyk.livejournal.com',
 'hotzed-livejournal-com.turbopages.org': 'hotzed.livejournal.com',
 'serge--er-livejournal-com.turbopages.org': 'serge-er.livejournal.com',
 'tcsbank-livejournal-com.turbopages.org': 'tcsbank.livejournal.com',
 'vukol-ru.turbopages.org': 'vukol.ru',
 'siddgames-ru.turbopages.org': 'siddgames.ru',
 'delllice-livejournal-com.turbopages.org': 'delllice.livejournal.com',
 'bulgakovatodays-livejournal-com.turbopages.org': 'bulgakovatodays.livejournal.com',
 'aleks--melnikov-livejournal-com.turbopages.org': 'aleks-melnikov.livejournal.com',
 'rby--foto-livejournal-com.turbopages.org': 'rby-foto.livejournal.com',
 'wot--invite-ru.turbopages.org': 'wot-invite.ru',
 'haimashkir-livejournal-com.turbopages.org': 'haimashkir.livejournal.com',
 'neryungri-bezformata-com.turbopages.org': 'neryungri.bezformata.com',
 'alifon-livejournal-com.turbopages.org': 'alifon.livejournal.com',
 'nyam--nyamki-livejournal-com.turbopages.org': 'nyam-nyamki.livejournal.com',
 'pliskina-livejournal-com.turbopages.org': 'pliskina.livejournal.com',
 'gayaz--samigulov-livejournal-com.turbopages.org': 'gayaz-samigulov.livejournal.com',
 'alexcrim-livejournal-com.turbopages.org': 'alexcrim.livejournal.com',
 'avtor--vm-ru.turbopages.org': 'avtor-vm.ru',
 'domass-ru.turbopages.org': 'domass.ru',
 'socpravo-ru.turbopages.org': 'socpravo.ru',
 'naff--off-livejournal-com.turbopages.org': 'naff-off.livejournal.com',
 'negribok-ru.turbopages.org': 'negribok.ru',
 'annagerrets-livejournal-com.turbopages.org': 'annagerrets.livejournal.com',
 'folkremedy-ru.turbopages.org': 'folkremedy.ru',
 'drova--pil-ru.turbopages.org': 'drova-pil.ru',
 'vilvetta-livejournal-com.turbopages.org': 'vilvetta.livejournal.com',
 'marossiya--ya-livejournal-com.turbopages.org': 'marossiya-ya.livejournal.com',
 'lacr1ma-livejournal-com.turbopages.org': 'lacr1ma.livejournal.com',
 'e--vikyra-livejournal-com.turbopages.org': 'e-vikyra.livejournal.com',
 'salise-livejournal-com.turbopages.org': 'salise.livejournal.com',
 'volma-livejournal-com.turbopages.org': 'volma.livejournal.com',
 'ddom47-ru.turbopages.org': 'ddom47.ru',
 'liteinoeproizvodstvo-ru.turbopages.org': 'liteinoeproizvodstvo.ru',
 'm--3713-livejournal-com.turbopages.org': 'm-3713.livejournal.com',
 'nevzdrasmion-livejournal-com.turbopages.org': 'nevzdrasmion.livejournal.com',
 'igor--levit-livejournal-com.turbopages.org': 'igor-levit.livejournal.com',
 'vika--pekarskaya-livejournal-com.turbopages.org': 'vika-pekarskaya.livejournal.com',
 'tekstus-livejournal-com.turbopages.org': 'tekstus.livejournal.com',
 'lestnica-guru.turbopages.org': 'lestnica.guru',
 'st--konstantin1-livejournal-com.turbopages.org': 'st-konstantin1.livejournal.com',
 'evizatanet-livejournal-com.turbopages.org': 'evizatanet.livejournal.com',
 'prostovita-ru.turbopages.org': 'prostovita.ru',
 'hhott--pepper-livejournal-com.turbopages.org': 'hhott-pepper.livejournal.com',
 'vonnaz-livejournal-com.turbopages.org': 'vonnaz.livejournal.com',
 'kaidzin-livejournal-com.turbopages.org': 'kaidzin.livejournal.com',
 'igor--salnikov-livejournal-com.turbopages.org': 'igor-salnikov.livejournal.com',
 'beinten-ru.turbopages.org': 'beinten.ru',
 'junkyard-livejournal-com.turbopages.org': 'junkyard.livejournal.com',
 'monokio-livejournal-com.turbopages.org': 'monokio.livejournal.com',
 'history--foto-livejournal-com.turbopages.org': 'history-foto.livejournal.com',
 'sberbank-bz.turbopages.org': 'sberbank.bz',
 'postroju--dom-ru.turbopages.org': 'postroju-dom.ru',
 'gubarevan-livejournal-com.turbopages.org': 'gubarevan.livejournal.com',
 'ru--uzd-livejournal-com.turbopages.org': 'ru-uzd.livejournal.com',
 'nskfamily-livejournal-com.turbopages.org': 'nskfamily.livejournal.com',
 'nalogi--cons-ru.turbopages.org': 'nalogi-cons.ru',
 'trueview-livejournal-com.turbopages.org': 'trueview.livejournal.com',
 'soviet--books-livejournal-com.turbopages.org': 'soviet-books.livejournal.com',
 'tve24-ru.turbopages.org': 'tve24.ru',
 'grace888-livejournal-com.turbopages.org': 'grace888.livejournal.com',
 'i--korotchenko-livejournal-com.turbopages.org': 'i-korotchenko.livejournal.com',
 'elka55-livejournal-com.turbopages.org': 'elka55.livejournal.com',
 'magicshop--in--ua-livejournal-com.turbopages.org': 'magicshop-in-ua.livejournal.com',
 'cool--shina-ru.turbopages.org': 'cool-shina.ru',
 'yuri--kosagovsky-livejournal-com.turbopages.org': 'yuri-kosagovsky.livejournal.com',
 'grafic747-livejournal-com.turbopages.org': 'grafic747.livejournal.com',
 'nira--tae-livejournal-com.turbopages.org': 'nira-tae.livejournal.com',
 'tproekt-com.turbopages.org': 'tproekt.com',
 'major--kalter-livejournal-com.turbopages.org': 'major-kalter.livejournal.com',
 'doroshenko--us-livejournal-com.turbopages.org': 'doroshenko-us.livejournal.com',
 'nashiptichki-ru.turbopages.org': 'nashiptichki.ru',
 'bigphils-livejournal-com.turbopages.org': 'bigphils.livejournal.com',
 'mirvn-livejournal-com.turbopages.org': 'mirvn.livejournal.com',
 'kbberkut-ru.turbopages.org': 'kbberkut.ru',
 'vvv--ig-livejournal-com.turbopages.org': 'vvv-ig.livejournal.com',
 '220v-guru.turbopages.org': '220v.guru',
 'mmv13-livejournal-com.turbopages.org': 'mmv13.livejournal.com',
 'fish54-ru.turbopages.org': 'fish54.ru',
 'troiskmsk-bezformata-com.turbopages.org': 'troiskmsk.bezformata.com',
 'planeta--zemly-ru.turbopages.org': 'planeta-zemly.ru',
 'plan--pu-livejournal-com.turbopages.org': 'plan-pu.livejournal.com',
 'verydveri-ru.turbopages.org': 'verydveri.ru',
 'ru--travelers-livejournal-com.turbopages.org': 'ru-travelers.livejournal.com',
 'goodspb-livejournal-com.turbopages.org': 'goodspb.livejournal.com',
 'oldden-livejournal-com.turbopages.org': 'oldden.livejournal.com',
 'russianinterest-livejournal-com.turbopages.org': 'russianinterest.livejournal.com',
 'igdud-livejournal-com.turbopages.org': 'igdud.livejournal.com',
 'subbotin-digital.turbopages.org': 'subbotin.digital',
 'yaromir191-livejournal-com.turbopages.org': 'yaromir191.livejournal.com',
 'daryssimo-livejournal-com.turbopages.org': 'daryssimo.livejournal.com',
 'writing365-livejournal-com.turbopages.org': 'writing365.livejournal.com',
 'melanyja-livejournal-com.turbopages.org': 'melanyja.livejournal.com',
 'vivikitoria-livejournal-com.turbopages.org': 'vivikitoria.livejournal.com',
 'booqua-livejournal-com.turbopages.org': 'booqua.livejournal.com',
 'yatsanav-livejournal-com.turbopages.org': 'yatsanav.livejournal.com',
 'eimage-livejournal-com.turbopages.org': 'eimage.livejournal.com',
 'desssa-livejournal-com.turbopages.org': 'desssa.livejournal.com',
 'innachumichkina-livejournal-com.turbopages.org': 'innachumichkina.livejournal.com',
 'emper1or-livejournal-com.turbopages.org': 'emper1or.livejournal.com',
 'svmensh-livejournal-com.turbopages.org': 'svmensh.livejournal.com',
 'drwats-livejournal-com.turbopages.org': 'drwats.livejournal.com',
 'certificatione-ru.turbopages.org': 'certificatione.ru',
 'dr--van--mogg-livejournal-com.turbopages.org': 'dr-van-mogg.livejournal.com',
 'ulejpchel-ru.turbopages.org': 'ulejpchel.ru',
 'stariyoskol-bezformata-com.turbopages.org': 'stariyoskol.bezformata.com',
 'china--shore-livejournal-com.turbopages.org': 'china-shore.livejournal.com',
 'swg54-livejournal-com.turbopages.org': 'swg54.livejournal.com',
 'tricsterr-livejournal-com.turbopages.org': 'tricsterr.livejournal.com',
 'zelenyikot-livejournal-com.turbopages.org': 'zelenyikot.livejournal.com',
 'myfavoritepets-ru.turbopages.org': 'myfavoritepets.ru',
 'replet-ru.turbopages.org': 'replet.ru',
 'tsiganash-livejournal-com.turbopages.org': 'tsiganash.livejournal.com',
 'repovesi-livejournal-com.turbopages.org': 'repovesi.livejournal.com',
 'iqmena-livejournal-com.turbopages.org': 'iqmena.livejournal.com',
 'poembook-ru.turbopages.org': 'poembook.ru',
 'colour--rain-livejournal-com.turbopages.org': 'colour-rain.livejournal.com',
 'architectguide-ru.turbopages.org': 'architectguide.ru',
 'hunter007-livejournal-com.turbopages.org': 'hunter007.livejournal.com',
 'tomat-guru.turbopages.org': 'tomat.guru',
 'uralfish-ru.turbopages.org': 'uralfish.ru',
 'prospinning-ru.turbopages.org': 'prospinning.ru',
 'surgut-bezformata-com.turbopages.org': 'surgut.bezformata.com',
 'vitaviktoria-livejournal-com.turbopages.org': 'vitaviktoria.livejournal.com',
 'izbook-livejournal-com.turbopages.org': 'izbook.livejournal.com',
 'vestic-livejournal-com.turbopages.org': 'vestic.livejournal.com',
 'kvarkus-livejournal-com.turbopages.org': 'kvarkus.livejournal.com',
 'ludosopher-livejournal-com.turbopages.org': 'ludosopher.livejournal.com',
 'physiotherapevt-livejournal-com.turbopages.org': 'physiotherapevt.livejournal.com',
 'vartonx-livejournal-com.turbopages.org': 'vartonx.livejournal.com',
 'ls--gramota-livejournal-com.turbopages.org': 'ls-gramota.livejournal.com',
 'polesh--chuk-livejournal-com.turbopages.org': 'polesh-chuk.livejournal.com',
 'vorcuta-bezformata-com.turbopages.org': 'vorcuta.bezformata.com',
 'conncept-ru.turbopages.org': 'conncept.ru',
 'chuch007-livejournal-com.turbopages.org': 'chuch007.livejournal.com',
 'sago--cuisine-livejournal-com.turbopages.org': 'sago-cuisine.livejournal.com',
 'smile--ontheroad-livejournal-com.turbopages.org': 'smile-ontheroad.livejournal.com',
 'fuchik2-livejournal-com.turbopages.org': 'fuchik2.livejournal.com',
 'viktor--ch-livejournal-com.turbopages.org': 'viktor-ch.livejournal.com',
 'tanya--morozova-livejournal-com.turbopages.org': 'tanya-morozova.livejournal.com',
 'dunadana-livejournal-com.turbopages.org': 'dunadana.livejournal.com',
 'pentad-ru.turbopages.org': 'pentad.ru',
 'loyal--fr-livejournal-com.turbopages.org': 'loyal-fr.livejournal.com',
 'rabota--polska-ru.turbopages.org': 'rabota-polska.ru',
 'kulimania-livejournal-com.turbopages.org': 'kulimania.livejournal.com',
 'ns--resistance-livejournal-com.turbopages.org': 'ns-resistance.livejournal.com',
 'adam--a--nt-livejournal-com.turbopages.org': 'adam-a-nt.livejournal.com',
 'ribakov-livejournal-com.turbopages.org': 'ribakov.livejournal.com',
 'kamyshin--volga-livejournal-com.turbopages.org': 'kamyshin-volga.livejournal.com',
 'pat--info--kam-livejournal-com.turbopages.org': 'pat-info-kam.livejournal.com',
 'tim--taller15-livejournal-com.turbopages.org': 'tim-taller15.livejournal.com',
 'ketiiiiiiii-livejournal-com.turbopages.org': 'ketiiiiiiii.livejournal.com',
 'expert--prava-online.turbopages.org': 'expert-prava.online',
 '707-livejournal-com.turbopages.org': '707.livejournal.com',
 'karhu1977-livejournal-com.turbopages.org': 'karhu1977.livejournal.com',
 'solnzepodobny-livejournal-com.turbopages.org': 'solnzepodobny.livejournal.com',
 'gruzdov-livejournal-com.turbopages.org': 'gruzdov.livejournal.com',
 'sandinist-livejournal-com.turbopages.org': 'sandinist.livejournal.com',
 'tedeash-livejournal-com.turbopages.org': 'tedeash.livejournal.com',
 'rencus-livejournal-com.turbopages.org': 'rencus.livejournal.com',
 'krasnodar-big--book--med-ru.turbopages.org': 'krasnodar.big-book-med.ru',
 'mmeshkov-livejournal-com.turbopages.org': 'mmeshkov.livejournal.com',
 'drukin-livejournal-com.turbopages.org': 'drukin.livejournal.com',
 'fish--owner-livejournal-com.turbopages.org': 'fish-owner.livejournal.com',
 'tcperovo-ru.turbopages.org': 'tcperovo.ru',
 'galynash-livejournal-com.turbopages.org': 'galynash.livejournal.com',
 'rons--inform-livejournal-com.turbopages.org': 'rons-inform.livejournal.com',
 'alexandr--acov-livejournal-com.turbopages.org': 'alexandr-acov.livejournal.com',
 'zaloginen-livejournal-com.turbopages.org': 'zaloginen.livejournal.com',
 'yuribox-livejournal-com.turbopages.org': 'yuribox.livejournal.com',
 'lena--talaeva-livejournal-com.turbopages.org': 'lena-talaeva.livejournal.com',
 'chatlanin-livejournal-com.turbopages.org': 'chatlanin.livejournal.com',
 'seloved-ru.turbopages.org': 'seloved.ru',
 'mix14-livejournal-com.turbopages.org': 'mix14.livejournal.com',
 'altsarev-livejournal-com.turbopages.org': 'altsarev.livejournal.com',
 'kruglov-livejournal-com.turbopages.org': 'kruglov.livejournal.com',
 'ldpr-livejournal-com.turbopages.org': 'ldpr.livejournal.com',
 'gennadiydubovoy-livejournal-com.turbopages.org': 'gennadiydubovoy.livejournal.com',
 'stafford--k-livejournal-com.turbopages.org': 'stafford-k.livejournal.com',
 'vamoisej-livejournal-com.turbopages.org': 'vamoisej.livejournal.com',
 'avtoremontnikam-ru.turbopages.org': 'avtoremontnikam.ru',
 'shipreck--s-livejournal-com.turbopages.org': 'shipreck-s.livejournal.com',
 'nortwolf--sam-livejournal-com.turbopages.org': 'nortwolf-sam.livejournal.com',
 'gidroinfo-ru.turbopages.org': 'gidroinfo.ru',
 'auvasilev-livejournal-com.turbopages.org': 'auvasilev.livejournal.com',
 'nologic-ru.turbopages.org': 'nologic.ru',
 'sudak-guru.turbopages.org': 'sudak.guru',
 'liski-bezformata-com.turbopages.org': 'liski.bezformata.com',
 'siletskayanatty-livejournal-com.turbopages.org': 'siletskayanatty.livejournal.com',
 'zoowka-com.turbopages.org': 'zoowka.com',
 'hypegamenews-ru.turbopages.org': 'hypegamenews.ru',
 'ru--tolk-livejournal-com.turbopages.org': 'ru-tolk.livejournal.com',
 'moremeda-com.turbopages.org': 'moremeda.com',
 'ctrannik002-livejournal-com.turbopages.org': 'ctrannik002.livejournal.com',
 'elisa--liz-livejournal-com.turbopages.org': 'elisa-liz.livejournal.com',
 'deva--terra-livejournal-com.turbopages.org': 'deva-terra.livejournal.com',
 'rosreesstr-ru.turbopages.org': 'rosreesstr.ru',
 'vzletit-livejournal-com.turbopages.org': 'vzletit.livejournal.com',
 'shiva16-livejournal-com.turbopages.org': 'shiva16.livejournal.com',
 'hel--sim-livejournal-com.turbopages.org': 'hel-sim.livejournal.com',
 'bel--gorod31-livejournal-com.turbopages.org': 'bel-gorod31.livejournal.com',
 'lovigin-livejournal-com.turbopages.org': 'lovigin.livejournal.com',
 'perevozchik00-livejournal-com.turbopages.org': 'perevozchik00.livejournal.com',
 'oblakastudio-livejournal-com.turbopages.org': 'oblakastudio.livejournal.com',
 'ru--tourism-livejournal-com.turbopages.org': 'ru-tourism.livejournal.com',
 'tvojkishechnik-ru.turbopages.org': 'tvojkishechnik.ru',
 'misscaprizzz-livejournal-com.turbopages.org': 'misscaprizzz.livejournal.com',
 'zaitsevdennis-livejournal-com.turbopages.org': 'zaitsevdennis.livejournal.com',
 'yestravel-ru.turbopages.org': 'yestravel.ru',
 'val--officinalis-livejournal-com.turbopages.org': 'val-officinalis.livejournal.com',
 'alonestalker-livejournal-com.turbopages.org': 'alonestalker.livejournal.com',
 'solomaestro-livejournal-com.turbopages.org': 'solomaestro.livejournal.com',
 'sergei--1956-livejournal-com.turbopages.org': 'sergei-1956.livejournal.com',
 'smska-livejournal-com.turbopages.org': 'smska.livejournal.com',
 'stormax-livejournal-com.turbopages.org': 'stormax.livejournal.com',
 'vagif19-livejournal-com.turbopages.org': 'vagif19.livejournal.com',
 'vcooking-livejournal-com.turbopages.org': 'vcooking.livejournal.com',
 'aerotruba-su.turbopages.org': 'aerotruba.su',
 'ltraditionalist-livejournal-com.turbopages.org': 'ltraditionalist.livejournal.com',
 'b737-livejournal-com.turbopages.org': 'b737.livejournal.com',
 'l8sae--rexhn1-livejournal-com.turbopages.org': 'l8sae-rexhn1.livejournal.com',
 'sravni--motor-ru.turbopages.org': 'sravni-motor.ru',
 'geo360-ru.turbopages.org': 'geo360.ru',
 'mittatiana-livejournal-com.turbopages.org': 'mittatiana.livejournal.com',
 'ivanovska--gorka-livejournal-com.turbopages.org': 'ivanovska-gorka.livejournal.com',
 'aborigenarbata-livejournal-com.turbopages.org': 'aborigenarbata.livejournal.com',
 'rsnnews-ru.turbopages.org': 'rsnnews.ru',
 'shaltay-livejournal-com.turbopages.org': 'shaltay.livejournal.com',
 'retto-livejournal-com.turbopages.org': 'retto.livejournal.com',
 'mon--plaisir-livejournal-com.turbopages.org': 'mon-plaisir.livejournal.com',
 'vikonst-livejournal-com.turbopages.org': 'vikonst.livejournal.com',
 'pressa--tof-livejournal-com.turbopages.org': 'pressa-tof.livejournal.com',
 'rockcult-ru.turbopages.org': 'rockcult.ru',
 'hotarukajiya-livejournal-com.turbopages.org': 'hotarukajiya.livejournal.com',
 'cherepanovhellp-livejournal-com.turbopages.org': 'cherepanovhellp.livejournal.com',
 'ibsorath-livejournal-com.turbopages.org': 'ibsorath.livejournal.com',
 'tihenchik-livejournal-com.turbopages.org': 'tihenchik.livejournal.com',
 'arz--top-ru.turbopages.org': 'arz-top.ru',
 '777ru-livejournal-com.turbopages.org': '777ru.livejournal.com',
 'hoochecoocheman-livejournal-com.turbopages.org': 'hoochecoocheman.livejournal.com',
 'los--oxuenos-livejournal-com.turbopages.org': 'los-oxuenos.livejournal.com',
 'annazv77-livejournal-com.turbopages.org': 'annazv77.livejournal.com',
 'dalnevostochnaya--ipoteka-ru.turbopages.org': 'dalnevostochnaya-ipoteka.ru',
 'daynice-livejournal-com.turbopages.org': 'daynice.livejournal.com',
 'zambek55-livejournal-com.turbopages.org': 'zambek55.livejournal.com',
 'koka--lermont-livejournal-com.turbopages.org': 'koka-lermont.livejournal.com',
 'featherygold-livejournal-com.turbopages.org': 'featherygold.livejournal.com',
 'ladaevolution-ru.turbopages.org': 'ladaevolution.ru',
 'bikepassion-livejournal-com.turbopages.org': 'bikepassion.livejournal.com',
 'glubinka--ru-livejournal-com.turbopages.org': 'glubinka-ru.livejournal.com',
 'aradanthearcher-livejournal-com.turbopages.org': 'aradanthearcher.livejournal.com',
 'shurov--oleg-livejournal-com.turbopages.org': 'shurov-oleg.livejournal.com',
 'uncle--doc-livejournal-com.turbopages.org': 'uncle-doc.livejournal.com',
 'catstreet-ru.turbopages.org': 'catstreet.ru',
 'gutta--honey-livejournal-com.turbopages.org': 'gutta-honey.livejournal.com',
 'bokariki-livejournal-com.turbopages.org': 'bokariki.livejournal.com',
 'ponaehal-livejournal-com.turbopages.org': 'ponaehal.livejournal.com',
 'beremena-ru.turbopages.org': 'beremena.ru',
 'orvill-livejournal-com.turbopages.org': 'orvill.livejournal.com',
 'greendom74-ru.turbopages.org': 'greendom74.ru',
 'vassia-livejournal-com.turbopages.org': 'vassia.livejournal.com',
 'rblaze-livejournal-com.turbopages.org': 'rblaze.livejournal.com',
 'tele--sat-livejournal-com.turbopages.org': 'tele-sat.livejournal.com',
 'prime--shoper-ru.turbopages.org': 'prime-shoper.ru',
 'dzen--rusist-livejournal-com.turbopages.org': 'dzen-rusist.livejournal.com',
 'donetsk--elenka-livejournal-com.turbopages.org': 'donetsk-elenka.livejournal.com',
 'zhagat--dadian-livejournal-com.turbopages.org': 'zhagat-dadian.livejournal.com',
 'nagval--22-livejournal-com.turbopages.org': 'nagval-22.livejournal.com',
 'ty--style-livejournal-com.turbopages.org': 'ty-style.livejournal.com',
 'sort--kartofel-ru.turbopages.org': 'sort-kartofel.ru',
 'vinokurov--roman-livejournal-com.turbopages.org': 'vinokurov-roman.livejournal.com',
 'arbodhy-livejournal-com.turbopages.org': 'arbodhy.livejournal.com',
 'mr--henry--m-livejournal-com.turbopages.org': 'mr-henry-m.livejournal.com',
 'ucmopucm-livejournal-com.turbopages.org': 'ucmopucm.livejournal.com',
 'bird--in--sky58-livejournal-com.turbopages.org': 'bird-in-sky58.livejournal.com',
 'mikhailove-livejournal-com.turbopages.org': 'mikhailove.livejournal.com',
 'wolgodacha-ru.turbopages.org': 'wolgodacha.ru',
 'estellemoda-ru.turbopages.org': 'estellemoda.ru',
 'zdorovajagolova-ru.turbopages.org': 'zdorovajagolova.ru',
 'arhangel-online.turbopages.org': 'arhangel.online',
 'brodsky-livejournal-com.turbopages.org': 'brodsky.livejournal.com',
 'annika--fluffy-livejournal-com.turbopages.org': 'annika-fluffy.livejournal.com',
 'juliana-livejournal-com.turbopages.org': 'juliana.livejournal.com',
 'nadin--86-livejournal-com.turbopages.org': 'nadin-86.livejournal.com',
 'narcofree-ru.turbopages.org': 'narcofree.ru',
 'zhiliekompleksi-ru.turbopages.org': 'zhiliekompleksi.ru',
 'svlasov-livejournal-com.turbopages.org': 'svlasov.livejournal.com',
 'moris--levran-livejournal-com.turbopages.org': 'moris-levran.livejournal.com',
 'shiloffm-livejournal-com.turbopages.org': 'shiloffm.livejournal.com',
 'mumuka-livejournal-com.turbopages.org': 'mumuka.livejournal.com',
 'kssernik-livejournal-com.turbopages.org': 'kssernik.livejournal.com',
 'annyshka--travel-livejournal-com.turbopages.org': 'annyshka-travel.livejournal.com',
 'militarizm-livejournal-com.turbopages.org': 'militarizm.livejournal.com',
 'joker000-livejournal-com.turbopages.org': 'joker000.livejournal.com',
 'boobletrouble-livejournal-com.turbopages.org': 'boobletrouble.livejournal.com',
 'huan--de--vsad-livejournal-com.turbopages.org': 'huan-de-vsad.livejournal.com',
 'serg--illiev-livejournal-com.turbopages.org': 'serg-illiev.livejournal.com',
 'specnaz777-livejournal-com.turbopages.org': 'specnaz777.livejournal.com',
 'lifekostroma-ru.turbopages.org': 'lifekostroma.ru',
 'bubligum9000-livejournal-com.turbopages.org': 'bubligum9000.livejournal.com',
 'sti--club-su.turbopages.org': 'sti-club.su',
 'realistromantic-livejournal-com.turbopages.org': 'realistromantic.livejournal.com',
 'lastwitch-livejournal-com.turbopages.org': 'lastwitch.livejournal.com',
 'svrudyuk-livejournal-com.turbopages.org': 'svrudyuk.livejournal.com',
 'habarnew-livejournal-com.turbopages.org': 'habarnew.livejournal.com',
 'backfourblood-ru.turbopages.org': 'backfourblood.ru',
 'na--novyj--god-ru.turbopages.org': 'na-novyj-god.ru',
 'blender--chat-livejournal-com.turbopages.org': 'blender-chat.livejournal.com',
 'loreley10-livejournal-com.turbopages.org': 'loreley10.livejournal.com',
 'utrennayazvezda-livejournal-com.turbopages.org': 'utrennayazvezda.livejournal.com',
 'neebler-livejournal-com.turbopages.org': 'neebler.livejournal.com',
 'otto--pilot-livejournal-com.turbopages.org': 'otto-pilot.livejournal.com',
 'zdorovie--volos-ru.turbopages.org': 'zdorovie-volos.ru',
 'meerkat--wise-livejournal-com.turbopages.org': 'meerkat-wise.livejournal.com',
 'turizm--ru-livejournal-com.turbopages.org': 'turizm-ru.livejournal.com',
 'foto--sochi-livejournal-com.turbopages.org': 'foto-sochi.livejournal.com',
 'doramax-ru.turbopages.org': 'doramax.ru',
 'interesniy--kiev-livejournal-com.turbopages.org': 'interesniy-kiev.livejournal.com',
 'dveia-livejournal-com.turbopages.org': 'dveia.livejournal.com',
 'webdelphi-ru.turbopages.org': 'webdelphi.ru',
 'vladimirsychev-livejournal-com.turbopages.org': 'vladimirsychev.livejournal.com',
 'natta--morozova-livejournal-com.turbopages.org': 'natta-morozova.livejournal.com',
 'nazpat-livejournal-com.turbopages.org': 'nazpat.livejournal.com',
 'topfermer-ru.turbopages.org': 'topfermer.ru',
 'ypopoff-livejournal-com.turbopages.org': 'ypopoff.livejournal.com',
 'bondarik-livejournal-com.turbopages.org': 'bondarik.livejournal.com',
 'multipovara-ru.turbopages.org': 'multipovara.ru',
 'inceptive-ru.turbopages.org': 'inceptive.ru',
 'livni-bezformata-com.turbopages.org': 'livni.bezformata.com',
 '1mile-cc.turbopages.org': '1mile.cc',
 'comnarcon-livejournal-com.turbopages.org': 'comnarcon.livejournal.com',
 've4naja--pamjat6-livejournal-com.turbopages.org': 've4naja-pamjat6.livejournal.com',
 'greenway--schop-ru.turbopages.org': 'greenway-schop.ru',
 'tropicbirds-ru.turbopages.org': 'tropicbirds.ru',
 'psychoanalysis-livejournal-com.turbopages.org': 'psychoanalysis.livejournal.com',
 'radomirkka-livejournal-com.turbopages.org': 'radomirkka.livejournal.com',
 'trank-livejournal-com.turbopages.org': 'trank.livejournal.com',
 '6a3apob--alex-livejournal-com.turbopages.org': '6a3apob-alex.livejournal.com',
 'pavel--vish-livejournal-com.turbopages.org': 'pavel-vish.livejournal.com',
 '15auto-livejournal-com.turbopages.org': '15auto.livejournal.com',
 'blange--blange-livejournal-com.turbopages.org': 'blange-blange.livejournal.com',
 'retsept--prigotovleniya-ru.turbopages.org': 'retsept-prigotovleniya.ru',
 'nostrov68-livejournal-com.turbopages.org': 'nostrov68.livejournal.com',
 'anecdotes-info.turbopages.org': 'anecdotes.info',
 'trassa--m4-ru.turbopages.org': 'trassa-m4.ru',
 'dmitry--birin-livejournal-com.turbopages.org': 'dmitry-birin.livejournal.com',
 'vse--vkusnoe-livejournal-com.turbopages.org': 'vse-vkusnoe.livejournal.com',
 'pesiq-ru.turbopages.org': 'pesiq.ru',
 'safehair-ru.turbopages.org': 'safehair.ru',
 'tatiaz-livejournal-com.turbopages.org': 'tatiaz.livejournal.com',
 'vashaspina-ru.turbopages.org': 'vashaspina.ru',
 'stomach--info-ru.turbopages.org': 'stomach-info.ru',
 'myfireshop-ru.turbopages.org': 'myfireshop.ru',
 '7cveta-livejournal-com.turbopages.org': '7cveta.livejournal.com',
 'klimat-livejournal-com.turbopages.org': 'klimat.livejournal.com',
 'dom--pod--goroy-livejournal-com.turbopages.org': 'dom-pod-goroy.livejournal.com',
 'antonhd-livejournal-com.turbopages.org': 'antonhd.livejournal.com',
 'yaostrov-ru.turbopages.org': 'yaostrov.ru',
 'jesfor-livejournal-com.turbopages.org': 'jesfor.livejournal.com',
 'markych-livejournal-com.turbopages.org': 'markych.livejournal.com',
 'oldporuchik-livejournal-com.turbopages.org': 'oldporuchik.livejournal.com',
 'travel--baby--org-livejournal-com.turbopages.org': 'travel-baby-org.livejournal.com',
 'yurykrasilnikov-livejournal-com.turbopages.org': 'yurykrasilnikov.livejournal.com',
 'world--cookbook-livejournal-com.turbopages.org': 'world-cookbook.livejournal.com',
 'volgo--dom--stroy-ru.turbopages.org': 'volgo-dom-stroy.ru',
 'mpopenker-livejournal-com.turbopages.org': 'mpopenker.livejournal.com',
 'budmuzhikom-livejournal-com.turbopages.org': 'budmuzhikom.livejournal.com',
 'gregoryens--ru-livejournal-com.turbopages.org': 'gregoryens-ru.livejournal.com',
 'severok1979-livejournal-com.turbopages.org': 'severok1979.livejournal.com',
 'kazyaffka-livejournal-com.turbopages.org': 'kazyaffka.livejournal.com',
 'homophobia--ru-livejournal-com.turbopages.org': 'homophobia-ru.livejournal.com',
 'pet--and-livejournal-com.turbopages.org': 'pet-and.livejournal.com',
 'mazayka--urza-livejournal-com.turbopages.org': 'mazayka-urza.livejournal.com',
 'musllimaa-livejournal-com.turbopages.org': 'musllimaa.livejournal.com',
 'vainahsveras-livejournal-com.turbopages.org': 'vainahsveras.livejournal.com',
 'formatag-ru.turbopages.org': 'formatag.ru',
 'bar--su--chok-livejournal-com.turbopages.org': 'bar-su-chok.livejournal.com',
 'inttera-livejournal-com.turbopages.org': 'inttera.livejournal.com',
 'biznes--on--line-biz.turbopages.org': 'biznes-on-line.biz',
 'liktv-org.turbopages.org': 'liktv.org',
 'nationalism--org-livejournal-com.turbopages.org': 'nationalism-org.livejournal.com',
 'dmitriy-livejournal-com.turbopages.org': 'dmitriy.livejournal.com',
 'rintra-livejournal-com.turbopages.org': 'rintra.livejournal.com',
 'gorbatka-ru.turbopages.org': 'gorbatka.ru',
 'crazyflesh-livejournal-com.turbopages.org': 'crazyflesh.livejournal.com',
 'ticovkusom-ru.turbopages.org': 'ticovkusom.ru',
 'nagmaa-livejournal-com.turbopages.org': 'nagmaa.livejournal.com',
 'rus--luck-livejournal-com.turbopages.org': 'rus-luck.livejournal.com',
 'selenadia-livejournal-com.turbopages.org': 'selenadia.livejournal.com',
 'gushrustalniy-bezformata-com.turbopages.org': 'gushrustalniy.bezformata.com',
 'leib--hussar-livejournal-com.turbopages.org': 'leib-hussar.livejournal.com',
 'alladvice-ru.turbopages.org': 'alladvice.ru',
 'lotos50a-livejournal-com.turbopages.org': 'lotos50a.livejournal.com',
 'tuxic-livejournal-com.turbopages.org': 'tuxic.livejournal.com',
 'psy--baby-livejournal-com.turbopages.org': 'psy-baby.livejournal.com',
 'lilac2012-livejournal-com.turbopages.org': 'lilac2012.livejournal.com',
 'samar--luka-livejournal-com.turbopages.org': 'samar-luka.livejournal.com',
 'ua--antivisa-livejournal-com.turbopages.org': 'ua-antivisa.livejournal.com',
 'nordprod-livejournal-com.turbopages.org': 'nordprod.livejournal.com',
 'me--lamazi-livejournal-com.turbopages.org': 'me-lamazi.livejournal.com',
 'anjerosudjensk-bezformata-com.turbopages.org': 'anjerosudjensk.bezformata.com',
 'pisma08-livejournal-com.turbopages.org': 'pisma08.livejournal.com',
 'ru--philosophy-livejournal-com.turbopages.org': 'ru-philosophy.livejournal.com',
 'juliet--r-livejournal-com.turbopages.org': 'juliet-r.livejournal.com',
 'apfomin-livejournal-com.turbopages.org': 'apfomin.livejournal.com',
 'krug--druzei-ru.turbopages.org': 'krug-druzei.ru',
 'barucaba-livejournal-com.turbopages.org': 'barucaba.livejournal.com',
 'gialuron-com.turbopages.org': 'gialuron.com',
 'adzhaya-livejournal-com.turbopages.org': 'adzhaya.livejournal.com',
 'ru--monopoly-livejournal-com.turbopages.org': 'ru-monopoly.livejournal.com',
 'bezlepkina-livejournal-com.turbopages.org': 'bezlepkina.livejournal.com',
 'skrip--mk-livejournal-com.turbopages.org': 'skrip-mk.livejournal.com',
 'selena2264-livejournal-com.turbopages.org': 'selena2264.livejournal.com',
 'free--maiden-livejournal-com.turbopages.org': 'free-maiden.livejournal.com',
 'freeedu-livejournal-com.turbopages.org': 'freeedu.livejournal.com',
 'business--digest-livejournal-com.turbopages.org': 'business-digest.livejournal.com',
 'aviator9298-livejournal-com.turbopages.org': 'aviator9298.livejournal.com',
 'prigotovdoma-ru.turbopages.org': 'prigotovdoma.ru',
 'ivanov--petrov-livejournal-com.turbopages.org': 'ivanov-petrov.livejournal.com',
 'new--vodokachkin-livejournal-com.turbopages.org': 'new-vodokachkin.livejournal.com',
 'expp-livejournal-com.turbopages.org': 'expp.livejournal.com',
 'doncunita-livejournal-com.turbopages.org': 'doncunita.livejournal.com',
 'george--rooke-livejournal-com.turbopages.org': 'george-rooke.livejournal.com',
 'juri--rust-livejournal-com.turbopages.org': 'juri-rust.livejournal.com',
 'womplanet-ru.turbopages.org': 'womplanet.ru',
 'learn--chinese-ru.turbopages.org': 'learn-chinese.ru',
 'pravdep-livejournal-com.turbopages.org': 'pravdep.livejournal.com',
 'vladimir--elkin-livejournal-com.turbopages.org': 'vladimir-elkin.livejournal.com',
 'p--krysa-livejournal-com.turbopages.org': 'p-krysa.livejournal.com',
 'ezy--life-ru.turbopages.org': 'ezy-life.ru',
 'pro--lico-ru.turbopages.org': 'pro-lico.ru',
 'vse--posobia-ru.turbopages.org': 'vse-posobia.ru',
 'lleo--kaganov-livejournal-com.turbopages.org': 'lleo-kaganov.livejournal.com',
 '2nikgar2-livejournal-com.turbopages.org': '2nikgar2.livejournal.com',
 '2magus-livejournal-com.turbopages.org': '2magus.livejournal.com',
 'being--for--dream-livejournal-com.turbopages.org': 'being-for-dream.livejournal.com',
 'marina--horte-livejournal-com.turbopages.org': 'marina-horte.livejournal.com',
 'pechkeen-livejournal-com.turbopages.org': 'pechkeen.livejournal.com',
 'osetinskie--pirogi-com.turbopages.org': 'osetinskie-pirogi.com',
 'altituda-livejournal-com.turbopages.org': 'altituda.livejournal.com',
 'nebitie-livejournal-com.turbopages.org': 'nebitie.livejournal.com',
 'dobriy--doktor-livejournal-com.turbopages.org': 'dobriy-doktor.livejournal.com',
 'ikvs-livejournal-com.turbopages.org': 'ikvs.livejournal.com',
 'karatel--medicin-livejournal-com.turbopages.org': 'karatel-medicin.livejournal.com',
 'sosizrygalov-livejournal-com.turbopages.org': 'sosizrygalov.livejournal.com',
 'kraeved74-livejournal-com.turbopages.org': 'kraeved74.livejournal.com',
 'rulenta-com.turbopages.org': 'rulenta.com',
 'luxovitsy-ru.turbopages.org': 'luxovitsy.ru',
 'perebabaevvalua-livejournal-com.turbopages.org': 'perebabaevvalua.livejournal.com',
 'garry--kh-livejournal-com.turbopages.org': 'garry-kh.livejournal.com',
 'mirty-livejournal-com.turbopages.org': 'mirty.livejournal.com',
 'jjolyk-livejournal-com.turbopages.org': 'jjolyk.livejournal.com',
 'vahemart-livejournal-com.turbopages.org': 'vahemart.livejournal.com',
 'beacon1143-livejournal-com.turbopages.org': 'beacon1143.livejournal.com',
 'ru--perinatal-livejournal-com.turbopages.org': 'ru-perinatal.livejournal.com',
 'pechnoi-ru.turbopages.org': 'pechnoi.ru',
 'slavikap--2-livejournal-com.turbopages.org': 'slavikap-2.livejournal.com',
 'xochunadachu-ru.turbopages.org': 'xochunadachu.ru',
 'andresfst-livejournal-com.turbopages.org': 'andresfst.livejournal.com',
 'music--of--colors-livejournal-com.turbopages.org': 'music-of-colors.livejournal.com',
 'infox-life.turbopages.org': 'infox.life',
 'nataassa-livejournal-com.turbopages.org': 'nataassa.livejournal.com',
 'viking--nord-livejournal-com.turbopages.org': 'viking-nord.livejournal.com',
 'xn----otbadddfqiihkcavh5a.turbopages.org': 'xn--otbadddfqiihkcavh5a',
 'giorgi--chachba-livejournal-com.turbopages.org': 'giorgi-chachba.livejournal.com',
 'bellamar-livejournal-com.turbopages.org': 'bellamar.livejournal.com',
 'klukva--sugar-livejournal-com.turbopages.org': 'klukva-sugar.livejournal.com',
 'kashevarya-livejournal-com.turbopages.org': 'kashevarya.livejournal.com',
 'izborskiy--club-livejournal-com.turbopages.org': 'izborskiy-club.livejournal.com',
 'showbizzz-net.turbopages.org': 'showbizzz.net',
 'ru--sysadmins-livejournal-com.turbopages.org': 'ru-sysadmins.livejournal.com',
 'proizvolvlastei-livejournal-com.turbopages.org': 'proizvolvlastei.livejournal.com',
 'orehov--ozuevo-livejournal-com.turbopages.org': 'orehov-ozuevo.livejournal.com',
 '0pexo6o--3ye6o-livejournal-com.turbopages.org': '0pexo6o-3ye6o.livejournal.com',
 'minsky--87-livejournal-com.turbopages.org': 'minsky-87.livejournal.com',
 'grigoruk-livejournal-com.turbopages.org': 'grigoruk.livejournal.com',
 'melodie--del--mar-livejournal-com.turbopages.org': 'melodie-del-mar.livejournal.com',
 'vikni-livejournal-com.turbopages.org': 'vikni.livejournal.com',
 'volk-livejournal-com.turbopages.org': 'volk.livejournal.com',
 'talifa88-livejournal-com.turbopages.org': 'talifa88.livejournal.com',
 'aviaslovar-ru.turbopages.org': 'aviaslovar.ru',
 'pyhalov-livejournal-com.turbopages.org': 'pyhalov.livejournal.com',
 'shvp-livejournal-com.turbopages.org': 'shvp.livejournal.com',
 'rezkadecor-ru.turbopages.org': 'rezkadecor.ru',
 'sefeol-livejournal-com.turbopages.org': 'sefeol.livejournal.com',
 'alexzirya-livejournal-com.turbopages.org': 'alexzirya.livejournal.com',
 'borisovdimitry-livejournal-com.turbopages.org': 'borisovdimitry.livejournal.com',
 'rabota--electrikom-ru.turbopages.org': 'rabota-electrikom.ru',
 'si--speech-ru.turbopages.org': 'si-speech.ru',
 'velkoldin-livejournal-com.turbopages.org': 'velkoldin.livejournal.com',
 'piter--places-livejournal-com.turbopages.org': 'piter-places.livejournal.com',
 '41svadba-ru.turbopages.org': '41svadba.ru',
 'viktoriashilova-livejournal-com.turbopages.org': 'viktoriashilova.livejournal.com',
 'vgf--tag-livejournal-com.turbopages.org': 'vgf-tag.livejournal.com',
 'kindzadzaa-livejournal-com.turbopages.org': 'kindzadzaa.livejournal.com',
 'daybook-info.turbopages.org': 'daybook.info',
 'congregatio-livejournal-com.turbopages.org': 'congregatio.livejournal.com',
 'ru--psya-livejournal-com.turbopages.org': 'ru-psya.livejournal.com',
 'telecomtimes-ru.turbopages.org': 'telecomtimes.ru',
 'bedolaga--iz--che-livejournal-com.turbopages.org': 'bedolaga-iz-che.livejournal.com',
 'andrey--andreew-livejournal-com.turbopages.org': 'andrey-andreew.livejournal.com',
 'happydolls-livejournal-com.turbopages.org': 'happydolls.livejournal.com',
 'mexsika-livejournal-com.turbopages.org': 'mexsika.livejournal.com',
 'czarnysmok-livejournal-com.turbopages.org': 'czarnysmok.livejournal.com',
 'vvdom-livejournal-com.turbopages.org': 'vvdom.livejournal.com',
 'autobotanik-livejournal-com.turbopages.org': 'autobotanik.livejournal.com',
 'punya--1-livejournal-com.turbopages.org': 'punya-1.livejournal.com',
 'toprating-livejournal-com.turbopages.org': 'toprating.livejournal.com',
 'jurikan-livejournal-com.turbopages.org': 'jurikan.livejournal.com',
 'skazka--lbt-ru.turbopages.org': 'skazka-lbt.ru',
 'vaz--2115i-ru.turbopages.org': 'vaz-2115i.ru',
 'aspar2-livejournal-com.turbopages.org': 'aspar2.livejournal.com',
 'oksiv-livejournal-com.turbopages.org': 'oksiv.livejournal.com',
 'valerytim-livejournal-com.turbopages.org': 'valerytim.livejournal.com',
 'pestraya--skazka-livejournal-com.turbopages.org': 'pestraya-skazka.livejournal.com',
 'ladamaran-livejournal-com.turbopages.org': 'ladamaran.livejournal.com',
 'geraklit--eff-livejournal-com.turbopages.org': 'geraklit-eff.livejournal.com',
 'vtaysaev-livejournal-com.turbopages.org': 'vtaysaev.livejournal.com',
 'oleg--volkov-livejournal-com.turbopages.org': 'oleg-volkov.livejournal.com',
 '5ttrg-livejournal-com.turbopages.org': '5ttrg.livejournal.com',
 'karl--lvovich-livejournal-com.turbopages.org': 'karl-lvovich.livejournal.com',
 'yamobi-ru.turbopages.org': 'yamobi.ru',
 'dexamethasone-livejournal-com.turbopages.org': 'dexamethasone.livejournal.com',
 'compinf-ru.turbopages.org': 'compinf.ru',
 'captainevery-livejournal-com.turbopages.org': 'captainevery.livejournal.com',
 '4x4--life-livejournal-com.turbopages.org': '4x4-life.livejournal.com',
 'greedyspeedy-livejournal-com.turbopages.org': 'greedyspeedy.livejournal.com',
 'botsman--m-livejournal-com.turbopages.org': 'botsman-m.livejournal.com',
 'melomanchick-ru.turbopages.org': 'melomanchick.ru',
 'oxic2000-livejournal-com.turbopages.org': 'oxic2000.livejournal.com',
 'fduchun76-livejournal-com.turbopages.org': 'fduchun76.livejournal.com',
 'moskvameniaetsy-livejournal-com.turbopages.org': 'moskvameniaetsy.livejournal.com',
 'olga--fink-livejournal-com.turbopages.org': 'olga-fink.livejournal.com',
 'gerynych-livejournal-com.turbopages.org': 'gerynych.livejournal.com',
 'xn---su-5cdae2a4b2a1a.turbopages.org': 'xn-.su.5cdae2a4b2a1a',
 'masterka--zl-livejournal-com.turbopages.org': 'masterka-zl.livejournal.com',
 'tebenko-livejournal-com.turbopages.org': 'tebenko.livejournal.com',
 'business--notepad-ru.turbopages.org': 'business-notepad.ru',
 'alex--anv-livejournal-com.turbopages.org': 'alex-anv.livejournal.com',
 'krsn-octagon-media.turbopages.org': 'krsn.octagon.media',
 'web--revenue-ru.turbopages.org': 'web-revenue.ru',
 'seversk-bezformata-com.turbopages.org': 'seversk.bezformata.com',
 'vadymych-livejournal-com.turbopages.org': 'vadymych.livejournal.com',
 'ru--matriarchy-livejournal-com.turbopages.org': 'ru-matriarchy.livejournal.com',
 'stilett--1-livejournal-com.turbopages.org': 'stilett-1.livejournal.com',
 'troitsk--eco-livejournal-com.turbopages.org': 'troitsk-eco.livejournal.com',
 'brndk-livejournal-com.turbopages.org': 'brndk.livejournal.com',
 'iz--bumagi--svoimi--rukami-ru.turbopages.org': 'iz-bumagi-svoimi-rukami.ru',
 'penoplastova-livejournal-com.turbopages.org': 'penoplastova.livejournal.com',
 'parusa--chg-livejournal-com.turbopages.org': 'parusa-chg.livejournal.com',
 '21242353-livejournal-com.turbopages.org': '21242353.livejournal.com',
 'helix--lab-livejournal-com.turbopages.org': 'helix-lab.livejournal.com',
 'inomoderator-livejournal-com.turbopages.org': 'inomoderator.livejournal.com',
 'escapistus-livejournal-com.turbopages.org': 'escapistus.livejournal.com',
 'kokcamokca-livejournal-com.turbopages.org': 'kokcamokca.livejournal.com',
 'arturborunov-livejournal-com.turbopages.org': 'arturborunov.livejournal.com',
 'sardor-livejournal-com.turbopages.org': 'sardor.livejournal.com',
 'vladimvu777-livejournal-com.turbopages.org': 'vladimvu777.livejournal.com',
 'rumpus-ru.turbopages.org': 'rumpus.ru',
 'djavid-livejournal-com.turbopages.org': 'djavid.livejournal.com',
 'andywendy-livejournal-com.turbopages.org': 'andywendy.livejournal.com',
 'mager-livejournal-com.turbopages.org': 'mager.livejournal.com',
 'nionilas--baby-livejournal-com.turbopages.org': 'nionilas-baby.livejournal.com',
 'voprak-livejournal-com.turbopages.org': 'voprak.livejournal.com',
 'meteo-livejournal-com.turbopages.org': 'meteo.livejournal.com',
 'arcanum--hominem-livejournal-com.turbopages.org': 'arcanum-hominem.livejournal.com',
 'avtoram--com-livejournal-com.turbopages.org': 'avtoram-com.livejournal.com',
 'testosam-ru.turbopages.org': 'testosam.ru',
 'fondpr-livejournal-com.turbopages.org': 'fondpr.livejournal.com',
 'chehov-bezformata-com.turbopages.org': 'chehov.bezformata.com',
 'ole--si-livejournal-com.turbopages.org': 'ole-si.livejournal.com',
 'kolibri-press.turbopages.org': 'kolibri.press',
 'puls-spb-ru.turbopages.org': 'puls.spb.ru',
 'lopatov--45-livejournal-com.turbopages.org': 'lopatov-45.livejournal.com',
 'rostov--compro-livejournal-com.turbopages.org': 'rostov-compro.livejournal.com',
 'myrra--zet-livejournal-com.turbopages.org': 'myrra-zet.livejournal.com',
 'mcdowns-livejournal-com.turbopages.org': 'mcdowns.livejournal.com',
 'gekata-livejournal-com.turbopages.org': 'gekata.livejournal.com',
 'yu--de--ki-livejournal-com.turbopages.org': 'yu-de-ki.livejournal.com',
 'mefodiy2009-livejournal-com.turbopages.org': 'mefodiy2009.livejournal.com',
 'selectgames-xyz.turbopages.org': 'selectgames.xyz',
 'aeroo-ru.turbopages.org': 'aeroo.ru',
 'anton--franchuk-livejournal-com.turbopages.org': 'anton-franchuk.livejournal.com',
 'bukar-ru.turbopages.org': 'bukar.ru',
 'everytips-ru.turbopages.org': 'everytips.ru',
 'sportkv-com.turbopages.org': 'sportkv.com',
 'barometr-livejournal-com.turbopages.org': 'barometr.livejournal.com',
 'nati--flame-livejournal-com.turbopages.org': 'nati-flame.livejournal.com',
 'potreb--prav-livejournal-com.turbopages.org': 'potreb-prav.livejournal.com',
 'marisha--amber-livejournal-com.turbopages.org': 'marisha-amber.livejournal.com',
 'kotobormot-ru.turbopages.org': 'kotobormot.ru',
 'vaz--2109-ru.turbopages.org': 'vaz-2109.ru',
 'iereyviktor81-livejournal-com.turbopages.org': 'iereyviktor81.livejournal.com',
 'puli--snegopaqa-livejournal-com.turbopages.org': 'puli-snegopaqa.livejournal.com',
 'klimentievich-livejournal-com.turbopages.org': 'klimentievich.livejournal.com',
 'krre31-livejournal-com.turbopages.org': 'krre31.livejournal.com',
 'platina--ru-livejournal-com.turbopages.org': 'platina-ru.livejournal.com',
 'vadimrm-livejournal-com.turbopages.org': 'vadimrm.livejournal.com',
 'black--merki-livejournal-com.turbopages.org': 'black-merki.livejournal.com',
 'tactical-livejournal-com.turbopages.org': 'tactical.livejournal.com',
 'moyaikea-livejournal-com.turbopages.org': 'moyaikea.livejournal.com',
 'xen0n-livejournal-com.turbopages.org': 'xen0n.livejournal.com',
 'ace--alex-livejournal-com.turbopages.org': 'ace-alex.livejournal.com',
 'angarsk38-ru.turbopages.org': 'angarsk38.ru',
 'atkarsk-bezformata-com.turbopages.org': 'atkarsk.bezformata.com',
 'iristoninfo-livejournal-com.turbopages.org': 'iristoninfo.livejournal.com',
 'inryko-livejournal-com.turbopages.org': 'inryko.livejournal.com',
 'old--soldier1982-livejournal-com.turbopages.org': 'old-soldier1982.livejournal.com',
 'paek21-livejournal-com.turbopages.org': 'paek21.livejournal.com',
 'pa--ek-livejournal-com.turbopages.org': 'pa-ek.livejournal.com',
 'manjonga-livejournal-com.turbopages.org': 'manjonga.livejournal.com',
 'ts--uzao-livejournal-com.turbopages.org': 'ts-uzao.livejournal.com',
 'dudukka-livejournal-com.turbopages.org': 'dudukka.livejournal.com',
 'mamlen-livejournal-com.turbopages.org': 'mamlen.livejournal.com',
 'darft-livejournal-com.turbopages.org': 'darft.livejournal.com',
 'transphoto2007-livejournal-com.turbopages.org': 'transphoto2007.livejournal.com',
 'tattoo--shop55-ru.turbopages.org': 'tattoo-shop55.ru',
 'yseverinova-livejournal-com.turbopages.org': 'yseverinova.livejournal.com',
 'bitsa--les-livejournal-com.turbopages.org': 'bitsa-les.livejournal.com',
 'ihedgehogi-livejournal-com.turbopages.org': 'ihedgehogi.livejournal.com',
 'andrey--stifeev-livejournal-com.turbopages.org': 'andrey-stifeev.livejournal.com',
 'ottosteinach-ru.turbopages.org': 'ottosteinach.ru',
 'majorfonskrip-livejournal-com.turbopages.org': 'majorfonskrip.livejournal.com',
 'baraholka-mastergrad-com.turbopages.org': 'baraholka.mastergrad.com',
 'serp--po-livejournal-com.turbopages.org': 'serp-po.livejournal.com',
 'nialma-livejournal-com.turbopages.org': 'nialma.livejournal.com',
 'chatcher2012-livejournal-com.turbopages.org': 'chatcher2012.livejournal.com',
 'ornament--i--stil-livejournal-com.turbopages.org': 'ornament-i-stil.livejournal.com',
 'ushakovpetr-livejournal-com.turbopages.org': 'ushakovpetr.livejournal.com',
 'proroslo-ru.turbopages.org': 'proroslo.ru',
 'kapitonovih-livejournal-com.turbopages.org': 'kapitonovih.livejournal.com',
 'moyugolok-livejournal-com.turbopages.org': 'moyugolok.livejournal.com',
 'rt--russian-livejournal-com.turbopages.org': 'rt-russian.livejournal.com',
 'budzdorovstarina-ru.turbopages.org': 'budzdorovstarina.ru',
 'neurograff-livejournal-com.turbopages.org': 'neurograff.livejournal.com',
 'wine--street-livejournal-com.turbopages.org': 'wine-street.livejournal.com',
 'lumbermoscow-livejournal-com.turbopages.org': 'lumbermoscow.livejournal.com',
 'krut--art-ru.turbopages.org': 'krut-art.ru',
 'moniteur-ru.turbopages.org': 'moniteur.ru',
 'kanash-bezformata-com.turbopages.org': 'kanash.bezformata.com',
 'perm--news-net.turbopages.org': 'perm-news.net',
 'jollyphonics-ru.turbopages.org': 'jollyphonics.ru',
 'mu--rena-livejournal-com.turbopages.org': 'mu-rena.livejournal.com',
 '1mim-livejournal-com.turbopages.org': '1mim.livejournal.com',
 'mister65-livejournal-com.turbopages.org': 'mister65.livejournal.com',
 'corgipembroke-ru.turbopages.org': 'corgipembroke.ru',
 'viardo--polina-livejournal-com.turbopages.org': 'viardo-polina.livejournal.com',
 'pilar--lar--lar-livejournal-com.turbopages.org': 'pilar-lar-lar.livejournal.com',
 'smerchenko-livejournal-com.turbopages.org': 'smerchenko.livejournal.com',
 'all--about--world-livejournal-com.turbopages.org': 'all-about-world.livejournal.com',
 'shpora-org.turbopages.org': 'shpora.org',
 'niunun-livejournal-com.turbopages.org': 'niunun.livejournal.com',
 'frendches-livejournal-com.turbopages.org': 'frendches.livejournal.com',
 'timonia-livejournal-com.turbopages.org': 'timonia.livejournal.com',
 'shtish-livejournal-com.turbopages.org': 'shtish.livejournal.com',
 'vladbabayan-livejournal-com.turbopages.org': 'vladbabayan.livejournal.com',
 'anna--ufa-livejournal-com.turbopages.org': 'anna-ufa.livejournal.com',
 'holit--i--leleyat-livejournal-com.turbopages.org': 'holit-i-leleyat.livejournal.com',
 'inbio-livejournal-com.turbopages.org': 'inbio.livejournal.com',
 'treshnik-livejournal-com.turbopages.org': 'treshnik.livejournal.com',
 'tramtek-livejournal-com.turbopages.org': 'tramtek.livejournal.com',
 'dmitriy16dndz-livejournal-com.turbopages.org': 'dmitriy16dndz.livejournal.com',
 'my--z--i--l-livejournal-com.turbopages.org': 'my-z-i-l.livejournal.com',
 'gorotskop-livejournal-com.turbopages.org': 'gorotskop.livejournal.com',
 'doll--perian-livejournal-com.turbopages.org': 'doll-perian.livejournal.com',
 'delvaneo-ru.turbopages.org': 'delvaneo.ru',
 'suomilarissa-livejournal-com.turbopages.org': 'suomilarissa.livejournal.com',
 'vuspeh-ru.turbopages.org': 'vuspeh.ru',
 'timon--timonich-livejournal-com.turbopages.org': 'timon-timonich.livejournal.com',
 'altaienergosbyt-livejournal-com.turbopages.org': 'altaienergosbyt.livejournal.com',
 'serial--da-livejournal-com.turbopages.org': 'serial-da.livejournal.com',
 'sergeyka--ang-livejournal-com.turbopages.org': 'sergeyka-ang.livejournal.com',
 'a--garvey-livejournal-com.turbopages.org': 'a-garvey.livejournal.com',
 'ulfofan-livejournal-com.turbopages.org': 'ulfofan.livejournal.com',
 'knigofile-livejournal-com.turbopages.org': 'knigofile.livejournal.com',
 'school--planet-ru.turbopages.org': 'school-planet.ru',
 'andrycorp-livejournal-com.turbopages.org': 'andrycorp.livejournal.com',
 'drakonit-livejournal-com.turbopages.org': 'drakonit.livejournal.com',
 'fhwl-ru.turbopages.org': 'fhwl.ru',
 'drakoniha-livejournal-com.turbopages.org': 'drakoniha.livejournal.com',
 'sport--iv-ru.turbopages.org': 'sport-iv.ru',
 'kobeleff-livejournal-com.turbopages.org': 'kobeleff.livejournal.com',
 'dostalo-livejournal-com.turbopages.org': 'dostalo.livejournal.com',
 'stella--pop-livejournal-com.turbopages.org': 'stella-pop.livejournal.com',
 'dimolution-livejournal-com.turbopages.org': 'dimolution.livejournal.com',
 'o--alexandr75-livejournal-com.turbopages.org': 'o-alexandr75.livejournal.com',
 '9orblnbl4-livejournal-com.turbopages.org': '9orblnbl4.livejournal.com',
 'mux22-livejournal-com.turbopages.org': 'mux22.livejournal.com',
 'gisellevv-livejournal-com.turbopages.org': 'gisellevv.livejournal.com',
 'family365-ru.turbopages.org': 'family365.ru',
 'moto13-ru.turbopages.org': 'moto13.ru',
 'anchoret-livejournal-com.turbopages.org': 'anchoret.livejournal.com',
 'effect--diet-livejournal-com.turbopages.org': 'effect-diet.livejournal.com',
 'mr--anderson22-livejournal-com.turbopages.org': 'mr-anderson22.livejournal.com',
 'gmaxbu-livejournal-com.turbopages.org': 'gmaxbu.livejournal.com',
 'tele--kadr-ru.turbopages.org': 'tele-kadr.ru',
 'klikunov--nd-livejournal-com.turbopages.org': 'klikunov-nd.livejournal.com',
 '5cek-livejournal-com.turbopages.org': '5cek.livejournal.com',
 'zastris-livejournal-com.turbopages.org': 'zastris.livejournal.com',
 'express--pol-ru.turbopages.org': 'express-pol.ru',
 'pirogu-ru.turbopages.org': 'pirogu.ru',
 'bar69marisha-livejournal-com.turbopages.org': 'bar69marisha.livejournal.com',
 'snowlass-livejournal-com.turbopages.org': 'snowlass.livejournal.com',
 'knigo--man-livejournal-com.turbopages.org': 'knigo-man.livejournal.com',
 'osfery-livejournal-com.turbopages.org': 'osfery.livejournal.com',
 'ageev--vadim-livejournal-com.turbopages.org': 'ageev-vadim.livejournal.com',
 'xn----7sba0bilddqim0ai7bxc.turbopages.org': 'xn--7sba0bilddqim0ai7bxc',
 'gl--lib-ru.turbopages.org': 'gl-lib.ru',
 'advokat--bondarchuk-ru.turbopages.org': 'advokat-bondarchuk.ru',
 'hilazhev-livejournal-com.turbopages.org': 'hilazhev.livejournal.com',
 'sergievposad-bezformata-com.turbopages.org': 'sergievposad.bezformata.com',
 'sergievgrad-ru.turbopages.org': 'sergievgrad.ru',
 'kompromatru-livejournal-com.turbopages.org': 'kompromatru.livejournal.com',
 'krd-livejournal-com.turbopages.org': 'krd.livejournal.com',
 'antergen-livejournal-com.turbopages.org': 'antergen.livejournal.com',
 'grafcard-ru.turbopages.org': 'grafcard.ru',
 'mageia-ru.turbopages.org': 'mageia.ru',
 'master--designer-livejournal-com.turbopages.org': 'master-designer.livejournal.com',
 'buguruslan-bezformata-com.turbopages.org': 'buguruslan.bezformata.com',
 'ludovit-livejournal-com.turbopages.org': 'ludovit.livejournal.com',
 'sky--hope-livejournal-com.turbopages.org': 'sky-hope.livejournal.com',
 'alex--mironov-livejournal-com.turbopages.org': 'alex-mironov.livejournal.com',
 'alexey--matveev-livejournal-com.turbopages.org': 'alexey-matveev.livejournal.com',
 'ecobeltex-ru.turbopages.org': 'ecobeltex.ru',
 'bougaev-livejournal-com.turbopages.org': 'bougaev.livejournal.com',
 'child--gifts-ru.turbopages.org': 'child-gifts.ru',
 'korepov1986-livejournal-com.turbopages.org': 'korepov1986.livejournal.com',
 'zelenoglazkaira-livejournal-com.turbopages.org': 'zelenoglazkaira.livejournal.com',
 'poparimsya-com.turbopages.org': 'poparimsya.com',
 'bell--mess-livejournal-com.turbopages.org': 'bell-mess.livejournal.com',
 'mezhdunami-net.turbopages.org': 'mezhdunami.net',
 'lsmodelclub-livejournal-com.turbopages.org': 'lsmodelclub.livejournal.com',
 'vlasovdenis-livejournal-com.turbopages.org': 'vlasovdenis.livejournal.com',
 'omnomad-com.turbopages.org': 'omnomad.com',
 't--moshkin-livejournal-com.turbopages.org': 't-moshkin.livejournal.com',
 'shashlichnii--dvorik-ru.turbopages.org': 'shashlichnii-dvorik.ru',
 'vnovomsvete-com.turbopages.org': 'vnovomsvete.com',
 'kirov-big--book--med-ru.turbopages.org': 'kirov.big-book-med.ru',
 'shinkarenkostas-livejournal-com.turbopages.org': 'shinkarenkostas.livejournal.com',
 'tiranozavrrex-livejournal-com.turbopages.org': 'tiranozavrrex.livejournal.com',
 'one--bonodono-livejournal-com.turbopages.org': 'one-bonodono.livejournal.com',
 'prosodia-ru.turbopages.org': 'prosodia.ru',
 'liniya--jizni-livejournal-com.turbopages.org': 'liniya-jizni.livejournal.com',
 'rostovdon-livejournal-com.turbopages.org': 'rostovdon.livejournal.com',
 'das--rus-livejournal-com.turbopages.org': 'das-rus.livejournal.com',
 'moskalkov--opera-livejournal-com.turbopages.org': 'moskalkov-opera.livejournal.com',
 'kent12321-livejournal-com.turbopages.org': 'kent12321.livejournal.com',
 'ment52-livejournal-com.turbopages.org': 'ment52.livejournal.com',
 'v--a--l--e--k-livejournal-com.turbopages.org': 'v-a-l-e-k.livejournal.com',
 'iosaaris-livejournal-com.turbopages.org': 'iosaaris.livejournal.com',
 'zoosovet-livejournal-com.turbopages.org': 'zoosovet.livejournal.com',
 'krasunia-ru.turbopages.org': 'krasunia.ru',
 'raseyskiy-livejournal-com.turbopages.org': 'raseyskiy.livejournal.com',
 'agvendir-livejournal-com.turbopages.org': 'agvendir.livejournal.com',
 'spydell-livejournal-com.turbopages.org': 'spydell.livejournal.com',
 'marvianna-livejournal-com.turbopages.org': 'marvianna.livejournal.com',
 'volgograd--trv-ru.turbopages.org': 'volgograd-trv.ru',
 'salvador--n69-livejournal-com.turbopages.org': 'salvador-n69.livejournal.com',
 'cat--brasil-livejournal-com.turbopages.org': 'cat-brasil.livejournal.com',
 'lucas--v--leyden-livejournal-com.turbopages.org': 'lucas-v-leyden.livejournal.com',
 'nastasyia-livejournal-com.turbopages.org': 'nastasyia.livejournal.com',
 'edanaprobu-ru.turbopages.org': 'edanaprobu.ru',
 'videokenigsberg-ru.turbopages.org': 'videokenigsberg.ru',
 'cbsmedia-ru.turbopages.org': 'cbsmedia.ru',
 'redyarsk-ru.turbopages.org': 'redyarsk.ru',
 'olbori--lu-livejournal-com.turbopages.org': 'olbori-lu.livejournal.com',
 'sociomodel-ru.turbopages.org': 'sociomodel.ru',
 'colonist-livejournal-com.turbopages.org': 'colonist.livejournal.com',
 'ruspravda-livejournal-com.turbopages.org': 'ruspravda.livejournal.com',
 'eucariot-livejournal-com.turbopages.org': 'eucariot.livejournal.com',
 'garpia-livejournal-com.turbopages.org': 'garpia.livejournal.com',
 'geokhar-livejournal-com.turbopages.org': 'geokhar.livejournal.com',
 'jenyabril-livejournal-com.turbopages.org': 'jenyabril.livejournal.com',
 'gg--hohia-livejournal-com.turbopages.org': 'gg-hohia.livejournal.com',
 'food4brain-ru.turbopages.org': 'food4brain.ru',
 'su--bodybuilding-livejournal-com.turbopages.org': 'su-bodybuilding.livejournal.com',
 'samsung--galaxy-mobi.turbopages.org': 'samsung-galaxy.mobi',
 'kolobok1973-livejournal-com.turbopages.org': 'kolobok1973.livejournal.com',
 'levas555-livejournal-com.turbopages.org': 'levas555.livejournal.com',
 'otponosa-com.turbopages.org': 'otponosa.com',
 'de--strega-livejournal-com.turbopages.org': 'de-strega.livejournal.com',
 'fimpuls-ru.turbopages.org': 'fimpuls.ru',
 'handynews-ru.turbopages.org': 'handynews.ru',
 '10dogs-ru.turbopages.org': '10dogs.ru',
 'elnur-livejournal-com.turbopages.org': 'elnur.livejournal.com',
 'erofey--manager-livejournal-com.turbopages.org': 'erofey-manager.livejournal.com',
 '7strannik-livejournal-com.turbopages.org': '7strannik.livejournal.com',
 'a--dyukov-livejournal-com.turbopages.org': 'a-dyukov.livejournal.com',
 'halvita-ru.turbopages.org': 'halvita.ru',
 'dondub-com.turbopages.org': 'dondub.com',
 'modmap-ru.turbopages.org': 'modmap.ru',
 'vsebonuskarti-ru.turbopages.org': 'vsebonuskarti.ru',
    'glagolas-livejournal-com.turbopages.org': 'glagolas.livejournal.com',
 'meddoctr-ru.turbopages.org': 'meddoctr.ru',
 'artur--s-livejournal-com.turbopages.org': 'artur-s.livejournal.com',
 'musulmon-livejournal-com.turbopages.org': 'musulmon.livejournal.com',
 'andy--sim-livejournal-com.turbopages.org': 'andy-sim.livejournal.com',
 'americanlegends-livejournal-com.turbopages.org': 'americanlegends.livejournal.com',
 'all--origami-ru.turbopages.org': 'all-origami.ru',
 'cat--expert-ru.turbopages.org': 'cat-expert.ru',
 'sdvoryanchikov-livejournal-com.turbopages.org': 'sdvoryanchikov.livejournal.com',
 'fashionofspain-livejournal-com.turbopages.org': 'fashionofspain.livejournal.com',
 'olgasukhova-livejournal-com.turbopages.org': 'olgasukhova.livejournal.com',
 'dobrokoshka-livejournal-com.turbopages.org': 'dobrokoshka.livejournal.com',
 'nai2008-livejournal-com.turbopages.org': 'nai2008.livejournal.com',
 'badgun159-livejournal-com.turbopages.org': 'badgun159.livejournal.com',
 'polarman-livejournal-com.turbopages.org': 'polarman.livejournal.com',
 'lytkin--pavel-livejournal-com.turbopages.org': 'lytkin-pavel.livejournal.com',
 'krolik--pp-livejournal-com.turbopages.org': 'krolik-pp.livejournal.com',
 'l--boris-livejournal-com.turbopages.org': 'l-boris.livejournal.com',
 'ruk--boi-livejournal-com.turbopages.org': 'ruk-boi.livejournal.com',
 'ddavidyan-livejournal-com.turbopages.org': 'ddavidyan.livejournal.com',
 'crazy--reader-livejournal-com.turbopages.org': 'crazy-reader.livejournal.com',
 'zahar-livejournal-com.turbopages.org': 'zahar.livejournal.com',
 'ourania--m-livejournal-com.turbopages.org': 'ourania-m.livejournal.com',
 'orenhostel-livejournal-com.turbopages.org': 'orenhostel.livejournal.com',
 'akimus--ra-livejournal-com.turbopages.org': 'akimus-ra.livejournal.com',
 'mielomi-livejournal-com.turbopages.org': 'mielomi.livejournal.com',
 'a--eliseev-livejournal-com.turbopages.org': 'a-eliseev.livejournal.com',
 'zzelenskaya-livejournal-com.turbopages.org': 'zzelenskaya.livejournal.com',
 'samogon--stok-ru.turbopages.org': 'samogon-stok.ru',
 'soc--factor-ru.turbopages.org': 'soc-factor.ru',
 'spirinmusic-ru.turbopages.org': 'spirinmusic.ru',
 'maloyaroslavec-bezformata-com.turbopages.org': 'maloyaroslavec.bezformata.com',
 'my--en-ru.turbopages.org': 'my-en.ru',
 'ru--chp-livejournal-com.turbopages.org': 'ru-chp.livejournal.com',
 'alatus--serpens-livejournal-com.turbopages.org': 'alatus-serpens.livejournal.com',
 'vitalymorozov-livejournal-com.turbopages.org': 'vitalymorozov.livejournal.com',
 'nastennnnnnnka-livejournal-com.turbopages.org': 'nastennnnnnnka.livejournal.com',
 'routir-livejournal-com.turbopages.org': 'routir.livejournal.com',
 'korolew-livejournal-com.turbopages.org': 'korolew.livejournal.com',
 'aos1986-livejournal-com.turbopages.org': 'aos1986.livejournal.com',
 'konditer--club-ru.turbopages.org': 'konditer-club.ru',
 'vdmironov-livejournal-com.turbopages.org': 'vdmironov.livejournal.com',
 'zoo--sarov-ru.turbopages.org': 'zoo-sarov.ru',
 'terskiykazak-livejournal-com.turbopages.org': 'terskiykazak.livejournal.com',
 'sergastra-livejournal-com.turbopages.org': 'sergastra.livejournal.com',
 'm--kruglov-livejournal-com.turbopages.org': 'm-kruglov.livejournal.com',
 'geophoto-livejournal-com.turbopages.org': 'geophoto.livejournal.com',
 'folkvald-livejournal-com.turbopages.org': 'folkvald.livejournal.com',
 '2001photo-livejournal-com.turbopages.org': '2001photo.livejournal.com',
 'plohoy--zyalt-livejournal-com.turbopages.org': 'plohoy-zyalt.livejournal.com',
 'ognersan-livejournal-com.turbopages.org': 'ognersan.livejournal.com',
 'numach-livejournal-com.turbopages.org': 'numach.livejournal.com',
 'profe--12-livejournal-com.turbopages.org': 'profe-12.livejournal.com',
 'okkomovies-livejournal-com.turbopages.org': 'okkomovies.livejournal.com',
 'tezska1-livejournal-com.turbopages.org': 'tezska1.livejournal.com',
 'vechniyzov-livejournal-com.turbopages.org': 'vechniyzov.livejournal.com',
 'iskusstvo-livejournal-com.turbopages.org': 'iskusstvo.livejournal.com',
 'l--aretta-livejournal-com.turbopages.org': 'l-aretta.livejournal.com',
 'dumspirospero--l-livejournal-com.turbopages.org': 'dumspirospero-l.livejournal.com',
 'flower--kalinka-livejournal-com.turbopages.org': 'flower-kalinka.livejournal.com',
 'shunrasher-livejournal-com.turbopages.org': 'shunrasher.livejournal.com',
 'christ--vs--judai-livejournal-com.turbopages.org': 'christ-vs-judai.livejournal.com',
 'zarow-livejournal-com.turbopages.org': 'zarow.livejournal.com',
 'tivir-livejournal-com.turbopages.org': 'tivir.livejournal.com',
 'kosmetichka-livejournal-com.turbopages.org': 'kosmetichka.livejournal.com',
 'vitaminki-guru.turbopages.org': 'vitaminki.guru',
 'dream--catch--me-livejournal-com.turbopages.org': 'dream-catch-me.livejournal.com',
 'sau--rina-livejournal-com.turbopages.org': 'sau-rina.livejournal.com',
 'katerinaelmar-livejournal-com.turbopages.org': 'katerinaelmar.livejournal.com',
 'worldwebinfo-ru.turbopages.org': 'worldwebinfo.ru',
 'mari--de--boshir-livejournal-com.turbopages.org': 'mari-de-boshir.livejournal.com',
 'avtoidei-ru.turbopages.org': 'avtoidei.ru',
 'xn------5cdj9bc0bpcdhn7au.turbopages.org': 'xn---5cdj9bc0bpcdhn7au',
 'reemont-ru.turbopages.org': 'reemont.ru',
 'biserlace-livejournal-com.turbopages.org': 'biserlace.livejournal.com',
 'pivo--msu-livejournal-com.turbopages.org': 'pivo-msu.livejournal.com',
 'agidel--town-livejournal-com.turbopages.org': 'agidel-town.livejournal.com',
 'available--cook-livejournal-com.turbopages.org': 'available-cook.livejournal.com',
 'jrtmy-ru.turbopages.org': 'jrtmy.ru',
 'doktorpravda-livejournal-com.turbopages.org': 'doktorpravda.livejournal.com',
 'leo--mosk-livejournal-com.turbopages.org': 'leo-mosk.livejournal.com',
 'i--v--zhuravlyov-livejournal-com.turbopages.org': 'i-v-zhuravlyov.livejournal.com',
 'ketomalina-ru.turbopages.org': 'ketomalina.ru',
 'vit--erofeev-livejournal-com.turbopages.org': 'vit-erofeev.livejournal.com',
 'kovoron-livejournal-com.turbopages.org': 'kovoron.livejournal.com',
 'isilgan-livejournal-com.turbopages.org': 'isilgan.livejournal.com',
 'ahiin-livejournal-com.turbopages.org': 'ahiin.livejournal.com',
 'razdolbaypegas-livejournal-com.turbopages.org': 'razdolbaypegas.livejournal.com',
 'sun--ai-livejournal-com.turbopages.org': 'sun-ai.livejournal.com',
 'dizzwizz-ru.turbopages.org': 'dizzwizz.ru',
 'iocube-livejournal-com.turbopages.org': 'iocube.livejournal.com',
 'antimantikora-livejournal-com.turbopages.org': 'antimantikora.livejournal.com',
 'artem--kashkanov-ru.turbopages.org': 'artem-kashkanov.ru',
 'sovkollekcioner-livejournal-com.turbopages.org': 'sovkollekcioner.livejournal.com',
 'nemologos-livejournal-com.turbopages.org': 'nemologos.livejournal.com',
 'narianmar-bezformata-com.turbopages.org': 'narianmar.bezformata.com',
 'mostovskoy-bezformata-com.turbopages.org': 'mostovskoy.bezformata.com',
 'gastrot-ru.turbopages.org': 'gastrot.ru',
 'ru--knitting-livejournal-com.turbopages.org': 'ru-knitting.livejournal.com',
 'gotanki-com.turbopages.org': 'gotanki.com',
 'wetfield-livejournal-com.turbopages.org': 'wetfield.livejournal.com',
 'marmara--calypso-livejournal-com.turbopages.org': 'marmara-calypso.livejournal.com',
 'balio-livejournal-com.turbopages.org': 'balio.livejournal.com',
 'kavkaz-rbc-ru.turbopages.org': 'kavkaz.rbc.ru',
 'ahabibti-livejournal-com.turbopages.org': 'ahabibti.livejournal.com',
 'playroom-ru.turbopages.org': 'playroom.ru',
 'yulf74-livejournal-com.turbopages.org': 'yulf74.livejournal.com',
 'amsterdamtravel-ru.turbopages.org': 'amsterdamtravel.ru',
 'sergei000-livejournal-com.turbopages.org': 'sergei000.livejournal.com',
 'pivokot-livejournal-com.turbopages.org': 'pivokot.livejournal.com',
 'mik--nik-livejournal-com.turbopages.org': 'mik-nik.livejournal.com',
 'bestpechi-ru.turbopages.org': 'bestpechi.ru',
 'rybinsk--once-ru.turbopages.org': 'rybinsk-once.ru',
 'iratn81-livejournal-com.turbopages.org': 'iratn81.livejournal.com',
 'my--niva-ru.turbopages.org': 'my-niva.ru',
 'koliadosik-livejournal-com.turbopages.org': 'koliadosik.livejournal.com',
 'a2--finance-com.turbopages.org': 'a2-finance.com',
 'ladyinfantahealth-ru.turbopages.org': 'ladyinfantahealth.ru',
 'stopvsd-net.turbopages.org': 'stopvsd.net',
 'english--city-ru.turbopages.org': 'english-city.ru',
 'homepitomec-ru.turbopages.org': 'homepitomec.ru',
 'oksador-livejournal-com.turbopages.org': 'oksador.livejournal.com',
 'belovdmitry-livejournal-com.turbopages.org': 'belovdmitry.livejournal.com',
 'vse--podmoskovje-livejournal-com.turbopages.org': 'vse-podmoskovje.livejournal.com',
 'imgsrc--ru-livejournal-com.turbopages.org': 'imgsrc-ru.livejournal.com',
 'brulant-ru.turbopages.org': 'brulant.ru',
 'syrmaepon-livejournal-com.turbopages.org': 'syrmaepon.livejournal.com',
 'gladkeeh-livejournal-com.turbopages.org': 'gladkeeh.livejournal.com',
 'ilyachalov-livejournal-com.turbopages.org': 'ilyachalov.livejournal.com',
 'alvimo-livejournal-com.turbopages.org': 'alvimo.livejournal.com',
 'easyneuro-livejournal-com.turbopages.org': 'easyneuro.livejournal.com',
 'greenners-livejournal-com.turbopages.org': 'greenners.livejournal.com',
 'sivilia--1-livejournal-com.turbopages.org': 'sivilia-1.livejournal.com',
 'dilijan-livejournal-com.turbopages.org': 'dilijan.livejournal.com',
 'xn----9sbgskednmwfwv.turbopages.org': 'xn--9sbgskednmwfwv',
 'aram--hakopian-livejournal-com.turbopages.org': 'aram-hakopian.livejournal.com',
 'mir--mak-livejournal-com.turbopages.org': 'mir-mak.livejournal.com',
 'kommynist-livejournal-com.turbopages.org': 'kommynist.livejournal.com',
 'ru--scooters-livejournal-com.turbopages.org': 'ru-scooters.livejournal.com',
 'acne911-ru.turbopages.org': 'acne911.ru',
 'kuban--swim-ru.turbopages.org': 'kuban-swim.ru',
 'krym-livejournal-com.turbopages.org': 'krym.livejournal.com',
 'igorsamusenko-livejournal-com.turbopages.org': 'igorsamusenko.livejournal.com',
 'best--design-livejournal-com.turbopages.org': 'best-design.livejournal.com',
 'l2ns-info.turbopages.org': 'l2ns.info',
 'englishforeducation-ru.turbopages.org': 'englishforeducation.ru',
 'unmonument-livejournal-com.turbopages.org': 'unmonument.livejournal.com',
 'mnenevlom-livejournal-com.turbopages.org': 'mnenevlom.livejournal.com',
 'avisi-ru.turbopages.org': 'avisi.ru',
 'crimeahistor-livejournal-com.turbopages.org': 'crimeahistor.livejournal.com',
 'babiy--bunt-ru.turbopages.org': 'babiy-bunt.ru',
 'badevlad-livejournal-com.turbopages.org': 'badevlad.livejournal.com',
 'gfaq-ru.turbopages.org': 'gfaq.ru',
 'sdelairukami-ru.turbopages.org': 'sdelairukami.ru',
 'chanych--85-livejournal-com.turbopages.org': 'chanych-85.livejournal.com',
 'deletant-livejournal-com.turbopages.org': 'deletant.livejournal.com',
 'tkaner-info.turbopages.org': 'tkaner.info',
 'ksenonia-livejournal-com.turbopages.org': 'ksenonia.livejournal.com',
 'sova74-ru.turbopages.org': 'sova74.ru',
 'lovlyavsem-ru.turbopages.org': 'lovlyavsem.ru',
 'ae--nekrasov-ru.turbopages.org': 'ae-nekrasov.ru',
 'rodgor--vlg-ru.turbopages.org': 'rodgor-vlg.ru',
 'serpompo2018-livejournal-com.turbopages.org': 'serpompo2018.livejournal.com',
 'ihistorian-livejournal-com.turbopages.org': 'ihistorian.livejournal.com',
 'zemfort1983-livejournal-com.turbopages.org': 'zemfort1983.livejournal.com',
 'licka--nicka-livejournal-com.turbopages.org': 'licka-nicka.livejournal.com',
 'ta--tu--sho-livejournal-com.turbopages.org': 'ta-tu-sho.livejournal.com',
 'pravdovanych-livejournal-com.turbopages.org': 'pravdovanych.livejournal.com',
 'megavolt--lab-livejournal-com.turbopages.org': 'megavolt-lab.livejournal.com',
 'ru--universe-livejournal-com.turbopages.org': 'ru-universe.livejournal.com',
 'warlen-livejournal-com.turbopages.org': 'warlen.livejournal.com',
 'appmaxx-com.turbopages.org': 'appmaxx.com',
 'vkdoc-livejournal-com.turbopages.org': 'vkdoc.livejournal.com',
 'fernandez--lj-livejournal-com.turbopages.org': 'fernandez-lj.livejournal.com',
 'foto--travel-net.turbopages.org': 'foto-travel.net',
 'nodima-livejournal-com.turbopages.org': 'nodima.livejournal.com',
 'andresol-livejournal-com.turbopages.org': 'andresol.livejournal.com',
 'materinstvo2-com.turbopages.org': 'materinstvo2.com',
 'russ--kosmos-livejournal-com.turbopages.org': 'russ-kosmos.livejournal.com',
 'pero--jarptizy-livejournal-com.turbopages.org': 'pero-jarptizy.livejournal.com',
 'theroco-com.turbopages.org': 'theroco.com',
 'alfafilatov-livejournal-com.turbopages.org': 'alfafilatov.livejournal.com',
 'garetty-livejournal-com.turbopages.org': 'garetty.livejournal.com',
 'dem--2011-livejournal-com.turbopages.org': 'dem-2011.livejournal.com',
 'glonasstravel-com.turbopages.org': 'glonasstravel.com',
 'yavelik-livejournal-com.turbopages.org': 'yavelik.livejournal.com',
 'world--sovet-info.turbopages.org': 'world-sovet.info',
 'ru--muslima-livejournal-com.turbopages.org': 'ru-muslima.livejournal.com',
 'cherepovec-bezformata-com.turbopages.org': 'cherepovec.bezformata.com',
 'yurkovskaya-livejournal-com.turbopages.org': 'yurkovskaya.livejournal.com',
 'medoboz-com.turbopages.org': 'medoboz.com',
 'rodaradi-ru.turbopages.org': 'rodaradi.ru',
 'tver-bezformata-com.turbopages.org': 'tver.bezformata.com',
 'serzigzagser-livejournal-com.turbopages.org': 'serzigzagser.livejournal.com',
 'coronavirus--hub-ru.turbopages.org': 'coronavirus-hub.ru',
 'yulianovsemen-livejournal-com.turbopages.org': 'yulianovsemen.livejournal.com',
 'samsebeadmin-ru.turbopages.org': 'samsebeadmin.ru',
 'hr-livejournal-com.turbopages.org': 'hr.livejournal.com',
 'syringa147-livejournal-com.turbopages.org': 'syringa147.livejournal.com',
 'denis--polozhaev-livejournal-com.turbopages.org': 'denis-polozhaev.livejournal.com',
 'glavbuhdudin-livejournal-com.turbopages.org': 'glavbuhdudin.livejournal.com',
 'ukraine--russia-livejournal-com.turbopages.org': 'ukraine-russia.livejournal.com',
 'realmoon-livejournal-com.turbopages.org': 'realmoon.livejournal.com',
 'mr--ulado-livejournal-com.turbopages.org': 'mr-ulado.livejournal.com',
 'wvtg-ru.turbopages.org': 'wvtg.ru',
 'chyvack-livejournal-com.turbopages.org': 'chyvack.livejournal.com',
 'otvet14-livejournal-com.turbopages.org': 'otvet14.livejournal.com',
 'politics--ru-livejournal-com.turbopages.org': 'politics-ru.livejournal.com',
 'visual--archive-livejournal-com.turbopages.org': 'visual-archive.livejournal.com',
 'strenger-livejournal-com.turbopages.org': 'strenger.livejournal.com',
 'wordpress--book-ru.turbopages.org': 'wordpress-book.ru',
 'tendr-guru.turbopages.org': 'tendr.guru',
 'truvor-livejournal-com.turbopages.org': 'truvor.livejournal.com',
 'krbank-ru.turbopages.org': 'krbank.ru',
 'vg147-livejournal-com.turbopages.org': 'vg147.livejournal.com',
 'sweet--aksenia-livejournal-com.turbopages.org': 'sweet-aksenia.livejournal.com',
 'magafurova-com.turbopages.org': 'magafurova.com',
 'mirvokryg-livejournal-com.turbopages.org': 'mirvokryg.livejournal.com',
 'if--i--were--rain-livejournal-com.turbopages.org': 'if-i-were-rain.livejournal.com',
 'xn----ctbshkfhsbdvu.turbopages.org': 'xn--ctbshkfhsbdvu',
 'weeleksa-livejournal-com.turbopages.org': 'weeleksa.livejournal.com',
 'theallexbekk-livejournal-com.turbopages.org': 'theallexbekk.livejournal.com',
 'polrecept-com.turbopages.org': 'polrecept.com',
 'designer--info-ru.turbopages.org': 'designer-info.ru',
 'lesnyanskiy-livejournal-com.turbopages.org': 'lesnyanskiy.livejournal.com',
 'elenahj-livejournal-com.turbopages.org': 'elenahj.livejournal.com',
 'kromanionez-livejournal-com.turbopages.org': 'kromanionez.livejournal.com',
 'darkelf--stalker-livejournal-com.turbopages.org': 'darkelf-stalker.livejournal.com',
 'obsudim--serialy-livejournal-com.turbopages.org': 'obsudim-serialy.livejournal.com',
 'ja--va-livejournal-com.turbopages.org': 'ja-va.livejournal.com',
 'pigeons--ru-livejournal-com.turbopages.org': 'pigeons-ru.livejournal.com',
 'lovewriter-livejournal-com.turbopages.org': 'lovewriter.livejournal.com',
 'nenazi--64-livejournal-com.turbopages.org': 'nenazi-64.livejournal.com',
 'ramenskoe-bezformata-com.turbopages.org': 'ramenskoe.bezformata.com',
 'dk-ru.turbopages.org': 'dk.ru',
 'klio123-livejournal-com.turbopages.org': 'klio123.livejournal.com',
 'holera--ham-livejournal-com.turbopages.org': 'holera-ham.livejournal.com',
 'prof--eug-livejournal-com.turbopages.org': 'prof-eug.livejournal.com',
 'map1983-livejournal-com.turbopages.org': 'map1983.livejournal.com',
 'eot--su-livejournal-com.turbopages.org': 'eot-su.livejournal.com',
 'vihlyn-livejournal-com.turbopages.org': 'vihlyn.livejournal.com',
 'marxxm-livejournal-com.turbopages.org': 'marxxm.livejournal.com',
 'politcreative-livejournal-com.turbopages.org': 'politcreative.livejournal.com',
 'molonlabe-livejournal-com.turbopages.org': 'molonlabe.livejournal.com',
 'manon--gabrielle-livejournal-com.turbopages.org': 'manon-gabrielle.livejournal.com',
 'wrc--info-ru.turbopages.org': 'wrc-info.ru',
 'budtezdorovy-net.turbopages.org': 'budtezdorovy.net',
 'hubrating-com.turbopages.org': 'hubrating.com',
 'harly--c-livejournal-com.turbopages.org': 'harly-c.livejournal.com',
 'maksim--kot-livejournal-com.turbopages.org': 'maksim-kot.livejournal.com',
 'oglaze-livejournal-com.turbopages.org': 'oglaze.livejournal.com',
 'timgud-livejournal-com.turbopages.org': 'timgud.livejournal.com',
 'anna--zhulidova-livejournal-com.turbopages.org': 'anna-zhulidova.livejournal.com',
 'imja-livejournal-com.turbopages.org': 'imja.livejournal.com',
 'jazzstone-ru.turbopages.org': 'jazzstone.ru',
 '2gusia-livejournal-com.turbopages.org': '2gusia.livejournal.com',
 'nikitayev-livejournal-com.turbopages.org': 'nikitayev.livejournal.com',
 'fotospalni-ru.turbopages.org': 'fotospalni.ru',
 'k--assir-livejournal-com.turbopages.org': 'k-assir.livejournal.com',
 'ben--landa-livejournal-com.turbopages.org': 'ben-landa.livejournal.com',
 'motti-ru.turbopages.org': 'motti.ru',
 'sv--makovkina-livejournal-com.turbopages.org': 'sv-makovkina.livejournal.com',
 'brodv-ru.turbopages.org': 'brodv.ru',
 'saali-livejournal-com.turbopages.org': 'saali.livejournal.com',
 'netcellulita-ru.turbopages.org': 'netcellulita.ru',
 'ldpr--elets-livejournal-com.turbopages.org': 'ldpr-elets.livejournal.com',
 'domik--sroy-ru.turbopages.org': 'domik-sroy.ru',
 'petrovsk-bezformata-com.turbopages.org': 'petrovsk.bezformata.com',
 'hallas-livejournal-com.turbopages.org': 'hallas.livejournal.com',
 'lumor-livejournal-com.turbopages.org': 'lumor.livejournal.com',
 'turandot5-livejournal-com.turbopages.org': 'turandot5.livejournal.com',
 'nefertiti--khv-ru.turbopages.org': 'nefertiti-khv.ru',
 's--yaroslav-livejournal-com.turbopages.org': 's-yaroslav.livejournal.com',
 'tichy-livejournal-com.turbopages.org': 'tichy.livejournal.com',
 'overtonwindow-livejournal-com.turbopages.org': 'overtonwindow.livejournal.com',
 'batya27-livejournal-com.turbopages.org': 'batya27.livejournal.com',
 'gbhfn-livejournal-com.turbopages.org': 'gbhfn.livejournal.com',
 'newsmaker--su-livejournal-com.turbopages.org': 'newsmaker-su.livejournal.com',
 'floor--mice-livejournal-com.turbopages.org': 'floor-mice.livejournal.com',
 'vodnik--ru-livejournal-com.turbopages.org': 'vodnik-ru.livejournal.com',
 'ru--gnostic-livejournal-com.turbopages.org': 'ru-gnostic.livejournal.com',
 'fryazino-bezformata-com.turbopages.org': 'fryazino.bezformata.com',
 'voutsen--cv-livejournal-com.turbopages.org': 'voutsen-cv.livejournal.com',
 'varka-livejournal-com.turbopages.org': 'varka.livejournal.com',
 'david--mendel-livejournal-com.turbopages.org': 'david-mendel.livejournal.com',
 'zhonga-ru.turbopages.org': 'zhonga.ru',
 'mikul--a-livejournal-com.turbopages.org': 'mikul-a.livejournal.com',
 'potionsquirrel-ru.turbopages.org': 'potionsquirrel.ru',
 'crusandr-livejournal-com.turbopages.org': 'crusandr.livejournal.com',
 '9231930-livejournal-com.turbopages.org': '9231930.livejournal.com',
 'ir--maverick-livejournal-com.turbopages.org': 'ir-maverick.livejournal.com',
 'moryakukrainy-livejournal-com.turbopages.org': 'moryakukrainy.livejournal.com',
 'kavkaz-versia-ru.turbopages.org': 'kavkaz.versia.ru',
 'mensstyl-com.turbopages.org': 'mensstyl.com',
 'euroskoda-ru.turbopages.org': 'euroskoda.ru',
 'dorama--ru-livejournal-com.turbopages.org': 'dorama-ru.livejournal.com',
 'valeriy--nsk-livejournal-com.turbopages.org': 'valeriy-nsk.livejournal.com',
 'shegolll-livejournal-com.turbopages.org': 'shegolll.livejournal.com',
 'privratnik-livejournal-com.turbopages.org': 'privratnik.livejournal.com',
 'videohit-info.turbopages.org': 'videohit.info',
 'karabai96-livejournal-com.turbopages.org': 'karabai96.livejournal.com',
 'nemankurt-livejournal-com.turbopages.org': 'nemankurt.livejournal.com',
 'rostov--na--donu-big--book--med-ru.turbopages.org': 'rostov-na-donu.big-book-med.ru',
 'aleks--rum-livejournal-com.turbopages.org': 'aleks-rum.livejournal.com',
 'galich89-livejournal-com.turbopages.org': 'galich89.livejournal.com',
 'mr--pw-livejournal-com.turbopages.org': 'mr-pw.livejournal.com',
 'snorska-livejournal-com.turbopages.org': 'snorska.livejournal.com',
 'svetalar-livejournal-com.turbopages.org': 'svetalar.livejournal.com',
 'palborum-livejournal-com.turbopages.org': 'palborum.livejournal.com',
 'art--of--arts-livejournal-com.turbopages.org': 'art-of-arts.livejournal.com',
 'ziryaell-livejournal-com.turbopages.org': 'ziryaell.livejournal.com',
 'studialarouge-ru.turbopages.org': 'studialarouge.ru',
 'sartana-livejournal-com.turbopages.org': 'sartana.livejournal.com',
 'vsekolesa-ru.turbopages.org': 'vsekolesa.ru',
 'mil--history-livejournal-com.turbopages.org': 'mil-history.livejournal.com',
 'gelio-livejournal-com.turbopages.org': 'gelio.livejournal.com',
 'inschooler-ru.turbopages.org': 'inschooler.ru',
 'otyrar-kz.turbopages.org': 'otyrar.kz',
 'amigo54-livejournal-com.turbopages.org': 'amigo54.livejournal.com',
 'maratguelman-livejournal-com.turbopages.org': 'maratguelman.livejournal.com',
 'moykariver-livejournal-com.turbopages.org': 'moykariver.livejournal.com',
 'woblog-ru.turbopages.org': 'woblog.ru',
 'zet09-livejournal-com.turbopages.org': 'zet09.livejournal.com',
 'arm--wanderer-livejournal-com.turbopages.org': 'arm-wanderer.livejournal.com',
 'planeta--tyva-livejournal-com.turbopages.org': 'planeta-tyva.livejournal.com',
 'gazetavechorka-ru.turbopages.org': 'gazetavechorka.ru',
 'gudkov--gennadij-livejournal-com.turbopages.org': 'gudkov-gennadij.livejournal.com',
 'urhelp-guru.turbopages.org': 'urhelp.guru',
 'shatilin-livejournal-com.turbopages.org': 'shatilin.livejournal.com',
 'newbabe-ru.turbopages.org': 'newbabe.ru',
 'oktyabrskiy-bezformata-com.turbopages.org': 'oktyabrskiy.bezformata.com',
 'vishwananda-livejournal-com.turbopages.org': 'vishwananda.livejournal.com',
 'projivot-ru.turbopages.org': 'projivot.ru',
 'like--rus-ru.turbopages.org': 'like-rus.ru',
 'aleonkin-livejournal-com.turbopages.org': 'aleonkin.livejournal.com',
 'monkeymella-livejournal-com.turbopages.org': 'monkeymella.livejournal.com',
 'faktypro-ru.turbopages.org': 'faktypro.ru',
 'military--museum-livejournal-com.turbopages.org': 'military-museum.livejournal.com',
 'drunkhunter1-livejournal-com.turbopages.org': 'drunkhunter1.livejournal.com',
 'alex--emilsson-livejournal-com.turbopages.org': 'alex-emilsson.livejournal.com',
 'verhnehavskiy-livejournal-com.turbopages.org': 'verhnehavskiy.livejournal.com',
 'voloskam-net.turbopages.org': 'voloskam.net',
 'dollmagic-ru.turbopages.org': 'dollmagic.ru',
 'herbalinfo-ru.turbopages.org': 'herbalinfo.ru',
 'fajnafajna-livejournal-com.turbopages.org': 'fajnafajna.livejournal.com',
 'ktotak-ru.turbopages.org': 'ktotak.ru',
 '2020--years-ru.turbopages.org': '2020-years.ru',
 'prometei--lj-livejournal-com.turbopages.org': 'prometei-lj.livejournal.com',
 'pozitiv-guru.turbopages.org': 'pozitiv.guru',
 'msk--sokol-livejournal-com.turbopages.org': 'msk-sokol.livejournal.com',
 'expertus-su.turbopages.org': 'expertus.su',
 'inva-news.turbopages.org': 'inva.news',
 'careerist-ru.turbopages.org': 'careerist.ru',
 'zglaza-net.turbopages.org': 'zglaza.net',
 'baiyt-livejournal-com.turbopages.org': 'baiyt.livejournal.com',
 'malinova--marina-livejournal-com.turbopages.org': 'malinova-marina.livejournal.com',
 'mosvodokanal-livejournal-com.turbopages.org': 'mosvodokanal.livejournal.com',
 'netolink-ru.turbopages.org': 'netolink.ru',
 'werwol-livejournal-com.turbopages.org': 'werwol.livejournal.com',
 'the--geek-ru.turbopages.org': 'the-geek.ru',
 'giocatore--spb-livejournal-com.turbopages.org': 'giocatore-spb.livejournal.com',
 'vseslavd-livejournal-com.turbopages.org': 'vseslavd.livejournal.com',
 'antitrole-livejournal-com.turbopages.org': 'antitrole.livejournal.com',
 'moyaboroda-ru.turbopages.org': 'moyaboroda.ru',
 'oktyabrskoepole-livejournal-com.turbopages.org': 'oktyabrskoepole.livejournal.com',
 'dogmasya-ru.turbopages.org': 'dogmasya.ru',
 'mitsu--market-ru.turbopages.org': 'mitsu-market.ru',
 'artofwars-livejournal-com.turbopages.org': 'artofwars.livejournal.com',
 'ferma-guru.turbopages.org': 'ferma.guru',
 'donskieogni-ru.turbopages.org': 'donskieogni.ru',
 'atomadog-livejournal-com.turbopages.org': 'atomadog.livejournal.com',
 'gorod--n-livejournal-com.turbopages.org': 'gorod-n.livejournal.com',
 'm--athanasios-livejournal-com.turbopages.org': 'm-athanasios.livejournal.com',
 'nacburo-livejournal-com.turbopages.org': 'nacburo.livejournal.com',
 'edwinbell-livejournal-com.turbopages.org': 'edwinbell.livejournal.com',
 'splar-ru.turbopages.org': 'splar.ru',
 'aaz1z-livejournal-com.turbopages.org': 'aaz1z.livejournal.com',
 'kaankerede-livejournal-com.turbopages.org': 'kaankerede.livejournal.com',
 'krisha0703-livejournal-com.turbopages.org': 'krisha0703.livejournal.com',
 'vol--majya-livejournal-com.turbopages.org': 'vol-majya.livejournal.com',
 'myhyundaicreta-ru.turbopages.org': 'myhyundaicreta.ru',
 'fanfanews-livejournal-com.turbopages.org': 'fanfanews.livejournal.com',
 'imbaaa-livejournal-com.turbopages.org': 'imbaaa.livejournal.com',
 'olga--moses-livejournal-com.turbopages.org': 'olga-moses.livejournal.com',
 'xn--24--5cdexb2cff1cg9b0g.turbopages.org': 'xn-24-5cdexb2cff1cg9b0g',
 'baltvilks-livejournal-com.turbopages.org': 'baltvilks.livejournal.com',
 'lutik--12-livejournal-com.turbopages.org': 'lutik-12.livejournal.com',
 '0lesja--0lgerd-livejournal-com.turbopages.org': '0lesja-0lgerd.livejournal.com',
 'evgenyart-livejournal-com.turbopages.org': 'evgenyart.livejournal.com',
 'sergbrezhnev-livejournal-com.turbopages.org': 'sergbrezhnev.livejournal.com',
 'modmakeup-ru.turbopages.org': 'modmakeup.ru',
 'urasinova-livejournal-com.turbopages.org': 'urasinova.livejournal.com',
 'fni--talya-livejournal-com.turbopages.org': 'fni-talya.livejournal.com',
 'museum--tarhany-livejournal-com.turbopages.org': 'museum-tarhany.livejournal.com',
 'michel--recamier-livejournal-com.turbopages.org': 'michel-recamier.livejournal.com',
 'magas--dedyakov-livejournal-com.turbopages.org': 'magas-dedyakov.livejournal.com',
 'veronasunrise-livejournal-com.turbopages.org': 'veronasunrise.livejournal.com',
 'ponchik--k-livejournal-com.turbopages.org': 'ponchik-k.livejournal.com',
 'all--arduino-ru.turbopages.org': 'all-arduino.ru',
 'trunk--2007-livejournal-com.turbopages.org': 'trunk-2007.livejournal.com',
 'aquaformula-ru.turbopages.org': 'aquaformula.ru',
 'labequip-ru.turbopages.org': 'labequip.ru',
 'galina--kayumova-livejournal-com.turbopages.org': 'galina-kayumova.livejournal.com',
 'queen--victory-livejournal-com.turbopages.org': 'queen-victory.livejournal.com',
 'roksolana2010-livejournal-com.turbopages.org': 'roksolana2010.livejournal.com',
 'hithlin-livejournal-com.turbopages.org': 'hithlin.livejournal.com',
 'dmitrij--sergeev-livejournal-com.turbopages.org': 'dmitrij-sergeev.livejournal.com',
 'aleksid-livejournal-com.turbopages.org': 'aleksid.livejournal.com',
 'soborianelj-livejournal-com.turbopages.org': 'soborianelj.livejournal.com',
 'igornasa-livejournal-com.turbopages.org': 'igornasa.livejournal.com',
 'ru--gnostik-livejournal-com.turbopages.org': 'ru-gnostik.livejournal.com',
 'alexeytretyakov-livejournal-com.turbopages.org': 'alexeytretyakov.livejournal.com',
 'zoritabasarana-ru.turbopages.org': 'zoritabasarana.ru',
 'simon--vit-livejournal-com.turbopages.org': 'simon-vit.livejournal.com',
 'asmokeblog-livejournal-com.turbopages.org': 'asmokeblog.livejournal.com',
 'cryptopolicy-livejournal-com.turbopages.org': 'cryptopolicy.livejournal.com',
 'outside--flo-livejournal-com.turbopages.org': 'outside-flo.livejournal.com',
 'vintage--dream--s-livejournal-com.turbopages.org': 'vintage-dream-s.livejournal.com',
 'norg--norg-livejournal-com.turbopages.org': 'norg-norg.livejournal.com',
 'vitaminy-expert.turbopages.org': 'vitaminy.expert',
 'monsterface-ru.turbopages.org': 'monsterface.ru',
 'denalt-livejournal-com.turbopages.org': 'denalt.livejournal.com',
 'nafnafa-livejournal-com.turbopages.org': 'nafnafa.livejournal.com',
 'little--shoes-livejournal-com.turbopages.org': 'little-shoes.livejournal.com',
 'english--art-livejournal-com.turbopages.org': 'english-art.livejournal.com',
 'napricedala-ru.turbopages.org': 'napricedala.ru',
 'glavtema-ru.turbopages.org': 'glavtema.ru',
 'prooverwatch-ru.turbopages.org': 'prooverwatch.ru',
 'krasnouhov-livejournal-com.turbopages.org': 'krasnouhov.livejournal.com',
 'spb--spotting-livejournal-com.turbopages.org': 'spb-spotting.livejournal.com',
 'snatalya-livejournal-com.turbopages.org': 'snatalya.livejournal.com',
 'chest--i--razym-livejournal-com.turbopages.org': 'chest-i-razym.livejournal.com',
 'incopolis-livejournal-com.turbopages.org': 'incopolis.livejournal.com',
 'korkin--dreams-livejournal-com.turbopages.org': 'korkin-dreams.livejournal.com',
 'natalija--khaf-livejournal-com.turbopages.org': 'natalija-khaf.livejournal.com',
 'eysk-bezformata-com.turbopages.org': 'eysk.bezformata.com',
 'pirs--marinplast-livejournal-com.turbopages.org': 'pirs-marinplast.livejournal.com',
 'nuancesprog-ru.turbopages.org': 'nuancesprog.ru',
 'irpenin-livejournal-com.turbopages.org': 'irpenin.livejournal.com',
 'rndonline-ru.turbopages.org': 'rndonline.ru',
 'nevinnomissk-bezformata-com.turbopages.org': 'nevinnomissk.bezformata.com',
 'mariel--98-livejournal-com.turbopages.org': 'mariel-98.livejournal.com',
 'lezhnev--sergey-livejournal-com.turbopages.org': 'lezhnev-sergey.livejournal.com',
 'survivalpanda-livejournal-com.turbopages.org': 'survivalpanda.livejournal.com',
 'art--expiration-livejournal-com.turbopages.org': 'art-expiration.livejournal.com',
 'kayri1966-livejournal-com.turbopages.org': 'kayri1966.livejournal.com',
 'allscitech-ru.turbopages.org': 'allscitech.ru',
 'prosto--dimka-livejournal-com.turbopages.org': 'prosto-dimka.livejournal.com',
 'pikalevo-bezformata-com.turbopages.org': 'pikalevo.bezformata.com',
 'azimutlife-ru.turbopages.org': 'azimutlife.ru',
 'convaliya-livejournal-com.turbopages.org': 'convaliya.livejournal.com',
 'elshin-livejournal-com.turbopages.org': 'elshin.livejournal.com',
 'giovinco-livejournal-com.turbopages.org': 'giovinco.livejournal.com',
 'ivagkin-livejournal-com.turbopages.org': 'ivagkin.livejournal.com',
 'yurymif-livejournal-com.turbopages.org': 'yurymif.livejournal.com',
 'kompanion-online.turbopages.org': 'kompanion.online',
 'artmarshals-livejournal-com.turbopages.org': 'artmarshals.livejournal.com',
 'oblachko75-livejournal-com.turbopages.org': 'oblachko75.livejournal.com',
 'ort7000-livejournal-com.turbopages.org': 'ort7000.livejournal.com',
 'metalchemist-livejournal-com.turbopages.org': 'metalchemist.livejournal.com',
 'fossegrim--july-livejournal-com.turbopages.org': 'fossegrim-july.livejournal.com',
 'askepak-livejournal-com.turbopages.org': 'askepak.livejournal.com',
 'simula--cra-livejournal-com.turbopages.org': 'simula-cra.livejournal.com',
 'tsharenko-livejournal-com.turbopages.org': 'tsharenko.livejournal.com',
 'ohotzaimka-ru.turbopages.org': 'ohotzaimka.ru',
 'sunely--tales-livejournal-com.turbopages.org': 'sunely-tales.livejournal.com',
 'telemax--spb-livejournal-com.turbopages.org': 'telemax-spb.livejournal.com',
 'navsifoy-livejournal-com.turbopages.org': 'navsifoy.livejournal.com',
 'telcode-su.turbopages.org': 'telcode.su',
 'etno--seminar-livejournal-com.turbopages.org': 'etno-seminar.livejournal.com',
 'grazdanstvo--rf-ru.turbopages.org': 'grazdanstvo-rf.ru',
 'maxfisher79-livejournal-com.turbopages.org': 'maxfisher79.livejournal.com',
 'nl--int-ru.turbopages.org': 'nl-int.ru',
 'houseussr-ru.turbopages.org': 'houseussr.ru',
 'tobolsk-bezformata-com.turbopages.org': 'tobolsk.bezformata.com',
 'silverlj-livejournal-com.turbopages.org': 'silverlj.livejournal.com',
 'apple--rep-ru.turbopages.org': 'apple-rep.ru',
 'amedeya-com.turbopages.org': 'amedeya.com',
 'allexell-livejournal-com.turbopages.org': 'allexell.livejournal.com',
 'milana--new-livejournal-com.turbopages.org': 'milana-new.livejournal.com',
 'maybe--s0--true-livejournal-com.turbopages.org': 'maybe-s0-true.livejournal.com',
 'luanda--ru-livejournal-com.turbopages.org': 'luanda-ru.livejournal.com',
 'zarava-livejournal-com.turbopages.org': 'zarava.livejournal.com',
 'being--fox-livejournal-com.turbopages.org': 'being-fox.livejournal.com',
 'idelbaev--artur-livejournal-com.turbopages.org': 'idelbaev-artur.livejournal.com',
 'obninsk-bezformata-com.turbopages.org': 'obninsk.bezformata.com',
 'pan--dohva-livejournal-com.turbopages.org': 'pan-dohva.livejournal.com',
 'sovetip-ru.turbopages.org': 'sovetip.ru',
 'sonikka--s-livejournal-com.turbopages.org': 'sonikka-s.livejournal.com',
 'i--am--a--programmer-ru.turbopages.org': 'i-am-a-programmer.ru',
 'liorelin-livejournal-com.turbopages.org': 'liorelin.livejournal.com',
 'levashove-livejournal-com.turbopages.org': 'levashove.livejournal.com',
 'veronika--stef-livejournal-com.turbopages.org': 'veronika-stef.livejournal.com',
 'olga--arefieva-livejournal-com.turbopages.org': 'olga-arefieva.livejournal.com',
 'spicesguide-ru.turbopages.org': 'spicesguide.ru',
 'doll--art--bjd-livejournal-com.turbopages.org': 'doll-art-bjd.livejournal.com',
 'inna--frank-livejournal-com.turbopages.org': 'inna-frank.livejournal.com',
 'annacheft-livejournal-com.turbopages.org': 'annacheft.livejournal.com',
 'compromat--amur-livejournal-com.turbopages.org': 'compromat-amur.livejournal.com',
 'm--simonyan-livejournal-com.turbopages.org': 'm-simonyan.livejournal.com',
 'strejevoy-bezformata-com.turbopages.org': 'strejevoy.bezformata.com',
 'greenstartpoint-ru.turbopages.org': 'greenstartpoint.ru',
 'gelena--eva-livejournal-com.turbopages.org': 'gelena-eva.livejournal.com',
 'poruchik--sk-livejournal-com.turbopages.org': 'poruchik-sk.livejournal.com',
 'mylivingfood-ru.turbopages.org': 'mylivingfood.ru',
 'ufa--nagaevo-ru.turbopages.org': 'ufa-nagaevo.ru',
 'chelovek888-livejournal-com.turbopages.org': 'chelovek888.livejournal.com',
 'kinocitatnik-ru.turbopages.org': 'kinocitatnik.ru',
 'daralian-livejournal-com.turbopages.org': 'daralian.livejournal.com',
 'vergoti-livejournal-com.turbopages.org': 'vergoti.livejournal.com',
 'nightnat-livejournal-com.turbopages.org': 'nightnat.livejournal.com',
 'lit--street-livejournal-com.turbopages.org': 'lit-street.livejournal.com',
 'kalinchevse-livejournal-com.turbopages.org': 'kalinchevse.livejournal.com',
 'bagee-livejournal-com.turbopages.org': 'bagee.livejournal.com',
 'earthy--cat-livejournal-com.turbopages.org': 'earthy-cat.livejournal.com',
 'albinos76-livejournal-com.turbopages.org': 'albinos76.livejournal.com',
 'che--telcontar-livejournal-com.turbopages.org': 'che-telcontar.livejournal.com',
 'milowa-livejournal-com.turbopages.org': 'milowa.livejournal.com',
 'my--question-ru.turbopages.org': 'my-question.ru',
 'kontora-pro.turbopages.org': 'kontora.pro',
 'ort--palomnik-livejournal-com.turbopages.org': 'ort-palomnik.livejournal.com',
 'grandfart-ru.turbopages.org': 'grandfart.ru',
 'nikolay--istomin-livejournal-com.turbopages.org': 'nikolay-istomin.livejournal.com',
 'bestposter-livejournal-com.turbopages.org': 'bestposter.livejournal.com',
 'compone-ru.turbopages.org': 'compone.ru',
 'astapova--v-livejournal-com.turbopages.org': 'astapova-v.livejournal.com',
 'komitet17-livejournal-com.turbopages.org': 'komitet17.livejournal.com',
 'kamajuki-livejournal-com.turbopages.org': 'kamajuki.livejournal.com',
 '1957--anti-livejournal-com.turbopages.org': '1957-anti.livejournal.com',
 'tatjanagord-livejournal-com.turbopages.org': 'tatjanagord.livejournal.com',
 'alpinistnew-livejournal-com.turbopages.org': 'alpinistnew.livejournal.com',
 'oftalmologiya-info.turbopages.org': 'oftalmologiya.info',
 'unlimmobile-livejournal-com.turbopages.org': 'unlimmobile.livejournal.com',
 'mestniy--bozhok-livejournal-com.turbopages.org': 'mestniy-bozhok.livejournal.com',
 'si74-ru.turbopages.org': 'si74.ru',
 'gvardei-livejournal-com.turbopages.org': 'gvardei.livejournal.com',
 'imago--666-livejournal-com.turbopages.org': 'imago-666.livejournal.com',
 'travelfoxes-livejournal-com.turbopages.org': 'travelfoxes.livejournal.com',
 'vit1-ru.turbopages.org': 'vit1.ru',
 'i--usa-livejournal-com.turbopages.org': 'i-usa.livejournal.com',
 'balabin--1712-livejournal-com.turbopages.org': 'balabin-1712.livejournal.com',
 'shienok-livejournal-com.turbopages.org': 'shienok.livejournal.com',
 'm--verigina-livejournal-com.turbopages.org': 'm-verigina.livejournal.com',
 'did--u--know-livejournal-com.turbopages.org': 'did-u-know.livejournal.com',
 'alexsolovyev-livejournal-com.turbopages.org': 'alexsolovyev.livejournal.com',
 'rodoslava-livejournal-com.turbopages.org': 'rodoslava.livejournal.com',
 'gametech-ru.turbopages.org': 'gametech.ru',
 'teplicainfo-ru.turbopages.org': 'teplicainfo.ru',
 'yul--chik-livejournal-com.turbopages.org': 'yul-chik.livejournal.com',
 'yaharu-livejournal-com.turbopages.org': 'yaharu.livejournal.com',
 'vzube-com.turbopages.org': 'vzube.com',
 'spk--kovka-ru.turbopages.org': 'spk-kovka.ru',
 'a--ukraina-livejournal-com.turbopages.org': 'a-ukraina.livejournal.com',
 'grannushka-livejournal-com.turbopages.org': 'grannushka.livejournal.com',
 'donish-su.turbopages.org': 'donish.su',
 'mypets-by.turbopages.org': 'mypets.by',
 'onreal-livejournal-com.turbopages.org': 'onreal.livejournal.com',
 'maxfux-livejournal-com.turbopages.org': 'maxfux.livejournal.com',
 'antushka-ru.turbopages.org': 'antushka.ru',
 'vinoteka--in--ua-livejournal-com.turbopages.org': 'vinoteka-in-ua.livejournal.com',
 '5elementum-ru.turbopages.org': '5elementum.ru',
 'regane-livejournal-com.turbopages.org': 'regane.livejournal.com',
 'jobims-livejournal-com.turbopages.org': 'jobims.livejournal.com',
 'optimusmedicus-com.turbopages.org': 'optimusmedicus.com',
 'strelka--detector-ru.turbopages.org': 'strelka-detector.ru',
 'writing--cures-livejournal-com.turbopages.org': 'writing-cures.livejournal.com',
 'pensionnij--fond-livejournal-com.turbopages.org': 'pensionnij-fond.livejournal.com',
 'fonzeppelin-livejournal-com.turbopages.org': 'fonzeppelin.livejournal.com',
 'vasilyev--eklips-livejournal-com.turbopages.org': 'vasilyev-eklips.livejournal.com',
 'amico--di--amici-livejournal-com.turbopages.org': 'amico-di-amici.livejournal.com',
 'akirich--pcroom-livejournal-com.turbopages.org': 'akirich-pcroom.livejournal.com',
 'hr--metod-com.turbopages.org': 'hr-metod.com',
 'rus--koldovstvo-livejournal-com.turbopages.org': 'rus-koldovstvo.livejournal.com',
 'odin--moy--den-livejournal-com.turbopages.org': 'odin-moy-den.livejournal.com',
 'navalnenok-livejournal-com.turbopages.org': 'navalnenok.livejournal.com',
 'drozdl-livejournal-com.turbopages.org': 'drozdl.livejournal.com',
 'pochke-ru.turbopages.org': 'pochke.ru',
 'prokopevsk-bezformata-com.turbopages.org': 'prokopevsk.bezformata.com',
 '1hello-ru.turbopages.org': '1hello.ru',
 'antonkachinskiy-livejournal-com.turbopages.org': 'antonkachinskiy.livejournal.com',
 'marenzhi-livejournal-com.turbopages.org': 'marenzhi.livejournal.com',
 'vlatisa-livejournal-com.turbopages.org': 'vlatisa.livejournal.com',
 'dima--chatrov-livejournal-com.turbopages.org': 'dima-chatrov.livejournal.com',
 'observet-livejournal-com.turbopages.org': 'observet.livejournal.com',
 'hanmooye-livejournal-com.turbopages.org': 'hanmooye.livejournal.com',
 'titov--serge-livejournal-com.turbopages.org': 'titov-serge.livejournal.com',
 'bilboleggins-livejournal-com.turbopages.org': 'bilboleggins.livejournal.com',
 'silavitamina-ru.turbopages.org': 'silavitamina.ru',
 'pro--spain-net.turbopages.org': 'pro-spain.net',
 'necro--tv-livejournal-com.turbopages.org': 'necro-tv.livejournal.com',
 'svolod--gm-livejournal-com.turbopages.org': 'svolod-gm.livejournal.com',
 'a--bakulin-livejournal-com.turbopages.org': 'a-bakulin.livejournal.com',
 'stroypraym-ru.turbopages.org': 'stroypraym.ru',
 'mojga-bezformata-com.turbopages.org': 'mojga.bezformata.com',
 'xn------5cdybacwbeskal3duin0d9bj.turbopages.org': 'xn---5cdybacwbeskal3duin0d9bj',
 'chihayuschiy-livejournal-com.turbopages.org': 'chihayuschiy.livejournal.com',
 'trenos-livejournal-com.turbopages.org': 'trenos.livejournal.com',
 'newsmart-livejournal-com.turbopages.org': 'newsmart.livejournal.com',
 'stetson--hat-livejournal-com.turbopages.org': 'stetson-hat.livejournal.com',
 'blog--acumed-kz.turbopages.org': 'blog-acumed.kz',
 'volandband-livejournal-com.turbopages.org': 'volandband.livejournal.com',
 'gorodkovrov-ru.turbopages.org': 'gorodkovrov.ru',
 'tur--katalog-ru.turbopages.org': 'tur-katalog.ru',
 'irga101-livejournal-com.turbopages.org': 'irga101.livejournal.com',
 'seovpmr-ru.turbopages.org': 'seovpmr.ru',
 'anthony--bf-livejournal-com.turbopages.org': 'anthony-bf.livejournal.com',
 'karachev-bezformata-com.turbopages.org': 'karachev.bezformata.com',
 'wanderv-livejournal-com.turbopages.org': 'wanderv.livejournal.com',
 'kerrangjke-livejournal-com.turbopages.org': 'kerrangjke.livejournal.com',
 'e-gazeta--unp-ru.turbopages.org': 'e.gazeta-unp.ru',
 'intuit--school-livejournal-com.turbopages.org': 'intuit-school.livejournal.com',
 'wla--dimir-livejournal-com.turbopages.org': 'wla-dimir.livejournal.com',
 'balashiha--ru-livejournal-com.turbopages.org': 'balashiha-ru.livejournal.com',
 'zoopriut-ru.turbopages.org': 'zoopriut.ru',
 'timemechanic-livejournal-com.turbopages.org': 'timemechanic.livejournal.com',
 'aboutrentgen-ru.turbopages.org': 'aboutrentgen.ru',
 'ya--opisalas-livejournal-com.turbopages.org': 'ya-opisalas.livejournal.com',
 'resheniya--sudov-ru.turbopages.org': 'resheniya-sudov.ru',
 'vybor--prost-ru.turbopages.org': 'vybor-prost.ru',
 'germanygid-ru.turbopages.org': 'germanygid.ru',
 'damoklov-livejournal-com.turbopages.org': 'damoklov.livejournal.com',
 'oboznik-ru.turbopages.org': 'oboznik.ru',
 'anekdot-livejournal-com.turbopages.org': 'anekdot.livejournal.com',
 'gur--095-livejournal-com.turbopages.org': 'gur-095.livejournal.com',
 'krasavchik-livejournal-com.turbopages.org': 'krasavchik.livejournal.com',
 'prmarina-livejournal-com.turbopages.org': 'prmarina.livejournal.com',
 'vsiakie--igri-livejournal-com.turbopages.org': 'vsiakie-igri.livejournal.com',
 'civilizasian-livejournal-com.turbopages.org': 'civilizasian.livejournal.com',
 '5rochka-ru.turbopages.org': '5rochka.ru',
 'sam--sdelay-ru.turbopages.org': 'sam-sdelay.ru',
 'kubirubi-livejournal-com.turbopages.org': 'kubirubi.livejournal.com',
 'nyamnyamych-ru.turbopages.org': 'nyamnyamych.ru',
 'zydog-livejournal-com.turbopages.org': 'zydog.livejournal.com',
 'daria--zarina-livejournal-com.turbopages.org': 'daria-zarina.livejournal.com',
 'kuda--edem-livejournal-com.turbopages.org': 'kuda-edem.livejournal.com',
 'xn------7cdal2be1adwpng4d.turbopages.org': 'xn---7cdal2be1adwpng4d',
 'eis--gen-livejournal-com.turbopages.org': 'eis-gen.livejournal.com',
 'my--planeta-livejournal-com.turbopages.org': 'my-planeta.livejournal.com',
 'nickmoshkov-livejournal-com.turbopages.org': 'nickmoshkov.livejournal.com',
 'selhoz-guru.turbopages.org': 'selhoz.guru',
 'setiwik-ru.turbopages.org': 'setiwik.ru',
 'galery--m-ru.turbopages.org': 'galery-m.ru',
 'jgtexport-ru.turbopages.org': 'jgtexport.ru',
 'microft-livejournal-com.turbopages.org': 'microft.livejournal.com',
 'andreyvadjra-livejournal-com.turbopages.org': 'andreyvadjra.livejournal.com',
 'smoliarm-livejournal-com.turbopages.org': 'smoliarm.livejournal.com',
 'zdrager-livejournal-com.turbopages.org': 'zdrager.livejournal.com',
 'beauty--spirit-livejournal-com.turbopages.org': 'beauty-spirit.livejournal.com',
 'brandexpert-livejournal-com.turbopages.org': 'brandexpert.livejournal.com',
 'karavan--plus-livejournal-com.turbopages.org': 'karavan-plus.livejournal.com',
 's--savina-livejournal-com.turbopages.org': 's-savina.livejournal.com',
 'sober--deer-livejournal-com.turbopages.org': 'sober-deer.livejournal.com',
 'ru--psicholog-livejournal-com.turbopages.org': 'ru-psicholog.livejournal.com',
 'anna--nik0laeva-livejournal-com.turbopages.org': 'anna-nik0laeva.livejournal.com',
 'eot--leningrad-livejournal-com.turbopages.org': 'eot-leningrad.livejournal.com',
 'risova-livejournal-com.turbopages.org': 'risova.livejournal.com',
 'so73-livejournal-com.turbopages.org': 'so73.livejournal.com',
 'monna--vanna-livejournal-com.turbopages.org': 'monna-vanna.livejournal.com',
 'mary--tbilisi-livejournal-com.turbopages.org': 'mary-tbilisi.livejournal.com',
 'egorov-livejournal-com.turbopages.org': 'egorov.livejournal.com',
 'dima--ratikhin-livejournal-com.turbopages.org': 'dima-ratikhin.livejournal.com',
 'saharisha-livejournal-com.turbopages.org': 'saharisha.livejournal.com',
 'nikolai--endegor-livejournal-com.turbopages.org': 'nikolai-endegor.livejournal.com',
 'barrakuda-biz.turbopages.org': 'barrakuda.biz',
 'pustotel-livejournal-com.turbopages.org': 'pustotel.livejournal.com',
 'sirvana-livejournal-com.turbopages.org': 'sirvana.livejournal.com',
 'astana--1986-livejournal-com.turbopages.org': 'astana-1986.livejournal.com',
 'kras--yar-livejournal-com.turbopages.org': 'kras-yar.livejournal.com',
 'gallerix-ru.turbopages.org': 'gallerix.ru',
 'koffkindom-ru.turbopages.org': 'koffkindom.ru',
 'yakudze-livejournal-com.turbopages.org': 'yakudze.livejournal.com',
 'innakin-ru.turbopages.org': 'innakin.ru',
 'mamayanka-livejournal-com.turbopages.org': 'mamayanka.livejournal.com',
 'auto-sevastopol-su.turbopages.org': 'auto.sevastopol.su',
 'immrill-livejournal-com.turbopages.org': 'immrill.livejournal.com',
 'scarystory-ru.turbopages.org': 'scarystory.ru',
 'elen--mos-livejournal-com.turbopages.org': 'elen-mos.livejournal.com',
 'autodeutschland-livejournal-com.turbopages.org': 'autodeutschland.livejournal.com',
 'alexhitrov-livejournal-com.turbopages.org': 'alexhitrov.livejournal.com',
 'pitertransport-livejournal-com.turbopages.org': 'pitertransport.livejournal.com',
 'recipesbook-livejournal-com.turbopages.org': 'recipesbook.livejournal.com',
 'harmoniq--app-livejournal-com.turbopages.org': 'harmoniq-app.livejournal.com',
 'praiser-livejournal-com.turbopages.org': 'praiser.livejournal.com',
 'meriber-livejournal-com.turbopages.org': 'meriber.livejournal.com',
 'olga--krav4enko-livejournal-com.turbopages.org': 'olga-krav4enko.livejournal.com',
 'newsbel-by.turbopages.org': 'newsbel.by',
 'nevadm-ru.turbopages.org': 'nevadm.ru',
 'engisdottir-livejournal-com.turbopages.org': 'engisdottir.livejournal.com',
 'norilsk--history-livejournal-com.turbopages.org': 'norilsk-history.livejournal.com',
 'bb--mania-livejournal-com.turbopages.org': 'bb-mania.livejournal.com',
 'kreditnie--donory--rostov-ru.turbopages.org': 'kreditnie-donory-rostov.ru',
 'shamanov--sergey-livejournal-com.turbopages.org': 'shamanov-sergey.livejournal.com',
 'dmsutyagin-livejournal-com.turbopages.org': 'dmsutyagin.livejournal.com',
 'ein--arzt-livejournal-com.turbopages.org': 'ein-arzt.livejournal.com',
 'bodystorm-ru.turbopages.org': 'bodystorm.ru',
 'mramorelite-ru.turbopages.org': 'mramorelite.ru',
 'magnitogorsk-bezformata-com.turbopages.org': 'magnitogorsk.bezformata.com',
 'cube--wan-livejournal-com.turbopages.org': 'cube-wan.livejournal.com',
 'tykywak-livejournal-com.turbopages.org': 'tykywak.livejournal.com',
 'asya--fly-livejournal-com.turbopages.org': 'asya-fly.livejournal.com',
 'logazinvest-ru.turbopages.org': 'logazinvest.ru',
 'gyroscooter--izhevsk-ru.turbopages.org': 'gyroscooter-izhevsk.ru',
 'viktorkotl-livejournal-com.turbopages.org': 'viktorkotl.livejournal.com',
 'stopqnet-livejournal-com.turbopages.org': 'stopqnet.livejournal.com',
 'vi--silver-livejournal-com.turbopages.org': 'vi-silver.livejournal.com',
 'kumir--millionof-livejournal-com.turbopages.org': 'kumir-millionof.livejournal.com',
 'kazakdona-ru.turbopages.org': 'kazakdona.ru',
 'dillmart-ru.turbopages.org': 'dillmart.ru',
 'riba--lka-ru.turbopages.org': 'riba-lka.ru',
 'ru--crazy--news-livejournal-com.turbopages.org': 'ru-crazy-news.livejournal.com',
 'moscow-big--book--med-ru.turbopages.org': 'moscow.big-book-med.ru',
 'zamos-livejournal-com.turbopages.org': 'zamos.livejournal.com',
 'depoterminal-livejournal-com.turbopages.org': 'depoterminal.livejournal.com',
 'mrandroid-ru.turbopages.org': 'mrandroid.ru',
 'vorodis-livejournal-com.turbopages.org': 'vorodis.livejournal.com',
 'mirrorboy2-livejournal-com.turbopages.org': 'mirrorboy2.livejournal.com',
 'kissa--redissa-livejournal-com.turbopages.org': 'kissa-redissa.livejournal.com',
 'klik--zarabotok-ru.turbopages.org': 'klik-zarabotok.ru',
 'tatititata-livejournal-com.turbopages.org': 'tatititata.livejournal.com',
 'vasiliygf-livejournal-com.turbopages.org': 'vasiliygf.livejournal.com',
 'ange--ou--de--moon-livejournal-com.turbopages.org': 'ange-ou-de-moon.livejournal.com',
 'kwartira--n13-livejournal-com.turbopages.org': 'kwartira-n13.livejournal.com',
 'irene--dragon-livejournal-com.turbopages.org': 'irene-dragon.livejournal.com',
 'dr--psix-livejournal-com.turbopages.org': 'dr-psix.livejournal.com',
 'alandinna-livejournal-com.turbopages.org': 'alandinna.livejournal.com',
 'guidehins-ru.turbopages.org': 'guidehins.ru',
 'nedelka--klin-ru.turbopages.org': 'nedelka-klin.ru',
 'peaky--blinders-ru.turbopages.org': 'peaky-blinders.ru',
 'actual--phlebology-ru.turbopages.org': 'actual-phlebology.ru',
 'd--garmonenko-livejournal-com.turbopages.org': 'd-garmonenko.livejournal.com',
 'mehanoid-livejournal-com.turbopages.org': 'mehanoid.livejournal.com',
 'lapatissiere-livejournal-com.turbopages.org': 'lapatissiere.livejournal.com',
 'kotyara--71-livejournal-com.turbopages.org': 'kotyara-71.livejournal.com',
 'amurogorod-ru.turbopages.org': 'amurogorod.ru',
 'simtour-livejournal-com.turbopages.org': 'simtour.livejournal.com',
 'mahsar-ru.turbopages.org': 'mahsar.ru',
 'uspevai--s--detmi-livejournal-com.turbopages.org': 'uspevai-s-detmi.livejournal.com',
 'news--hunter-pro.turbopages.org': 'news-hunter.pro',
 'hunter--cbx-livejournal-com.turbopages.org': 'hunter-cbx.livejournal.com',
 'nat--velikanova-livejournal-com.turbopages.org': 'nat-velikanova.livejournal.com',
 'lyubovdetyam-livejournal-com.turbopages.org': 'lyubovdetyam.livejournal.com',
 'jread-top.turbopages.org': 'jread.top',
 'grenoble-livejournal-com.turbopages.org': 'grenoble.livejournal.com',
 'mymaster-livejournal-com.turbopages.org': 'mymaster.livejournal.com',
 'lizardian-livejournal-com.turbopages.org': 'lizardian.livejournal.com',
 'oleg--a-livejournal-com.turbopages.org': 'oleg-a.livejournal.com',
 'autoclub99-ru.turbopages.org': 'autoclub99.ru',
 'level--meter-livejournal-com.turbopages.org': 'level-meter.livejournal.com',
 'xn------7cdkfke0egmzjnb0b.turbopages.org': 'xn---7cdkfke0egmzjnb0b',
 'ststalin-livejournal-com.turbopages.org': 'ststalin.livejournal.com',
 'porodikoshek-ru.turbopages.org': 'porodikoshek.ru',
 'sovremennikk-livejournal-com.turbopages.org': 'sovremennikk.livejournal.com',
 'itditpspb-livejournal-com.turbopages.org': 'itditpspb.livejournal.com',
 'inbelousov-livejournal-com.turbopages.org': 'inbelousov.livejournal.com',
 'tiflist-ru.turbopages.org': 'tiflist.ru',
 'zapadnik-livejournal-com.turbopages.org': 'zapadnik.livejournal.com',
 'mega--povar-ru.turbopages.org': 'mega-povar.ru',
 'mari--ya--77-livejournal-com.turbopages.org': 'mari-ya-77.livejournal.com',
 'walter--simons-livejournal-com.turbopages.org': 'walter-simons.livejournal.com',
 'vinauto777-livejournal-com.turbopages.org': 'vinauto777.livejournal.com',
 'serjio--pereira-livejournal-com.turbopages.org': 'serjio-pereira.livejournal.com',
 'infohome-com-ua.turbopages.org': 'infohome.com.ua',
 'rivershkiper-livejournal-com.turbopages.org': 'rivershkiper.livejournal.com',
 'olster28-livejournal-com.turbopages.org': 'olster28.livejournal.com',
 'azizaholga-livejournal-com.turbopages.org': 'azizaholga.livejournal.com',
 'zakaztaxi-info.turbopages.org': 'zakaztaxi.info',
 'yanaubka-livejournal-com.turbopages.org': 'yanaubka.livejournal.com',
 'organika-livejournal-com.turbopages.org': 'organika.livejournal.com',
 'konkurs--dlya--pedagogov-info.turbopages.org': 'konkurs-dlya-pedagogov.info',
 'bronnici-bezformata-com.turbopages.org': 'bronnici.bezformata.com',
 'beaf-pro.turbopages.org': 'beaf.pro',
 'serh-livejournal-com.turbopages.org': 'serh.livejournal.com',
 'tyumen-big--book--med-ru.turbopages.org': 'tyumen.big-book-med.ru',
 'nyagan-life.turbopages.org': 'nyagan.life',
 'meangel-livejournal-com.turbopages.org': 'meangel.livejournal.com',
 'prikolnypodarok-livejournal-com.turbopages.org': 'prikolnypodarok.livejournal.com',
 'rajskij--sad-livejournal-com.turbopages.org': 'rajskij-sad.livejournal.com',
 'veraneo-livejournal-com.turbopages.org': 'veraneo.livejournal.com',
 'gollashkino-livejournal-com.turbopages.org': 'gollashkino.livejournal.com',
 'reality--surfing-livejournal-com.turbopages.org': 'reality-surfing.livejournal.com',
 'steppentiger-livejournal-com.turbopages.org': 'steppentiger.livejournal.com',
 'altertravel-ru.turbopages.org': 'altertravel.ru',
 'dr--mart-livejournal-com.turbopages.org': 'dr-mart.livejournal.com',
 'mashutka--alfi-livejournal-com.turbopages.org': 'mashutka-alfi.livejournal.com',
 'dmfff-livejournal-com.turbopages.org': 'dmfff.livejournal.com',
 'costume--history-livejournal-com.turbopages.org': 'costume-history.livejournal.com',
 'intizar-livejournal-com.turbopages.org': 'intizar.livejournal.com',
 'pricepblog-ru.turbopages.org': 'pricepblog.ru',
 'flowerspbmax-livejournal-com.turbopages.org': 'flowerspbmax.livejournal.com',
 'tumentcev--boris-livejournal-com.turbopages.org': 'tumentcev-boris.livejournal.com',
 'tania--selezneva-livejournal-com.turbopages.org': 'tania-selezneva.livejournal.com',
 'pavel--sviridov-livejournal-com.turbopages.org': 'pavel-sviridov.livejournal.com',
 'teplores-ru.turbopages.org': 'teplores.ru',
 'doctor--notes-livejournal-com.turbopages.org': 'doctor-notes.livejournal.com',
 'barilla--mania-livejournal-com.turbopages.org': 'barilla-mania.livejournal.com',
 'vpervye1-livejournal-com.turbopages.org': 'vpervye1.livejournal.com',
 'vseshokeri-ru.turbopages.org': 'vseshokeri.ru',
 'visit--plus-com.turbopages.org': 'visit-plus.com',
 'icdialogue-ru.turbopages.org': 'icdialogue.ru',
 'catnikiforova-livejournal-com.turbopages.org': 'catnikiforova.livejournal.com',
 'kilativ-livejournal-com.turbopages.org': 'kilativ.livejournal.com',
 'kitaiplus-livejournal-com.turbopages.org': 'kitaiplus.livejournal.com',
 'mega--faza-ru.turbopages.org': 'mega-faza.ru',
 'dyadka--zakharov-livejournal-com.turbopages.org': 'dyadka-zakharov.livejournal.com',
 'pavel--kolesov-livejournal-com.turbopages.org': 'pavel-kolesov.livejournal.com',
 'adieusovok-com.turbopages.org': 'adieusovok.com',
 'ladysunrise-livejournal-com.turbopages.org': 'ladysunrise.livejournal.com',
 'samoylovskaya-ru.turbopages.org': 'samoylovskaya.ru',
 'nes--i-livejournal-com.turbopages.org': 'nes-i.livejournal.com',
 'mar--amirchanyan-livejournal-com.turbopages.org': 'mar-amirchanyan.livejournal.com',
 'mrcynognathus-livejournal-com.turbopages.org': 'mrcynognathus.livejournal.com',
 'fontanka167-livejournal-com.turbopages.org': 'fontanka167.livejournal.com',
 'very--old--life-livejournal-com.turbopages.org': 'very-old-life.livejournal.com',
 'rais-livejournal-com.turbopages.org': 'rais.livejournal.com',
 'lactoriacornuta-livejournal-com.turbopages.org': 'lactoriacornuta.livejournal.com',
 'pravda33-livejournal-com.turbopages.org': 'pravda33.livejournal.com',
 'babaecalus-livejournal-com.turbopages.org': 'babaecalus.livejournal.com',
 'neva-versia-ru.turbopages.org': 'neva.versia.ru',
 'yusev--alexei-livejournal-com.turbopages.org': 'yusev-alexei.livejournal.com',
 'zargarov-livejournal-com.turbopages.org': 'zargarov.livejournal.com',
 'feo--man-livejournal-com.turbopages.org': 'feo-man.livejournal.com',
 'mahtalcar-livejournal-com.turbopages.org': 'mahtalcar.livejournal.com',
 'budetlyanin108-livejournal-com.turbopages.org': 'budetlyanin108.livejournal.com',
 'sustavoved-com.turbopages.org': 'sustavoved.com',
 'kak--svoimi--rukami-com.turbopages.org': 'kak-svoimi-rukami.com',
 'vvzaitcev-livejournal-com.turbopages.org': 'vvzaitcev.livejournal.com',
 '108km-livejournal-com.turbopages.org': '108km.livejournal.com',
 'art--afrodiziak-livejournal-com.turbopages.org': 'art-afrodiziak.livejournal.com',
 'bob--many-livejournal-com.turbopages.org': 'bob-many.livejournal.com',
 'kiborg-pro.turbopages.org': 'kiborg.pro',
 'optimist888-livejournal-com.turbopages.org': 'optimist888.livejournal.com',
 'shaulreznik-livejournal-com.turbopages.org': 'shaulreznik.livejournal.com',
 'pulmonologiya-com.turbopages.org': 'pulmonologiya.com',
 'russellab-livejournal-com.turbopages.org': 'russellab.livejournal.com',
 'turkish--r-livejournal-com.turbopages.org': 'turkish-r.livejournal.com',
 'kuzma--prutkoff-livejournal-com.turbopages.org': 'kuzma-prutkoff.livejournal.com',
 'radikal282-livejournal-com.turbopages.org': 'radikal282.livejournal.com',
 'be--happywoman-ru.turbopages.org': 'be-happywoman.ru',
 'apple--w-livejournal-com.turbopages.org': 'apple-w.livejournal.com',
 'sgtmadcat-livejournal-com.turbopages.org': 'sgtmadcat.livejournal.com',
 'lenulja79-livejournal-com.turbopages.org': 'lenulja79.livejournal.com',
 'witeck-livejournal-com.turbopages.org': 'witeck.livejournal.com',
 'juste--rus-livejournal-com.turbopages.org': 'juste-rus.livejournal.com',
 'radvi--sasha-livejournal-com.turbopages.org': 'radvi-sasha.livejournal.com',
 'york--teryer-ru.turbopages.org': 'york-teryer.ru',
 'nicsky-livejournal-com.turbopages.org': 'nicsky.livejournal.com',
 'segolene--royal-livejournal-com.turbopages.org': 'segolene-royal.livejournal.com',
 'magazinov--24-ru.turbopages.org': 'magazinov-24.ru',
 'damreceptic-ru.turbopages.org': 'damreceptic.ru',
 'valorantexpert-ru.turbopages.org': 'valorantexpert.ru',
 'yuriy--yum-livejournal-com.turbopages.org': 'yuriy-yum.livejournal.com',
 'asdf232-livejournal-com.turbopages.org': 'asdf232.livejournal.com',
 'str--bio-livejournal-com.turbopages.org': 'str-bio.livejournal.com',
 'sadtranslations-livejournal-com.turbopages.org': 'sadtranslations.livejournal.com',
 'pablik--kozhevnika-ru.turbopages.org': 'pablik-kozhevnika.ru',
 'tanuffka-livejournal-com.turbopages.org': 'tanuffka.livejournal.com',
 'vitaliymih-livejournal-com.turbopages.org': 'vitaliymih.livejournal.com',
 'big5-ru.turbopages.org': 'big5.ru',
 'lost--fortresses-livejournal-com.turbopages.org': 'lost-fortresses.livejournal.com',
 'swann74-livejournal-com.turbopages.org': 'swann74.livejournal.com',
 'mechanismone-livejournal-com.turbopages.org': 'mechanismone.livejournal.com',
 'phontanka-livejournal-com.turbopages.org': 'phontanka.livejournal.com',
 'ewushka-livejournal-com.turbopages.org': 'ewushka.livejournal.com',
 'natasha--tuk-livejournal-com.turbopages.org': 'natasha-tuk.livejournal.com',
 'energomashbank-livejournal-com.turbopages.org': 'energomashbank.livejournal.com',
 'tollique-livejournal-com.turbopages.org': 'tollique.livejournal.com',
 'jenygreate-livejournal-com.turbopages.org': 'jenygreate.livejournal.com',
 'museum--ru-livejournal-com.turbopages.org': 'museum-ru.livejournal.com',
 'srk-livejournal-com.turbopages.org': 'srk.livejournal.com',
 'fotoserg-livejournal-com.turbopages.org': 'fotoserg.livejournal.com',
 'almor1998-livejournal-com.turbopages.org': 'almor1998.livejournal.com',
 'yaroslav--gunin-livejournal-com.turbopages.org': 'yaroslav-gunin.livejournal.com',
 'dymovskiy--name-livejournal-com.turbopages.org': 'dymovskiy-name.livejournal.com',
 'mixail--shagurin-livejournal-com.turbopages.org': 'mixail-shagurin.livejournal.com',
 'alex999k-livejournal-com.turbopages.org': 'alex999k.livejournal.com',
 'poetry--in--life-livejournal-com.turbopages.org': 'poetry-in-life.livejournal.com',
 'who--stole--tarts-livejournal-com.turbopages.org': 'who-stole-tarts.livejournal.com',
 'chipstone-livejournal-com.turbopages.org': 'chipstone.livejournal.com',
 'cyber125-livejournal-com.turbopages.org': 'cyber125.livejournal.com',
 'anti--chief-livejournal-com.turbopages.org': 'anti-chief.livejournal.com',
 'manma-livejournal-com.turbopages.org': 'manma.livejournal.com',
 'lemiki-ru.turbopages.org': 'lemiki.ru',
 'rousseau-livejournal-com.turbopages.org': 'rousseau.livejournal.com',
 'gotovim-uz.turbopages.org': 'gotovim.uz',
 'potamophylax-livejournal-com.turbopages.org': 'potamophylax.livejournal.com',
 'komen--dant-livejournal-com.turbopages.org': 'komen-dant.livejournal.com',
 'stillavin-livejournal-com.turbopages.org': 'stillavin.livejournal.com',
 'kostadm-livejournal-com.turbopages.org': 'kostadm.livejournal.com',
 'tomsoer-livejournal-com.turbopages.org': 'tomsoer.livejournal.com',
 'arheolog68-livejournal-com.turbopages.org': 'arheolog68.livejournal.com',
 'rutralet-livejournal-com.turbopages.org': 'rutralet.livejournal.com',
 'rimona-livejournal-com.turbopages.org': 'rimona.livejournal.com',
 'vsesam-org.turbopages.org': 'vsesam.org',
 'leo--tvorit-livejournal-com.turbopages.org': 'leo-tvorit.livejournal.com',
 'sketchschool-livejournal-com.turbopages.org': 'sketchschool.livejournal.com',
 'krd-kudago-com.turbopages.org': 'krd.kudago.com',
 'birorka-livejournal-com.turbopages.org': 'birorka.livejournal.com',
 'tbiliseli-livejournal-com.turbopages.org': 'tbiliseli.livejournal.com',
 'kot--kam-livejournal-com.turbopages.org': 'kot-kam.livejournal.com',
 'vkusnogotovym-ru.turbopages.org': 'vkusnogotovym.ru',
 'karankee-livejournal-com.turbopages.org': 'karankee.livejournal.com',
 'soraman-livejournal-com.turbopages.org': 'soraman.livejournal.com',
 'mpolyakov-livejournal-com.turbopages.org': 'mpolyakov.livejournal.com',
 'drakosh--a-livejournal-com.turbopages.org': 'drakosh-a.livejournal.com',
 'stay--in--bel-livejournal-com.turbopages.org': 'stay-in-bel.livejournal.com',
 'tvoiugolok-ru.turbopages.org': 'tvoiugolok.ru',
 'vepoqinfidal-livejournal-com.turbopages.org': 'vepoqinfidal.livejournal.com',
 'moj--maskarad-livejournal-com.turbopages.org': 'moj-maskarad.livejournal.com',
 'kansk-bezformata-com.turbopages.org': 'kansk.bezformata.com',
 'zina--korzina-livejournal-com.turbopages.org': 'zina-korzina.livejournal.com',
 'kino--tops-livejournal-com.turbopages.org': 'kino-tops.livejournal.com',
 'aftershock--2-livejournal-com.turbopages.org': 'aftershock-2.livejournal.com',
 'julietta--bella-livejournal-com.turbopages.org': 'julietta-bella.livejournal.com',
 'donbassrus-livejournal-com.turbopages.org': 'donbassrus.livejournal.com',
 'mikhailzhukov-livejournal-com.turbopages.org': 'mikhailzhukov.livejournal.com',
 'page-maple4-ru.turbopages.org': 'page.maple4.ru',
 'georgith55-livejournal-com.turbopages.org': 'georgith55.livejournal.com',
 'akfiti-livejournal-com.turbopages.org': 'akfiti.livejournal.com',
 'genrixll-livejournal-com.turbopages.org': 'genrixll.livejournal.com',
 'universal--inf-livejournal-com.turbopages.org': 'universal-inf.livejournal.com',
 'ragazzo--2009-livejournal-com.turbopages.org': 'ragazzo-2009.livejournal.com',
 'fox--cherry--lee-livejournal-com.turbopages.org': 'fox-cherry-lee.livejournal.com',
 'watermelon83-livejournal-com.turbopages.org': 'watermelon83.livejournal.com',
 'jokesland-net-ru.turbopages.org': 'jokesland.net.ru',
 'lyutov70-livejournal-com.turbopages.org': 'lyutov70.livejournal.com',
 'olxa2-livejournal-com.turbopages.org': 'olxa2.livejournal.com',
 'm--d--n-livejournal-com.turbopages.org': 'm-d-n.livejournal.com',
 'remont-livejournal-com.turbopages.org': 'remont.livejournal.com',
 'wedma--la71-livejournal-com.turbopages.org': 'wedma-la71.livejournal.com',
 'arkras-com.turbopages.org': 'arkras.com',
 'evstoliya--3-livejournal-com.turbopages.org': 'evstoliya-3.livejournal.com',
 'priazove-ru.turbopages.org': 'priazove.ru',
 'antonio--j-livejournal-com.turbopages.org': 'antonio-j.livejournal.com',
 'sunnit05-livejournal-com.turbopages.org': 'sunnit05.livejournal.com',
 'vampio-livejournal-com.turbopages.org': 'vampio.livejournal.com',
 'tourism--il-livejournal-com.turbopages.org': 'tourism-il.livejournal.com',
 'zeynepdersaadet-livejournal-com.turbopages.org': 'zeynepdersaadet.livejournal.com',
 'rebeka--r-livejournal-com.turbopages.org': 'rebeka-r.livejournal.com',
 'kb--aero-ru.turbopages.org': 'kb-aero.ru',
 'gorod--mytischi-livejournal-com.turbopages.org': 'gorod-mytischi.livejournal.com',
 'gertmany-ru.turbopages.org': 'gertmany.ru',
 'kpss--ru-livejournal-com.turbopages.org': 'kpss-ru.livejournal.com',
 'irek--tattarich-livejournal-com.turbopages.org': 'irek-tattarich.livejournal.com',
 'masha--aiva-livejournal-com.turbopages.org': 'masha-aiva.livejournal.com',
 'boldachev-livejournal-com.turbopages.org': 'boldachev.livejournal.com',
 'krylov-livejournal-com.turbopages.org': 'krylov.livejournal.com',
 'redmond--russia-livejournal-com.turbopages.org': 'redmond-russia.livejournal.com',
 'orfis--sakarna-livejournal-com.turbopages.org': 'orfis-sakarna.livejournal.com',
 'olga--mezenceva-livejournal-com.turbopages.org': 'olga-mezenceva.livejournal.com',
 'nle2004-livejournal-com.turbopages.org': 'nle2004.livejournal.com',
 'stop--smoking--ru-livejournal-com.turbopages.org': 'stop-smoking-ru.livejournal.com',
 'satka-bezformata-com.turbopages.org': 'satka.bezformata.com',
 '777hawk-livejournal-com.turbopages.org': '777hawk.livejournal.com',
 'danmas-livejournal-com.turbopages.org': 'danmas.livejournal.com',
 'mr--fors-livejournal-com.turbopages.org': 'mr-fors.livejournal.com',
 'nina--minina-livejournal-com.turbopages.org': 'nina-minina.livejournal.com',
 'drugie--drugie-livejournal-com.turbopages.org': 'drugie-drugie.livejournal.com',
 'naginata-livejournal-com.turbopages.org': 'naginata.livejournal.com',
 'gps--echolot-ru.turbopages.org': 'gps-echolot.ru',
 'serg--was-livejournal-com.turbopages.org': 'serg-was.livejournal.com',
 'ivanteevec-livejournal-com.turbopages.org': 'ivanteevec.livejournal.com',
 'langtown-ru.turbopages.org': 'langtown.ru',
 'znaesh--kak-com.turbopages.org': 'znaesh-kak.com',
 'mul9l-livejournal-com.turbopages.org': 'mul9l.livejournal.com',
 'govorilkin-livejournal-com.turbopages.org': 'govorilkin.livejournal.com',
 'old--world--blues-livejournal-com.turbopages.org': 'old-world-blues.livejournal.com',
 'pavorit-livejournal-com.turbopages.org': 'pavorit.livejournal.com',
 'lonsom-livejournal-com.turbopages.org': 'lonsom.livejournal.com',
 'kedrovy--ruchey-livejournal-com.turbopages.org': 'kedrovy-ruchey.livejournal.com',
 'dokument24-livejournal-com.turbopages.org': 'dokument24.livejournal.com',
 'nyurok88-livejournal-com.turbopages.org': 'nyurok88.livejournal.com',
 'fan--female-ru.turbopages.org': 'fan-female.ru',
 'orlenokvolga-ru.turbopages.org': 'orlenokvolga.ru',
 'fleur--de--cassie-livejournal-com.turbopages.org': 'fleur-de-cassie.livejournal.com',
 'audi0sam-livejournal-com.turbopages.org': 'audi0sam.livejournal.com',
 'ivanskobey-livejournal-com.turbopages.org': 'ivanskobey.livejournal.com',
 'azamat--ulbashev-livejournal-com.turbopages.org': 'azamat-ulbashev.livejournal.com',
 'kosha--ann-livejournal-com.turbopages.org': 'kosha-ann.livejournal.com',
 'imur-livejournal-com.turbopages.org': 'imur.livejournal.com',
 'sparex-livejournal-com.turbopages.org': 'sparex.livejournal.com',
 'aazz--z-livejournal-com.turbopages.org': 'aazz-z.livejournal.com',
 'basilius3-livejournal-com.turbopages.org': 'basilius3.livejournal.com',
 'malamant-livejournal-com.turbopages.org': 'malamant.livejournal.com',
 'bigimotoff-livejournal-com.turbopages.org': 'bigimotoff.livejournal.com',
 'alexander--vasil-livejournal-com.turbopages.org': 'alexander-vasil.livejournal.com',
 'ok--sun--kaa-livejournal-com.turbopages.org': 'ok-sun-kaa.livejournal.com',
 'bskam-livejournal-com.turbopages.org': 'bskam.livejournal.com',
 'dominionmind-ru.turbopages.org': 'dominionmind.ru',
 'balkonblog-ru.turbopages.org': 'balkonblog.ru',
 'leksandrych-livejournal-com.turbopages.org': 'leksandrych.livejournal.com',
 'prosto--ekonom-ru.turbopages.org': 'prosto-ekonom.ru',
 'snezinka11-livejournal-com.turbopages.org': 'snezinka11.livejournal.com',
 'robinfeelgood-livejournal-com.turbopages.org': 'robinfeelgood.livejournal.com',
 'stariy--khren-livejournal-com.turbopages.org': 'stariy-khren.livejournal.com',
 'bestcattery-ru.turbopages.org': 'bestcattery.ru',
 'angel--avvadon-livejournal-com.turbopages.org': 'angel-avvadon.livejournal.com',
 'animals--rights-livejournal-com.turbopages.org': 'animals-rights.livejournal.com',
 'rusday-com.turbopages.org': 'rusday.com',
 'npoet-ru.turbopages.org': 'npoet.ru',
 'komitetzaprava-livejournal-com.turbopages.org': 'komitetzaprava.livejournal.com',
    'nostradamvs-livejournal-com.turbopages.org': 'nostradamvs.livejournal.com',
 'coffee--mood-livejournal-com.turbopages.org': 'coffee-mood.livejournal.com',
 'pro--tyres-ru.turbopages.org': 'pro-tyres.ru',
 'sevastian--mos-livejournal-com.turbopages.org': 'sevastian-mos.livejournal.com',
 'barcaffe-ru.turbopages.org': 'barcaffe.ru',
 'ru--filpark-livejournal-com.turbopages.org': 'ru-filpark.livejournal.com',
 'zepete-livejournal-com.turbopages.org': 'zepete.livejournal.com',
 'domashniirestoran-ru.turbopages.org': 'domashniirestoran.ru',
 'pavlovskyposad-livejournal-com.turbopages.org': 'pavlovskyposad.livejournal.com',
 'zooterritoria-ru.turbopages.org': 'zooterritoria.ru',
 'remmachserp-ru.turbopages.org': 'remmachserp.ru',
 'galkyk-livejournal-com.turbopages.org': 'galkyk.livejournal.com',
 'zvon4ekova-livejournal-com.turbopages.org': 'zvon4ekova.livejournal.com',
 'kinokpk-livejournal-com.turbopages.org': 'kinokpk.livejournal.com',
 'dubna-bezformata-com.turbopages.org': 'dubna.bezformata.com',
 '100--primet-ru.turbopages.org': '100-primet.ru',
 'avmalgin-livejournal-com.turbopages.org': 'avmalgin.livejournal.com',
 'andrey4-livejournal-com.turbopages.org': 'andrey4.livejournal.com',
 'leonid--martinyk-livejournal-com.turbopages.org': 'leonid-martinyk.livejournal.com',
 'p--senko-livejournal-com.turbopages.org': 'p-senko.livejournal.com',
 'allavasenka-livejournal-com.turbopages.org': 'allavasenka.livejournal.com',
 'vovkazzz-livejournal-com.turbopages.org': 'vovkazzz.livejournal.com',
 'ronin--077-livejournal-com.turbopages.org': 'ronin-077.livejournal.com',
 'cd--player-livejournal-com.turbopages.org': 'cd-player.livejournal.com',
 'sav--in-livejournal-com.turbopages.org': 'sav-in.livejournal.com',
 'sandreyo-livejournal-com.turbopages.org': 'sandreyo.livejournal.com',
 'aviablogger-livejournal-com.turbopages.org': 'aviablogger.livejournal.com',
 'aaavolkov-livejournal-com.turbopages.org': 'aaavolkov.livejournal.com',
 'coffeebreak24-ru.turbopages.org': 'coffeebreak24.ru',
 'kvanbrud-livejournal-com.turbopages.org': 'kvanbrud.livejournal.com',
 'zsm--miass-ru.turbopages.org': 'zsm-miass.ru',
 'jd--com-ru.turbopages.org': 'jd-com.ru',
 'detskoekino-livejournal-com.turbopages.org': 'detskoekino.livejournal.com',
 'mka-livejournal-com.turbopages.org': 'mka.livejournal.com',
 'lacmus-life.turbopages.org': 'lacmus.life',
 'triada--web-ru.turbopages.org': 'triada-web.ru',
 'kateryna--dolgiy-livejournal-com.turbopages.org': 'kateryna-dolgiy.livejournal.com',
 'linza-guru.turbopages.org': 'linza.guru',
 '32sm-ru.turbopages.org': '32sm.ru',
 '1000voltt-ru.turbopages.org': '1000voltt.ru',
 'eliseevna--50-livejournal-com.turbopages.org': 'eliseevna-50.livejournal.com',
 'niktan--gullsea-livejournal-com.turbopages.org': 'niktan-gullsea.livejournal.com',
 'electric--blogger-ru.turbopages.org': 'electric-blogger.ru',
 'paratype-livejournal-com.turbopages.org': 'paratype.livejournal.com',
 'vertebrata-ru.turbopages.org': 'vertebrata.ru',
 'voronkov--kirill-livejournal-com.turbopages.org': 'voronkov-kirill.livejournal.com',
 'malorossijanin-livejournal-com.turbopages.org': 'malorossijanin.livejournal.com',
 'immusic-livejournal-com.turbopages.org': 'immusic.livejournal.com',
 'andp2027-livejournal-com.turbopages.org': 'andp2027.livejournal.com',
 'juche--songun-livejournal-com.turbopages.org': 'juche-songun.livejournal.com',
 'alexey43-livejournal-com.turbopages.org': 'alexey43.livejournal.com',
 'travel247-ru.turbopages.org': 'travel247.ru',
 'ffedd--ya-livejournal-com.turbopages.org': 'ffedd-ya.livejournal.com',
 'dok34ru-livejournal-com.turbopages.org': 'dok34ru.livejournal.com',
 'with--astronotus-livejournal-com.turbopages.org': 'with-astronotus.livejournal.com',
 'valery--brest--by-livejournal-com.turbopages.org': 'valery-brest-by.livejournal.com',
 'olga1982a-livejournal-com.turbopages.org': 'olga1982a.livejournal.com',
 'stapelia2784-livejournal-com.turbopages.org': 'stapelia2784.livejournal.com',
 'holydiver--777-livejournal-com.turbopages.org': 'holydiver-777.livejournal.com',
 'mogochadp-livejournal-com.turbopages.org': 'mogochadp.livejournal.com',
 'temhuk-livejournal-com.turbopages.org': 'temhuk.livejournal.com',
 'vestiukraine-livejournal-com.turbopages.org': 'vestiukraine.livejournal.com',
 'avtovodu-livejournal-com.turbopages.org': 'avtovodu.livejournal.com',
 'mihailpsk-livejournal-com.turbopages.org': 'mihailpsk.livejournal.com',
 'mydesigngroup-ru.turbopages.org': 'mydesigngroup.ru',
 'hotjuice-livejournal-com.turbopages.org': 'hotjuice.livejournal.com',
 'farniev--kostya-livejournal-com.turbopages.org': 'farniev-kostya.livejournal.com',
 'al--cherkasov-livejournal-com.turbopages.org': 'al-cherkasov.livejournal.com',
 'avp23649-livejournal-com.turbopages.org': 'avp23649.livejournal.com',
 'gulaytour-ru.turbopages.org': 'gulaytour.ru',
 'stylehunters--ru-livejournal-com.turbopages.org': 'stylehunters-ru.livejournal.com',
 'tmmc-livejournal-com.turbopages.org': 'tmmc.livejournal.com',
 'jekkyv-ru.turbopages.org': 'jekkyv.ru',
 'spb--09-livejournal-com.turbopages.org': 'spb-09.livejournal.com',
 'wordpress--abc-ru.turbopages.org': 'wordpress-abc.ru',
 'lohmatiy77-livejournal-com.turbopages.org': 'lohmatiy77.livejournal.com',
 'winsin-livejournal-com.turbopages.org': 'winsin.livejournal.com',
 'remontoteka-net.turbopages.org': 'remontoteka.net',
 'green--hands-livejournal-com.turbopages.org': 'green-hands.livejournal.com',
 'jiwo-ru.turbopages.org': 'jiwo.ru',
 'atrizno-livejournal-com.turbopages.org': 'atrizno.livejournal.com',
 'vladislav--357-livejournal-com.turbopages.org': 'vladislav-357.livejournal.com',
 'orionstroy-group.turbopages.org': 'orionstroy.group',
 'novopashin-livejournal-com.turbopages.org': 'novopashin.livejournal.com',
 'qbik-club.turbopages.org': 'qbik.club',
 'opupel-livejournal-com.turbopages.org': 'opupel.livejournal.com',
 'lada--matushka-livejournal-com.turbopages.org': 'lada-matushka.livejournal.com',
 'cheeseinfo-ru.turbopages.org': 'cheeseinfo.ru',
 'alexnovember-livejournal-com.turbopages.org': 'alexnovember.livejournal.com',
 'reich--erwacht-livejournal-com.turbopages.org': 'reich-erwacht.livejournal.com',
 'barankin1000-livejournal-com.turbopages.org': 'barankin1000.livejournal.com',
 'les--details-livejournal-com.turbopages.org': 'les-details.livejournal.com',
 'rugamenews-ru.turbopages.org': 'rugamenews.ru',
 'catrina--burana-livejournal-com.turbopages.org': 'catrina-burana.livejournal.com',
 'just--meller-livejournal-com.turbopages.org': 'just-meller.livejournal.com',
 'roza--zanoza-ru.turbopages.org': 'roza-zanoza.ru',
 'sayanarus-livejournal-com.turbopages.org': 'sayanarus.livejournal.com',
 'nngan-livejournal-com.turbopages.org': 'nngan.livejournal.com',
 'kiprioman-livejournal-com.turbopages.org': 'kiprioman.livejournal.com',
 'handmade--paradise-ru.turbopages.org': 'handmade-paradise.ru',
 'ostroglyad-livejournal-com.turbopages.org': 'ostroglyad.livejournal.com',
 'vortaro-ru.turbopages.org': 'vortaro.ru',
 'gutsuland-livejournal-com.turbopages.org': 'gutsuland.livejournal.com',
 'eurokafel-ru.turbopages.org': 'eurokafel.ru',
 'herbogolik-livejournal-com.turbopages.org': 'herbogolik.livejournal.com',
 'donday--novocherkassk-ru.turbopages.org': 'donday-novocherkassk.ru',
 'mybelovo-ru.turbopages.org': 'mybelovo.ru',
 'nadia--yacik-livejournal-com.turbopages.org': 'nadia-yacik.livejournal.com',
 'ponchikov-net.turbopages.org': 'ponchikov.net',
 'azned-net.turbopages.org': 'azned.net',
 'karagajskij--bor-ru.turbopages.org': 'karagajskij-bor.ru',
 'nnatalia257-livejournal-com.turbopages.org': 'nnatalia257.livejournal.com',
 'av--klement-livejournal-com.turbopages.org': 'av-klement.livejournal.com',
 'mistervik-livejournal-com.turbopages.org': 'mistervik.livejournal.com',
 'innabuybeauty-livejournal-com.turbopages.org': 'innabuybeauty.livejournal.com',
 'kuzmanafanya-livejournal-com.turbopages.org': 'kuzmanafanya.livejournal.com',
 'xn------7cdeb4esagclj6c.turbopages.org': 'xn---7cdeb4esagclj6c',
 'itechinfo-ru.turbopages.org': 'itechinfo.ru',
 'bjorn--varulv-livejournal-com.turbopages.org': 'bjorn-varulv.livejournal.com',
 'domdo-ru.turbopages.org': 'domdo.ru',
 '3652-ru.turbopages.org': '3652.ru',
 'vina--doma-ru.turbopages.org': 'vina-doma.ru',
 'holodesign-space.turbopages.org': 'holodesign.space',
 'sergiovillaggio-livejournal-com.turbopages.org': 'sergiovillaggio.livejournal.com',
 'xn---online-1fgauinz4hqazws.turbopages.org': 'xn-.online.1fgauinz4hqazws',
 'iskander1313-livejournal-com.turbopages.org': 'iskander1313.livejournal.com',
 'serg32sem2-livejournal-com.turbopages.org': 'serg32sem2.livejournal.com',
 'xn----otbahljfmcbc2bf.turbopages.org': 'xn--otbahljfmcbc2bf',
 'horo-uaset-com.turbopages.org': 'horo.uaset.com',
 'life--ussr-ru.turbopages.org': 'life-ussr.ru',
 'cheralpa-livejournal-com.turbopages.org': 'cheralpa.livejournal.com',
 'eot56-livejournal-com.turbopages.org': 'eot56.livejournal.com',
 'givotinki-ru.turbopages.org': 'givotinki.ru',
 'promislovik-info.turbopages.org': 'promislovik.info',
 'nevinnomissk-big--book--med-ru.turbopages.org': 'nevinnomissk.big-book-med.ru',
 'nickol1975-livejournal-com.turbopages.org': 'nickol1975.livejournal.com',
 'spinningist34-livejournal-com.turbopages.org': 'spinningist34.livejournal.com',
 'tany96-livejournal-com.turbopages.org': 'tany96.livejournal.com',
 'yagrigorieva-livejournal-com.turbopages.org': 'yagrigorieva.livejournal.com',
 'apelsinovitsh-livejournal-com.turbopages.org': 'apelsinovitsh.livejournal.com',
 'santehkrug-ru.turbopages.org': 'santehkrug.ru',
 'artroz--kolennih--sustavov-ru.turbopages.org': 'artroz-kolennih-sustavov.ru',
 'oldcolor-livejournal-com.turbopages.org': 'oldcolor.livejournal.com',
 'mayorgb-livejournal-com.turbopages.org': 'mayorgb.livejournal.com',
 'smapse-livejournal-com.turbopages.org': 'smapse.livejournal.com',
 'alexfisich-livejournal-com.turbopages.org': 'alexfisich.livejournal.com',
 'prostie--istorii-ru.turbopages.org': 'prostie-istorii.ru',
 'photomatika-livejournal-com.turbopages.org': 'photomatika.livejournal.com',
 'ru--fraud-livejournal-com.turbopages.org': 'ru-fraud.livejournal.com',
 'trubyda-ru.turbopages.org': 'trubyda.ru',
 'yols-livejournal-com.turbopages.org': 'yols.livejournal.com',
 'rcross-livejournal-com.turbopages.org': 'rcross.livejournal.com',
 'photonoid-livejournal-com.turbopages.org': 'photonoid.livejournal.com',
 'inform--24-com.turbopages.org': 'inform-24.com',
 'infa--stars-ru.turbopages.org': 'infa-stars.ru',
 'ilovefreebies-ru.turbopages.org': 'ilovefreebies.ru',
 'katerina--sapyan-livejournal-com.turbopages.org': 'katerina-sapyan.livejournal.com',
 'skkdc-ru.turbopages.org': 'skkdc.ru',
 'spratto-livejournal-com.turbopages.org': 'spratto.livejournal.com',
 'thunder--games-livejournal-com.turbopages.org': 'thunder-games.livejournal.com',
 'fejanor-livejournal-com.turbopages.org': 'fejanor.livejournal.com',
 'kazan-big--book--med-ru.turbopages.org': 'kazan.big-book-med.ru',
 'makaryshka-livejournal-com.turbopages.org': 'makaryshka.livejournal.com',
 'ajbolit444-livejournal-com.turbopages.org': 'ajbolit444.livejournal.com',
 'istochniki--tyumeni-ru.turbopages.org': 'istochniki-tyumeni.ru',
 'habarolog-ru.turbopages.org': 'habarolog.ru',
 'zoolog-guru.turbopages.org': 'zoolog.guru',
 'yurij--burlan-livejournal-com.turbopages.org': 'yurij-burlan.livejournal.com',
 'chiptuninger-com.turbopages.org': 'chiptuninger.com',
 'taki--net-livejournal-com.turbopages.org': 'taki-net.livejournal.com',
 'kleomen-livejournal-com.turbopages.org': 'kleomen.livejournal.com',
 'galinagordeeva-livejournal-com.turbopages.org': 'galinagordeeva.livejournal.com',
 '7daytravel-ru.turbopages.org': '7daytravel.ru',
 'stupino--24-ru.turbopages.org': 'stupino-24.ru',
 'nabbla1-livejournal-com.turbopages.org': 'nabbla1.livejournal.com',
 'urouter-ru.turbopages.org': 'urouter.ru',
 'web--nomad-ru.turbopages.org': 'web-nomad.ru',
 'alekseevich56-livejournal-com.turbopages.org': 'alekseevich56.livejournal.com',
 'y--kharechko-livejournal-com.turbopages.org': 'y-kharechko.livejournal.com',
 'tanya--zatikyan-livejournal-com.turbopages.org': 'tanya-zatikyan.livejournal.com',
 'pockomhad3op-livejournal-com.turbopages.org': 'pockomhad3op.livejournal.com',
 'housedsgn-ru.turbopages.org': 'housedsgn.ru',
 'redkoshka-livejournal-com.turbopages.org': 'redkoshka.livejournal.com',
 'parazit--off-ru.turbopages.org': 'parazit-off.ru',
 'computerjet-ru.turbopages.org': 'computerjet.ru',
 'vkarpinsk-info.turbopages.org': 'vkarpinsk.info',
 'ekb-rbc-ru.turbopages.org': 'ekb.rbc.ru',
 'shadrinsk-bezformata-com.turbopages.org': 'shadrinsk.bezformata.com',
 'evgenyageev-livejournal-com.turbopages.org': 'evgenyageev.livejournal.com',
 'metekspert-ru.turbopages.org': 'metekspert.ru',
 'legir--mk-ru.turbopages.org': 'legir-mk.ru',
 'gidlady-ru.turbopages.org': 'gidlady.ru',
 'hameleon7-livejournal-com.turbopages.org': 'hameleon7.livejournal.com',
 't--brass-livejournal-com.turbopages.org': 't-brass.livejournal.com',
 'bazamoto-ru.turbopages.org': 'bazamoto.ru',
 'alboros-livejournal-com.turbopages.org': 'alboros.livejournal.com',
 'oskolki--chesti-livejournal-com.turbopages.org': 'oskolki-chesti.livejournal.com',
 'igor--piterskiy-livejournal-com.turbopages.org': 'igor-piterskiy.livejournal.com',
 'studio--valentine-ru.turbopages.org': 'studio-valentine.ru',
 'svet--frau-livejournal-com.turbopages.org': 'svet-frau.livejournal.com',
 'los123-livejournal-com.turbopages.org': 'los123.livejournal.com',
 'easierisbetter-livejournal-com.turbopages.org': 'easierisbetter.livejournal.com',
 'admarkelov-ru.turbopages.org': 'admarkelov.ru',
 'obzorinstrumentov-ru.turbopages.org': 'obzorinstrumentov.ru',
 'pechinfo-ru.turbopages.org': 'pechinfo.ru',
 'skkilla-livejournal-com.turbopages.org': 'skkilla.livejournal.com',
 'cie--cie-livejournal-com.turbopages.org': 'cie-cie.livejournal.com',
 'xieergai29-livejournal-com.turbopages.org': 'xieergai29.livejournal.com',
 'ya--kadyrov-livejournal-com.turbopages.org': 'ya-kadyrov.livejournal.com',
 'u1ver-livejournal-com.turbopages.org': 'u1ver.livejournal.com',
 'drs--radchenko-livejournal-com.turbopages.org': 'drs-radchenko.livejournal.com',
 'vadims3233-livejournal-com.turbopages.org': 'vadims3233.livejournal.com',
 'imagadan-livejournal-com.turbopages.org': 'imagadan.livejournal.com',
 'ilskiy-su.turbopages.org': 'ilskiy.su',
 'kinoakula-ru.turbopages.org': 'kinoakula.ru',
 'house--fitness-ru.turbopages.org': 'house-fitness.ru',
 'dreamsmedic-com.turbopages.org': 'dreamsmedic.com',
 'lana--vita-livejournal-com.turbopages.org': 'lana-vita.livejournal.com',
 'danilovaalena-livejournal-com.turbopages.org': 'danilovaalena.livejournal.com',
 'katze75-livejournal-com.turbopages.org': 'katze75.livejournal.com',
 'hypertabloid-livejournal-com.turbopages.org': 'hypertabloid.livejournal.com',
 'blockstrike-ru.turbopages.org': 'blockstrike.ru',
 'ar--citygym-ru.turbopages.org': 'ar-citygym.ru',
 'pc--miner-com.turbopages.org': 'pc-miner.com',
 'alex--west-livejournal-com.turbopages.org': 'alex-west.livejournal.com',
 'fotomeridian-livejournal-com.turbopages.org': 'fotomeridian.livejournal.com',
 'ivannovikov-livejournal-com.turbopages.org': 'ivannovikov.livejournal.com',
 'helgimat-livejournal-com.turbopages.org': 'helgimat.livejournal.com',
 'bestof--woman--ru-livejournal-com.turbopages.org': 'bestof-woman-ru.livejournal.com',
 'keramabratsk-ru.turbopages.org': 'keramabratsk.ru',
 'natalya--maslova-livejournal-com.turbopages.org': 'natalya-maslova.livejournal.com',
 'livecity63-livejournal-com.turbopages.org': 'livecity63.livejournal.com',
 'makcon-livejournal-com.turbopages.org': 'makcon.livejournal.com',
 'domashnyaya--vypechka-ru.turbopages.org': 'domashnyaya-vypechka.ru',
 'nevashamasha-livejournal-com.turbopages.org': 'nevashamasha.livejournal.com',
 'crystalsoap-ru.turbopages.org': 'crystalsoap.ru',
 'abu--musa--ashari-livejournal-com.turbopages.org': 'abu-musa-ashari.livejournal.com',
 'donna--benta-livejournal-com.turbopages.org': 'donna-benta.livejournal.com',
 'drumsmen-livejournal-com.turbopages.org': 'drumsmen.livejournal.com',
 'm--a--r--y--l--a-livejournal-com.turbopages.org': 'm-a-r-y-l-a.livejournal.com',
 'novochvedomosti-ru.turbopages.org': 'novochvedomosti.ru',
 'wlodzimiz-livejournal-com.turbopages.org': 'wlodzimiz.livejournal.com',
 'dalhamun-livejournal-com.turbopages.org': 'dalhamun.livejournal.com',
 'inhumane13-livejournal-com.turbopages.org': 'inhumane13.livejournal.com',
 'svoe--hozyaistvo-ru.turbopages.org': 'svoe-hozyaistvo.ru',
 'greenapteka-ru.turbopages.org': 'greenapteka.ru',
 'xn----ctbaex2ad0a0b.turbopages.org': 'xn--ctbaex2ad0a0b',
 'dmitr--shmelyov-livejournal-com.turbopages.org': 'dmitr-shmelyov.livejournal.com',
 'ruskemping-ru.turbopages.org': 'ruskemping.ru',
 'karpenko--sasha-livejournal-com.turbopages.org': 'karpenko-sasha.livejournal.com',
 'captain--ts-livejournal-com.turbopages.org': 'captain-ts.livejournal.com',
 'farych-livejournal-com.turbopages.org': 'farych.livejournal.com',
 'alisha--96-livejournal-com.turbopages.org': 'alisha-96.livejournal.com',
 'allowerdy-livejournal-com.turbopages.org': 'allowerdy.livejournal.com',
 'borshkovsm-livejournal-com.turbopages.org': 'borshkovsm.livejournal.com',
 'kryuchkom-ru.turbopages.org': 'kryuchkom.ru',
 'agorohov-com.turbopages.org': 'agorohov.com',
 'eurolabinstrument-ru.turbopages.org': 'eurolabinstrument.ru',
 'mayakleather-ru.turbopages.org': 'mayakleather.ru',
 'miya--mu-livejournal-com.turbopages.org': 'miya-mu.livejournal.com',
 'a--lamtyugov-livejournal-com.turbopages.org': 'a-lamtyugov.livejournal.com',
 'kadykchanskiy-livejournal-com.turbopages.org': 'kadykchanskiy.livejournal.com',
 'teblog-ru.turbopages.org': 'teblog.ru',
 'aprosh-livejournal-com.turbopages.org': 'aprosh.livejournal.com',
 'huholya-livejournal-com.turbopages.org': 'huholya.livejournal.com',
 'impr0mptu-livejournal-com.turbopages.org': 'impr0mptu.livejournal.com',
 'dara--j-livejournal-com.turbopages.org': 'dara-j.livejournal.com',
 'major--c-livejournal-com.turbopages.org': 'major-c.livejournal.com',
 'vengersky-livejournal-com.turbopages.org': 'vengersky.livejournal.com',
 'adm--krgl-livejournal-com.turbopages.org': 'adm-krgl.livejournal.com',
 'stroyjurnal-livejournal-com.turbopages.org': 'stroyjurnal.livejournal.com',
 'stepandurnev-livejournal-com.turbopages.org': 'stepandurnev.livejournal.com',
 'alex--des-livejournal-com.turbopages.org': 'alex-des.livejournal.com',
 'ru--ecig-livejournal-com.turbopages.org': 'ru-ecig.livejournal.com',
 'inastana-kz.turbopages.org': 'inastana.kz',
 'volynska-livejournal-com.turbopages.org': 'volynska.livejournal.com',
 'leska--man-livejournal-com.turbopages.org': 'leska-man.livejournal.com',
 'svetkuban-livejournal-com.turbopages.org': 'svetkuban.livejournal.com',
 'chem--polezno-com.turbopages.org': 'chem-polezno.com',
 'bspb-livejournal-com.turbopages.org': 'bspb.livejournal.com',
 'enigmatic--past-livejournal-com.turbopages.org': 'enigmatic-past.livejournal.com',
 'le2009-livejournal-com.turbopages.org': 'le2009.livejournal.com',
 'kamsaddeco-com.turbopages.org': 'kamsaddeco.com',
 'juridicheskaja--konsultacija--online-ru.turbopages.org': 'juridicheskaja-konsultacija-online.ru',
 'yconsult-ru.turbopages.org': 'yconsult.ru',
 'kot--matraskin-livejournal-com.turbopages.org': 'kot-matraskin.livejournal.com',
 'glamyr--znaeteli-livejournal-com.turbopages.org': 'glamyr-znaeteli.livejournal.com',
 'zhd-online.turbopages.org': 'zhd.online',
 'markellov-livejournal-com.turbopages.org': 'markellov.livejournal.com',
 'vseotravmah-ru.turbopages.org': 'vseotravmah.ru',
 'imerica-ru.turbopages.org': 'imerica.ru',
 'prostosir-livejournal-com.turbopages.org': 'prostosir.livejournal.com',
 'zajcev--ushastyj-livejournal-com.turbopages.org': 'zajcev-ushastyj.livejournal.com',
 'kotlas-bezformata-com.turbopages.org': 'kotlas.bezformata.com',
 'volynko-livejournal-com.turbopages.org': 'volynko.livejournal.com',
 'uceps-ru.turbopages.org': 'uceps.ru',
 'mk--sakhalin-ru.turbopages.org': 'mk-sakhalin.ru',
 'ingwall-livejournal-com.turbopages.org': 'ingwall.livejournal.com',
 'tamara041-livejournal-com.turbopages.org': 'tamara041.livejournal.com',
 'diecast43-livejournal-com.turbopages.org': 'diecast43.livejournal.com',
 'artoul-livejournal-com.turbopages.org': 'artoul.livejournal.com',
 'biobalast-livejournal-com.turbopages.org': 'biobalast.livejournal.com',
 'uvo-ru.turbopages.org': 'uvo.ru',
 'd--34-livejournal-com.turbopages.org': 'd-34.livejournal.com',
 'realika-ru.turbopages.org': 'realika.ru',
 'dom--penoblokov-ru.turbopages.org': 'dom-penoblokov.ru',
 'sorokinaanna-livejournal-com.turbopages.org': 'sorokinaanna.livejournal.com',
 'decor-design.turbopages.org': 'decor.design',
 'tomsk-anumis-ru.turbopages.org': 'tomsk.anumis.ru',
 'vitri-com-ua.turbopages.org': 'vitri.com.ua',
 'elenapvjapan-livejournal-com.turbopages.org': 'elenapvjapan.livejournal.com',
 'mperial-livejournal-com.turbopages.org': 'mperial.livejournal.com',
 '2banket-ru.turbopages.org': '2banket.ru',
 'myvisato-ru.turbopages.org': 'myvisato.ru',
 'wi--tech-ru.turbopages.org': 'wi-tech.ru',
 'banki--dolgi-ru.turbopages.org': 'banki-dolgi.ru',
 'kshibaev-livejournal-com.turbopages.org': 'kshibaev.livejournal.com',
 'ikaketosdelano-ru.turbopages.org': 'ikaketosdelano.ru',
 'lk29-livejournal-com.turbopages.org': 'lk29.livejournal.com',
 'trolleybassist-livejournal-com.turbopages.org': 'trolleybassist.livejournal.com',
 'moj--oduvan4ik-livejournal-com.turbopages.org': 'moj-oduvan4ik.livejournal.com',
 'rusgay-livejournal-com.turbopages.org': 'rusgay.livejournal.com',
 '4int-livejournal-com.turbopages.org': '4int.livejournal.com',
 'bokovina-ru.turbopages.org': 'bokovina.ru',
 'car--el-livejournal-com.turbopages.org': 'car-el.livejournal.com',
 'klopabei-ru.turbopages.org': 'klopabei.ru',
 'timurprosto-livejournal-com.turbopages.org': 'timurprosto.livejournal.com',
 'poraionu-ru.turbopages.org': 'poraionu.ru',
 'siteogorod-ru.turbopages.org': 'siteogorod.ru',
 'w--o--f-ru.turbopages.org': 'w-o-f.ru',
 'dr--slabinsky-livejournal-com.turbopages.org': 'dr-slabinsky.livejournal.com',
 'tom--bolton-livejournal-com.turbopages.org': 'tom-bolton.livejournal.com',
 'salome--lou-livejournal-com.turbopages.org': 'salome-lou.livejournal.com',
 'juicepost-ru.turbopages.org': 'juicepost.ru',
 'cryptonews-website.turbopages.org': 'cryptonews.website',
 'yu--stesh-livejournal-com.turbopages.org': 'yu-stesh.livejournal.com',
 'vse--investicii-ru.turbopages.org': 'vse-investicii.ru',
 'evgggen-livejournal-com.turbopages.org': 'evgggen.livejournal.com',
 'otto--cazz-livejournal-com.turbopages.org': 'otto-cazz.livejournal.com',
 'qnet-livejournal-com.turbopages.org': 'qnet.livejournal.com',
 'aramaia-livejournal-com.turbopages.org': 'aramaia.livejournal.com',
 '101-livejournal-com.turbopages.org': '101.livejournal.com',
 'dr--rusi4-livejournal-com.turbopages.org': 'dr-rusi4.livejournal.com',
 'radiochipi-ru.turbopages.org': 'radiochipi.ru',
 'janunova-livejournal-com.turbopages.org': 'janunova.livejournal.com',
 'okartoshke-ru.turbopages.org': 'okartoshke.ru',
 'ptah57-livejournal-com.turbopages.org': 'ptah57.livejournal.com',
 'arno1251-livejournal-com.turbopages.org': 'arno1251.livejournal.com',
 'lesnoyskitalets-livejournal-com.turbopages.org': 'lesnoyskitalets.livejournal.com',
 'frolovchik-livejournal-com.turbopages.org': 'frolovchik.livejournal.com',
 'onethree-ru.turbopages.org': 'onethree.ru',
 'almazkayumov-livejournal-com.turbopages.org': 'almazkayumov.livejournal.com',
 'angevine--russe-livejournal-com.turbopages.org': 'angevine-russe.livejournal.com',
 'legarhan-livejournal-com.turbopages.org': 'legarhan.livejournal.com',
 'popartblog-ru.turbopages.org': 'popartblog.ru',
 'kapitan--flint1-livejournal-com.turbopages.org': 'kapitan-flint1.livejournal.com',
 'alco--gourmet-livejournal-com.turbopages.org': 'alco-gourmet.livejournal.com',
 'linalina20-livejournal-com.turbopages.org': 'linalina20.livejournal.com',
 'pink--mathilda-livejournal-com.turbopages.org': 'pink-mathilda.livejournal.com',
 'rozenbum-livejournal-com.turbopages.org': 'rozenbum.livejournal.com',
 'droband1975-livejournal-com.turbopages.org': 'droband1975.livejournal.com',
 'vbaden-livejournal-com.turbopages.org': 'vbaden.livejournal.com',
 'natalyalakshmi-livejournal-com.turbopages.org': 'natalyalakshmi.livejournal.com',
 'elena--gadanie-ru.turbopages.org': 'elena-gadanie.ru',
 'egorka--datskij-livejournal-com.turbopages.org': 'egorka-datskij.livejournal.com',
 'paslen-livejournal-com.turbopages.org': 'paslen.livejournal.com',
 'mogue-ru.turbopages.org': 'mogue.ru',
 'general--ivanov-livejournal-com.turbopages.org': 'general-ivanov.livejournal.com',
 'pavluhinoleg-livejournal-com.turbopages.org': 'pavluhinoleg.livejournal.com',
 'xotab--81-livejournal-com.turbopages.org': 'xotab-81.livejournal.com',
 'xpanyhovo-livejournal-com.turbopages.org': 'xpanyhovo.livejournal.com',
 'excursovod--ru-livejournal-com.turbopages.org': 'excursovod-ru.livejournal.com',
 'linch--ekb-livejournal-com.turbopages.org': 'linch-ekb.livejournal.com',
 'ontele2-ru.turbopages.org': 'ontele2.ru',
 'araratkafe-ru.turbopages.org': 'araratkafe.ru',
 'sdelaitak24-ru.turbopages.org': 'sdelaitak24.ru',
 'advokatbudko-livejournal-com.turbopages.org': 'advokatbudko.livejournal.com',
 'eska-livejournal-com.turbopages.org': 'eska.livejournal.com',
 'belomor999-livejournal-com.turbopages.org': 'belomor999.livejournal.com',
 'mozhga18-ru.turbopages.org': 'mozhga18.ru',
 'kuybishev-bezformata-com.turbopages.org': 'kuybishev.bezformata.com',
 'sredigor--01-livejournal-com.turbopages.org': 'sredigor-01.livejournal.com',
 'arnyfrazer-livejournal-com.turbopages.org': 'arnyfrazer.livejournal.com',
 'matveev12345-livejournal-com.turbopages.org': 'matveev12345.livejournal.com',
 'senorita--angie-livejournal-com.turbopages.org': 'senorita-angie.livejournal.com',
 'arky--titan-livejournal-com.turbopages.org': 'arky-titan.livejournal.com',
 'blinovyury-livejournal-com.turbopages.org': 'blinovyury.livejournal.com',
 'azlok-livejournal-com.turbopages.org': 'azlok.livejournal.com',
 'pra3dnuk-ru.turbopages.org': 'pra3dnuk.ru',
 'zaton50-livejournal-com.turbopages.org': 'zaton50.livejournal.com',
 'red-msk-ru.turbopages.org': 'red.msk.ru',
 'estel--oscora-livejournal-com.turbopages.org': 'estel-oscora.livejournal.com',
 'garmoniyastoron-ru.turbopages.org': 'garmoniyastoron.ru',
 'rohanwarrior-livejournal-com.turbopages.org': 'rohanwarrior.livejournal.com',
 'decker--ul-livejournal-com.turbopages.org': 'decker-ul.livejournal.com',
 'rtavto-livejournal-com.turbopages.org': 'rtavto.livejournal.com',
 'tistoriya-livejournal-com.turbopages.org': 'tistoriya.livejournal.com',
 'nashsovetik-ru.turbopages.org': 'nashsovetik.ru',
 'aselajn-livejournal-com.turbopages.org': 'aselajn.livejournal.com',
 'filin--dimitry-livejournal-com.turbopages.org': 'filin-dimitry.livejournal.com',
 'sechche-livejournal-com.turbopages.org': 'sechche.livejournal.com',
 'alexxa--moscow-livejournal-com.turbopages.org': 'alexxa-moscow.livejournal.com',
 'shantinathini-livejournal-com.turbopages.org': 'shantinathini.livejournal.com',
 'jewish--family-livejournal-com.turbopages.org': 'jewish-family.livejournal.com',
 'simptomov-com.turbopages.org': 'simptomov.com',
 'rating--credit--broker-ru.turbopages.org': 'rating-credit-broker.ru',
 'gimpart-org.turbopages.org': 'gimpart.org',
 'novorossiysk2-livejournal-com.turbopages.org': 'novorossiysk2.livejournal.com',
 'deit-name.turbopages.org': 'deit.name',
 'autozona74-ru.turbopages.org': 'autozona74.ru',
 'veteran--info-ru.turbopages.org': 'veteran-info.ru',
 'hazarinn-livejournal-com.turbopages.org': 'hazarinn.livejournal.com',
 'aqvium-ru.turbopages.org': 'aqvium.ru',
 'dima--pashchenko-livejournal-com.turbopages.org': 'dima-pashchenko.livejournal.com',
 'vladimir--l--s-livejournal-com.turbopages.org': 'vladimir-l-s.livejournal.com',
 'a--sharkov-livejournal-com.turbopages.org': 'a-sharkov.livejournal.com',
 'turtle--t-livejournal-com.turbopages.org': 'turtle-t.livejournal.com',
 'ustaliy-ru.turbopages.org': 'ustaliy.ru',
 'mrpossessor-livejournal-com.turbopages.org': 'mrpossessor.livejournal.com',
 'buljon-ru.turbopages.org': 'buljon.ru',
 'hobbikulinar-ru.turbopages.org': 'hobbikulinar.ru',
 'kipha-livejournal-com.turbopages.org': 'kipha.livejournal.com',
 'newsnhk-com.turbopages.org': 'newsnhk.com',
 'moiarussia-ru.turbopages.org': 'moiarussia.ru',
 'omega45-livejournal-com.turbopages.org': 'omega45.livejournal.com',
 'mountain--altay-livejournal-com.turbopages.org': 'mountain-altay.livejournal.com',
 'funnyking-livejournal-com.turbopages.org': 'funnyking.livejournal.com',
 'kurgan-bezformata-com.turbopages.org': 'kurgan.bezformata.com',
 'aruna-website.turbopages.org': 'aruna.website',
 'russianpost-livejournal-com.turbopages.org': 'russianpost.livejournal.com',
 'mirogoroda-com.turbopages.org': 'mirogoroda.com',
 'anthrax--urbex-livejournal-com.turbopages.org': 'anthrax-urbex.livejournal.com',
 'b1zz4r-livejournal-com.turbopages.org': 'b1zz4r.livejournal.com',
 'timotv-livejournal-com.turbopages.org': 'timotv.livejournal.com',
 'honzales-livejournal-com.turbopages.org': 'honzales.livejournal.com',
 'newarchit-ru.turbopages.org': 'newarchit.ru',
 'pawnstars-ru.turbopages.org': 'pawnstars.ru',
 'orexoed-ru.turbopages.org': 'orexoed.ru',
 'itsitizen-livejournal-com.turbopages.org': 'itsitizen.livejournal.com',
 'winejumper-livejournal-com.turbopages.org': 'winejumper.livejournal.com',
 'janis60-livejournal-com.turbopages.org': 'janis60.livejournal.com',
 'visionarys-livejournal-com.turbopages.org': 'visionarys.livejournal.com',
 'far01-livejournal-com.turbopages.org': 'far01.livejournal.com',
 'shtrafy-kz.turbopages.org': 'shtrafy.kz',
 'simkinbh-livejournal-com.turbopages.org': 'simkinbh.livejournal.com',
 'kreditnie--donory--ekaterinburg-ru.turbopages.org': 'kreditnie-donory-ekaterinburg.ru',
 'oldgoro-livejournal-com.turbopages.org': 'oldgoro.livejournal.com',
 'tattootoday-org.turbopages.org': 'tattootoday.org',
 'remidios--fine-livejournal-com.turbopages.org': 'remidios-fine.livejournal.com',
 'arny-ru.turbopages.org': 'arny.ru',
 'chesnochny-ru.turbopages.org': 'chesnochny.ru',
 'zipporu-livejournal-com.turbopages.org': 'zipporu.livejournal.com',
 'nikitapatison-livejournal-com.turbopages.org': 'nikitapatison.livejournal.com',
 'baikinumerologa-livejournal-com.turbopages.org': 'baikinumerologa.livejournal.com',
 'build5-ru.turbopages.org': 'build5.ru',
 'catherine--catty-livejournal-com.turbopages.org': 'catherine-catty.livejournal.com',
 'kitchendizajn-ru.turbopages.org': 'kitchendizajn.ru',
 'makeev--dv-livejournal-com.turbopages.org': 'makeev-dv.livejournal.com',
 'sixbisix-livejournal-com.turbopages.org': 'sixbisix.livejournal.com',
 'qqurl-ru.turbopages.org': 'qqurl.ru',
 'iphoneos-ru.turbopages.org': 'iphoneos.ru',
 'mineralnievodi-bezformata-com.turbopages.org': 'mineralnievodi.bezformata.com',
 'kudaufa-ru.turbopages.org': 'kudaufa.ru',
 'instylewithanna-livejournal-com.turbopages.org': 'instylewithanna.livejournal.com',
 'evgenij--lawyer-livejournal-com.turbopages.org': 'evgenij-lawyer.livejournal.com',
 'titan--spec-ru.turbopages.org': 'titan-spec.ru',
 'vikoworld-ru.turbopages.org': 'vikoworld.ru',
 'maksevg-livejournal-com.turbopages.org': 'maksevg.livejournal.com',
 'armystandard-ru.turbopages.org': 'armystandard.ru',
 'jonny--begood-livejournal-com.turbopages.org': 'jonny-begood.livejournal.com',
 'specavto-ru.turbopages.org': 'specavto.ru',
 'thebcj-ru.turbopages.org': 'thebcj.ru',
 'russianstoday-ru.turbopages.org': 'russianstoday.ru',
 'dinoinfo-ru.turbopages.org': 'dinoinfo.ru',
 'kialife-ru.turbopages.org': 'kialife.ru',
 'svetlyachok-livejournal-com.turbopages.org': 'svetlyachok.livejournal.com',
 'kottoblog-livejournal-com.turbopages.org': 'kottoblog.livejournal.com',
 'cryptor-net.turbopages.org': 'cryptor.net',
 'east--slavonians-livejournal-com.turbopages.org': 'east-slavonians.livejournal.com',
 'pute6vennik-livejournal-com.turbopages.org': 'pute6vennik.livejournal.com',
 'lena--frizz-livejournal-com.turbopages.org': 'lena-frizz.livejournal.com',
 'onepamop-livejournal-com.turbopages.org': 'onepamop.livejournal.com',
 'adelanta-livejournal-com.turbopages.org': 'adelanta.livejournal.com',
 'travelradar-world.turbopages.org': 'travelradar.world',
 'ust--kut-bezformata-com.turbopages.org': 'ust-kut.bezformata.com',
 'roman--novoselov-livejournal-com.turbopages.org': 'roman-novoselov.livejournal.com',
 'anna--kulakowska-livejournal-com.turbopages.org': 'anna-kulakowska.livejournal.com',
 'navimann-livejournal-com.turbopages.org': 'navimann.livejournal.com',
 'austria--today-ru.turbopages.org': 'austria-today.ru',
 'tap--ac-livejournal-com.turbopages.org': 'tap-ac.livejournal.com',
 'kunashak-bezformata-com.turbopages.org': 'kunashak.bezformata.com',
 'maximsjaber-livejournal-com.turbopages.org': 'maximsjaber.livejournal.com',
 'pnz-ru.turbopages.org': 'pnz.ru',
 'belan--olga-livejournal-com.turbopages.org': 'belan-olga.livejournal.com',
 'granvini-livejournal-com.turbopages.org': 'granvini.livejournal.com',
 'kseniya--1488-livejournal-com.turbopages.org': 'kseniya-1488.livejournal.com',
 'mk--tuva-ru.turbopages.org': 'mk-tuva.ru',
 'graqdanin-livejournal-com.turbopages.org': 'graqdanin.livejournal.com',
 'eldercare-ru.turbopages.org': 'eldercare.ru',
 'gkh11-ru.turbopages.org': 'gkh11.ru',
 'lubernet-ru.turbopages.org': 'lubernet.ru',
 'pulemjotov-livejournal-com.turbopages.org': 'pulemjotov.livejournal.com',
 'vrach--med-ru.turbopages.org': 'vrach-med.ru',
 'prorok70-livejournal-com.turbopages.org': 'prorok70.livejournal.com',
 'masteriks-livejournal-com.turbopages.org': 'masteriks.livejournal.com',
 'autodvs-ru.turbopages.org': 'autodvs.ru',
 'srostova-ru.turbopages.org': 'srostova.ru',
 'vsekidki-ru.turbopages.org': 'vsekidki.ru',
 'chastnie--zaimy--ufa-ru.turbopages.org': 'chastnie-zaimy-ufa.ru',
 'azart--ude-livejournal-com.turbopages.org': 'azart-ude.livejournal.com',
 'energyfm-ru.turbopages.org': 'energyfm.ru',
 'electricavdome-ru.turbopages.org': 'electricavdome.ru',
 'funwood-livejournal-com.turbopages.org': 'funwood.livejournal.com',
 'basil76--moscow-livejournal-com.turbopages.org': 'basil76-moscow.livejournal.com',
 'tesoro82-livejournal-com.turbopages.org': 'tesoro82.livejournal.com',
 'alexas21-livejournal-com.turbopages.org': 'alexas21.livejournal.com',
 'dim--sale-livejournal-com.turbopages.org': 'dim-sale.livejournal.com',
 'veg--article-livejournal-com.turbopages.org': 'veg-article.livejournal.com',
 'shurigin-livejournal-com.turbopages.org': 'shurigin.livejournal.com',
 'vikt--myasnikov-livejournal-com.turbopages.org': 'vikt-myasnikov.livejournal.com',
 'rasskazchic93-livejournal-com.turbopages.org': 'rasskazchic93.livejournal.com',
 'donrf-livejournal-com.turbopages.org': 'donrf.livejournal.com',
 'strangernn-livejournal-com.turbopages.org': 'strangernn.livejournal.com',
 'gelya--leopold-livejournal-com.turbopages.org': 'gelya-leopold.livejournal.com',
 'jrchernik-livejournal-com.turbopages.org': 'jrchernik.livejournal.com',
 'rentalhome-ru.turbopages.org': 'rentalhome.ru',
 'illumon-ru.turbopages.org': 'illumon.ru',
 'medic--23-ru.turbopages.org': 'medic-23.ru',
 'islamecology-livejournal-com.turbopages.org': 'islamecology.livejournal.com',
 'odessa--avant-livejournal-com.turbopages.org': 'odessa-avant.livejournal.com',
 'kahhar--786-livejournal-com.turbopages.org': 'kahhar-786.livejournal.com',
 'kogdavyydet-ru.turbopages.org': 'kogdavyydet.ru',
 'nlpt-livejournal-com.turbopages.org': 'nlpt.livejournal.com',
 'annakosticina-livejournal-com.turbopages.org': 'annakosticina.livejournal.com',
 'prostatitno-ru.turbopages.org': 'prostatitno.ru',
 'ge17-livejournal-com.turbopages.org': 'ge17.livejournal.com',
 'brekotin-livejournal-com.turbopages.org': 'brekotin.livejournal.com',
 'nikutyonok-livejournal-com.turbopages.org': 'nikutyonok.livejournal.com',
 'plita-guru.turbopages.org': 'plita.guru',
 'pmn--2-livejournal-com.turbopages.org': 'pmn-2.livejournal.com',
 'nijnevartovsk-bezformata-com.turbopages.org': 'nijnevartovsk.bezformata.com',
 'wire--phones-ru.turbopages.org': 'wire-phones.ru',
 'koreanst-livejournal-com.turbopages.org': 'koreanst.livejournal.com',
 'pioneer--lj-livejournal-com.turbopages.org': 'pioneer-lj.livejournal.com',
 'rio--travel-livejournal-com.turbopages.org': 'rio-travel.livejournal.com',
 'fan--project-livejournal-com.turbopages.org': 'fan-project.livejournal.com',
 'forex--total-ru.turbopages.org': 'forex-total.ru',
 'elabuga-bezformata-com.turbopages.org': 'elabuga.bezformata.com',
 'radyjnai--ovca-livejournal-com.turbopages.org': 'radyjnai-ovca.livejournal.com',
 'velma--l-ru.turbopages.org': 'velma-l.ru',
 'rorocho-livejournal-com.turbopages.org': 'rorocho.livejournal.com',
 'hotgeo-ru.turbopages.org': 'hotgeo.ru',
 'book--vs--life-livejournal-com.turbopages.org': 'book-vs-life.livejournal.com',
 'ntagil-bezformata-com.turbopages.org': 'ntagil.bezformata.com',
 'gaikin512-livejournal-com.turbopages.org': 'gaikin512.livejournal.com',
 'your--food--today-livejournal-com.turbopages.org': 'your-food-today.livejournal.com',
 'wire--ru-livejournal-com.turbopages.org': 'wire-ru.livejournal.com',
 'seozp-ru.turbopages.org': 'seozp.ru',
 'cymoorka-livejournal-com.turbopages.org': 'cymoorka.livejournal.com',
 'concluder-livejournal-com.turbopages.org': 'concluder.livejournal.com',
 'vethij-livejournal-com.turbopages.org': 'vethij.livejournal.com',
 'ya--kristinka-livejournal-com.turbopages.org': 'ya-kristinka.livejournal.com',
 'shelkovo-bezformata-com.turbopages.org': 'shelkovo.bezformata.com',
 'christ--civ-livejournal-com.turbopages.org': 'christ-civ.livejournal.com',
 'oleg--devyatkin-livejournal-com.turbopages.org': 'oleg-devyatkin.livejournal.com',
 'turportal63-ru.turbopages.org': 'turportal63.ru',
 'elista-bezformata-com.turbopages.org': 'elista.bezformata.com',
 'drova--darom-ru.turbopages.org': 'drova-darom.ru',
 'freaks--fanclub-livejournal-com.turbopages.org': 'freaks-fanclub.livejournal.com',
 'msovetov-ru.turbopages.org': 'msovetov.ru',
 'orehi--zerna-ru.turbopages.org': 'orehi-zerna.ru',
 'anonyymm-livejournal-com.turbopages.org': 'anonyymm.livejournal.com',
 'kotcredit-com-ua.turbopages.org': 'kotcredit.com.ua',
 'moinogi-ru.turbopages.org': 'moinogi.ru',
 'klondike-livejournal-com.turbopages.org': 'klondike.livejournal.com',
 'tatyana--ayupova-livejournal-com.turbopages.org': 'tatyana-ayupova.livejournal.com',
 'vasilisa--prekrasnaja-ru.turbopages.org': 'vasilisa-prekrasnaja.ru',
 'anna--frid-livejournal-com.turbopages.org': 'anna-frid.livejournal.com',
 'feanoturi-livejournal-com.turbopages.org': 'feanoturi.livejournal.com',
 'orhidea777-livejournal-com.turbopages.org': 'orhidea777.livejournal.com',
 'ary-livejournal-com.turbopages.org': 'ary.livejournal.com',
 'zeleninsergey-livejournal-com.turbopages.org': 'zeleninsergey.livejournal.com',
 'cyrillitsa-ru.turbopages.org': 'cyrillitsa.ru',
 'elegantchikova-livejournal-com.turbopages.org': 'elegantchikova.livejournal.com',
 'jjenson-livejournal-com.turbopages.org': 'jjenson.livejournal.com',
 'gribkovye--zabolevaniya-com.turbopages.org': 'gribkovye-zabolevaniya.com',
 'anlazz-livejournal-com.turbopages.org': 'anlazz.livejournal.com',
 'yanosha-livejournal-com.turbopages.org': 'yanosha.livejournal.com',
 'zheldor--city-livejournal-com.turbopages.org': 'zheldor-city.livejournal.com',
 'rushtraf-com.turbopages.org': 'rushtraf.com',
 'zapret--no-livejournal-com.turbopages.org': 'zapret-no.livejournal.com',
 'dervish666-livejournal-com.turbopages.org': 'dervish666.livejournal.com',
 'comp--profi-com.turbopages.org': 'comp-profi.com',
 'port--alexandra3-livejournal-com.turbopages.org': 'port-alexandra3.livejournal.com',
 'anapa-bezformata-com.turbopages.org': 'anapa.bezformata.com',
 'healthywomen-ru.turbopages.org': 'healthywomen.ru',
 'pavell743-livejournal-com.turbopages.org': 'pavell743.livejournal.com',
 'sensi--46-livejournal-com.turbopages.org': 'sensi-46.livejournal.com',
 'xn--86--eddakcg1a6bvui0d.turbopages.org': 'xn-86-eddakcg1a6bvui0d',
 'rogwu-livejournal-com.turbopages.org': 'rogwu.livejournal.com',
 'budtezzdorovy-ru.turbopages.org': 'budtezzdorovy.ru',
 'makdim-livejournal-com.turbopages.org': 'makdim.livejournal.com',
 'marketsignal-ru.turbopages.org': 'marketsignal.ru',
 'baby--gourmand-livejournal-com.turbopages.org': 'baby-gourmand.livejournal.com',
 'taxi--reestr-ru.turbopages.org': 'taxi-reestr.ru',
 'whiskytalk-ru.turbopages.org': 'whiskytalk.ru',
 'superbarok-livejournal-com.turbopages.org': 'superbarok.livejournal.com',
 'tservice23-ru.turbopages.org': 'tservice23.ru',
 'zhig--ailo-livejournal-com.turbopages.org': 'zhig-ailo.livejournal.com',
 'edem88-livejournal-com.turbopages.org': 'edem88.livejournal.com',
 'archerowiki-ru.turbopages.org': 'archerowiki.ru',
 'cheaptrip-livejournal-com.turbopages.org': 'cheaptrip.livejournal.com',
 'voronezh-big--book--med-ru.turbopages.org': 'voronezh.big-book-med.ru',
 'niknik--2014-livejournal-com.turbopages.org': 'niknik-2014.livejournal.com',
 'dibr-livejournal-com.turbopages.org': 'dibr.livejournal.com',
 'bardokin-livejournal-com.turbopages.org': 'bardokin.livejournal.com',
 'forum-rus--outboard-ru.turbopages.org': 'forum.rus-outboard.ru',
 'narofominsk-bezformata-com.turbopages.org': 'narofominsk.bezformata.com',
 'sventameat-ru.turbopages.org': 'sventameat.ru',
 'libier-livejournal-com.turbopages.org': 'libier.livejournal.com',
 'lashevchenko-livejournal-com.turbopages.org': 'lashevchenko.livejournal.com',
 'bezprivivok-livejournal-com.turbopages.org': 'bezprivivok.livejournal.com',
 'viva--krasota-ru.turbopages.org': 'viva-krasota.ru',
 'gluhovski--igor-livejournal-com.turbopages.org': 'gluhovski-igor.livejournal.com',
 'trudikk-livejournal-com.turbopages.org': 'trudikk.livejournal.com',
 'note--buki-livejournal-com.turbopages.org': 'note-buki.livejournal.com',
 'd--chebatkov-livejournal-com.turbopages.org': 'd-chebatkov.livejournal.com',
 'lunteg-livejournal-com.turbopages.org': 'lunteg.livejournal.com',
 'ladycleo-ru.turbopages.org': 'ladycleo.ru',
 'judepinguin-livejournal-com.turbopages.org': 'judepinguin.livejournal.com',
 'isaak--rozovsky-livejournal-com.turbopages.org': 'isaak-rozovsky.livejournal.com',
 'engtopic-ru.turbopages.org': 'engtopic.ru',
 'android101-ru.turbopages.org': 'android101.ru',
 'ekobatarei-ru.turbopages.org': 'ekobatarei.ru',
 'renen-ru.turbopages.org': 'renen.ru',
 'aldusku-livejournal-com.turbopages.org': 'aldusku.livejournal.com',
 'piragis-ru.turbopages.org': 'piragis.ru',
 'altruii-livejournal-com.turbopages.org': 'altruii.livejournal.com',
 'sr--maks-livejournal-com.turbopages.org': 'sr-maks.livejournal.com',
 'bez--santehnika-ru.turbopages.org': 'bez-santehnika.ru',
 'astrahanfoto-livejournal-com.turbopages.org': 'astrahanfoto.livejournal.com',
 'tarabanki-ru.turbopages.org': 'tarabanki.ru',
 'steblya--kam-livejournal-com.turbopages.org': 'steblya-kam.livejournal.com',
 'icineman-livejournal-com.turbopages.org': 'icineman.livejournal.com',
 'valuh-livejournal-com.turbopages.org': 'valuh.livejournal.com',
 'online--dendy-ru.turbopages.org': 'online-dendy.ru',
 'shockplus-livejournal-com.turbopages.org': 'shockplus.livejournal.com',
 '1--11-info.turbopages.org': '1-11.info',
 'arunia--studio-livejournal-com.turbopages.org': 'arunia-studio.livejournal.com',
 'dikan2010-livejournal-com.turbopages.org': 'dikan2010.livejournal.com',
 'traveler62rus-livejournal-com.turbopages.org': 'traveler62rus.livejournal.com',
 'pishchulin-livejournal-com.turbopages.org': 'pishchulin.livejournal.com',
 'arisu117-livejournal-com.turbopages.org': 'arisu117.livejournal.com',
 'lenar--huzhin-livejournal-com.turbopages.org': 'lenar-huzhin.livejournal.com',
 'armyreporter--ru-livejournal-com.turbopages.org': 'armyreporter-ru.livejournal.com',
 'drevniy--daos-livejournal-com.turbopages.org': 'drevniy-daos.livejournal.com',
 'grafo--navigator-livejournal-com.turbopages.org': 'grafo-navigator.livejournal.com',
 'solncepoksa-livejournal-com.turbopages.org': 'solncepoksa.livejournal.com',
 '1epilyaciya-ru.turbopages.org': '1epilyaciya.ru',
 'anatoly--tokarev-livejournal-com.turbopages.org': 'anatoly-tokarev.livejournal.com',
 'lyamusik-ru.turbopages.org': 'lyamusik.ru',
 'alexborodin-livejournal-com.turbopages.org': 'alexborodin.livejournal.com',
 'danitta-livejournal-com.turbopages.org': 'danitta.livejournal.com',
 'leon--po-livejournal-com.turbopages.org': 'leon-po.livejournal.com',
 'hortus--ru-livejournal-com.turbopages.org': 'hortus-ru.livejournal.com',
 'man--go--ost-livejournal-com.turbopages.org': 'man-go-ost.livejournal.com',
 'politike-ru.turbopages.org': 'politike.ru',
 'temryuk-bezformata-com.turbopages.org': 'temryuk.bezformata.com',
 'dormouse--a-livejournal-com.turbopages.org': 'dormouse-a.livejournal.com',
 'yuvit-livejournal-com.turbopages.org': 'yuvit.livejournal.com',
 'ostrovok-livejournal-com.turbopages.org': 'ostrovok.livejournal.com',
 'tasting--of--life-livejournal-com.turbopages.org': 'tasting-of-life.livejournal.com',
 'best--crm-ru.turbopages.org': 'best-crm.ru',
 'kirill--rozhkov-livejournal-com.turbopages.org': 'kirill-rozhkov.livejournal.com',
 'intellectology-livejournal-com.turbopages.org': 'intellectology.livejournal.com',
 'rus--chinese-livejournal-com.turbopages.org': 'rus-chinese.livejournal.com',
 'marawarren-livejournal-com.turbopages.org': 'marawarren.livejournal.com',
 'lazarevs--007-livejournal-com.turbopages.org': 'lazarevs-007.livejournal.com',
 'zodchi1-livejournal-com.turbopages.org': 'zodchi1.livejournal.com',
 'natalija44-livejournal-com.turbopages.org': 'natalija44.livejournal.com',
 'nicksob-livejournal-com.turbopages.org': 'nicksob.livejournal.com',
 'radiant--kristal-livejournal-com.turbopages.org': 'radiant-kristal.livejournal.com',
 'hypnosismoscow-livejournal-com.turbopages.org': 'hypnosismoscow.livejournal.com',
 'kromak-livejournal-com.turbopages.org': 'kromak.livejournal.com',
 'pobeda--nvr-ru.turbopages.org': 'pobeda-nvr.ru',
 'andreimos-livejournal-com.turbopages.org': 'andreimos.livejournal.com',
 'vish--m-livejournal-com.turbopages.org': 'vish-m.livejournal.com',
 'sapunov-livejournal-com.turbopages.org': 'sapunov.livejournal.com',
 'blagogon--ru-livejournal-com.turbopages.org': 'blagogon-ru.livejournal.com',
 'gypergidroz-ru.turbopages.org': 'gypergidroz.ru',
 'millerovo-bezformata-com.turbopages.org': 'millerovo.bezformata.com',
 'sergeevav11-ru.turbopages.org': 'sergeevav11.ru',
 'ledibibliotekar-livejournal-com.turbopages.org': 'ledibibliotekar.livejournal.com',
 'pokupka--5-ru.turbopages.org': 'pokupka-5.ru',
 'hobbylaby-ru.turbopages.org': 'hobbylaby.ru',
 'irena--handmade-ru.turbopages.org': 'irena-handmade.ru',
 'crawford--ghast-livejournal-com.turbopages.org': 'crawford-ghast.livejournal.com',
 'kazakh--zerno-net.turbopages.org': 'kazakh-zerno.net',
 'hosta-discus--club-ru.turbopages.org': 'hosta.discus-club.ru',
 'kirpichspec-ru.turbopages.org': 'kirpichspec.ru',
 'gen--shtab-livejournal-com.turbopages.org': 'gen-shtab.livejournal.com',
 'eniki--beniki-livejournal-com.turbopages.org': 'eniki-beniki.livejournal.com',
 'proweb63-ru.turbopages.org': 'proweb63.ru',
 'gamer005-livejournal-com.turbopages.org': 'gamer005.livejournal.com',
 'teh--int-ru.turbopages.org': 'teh-int.ru',
 'bloguser-ru.turbopages.org': 'bloguser.ru',
 'allmessengers-ru.turbopages.org': 'allmessengers.ru',
 'agrobiotech-livejournal-com.turbopages.org': 'agrobiotech.livejournal.com',
 'podkormka-guru.turbopages.org': 'podkormka.guru',
 'pol--hot-ru.turbopages.org': 'pol-hot.ru',
 'feelsmell-ru.turbopages.org': 'feelsmell.ru',
 'zvezdochka--ru-livejournal-com.turbopages.org': 'zvezdochka-ru.livejournal.com',
 'alexpsw-livejournal-com.turbopages.org': 'alexpsw.livejournal.com',
 'razomir-livejournal-com.turbopages.org': 'razomir.livejournal.com',
 'mapdesign-livejournal-com.turbopages.org': 'mapdesign.livejournal.com',
 'zoryana--1955-livejournal-com.turbopages.org': 'zoryana-1955.livejournal.com',
 'moyaindia-ru.turbopages.org': 'moyaindia.ru',
 'kaluganews-com.turbopages.org': 'kaluganews.com',
 'is--pl89-livejournal-com.turbopages.org': 'is-pl89.livejournal.com',
 'consumerista-livejournal-com.turbopages.org': 'consumerista.livejournal.com',
 'ele--na--61-livejournal-com.turbopages.org': 'ele-na-61.livejournal.com',
 'kostya--bolotov-livejournal-com.turbopages.org': 'kostya-bolotov.livejournal.com',
 'wik--end-com.turbopages.org': 'wik-end.com',
 'mirskim-livejournal-com.turbopages.org': 'mirskim.livejournal.com',
 'dimka--jd-livejournal-com.turbopages.org': 'dimka-jd.livejournal.com',
 'vdekrete-info.turbopages.org': 'vdekrete.info',
 'viberer-site.turbopages.org': 'viberer.site',
 'n--perfilova-livejournal-com.turbopages.org': 'n-perfilova.livejournal.com',
 'yaremchuk--v-livejournal-com.turbopages.org': 'yaremchuk-v.livejournal.com',
 'otovarah-ru.turbopages.org': 'otovarah.ru',
 'alco--safe-ru.turbopages.org': 'alco-safe.ru',
 'nik--online-ru.turbopages.org': 'nik-online.ru',
 'survincity-ru.turbopages.org': 'survincity.ru',
 'philip--marlow-livejournal-com.turbopages.org': 'philip-marlow.livejournal.com',
 'altereos-livejournal-com.turbopages.org': 'altereos.livejournal.com',
 'homecosmetics-livejournal-com.turbopages.org': 'homecosmetics.livejournal.com',
 'khimki-livejournal-com.turbopages.org': 'khimki.livejournal.com',
 '2012sillybilly-livejournal-com.turbopages.org': '2012sillybilly.livejournal.com',
 'chehov--life-ru.turbopages.org': 'chehov-life.ru',
 'msk--tsaritsino-livejournal-com.turbopages.org': 'msk-tsaritsino.livejournal.com',
 'kodecs--rf-ru.turbopages.org': 'kodecs-rf.ru',
 'shuwany-livejournal-com.turbopages.org': 'shuwany.livejournal.com',
 'vse--pro--detstvo-ru.turbopages.org': 'vse-pro-detstvo.ru',
 'ekaterina--demko-livejournal-com.turbopages.org': 'ekaterina-demko.livejournal.com',
 'fashionlife-livejournal-com.turbopages.org': 'fashionlife.livejournal.com',
 'successmed-ru.turbopages.org': 'successmed.ru',
 'dobergroup-livejournal-com.turbopages.org': 'dobergroup.livejournal.com',
 'andy--777-livejournal-com.turbopages.org': 'andy-777.livejournal.com',
 'itnote--tlt-ru.turbopages.org': 'itnote-tlt.ru',
 'onyx007-livejournal-com.turbopages.org': 'onyx007.livejournal.com',
 'world--bar-livejournal-com.turbopages.org': 'world-bar.livejournal.com',
 'mish--gunner-livejournal-com.turbopages.org': 'mish-gunner.livejournal.com',
 'inspector--torez-livejournal-com.turbopages.org': 'inspector-torez.livejournal.com',
 'corex12-livejournal-com.turbopages.org': 'corex12.livejournal.com',
 'marina--shandar-livejournal-com.turbopages.org': 'marina-shandar.livejournal.com',
 'ilyabim-livejournal-com.turbopages.org': 'ilyabim.livejournal.com',
 'ru--postcrossing-livejournal-com.turbopages.org': 'ru-postcrossing.livejournal.com',
 'd--stroz-livejournal-com.turbopages.org': 'd-stroz.livejournal.com',
 'reallife-kz.turbopages.org': 'reallife.kz',
 'chtets--antony-livejournal-com.turbopages.org': 'chtets-antony.livejournal.com',
 'andronnik26-livejournal-com.turbopages.org': 'andronnik26.livejournal.com',
 'galaxy2012-livejournal-com.turbopages.org': 'galaxy2012.livejournal.com',
 'taravarma-livejournal-com.turbopages.org': 'taravarma.livejournal.com',
 'f--school-livejournal-com.turbopages.org': 'f-school.livejournal.com',
 'chern--molnija-livejournal-com.turbopages.org': 'chern-molnija.livejournal.com',
 'ru-hrodna-life.turbopages.org': 'ru.hrodna.life',
 'cheepok-livejournal-com.turbopages.org': 'cheepok.livejournal.com',
 'time--and--life-livejournal-com.turbopages.org': 'time-and-life.livejournal.com',
 'la--cruz-livejournal-com.turbopages.org': 'la-cruz.livejournal.com',
 'alecktimakov-livejournal-com.turbopages.org': 'alecktimakov.livejournal.com',
 'vodani4--ey-livejournal-com.turbopages.org': 'vodani4-ey.livejournal.com',
 'medoperacii-com.turbopages.org': 'medoperacii.com',
 'masterkrasok-ru.turbopages.org': 'masterkrasok.ru',
 'welcometoaltai-livejournal-com.turbopages.org': 'welcometoaltai.livejournal.com',
 'goodspider-livejournal-com.turbopages.org': 'goodspider.livejournal.com',
 'tebedam-livejournal-com.turbopages.org': 'tebedam.livejournal.com',
 'testanaliz-ru.turbopages.org': 'testanaliz.ru',
 'old--bubble--gum-livejournal-com.turbopages.org': 'old-bubble-gum.livejournal.com',
 'memo--projects-livejournal-com.turbopages.org': 'memo-projects.livejournal.com',
 'kpms-ru.turbopages.org': 'kpms.ru',
 'naissur1986-livejournal-com.turbopages.org': 'naissur1986.livejournal.com',
 'skunk--69-livejournal-com.turbopages.org': 'skunk-69.livejournal.com',
 'my--mangaanime-livejournal-com.turbopages.org': 'my-mangaanime.livejournal.com',
 'irina--seriel-livejournal-com.turbopages.org': 'irina-seriel.livejournal.com',
 'antidictatura-livejournal-com.turbopages.org': 'antidictatura.livejournal.com',
 'cryptorussia-ru.turbopages.org': 'cryptorussia.ru',
 'dmitriysh-livejournal-com.turbopages.org': 'dmitriysh.livejournal.com',
 'ktovdele-ru.turbopages.org': 'ktovdele.ru',
 'elevatorman--ua-livejournal-com.turbopages.org': 'elevatorman-ua.livejournal.com',
 'oleg--lisowski-livejournal-com.turbopages.org': 'oleg-lisowski.livejournal.com',
 'interface31-livejournal-com.turbopages.org': 'interface31.livejournal.com',
 'ramer-livejournal-com.turbopages.org': 'ramer.livejournal.com',
 'svhostel-ru.turbopages.org': 'svhostel.ru',
 'seoded-livejournal-com.turbopages.org': 'seoded.livejournal.com',
 'pandukht-livejournal-com.turbopages.org': 'pandukht.livejournal.com',
 'nilulin-livejournal-com.turbopages.org': 'nilulin.livejournal.com',
 'nuvba-livejournal-com.turbopages.org': 'nuvba.livejournal.com',
 'otradnaya-bezformata-com.turbopages.org': 'otradnaya.bezformata.com',
 'dmgusev-livejournal-com.turbopages.org': 'dmgusev.livejournal.com',
 'dolgorukiy-livejournal-com.turbopages.org': 'dolgorukiy.livejournal.com',
 'bottgg-livejournal-com.turbopages.org': 'bottgg.livejournal.com',
 'anti--mrakobes-livejournal-com.turbopages.org': 'anti-mrakobes.livejournal.com',
 'yrswoodwork-ru.turbopages.org': 'yrswoodwork.ru',
 'techguru--remont-ru.turbopages.org': 'techguru-remont.ru',
 'sp--off-livejournal-com.turbopages.org': 'sp-off.livejournal.com',
 'tytyvainen-livejournal-com.turbopages.org': 'tytyvainen.livejournal.com',
 'usolie-irk-today.turbopages.org': 'usolie.irk.today',
 'eskulap--plus-ru.turbopages.org': 'eskulap-plus.ru',
 'brjunetka-ru.turbopages.org': 'brjunetka.ru',
 'maikop-bezformata-com.turbopages.org': 'maikop.bezformata.com',
 'nvk63-livejournal-com.turbopages.org': 'nvk63.livejournal.com',
 'lovelyman111-livejournal-com.turbopages.org': 'lovelyman111.livejournal.com',
 'osnaz--cikle-livejournal-com.turbopages.org': 'osnaz-cikle.livejournal.com',
 '0642-ua.turbopages.org': '0642.ua',
 'totalhub-ru.turbopages.org': 'totalhub.ru',
 'solomarina2002-livejournal-com.turbopages.org': 'solomarina2002.livejournal.com',
 'ams--1-livejournal-com.turbopages.org': 'ams-1.livejournal.com',
 'kriminalisty-ru.turbopages.org': 'kriminalisty.ru',
 '30--top-ru.turbopages.org': '30-top.ru',
 'autoprodor-ru.turbopages.org': 'autoprodor.ru',
 'megard-livejournal-com.turbopages.org': 'megard.livejournal.com',
 'readfootball-com.turbopages.org': 'readfootball.com',
 'tarot--aenigma-livejournal-com.turbopages.org': 'tarot-aenigma.livejournal.com',
 'ru--jogging-livejournal-com.turbopages.org': 'ru-jogging.livejournal.com',
 'colonel--baranez-livejournal-com.turbopages.org': 'colonel-baranez.livejournal.com',
 'alenakogotkova-livejournal-com.turbopages.org': 'alenakogotkova.livejournal.com',
 'onrockwave-com.turbopages.org': 'onrockwave.com',
 'massaraksh10-livejournal-com.turbopages.org': 'massaraksh10.livejournal.com',
 'komodo74-livejournal-com.turbopages.org': 'komodo74.livejournal.com',
 'nimfeechka-livejournal-com.turbopages.org': 'nimfeechka.livejournal.com',
 'giedrius--ru-livejournal-com.turbopages.org': 'giedrius-ru.livejournal.com',
 'hautiev--sh-livejournal-com.turbopages.org': 'hautiev-sh.livejournal.com',
 'dobropesik-ru.turbopages.org': 'dobropesik.ru',
 'we--checked-livejournal-com.turbopages.org': 'we-checked.livejournal.com',
 'm--fishman-livejournal-com.turbopages.org': 'm-fishman.livejournal.com',
 'queens--of--old-livejournal-com.turbopages.org': 'queens-of-old.livejournal.com',
 'crithin-livejournal-com.turbopages.org': 'crithin.livejournal.com',
 'detitambov-ru.turbopages.org': 'detitambov.ru',
 'minecool-ru.turbopages.org': 'minecool.ru',
 'genshinmania-ru.turbopages.org': 'genshinmania.ru',
 'kat--dallas-livejournal-com.turbopages.org': 'kat-dallas.livejournal.com',
 'lisenock-livejournal-com.turbopages.org': 'lisenock.livejournal.com',
 'alex--ermak-livejournal-com.turbopages.org': 'alex-ermak.livejournal.com',
 'gudea-livejournal-com.turbopages.org': 'gudea.livejournal.com',
 'michaelson--1-livejournal-com.turbopages.org': 'michaelson-1.livejournal.com',
 'belorechensk-bezformata-com.turbopages.org': 'belorechensk.bezformata.com',
 'timbuktoo-livejournal-com.turbopages.org': 'timbuktoo.livejournal.com',
 'evo--lutio2-livejournal-com.turbopages.org': 'evo-lutio2.livejournal.com',
 'sviridenkova-livejournal-com.turbopages.org': 'sviridenkova.livejournal.com',
 'facte-ru.turbopages.org': 'facte.ru',
 'respublika--news-livejournal-com.turbopages.org': 'respublika-news.livejournal.com',
 'ella--lutio-livejournal-com.turbopages.org': 'ella-lutio.livejournal.com',
 'selena--to--alex-livejournal-com.turbopages.org': 'selena-to-alex.livejournal.com',
 'ksun--xxx-livejournal-com.turbopages.org': 'ksun-xxx.livejournal.com',
 'konkovo--msk-livejournal-com.turbopages.org': 'konkovo-msk.livejournal.com',
 'ltybc278-livejournal-com.turbopages.org': 'ltybc278.livejournal.com',
 'viplissa-livejournal-com.turbopages.org': 'viplissa.livejournal.com',
 'arefiev--dm-livejournal-com.turbopages.org': 'arefiev-dm.livejournal.com',
 'lider99-livejournal-com.turbopages.org': 'lider99.livejournal.com',
 'kaminproekt-ru.turbopages.org': 'kaminproekt.ru',
 'merengue-livejournal-com.turbopages.org': 'merengue.livejournal.com',
 'ramha39-livejournal-com.turbopages.org': 'ramha39.livejournal.com',
 'ladariy-livejournal-com.turbopages.org': 'ladariy.livejournal.com',
 'roeline-livejournal-com.turbopages.org': 'roeline.livejournal.com',
 'taksa--sobaka-ru.turbopages.org': 'taksa-sobaka.ru',
 'tommy--mor-livejournal-com.turbopages.org': 'tommy-mor.livejournal.com',
 'yaturistka-ru.turbopages.org': 'yaturistka.ru',
 'sarnitskaya-livejournal-com.turbopages.org': 'sarnitskaya.livejournal.com',
 'vilisov--andrey-livejournal-com.turbopages.org': 'vilisov-andrey.livejournal.com',
 'syshilka-livejournal-com.turbopages.org': 'syshilka.livejournal.com',
 'chernovik-livejournal-com.turbopages.org': 'chernovik.livejournal.com',
 'niramas-livejournal-com.turbopages.org': 'niramas.livejournal.com',
 'save--sp--burg-livejournal-com.turbopages.org': 'save-sp-burg.livejournal.com',
 'e--vajs-livejournal-com.turbopages.org': 'e-vajs.livejournal.com',
 'alexkaravaev-livejournal-com.turbopages.org': 'alexkaravaev.livejournal.com',
 'biglik-ru.turbopages.org': 'biglik.ru',
 'alexrayu-livejournal-com.turbopages.org': 'alexrayu.livejournal.com',
 'unibrom-livejournal-com.turbopages.org': 'unibrom.livejournal.com',
 'wikidengi-ru.turbopages.org': 'wikidengi.ru',
 'kseniapo-livejournal-com.turbopages.org': 'kseniapo.livejournal.com',
 'neprostogluk-livejournal-com.turbopages.org': 'neprostogluk.livejournal.com',
 'isla--margarita-livejournal-com.turbopages.org': 'isla-margarita.livejournal.com',
 'lubimoe-menu.turbopages.org': 'lubimoe.menu',
 'ozeroff-livejournal-com.turbopages.org': 'ozeroff.livejournal.com',
 'igfn-livejournal-com.turbopages.org': 'igfn.livejournal.com',
 'mvdgaiussr-livejournal-com.turbopages.org': 'mvdgaiussr.livejournal.com',
 'avelisov-livejournal-com.turbopages.org': 'avelisov.livejournal.com',
 'tchin--drugitche-livejournal-com.turbopages.org': 'tchin-drugitche.livejournal.com',
 'barskaya-livejournal-com.turbopages.org': 'barskaya.livejournal.com',
 'cups4women-livejournal-com.turbopages.org': 'cups4women.livejournal.com',
 'anirish-livejournal-com.turbopages.org': 'anirish.livejournal.com',
 'visionfor-livejournal-com.turbopages.org': 'visionfor.livejournal.com',
 'milanist88-livejournal-com.turbopages.org': 'milanist88.livejournal.com',
 'vasneverov-livejournal-com.turbopages.org': 'vasneverov.livejournal.com',
 'ilim24-ru.turbopages.org': 'ilim24.ru',
 'vitalv89-livejournal-com.turbopages.org': 'vitalv89.livejournal.com',
 'mejdurechensk-bezformata-com.turbopages.org': 'mejdurechensk.bezformata.com',
 'nash--dvor-livejournal-com.turbopages.org': 'nash-dvor.livejournal.com',
 'vovik--ptz-livejournal-com.turbopages.org': 'vovik-ptz.livejournal.com',
 'paszekpielawski-livejournal-com.turbopages.org': 'paszekpielawski.livejournal.com',
 'leo--nafta-livejournal-com.turbopages.org': 'leo-nafta.livejournal.com',
 'liudprando-livejournal-com.turbopages.org': 'liudprando.livejournal.com',
 'vazart-livejournal-com.turbopages.org': 'vazart.livejournal.com',
 'uchitelj-livejournal-com.turbopages.org': 'uchitelj.livejournal.com',
 'vitamita-net.turbopages.org': 'vitamita.net',
 'litkarino-bezformata-com.turbopages.org': 'litkarino.bezformata.com',
 'onosamo-livejournal-com.turbopages.org': 'onosamo.livejournal.com',
 'keyko--sansara-livejournal-com.turbopages.org': 'keyko-sansara.livejournal.com',
 'vsevodintsovo-ru.turbopages.org': 'vsevodintsovo.ru',
 'around--mira-ru.turbopages.org': 'around-mira.ru',
 'nickmix01-livejournal-com.turbopages.org': 'nickmix01.livejournal.com',
 'trueyalta-livejournal-com.turbopages.org': 'trueyalta.livejournal.com',
 'serweb-ru.turbopages.org': 'serweb.ru',
 'nizhnekamsk-big--book--med-ru.turbopages.org': 'nizhnekamsk.big-book-med.ru',
 'agrodoctor-livejournal-com.turbopages.org': 'agrodoctor.livejournal.com',
 'fregat3005-livejournal-com.turbopages.org': 'fregat3005.livejournal.com',
 'ag--smith-livejournal-com.turbopages.org': 'ag-smith.livejournal.com',
 'redburda--ru-livejournal-com.turbopages.org': 'redburda-ru.livejournal.com',
 'teddy--boar-livejournal-com.turbopages.org': 'teddy-boar.livejournal.com',
 'valera--kolpakov-livejournal-com.turbopages.org': 'valera-kolpakov.livejournal.com',
 'domagonim-ru.turbopages.org': 'domagonim.ru',
 'fabiavod-ru.turbopages.org': 'fabiavod.ru',
 'toshaleb-livejournal-com.turbopages.org': 'toshaleb.livejournal.com',
 '307elena-livejournal-com.turbopages.org': '307elena.livejournal.com',
 'etoonda-livejournal-com.turbopages.org': 'etoonda.livejournal.com',
 'johni--d-livejournal-com.turbopages.org': 'johni-d.livejournal.com',
 'kapustablog-livejournal-com.turbopages.org': 'kapustablog.livejournal.com',
 'digit--tv-ru.turbopages.org': 'digit-tv.ru',
 'zdorsustav-ru.turbopages.org': 'zdorsustav.ru',
 'manga--sempai-ru.turbopages.org': 'manga-sempai.ru',
 'youngspace-ru.turbopages.org': 'youngspace.ru',
 'a--fixx-livejournal-com.turbopages.org': 'a-fixx.livejournal.com',
 'ru--red--lights-livejournal-com.turbopages.org': 'ru-red-lights.livejournal.com',
 'pike-livejournal-com.turbopages.org': 'pike.livejournal.com',
 'aqhome-ru.turbopages.org': 'aqhome.ru',
 'midnike-livejournal-com.turbopages.org': 'midnike.livejournal.com',
 'kronshtadtkniga-ru.turbopages.org': 'kronshtadtkniga.ru',
 'gorod62-livejournal-com.turbopages.org': 'gorod62.livejournal.com',
 'rumydom-ru.turbopages.org': 'rumydom.ru',
 'chastnydom-com.turbopages.org': 'chastnydom.com',
 'web--compromat-livejournal-com.turbopages.org': 'web-compromat.livejournal.com',
 'zombilol-livejournal-com.turbopages.org': 'zombilol.livejournal.com',
 'photoguide-livejournal-com.turbopages.org': 'photoguide.livejournal.com',
 'glikery-livejournal-com.turbopages.org': 'glikery.livejournal.com',
 'big--game-livejournal-com.turbopages.org': 'big-game.livejournal.com',
 'gorshkov--sergey-livejournal-com.turbopages.org': 'gorshkov-sergey.livejournal.com',
 'stephan--dupo-livejournal-com.turbopages.org': 'stephan-dupo.livejournal.com',
 'aspasiaroma-livejournal-com.turbopages.org': 'aspasiaroma.livejournal.com',
 'leonid--orlan-livejournal-com.turbopages.org': 'leonid-orlan.livejournal.com',
 'romanenko-livejournal-com.turbopages.org': 'romanenko.livejournal.com',
 'matholimp-livejournal-com.turbopages.org': 'matholimp.livejournal.com',
 'litra-online.turbopages.org': 'litra.online',
 'kuka--ra-livejournal-com.turbopages.org': 'kuka-ra.livejournal.com',
 'yarowind-livejournal-com.turbopages.org': 'yarowind.livejournal.com',
 'mij--gun-livejournal-com.turbopages.org': 'mij-gun.livejournal.com',
 'darkmeister-livejournal-com.turbopages.org': 'darkmeister.livejournal.com',
 'papkin1-livejournal-com.turbopages.org': 'papkin1.livejournal.com',
 'automediapro-ru.turbopages.org': 'automediapro.ru',
 'arashi--opera-livejournal-com.turbopages.org': 'arashi-opera.livejournal.com',
 'haphaper-livejournal-com.turbopages.org': 'haphaper.livejournal.com',
 'rubin65-livejournal-com.turbopages.org': 'rubin65.livejournal.com',
 'g--sarkisyan-livejournal-com.turbopages.org': 'g-sarkisyan.livejournal.com',
 'midgardets-livejournal-com.turbopages.org': 'midgardets.livejournal.com',
 'greenvegan-ru.turbopages.org': 'greenvegan.ru',
 'ratrussian-livejournal-com.turbopages.org': 'ratrussian.livejournal.com',
 '2fixika-livejournal-com.turbopages.org': '2fixika.livejournal.com',
 'pushkino-bezformata-com.turbopages.org': 'pushkino.bezformata.com',
 'anagaminx-livejournal-com.turbopages.org': 'anagaminx.livejournal.com',
 'greypey-ru.turbopages.org': 'greypey.ru',
 'zaphastivaz-ru.turbopages.org': 'zaphastivaz.ru',
 'maikl--rus-livejournal-com.turbopages.org': 'maikl-rus.livejournal.com',
 'natasha--fisher-livejournal-com.turbopages.org': 'natasha-fisher.livejournal.com',
 'telec-livejournal-com.turbopages.org': 'telec.livejournal.com',
 'georgy--lisov-livejournal-com.turbopages.org': 'georgy-lisov.livejournal.com',
 'severin60-livejournal-com.turbopages.org': 'severin60.livejournal.com',
 'rai77-com.turbopages.org': 'rai77.com',
 'erekto-ru.turbopages.org': 'erekto.ru',
 'electronic--payment-ru.turbopages.org': 'electronic-payment.ru',
 'la--vidaloca-com.turbopages.org': 'la-vidaloca.com',
 'ratengoods-com.turbopages.org': 'ratengoods.com',
 'grimsound-ru.turbopages.org': 'grimsound.ru',
 'iwar-livejournal-com.turbopages.org': 'iwar.livejournal.com',
 'raelrulez-livejournal-com.turbopages.org': 'raelrulez.livejournal.com',
 'service--drive-ru.turbopages.org': 'service-drive.ru',
 'amio-livejournal-com.turbopages.org': 'amio.livejournal.com',
 'cert--ua-livejournal-com.turbopages.org': 'cert-ua.livejournal.com',
 'neperm-ru.turbopages.org': 'neperm.ru',
 'techrocks-ru.turbopages.org': 'techrocks.ru',
 'andrewrostov-livejournal-com.turbopages.org': 'andrewrostov.livejournal.com',
 'rt--simulators-ru.turbopages.org': 'rt-simulators.ru',
 'azbukatreydera-ru.turbopages.org': 'azbukatreydera.ru',
 'printeros-ru.turbopages.org': 'printeros.ru',
 'prohladniy-bezformata-com.turbopages.org': 'prohladniy.bezformata.com',
 'gelpriboy-ru.turbopages.org': 'gelpriboy.ru',
 'krckorabl-ru.turbopages.org': 'krckorabl.ru',
 'neveev-livejournal-com.turbopages.org': 'neveev.livejournal.com',
 'enjoy--england-livejournal-com.turbopages.org': 'enjoy-england.livejournal.com',
 'leon--antikvar-livejournal-com.turbopages.org': 'leon-antikvar.livejournal.com',
 'malishichki-ru.turbopages.org': 'malishichki.ru',
 'pro--vkusnyashki-ru.turbopages.org': 'pro-vkusnyashki.ru',
    'koto--mafia-livejournal-com.turbopages.org': 'koto-mafia.livejournal.com',
 'chinese--cuisine-livejournal-com.turbopages.org': 'chinese-cuisine.livejournal.com',
 'lifejourney-club.turbopages.org': 'lifejourney.club',
 'spasibo--sberbank-com.turbopages.org': 'spasibo-sberbank.com',
 'katty--wise-livejournal-com.turbopages.org': 'katty-wise.livejournal.com',
 'alek--morse-livejournal-com.turbopages.org': 'alek-morse.livejournal.com',
 'gcup-ru.turbopages.org': 'gcup.ru',
 'doktor--ok-com.turbopages.org': 'doktor-ok.com',
 'ffix1975-livejournal-com.turbopages.org': 'ffix1975.livejournal.com',
 'ege--legko-livejournal-com.turbopages.org': 'ege-legko.livejournal.com',
 'cyaness-livejournal-com.turbopages.org': 'cyaness.livejournal.com',
 'bezrulya-ru.turbopages.org': 'bezrulya.ru',
 'maximus67-livejournal-com.turbopages.org': 'maximus67.livejournal.com',
 'napev-livejournal-com.turbopages.org': 'napev.livejournal.com',
 'stone--stream-ru.turbopages.org': 'stone-stream.ru',
 'hideguard-ru.turbopages.org': 'hideguard.ru',
 'm--shahidov-livejournal-com.turbopages.org': 'm-shahidov.livejournal.com',
 'shahti-bezformata-com.turbopages.org': 'shahti.bezformata.com',
 'kdv-livejournal-com.turbopages.org': 'kdv.livejournal.com',
 'murmia-livejournal-com.turbopages.org': 'murmia.livejournal.com',
 'frankensstein-livejournal-com.turbopages.org': 'frankensstein.livejournal.com',
 'kardaeva-ru.turbopages.org': 'kardaeva.ru',
 'nastroike-com.turbopages.org': 'nastroike.com',
 'husainov-livejournal-com.turbopages.org': 'husainov.livejournal.com',
 'esdra-livejournal-com.turbopages.org': 'esdra.livejournal.com',
 'kakieshtory-ru.turbopages.org': 'kakieshtory.ru',
 'gunsfriend-ru.turbopages.org': 'gunsfriend.ru',
 'thermalinfo-ru.turbopages.org': 'thermalinfo.ru',
 'fermasadogorod-ru.turbopages.org': 'fermasadogorod.ru',
 'tpaservice-ru.turbopages.org': 'tpaservice.ru',
 'labkovskiy-livejournal-com.turbopages.org': 'labkovskiy.livejournal.com',
 'nn-kudago-com.turbopages.org': 'nn.kudago.com',
 'dmitry--gubsky-livejournal-com.turbopages.org': 'dmitry-gubsky.livejournal.com',
 'mosoptika--servis-ru.turbopages.org': 'mosoptika-servis.ru',
 'kbmk45-ru.turbopages.org': 'kbmk45.ru',
 'kurbatishka-livejournal-com.turbopages.org': 'kurbatishka.livejournal.com',
 'oblefaroplastike-ru.turbopages.org': 'oblefaroplastike.ru',
 'relatoxfh-ru.turbopages.org': 'relatoxfh.ru',
 'kosmacz-livejournal-com.turbopages.org': 'kosmacz.livejournal.com',
 'zheka--sch-livejournal-com.turbopages.org': 'zheka-sch.livejournal.com',
 'sokolov9686-livejournal-com.turbopages.org': 'sokolov9686.livejournal.com',
 'yyprst-livejournal-com.turbopages.org': 'yyprst.livejournal.com',
 'sprut-ai.turbopages.org': 'sprut.ai',
 'cherkasova--lena-livejournal-com.turbopages.org': 'cherkasova-lena.livejournal.com',
 'domananeve-ru.turbopages.org': 'domananeve.ru',
 'livecars-ru.turbopages.org': 'livecars.ru',
 'bugumich-livejournal-com.turbopages.org': 'bugumich.livejournal.com',
 'kolovrat25-livejournal-com.turbopages.org': 'kolovrat25.livejournal.com',
 '100realt-ru.turbopages.org': '100realt.ru',
 'garazhmechti-ru.turbopages.org': 'garazhmechti.ru',
 'o--daeda-ru.turbopages.org': 'o-daeda.ru',
 'kiowa--mike-livejournal-com.turbopages.org': 'kiowa-mike.livejournal.com',
 'zamesbetona-ru.turbopages.org': 'zamesbetona.ru',
 'azbuka--zvezd-livejournal-com.turbopages.org': 'azbuka-zvezd.livejournal.com',
 'ukusta-ru.turbopages.org': 'ukusta.ru',
 'krasa--blog-ru.turbopages.org': 'krasa-blog.ru',
 'rubtsovskmv-ru.turbopages.org': 'rubtsovskmv.ru',
 'chechnyatoday-com.turbopages.org': 'chechnyatoday.com',
 'pro--wipers-ru.turbopages.org': 'pro-wipers.ru',
 'nana--95-livejournal-com.turbopages.org': 'nana-95.livejournal.com',
 'oopt--msk-livejournal-com.turbopages.org': 'oopt-msk.livejournal.com',
 'power--boat-ru.turbopages.org': 'power-boat.ru',
 'money--creditor-ru.turbopages.org': 'money-creditor.ru',
 'pyanaya--vi6nya-livejournal-com.turbopages.org': 'pyanaya-vi6nya.livejournal.com',
 'bemysweet-com.turbopages.org': 'bemysweet.com',
 'vologda-aif-ru.turbopages.org': 'vologda.aif.ru',
 'hobiz-ru.turbopages.org': 'hobiz.ru',
 'ext--331676-livejournal-com.turbopages.org': 'ext-331676.livejournal.com',
 'gta5-su.turbopages.org': 'gta5.su',
 'podarkimaster-ru.turbopages.org': 'podarkimaster.ru',
 'meast--ru-livejournal-com.turbopages.org': 'meast-ru.livejournal.com',
 'b--a--n--s--h--e--e-livejournal-com.turbopages.org': 'b-a-n-s-h-e-e.livejournal.com',
 'alexpashkov-livejournal-com.turbopages.org': 'alexpashkov.livejournal.com',
 'ipinform-ru.turbopages.org': 'ipinform.ru',
 'agagen-livejournal-com.turbopages.org': 'agagen.livejournal.com',
 'olgakl1971-livejournal-com.turbopages.org': 'olgakl1971.livejournal.com',
 'chilyaeva-livejournal-com.turbopages.org': 'chilyaeva.livejournal.com',
 'koshmau-livejournal-com.turbopages.org': 'koshmau.livejournal.com',
 'beginogi-ru.turbopages.org': 'beginogi.ru',
 'gallago-livejournal-com.turbopages.org': 'gallago.livejournal.com',
 'tasty--mama-livejournal-com.turbopages.org': 'tasty-mama.livejournal.com',
 'nornegest-livejournal-com.turbopages.org': 'nornegest.livejournal.com',
 'dostoevskiyfm-ru.turbopages.org': 'dostoevskiyfm.ru',
 'gorshkovsasha-livejournal-com.turbopages.org': 'gorshkovsasha.livejournal.com',
 'cavetravel-livejournal-com.turbopages.org': 'cavetravel.livejournal.com',
 'kuznetsov--ru-livejournal-com.turbopages.org': 'kuznetsov-ru.livejournal.com',
 'genm143-livejournal-com.turbopages.org': 'genm143.livejournal.com',
 'zee--tv-ru.turbopages.org': 'zee-tv.ru',
 'honestlil-livejournal-com.turbopages.org': 'honestlil.livejournal.com',
 'serpuhov-bezformata-com.turbopages.org': 'serpuhov.bezformata.com',
 'erimid-site.turbopages.org': 'erimid.site',
 'truckmix-ru.turbopages.org': 'truckmix.ru',
 'ntel--nsk-ru.turbopages.org': 'ntel-nsk.ru',
 'vseprogarazh-ru.turbopages.org': 'vseprogarazh.ru',
 'techwork-livejournal-com.turbopages.org': 'techwork.livejournal.com',
 'talinkas-livejournal-com.turbopages.org': 'talinkas.livejournal.com',
 'yesin64-livejournal-com.turbopages.org': 'yesin64.livejournal.com',
 'rucheek4-ru.turbopages.org': 'rucheek4.ru',
 'tushinetc-livejournal-com.turbopages.org': 'tushinetc.livejournal.com',
 'sagasumerki-livejournal-com.turbopages.org': 'sagasumerki.livejournal.com',
 'yakutsk-bezformata-com.turbopages.org': 'yakutsk.bezformata.com',
 'armour--shina-ru.turbopages.org': 'armour-shina.ru',
 'novouralsk--2014-livejournal-com.turbopages.org': 'novouralsk-2014.livejournal.com',
 'guriny-livejournal-com.turbopages.org': 'guriny.livejournal.com',
 'vlad--vinogradov-livejournal-com.turbopages.org': 'vlad-vinogradov.livejournal.com',
 'xn--56--eddobvhbi5bmlki9c.turbopages.org': 'xn-56-eddobvhbi5bmlki9c',
 'turlog-ru.turbopages.org': 'turlog.ru',
 'neizvestniy--geniy-ru.turbopages.org': 'neizvestniy-geniy.ru',
 'shadow--ru-livejournal-com.turbopages.org': 'shadow-ru.livejournal.com',
 'novayagazeta-livejournal-com.turbopages.org': 'novayagazeta.livejournal.com',
 'martinian-livejournal-com.turbopages.org': 'martinian.livejournal.com',
 'suntoriya-livejournal-com.turbopages.org': 'suntoriya.livejournal.com',
 'godleif-livejournal-com.turbopages.org': 'godleif.livejournal.com',
 'klub--krik-livejournal-com.turbopages.org': 'klub-krik.livejournal.com',
 'etops-livejournal-com.turbopages.org': 'etops.livejournal.com',
 'jamallutdin-livejournal-com.turbopages.org': 'jamallutdin.livejournal.com',
 'leon--spb67-livejournal-com.turbopages.org': 'leon-spb67.livejournal.com',
 'tsume--wr-livejournal-com.turbopages.org': 'tsume-wr.livejournal.com',
 'deti57-ru.turbopages.org': 'deti57.ru',
 'pavelprokh-livejournal-com.turbopages.org': 'pavelprokh.livejournal.com',
 'kancguru-livejournal-com.turbopages.org': 'kancguru.livejournal.com',
 'a--mediamed-ru.turbopages.org': 'a-mediamed.ru',
 'shark--er-livejournal-com.turbopages.org': 'shark-er.livejournal.com',
 'fatra-su.turbopages.org': 'fatra.su',
 'alenushka--rus-livejournal-com.turbopages.org': 'alenushka-rus.livejournal.com',
 'proiron-ru.turbopages.org': 'proiron.ru',
 'rayon-irk-today.turbopages.org': 'rayon.irk.today',
 'rastamozhitvspb-ru.turbopages.org': 'rastamozhitvspb.ru',
 'macdays-ru.turbopages.org': 'macdays.ru',
 'novostimosoblasti-ru.turbopages.org': 'novostimosoblasti.ru',
 'museum--guide-livejournal-com.turbopages.org': 'museum-guide.livejournal.com',
 'brodaga--2-livejournal-com.turbopages.org': 'brodaga-2.livejournal.com',
 'dolean-ru.turbopages.org': 'dolean.ru',
 'vet--life-livejournal-com.turbopages.org': 'vet-life.livejournal.com',
 'zezyaa-livejournal-com.turbopages.org': 'zezyaa.livejournal.com',
 'rostov--don-livejournal-com.turbopages.org': 'rostov-don.livejournal.com',
 'vibor--a-ru.turbopages.org': 'vibor-a.ru',
 'walera11-livejournal-com.turbopages.org': 'walera11.livejournal.com',
 'south-life.turbopages.org': 'south.life',
 'yamato999-livejournal-com.turbopages.org': 'yamato999.livejournal.com',
 'alex--oil-livejournal-com.turbopages.org': 'alex-oil.livejournal.com',
 'saygotakamori-livejournal-com.turbopages.org': 'saygotakamori.livejournal.com',
 'mprostata-com.turbopages.org': 'mprostata.com',
 'shumilov-livejournal-com.turbopages.org': 'shumilov.livejournal.com',
 'horseportal-ru.turbopages.org': 'horseportal.ru',
 'kaffeemaus-livejournal-com.turbopages.org': 'kaffeemaus.livejournal.com',
 'wond--world-livejournal-com.turbopages.org': 'wond-world.livejournal.com',
 'ja--potamchik-livejournal-com.turbopages.org': 'ja-potamchik.livejournal.com',
 'misha--jergenia-livejournal-com.turbopages.org': 'misha-jergenia.livejournal.com',
 'tsibirinka-livejournal-com.turbopages.org': 'tsibirinka.livejournal.com',
 'fluger19-livejournal-com.turbopages.org': 'fluger19.livejournal.com',
 'ieris--m-livejournal-com.turbopages.org': 'ieris-m.livejournal.com',
 'kolibry108-livejournal-com.turbopages.org': 'kolibry108.livejournal.com',
 'ru--metro-livejournal-com.turbopages.org': 'ru-metro.livejournal.com',
 'mrakorius-livejournal-com.turbopages.org': 'mrakorius.livejournal.com',
 'skfoonline-livejournal-com.turbopages.org': 'skfoonline.livejournal.com',
 'michailov--na-livejournal-com.turbopages.org': 'michailov-na.livejournal.com',
 'ru--anomalia-livejournal-com.turbopages.org': 'ru-anomalia.livejournal.com',
 'alex--druz-livejournal-com.turbopages.org': 'alex-druz.livejournal.com',
 'malay--otter-livejournal-com.turbopages.org': 'malay-otter.livejournal.com',
 'ratbud-livejournal-com.turbopages.org': 'ratbud.livejournal.com',
 'romali83-livejournal-com.turbopages.org': 'romali83.livejournal.com',
 'pelato-livejournal-com.turbopages.org': 'pelato.livejournal.com',
 'krilvic-livejournal-com.turbopages.org': 'krilvic.livejournal.com',
 'photo--ilius-livejournal-com.turbopages.org': 'photo-ilius.livejournal.com',
 'pavel--kuchma-livejournal-com.turbopages.org': 'pavel-kuchma.livejournal.com',
 'istra-bezformata-com.turbopages.org': 'istra.bezformata.com',
 'yasen-su.turbopages.org': 'yasen.su',
 'gunslaw-ru.turbopages.org': 'gunslaw.ru',
 'gemoparazit-ru.turbopages.org': 'gemoparazit.ru',
 'agnks-ru.turbopages.org': 'agnks.ru',
 'perm-big--book--med-ru.turbopages.org': 'perm.big-book-med.ru',
 'russkaya--usadba-livejournal-com.turbopages.org': 'russkaya-usadba.livejournal.com',
 'viktoriya--angel-livejournal-com.turbopages.org': 'viktoriya-angel.livejournal.com',
 'atlanta--s-livejournal-com.turbopages.org': 'atlanta-s.livejournal.com',
 'shakkar-livejournal-com.turbopages.org': 'shakkar.livejournal.com',
 'gorodokboxing-com.turbopages.org': 'gorodokboxing.com',
 'nikolaeva--ira-livejournal-com.turbopages.org': 'nikolaeva-ira.livejournal.com',
 'karmelpilates-com.turbopages.org': 'karmelpilates.com',
 'oldbotanik-livejournal-com.turbopages.org': 'oldbotanik.livejournal.com',
 'dburtsev-livejournal-com.turbopages.org': 'dburtsev.livejournal.com',
 'galkovsky-livejournal-com.turbopages.org': 'galkovsky.livejournal.com',
 'ucmopuockon-livejournal-com.turbopages.org': 'ucmopuockon.livejournal.com',
 'vkusnoe--utro-livejournal-com.turbopages.org': 'vkusnoe-utro.livejournal.com',
 'make--your--style-livejournal-com.turbopages.org': 'make-your-style.livejournal.com',
 'zoobusiness-kiev-ua.turbopages.org': 'zoobusiness.kiev.ua',
 'rubcovsk-bezformata-com.turbopages.org': 'rubcovsk.bezformata.com',
 'hoove-ru.turbopages.org': 'hoove.ru',
 'appsgames-ru.turbopages.org': 'appsgames.ru',
 'crusoe-livejournal-com.turbopages.org': 'crusoe.livejournal.com',
 'blogs-masterclassy-ru.turbopages.org': 'blogs.masterclassy.ru',
 'elfi--doll-livejournal-com.turbopages.org': 'elfi-doll.livejournal.com',
 'bank--explorer-ru.turbopages.org': 'bank-explorer.ru',
 'marine18--yes-livejournal-com.turbopages.org': 'marine18-yes.livejournal.com',
 'astrologe-livejournal-com.turbopages.org': 'astrologe.livejournal.com',
 'artyom--ferrier-livejournal-com.turbopages.org': 'artyom-ferrier.livejournal.com',
 'balty--slavjane-livejournal-com.turbopages.org': 'balty-slavjane.livejournal.com',
 'chtodelat24-ru.turbopages.org': 'chtodelat24.ru',
 '10--windows-ru.turbopages.org': '10-windows.ru',
 'bely--den-livejournal-com.turbopages.org': 'bely-den.livejournal.com',
 'userman-ru.turbopages.org': 'userman.ru',
 'staryikot-livejournal-com.turbopages.org': 'staryikot.livejournal.com',
 'ya--regisha-livejournal-com.turbopages.org': 'ya-regisha.livejournal.com',
 'fomaru-livejournal-com.turbopages.org': 'fomaru.livejournal.com',
 'balproton-ru.turbopages.org': 'balproton.ru',
 'red--balls-livejournal-com.turbopages.org': 'red-balls.livejournal.com',
 'crimea--hotels-ru.turbopages.org': 'crimea-hotels.ru',
 'alexlotov-livejournal-com.turbopages.org': 'alexlotov.livejournal.com',
 'itarasov-livejournal-com.turbopages.org': 'itarasov.livejournal.com',
 'valermon-livejournal-com.turbopages.org': 'valermon.livejournal.com',
 'moscultura-livejournal-com.turbopages.org': 'moscultura.livejournal.com',
 'shkolnoepitanie-livejournal-com.turbopages.org': 'shkolnoepitanie.livejournal.com',
 'denturex-ru.turbopages.org': 'denturex.ru',
 'horstveps-livejournal-com.turbopages.org': 'horstveps.livejournal.com',
 'supron--licvin-livejournal-com.turbopages.org': 'supron-licvin.livejournal.com',
 '1981dn-livejournal-com.turbopages.org': '1981dn.livejournal.com',
 'nekrasov1979-livejournal-com.turbopages.org': 'nekrasov1979.livejournal.com',
 'kleopina-livejournal-com.turbopages.org': 'kleopina.livejournal.com',
 'pravoslavnij-livejournal-com.turbopages.org': 'pravoslavnij.livejournal.com',
 'rustavi2-livejournal-com.turbopages.org': 'rustavi2.livejournal.com',
 'olga--kulikoff-livejournal-com.turbopages.org': 'olga-kulikoff.livejournal.com',
 'mtbike-livejournal-com.turbopages.org': 'mtbike.livejournal.com',
 'world--china-livejournal-com.turbopages.org': 'world-china.livejournal.com',
 'pro--novyj--god-ru.turbopages.org': 'pro-novyj-god.ru',
 'steprecept-ru.turbopages.org': 'steprecept.ru',
 'my--dog--friend-livejournal-com.turbopages.org': 'my-dog-friend.livejournal.com',
 'rednyrg721-livejournal-com.turbopages.org': 'rednyrg721.livejournal.com',
 'rkkawarrior-livejournal-com.turbopages.org': 'rkkawarrior.livejournal.com',
 'ffanatt-livejournal-com.turbopages.org': 'ffanatt.livejournal.com',
 'alexey1789-livejournal-com.turbopages.org': 'alexey1789.livejournal.com',
 'ozzeoz-livejournal-com.turbopages.org': 'ozzeoz.livejournal.com',
 'shugaring-pro.turbopages.org': 'shugaring.pro',
 'mirtenei-ru.turbopages.org': 'mirtenei.ru',
 'android--magazine-ru.turbopages.org': 'android-magazine.ru',
 'bemen-ru.turbopages.org': 'bemen.ru',
 'gavrilenko-livejournal-com.turbopages.org': 'gavrilenko.livejournal.com',
 'pirogi-guru.turbopages.org': 'pirogi.guru',
 'svirelin-ru.turbopages.org': 'svirelin.ru',
 'ruslanosharov-com.turbopages.org': 'ruslanosharov.com',
 'terranova2017-livejournal-com.turbopages.org': 'terranova2017.livejournal.com',
 'chydesa--mira-ru.turbopages.org': 'chydesa-mira.ru',
 'toivoa-livejournal-com.turbopages.org': 'toivoa.livejournal.com',
 'prazdniktyt-ru.turbopages.org': 'prazdniktyt.ru',
 'kim373-livejournal-com.turbopages.org': 'kim373.livejournal.com',
 'xn----7sbfbw5berlez0f.turbopages.org': 'xn--7sbfbw5berlez0f',
 'prosto--krys-livejournal-com.turbopages.org': 'prosto-krys.livejournal.com',
 'legkopolezno-ru.turbopages.org': 'legkopolezno.ru',
 'factum--info-net.turbopages.org': 'factum-info.net',
 'vibandaka-livejournal-com.turbopages.org': 'vibandaka.livejournal.com',
 'doc--serg-livejournal-com.turbopages.org': 'doc-serg.livejournal.com',
 'infosort-ru.turbopages.org': 'infosort.ru',
 'hvostat--hvostat-livejournal-com.turbopages.org': 'hvostat-hvostat.livejournal.com',
 'rembvka-ru.turbopages.org': 'rembvka.ru',
 'velostrana-livejournal-com.turbopages.org': 'velostrana.livejournal.com',
 'farmacia24-ru.turbopages.org': 'farmacia24.ru',
 'a--kudryavets-livejournal-com.turbopages.org': 'a-kudryavets.livejournal.com',
 'gtrk--kostroma-ru.turbopages.org': 'gtrk-kostroma.ru',
 'vgr-by.turbopages.org': 'vgr.by',
 'chudo--udo-info.turbopages.org': 'chudo-udo.info',
 'stanki--snab-ru.turbopages.org': 'stanki-snab.ru',
 'in--es-livejournal-com.turbopages.org': 'in-es.livejournal.com',
 'uka--j-livejournal-com.turbopages.org': 'uka-j.livejournal.com',
 'raul--safa-livejournal-com.turbopages.org': 'raul-safa.livejournal.com',
 'ecuared-livejournal-com.turbopages.org': 'ecuared.livejournal.com',
 'lallika-livejournal-com.turbopages.org': 'lallika.livejournal.com',
 'aidatiflis7-livejournal-com.turbopages.org': 'aidatiflis7.livejournal.com',
 'loco--bird-livejournal-com.turbopages.org': 'loco-bird.livejournal.com',
 'sterh1973-livejournal-com.turbopages.org': 'sterh1973.livejournal.com',
 'hyundai--kona24-ru.turbopages.org': 'hyundai-kona24.ru',
 'samovarchiki-ru.turbopages.org': 'samovarchiki.ru',
 'childhood--ru-livejournal-com.turbopages.org': 'childhood-ru.livejournal.com',
 'masakaru-ru.turbopages.org': 'masakaru.ru',
 'prozhony-livejournal-com.turbopages.org': 'prozhony.livejournal.com',
 'petitnicolas-livejournal-com.turbopages.org': 'petitnicolas.livejournal.com',
 'manualza-ru.turbopages.org': 'manualza.ru',
 'tridentt-livejournal-com.turbopages.org': 'tridentt.livejournal.com',
 '3ve3do4et-livejournal-com.turbopages.org': '3ve3do4et.livejournal.com',
 'dyvo-livejournal-com.turbopages.org': 'dyvo.livejournal.com',
 'dent--a--med-ru.turbopages.org': 'dent-a-med.ru',
 'eleonorar-livejournal-com.turbopages.org': 'eleonorar.livejournal.com',
 'yashin-livejournal-com.turbopages.org': 'yashin.livejournal.com',
 'vbulahtin-livejournal-com.turbopages.org': 'vbulahtin.livejournal.com',
 'quis--quaeritis-livejournal-com.turbopages.org': 'quis-quaeritis.livejournal.com',
 'ollender-livejournal-com.turbopages.org': 'ollender.livejournal.com',
 'sdelaem--podelki-ru.turbopages.org': 'sdelaem-podelki.ru',
 'taberko-livejournal-com.turbopages.org': 'taberko.livejournal.com',
 'semya--centr-ru.turbopages.org': 'semya-centr.ru',
 't--kudelina-livejournal-com.turbopages.org': 't-kudelina.livejournal.com',
 'vaz2109-net.turbopages.org': 'vaz2109.net',
 'babs71-livejournal-com.turbopages.org': 'babs71.livejournal.com',
 'luarvik--1221-livejournal-com.turbopages.org': 'luarvik-1221.livejournal.com',
 'infodacha-com.turbopages.org': 'infodacha.com',
 'trayan--creator-livejournal-com.turbopages.org': 'trayan-creator.livejournal.com',
 'internet--provaider-ru.turbopages.org': 'internet-provaider.ru',
 'urist50-ru.turbopages.org': 'urist50.ru',
 'mr--gaijin-livejournal-com.turbopages.org': 'mr-gaijin.livejournal.com',
 'litlli--tlu-livejournal-com.turbopages.org': 'litlli-tlu.livejournal.com',
 'linuxsam-livejournal-com.turbopages.org': 'linuxsam.livejournal.com',
 'vash--ortoped-com.turbopages.org': 'vash-ortoped.com',
 'anaskon-livejournal-com.turbopages.org': 'anaskon.livejournal.com',
 'v--zabugin-livejournal-com.turbopages.org': 'v-zabugin.livejournal.com',
 'astra--wizard-livejournal-com.turbopages.org': 'astra-wizard.livejournal.com',
 'alfabankru-livejournal-com.turbopages.org': 'alfabankru.livejournal.com',
 'mitrokhin--alex-livejournal-com.turbopages.org': 'mitrokhin-alex.livejournal.com',
 'papa--karl0-livejournal-com.turbopages.org': 'papa-karl0.livejournal.com',
 'fanny--lune-livejournal-com.turbopages.org': 'fanny-lune.livejournal.com',
 'mikhailtula-livejournal-com.turbopages.org': 'mikhailtula.livejournal.com',
 'ontet-ru.turbopages.org': 'ontet.ru',
 'b2b--instrument-ru.turbopages.org': 'b2b-instrument.ru',
 'konstolkh2016-livejournal-com.turbopages.org': 'konstolkh2016.livejournal.com',
 'woman--dreams-livejournal-com.turbopages.org': 'woman-dreams.livejournal.com',
 'andrologmed-ru.turbopages.org': 'andrologmed.ru',
 'dev9tka-ru.turbopages.org': 'dev9tka.ru',
 'revmatolog-org.turbopages.org': 'revmatolog.org',
 'doktor--killer-livejournal-com.turbopages.org': 'doktor-killer.livejournal.com',
 'yes--fitness-ru.turbopages.org': 'yes-fitness.ru',
 'zauber--russ-ru.turbopages.org': 'zauber-russ.ru',
 'aptekatamara-ru.turbopages.org': 'aptekatamara.ru',
 'elchupanibrei-livejournal-com.turbopages.org': 'elchupanibrei.livejournal.com',
 'ladyvrn-ru.turbopages.org': 'ladyvrn.ru',
 'dovir--finance-ru.turbopages.org': 'dovir-finance.ru',
 'rossica--antiqua-livejournal-com.turbopages.org': 'rossica-antiqua.livejournal.com',
 'jenich-livejournal-com.turbopages.org': 'jenich.livejournal.com',
 'kamrti-com.turbopages.org': 'kamrti.com',
 'rynda--org-livejournal-com.turbopages.org': 'rynda-org.livejournal.com',
 'bhaktayog-livejournal-com.turbopages.org': 'bhaktayog.livejournal.com',
 'sarlynx-livejournal-com.turbopages.org': 'sarlynx.livejournal.com',
 'saratov-versia-ru.turbopages.org': 'saratov.versia.ru',
 'an--zss-ru.turbopages.org': 'an-zss.ru',
 'ufacitynews-ru.turbopages.org': 'ufacitynews.ru',
 'civildefense48-livejournal-com.turbopages.org': 'civildefense48.livejournal.com',
 'kamenskuralsk-bezformata-com.turbopages.org': 'kamenskuralsk.bezformata.com',
 'si--ven-livejournal-com.turbopages.org': 'si-ven.livejournal.com',
 'readingsouls-livejournal-com.turbopages.org': 'readingsouls.livejournal.com',
 'china--auto2-ru.turbopages.org': 'china-auto2.ru',
 'lifenail-ru.turbopages.org': 'lifenail.ru',
 'pcgame-in-ua.turbopages.org': 'pcgame.in.ua',
 'v--top100-livejournal-com.turbopages.org': 'v-top100.livejournal.com',
 'tyt--sxemi-ru.turbopages.org': 'tyt-sxemi.ru',
 'scopica-ru.turbopages.org': 'scopica.ru',
 'safezone-cc.turbopages.org': 'safezone.cc',
 'igreksovest-livejournal-com.turbopages.org': 'igreksovest.livejournal.com',
 'morozofkk-ru.turbopages.org': 'morozofkk.ru',
 'tosno-bezformata-com.turbopages.org': 'tosno.bezformata.com',
 'tastyblog-livejournal-com.turbopages.org': 'tastyblog.livejournal.com',
 'yuliamass-livejournal-com.turbopages.org': 'yuliamass.livejournal.com',
 'foodhere-ru.turbopages.org': 'foodhere.ru',
 'tankushin-ru.turbopages.org': 'tankushin.ru',
 'old-rk-karelia-ru.turbopages.org': 'old.rk.karelia.ru',
 'viborg-bezformata-com.turbopages.org': 'viborg.bezformata.com',
 'komanda--sibiri-ru.turbopages.org': 'komanda-sibiri.ru',
 'ivan--59-livejournal-com.turbopages.org': 'ivan-59.livejournal.com',
 'tursam-ru.turbopages.org': 'tursam.ru',
 'all--telecom-ru.turbopages.org': 'all-telecom.ru',
 'stainlesstlrat-livejournal-com.turbopages.org': 'stainlesstlrat.livejournal.com',
 'ultima--cruzado-livejournal-com.turbopages.org': 'ultima-cruzado.livejournal.com',
 'holy--matriarchy-livejournal-com.turbopages.org': 'holy-matriarchy.livejournal.com',
 'doskado-livejournal-com.turbopages.org': 'doskado.livejournal.com',
 'old--v-livejournal-com.turbopages.org': 'old-v.livejournal.com',
 'dobraya--bagira-livejournal-com.turbopages.org': 'dobraya-bagira.livejournal.com',
 'sv--kashin-livejournal-com.turbopages.org': 'sv-kashin.livejournal.com',
 'luv--vie-livejournal-com.turbopages.org': 'luv-vie.livejournal.com',
 'south--moskovia-livejournal-com.turbopages.org': 'south-moskovia.livejournal.com',
 'mikhaelkatz-livejournal-com.turbopages.org': 'mikhaelkatz.livejournal.com',
 'yourmetaldreams-livejournal-com.turbopages.org': 'yourmetaldreams.livejournal.com',
 'light--cook-ru.turbopages.org': 'light-cook.ru',
 'goodcoins-su.turbopages.org': 'goodcoins.su',
 'aleks--aberdin-livejournal-com.turbopages.org': 'aleks-aberdin.livejournal.com',
 'mirnuyatom--olga-livejournal-com.turbopages.org': 'mirnuyatom-olga.livejournal.com',
 'filosof--a-livejournal-com.turbopages.org': 'filosof-a.livejournal.com',
 'finexg-ru.turbopages.org': 'finexg.ru',
 'liwli-ru.turbopages.org': 'liwli.ru',
 'cleanhack-ru.turbopages.org': 'cleanhack.ru',
 'neo125-livejournal-com.turbopages.org': 'neo125.livejournal.com',
 'master--n-livejournal-com.turbopages.org': 'master-n.livejournal.com',
 'salut--sveta-livejournal-com.turbopages.org': 'salut-sveta.livejournal.com',
 'td--np-ru.turbopages.org': 'td-np.ru',
 'eventsanalytics-livejournal-com.turbopages.org': 'eventsanalytics.livejournal.com',
 'romti-livejournal-com.turbopages.org': 'romti.livejournal.com',
 'daniel--grishin-livejournal-com.turbopages.org': 'daniel-grishin.livejournal.com',
 'eizhru-livejournal-com.turbopages.org': 'eizhru.livejournal.com',
 'alienordis-livejournal-com.turbopages.org': 'alienordis.livejournal.com',
 'trunin-com.turbopages.org': 'trunin.com',
 'ubu-ru.turbopages.org': 'ubu.ru',
 'dv--leonov-livejournal-com.turbopages.org': 'dv-leonov.livejournal.com',
 'stelazin-livejournal-com.turbopages.org': 'stelazin.livejournal.com',
 'geliy-livejournal-com.turbopages.org': 'geliy.livejournal.com',
 'xn----itbvilb0ac.turbopages.org': 'xn--itbvilb0ac',
 'truppi--lerneya-livejournal-com.turbopages.org': 'truppi-lerneya.livejournal.com',
 'ru--4g-livejournal-com.turbopages.org': 'ru-4g.livejournal.com',
 'aydanil-ru.turbopages.org': 'aydanil.ru',
 'sevtao-livejournal-com.turbopages.org': 'sevtao.livejournal.com',
 'langart-ru.turbopages.org': 'langart.ru',
 'iv--g-livejournal-com.turbopages.org': 'iv-g.livejournal.com',
 'tvoiyapravda-ru.turbopages.org': 'tvoiyapravda.ru',
 'komariv-livejournal-com.turbopages.org': 'komariv.livejournal.com',
 'boogie--sapiens-livejournal-com.turbopages.org': 'boogie-sapiens.livejournal.com',
 'lovensa-ru.turbopages.org': 'lovensa.ru',
 'car--travels-livejournal-com.turbopages.org': 'car-travels.livejournal.com',
 'tripsaway-ru.turbopages.org': 'tripsaway.ru',
 'hvac-livejournal-com.turbopages.org': 'hvac.livejournal.com',
 'povar--life-ru.turbopages.org': 'povar-life.ru',
 'erpstat-ru.turbopages.org': 'erpstat.ru',
 'lidik2012-livejournal-com.turbopages.org': 'lidik2012.livejournal.com',
 'foto--travel-livejournal-com.turbopages.org': 'foto-travel.livejournal.com',
 'teploleta-livejournal-com.turbopages.org': 'teploleta.livejournal.com',
 'prokhozhyj-livejournal-com.turbopages.org': 'prokhozhyj.livejournal.com',
 'gawrusha-livejournal-com.turbopages.org': 'gawrusha.livejournal.com',
 'ran--ger-livejournal-com.turbopages.org': 'ran-ger.livejournal.com',
 'vash--gipnoz-livejournal-com.turbopages.org': 'vash-gipnoz.livejournal.com',
 'admin--minsk-livejournal-com.turbopages.org': 'admin-minsk.livejournal.com',
 'natamax-livejournal-com.turbopages.org': 'natamax.livejournal.com',
 'mnemodno-ru.turbopages.org': 'mnemodno.ru',
 'smi74-ru.turbopages.org': 'smi74.ru',
 'sochi-big--book--med-ru.turbopages.org': 'sochi.big-book-med.ru',
 '6a6a--yaga-livejournal-com.turbopages.org': '6a6a-yaga.livejournal.com',
 'allergo-guru.turbopages.org': 'allergo.guru',
 'vihrbudushego-livejournal-com.turbopages.org': 'vihrbudushego.livejournal.com',
 'ustav-livejournal-com.turbopages.org': 'ustav.livejournal.com',
 'craftband-ru.turbopages.org': 'craftband.ru',
 'alex--pro--1-livejournal-com.turbopages.org': 'alex-pro-1.livejournal.com',
 'romus77-livejournal-com.turbopages.org': 'romus77.livejournal.com',
 'ru--podorojniy-livejournal-com.turbopages.org': 'ru-podorojniy.livejournal.com',
 'anton7777777-livejournal-com.turbopages.org': 'anton7777777.livejournal.com',
 'wilama--taro-livejournal-com.turbopages.org': 'wilama-taro.livejournal.com',
 'muromlena57-livejournal-com.turbopages.org': 'muromlena57.livejournal.com',
 'mvpravo-ru.turbopages.org': 'mvpravo.ru',
 'gella--che-livejournal-com.turbopages.org': 'gella-che.livejournal.com',
 'moluccella-livejournal-com.turbopages.org': 'moluccella.livejournal.com',
 'erecisrael-livejournal-com.turbopages.org': 'erecisrael.livejournal.com',
 'san--gennaro-livejournal-com.turbopages.org': 'san-gennaro.livejournal.com',
 'marv-livejournal-com.turbopages.org': 'marv.livejournal.com',
 'xn----otbamiicigee2b3g.turbopages.org': 'xn--otbamiicigee2b3g',
 'poroda--mops-ru.turbopages.org': 'poroda-mops.ru',
 'antenna102-ru.turbopages.org': 'antenna102.ru',
 'kojemyakin-livejournal-com.turbopages.org': 'kojemyakin.livejournal.com',
 'geser-livejournal-com.turbopages.org': 'geser.livejournal.com',
 'a--lex--7-livejournal-com.turbopages.org': 'a-lex-7.livejournal.com',
 'wvape-ru.turbopages.org': 'wvape.ru',
 'titovsergei-ru.turbopages.org': 'titovsergei.ru',
 'anrika-livejournal-com.turbopages.org': 'anrika.livejournal.com',
 'kantemirov--r-livejournal-com.turbopages.org': 'kantemirov-r.livejournal.com',
 'nevar-livejournal-com.turbopages.org': 'nevar.livejournal.com',
 'mariya--volik-livejournal-com.turbopages.org': 'mariya-volik.livejournal.com',
 'tehpodderzka-ru.turbopages.org': 'tehpodderzka.ru',
 'topkpop-ru.turbopages.org': 'topkpop.ru',
 'ru--antivisa-livejournal-com.turbopages.org': 'ru-antivisa.livejournal.com',
 'harryhaller-livejournal-com.turbopages.org': 'harryhaller.livejournal.com',
 'shwanoff-ru.turbopages.org': 'shwanoff.ru',
 'pol--ektof-livejournal-com.turbopages.org': 'pol-ektof.livejournal.com',
 'annamann-livejournal-com.turbopages.org': 'annamann.livejournal.com',
 'qwertysirius-livejournal-com.turbopages.org': 'qwertysirius.livejournal.com',
 'belgorod-bezformata-com.turbopages.org': 'belgorod.bezformata.com',
 'voenjur-livejournal-com.turbopages.org': 'voenjur.livejournal.com',
 'lion--house-livejournal-com.turbopages.org': 'lion-house.livejournal.com',
 'eugzolotuhin-livejournal-com.turbopages.org': 'eugzolotuhin.livejournal.com',
 'footbolist-livejournal-com.turbopages.org': 'footbolist.livejournal.com',
 'aragont-livejournal-com.turbopages.org': 'aragont.livejournal.com',
 'cosmic--kettle--0-livejournal-com.turbopages.org': 'cosmic-kettle-0.livejournal.com',
 'elektroklub--nn-ru.turbopages.org': 'elektroklub-nn.ru',
 'ksherbi-livejournal-com.turbopages.org': 'ksherbi.livejournal.com',
 'udikov-livejournal-com.turbopages.org': 'udikov.livejournal.com',
 'ngasanova-livejournal-com.turbopages.org': 'ngasanova.livejournal.com',
 'biver64-livejournal-com.turbopages.org': 'biver64.livejournal.com',
 'arturindia-livejournal-com.turbopages.org': 'arturindia.livejournal.com',
 'zazerkaliya-livejournal-com.turbopages.org': 'zazerkaliya.livejournal.com',
 'akanti-livejournal-com.turbopages.org': 'akanti.livejournal.com',
 'sergeyaprum-livejournal-com.turbopages.org': 'sergeyaprum.livejournal.com',
 'pricepinfo-ru.turbopages.org': 'pricepinfo.ru',
 'dariaxxx-livejournal-com.turbopages.org': 'dariaxxx.livejournal.com',
 'vagrant--91-livejournal-com.turbopages.org': 'vagrant-91.livejournal.com',
 'elena--tendler-livejournal-com.turbopages.org': 'elena-tendler.livejournal.com',
 'diecast--43-livejournal-com.turbopages.org': 'diecast-43.livejournal.com',
 'moto--port-ru.turbopages.org': 'moto-port.ru',
 'putnik1-livejournal-com.turbopages.org': 'putnik1.livejournal.com',
 'kupifranshizu-ru.turbopages.org': 'kupifranshizu.ru',
 'dikiy--m-livejournal-com.turbopages.org': 'dikiy-m.livejournal.com',
 'townmen-livejournal-com.turbopages.org': 'townmen.livejournal.com',
 'nevworker-ru.turbopages.org': 'nevworker.ru',
 'kondratio-livejournal-com.turbopages.org': 'kondratio.livejournal.com',
 'lidka4t-livejournal-com.turbopages.org': 'lidka4t.livejournal.com',
 'kinocritic-livejournal-com.turbopages.org': 'kinocritic.livejournal.com',
 'ugpr-ru.turbopages.org': 'ugpr.ru',
 'fapvdo-ru.turbopages.org': 'fapvdo.ru',
 'm--jake-livejournal-com.turbopages.org': 'm-jake.livejournal.com',
 'minemik-ru.turbopages.org': 'minemik.ru',
 'ext--5787874-livejournal-com.turbopages.org': 'ext-5787874.livejournal.com',
 'lad--unity-com.turbopages.org': 'lad-unity.com',
 'stroy--good-ru.turbopages.org': 'stroy-good.ru',
 'lisiy--ml-livejournal-com.turbopages.org': 'lisiy-ml.livejournal.com',
 'i--bormey-livejournal-com.turbopages.org': 'i-bormey.livejournal.com',
 'shop--argo-ru.turbopages.org': 'shop-argo.ru',
 'elisaveta--neru-livejournal-com.turbopages.org': 'elisaveta-neru.livejournal.com',
 'facelift-guru.turbopages.org': 'facelift.guru',
 'zametilprosto-livejournal-com.turbopages.org': 'zametilprosto.livejournal.com',
 'brock--msc-livejournal-com.turbopages.org': 'brock-msc.livejournal.com',
 'gabtatiana-livejournal-com.turbopages.org': 'gabtatiana.livejournal.com',
 'goldserfer-ru.turbopages.org': 'goldserfer.ru',
 'taxi--mytishchi-ru.turbopages.org': 'taxi-mytishchi.ru',
 'tandyr34-ru.turbopages.org': 'tandyr34.ru',
 'buhgalterov-ru.turbopages.org': 'buhgalterov.ru',
 'feldsherstvo-ru.turbopages.org': 'feldsherstvo.ru',
 'al--kamensky-livejournal-com.turbopages.org': 'al-kamensky.livejournal.com',
 'reedr-ru.turbopages.org': 'reedr.ru',
 'huligan--av-livejournal-com.turbopages.org': 'huligan-av.livejournal.com',
 'zaborstroj-ru.turbopages.org': 'zaborstroj.ru',
 'foto--konkursy-ru.turbopages.org': 'foto-konkursy.ru',
 'man--from--nord-livejournal-com.turbopages.org': 'man-from-nord.livejournal.com',
 'wladimirez-livejournal-com.turbopages.org': 'wladimirez.livejournal.com',
 'martosennij-livejournal-com.turbopages.org': 'martosennij.livejournal.com',
 'apuhtin18-livejournal-com.turbopages.org': 'apuhtin18.livejournal.com',
 'svetoch31-livejournal-com.turbopages.org': 'svetoch31.livejournal.com',
 'atomycosmetic-com.turbopages.org': 'atomycosmetic.com',
 'silviapons-livejournal-com.turbopages.org': 'silviapons.livejournal.com',
 'crimerussia-livejournal-com.turbopages.org': 'crimerussia.livejournal.com',
 'amfora-livejournal-com.turbopages.org': 'amfora.livejournal.com',
 'shop--kombez-ru.turbopages.org': 'shop-kombez.ru',
 'proavto21-ru.turbopages.org': 'proavto21.ru',
 'robinzon25-livejournal-com.turbopages.org': 'robinzon25.livejournal.com',
 'no--idiot--toys-livejournal-com.turbopages.org': 'no-idiot-toys.livejournal.com',
 'tvoyaparallel-livejournal-com.turbopages.org': 'tvoyaparallel.livejournal.com',
 'pluto9999-livejournal-com.turbopages.org': 'pluto9999.livejournal.com',
 'techmonuments-livejournal-com.turbopages.org': 'techmonuments.livejournal.com',
 'reg--813-livejournal-com.turbopages.org': 'reg-813.livejournal.com',
 'hunter--yv-livejournal-com.turbopages.org': 'hunter-yv.livejournal.com',
 'dizain--cheloveka21-ru.turbopages.org': 'dizain-cheloveka21.ru',
 'lifeel-livejournal-com.turbopages.org': 'lifeel.livejournal.com',
 'medotvet-com.turbopages.org': 'medotvet.com',
 'v--grebennikov-livejournal-com.turbopages.org': 'v-grebennikov.livejournal.com',
 'ankol1-livejournal-com.turbopages.org': 'ankol1.livejournal.com',
 'ulia--ukke-livejournal-com.turbopages.org': 'ulia-ukke.livejournal.com',
 'vnk--dx-livejournal-com.turbopages.org': 'vnk-dx.livejournal.com',
 'adjedan-livejournal-com.turbopages.org': 'adjedan.livejournal.com',
 'vitus-livejournal-com.turbopages.org': 'vitus.livejournal.com',
 'stream--center-ru.turbopages.org': 'stream-center.ru',
 'pvd--yug-livejournal-com.turbopages.org': 'pvd-yug.livejournal.com',
 'prepod2000-livejournal-com.turbopages.org': 'prepod2000.livejournal.com',
 'veatfalika-livejournal-com.turbopages.org': 'veatfalika.livejournal.com',
 'naukatv-ru.turbopages.org': 'naukatv.ru',
 'felbert-livejournal-com.turbopages.org': 'felbert.livejournal.com',
 'andanton-livejournal-com.turbopages.org': 'andanton.livejournal.com',
 'podoprigora74-livejournal-com.turbopages.org': 'podoprigora74.livejournal.com',
 'victor--shyaga-livejournal-com.turbopages.org': 'victor-shyaga.livejournal.com',
 'peteronline-ru.turbopages.org': 'peteronline.ru',
 'mahila--dakini-livejournal-com.turbopages.org': 'mahila-dakini.livejournal.com',
 'pravdoiskanie-livejournal-com.turbopages.org': 'pravdoiskanie.livejournal.com',
 'sport--rekord-livejournal-com.turbopages.org': 'sport-rekord.livejournal.com',
 'koroleffsov-ru.turbopages.org': 'koroleffsov.ru',
 'ekburg-tv.turbopages.org': 'ekburg.tv',
 'kiev--paparazzi-livejournal-com.turbopages.org': 'kiev-paparazzi.livejournal.com',
 'windtravel-livejournal-com.turbopages.org': 'windtravel.livejournal.com',
 'budan-livejournal-com.turbopages.org': 'budan.livejournal.com',
 'chukovskij-livejournal-com.turbopages.org': 'chukovskij.livejournal.com',
 'p--o--s--t--c--a--r--d-livejournal-com.turbopages.org': 'p-o-s-t-c-a-r-d.livejournal.com',
 'znbm-ru.turbopages.org': 'znbm.ru',
 'dfcbktyj-livejournal-com.turbopages.org': 'dfcbktyj.livejournal.com',
 'valya--15-livejournal-com.turbopages.org': 'valya-15.livejournal.com',
 'ak--su-livejournal-com.turbopages.org': 'ak-su.livejournal.com',
 'bufetum-livejournal-com.turbopages.org': 'bufetum.livejournal.com',
 'chantfleuri-livejournal-com.turbopages.org': 'chantfleuri.livejournal.com',
 'vitaminka2012k-livejournal-com.turbopages.org': 'vitaminka2012k.livejournal.com',
 'mirina--arktus-livejournal-com.turbopages.org': 'mirina-arktus.livejournal.com',
 'kagemushanatako-livejournal-com.turbopages.org': 'kagemushanatako.livejournal.com',
 'ilovegarazh-ru.turbopages.org': 'ilovegarazh.ru',
 'xn------iddhce3bsfgivk2amx.turbopages.org': 'xn---iddhce3bsfgivk2amx',
 'frocush-livejournal-com.turbopages.org': 'frocush.livejournal.com',
 'asosnin-livejournal-com.turbopages.org': 'asosnin.livejournal.com',
 'matsam-livejournal-com.turbopages.org': 'matsam.livejournal.com',
 'ctac-livejournal-com.turbopages.org': 'ctac.livejournal.com',
 'alexvess-livejournal-com.turbopages.org': 'alexvess.livejournal.com',
 'eto--moyo--mesto-livejournal-com.turbopages.org': 'eto-moyo-mesto.livejournal.com',
 'djdiplomat-ru.turbopages.org': 'djdiplomat.ru',
 'zoneplanet-ru.turbopages.org': 'zoneplanet.ru',
 'skodakodiaq-club.turbopages.org': 'skodakodiaq.club',
 'workandhobby-ru.turbopages.org': 'workandhobby.ru',
 'anni--sanni-livejournal-com.turbopages.org': 'anni-sanni.livejournal.com',
 'ebrk-livejournal-com.turbopages.org': 'ebrk.livejournal.com',
 'foodclub--ru-livejournal-com.turbopages.org': 'foodclub-ru.livejournal.com',
 'antinormanist-livejournal-com.turbopages.org': 'antinormanist.livejournal.com',
 'soul--fishing-ru.turbopages.org': 'soul-fishing.ru',
 'vkusnodarka-ru.turbopages.org': 'vkusnodarka.ru',
 'lesstroy-net.turbopages.org': 'lesstroy.net',
 '1941--45-livejournal-com.turbopages.org': '1941-45.livejournal.com',
 'inna--budapest-livejournal-com.turbopages.org': 'inna-budapest.livejournal.com',
 'netdogg-livejournal-com.turbopages.org': 'netdogg.livejournal.com',
 'molly--inside-livejournal-com.turbopages.org': 'molly-inside.livejournal.com',
 'semo--sankus-livejournal-com.turbopages.org': 'semo-sankus.livejournal.com',
 'besedki-fun.turbopages.org': 'besedki.fun',
 'rodeland-livejournal-com.turbopages.org': 'rodeland.livejournal.com',
 'poundpig-ru.turbopages.org': 'poundpig.ru',
 'oblakos-ru.turbopages.org': 'oblakos.ru',
 'rusimage--ru-livejournal-com.turbopages.org': 'rusimage-ru.livejournal.com',
 'unetentation-livejournal-com.turbopages.org': 'unetentation.livejournal.com',
 'grundik-livejournal-com.turbopages.org': 'grundik.livejournal.com',
 'nataspace-livejournal-com.turbopages.org': 'nataspace.livejournal.com',
 'dzhotti-livejournal-com.turbopages.org': 'dzhotti.livejournal.com',
 'buddhistka-livejournal-com.turbopages.org': 'buddhistka.livejournal.com',
 'stupaman-livejournal-com.turbopages.org': 'stupaman.livejournal.com',
 'monastyri--mira-livejournal-com.turbopages.org': 'monastyri-mira.livejournal.com',
 'kerzak--1-livejournal-com.turbopages.org': 'kerzak-1.livejournal.com',
 'svoimi--rukami2-ru.turbopages.org': 'svoimi-rukami2.ru',
 'pensnews-ru.turbopages.org': 'pensnews.ru',
 'takoe--nebo-livejournal-com.turbopages.org': 'takoe-nebo.livejournal.com',
 'julitta-ru.turbopages.org': 'julitta.ru',
 'altyn73-livejournal-com.turbopages.org': 'altyn73.livejournal.com',
 'banyamechta-ru.turbopages.org': 'banyamechta.ru',
 'kraskainfo-ru.turbopages.org': 'kraskainfo.ru',
 'gipsylilya-livejournal-com.turbopages.org': 'gipsylilya.livejournal.com',
 'kljuet-ru.turbopages.org': 'kljuet.ru',
 'protendersi-livejournal-com.turbopages.org': 'protendersi.livejournal.com',
 'tihoreck-bezformata-com.turbopages.org': 'tihoreck.bezformata.com',
 'welder--history-livejournal-com.turbopages.org': 'welder-history.livejournal.com',
 'haritonoff-livejournal-com.turbopages.org': 'haritonoff.livejournal.com',
 'pomogach-livejournal-com.turbopages.org': 'pomogach.livejournal.com',
 'tohasss-livejournal-com.turbopages.org': 'tohasss.livejournal.com',
 'vseokyxne-com.turbopages.org': 'vseokyxne.com',
 'bobr-pw.turbopages.org': 'bobr.pw',
 'minemod-ru.turbopages.org': 'minemod.ru',
 '4x4funclub-ru.turbopages.org': '4x4funclub.ru',
 'vita--colorata-livejournal-com.turbopages.org': 'vita-colorata.livejournal.com',
 'zhevskaya-livejournal-com.turbopages.org': 'zhevskaya.livejournal.com',
 'astra--journal-livejournal-com.turbopages.org': 'astra-journal.livejournal.com',
 'v--korsak-livejournal-com.turbopages.org': 'v-korsak.livejournal.com',
 'theslim-livejournal-com.turbopages.org': 'theslim.livejournal.com',
 'irina88852-livejournal-com.turbopages.org': 'irina88852.livejournal.com',
 '3d--shka-livejournal-com.turbopages.org': '3d-shka.livejournal.com',
 'xn------0ddmlrsf8a3h.turbopages.org': 'xn---0ddmlrsf8a3h',
 'sergedid-livejournal-com.turbopages.org': 'sergedid.livejournal.com',
 'flickering4-livejournal-com.turbopages.org': 'flickering4.livejournal.com',
 'fyunt--converse-livejournal-com.turbopages.org': 'fyunt-converse.livejournal.com',
 'maryika-livejournal-com.turbopages.org': 'maryika.livejournal.com',
 'pavelval-livejournal-com.turbopages.org': 'pavelval.livejournal.com',
 'ladylubov-livejournal-com.turbopages.org': 'ladylubov.livejournal.com',
 'otvet--plus-livejournal-com.turbopages.org': 'otvet-plus.livejournal.com',
 'anordist1-livejournal-com.turbopages.org': 'anordist1.livejournal.com',
 'ottenki--serogo-livejournal-com.turbopages.org': 'ottenki-serogo.livejournal.com',
 'bakhmetev-livejournal-com.turbopages.org': 'bakhmetev.livejournal.com',
 'my--orchids-livejournal-com.turbopages.org': 'my-orchids.livejournal.com',
 'nichren-livejournal-com.turbopages.org': 'nichren.livejournal.com',
 'armsline-livejournal-com.turbopages.org': 'armsline.livejournal.com',
 'annakora-livejournal-com.turbopages.org': 'annakora.livejournal.com',
 'moskvadeluxe-livejournal-com.turbopages.org': 'moskvadeluxe.livejournal.com',
 'e--lektrik-livejournal-com.turbopages.org': 'e-lektrik.livejournal.com',
 'mendkovich-livejournal-com.turbopages.org': 'mendkovich.livejournal.com',
 'miha--top-livejournal-com.turbopages.org': 'miha-top.livejournal.com',
 'marieclaire-ru.turbopages.org': 'marieclaire.ru',
 'botalex-livejournal-com.turbopages.org': 'botalex.livejournal.com',
 'greenarine-livejournal-com.turbopages.org': 'greenarine.livejournal.com',
 'borisfen70-livejournal-com.turbopages.org': 'borisfen70.livejournal.com',
 'ru--darts-livejournal-com.turbopages.org': 'ru-darts.livejournal.com',
 'shivelier-livejournal-com.turbopages.org': 'shivelier.livejournal.com',
 'ru--guitar-livejournal-com.turbopages.org': 'ru-guitar.livejournal.com',
 'ir--cherry-livejournal-com.turbopages.org': 'ir-cherry.livejournal.com',
 'ns--white--ns-livejournal-com.turbopages.org': 'ns-white-ns.livejournal.com',
 'shiropaev-livejournal-com.turbopages.org': 'shiropaev.livejournal.com',
 'famous--chechens-livejournal-com.turbopages.org': 'famous-chechens.livejournal.com',
 'sverc-livejournal-com.turbopages.org': 'sverc.livejournal.com',
 'gazeta--pkrest-livejournal-com.turbopages.org': 'gazeta-pkrest.livejournal.com',
 'afranius-livejournal-com.turbopages.org': 'afranius.livejournal.com',
 'tankist--31-livejournal-com.turbopages.org': 'tankist-31.livejournal.com',
 'marinazen-livejournal-com.turbopages.org': 'marinazen.livejournal.com',
 'avdysh--oleg-livejournal-com.turbopages.org': 'avdysh-oleg.livejournal.com',
 'bbb-livejournal-com.turbopages.org': 'bbb.livejournal.com',
 'kitubijca-livejournal-com.turbopages.org': 'kitubijca.livejournal.com',
 'verchitel22-livejournal-com.turbopages.org': 'verchitel22.livejournal.com',
 'bibe-ru.turbopages.org': 'bibe.ru',
 'konditermagazin-ru.turbopages.org': 'konditermagazin.ru',
 'jjnatt-livejournal-com.turbopages.org': 'jjnatt.livejournal.com',
 'vitimbabi4ev-livejournal-com.turbopages.org': 'vitimbabi4ev.livejournal.com',
 'kaminec-livejournal-com.turbopages.org': 'kaminec.livejournal.com',
 'noa--kinn-livejournal-com.turbopages.org': 'noa-kinn.livejournal.com',
 'basov--chukotka-livejournal-com.turbopages.org': 'basov-chukotka.livejournal.com',
 'pavel--petukhov-livejournal-com.turbopages.org': 'pavel-petukhov.livejournal.com',
 'traveltimes-ru.turbopages.org': 'traveltimes.ru',
 'ochki-guru.turbopages.org': 'ochki.guru',
 'stihirus24-ru.turbopages.org': 'stihirus24.ru',
 'za--gogu--lina-livejournal-com.turbopages.org': 'za-gogu-lina.livejournal.com',
 'leonovvaleri-livejournal-com.turbopages.org': 'leonovvaleri.livejournal.com',
 'inok--arkadiy-livejournal-com.turbopages.org': 'inok-arkadiy.livejournal.com',
 'cinik--ru-livejournal-com.turbopages.org': 'cinik-ru.livejournal.com',
 'elshanec-livejournal-com.turbopages.org': 'elshanec.livejournal.com',
 'dedmaxopka-livejournal-com.turbopages.org': 'dedmaxopka.livejournal.com',
 'golovko-livejournal-com.turbopages.org': 'golovko.livejournal.com',
 'arrsagro-ru.turbopages.org': 'arrsagro.ru',
 'thobias-livejournal-com.turbopages.org': 'thobias.livejournal.com',
 'kareliya--piter-livejournal-com.turbopages.org': 'kareliya-piter.livejournal.com',
 'arkhimtikhon-livejournal-com.turbopages.org': 'arkhimtikhon.livejournal.com',
 'yurij--maximov-livejournal-com.turbopages.org': 'yurij-maximov.livejournal.com',
 'avvakumpetrov-livejournal-com.turbopages.org': 'avvakumpetrov.livejournal.com',
 'bumerang90-livejournal-com.turbopages.org': 'bumerang90.livejournal.com',
 'parmanews-ru.turbopages.org': 'parmanews.ru',
 'judeomasson-livejournal-com.turbopages.org': 'judeomasson.livejournal.com',
 'gamrus-ru.turbopages.org': 'gamrus.ru',
 'engelgardt--lj-livejournal-com.turbopages.org': 'engelgardt-lj.livejournal.com',
 'privalovgame-livejournal-com.turbopages.org': 'privalovgame.livejournal.com',
 'ishvyrkov-livejournal-com.turbopages.org': 'ishvyrkov.livejournal.com',
 'slav--drevnosti-livejournal-com.turbopages.org': 'slav-drevnosti.livejournal.com',
 'gidrotgv-ru.turbopages.org': 'gidrotgv.ru',
 'elenawal-livejournal-com.turbopages.org': 'elenawal.livejournal.com',
 'bartov--e-livejournal-com.turbopages.org': 'bartov-e.livejournal.com',
 'ru--etymology-livejournal-com.turbopages.org': 'ru-etymology.livejournal.com',
 'gorizont--it-ru.turbopages.org': 'gorizont-it.ru',
 'bathet-livejournal-com.turbopages.org': 'bathet.livejournal.com',
 't24-su.turbopages.org': 't24.su',
 'mib55-livejournal-com.turbopages.org': 'mib55.livejournal.com',
 'tourbyself-ru.turbopages.org': 'tourbyself.ru',
 'discoman-livejournal-com.turbopages.org': 'discoman.livejournal.com',
 'sumy--grad-livejournal-com.turbopages.org': 'sumy-grad.livejournal.com',
 'astro-sibnet-ru.turbopages.org': 'astro.sibnet.ru',
 'dekarmi-livejournal-com.turbopages.org': 'dekarmi.livejournal.com',
 'swan07-livejournal-com.turbopages.org': 'swan07.livejournal.com',
 'dendy-club.turbopages.org': 'dendy.club',
 'kamchatkamedia-ru.turbopages.org': 'kamchatkamedia.ru',
 'kompilya--tor-livejournal-com.turbopages.org': 'kompilya-tor.livejournal.com',
 'mvestnik-ru.turbopages.org': 'mvestnik.ru',
 'soko1-livejournal-com.turbopages.org': 'soko1.livejournal.com',
 'aquael--russia-livejournal-com.turbopages.org': 'aquael-russia.livejournal.com',
 'cocosov-net.turbopages.org': 'cocosov.net',
 'nikozlov-livejournal-com.turbopages.org': 'nikozlov.livejournal.com',
 '2dar-livejournal-com.turbopages.org': '2dar.livejournal.com',
 'barsovy--travels-livejournal-com.turbopages.org': 'barsovy-travels.livejournal.com',
 't--trade-livejournal-com.turbopages.org': 't-trade.livejournal.com',
 'earthquaketrack-ru.turbopages.org': 'earthquaketrack.ru',
 'karlovypivovary-ru.turbopages.org': 'karlovypivovary.ru',
 'serovglobus-ru.turbopages.org': 'serovglobus.ru',
 'informer-news--kmv-ru.turbopages.org': 'informer.news-kmv.ru',
 'dachanaladoni-ru.turbopages.org': 'dachanaladoni.ru',
 'dvaraza-ru.turbopages.org': 'dvaraza.ru',
 'nw24-ru.turbopages.org': 'nw24.ru',
 'mastremont-ru.turbopages.org': 'mastremont.ru',
 'ruqrz-com.turbopages.org': 'ruqrz.com',
 'anntre-livejournal-com.turbopages.org': 'anntre.livejournal.com',
 'galkagalka-livejournal-com.turbopages.org': 'galkagalka.livejournal.com',
 'inna1903gr-livejournal-com.turbopages.org': 'inna1903gr.livejournal.com',
 'maria--gorynceva-livejournal-com.turbopages.org': 'maria-gorynceva.livejournal.com',
 'alfo--6in-livejournal-com.turbopages.org': 'alfo-6in.livejournal.com',
 'truecrime--lj-livejournal-com.turbopages.org': 'truecrime-lj.livejournal.com',
 'yuri--ne--gagarin-livejournal-com.turbopages.org': 'yuri-ne-gagarin.livejournal.com',
 'aaaandreww-livejournal-com.turbopages.org': 'aaaandreww.livejournal.com',
 'mi--ta--pe-livejournal-com.turbopages.org': 'mi-ta-pe.livejournal.com',
 'vova--91-livejournal-com.turbopages.org': 'vova-91.livejournal.com',
 'ho--hol-livejournal-com.turbopages.org': 'ho-hol.livejournal.com',
 'pd--v--np-livejournal-com.turbopages.org': 'pd-v-np.livejournal.com',
 'lexwork--page-livejournal-com.turbopages.org': 'lexwork-page.livejournal.com',
 'thelifeissimple-livejournal-com.turbopages.org': 'thelifeissimple.livejournal.com',
 'alon--delon-livejournal-com.turbopages.org': 'alon-delon.livejournal.com',
 'deni--m-livejournal-com.turbopages.org': 'deni-m.livejournal.com',
 'aaalyaska-livejournal-com.turbopages.org': 'aaalyaska.livejournal.com',
 'sika--sochi-ru.turbopages.org': 'sika-sochi.ru',
 'stimfaly--birdy-livejournal-com.turbopages.org': 'stimfaly-birdy.livejournal.com',
 'drunken--gunzz-livejournal-com.turbopages.org': 'drunken-gunzz.livejournal.com',
 'iry4-livejournal-com.turbopages.org': 'iry4.livejournal.com',
 'world--japan-livejournal-com.turbopages.org': 'world-japan.livejournal.com',
 'mystichrist-livejournal-com.turbopages.org': 'mystichrist.livejournal.com',
 'ipkisss-livejournal-com.turbopages.org': 'ipkisss.livejournal.com',
 'dimastuui-livejournal-com.turbopages.org': 'dimastuui.livejournal.com',
 'mister--sadovnik-ru.turbopages.org': 'mister-sadovnik.ru',
 'vremjadelat-ru.turbopages.org': 'vremjadelat.ru',
 'musecube-org.turbopages.org': 'musecube.org',
 'kayros--81-livejournal-com.turbopages.org': 'kayros-81.livejournal.com',
 'alexey--mochalov-livejournal-com.turbopages.org': 'alexey-mochalov.livejournal.com',
 'taimkofeinya-ru.turbopages.org': 'taimkofeinya.ru',
 'egor--bychkov-livejournal-com.turbopages.org': 'egor-bychkov.livejournal.com',
 'alex--rasceskin-livejournal-com.turbopages.org': 'alex-rasceskin.livejournal.com',
 'street--story-livejournal-com.turbopages.org': 'street-story.livejournal.com',
 'hawk--assinore-livejournal-com.turbopages.org': 'hawk-assinore.livejournal.com',
 'unico--unicornio-livejournal-com.turbopages.org': 'unico-unicornio.livejournal.com',
 'weeip-livejournal-com.turbopages.org': 'weeip.livejournal.com',
 'zhelty1-livejournal-com.turbopages.org': 'zhelty1.livejournal.com',
 'cena--auto-ru.turbopages.org': 'cena-auto.ru',
 'yuri--rus-livejournal-com.turbopages.org': 'yuri-rus.livejournal.com',
 'torbasow-livejournal-com.turbopages.org': 'torbasow.livejournal.com',
 'mickewich--w-livejournal-com.turbopages.org': 'mickewich-w.livejournal.com',
 'voytsekhovskiy-livejournal-com.turbopages.org': 'voytsekhovskiy.livejournal.com',
 'madlena--valeeva-livejournal-com.turbopages.org': 'madlena-valeeva.livejournal.com',
 'bubbakar-livejournal-com.turbopages.org': 'bubbakar.livejournal.com',
 'dnevniksite-ru.turbopages.org': 'dnevniksite.ru',
 'zig--zag63-livejournal-com.turbopages.org': 'zig-zag63.livejournal.com',
 'olika2011-livejournal-com.turbopages.org': 'olika2011.livejournal.com',
 'evlebedev3-livejournal-com.turbopages.org': 'evlebedev3.livejournal.com',
 'safety--il-livejournal-com.turbopages.org': 'safety-il.livejournal.com',
 'tarastrans-livejournal-com.turbopages.org': 'tarastrans.livejournal.com',
 'serfilatov-livejournal-com.turbopages.org': 'serfilatov.livejournal.com',
 '44-ua.turbopages.org': '44.ua',
 'lelikzvena-livejournal-com.turbopages.org': 'lelikzvena.livejournal.com',
 'marina--pavlova-livejournal-com.turbopages.org': 'marina-pavlova.livejournal.com',
 'penza-big--book--med-ru.turbopages.org': 'penza.big-book-med.ru',
 'stopparodontoz-ru.turbopages.org': 'stopparodontoz.ru',
 'dpmmax-livejournal-com.turbopages.org': 'dpmmax.livejournal.com',
 'chevrolet--niva-ru.turbopages.org': 'chevrolet-niva.ru',
 'pro--sousi-ru.turbopages.org': 'pro-sousi.ru',
 'krasivayagrud-ru.turbopages.org': 'krasivayagrud.ru',
 'cosmo--expo-livejournal-com.turbopages.org': 'cosmo-expo.livejournal.com',
 'ges--tapochka-livejournal-com.turbopages.org': 'ges-tapochka.livejournal.com',
 'rusdozor-ru.turbopages.org': 'rusdozor.ru',
 'tatiana--akko-livejournal-com.turbopages.org': 'tatiana-akko.livejournal.com',
 'zagar--ok-ru.turbopages.org': 'zagar-ok.ru',
 'euro--vip-livejournal-com.turbopages.org': 'euro-vip.livejournal.com',
 'artur--murzahan-livejournal-com.turbopages.org': 'artur-murzahan.livejournal.com',
 'd--konstantinov-livejournal-com.turbopages.org': 'd-konstantinov.livejournal.com',
 'volchuksergei-livejournal-com.turbopages.org': 'volchuksergei.livejournal.com',
 'snayper--poet-livejournal-com.turbopages.org': 'snayper-poet.livejournal.com',
 'opd--voshod-livejournal-com.turbopages.org': 'opd-voshod.livejournal.com',
 'imed3-livejournal-com.turbopages.org': 'imed3.livejournal.com',
 'sovetpodolska-livejournal-com.turbopages.org': 'sovetpodolska.livejournal.com',
 'taxinu-livejournal-com.turbopages.org': 'taxinu.livejournal.com',
 'yaznal-ru.turbopages.org': 'yaznal.ru',
 'vedmed1969-livejournal-com.turbopages.org': 'vedmed1969.livejournal.com',
 'poimaitur-ru.turbopages.org': 'poimaitur.ru',
 'pavel--kosenko-livejournal-com.turbopages.org': 'pavel-kosenko.livejournal.com',
 'ya--lyublyu-ru.turbopages.org': 'ya-lyublyu.ru',
 'fanasia-ru.turbopages.org': 'fanasia.ru',
 'bellezza--storia-livejournal-com.turbopages.org': 'bellezza-storia.livejournal.com',
 'myphototravel-livejournal-com.turbopages.org': 'myphototravel.livejournal.com',
 'samara--ru-livejournal-com.turbopages.org': 'samara-ru.livejournal.com',
 'bezymyansky-livejournal-com.turbopages.org': 'bezymyansky.livejournal.com',
 'zakshi-livejournal-com.turbopages.org': 'zakshi.livejournal.com',
 'ele--n--a-livejournal-com.turbopages.org': 'ele-n-a.livejournal.com',
 'prohojyi-livejournal-com.turbopages.org': 'prohojyi.livejournal.com',
 'kyl-guru.turbopages.org': 'kyl.guru',
 'rumyantseva--t-livejournal-com.turbopages.org': 'rumyantseva-t.livejournal.com',
 'zlatogora-livejournal-com.turbopages.org': 'zlatogora.livejournal.com',
 'moyarozochka-livejournal-com.turbopages.org': 'moyarozochka.livejournal.com',
 'pechi-guru.turbopages.org': 'pechi.guru',
 'uchebnyy--tsentr-ru.turbopages.org': 'uchebnyy-tsentr.ru',
 'fiestino-ru.turbopages.org': 'fiestino.ru',
 'ramis--nazmiev-livejournal-com.turbopages.org': 'ramis-nazmiev.livejournal.com',
 'marina--klimkova-livejournal-com.turbopages.org': 'marina-klimkova.livejournal.com',
 'nevrozanet-ru.turbopages.org': 'nevrozanet.ru',
 'alex54sar-livejournal-com.turbopages.org': 'alex54sar.livejournal.com',
 'kibika-com.turbopages.org': 'kibika.com',
 'potolokspec-com.turbopages.org': 'potolokspec.com',
 'artemu238-livejournal-com.turbopages.org': 'artemu238.livejournal.com',
 'pskov-big--book--med-ru.turbopages.org': 'pskov.big-book-med.ru',
 'barnas-livejournal-com.turbopages.org': 'barnas.livejournal.com',
 'irwi99-livejournal-com.turbopages.org': 'irwi99.livejournal.com',
 'ege--news-livejournal-com.turbopages.org': 'ege-news.livejournal.com',
 'slon--kirill-livejournal-com.turbopages.org': 'slon-kirill.livejournal.com',
 'cosmetic--oil-com.turbopages.org': 'cosmetic-oil.com',
 'film--gde-ru.turbopages.org': 'film-gde.ru',
 'dmitry--novak-livejournal-com.turbopages.org': 'dmitry-novak.livejournal.com',
 'mariinskiyposad-bezformata-com.turbopages.org': 'mariinskiyposad.bezformata.com',
 'freemedia-io.turbopages.org': 'freemedia.io',
 'romanovz-livejournal-com.turbopages.org': 'romanovz.livejournal.com',
 'nastya--maks-livejournal-com.turbopages.org': 'nastya-maks.livejournal.com',
 'ruslan--rb--kg-livejournal-com.turbopages.org': 'ruslan-rb-kg.livejournal.com',
 'rumol--leaks-livejournal-com.turbopages.org': 'rumol-leaks.livejournal.com',
 'twower-livejournal-com.turbopages.org': 'twower.livejournal.com',
 'muamuk-livejournal-com.turbopages.org': 'muamuk.livejournal.com',
 'siverkin-livejournal-com.turbopages.org': 'siverkin.livejournal.com',
 'atelman-livejournal-com.turbopages.org': 'atelman.livejournal.com',
 'politredaktor-livejournal-com.turbopages.org': 'politredaktor.livejournal.com',
 'koudesnik-livejournal-com.turbopages.org': 'koudesnik.livejournal.com',
 'giper-livejournal-com.turbopages.org': 'giper.livejournal.com',
 'sizran-bezformata-com.turbopages.org': 'sizran.bezformata.com',
 'bagira--belaya-livejournal-com.turbopages.org': 'bagira-belaya.livejournal.com',
 'yashka7-livejournal-com.turbopages.org': 'yashka7.livejournal.com',
 'sviyaga-livejournal-com.turbopages.org': 'sviyaga.livejournal.com',
 'natalyarchi-livejournal-com.turbopages.org': 'natalyarchi.livejournal.com',
 'podmoskva-livejournal-com.turbopages.org': 'podmoskva.livejournal.com',
 'bigbucks-com-ua.turbopages.org': 'bigbucks.com.ua',
 'cooknotes-livejournal-com.turbopages.org': 'cooknotes.livejournal.com',
 'lina--001-livejournal-com.turbopages.org': 'lina-001.livejournal.com',
 'avgulen-com.turbopages.org': 'avgulen.com',
 'krasivochka-ru.turbopages.org': 'krasivochka.ru',
 'alex--gar-livejournal-com.turbopages.org': 'alex-gar.livejournal.com',
 'mane4kasakh-livejournal-com.turbopages.org': 'mane4kasakh.livejournal.com',
 'viatcheslav-livejournal-com.turbopages.org': 'viatcheslav.livejournal.com',
 'monco83-livejournal-com.turbopages.org': 'monco83.livejournal.com',
 'blog--trepashkin-livejournal-com.turbopages.org': 'blog-trepashkin.livejournal.com',
 'olga--anz-livejournal-com.turbopages.org': 'olga-anz.livejournal.com',
 'stepankapusta-livejournal-com.turbopages.org': 'stepankapusta.livejournal.com',
 'pochemunet-ru.turbopages.org': 'pochemunet.ru',
 'travel--pfo-livejournal-com.turbopages.org': 'travel-pfo.livejournal.com',
 'fitonyashka-net.turbopages.org': 'fitonyashka.net',
 'lifesouldeath-livejournal-com.turbopages.org': 'lifesouldeath.livejournal.com',
 'fort--i--ko-livejournal-com.turbopages.org': 'fort-i-ko.livejournal.com',
 'miragerry-livejournal-com.turbopages.org': 'miragerry.livejournal.com',
 'yakimovskie-livejournal-com.turbopages.org': 'yakimovskie.livejournal.com',
 'spasskajaola-livejournal-com.turbopages.org': 'spasskajaola.livejournal.com',
 'coream-ru.turbopages.org': 'coream.ru',
 'prilepin-livejournal-com.turbopages.org': 'prilepin.livejournal.com',
 'litsenzirovanie-livejournal-com.turbopages.org': 'litsenzirovanie.livejournal.com',
 'russ--ad-livejournal-com.turbopages.org': 'russ-ad.livejournal.com',
 'eduard--456-livejournal-com.turbopages.org': 'eduard-456.livejournal.com',
 'malecomunity-livejournal-com.turbopages.org': 'malecomunity.livejournal.com',
 'novosibirsk-big--book--med-ru.turbopages.org': 'novosibirsk.big-book-med.ru',
 'i--sanin-livejournal-com.turbopages.org': 'i-sanin.livejournal.com',
 'golos--dobra-livejournal-com.turbopages.org': 'golos-dobra.livejournal.com',
 'training--dog-livejournal-com.turbopages.org': 'training-dog.livejournal.com',
 'car--news-info.turbopages.org': 'car-news.info',
 'kakgevkusno-ru.turbopages.org': 'kakgevkusno.ru',
 'kizlyar--cognac-ru.turbopages.org': 'kizlyar-cognac.ru',
 'nationmagazine-ru.turbopages.org': 'nationmagazine.ru',
 'kurs2015-ru.turbopages.org': 'kurs2015.ru',
 'shop-christmas--plus-ru.turbopages.org': 'shop.christmas-plus.ru',
 'mikka-livejournal-com.turbopages.org': 'mikka.livejournal.com',
 'goldpekary-livejournal-com.turbopages.org': 'goldpekary.livejournal.com',
 'amonov-livejournal-com.turbopages.org': 'amonov.livejournal.com',
 'nu--today-ru.turbopages.org': 'nu-today.ru',
 'fvsd-ru.turbopages.org': 'fvsd.ru',
 'dmurashev-livejournal-com.turbopages.org': 'dmurashev.livejournal.com',
 'ortnit-livejournal-com.turbopages.org': 'ortnit.livejournal.com',
 'calendum-ru.turbopages.org': 'calendum.ru',
 'stepvweb-ru.turbopages.org': 'stepvweb.ru',
 'prosad-ru.turbopages.org': 'prosad.ru',
 'natalya--kiriche-livejournal-com.turbopages.org': 'natalya-kiriche.livejournal.com',
 'kotmusico-livejournal-com.turbopages.org': 'kotmusico.livejournal.com',
 'v--ozhidanii--chuda-ru.turbopages.org': 'v-ozhidanii-chuda.ru',
 'romanshlyapin-livejournal-com.turbopages.org': 'romanshlyapin.livejournal.com',
 'hunghutzu-livejournal-com.turbopages.org': 'hunghutzu.livejournal.com',
 'zharovnya-club.turbopages.org': 'zharovnya.club',
 'archi--fact-livejournal-com.turbopages.org': 'archi-fact.livejournal.com',
 'itbase-livejournal-com.turbopages.org': 'itbase.livejournal.com',
 'masharinka-livejournal-com.turbopages.org': 'masharinka.livejournal.com',
 'drug--goy-livejournal-com.turbopages.org': 'drug-goy.livejournal.com',
 'osolntseva-livejournal-com.turbopages.org': 'osolntseva.livejournal.com',
 '10diet-ru.turbopages.org': '10diet.ru',
 'irina--rm-livejournal-com.turbopages.org': 'irina-rm.livejournal.com',
 'kakzarabativat-ru.turbopages.org': 'kakzarabativat.ru',
 'deni--spiri-livejournal-com.turbopages.org': 'deni-spiri.livejournal.com',
 'sub--zero-livejournal-com.turbopages.org': 'sub-zero.livejournal.com',
 'pivobzor-ru.turbopages.org': 'pivobzor.ru',
 'sonnik--expert-ru.turbopages.org': 'sonnik-expert.ru',
 'gu--mvd63-livejournal-com.turbopages.org': 'gu-mvd63.livejournal.com',
 'kristinkama-livejournal-com.turbopages.org': 'kristinkama.livejournal.com',
 'njkntr-livejournal-com.turbopages.org': 'njkntr.livejournal.com',
 'gadkaya--mau-livejournal-com.turbopages.org': 'gadkaya-mau.livejournal.com',
 'bodybalet-ru.turbopages.org': 'bodybalet.ru',
 'natasheba-livejournal-com.turbopages.org': 'natasheba.livejournal.com',
 'riadmar-livejournal-com.turbopages.org': 'riadmar.livejournal.com',
 'lenny--van--ross-livejournal-com.turbopages.org': 'lenny-van-ross.livejournal.com',
 'floridei-ru.turbopages.org': 'floridei.ru',
 'xatarov-livejournal-com.turbopages.org': 'xatarov.livejournal.com',
 'anna--prokofeva-livejournal-com.turbopages.org': 'anna-prokofeva.livejournal.com',
 'arlyonka-livejournal-com.turbopages.org': 'arlyonka.livejournal.com',
 'goryachev-livejournal-com.turbopages.org': 'goryachev.livejournal.com',
 'labazov-livejournal-com.turbopages.org': 'labazov.livejournal.com',
 'caucasian--dress-livejournal-com.turbopages.org': 'caucasian-dress.livejournal.com',
 'energa-livejournal-com.turbopages.org': 'energa.livejournal.com',
 'arman71-livejournal-com.turbopages.org': 'arman71.livejournal.com',
 'kaspijs-livejournal-com.turbopages.org': 'kaspijs.livejournal.com',
 'atrey-livejournal-com.turbopages.org': 'atrey.livejournal.com',
 'go2all-livejournal-com.turbopages.org': 'go2all.livejournal.com',
 'z--yanson-livejournal-com.turbopages.org': 'z-yanson.livejournal.com',
 'sahoganin-livejournal-com.turbopages.org': 'sahoganin.livejournal.com',
 'sadvokrug-ru.turbopages.org': 'sadvokrug.ru',
 'appassionata--lr-livejournal-com.turbopages.org': 'appassionata-lr.livejournal.com',
 'velikiynovgorod-bezformata-com.turbopages.org': 'velikiynovgorod.bezformata.com',
 'gilljan-livejournal-com.turbopages.org': 'gilljan.livejournal.com',
 'linod-livejournal-com.turbopages.org': 'linod.livejournal.com',
 'sewingworld-livejournal-com.turbopages.org': 'sewingworld.livejournal.com',
 'akeelow-ru.turbopages.org': 'akeelow.ru',
 'alex--mclydy-livejournal-com.turbopages.org': 'alex-mclydy.livejournal.com',
 'xn------5cd1a0adji7afenql3q.turbopages.org': 'xn---5cd1a0adji7afenql3q',
 'nemabazara-ru.turbopages.org': 'nemabazara.ru',
 'sobolev82-livejournal-com.turbopages.org': 'sobolev82.livejournal.com',
 'boho--area-livejournal-com.turbopages.org': 'boho-area.livejournal.com',
 'fyunt-livejournal-com.turbopages.org': 'fyunt.livejournal.com',
 'ns--tribunal-livejournal-com.turbopages.org': 'ns-tribunal.livejournal.com',
 'zakusilov-livejournal-com.turbopages.org': 'zakusilov.livejournal.com',
 'martinis09-livejournal-com.turbopages.org': 'martinis09.livejournal.com',
 'anti--kod-livejournal-com.turbopages.org': 'anti-kod.livejournal.com',
 'evizvarina-livejournal-com.turbopages.org': 'evizvarina.livejournal.com',
 'pppirate-livejournal-com.turbopages.org': 'pppirate.livejournal.com',
 'starlife--tv-ru.turbopages.org': 'starlife-tv.ru',
 'fem--books-livejournal-com.turbopages.org': 'fem-books.livejournal.com',
 'death2032-livejournal-com.turbopages.org': 'death2032.livejournal.com',
 'swoonduchok-livejournal-com.turbopages.org': 'swoonduchok.livejournal.com',
 'mossudmed-livejournal-com.turbopages.org': 'mossudmed.livejournal.com',
 'nebogatoff-ru.turbopages.org': 'nebogatoff.ru',
 'detskaya-info.turbopages.org': 'detskaya.info',
 'masloauto-ru.turbopages.org': 'masloauto.ru',
 'insaneideas-livejournal-com.turbopages.org': 'insaneideas.livejournal.com',
 'su--maloetazhki-livejournal-com.turbopages.org': 'su-maloetazhki.livejournal.com',
 'khabarovsk-big--book--med-ru.turbopages.org': 'khabarovsk.big-book-med.ru',
 'lyashenko--serg-livejournal-com.turbopages.org': 'lyashenko-serg.livejournal.com',
 'gdeklad-ru.turbopages.org': 'gdeklad.ru',
 'hiromantiya-livejournal-com.turbopages.org': 'hiromantiya.livejournal.com',
 'mycoins-info.turbopages.org': 'mycoins.info',
 'elhombresombro-livejournal-com.turbopages.org': 'elhombresombro.livejournal.com',
 'arhview-livejournal-com.turbopages.org': 'arhview.livejournal.com',
 'moto--fix-livejournal-com.turbopages.org': 'moto-fix.livejournal.com',
 'yu--dzin-livejournal-com.turbopages.org': 'yu-dzin.livejournal.com',
 'varikap-ru.turbopages.org': 'varikap.ru',
 'le--flowerbird-livejournal-com.turbopages.org': 'le-flowerbird.livejournal.com',
 'moskva-vbr-ru.turbopages.org': 'moskva.vbr.ru',
 'dvprogram-us.turbopages.org': 'dvprogram.us',
 'afisha--omsk-ru.turbopages.org': 'afisha-omsk.ru',
 'obzorstroy-ru.turbopages.org': 'obzorstroy.ru',
 'msk--horoshevka-livejournal-com.turbopages.org': 'msk-horoshevka.livejournal.com',
 'funnew-ru.turbopages.org': 'funnew.ru',
 'byuanov--ed-ru.turbopages.org': 'byuanov-ed.ru',
 'jury--a--kap-livejournal-com.turbopages.org': 'jury-a-kap.livejournal.com',
 'r--nedo-livejournal-com.turbopages.org': 'r-nedo.livejournal.com',
 'max--fedosov-livejournal-com.turbopages.org': 'max-fedosov.livejournal.com',
 'anyasans-livejournal-com.turbopages.org': 'anyasans.livejournal.com',
 'itcblog-ru.turbopages.org': 'itcblog.ru',
 'missalgir-livejournal-com.turbopages.org': 'missalgir.livejournal.com',
 'baby--tour-livejournal-com.turbopages.org': 'baby-tour.livejournal.com',
    'blog-cjbars-ru.turbopages.org': 'blog.cjbars.ru',
 'b8x-ru.turbopages.org': 'b8x.ru',
 'kotelsibir-ru.turbopages.org': 'kotelsibir.ru',
 'winter--fishing-ru.turbopages.org': 'winter-fishing.ru',
 'dedgnat-livejournal-com.turbopages.org': 'dedgnat.livejournal.com',
 'otravlenym-ru.turbopages.org': 'otravlenym.ru',
 'evgenia2108-livejournal-com.turbopages.org': 'evgenia2108.livejournal.com',
 'gribokexpert-ru.turbopages.org': 'gribokexpert.ru',
 'turbogaga-livejournal-com.turbopages.org': 'turbogaga.livejournal.com',
 'mysea-livejournal-com.turbopages.org': 'mysea.livejournal.com',
 'prolimfouzly-ru.turbopages.org': 'prolimfouzly.ru',
 'rasklad-net.turbopages.org': 'rasklad.net',
 'ufa-big--book--med-ru.turbopages.org': 'ufa.big-book-med.ru',
 'samoshvejka-ru.turbopages.org': 'samoshvejka.ru',
 'updron-ru.turbopages.org': 'updron.ru',
 'leto--kat-livejournal-com.turbopages.org': 'leto-kat.livejournal.com',
 'maslogsm-ru.turbopages.org': 'maslogsm.ru',
 'ssgen-livejournal-com.turbopages.org': 'ssgen.livejournal.com',
 'killermurder-livejournal-com.turbopages.org': 'killermurder.livejournal.com',
 'begamot--74-livejournal-com.turbopages.org': 'begamot-74.livejournal.com',
 'prosto--mariya27-livejournal-com.turbopages.org': 'prosto-mariya27.livejournal.com',
 'sterligov-livejournal-com.turbopages.org': 'sterligov.livejournal.com',
 'roman--n-livejournal-com.turbopages.org': 'roman-n.livejournal.com',
 'vitkvv2017-livejournal-com.turbopages.org': 'vitkvv2017.livejournal.com',
 'aleksej--cccp-livejournal-com.turbopages.org': 'aleksej-cccp.livejournal.com',
 'ashudad-livejournal-com.turbopages.org': 'ashudad.livejournal.com',
 'vagant-livejournal-com.turbopages.org': 'vagant.livejournal.com',
 'astori--18-livejournal-com.turbopages.org': 'astori-18.livejournal.com',
 'dorogisk-livejournal-com.turbopages.org': 'dorogisk.livejournal.com',
 'traveller2-livejournal-com.turbopages.org': 'traveller2.livejournal.com',
 'anton--raevsky-livejournal-com.turbopages.org': 'anton-raevsky.livejournal.com',
 'singing--bowls-ru.turbopages.org': 'singing-bowls.ru',
 'astlena-livejournal-com.turbopages.org': 'astlena.livejournal.com',
 'mirgrudnichka-ru.turbopages.org': 'mirgrudnichka.ru',
 'raduga--oksana-livejournal-com.turbopages.org': 'raduga-oksana.livejournal.com',
 'jorra--jorra-livejournal-com.turbopages.org': 'jorra-jorra.livejournal.com',
 'barrakuda63-livejournal-com.turbopages.org': 'barrakuda63.livejournal.com',
 'allcharms-ru.turbopages.org': 'allcharms.ru',
 'omsk--news-net.turbopages.org': 'omsk-news.net',
 'v--pc-ru.turbopages.org': 'v-pc.ru',
 'olegkir2002-livejournal-com.turbopages.org': 'olegkir2002.livejournal.com',
 'pltrk-ru.turbopages.org': 'pltrk.ru',
 'tojedengi-ru.turbopages.org': 'tojedengi.ru',
 'rimeyer-livejournal-com.turbopages.org': 'rimeyer.livejournal.com',
 'mks--c-ru.turbopages.org': 'mks-c.ru',
 'studopedya-ru.turbopages.org': 'studopedya.ru',
 'galkina133-livejournal-com.turbopages.org': 'galkina133.livejournal.com',
 'povarewka-ru.turbopages.org': 'povarewka.ru',
 'kulinarium-livejournal-com.turbopages.org': 'kulinarium.livejournal.com',
 'worldofhistory-ru.turbopages.org': 'worldofhistory.ru',
 'sonoffstore-ru.turbopages.org': 'sonoffstore.ru',
 '9161728639-livejournal-com.turbopages.org': '9161728639.livejournal.com',
 'turkis--ser-ru.turbopages.org': 'turkis-ser.ru',
 'anty--big--game-livejournal-com.turbopages.org': 'anty-big-game.livejournal.com',
 'burckina--faso-livejournal-com.turbopages.org': 'burckina-faso.livejournal.com',
 'primorye24-ru.turbopages.org': 'primorye24.ru',
 'etm--club-site.turbopages.org': 'etm-club.site',
 'luhovici-bezformata-com.turbopages.org': 'luhovici.bezformata.com',
 'lekso79-livejournal-com.turbopages.org': 'lekso79.livejournal.com',
 'vashmedsovetnik-com.turbopages.org': 'vashmedsovetnik.com',
 'isl-livejournal-com.turbopages.org': 'isl.livejournal.com',
 'alfa--delta-livejournal-com.turbopages.org': 'alfa-delta.livejournal.com',
 'new--rabochy-livejournal-com.turbopages.org': 'new-rabochy.livejournal.com',
 'kohama--natalia-livejournal-com.turbopages.org': 'kohama-natalia.livejournal.com',
 'revizornews-ru.turbopages.org': 'revizornews.ru',
 'fakestop-livejournal-com.turbopages.org': 'fakestop.livejournal.com',
 'kyhni-guru.turbopages.org': 'kyhni.guru',
 'politcentr-ru.turbopages.org': 'politcentr.ru',
 'nash--krai-ru.turbopages.org': 'nash-krai.ru',
 'market--crimea-com.turbopages.org': 'market-crimea.com',
 'ja--sama--ru-livejournal-com.turbopages.org': 'ja-sama-ru.livejournal.com',
 'myakota-livejournal-com.turbopages.org': 'myakota.livejournal.com',
 'qebedo-livejournal-com.turbopages.org': 'qebedo.livejournal.com',
 'miguru-ru.turbopages.org': 'miguru.ru',
 'rodina--znaet-livejournal-com.turbopages.org': 'rodina-znaet.livejournal.com',
 'kuznets-livejournal-com.turbopages.org': 'kuznets.livejournal.com',
 'nk--tv-com.turbopages.org': 'nk-tv.com',
 'kids--in--trips-ru.turbopages.org': 'kids-in-trips.ru',
 'kizil-bezformata-com.turbopages.org': 'kizil.bezformata.com',
 'avto--moto--shtuchki-ru.turbopages.org': 'avto-moto-shtuchki.ru',
 'kamnedeloff-ru.turbopages.org': 'kamnedeloff.ru',
 'yura--turist-livejournal-com.turbopages.org': 'yura-turist.livejournal.com',
 'iguana--coach-livejournal-com.turbopages.org': 'iguana-coach.livejournal.com',
 'light2811-livejournal-com.turbopages.org': 'light2811.livejournal.com',
 'talimilenkaya-livejournal-com.turbopages.org': 'talimilenkaya.livejournal.com',
 'lunnay--mama-livejournal-com.turbopages.org': 'lunnay-mama.livejournal.com',
 'maria--vlasova-livejournal-com.turbopages.org': 'maria-vlasova.livejournal.com',
 'elena--boel-livejournal-com.turbopages.org': 'elena-boel.livejournal.com',
 'kuzya--07-livejournal-com.turbopages.org': 'kuzya-07.livejournal.com',
 'astro--club-com-ua.turbopages.org': 'astro-club.com.ua',
 'utrokrasit-livejournal-com.turbopages.org': 'utrokrasit.livejournal.com',
 'gulnaz528-livejournal-com.turbopages.org': 'gulnaz528.livejournal.com',
 'sta--rep-ru.turbopages.org': 'sta-rep.ru',
 'grandgames-net.turbopages.org': 'grandgames.net',
 'casatic-livejournal-com.turbopages.org': 'casatic.livejournal.com',
 'dkvitin-ru.turbopages.org': 'dkvitin.ru',
 'showbiz--life-ru.turbopages.org': 'showbiz-life.ru',
 'articles-opexflow-com.turbopages.org': 'articles.opexflow.com',
 'navy--korabel-livejournal-com.turbopages.org': 'navy-korabel.livejournal.com',
 'tambov-mk-ru.turbopages.org': 'tambov.mk.ru',
 'maxim--ryzhov-livejournal-com.turbopages.org': 'maxim-ryzhov.livejournal.com',
 'lamazo-livejournal-com.turbopages.org': 'lamazo.livejournal.com',
 'petropavlovskkamchatskiy-bezformata-com.turbopages.org': 'petropavlovskkamchatskiy.bezformata.com',
 'nehludoff-livejournal-com.turbopages.org': 'nehludoff.livejournal.com',
 'voencomuezd-livejournal-com.turbopages.org': 'voencomuezd.livejournal.com',
 'banki--card-ru.turbopages.org': 'banki-card.ru',
 'jeleznodorojniy-bezformata-com.turbopages.org': 'jeleznodorojniy.bezformata.com',
 'topwoman-info.turbopages.org': 'topwoman.info',
 'laikovo-info.turbopages.org': 'laikovo.info',
 'belyi--mitia-livejournal-com.turbopages.org': 'belyi-mitia.livejournal.com',
 'freeseller-ru.turbopages.org': 'freeseller.ru',
 'green--wiki-ru.turbopages.org': 'green-wiki.ru',
 'xn----ktbiidmil0b8g.turbopages.org': 'xn--ktbiidmil0b8g',
 'design--bathroom-ru.turbopages.org': 'design-bathroom.ru',
 'vhod-guru.turbopages.org': 'vhod.guru',
 'analiz--imeni-ru.turbopages.org': 'analiz-imeni.ru',
 'yahunter-livejournal-com.turbopages.org': 'yahunter.livejournal.com',
 'swarogfond-livejournal-com.turbopages.org': 'swarogfond.livejournal.com',
 'tapirr-livejournal-com.turbopages.org': 'tapirr.livejournal.com',
 'donnightman-livejournal-com.turbopages.org': 'donnightman.livejournal.com',
 'ru--hidden-livejournal-com.turbopages.org': 'ru-hidden.livejournal.com',
 '101benzopila-ru.turbopages.org': '101benzopila.ru',
 'dobriifin-livejournal-com.turbopages.org': 'dobriifin.livejournal.com',
 'k3y-ru.turbopages.org': 'k3y.ru',
 'trara-ru.turbopages.org': 'trara.ru',
 'mstrkrft-ru.turbopages.org': 'mstrkrft.ru',
 'mozgokratia-ru.turbopages.org': 'mozgokratia.ru',
 'inthai-livejournal-com.turbopages.org': 'inthai.livejournal.com',
 'vedaveta-livejournal-com.turbopages.org': 'vedaveta.livejournal.com',
 'refite-ru.turbopages.org': 'refite.ru',
 'kadochnikoff-livejournal-com.turbopages.org': 'kadochnikoff.livejournal.com',
 '1ladapriora-ru.turbopages.org': '1ladapriora.ru',
 'anderson--mike-livejournal-com.turbopages.org': 'anderson-mike.livejournal.com',
 'iron--vobla-livejournal-com.turbopages.org': 'iron-vobla.livejournal.com',
 'accauntlol-ru.turbopages.org': 'accauntlol.ru',
 'valgaming-com.turbopages.org': 'valgaming.com',
 'ru--mac-livejournal-com.turbopages.org': 'ru-mac.livejournal.com',
 'spalexxx-livejournal-com.turbopages.org': 'spalexxx.livejournal.com',
 'meliell17-livejournal-com.turbopages.org': 'meliell17.livejournal.com',
 'galartemenko-livejournal-com.turbopages.org': 'galartemenko.livejournal.com',
 'zagotovochkj-ru.turbopages.org': 'zagotovochkj.ru',
 'mrdou-livejournal-com.turbopages.org': 'mrdou.livejournal.com',
 'pozelanie-ru.turbopages.org': 'pozelanie.ru',
 'diviola-livejournal-com.turbopages.org': 'diviola.livejournal.com',
 'yaslyam--da-livejournal-com.turbopages.org': 'yaslyam-da.livejournal.com',
 'doctor--2000-livejournal-com.turbopages.org': 'doctor-2000.livejournal.com',
 'silazdorovya-ru.turbopages.org': 'silazdorovya.ru',
 'strravaganza-livejournal-com.turbopages.org': 'strravaganza.livejournal.com',
 'chet--nik-livejournal-com.turbopages.org': 'chet-nik.livejournal.com',
 'brilliant--life-livejournal-com.turbopages.org': 'brilliant-life.livejournal.com',
 'stroi--baza-ru.turbopages.org': 'stroi-baza.ru',
 'magiya-guru.turbopages.org': 'magiya.guru',
 'samolet-media.turbopages.org': 'samolet.media',
 'vallentinn-livejournal-com.turbopages.org': 'vallentinn.livejournal.com',
 'alkogol--doma-ru.turbopages.org': 'alkogol-doma.ru',
 'etymology--ru-livejournal-com.turbopages.org': 'etymology-ru.livejournal.com',
 'joe--cooker-livejournal-com.turbopages.org': 'joe-cooker.livejournal.com',
 'egoryevsk-bezformata-com.turbopages.org': 'egoryevsk.bezformata.com',
 'anatoligreen-livejournal-com.turbopages.org': 'anatoligreen.livejournal.com',
 'edgi-livejournal-com.turbopages.org': 'edgi.livejournal.com',
 'drumbumbum-livejournal-com.turbopages.org': 'drumbumbum.livejournal.com',
 'vyshemir-livejournal-com.turbopages.org': 'vyshemir.livejournal.com',
 'wt--blog-net.turbopages.org': 'wt-blog.net',
 'general--skokov-livejournal-com.turbopages.org': 'general-skokov.livejournal.com',
 'antona1976-livejournal-com.turbopages.org': 'antona1976.livejournal.com',
 'turagentovnet-ru.turbopages.org': 'turagentovnet.ru',
 'yageru-livejournal-com.turbopages.org': 'yageru.livejournal.com',
 'hardware--museum-livejournal-com.turbopages.org': 'hardware-museum.livejournal.com',
 'beincrypto-ru.turbopages.org': 'beincrypto.ru',
 'lelady-ru.turbopages.org': 'lelady.ru',
 'maggala-livejournal-com.turbopages.org': 'maggala.livejournal.com',
 'mebel--v--nsk-ru.turbopages.org': 'mebel-v-nsk.ru',
 'ots--v--v-livejournal-com.turbopages.org': 'ots-v-v.livejournal.com',
 'np0cmo--ma-livejournal-com.turbopages.org': 'np0cmo-ma.livejournal.com',
 'foto--sib-livejournal-com.turbopages.org': 'foto-sib.livejournal.com',
 'zverushka--sos-livejournal-com.turbopages.org': 'zverushka-sos.livejournal.com',
 'andrewleslie-livejournal-com.turbopages.org': 'andrewleslie.livejournal.com',
 'endocrin--patient-com.turbopages.org': 'endocrin-patient.com',
 'xn----otbohaj2a.turbopages.org': 'xn--otbohaj2a',
 'family--doctor13-ru.turbopages.org': 'family-doctor13.ru',
 'psymod-ru.turbopages.org': 'psymod.ru',
 'chelchel--ru-livejournal-com.turbopages.org': 'chelchel-ru.livejournal.com',
 'palata--namber--6-livejournal-com.turbopages.org': 'palata-namber-6.livejournal.com',
 'up--to--snuff-livejournal-com.turbopages.org': 'up-to-snuff.livejournal.com',
 'evgenyastasaid-livejournal-com.turbopages.org': 'evgenyastasaid.livejournal.com',
 'weblake-ru.turbopages.org': 'weblake.ru',
 'sizif73-livejournal-com.turbopages.org': 'sizif73.livejournal.com',
 'ondatra-livejournal-com.turbopages.org': 'ondatra.livejournal.com',
 'electronicsblog-ru.turbopages.org': 'electronicsblog.ru',
 'varyag--nord-livejournal-com.turbopages.org': 'varyag-nord.livejournal.com',
 'tehnopomosh-com.turbopages.org': 'tehnopomosh.com',
 'to4et-livejournal-com.turbopages.org': 'to4et.livejournal.com',
 'telecomdom-ru.turbopages.org': 'telecomdom.ru',
 'charmelle--co-ru.turbopages.org': 'charmelle-co.ru',
 'meditate-fun.turbopages.org': 'meditate.fun',
 'imperia--vrn-ru.turbopages.org': 'imperia-vrn.ru',
 'krylatskoye-ru.turbopages.org': 'krylatskoye.ru',
 'top--raitingi-ru.turbopages.org': 'top-raitingi.ru',
 'fingaring-livejournal-com.turbopages.org': 'fingaring.livejournal.com',
 'matma-ru.turbopages.org': 'matma.ru',
 'smoking--devices-livejournal-com.turbopages.org': 'smoking-devices.livejournal.com',
 'podarijizn-livejournal-com.turbopages.org': 'podarijizn.livejournal.com',
 'verola-livejournal-com.turbopages.org': 'verola.livejournal.com',
 'shagirt-livejournal-com.turbopages.org': 'shagirt.livejournal.com',
 'a--glazunov-livejournal-com.turbopages.org': 'a-glazunov.livejournal.com',
 'edalnya-com.turbopages.org': 'edalnya.com',
 'legatus-livejournal-com.turbopages.org': 'legatus.livejournal.com',
 'anadyr-org.turbopages.org': 'anadyr.org',
 'city--pages-info.turbopages.org': 'city-pages.info',
 'my--zenit-ru.turbopages.org': 'my-zenit.ru',
 'yilmazsoy-livejournal-com.turbopages.org': 'yilmazsoy.livejournal.com',
 'master--houses-ru.turbopages.org': 'master-houses.ru',
 'anastassia--jm-livejournal-com.turbopages.org': 'anastassia-jm.livejournal.com',
 'e--aleks-livejournal-com.turbopages.org': 'e-aleks.livejournal.com',
 'liveracing-livejournal-com.turbopages.org': 'liveracing.livejournal.com',
 'rohal--emarat-livejournal-com.turbopages.org': 'rohal-emarat.livejournal.com',
 'portalrebenka-com.turbopages.org': 'portalrebenka.com',
 'detonator666-livejournal-com.turbopages.org': 'detonator666.livejournal.com',
 'gelendzhik-kurort--pro-ru.turbopages.org': 'gelendzhik.kurort-pro.ru',
 'elinastyle-livejournal-com.turbopages.org': 'elinastyle.livejournal.com',
 'alex--888555-livejournal-com.turbopages.org': 'alex-888555.livejournal.com',
 'mkala-mk-ru.turbopages.org': 'mkala.mk.ru',
 'aviator--ru-livejournal-com.turbopages.org': 'aviator-ru.livejournal.com',
 'avon--061-ru.turbopages.org': 'avon-061.ru',
 'lopasnya-ru.turbopages.org': 'lopasnya.ru',
 'kryptobike-ru.turbopages.org': 'kryptobike.ru',
 'skif1959-livejournal-com.turbopages.org': 'skif1959.livejournal.com',
 'rinka-livejournal-com.turbopages.org': 'rinka.livejournal.com',
 'delyagin-livejournal-com.turbopages.org': 'delyagin.livejournal.com',
 'anni--meranni-livejournal-com.turbopages.org': 'anni-meranni.livejournal.com',
 'uvova-livejournal-com.turbopages.org': 'uvova.livejournal.com',
 'obzorus-com.turbopages.org': 'obzorus.com',
 'nposg2016-livejournal-com.turbopages.org': 'nposg2016.livejournal.com',
 'ego--drakonis-livejournal-com.turbopages.org': 'ego-drakonis.livejournal.com',
 'blogfilosofa-livejournal-com.turbopages.org': 'blogfilosofa.livejournal.com',
 'melon--panda-livejournal-com.turbopages.org': 'melon-panda.livejournal.com',
 'politinform-su.turbopages.org': 'politinform.su',
 'hist--etnol-livejournal-com.turbopages.org': 'hist-etnol.livejournal.com',
 'pelmax-livejournal-com.turbopages.org': 'pelmax.livejournal.com',
 'vodablog-livejournal-com.turbopages.org': 'vodablog.livejournal.com',
 'tyhomir-livejournal-com.turbopages.org': 'tyhomir.livejournal.com',
 'golovin1970-livejournal-com.turbopages.org': 'golovin1970.livejournal.com',
 'xn------9cdai3b7bfavdio9ab.turbopages.org': 'xn---9cdai3b7bfavdio9ab',
 'stasya--lerkina-livejournal-com.turbopages.org': 'stasya-lerkina.livejournal.com',
 'perisher--13-livejournal-com.turbopages.org': 'perisher-13.livejournal.com',
 'solod--suslo-ru.turbopages.org': 'solod-suslo.ru',
 'v--strane--i--mire-livejournal-com.turbopages.org': 'v-strane-i-mire.livejournal.com',
 'prof--sovet-ru.turbopages.org': 'prof-sovet.ru',
 'iskra0000-livejournal-com.turbopages.org': 'iskra0000.livejournal.com',
 'houses-ru.turbopages.org': 'houses.ru',
 'haloterapia-ru.turbopages.org': 'haloterapia.ru',
 'chev--chelios--09-livejournal-com.turbopages.org': 'chev-chelios-09.livejournal.com',
 'korolev-bezformata-com.turbopages.org': 'korolev.bezformata.com',
 'made--inussr-livejournal-com.turbopages.org': 'made-inussr.livejournal.com',
 'mvdiary-livejournal-com.turbopages.org': 'mvdiary.livejournal.com',
 'tovar-vip.turbopages.org': 'tovar.vip',
 'stradivari-livejournal-com.turbopages.org': 'stradivari.livejournal.com',
 'sudenko-livejournal-com.turbopages.org': 'sudenko.livejournal.com',
 'alfred--shnobel-livejournal-com.turbopages.org': 'alfred-shnobel.livejournal.com',
 'zdrav--grud-ru.turbopages.org': 'zdrav-grud.ru',
 'nickned-livejournal-com.turbopages.org': 'nickned.livejournal.com',
 'timeskhan-livejournal-com.turbopages.org': 'timeskhan.livejournal.com',
 'guardofword-com.turbopages.org': 'guardofword.com',
 'asmithmtrx-livejournal-com.turbopages.org': 'asmithmtrx.livejournal.com',
 'olga--al-livejournal-com.turbopages.org': 'olga-al.livejournal.com',
 'astrodeva-ru.turbopages.org': 'astrodeva.ru',
 'irina--calish-livejournal-com.turbopages.org': 'irina-calish.livejournal.com',
 'otolm-livejournal-com.turbopages.org': 'otolm.livejournal.com',
 'raduga--astro-livejournal-com.turbopages.org': 'raduga-astro.livejournal.com',
 'vlada--bern-livejournal-com.turbopages.org': 'vlada-bern.livejournal.com',
 'powerforpeople-livejournal-com.turbopages.org': 'powerforpeople.livejournal.com',
 'plantatravel-livejournal-com.turbopages.org': 'plantatravel.livejournal.com',
 'ilya--shpankov-livejournal-com.turbopages.org': 'ilya-shpankov.livejournal.com',
 'ecofit-info.turbopages.org': 'ecofit.info',
 'proizvoditely--dverey--spb-ru.turbopages.org': 'proizvoditely-dverey-spb.ru',
 'deti--v--semje-livejournal-com.turbopages.org': 'deti-v-semje.livejournal.com',
 'taani--a-livejournal-com.turbopages.org': 'taani-a.livejournal.com',
 'englishforcing-ru.turbopages.org': 'englishforcing.ru',
 'cont-ws.turbopages.org': 'cont.ws',
 'voin--karabakha-livejournal-com.turbopages.org': 'voin-karabakha.livejournal.com',
 'denkirish-livejournal-com.turbopages.org': 'denkirish.livejournal.com',
 'katzenkampf-livejournal-com.turbopages.org': 'katzenkampf.livejournal.com',
 'ali--help-ru.turbopages.org': 'ali-help.ru',
 'voskresensk--24-ru.turbopages.org': 'voskresensk-24.ru',
 'alshevskix-livejournal-com.turbopages.org': 'alshevskix.livejournal.com',
 'mila--kanari-livejournal-com.turbopages.org': 'mila-kanari.livejournal.com',
 'shushenskoe-bezformata-com.turbopages.org': 'shushenskoe.bezformata.com',
 'rb-versia-ru.turbopages.org': 'rb.versia.ru',
 'ilovemoscow-livejournal-com.turbopages.org': 'ilovemoscow.livejournal.com',
 'paaladin-livejournal-com.turbopages.org': 'paaladin.livejournal.com',
 'g--victor-livejournal-com.turbopages.org': 'g-victor.livejournal.com',
 'baby--il-livejournal-com.turbopages.org': 'baby-il.livejournal.com',
 'krasotizemli-ru.turbopages.org': 'krasotizemli.ru',
 'tito0107-livejournal-com.turbopages.org': 'tito0107.livejournal.com',
 'tvoysaratov-ru.turbopages.org': 'tvoysaratov.ru',
 'rb--str-ru.turbopages.org': 'rb-str.ru',
 'retro--ladies-livejournal-com.turbopages.org': 'retro-ladies.livejournal.com',
 'dubva1-livejournal-com.turbopages.org': 'dubva1.livejournal.com',
 'beef--and--smoked-livejournal-com.turbopages.org': 'beef-and-smoked.livejournal.com',
 'jumadilov-kz.turbopages.org': 'jumadilov.kz',
 'botter-livejournal-com.turbopages.org': 'botter.livejournal.com',
 'uprayser-livejournal-com.turbopages.org': 'uprayser.livejournal.com',
 'wowmoon-ru.turbopages.org': 'wowmoon.ru',
 'la--gatta--ciara-livejournal-com.turbopages.org': 'la-gatta-ciara.livejournal.com',
 'remodelo-ru.turbopages.org': 'remodelo.ru',
 'evgensemenihin-livejournal-com.turbopages.org': 'evgensemenihin.livejournal.com',
 '061-ua.turbopages.org': '061.ua',
 'povalixa-livejournal-com.turbopages.org': 'povalixa.livejournal.com',
 'ecmoru-livejournal-com.turbopages.org': 'ecmoru.livejournal.com',
 'kirill--potapov-livejournal-com.turbopages.org': 'kirill-potapov.livejournal.com',
 'scinquisitor-livejournal-com.turbopages.org': 'scinquisitor.livejournal.com',
 'komplemir-by.turbopages.org': 'komplemir.by',
 '1--pp-ru.turbopages.org': '1-pp.ru',
 'flower--plus-ru.turbopages.org': 'flower-plus.ru',
 'strikun-ru.turbopages.org': 'strikun.ru',
 'patriot--af-livejournal-com.turbopages.org': 'patriot-af.livejournal.com',
 'mboyarskov-livejournal-com.turbopages.org': 'mboyarskov.livejournal.com',
 'alexandragor-livejournal-com.turbopages.org': 'alexandragor.livejournal.com',
 'tolyas83-livejournal-com.turbopages.org': 'tolyas83.livejournal.com',
 'sampomiru-ru.turbopages.org': 'sampomiru.ru',
 'ukrainerussia-livejournal-com.turbopages.org': 'ukrainerussia.livejournal.com',
 'amsrus-ru.turbopages.org': 'amsrus.ru',
 'len--ol--kat-livejournal-com.turbopages.org': 'len-ol-kat.livejournal.com',
 'knevz-livejournal-com.turbopages.org': 'knevz.livejournal.com',
 'orel-bezformata-com.turbopages.org': 'orel.bezformata.com',
 'pauluskp-livejournal-com.turbopages.org': 'pauluskp.livejournal.com',
 'newislamru-livejournal-com.turbopages.org': 'newislamru.livejournal.com',
 'ahmadmuslim-livejournal-com.turbopages.org': 'ahmadmuslim.livejournal.com',
 'asharit7-livejournal-com.turbopages.org': 'asharit7.livejournal.com',
 'dfedbees-livejournal-com.turbopages.org': 'dfedbees.livejournal.com',
 'yuritkachev-livejournal-com.turbopages.org': 'yuritkachev.livejournal.com',
 'norilsk-bezformata-com.turbopages.org': 'norilsk.bezformata.com',
 'sanaei-livejournal-com.turbopages.org': 'sanaei.livejournal.com',
 'laruche-livejournal-com.turbopages.org': 'laruche.livejournal.com',
 'magbook-net.turbopages.org': 'magbook.net',
 'test--ekzamen-ru.turbopages.org': 'test-ekzamen.ru',
 'nizhniy--tagil-big--book--med-ru.turbopages.org': 'nizhniy-tagil.big-book-med.ru',
 'fitnesstop-ru.turbopages.org': 'fitnesstop.ru',
 'fisher--fish-ru.turbopages.org': 'fisher-fish.ru',
 'thailandi-info.turbopages.org': 'thailandi.info',
 'smokehookah-livejournal-com.turbopages.org': 'smokehookah.livejournal.com',
 'volokolamsk-bezformata-com.turbopages.org': 'volokolamsk.bezformata.com',
 'maintransport-ru.turbopages.org': 'maintransport.ru',
 'svart--ulfr-livejournal-com.turbopages.org': 'svart-ulfr.livejournal.com',
 'andy7290-livejournal-com.turbopages.org': 'andy7290.livejournal.com',
 'uchenitsa--iya-livejournal-com.turbopages.org': 'uchenitsa-iya.livejournal.com',
 'b0gus-livejournal-com.turbopages.org': 'b0gus.livejournal.com',
 'study--anatomy-livejournal-com.turbopages.org': 'study-anatomy.livejournal.com',
 'madou169-ru.turbopages.org': 'madou169.ru',
 'dom--sad--og-ru.turbopages.org': 'dom-sad-og.ru',
 'mturnn-livejournal-com.turbopages.org': 'mturnn.livejournal.com',
 'enia-livejournal-com.turbopages.org': 'enia.livejournal.com',
 'armrinok-ru.turbopages.org': 'armrinok.ru',
 'xn--24--idd3dkder.turbopages.org': 'xn-24-idd3dkder',
 'hgr-livejournal-com.turbopages.org': 'hgr.livejournal.com',
 'vovikte-livejournal-com.turbopages.org': 'vovikte.livejournal.com',
 'medgyna-livejournal-com.turbopages.org': 'medgyna.livejournal.com',
 'verbitsky-livejournal-com.turbopages.org': 'verbitsky.livejournal.com',
 'elektrostal-bezformata-com.turbopages.org': 'elektrostal.bezformata.com',
 'gatoazul-livejournal-com.turbopages.org': 'gatoazul.livejournal.com',
 'boris--gouts-livejournal-com.turbopages.org': 'boris-gouts.livejournal.com',
 'makemytravel-ru.turbopages.org': 'makemytravel.ru',
 'seductrice-ru.turbopages.org': 'seductrice.ru',
 'pelican--travel-livejournal-com.turbopages.org': 'pelican-travel.livejournal.com',
 'chi--lll-livejournal-com.turbopages.org': 'chi-lll.livejournal.com',
 'stas--senkin-livejournal-com.turbopages.org': 'stas-senkin.livejournal.com',
 'svoimi--rukamy-com.turbopages.org': 'svoimi-rukamy.com',
 'gym--people-ru.turbopages.org': 'gym-people.ru',
 'shibari--by-livejournal-com.turbopages.org': 'shibari-by.livejournal.com',
 'brejestovski-livejournal-com.turbopages.org': 'brejestovski.livejournal.com',
 'by--enigma-livejournal-com.turbopages.org': 'by-enigma.livejournal.com',
 'mahavam-livejournal-com.turbopages.org': 'mahavam.livejournal.com',
 'ihockey-bet.turbopages.org': 'ihockey.bet',
 'antisemitism-livejournal-com.turbopages.org': 'antisemitism.livejournal.com',
 'paradoxov-livejournal-com.turbopages.org': 'paradoxov.livejournal.com',
 'mamaasia-info.turbopages.org': 'mamaasia.info',
 'webgamer-ru.turbopages.org': 'webgamer.ru',
 'toolsmagic-ru.turbopages.org': 'toolsmagic.ru',
 'riverpilgrim-livejournal-com.turbopages.org': 'riverpilgrim.livejournal.com',
 'kwakin--misha-livejournal-com.turbopages.org': 'kwakin-misha.livejournal.com',
 'lezgigazet-ru.turbopages.org': 'lezgigazet.ru',
 'remont--i--dizajn-ru.turbopages.org': 'remont-i-dizajn.ru',
 's7--direct--line-livejournal-com.turbopages.org': 's7-direct-line.livejournal.com',
 'avro--live-livejournal-com.turbopages.org': 'avro-live.livejournal.com',
 'vipdis-ru.turbopages.org': 'vipdis.ru',
 'von--buddenbrock-livejournal-com.turbopages.org': 'von-buddenbrock.livejournal.com',
 'ognikavkaza-ru.turbopages.org': 'ognikavkaza.ru',
 'physiologynorma-ru.turbopages.org': 'physiologynorma.ru',
 'julifern-livejournal-com.turbopages.org': 'julifern.livejournal.com',
 'incopula-livejournal-com.turbopages.org': 'incopula.livejournal.com',
 'scuir-livejournal-com.turbopages.org': 'scuir.livejournal.com',
 'oktzd-ru.turbopages.org': 'oktzd.ru',
 'dima--ignatiev-livejournal-com.turbopages.org': 'dima-ignatiev.livejournal.com',
 'rajon--pride-livejournal-com.turbopages.org': 'rajon-pride.livejournal.com',
 'okrsredkchr-ru.turbopages.org': 'okrsredkchr.ru',
 'kozandrevna-livejournal-com.turbopages.org': 'kozandrevna.livejournal.com',
 'maglipogoda-ru.turbopages.org': 'maglipogoda.ru',
 'laufercnc-livejournal-com.turbopages.org': 'laufercnc.livejournal.com',
 'arh-mk-ru.turbopages.org': 'arh.mk.ru',
 'koronavirus-center.turbopages.org': 'koronavirus.center',
 'amotor--13-livejournal-com.turbopages.org': 'amotor-13.livejournal.com',
 'lavrushka-info.turbopages.org': 'lavrushka.info',
 'imperators87-livejournal-com.turbopages.org': 'imperators87.livejournal.com',
 'lechisustavv-ru.turbopages.org': 'lechisustavv.ru',
 'kostroma-bezformata-com.turbopages.org': 'kostroma.bezformata.com',
 'ex--hort-ru.turbopages.org': 'ex-hort.ru',
 'sa--rus-livejournal-com.turbopages.org': 'sa-rus.livejournal.com',
 'zharkov-livejournal-com.turbopages.org': 'zharkov.livejournal.com',
 'netrmed-livejournal-com.turbopages.org': 'netrmed.livejournal.com',
 'nicstroy-ru.turbopages.org': 'nicstroy.ru',
 'tipstrips-ru.turbopages.org': 'tipstrips.ru',
 'ajlis-livejournal-com.turbopages.org': 'ajlis.livejournal.com',
 'camry--v50-ru.turbopages.org': 'camry-v50.ru',
 'parentbook-ru.turbopages.org': 'parentbook.ru',
 '1cguide-ru.turbopages.org': '1cguide.ru',
 'zheniavasilievv-livejournal-com.turbopages.org': 'zheniavasilievv.livejournal.com',
 'h--i-su.turbopages.org': 'h-i.su',
 'radoslava-ru.turbopages.org': 'radoslava.ru',
 'kinoclub-livejournal-com.turbopages.org': 'kinoclub.livejournal.com',
 'stroitelstvo-expert.turbopages.org': 'stroitelstvo.expert',
 'antipa62-livejournal-com.turbopages.org': 'antipa62.livejournal.com',
 '5f--media-livejournal-com.turbopages.org': '5f-media.livejournal.com',
 'fermerbezhlopot-ru.turbopages.org': 'fermerbezhlopot.ru',
 'helgpo-livejournal-com.turbopages.org': 'helgpo.livejournal.com',
 'kreditnie--donory--novosibirsk-ru.turbopages.org': 'kreditnie-donory-novosibirsk.ru',
 'fea--sofia-livejournal-com.turbopages.org': 'fea-sofia.livejournal.com',
 'likasmileful-livejournal-com.turbopages.org': 'likasmileful.livejournal.com',
 'abuasad-livejournal-com.turbopages.org': 'abuasad.livejournal.com',
 'komandorva-livejournal-com.turbopages.org': 'komandorva.livejournal.com',
 'health-omskinform-ru.turbopages.org': 'health.omskinform.ru',
 'kalambur78-livejournal-com.turbopages.org': 'kalambur78.livejournal.com',
 'doctrinaire1802-livejournal-com.turbopages.org': 'doctrinaire1802.livejournal.com',
 'youknow-by.turbopages.org': 'youknow.by',
 'rnk-ru.turbopages.org': 'rnk.ru',
 'animal--zordok-livejournal-com.turbopages.org': 'animal-zordok.livejournal.com',
 'caverin2008-livejournal-com.turbopages.org': 'caverin2008.livejournal.com',
 'tov--mauser2--0-livejournal-com.turbopages.org': 'tov-mauser2-0.livejournal.com',
 'sviridenkov-livejournal-com.turbopages.org': 'sviridenkov.livejournal.com',
 'kem--live-ru.turbopages.org': 'kem-live.ru',
 'vyazanie--kis-ru.turbopages.org': 'vyazanie-kis.ru',
 'ifc-livejournal-com.turbopages.org': 'ifc.livejournal.com',
 'mi--band-fix--note-ru.turbopages.org': 'mi-band.fix-note.ru',
 'ckychnovosti-livejournal-com.turbopages.org': 'ckychnovosti.livejournal.com',
 'ahtubpravda-livejournal-com.turbopages.org': 'ahtubpravda.livejournal.com',
 'bestpeople2010-livejournal-com.turbopages.org': 'bestpeople2010.livejournal.com',
 'muchampion-ru.turbopages.org': 'muchampion.ru',
 'artemdragunov-livejournal-com.turbopages.org': 'artemdragunov.livejournal.com',
 'lgbt--vmeste-livejournal-com.turbopages.org': 'lgbt-vmeste.livejournal.com',
 'm--arch-livejournal-com.turbopages.org': 'm-arch.livejournal.com',
 'progoroduhta-ru.turbopages.org': 'progoroduhta.ru',
 'berg--press-livejournal-com.turbopages.org': 'berg-press.livejournal.com',
 'kursk-big--book--med-ru.turbopages.org': 'kursk.big-book-med.ru',
 'gela--guralia-livejournal-com.turbopages.org': 'gela-guralia.livejournal.com',
 'chto--chitat-livejournal-com.turbopages.org': 'chto-chitat.livejournal.com',
 'ru--verses-livejournal-com.turbopages.org': 'ru-verses.livejournal.com',
 'begemusja-livejournal-com.turbopages.org': 'begemusja.livejournal.com',
 'proshakov-livejournal-com.turbopages.org': 'proshakov.livejournal.com',
 'bulgakovdnevnik-livejournal-com.turbopages.org': 'bulgakovdnevnik.livejournal.com',
 'mini--puppy-ru.turbopages.org': 'mini-puppy.ru',
 'micro--solution-ru.turbopages.org': 'micro-solution.ru',
 'svitla--4ok-livejournal-com.turbopages.org': 'svitla-4ok.livejournal.com',
 'dimlix-com.turbopages.org': 'dimlix.com',
 'wisaev-livejournal-com.turbopages.org': 'wisaev.livejournal.com',
 'skif09-livejournal-com.turbopages.org': 'skif09.livejournal.com',
 'zonavannoi-ru.turbopages.org': 'zonavannoi.ru',
 'firs--away-livejournal-com.turbopages.org': 'firs-away.livejournal.com',
 'historian30h-livejournal-com.turbopages.org': 'historian30h.livejournal.com',
 'aromachem-livejournal-com.turbopages.org': 'aromachem.livejournal.com',
 'maliy-livejournal-com.turbopages.org': 'maliy.livejournal.com',
 'lileeva--e-livejournal-com.turbopages.org': 'lileeva-e.livejournal.com',
 'sky--hexe-livejournal-com.turbopages.org': 'sky-hexe.livejournal.com',
 '59i-ru.turbopages.org': '59i.ru',
 'rnbee-kiev-ua.turbopages.org': 'rnbee.kiev.ua',
 'y--blaz-livejournal-com.turbopages.org': 'y-blaz.livejournal.com',
 'alco--40-livejournal-com.turbopages.org': 'alco-40.livejournal.com',
 'sgdw-livejournal-com.turbopages.org': 'sgdw.livejournal.com',
 'germiones--muzh-livejournal-com.turbopages.org': 'germiones-muzh.livejournal.com',
 'perevestnik-livejournal-com.turbopages.org': 'perevestnik.livejournal.com',
 'world--of--sports-ru.turbopages.org': 'world-of-sports.ru',
 'e--polonskiy-livejournal-com.turbopages.org': 'e-polonskiy.livejournal.com',
 'stabbut-livejournal-com.turbopages.org': 'stabbut.livejournal.com',
 'ardm-livejournal-com.turbopages.org': 'ardm.livejournal.com',
 'luntik--n-livejournal-com.turbopages.org': 'luntik-n.livejournal.com',
 'prestigpol-ru.turbopages.org': 'prestigpol.ru',
 'lopuhsurepkin-livejournal-com.turbopages.org': 'lopuhsurepkin.livejournal.com',
 'berg--man-livejournal-com.turbopages.org': 'berg-man.livejournal.com',
 'sadda-ru.turbopages.org': 'sadda.ru',
 'skill--wood-ru.turbopages.org': 'skill-wood.ru',
 'shpy-livejournal-com.turbopages.org': 'shpy.livejournal.com',
 'denezhnye--ruchejki-ru.turbopages.org': 'denezhnye-ruchejki.ru',
 'megaz0id-livejournal-com.turbopages.org': 'megaz0id.livejournal.com',
 'batal-livejournal-com.turbopages.org': 'batal.livejournal.com',
 'save--kolbich-livejournal-com.turbopages.org': 'save-kolbich.livejournal.com',
 'ararat--moskva-ru.turbopages.org': 'ararat-moskva.ru',
 'ru--melenki-livejournal-com.turbopages.org': 'ru-melenki.livejournal.com',
 'history--russia-livejournal-com.turbopages.org': 'history-russia.livejournal.com',
 'likeni-ru.turbopages.org': 'likeni.ru',
 'yaroslavl-big--book--med-ru.turbopages.org': 'yaroslavl.big-book-med.ru',
 'bantomaniya-ru.turbopages.org': 'bantomaniya.ru',
 'jesus--allahov-livejournal-com.turbopages.org': 'jesus-allahov.livejournal.com',
 'vietnamix-ru.turbopages.org': 'vietnamix.ru',
 '1nsk-ru.turbopages.org': '1nsk.ru',
 'teremizbrusa-ru.turbopages.org': 'teremizbrusa.ru',
 'leolion--1-livejournal-com.turbopages.org': 'leolion-1.livejournal.com',
 'lovesun-by.turbopages.org': 'lovesun.by',
 'gadsclub-livejournal-com.turbopages.org': 'gadsclub.livejournal.com',
 'ru--fitfood-livejournal-com.turbopages.org': 'ru-fitfood.livejournal.com',
 'vamsterdame-com.turbopages.org': 'vamsterdame.com',
 'krymshina-ru.turbopages.org': 'krymshina.ru',
 'solisprime-livejournal-com.turbopages.org': 'solisprime.livejournal.com',
 'tvsher-livejournal-com.turbopages.org': 'tvsher.livejournal.com',
 'znichk--a-livejournal-com.turbopages.org': 'znichk-a.livejournal.com',
 'evia1-livejournal-com.turbopages.org': 'evia1.livejournal.com',
 'chadeyka-livejournal-com.turbopages.org': 'chadeyka.livejournal.com',
 'comfortoria-ru.turbopages.org': 'comfortoria.ru',
 'polynkov-livejournal-com.turbopages.org': 'polynkov.livejournal.com',
 'marina--haifa-livejournal-com.turbopages.org': 'marina-haifa.livejournal.com',
 'maximkoo-livejournal-com.turbopages.org': 'maximkoo.livejournal.com',
 'gorgaznn-ru.turbopages.org': 'gorgaznn.ru',
 'sopass-livejournal-com.turbopages.org': 'sopass.livejournal.com',
 'sfwss-livejournal-com.turbopages.org': 'sfwss.livejournal.com',
 'za--slanetz-livejournal-com.turbopages.org': 'za-slanetz.livejournal.com',
 'rotovirus-ru.turbopages.org': 'rotovirus.ru',
 'chasogor-livejournal-com.turbopages.org': 'chasogor.livejournal.com',
 'i--mar--a-livejournal-com.turbopages.org': 'i-mar-a.livejournal.com',
 'evri4ka-livejournal-com.turbopages.org': 'evri4ka.livejournal.com',
 'yu--butovo-livejournal-com.turbopages.org': 'yu-butovo.livejournal.com',
 'naemi-ru.turbopages.org': 'naemi.ru',
 'dev-infox-ru.turbopages.org': 'dev.infox.ru',
 'transurfer-livejournal-com.turbopages.org': 'transurfer.livejournal.com',
 'gribokstop-com.turbopages.org': 'gribokstop.com',
 'lychik--school-livejournal-com.turbopages.org': 'lychik-school.livejournal.com',
 'lll22021918-livejournal-com.turbopages.org': 'lll22021918.livejournal.com',
 'bolyatpochki-ru.turbopages.org': 'bolyatpochki.ru',
 'kuhniug-ru.turbopages.org': 'kuhniug.ru',
 'charlifox-ru.turbopages.org': 'charlifox.ru',
 'vnpru-livejournal-com.turbopages.org': 'vnpru.livejournal.com',
 'sovet--ok-ru.turbopages.org': 'sovet-ok.ru',
 'house--perm-ru.turbopages.org': 'house-perm.ru',
 'gasimova-livejournal-com.turbopages.org': 'gasimova.livejournal.com',
 'vodolei--13-livejournal-com.turbopages.org': 'vodolei-13.livejournal.com',
 'nika--po-livejournal-com.turbopages.org': 'nika-po.livejournal.com',
 'ellis--cu-livejournal-com.turbopages.org': 'ellis-cu.livejournal.com',
 'no--korrupcia-livejournal-com.turbopages.org': 'no-korrupcia.livejournal.com',
 'redpunch-ru.turbopages.org': 'redpunch.ru',
 'sva--vrn-ru.turbopages.org': 'sva-vrn.ru',
 'komigor-com.turbopages.org': 'komigor.com',
 'malcovsky-livejournal-com.turbopages.org': 'malcovsky.livejournal.com',
 'resiverplus-ru.turbopages.org': 'resiverplus.ru',
 'kakdobratsyado-ru.turbopages.org': 'kakdobratsyado.ru',
 'goodubai-livejournal-com.turbopages.org': 'goodubai.livejournal.com',
 'psyberia-livejournal-com.turbopages.org': 'psyberia.livejournal.com',
 'lamanhoy-livejournal-com.turbopages.org': 'lamanhoy.livejournal.com',
 'gotovim--doma--retsepty-ru.turbopages.org': 'gotovim-doma-retsepty.ru',
 'iks--online-net.turbopages.org': 'iks-online.net',
 'sntdpni-livejournal-com.turbopages.org': 'sntdpni.livejournal.com',
 'pmaki-ru.turbopages.org': 'pmaki.ru',
 'ibn--ibrahim-livejournal-com.turbopages.org': 'ibn-ibrahim.livejournal.com',
 'nadya--petrova-livejournal-com.turbopages.org': 'nadya-petrova.livejournal.com',
 'teatralka--tusa-livejournal-com.turbopages.org': 'teatralka-tusa.livejournal.com',
 'xn----itbmgldddjhg5b.turbopages.org': 'xn--itbmgldddjhg5b',
 'ushistory-ru.turbopages.org': 'ushistory.ru',
 'sk--alla-livejournal-com.turbopages.org': 'sk-alla.livejournal.com',
 'huskyhome-ru.turbopages.org': 'huskyhome.ru',
 'budgetnik-ru.turbopages.org': 'budgetnik.ru',
 'mary--hr5-livejournal-com.turbopages.org': 'mary-hr5.livejournal.com',
 'ot--lukavogo-ru.turbopages.org': 'ot-lukavogo.ru',
 'volsk-bezformata-com.turbopages.org': 'volsk.bezformata.com',
 'vodnoklassnikax-ru.turbopages.org': 'vodnoklassnikax.ru',
 'vitr--ru-livejournal-com.turbopages.org': 'vitr-ru.livejournal.com',
 'gorpark-livejournal-com.turbopages.org': 'gorpark.livejournal.com',
 'englishforbeginner-ru.turbopages.org': 'englishforbeginner.ru',
 'rkc--parus-ru.turbopages.org': 'rkc-parus.ru',
 'liabannikova-livejournal-com.turbopages.org': 'liabannikova.livejournal.com',
 'for--artaddicted-livejournal-com.turbopages.org': 'for-artaddicted.livejournal.com',
 'ant--bravo-livejournal-com.turbopages.org': 'ant-bravo.livejournal.com',
 'dushlik-livejournal-com.turbopages.org': 'dushlik.livejournal.com',
 'savin--andrey-livejournal-com.turbopages.org': 'savin-andrey.livejournal.com',
 'sangit-livejournal-com.turbopages.org': 'sangit.livejournal.com',
 'wkrasote-ru.turbopages.org': 'wkrasote.ru',
 'gulkevichi-livejournal-com.turbopages.org': 'gulkevichi.livejournal.com',
 'zaim-com.turbopages.org': 'zaim.com',
 'alreut1-livejournal-com.turbopages.org': 'alreut1.livejournal.com',
 'womanclub-online.turbopages.org': 'womanclub.online',
 'omsk-bezformata-com.turbopages.org': 'omsk.bezformata.com',
 'mnogo--skazok-ru.turbopages.org': 'mnogo-skazok.ru',
 'popolo4kam-ru.turbopages.org': 'popolo4kam.ru',
 'psychology--ua-livejournal-com.turbopages.org': 'psychology-ua.livejournal.com',
 'vakhnenko-livejournal-com.turbopages.org': 'vakhnenko.livejournal.com',
 'vovkodav-livejournal-com.turbopages.org': 'vovkodav.livejournal.com',
 'saarem-livejournal-com.turbopages.org': 'saarem.livejournal.com',
 'yoshka--live-ru.turbopages.org': 'yoshka-live.ru',
 'tumbalele-livejournal-com.turbopages.org': 'tumbalele.livejournal.com',
 'murmansk-bezformata-com.turbopages.org': 'murmansk.bezformata.com',
 'maryannfoto-livejournal-com.turbopages.org': 'maryannfoto.livejournal.com',
 'abissinsk-ru.turbopages.org': 'abissinsk.ru',
 'lorddreadnought-livejournal-com.turbopages.org': 'lorddreadnought.livejournal.com',
 'diagnostic--nt-ru.turbopages.org': 'diagnostic-nt.ru',
 'nina--zaryana-livejournal-com.turbopages.org': 'nina-zaryana.livejournal.com',
 'golubyeskazki-livejournal-com.turbopages.org': 'golubyeskazki.livejournal.com',
 'yerofea-livejournal-com.turbopages.org': 'yerofea.livejournal.com',
 'halidahamid-livejournal-com.turbopages.org': 'halidahamid.livejournal.com',
 'gar--remont-livejournal-com.turbopages.org': 'gar-remont.livejournal.com',
 'xn----etbfmbaaktlnhrela9c.turbopages.org': 'xn--etbfmbaaktlnhrela9c',
 'my--optika-livejournal-com.turbopages.org': 'my-optika.livejournal.com',
 'volklarson-livejournal-com.turbopages.org': 'volklarson.livejournal.com',
 'galina--nagornay-livejournal-com.turbopages.org': 'galina-nagornay.livejournal.com',
 'american--gangst-livejournal-com.turbopages.org': 'american-gangst.livejournal.com',
 'novozybkov-su.turbopages.org': 'novozybkov.su',
 'wayfarer--76-livejournal-com.turbopages.org': 'wayfarer-76.livejournal.com',
 'okean--v--butylke-livejournal-com.turbopages.org': 'okean-v-butylke.livejournal.com',
 'marina--kinosnob-livejournal-com.turbopages.org': 'marina-kinosnob.livejournal.com',
 'pcrentgen-ru.turbopages.org': 'pcrentgen.ru',
 'ru--kosi4ki-livejournal-com.turbopages.org': 'ru-kosi4ki.livejournal.com',
 'irondizain-livejournal-com.turbopages.org': 'irondizain.livejournal.com',
 'leonov--do-ru.turbopages.org': 'leonov-do.ru',
 'xn--63--5cdae8dh6bi8b9d.turbopages.org': 'xn-63-5cdae8dh6bi8b9d',
 'olesiny--igry-livejournal-com.turbopages.org': 'olesiny-igry.livejournal.com',
 'ruslesnik888-livejournal-com.turbopages.org': 'ruslesnik888.livejournal.com',
 'podolsk-bezformata-com.turbopages.org': 'podolsk.bezformata.com',
 'psorium-ru.turbopages.org': 'psorium.ru',
 'silver--slider-livejournal-com.turbopages.org': 'silver-slider.livejournal.com',
 'poisk-livejournal-com.turbopages.org': 'poisk.livejournal.com',
 'martavalieva-livejournal-com.turbopages.org': 'martavalieva.livejournal.com',
 'pharmprom-ru.turbopages.org': 'pharmprom.ru',
 'milalexx-livejournal-com.turbopages.org': 'milalexx.livejournal.com',
 'frantsouzov-livejournal-com.turbopages.org': 'frantsouzov.livejournal.com',
 'lora--w-livejournal-com.turbopages.org': 'lora-w.livejournal.com',
 'patriotka-livejournal-com.turbopages.org': 'patriotka.livejournal.com',
 'xn----7sburiviw0d.turbopages.org': 'xn--7sburiviw0d',
 'showtopmodel-ru.turbopages.org': 'showtopmodel.ru',
 'chuvashsky-livejournal-com.turbopages.org': 'chuvashsky.livejournal.com',
 'anticor--21-livejournal-com.turbopages.org': 'anticor-21.livejournal.com',
 'prosound-ixbt-com.turbopages.org': 'prosound.ixbt.com',
 'cateyeru-livejournal-com.turbopages.org': 'cateyeru.livejournal.com',
 'nikitinskii-livejournal-com.turbopages.org': 'nikitinskii.livejournal.com',
 'stasy--sunkissed-livejournal-com.turbopages.org': 'stasy-sunkissed.livejournal.com',
 'vykza-bezformata-com.turbopages.org': 'vykza.bezformata.com',
 'nail--trend-ru.turbopages.org': 'nail-trend.ru',
 'arza--zoevna-livejournal-com.turbopages.org': 'arza-zoevna.livejournal.com',
 'roman--myskhako-livejournal-com.turbopages.org': 'roman-myskhako.livejournal.com',
 'ivanovo-big--book--med-ru.turbopages.org': 'ivanovo.big-book-med.ru',
 'mysikmi-livejournal-com.turbopages.org': 'mysikmi.livejournal.com',
 'mk--lenobl-ru.turbopages.org': 'mk-lenobl.ru',
 'camcebekulinar-ru.turbopages.org': 'camcebekulinar.ru',
 'vadim88-livejournal-com.turbopages.org': 'vadim88.livejournal.com',
 'mkosinova-livejournal-com.turbopages.org': 'mkosinova.livejournal.com',
 'color2009-livejournal-com.turbopages.org': 'color2009.livejournal.com',
 'cinema-pravda-ru.turbopages.org': 'cinema.pravda.ru',
 'fifarus-ru.turbopages.org': 'fifarus.ru',
 'kirill--moiseev-livejournal-com.turbopages.org': 'kirill-moiseev.livejournal.com',
 'myfilology-ru.turbopages.org': 'myfilology.ru',
 'kinosiski-livejournal-com.turbopages.org': 'kinosiski.livejournal.com',
 'neofeliya-livejournal-com.turbopages.org': 'neofeliya.livejournal.com',
 'artritu-net.turbopages.org': 'artritu.net',
 'naturehacks-livejournal-com.turbopages.org': 'naturehacks.livejournal.com',
 'kerb-livejournal-com.turbopages.org': 'kerb.livejournal.com',
 'dilato-ru.turbopages.org': 'dilato.ru',
 '1damer-livejournal-com.turbopages.org': '1damer.livejournal.com',
 'lushagina-livejournal-com.turbopages.org': 'lushagina.livejournal.com',
 'blog--svp--8-livejournal-com.turbopages.org': 'blog-svp-8.livejournal.com',
 'mops--sobaka-ru.turbopages.org': 'mops-sobaka.ru',
 'alex--military-livejournal-com.turbopages.org': 'alex-military.livejournal.com',
 'impotencija-net.turbopages.org': 'impotencija.net',
 'ru--taxi-livejournal-com.turbopages.org': 'ru-taxi.livejournal.com',
 'history4u-livejournal-com.turbopages.org': 'history4u.livejournal.com',
 'helghelg2-livejournal-com.turbopages.org': 'helghelg2.livejournal.com',
 'grey--olli-livejournal-com.turbopages.org': 'grey-olli.livejournal.com',
 'elcomienzo-ru.turbopages.org': 'elcomienzo.ru',
 'ae--krylov-livejournal-com.turbopages.org': 'ae-krylov.livejournal.com',
 'goromed-livejournal-com.turbopages.org': 'goromed.livejournal.com',
 'observer--l-livejournal-com.turbopages.org': 'observer-l.livejournal.com',
 'sled--vzayt-livejournal-com.turbopages.org': 'sled-vzayt.livejournal.com',
 'schmusic-ru.turbopages.org': 'schmusic.ru',
 'marinaizminska-livejournal-com.turbopages.org': 'marinaizminska.livejournal.com',
 'tulyka-livejournal-com.turbopages.org': 'tulyka.livejournal.com',
 'vadro-livejournal-com.turbopages.org': 'vadro.livejournal.com',
 'liorasun55-livejournal-com.turbopages.org': 'liorasun55.livejournal.com',
 'wutheringkites-livejournal-com.turbopages.org': 'wutheringkites.livejournal.com',
 'viktoriiako-livejournal-com.turbopages.org': 'viktoriiako.livejournal.com',
 'alex--avr2-livejournal-com.turbopages.org': 'alex-avr2.livejournal.com',
 'domklubka-ru.turbopages.org': 'domklubka.ru',
 'electrongrad-ru.turbopages.org': 'electrongrad.ru',
 'mostovoy-livejournal-com.turbopages.org': 'mostovoy.livejournal.com',
 'numetalboy1990-livejournal-com.turbopages.org': 'numetalboy1990.livejournal.com',
 'vitaliverba-livejournal-com.turbopages.org': 'vitaliverba.livejournal.com',
 'orlinoe-org.turbopages.org': 'orlinoe.org',
 'poisk--pravdy-livejournal-com.turbopages.org': 'poisk-pravdy.livejournal.com',
 'spjasshka-livejournal-com.turbopages.org': 'spjasshka.livejournal.com',
 'agritura-livejournal-com.turbopages.org': 'agritura.livejournal.com',
 '66sean99-livejournal-com.turbopages.org': '66sean99.livejournal.com',
 'oldtmb-livejournal-com.turbopages.org': 'oldtmb.livejournal.com',
 'ivin--vv-livejournal-com.turbopages.org': 'ivin-vv.livejournal.com',
 'taxi24--sochi-ru.turbopages.org': 'taxi24-sochi.ru',
 'mnogochernil-livejournal-com.turbopages.org': 'mnogochernil.livejournal.com',
 'windowsprofi-ru.turbopages.org': 'windowsprofi.ru',
 'aktivnyj--otdykh-ru.turbopages.org': 'aktivnyj-otdykh.ru',
 'kirulya-livejournal-com.turbopages.org': 'kirulya.livejournal.com',
 'evse7-livejournal-com.turbopages.org': 'evse7.livejournal.com',
 'myflowersdream-ru.turbopages.org': 'myflowersdream.ru',
 'krausen-livejournal-com.turbopages.org': 'krausen.livejournal.com',
 'geliny-livejournal-com.turbopages.org': 'geliny.livejournal.com',
 'corfu--guide-ru.turbopages.org': 'corfu-guide.ru',
 '13marysia--k-livejournal-com.turbopages.org': '13marysia-k.livejournal.com',
 'ancient--ru-livejournal-com.turbopages.org': 'ancient-ru.livejournal.com',
 'xn--34--eddo5agxrgh4b1i.turbopages.org': 'xn-34-eddo5agxrgh4b1i',
 '4words-ru.turbopages.org': '4words.ru',
 'yana--anders-livejournal-com.turbopages.org': 'yana-anders.livejournal.com',
 'miss--juliettef-livejournal-com.turbopages.org': 'miss-juliettef.livejournal.com',
 'regionvoice-ru.turbopages.org': 'regionvoice.ru',
 'shlisselburg-bezformata-com.turbopages.org': 'shlisselburg.bezformata.com',
 'ingrian-livejournal-com.turbopages.org': 'ingrian.livejournal.com',
 'd--v--sokolov-livejournal-com.turbopages.org': 'd-v-sokolov.livejournal.com',
 '63r-livejournal-com.turbopages.org': '63r.livejournal.com',
 'startiktok-ru.turbopages.org': 'startiktok.ru',
 'flash--a-livejournal-com.turbopages.org': 'flash-a.livejournal.com',
 'solovyeva-livejournal-com.turbopages.org': 'solovyeva.livejournal.com',
 'beauty--portal-ru.turbopages.org': 'beauty-portal.ru',
 'supersmeh-ru.turbopages.org': 'supersmeh.ru',
 'iskusstvo--zvuka-livejournal-com.turbopages.org': 'iskusstvo-zvuka.livejournal.com',
 'immunologiya-info.turbopages.org': 'immunologiya.info',
 'lifepodium-ru.turbopages.org': 'lifepodium.ru',
 'lgram-ru.turbopages.org': 'lgram.ru',
 'kegel--pf-ru.turbopages.org': 'kegel-pf.ru',
 'don--hyena-livejournal-com.turbopages.org': 'don-hyena.livejournal.com',
 'poleznyesvojstva-org.turbopages.org': 'poleznyesvojstva.org',
 'etfbaza-ru.turbopages.org': 'etfbaza.ru',
 'ktaara-livejournal-com.turbopages.org': 'ktaara.livejournal.com',
 'ctakan--divanych-livejournal-com.turbopages.org': 'ctakan-divanych.livejournal.com',
 'servicefree-livejournal-com.turbopages.org': 'servicefree.livejournal.com',
 'mos--kreml-livejournal-com.turbopages.org': 'mos-kreml.livejournal.com',
 'kotkovalexey-livejournal-com.turbopages.org': 'kotkovalexey.livejournal.com',
 'polubotko1950-livejournal-com.turbopages.org': 'polubotko1950.livejournal.com',
 'medn-ru.turbopages.org': 'medn.ru',
 'nmedik-org.turbopages.org': 'nmedik.org',
 'kropotkin-bezformata-com.turbopages.org': 'kropotkin.bezformata.com',
 'magazeta-com.turbopages.org': 'magazeta.com',
 'wsn01-livejournal-com.turbopages.org': 'wsn01.livejournal.com',
 'baikalgo-ru.turbopages.org': 'baikalgo.ru',
 'valis--mebel-ru.turbopages.org': 'valis-mebel.ru',
 'trapeznikov--30-livejournal-com.turbopages.org': 'trapeznikov-30.livejournal.com',
 'kaliningradnews-ru.turbopages.org': 'kaliningradnews.ru',
 'arh-aif-ru.turbopages.org': 'arh.aif.ru',
 'eugen--buzin-livejournal-com.turbopages.org': 'eugen-buzin.livejournal.com',
 'ipnovus--146-livejournal-com.turbopages.org': 'ipnovus-146.livejournal.com',
 'tn--club-ru.turbopages.org': 'tn-club.ru',
 'littlehirosima-livejournal-com.turbopages.org': 'littlehirosima.livejournal.com',
 'hotkovo-net-ru.turbopages.org': 'hotkovo.net.ru',
 'lyarder-livejournal-com.turbopages.org': 'lyarder.livejournal.com',
 'stroy54-ru.turbopages.org': 'stroy54.ru',
 'vse--v--sochi-ru.turbopages.org': 'vse-v-sochi.ru',
 'ecovedic-livejournal-com.turbopages.org': 'ecovedic.livejournal.com',
 'ulanude-bezformata-com.turbopages.org': 'ulanude.bezformata.com',
 'kupi--krasku-ru.turbopages.org': 'kupi-krasku.ru',
 'genby-livejournal-com.turbopages.org': 'genby.livejournal.com',
 'nol21-livejournal-com.turbopages.org': 'nol21.livejournal.com',
 'gril--avp-livejournal-com.turbopages.org': 'gril-avp.livejournal.com',
 'zelenyislon-livejournal-com.turbopages.org': 'zelenyislon.livejournal.com',
 'yursh-livejournal-com.turbopages.org': 'yursh.livejournal.com',
 'sex--bez--zppp-livejournal-com.turbopages.org': 'sex-bez-zppp.livejournal.com',
 'kravcov--sergey-ru.turbopages.org': 'kravcov-sergey.ru',
 'maximus101-livejournal-com.turbopages.org': 'maximus101.livejournal.com',
 'mova-livejournal-com.turbopages.org': 'mova.livejournal.com',
 'myuspehlife-ru.turbopages.org': 'myuspehlife.ru',
 'superkosmetolog-ru.turbopages.org': 'superkosmetolog.ru',
 'silaserdca-ru.turbopages.org': 'silaserdca.ru',
 'voskresensk-bezformata-com.turbopages.org': 'voskresensk.bezformata.com',
 'helgabell-livejournal-com.turbopages.org': 'helgabell.livejournal.com',
 'plemah-livejournal-com.turbopages.org': 'plemah.livejournal.com',
 'gam--ga-ru.turbopages.org': 'gam-ga.ru',
 'bella--shmella-livejournal-com.turbopages.org': 'bella-shmella.livejournal.com',
 'acarin--19-livejournal-com.turbopages.org': 'acarin-19.livejournal.com',
 'pomidorchik-com.turbopages.org': 'pomidorchik.com',
 'zulfia-livejournal-com.turbopages.org': 'zulfia.livejournal.com',
 'krasiv--i--zdorov-ru.turbopages.org': 'krasiv-i-zdorov.ru',
 'gugegot-livejournal-com.turbopages.org': 'gugegot.livejournal.com',
 'mmet-livejournal-com.turbopages.org': 'mmet.livejournal.com',
 'rublev--museum-livejournal-com.turbopages.org': 'rublev-museum.livejournal.com',
 'gessor-livejournal-com.turbopages.org': 'gessor.livejournal.com',
 'xn--54--5cdagnqb4apoek2a6bu4c.turbopages.org': 'xn-54-5cdagnqb4apoek2a6bu4c',
 'znanie--svet-ru.turbopages.org': 'znanie-svet.ru',
 'kolhozsity-livejournal-com.turbopages.org': 'kolhozsity.livejournal.com',
 'shkesper-livejournal-com.turbopages.org': 'shkesper.livejournal.com',
 'white--nephilim-livejournal-com.turbopages.org': 'white-nephilim.livejournal.com',
 'naffanja-livejournal-com.turbopages.org': 'naffanja.livejournal.com',
 'andy--sinclair-livejournal-com.turbopages.org': 'andy-sinclair.livejournal.com',
 'abrezd-livejournal-com.turbopages.org': 'abrezd.livejournal.com',
 'zdaccess-livejournal-com.turbopages.org': 'zdaccess.livejournal.com',
 'monkaster-livejournal-com.turbopages.org': 'monkaster.livejournal.com',
 'rodina-livejournal-com.turbopages.org': 'rodina.livejournal.com',
 'rusdrus-livejournal-com.turbopages.org': 'rusdrus.livejournal.com',
 'serj--32-livejournal-com.turbopages.org': 'serj-32.livejournal.com',
 'rigort-livejournal-com.turbopages.org': 'rigort.livejournal.com',
 'medvedevphoto-livejournal-com.turbopages.org': 'medvedevphoto.livejournal.com',
 'nukemall-livejournal-com.turbopages.org': 'nukemall.livejournal.com',
 'aran7-livejournal-com.turbopages.org': 'aran7.livejournal.com',
 'mil--avia-livejournal-com.turbopages.org': 'mil-avia.livejournal.com',
 'eugene--zando-livejournal-com.turbopages.org': 'eugene-zando.livejournal.com',
 'woozrael-livejournal-com.turbopages.org': 'woozrael.livejournal.com',
 'news-fidller-com.turbopages.org': 'news.fidller.com',
 'nesovet-livejournal-com.turbopages.org': 'nesovet.livejournal.com',
 'lazarudin-livejournal-com.turbopages.org': 'lazarudin.livejournal.com',
 'urpflanze-livejournal-com.turbopages.org': 'urpflanze.livejournal.com',
 'ru--indesign-livejournal-com.turbopages.org': 'ru-indesign.livejournal.com',
 'stroysmi-ru.turbopages.org': 'stroysmi.ru',
 'ledi--oks-livejournal-com.turbopages.org': 'ledi-oks.livejournal.com',
 'sdelaysamodelku-ru.turbopages.org': 'sdelaysamodelku.ru',
 'xn----7sbpwlbvbpxp.turbopages.org': 'xn--7sbpwlbvbpxp',
 'vv--34-ru.turbopages.org': 'vv-34.ru',
 'andronic-livejournal-com.turbopages.org': 'andronic.livejournal.com',
 'korrektor--ru-livejournal-com.turbopages.org': 'korrektor-ru.livejournal.com',
 'salon--doma-livejournal-com.turbopages.org': 'salon-doma.livejournal.com',
 'balkonidea-ru.turbopages.org': 'balkonidea.ru',
 'merkazit-livejournal-com.turbopages.org': 'merkazit.livejournal.com',
 'litlib-net.turbopages.org': 'litlib.net',
 'vasya--vasechkin-livejournal-com.turbopages.org': 'vasya-vasechkin.livejournal.com',
 'alexeykass-livejournal-com.turbopages.org': 'alexeykass.livejournal.com',
 'syktyvkar-big--book--med-ru.turbopages.org': 'syktyvkar.big-book-med.ru',
 'roman983-livejournal-com.turbopages.org': 'roman983.livejournal.com',
 'gorlanovig-livejournal-com.turbopages.org': 'gorlanovig.livejournal.com',
 'bskamalov-livejournal-com.turbopages.org': 'bskamalov.livejournal.com',
 'a--krotov-livejournal-com.turbopages.org': 'a-krotov.livejournal.com',
 'ana--lee-livejournal-com.turbopages.org': 'ana-lee.livejournal.com',
 'tigriska-livejournal-com.turbopages.org': 'tigriska.livejournal.com',
 'julia--bannykh-livejournal-com.turbopages.org': 'julia-bannykh.livejournal.com',
 'soc--armen-livejournal-com.turbopages.org': 'soc-armen.livejournal.com',
 'bog--oslovskij-livejournal-com.turbopages.org': 'bog-oslovskij.livejournal.com',
 'les--tish-ru.turbopages.org': 'les-tish.ru',
 'anithing-livejournal-com.turbopages.org': 'anithing.livejournal.com',
 'galitsin--news-livejournal-com.turbopages.org': 'galitsin-news.livejournal.com',
 'nhl--news-ru.turbopages.org': 'nhl-news.ru',
 'chuma3-livejournal-com.turbopages.org': 'chuma3.livejournal.com',
 'zohe-ru.turbopages.org': 'zohe.ru',
 '9psy-ru.turbopages.org': '9psy.ru',
 'kuhnyaterapiya-livejournal-com.turbopages.org': 'kuhnyaterapiya.livejournal.com',
 'radioamateur-livejournal-com.turbopages.org': 'radioamateur.livejournal.com',
 'fm--archive-livejournal-com.turbopages.org': 'fm-archive.livejournal.com',
 'mishajp-livejournal-com.turbopages.org': 'mishajp.livejournal.com',
 'xn----9sb6aifi2a.turbopages.org': 'xn--9sb6aifi2a',
 'serdce-biz.turbopages.org': 'serdce.biz',
 'ekovsem-ru.turbopages.org': 'ekovsem.ru',
 'vladikoms-livejournal-com.turbopages.org': 'vladikoms.livejournal.com',
 'marss2-livejournal-com.turbopages.org': 'marss2.livejournal.com',
 'kurgan-big--book--med-ru.turbopages.org': 'kurgan.big-book-med.ru',
 'imbg-livejournal-com.turbopages.org': 'imbg.livejournal.com',
 'rodom--iz--tiflis-livejournal-com.turbopages.org': 'rodom-iz-tiflis.livejournal.com',
 'hataiiia-livejournal-com.turbopages.org': 'hataiiia.livejournal.com',
 'kjeyly-livejournal-com.turbopages.org': 'kjeyly.livejournal.com',
 'triplinks-ru.turbopages.org': 'triplinks.ru',
 'imhotype-livejournal-com.turbopages.org': 'imhotype.livejournal.com',
 'ap6y3uk87-livejournal-com.turbopages.org': 'ap6y3uk87.livejournal.com',
 'a--zatvornik-livejournal-com.turbopages.org': 'a-zatvornik.livejournal.com',
 'market--mobi-ru.turbopages.org': 'market-mobi.ru',
 'vegnews-ru.turbopages.org': 'vegnews.ru',
 'fikus-guru.turbopages.org': 'fikus.guru',
 'avto1000-com.turbopages.org': 'avto1000.com',
 'alex--sobolev--82-livejournal-com.turbopages.org': 'alex-sobolev-82.livejournal.com',
 'gusev--a--v-livejournal-com.turbopages.org': 'gusev-a-v.livejournal.com',
 'art--nuinu-livejournal-com.turbopages.org': 'art-nuinu.livejournal.com',
 'himki-bezformata-com.turbopages.org': 'himki.bezformata.com',
 'red--gremlin-livejournal-com.turbopages.org': 'red-gremlin.livejournal.com',
 'jerboa--wee-livejournal-com.turbopages.org': 'jerboa-wee.livejournal.com',
 'oboinovye-ru.turbopages.org': 'oboinovye.ru',
 'androloginfo-ru.turbopages.org': 'androloginfo.ru',
 'vesninalj-livejournal-com.turbopages.org': 'vesninalj.livejournal.com',
 'gidanaliz-ru.turbopages.org': 'gidanaliz.ru',
 'cosmicsucculents-ru.turbopages.org': 'cosmicsucculents.ru',
 'mtv59-livejournal-com.turbopages.org': 'mtv59.livejournal.com',
 'nail--ru-livejournal-com.turbopages.org': 'nail-ru.livejournal.com',
 'nailcouture-livejournal-com.turbopages.org': 'nailcouture.livejournal.com',
 'sergeyk-livejournal-com.turbopages.org': 'sergeyk.livejournal.com',
 'journeyismyhome-ru.turbopages.org': 'journeyismyhome.ru',
 'grazdano4ka-livejournal-com.turbopages.org': 'grazdano4ka.livejournal.com',
 'interesniy--spb-livejournal-com.turbopages.org': 'interesniy-spb.livejournal.com',
 'michelleka-livejournal-com.turbopages.org': 'michelleka.livejournal.com',
 'diaryrh-ru.turbopages.org': 'diaryrh.ru',
 'vsyoprosad-ru.turbopages.org': 'vsyoprosad.ru',
 'birosss-livejournal-com.turbopages.org': 'birosss.livejournal.com',
 'drink--drink-ru.turbopages.org': 'drink-drink.ru',
 'okoshkah-com.turbopages.org': 'okoshkah.com',
 'rgo--penza-livejournal-com.turbopages.org': 'rgo-penza.livejournal.com',
 'penza--tour-livejournal-com.turbopages.org': 'penza-tour.livejournal.com',
 'zavertkinas2-livejournal-com.turbopages.org': 'zavertkinas2.livejournal.com',
 'larichev-livejournal-com.turbopages.org': 'larichev.livejournal.com',
 'forex--grail-ru.turbopages.org': 'forex-grail.ru',
 'pravslov-ru.turbopages.org': 'pravslov.ru',
 'abdul--vakhed-livejournal-com.turbopages.org': 'abdul-vakhed.livejournal.com',
 'mentorna--blog-livejournal-com.turbopages.org': 'mentorna-blog.livejournal.com',
 'peterburzec-livejournal-com.turbopages.org': 'peterburzec.livejournal.com',
 'kkk123-ru.turbopages.org': 'kkk123.ru',
 'katyajtj-livejournal-com.turbopages.org': 'katyajtj.livejournal.com',
 'glazaexpert-ru.turbopages.org': 'glazaexpert.ru',
 'antisektaperm-livejournal-com.turbopages.org': 'antisektaperm.livejournal.com',
 'honeylife24-ru.turbopages.org': 'honeylife24.ru',
 'kreditnie--donori--moskvy-ru.turbopages.org': 'kreditnie-donori-moskvy.ru',
 'dron285-livejournal-com.turbopages.org': 'dron285.livejournal.com',
 'dm--kalashnikov-livejournal-com.turbopages.org': 'dm-kalashnikov.livejournal.com',
 'vaga--land-livejournal-com.turbopages.org': 'vaga-land.livejournal.com',
 'townsman-livejournal-com.turbopages.org': 'townsman.livejournal.com',
 'ozareniye-livejournal-com.turbopages.org': 'ozareniye.livejournal.com',
 'xn----ctblibg4cunz.turbopages.org': 'xn--ctblibg4cunz',
 'ludmilapsyholog-livejournal-com.turbopages.org': 'ludmilapsyholog.livejournal.com',
 'ogon--777-livejournal-com.turbopages.org': 'ogon-777.livejournal.com',
 'pepelnaya-livejournal-com.turbopages.org': 'pepelnaya.livejournal.com',
 'kavkaz-news--kmv-ru.turbopages.org': 'kavkaz.news-kmv.ru',
 'vkamen-ru.turbopages.org': 'vkamen.ru',
 'vilenakopov-livejournal-com.turbopages.org': 'vilenakopov.livejournal.com',
 'xn---su-iddzy9a.turbopages.org': 'xn-.su.iddzy9a',
 'stroyremkom-ru.turbopages.org': 'stroyremkom.ru',
 'hayduk-livejournal-com.turbopages.org': 'hayduk.livejournal.com',
 'dimabalakirev-livejournal-com.turbopages.org': 'dimabalakirev.livejournal.com',
 'dennis--rodman-livejournal-com.turbopages.org': 'dennis-rodman.livejournal.com',
 'axiona-ru.turbopages.org': 'axiona.ru',
 'beresta-xyz.turbopages.org': 'beresta.xyz',
 'kua1102-livejournal-com.turbopages.org': 'kua1102.livejournal.com',
 'stringbasso-livejournal-com.turbopages.org': 'stringbasso.livejournal.com',
 'margoritka-ru.turbopages.org': 'margoritka.ru',
 'sergeysidorof-livejournal-com.turbopages.org': 'sergeysidorof.livejournal.com',
 'taii--liira-livejournal-com.turbopages.org': 'taii-liira.livejournal.com',
 'wolsk-ru.turbopages.org': 'wolsk.ru',
 'vladimir--chub-livejournal-com.turbopages.org': 'vladimir-chub.livejournal.com',
 'mens--physic-ru.turbopages.org': 'mens-physic.ru',
 'yakutia-aif-ru.turbopages.org': 'yakutia.aif.ru',
 'aboutwine-online.turbopages.org': 'aboutwine.online',
 'tridevici-com.turbopages.org': 'tridevici.com',
 'voda--78--ru-livejournal-com.turbopages.org': 'voda-78-ru.livejournal.com',
 'tar--s-livejournal-com.turbopages.org': 'tar-s.livejournal.com',
 'gorceoptom-livejournal-com.turbopages.org': 'gorceoptom.livejournal.com',
 'igumen--nektariy-livejournal-com.turbopages.org': 'igumen-nektariy.livejournal.com',
 'archygoncharov-livejournal-com.turbopages.org': 'archygoncharov.livejournal.com',
 'rebrik-livejournal-com.turbopages.org': 'rebrik.livejournal.com',
 'all--for--woman-com.turbopages.org': 'all-for-woman.com',
 'simeonn-livejournal-com.turbopages.org': 'simeonn.livejournal.com',
 'pudgik-livejournal-com.turbopages.org': 'pudgik.livejournal.com',
 'iphs-livejournal-com.turbopages.org': 'iphs.livejournal.com',
 'rudolf--khb-livejournal-com.turbopages.org': 'rudolf-khb.livejournal.com',
 'flylady--ru-livejournal-com.turbopages.org': 'flylady-ru.livejournal.com',
 'aksanova-livejournal-com.turbopages.org': 'aksanova.livejournal.com',
 'a--murzilka-livejournal-com.turbopages.org': 'a-murzilka.livejournal.com',
 'meaerror-livejournal-com.turbopages.org': 'meaerror.livejournal.com',
 'canningclub-livejournal-com.turbopages.org': 'canningclub.livejournal.com',
 'ktotamm-livejournal-com.turbopages.org': 'ktotamm.livejournal.com',
 'stepaniya-livejournal-com.turbopages.org': 'stepaniya.livejournal.com',
 'lugaland-com.turbopages.org': 'lugaland.com',
 'sir--prikol-livejournal-com.turbopages.org': 'sir-prikol.livejournal.com',
 'studsouz-mgimo-ru.turbopages.org': 'studsouz.mgimo.ru',
 'boxing--cat-livejournal-com.turbopages.org': 'boxing-cat.livejournal.com',
 'krym-aif-ru.turbopages.org': 'krym.aif.ru',
 'alyona--yarripko-livejournal-com.turbopages.org': 'alyona-yarripko.livejournal.com',
 'shigona-livejournal-com.turbopages.org': 'shigona.livejournal.com',
 'gotovim--edim-ru.turbopages.org': 'gotovim-edim.ru',
 'dobro--narodnoe-livejournal-com.turbopages.org': 'dobro-narodnoe.livejournal.com',
 'inoyan-livejournal-com.turbopages.org': 'inoyan.livejournal.com',
 'k0m2375-livejournal-com.turbopages.org': 'k0m2375.livejournal.com',
 'filibuster60-livejournal-com.turbopages.org': 'filibuster60.livejournal.com',
 'mr--volkov-livejournal-com.turbopages.org': 'mr-volkov.livejournal.com',
 'avtobeginner-ru.turbopages.org': 'avtobeginner.ru',
 'andrey--eroshin-livejournal-com.turbopages.org': 'andrey-eroshin.livejournal.com',
 'voiks-livejournal-com.turbopages.org': 'voiks.livejournal.com',
 'basilisk-livejournal-com.turbopages.org': 'basilisk.livejournal.com',
 'argonov-livejournal-com.turbopages.org': 'argonov.livejournal.com',
 'svadbalist-ru.turbopages.org': 'svadbalist.ru',
 'tisha--the--cat-livejournal-com.turbopages.org': 'tisha-the-cat.livejournal.com',
 'old--penza-livejournal-com.turbopages.org': 'old-penza.livejournal.com',
 'dronova--nona-livejournal-com.turbopages.org': 'dronova-nona.livejournal.com',
 'iammilitary--by-livejournal-com.turbopages.org': 'iammilitary-by.livejournal.com',
 'av--razumovsky-livejournal-com.turbopages.org': 'av-razumovsky.livejournal.com',
 'mobilelords-ru.turbopages.org': 'mobilelords.ru',
 'pskov--land-livejournal-com.turbopages.org': 'pskov-land.livejournal.com',
 'tavanen-ru.turbopages.org': 'tavanen.ru',
 'rebenok-online.turbopages.org': 'rebenok.online',
 'sergeyk2006-livejournal-com.turbopages.org': 'sergeyk2006.livejournal.com',
 'iarosvetsvetlov-livejournal-com.turbopages.org': 'iarosvetsvetlov.livejournal.com',
 'dfktyf-livejournal-com.turbopages.org': 'dfktyf.livejournal.com',
 'wind10-ru.turbopages.org': 'wind10.ru',
 'myparallel-ru.turbopages.org': 'myparallel.ru',
 'drugoj--m-livejournal-com.turbopages.org': 'drugoj-m.livejournal.com',
 'photozone--t-livejournal-com.turbopages.org': 'photozone-t.livejournal.com',
 'era--group-livejournal-com.turbopages.org': 'era-group.livejournal.com',
 'adis--71-livejournal-com.turbopages.org': 'adis-71.livejournal.com',
 'pvn123-livejournal-com.turbopages.org': 'pvn123.livejournal.com',
 'lugovskaya--l-livejournal-com.turbopages.org': 'lugovskaya-l.livejournal.com',
 'aquilaaquilonis-livejournal-com.turbopages.org': 'aquilaaquilonis.livejournal.com',
 'oil--beauty-livejournal-com.turbopages.org': 'oil-beauty.livejournal.com',
 'more--travel-livejournal-com.turbopages.org': 'more-travel.livejournal.com',
 'natural--colours-livejournal-com.turbopages.org': 'natural-colours.livejournal.com',
 'ivanteevka-bezformata-com.turbopages.org': 'ivanteevka.bezformata.com',
 'ru--compromat-livejournal-com.turbopages.org': 'ru-compromat.livejournal.com',
 'mobileoc-ru.turbopages.org': 'mobileoc.ru',
 'potrebnadzor-online.turbopages.org': 'potrebnadzor.online',
 'blackwot-ru.turbopages.org': 'blackwot.ru',
 'shaherezada-livejournal-com.turbopages.org': 'shaherezada.livejournal.com',
 'adlanter-livejournal-com.turbopages.org': 'adlanter.livejournal.com',
 'ru--socionics-livejournal-com.turbopages.org': 'ru-socionics.livejournal.com',
 'sanya--de--costa-livejournal-com.turbopages.org': 'sanya-de-costa.livejournal.com',
 'high--law-livejournal-com.turbopages.org': 'high-law.livejournal.com',
 'accross-livejournal-com.turbopages.org': 'accross.livejournal.com',
 'frederik--2520-livejournal-com.turbopages.org': 'frederik-2520.livejournal.com',
 'prostogv-ru.turbopages.org': 'prostogv.ru',
 'catarakta-ru.turbopages.org': 'catarakta.ru',
    'yasoldat-ru.turbopages.org': 'yasoldat.ru',
 'sosedgeorg-livejournal-com.turbopages.org': 'sosedgeorg.livejournal.com',
 'ru--uk-livejournal-com.turbopages.org': 'ru-uk.livejournal.com',
 'anti--terror--rf-livejournal-com.turbopages.org': 'anti-terror-rf.livejournal.com',
 'calculatorfs-ru.turbopages.org': 'calculatorfs.ru',
 'eruditochka-ru.turbopages.org': 'eruditochka.ru',
 'luckclose-ru.turbopages.org': 'luckclose.ru',
 'anna--y-livejournal-com.turbopages.org': 'anna-y.livejournal.com',
 'alf5-livejournal-com.turbopages.org': 'alf5.livejournal.com',
 'max--khlestunov-livejournal-com.turbopages.org': 'max-khlestunov.livejournal.com',
 'man--woman-livejournal-com.turbopages.org': 'man-woman.livejournal.com',
 'ivhw-livejournal-com.turbopages.org': 'ivhw.livejournal.com',
 'bratya--zabaday-livejournal-com.turbopages.org': 'bratya-zabaday.livejournal.com',
 'gukkur-ru.turbopages.org': 'gukkur.ru',
 'stogova-livejournal-com.turbopages.org': 'stogova.livejournal.com',
 'russ--muslim-livejournal-com.turbopages.org': 'russ-muslim.livejournal.com',
 'sj--aquaria-livejournal-com.turbopages.org': 'sj-aquaria.livejournal.com',
 'aritkin-livejournal-com.turbopages.org': 'aritkin.livejournal.com',
 'svetikart-ru.turbopages.org': 'svetikart.ru',
 'poroda--french--bulldogs-ru.turbopages.org': 'poroda-french-bulldogs.ru',
 'progressio-me.turbopages.org': 'progressio.me',
 'kulturolog--ia-livejournal-com.turbopages.org': 'kulturolog-ia.livejournal.com',
 'en365-ru.turbopages.org': 'en365.ru',
 'winnernickname-ru.turbopages.org': 'winnernickname.ru',
 'bluda--doma-ru.turbopages.org': 'bluda-doma.ru',
 'grzegorz-livejournal-com.turbopages.org': 'grzegorz.livejournal.com',
 'aleksei--44-livejournal-com.turbopages.org': 'aleksei-44.livejournal.com',
 'dobrosvet108-livejournal-com.turbopages.org': 'dobrosvet108.livejournal.com',
 'vkosmetichke-net.turbopages.org': 'vkosmetichke.net',
 'ganfayter-livejournal-com.turbopages.org': 'ganfayter.livejournal.com',
 'krasivo-biz.turbopages.org': 'krasivo.biz',
 'newsparky-livejournal-com.turbopages.org': 'newsparky.livejournal.com',
 'med--history-livejournal-com.turbopages.org': 'med-history.livejournal.com',
 'zverjata-ru.turbopages.org': 'zverjata.ru',
 'fee--alka-livejournal-com.turbopages.org': 'fee-alka.livejournal.com',
 'xaliavschik-livejournal-com.turbopages.org': 'xaliavschik.livejournal.com',
 'varjag--2007-livejournal-com.turbopages.org': 'varjag-2007.livejournal.com',
 'jelchnet-ru.turbopages.org': 'jelchnet.ru',
 'leninskkuzneckiy-bezformata-com.turbopages.org': 'leninskkuzneckiy.bezformata.com',
 'teamspro-ru.turbopages.org': 'teamspro.ru',
 'drevologia-ru.turbopages.org': 'drevologia.ru',
 'home--bread-livejournal-com.turbopages.org': 'home-bread.livejournal.com',
 'vladivostok-livejournal-com.turbopages.org': 'vladivostok.livejournal.com',
 'untt-livejournal-com.turbopages.org': 'untt.livejournal.com',
 'starnatali-livejournal-com.turbopages.org': 'starnatali.livejournal.com',
 'fly2trip-net.turbopages.org': 'fly2trip.net',
 'kinocalendar-livejournal-com.turbopages.org': 'kinocalendar.livejournal.com',
 'banana-by.turbopages.org': 'banana.by',
 'obetomnegovoryat-ru.turbopages.org': 'obetomnegovoryat.ru',
 'vecherusia-livejournal-com.turbopages.org': 'vecherusia.livejournal.com',
 'delis--avto-ru.turbopages.org': 'delis-avto.ru',
 'vartumashvili-livejournal-com.turbopages.org': 'vartumashvili.livejournal.com',
 'dikarem--ge-livejournal-com.turbopages.org': 'dikarem-ge.livejournal.com',
 'aleximedashvili-livejournal-com.turbopages.org': 'aleximedashvili.livejournal.com',
 'nalog--expert-com.turbopages.org': 'nalog-expert.com',
 'ievon-livejournal-com.turbopages.org': 'ievon.livejournal.com',
 'ufk34-ru.turbopages.org': 'ufk34.ru',
 'tkanexpert-ru.turbopages.org': 'tkanexpert.ru',
 'pamsik-livejournal-com.turbopages.org': 'pamsik.livejournal.com',
 'ulitka--kg-livejournal-com.turbopages.org': 'ulitka-kg.livejournal.com',
 'telegram--katalog-ru.turbopages.org': 'telegram-katalog.ru',
 'beremennostnedeli-ru.turbopages.org': 'beremennostnedeli.ru',
 'nemo2001-livejournal-com.turbopages.org': 'nemo2001.livejournal.com',
 'komlark-ru.turbopages.org': 'komlark.ru',
 'sis--2007-livejournal-com.turbopages.org': 'sis-2007.livejournal.com',
 'miriy-livejournal-com.turbopages.org': 'miriy.livejournal.com',
 'moseco-livejournal-com.turbopages.org': 'moseco.livejournal.com',
 'tambov-kp-ru.turbopages.org': 'tambov.kp.ru',
 'uslugi--mfc-ru.turbopages.org': 'uslugi-mfc.ru',
 'popugy-ru.turbopages.org': 'popugy.ru',
 'simbiblis-livejournal-com.turbopages.org': 'simbiblis.livejournal.com',
 'anar--gasimov-livejournal-com.turbopages.org': 'anar-gasimov.livejournal.com',
 'haydamak-livejournal-com.turbopages.org': 'haydamak.livejournal.com',
 'rus--48-livejournal-com.turbopages.org': 'rus-48.livejournal.com',
 'serge--novikov-livejournal-com.turbopages.org': 'serge-novikov.livejournal.com',
 'literatura--vda-livejournal-com.turbopages.org': 'literatura-vda.livejournal.com',
 'angliyskiyazik-ru.turbopages.org': 'angliyskiyazik.ru',
 'knigipoistorii-livejournal-com.turbopages.org': 'knigipoistorii.livejournal.com',
 'gignomai-livejournal-com.turbopages.org': 'gignomai.livejournal.com',
 'incognito--from-livejournal-com.turbopages.org': 'incognito-from.livejournal.com',
 'vdryndine1939-livejournal-com.turbopages.org': 'vdryndine1939.livejournal.com',
 'm--power-livejournal-com.turbopages.org': 'm-power.livejournal.com',
 'armavir-bezformata-com.turbopages.org': 'armavir.bezformata.com',
 'sevkuban--station-ru.turbopages.org': 'sevkuban-station.ru',
 'sovetdomu-ru.turbopages.org': 'sovetdomu.ru',
 'helpyourheart-ru.turbopages.org': 'helpyourheart.ru',
 'ljpromo-livejournal-com.turbopages.org': 'ljpromo.livejournal.com',
 'tambov-bezformata-com.turbopages.org': 'tambov.bezformata.com',
 'kibernetika-livejournal-com.turbopages.org': 'kibernetika.livejournal.com',
 'pristalnaya-livejournal-com.turbopages.org': 'pristalnaya.livejournal.com',
 'tourirana-ru.turbopages.org': 'tourirana.ru',
 'timatkov-livejournal-com.turbopages.org': 'timatkov.livejournal.com',
 'tot--samy--yustas-livejournal-com.turbopages.org': 'tot-samy-yustas.livejournal.com',
 'aslan-livejournal-com.turbopages.org': 'aslan.livejournal.com',
 'liya--fa-livejournal-com.turbopages.org': 'liya-fa.livejournal.com',
 'alikhanov-livejournal-com.turbopages.org': 'alikhanov.livejournal.com',
 'crazy555-livejournal-com.turbopages.org': 'crazy555.livejournal.com',
 'manexpert-ru.turbopages.org': 'manexpert.ru',
 'moycapital-com.turbopages.org': 'moycapital.com',
 'vowen346-livejournal-com.turbopages.org': 'vowen346.livejournal.com',
 'dkarpunin-livejournal-com.turbopages.org': 'dkarpunin.livejournal.com',
 'notkinastya-ru.turbopages.org': 'notkinastya.ru',
 'elenergi-ru.turbopages.org': 'elenergi.ru',
 'crazykuzya-livejournal-com.turbopages.org': 'crazykuzya.livejournal.com',
 'pumbik-ru.turbopages.org': 'pumbik.ru',
 'moloko--i-ru.turbopages.org': 'moloko-i.ru',
 'pro--xvost-ru.turbopages.org': 'pro-xvost.ru',
 'enigma--vita-livejournal-com.turbopages.org': 'enigma-vita.livejournal.com',
 'denegnerad-livejournal-com.turbopages.org': 'denegnerad.livejournal.com',
 'start--yoga-livejournal-com.turbopages.org': 'start-yoga.livejournal.com',
 'fish-team.turbopages.org': 'fish.team',
 'sde-in-ua.turbopages.org': 'sde.in.ua',
 'newduster-ru.turbopages.org': 'newduster.ru',
 'aminora-livejournal-com.turbopages.org': 'aminora.livejournal.com',
 'romanetto-livejournal-com.turbopages.org': 'romanetto.livejournal.com',
 'rositsa-livejournal-com.turbopages.org': 'rositsa.livejournal.com',
 'olgaafon-livejournal-com.turbopages.org': 'olgaafon.livejournal.com',
 'kuda--kazan-ru.turbopages.org': 'kuda-kazan.ru',
 'natali--ya-livejournal-com.turbopages.org': 'natali-ya.livejournal.com',
 'autbar-ru.turbopages.org': 'autbar.ru',
 'kate--goria-livejournal-com.turbopages.org': 'kate-goria.livejournal.com',
 'amexc-ru.turbopages.org': 'amexc.ru',
 'meclean-ru.turbopages.org': 'meclean.ru',
 'vb-by.turbopages.org': 'vb.by',
 'arhangelsk-bezformata-com.turbopages.org': 'arhangelsk.bezformata.com',
 'dommagii-org.turbopages.org': 'dommagii.org',
 'artm--yug-ru.turbopages.org': 'artm-yug.ru',
 'webohrannik-ru.turbopages.org': 'webohrannik.ru',
 'ololo-tv.turbopages.org': 'ololo.tv',
 'topnewsrussia-ru.turbopages.org': 'topnewsrussia.ru',
 'urbanexplorer61-livejournal-com.turbopages.org': 'urbanexplorer61.livejournal.com',
 'amigooo-livejournal-com.turbopages.org': 'amigooo.livejournal.com',
 'kozak--pravdorub-livejournal-com.turbopages.org': 'kozak-pravdorub.livejournal.com',
 'german--shepherd--dog-ru.turbopages.org': 'german-shepherd-dog.ru',
 'shikiogor-ru.turbopages.org': 'shikiogor.ru',
 'alex--aka--jj-livejournal-com.turbopages.org': 'alex-aka-jj.livejournal.com',
 'postlink2008-livejournal-com.turbopages.org': 'postlink2008.livejournal.com',
 'marrch--caat-livejournal-com.turbopages.org': 'marrch-caat.livejournal.com',
 '2compa-ru.turbopages.org': '2compa.ru',
 'russamara-livejournal-com.turbopages.org': 'russamara.livejournal.com',
 'anastasiatruth-livejournal-com.turbopages.org': 'anastasiatruth.livejournal.com',
 'pricecoins-ru.turbopages.org': 'pricecoins.ru',
 'brestcity-com.turbopages.org': 'brestcity.com',
 'motorchina--online-ru.turbopages.org': 'motorchina-online.ru',
 '2775040-livejournal-com.turbopages.org': '2775040.livejournal.com',
 'avtobasarf-ru.turbopages.org': 'avtobasarf.ru',
 '100wines-net.turbopages.org': '100wines.net',
 'wiki-pro--voip-ru.turbopages.org': 'wiki.pro-voip.ru',
 'posle--60--let-ru.turbopages.org': 'posle-60-let.ru',
 'medknsltant-com.turbopages.org': 'medknsltant.com',
 'moloczheleza-ru.turbopages.org': 'moloczheleza.ru',
 'vashakasha-com.turbopages.org': 'vashakasha.com',
 'licuala-livejournal-com.turbopages.org': 'licuala.livejournal.com',
 'ismilady-livejournal-com.turbopages.org': 'ismilady.livejournal.com',
 'intellifishing-ru.turbopages.org': 'intellifishing.ru',
 'ford--trucks-club.turbopages.org': 'ford-trucks.club',
 'bigra-livejournal-com.turbopages.org': 'bigra.livejournal.com',
 'nonbovi-livejournal-com.turbopages.org': 'nonbovi.livejournal.com',
 'o--cz-ru.turbopages.org': 'o-cz.ru',
 'svyatoslav-livejournal-com.turbopages.org': 'svyatoslav.livejournal.com',
 'zzizz-ru.turbopages.org': 'zzizz.ru',
 'tsarvar-com.turbopages.org': 'tsarvar.com',
 'miosalute-ru.turbopages.org': 'miosalute.ru',
 'oreptiliya-ru.turbopages.org': 'oreptiliya.ru',
 'konreal-livejournal-com.turbopages.org': 'konreal.livejournal.com',
 'robinbobin-livejournal-com.turbopages.org': 'robinbobin.livejournal.com',
 'fusionstyle--cool-ru.turbopages.org': 'fusionstyle-cool.ru',
 'ladyup-online.turbopages.org': 'ladyup.online',
 'retroa-livejournal-com.turbopages.org': 'retroa.livejournal.com',
 'khmelikvictor-livejournal-com.turbopages.org': 'khmelikvictor.livejournal.com',
 'nikonofficial-livejournal-com.turbopages.org': 'nikonofficial.livejournal.com',
 'finansypress-ru.turbopages.org': 'finansypress.ru',
 'charovni-ru.turbopages.org': 'charovni.ru',
 'apatiti-bezformata-com.turbopages.org': 'apatiti.bezformata.com',
 'kometa-red.turbopages.org': 'kometa.red',
 'deni--didro-livejournal-com.turbopages.org': 'deni-didro.livejournal.com',
 'ivangogh-livejournal-com.turbopages.org': 'ivangogh.livejournal.com',
 'banyabest-ru.turbopages.org': 'banyabest.ru',
 'bannyi--den-ru.turbopages.org': 'bannyi-den.ru',
 'wg--lj-livejournal-com.turbopages.org': 'wg-lj.livejournal.com',
 'assalam786-livejournal-com.turbopages.org': 'assalam786.livejournal.com',
 'spirtyaga-ru.turbopages.org': 'spirtyaga.ru',
 'alternathistory-livejournal-com.turbopages.org': 'alternathistory.livejournal.com',
 'blog-altuninvv-ru.turbopages.org': 'blog.altuninvv.ru',
 'dlt--spb-livejournal-com.turbopages.org': 'dlt-spb.livejournal.com',
 'proboknet-livejournal-com.turbopages.org': 'proboknet.livejournal.com',
 'desants-livejournal-com.turbopages.org': 'desants.livejournal.com',
 'arbatovagidepar-livejournal-com.turbopages.org': 'arbatovagidepar.livejournal.com',
 'le--kot-livejournal-com.turbopages.org': 'le-kot.livejournal.com',
 'fotograf--1-livejournal-com.turbopages.org': 'fotograf-1.livejournal.com',
 'oktyabrsky-big--book--med-ru.turbopages.org': 'oktyabrsky.big-book-med.ru',
 'moepravo-guru.turbopages.org': 'moepravo.guru',
 'home--and--garden-livejournal-com.turbopages.org': 'home-and-garden.livejournal.com',
 'feodora3368-livejournal-com.turbopages.org': 'feodora3368.livejournal.com',
 'kotobum-ru.turbopages.org': 'kotobum.ru',
 'photo--prichesok-ru.turbopages.org': 'photo-prichesok.ru',
 'andreyivanichsmirnov-ru.turbopages.org': 'andreyivanichsmirnov.ru',
 'mirinda--38-livejournal-com.turbopages.org': 'mirinda-38.livejournal.com',
 'marketologiya-livejournal-com.turbopages.org': 'marketologiya.livejournal.com',
 'trunov--dmitry-livejournal-com.turbopages.org': 'trunov-dmitry.livejournal.com',
 'niejournal-ru.turbopages.org': 'niejournal.ru',
 'gavailer-livejournal-com.turbopages.org': 'gavailer.livejournal.com',
 'invirostov-livejournal-com.turbopages.org': 'invirostov.livejournal.com',
 'step--to--joy-livejournal-com.turbopages.org': 'step-to-joy.livejournal.com',
 'never--no-livejournal-com.turbopages.org': 'never-no.livejournal.com',
 'birdchuvashia-livejournal-com.turbopages.org': 'birdchuvashia.livejournal.com',
 'gallika-livejournal-com.turbopages.org': 'gallika.livejournal.com',
 'shakko--kitsune-livejournal-com.turbopages.org': 'shakko-kitsune.livejournal.com',
 'kcherepanov-livejournal-com.turbopages.org': 'kcherepanov.livejournal.com',
 'egyptpro-ru.turbopages.org': 'egyptpro.ru',
 'musorexpert-ru.turbopages.org': 'musorexpert.ru',
 'service--193-livejournal-com.turbopages.org': 'service-193.livejournal.com',
 'deletant--co-livejournal-com.turbopages.org': 'deletant-co.livejournal.com',
 'fms--forum-ru.turbopages.org': 'fms-forum.ru',
 'milkmomstory-livejournal-com.turbopages.org': 'milkmomstory.livejournal.com',
 'alifammara-livejournal-com.turbopages.org': 'alifammara.livejournal.com',
 'bildsnab--yug-ru.turbopages.org': 'bildsnab-yug.ru',
 'chita-bezformata-com.turbopages.org': 'chita.bezformata.com',
 'afisha-yuga-ru.turbopages.org': 'afisha.yuga.ru',
 'lenoblast-bezformata-com.turbopages.org': 'lenoblast.bezformata.com',
 'lubelia-livejournal-com.turbopages.org': 'lubelia.livejournal.com',
 'artsad--manko-ru.turbopages.org': 'artsad-manko.ru',
 'erich--springer-livejournal-com.turbopages.org': 'erich-springer.livejournal.com',
 'mosobl-livejournal-com.turbopages.org': 'mosobl.livejournal.com',
 'uchvatovsb-livejournal-com.turbopages.org': 'uchvatovsb.livejournal.com',
 'spichechka-livejournal-com.turbopages.org': 'spichechka.livejournal.com',
 'oktlife-ru.turbopages.org': 'oktlife.ru',
 'nandzed-livejournal-com.turbopages.org': 'nandzed.livejournal.com',
 'karmaraksha-livejournal-com.turbopages.org': 'karmaraksha.livejournal.com',
 'ru--med-livejournal-com.turbopages.org': 'ru-med.livejournal.com',
 'lady--ikx-livejournal-com.turbopages.org': 'lady-ikx.livejournal.com',
 'gusewww-livejournal-com.turbopages.org': 'gusewww.livejournal.com',
 'miravto21-ru.turbopages.org': 'miravto21.ru',
 'terme-ru.turbopages.org': 'terme.ru',
 'biblionovolaba-ru.turbopages.org': 'biblionovolaba.ru',
 'historyonline-livejournal-com.turbopages.org': 'historyonline.livejournal.com',
 'moskva--bez--prik-livejournal-com.turbopages.org': 'moskva-bez-prik.livejournal.com',
 'andry--astashkin-livejournal-com.turbopages.org': 'andry-astashkin.livejournal.com',
 'makekaresus19-livejournal-com.turbopages.org': 'makekaresus19.livejournal.com',
 'moya--moskva-livejournal-com.turbopages.org': 'moya-moskva.livejournal.com',
 'pharmateacher-livejournal-com.turbopages.org': 'pharmateacher.livejournal.com',
 'gde--notarius-ru.turbopages.org': 'gde-notarius.ru',
 'liza--photo-livejournal-com.turbopages.org': 'liza-photo.livejournal.com',
 'auto--history-livejournal-com.turbopages.org': 'auto-history.livejournal.com',
 'galina--59-livejournal-com.turbopages.org': 'galina-59.livejournal.com',
 'mordig81-livejournal-com.turbopages.org': 'mordig81.livejournal.com',
 'bioplant-livejournal-com.turbopages.org': 'bioplant.livejournal.com',
 'nefteyugansk-bezformata-com.turbopages.org': 'nefteyugansk.bezformata.com',
 'sadiklumba-ru.turbopages.org': 'sadiklumba.ru',
 'floradoc-com.turbopages.org': 'floradoc.com',
 'polymerclayfimo-livejournal-com.turbopages.org': 'polymerclayfimo.livejournal.com',
 'blog--10101-livejournal-com.turbopages.org': 'blog-10101.livejournal.com',
 'elhutto-livejournal-com.turbopages.org': 'elhutto.livejournal.com',
 'athlonangel-livejournal-com.turbopages.org': 'athlonangel.livejournal.com',
 'oleg--ivanov-livejournal-com.turbopages.org': 'oleg-ivanov.livejournal.com',
 'yellowcross-ru.turbopages.org': 'yellowcross.ru',
 'kaluga-bezformata-com.turbopages.org': 'kaluga.bezformata.com',
 'serpuhov-big--book--med-ru.turbopages.org': 'serpuhov.big-book-med.ru',
 'osk82-ru.turbopages.org': 'osk82.ru',
 'saki--remont-ru.turbopages.org': 'saki-remont.ru',
 'mir--i--ljubov-livejournal-com.turbopages.org': 'mir-i-ljubov.livejournal.com',
 'oren--reader-livejournal-com.turbopages.org': 'oren-reader.livejournal.com',
 'socker--zucchero-livejournal-com.turbopages.org': 'socker-zucchero.livejournal.com',
 'balamut4uma-livejournal-com.turbopages.org': 'balamut4uma.livejournal.com',
 'dialchu-ru.turbopages.org': 'dialchu.ru',
 'prosto--interesno-ru.turbopages.org': 'prosto-interesno.ru',
 'countries1-ru.turbopages.org': 'countries1.ru',
 'tehnopub-ru.turbopages.org': 'tehnopub.ru',
 'shumerlya-bezformata-com.turbopages.org': 'shumerlya.bezformata.com',
 'mobilenotes-ru.turbopages.org': 'mobilenotes.ru',
 'auto-vercity-ru.turbopages.org': 'auto.vercity.ru',
 'pythonru-com.turbopages.org': 'pythonru.com',
 'under--prog-ru.turbopages.org': 'under-prog.ru',
 'vlade--mir-livejournal-com.turbopages.org': 'vlade-mir.livejournal.com',
 'rasteniyadom-ru.turbopages.org': 'rasteniyadom.ru',
 'photo--discovery-livejournal-com.turbopages.org': 'photo-discovery.livejournal.com',
 'ervix-livejournal-com.turbopages.org': 'ervix.livejournal.com',
 'mrgoodween-livejournal-com.turbopages.org': 'mrgoodween.livejournal.com',
 'yourmotor-ru.turbopages.org': 'yourmotor.ru',
 'bloknot--turista-ru.turbopages.org': 'bloknot-turista.ru',
 'mymood-ru.turbopages.org': 'mymood.ru',
 'handbookbride-ru.turbopages.org': 'handbookbride.ru',
 'bebiklad-ru.turbopages.org': 'bebiklad.ru',
 'man--with--dogs-livejournal-com.turbopages.org': 'man-with-dogs.livejournal.com',
 'starkov--blues-livejournal-com.turbopages.org': 'starkov-blues.livejournal.com',
 '13chairs-ru.turbopages.org': '13chairs.ru',
 'insider--dnr-livejournal-com.turbopages.org': 'insider-dnr.livejournal.com',
 'tsuslik-ru.turbopages.org': 'tsuslik.ru',
 'rodinaussr-livejournal-com.turbopages.org': 'rodinaussr.livejournal.com',
 'etocvetochki-com.turbopages.org': 'etocvetochki.com',
 'bemmby-livejournal-com.turbopages.org': 'bemmby.livejournal.com',
 'kmory-ru.turbopages.org': 'kmory.ru',
 'skincenter-ru.turbopages.org': 'skincenter.ru',
 'novorossiysk-bezformata-com.turbopages.org': 'novorossiysk.bezformata.com',
 'sattvik-livejournal-com.turbopages.org': 'sattvik.livejournal.com',
 'tsarkov--peter-livejournal-com.turbopages.org': 'tsarkov-peter.livejournal.com',
 'rus--kukurus-livejournal-com.turbopages.org': 'rus-kukurus.livejournal.com',
 'igor--journal-livejournal-com.turbopages.org': 'igor-journal.livejournal.com',
 'bigstonedragon-livejournal-com.turbopages.org': 'bigstonedragon.livejournal.com',
 'falcon63-livejournal-com.turbopages.org': 'falcon63.livejournal.com',
 'foundmaster-ru.turbopages.org': 'foundmaster.ru',
 'wowka13-livejournal-com.turbopages.org': 'wowka13.livejournal.com',
 'avianikitoss-livejournal-com.turbopages.org': 'avianikitoss.livejournal.com',
 'convert--me-com.turbopages.org': 'convert-me.com',
 'history--club-livejournal-com.turbopages.org': 'history-club.livejournal.com',
 'kovaleva-gallery.turbopages.org': 'kovaleva.gallery',
 'russsoprot-livejournal-com.turbopages.org': 'russsoprot.livejournal.com',
 'williamfaulkner-ru.turbopages.org': 'williamfaulkner.ru',
 'paulus--raul-livejournal-com.turbopages.org': 'paulus-raul.livejournal.com',
 'yasnji-livejournal-com.turbopages.org': 'yasnji.livejournal.com',
 'lamaruo-livejournal-com.turbopages.org': 'lamaruo.livejournal.com',
 'mryabinin-livejournal-com.turbopages.org': 'mryabinin.livejournal.com',
 'alexeygoncharov-livejournal-com.turbopages.org': 'alexeygoncharov.livejournal.com',
 'davnym--davno-livejournal-com.turbopages.org': 'davnym-davno.livejournal.com',
 'van--gogha-livejournal-com.turbopages.org': 'van-gogha.livejournal.com',
 'modeland-ru.turbopages.org': 'modeland.ru',
 'montrealex-livejournal-com.turbopages.org': 'montrealex.livejournal.com',
 'gal--an-livejournal-com.turbopages.org': 'gal-an.livejournal.com',
 'stroycentr96-ru.turbopages.org': 'stroycentr96.ru',
 'roman--yhnovec-livejournal-com.turbopages.org': 'roman-yhnovec.livejournal.com',
 'ideireceptov-ru.turbopages.org': 'ideireceptov.ru',
 'kalgad-livejournal-com.turbopages.org': 'kalgad.livejournal.com',
 'donday--shakhty-ru.turbopages.org': 'donday-shakhty.ru',
 'podelochkin-ru.turbopages.org': 'podelochkin.ru',
 'yarvelesov-livejournal-com.turbopages.org': 'yarvelesov.livejournal.com',
 'rex711-livejournal-com.turbopages.org': 'rex711.livejournal.com',
 'marybrows-ru.turbopages.org': 'marybrows.ru',
 'petropressa-ru.turbopages.org': 'petropressa.ru',
 'spuzom-com.turbopages.org': 'spuzom.com',
 'olejka-livejournal-com.turbopages.org': 'olejka.livejournal.com',
 'old--crocodile-livejournal-com.turbopages.org': 'old-crocodile.livejournal.com',
 'smolegov-livejournal-com.turbopages.org': 'smolegov.livejournal.com',
 'yurgita-livejournal-com.turbopages.org': 'yurgita.livejournal.com',
 'svetla--k-livejournal-com.turbopages.org': 'svetla-k.livejournal.com',
 'koshkanasha-ru.turbopages.org': 'koshkanasha.ru',
 'neirolog-livejournal-com.turbopages.org': 'neirolog.livejournal.com',
 'visitor--fudo-livejournal-com.turbopages.org': 'visitor-fudo.livejournal.com',
 'show--me--ivanovo-livejournal-com.turbopages.org': 'show-me-ivanovo.livejournal.com',
 'michael101063-livejournal-com.turbopages.org': 'michael101063.livejournal.com',
 'galina--s-livejournal-com.turbopages.org': 'galina-s.livejournal.com',
 'olesia--organic-livejournal-com.turbopages.org': 'olesia-organic.livejournal.com',
 'mark--twen-livejournal-com.turbopages.org': 'mark-twen.livejournal.com',
 'media-price-ua.turbopages.org': 'media.price.ua',
 'tech--m-livejournal-com.turbopages.org': 'tech-m.livejournal.com',
 'pro--platje-ru.turbopages.org': 'pro-platje.ru',
 '9news-livejournal-com.turbopages.org': '9news.livejournal.com',
 'misha--samarsky-livejournal-com.turbopages.org': 'misha-samarsky.livejournal.com',
 'bp21-livejournal-com.turbopages.org': 'bp21.livejournal.com',
 'andrei--bt-livejournal-com.turbopages.org': 'andrei-bt.livejournal.com',
 'obuv--patriotplus-ru.turbopages.org': 'obuv-patriotplus.ru',
 'yoursong-ru.turbopages.org': 'yoursong.ru',
 'dachaklub-ru.turbopages.org': 'dachaklub.ru',
 'limonov74-livejournal-com.turbopages.org': 'limonov74.livejournal.com',
 'p--ster-livejournal-com.turbopages.org': 'p-ster.livejournal.com',
 'replicadisk-ru.turbopages.org': 'replicadisk.ru',
 'starina--chuk-livejournal-com.turbopages.org': 'starina-chuk.livejournal.com',
 'akolesov-livejournal-com.turbopages.org': 'akolesov.livejournal.com',
 'ihavesage-livejournal-com.turbopages.org': 'ihavesage.livejournal.com',
 'mediastrazh-ru.turbopages.org': 'mediastrazh.ru',
 'xorolik-livejournal-com.turbopages.org': 'xorolik.livejournal.com',
 'sibved-livejournal-com.turbopages.org': 'sibved.livejournal.com',
 'ru--remont-com.turbopages.org': 'ru-remont.com',
 'rainy--sunny-livejournal-com.turbopages.org': 'rainy-sunny.livejournal.com',
 'ask--robert-livejournal-com.turbopages.org': 'ask-robert.livejournal.com',
 'shweda-livejournal-com.turbopages.org': 'shweda.livejournal.com',
 'tula-big--book--med-ru.turbopages.org': 'tula.big-book-med.ru',
 'tatiana--gayduk-livejournal-com.turbopages.org': 'tatiana-gayduk.livejournal.com',
 'zapechen-ru.turbopages.org': 'zapechen.ru',
 'petzona-ru.turbopages.org': 'petzona.ru',
 'alisa--morozova-livejournal-com.turbopages.org': 'alisa-morozova.livejournal.com',
 'svetliy--cgb-ru.turbopages.org': 'svetliy-cgb.ru',
 'imgame-kz.turbopages.org': 'imgame.kz',
 'cmhelp-ru.turbopages.org': 'cmhelp.ru',
 'neshitoff-livejournal-com.turbopages.org': 'neshitoff.livejournal.com',
 'lilya--shuk-livejournal-com.turbopages.org': 'lilya-shuk.livejournal.com',
 'andrmagia-livejournal-com.turbopages.org': 'andrmagia.livejournal.com',
 'alexandr2201-livejournal-com.turbopages.org': 'alexandr2201.livejournal.com',
 'golosoff-livejournal-com.turbopages.org': 'golosoff.livejournal.com',
 'km-ru.turbopages.org': 'km.ru',
 'seohotmix-ru.turbopages.org': 'seohotmix.ru',
 'reshit-ru.turbopages.org': 'reshit.ru',
 'fishgorod-ru.turbopages.org': 'fishgorod.ru',
 'batytnie--centry--spb-ru.turbopages.org': 'batytnie-centry-spb.ru',
 'sled--udmproc-ru.turbopages.org': 'sled-udmproc.ru',
 'tnenergy-livejournal-com.turbopages.org': 'tnenergy.livejournal.com',
 'nely--snork-livejournal-com.turbopages.org': 'nely-snork.livejournal.com',
 'kaluga--apx-livejournal-com.turbopages.org': 'kaluga-apx.livejournal.com',
 'texcontem-livejournal-com.turbopages.org': 'texcontem.livejournal.com',
 'panzerjack-livejournal-com.turbopages.org': 'panzerjack.livejournal.com',
 'myprikol-com.turbopages.org': 'myprikol.com',
 'lyasha-livejournal-com.turbopages.org': 'lyasha.livejournal.com',
 'tyulka-livejournal-com.turbopages.org': 'tyulka.livejournal.com',
 'k--larabell-livejournal-com.turbopages.org': 'k-larabell.livejournal.com',
 'chistota-guru.turbopages.org': 'chistota.guru',
 'assa777-livejournal-com.turbopages.org': 'assa777.livejournal.com',
 'delta-news.turbopages.org': 'delta.news',
 'espen31-livejournal-com.turbopages.org': 'espen31.livejournal.com',
 'topof-livejournal-com.turbopages.org': 'topof.livejournal.com',
 'ru--popugai-livejournal-com.turbopages.org': 'ru-popugai.livejournal.com',
 'ornitoterapiya-ru.turbopages.org': 'ornitoterapiya.ru',
 'perrotcorella-livejournal-com.turbopages.org': 'perrotcorella.livejournal.com',
 'autosmotr-ru.turbopages.org': 'autosmotr.ru',
 'atsinis-livejournal-com.turbopages.org': 'atsinis.livejournal.com',
 'goldperepel-ru.turbopages.org': 'goldperepel.ru',
 'ketrin--mel-livejournal-com.turbopages.org': 'ketrin-mel.livejournal.com',
 'proagregat-com.turbopages.org': 'proagregat.com',
 'konono-livejournal-com.turbopages.org': 'konono.livejournal.com',
 'samtime-online.turbopages.org': 'samtime.online',
 'and--ekb-livejournal-com.turbopages.org': 'and-ekb.livejournal.com',
 'instrument-guru.turbopages.org': 'instrument.guru',
 'chelyabinsk-octagon-media.turbopages.org': 'chelyabinsk.octagon.media',
 'kemerovo-bezformata-com.turbopages.org': 'kemerovo.bezformata.com',
 'miodimore-ru.turbopages.org': 'miodimore.ru',
 'youloy--ruslan-livejournal-com.turbopages.org': 'youloy-ruslan.livejournal.com',
 'jaaj-club.turbopages.org': 'jaaj.club',
 'mr--gsv-livejournal-com.turbopages.org': 'mr-gsv.livejournal.com',
 'apjournal-ru.turbopages.org': 'apjournal.ru',
 'gotskaya-livejournal-com.turbopages.org': 'gotskaya.livejournal.com',
 'riaderbent-ru.turbopages.org': 'riaderbent.ru',
 'doctor--cardiologist-ru.turbopages.org': 'doctor-cardiologist.ru',
 'mara--kar-livejournal-com.turbopages.org': 'mara-kar.livejournal.com',
 'latinsk-ru.turbopages.org': 'latinsk.ru',
 'rugraphics-ru.turbopages.org': 'rugraphics.ru',
 'letozimoi-ru.turbopages.org': 'letozimoi.ru',
 'diabeta--net-ru.turbopages.org': 'diabeta-net.ru',
 'talkaboutcre-livejournal-com.turbopages.org': 'talkaboutcre.livejournal.com',
 'gilliotinus-livejournal-com.turbopages.org': 'gilliotinus.livejournal.com',
 'napitok--eco-ru.turbopages.org': 'napitok-eco.ru',
 'life14-ru.turbopages.org': 'life14.ru',
 'bolezney-net.turbopages.org': 'bolezney.net',
 'png2-ru.turbopages.org': 'png2.ru',
 'kostroma-aif-ru.turbopages.org': 'kostroma.aif.ru',
 'simdental-ru.turbopages.org': 'simdental.ru',
 'aogarden-ru.turbopages.org': 'aogarden.ru',
 'orchids--lux-livejournal-com.turbopages.org': 'orchids-lux.livejournal.com',
 'majakovskij-livejournal-com.turbopages.org': 'majakovskij.livejournal.com',
 'dima--piterski-livejournal-com.turbopages.org': 'dima-piterski.livejournal.com',
 'vuk--srpski-livejournal-com.turbopages.org': 'vuk-srpski.livejournal.com',
 'mkisrael-co-il.turbopages.org': 'mkisrael.co.il',
 'pogovorim-by.turbopages.org': 'pogovorim.by',
 'bagdasarov--lj-livejournal-com.turbopages.org': 'bagdasarov-lj.livejournal.com',
 'shichengaru-livejournal-com.turbopages.org': 'shichengaru.livejournal.com',
 'x--a--n--k--a-livejournal-com.turbopages.org': 'x-a-n-k-a.livejournal.com',
 'ru--linux-livejournal-com.turbopages.org': 'ru-linux.livejournal.com',
 'old--lekar-com.turbopages.org': 'old-lekar.com',
 'foxfan-ru.turbopages.org': 'foxfan.ru',
 'lechiserdce-ru.turbopages.org': 'lechiserdce.ru',
 'selfmade--lucy-livejournal-com.turbopages.org': 'selfmade-lucy.livejournal.com',
 'viver77-livejournal-com.turbopages.org': 'viver77.livejournal.com',
 'pronabludenie-com.turbopages.org': 'pronabludenie.com',
 'swedinfo-ru.turbopages.org': 'swedinfo.ru',
 '110km-ru.turbopages.org': '110km.ru',
 'k--poli-livejournal-com.turbopages.org': 'k-poli.livejournal.com',
 'b44-ru.turbopages.org': 'b44.ru',
 '12kotov-ru.turbopages.org': '12kotov.ru',
 'next--brands-com.turbopages.org': 'next-brands.com',
 'marchenk-livejournal-com.turbopages.org': 'marchenk.livejournal.com',
 'nasledstvo--ru-ru.turbopages.org': 'nasledstvo-ru.ru',
 'arctus-livejournal-com.turbopages.org': 'arctus.livejournal.com',
 'mitino-livejournal-com.turbopages.org': 'mitino.livejournal.com',
 'runaruna-ru.turbopages.org': 'runaruna.ru',
 '3ojlotou-livejournal-com.turbopages.org': '3ojlotou.livejournal.com',
 'v--inhelo-livejournal-com.turbopages.org': 'v-inhelo.livejournal.com',
 'alexzsoft-ru.turbopages.org': 'alexzsoft.ru',
 'barzy-livejournal-com.turbopages.org': 'barzy.livejournal.com',
 'world--jewellery-livejournal-com.turbopages.org': 'world-jewellery.livejournal.com',
 'lechigemor-ru.turbopages.org': 'lechigemor.ru',
 'ural--ga-livejournal-com.turbopages.org': 'ural-ga.livejournal.com',
 'infovend-ru.turbopages.org': 'infovend.ru',
 'norma--pb-ru.turbopages.org': 'norma-pb.ru',
 'hk--krasnodar-ru.turbopages.org': 'hk-krasnodar.ru',
 'unis-livejournal-com.turbopages.org': 'unis.livejournal.com',
 'naveselegaming-ru.turbopages.org': 'naveselegaming.ru',
 'krasnaya--femka-livejournal-com.turbopages.org': 'krasnaya-femka.livejournal.com',
 'activeinvestor-pro.turbopages.org': 'activeinvestor.pro',
 'tiana--verner-livejournal-com.turbopages.org': 'tiana-verner.livejournal.com',
 'katyakatya-livejournal-com.turbopages.org': 'katyakatya.livejournal.com',
 'world--dogs-ru.turbopages.org': 'world-dogs.ru',
 'un--tal--leetah-livejournal-com.turbopages.org': 'un-tal-leetah.livejournal.com',
 'povaresh-com.turbopages.org': 'povaresh.com',
 'pracooking-livejournal-com.turbopages.org': 'pracooking.livejournal.com',
 'plemyash--sergei-livejournal-com.turbopages.org': 'plemyash-sergei.livejournal.com',
 'tvojajbolit-ru.turbopages.org': 'tvojajbolit.ru',
 'rnto-livejournal-com.turbopages.org': 'rnto.livejournal.com',
 'peaceinukraine-livejournal-com.turbopages.org': 'peaceinukraine.livejournal.com',
 'sovety-guru.turbopages.org': 'sovety.guru',
 'taliyahco-livejournal-com.turbopages.org': 'taliyahco.livejournal.com',
 'sintezoil-ru.turbopages.org': 'sintezoil.ru',
 'mexanizm-livejournal-com.turbopages.org': 'mexanizm.livejournal.com',
 'jury--tver-livejournal-com.turbopages.org': 'jury-tver.livejournal.com',
 'egor--23-livejournal-com.turbopages.org': 'egor-23.livejournal.com',
 'prosto--eda-ru.turbopages.org': 'prosto-eda.ru',
 'ru--history-livejournal-com.turbopages.org': 'ru-history.livejournal.com',
 'igorek44-livejournal-com.turbopages.org': 'igorek44.livejournal.com',
 'e--strannik-livejournal-com.turbopages.org': 'e-strannik.livejournal.com',
 'funtofil-livejournal-com.turbopages.org': 'funtofil.livejournal.com',
 'garyh--jet-livejournal-com.turbopages.org': 'garyh-jet.livejournal.com',
 'lichnyjcredit-ru.turbopages.org': 'lichnyjcredit.ru',
 'live--imho-livejournal-com.turbopages.org': 'live-imho.livejournal.com',
 'fun-livejournal-com.turbopages.org': 'fun.livejournal.com',
 'jazvahodjachaja-livejournal-com.turbopages.org': 'jazvahodjachaja.livejournal.com',
 'marina--aqua-livejournal-com.turbopages.org': 'marina-aqua.livejournal.com',
 'soviet--people-livejournal-com.turbopages.org': 'soviet-people.livejournal.com',
 'cand--orel-livejournal-com.turbopages.org': 'cand-orel.livejournal.com',
 'iolla-livejournal-com.turbopages.org': 'iolla.livejournal.com',
 'sfrolov-livejournal-com.turbopages.org': 'sfrolov.livejournal.com',
 'cat--779-livejournal-com.turbopages.org': 'cat-779.livejournal.com',
 'inlugansk-ru.turbopages.org': 'inlugansk.ru',
 'skarty-ru.turbopages.org': 'skarty.ru',
 'shusek-livejournal-com.turbopages.org': 'shusek.livejournal.com',
 'darenok--market-ru.turbopages.org': 'darenok-market.ru',
 'zihuatanexo-livejournal-com.turbopages.org': 'zihuatanexo.livejournal.com',
 'otzvezd-ru.turbopages.org': 'otzvezd.ru',
 'parazitron-ru.turbopages.org': 'parazitron.ru',
 'bezmedikov-ru.turbopages.org': 'bezmedikov.ru',
 'moskray-livejournal-com.turbopages.org': 'moskray.livejournal.com',
 'uzao-aif-ru.turbopages.org': 'uzao.aif.ru',
 'dynalib-ru.turbopages.org': 'dynalib.ru',
 'vlad--yulia-livejournal-com.turbopages.org': 'vlad-yulia.livejournal.com',
 'angela--kvitka-livejournal-com.turbopages.org': 'angela-kvitka.livejournal.com',
 'panarina--nadya-livejournal-com.turbopages.org': 'panarina-nadya.livejournal.com',
 'flo-discus--club-ru.turbopages.org': 'flo.discus-club.ru',
 'komkur-info.turbopages.org': 'komkur.info',
 'good--cook-ru.turbopages.org': 'good-cook.ru',
 'm--thunderbird-ru.turbopages.org': 'm-thunderbird.ru',
 'strelec22-livejournal-com.turbopages.org': 'strelec22.livejournal.com',
 'eagle--rost-livejournal-com.turbopages.org': 'eagle-rost.livejournal.com',
 'nepohui-livejournal-com.turbopages.org': 'nepohui.livejournal.com',
 'ruh666-livejournal-com.turbopages.org': 'ruh666.livejournal.com',
 'foto--tattoo-ru.turbopages.org': 'foto-tattoo.ru',
 'daskonto-livejournal-com.turbopages.org': 'daskonto.livejournal.com',
 '64vlad-livejournal-com.turbopages.org': '64vlad.livejournal.com',
 'valerongrach-livejournal-com.turbopages.org': 'valerongrach.livejournal.com',
 'olenalex-livejournal-com.turbopages.org': 'olenalex.livejournal.com',
 'skazo4nik1943-livejournal-com.turbopages.org': 'skazo4nik1943.livejournal.com',
 'michaelsvoboda-livejournal-com.turbopages.org': 'michaelsvoboda.livejournal.com',
 'subbotin--alex-livejournal-com.turbopages.org': 'subbotin-alex.livejournal.com',
 'hivlove-ru.turbopages.org': 'hivlove.ru',
 '1omsk-ru.turbopages.org': '1omsk.ru',
 'invaworld-ru.turbopages.org': 'invaworld.ru',
 'tips--people-ru.turbopages.org': 'tips-people.ru',
 'miihaaiil-livejournal-com.turbopages.org': 'miihaaiil.livejournal.com',
 'berezniki-bezformata-com.turbopages.org': 'berezniki.bezformata.com',
 'mk-kn-md.turbopages.org': 'mk.kn.md',
 'moj--golos-livejournal-com.turbopages.org': 'moj-golos.livejournal.com',
 'legendapress-ru.turbopages.org': 'legendapress.ru',
 'alko-pro.turbopages.org': 'alko.pro',
 'worldgamenews-com.turbopages.org': 'worldgamenews.com',
 'pavel--spasatel-livejournal-com.turbopages.org': 'pavel-spasatel.livejournal.com',
 'ex--roadmedia-ru.turbopages.org': 'ex-roadmedia.ru',
 'itvist-livejournal-com.turbopages.org': 'itvist.livejournal.com',
 'vodasila-ru.turbopages.org': 'vodasila.ru',
 'kamtehnopark-ru.turbopages.org': 'kamtehnopark.ru',
 'koryo--reporter1-livejournal-com.turbopages.org': 'koryo-reporter1.livejournal.com',
 'arkay97-livejournal-com.turbopages.org': 'arkay97.livejournal.com',
 'nashural-livejournal-com.turbopages.org': 'nashural.livejournal.com',
 'pripyanye-livejournal-com.turbopages.org': 'pripyanye.livejournal.com',
 'photozov-livejournal-com.turbopages.org': 'photozov.livejournal.com',
 'fishingsib-ru.turbopages.org': 'fishingsib.ru',
 'sergey--krapivin-livejournal-com.turbopages.org': 'sergey-krapivin.livejournal.com',
 'psiheya18-livejournal-com.turbopages.org': 'psiheya18.livejournal.com',
 'drzubastik-ru.turbopages.org': 'drzubastik.ru',
 'portscaner-ru.turbopages.org': 'portscaner.ru',
 'gazelleclub-ru.turbopages.org': 'gazelleclub.ru',
 'kali76-ru.turbopages.org': 'kali76.ru',
 'tfzp-ru.turbopages.org': 'tfzp.ru',
 'kalakazo-livejournal-com.turbopages.org': 'kalakazo.livejournal.com',
 'arzik-livejournal-com.turbopages.org': 'arzik.livejournal.com',
 'mitrofanova--m-livejournal-com.turbopages.org': 'mitrofanova-m.livejournal.com',
 'oneparatrooper-livejournal-com.turbopages.org': 'oneparatrooper.livejournal.com',
 'elgri-livejournal-com.turbopages.org': 'elgri.livejournal.com',
 'mrdachnik-ru.turbopages.org': 'mrdachnik.ru',
 'ru--biology-livejournal-com.turbopages.org': 'ru-biology.livejournal.com',
 'vgannaa-livejournal-com.turbopages.org': 'vgannaa.livejournal.com',
 'natalya--orlenko-livejournal-com.turbopages.org': 'natalya-orlenko.livejournal.com',
 'storydoma-ru.turbopages.org': 'storydoma.ru',
 'poznavasha-ru.turbopages.org': 'poznavasha.ru',
 'zasushim-ru.turbopages.org': 'zasushim.ru',
 'dobrazdrava-ru.turbopages.org': 'dobrazdrava.ru',
 'tg--journal-com.turbopages.org': 'tg-journal.com',
 'alexbedrin-com.turbopages.org': 'alexbedrin.com',
 'heal--skin-com.turbopages.org': 'heal-skin.com',
 'medicalserviceplus-ru.turbopages.org': 'medicalserviceplus.ru',
 'vklader-com.turbopages.org': 'vklader.com',
 'potsreotizm-livejournal-com.turbopages.org': 'potsreotizm.livejournal.com',
 'buhpressa-ru.turbopages.org': 'buhpressa.ru',
 'major--colville-livejournal-com.turbopages.org': 'major-colville.livejournal.com',
 'ayrat72-livejournal-com.turbopages.org': 'ayrat72.livejournal.com',
 'vyatkakirov-ru.turbopages.org': 'vyatkakirov.ru',
 'vadim--i--z-livejournal-com.turbopages.org': 'vadim-i-z.livejournal.com',
 'daiyan--19-livejournal-com.turbopages.org': 'daiyan-19.livejournal.com',
 'alko-expert.turbopages.org': 'alko.expert',
 'terrablag-ru.turbopages.org': 'terrablag.ru',
 'agk24-ru.turbopages.org': 'agk24.ru',
 'beerbottle-ru.turbopages.org': 'beerbottle.ru',
 'linguistpro-net.turbopages.org': 'linguistpro.net',
 'anecdoty-ru.turbopages.org': 'anecdoty.ru',
 'laura--mz-livejournal-com.turbopages.org': 'laura-mz.livejournal.com',
 'vikond65-livejournal-com.turbopages.org': 'vikond65.livejournal.com',
 'kiapublic-ru.turbopages.org': 'kiapublic.ru',
 'investbrok-ru.turbopages.org': 'investbrok.ru',
 'pan--sapunov-livejournal-com.turbopages.org': 'pan-sapunov.livejournal.com',
 'autoprokat--rentmotors-ru.turbopages.org': 'autoprokat-rentmotors.ru',
 'dentalvbg-ru.turbopages.org': 'dentalvbg.ru',
 'labote-ru.turbopages.org': 'labote.ru',
 'sana67-ru.turbopages.org': 'sana67.ru',
 'russ--politics-livejournal-com.turbopages.org': 'russ-politics.livejournal.com',
 'wise--maga-livejournal-com.turbopages.org': 'wise-maga.livejournal.com',
 'admai-livejournal-com.turbopages.org': 'admai.livejournal.com',
 'parmaday-ru.turbopages.org': 'parmaday.ru',
 'kartinka--kartinki-ru.turbopages.org': 'kartinka-kartinki.ru',
 'engineering--ru-livejournal-com.turbopages.org': 'engineering-ru.livejournal.com',
 'vkusss-ru.turbopages.org': 'vkusss.ru',
 'sekunda-media.turbopages.org': 'sekunda.media',
 'burnyashev-ru.turbopages.org': 'burnyashev.ru',
 'dimitrovy-livejournal-com.turbopages.org': 'dimitrovy.livejournal.com',
 'bitovki-info.turbopages.org': 'bitovki.info',
 'goods--for--art-livejournal-com.turbopages.org': 'goods-for-art.livejournal.com',
 'dokoli-livejournal-com.turbopages.org': 'dokoli.livejournal.com',
 'dandorfman-livejournal-com.turbopages.org': 'dandorfman.livejournal.com',
 'krasnoeselo-su.turbopages.org': 'krasnoeselo.su',
 'konstantinna-livejournal-com.turbopages.org': 'konstantinna.livejournal.com',
 'murleka-livejournal-com.turbopages.org': 'murleka.livejournal.com',
 'ru--auto-livejournal-com.turbopages.org': 'ru-auto.livejournal.com',
 'demkristo-livejournal-com.turbopages.org': 'demkristo.livejournal.com',
 'tommycus-livejournal-com.turbopages.org': 'tommycus.livejournal.com',
 'ryabovol-livejournal-com.turbopages.org': 'ryabovol.livejournal.com',
 'sryl-livejournal-com.turbopages.org': 'sryl.livejournal.com',
 'pryaha-livejournal-com.turbopages.org': 'pryaha.livejournal.com',
 'family--counselor-ru.turbopages.org': 'family-counselor.ru',
 'superurozhay-ru.turbopages.org': 'superurozhay.ru',
 'idei--biz-com.turbopages.org': 'idei-biz.com',
 'chemicalstudy-ru.turbopages.org': 'chemicalstudy.ru',
 'solnishka--ra-livejournal-com.turbopages.org': 'solnishka-ra.livejournal.com',
 'efremovrf-livejournal-com.turbopages.org': 'efremovrf.livejournal.com',
 'lor-guru.turbopages.org': 'lor.guru',
 'ryzhij--kotik-livejournal-com.turbopages.org': 'ryzhij-kotik.livejournal.com',
 'automobile--zip-ru.turbopages.org': 'automobile-zip.ru',
 'swking-ru.turbopages.org': 'swking.ru',
 'sharp--worlds-ru.turbopages.org': 'sharp-worlds.ru',
 'vseosemye-ru.turbopages.org': 'vseosemye.ru',
 'artyushenkooleg-livejournal-com.turbopages.org': 'artyushenkooleg.livejournal.com',
 'acheleranda-livejournal-com.turbopages.org': 'acheleranda.livejournal.com',
 'oleganisimov-livejournal-com.turbopages.org': 'oleganisimov.livejournal.com',
 'airinsail-ru.turbopages.org': 'airinsail.ru',
 'wikitrucks-ru.turbopages.org': 'wikitrucks.ru',
 'alvere-livejournal-com.turbopages.org': 'alvere.livejournal.com',
 'prosmazku-ru.turbopages.org': 'prosmazku.ru',
 's0no-livejournal-com.turbopages.org': 's0no.livejournal.com',
 'senib-livejournal-com.turbopages.org': 'senib.livejournal.com',
 'r--l--stevenson-livejournal-com.turbopages.org': 'r-l-stevenson.livejournal.com',
 'rostov--news-net.turbopages.org': 'rostov-news.net',
 'kompy-guru.turbopages.org': 'kompy.guru',
 'tatka--olk-livejournal-com.turbopages.org': 'tatka-olk.livejournal.com',
 'elena--sem-livejournal-com.turbopages.org': 'elena-sem.livejournal.com',
 'zavod007-livejournal-com.turbopages.org': 'zavod007.livejournal.com',
 'mius--front-livejournal-com.turbopages.org': 'mius-front.livejournal.com',
 'dimoheha-livejournal-com.turbopages.org': 'dimoheha.livejournal.com',
 'padonak39-livejournal-com.turbopages.org': 'padonak39.livejournal.com',
 'theorangecolor-livejournal-com.turbopages.org': 'theorangecolor.livejournal.com',
 'shmulperzon-livejournal-com.turbopages.org': 'shmulperzon.livejournal.com',
 'fixin-livejournal-com.turbopages.org': 'fixin.livejournal.com',
 'mdhata-livejournal-com.turbopages.org': 'mdhata.livejournal.com',
 'zov--24-livejournal-com.turbopages.org': 'zov-24.livejournal.com',
 'mihailgurevitch-livejournal-com.turbopages.org': 'mihailgurevitch.livejournal.com',
 'brainandsoma-ru.turbopages.org': 'brainandsoma.ru',
 'mkpenza-ru.turbopages.org': 'mkpenza.ru',
 'delat--mozg-livejournal-com.turbopages.org': 'delat-mozg.livejournal.com',
 'shaltay--boltay-livejournal-com.turbopages.org': 'shaltay-boltay.livejournal.com',
 'imit--omsu-livejournal-com.turbopages.org': 'imit-omsu.livejournal.com',
 'kolobashkinalex-livejournal-com.turbopages.org': 'kolobashkinalex.livejournal.com',
 '191966v-livejournal-com.turbopages.org': '191966v.livejournal.com',
 'prokatmsk-com.turbopages.org': 'prokatmsk.com',
 'jeka--jj-livejournal-com.turbopages.org': 'jeka-jj.livejournal.com',
 'forcopywriters-ru.turbopages.org': 'forcopywriters.ru',
 'proofreader--z-livejournal-com.turbopages.org': 'proofreader-z.livejournal.com',
 'sanvmf-ru.turbopages.org': 'sanvmf.ru',
 'photooniphone-ru.turbopages.org': 'photooniphone.ru',
 'womenitems-ru.turbopages.org': 'womenitems.ru',
 'beautypatrol-ru.turbopages.org': 'beautypatrol.ru',
 'xn----7sbaaajmeb9ccoirgi3bdng2dvk.turbopages.org': 'xn--7sbaaajmeb9ccoirgi3bdng2dvk',
 'tsa--tsa-livejournal-com.turbopages.org': 'tsa-tsa.livejournal.com',
 'ruslandya-livejournal-com.turbopages.org': 'ruslandya.livejournal.com',
 'alexeyskydan-livejournal-com.turbopages.org': 'alexeyskydan.livejournal.com',
 'gpvn-ru.turbopages.org': 'gpvn.ru',
 'russianbeauty-ru.turbopages.org': 'russianbeauty.ru',
 'superman2014-livejournal-com.turbopages.org': 'superman2014.livejournal.com',
 'progsva-ru.turbopages.org': 'progsva.ru',
 'zapiski--agronoma-ru.turbopages.org': 'zapiski-agronoma.ru',
 'mk--kaliningrad-ru.turbopages.org': 'mk-kaliningrad.ru',
 'pioneeraka-livejournal-com.turbopages.org': 'pioneeraka.livejournal.com',
 'vseon-com.turbopages.org': 'vseon.com',
 'billionnews-ru.turbopages.org': 'billionnews.ru',
 'marimbu-livejournal-com.turbopages.org': 'marimbu.livejournal.com',
 'el--magico-livejournal-com.turbopages.org': 'el-magico.livejournal.com',
 'mohnatyj-livejournal-com.turbopages.org': 'mohnatyj.livejournal.com',
 'rust1964-livejournal-com.turbopages.org': 'rust1964.livejournal.com',
 'victorsolkin-livejournal-com.turbopages.org': 'victorsolkin.livejournal.com',
 'venagid-ru.turbopages.org': 'venagid.ru',
 'seoforyou-livejournal-com.turbopages.org': 'seoforyou.livejournal.com',
 'favor--in--all-livejournal-com.turbopages.org': 'favor-in-all.livejournal.com',
 'tanjand-livejournal-com.turbopages.org': 'tanjand.livejournal.com',
 'romanovtoday-livejournal-com.turbopages.org': 'romanovtoday.livejournal.com',
 '63ru-livejournal-com.turbopages.org': '63ru.livejournal.com',
 'carry--kaeterry-livejournal-com.turbopages.org': 'carry-kaeterry.livejournal.com',
 'budylai-livejournal-com.turbopages.org': 'budylai.livejournal.com',
 'deadokey-livejournal-com.turbopages.org': 'deadokey.livejournal.com',
 'alik--morozov-livejournal-com.turbopages.org': 'alik-morozov.livejournal.com',
 'evstratov--and-livejournal-com.turbopages.org': 'evstratov-and.livejournal.com',
 'antno-livejournal-com.turbopages.org': 'antno.livejournal.com',
 'abrakadabra19-livejournal-com.turbopages.org': 'abrakadabra19.livejournal.com',
 'zap--online-ru.turbopages.org': 'zap-online.ru',
 'cryptos-tv.turbopages.org': 'cryptos.tv',
 'goldren-livejournal-com.turbopages.org': 'goldren.livejournal.com',
 'matkamed-ru.turbopages.org': 'matkamed.ru',
 'conti--group-ru.turbopages.org': 'conti-group.ru',
 'doktorredko-livejournal-com.turbopages.org': 'doktorredko.livejournal.com',
 'bootsector-livejournal-com.turbopages.org': 'bootsector.livejournal.com',
 'kudatumen-ru.turbopages.org': 'kudatumen.ru',
 'starmidgard-livejournal-com.turbopages.org': 'starmidgard.livejournal.com',
 '048-ua.turbopages.org': '048.ua',
 'civil--engineer-livejournal-com.turbopages.org': 'civil-engineer.livejournal.com',
 '5--razones-livejournal-com.turbopages.org': '5-razones.livejournal.com',
 'oscar1985-livejournal-com.turbopages.org': 'oscar1985.livejournal.com',
 'chumakin-livejournal-com.turbopages.org': 'chumakin.livejournal.com',
 'tyler78-livejournal-com.turbopages.org': 'tyler78.livejournal.com',
 'ru--bykov-livejournal-com.turbopages.org': 'ru-bykov.livejournal.com',
 'kopeysk-bezformata-com.turbopages.org': 'kopeysk.bezformata.com',
 'prostopasha1914-livejournal-com.turbopages.org': 'prostopasha1914.livejournal.com',
 'alexey--ap-livejournal-com.turbopages.org': 'alexey-ap.livejournal.com',
 'sferastroiki-ru.turbopages.org': 'sferastroiki.ru',
 'priamurmedia-ru.turbopages.org': 'priamurmedia.ru',
 'moscomeco-livejournal-com.turbopages.org': 'moscomeco.livejournal.com',
 'sergiy06-livejournal-com.turbopages.org': 'sergiy06.livejournal.com',
 'absurdman-livejournal-com.turbopages.org': 'absurdman.livejournal.com',
 'yurisokolov-livejournal-com.turbopages.org': 'yurisokolov.livejournal.com',
 'taila--75-livejournal-com.turbopages.org': 'taila-75.livejournal.com',
 'sp--kupavna-ru.turbopages.org': 'sp-kupavna.ru',
 'boxing78-ru.turbopages.org': 'boxing78.ru',
 'boxingblog-ru.turbopages.org': 'boxingblog.ru',
 'afk--arena-com.turbopages.org': 'afk-arena.com',
 'red--army--1917-livejournal-com.turbopages.org': 'red-army-1917.livejournal.com',
 'ru--healthlife-livejournal-com.turbopages.org': 'ru-healthlife.livejournal.com',
 'girevoy--fitness-livejournal-com.turbopages.org': 'girevoy-fitness.livejournal.com',
 'newslipetsk-ru.turbopages.org': 'newslipetsk.ru',
 'ru--aviation-livejournal-com.turbopages.org': 'ru-aviation.livejournal.com',
 'cesim-livejournal-com.turbopages.org': 'cesim.livejournal.com',
 'ufanewsrb-ru.turbopages.org': 'ufanewsrb.ru',
 'vvprohvatilov-livejournal-com.turbopages.org': 'vvprohvatilov.livejournal.com',
 'ryjla-livejournal-com.turbopages.org': 'ryjla.livejournal.com',
 'fotomanya-livejournal-com.turbopages.org': 'fotomanya.livejournal.com',
 'shkola30-livejournal-com.turbopages.org': 'shkola30.livejournal.com',
 'samodelki--n-ru.turbopages.org': 'samodelki-n.ru',
 'sembond-livejournal-com.turbopages.org': 'sembond.livejournal.com',
 'oldvladikavkaz-livejournal-com.turbopages.org': 'oldvladikavkaz.livejournal.com',
 'karpukhins-livejournal-com.turbopages.org': 'karpukhins.livejournal.com',
 'lex--kravetski-livejournal-com.turbopages.org': 'lex-kravetski.livejournal.com',
 'yolka--izh-livejournal-com.turbopages.org': 'yolka-izh.livejournal.com',
 'zakonometr-ru.turbopages.org': 'zakonometr.ru',
 'spitot-livejournal-com.turbopages.org': 'spitot.livejournal.com',
 'maga--gorec-livejournal-com.turbopages.org': 'maga-gorec.livejournal.com',
 'agrarnyisector-ru.turbopages.org': 'agrarnyisector.ru',
 'neihimoon-livejournal-com.turbopages.org': 'neihimoon.livejournal.com',
 'tumdo-livejournal-com.turbopages.org': 'tumdo.livejournal.com',
 'ps13-ru.turbopages.org': 'ps13.ru',
 'sanpros-ru.turbopages.org': 'sanpros.ru',
 'space--of--spice-livejournal-com.turbopages.org': 'space-of-spice.livejournal.com',
 'kireev--andrei-livejournal-com.turbopages.org': 'kireev-andrei.livejournal.com',
 'gorbutovich-livejournal-com.turbopages.org': 'gorbutovich.livejournal.com',
 'lunfa-livejournal-com.turbopages.org': 'lunfa.livejournal.com',
 'vseosmile-ru.turbopages.org': 'vseosmile.ru',
 'openheavens--spb-livejournal-com.turbopages.org': 'openheavens-spb.livejournal.com',
 'obplitke-ru.turbopages.org': 'obplitke.ru',
 'shakhty--media-ru.turbopages.org': 'shakhty-media.ru',
 'cooptech-ru.turbopages.org': 'cooptech.ru',
 'kino--tv--forum-ru.turbopages.org': 'kino-tv-forum.ru',
 'travelway-livejournal-com.turbopages.org': 'travelway.livejournal.com',
 'weacom-ru.turbopages.org': 'weacom.ru',
 'asia--compot-livejournal-com.turbopages.org': 'asia-compot.livejournal.com',
 'escportal-ru.turbopages.org': 'escportal.ru',
 'noginsk-bezformata-com.turbopages.org': 'noginsk.bezformata.com',
 'gt--models-ru.turbopages.org': 'gt-models.ru',
 'gribokbolezn-ru.turbopages.org': 'gribokbolezn.ru',
 'xn------gddpaqao7bwico5b1eze.turbopages.org': 'xn---gddpaqao7bwico5b1eze',
 'kindwolf76-livejournal-com.turbopages.org': 'kindwolf76.livejournal.com',
 'i--m--ho-livejournal-com.turbopages.org': 'i-m-ho.livejournal.com',
 'filosofka-ru.turbopages.org': 'filosofka.ru',
 'stop--fat-ru.turbopages.org': 'stop-fat.ru',
 'w--bodybuilding-ru.turbopages.org': 'w-bodybuilding.ru',
 'maxdohod-info.turbopages.org': 'maxdohod.info',
 'expo--sib-ru.turbopages.org': 'expo-sib.ru',
 'rank--top-ru.turbopages.org': 'rank-top.ru',
 'vcctv-ru.turbopages.org': 'vcctv.ru',
 'nikulochkin-livejournal-com.turbopages.org': 'nikulochkin.livejournal.com',
 'sub--cult-ru.turbopages.org': 'sub-cult.ru',
 'vpishma-bezformata-com.turbopages.org': 'vpishma.bezformata.com',
 'zonasg4-livejournal-com.turbopages.org': 'zonasg4.livejournal.com',
 'pozharanet-com.turbopages.org': 'pozharanet.com',
 'ua--bal-livejournal-com.turbopages.org': 'ua-bal.livejournal.com',
 'priazovstep-ru.turbopages.org': 'priazovstep.ru',
 'kranigruz-ru.turbopages.org': 'kranigruz.ru',
 'dengi--moskva-ru.turbopages.org': 'dengi-moskva.ru',
 'razvod--guru-com.turbopages.org': 'razvod-guru.com',
 'astrakhan-kp-ru.turbopages.org': 'astrakhan.kp.ru',
 'k2--3300-livejournal-com.turbopages.org': 'k2-3300.livejournal.com',
 'dimgo--lee-livejournal-com.turbopages.org': 'dimgo-lee.livejournal.com',
 'voogg-livejournal-com.turbopages.org': 'voogg.livejournal.com',
 'titaniao-livejournal-com.turbopages.org': 'titaniao.livejournal.com',
 'lechenie--tut-ru.turbopages.org': 'lechenie-tut.ru',
 'chita-tsargrad-tv.turbopages.org': 'chita.tsargrad.tv',
 'sharapov--m-livejournal-com.turbopages.org': 'sharapov-m.livejournal.com',
 'kamss-ru.turbopages.org': 'kamss.ru',
 'vsk--10-livejournal-com.turbopages.org': 'vsk-10.livejournal.com',
 'putnik--pro-livejournal-com.turbopages.org': 'putnik-pro.livejournal.com',
 'rostovnadonu-bezformata-com.turbopages.org': 'rostovnadonu.bezformata.com',
 'lobzik--azikv-livejournal-com.turbopages.org': 'lobzik-azikv.livejournal.com',
 'kakinfo-ru.turbopages.org': 'kakinfo.ru',
 'stydopedia-ru.turbopages.org': 'stydopedia.ru',
 'alphagifts-livejournal-com.turbopages.org': 'alphagifts.livejournal.com',
 'korolevriamo-ru.turbopages.org': 'korolevriamo.ru',
 'msk-live24-ru.turbopages.org': 'msk.live24.ru',
 'vosledoma-com.turbopages.org': 'vosledoma.com',
 'mysadiogorod-com.turbopages.org': 'mysadiogorod.com',
 'gidpodelok-ru.turbopages.org': 'gidpodelok.ru',
 'alpha--parenting-livejournal-com.turbopages.org': 'alpha-parenting.livejournal.com',
 'zlobnig-livejournal-com.turbopages.org': 'zlobnig.livejournal.com',
 'myfashion--diary-livejournal-com.turbopages.org': 'myfashion-diary.livejournal.com',
 'k--lvk-livejournal-com.turbopages.org': 'k-lvk.livejournal.com',
 'gogoled-ru.turbopages.org': 'gogoled.ru',
 'insideevs-ru.turbopages.org': 'insideevs.ru',
 'zilm-livejournal-com.turbopages.org': 'zilm.livejournal.com',
 'gateinbest-ru.turbopages.org': 'gateinbest.ru',
 'sushidom--lk-ru.turbopages.org': 'sushidom-lk.ru',
 'hitmanah-livejournal-com.turbopages.org': 'hitmanah.livejournal.com',
 'kemaclubru-livejournal-com.turbopages.org': 'kemaclubru.livejournal.com',
 'leds--test-ru.turbopages.org': 'leds-test.ru',
 'tvoyalady-com.turbopages.org': 'tvoyalady.com',
 'new--kodiaq-ru.turbopages.org': 'new-kodiaq.ru',
 'chel-prufy-ru.turbopages.org': 'chel.prufy.ru',
 'decorinfo-org.turbopages.org': 'decorinfo.org',
 'power--body-ru.turbopages.org': 'power-body.ru',
 'zdesvamnetut0-livejournal-com.turbopages.org': 'zdesvamnetut0.livejournal.com',
 'wildlife--ru-livejournal-com.turbopages.org': 'wildlife-ru.livejournal.com',
 'windows--ten-ru.turbopages.org': 'windows-ten.ru',
 'vosmarket-ru.turbopages.org': 'vosmarket.ru',
 'cal--len-livejournal-com.turbopages.org': 'cal-len.livejournal.com',
 'art--buz2011-livejournal-com.turbopages.org': 'art-buz2011.livejournal.com',
 'no--tuberculosis-ru.turbopages.org': 'no-tuberculosis.ru',
 'farmcraft-su.turbopages.org': 'farmcraft.su',
 'feniks--mebel-ru.turbopages.org': 'feniks-mebel.ru',
 'dorogoipodarok-livejournal-com.turbopages.org': 'dorogoipodarok.livejournal.com',
 'bloody--icon-livejournal-com.turbopages.org': 'bloody-icon.livejournal.com',
 'callmjoker-livejournal-com.turbopages.org': 'callmjoker.livejournal.com',
 'nordtravel-art.turbopages.org': 'nordtravel.art',
 'my--swiss-ru.turbopages.org': 'my-swiss.ru',
 'doroga--peremen-livejournal-com.turbopages.org': 'doroga-peremen.livejournal.com',
 'domodedovo-bezformata-com.turbopages.org': 'domodedovo.bezformata.com',
 'formusical-ru.turbopages.org': 'formusical.ru',
 'xn----7sbgxsbiiu4b.turbopages.org': 'xn--7sbgxsbiiu4b',
 'propianino-ru.turbopages.org': 'propianino.ru',
 'vdomodedovo-info.turbopages.org': 'vdomodedovo.info',
 'puzoterok-net.turbopages.org': 'puzoterok.net',
 'politichanka-livejournal-com.turbopages.org': 'politichanka.livejournal.com',
 'garage--ru-livejournal-com.turbopages.org': 'garage-ru.livejournal.com',
 'purmix-ru.turbopages.org': 'purmix.ru',
 'petrovich607-livejournal-com.turbopages.org': 'petrovich607.livejournal.com',
 'mari--marian-livejournal-com.turbopages.org': 'mari-marian.livejournal.com',
 'kitchen--tech-livejournal-com.turbopages.org': 'kitchen-tech.livejournal.com',
 'puristka--maga-livejournal-com.turbopages.org': 'puristka-maga.livejournal.com',
 'kartovedru-livejournal-com.turbopages.org': 'kartovedru.livejournal.com',
 'mknews-de.turbopages.org': 'mknews.de',
 'kenigtiger-livejournal-com.turbopages.org': 'kenigtiger.livejournal.com',
 '6element-livejournal-com.turbopages.org': '6element.livejournal.com',
 'gidotdelki-ru.turbopages.org': 'gidotdelki.ru',
 'olhanninen-livejournal-com.turbopages.org': 'olhanninen.livejournal.com',
 'studcity-livejournal-com.turbopages.org': 'studcity.livejournal.com',
 'mari--batinak-livejournal-com.turbopages.org': 'mari-batinak.livejournal.com',
 'russ5-livejournal-com.turbopages.org': 'russ5.livejournal.com',
 'solitaire17-livejournal-com.turbopages.org': 'solitaire17.livejournal.com',
 'zarlegko-ru.turbopages.org': 'zarlegko.ru',
 'sev--izm-livejournal-com.turbopages.org': 'sev-izm.livejournal.com',
 'cheboksari-bezformata-com.turbopages.org': 'cheboksari.bezformata.com',
 'zsbooka-livejournal-com.turbopages.org': 'zsbooka.livejournal.com',
 'bloha-ru.turbopages.org': 'bloha.ru',
 'greenchelman--3-livejournal-com.turbopages.org': 'greenchelman-3.livejournal.com',
 'guralyuk-livejournal-com.turbopages.org': 'guralyuk.livejournal.com',
 'rinadavis-livejournal-com.turbopages.org': 'rinadavis.livejournal.com',
 'welcome--belarus-ru.turbopages.org': 'welcome-belarus.ru',
 '2wars-ru.turbopages.org': '2wars.ru',
 'volandkovna-livejournal-com.turbopages.org': 'volandkovna.livejournal.com',
 'leg0ner-livejournal-com.turbopages.org': 'leg0ner.livejournal.com',
 'ser--stepanov-livejournal-com.turbopages.org': 'ser-stepanov.livejournal.com',
 'lugansk--lg--ua-livejournal-com.turbopages.org': 'lugansk-lg-ua.livejournal.com',
 'ru--chevron-livejournal-com.turbopages.org': 'ru-chevron.livejournal.com',
 'rnmelnikov-livejournal-com.turbopages.org': 'rnmelnikov.livejournal.com',
 'modmcpe-net.turbopages.org': 'modmcpe.net',
 '100minecraft-ru.turbopages.org': '100minecraft.ru',
 'aqualena-livejournal-com.turbopages.org': 'aqualena.livejournal.com',
 'sonnik-guru.turbopages.org': 'sonnik.guru',
 'firstzuzuchka-livejournal-com.turbopages.org': 'firstzuzuchka.livejournal.com',
 'pluk-livejournal-com.turbopages.org': 'pluk.livejournal.com',
 'lelavadee-livejournal-com.turbopages.org': 'lelavadee.livejournal.com',
 'dementievskiy-livejournal-com.turbopages.org': 'dementievskiy.livejournal.com',
 'piter--news-net.turbopages.org': 'piter-news.net',
 'direct--settings-ru.turbopages.org': 'direct-settings.ru',
 'babaev--elshad-livejournal-com.turbopages.org': 'babaev-elshad.livejournal.com',
 'xekate-livejournal-com.turbopages.org': 'xekate.livejournal.com',
 'punakettu-me.turbopages.org': 'punakettu.me',
 'alrina-livejournal-com.turbopages.org': 'alrina.livejournal.com',
 'lookmystyle-club.turbopages.org': 'lookmystyle.club',
 'kukina--kat-livejournal-com.turbopages.org': 'kukina-kat.livejournal.com',
 'nashenasledie-livejournal-com.turbopages.org': 'nashenasledie.livejournal.com',
 'slavyansknakubani-bezformata-com.turbopages.org': 'slavyansknakubani.bezformata.com',
 'natasha--lav-livejournal-com.turbopages.org': 'natasha-lav.livejournal.com',
 'oceanbank-ru.turbopages.org': 'oceanbank.ru',
 'tatianayelkina-livejournal-com.turbopages.org': 'tatianayelkina.livejournal.com',
 'si--foto-com.turbopages.org': 'si-foto.com',
 'anzor--m-livejournal-com.turbopages.org': 'anzor-m.livejournal.com',
 'artshot-ru.turbopages.org': 'artshot.ru',
 'transliteration-pro.turbopages.org': 'transliteration.pro',
 'tonygcoolioff-livejournal-com.turbopages.org': 'tonygcoolioff.livejournal.com',
 'odi--city-livejournal-com.turbopages.org': 'odi-city.livejournal.com',
 'fotopitera-livejournal-com.turbopages.org': 'fotopitera.livejournal.com',
 'flatinfo-ru.turbopages.org': 'flatinfo.ru',
 'tiina-livejournal-com.turbopages.org': 'tiina.livejournal.com',
 'blogomarathon-livejournal-com.turbopages.org': 'blogomarathon.livejournal.com',
 'var--veka-ru.turbopages.org': 'var-veka.ru',
 'women--startup-ru.turbopages.org': 'women-startup.ru',
 'nasedkin-livejournal-com.turbopages.org': 'nasedkin.livejournal.com',
 'stream--trader-ru.turbopages.org': 'stream-trader.ru',
 'moneymr-livejournal-com.turbopages.org': 'moneymr.livejournal.com',
 'maxnicol-livejournal-com.turbopages.org': 'maxnicol.livejournal.com',
 'magpie73-livejournal-com.turbopages.org': 'magpie73.livejournal.com',
 'zagatin--svv-livejournal-com.turbopages.org': 'zagatin-svv.livejournal.com',
 'windows--school-ru.turbopages.org': 'windows-school.ru',
 'natalba-livejournal-com.turbopages.org': 'natalba.livejournal.com',
 'fiona--amos-livejournal-com.turbopages.org': 'fiona-amos.livejournal.com',
 'forever--michael-livejournal-com.turbopages.org': 'forever-michael.livejournal.com',
 'olgeya--humayer-livejournal-com.turbopages.org': 'olgeya-humayer.livejournal.com',
 'giddizajn-ru.turbopages.org': 'giddizajn.ru',
 'stassavenkov-livejournal-com.turbopages.org': 'stassavenkov.livejournal.com',
 'gorod-ryazan-ru.turbopages.org': 'gorod.ryazan.ru',
 'galchi-livejournal-com.turbopages.org': 'galchi.livejournal.com',
 'samlady-livejournal-com.turbopages.org': 'samlady.livejournal.com',
 'coal--liza-livejournal-com.turbopages.org': 'coal-liza.livejournal.com',
 'prokrys-ru.turbopages.org': 'prokrys.ru',
 'warsh-livejournal-com.turbopages.org': 'warsh.livejournal.com',
 'dd--stroi-ru.turbopages.org': 'dd-stroi.ru',
 'pozdravlenie1-ru.turbopages.org': 'pozdravlenie1.ru',
 'promesyachnye-com.turbopages.org': 'promesyachnye.com',
 'denokan-livejournal-com.turbopages.org': 'denokan.livejournal.com',
 'deepshifting-ru.turbopages.org': 'deepshifting.ru',
 'vadhor-livejournal-com.turbopages.org': 'vadhor.livejournal.com',
 'lavnik-net.turbopages.org': 'lavnik.net',
 'lotos--n-livejournal-com.turbopages.org': 'lotos-n.livejournal.com',
 'yulsana-livejournal-com.turbopages.org': 'yulsana.livejournal.com',
 'kakhack-ru.turbopages.org': 'kakhack.ru',
 'nahnews-org.turbopages.org': 'nahnews.org',
 'wikistroi-ru.turbopages.org': 'wikistroi.ru',
 'zakupki-plus.turbopages.org': 'zakupki.plus',
 'whoyougle-ru.turbopages.org': 'whoyougle.ru',
 'omskzdes-ru.turbopages.org': 'omskzdes.ru',
 'glorioza3-livejournal-com.turbopages.org': 'glorioza3.livejournal.com',
 'am555-livejournal-com.turbopages.org': 'am555.livejournal.com',
 'zason-ru.turbopages.org': 'zason.ru',
 'olgarukenglaz-livejournal-com.turbopages.org': 'olgarukenglaz.livejournal.com',
 'my--first--time-livejournal-com.turbopages.org': 'my-first-time.livejournal.com',
 'grandt73-livejournal-com.turbopages.org': 'grandt73.livejournal.com',
 'judgesuhov-livejournal-com.turbopages.org': 'judgesuhov.livejournal.com',
 'ludmila--shilova-livejournal-com.turbopages.org': 'ludmila-shilova.livejournal.com',
 'barnaul-bezformata-com.turbopages.org': 'barnaul.bezformata.com',
 'xahopuk-livejournal-com.turbopages.org': 'xahopuk.livejournal.com',
 'bl1ch-livejournal-com.turbopages.org': 'bl1ch.livejournal.com',
 'dindin4-livejournal-com.turbopages.org': 'dindin4.livejournal.com',
 'nashpol-com.turbopages.org': 'nashpol.com',
 'darriuss-livejournal-com.turbopages.org': 'darriuss.livejournal.com',
 'wysedevice-ru.turbopages.org': 'wysedevice.ru',
 'displus-livejournal-com.turbopages.org': 'displus.livejournal.com',
 'ru--autostop-livejournal-com.turbopages.org': 'ru-autostop.livejournal.com',
 'logik--logik-livejournal-com.turbopages.org': 'logik-logik.livejournal.com',
 'dmit--chiginskiy-livejournal-com.turbopages.org': 'dmit-chiginskiy.livejournal.com',
 'roofs-club.turbopages.org': 'roofs.club',
 'praktibuh-ru.turbopages.org': 'praktibuh.ru',
 'mellanzh-livejournal-com.turbopages.org': 'mellanzh.livejournal.com',
 'olga74ru-livejournal-com.turbopages.org': 'olga74ru.livejournal.com',
 'holmogor-livejournal-com.turbopages.org': 'holmogor.livejournal.com',
 'hojja--nusreddin-livejournal-com.turbopages.org': 'hojja-nusreddin.livejournal.com',
 'obrazok-livejournal-com.turbopages.org': 'obrazok.livejournal.com',
 'perfectlyhappy-ru.turbopages.org': 'perfectlyhappy.ru',
 'oaudi-ru.turbopages.org': 'oaudi.ru',
 'danzeon-livejournal-com.turbopages.org': 'danzeon.livejournal.com',
 '13--century--ru-livejournal-com.turbopages.org': '13-century-ru.livejournal.com',
 'badschool-ru.turbopages.org': 'badschool.ru',
 'aleforion-livejournal-com.turbopages.org': 'aleforion.livejournal.com',
 'orion4-livejournal-com.turbopages.org': 'orion4.livejournal.com',
 'tati2705-livejournal-com.turbopages.org': 'tati2705.livejournal.com',
 'vyazanie-info.turbopages.org': 'vyazanie.info',
 'yarpdd72-ru.turbopages.org': 'yarpdd72.ru',
 'valuev-livejournal-com.turbopages.org': 'valuev.livejournal.com',
 'tanya--sum--mer-livejournal-com.turbopages.org': 'tanya-sum-mer.livejournal.com',
    'malinoviy--chay-livejournal-com.turbopages.org': 'malinoviy-chay.livejournal.com',
 'yuripasholok-livejournal-com.turbopages.org': 'yuripasholok.livejournal.com',
 'bestforwoman24-livejournal-com.turbopages.org': 'bestforwoman24.livejournal.com',
 'svetlyachok4-livejournal-com.turbopages.org': 'svetlyachok4.livejournal.com',
 'putidorogi--nn-ru.turbopages.org': 'putidorogi-nn.ru',
 'kam24-ru.turbopages.org': 'kam24.ru',
 'gornoaltaysk-bezformata-com.turbopages.org': 'gornoaltaysk.bezformata.com',
 'guitarissimo-ru.turbopages.org': 'guitarissimo.ru',
 'halal--eko-ru.turbopages.org': 'halal-eko.ru',
 'loader--blog-livejournal-com.turbopages.org': 'loader-blog.livejournal.com',
 'realhemul-livejournal-com.turbopages.org': 'realhemul.livejournal.com',
 'tomsk-bezformata-com.turbopages.org': 'tomsk.bezformata.com',
 'poliklinika73-ru.turbopages.org': 'poliklinika73.ru',
 'murzillo-livejournal-com.turbopages.org': 'murzillo.livejournal.com',
 'mojsrpski-org.turbopages.org': 'mojsrpski.org',
 'olly--ru-livejournal-com.turbopages.org': 'olly-ru.livejournal.com',
 'vesti--sevastopol-ru.turbopages.org': 'vesti-sevastopol.ru',
 'xranitelochaga-ru.turbopages.org': 'xranitelochaga.ru',
 'amygdally-livejournal-com.turbopages.org': 'amygdally.livejournal.com',
 'ribxoz-ru.turbopages.org': 'ribxoz.ru',
 'obliqo-ru.turbopages.org': 'obliqo.ru',
 'inesacipa-livejournal-com.turbopages.org': 'inesacipa.livejournal.com',
 'menobr-ru.turbopages.org': 'menobr.ru',
 'piterbu-ru.turbopages.org': 'piterbu.ru',
 'gardenidea-ru.turbopages.org': 'gardenidea.ru',
 'sipaevakaterina-livejournal-com.turbopages.org': 'sipaevakaterina.livejournal.com',
 'hr--director-ru.turbopages.org': 'hr-director.ru',
 'potokudach-ru.turbopages.org': 'potokudach.ru',
 'qrz-ru.turbopages.org': 'qrz.ru',
 'podmoscowe-ru.turbopages.org': 'podmoscowe.ru',
 'wapkin-livejournal-com.turbopages.org': 'wapkin.livejournal.com',
 'habinfo-ru.turbopages.org': 'habinfo.ru',
 'ballony-com-ua.turbopages.org': 'ballony.com.ua',
 'tekhnotop-ru.turbopages.org': 'tekhnotop.ru',
 'murmansk-aif-ru.turbopages.org': 'murmansk.aif.ru',
 'kusochek--raya-ru.turbopages.org': 'kusochek-raya.ru',
 'anthropology--ru-livejournal-com.turbopages.org': 'anthropology-ru.livejournal.com',
 'redikraed-ru.turbopages.org': 'redikraed.ru',
 'good--tips-pro.turbopages.org': 'good-tips.pro',
 'candoru-ru.turbopages.org': 'candoru.ru',
 'ouranopolis-livejournal-com.turbopages.org': 'ouranopolis.livejournal.com',
 'liberwill-livejournal-com.turbopages.org': 'liberwill.livejournal.com',
 'ufa-octagon-media.turbopages.org': 'ufa.octagon.media',
 'rama909-livejournal-com.turbopages.org': 'rama909.livejournal.com',
 'leysa-ru.turbopages.org': 'leysa.ru',
 'tandem--psy-ru.turbopages.org': 'tandem-psy.ru',
 'rukodelie-expert.turbopages.org': 'rukodelie.expert',
 'zdorovzhivi-ru.turbopages.org': 'zdorovzhivi.ru',
 'nuker82-livejournal-com.turbopages.org': 'nuker82.livejournal.com',
 'zhenziyou-livejournal-com.turbopages.org': 'zhenziyou.livejournal.com',
 'calendar-fontanka-ru.turbopages.org': 'calendar.fontanka.ru',
 'kudamoscow-ru.turbopages.org': 'kudamoscow.ru',
 'gramkin-livejournal-com.turbopages.org': 'gramkin.livejournal.com',
 'grila-ru.turbopages.org': 'grila.ru',
 'krutina-livejournal-com.turbopages.org': 'krutina.livejournal.com',
 'totemi--ru-livejournal-com.turbopages.org': 'totemi-ru.livejournal.com',
 'russianwife-livejournal-com.turbopages.org': 'russianwife.livejournal.com',
 'hitkiller-com.turbopages.org': 'hitkiller.com',
 'warspot-livejournal-com.turbopages.org': 'warspot.livejournal.com',
 'igor--ullman-livejournal-com.turbopages.org': 'igor-ullman.livejournal.com',
 'arkadiy1969-livejournal-com.turbopages.org': 'arkadiy1969.livejournal.com',
 'novosti--pro--kino-ru.turbopages.org': 'novosti-pro-kino.ru',
 'perfumero-livejournal-com.turbopages.org': 'perfumero.livejournal.com',
 'polkan--a-livejournal-com.turbopages.org': 'polkan-a.livejournal.com',
 'tskad-riamo-ru.turbopages.org': 'tskad.riamo.ru',
 'gormonyinfo-com.turbopages.org': 'gormonyinfo.com',
 'postcrossing--ua-livejournal-com.turbopages.org': 'postcrossing-ua.livejournal.com',
 'solee4ka-livejournal-com.turbopages.org': 'solee4ka.livejournal.com',
 '2klass-ru.turbopages.org': '2klass.ru',
 'gelen-pro.turbopages.org': 'gelen.pro',
 'planshet--info-ru.turbopages.org': 'planshet-info.ru',
 'kolyaseg-livejournal-com.turbopages.org': 'kolyaseg.livejournal.com',
 'uppetite-ru.turbopages.org': 'uppetite.ru',
 'lana--sator-livejournal-com.turbopages.org': 'lana-sator.livejournal.com',
 '056-ua.turbopages.org': '056.ua',
 'newsroyal-ru.turbopages.org': 'newsroyal.ru',
 'otkritayagazeta-livejournal-com.turbopages.org': 'otkritayagazeta.livejournal.com',
 'stagila-ru.turbopages.org': 'stagila.ru',
 'pod--potol-com.turbopages.org': 'pod-potol.com',
 'jabber777-livejournal-com.turbopages.org': 'jabber777.livejournal.com',
 'kos1chka-ru.turbopages.org': 'kos1chka.ru',
 'emi--grant-ru.turbopages.org': 'emi-grant.ru',
 'remont--samomy-ru.turbopages.org': 'remont-samomy.ru',
 'ruslanostashko-livejournal-com.turbopages.org': 'ruslanostashko.livejournal.com',
 'kratko--obo--vsem-ru.turbopages.org': 'kratko-obo-vsem.ru',
 'vladis--baxman-livejournal-com.turbopages.org': 'vladis-baxman.livejournal.com',
 'chyyr-livejournal-com.turbopages.org': 'chyyr.livejournal.com',
 'pteat-ru.turbopages.org': 'pteat.ru',
 'ak747-livejournal-com.turbopages.org': 'ak747.livejournal.com',
 'dmitriy--sivak-livejournal-com.turbopages.org': 'dmitriy-sivak.livejournal.com',
 'yuliazaiceva-livejournal-com.turbopages.org': 'yuliazaiceva.livejournal.com',
 'annastorm-livejournal-com.turbopages.org': 'annastorm.livejournal.com',
 'roni--elman-livejournal-com.turbopages.org': 'roni-elman.livejournal.com',
 'trinixy--ru-livejournal-com.turbopages.org': 'trinixy-ru.livejournal.com',
 'suzuki--club-ru.turbopages.org': 'suzuki-club.ru',
 'uralmines-ru.turbopages.org': 'uralmines.ru',
 'mrybakov-livejournal-com.turbopages.org': 'mrybakov.livejournal.com',
 'klinikanz-ru.turbopages.org': 'klinikanz.ru',
 'svpodlaskina-livejournal-com.turbopages.org': 'svpodlaskina.livejournal.com',
 'tajikistan24-com.turbopages.org': 'tajikistan24.com',
 'plastikaokon-livejournal-com.turbopages.org': 'plastikaokon.livejournal.com',
 'kardiodocs-ru.turbopages.org': 'kardiodocs.ru',
 'nataly--r-livejournal-com.turbopages.org': 'nataly-r.livejournal.com',
 'novosti-tj.turbopages.org': 'novosti.tj',
 'dubinkind-livejournal-com.turbopages.org': 'dubinkind.livejournal.com',
 'kirakin-livejournal-com.turbopages.org': 'kirakin.livejournal.com',
 'asia-msk-ru.turbopages.org': 'asia.msk.ru',
 'oknologiya-ru.turbopages.org': 'oknologiya.ru',
 'kot--sapog-livejournal-com.turbopages.org': 'kot-sapog.livejournal.com',
 'frallik-livejournal-com.turbopages.org': 'frallik.livejournal.com',
 'gorlis--gorsky-livejournal-com.turbopages.org': 'gorlis-gorsky.livejournal.com',
 'sdmeb-ru.turbopages.org': 'sdmeb.ru',
 'kosolapov-livejournal-com.turbopages.org': 'kosolapov.livejournal.com',
 'shvetsovmn-livejournal-com.turbopages.org': 'shvetsovmn.livejournal.com',
 'socprav-ru.turbopages.org': 'socprav.ru',
 '1gatta--felice-livejournal-com.turbopages.org': '1gatta-felice.livejournal.com',
 'pravdoryb-info.turbopages.org': 'pravdoryb.info',
 'polycarbonate--macro-ru.turbopages.org': 'polycarbonate-macro.ru',
 'uliyanovsk-bezformata-com.turbopages.org': 'uliyanovsk.bezformata.com',
 'dmugtasimov--pro-livejournal-com.turbopages.org': 'dmugtasimov-pro.livejournal.com',
 'st--tsourkan-livejournal-com.turbopages.org': 'st-tsourkan.livejournal.com',
 'turako-ru.turbopages.org': 'turako.ru',
 'tarkhagro-ru.turbopages.org': 'tarkhagro.ru',
 'baikal-mk-ru.turbopages.org': 'baikal.mk.ru',
 'arcanus-ru.turbopages.org': 'arcanus.ru',
 'mkivanovo-ru.turbopages.org': 'mkivanovo.ru',
 'stribuna-ru.turbopages.org': 'stribuna.ru',
 'malyutka--e-livejournal-com.turbopages.org': 'malyutka-e.livejournal.com',
 'nadomkrat-ru.turbopages.org': 'nadomkrat.ru',
 'skeletopora-ru.turbopages.org': 'skeletopora.ru',
 'mantsurov-livejournal-com.turbopages.org': 'mantsurov.livejournal.com',
 'dimanos-livejournal-com.turbopages.org': 'dimanos.livejournal.com',
 'alexey--fok-livejournal-com.turbopages.org': 'alexey-fok.livejournal.com',
 'tvdigitally-ru.turbopages.org': 'tvdigitally.ru',
 'bludakchr-ru.turbopages.org': 'bludakchr.ru',
 'afishapskov-ru.turbopages.org': 'afishapskov.ru',
 'russianemirates-com.turbopages.org': 'russianemirates.com',
 'arktal-livejournal-com.turbopages.org': 'arktal.livejournal.com',
 'poxmel-livejournal-com.turbopages.org': 'poxmel.livejournal.com',
 'igorol-livejournal-com.turbopages.org': 'igorol.livejournal.com',
 'otransformatore-ru.turbopages.org': 'otransformatore.ru',
 'tel--spb-ru.turbopages.org': 'tel-spb.ru',
 'ledsshop-ru.turbopages.org': 'ledsshop.ru',
 'vlad--vlz-livejournal-com.turbopages.org': 'vlad-vlz.livejournal.com',
 'polittechno-livejournal-com.turbopages.org': 'polittechno.livejournal.com',
 'vanda--rakitska-livejournal-com.turbopages.org': 'vanda-rakitska.livejournal.com',
 'nyama--nyama-ru.turbopages.org': 'nyama-nyama.ru',
 'digger-ru.turbopages.org': 'digger.ru',
 'marjulia-livejournal-com.turbopages.org': 'marjulia.livejournal.com',
 'metodika--rki-livejournal-com.turbopages.org': 'metodika-rki.livejournal.com',
 'sevprostor-livejournal-com.turbopages.org': 'sevprostor.livejournal.com',
 'eyeclinics-ru.turbopages.org': 'eyeclinics.ru',
 'rybnoe-net.turbopages.org': 'rybnoe.net',
 'kozhakoshek-com.turbopages.org': 'kozhakoshek.com',
 'in--moto-ru.turbopages.org': 'in-moto.ru',
 'marieclaire-ua.turbopages.org': 'marieclaire.ua',
 '1internettv-livejournal-com.turbopages.org': '1internettv.livejournal.com',
 'avn--msk-livejournal-com.turbopages.org': 'avn-msk.livejournal.com',
 'russboss-livejournal-com.turbopages.org': 'russboss.livejournal.com',
 'voprosov--net-ru.turbopages.org': 'voprosov-net.ru',
 'kirssdiana-livejournal-com.turbopages.org': 'kirssdiana.livejournal.com',
 'schelkunov-livejournal-com.turbopages.org': 'schelkunov.livejournal.com',
 'mikes68-livejournal-com.turbopages.org': 'mikes68.livejournal.com',
 'v0lk-ru.turbopages.org': 'v0lk.ru',
 'chevroletov-ru.turbopages.org': 'chevroletov.ru',
 'nikkto59-livejournal-com.turbopages.org': 'nikkto59.livejournal.com',
 'damir--hazrat-livejournal-com.turbopages.org': 'damir-hazrat.livejournal.com',
 'myroad-livejournal-com.turbopages.org': 'myroad.livejournal.com',
 'andrey58tom-livejournal-com.turbopages.org': 'andrey58tom.livejournal.com',
 'sterlitamak-bezformata-com.turbopages.org': 'sterlitamak.bezformata.com',
 'nathoncharova-livejournal-com.turbopages.org': 'nathoncharova.livejournal.com',
 'gmichailov-livejournal-com.turbopages.org': 'gmichailov.livejournal.com',
 'id77-livejournal-com.turbopages.org': 'id77.livejournal.com',
 'pl--yum-livejournal-com.turbopages.org': 'pl-yum.livejournal.com',
 'minakovas-livejournal-com.turbopages.org': 'minakovas.livejournal.com',
 'oleg--aliyev-livejournal-com.turbopages.org': 'oleg-aliyev.livejournal.com',
 'iskander--bel-livejournal-com.turbopages.org': 'iskander-bel.livejournal.com',
 'vsostrovski-livejournal-com.turbopages.org': 'vsostrovski.livejournal.com',
 'pogoworim-ru.turbopages.org': 'pogoworim.ru',
 'prohranenie-ru.turbopages.org': 'prohranenie.ru',
 'safari--in--africa-ru.turbopages.org': 'safari-in-africa.ru',
 'oksa--sun-livejournal-com.turbopages.org': 'oksa-sun.livejournal.com',
 'summercamp-ru.turbopages.org': 'summercamp.ru',
 '1akm-ru.turbopages.org': '1akm.ru',
 'arhistrazh-livejournal-com.turbopages.org': 'arhistrazh.livejournal.com',
 'chelyabinsk--news-net.turbopages.org': 'chelyabinsk-news.net',
 'fem--city-livejournal-com.turbopages.org': 'fem-city.livejournal.com',
 'interesnyefakty-org.turbopages.org': 'interesnyefakty.org',
 'silver--mania-ru.turbopages.org': 'silver-mania.ru',
 'math--nttt-ru.turbopages.org': 'math-nttt.ru',
 'mrminiver-livejournal-com.turbopages.org': 'mrminiver.livejournal.com',
 'ssokkill-livejournal-com.turbopages.org': 'ssokkill.livejournal.com',
 'friend-livejournal-com.turbopages.org': 'friend.livejournal.com',
 'life--godo-com.turbopages.org': 'life-godo.com',
 'british--cinema-livejournal-com.turbopages.org': 'british-cinema.livejournal.com',
 'samarasport-livejournal-com.turbopages.org': 'samarasport.livejournal.com',
 'culturfit-ru.turbopages.org': 'culturfit.ru',
 'hrenovina-livejournal-com.turbopages.org': 'hrenovina.livejournal.com',
 'seonews-ru.turbopages.org': 'seonews.ru',
 'malinovii--rai-ru.turbopages.org': 'malinovii-rai.ru',
 'pro--psycholog-ru.turbopages.org': 'pro-psycholog.ru',
 'mosautorent-ru.turbopages.org': 'mosautorent.ru',
 'andresanda-ru.turbopages.org': 'andresanda.ru',
 'remont2114-club.turbopages.org': 'remont2114.club',
 'serialson-ru.turbopages.org': 'serialson.ru',
 'netzhukam-ru.turbopages.org': 'netzhukam.ru',
 'rogveles-livejournal-com.turbopages.org': 'rogveles.livejournal.com',
 'oleka-livejournal-com.turbopages.org': 'oleka.livejournal.com',
 'upsya-livejournal-com.turbopages.org': 'upsya.livejournal.com',
 'tukoff-ru.turbopages.org': 'tukoff.ru',
 'domikdomovenka-ru.turbopages.org': 'domikdomovenka.ru',
 'ipoteka--nedvizhimost-ru.turbopages.org': 'ipoteka-nedvizhimost.ru',
 'perm-bezformata-com.turbopages.org': 'perm.bezformata.com',
 '210101-ru.turbopages.org': '210101.ru',
 'wine--friends-livejournal-com.turbopages.org': 'wine-friends.livejournal.com',
 'zelenka-ru.turbopages.org': 'zelenka.ru',
 'moscowteslaclub-ru.turbopages.org': 'moscowteslaclub.ru',
 'mahachkala-bezformata-com.turbopages.org': 'mahachkala.bezformata.com',
 'vkysnoemenu-com.turbopages.org': 'vkysnoemenu.com',
 'skolko--stoyat-ru.turbopages.org': 'skolko-stoyat.ru',
 'albert--lex-livejournal-com.turbopages.org': 'albert-lex.livejournal.com',
 'moviespotting-ru.turbopages.org': 'moviespotting.ru',
 'profibeer-ru.turbopages.org': 'profibeer.ru',
 'podelki--doma-ru.turbopages.org': 'podelki-doma.ru',
 'vazm6-livejournal-com.turbopages.org': 'vazm6.livejournal.com',
 'api--ano-livejournal-com.turbopages.org': 'api-ano.livejournal.com',
 'moto--idioto-livejournal-com.turbopages.org': 'moto-idioto.livejournal.com',
 'tonna--sv-ru.turbopages.org': 'tonna-sv.ru',
 'sotka-guru.turbopages.org': 'sotka.guru',
 'obag--store-ru.turbopages.org': 'obag-store.ru',
 '4lapki-com.turbopages.org': '4lapki.com',
 'waysi-ru.turbopages.org': 'waysi.ru',
 'rustemal-livejournal-com.turbopages.org': 'rustemal.livejournal.com',
 'urlaw03-ru.turbopages.org': 'urlaw03.ru',
 'vtop21-ru.turbopages.org': 'vtop21.ru',
 'fitslife-ru.turbopages.org': 'fitslife.ru',
 'sdstudia-ru.turbopages.org': 'sdstudia.ru',
 'dvenashka-ru.turbopages.org': 'dvenashka.ru',
 'amber--info-ru.turbopages.org': 'amber-info.ru',
 'crystal--wow-ru.turbopages.org': 'crystal-wow.ru',
 'windoffields-livejournal-com.turbopages.org': 'windoffields.livejournal.com',
 'hleby-ru.turbopages.org': 'hleby.ru',
 'ruargentina-com.turbopages.org': 'ruargentina.com',
 'mariana--aga-livejournal-com.turbopages.org': 'mariana-aga.livejournal.com',
 'svoimirukamivdome-ru.turbopages.org': 'svoimirukamivdome.ru',
 'stroystandart-info.turbopages.org': 'stroystandart.info',
 'dachnikinfo-ru.turbopages.org': 'dachnikinfo.ru',
 'hot--chocolate-ru.turbopages.org': 'hot-chocolate.ru',
 'rusinfo-info.turbopages.org': 'rusinfo.info',
 'recommendyou-ru.turbopages.org': 'recommendyou.ru',
 'mediabrest-by.turbopages.org': 'mediabrest.by',
 'techomagic-livejournal-com.turbopages.org': 'techomagic.livejournal.com',
 'victorinoxru-livejournal-com.turbopages.org': 'victorinoxru.livejournal.com',
 'america--europa-ru.turbopages.org': 'america-europa.ru',
 'mr--512-ru.turbopages.org': 'mr-512.ru',
 'electrod-biz.turbopages.org': 'electrod.biz',
 'xn----7sbj2aouf0a.turbopages.org': 'xn--7sbj2aouf0a',
 'kakdelateto-ru.turbopages.org': 'kakdelateto.ru',
 'eugenpr-livejournal-com.turbopages.org': 'eugenpr.livejournal.com',
 'coolhealth-ru.turbopages.org': 'coolhealth.ru',
 'smi58-ru.turbopages.org': 'smi58.ru',
 'pctune-ru.turbopages.org': 'pctune.ru',
 'prof--afv-livejournal-com.turbopages.org': 'prof-afv.livejournal.com',
 'vospitatelyu-ru.turbopages.org': 'vospitatelyu.ru',
 'aababy-ru.turbopages.org': 'aababy.ru',
 'zclub--caspian-ru.turbopages.org': 'zclub-caspian.ru',
 'nizzaev-livejournal-com.turbopages.org': 'nizzaev.livejournal.com',
 'nau--spb-livejournal-com.turbopages.org': 'nau-spb.livejournal.com',
 'boox--chakmool-livejournal-com.turbopages.org': 'boox-chakmool.livejournal.com',
 'vagro14-livejournal-com.turbopages.org': 'vagro14.livejournal.com',
 'rusisrael-livejournal-com.turbopages.org': 'rusisrael.livejournal.com',
 'volnodum-livejournal-com.turbopages.org': 'volnodum.livejournal.com',
 'tashman-livejournal-com.turbopages.org': 'tashman.livejournal.com',
 'skazkidlyadetey-ru.turbopages.org': 'skazkidlyadetey.ru',
 'happyb00k-livejournal-com.turbopages.org': 'happyb00k.livejournal.com',
 'vera--rb-livejournal-com.turbopages.org': 'vera-rb.livejournal.com',
 'kak--otdelat-ru.turbopages.org': 'kak-otdelat.ru',
 'mirax-space.turbopages.org': 'mirax.space',
 'znatoki-org.turbopages.org': 'znatoki.org',
 'stupino-bezformata-com.turbopages.org': 'stupino.bezformata.com',
 'kiaseltos2-ru.turbopages.org': 'kiaseltos2.ru',
 'air--dots-info.turbopages.org': 'air-dots.info',
 'rap--area-ru.turbopages.org': 'rap-area.ru',
 'kot--koshka-ru.turbopages.org': 'kot-koshka.ru',
 'stranger233-livejournal-com.turbopages.org': 'stranger233.livejournal.com',
 'mosaic--holiday-ru.turbopages.org': 'mosaic-holiday.ru',
 'gamerant-ru.turbopages.org': 'gamerant.ru',
 'romhacking-ru.turbopages.org': 'romhacking.ru',
 'reliz-org.turbopages.org': 'reliz.org',
 'botalisa-ru.turbopages.org': 'botalisa.ru',
 'florabank-ru.turbopages.org': 'florabank.ru',
 'vladimirtan-livejournal-com.turbopages.org': 'vladimirtan.livejournal.com',
 'vdvoem--s--toboi-ru.turbopages.org': 'vdvoem-s-toboi.ru',
 'banktvi-ru.turbopages.org': 'banktvi.ru',
 'englishmix-ru.turbopages.org': 'englishmix.ru',
 'ru--learnenglish-livejournal-com.turbopages.org': 'ru-learnenglish.livejournal.com',
 'gadget--bit-ru.turbopages.org': 'gadget-bit.ru',
 'belarus-kp-ru.turbopages.org': 'belarus.kp.ru',
 'mixatronik-ru.turbopages.org': 'mixatronik.ru',
 'trackt-ru.turbopages.org': 'trackt.ru',
 'house--animals-ru.turbopages.org': 'house-animals.ru',
 'realmassage-livejournal-com.turbopages.org': 'realmassage.livejournal.com',
 'pro--massage-livejournal-com.turbopages.org': 'pro-massage.livejournal.com',
 'lady--dalet-livejournal-com.turbopages.org': 'lady-dalet.livejournal.com',
 'akozmin--7-livejournal-com.turbopages.org': 'akozmin-7.livejournal.com',
 'sminews-ru.turbopages.org': 'sminews.ru',
 'joeck--12-livejournal-com.turbopages.org': 'joeck-12.livejournal.com',
 'si--sv-com.turbopages.org': 'si-sv.com',
 'vzorsgor-ru.turbopages.org': 'vzorsgor.ru',
 'madmundt-livejournal-com.turbopages.org': 'madmundt.livejournal.com',
 'steel--c-livejournal-com.turbopages.org': 'steel-c.livejournal.com',
 'versusb-ru.turbopages.org': 'versusb.ru',
 'karpunina-livejournal-com.turbopages.org': 'karpunina.livejournal.com',
 'stalist-livejournal-com.turbopages.org': 'stalist.livejournal.com',
 'dezzky--mat-livejournal-com.turbopages.org': 'dezzky-mat.livejournal.com',
 'edkin-livejournal-com.turbopages.org': 'edkin.livejournal.com',
 'anuta-livejournal-com.turbopages.org': 'anuta.livejournal.com',
 'propovyazki-ru.turbopages.org': 'propovyazki.ru',
 'portniha-com.turbopages.org': 'portniha.com',
 'arenophile-livejournal-com.turbopages.org': 'arenophile.livejournal.com',
 'fringilla--pinso-livejournal-com.turbopages.org': 'fringilla-pinso.livejournal.com',
 'semenalna-ru.turbopages.org': 'semenalna.ru',
 'buildip-ru.turbopages.org': 'buildip.ru',
 'dwarf--zero-livejournal-com.turbopages.org': 'dwarf-zero.livejournal.com',
 'ternovskiy-livejournal-com.turbopages.org': 'ternovskiy.livejournal.com',
 'urb--a-livejournal-com.turbopages.org': 'urb-a.livejournal.com',
 'belcrb-ru.turbopages.org': 'belcrb.ru',
 'diabetnorma-ru.turbopages.org': 'diabetnorma.ru',
 'skorogovo-ru.turbopages.org': 'skorogovo.ru',
 'istpzr-livejournal-com.turbopages.org': 'istpzr.livejournal.com',
 'dralexandra-livejournal-com.turbopages.org': 'dralexandra.livejournal.com',
 'ponosa-net.turbopages.org': 'ponosa.net',
 'oesolod-livejournal-com.turbopages.org': 'oesolod.livejournal.com',
 'spielkarten-livejournal-com.turbopages.org': 'spielkarten.livejournal.com',
 'helg82-livejournal-com.turbopages.org': 'helg82.livejournal.com',
 'radomysl-livejournal-com.turbopages.org': 'radomysl.livejournal.com',
 'taleb-ru.turbopages.org': 'taleb.ru',
 'audiobooks-su.turbopages.org': 'audiobooks.su',
 'ratings-7ya-ru.turbopages.org': 'ratings.7ya.ru',
 'xn----7sbrvkcgoh6a2frb.turbopages.org': 'xn--7sbrvkcgoh6a2frb',
 'kartarassrochky-ru.turbopages.org': 'kartarassrochky.ru',
 'irina--177rus-livejournal-com.turbopages.org': 'irina-177rus.livejournal.com',
 'moneyzz-ru.turbopages.org': 'moneyzz.ru',
 'otaxiyandex-ru.turbopages.org': 'otaxiyandex.ru',
 'unknownrussia-livejournal-com.turbopages.org': 'unknownrussia.livejournal.com',
 'best--3--0-livejournal-com.turbopages.org': 'best-3-0.livejournal.com',
 'darsik--dasha-livejournal-com.turbopages.org': 'darsik-dasha.livejournal.com',
 'gipertonija-ru.turbopages.org': 'gipertonija.ru',
 'medok--spb-ru.turbopages.org': 'medok-spb.ru',
 'moscowatelier-ru.turbopages.org': 'moscowatelier.ru',
 'olgasingularia-livejournal-com.turbopages.org': 'olgasingularia.livejournal.com',
 'vash--ng-ru.turbopages.org': 'vash-ng.ru',
 'ibsmoscow-ru.turbopages.org': 'ibsmoscow.ru',
 'dibora-site.turbopages.org': 'dibora.site',
 'narodovedia-ru.turbopages.org': 'narodovedia.ru',
 'fudao-livejournal-com.turbopages.org': 'fudao.livejournal.com',
 'my--pk-ru.turbopages.org': 'my-pk.ru',
 'chocolategid-ru.turbopages.org': 'chocolategid.ru',
 'artelyero-livejournal-com.turbopages.org': 'artelyero.livejournal.com',
 'donpress-ru.turbopages.org': 'donpress.ru',
 'infofisher-ru.turbopages.org': 'infofisher.ru',
 'bluesrock-livejournal-com.turbopages.org': 'bluesrock.livejournal.com',
 'volgograd-bezformata-com.turbopages.org': 'volgograd.bezformata.com',
 'add--hobby-ru.turbopages.org': 'add-hobby.ru',
 'nrtv-ru.turbopages.org': 'nrtv.ru',
 'igamer-biz.turbopages.org': 'igamer.biz',
 'lost--buddha-livejournal-com.turbopages.org': 'lost-buddha.livejournal.com',
 'shturman1922-livejournal-com.turbopages.org': 'shturman1922.livejournal.com',
 'alkomen-ru.turbopages.org': 'alkomen.ru',
 'dmitriyshimko-livejournal-com.turbopages.org': 'dmitriyshimko.livejournal.com',
 'systemlines-ru.turbopages.org': 'systemlines.ru',
 'avtogai-ru.turbopages.org': 'avtogai.ru',
 'stranniksenya-livejournal-com.turbopages.org': 'stranniksenya.livejournal.com',
 'vene--ro4ka-livejournal-com.turbopages.org': 'vene-ro4ka.livejournal.com',
 'soft--setup-ru.turbopages.org': 'soft-setup.ru',
 'tc--nabegovoy-ru.turbopages.org': 'tc-nabegovoy.ru',
 'sarvelo-ru.turbopages.org': 'sarvelo.ru',
 'sakhalin-aif-ru.turbopages.org': 'sakhalin.aif.ru',
 'spp--photo-ru.turbopages.org': 'spp-photo.ru',
 'turvopros-com.turbopages.org': 'turvopros.com',
 'traid24-com.turbopages.org': 'traid24.com',
 'vashbaks-ru.turbopages.org': 'vashbaks.ru',
 'voronezh-domostroyrf-ru.turbopages.org': 'voronezh.domostroyrf.ru',
 'automatz-livejournal-com.turbopages.org': 'automatz.livejournal.com',
 'artemspec-livejournal-com.turbopages.org': 'artemspec.livejournal.com',
 'mycollages-ru.turbopages.org': 'mycollages.ru',
 'herbana-world.turbopages.org': 'herbana.world',
 'separett-su.turbopages.org': 'separett.su',
 'volzsky-ru.turbopages.org': 'volzsky.ru',
 'web--rynok-ru.turbopages.org': 'web-rynok.ru',
 'tereshonok-livejournal-com.turbopages.org': 'tereshonok.livejournal.com',
 'prizivaut-ru.turbopages.org': 'prizivaut.ru',
 'kezling-ru.turbopages.org': 'kezling.ru',
 'v--stranstviyorg-livejournal-com.turbopages.org': 'v-stranstviyorg.livejournal.com',
 'kir--bor-livejournal-com.turbopages.org': 'kir-bor.livejournal.com',
 'cmotp10-livejournal-com.turbopages.org': 'cmotp10.livejournal.com',
 'islamunveiled-livejournal-com.turbopages.org': 'islamunveiled.livejournal.com',
 'jago52-livejournal-com.turbopages.org': 'jago52.livejournal.com',
 'lusia--lusi-livejournal-com.turbopages.org': 'lusia-lusi.livejournal.com',
 'biblecomua-livejournal-com.turbopages.org': 'biblecomua.livejournal.com',
 '4elovek--vtumane-livejournal-com.turbopages.org': '4elovek-vtumane.livejournal.com',
 'vseobislame-livejournal-com.turbopages.org': 'vseobislame.livejournal.com',
 'muranochka-livejournal-com.turbopages.org': 'muranochka.livejournal.com',
 'mislpronzaya-livejournal-com.turbopages.org': 'mislpronzaya.livejournal.com',
 'krugom--eda-livejournal-com.turbopages.org': 'krugom-eda.livejournal.com',
 'burgernews--ru-livejournal-com.turbopages.org': 'burgernews-ru.livejournal.com',
 'prodlenka-org.turbopages.org': 'prodlenka.org',
 'anqellika-livejournal-com.turbopages.org': 'anqellika.livejournal.com',
 'steam--helper-ru.turbopages.org': 'steam-helper.ru',
 'trampeador-livejournal-com.turbopages.org': 'trampeador.livejournal.com',
 'owkorr79-livejournal-com.turbopages.org': 'owkorr79.livejournal.com',
 'cotlix-com.turbopages.org': 'cotlix.com',
 'sayanagorsk-bezformata-com.turbopages.org': 'sayanagorsk.bezformata.com',
 'zomerm-livejournal-com.turbopages.org': 'zomerm.livejournal.com',
 'czeslaw--list-livejournal-com.turbopages.org': 'czeslaw-list.livejournal.com',
 'kado4nikov-livejournal-com.turbopages.org': 'kado4nikov.livejournal.com',
 'stockinfocus-ru.turbopages.org': 'stockinfocus.ru',
 'alex--leit-livejournal-com.turbopages.org': 'alex-leit.livejournal.com',
 'sobaki-guru.turbopages.org': 'sobaki.guru',
 'fermeragronom-ru.turbopages.org': 'fermeragronom.ru',
 'izbloka-com.turbopages.org': 'izbloka.com',
 'maminyzaboty-com.turbopages.org': 'maminyzaboty.com',
 '7kul-ru.turbopages.org': '7kul.ru',
 'general--ivanoff-livejournal-com.turbopages.org': 'general-ivanoff.livejournal.com',
 'cheslav--kara-livejournal-com.turbopages.org': 'cheslav-kara.livejournal.com',
 'saucepizza-ru.turbopages.org': 'saucepizza.ru',
 'glkey-ru.turbopages.org': 'glkey.ru',
 'koyrakh-livejournal-com.turbopages.org': 'koyrakh.livejournal.com',
 'mihael--uwoo-livejournal-com.turbopages.org': 'mihael-uwoo.livejournal.com',
 'minitraktor34-ru.turbopages.org': 'minitraktor34.ru',
 'kiauto-ru.turbopages.org': 'kiauto.ru',
 'pcask-ru.turbopages.org': 'pcask.ru',
 'analogi-net.turbopages.org': 'analogi.net',
 'dag-aif-ru.turbopages.org': 'dag.aif.ru',
 'senat--perm-livejournal-com.turbopages.org': 'senat-perm.livejournal.com',
 'oldeem-ru.turbopages.org': 'oldeem.ru',
 'mnogovoprosov-ru.turbopages.org': 'mnogovoprosov.ru',
 'psypedprofi-ru.turbopages.org': 'psypedprofi.ru',
 'all--events-ru.turbopages.org': 'all-events.ru',
 'cheshirrrko-livejournal-com.turbopages.org': 'cheshirrrko.livejournal.com',
 'sciencepop-ru.turbopages.org': 'sciencepop.ru',
 'kraevushka-livejournal-com.turbopages.org': 'kraevushka.livejournal.com',
 'milutkin-livejournal-com.turbopages.org': 'milutkin.livejournal.com',
 'podborkiserialov-ru.turbopages.org': 'podborkiserialov.ru',
 'pivopotam-livejournal-com.turbopages.org': 'pivopotam.livejournal.com',
 'donkraeved-livejournal-com.turbopages.org': 'donkraeved.livejournal.com',
 'kalmayur-livejournal-com.turbopages.org': 'kalmayur.livejournal.com',
 'terminator--slv-livejournal-com.turbopages.org': 'terminator-slv.livejournal.com',
 'aroundsew-ru.turbopages.org': 'aroundsew.ru',
 'persian--poetry-livejournal-com.turbopages.org': 'persian-poetry.livejournal.com',
 'aba--g-livejournal-com.turbopages.org': 'aba-g.livejournal.com',
 'greatiptv-ru.turbopages.org': 'greatiptv.ru',
 'hyperprapor-livejournal-com.turbopages.org': 'hyperprapor.livejournal.com',
 'yurovskiy-livejournal-com.turbopages.org': 'yurovskiy.livejournal.com',
 'urtica--monoica-livejournal-com.turbopages.org': 'urtica-monoica.livejournal.com',
 'padolski-livejournal-com.turbopages.org': 'padolski.livejournal.com',
 'egorkurguzov-livejournal-com.turbopages.org': 'egorkurguzov.livejournal.com',
 'moetatu-ru.turbopages.org': 'moetatu.ru',
 'kraeham-livejournal-com.turbopages.org': 'kraeham.livejournal.com',
 'tourist0515-livejournal-com.turbopages.org': 'tourist0515.livejournal.com',
 'mehrzod1988-livejournal-com.turbopages.org': 'mehrzod1988.livejournal.com',
 'stalkersafe-ru.turbopages.org': 'stalkersafe.ru',
 'gtn--pravda-ru.turbopages.org': 'gtn-pravda.ru',
 'kelmochka-ru.turbopages.org': 'kelmochka.ru',
 'teplota-guru.turbopages.org': 'teplota.guru',
 'rtiivaz-ru.turbopages.org': 'rtiivaz.ru',
 'ujnosahalinsk-bezformata-com.turbopages.org': 'ujnosahalinsk.bezformata.com',
 'irinamyzikina-livejournal-com.turbopages.org': 'irinamyzikina.livejournal.com',
 'bookpollen-livejournal-com.turbopages.org': 'bookpollen.livejournal.com',
 'val000-livejournal-com.turbopages.org': 'val000.livejournal.com',
 'nisorinki-net.turbopages.org': 'nisorinki.net',
 'patasak-livejournal-com.turbopages.org': 'patasak.livejournal.com',
 'bodibilding--free-ru.turbopages.org': 'bodibilding-free.ru',
 'designersfromrussia-ru.turbopages.org': 'designersfromrussia.ru',
 'krasnogorsk-bezformata-com.turbopages.org': 'krasnogorsk.bezformata.com',
 'johanajollygirl-livejournal-com.turbopages.org': 'johanajollygirl.livejournal.com',
 'tankist--sssr-livejournal-com.turbopages.org': 'tankist-sssr.livejournal.com',
 'pers--pektiva-livejournal-com.turbopages.org': 'pers-pektiva.livejournal.com',
 'tonmeta-com.turbopages.org': 'tonmeta.com',
 'dezona-ru.turbopages.org': 'dezona.ru',
 'kurjer-info.turbopages.org': 'kurjer.info',
 'batteryk-com.turbopages.org': 'batteryk.com',
 'brodjaga--it-ru.turbopages.org': 'brodjaga-it.ru',
 'jtele2-ru.turbopages.org': 'jtele2.ru',
 'cattur-ru.turbopages.org': 'cattur.ru',
 'jliza-ru.turbopages.org': 'jliza.ru',
 'mrs--emerald-livejournal-com.turbopages.org': 'mrs-emerald.livejournal.com',
 'virlad-livejournal-com.turbopages.org': 'virlad.livejournal.com',
 'pcbdesigner-ru.turbopages.org': 'pcbdesigner.ru',
 'nekrasivyihnet-ru.turbopages.org': 'nekrasivyihnet.ru',
 'skinali54-ru.turbopages.org': 'skinali54.ru',
 'silverwind3000-livejournal-com.turbopages.org': 'silverwind3000.livejournal.com',
 'pontokot-livejournal-com.turbopages.org': 'pontokot.livejournal.com',
 'protorsunova-livejournal-com.turbopages.org': 'protorsunova.livejournal.com',
 'natakholina-livejournal-com.turbopages.org': 'natakholina.livejournal.com',
 'u81-livejournal-com.turbopages.org': 'u81.livejournal.com',
 'horki-info.turbopages.org': 'horki.info',
 'koipark-livejournal-com.turbopages.org': 'koipark.livejournal.com',
 'dermatologinfo-ru.turbopages.org': 'dermatologinfo.ru',
 'obsleduem-net.turbopages.org': 'obsleduem.net',
 'russrybalka-ru.turbopages.org': 'russrybalka.ru',
 'riusv-livejournal-com.turbopages.org': 'riusv.livejournal.com',
 'vnuk--sapera-livejournal-com.turbopages.org': 'vnuk-sapera.livejournal.com',
 'smitsmitty-livejournal-com.turbopages.org': 'smitsmitty.livejournal.com',
 'iwtkl-livejournal-com.turbopages.org': 'iwtkl.livejournal.com',
 'nochnaya--ximera-livejournal-com.turbopages.org': 'nochnaya-ximera.livejournal.com',
 'parashutov-livejournal-com.turbopages.org': 'parashutov.livejournal.com',
 'hasid-livejournal-com.turbopages.org': 'hasid.livejournal.com',
 'rjssianin-livejournal-com.turbopages.org': 'rjssianin.livejournal.com',
 'nastyono4ka-livejournal-com.turbopages.org': 'nastyono4ka.livejournal.com',
 '120-su.turbopages.org': '120.su',
 'pushkino-tv.turbopages.org': 'pushkino.tv',
 'uspei-com.turbopages.org': 'uspei.com',
 'otrageniya-livejournal-com.turbopages.org': 'otrageniya.livejournal.com',
 'finna11-livejournal-com.turbopages.org': 'finna11.livejournal.com',
 'alina--sharapova-ru.turbopages.org': 'alina-sharapova.ru',
 'murmansk-ru.turbopages.org': 'murmansk.ru',
 'ru--museum70-livejournal-com.turbopages.org': 'ru-museum70.livejournal.com',
 'tttkkk-livejournal-com.turbopages.org': 'tttkkk.livejournal.com',
 'sly2m-livejournal-com.turbopages.org': 'sly2m.livejournal.com',
 'vitasite-ru.turbopages.org': 'vitasite.ru',
 'tomsk--time-ru.turbopages.org': 'tomsk-time.ru',
 'freedom-livejournal-com.turbopages.org': 'freedom.livejournal.com',
 'grozny-tv.turbopages.org': 'grozny.tv',
 'xn----ctbahfbr0atjzi7b.turbopages.org': 'xn--ctbahfbr0atjzi7b',
 'asebo-ru.turbopages.org': 'asebo.ru',
 'trueadviser-livejournal-com.turbopages.org': 'trueadviser.livejournal.com',
 'moscow-news--kmv-ru.turbopages.org': 'moscow.news-kmv.ru',
 'guy--for--buy-livejournal-com.turbopages.org': 'guy-for-buy.livejournal.com',
 'travel--secrets-ru.turbopages.org': 'travel-secrets.ru',
 'masha--blade-livejournal-com.turbopages.org': 'masha-blade.livejournal.com',
 'bmwservice-livejournal-com.turbopages.org': 'bmwservice.livejournal.com',
 'permjenergosbyt--lk-ru.turbopages.org': 'permjenergosbyt-lk.ru',
 'mebel--trade-su.turbopages.org': 'mebel-trade.su',
 'sdelatrykami-ru.turbopages.org': 'sdelatrykami.ru',
 'domovodstvo-livejournal-com.turbopages.org': 'domovodstvo.livejournal.com',
 'lady--biznes-ru.turbopages.org': 'lady-biznes.ru',
 'beauty--shmuty-livejournal-com.turbopages.org': 'beauty-shmuty.livejournal.com',
 'kanaev55-livejournal-com.turbopages.org': 'kanaev55.livejournal.com',
 'xoxu-ru.turbopages.org': 'xoxu.ru',
 'tolik--bars-livejournal-com.turbopages.org': 'tolik-bars.livejournal.com',
 'rrnews-ru.turbopages.org': 'rrnews.ru',
 '3drive-ru.turbopages.org': '3drive.ru',
 'simbpravoslavie-livejournal-com.turbopages.org': 'simbpravoslavie.livejournal.com',
 'animal--book-ru.turbopages.org': 'animal-book.ru',
 'vladimir-big--book--med-ru.turbopages.org': 'vladimir.big-book-med.ru',
 'totalexpo-ru.turbopages.org': 'totalexpo.ru',
 'dermatolog-guru.turbopages.org': 'dermatolog.guru',
 'primepress-by.turbopages.org': 'primepress.by',
 'szpilka-ru.turbopages.org': 'szpilka.ru',
 'mytopfinance-ru.turbopages.org': 'mytopfinance.ru',
 'smartmoney-today.turbopages.org': 'smartmoney.today',
 'ochg-ru.turbopages.org': 'ochg.ru',
 'ru--radio--electr-livejournal-com.turbopages.org': 'ru-radio-electr.livejournal.com',
 'd--r--agon-livejournal-com.turbopages.org': 'd-r-agon.livejournal.com',
 'dawahsister-livejournal-com.turbopages.org': 'dawahsister.livejournal.com',
 'omgubuntu-ru.turbopages.org': 'omgubuntu.ru',
 'iris--arctic-livejournal-com.turbopages.org': 'iris-arctic.livejournal.com',
 'myauu-livejournal-com.turbopages.org': 'myauu.livejournal.com',
 'luxvaz-ru.turbopages.org': 'luxvaz.ru',
 'blondzinko-livejournal-com.turbopages.org': 'blondzinko.livejournal.com',
 '2--oi--shans-livejournal-com.turbopages.org': '2-oi-shans.livejournal.com',
 'prohorov-livejournal-com.turbopages.org': 'prohorov.livejournal.com',
 'mk--novgorod-ru.turbopages.org': 'mk-novgorod.ru',
 'moblegends-ru.turbopages.org': 'moblegends.ru',
 'labinsk--remont-ru.turbopages.org': 'labinsk-remont.ru',
 'santech--info-ru.turbopages.org': 'santech-info.ru',
 'rostovsad-livejournal-com.turbopages.org': 'rostovsad.livejournal.com',
 'nemez--06-livejournal-com.turbopages.org': 'nemez-06.livejournal.com',
 'cpv-ru.turbopages.org': 'cpv.ru',
 'yourwriter-ru.turbopages.org': 'yourwriter.ru',
 'zapchasti--isuzu-ru.turbopages.org': 'zapchasti-isuzu.ru',
 'ktk-kz.turbopages.org': 'ktk.kz',
 'wow--lab-ru.turbopages.org': 'wow-lab.ru',
 'ginger--at--heart-livejournal-com.turbopages.org': 'ginger-at-heart.livejournal.com',
 'my--renault2-ru.turbopages.org': 'my-renault2.ru',
 'history--of--art-livejournal-com.turbopages.org': 'history-of-art.livejournal.com',
 'euro--royals-livejournal-com.turbopages.org': 'euro-royals.livejournal.com',
 'lavande--la-livejournal-com.turbopages.org': 'lavande-la.livejournal.com',
 'gosuslugi-livejournal-com.turbopages.org': 'gosuslugi.livejournal.com',
 'world--of--whisky-livejournal-com.turbopages.org': 'world-of-whisky.livejournal.com',
 'bluxer-livejournal-com.turbopages.org': 'bluxer.livejournal.com',
 '24-kz.turbopages.org': '24.kz',
 'dionika--club-ru.turbopages.org': 'dionika-club.ru',
 'nijnekamsk-bezformata-com.turbopages.org': 'nijnekamsk.bezformata.com',
 'zamok--garibaldi-livejournal-com.turbopages.org': 'zamok-garibaldi.livejournal.com',
 'fakty--o-ru.turbopages.org': 'fakty-o.ru',
 'vashnarkolog-com.turbopages.org': 'vashnarkolog.com',
 'my--calend-ru.turbopages.org': 'my-calend.ru',
 'lakhtacenter-livejournal-com.turbopages.org': 'lakhtacenter.livejournal.com',
 'ufa--news-net.turbopages.org': 'ufa-news.net',
 'avramenko--konst-livejournal-com.turbopages.org': 'avramenko-konst.livejournal.com',
 'bahadur--singh-livejournal-com.turbopages.org': 'bahadur-singh.livejournal.com',
 'masterfibre03-ru.turbopages.org': 'masterfibre03.ru',
 'baza--polimerov-ru.turbopages.org': 'baza-polimerov.ru',
 'don--katalan-livejournal-com.turbopages.org': 'don-katalan.livejournal.com',
 'q3d-livejournal-com.turbopages.org': 'q3d.livejournal.com',
 'kkoop-ru.turbopages.org': 'kkoop.ru',
 'str--albatros-livejournal-com.turbopages.org': 'str-albatros.livejournal.com',
 'kirjach-bezformata-com.turbopages.org': 'kirjach.bezformata.com',
 'aquaspecial-ru.turbopages.org': 'aquaspecial.ru',
 'polostudy-livejournal-com.turbopages.org': 'polostudy.livejournal.com',
 'dogs4you-ru.turbopages.org': 'dogs4you.ru',
 'prorenault2-ru.turbopages.org': 'prorenault2.ru',
 'prikol32-ru.turbopages.org': 'prikol32.ru',
 'tbdoc-ru.turbopages.org': 'tbdoc.ru',
 'lobnya-bezformata-com.turbopages.org': 'lobnya.bezformata.com',
 'vsedetstvo-ru.turbopages.org': 'vsedetstvo.ru',
 'zhivotnyeplanety-ru.turbopages.org': 'zhivotnyeplanety.ru',
 'arina--ross-livejournal-com.turbopages.org': 'arina-ross.livejournal.com',
 'altuhov-livejournal-com.turbopages.org': 'altuhov.livejournal.com',
 'vestitula-ru.turbopages.org': 'vestitula.ru',
 'samnamos-livejournal-com.turbopages.org': 'samnamos.livejournal.com',
 'postalovsky--a-livejournal-com.turbopages.org': 'postalovsky-a.livejournal.com',
 'enjoyenglish--blog-com.turbopages.org': 'enjoyenglish-blog.com',
 'dietdo-ru.turbopages.org': 'dietdo.ru',
 'nettravm-ru.turbopages.org': 'nettravm.ru',
 'vilgun-livejournal-com.turbopages.org': 'vilgun.livejournal.com',
 'post1-su.turbopages.org': 'post1.su',
 'svobodniy-bezformata-com.turbopages.org': 'svobodniy.bezformata.com',
 'alex--oleyni-livejournal-com.turbopages.org': 'alex-oleyni.livejournal.com',
 'zveri-guru.turbopages.org': 'zveri.guru',
 'schurik2-livejournal-com.turbopages.org': 'schurik2.livejournal.com',
 'projectzakon-ru.turbopages.org': 'projectzakon.ru',
 'tonalli-livejournal-com.turbopages.org': 'tonalli.livejournal.com',
 'renault--kaptur-ru.turbopages.org': 'renault-kaptur.ru',
 'kupi2metr-ru.turbopages.org': 'kupi2metr.ru',
 'xn----7sbfmq1ccckxc5f.turbopages.org': 'xn--7sbfmq1ccckxc5f',
 'ree4-livejournal-com.turbopages.org': 'ree4.livejournal.com',
 'ilya--kudriashov-livejournal-com.turbopages.org': 'ilya-kudriashov.livejournal.com',
 'vmikhailuk-livejournal-com.turbopages.org': 'vmikhailuk.livejournal.com',
 'suhib-livejournal-com.turbopages.org': 'suhib.livejournal.com',
 'frukti--yagodi-ru.turbopages.org': 'frukti-yagodi.ru',
 'nad--suetoi-livejournal-com.turbopages.org': 'nad-suetoi.livejournal.com',
 'natalvital-livejournal-com.turbopages.org': 'natalvital.livejournal.com',
 'clubmobilelegends-com.turbopages.org': 'clubmobilelegends.com',
 'vseanekdoty-ru.turbopages.org': 'vseanekdoty.ru',
 'factsinter-com.turbopages.org': 'factsinter.com',
 'denis--demakhin-livejournal-com.turbopages.org': 'denis-demakhin.livejournal.com',
 'gramopod-ru.turbopages.org': 'gramopod.ru',
 'olegradul-livejournal-com.turbopages.org': 'olegradul.livejournal.com',
 'hrapless-ru.turbopages.org': 'hrapless.ru',
 'onkuhnya-ru.turbopages.org': 'onkuhnya.ru',
 'newtimes-kz.turbopages.org': 'newtimes.kz',
 'yatzenko-livejournal-com.turbopages.org': 'yatzenko.livejournal.com',
 'bookflowers-ru.turbopages.org': 'bookflowers.ru',
 'yuliya212-livejournal-com.turbopages.org': 'yuliya212.livejournal.com',
 'yk24-ru.turbopages.org': 'yk24.ru',
 'ru--translate-livejournal-com.turbopages.org': 'ru-translate.livejournal.com',
 'knopka--122-livejournal-com.turbopages.org': 'knopka-122.livejournal.com',
 'sochi-mk-ru.turbopages.org': 'sochi.mk.ru',
 'bolshaya--byaka-livejournal-com.turbopages.org': 'bolshaya-byaka.livejournal.com',
 'davidpestretsov-livejournal-com.turbopages.org': 'davidpestretsov.livejournal.com',
 'mijgona-livejournal-com.turbopages.org': 'mijgona.livejournal.com',
 'xn------5cdc2cn9ajhbhki1a.turbopages.org': 'xn---5cdc2cn9ajhbhki1a',
 'nash--surgut-ru.turbopages.org': 'nash-surgut.ru',
 'mono--polist-livejournal-com.turbopages.org': 'mono-polist.livejournal.com',
 'hanush--jr-livejournal-com.turbopages.org': 'hanush-jr.livejournal.com',
 'sevabashirov-livejournal-com.turbopages.org': 'sevabashirov.livejournal.com',
 'vokrug3d-ru.turbopages.org': 'vokrug3d.ru',
 'stavropol-bezformata-com.turbopages.org': 'stavropol.bezformata.com',
 'traktorspec-ru.turbopages.org': 'traktorspec.ru',
 'taoberry-ru.turbopages.org': 'taoberry.ru',
 'izbavitsya--ot--vrediteley-ru.turbopages.org': 'izbavitsya-ot-vrediteley.ru',
 'divo--dacha-ru.turbopages.org': 'divo-dacha.ru',
 'tripchik-ru.turbopages.org': 'tripchik.ru',
 'mopera66-ru.turbopages.org': 'mopera66.ru',
 'bullfinch-livejournal-com.turbopages.org': 'bullfinch.livejournal.com',
 'qostanay-tv.turbopages.org': 'qostanay.tv',
 'recept--napitok-ru.turbopages.org': 'recept-napitok.ru',
 'noutboouk-ru.turbopages.org': 'noutboouk.ru',
 'allinweb-ru.turbopages.org': 'allinweb.ru',
 'kodola-livejournal-com.turbopages.org': 'kodola.livejournal.com',
 'tyumen--street-livejournal-com.turbopages.org': 'tyumen-street.livejournal.com',
 'one--versus--all-livejournal-com.turbopages.org': 'one-versus-all.livejournal.com',
 'uva-livejournal-com.turbopages.org': 'uva.livejournal.com',
 'krambambyly-livejournal-com.turbopages.org': 'krambambyly.livejournal.com',
 'ymora--net-livejournal-com.turbopages.org': 'ymora-net.livejournal.com',
 'prokaljan-ru.turbopages.org': 'prokaljan.ru',
 'goodfit-ru-com.turbopages.org': 'goodfit.ru.com',
 'spinyful-livejournal-com.turbopages.org': 'spinyful.livejournal.com',
 'neferjournal-livejournal-com.turbopages.org': 'neferjournal.livejournal.com',
 'wildcompany-livejournal-com.turbopages.org': 'wildcompany.livejournal.com',
 'nadezhda--0--0-livejournal-com.turbopages.org': 'nadezhda-0-0.livejournal.com',
 'don--katar-livejournal-com.turbopages.org': 'don-katar.livejournal.com',
 'polspec-com.turbopages.org': 'polspec.com',
 'ugra-mk-ru.turbopages.org': 'ugra.mk.ru',
 'kartofell-ru.turbopages.org': 'kartofell.ru',
 'izbrus-ru.turbopages.org': 'izbrus.ru',
 'levpadalko-livejournal-com.turbopages.org': 'levpadalko.livejournal.com',
 'klausnick-livejournal-com.turbopages.org': 'klausnick.livejournal.com',
 'bryansk-bezformata-com.turbopages.org': 'bryansk.bezformata.com',
 'zaggo-ru.turbopages.org': 'zaggo.ru',
 'popoffich-livejournal-com.turbopages.org': 'popoffich.livejournal.com',
 'podelkiexpert-ru.turbopages.org': 'podelkiexpert.ru',
 'apotapov777-livejournal-com.turbopages.org': 'apotapov777.livejournal.com',
 'yoshkarola-bezformata-com.turbopages.org': 'yoshkarola.bezformata.com',
 'kisitnikov-livejournal-com.turbopages.org': 'kisitnikov.livejournal.com',
 'vare4ka70-livejournal-com.turbopages.org': 'vare4ka70.livejournal.com',
 'elenatruhina-livejournal-com.turbopages.org': 'elenatruhina.livejournal.com',
 'porodykotov-ru.turbopages.org': 'porodykotov.ru',
 'popugaichik-livejournal-com.turbopages.org': 'popugaichik.livejournal.com',
 'okolostand--up-livejournal-com.turbopages.org': 'okolostand-up.livejournal.com',
 'zeir-ru.turbopages.org': 'zeir.ru',
 'gips--golova-livejournal-com.turbopages.org': 'gips-golova.livejournal.com',
 'thunderbreaker-livejournal-com.turbopages.org': 'thunderbreaker.livejournal.com',
 'evalinger-livejournal-com.turbopages.org': 'evalinger.livejournal.com',
 'voenternet-livejournal-com.turbopages.org': 'voenternet.livejournal.com',
 'cosharel-livejournal-com.turbopages.org': 'cosharel.livejournal.com',
 't--magazine-ru.turbopages.org': 't-magazine.ru',
 'yamal-aif-ru.turbopages.org': 'yamal.aif.ru',
 'edu-1sept-ru.turbopages.org': 'edu.1sept.ru',
 'diets-guru.turbopages.org': 'diets.guru',
 'stroynaya--zhizn-ru.turbopages.org': 'stroynaya-zhizn.ru',
 'zaryadka-guru.turbopages.org': 'zaryadka.guru',
 'xudenkaya-ru.turbopages.org': 'xudenkaya.ru',
 'mag-103-kz.turbopages.org': 'mag.103.kz',
 'myvegan--club-ru.turbopages.org': 'myvegan-club.ru',
 'snianna-ru.turbopages.org': 'snianna.ru',
 'spalnidizajn-ru.turbopages.org': 'spalnidizajn.ru',
 'evrica--taurica-livejournal-com.turbopages.org': 'evrica-taurica.livejournal.com',
 'brawlstars-softodrom-by.turbopages.org': 'brawlstars.softodrom.by',
 'ru-pickgamer-com.turbopages.org': 'ru.pickgamer.com',
 'korolevvlad-livejournal-com.turbopages.org': 'korolevvlad.livejournal.com',
 'kazlenta-kz.turbopages.org': 'kazlenta.kz',
 'astro--ru-ru.turbopages.org': 'astro-ru.ru',
 'yuliya--tarolog-livejournal-com.turbopages.org': 'yuliya-tarolog.livejournal.com',
 'missmarymorstan-livejournal-com.turbopages.org': 'missmarymorstan.livejournal.com',
 'vikasys-livejournal-com.turbopages.org': 'vikasys.livejournal.com',
 'ura--povara-ru.turbopages.org': 'ura-povara.ru',
 'velgal-livejournal-com.turbopages.org': 'velgal.livejournal.com',
 'sa--va--artpantry-livejournal-com.turbopages.org': 'sa-va-artpantry.livejournal.com',
 'chersonesos--sev-ru.turbopages.org': 'chersonesos-sev.ru',
 'sovenok101-livejournal-com.turbopages.org': 'sovenok101.livejournal.com',
 'camoe-livejournal-com.turbopages.org': 'camoe.livejournal.com',
 'hochu--dom-info.turbopages.org': 'hochu-dom.info',
 'barikripke-livejournal-com.turbopages.org': 'barikripke.livejournal.com',
 'inna--sharfik-livejournal-com.turbopages.org': 'inna-sharfik.livejournal.com',
 'zol--dol-livejournal-com.turbopages.org': 'zol-dol.livejournal.com',
 'ribler-ru.turbopages.org': 'ribler.ru',
 'e--ecolog-ru.turbopages.org': 'e-ecolog.ru',
 'auto--master-su.turbopages.org': 'auto-master.su',
 'nedorazumenie-livejournal-com.turbopages.org': 'nedorazumenie.livejournal.com',
 'puzyrzhelchnyj-ru.turbopages.org': 'puzyrzhelchnyj.ru',
 'habarovsk-bezformata-com.turbopages.org': 'habarovsk.bezformata.com',
 'blog-pagbac-ru.turbopages.org': 'blog.pagbac.ru',
 'pechatnick-com.turbopages.org': 'pechatnick.com',
 'kumon--deti-com.turbopages.org': 'kumon-deti.com',
 'detskiystih-ru.turbopages.org': 'detskiystih.ru',
 'ng72-ru.turbopages.org': 'ng72.ru',
 'gdevise-ru.turbopages.org': 'gdevise.ru',
 'dailymoscow-ru.turbopages.org': 'dailymoscow.ru',
 'druskeit-livejournal-com.turbopages.org': 'druskeit.livejournal.com',
 'krasnoturinsk-info.turbopages.org': 'krasnoturinsk.info',
 'sysadmin--note-ru.turbopages.org': 'sysadmin-note.ru',
 'zaurmag-ru.turbopages.org': 'zaurmag.ru',
 'little--lilia-livejournal-com.turbopages.org': 'little-lilia.livejournal.com',
 'grelectric-livejournal-com.turbopages.org': 'grelectric.livejournal.com',
 'recepty--s--foto-ru.turbopages.org': 'recepty-s-foto.ru',
 'neo--medical-ru.turbopages.org': 'neo-medical.ru',
 'alcozavr-com.turbopages.org': 'alcozavr.com',
 'komnatnyecvety-ru.turbopages.org': 'komnatnyecvety.ru',
 'ferrochrome-livejournal-com.turbopages.org': 'ferrochrome.livejournal.com',
 'pratina-livejournal-com.turbopages.org': 'pratina.livejournal.com',
 'moy--garazh-ru.turbopages.org': 'moy-garazh.ru',
 'strange--bob-livejournal-com.turbopages.org': 'strange-bob.livejournal.com',
 'alvasar--num-livejournal-com.turbopages.org': 'alvasar-num.livejournal.com',
 'lazystranger-livejournal-com.turbopages.org': 'lazystranger.livejournal.com',
 'kot--i--koshka-ru.turbopages.org': 'kot-i-koshka.ru',
 'shok--darvina-livejournal-com.turbopages.org': 'shok-darvina.livejournal.com',
 'flaska-ru.turbopages.org': 'flaska.ru',
 'sanchoroz-livejournal-com.turbopages.org': 'sanchoroz.livejournal.com',
 'irchi--k-livejournal-com.turbopages.org': 'irchi-k.livejournal.com',
 '143900-ru.turbopages.org': '143900.ru',
 'oficianty-com.turbopages.org': 'oficianty.com',
 'son--prisnilos-com.turbopages.org': 'son-prisnilos.com',
 'alex--sincov-livejournal-com.turbopages.org': 'alex-sincov.livejournal.com',
 'anonimusi-livejournal-com.turbopages.org': 'anonimusi.livejournal.com',
 'misharoby-livejournal-com.turbopages.org': 'misharoby.livejournal.com',
 'smartreality-ru.turbopages.org': 'smartreality.ru',
 'polnyi--privod-ru.turbopages.org': 'polnyi-privod.ru',
 'kuzbass--today-ru.turbopages.org': 'kuzbass-today.ru',
 'dixinews-ru.turbopages.org': 'dixinews.ru',
 'littlehobby-livejournal-com.turbopages.org': 'littlehobby.livejournal.com',
 'sd--terminal-ru.turbopages.org': 'sd-terminal.ru',
 'nails--class-ru.turbopages.org': 'nails-class.ru',
 'spb-dixinews-ru.turbopages.org': 'spb.dixinews.ru',
 'ksenya--in--past-livejournal-com.turbopages.org': 'ksenya-in-past.livejournal.com',
 'leon--rumata-livejournal-com.turbopages.org': 'leon-rumata.livejournal.com',
 'mkoinov-livejournal-com.turbopages.org': 'mkoinov.livejournal.com',
 'kovrov-big--book--med-ru.turbopages.org': 'kovrov.big-book-med.ru',
 'naukateh-ru.turbopages.org': 'naukateh.ru',
 'polonsil-ru.turbopages.org': 'polonsil.ru',
 'ru--harrypotter-livejournal-com.turbopages.org': 'ru-harrypotter.livejournal.com',
 'didacts-ru.turbopages.org': 'didacts.ru',
 'hunhuz71-livejournal-com.turbopages.org': 'hunhuz71.livejournal.com',
 'svetashlyakhta-livejournal-com.turbopages.org': 'svetashlyakhta.livejournal.com',
 'ru--japan-livejournal-com.turbopages.org': 'ru-japan.livejournal.com',
 'dimagrib-livejournal-com.turbopages.org': 'dimagrib.livejournal.com',
 'yushchuk-livejournal-com.turbopages.org': 'yushchuk.livejournal.com',
 'authoritynutrition-net.turbopages.org': 'authoritynutrition.net',
 'lovelyspain-ru.turbopages.org': 'lovelyspain.ru',
 'igor113-livejournal-com.turbopages.org': 'igor113.livejournal.com',
 'samoljot-livejournal-com.turbopages.org': 'samoljot.livejournal.com',
 'shadberry--game-livejournal-com.turbopages.org': 'shadberry-game.livejournal.com',
 'ravreba--makc-livejournal-com.turbopages.org': 'ravreba-makc.livejournal.com',
 'lvoropaeva-livejournal-com.turbopages.org': 'lvoropaeva.livejournal.com',
 'tverdyi--znak-livejournal-com.turbopages.org': 'tverdyi-znak.livejournal.com',
 'klasson-livejournal-com.turbopages.org': 'klasson.livejournal.com',
 'ataman--golovko-livejournal-com.turbopages.org': 'ataman-golovko.livejournal.com',
 'sadcrixivan-livejournal-com.turbopages.org': 'sadcrixivan.livejournal.com',
 'antonika30-livejournal-com.turbopages.org': 'antonika30.livejournal.com',
 'morseanen-livejournal-com.turbopages.org': 'morseanen.livejournal.com',
 'stariy--voin-livejournal-com.turbopages.org': 'stariy-voin.livejournal.com',
 'samsebeskazal-livejournal-com.turbopages.org': 'samsebeskazal.livejournal.com',
 'wod--1958-livejournal-com.turbopages.org': 'wod-1958.livejournal.com',
 'neyva--news-ru.turbopages.org': 'neyva-news.ru',
 'novokuzneck-bezformata-com.turbopages.org': 'novokuzneck.bezformata.com',
 'fanisovich-livejournal-com.turbopages.org': 'fanisovich.livejournal.com',
 'kultovoe--kino-livejournal-com.turbopages.org': 'kultovoe-kino.livejournal.com',
 'propiska-org.turbopages.org': 'propiska.org',
 'narkop-livejournal-com.turbopages.org': 'narkop.livejournal.com',
 'virtualshopping-livejournal-com.turbopages.org': 'virtualshopping.livejournal.com',
 'studio--snab-ru.turbopages.org': 'studio-snab.ru',
 'alisila-ru.turbopages.org': 'alisila.ru',
 'fragramator-livejournal-com.turbopages.org': 'fragramator.livejournal.com',
 'elektrobiz-ru.turbopages.org': 'elektrobiz.ru',
 'gorlenka-livejournal-com.turbopages.org': 'gorlenka.livejournal.com',
 'icitata-ru.turbopages.org': 'icitata.ru',
 'nicolaitroitsky-livejournal-com.turbopages.org': 'nicolaitroitsky.livejournal.com',
 'collarmaster-livejournal-com.turbopages.org': 'collarmaster.livejournal.com',
 'rudy--ogon-livejournal-com.turbopages.org': 'rudy-ogon.livejournal.com',
 'hooka--mimi-ru.turbopages.org': 'hooka-mimi.ru',
 'prostoi--recept-ru.turbopages.org': 'prostoi-recept.ru',
 'gelendjik-bezformata-com.turbopages.org': 'gelendjik.bezformata.com',
 'masterkin-ru.turbopages.org': 'masterkin.ru',
 'podmasterij-ru.turbopages.org': 'podmasterij.ru',
 'afro--kosu-livejournal-com.turbopages.org': 'afro-kosu.livejournal.com',
 'jeteraconte-livejournal-com.turbopages.org': 'jeteraconte.livejournal.com',
 'vision--health-ru.turbopages.org': 'vision-health.ru',
 'eaquilla-livejournal-com.turbopages.org': 'eaquilla.livejournal.com',
 'priangine-ru.turbopages.org': 'priangine.ru',
 'windows10all-ru.turbopages.org': 'windows10all.ru',
 'domastroika-com.turbopages.org': 'domastroika.com',
 'dobvesti-ru.turbopages.org': 'dobvesti.ru',
 'lecheniedetej-ru.turbopages.org': 'lecheniedetej.ru',
 'sovetmame-online.turbopages.org': 'sovetmame.online',
 'aktinfo-com.turbopages.org': 'aktinfo.com',
 'interesportal-ru.turbopages.org': 'interesportal.ru',
 'fishingjournal-ru.turbopages.org': 'fishingjournal.ru',
 'lenoblast--pohod-livejournal-com.turbopages.org': 'lenoblast-pohod.livejournal.com',
 'jeanix-livejournal-com.turbopages.org': 'jeanix.livejournal.com',
 'pastuh83-livejournal-com.turbopages.org': 'pastuh83.livejournal.com',
 'starp-livejournal-com.turbopages.org': 'starp.livejournal.com',
 'sovietdetstvo-livejournal-com.turbopages.org': 'sovietdetstvo.livejournal.com',
 'dervishv-livejournal-com.turbopages.org': 'dervishv.livejournal.com',
 'vasiliy--okochka-livejournal-com.turbopages.org': 'vasiliy-okochka.livejournal.com',
 'becky--sharpe-livejournal-com.turbopages.org': 'becky-sharpe.livejournal.com',
 'proactions-ru.turbopages.org': 'proactions.ru',
 'mirsadovodnik-ru.turbopages.org': 'mirsadovodnik.ru',
 'khvkedr-ru.turbopages.org': 'khvkedr.ru',
 'yagoda-guru.turbopages.org': 'yagoda.guru',
 'vatchenko-livejournal-com.turbopages.org': 'vatchenko.livejournal.com',
 'orden--bezdna-livejournal-com.turbopages.org': 'orden-bezdna.livejournal.com',
 'tut-expert.turbopages.org': 'tut.expert',
 'theofilakt-livejournal-com.turbopages.org': 'theofilakt.livejournal.com',
 'airguide-livejournal-com.turbopages.org': 'airguide.livejournal.com',
 'in--power-ru.turbopages.org': 'in-power.ru',
 'crypto--fox-ru.turbopages.org': 'crypto-fox.ru',
 'miriam--shopping-livejournal-com.turbopages.org': 'miriam-shopping.livejournal.com',
 'newsru-co-il.turbopages.org': 'newsru.co.il',
 'av--petrov-livejournal-com.turbopages.org': 'av-petrov.livejournal.com',
 'beslan-bezformata-com.turbopages.org': 'beslan.bezformata.com',
 'prekrasny--mir-ru.turbopages.org': 'prekrasny-mir.ru',
 'r76-su.turbopages.org': 'r76.su',
 'turpiter-ru.turbopages.org': 'turpiter.ru',
 '4gazon-ru.turbopages.org': '4gazon.ru',
 'ru--birds-livejournal-com.turbopages.org': 'ru-birds.livejournal.com',
 'gazetapik-ru.turbopages.org': 'gazetapik.ru',
 'alionushka1-livejournal-com.turbopages.org': 'alionushka1.livejournal.com',
 'gbuz--stoma-ru.turbopages.org': 'gbuz-stoma.ru',
 'abkhazski--paren-livejournal-com.turbopages.org': 'abkhazski-paren.livejournal.com',
 'vicvek-livejournal-com.turbopages.org': 'vicvek.livejournal.com',
 'mathematics--tests-com.turbopages.org': 'mathematics-tests.com',
 'kazandoctor-ru.turbopages.org': 'kazandoctor.ru',
 'i--sergeev-livejournal-com.turbopages.org': 'i-sergeev.livejournal.com',
 't--itanium-livejournal-com.turbopages.org': 't-itanium.livejournal.com',
 'vkysnayakyxnya-ru.turbopages.org': 'vkysnayakyxnya.ru',
 'fitnessbraslet-com.turbopages.org': 'fitnessbraslet.com',
 'aleks--gruzdev-livejournal-com.turbopages.org': 'aleks-gruzdev.livejournal.com',
 'kalivod-livejournal-com.turbopages.org': 'kalivod.livejournal.com',
 'stoki-guru.turbopages.org': 'stoki.guru',
 '7odezhek-livejournal-com.turbopages.org': '7odezhek.livejournal.com',
 'little--chudo-ru.turbopages.org': 'little-chudo.ru',
 'sunsinta-livejournal-com.turbopages.org': 'sunsinta.livejournal.com',
 'porosolka--balt-livejournal-com.turbopages.org': 'porosolka-balt.livejournal.com',
 'vine--gret24-ru.turbopages.org': 'vine-gret24.ru',
 'building--plot-ru.turbopages.org': 'building-plot.ru',
 'calmandveggi-ru.turbopages.org': 'calmandveggi.ru',
 'muzhik--v--dome-ru.turbopages.org': 'muzhik-v-dome.ru',
 'nonano-ru.turbopages.org': 'nonano.ru',
 '9oodw1n-livejournal-com.turbopages.org': '9oodw1n.livejournal.com',
 '4pda-tech.turbopages.org': '4pda.tech',
 'fillum-livejournal-com.turbopages.org': 'fillum.livejournal.com',
 'rwlib-net.turbopages.org': 'rwlib.net',
 'licbook-ru.turbopages.org': 'licbook.ru',
 'otevalm-livejournal-com.turbopages.org': 'otevalm.livejournal.com',
 'moi--stroki-livejournal-com.turbopages.org': 'moi-stroki.livejournal.com',
 'fishingsblog-ru.turbopages.org': 'fishingsblog.ru',
 'avtomat--kx-livejournal-com.turbopages.org': 'avtomat-kx.livejournal.com',
 'bagullnik-livejournal-com.turbopages.org': 'bagullnik.livejournal.com',
 'spinneres-ru.turbopages.org': 'spinneres.ru',
 'komakplat-ru.turbopages.org': 'komakplat.ru',
 'kot--begemott-livejournal-com.turbopages.org': 'kot-begemott.livejournal.com',
 'shabanov--f-livejournal-com.turbopages.org': 'shabanov-f.livejournal.com',
 'olgasam009-livejournal-com.turbopages.org': 'olgasam009.livejournal.com',
 'bogdan--63-livejournal-com.turbopages.org': 'bogdan-63.livejournal.com',
 'albert--motsar-livejournal-com.turbopages.org': 'albert-motsar.livejournal.com',
 'warhistory-livejournal-com.turbopages.org': 'warhistory.livejournal.com',
 'corporatelie-livejournal-com.turbopages.org': 'corporatelie.livejournal.com',
 'kursk-bezformata-com.turbopages.org': 'kursk.bezformata.com',
 'gena--kr-livejournal-com.turbopages.org': 'gena-kr.livejournal.com',
 'pereklichka-livejournal-com.turbopages.org': 'pereklichka.livejournal.com',
 'andvari5-livejournal-com.turbopages.org': 'andvari5.livejournal.com',
 'skif--tag-livejournal-com.turbopages.org': 'skif-tag.livejournal.com',
 'ulia--g-livejournal-com.turbopages.org': 'ulia-g.livejournal.com',
 'i--algida-livejournal-com.turbopages.org': 'i-algida.livejournal.com',
 'padunskiy-livejournal-com.turbopages.org': 'padunskiy.livejournal.com',
 'towmater--76-livejournal-com.turbopages.org': 'towmater-76.livejournal.com',
 'tosno--vestnik-ru.turbopages.org': 'tosno-vestnik.ru',
 'falyosa-livejournal-com.turbopages.org': 'falyosa.livejournal.com',
 'kuba-livejournal-com.turbopages.org': 'kuba.livejournal.com',
 'mikat75-livejournal-com.turbopages.org': 'mikat75.livejournal.com',
 'noel--17-livejournal-com.turbopages.org': 'noel-17.livejournal.com',
 'tokblog-ru.turbopages.org': 'tokblog.ru',
 'help--gadget-ru.turbopages.org': 'help-gadget.ru',
 'esovetnik-ru.turbopages.org': 'esovetnik.ru',
 'murom-bezformata-com.turbopages.org': 'murom.bezformata.com',
 'archistorik-livejournal-com.turbopages.org': 'archistorik.livejournal.com',
 'dedugan530-livejournal-com.turbopages.org': 'dedugan530.livejournal.com',
 'xn----ctbjkjyhmqwq.turbopages.org': 'xn--ctbjkjyhmqwq',
 'velikieluki-bezformata-com.turbopages.org': 'velikieluki.bezformata.com',
 'sestra--milo-livejournal-com.turbopages.org': 'sestra-milo.livejournal.com',
 'zoo--farm-ru.turbopages.org': 'zoo-farm.ru',
 'samvguvt--4level-livejournal-com.turbopages.org': 'samvguvt-4level.livejournal.com',
 'poliestera-ru.turbopages.org': 'poliestera.ru',
 'carfactum-ru.turbopages.org': 'carfactum.ru',
 'mylektsii-su.turbopages.org': 'mylektsii.su',
 'diabetiya-ru.turbopages.org': 'diabetiya.ru',
 'etalon73-ru.turbopages.org': 'etalon73.ru',
 'vyazma-bezformata-com.turbopages.org': 'vyazma.bezformata.com',
 'autoiwc-ru.turbopages.org': 'autoiwc.ru',
 'dlebol-ru.turbopages.org': 'dlebol.ru',
 'gala--gala15-livejournal-com.turbopages.org': 'gala-gala15.livejournal.com',
 'youvteme-online.turbopages.org': 'youvteme.online',
 'thomas--cranmer-livejournal-com.turbopages.org': 'thomas-cranmer.livejournal.com',
 'iskateligr-ru.turbopages.org': 'iskateligr.ru',
 'brain--food-ru.turbopages.org': 'brain-food.ru',
 'moe--lipetsk-ru.turbopages.org': 'moe-lipetsk.ru',
 'denis--balin-livejournal-com.turbopages.org': 'denis-balin.livejournal.com',
 'chispa1707-livejournal-com.turbopages.org': 'chispa1707.livejournal.com',
 'tvrepair-ru.turbopages.org': 'tvrepair.ru',
 'stodvach-livejournal-com.turbopages.org': 'stodvach.livejournal.com',
 'obritye-ru.turbopages.org': 'obritye.ru',
 'osp--sakhalin-ru.turbopages.org': 'osp-sakhalin.ru',
 'ppc-world.turbopages.org': 'ppc.world',
 'chevalry-livejournal-com.turbopages.org': 'chevalry.livejournal.com',
 'sokura-livejournal-com.turbopages.org': 'sokura.livejournal.com',
 'idaten-ru.turbopages.org': 'idaten.ru',
 'army--reporter-livejournal-com.turbopages.org': 'army-reporter.livejournal.com',
 'tromentano-livejournal-com.turbopages.org': 'tromentano.livejournal.com',
 'offroadrest-ru.turbopages.org': 'offroadrest.ru',
 'kogda-info.turbopages.org': 'kogda.info',
 'samvsestroy-ru.turbopages.org': 'samvsestroy.ru',
 'nomisma-livejournal-com.turbopages.org': 'nomisma.livejournal.com',
 'kzn-kudago-com.turbopages.org': 'kzn.kudago.com',
 'biser--mania-ru.turbopages.org': 'biser-mania.ru',
 'mir--sousov-ru.turbopages.org': 'mir-sousov.ru',
 'lex--sorokin-livejournal-com.turbopages.org': 'lex-sorokin.livejournal.com',
 'aleksey--delfinn-livejournal-com.turbopages.org': 'aleksey-delfinn.livejournal.com',
 'grey--croco-livejournal-com.turbopages.org': 'grey-croco.livejournal.com',
 'mebeltrends-ru.turbopages.org': 'mebeltrends.ru',
 'auto--electronica-ru.turbopages.org': 'auto-electronica.ru',
 'ru--pelevin-livejournal-com.turbopages.org': 'ru-pelevin.livejournal.com',
 'hyundai--getz-net.turbopages.org': 'hyundai-getz.net',
 'redfoxday-ru.turbopages.org': 'redfoxday.ru',
 'agroved-org.turbopages.org': 'agroved.org',
 'lx--photos-livejournal-com.turbopages.org': 'lx-photos.livejournal.com',
 'avtozakony-ru.turbopages.org': 'avtozakony.ru',
 'hyek-livejournal-com.turbopages.org': 'hyek.livejournal.com',
 'sudarshana-ru.turbopages.org': 'sudarshana.ru',
 'alisa1lisa-livejournal-com.turbopages.org': 'alisa1lisa.livejournal.com',
 'nickfilin-livejournal-com.turbopages.org': 'nickfilin.livejournal.com',
 'kachok--alyosha-livejournal-com.turbopages.org': 'kachok-alyosha.livejournal.com',
 'flamingovv-livejournal-com.turbopages.org': 'flamingovv.livejournal.com',
 'toxica666-livejournal-com.turbopages.org': 'toxica666.livejournal.com',
 'werewolf0001-livejournal-com.turbopages.org': 'werewolf0001.livejournal.com',
 'pbs990-livejournal-com.turbopages.org': 'pbs990.livejournal.com',
 'dvigatels-ru.turbopages.org': 'dvigatels.ru',
 'strannikdori-livejournal-com.turbopages.org': 'strannikdori.livejournal.com',
    'bealiveinlove-livejournal-com.turbopages.org': 'bealiveinlove.livejournal.com',
 'kuban-rbc-ru.turbopages.org': 'kuban.rbc.ru',
 'beznarostov-ru.turbopages.org': 'beznarostov.ru',
 'wildfauna-ru.turbopages.org': 'wildfauna.ru',
 'trk7-ru.turbopages.org': 'trk7.ru',
 'mcmag-ru.turbopages.org': 'mcmag.ru',
 'pilip--pilipich-livejournal-com.turbopages.org': 'pilip-pilipich.livejournal.com',
 'zabrab75-ru.turbopages.org': 'zabrab75.ru',
 'uvarov0-livejournal-com.turbopages.org': 'uvarov0.livejournal.com',
 'shelehov-irk-today.turbopages.org': 'shelehov.irk.today',
 'mfc-zone.turbopages.org': 'mfc.zone',
 'b2bmaster-ru.turbopages.org': 'b2bmaster.ru',
 'podushka-net.turbopages.org': 'podushka.net',
 'ne--skazu-livejournal-com.turbopages.org': 'ne-skazu.livejournal.com',
 'slime--recipes-com.turbopages.org': 'slime-recipes.com',
 'astrakhanfm-ru.turbopages.org': 'astrakhanfm.ru',
 'rosng-ru.turbopages.org': 'rosng.ru',
 'promigreni-com.turbopages.org': 'promigreni.com',
 'sochi-com.turbopages.org': 'sochi.com',
 'poluostrov--kamchatka-ru.turbopages.org': 'poluostrov-kamchatka.ru',
 'shelomova-livejournal-com.turbopages.org': 'shelomova.livejournal.com',
 'beelead-com.turbopages.org': 'beelead.com',
 'odeon--group-ru.turbopages.org': 'odeon-group.ru',
 'vkusnyjstol-ru.turbopages.org': 'vkusnyjstol.ru',
 'sobaki--pesiki-ru.turbopages.org': 'sobaki-pesiki.ru',
 'relife-blog.turbopages.org': 'relife.blog',
 'qwizz-ru.turbopages.org': 'qwizz.ru',
 'alexeyosokin-livejournal-com.turbopages.org': 'alexeyosokin.livejournal.com',
 'tavto-ru.turbopages.org': 'tavto.ru',
 'chellove4ishe-livejournal-com.turbopages.org': 'chellove4ishe.livejournal.com',
 'grand--haus-ru.turbopages.org': 'grand-haus.ru',
 'tolkovkysno-ru.turbopages.org': 'tolkovkysno.ru',
 'shokoladd-ru.turbopages.org': 'shokoladd.ru',
 '3dgame--creator-ru.turbopages.org': '3dgame-creator.ru',
 'vologda-mk-ru.turbopages.org': 'vologda.mk.ru',
 'sankt--peterburg-splar-ru.turbopages.org': 'sankt-peterburg.splar.ru',
 'provladimir-ru.turbopages.org': 'provladimir.ru',
 'posamogonu-ru.turbopages.org': 'posamogonu.ru',
 'patriotov-net.turbopages.org': 'patriotov.net',
 't--demi-livejournal-com.turbopages.org': 't-demi.livejournal.com',
 'cracklover-livejournal-com.turbopages.org': 'cracklover.livejournal.com',
 'tehnologi-su.turbopages.org': 'tehnologi.su',
 'dosuga-net.turbopages.org': 'dosuga.net',
 'yell-ru.turbopages.org': 'yell.ru',
 'oleg--butenko-livejournal-com.turbopages.org': 'oleg-butenko.livejournal.com',
 'darkind-livejournal-com.turbopages.org': 'darkind.livejournal.com',
 'don--art-ru.turbopages.org': 'don-art.ru',
 'natalcarta-ru.turbopages.org': 'natalcarta.ru',
 'penza-bezformata-com.turbopages.org': 'penza.bezformata.com',
 'penza-aif-ru.turbopages.org': 'penza.aif.ru',
 'howtotravel-ru.turbopages.org': 'howtotravel.ru',
 'feldsher-ru.turbopages.org': 'feldsher.ru',
 'seopulses-ru.turbopages.org': 'seopulses.ru',
 'kto--muzh-ru.turbopages.org': 'kto-muzh.ru',
 'preryvanie--beremennosti-ru.turbopages.org': 'preryvanie-beremennosti.ru',
 'nk96-ru.turbopages.org': 'nk96.ru',
 'ladoshka82-livejournal-com.turbopages.org': 'ladoshka82.livejournal.com',
 'm--i--kuznetsov-livejournal-com.turbopages.org': 'm-i-kuznetsov.livejournal.com',
 'trio--mia-livejournal-com.turbopages.org': 'trio-mia.livejournal.com',
 'armatool-ru.turbopages.org': 'armatool.ru',
 'laminatika-ru.turbopages.org': 'laminatika.ru',
 'safoelj-livejournal-com.turbopages.org': 'safoelj.livejournal.com',
 'chevniva-ru.turbopages.org': 'chevniva.ru',
 'maysuryan-livejournal-com.turbopages.org': 'maysuryan.livejournal.com',
 'materialyexpert-ru.turbopages.org': 'materialyexpert.ru',
 'vuborke-ru.turbopages.org': 'vuborke.ru',
 'avtotachki-com.turbopages.org': 'avtotachki.com',
 'prohfesor-livejournal-com.turbopages.org': 'prohfesor.livejournal.com',
 'mazikrem-ru.turbopages.org': 'mazikrem.ru',
 'tokar-guru.turbopages.org': 'tokar.guru',
 'mojkishechnik-ru.turbopages.org': 'mojkishechnik.ru',
 'pavenitur--lehta-livejournal-com.turbopages.org': 'pavenitur-lehta.livejournal.com',
 'erzian--folkmed-livejournal-com.turbopages.org': 'erzian-folkmed.livejournal.com',
 'nosikot-livejournal-com.turbopages.org': 'nosikot.livejournal.com',
 'micrusha-ru.turbopages.org': 'micrusha.ru',
 'astroskop-livejournal-com.turbopages.org': 'astroskop.livejournal.com',
 'shaxsu-ru.turbopages.org': 'shaxsu.ru',
 'shyan-ru.turbopages.org': 'shyan.ru',
 'doctor-livejournal-com.turbopages.org': 'doctor.livejournal.com',
 'krasivodel-ru.turbopages.org': 'krasivodel.ru',
 'saitzhenskiy-ru.turbopages.org': 'saitzhenskiy.ru',
 'tat-versia-ru.turbopages.org': 'tat.versia.ru',
 'mechtayte-ru.turbopages.org': 'mechtayte.ru',
 'master--azov-ru.turbopages.org': 'master-azov.ru',
 'nina--vidente-livejournal-com.turbopages.org': 'nina-vidente.livejournal.com',
 '45ll-net.turbopages.org': '45ll.net',
 'plitkahoff-ru.turbopages.org': 'plitkahoff.ru',
 'zvonili-net.turbopages.org': 'zvonili.net',
 'xcost-org.turbopages.org': 'xcost.org',
 'ketome-ru.turbopages.org': 'ketome.ru',
 'proscrap-livejournal-com.turbopages.org': 'proscrap.livejournal.com',
 'trg--market-ru.turbopages.org': 'trg-market.ru',
 'moy--kroha-info.turbopages.org': 'moy-kroha.info',
 'moguletaty-livejournal-com.turbopages.org': 'moguletaty.livejournal.com',
 'adamashek-livejournal-com.turbopages.org': 'adamashek.livejournal.com',
 'silentwesterner-livejournal-com.turbopages.org': 'silentwesterner.livejournal.com',
 'dosie-su.turbopages.org': 'dosie.su',
 'oleglurie--new-livejournal-com.turbopages.org': 'oleglurie-new.livejournal.com',
 'stasrus2009-livejournal-com.turbopages.org': 'stasrus2009.livejournal.com',
 'homo--ineruditus-livejournal-com.turbopages.org': 'homo-ineruditus.livejournal.com',
 'alekseysc-livejournal-com.turbopages.org': 'alekseysc.livejournal.com',
 'oboguev-livejournal-com.turbopages.org': 'oboguev.livejournal.com',
 'parashistay-ru.turbopages.org': 'parashistay.ru',
 'auto-1prime-ru.turbopages.org': 'auto.1prime.ru',
 'el--murid-livejournal-com.turbopages.org': 'el-murid.livejournal.com',
 'prokachestvo-ru.turbopages.org': 'prokachestvo.ru',
 'snipeclass-ru.turbopages.org': 'snipeclass.ru',
 'prolitso-ru.turbopages.org': 'prolitso.ru',
 'allcantrip-ru.turbopages.org': 'allcantrip.ru',
 'life--in--turkey-livejournal-com.turbopages.org': 'life-in-turkey.livejournal.com',
 'gorinkom-ru.turbopages.org': 'gorinkom.ru',
 'mainstro-ru.turbopages.org': 'mainstro.ru',
 'vyatka--grad-ru.turbopages.org': 'vyatka-grad.ru',
 'telegramfeed-com.turbopages.org': 'telegramfeed.com',
 'tgchief-com.turbopages.org': 'tgchief.com',
 'fleurlili-ru.turbopages.org': 'fleurlili.ru',
 'pitaniedetok-ru.turbopages.org': 'pitaniedetok.ru',
 'volkswagen2-ru.turbopages.org': 'volkswagen2.ru',
 'cyxymu-livejournal-com.turbopages.org': 'cyxymu.livejournal.com',
 'odinnaostrove-com.turbopages.org': 'odinnaostrove.com',
 '7factov-ru.turbopages.org': '7factov.ru',
 'chemodur-livejournal-com.turbopages.org': 'chemodur.livejournal.com',
 'system--psiholog-livejournal-com.turbopages.org': 'system-psiholog.livejournal.com',
 'blogs-bebeshka-info.turbopages.org': 'blogs.bebeshka.info',
 'determiner-ru.turbopages.org': 'determiner.ru',
 'dom--s--ymom-org.turbopages.org': 'dom-s-ymom.org',
 '21mm-ru.turbopages.org': '21mm.ru',
 'v--n--zb-livejournal-com.turbopages.org': 'v-n-zb.livejournal.com',
 'duet11-livejournal-com.turbopages.org': 'duet11.livejournal.com',
 'neznakomka--18-livejournal-com.turbopages.org': 'neznakomka-18.livejournal.com',
 'pobeda-life.turbopages.org': 'pobeda.life',
 'materialyinfo-ru.turbopages.org': 'materialyinfo.ru',
 'spid--vich--zppp-ru.turbopages.org': 'spid-vich-zppp.ru',
 'mjj-ru.turbopages.org': 'mjj.ru',
 'topor-info.turbopages.org': 'topor.info',
 'opervzakone-livejournal-com.turbopages.org': 'opervzakone.livejournal.com',
 'klin-bezformata-com.turbopages.org': 'klin.bezformata.com',
 'karniz--dv-ru.turbopages.org': 'karniz-dv.ru',
 'raschetniy--schet-ru.turbopages.org': 'raschetniy-schet.ru',
 'sedan--polo-ru.turbopages.org': 'sedan-polo.ru',
 'anatalia-ru.turbopages.org': 'anatalia.ru',
 'obltv-ru.turbopages.org': 'obltv.ru',
 'shopml-ru.turbopages.org': 'shopml.ru',
 'filevskayalinia-livejournal-com.turbopages.org': 'filevskayalinia.livejournal.com',
 'shenzhen--ru-livejournal-com.turbopages.org': 'shenzhen-ru.livejournal.com',
 'slave--gulianna-livejournal-com.turbopages.org': 'slave-gulianna.livejournal.com',
 'vash--massazh-ru.turbopages.org': 'vash-massazh.ru',
 'der--appetit-livejournal-com.turbopages.org': 'der-appetit.livejournal.com',
 'teplobloknn-ru.turbopages.org': 'teplobloknn.ru',
 'vulkan--avia-livejournal-com.turbopages.org': 'vulkan-avia.livejournal.com',
 'daily--winegraph-livejournal-com.turbopages.org': 'daily-winegraph.livejournal.com',
 'amantonio-livejournal-com.turbopages.org': 'amantonio.livejournal.com',
 'mamonino-livejournal-com.turbopages.org': 'mamonino.livejournal.com',
 'proizhs-ru.turbopages.org': 'proizhs.ru',
 'tytpodelki-ru.turbopages.org': 'tytpodelki.ru',
 'dachadesign-ru.turbopages.org': 'dachadesign.ru',
 'ekomebel-com.turbopages.org': 'ekomebel.com',
 'toolprokat43-ru.turbopages.org': 'toolprokat43.ru',
 'sdelat--dom-ru.turbopages.org': 'sdelat-dom.ru',
 'nerm--ykt-livejournal-com.turbopages.org': 'nerm-ykt.livejournal.com',
 'sforza--mcintosh-livejournal-com.turbopages.org': 'sforza-mcintosh.livejournal.com',
 'ajushka-livejournal-com.turbopages.org': 'ajushka.livejournal.com',
 'webinvestor-pro.turbopages.org': 'webinvestor.pro',
 'xn------5cdcidt5a3bxeoge8dxji.turbopages.org': 'xn---5cdcidt5a3bxeoge8dxji',
 'tht55-livejournal-com.turbopages.org': 'tht55.livejournal.com',
 'sladkowa-livejournal-com.turbopages.org': 'sladkowa.livejournal.com',
 'gkgazeta-ru.turbopages.org': 'gkgazeta.ru',
 'neprofessional-livejournal-com.turbopages.org': 'neprofessional.livejournal.com',
 'vchaspik-ua.turbopages.org': 'vchaspik.ua',
 'ivory--iv-livejournal-com.turbopages.org': 'ivory-iv.livejournal.com',
 'bulze-ru.turbopages.org': 'bulze.ru',
 'ekstraverty-ru.turbopages.org': 'ekstraverty.ru',
 'alena--korolkova-livejournal-com.turbopages.org': 'alena-korolkova.livejournal.com',
 'bad--breath-ru.turbopages.org': 'bad-breath.ru',
 'lipeck-bezformata-com.turbopages.org': 'lipeck.bezformata.com',
 'dshishatura-ru.turbopages.org': 'dshishatura.ru',
 'catholicsar-ru.turbopages.org': 'catholicsar.ru',
 'naperevoz-ru.turbopages.org': 'naperevoz.ru',
 'detroit--diesel8-livejournal-com.turbopages.org': 'detroit-diesel8.livejournal.com',
 'anvay-ru.turbopages.org': 'anvay.ru',
 'moloko--chr-ru.turbopages.org': 'moloko-chr.ru',
 'cosmo-kz.turbopages.org': 'cosmo.kz',
 'sakhalinmedia-ru.turbopages.org': 'sakhalinmedia.ru',
 'shadowerstark-livejournal-com.turbopages.org': 'shadowerstark.livejournal.com',
 'klg-aif-ru.turbopages.org': 'klg.aif.ru',
 'size--up-ru.turbopages.org': 'size-up.ru',
 '2hpc-ru.turbopages.org': '2hpc.ru',
 'za--pravdu17-livejournal-com.turbopages.org': 'za-pravdu17.livejournal.com',
 'smolgazeta-ru.turbopages.org': 'smolgazeta.ru',
 'osvr--site-livejournal-com.turbopages.org': 'osvr-site.livejournal.com',
 'vseofinansah-ru.turbopages.org': 'vseofinansah.ru',
 'familyr--pravo-livejournal-com.turbopages.org': 'familyr-pravo.livejournal.com',
 'womenmag-ru.turbopages.org': 'womenmag.ru',
 'melegimtekstil-ru.turbopages.org': 'melegimtekstil.ru',
 'sad6sotok-ru.turbopages.org': 'sad6sotok.ru',
 'enman-livejournal-com.turbopages.org': 'enman.livejournal.com',
 'my--pic-ru.turbopages.org': 'my-pic.ru',
 'krutipedaly-ru.turbopages.org': 'krutipedaly.ru',
 '2academy-livejournal-com.turbopages.org': '2academy.livejournal.com',
 'rus--vopros-livejournal-com.turbopages.org': 'rus-vopros.livejournal.com',
 'lenarudenko-livejournal-com.turbopages.org': 'lenarudenko.livejournal.com',
 '24rus-ru.turbopages.org': '24rus.ru',
 'dvoynik--nikolay-livejournal-com.turbopages.org': 'dvoynik-nikolay.livejournal.com',
 'freelance-today.turbopages.org': 'freelance.today',
 'pskov-kp-ru.turbopages.org': 'pskov.kp.ru',
 'wikipets-ru.turbopages.org': 'wikipets.ru',
 'sovhozik-ru.turbopages.org': 'sovhozik.ru',
 'saddrakona-ru.turbopages.org': 'saddrakona.ru',
 'alternativa--mc-ru.turbopages.org': 'alternativa-mc.ru',
 'budushchee-livejournal-com.turbopages.org': 'budushchee.livejournal.com',
 'catcher-fish.turbopages.org': 'catcher.fish',
 's--zametki-ru.turbopages.org': 's-zametki.ru',
 'potrebitel--il-livejournal-com.turbopages.org': 'potrebitel-il.livejournal.com',
 'boristayskey-livejournal-com.turbopages.org': 'boristayskey.livejournal.com',
 'missfit-ru.turbopages.org': 'missfit.ru',
 'mkchita-ru.turbopages.org': 'mkchita.ru',
 'howtosew-ru.turbopages.org': 'howtosew.ru',
 'kviten-livejournal-com.turbopages.org': 'kviten.livejournal.com',
 'balkon-guru.turbopages.org': 'balkon.guru',
 'novosibirsk-best--stroy-ru.turbopages.org': 'novosibirsk.best-stroy.ru',
 '9xxz-livejournal-com.turbopages.org': '9xxz.livejournal.com',
 'more--dom-livejournal-com.turbopages.org': 'more-dom.livejournal.com',
 'nawepodmoskovie-livejournal-com.turbopages.org': 'nawepodmoskovie.livejournal.com',
 'xn------7cdau3d8ak2a.turbopages.org': 'xn---7cdau3d8ak2a',
 'newzdorovie-ru.turbopages.org': 'newzdorovie.ru',
 'lena--miro-ru.turbopages.org': 'lena-miro.ru',
 'exclusivevdom-ru.turbopages.org': 'exclusivevdom.ru',
 'melissa--12-livejournal-com.turbopages.org': 'melissa-12.livejournal.com',
 'pavel--subbotin-livejournal-com.turbopages.org': 'pavel-subbotin.livejournal.com',
 'soviet--life-livejournal-com.turbopages.org': 'soviet-life.livejournal.com',
 'tricolor--lk-online.turbopages.org': 'tricolor-lk.online',
 'godzagodom-com.turbopages.org': 'godzagodom.com',
 'infobraz-ru.turbopages.org': 'infobraz.ru',
 'semenuk-livejournal-com.turbopages.org': 'semenuk.livejournal.com',
 'silmagor-livejournal-com.turbopages.org': 'silmagor.livejournal.com',
 'kot--de--azur-livejournal-com.turbopages.org': 'kot-de-azur.livejournal.com',
 'moonwalker004-livejournal-com.turbopages.org': 'moonwalker004.livejournal.com',
 'architectstyle-livejournal-com.turbopages.org': 'architectstyle.livejournal.com',
 'trip--guru-ru.turbopages.org': 'trip-guru.ru',
 'radioman07-livejournal-com.turbopages.org': 'radioman07.livejournal.com',
 'toplivoavto-ru.turbopages.org': 'toplivoavto.ru',
 'finkultura-com.turbopages.org': 'finkultura.com',
 'awrus7-livejournal-com.turbopages.org': 'awrus7.livejournal.com',
 'a--school5-ru.turbopages.org': 'a-school5.ru',
 'otprazdnuem-com.turbopages.org': 'otprazdnuem.com',
 'hendmeid-guru.turbopages.org': 'hendmeid.guru',
 'skeptimist-livejournal-com.turbopages.org': 'skeptimist.livejournal.com',
 'de--de--de-livejournal-com.turbopages.org': 'de-de-de.livejournal.com',
 'krym--portal-ru.turbopages.org': 'krym-portal.ru',
 'mingi--tay-livejournal-com.turbopages.org': 'mingi-tay.livejournal.com',
 'beerexpert-livejournal-com.turbopages.org': 'beerexpert.livejournal.com',
 'funsvadba-livejournal-com.turbopages.org': 'funsvadba.livejournal.com',
 'chameleon--girl-livejournal-com.turbopages.org': 'chameleon-girl.livejournal.com',
 '19ivolga67-livejournal-com.turbopages.org': '19ivolga67.livejournal.com',
 'vgulevich-livejournal-com.turbopages.org': 'vgulevich.livejournal.com',
 'infopro54-ru.turbopages.org': 'infopro54.ru',
 'german--kmw-livejournal-com.turbopages.org': 'german-kmw.livejournal.com',
 'mg5642-livejournal-com.turbopages.org': 'mg5642.livejournal.com',
 'victorborisov-livejournal-com.turbopages.org': 'victorborisov.livejournal.com',
 'torbina-livejournal-com.turbopages.org': 'torbina.livejournal.com',
 'trudovoi--kodeks-ru.turbopages.org': 'trudovoi-kodeks.ru',
 'niic--ekspert-ru.turbopages.org': 'niic-ekspert.ru',
 'sad24-ru.turbopages.org': 'sad24.ru',
 'i--won-ru.turbopages.org': 'i-won.ru',
 'diary--workout-ru.turbopages.org': 'diary-workout.ru',
 'neuezeiten-livejournal-com.turbopages.org': 'neuezeiten.livejournal.com',
 'chudinovandrei-livejournal-com.turbopages.org': 'chudinovandrei.livejournal.com',
 'my--agro-com.turbopages.org': 'my-agro.com',
 'spp--43-livejournal-com.turbopages.org': 'spp-43.livejournal.com',
 'tula-bezformata-com.turbopages.org': 'tula.bezformata.com',
 'duel--gazeta-livejournal-com.turbopages.org': 'duel-gazeta.livejournal.com',
 'ru--tarot-livejournal-com.turbopages.org': 'ru-tarot.livejournal.com',
 'tursputnik-com.turbopages.org': 'tursputnik.com',
 'krieger--88-livejournal-com.turbopages.org': 'krieger-88.livejournal.com',
 'lenormande-livejournal-com.turbopages.org': 'lenormande.livejournal.com',
 'vozduh2005-livejournal-com.turbopages.org': 'vozduh2005.livejournal.com',
 'xn------5cdty5aeghnk6c5i.turbopages.org': 'xn---5cdty5aeghnk6c5i',
 'deadlylaugh-ru.turbopages.org': 'deadlylaugh.ru',
 'magicpeace-ru.turbopages.org': 'magicpeace.ru',
 'aliexpres-sale.turbopages.org': 'aliexpres.sale',
 'migreview-com.turbopages.org': 'migreview.com',
 'seva--riga-livejournal-com.turbopages.org': 'seva-riga.livejournal.com',
 'edersaas-ru.turbopages.org': 'edersaas.ru',
 'nikitich-livejournal-com.turbopages.org': 'nikitich.livejournal.com',
 'skrepohistory-livejournal-com.turbopages.org': 'skrepohistory.livejournal.com',
 'trim--c-livejournal-com.turbopages.org': 'trim-c.livejournal.com',
 'yakutia-kp-ru.turbopages.org': 'yakutia.kp.ru',
 'yury--st-livejournal-com.turbopages.org': 'yury-st.livejournal.com',
 'kino--sssr-livejournal-com.turbopages.org': 'kino-sssr.livejournal.com',
 'satira--ua-livejournal-com.turbopages.org': 'satira-ua.livejournal.com',
 'kopilka--kulinara-ru.turbopages.org': 'kopilka-kulinara.ru',
 'ru--cats-livejournal-com.turbopages.org': 'ru-cats.livejournal.com',
 'vsegdanarybalke-ru.turbopages.org': 'vsegdanarybalke.ru',
 'dront-livejournal-com.turbopages.org': 'dront.livejournal.com',
 'zabolevanija-net.turbopages.org': 'zabolevanija.net',
 'moscowmap-ru.turbopages.org': 'moscowmap.ru',
 'kamchatka-aif-ru.turbopages.org': 'kamchatka.aif.ru',
 'smashno-ru.turbopages.org': 'smashno.ru',
 'tv--st-ru.turbopages.org': 'tv-st.ru',
 'georgytim-ru.turbopages.org': 'georgytim.ru',
 'oli--da-livejournal-com.turbopages.org': 'oli-da.livejournal.com',
 'goroskop-guru.turbopages.org': 'goroskop.guru',
 'vseogv-ru.turbopages.org': 'vseogv.ru',
 'sytayasemya-ru.turbopages.org': 'sytayasemya.ru',
 'iflife-ru.turbopages.org': 'iflife.ru',
 'amamam-ru.turbopages.org': 'amamam.ru',
 'qil-ru.turbopages.org': 'qil.ru',
 'plateginfo-com.turbopages.org': 'plateginfo.com',
 'autompv-ru.turbopages.org': 'autompv.ru',
 'xn----7sbgyzecrjdri.turbopages.org': 'xn--7sbgyzecrjdri',
 'newostrie-ru.turbopages.org': 'newostrie.ru',
 'diabet--cats-livejournal-com.turbopages.org': 'diabet-cats.livejournal.com',
 'natural--world-ru.turbopages.org': 'natural-world.ru',
 'latexdragon-livejournal-com.turbopages.org': 'latexdragon.livejournal.com',
 'enciclopediya--geografa-ru.turbopages.org': 'enciclopediya-geografa.ru',
 'vestitambov-ru.turbopages.org': 'vestitambov.ru',
 'vtambove-ru.turbopages.org': 'vtambove.ru',
 'molotok34-ru.turbopages.org': 'molotok34.ru',
 'iarex-ru.turbopages.org': 'iarex.ru',
 'olgalit-livejournal-com.turbopages.org': 'olgalit.livejournal.com',
 'yutka--k-livejournal-com.turbopages.org': 'yutka-k.livejournal.com',
 'znayi-ru.turbopages.org': 'znayi.ru',
 'oncobudni-livejournal-com.turbopages.org': 'oncobudni.livejournal.com',
 'napravdestoy-livejournal-com.turbopages.org': 'napravdestoy.livejournal.com',
 'allgrach-livejournal-com.turbopages.org': 'allgrach.livejournal.com',
 'asichek-livejournal-com.turbopages.org': 'asichek.livejournal.com',
 'pisez-com.turbopages.org': 'pisez.com',
 'nova--sun-ru.turbopages.org': 'nova-sun.ru',
 'psyson-ru.turbopages.org': 'psyson.ru',
 'sims--club-com.turbopages.org': 'sims-club.com',
 'pro--promotion-ru.turbopages.org': 'pro-promotion.ru',
 'multigo-ru.turbopages.org': 'multigo.ru',
 'reflex--vrn-ru.turbopages.org': 'reflex-vrn.ru',
 'adigea-aif-ru.turbopages.org': 'adigea.aif.ru',
 'tov--tob-livejournal-com.turbopages.org': 'tov-tob.livejournal.com',
 'rebro--a--dama-livejournal-com.turbopages.org': 'rebro-a-dama.livejournal.com',
 'catamish-livejournal-com.turbopages.org': 'catamish.livejournal.com',
 'universman-livejournal-com.turbopages.org': 'universman.livejournal.com',
 'newsnord-ru.turbopages.org': 'newsnord.ru',
 'krass56-ru.turbopages.org': 'krass56.ru',
 'dvizhenie24-ru.turbopages.org': 'dvizhenie24.ru',
 'timag82-livejournal-com.turbopages.org': 'timag82.livejournal.com',
 'caravan-kz.turbopages.org': 'caravan.kz',
 'prazdnikson-ru.turbopages.org': 'prazdnikson.ru',
 'start--luck-ru.turbopages.org': 'start-luck.ru',
 'spb-octagon-media.turbopages.org': 'spb.octagon.media',
 'milfarm-ru.turbopages.org': 'milfarm.ru',
 '0--stranger-livejournal-com.turbopages.org': '0-stranger.livejournal.com',
 'emigrant-expert.turbopages.org': 'emigrant.expert',
 'zaur05-livejournal-com.turbopages.org': 'zaur05.livejournal.com',
 'eluosi-livejournal-com.turbopages.org': 'eluosi.livejournal.com',
 'f1report-ru.turbopages.org': 'f1report.ru',
 'birds-su.turbopages.org': 'birds.su',
 'fotografersha-livejournal-com.turbopages.org': 'fotografersha.livejournal.com',
 'sportmaster--bonus-ru.turbopages.org': 'sportmaster-bonus.ru',
 'olenenyok-livejournal-com.turbopages.org': 'olenenyok.livejournal.com',
 'domotkn-ru.turbopages.org': 'domotkn.ru',
 'pythonz-net.turbopages.org': 'pythonz.net',
 'velobike--ru-livejournal-com.turbopages.org': 'velobike-ru.livejournal.com',
 'gruz0perevozki-ru.turbopages.org': 'gruz0perevozki.ru',
 'paveldefula-livejournal-com.turbopages.org': 'paveldefula.livejournal.com',
 'miss--hohotyn007-livejournal-com.turbopages.org': 'miss-hohotyn007.livejournal.com',
 'freshjournal--ru-livejournal-com.turbopages.org': 'freshjournal-ru.livejournal.com',
 'trendy--u-com.turbopages.org': 'trendy-u.com',
 'blues--in--rock-livejournal-com.turbopages.org': 'blues-in-rock.livejournal.com',
 'money--banki-ru.turbopages.org': 'money-banki.ru',
 'vottakaya-ru.turbopages.org': 'vottakaya.ru',
 'sofyapremudraya-livejournal-com.turbopages.org': 'sofyapremudraya.livejournal.com',
 'farmf-ru.turbopages.org': 'farmf.ru',
 'portal2010-com.turbopages.org': 'portal2010.com',
 'prosobak-fun.turbopages.org': 'prosobak.fun',
 'troika--ptah-livejournal-com.turbopages.org': 'troika-ptah.livejournal.com',
 'bukva--zakona-com.turbopages.org': 'bukva-zakona.com',
 'morena--morana-livejournal-com.turbopages.org': 'morena-morana.livejournal.com',
 'tula-aif-ru.turbopages.org': 'tula.aif.ru',
 'clubhistory-ru.turbopages.org': 'clubhistory.ru',
 'o--perov-livejournal-com.turbopages.org': 'o-perov.livejournal.com',
 'gdetoplivo-ru.turbopages.org': 'gdetoplivo.ru',
 'listofbest-livejournal-com.turbopages.org': 'listofbest.livejournal.com',
 'an--babushkin-livejournal-com.turbopages.org': 'an-babushkin.livejournal.com',
 'psmirnova1-livejournal-com.turbopages.org': 'psmirnova1.livejournal.com',
 'dilailah-livejournal-com.turbopages.org': 'dilailah.livejournal.com',
 'washerbest-com.turbopages.org': 'washerbest.com',
 'plaintest-com.turbopages.org': 'plaintest.com',
 'sousec-ru.turbopages.org': 'sousec.ru',
 'vlad--erem8-livejournal-com.turbopages.org': 'vlad-erem8.livejournal.com',
 'newsru-ca.turbopages.org': 'newsru.ca',
 'behoneybee-ru.turbopages.org': 'behoneybee.ru',
 'ru--shkola-ru.turbopages.org': 'ru-shkola.ru',
 'lubertsyriamo-ru.turbopages.org': 'lubertsyriamo.ru',
 'oryol-big--book--med-ru.turbopages.org': 'oryol.big-book-med.ru',
 'kinanet-livejournal-com.turbopages.org': 'kinanet.livejournal.com',
 'peruna2013-livejournal-com.turbopages.org': 'peruna2013.livejournal.com',
 'sibnarkomat-livejournal-com.turbopages.org': 'sibnarkomat.livejournal.com',
 'tipolog-livejournal-com.turbopages.org': 'tipolog.livejournal.com',
 'pesrus-ru.turbopages.org': 'pesrus.ru',
 'gramotno-livejournal-com.turbopages.org': 'gramotno.livejournal.com',
 'ua--travels-livejournal-com.turbopages.org': 'ua-travels.livejournal.com',
 'koshkiwiki-ru.turbopages.org': 'koshkiwiki.ru',
 'real--ist-livejournal-com.turbopages.org': 'real-ist.livejournal.com',
 'raiskiy-livejournal-com.turbopages.org': 'raiskiy.livejournal.com',
 'sustavbol-ru.turbopages.org': 'sustavbol.ru',
 'maxiwell83-livejournal-com.turbopages.org': 'maxiwell83.livejournal.com',
 'greenorc-livejournal-com.turbopages.org': 'greenorc.livejournal.com',
 'kids--tale-ru.turbopages.org': 'kids-tale.ru',
 'mingitau-livejournal-com.turbopages.org': 'mingitau.livejournal.com',
 'ugra-aif-ru.turbopages.org': 'ugra.aif.ru',
 'kak--stat--krasivoy-ru.turbopages.org': 'kak-stat-krasivoy.ru',
 'gorojanin--iz--b-livejournal-com.turbopages.org': 'gorojanin-iz-b.livejournal.com',
 'kuda--spb-ru.turbopages.org': 'kuda-spb.ru',
 'monteklever-livejournal-com.turbopages.org': 'monteklever.livejournal.com',
 'xn--------3vea0bgq5afd1aiwbbldrehjda7gwjj6a.turbopages.org': 'xn----3vea0bgq5afd1aiwbbldrehjda7gwjj6a',
 'sunisex-livejournal-com.turbopages.org': 'sunisex.livejournal.com',
 'erofotos-livejournal-com.turbopages.org': 'erofotos.livejournal.com',
 'azbuka--diabeta-com.turbopages.org': 'azbuka-diabeta.com',
 'glavnyecennosti-ru.turbopages.org': 'glavnyecennosti.ru',
 '1smm-info.turbopages.org': '1smm.info',
 'mebeldizajn-ru.turbopages.org': 'mebeldizajn.ru',
 'bvallejo-livejournal-com.turbopages.org': 'bvallejo.livejournal.com',
 'apple--iphone-ru.turbopages.org': 'apple-iphone.ru',
 'compmasterekb-livejournal-com.turbopages.org': 'compmasterekb.livejournal.com',
 'mkartoev-livejournal-com.turbopages.org': 'mkartoev.livejournal.com',
 'sojuz--slavjan-livejournal-com.turbopages.org': 'sojuz-slavjan.livejournal.com',
 'sergii--88-livejournal-com.turbopages.org': 'sergii-88.livejournal.com',
 'red--ptero-livejournal-com.turbopages.org': 'red-ptero.livejournal.com',
 'womendomain-ru.turbopages.org': 'womendomain.ru',
 'litgeroi-ru.turbopages.org': 'litgeroi.ru',
 'mutatorkvn-livejournal-com.turbopages.org': 'mutatorkvn.livejournal.com',
 'homaaxel-livejournal-com.turbopages.org': 'homaaxel.livejournal.com',
 'olegcheb-livejournal-com.turbopages.org': 'olegcheb.livejournal.com',
 'tatmiles-livejournal-com.turbopages.org': 'tatmiles.livejournal.com',
 'gameoko-ru.turbopages.org': 'gameoko.ru',
 'awo--onyshchenko-livejournal-com.turbopages.org': 'awo-onyshchenko.livejournal.com',
 'seloveselo-online.turbopages.org': 'seloveselo.online',
 'rasxodtopliva-ru.turbopages.org': 'rasxodtopliva.ru',
 'feelek-livejournal-com.turbopages.org': 'feelek.livejournal.com',
 'bibliophagus-livejournal-com.turbopages.org': 'bibliophagus.livejournal.com',
 'bigdrum-livejournal-com.turbopages.org': 'bigdrum.livejournal.com',
 'mashenka9024-livejournal-com.turbopages.org': 'mashenka9024.livejournal.com',
 'shandi1-livejournal-com.turbopages.org': 'shandi1.livejournal.com',
 'salatau-livejournal-com.turbopages.org': 'salatau.livejournal.com',
 'toyota--chr2-ru.turbopages.org': 'toyota-chr2.ru',
 'rebenokmama-ru.turbopages.org': 'rebenokmama.ru',
 'xn----9sbmaxbm9abwt5f.turbopages.org': 'xn--9sbmaxbm9abwt5f',
 '40a-media.turbopages.org': '40a.media',
 'sarkoidoza-ru.turbopages.org': 'sarkoidoza.ru',
 'noliquid-livejournal-com.turbopages.org': 'noliquid.livejournal.com',
 'bayoun-ru.turbopages.org': 'bayoun.ru',
 'sirjones-livejournal-com.turbopages.org': 'sirjones.livejournal.com',
 'dm--private-livejournal-com.turbopages.org': 'dm-private.livejournal.com',
 'zen23-ru.turbopages.org': 'zen23.ru',
 'michael--1976-livejournal-com.turbopages.org': 'michael-1976.livejournal.com',
 'mozg-expert.turbopages.org': 'mozg.expert',
 'karina--great-livejournal-com.turbopages.org': 'karina-great.livejournal.com',
 'kirillfedorov4-livejournal-com.turbopages.org': 'kirillfedorov4.livejournal.com',
 'gjabu-livejournal-com.turbopages.org': 'gjabu.livejournal.com',
 'ru--railway-livejournal-com.turbopages.org': 'ru-railway.livejournal.com',
 'vita--life777-livejournal-com.turbopages.org': 'vita-life777.livejournal.com',
 'daurskoe-ru.turbopages.org': 'daurskoe.ru',
 'davydov--index-livejournal-com.turbopages.org': 'davydov-index.livejournal.com',
 'victoryglas-livejournal-com.turbopages.org': 'victoryglas.livejournal.com',
 'eho--2013-livejournal-com.turbopages.org': 'eho-2013.livejournal.com',
 'favniya-info.turbopages.org': 'favniya.info',
 'pankratiev-livejournal-com.turbopages.org': 'pankratiev.livejournal.com',
 'sornyakov-net.turbopages.org': 'sornyakov.net',
 'am--world-ru.turbopages.org': 'am-world.ru',
 'rutvet-ru.turbopages.org': 'rutvet.ru',
 'byton-ru.turbopages.org': 'byton.ru',
 'cadiogorod-ru.turbopages.org': 'cadiogorod.ru',
 'compsch-com.turbopages.org': 'compsch.com',
 'lugacity-ru.turbopages.org': 'lugacity.ru',
 'dalza-ru.turbopages.org': 'dalza.ru',
 'afterdark2-livejournal-com.turbopages.org': 'afterdark2.livejournal.com',
 'russkij--sever-livejournal-com.turbopages.org': 'russkij-sever.livejournal.com',
 'bga68-livejournal-com.turbopages.org': 'bga68.livejournal.com',
 'blog-blizkolom-ru.turbopages.org': 'blog.blizkolom.ru',
 'work--in--internet-ru.turbopages.org': 'work-in-internet.ru',
 'structor--16-livejournal-com.turbopages.org': 'structor-16.livejournal.com',
 'laila50-livejournal-com.turbopages.org': 'laila50.livejournal.com',
 'v--shabunina-livejournal-com.turbopages.org': 'v-shabunina.livejournal.com',
 '24--00chasa-livejournal-com.turbopages.org': '24-00chasa.livejournal.com',
 '2ryak-livejournal-com.turbopages.org': '2ryak.livejournal.com',
 'ru--open-livejournal-com.turbopages.org': 'ru-open.livejournal.com',
 'vanna-guru.turbopages.org': 'vanna.guru',
 'poraest-ru.turbopages.org': 'poraest.ru',
 'zkd--smotret-ru.turbopages.org': 'zkd-smotret.ru',
 'zakharkiv--travel-ru.turbopages.org': 'zakharkiv-travel.ru',
 'smrebenok-ru.turbopages.org': 'smrebenok.ru',
 'pegasoff-livejournal-com.turbopages.org': 'pegasoff.livejournal.com',
 'prosto--mariya-ru.turbopages.org': 'prosto-mariya.ru',
 'health-sarbc-ru.turbopages.org': 'health.sarbc.ru',
 'frosya--hod-livejournal-com.turbopages.org': 'frosya-hod.livejournal.com',
 'popcrim-livejournal-com.turbopages.org': 'popcrim.livejournal.com',
 'fat--yankey-livejournal-com.turbopages.org': 'fat-yankey.livejournal.com',
 'family--auto-ru.turbopages.org': 'family-auto.ru',
 'kamaz1981-ru.turbopages.org': 'kamaz1981.ru',
 'spb--auto-livejournal-com.turbopages.org': 'spb-auto.livejournal.com',
 'autocentrum-ru.turbopages.org': 'autocentrum.ru',
 'pro--openbank-ru.turbopages.org': 'pro-openbank.ru',
 'zergulio-livejournal-com.turbopages.org': 'zergulio.livejournal.com',
 'elleonora-ru.turbopages.org': 'elleonora.ru',
 'just--interes-ru.turbopages.org': 'just-interes.ru',
 'luki--news-ru.turbopages.org': 'luki-news.ru',
 'abakan-bezformata-com.turbopages.org': 'abakan.bezformata.com',
 'uk--vertum-ru.turbopages.org': 'uk-vertum.ru',
 'prosadiogorod-ru.turbopages.org': 'prosadiogorod.ru',
 'silaoberega-ru.turbopages.org': 'silaoberega.ru',
 'mk--donbass-ru.turbopages.org': 'mk-donbass.ru',
 'questrum-livejournal-com.turbopages.org': 'questrum.livejournal.com',
 'alexandr--palkin-livejournal-com.turbopages.org': 'alexandr-palkin.livejournal.com',
 'baikalstom-ru.turbopages.org': 'baikalstom.ru',
 'imungu-livejournal-com.turbopages.org': 'imungu.livejournal.com',
 'easycooks-livejournal-com.turbopages.org': 'easycooks.livejournal.com',
 'fructify-ru.turbopages.org': 'fructify.ru',
 'coffeegravity-ru.turbopages.org': 'coffeegravity.ru',
 'mediapure-ru.turbopages.org': 'mediapure.ru',
 'remonteha-ru.turbopages.org': 'remonteha.ru',
 'k9shleyka-ru.turbopages.org': 'k9shleyka.ru',
 'new--lustra-ru.turbopages.org': 'new-lustra.ru',
 'turbo--tex-ru.turbopages.org': 'turbo-tex.ru',
 'shtorydekor-ru.turbopages.org': 'shtorydekor.ru',
 'raikovstudio-ru.turbopages.org': 'raikovstudio.ru',
 'dizajny-expert.turbopages.org': 'dizajny.expert',
 'rabota--psy-livejournal-com.turbopages.org': 'rabota-psy.livejournal.com',
 'borman--b-livejournal-com.turbopages.org': 'borman-b.livejournal.com',
 'sashabolshevik-livejournal-com.turbopages.org': 'sashabolshevik.livejournal.com',
 'euster-livejournal-com.turbopages.org': 'euster.livejournal.com',
 'malhaaza-livejournal-com.turbopages.org': 'malhaaza.livejournal.com',
 'voshod--chr-livejournal-com.turbopages.org': 'voshod-chr.livejournal.com',
 'niysa-livejournal-com.turbopages.org': 'niysa.livejournal.com',
 'wolk28-livejournal-com.turbopages.org': 'wolk28.livejournal.com',
 'slovobozhie-com.turbopages.org': 'slovobozhie.com',
 'kv--bear-livejournal-com.turbopages.org': 'kv-bear.livejournal.com',
 'troikatickets-ru.turbopages.org': 'troikatickets.ru',
 'solnechnogorsk-bezformata-com.turbopages.org': 'solnechnogorsk.bezformata.com',
 'kreditniyadvokat-ru.turbopages.org': 'kreditniyadvokat.ru',
 'mkkmz-ru.turbopages.org': 'mkkmz.ru',
 'o--instagram-ru.turbopages.org': 'o-instagram.ru',
 'iherbfans-livejournal-com.turbopages.org': 'iherbfans.livejournal.com',
 'infozet-ru.turbopages.org': 'infozet.ru',
 'kot--1983-livejournal-com.turbopages.org': 'kot-1983.livejournal.com',
 'byrnas-livejournal-com.turbopages.org': 'byrnas.livejournal.com',
 'sajtzarabotka-ru.turbopages.org': 'sajtzarabotka.ru',
 'aleks1966-livejournal-com.turbopages.org': 'aleks1966.livejournal.com',
 'znaiwifi-com.turbopages.org': 'znaiwifi.com',
 'navigator011-livejournal-com.turbopages.org': 'navigator011.livejournal.com',
 'edemnayug-com.turbopages.org': 'edemnayug.com',
 'triphint-ru.turbopages.org': 'triphint.ru',
 'aizen--tt-livejournal-com.turbopages.org': 'aizen-tt.livejournal.com',
 'ammiac-com.turbopages.org': 'ammiac.com',
 'lachat-ru.turbopages.org': 'lachat.ru',
 'chihuahuadog-ru.turbopages.org': 'chihuahuadog.ru',
 'cook--dream-ru.turbopages.org': 'cook-dream.ru',
 'yakaev-livejournal-com.turbopages.org': 'yakaev.livejournal.com',
 'marya--iskysnica-livejournal-com.turbopages.org': 'marya-iskysnica.livejournal.com',
 'partizan--ch-livejournal-com.turbopages.org': 'partizan-ch.livejournal.com',
 'izvestia-kiev-ua.turbopages.org': 'izvestia.kiev.ua',
 'movie--rippers-livejournal-com.turbopages.org': 'movie-rippers.livejournal.com',
 'azbuha-ru.turbopages.org': 'azbuha.ru',
 'kaliningrad-rbc-ru.turbopages.org': 'kaliningrad.rbc.ru',
 'techno--france-ru.turbopages.org': 'techno-france.ru',
 'gusev--online-ru.turbopages.org': 'gusev-online.ru',
 'balticnews-ru.turbopages.org': 'balticnews.ru',
 'minutamami-ru.turbopages.org': 'minutamami.ru',
 'mechnik-livejournal-com.turbopages.org': 'mechnik.livejournal.com',
 'recepty-7dach-ru.turbopages.org': 'recepty.7dach.ru',
 'byd--spokoen-ru.turbopages.org': 'byd-spokoen.ru',
 'evan--gcrm-livejournal-com.turbopages.org': 'evan-gcrm.livejournal.com',
 'raigap-livejournal-com.turbopages.org': 'raigap.livejournal.com',
 'nordlight--spb-livejournal-com.turbopages.org': 'nordlight-spb.livejournal.com',
 'tobgorod-livejournal-com.turbopages.org': 'tobgorod.livejournal.com',
 'pif-dn-ua.turbopages.org': 'pif.dn.ua',
 'pitomzy-com.turbopages.org': 'pitomzy.com',
 'subbotine-livejournal-com.turbopages.org': 'subbotine.livejournal.com',
 'pro--karkas-ru.turbopages.org': 'pro-karkas.ru',
 'estatevoronezh-ru.turbopages.org': 'estatevoronezh.ru',
 'shuraagap-livejournal-com.turbopages.org': 'shuraagap.livejournal.com',
 'baikal24-ru.turbopages.org': 'baikal24.ru',
 'business--magazine-online.turbopages.org': 'business-magazine.online',
 'raid--sl-ru.turbopages.org': 'raid-sl.ru',
 'morschinnet-ru.turbopages.org': 'morschinnet.ru',
 'adslike-ru.turbopages.org': 'adslike.ru',
 'protivklopov-ru.turbopages.org': 'protivklopov.ru',
 'gazeta--vp-ru.turbopages.org': 'gazeta-vp.ru',
 'uncle--sem-livejournal-com.turbopages.org': 'uncle-sem.livejournal.com',
 'slushaidom-ru.turbopages.org': 'slushaidom.ru',
 'cureprostate-ru.turbopages.org': 'cureprostate.ru',
 'doclvs-ru.turbopages.org': 'doclvs.ru',
 'tort--tort-ru.turbopages.org': 'tort-tort.ru',
 'ipoteker-ru.turbopages.org': 'ipoteker.ru',
 'mdomik-ru.turbopages.org': 'mdomik.ru',
 'investobox-ru.turbopages.org': 'investobox.ru',
 '33--podelki-ru.turbopages.org': '33-podelki.ru',
 'historical--fact-livejournal-com.turbopages.org': 'historical-fact.livejournal.com',
 'oleg--doren-livejournal-com.turbopages.org': 'oleg-doren.livejournal.com',
 'mrlycien-livejournal-com.turbopages.org': 'mrlycien.livejournal.com',
 'uleypchel-com-ua.turbopages.org': 'uleypchel.com.ua',
 'glav--ryba-livejournal-com.turbopages.org': 'glav-ryba.livejournal.com',
 'vse--horosho-livejournal-com.turbopages.org': 'vse-horosho.livejournal.com',
 'isa--abu--ali-livejournal-com.turbopages.org': 'isa-abu-ali.livejournal.com',
 'yael--shoshany-livejournal-com.turbopages.org': 'yael-shoshany.livejournal.com',
 'kshetunsky-livejournal-com.turbopages.org': 'kshetunsky.livejournal.com',
 'drugoe--kino-livejournal-com.turbopages.org': 'drugoe-kino.livejournal.com',
 'ruanaliz-ru.turbopages.org': 'ruanaliz.ru',
 'mk--kalm-ru.turbopages.org': 'mk-kalm.ru',
 'officelife-media.turbopages.org': 'officelife.media',
 'opodjeludochnoy-ru.turbopages.org': 'opodjeludochnoy.ru',
 'rusistka-livejournal-com.turbopages.org': 'rusistka.livejournal.com',
 't--4--erti--k-livejournal-com.turbopages.org': 't-4-erti-k.livejournal.com',
 'perstni-com.turbopages.org': 'perstni.com',
 'alekseletskih-livejournal-com.turbopages.org': 'alekseletskih.livejournal.com',
 'franchisespace-ru.turbopages.org': 'franchisespace.ru',
 'gorodovoy-ru.turbopages.org': 'gorodovoy.ru',
 'elektropovar-ru.turbopages.org': 'elektropovar.ru',
 'mobiat-ru.turbopages.org': 'mobiat.ru',
 'xn------gdd3avnendeinhmo5n.turbopages.org': 'xn---gdd3avnendeinhmo5n',
 'naavtotrasse-ru.turbopages.org': 'naavtotrasse.ru',
 'muravei--s-livejournal-com.turbopages.org': 'muravei-s.livejournal.com',
 'novostivoronezha-ru.turbopages.org': 'novostivoronezha.ru',
 'air--med-ru.turbopages.org': 'air-med.ru',
 'modniy--style-ru.turbopages.org': 'modniy-style.ru',
 'rybalka--v-ru.turbopages.org': 'rybalka-v.ru',
 'saratov-bezformata-com.turbopages.org': 'saratov.bezformata.com',
 'anytra-livejournal-com.turbopages.org': 'anytra.livejournal.com',
 'voltobzor-ru.turbopages.org': 'voltobzor.ru',
 'samuraikin-livejournal-com.turbopages.org': 'samuraikin.livejournal.com',
 'vladivostok-bezformata-com.turbopages.org': 'vladivostok.bezformata.com',
 'sect-ru.turbopages.org': 'sect.ru',
 'sibsisadmin-ru.turbopages.org': 'sibsisadmin.ru',
 'dennism-livejournal-com.turbopages.org': 'dennism.livejournal.com',
 'bryansknovosti-ru.turbopages.org': 'bryansknovosti.ru',
 'strhouse-ru.turbopages.org': 'strhouse.ru',
 'sadalskij-livejournal-com.turbopages.org': 'sadalskij.livejournal.com',
 'biysk-bezformata-com.turbopages.org': 'biysk.bezformata.com',
 'koparev-livejournal-com.turbopages.org': 'koparev.livejournal.com',
 'volvo--cars-livejournal-com.turbopages.org': 'volvo-cars.livejournal.com',
 'hali--gali-com.turbopages.org': 'hali-gali.com',
 'vrednyeprivychki-ru.turbopages.org': 'vrednyeprivychki.ru',
 'aged--hare-livejournal-com.turbopages.org': 'aged-hare.livejournal.com',
 'orgperevozok-ru.turbopages.org': 'orgperevozok.ru',
 'gadgetdream-ru.turbopages.org': 'gadgetdream.ru',
 'domodedovod-ru.turbopages.org': 'domodedovod.ru',
 'mir--ogorodik-ru.turbopages.org': 'mir-ogorodik.ru',
 'iranta-net.turbopages.org': 'iranta.net',
 '67viktor-livejournal-com.turbopages.org': '67viktor.livejournal.com',
 '03--pravo-ru.turbopages.org': '03-pravo.ru',
 'bessochi-livejournal-com.turbopages.org': 'bessochi.livejournal.com',
 'for--moms-ru.turbopages.org': 'for-moms.ru',
 'wumojetevse-ru.turbopages.org': 'wumojetevse.ru',
 'body--modify-livejournal-com.turbopages.org': 'body-modify.livejournal.com',
 'about--visotsky-livejournal-com.turbopages.org': 'about-visotsky.livejournal.com',
 'abrod-livejournal-com.turbopages.org': 'abrod.livejournal.com',
 'mirpodarkoff-ru.turbopages.org': 'mirpodarkoff.ru',
 'knigizhizni-ru.turbopages.org': 'knigizhizni.ru',
 'kino--data-online.turbopages.org': 'kino-data.online',
 'soba4nik-ru.turbopages.org': 'soba4nik.ru',
 'allerrorcodes-ru.turbopages.org': 'allerrorcodes.ru',
 'ekb-kudago-com.turbopages.org': 'ekb.kudago.com',
 '46tv-ru.turbopages.org': '46tv.ru',
 'ok--magazine-ru.turbopages.org': 'ok-magazine.ru',
 'vospitanie-guru.turbopages.org': 'vospitanie.guru',
 'irk-aif-ru.turbopages.org': 'irk.aif.ru',
 'uctopuockon--pyc-livejournal-com.turbopages.org': 'uctopuockon-pyc.livejournal.com',
 'sveta--konfeta1-livejournal-com.turbopages.org': 'sveta-konfeta1.livejournal.com',
 'uritsk-livejournal-com.turbopages.org': 'uritsk.livejournal.com',
 'well--p-livejournal-com.turbopages.org': 'well-p.livejournal.com',
 'limonada--net-livejournal-com.turbopages.org': 'limonada-net.livejournal.com',
 'lovers--of--art-livejournal-com.turbopages.org': 'lovers-of-art.livejournal.com',
 'tuning--mg-ru.turbopages.org': 'tuning-mg.ru',
 'nelly4726-livejournal-com.turbopages.org': 'nelly4726.livejournal.com',
 'drajver-ru.turbopages.org': 'drajver.ru',
 '7minutes-ru.turbopages.org': '7minutes.ru',
 'animaljournal-ru.turbopages.org': 'animaljournal.ru',
 'golibus-livejournal-com.turbopages.org': 'golibus.livejournal.com',
 'americandiary-livejournal-com.turbopages.org': 'americandiary.livejournal.com',
 'registrr-livejournal-com.turbopages.org': 'registrr.livejournal.com',
 'chiosta-livejournal-com.turbopages.org': 'chiosta.livejournal.com',
 'markosyan--usik-livejournal-com.turbopages.org': 'markosyan-usik.livejournal.com',
 'joinfor-ru.turbopages.org': 'joinfor.ru',
 'skysheep-livejournal-com.turbopages.org': 'skysheep.livejournal.com',
 'marjalutra-livejournal-com.turbopages.org': 'marjalutra.livejournal.com',
 'julia--ordaz-livejournal-com.turbopages.org': 'julia-ordaz.livejournal.com',
 'avto--kul-ru.turbopages.org': 'avto-kul.ru',
 'obrazcy--rezume-ru.turbopages.org': 'obrazcy-rezume.ru',
 'gradientr-com.turbopages.org': 'gradientr.com',
 'skurlatov-livejournal-com.turbopages.org': 'skurlatov.livejournal.com',
 'pro--kanalizaciju-ru.turbopages.org': 'pro-kanalizaciju.ru',
 'rosticus-livejournal-com.turbopages.org': 'rosticus.livejournal.com',
 'posleurokov-ru.turbopages.org': 'posleurokov.ru',
 'messengers-guru.turbopages.org': 'messengers.guru',
 'my--busines-ru.turbopages.org': 'my-busines.ru',
 'perm-mk-ru.turbopages.org': 'perm.mk.ru',
 'obrazec--rezyume-ru.turbopages.org': 'obrazec-rezyume.ru',
 'a--velezar14-livejournal-com.turbopages.org': 'a-velezar14.livejournal.com',
 'serg--slavorum-livejournal-com.turbopages.org': 'serg-slavorum.livejournal.com',
 'dona--anna-livejournal-com.turbopages.org': 'dona-anna.livejournal.com',
 'dekostom-ru.turbopages.org': 'dekostom.ru',
 '0llin--r-livejournal-com.turbopages.org': '0llin-r.livejournal.com',
 'drogo--of--athas-livejournal-com.turbopages.org': 'drogo-of-athas.livejournal.com',
 'yakushef-livejournal-com.turbopages.org': 'yakushef.livejournal.com',
 'vladimir--boroda-livejournal-com.turbopages.org': 'vladimir-boroda.livejournal.com',
 'kan--kendarat-livejournal-com.turbopages.org': 'kan-kendarat.livejournal.com',
 'citrus--site-ru.turbopages.org': 'citrus-site.ru',
 'skolko--serij-ru.turbopages.org': 'skolko-serij.ru',
 'nosoldat-net.turbopages.org': 'nosoldat.net',
 'konsul--777--999-livejournal-com.turbopages.org': 'konsul-777-999.livejournal.com',
 'natasha--laurel-livejournal-com.turbopages.org': 'natasha-laurel.livejournal.com',
 'troitsa1-livejournal-com.turbopages.org': 'troitsa1.livejournal.com',
 'barkalov--max-livejournal-com.turbopages.org': 'barkalov-max.livejournal.com',
 'polarrussia-livejournal-com.turbopages.org': 'polarrussia.livejournal.com',
 'rybalka-online.turbopages.org': 'rybalka.online',
 'kakayasvadba-com.turbopages.org': 'kakayasvadba.com',
 'bober--market-ru.turbopages.org': 'bober-market.ru',
 'dvhab-com.turbopages.org': 'dvhab.com',
 'privately-ru.turbopages.org': 'privately.ru',
 'irma2204-livejournal-com.turbopages.org': 'irma2204.livejournal.com',
 'kreditmoneya-ru.turbopages.org': 'kreditmoneya.ru',
 'andreykon-livejournal-com.turbopages.org': 'andreykon.livejournal.com',
 'stabmarket-ru.turbopages.org': 'stabmarket.ru',
 'beirutta-livejournal-com.turbopages.org': 'beirutta.livejournal.com',
 'script--ok-ru.turbopages.org': 'script-ok.ru',
 'severenin-livejournal-com.turbopages.org': 'severenin.livejournal.com',
 'rodnover--vedist-livejournal-com.turbopages.org': 'rodnover-vedist.livejournal.com',
 'kovka--svarka-net.turbopages.org': 'kovka-svarka.net',
 'spina--health-com.turbopages.org': 'spina-health.com',
 'bratsk-irk-today.turbopages.org': 'bratsk.irk.today',
 'prostislav1-livejournal-com.turbopages.org': 'prostislav1.livejournal.com',
 'my--journal--omsk-livejournal-com.turbopages.org': 'my-journal-omsk.livejournal.com',
 'achinsk-bezformata-com.turbopages.org': 'achinsk.bezformata.com',
 'dimarcello-livejournal-com.turbopages.org': 'dimarcello.livejournal.com',
 'smirnova--luda-livejournal-com.turbopages.org': 'smirnova-luda.livejournal.com',
 'grail58-ru.turbopages.org': 'grail58.ru',
 'solnechnaya--zai-livejournal-com.turbopages.org': 'solnechnaya-zai.livejournal.com',
 'mixtests-com.turbopages.org': 'mixtests.com',
 'ulovanet-ru.turbopages.org': 'ulovanet.ru',
 'advokatsidorov-ru.turbopages.org': 'advokatsidorov.ru',
 'atma--raga-livejournal-com.turbopages.org': 'atma-raga.livejournal.com',
 'la--belaga-livejournal-com.turbopages.org': 'la-belaga.livejournal.com',
 'narrata--photo-livejournal-com.turbopages.org': 'narrata-photo.livejournal.com',
 'strutinskaya-livejournal-com.turbopages.org': 'strutinskaya.livejournal.com',
 'arbuzov-livejournal-com.turbopages.org': 'arbuzov.livejournal.com',
 'liseykina-livejournal-com.turbopages.org': 'liseykina.livejournal.com',
 'ovelosipede-ru.turbopages.org': 'ovelosipede.ru',
 'makeladder-com.turbopages.org': 'makeladder.com',
 'victorvideo-livejournal-com.turbopages.org': 'victorvideo.livejournal.com',
 'stalic--kitchen-livejournal-com.turbopages.org': 'stalic-kitchen.livejournal.com',
 'vetmanol-ru.turbopages.org': 'vetmanol.ru',
 'masterim-guru.turbopages.org': 'masterim.guru',
 '7pravil-livejournal-com.turbopages.org': '7pravil.livejournal.com',
 'afisha--tyumen-ru.turbopages.org': 'afisha-tyumen.ru',
 'cs20-ru.turbopages.org': 'cs20.ru',
 'gornomari-livejournal-com.turbopages.org': 'gornomari.livejournal.com',
 'puteshestvievmirprirodi-com.turbopages.org': 'puteshestvievmirprirodi.com',
 'russkiy--malchik-livejournal-com.turbopages.org': 'russkiy-malchik.livejournal.com',
 'turisheva-ru.turbopages.org': 'turisheva.ru',
 'xn--100----mofa0a2gc0agr8a.turbopages.org': 'xn-100--mofa0a2gc0agr8a',
 'vladggu-ru.turbopages.org': 'vladggu.ru',
 'annatubten-livejournal-com.turbopages.org': 'annatubten.livejournal.com',
 'motildazoo-ru.turbopages.org': 'motildazoo.ru',
 'poz--ru-livejournal-com.turbopages.org': 'poz-ru.livejournal.com',
 'proglaza-net.turbopages.org': 'proglaza.net',
 'vzlomboy-livejournal-com.turbopages.org': 'vzlomboy.livejournal.com',
 'voronezh-livejournal-com.turbopages.org': 'voronezh.livejournal.com',
 'pavel--voronezh-livejournal-com.turbopages.org': 'pavel-voronezh.livejournal.com',
 'kotelnikov--ap-livejournal-com.turbopages.org': 'kotelnikov-ap.livejournal.com',
 'igorkurl-livejournal-com.turbopages.org': 'igorkurl.livejournal.com',
 'procol--harum-livejournal-com.turbopages.org': 'procol-harum.livejournal.com',
 'pushkinskij--dom-livejournal-com.turbopages.org': 'pushkinskij-dom.livejournal.com',
 'ogorod-plus.turbopages.org': 'ogorod.plus',
 'nat--zdorov-ru.turbopages.org': 'nat-zdorov.ru',
 'kdubonosov-ru.turbopages.org': 'kdubonosov.ru',
 'belyaev-livejournal-com.turbopages.org': 'belyaev.livejournal.com',
 'autolab72-ru.turbopages.org': 'autolab72.ru',
 'alek--sachka-livejournal-com.turbopages.org': 'alek-sachka.livejournal.com',
 'ant--ufa-com.turbopages.org': 'ant-ufa.com',
 'zai--chen--ko-livejournal-com.turbopages.org': 'zai-chen-ko.livejournal.com',
 'computer76-ru.turbopages.org': 'computer76.ru',
 'antonthelawyer-livejournal-com.turbopages.org': 'antonthelawyer.livejournal.com',
 'sochi-bezformata-com.turbopages.org': 'sochi.bezformata.com',
 'apostletim-livejournal-com.turbopages.org': 'apostletim.livejournal.com',
 'ierei-livejournal-com.turbopages.org': 'ierei.livejournal.com',
 'pesen--net-livejournal-com.turbopages.org': 'pesen-net.livejournal.com',
 'ribulki-ru.turbopages.org': 'ribulki.ru',
 'yagazeta-com.turbopages.org': 'yagazeta.com',
 'old--ufo-livejournal-com.turbopages.org': 'old-ufo.livejournal.com',
 'cheerss-ru.turbopages.org': 'cheerss.ru',
 'alexkolos-livejournal-com.turbopages.org': 'alexkolos.livejournal.com',
 'el--tolstyh-livejournal-com.turbopages.org': 'el-tolstyh.livejournal.com',
 'kaliningrad-bezformata-com.turbopages.org': 'kaliningrad.bezformata.com',
 'domunit-ru.turbopages.org': 'domunit.ru',
 'primeni-ru.turbopages.org': 'primeni.ru',
 'perpetuya--1-livejournal-com.turbopages.org': 'perpetuya-1.livejournal.com',
 'opt32-ru.turbopages.org': 'opt32.ru',
 'arborio-ru.turbopages.org': 'arborio.ru',
 'ru--elizarov-livejournal-com.turbopages.org': 'ru-elizarov.livejournal.com',
 'warchechnya-ru.turbopages.org': 'warchechnya.ru',
 'georgievsk-bezformata-com.turbopages.org': 'georgievsk.bezformata.com',
 'balashiha-bezformata-com.turbopages.org': 'balashiha.bezformata.com',
 'krasivozamuzh-ru.turbopages.org': 'krasivozamuzh.ru',
 'greenfieldparts-ru.turbopages.org': 'greenfieldparts.ru',
 'oskanov-livejournal-com.turbopages.org': 'oskanov.livejournal.com',
 'byslaikyr-livejournal-com.turbopages.org': 'byslaikyr.livejournal.com',
 'tatar--rulit-livejournal-com.turbopages.org': 'tatar-rulit.livejournal.com',
 'asaratov-livejournal-com.turbopages.org': 'asaratov.livejournal.com',
 'fito--center-ru.turbopages.org': 'fito-center.ru',
 'gtrkmariel-ru.turbopages.org': 'gtrkmariel.ru',
 'alllyr-ru.turbopages.org': 'alllyr.ru',
 'narvasadataa-livejournal-com.turbopages.org': 'narvasadataa.livejournal.com',
 'oadam-livejournal-com.turbopages.org': 'oadam.livejournal.com',
 'svoya--ptica-com.turbopages.org': 'svoya-ptica.com',
 'vaneevasdorove1-ru.turbopages.org': 'vaneevasdorove1.ru',
 'open-kg.turbopages.org': 'open.kg',
 'doctormakarova-ru.turbopages.org': 'doctormakarova.ru',
 'borianm-livejournal-com.turbopages.org': 'borianm.livejournal.com',
 'evrasiansk-livejournal-com.turbopages.org': 'evrasiansk.livejournal.com',
 'bdmbase-ru.turbopages.org': 'bdmbase.ru',
 'xeon--e5450-ru.turbopages.org': 'xeon-e5450.ru',
 'monster-livejournal-com.turbopages.org': 'monster.livejournal.com',
 'rtp--news-com.turbopages.org': 'rtp-news.com',
 'nbalanchak-livejournal-com.turbopages.org': 'nbalanchak.livejournal.com',
 'maxvl-livejournal-com.turbopages.org': 'maxvl.livejournal.com',
 'rostislavddd-livejournal-com.turbopages.org': 'rostislavddd.livejournal.com',
 'otattu-ru.turbopages.org': 'otattu.ru',
 'vladimirkrym-livejournal-com.turbopages.org': 'vladimirkrym.livejournal.com',
 'xn----7sb3a0ahafn8g.turbopages.org': 'xn--7sb3a0ahafn8g',
 'sonata-livejournal-com.turbopages.org': 'sonata.livejournal.com',
 'giperton-com.turbopages.org': 'giperton.com',
 'justfitnes-ru.turbopages.org': 'justfitnes.ru',
 'mylawyer-club.turbopages.org': 'mylawyer.club',
 'prodekorsten-com.turbopages.org': 'prodekorsten.com',
 'kaliningradblog-livejournal-com.turbopages.org': 'kaliningradblog.livejournal.com',
 'shkapyneta-livejournal-com.turbopages.org': 'shkapyneta.livejournal.com',
 'cennye--monety-ru.turbopages.org': 'cennye-monety.ru',
 'ran--dent-ru.turbopages.org': 'ran-dent.ru',
 'mylnikovdm-livejournal-com.turbopages.org': 'mylnikovdm.livejournal.com',
 'sarmik-livejournal-com.turbopages.org': 'sarmik.livejournal.com',
 'serovglobus-livejournal-com.turbopages.org': 'serovglobus.livejournal.com',
 'renault--drive-ru.turbopages.org': 'renault-drive.ru',
 'resobr-ru.turbopages.org': 'resobr.ru',
 'cvetoshki-ru.turbopages.org': 'cvetoshki.ru',
 'dm--matveev-livejournal-com.turbopages.org': 'dm-matveev.livejournal.com',
 'mysku-livejournal-com.turbopages.org': 'mysku.livejournal.com',
 'ravilov-media.turbopages.org': 'ravilov.media',
 'serzzze-livejournal-com.turbopages.org': 'serzzze.livejournal.com',
 'sioku-livejournal-com.turbopages.org': 'sioku.livejournal.com',
 'dimagubin-livejournal-com.turbopages.org': 'dimagubin.livejournal.com',
 'onlywed-ru.turbopages.org': 'onlywed.ru',
 'dolchev-livejournal-com.turbopages.org': 'dolchev.livejournal.com',
 '101jurist-com.turbopages.org': '101jurist.com',
 'hoz-guru.turbopages.org': 'hoz.guru',
 'tuapsevesti-ru.turbopages.org': 'tuapsevesti.ru',
 'homius-ru.turbopages.org': 'homius.ru',
 'ksenevka2012-livejournal-com.turbopages.org': 'ksenevka2012.livejournal.com',
 'metallurgprom-org.turbopages.org': 'metallurgprom.org',
 'xn----ctbfl0babnekv.turbopages.org': 'xn--ctbfl0babnekv',
 'ksani--mamontova-livejournal-com.turbopages.org': 'ksani-mamontova.livejournal.com',
 'huntportal-ru.turbopages.org': 'huntportal.ru',
 'nikefans-ru.turbopages.org': 'nikefans.ru',
 'pro--instrymenti-ru.turbopages.org': 'pro-instrymenti.ru',
 'mycrazystars-ru.turbopages.org': 'mycrazystars.ru',
 'zalakirovano-ru.turbopages.org': 'zalakirovano.ru',
 'kofechay-com.turbopages.org': 'kofechay.com',
 'vfront-ru.turbopages.org': 'vfront.ru',
 'solncepoklonnik-livejournal-com.turbopages.org': 'solncepoklonnik.livejournal.com',
 'youarex-ru.turbopages.org': 'youarex.ru',
 'creditar-ru.turbopages.org': 'creditar.ru',
 'advocat--perm-ru.turbopages.org': 'advocat-perm.ru',
 'urist--rostov-com.turbopages.org': 'urist-rostov.com',
 'victor--vos-livejournal-com.turbopages.org': 'victor-vos.livejournal.com',
 'klinikiinfo-ru.turbopages.org': 'klinikiinfo.ru',
 'krasgmu-net.turbopages.org': 'krasgmu.net',
 'shinomontazh--penza-ru.turbopages.org': 'shinomontazh-penza.ru',
 'irek--murtazin-livejournal-com.turbopages.org': 'irek-murtazin.livejournal.com',
 'znaniyaetosila-ru.turbopages.org': 'znaniyaetosila.ru',
 'lexzag-livejournal-com.turbopages.org': 'lexzag.livejournal.com',
 'detskie--raskraski-com.turbopages.org': 'detskie-raskraski.com',
 'dazachatie-ru.turbopages.org': 'dazachatie.ru',
 'itswat-ru.turbopages.org': 'itswat.ru',
 'givatri-livejournal-com.turbopages.org': 'givatri.livejournal.com',
 'livetotravel-ru.turbopages.org': 'livetotravel.ru',
 'prof--medstail-ru.turbopages.org': 'prof-medstail.ru',
 'alinka--minina-livejournal-com.turbopages.org': 'alinka-minina.livejournal.com',
 'milakamilla-livejournal-com.turbopages.org': 'milakamilla.livejournal.com',
 'btdrive-livejournal-com.turbopages.org': 'btdrive.livejournal.com',
 'drunkyhorse-com.turbopages.org': 'drunkyhorse.com',
 'clinicagernetic-ru.turbopages.org': 'clinicagernetic.ru',
 'kolomnaonline-ru.turbopages.org': 'kolomnaonline.ru',
 'sedov--05-livejournal-com.turbopages.org': 'sedov-05.livejournal.com',
 'volgodonsk-bezformata-com.turbopages.org': 'volgodonsk.bezformata.com',
 'shawlshopmsk-livejournal-com.turbopages.org': 'shawlshopmsk.livejournal.com',
 'dogcatdog-ru.turbopages.org': 'dogcatdog.ru',
 'vsatman888-livejournal-com.turbopages.org': 'vsatman888.livejournal.com',
 'otravlen-info.turbopages.org': 'otravlen.info',
 'samarski--kray-livejournal-com.turbopages.org': 'samarski-kray.livejournal.com',
 'fr--dc-ru.turbopages.org': 'fr-dc.ru',
 'big--brave-livejournal-com.turbopages.org': 'big-brave.livejournal.com',
 'tehnofaq-ru.turbopages.org': 'tehnofaq.ru',
 'untrikoy-livejournal-com.turbopages.org': 'untrikoy.livejournal.com',
 'shmandercheizer-livejournal-com.turbopages.org': 'shmandercheizer.livejournal.com',
 'nikonova--alina-livejournal-com.turbopages.org': 'nikonova-alina.livejournal.com',
 'avtopaper-ru.turbopages.org': 'avtopaper.ru',
 'anton2ov-livejournal-com.turbopages.org': 'anton2ov.livejournal.com',
 'socseti4you-ru.turbopages.org': 'socseti4you.ru',
 'ivan--melnic667-livejournal-com.turbopages.org': 'ivan-melnic667.livejournal.com',
 'tv--wwe-ru.turbopages.org': 'tv-wwe.ru',
 'telegros-ru.turbopages.org': 'telegros.ru',
 'luciusvorren-livejournal-com.turbopages.org': 'luciusvorren.livejournal.com',
 'imperialcommiss-livejournal-com.turbopages.org': 'imperialcommiss.livejournal.com',
 'domovouyasha-ru.turbopages.org': 'domovouyasha.ru',
 'yasnodar-ru.turbopages.org': 'yasnodar.ru',
 'anton--afanasev-livejournal-com.turbopages.org': 'anton-afanasev.livejournal.com',
 'vkusnota24-ru.turbopages.org': 'vkusnota24.ru',
 'kuda--sochi-ru.turbopages.org': 'kuda-sochi.ru',
 'energo--24-ru.turbopages.org': 'energo-24.ru',
 'pg13-ru.turbopages.org': 'pg13.ru',
 'swbkka-livejournal-com.turbopages.org': 'swbkka.livejournal.com',
 'antizoomby-livejournal-com.turbopages.org': 'antizoomby.livejournal.com',
 'bdv--collection-livejournal-com.turbopages.org': 'bdv-collection.livejournal.com',
 'germanych-livejournal-com.turbopages.org': 'germanych.livejournal.com',
 'mobinvest-ru.turbopages.org': 'mobinvest.ru',
 'fishcomm-livejournal-com.turbopages.org': 'fishcomm.livejournal.com',
 'tverlife-ru.turbopages.org': 'tverlife.ru',
 'tehnopage-ru.turbopages.org': 'tehnopage.ru',
 'oblast45-ru.turbopages.org': 'oblast45.ru',
 'binary2hex-ru.turbopages.org': 'binary2hex.ru',
 'almaty-tv.turbopages.org': 'almaty.tv',
 'm--ludenhoff-livejournal-com.turbopages.org': 'm-ludenhoff.livejournal.com',
 'atsoy-livejournal-com.turbopages.org': 'atsoy.livejournal.com',
 'kangen-ru.turbopages.org': 'kangen.ru',
 'pivunet-com.turbopages.org': 'pivunet.com',
 'stroitel-livejournal-com.turbopages.org': 'stroitel.livejournal.com',
 'ap--racing-livejournal-com.turbopages.org': 'ap-racing.livejournal.com',
 'housechief-ru.turbopages.org': 'housechief.ru',
 'my--chinese--phone-ru.turbopages.org': 'my-chinese-phone.ru',
 'corvinus--quint-livejournal-com.turbopages.org': 'corvinus-quint.livejournal.com',
 'propotolok-guru.turbopages.org': 'propotolok.guru',
 'womanss-ru.turbopages.org': 'womanss.ru',
 'sandra--nika-livejournal-com.turbopages.org': 'sandra-nika.livejournal.com',
 'cognac--ru-livejournal-com.turbopages.org': 'cognac-ru.livejournal.com',
 'nefer-livejournal-com.turbopages.org': 'nefer.livejournal.com',
 'samesound-ru.turbopages.org': 'samesound.ru',
 'incorrigible--81-livejournal-com.turbopages.org': 'incorrigible-81.livejournal.com',
 'mywonderfulhome-ru.turbopages.org': 'mywonderfulhome.ru',
 'joybay-ru.turbopages.org': 'joybay.ru',
 'foodman-club.turbopages.org': 'foodman.club',
 'bitsa--forest-livejournal-com.turbopages.org': 'bitsa-forest.livejournal.com',
 'jannet--yummy-livejournal-com.turbopages.org': 'jannet-yummy.livejournal.com',
 'bobrikstas-livejournal-com.turbopages.org': 'bobrikstas.livejournal.com',
 'plusworld-ru.turbopages.org': 'plusworld.ru',
 'nzs--rst-ru.turbopages.org': 'nzs-rst.ru',
 'agnus--dei--8-livejournal-com.turbopages.org': 'agnus-dei-8.livejournal.com',
 'seu--8-ru.turbopages.org': 'seu-8.ru',
 'biboroda-livejournal-com.turbopages.org': 'biboroda.livejournal.com',
 'otets--gennadiy-livejournal-com.turbopages.org': 'otets-gennadiy.livejournal.com',
 'dachi-guru.turbopages.org': 'dachi.guru',
 'lesorubexpert-ru.turbopages.org': 'lesorubexpert.ru',
 'gidstroitelstva-ru.turbopages.org': 'gidstroitelstva.ru',
 'inbrain-top.turbopages.org': 'inbrain.top',
 'korchev--karkas-ru.turbopages.org': 'korchev-karkas.ru',
 'culinarygallery-ru.turbopages.org': 'culinarygallery.ru',
 'black--skif-livejournal-com.turbopages.org': 'black-skif.livejournal.com',
 'aristarh--r-livejournal-com.turbopages.org': 'aristarh-r.livejournal.com',
 'trablin-livejournal-com.turbopages.org': 'trablin.livejournal.com',
 'cycyron-livejournal-com.turbopages.org': 'cycyron.livejournal.com',
 'sandra17-livejournal-com.turbopages.org': 'sandra17.livejournal.com',
 'pfo-volga-news.turbopages.org': 'pfo.volga.news',
 'mychevi-ru.turbopages.org': 'mychevi.ru',
 'game--tips-ru.turbopages.org': 'game-tips.ru',
 'enciklopediya--tehniki-ru.turbopages.org': 'enciklopediya-tehniki.ru',
 'bryansk-aif-ru.turbopages.org': 'bryansk.aif.ru',
 'pavarkel-livejournal-com.turbopages.org': 'pavarkel.livejournal.com',
 'spravka-vmarkse-ru.turbopages.org': 'spravka.vmarkse.ru',
 'mebel--designing-ru.turbopages.org': 'mebel-designing.ru',
 'carlasart-ru.turbopages.org': 'carlasart.ru',
 'maxim--bo-livejournal-com.turbopages.org': 'maxim-bo.livejournal.com',
 'ussuriysk-bezformata-com.turbopages.org': 'ussuriysk.bezformata.com',
 'nikolian-livejournal-com.turbopages.org': 'nikolian.livejournal.com',
 'puerrtto-livejournal-com.turbopages.org': 'puerrtto.livejournal.com',
 'moscow--walks-livejournal-com.turbopages.org': 'moscow-walks.livejournal.com',
 'strogino--time-livejournal-com.turbopages.org': 'strogino-time.livejournal.com',
 '5psy-ru.turbopages.org': '5psy.ru',
 'tasha--jardinier-livejournal-com.turbopages.org': 'tasha-jardinier.livejournal.com',
 'tolyatti-bezformata-com.turbopages.org': 'tolyatti.bezformata.com',
 'sapiens4media-livejournal-com.turbopages.org': 'sapiens4media.livejournal.com',
 'faq--comp-ru.turbopages.org': 'faq-comp.ru',
 'kvobzor-ru.turbopages.org': 'kvobzor.ru',
 'xn------eddmd2a1aek2ap9ap8f.turbopages.org': 'xn---eddmd2a1aek2ap9ap8f',
 'galeneastro-livejournal-com.turbopages.org': 'galeneastro.livejournal.com',
 'belayvolhiza-livejournal-com.turbopages.org': 'belayvolhiza.livejournal.com',
 'xn----7sbge3brbbtn3a.turbopages.org': 'xn--7sbge3brbbtn3a',
 'ekamer-livejournal-com.turbopages.org': 'ekamer.livejournal.com',
 'nampuom--pycu-livejournal-com.turbopages.org': 'nampuom-pycu.livejournal.com',
 'farta-livejournal-com.turbopages.org': 'farta.livejournal.com',
 'mydietlive-ru.turbopages.org': 'mydietlive.ru',
 'hitfm-ru.turbopages.org': 'hitfm.ru',
 'chaivo-livejournal-com.turbopages.org': 'chaivo.livejournal.com',
 'nektonnasos-ru.turbopages.org': 'nektonnasos.ru',
 'fashion101-ru.turbopages.org': 'fashion101.ru',
 'rjav-livejournal-com.turbopages.org': 'rjav.livejournal.com',
 'anton--malahov-livejournal-com.turbopages.org': 'anton-malahov.livejournal.com',
    'infokuzov-ru.turbopages.org': 'infokuzov.ru',
 'electromc-ru.turbopages.org': 'electromc.ru',
 'kazan-prufy-ru.turbopages.org': 'kazan.prufy.ru',
 '4inim-ru.turbopages.org': '4inim.ru',
 'delis--aniuta-livejournal-com.turbopages.org': 'delis-aniuta.livejournal.com',
 'astrakhan-aif-ru.turbopages.org': 'astrakhan.aif.ru',
 'mcpanacea-ru.turbopages.org': 'mcpanacea.ru',
 'mikel2--lagutin-livejournal-com.turbopages.org': 'mikel2-lagutin.livejournal.com',
 'gotovim--vmeste1-livejournal-com.turbopages.org': 'gotovim-vmeste1.livejournal.com',
 'harald--halti-livejournal-com.turbopages.org': 'harald-halti.livejournal.com',
 'urokirukodelie-ru.turbopages.org': 'urokirukodelie.ru',
 'muph-livejournal-com.turbopages.org': 'muph.livejournal.com',
 'zelen--na--podokonnike-ru.turbopages.org': 'zelen-na-podokonnike.ru',
 'gruziapro-ru.turbopages.org': 'gruziapro.ru',
 'freesoftenot69-livejournal-com.turbopages.org': 'freesoftenot69.livejournal.com',
 'alex--d-ru.turbopages.org': 'alex-d.ru',
 'uncle--pafnutiy-livejournal-com.turbopages.org': 'uncle-pafnutiy.livejournal.com',
 'moyage311-livejournal-com.turbopages.org': 'moyage311.livejournal.com',
 'kp-kg.turbopages.org': 'kp.kg',
 'k--sportracing-ru.turbopages.org': 'k-sportracing.ru',
 'linhai--russia-ru.turbopages.org': 'linhai-russia.ru',
 'vyshivka-mirtesen-ru.turbopages.org': 'vyshivka.mirtesen.ru',
 'retro--dress-ru.turbopages.org': 'retro-dress.ru',
 'ld58-ru.turbopages.org': 'ld58.ru',
 'master--banketov-ru.turbopages.org': 'master-banketov.ru',
 'bwiper-ru.turbopages.org': 'bwiper.ru',
 'migranews-ru.turbopages.org': 'migranews.ru',
 'empiresandpuzzles-ru.turbopages.org': 'empiresandpuzzles.ru',
 'z--grechka-ru.turbopages.org': 'z-grechka.ru',
 'moreulybok-ru.turbopages.org': 'moreulybok.ru',
 'shkolazhizni-ru.turbopages.org': 'shkolazhizni.ru',
 'motivashkins-ru.turbopages.org': 'motivashkins.ru',
 'style--me-ru.turbopages.org': 'style-me.ru',
 'grodno24-com.turbopages.org': 'grodno24.com',
 'news--turk-ru.turbopages.org': 'news-turk.ru',
 'mchs-fun.turbopages.org': 'mchs.fun',
 'aleksvik-livejournal-com.turbopages.org': 'aleksvik.livejournal.com',
 'news--r-ru.turbopages.org': 'news-r.ru',
 'delovoymir-biz.turbopages.org': 'delovoymir.biz',
 'infoogle-ru.turbopages.org': 'infoogle.ru',
 's3tr0n-livejournal-com.turbopages.org': 's3tr0n.livejournal.com',
 'chpsaratov-ru.turbopages.org': 'chpsaratov.ru',
 'kirov-bezformata-com.turbopages.org': 'kirov.bezformata.com',
 'tver-mk-ru.turbopages.org': 'tver.mk.ru',
 'gold--nostalgia-livejournal-com.turbopages.org': 'gold-nostalgia.livejournal.com',
 'zanmsk-ru.turbopages.org': 'zanmsk.ru',
 'vladimir-mk-ru.turbopages.org': 'vladimir.mk.ru',
 'denmark--surgery-livejournal-com.turbopages.org': 'denmark-surgery.livejournal.com',
 'nepropadu-com.turbopages.org': 'nepropadu.com',
 'minyt--ka-ru.turbopages.org': 'minyt-ka.ru',
 'kotofan-ru.turbopages.org': 'kotofan.ru',
 'topkarting-ru.turbopages.org': 'topkarting.ru',
 'psycop-ru.turbopages.org': 'psycop.ru',
 'shokolad-today.turbopages.org': 'shokolad.today',
 'stepn--wiki-ru.turbopages.org': 'stepn-wiki.ru',
 'vse--ochen--prosto-ru.turbopages.org': 'vse-ochen-prosto.ru',
 'news-1777-ru.turbopages.org': 'news.1777.ru',
 'kulinaroman-ru.turbopages.org': 'kulinaroman.ru',
 'novikovmaxim-livejournal-com.turbopages.org': 'novikovmaxim.livejournal.com',
 'wi10-ru.turbopages.org': 'wi10.ru',
 'brauzerok-ru.turbopages.org': 'brauzerok.ru',
 'zapusti-biz.turbopages.org': 'zapusti.biz',
 'kolesadom-ru.turbopages.org': 'kolesadom.ru',
 'foksevmash-ru.turbopages.org': 'foksevmash.ru',
 'nemihail-livejournal-com.turbopages.org': 'nemihail.livejournal.com',
 'ru--vet-livejournal-com.turbopages.org': 'ru-vet.livejournal.com',
 'ulytka-ru.turbopages.org': 'ulytka.ru',
 'dom--i--remont-info.turbopages.org': 'dom-i-remont.info',
 'moja--koshka-ru.turbopages.org': 'moja-koshka.ru',
 'stoppestinfo-com.turbopages.org': 'stoppestinfo.com',
 'archeonews-ru.turbopages.org': 'archeonews.ru',
 'newstula-ru.turbopages.org': 'newstula.ru',
 'sladkiy--dvor-ru.turbopages.org': 'sladkiy-dvor.ru',
 'dobryjson-ru.turbopages.org': 'dobryjson.ru',
 'chkola--gastronoma-ru.turbopages.org': 'chkola-gastronoma.ru',
 'proexpress-com-ua.turbopages.org': 'proexpress.com.ua',
 'findanimals-ru.turbopages.org': 'findanimals.ru',
 'pwo-su.turbopages.org': 'pwo.su',
 'allmednews-ru.turbopages.org': 'allmednews.ru',
 'allergija-com.turbopages.org': 'allergija.com',
 'province-ru.turbopages.org': 'province.ru',
 'evgechesnokov-livejournal-com.turbopages.org': 'evgechesnokov.livejournal.com',
 'm0rdashka-livejournal-com.turbopages.org': 'm0rdashka.livejournal.com',
 'seleste--rusa-livejournal-com.turbopages.org': 'seleste-rusa.livejournal.com',
 'rederika-livejournal-com.turbopages.org': 'rederika.livejournal.com',
 'y4astkoviu-livejournal-com.turbopages.org': 'y4astkoviu.livejournal.com',
 'vsemuzei-com.turbopages.org': 'vsemuzei.com',
 'zapzap-livejournal-com.turbopages.org': 'zapzap.livejournal.com',
 'ivvvv-livejournal-com.turbopages.org': 'ivvvv.livejournal.com',
 'yapet-livejournal-com.turbopages.org': 'yapet.livejournal.com',
 'moyaukraine-livejournal-com.turbopages.org': 'moyaukraine.livejournal.com',
 'devibhakta-livejournal-com.turbopages.org': 'devibhakta.livejournal.com',
 'valse--boston-livejournal-com.turbopages.org': 'valse-boston.livejournal.com',
 'photar-ru.turbopages.org': 'photar.ru',
 'xn----7sbldqbx2boys3i.turbopages.org': 'xn--7sbldqbx2boys3i',
 'poexaliroma-livejournal-com.turbopages.org': 'poexaliroma.livejournal.com',
 'swamp--lynx-livejournal-com.turbopages.org': 'swamp-lynx.livejournal.com',
 'inplanet-net.turbopages.org': 'inplanet.net',
 '0bovsemka-livejournal-com.turbopages.org': '0bovsemka.livejournal.com',
 'ulsa-livejournal-com.turbopages.org': 'ulsa.livejournal.com',
 'realconsult-ru.turbopages.org': 'realconsult.ru',
 'kazan-bezformata-com.turbopages.org': 'kazan.bezformata.com',
 'yurasumy-livejournal-com.turbopages.org': 'yurasumy.livejournal.com',
 'mnevkusnotut-ru.turbopages.org': 'mnevkusnotut.ru',
 'inopressa-ru.turbopages.org': 'inopressa.ru',
 'gazeta--perm-ru.turbopages.org': 'gazeta-perm.ru',
 'mastersamodelok-ru.turbopages.org': 'mastersamodelok.ru',
 'nadtex-ru.turbopages.org': 'nadtex.ru',
 'biosfera--ua-livejournal-com.turbopages.org': 'biosfera-ua.livejournal.com',
 'zoomax74-ru.turbopages.org': 'zoomax74.ru',
 'privolny-ru.turbopages.org': 'privolny.ru',
 'veles--vologda-ru.turbopages.org': 'veles-vologda.ru',
 'klikabol-com.turbopages.org': 'klikabol.com',
 'bestdoor-guru.turbopages.org': 'bestdoor.guru',
 'forgotten--tale-livejournal-com.turbopages.org': 'forgotten-tale.livejournal.com',
 'logoderm-ru.turbopages.org': 'logoderm.ru',
 'kaknazvat-com.turbopages.org': 'kaknazvat.com',
 'temperaturka-com.turbopages.org': 'temperaturka.com',
 'womenres-ru.turbopages.org': 'womenres.ru',
 'storm100-livejournal-com.turbopages.org': 'storm100.livejournal.com',
 'alyaeva-livejournal-com.turbopages.org': 'alyaeva.livejournal.com',
 'bookz-ru.turbopages.org': 'bookz.ru',
 'bosuell-livejournal-com.turbopages.org': 'bosuell.livejournal.com',
 'vanilla-su.turbopages.org': 'vanilla.su',
 'guzel54-livejournal-com.turbopages.org': 'guzel54.livejournal.com',
 'femina--vita-livejournal-com.turbopages.org': 'femina-vita.livejournal.com',
 'ket--holod-ru.turbopages.org': 'ket-holod.ru',
 'vulpesmontana-livejournal-com.turbopages.org': 'vulpesmontana.livejournal.com',
 'mma--spb-ru.turbopages.org': 'mma-spb.ru',
 'povarixa-ru.turbopages.org': 'povarixa.ru',
 'handskill-ru.turbopages.org': 'handskill.ru',
 'ruslana--k-livejournal-com.turbopages.org': 'ruslana-k.livejournal.com',
 'dmitriev--lgovskij-ru.turbopages.org': 'dmitriev-lgovskij.ru',
 'fps--up-ru.turbopages.org': 'fps-up.ru',
 'pailish-livejournal-com.turbopages.org': 'pailish.livejournal.com',
 'ru--royalty-livejournal-com.turbopages.org': 'ru-royalty.livejournal.com',
 'sergej--manit-livejournal-com.turbopages.org': 'sergej-manit.livejournal.com',
 'marfa--nikitina4-livejournal-com.turbopages.org': 'marfa-nikitina4.livejournal.com',
 'annaturko-livejournal-com.turbopages.org': 'annaturko.livejournal.com',
 'pridesaratov-ru.turbopages.org': 'pridesaratov.ru',
 'firefly--patrick-livejournal-com.turbopages.org': 'firefly-patrick.livejournal.com',
 'o--ladagranta-ru.turbopages.org': 'o-ladagranta.ru',
 'cooleat-ru.turbopages.org': 'cooleat.ru',
 'volga174-livejournal-com.turbopages.org': 'volga174.livejournal.com',
 'andrdi-livejournal-com.turbopages.org': 'andrdi.livejournal.com',
 'strajj-livejournal-com.turbopages.org': 'strajj.livejournal.com',
 'mc--ub-ru.turbopages.org': 'mc-ub.ru',
 'ladynumber1-com.turbopages.org': 'ladynumber1.com',
 'tekstil--doma-ru.turbopages.org': 'tekstil-doma.ru',
 'i38-ru.turbopages.org': 'i38.ru',
 'index--uliz-ru.turbopages.org': 'index-uliz.ru',
 'vpnscanner-com.turbopages.org': 'vpnscanner.com',
 'kuban-newizv-ru.turbopages.org': 'kuban.newizv.ru',
 'beers-su.turbopages.org': 'beers.su',
 'hyundai--auto2-ru.turbopages.org': 'hyundai-auto2.ru',
 'oilgazfontan-livejournal-com.turbopages.org': 'oilgazfontan.livejournal.com',
 'chistoprudov-livejournal-com.turbopages.org': 'chistoprudov.livejournal.com',
 'azov--leto-com.turbopages.org': 'azov-leto.com',
 'myfavoritepet-ru.turbopages.org': 'myfavoritepet.ru',
 'penza-kp-ru.turbopages.org': 'penza.kp.ru',
 'vesti92-ru.turbopages.org': 'vesti92.ru',
 'budtoturisto-ru.turbopages.org': 'budtoturisto.ru',
 'rahimovruslan-livejournal-com.turbopages.org': 'rahimovruslan.livejournal.com',
 'kogdakotika-net.turbopages.org': 'kogdakotika.net',
 'liski-riavrn-ru.turbopages.org': 'liski.riavrn.ru',
 'gala67-livejournal-com.turbopages.org': 'gala67.livejournal.com',
 'travelfrog-livejournal-com.turbopages.org': 'travelfrog.livejournal.com',
 'ru--kino-livejournal-com.turbopages.org': 'ru-kino.livejournal.com',
 'vlad--piskunov-livejournal-com.turbopages.org': 'vlad-piskunov.livejournal.com',
 'goodwine-livejournal-com.turbopages.org': 'goodwine.livejournal.com',
 'russiantowns-livejournal-com.turbopages.org': 'russiantowns.livejournal.com',
 'realmaluta-livejournal-com.turbopages.org': 'realmaluta.livejournal.com',
 'idoska--volgograd-ru.turbopages.org': 'idoska-volgograd.ru',
 'nstarikov-ru.turbopages.org': 'nstarikov.ru',
 'fertilizerdaily-ru.turbopages.org': 'fertilizerdaily.ru',
 'ufa-bezformata-com.turbopages.org': 'ufa.bezformata.com',
 'ardma-net.turbopages.org': 'ardma.net',
 'chelyabinsk-bezformata-com.turbopages.org': 'chelyabinsk.bezformata.com',
 'dcactus-ru.turbopages.org': 'dcactus.ru',
 'novosibirsk-bezformata-com.turbopages.org': 'novosibirsk.bezformata.com',
 'heavycook-livejournal-com.turbopages.org': 'heavycook.livejournal.com',
 'catshere-ru.turbopages.org': 'catshere.ru',
 'proficinema-ru.turbopages.org': 'proficinema.ru',
 'sekretizdorovya-ru.turbopages.org': 'sekretizdorovya.ru',
 'ruslanviktorov-livejournal-com.turbopages.org': 'ruslanviktorov.livejournal.com',
 'arbitr57-livejournal-com.turbopages.org': 'arbitr57.livejournal.com',
 'mishanik--210-livejournal-com.turbopages.org': 'mishanik-210.livejournal.com',
 'zdorovs-livejournal-com.turbopages.org': 'zdorovs.livejournal.com',
 'vesti--lipetsk-ru.turbopages.org': 'vesti-lipetsk.ru',
 'style313-ru.turbopages.org': 'style313.ru',
 'garim--parim-ru.turbopages.org': 'garim-parim.ru',
 'pushba-livejournal-com.turbopages.org': 'pushba.livejournal.com',
 'allergiik-ru.turbopages.org': 'allergiik.ru',
 'novser-livejournal-com.turbopages.org': 'novser.livejournal.com',
 'daria--iz--orla-livejournal-com.turbopages.org': 'daria-iz-orla.livejournal.com',
 'urbanomica-livejournal-com.turbopages.org': 'urbanomica.livejournal.com',
 'shtopor7-livejournal-com.turbopages.org': 'shtopor7.livejournal.com',
 'xn----7sbudalmsse8b.turbopages.org': 'xn--7sbudalmsse8b',
 'betonpro100-ru.turbopages.org': 'betonpro100.ru',
 'mon--sofia-livejournal-com.turbopages.org': 'mon-sofia.livejournal.com',
 'vseproip-com.turbopages.org': 'vseproip.com',
 'mp3obzor-ru.turbopages.org': 'mp3obzor.ru',
 'pechistroy-ru.turbopages.org': 'pechistroy.ru',
 'spravkavrn-ru.turbopages.org': 'spravkavrn.ru',
 'tihiomyt-livejournal-com.turbopages.org': 'tihiomyt.livejournal.com',
 'moskva-bezformata-com.turbopages.org': 'moskva.bezformata.com',
 'leto-today.turbopages.org': 'leto.today',
 'taganrogprav-ru.turbopages.org': 'taganrogprav.ru',
 'taganrog-bezformata-com.turbopages.org': 'taganrog.bezformata.com',
 'irksib-ru.turbopages.org': 'irksib.ru',
 'time33-ru.turbopages.org': 'time33.ru',
 'kolofan-livejournal-com.turbopages.org': 'kolofan.livejournal.com',
 'synchasvetlana-livejournal-com.turbopages.org': 'synchasvetlana.livejournal.com',
 'mk--kirov-ru.turbopages.org': 'mk-kirov.ru',
 'gold-ru.turbopages.org': 'gold.ru',
 'vaniagvozd-livejournal-com.turbopages.org': 'vaniagvozd.livejournal.com',
 'chronograph-livejournal-com.turbopages.org': 'chronograph.livejournal.com',
 'top68-ru.turbopages.org': 'top68.ru',
 'xn----7sbaamhw4aqvhj6c.turbopages.org': 'xn--7sbaamhw4aqvhj6c',
 'consultant--mos-ru.turbopages.org': 'consultant-mos.ru',
 'minskblog-livejournal-com.turbopages.org': 'minskblog.livejournal.com',
 'des--life-ru.turbopages.org': 'des-life.ru',
 'practeco-ru.turbopages.org': 'practeco.ru',
 'abc--modul-ru.turbopages.org': 'abc-modul.ru',
 'gatchina-bezformata-com.turbopages.org': 'gatchina.bezformata.com',
 'xn----7sbak0a4axidp2b.turbopages.org': 'xn--7sbak0a4axidp2b',
 'mir--logiki-ru.turbopages.org': 'mir-logiki.ru',
 'empathycenter-ru.turbopages.org': 'empathycenter.ru',
 'dilerscan-ru.turbopages.org': 'dilerscan.ru',
 'myworldwiki-com.turbopages.org': 'myworldwiki.com',
 'zdorova--krasiva-com.turbopages.org': 'zdorova-krasiva.com',
 'silver--ktulhu-livejournal-com.turbopages.org': 'silver-ktulhu.livejournal.com',
 'flamebelle-livejournal-com.turbopages.org': 'flamebelle.livejournal.com',
 'irina--co-livejournal-com.turbopages.org': 'irina-co.livejournal.com',
 'dlja--pohudenija-ru.turbopages.org': 'dlja-pohudenija.ru',
 'sove2u-ru.turbopages.org': 'sove2u.ru',
 'uravto-com.turbopages.org': 'uravto.com',
 'botanicoved-com.turbopages.org': 'botanicoved.com',
 'allmomente-livejournal-com.turbopages.org': 'allmomente.livejournal.com',
 'usl--k-livejournal-com.turbopages.org': 'usl-k.livejournal.com',
 'humanoidov-livejournal-com.turbopages.org': 'humanoidov.livejournal.com',
 'berni777-livejournal-com.turbopages.org': 'berni777.livejournal.com',
 'bonbonvivant-livejournal-com.turbopages.org': 'bonbonvivant.livejournal.com',
 'vexperience-ru.turbopages.org': 'vexperience.ru',
 'katok-su.turbopages.org': 'katok.su',
 'apexgame-ru.turbopages.org': 'apexgame.ru',
 'ammo1-livejournal-com.turbopages.org': 'ammo1.livejournal.com',
 'ext--5299091-livejournal-com.turbopages.org': 'ext-5299091.livejournal.com',
 'remch--ch-livejournal-com.turbopages.org': 'remch-ch.livejournal.com',
 'serg63ant-livejournal-com.turbopages.org': 'serg63ant.livejournal.com',
 'smetnoedelo-ru.turbopages.org': 'smetnoedelo.ru',
 'immense--sp-livejournal-com.turbopages.org': 'immense-sp.livejournal.com',
 'sestramobislame-livejournal-com.turbopages.org': 'sestramobislame.livejournal.com',
 'marykay--4u-ru.turbopages.org': 'marykay-4u.ru',
 'eug64-livejournal-com.turbopages.org': 'eug64.livejournal.com',
 'etnodrom-livejournal-com.turbopages.org': 'etnodrom.livejournal.com',
 'agronomius-livejournal-com.turbopages.org': 'agronomius.livejournal.com',
 'annetjohnson-livejournal-com.turbopages.org': 'annetjohnson.livejournal.com',
 'wolfschanze-livejournal-com.turbopages.org': 'wolfschanze.livejournal.com',
 'humus-livejournal-com.turbopages.org': 'humus.livejournal.com',
 'wek-ru.turbopages.org': 'wek.ru',
 'ru--polit-livejournal-com.turbopages.org': 'ru-polit.livejournal.com',
 'doctor--365-net.turbopages.org': 'doctor-365.net',
 'safaniuk-livejournal-com.turbopages.org': 'safaniuk.livejournal.com',
 'sverdlovskavia-livejournal-com.turbopages.org': 'sverdlovskavia.livejournal.com',
 'meataspa-ru.turbopages.org': 'meataspa.ru',
 '76--82-livejournal-com.turbopages.org': '76-82.livejournal.com',
 'pavon-kz.turbopages.org': 'pavon.kz',
 'kanalizaciyadoma-com.turbopages.org': 'kanalizaciyadoma.com',
 'alex--levitas-livejournal-com.turbopages.org': 'alex-levitas.livejournal.com',
 'ejidna-livejournal-com.turbopages.org': 'ejidna.livejournal.com',
 'anyuta--x-livejournal-com.turbopages.org': 'anyuta-x.livejournal.com',
 'frauwow-com.turbopages.org': 'frauwow.com',
 'top--lap-livejournal-com.turbopages.org': 'top-lap.livejournal.com',
 'in--matrix-livejournal-com.turbopages.org': 'in-matrix.livejournal.com',
 'dambiev-livejournal-com.turbopages.org': 'dambiev.livejournal.com',
 'navigamer-ru.turbopages.org': 'navigamer.ru',
 'med--ram-ru.turbopages.org': 'med-ram.ru',
 'floraprice-ru.turbopages.org': 'floraprice.ru',
 'ussurmedia-ru.turbopages.org': 'ussurmedia.ru',
 'diabetystop-com.turbopages.org': 'diabetystop.com',
 'ecworld-livejournal-com.turbopages.org': 'ecworld.livejournal.com',
 'medtelo-ru.turbopages.org': 'medtelo.ru',
 'myandrolog-ru.turbopages.org': 'myandrolog.ru',
 'ntr--24-ru.turbopages.org': 'ntr-24.ru',
 'ruinformer-com.turbopages.org': 'ruinformer.com',
 'razuznaika-livejournal-com.turbopages.org': 'razuznaika.livejournal.com',
 'vladimir-bezformata-com.turbopages.org': 'vladimir.bezformata.com',
 'novocherkassk--gorod-ru.turbopages.org': 'novocherkassk-gorod.ru',
 'rv--ryazan-ru.turbopages.org': 'rv-ryazan.ru',
 'lifekorea-ru.turbopages.org': 'lifekorea.ru',
 'pomidoroff-ru.turbopages.org': 'pomidoroff.ru',
 'selyanka1-livejournal-com.turbopages.org': 'selyanka1.livejournal.com',
 'lifetattoo-ru.turbopages.org': 'lifetattoo.ru',
 'magadanpravda-ru.turbopages.org': 'magadanpravda.ru',
 'khabarovsk-octagon-media.turbopages.org': 'khabarovsk.octagon.media',
 'kolymaplus-ru.turbopages.org': 'kolymaplus.ru',
 'zendiar-com.turbopages.org': 'zendiar.com',
 'knigaproavto-ru.turbopages.org': 'knigaproavto.ru',
 'xn----ctbrxekhp4b.turbopages.org': 'xn--ctbrxekhp4b',
 'sellvoyage-ru.turbopages.org': 'sellvoyage.ru',
 'oldfisher--mk-livejournal-com.turbopages.org': 'oldfisher-mk.livejournal.com',
 'bragin--sasha-livejournal-com.turbopages.org': 'bragin-sasha.livejournal.com',
 'dbortik-ru.turbopages.org': 'dbortik.ru',
 'ozhegova--slovar-ru.turbopages.org': 'ozhegova-slovar.ru',
 'kolybanov-livejournal-com.turbopages.org': 'kolybanov.livejournal.com',
 'lycoperdon-livejournal-com.turbopages.org': 'lycoperdon.livejournal.com',
 'and--stav-livejournal-com.turbopages.org': 'and-stav.livejournal.com',
 'bane-guru.turbopages.org': 'bane.guru',
 'karta--raionov-ru.turbopages.org': 'karta-raionov.ru',
 'kurai--31-livejournal-com.turbopages.org': 'kurai-31.livejournal.com',
 'mashinapro-ru.turbopages.org': 'mashinapro.ru',
 'oxrana1-ru.turbopages.org': 'oxrana1.ru',
 'manga24-ru.turbopages.org': 'manga24.ru',
 'pohmelje-ru.turbopages.org': 'pohmelje.ru',
 'baby--lifestyle-ru.turbopages.org': 'baby-lifestyle.ru',
 'none--smilodon-livejournal-com.turbopages.org': 'none-smilodon.livejournal.com',
 'lewhobotov-livejournal-com.turbopages.org': 'lewhobotov.livejournal.com',
 'marketing--course-ru.turbopages.org': 'marketing-course.ru',
 'business--mama-ru.turbopages.org': 'business-mama.ru',
 'yastroyu-ru.turbopages.org': 'yastroyu.ru',
 'hippy--end-livejournal-com.turbopages.org': 'hippy-end.livejournal.com',
 'moe--belgorod-ru.turbopages.org': 'moe-belgorod.ru',
 'individual--tour-livejournal-com.turbopages.org': 'individual-tour.livejournal.com',
 'zampolit--ru-livejournal-com.turbopages.org': 'zampolit-ru.livejournal.com',
 'gtrkkursk-ru.turbopages.org': 'gtrkkursk.ru',
 'sputnikkino-ru.turbopages.org': 'sputnikkino.ru',
 't-rbc-ru.turbopages.org': 't.rbc.ru',
 'irma--stream-ru.turbopages.org': 'irma-stream.ru',
 'tankoved34-livejournal-com.turbopages.org': 'tankoved34.livejournal.com',
 'interessno-ru.turbopages.org': 'interessno.ru',
 'rupolitshow-ru.turbopages.org': 'rupolitshow.ru',
 'vl--sokolov-livejournal-com.turbopages.org': 'vl-sokolov.livejournal.com',
 'pochemuu-ru.turbopages.org': 'pochemuu.ru',
 'narodna--pravda-ua.turbopages.org': 'narodna-pravda.ua',
 'newsua-ru.turbopages.org': 'newsua.ru',
 'kvladimirrr-livejournal-com.turbopages.org': 'kvladimirrr.livejournal.com',
 'ardm2016-livejournal-com.turbopages.org': 'ardm2016.livejournal.com',
 'm--zaxaroff-livejournal-com.turbopages.org': 'm-zaxaroff.livejournal.com',
 'ru-livejournal-com.turbopages.org': 'ru.livejournal.com',
 'nathalie--zh-livejournal-com.turbopages.org': 'nathalie-zh.livejournal.com',
 'gp--prsmah-ru.turbopages.org': 'gp-prsmah.ru',
 'motoiwc-ru.turbopages.org': 'motoiwc.ru',
 'belaruspartisan-by.turbopages.org': 'belaruspartisan.by',
 'anriasem-livejournal-com.turbopages.org': 'anriasem.livejournal.com',
 'd--marianna-livejournal-com.turbopages.org': 'd-marianna.livejournal.com',
 'ilya--prosto-livejournal-com.turbopages.org': 'ilya-prosto.livejournal.com',
 'utvamu-livejournal-com.turbopages.org': 'utvamu.livejournal.com',
 'rurik--l-livejournal-com.turbopages.org': 'rurik-l.livejournal.com',
 'dfm-ru.turbopages.org': 'dfm.ru',
 'panin1976-livejournal-com.turbopages.org': 'panin1976.livejournal.com',
 'korzik-net.turbopages.org': 'korzik.net',
 'beelinehelp24-ru.turbopages.org': 'beelinehelp24.ru',
 'darievna-ru.turbopages.org': 'darievna.ru',
 'petpop-cc.turbopages.org': 'petpop.cc',
 'donz--ru-livejournal-com.turbopages.org': 'donz-ru.livejournal.com',
 'dolinapozitiva-ru.turbopages.org': 'dolinapozitiva.ru',
 'skumila-livejournal-com.turbopages.org': 'skumila.livejournal.com',
 'volna888-livejournal-com.turbopages.org': 'volna888.livejournal.com',
 'pixmafia-com.turbopages.org': 'pixmafia.com',
 'ruxan-ru.turbopages.org': 'ruxan.ru',
 'kirov-aif-ru.turbopages.org': 'kirov.aif.ru',
 'autostuk-ru.turbopages.org': 'autostuk.ru',
 'hairface-ru.turbopages.org': 'hairface.ru',
 'careface-ru.turbopages.org': 'careface.ru',
 'agrarii-com.turbopages.org': 'agrarii.com',
 'xn----itbiakjjjkehgdu0a9a.turbopages.org': 'xn--itbiakjjjkehgdu0a9a',
 'prinmedia-ru.turbopages.org': 'prinmedia.ru',
 'veddana-livejournal-com.turbopages.org': 'veddana.livejournal.com',
 'zub-clinic.turbopages.org': 'zub.clinic',
 'selsov-ru.turbopages.org': 'selsov.ru',
 'pro--kur-ru.turbopages.org': 'pro-kur.ru',
 'healthy--back-livejournal-com.turbopages.org': 'healthy-back.livejournal.com',
 'aforisma-ru.turbopages.org': 'aforisma.ru',
 'rabotai--sam-ru.turbopages.org': 'rabotai-sam.ru',
 'proamulety-ru.turbopages.org': 'proamulety.ru',
 'xn------7cdhdra5eaidflpe2e.turbopages.org': 'xn---7cdhdra5eaidflpe2e',
 'kuzbass-aif-ru.turbopages.org': 'kuzbass.aif.ru',
 'lechinos-ru.turbopages.org': 'lechinos.ru',
 'runetmir-com.turbopages.org': 'runetmir.com',
 'pechen-infox-ru.turbopages.org': 'pechen.infox.ru',
 'spacream-ru.turbopages.org': 'spacream.ru',
 'iphonich-ru.turbopages.org': 'iphonich.ru',
 'mag-103-ua.turbopages.org': 'mag.103.ua',
 'roza--i--fialka-ru.turbopages.org': 'roza-i-fialka.ru',
 '100orchids-ru.turbopages.org': '100orchids.ru',
 'sweetcold-ru.turbopages.org': 'sweetcold.ru',
 'orenburg-kp-ru.turbopages.org': 'orenburg.kp.ru',
 'russkajakrasota-ru.turbopages.org': 'russkajakrasota.ru',
 'greensmm-livejournal-com.turbopages.org': 'greensmm.livejournal.com',
 'coptersworld-ru.turbopages.org': 'coptersworld.ru',
 'rio--restoran-ru.turbopages.org': 'rio-restoran.ru',
 'nn-versia-ru.turbopages.org': 'nn.versia.ru',
 'fishmanual-ru.turbopages.org': 'fishmanual.ru',
 'allboxing-ru.turbopages.org': 'allboxing.ru',
 'cnc--maniac-ru.turbopages.org': 'cnc-maniac.ru',
 'extxe-com.turbopages.org': 'extxe.com',
 'kraeved1147-ru.turbopages.org': 'kraeved1147.ru',
 'bizgaz-ru.turbopages.org': 'bizgaz.ru',
 'dayflowers-ru.turbopages.org': 'dayflowers.ru',
 'moftop-ru.turbopages.org': 'moftop.ru',
 '2009--da-livejournal-com.turbopages.org': '2009-da.livejournal.com',
 'imaman-ru.turbopages.org': 'imaman.ru',
 'likefishing-ru.turbopages.org': 'likefishing.ru',
 'f1news-ru.turbopages.org': 'f1news.ru',
 'kirra808i8-livejournal-com.turbopages.org': 'kirra808i8.livejournal.com',
 'soullaway-livejournal-com.turbopages.org': 'soullaway.livejournal.com',
 'idealtrip-ru.turbopages.org': 'idealtrip.ru',
 'imp--navigator-livejournal-com.turbopages.org': 'imp-navigator.livejournal.com',
 'orel-kp-ru.turbopages.org': 'orel.kp.ru',
 'frazy-su.turbopages.org': 'frazy.su',
 'matoos-livejournal-com.turbopages.org': 'matoos.livejournal.com',
 'pojaru-net-ru.turbopages.org': 'pojaru.net.ru',
 'ceim-ru.turbopages.org': 'ceim.ru',
 'avto--second-ru.turbopages.org': 'avto-second.ru',
 'room--plant-ru.turbopages.org': 'room-plant.ru',
 'vkaznu-ru.turbopages.org': 'vkaznu.ru',
 'stroitelexpert-ru.turbopages.org': 'stroitelexpert.ru',
 'kursktv-ru.turbopages.org': 'kursktv.ru',
 'sharov--a--s-livejournal-com.turbopages.org': 'sharov-a-s.livejournal.com',
 'mk--yamal-ru.turbopages.org': 'mk-yamal.ru',
 'fedorova--tl-livejournal-com.turbopages.org': 'fedorova-tl.livejournal.com',
 'kaschess-ru.turbopages.org': 'kaschess.ru',
 'crimeapress-info.turbopages.org': 'crimeapress.info',
 'skupkapiter-ru.turbopages.org': 'skupkapiter.ru',
 'xn----8sb1agibddyj.turbopages.org': 'xn--8sb1agibddyj',
 'vittasim-livejournal-com.turbopages.org': 'vittasim.livejournal.com',
 'novostic-ru.turbopages.org': 'novostic.ru',
 'zakon--i--poryadok-com.turbopages.org': 'zakon-i-poryadok.com',
 'burdacentr-ru.turbopages.org': 'burdacentr.ru',
 'polzavred-ru.turbopages.org': 'polzavred.ru',
 'dietologiya-info.turbopages.org': 'dietologiya.info',
 'prokat40-ru.turbopages.org': 'prokat40.ru',
 'ateist66-livejournal-com.turbopages.org': 'ateist66.livejournal.com',
 'ledikic-ru.turbopages.org': 'ledikic.ru',
 'foragro-ru.turbopages.org': 'foragro.ru',
 'kinosvami-ru.turbopages.org': 'kinosvami.ru',
 'howmeow-ru.turbopages.org': 'howmeow.ru',
 'veryclose-ru.turbopages.org': 'veryclose.ru',
 'krechet--d-livejournal-com.turbopages.org': 'krechet-d.livejournal.com',
 'kontinent--mu-livejournal-com.turbopages.org': 'kontinent-mu.livejournal.com',
 'lapku-ru.turbopages.org': 'lapku.ru',
 'xn------7cduuaqqkjkykh8b.turbopages.org': 'xn---7cduuaqqkjkykh8b',
 'rosavto--spb-ru.turbopages.org': 'rosavto-spb.ru',
 'apetrochenkov-livejournal-com.turbopages.org': 'apetrochenkov.livejournal.com',
 'rodline-livejournal-com.turbopages.org': 'rodline.livejournal.com',
 'yamal-kp-ru.turbopages.org': 'yamal.kp.ru',
 'rutvstar-ru.turbopages.org': 'rutvstar.ru',
 'teplodom24-ru.turbopages.org': 'teplodom24.ru',
 'okno--pro-ru.turbopages.org': 'okno-pro.ru',
 'avtomonitor-ru.turbopages.org': 'avtomonitor.ru',
 'orif64-ru.turbopages.org': 'orif64.ru',
 'xn----7sbfmb1cfoceem7a.turbopages.org': 'xn--7sbfmb1cfoceem7a',
 'transponder--info-ru.turbopages.org': 'transponder-info.ru',
 'hiterbober-ru.turbopages.org': 'hiterbober.ru',
 'zrkuban-ru.turbopages.org': 'zrkuban.ru',
 'cr2-livejournal-com.turbopages.org': 'cr2.livejournal.com',
 'vince--crane-livejournal-com.turbopages.org': 'vince-crane.livejournal.com',
 'pushkino--2009-livejournal-com.turbopages.org': 'pushkino-2009.livejournal.com',
 'gidrozanet-ru.turbopages.org': 'gidrozanet.ru',
 'bigfish1-ru.turbopages.org': 'bigfish1.ru',
 'kolovrat2017-livejournal-com.turbopages.org': 'kolovrat2017.livejournal.com',
 'mirbetona-info.turbopages.org': 'mirbetona.info',
 'photo--and--travels-ru.turbopages.org': 'photo-and-travels.ru',
 'news--kmv-ru.turbopages.org': 'news-kmv.ru',
 'vseglisty-ru.turbopages.org': 'vseglisty.ru',
 'letselsovet-ru.turbopages.org': 'letselsovet.ru',
 'gagz-ru.turbopages.org': 'gagz.ru',
 'leo--dikiy-livejournal-com.turbopages.org': 'leo-dikiy.livejournal.com',
 'toyota--auto2-ru.turbopages.org': 'toyota-auto2.ru',
 'ladaria-livejournal-com.turbopages.org': 'ladaria.livejournal.com',
 'crossarea-ru.turbopages.org': 'crossarea.ru',
 'progulkino-livejournal-com.turbopages.org': 'progulkino.livejournal.com',
 'avto--mira-livejournal-com.turbopages.org': 'avto-mira.livejournal.com',
 'vozduxx-livejournal-com.turbopages.org': 'vozduxx.livejournal.com',
 'yandex--station-ru.turbopages.org': 'yandex-station.ru',
 'kremll-info.turbopages.org': 'kremll.info',
 'carabaas-livejournal-com.turbopages.org': 'carabaas.livejournal.com',
 'a--nalgin-livejournal-com.turbopages.org': 'a-nalgin.livejournal.com',
 'ijevsk-bezformata-com.turbopages.org': 'ijevsk.bezformata.com',
 'pnzdrive-ru.turbopages.org': 'pnzdrive.ru',
 'blitz-plus.turbopages.org': 'blitz.plus',
 'vipetroff-livejournal-com.turbopages.org': 'vipetroff.livejournal.com',
 'polzaivrededy-ru.turbopages.org': 'polzaivrededy.ru',
 'earth--chronicles-ru.turbopages.org': 'earth-chronicles.ru',
 'klimaks24-ru.turbopages.org': 'klimaks24.ru',
 'vile--gnus-livejournal-com.turbopages.org': 'vile-gnus.livejournal.com',
 'resholeg-livejournal-com.turbopages.org': 'resholeg.livejournal.com',
 'asx--club-su.turbopages.org': 'asx-club.su',
 'zaim--click-ru.turbopages.org': 'zaim-click.ru',
 'viplawyer-ru.turbopages.org': 'viplawyer.ru',
 'dimon--porter-livejournal-com.turbopages.org': 'dimon-porter.livejournal.com',
 'teplosten24-ru.turbopages.org': 'teplosten24.ru',
 'otdelkadom--surgut-ru.turbopages.org': 'otdelkadom-surgut.ru',
 'vasilisc-com.turbopages.org': 'vasilisc.com',
 'ablexur-ru.turbopages.org': 'ablexur.ru',
 'dassie2001-livejournal-com.turbopages.org': 'dassie2001.livejournal.com',
 'antiguo--hidalgo-livejournal-com.turbopages.org': 'antiguo-hidalgo.livejournal.com',
 'p--o--c--o--m--a--h--a-livejournal-com.turbopages.org': 'p-o-c-o-m-a-h-a.livejournal.com',
 'naim-guru.turbopages.org': 'naim.guru',
 'resident--063-livejournal-com.turbopages.org': 'resident-063.livejournal.com',
 'akry-livejournal-com.turbopages.org': 'akry.livejournal.com',
 'ppp--kaluga-ru.turbopages.org': 'ppp-kaluga.ru',
 'instrukciya--otzyvy-ru.turbopages.org': 'instrukciya-otzyvy.ru',
 'teplospec-com.turbopages.org': 'teplospec.com',
 'vologda-bezformata-com.turbopages.org': 'vologda.bezformata.com',
 'anapa--ch-ru.turbopages.org': 'anapa-ch.ru',
 'krasnodar-bezformata-com.turbopages.org': 'krasnodar.bezformata.com',
 'delokuhni-ru.turbopages.org': 'delokuhni.ru',
 'lingust-ru.turbopages.org': 'lingust.ru',
 'gastroscan-livejournal-com.turbopages.org': 'gastroscan.livejournal.com',
 'masterclassy-ru.turbopages.org': 'masterclassy.ru',
 'etnoperm-ru.turbopages.org': 'etnoperm.ru',
 'avt1975-livejournal-com.turbopages.org': 'avt1975.livejournal.com',
 'exabait-ru.turbopages.org': 'exabait.ru',
 'byacs-livejournal-com.turbopages.org': 'byacs.livejournal.com',
 'qashqairus-ru.turbopages.org': 'qashqairus.ru',
 'netkoblog-livejournal-com.turbopages.org': 'netkoblog.livejournal.com',
 'felisata-livejournal-com.turbopages.org': 'felisata.livejournal.com',
 'burcevka-ru.turbopages.org': 'burcevka.ru',
 'amikron-livejournal-com.turbopages.org': 'amikron.livejournal.com',
 'avtodoc24-ru.turbopages.org': 'avtodoc24.ru',
 'dinozavriki-com.turbopages.org': 'dinozavriki.com',
 'pattisony-ru.turbopages.org': 'pattisony.ru',
 'ufa-rbc-ru.turbopages.org': 'ufa.rbc.ru',
 'proverkato-ru.turbopages.org': 'proverkato.ru',
 'vannajakomnata-ru.turbopages.org': 'vannajakomnata.ru',
 'autoxs-ru.turbopages.org': 'autoxs.ru',
 'driver--helper-ru.turbopages.org': 'driver-helper.ru',
 'gorodskoyportal-ru.turbopages.org': 'gorodskoyportal.ru',
 'uglich--jj-livejournal-com.turbopages.org': 'uglich-jj.livejournal.com',
 'vologda-kp-ru.turbopages.org': 'vologda.kp.ru',
 'diz--kitchen-ru.turbopages.org': 'diz-kitchen.ru',
 's--voi-ru.turbopages.org': 's-voi.ru',
 'dachnijsovet-ru.turbopages.org': 'dachnijsovet.ru',
 'essens--market-ru.turbopages.org': 'essens-market.ru',
 'slavikap-livejournal-com.turbopages.org': 'slavikap.livejournal.com',
 'filaretuos-livejournal-com.turbopages.org': 'filaretuos.livejournal.com',
 'ferma--biz-ru.turbopages.org': 'ferma-biz.ru',
 'domovodstvo--online-ru.turbopages.org': 'domovodstvo-online.ru',
 'newogorod-ru.turbopages.org': 'newogorod.ru',
 'm--introduction-livejournal-com.turbopages.org': 'm-introduction.livejournal.com',
 'perfume007-livejournal-com.turbopages.org': 'perfume007.livejournal.com',
 'usedcars-ru.turbopages.org': 'usedcars.ru',
 'uborka-co.turbopages.org': 'uborka.co',
 'nastroeniya-net.turbopages.org': 'nastroeniya.net',
 'krsk-aif-ru.turbopages.org': 'krsk.aif.ru',
 'avitodeli-ru.turbopages.org': 'avitodeli.ru',
 'str--mebel-ru.turbopages.org': 'str-mebel.ru',
 'avtorele24-ru.turbopages.org': 'avtorele24.ru',
 'yr--expert-com.turbopages.org': 'yr-expert.com',
 'p--syutkin-livejournal-com.turbopages.org': 'p-syutkin.livejournal.com',
 'kononenko-livejournal-com.turbopages.org': 'kononenko.livejournal.com',
 'vladimir--ses-ru.turbopages.org': 'vladimir-ses.ru',
 'perm-dixinews-ru.turbopages.org': 'perm.dixinews.ru',
 'bsodstop-ru.turbopages.org': 'bsodstop.ru',
 'socgramota-ru.turbopages.org': 'socgramota.ru',
 'aleksei--sobolev-livejournal-com.turbopages.org': 'aleksei-sobolev.livejournal.com',
 'vneklas--chas-ru.turbopages.org': 'vneklas-chas.ru',
 'mstervochka-livejournal-com.turbopages.org': 'mstervochka.livejournal.com',
 'immi--i-com.turbopages.org': 'immi-i.com',
 'ollforkids-ru.turbopages.org': 'ollforkids.ru',
 'nicespb-ru.turbopages.org': 'nicespb.ru',
 'starcom68-livejournal-com.turbopages.org': 'starcom68.livejournal.com',
 'roomavto-ru.turbopages.org': 'roomavto.ru',
 'ankon--perm-ru.turbopages.org': 'ankon-perm.ru',
 'pervyebljuda-ru.turbopages.org': 'pervyebljuda.ru',
 'ketrasha-ru.turbopages.org': 'ketrasha.ru',
 'ivanovo-aif-ru.turbopages.org': 'ivanovo.aif.ru',
 'otdeln-ru.turbopages.org': 'otdeln.ru',
 'xn------5cdmdjafea7d8aqvk7b0a3l.turbopages.org': 'xn---5cdmdjafea7d8aqvk7b0a3l',
 'nastalgya-livejournal-com.turbopages.org': 'nastalgya.livejournal.com',
 'rys--strategia-livejournal-com.turbopages.org': 'rys-strategia.livejournal.com',
 'zdesvamnetut100-livejournal-com.turbopages.org': 'zdesvamnetut100.livejournal.com',
 'taen--1-livejournal-com.turbopages.org': 'taen-1.livejournal.com',
 'zbaza-livejournal-com.turbopages.org': 'zbaza.livejournal.com',
 'banguerski--alex-livejournal-com.turbopages.org': 'banguerski-alex.livejournal.com',
 'diak--kuraev-livejournal-com.turbopages.org': 'diak-kuraev.livejournal.com',
 'uridich-ru.turbopages.org': 'uridich.ru',
 'expertchistoti-ru.turbopages.org': 'expertchistoti.ru',
 'kirr--zah-livejournal-com.turbopages.org': 'kirr-zah.livejournal.com',
 'topdialog-ru.turbopages.org': 'topdialog.ru',
 'nachkar-ru.turbopages.org': 'nachkar.ru',
 'prousadbi-ru.turbopages.org': 'prousadbi.ru',
 'qkempek-livejournal-com.turbopages.org': 'qkempek.livejournal.com',
 'olegmatveev-livejournal-com.turbopages.org': 'olegmatveev.livejournal.com',
 'medspecial-ru.turbopages.org': 'medspecial.ru',
 'ogbors-livejournal-com.turbopages.org': 'ogbors.livejournal.com',
 'aqua--rmnt-com.turbopages.org': 'aqua-rmnt.com',
 'decorobot-ru.turbopages.org': 'decorobot.ru',
 'mirivlad-ru.turbopages.org': 'mirivlad.ru',
 'soulim-livejournal-com.turbopages.org': 'soulim.livejournal.com',
 'lastochkanata-livejournal-com.turbopages.org': 'lastochkanata.livejournal.com',
 'grushenka-livejournal-com.turbopages.org': 'grushenka.livejournal.com',
 'sirin--from--shrm-livejournal-com.turbopages.org': 'sirin-from-shrm.livejournal.com',
 'd--popovskiy-livejournal-com.turbopages.org': 'd-popovskiy.livejournal.com',
 'computernotes-ru.turbopages.org': 'computernotes.ru',
 'yaroslavl-bezformata-com.turbopages.org': 'yaroslavl.bezformata.com',
 'panoramy--chernobylya-ru.turbopages.org': 'panoramy-chernobylya.ru',
 'mein--gott--inri-livejournal-com.turbopages.org': 'mein-gott-inri.livejournal.com',
 'klops-ru.turbopages.org': 'klops.ru',
 'momenty-org.turbopages.org': 'momenty.org',
 'xperia--droid-ru.turbopages.org': 'xperia-droid.ru',
 'skurlatov1-livejournal-com.turbopages.org': 'skurlatov1.livejournal.com',
 'ues--company-ru.turbopages.org': 'ues-company.ru',
 'oilcapital-ru.turbopages.org': 'oilcapital.ru',
 'samarski--forum-livejournal-com.turbopages.org': 'samarski-forum.livejournal.com',
 'olganechkina-livejournal-com.turbopages.org': 'olganechkina.livejournal.com',
 'youmanual-ru.turbopages.org': 'youmanual.ru',
 'sergeydolya-livejournal-com.turbopages.org': 'sergeydolya.livejournal.com',
 'newsorel-ru.turbopages.org': 'newsorel.ru',
 'yanson-livejournal-com.turbopages.org': 'yanson.livejournal.com',
 'colortours-ru.turbopages.org': 'colortours.ru',
 'beshbel-livejournal-com.turbopages.org': 'beshbel.livejournal.com',
 'rusfootball-info.turbopages.org': 'rusfootball.info',
 'luckyea77-livejournal-com.turbopages.org': 'luckyea77.livejournal.com',
 'gup--veles-ru.turbopages.org': 'gup-veles.ru',
 'omedvet-ru.turbopages.org': 'omedvet.ru',
 'itexn-com.turbopages.org': 'itexn.com',
 'princessovna-ru.turbopages.org': 'princessovna.ru',
 'putevojdnevnik-ru.turbopages.org': 'putevojdnevnik.ru',
 'med39-ru.turbopages.org': 'med39.ru',
 'severinform-ru.turbopages.org': 'severinform.ru',
 'gorodvo-ru.turbopages.org': 'gorodvo.ru',
 'sandizain-ru.turbopages.org': 'sandizain.ru',
 'automotokit-ru.turbopages.org': 'automotokit.ru',
 'andreas--zarus-livejournal-com.turbopages.org': 'andreas-zarus.livejournal.com',
 'opttorgsar-ru.turbopages.org': 'opttorgsar.ru',
 'inco--list-ru.turbopages.org': 'inco-list.ru',
 'xn----itbjh2ae2am3dza.turbopages.org': 'xn--itbjh2ae2am3dza',
 'ponomaryov--oleg-livejournal-com.turbopages.org': 'ponomaryov-oleg.livejournal.com',
 'the--satori-net.turbopages.org': 'the-satori.net',
 'bike--rampage-ru.turbopages.org': 'bike-rampage.ru',
 'whattonews-ru.turbopages.org': 'whattonews.ru',
 'autozona54-ru.turbopages.org': 'autozona54.ru',
 'svoysredychuzih-livejournal-com.turbopages.org': 'svoysredychuzih.livejournal.com',
 'allnari-livejournal-com.turbopages.org': 'allnari.livejournal.com',
 'modelman-livejournal-com.turbopages.org': 'modelman.livejournal.com',
 'rex--net-livejournal-com.turbopages.org': 'rex-net.livejournal.com',
 'stevekyiv-livejournal-com.turbopages.org': 'stevekyiv.livejournal.com',
 'anna--bpguide-livejournal-com.turbopages.org': 'anna-bpguide.livejournal.com',
 'pitanov-livejournal-com.turbopages.org': 'pitanov.livejournal.com',
 'kremlinvorota-ru.turbopages.org': 'kremlinvorota.ru',
 'bestmoto-pro.turbopages.org': 'bestmoto.pro',
 'misteria-livejournal-com.turbopages.org': 'misteria.livejournal.com',
 'prolestnici-com.turbopages.org': 'prolestnici.com',
 'pdds-ru.turbopages.org': 'pdds.ru',
 'moped--info-ru.turbopages.org': 'moped-info.ru',
 'vashnozh-com.turbopages.org': 'vashnozh.com',
 'mojcar-ru.turbopages.org': 'mojcar.ru',
 'tizdorov24-ru.turbopages.org': 'tizdorov24.ru',
 'anatomiya--zdoroviya-ru.turbopages.org': 'anatomiya-zdoroviya.ru',
 'masterinterera-ru.turbopages.org': 'masterinterera.ru',
 'innov-ru.turbopages.org': 'innov.ru',
 'dor--zhur-livejournal-com.turbopages.org': 'dor-zhur.livejournal.com',
 'dogkind-ru.turbopages.org': 'dogkind.ru',
 'dietaprosto-com.turbopages.org': 'dietaprosto.com',
 'bezdiabet-ru.turbopages.org': 'bezdiabet.ru',
 'herson-tsargrad-tv.turbopages.org': 'herson.tsargrad.tv',
 'zarplatawmz-ru.turbopages.org': 'zarplatawmz.ru',
 'iklubkov-livejournal-com.turbopages.org': 'iklubkov.livejournal.com',
 'akbinfo-ru.turbopages.org': 'akbinfo.ru',
 'rukivnogi-com.turbopages.org': 'rukivnogi.com',
 'terve-su.turbopages.org': 'terve.su',
 'eustress-ru.turbopages.org': 'eustress.ru',
 'kakvtiktoke-ru.turbopages.org': 'kakvtiktoke.ru',
 'rivka381-livejournal-com.turbopages.org': 'rivka381.livejournal.com',
 'sekutor-livejournal-com.turbopages.org': 'sekutor.livejournal.com',
 'tur--gid-net.turbopages.org': 'tur-gid.net',
 'beobaxter-livejournal-com.turbopages.org': 'beobaxter.livejournal.com',
 'do--slez-com.turbopages.org': 'do-slez.com',
 'goroskop2021-ru.turbopages.org': 'goroskop2021.ru',
 'life--steps-ru.turbopages.org': 'life-steps.ru',
 'topcor-ru.turbopages.org': 'topcor.ru',
 'antipan-livejournal-com.turbopages.org': 'antipan.livejournal.com',
 'pihovkin1-livejournal-com.turbopages.org': 'pihovkin1.livejournal.com',
 'ehrlichperson-livejournal-com.turbopages.org': 'ehrlichperson.livejournal.com',
 'gorkogo29-ru.turbopages.org': 'gorkogo29.ru',
 'murmansk-mk-ru.turbopages.org': 'murmansk.mk.ru',
 'zt29-ru.turbopages.org': 'zt29.ru',
 'nail--design--photos-ru.turbopages.org': 'nail-design-photos.ru',
 'churochkin-livejournal-com.turbopages.org': 'churochkin.livejournal.com',
 'doctorsemash-com.turbopages.org': 'doctorsemash.com',
 'ug--ur-com.turbopages.org': 'ug-ur.com',
 'azbukadiet-ru.turbopages.org': 'azbukadiet.ru',
 'maximum-ru.turbopages.org': 'maximum.ru',
 'eburg-mk-ru.turbopages.org': 'eburg.mk.ru',
 'nalogobzor-info.turbopages.org': 'nalogobzor.info',
 'dbs8-ru.turbopages.org': 'dbs8.ru',
 'alexsvatov-livejournal-com.turbopages.org': 'alexsvatov.livejournal.com',
 'calculatori-ru.turbopages.org': 'calculatori.ru',
 'ugra-kp-ru.turbopages.org': 'ugra.kp.ru',
 'kuda--mo-ru.turbopages.org': 'kuda-mo.ru',
 'chetvergvecher-livejournal-com.turbopages.org': 'chetvergvecher.livejournal.com',
 '4esnok-by.turbopages.org': '4esnok.by',
 'svoydom-info.turbopages.org': 'svoydom.info',
 'storyone-ru.turbopages.org': 'storyone.ru',
 'fere--expertt-livejournal-com.turbopages.org': 'fere-expertt.livejournal.com',
 'prorezume-com.turbopages.org': 'prorezume.com',
 'fishki--remonta-ru.turbopages.org': 'fishki-remonta.ru',
 'planeta--mhpi-livejournal-com.turbopages.org': 'planeta-mhpi.livejournal.com',
 'au--rella-livejournal-com.turbopages.org': 'au-rella.livejournal.com',
 'isfix-ru.turbopages.org': 'isfix.ru',
 'posuda--inet-ru.turbopages.org': 'posuda-inet.ru',
 'xn----------ogglccgdgg8a6c0afq5bpu9ewa7nsde.turbopages.org': 'xn-----ogglccgdgg8a6c0afq5bpu9ewa7nsde',
 'rvsmaster-ru.turbopages.org': 'rvsmaster.ru',
 'hmelectro-ru.turbopages.org': 'hmelectro.ru',
 'encom74-ru.turbopages.org': 'encom74.ru',
 'postroika-biz.turbopages.org': 'postroika.biz',
 'jirosjigateli--dlya--myjchin-ru.turbopages.org': 'jirosjigateli-dlya-myjchin.ru',
 'fc--dynamo-ru.turbopages.org': 'fc-dynamo.ru',
 '2auto-su.turbopages.org': '2auto.su',
 'piskunov--vitaly-livejournal-com.turbopages.org': 'piskunov-vitaly.livejournal.com',
 'wlgames-ru.turbopages.org': 'wlgames.ru',
 'grozny--inform-ru.turbopages.org': 'grozny-inform.ru',
 'utro--news-ru.turbopages.org': 'utro-news.ru',
 'interes2012-livejournal-com.turbopages.org': 'interes2012.livejournal.com',
 'hwp-ru.turbopages.org': 'hwp.ru',
 'guidepc-ru.turbopages.org': 'guidepc.ru',
 'remontnik--pk-ru.turbopages.org': 'remontnik-pk.ru',
 '3ruble-livejournal-com.turbopages.org': '3ruble.livejournal.com',
 'samchef-ru.turbopages.org': 'samchef.ru',
 'vkys-info.turbopages.org': 'vkys.info',
 'sevastopol-kp-ru.turbopages.org': 'sevastopol.kp.ru',
 'cyber-pressball-by.turbopages.org': 'cyber.pressball.by',
 'dailyfitnessblog-ru.turbopages.org': 'dailyfitnessblog.ru',
 'voditelexpert-ru.turbopages.org': 'voditelexpert.ru',
 'xn---com-u4dr9a2bpo.turbopages.org': 'xn-.com.u4dr9a2bpo',
 'mrmetall-ru.turbopages.org': 'mrmetall.ru',
 'ezoterik--page-com.turbopages.org': 'ezoterik-page.com',
 'migrant--fms-ru.turbopages.org': 'migrant-fms.ru',
 'kommari-livejournal-com.turbopages.org': 'kommari.livejournal.com',
 'rkzsp-ru.turbopages.org': 'rkzsp.ru',
 'tolstiyyoj-livejournal-com.turbopages.org': 'tolstiyyoj.livejournal.com',
 'kem-kp-ru.turbopages.org': 'kem.kp.ru',
 'frengs-livejournal-com.turbopages.org': 'frengs.livejournal.com',
 'uta--kryakva-livejournal-com.turbopages.org': 'uta-kryakva.livejournal.com',
 'sherlock--serial-livejournal-com.turbopages.org': 'sherlock-serial.livejournal.com',
 'ufa--medtehnika-ru.turbopages.org': 'ufa-medtehnika.ru',
 'mymammy-info.turbopages.org': 'mymammy.info',
 'sdelano--u--nas-livejournal-com.turbopages.org': 'sdelano-u-nas.livejournal.com',
 'yakutia-mk-ru.turbopages.org': 'yakutia.mk.ru',
 'petrozavodsk-bezformata-com.turbopages.org': 'petrozavodsk.bezformata.com',
 'fergana-ru.turbopages.org': 'fergana.ru',
 'modnyeidei-ru.turbopages.org': 'modnyeidei.ru',
 'udm-aif-ru.turbopages.org': 'udm.aif.ru',
 'it88-ru.turbopages.org': 'it88.ru',
 'vmeste--po--uralu-livejournal-com.turbopages.org': 'vmeste-po-uralu.livejournal.com',
 'peremogi-livejournal-com.turbopages.org': 'peremogi.livejournal.com',
 'money-onliner-by.turbopages.org': 'money.onliner.by',
 'crazydog--lady-livejournal-com.turbopages.org': 'crazydog-lady.livejournal.com',
 'postirajka-ru.turbopages.org': 'postirajka.ru',
 'senseisekai-livejournal-com.turbopages.org': 'senseisekai.livejournal.com',
 'polevskoy-bezformata-com.turbopages.org': 'polevskoy.bezformata.com',
 'lasus-ru.turbopages.org': 'lasus.ru',
 'chinese--projectors-ru.turbopages.org': 'chinese-projectors.ru',
 'instrukzii-ru.turbopages.org': 'instrukzii.ru',
 'domovik-guru.turbopages.org': 'domovik.guru',
 'hair--ok-ru.turbopages.org': 'hair-ok.ru',
 'forsite--spb-ru.turbopages.org': 'forsite-spb.ru',
 'autozam-ru.turbopages.org': 'autozam.ru',
 'dmrog-livejournal-com.turbopages.org': 'dmrog.livejournal.com',
 'bryansk-kp-ru.turbopages.org': 'bryansk.kp.ru',
 'pani--yulya-livejournal-com.turbopages.org': 'pani-yulya.livejournal.com',
 'magic--mir-ru.turbopages.org': 'magic-mir.ru',
 'iralenejewellry-livejournal-com.turbopages.org': 'iralenejewellry.livejournal.com',
 'glukovarenik-livejournal-com.turbopages.org': 'glukovarenik.livejournal.com',
 'podmoskovye-bezformata-com.turbopages.org': 'podmoskovye.bezformata.com',
 'vrn-mk-ru.turbopages.org': 'vrn.mk.ru',
 '133bpm-livejournal-com.turbopages.org': '133bpm.livejournal.com',
 'irewi-livejournal-com.turbopages.org': 'irewi.livejournal.com',
 'amrom-ru.turbopages.org': 'amrom.ru',
 'wiki-merionet-ru.turbopages.org': 'wiki.merionet.ru',
 'excel2-ru.turbopages.org': 'excel2.ru',
 'domoticzfaq-ru.turbopages.org': 'domoticzfaq.ru',
 'lady-pravda-ru.turbopages.org': 'lady.pravda.ru',
 'bookinfa-ru.turbopages.org': 'bookinfa.ru',
 'srokigodnosti-ru.turbopages.org': 'srokigodnosti.ru',
 'thabank-ru.turbopages.org': 'thabank.ru',
 'uznayki-ru.turbopages.org': 'uznayki.ru',
 'clubshuttle-ru.turbopages.org': 'clubshuttle.ru',
 'yujnyj51-livejournal-com.turbopages.org': 'yujnyj51.livejournal.com',
 'berdsk--online-ru.turbopages.org': 'berdsk-online.ru',
 'berdsk-bezformata-com.turbopages.org': 'berdsk.bezformata.com',
 'dachnye--sovety-ru.turbopages.org': 'dachnye-sovety.ru',
 'sait--pro--dachu-ru.turbopages.org': 'sait-pro-dachu.ru',
 'dacha2u-ru.turbopages.org': 'dacha2u.ru',
 'home--ideas-ru.turbopages.org': 'home-ideas.ru',
 'aobe-ru.turbopages.org': 'aobe.ru',
 'k--markarian-livejournal-com.turbopages.org': 'k-markarian.livejournal.com',
 'muzhik-guru.turbopages.org': 'muzhik.guru',
 'hr--elearning-ru.turbopages.org': 'hr-elearning.ru',
 'its--city-ru.turbopages.org': 'its-city.ru',
 'pavlovskiyposad-bezformata-com.turbopages.org': 'pavlovskiyposad.bezformata.com',
 'ryzhaya--kysya-livejournal-com.turbopages.org': 'ryzhaya-kysya.livejournal.com',
 'luberci-bezformata-com.turbopages.org': 'luberci.bezformata.com',
 'bankiclub-ru.turbopages.org': 'bankiclub.ru',
 'plush--koala-livejournal-com.turbopages.org': 'plush-koala.livejournal.com',
 'surgut-big--book--edu-ru.turbopages.org': 'surgut.big-book-edu.ru',
 'vgae-ru.turbopages.org': 'vgae.ru',
 'max--vps-ru.turbopages.org': 'max-vps.ru',
 'mnogodetki--msk-livejournal-com.turbopages.org': 'mnogodetki-msk.livejournal.com',
 'lifeinsect-ru.turbopages.org': 'lifeinsect.ru',
 'flagof-ru.turbopages.org': 'flagof.ru',
 'chtozainstrument-ru.turbopages.org': 'chtozainstrument.ru',
 'lentv24-ru.turbopages.org': 'lentv24.ru',
 'tolyatti--news-net.turbopages.org': 'tolyatti-news.net',
 'myrtus--niveus-livejournal-com.turbopages.org': 'myrtus-niveus.livejournal.com',
 'kazved-ru.turbopages.org': 'kazved.ru',
 'spectr--lkm-ru.turbopages.org': 'spectr-lkm.ru',
 'psiheya--market-ru.turbopages.org': 'psiheya-market.ru',
 'vusadebke-com.turbopages.org': 'vusadebke.com',
 'konspektiruem-ru.turbopages.org': 'konspektiruem.ru',
 'it--is--too--much-livejournal-com.turbopages.org': 'it-is-too-much.livejournal.com',
 'svoya--shveyka-livejournal-com.turbopages.org': 'svoya-shveyka.livejournal.com',
 'spirtov-net.turbopages.org': 'spirtov.net',
 'yakutia--daily-ru.turbopages.org': 'yakutia-daily.ru',
 'luojatar-ru.turbopages.org': 'luojatar.ru',
 'raven--yellow-livejournal-com.turbopages.org': 'raven-yellow.livejournal.com',
 'tragemata-livejournal-com.turbopages.org': 'tragemata.livejournal.com',
 'pushcity-ru.turbopages.org': 'pushcity.ru',
 'deputat--morozov-livejournal-com.turbopages.org': 'deputat-morozov.livejournal.com',
 'useful--faq-livejournal-com.turbopages.org': 'useful-faq.livejournal.com',
 'koshke-info.turbopages.org': 'koshke.info',
 'kaluga-kp-ru.turbopages.org': 'kaluga.kp.ru',
 'kotlausaite-ru.turbopages.org': 'kotlausaite.ru',
 'otravleniya-net.turbopages.org': 'otravleniya.net',
 'alex--rozoff-livejournal-com.turbopages.org': 'alex-rozoff.livejournal.com',
 'pryf-livejournal-com.turbopages.org': 'pryf.livejournal.com',
 'fitness--star74-ru.turbopages.org': 'fitness-star74.ru',
 'mibands-ru.turbopages.org': 'mibands.ru',
 'tests24-ru.turbopages.org': 'tests24.ru',
 'danuvius-livejournal-com.turbopages.org': 'danuvius.livejournal.com',
 'filmografiatv-ru.turbopages.org': 'filmografiatv.ru',
 'livebir-ru.turbopages.org': 'livebir.ru',
 'globusmira-ru.turbopages.org': 'globusmira.ru',
 'annawwts-com.turbopages.org': 'annawwts.com',
 'prohandmade-ru.turbopages.org': 'prohandmade.ru',
 'anygreece-com.turbopages.org': 'anygreece.com',
 'akriderm-com.turbopages.org': 'akriderm.com',
 'freevilladge-com.turbopages.org': 'freevilladge.com',
 'sci--hit-com.turbopages.org': 'sci-hit.com',
 'ru--spacegiraffe-livejournal-com.turbopages.org': 'ru-spacegiraffe.livejournal.com',
 'rosgvardiya-info.turbopages.org': 'rosgvardiya.info',
 'buhproffi-ru.turbopages.org': 'buhproffi.ru',
 'skolkozarabativaet-ru.turbopages.org': 'skolkozarabativaet.ru',
 'fermabusines-ru.turbopages.org': 'fermabusines.ru',
 'velolive-com.turbopages.org': 'velolive.com',
 'avto--all-com.turbopages.org': 'avto-all.com',
 'izich-info.turbopages.org': 'izich.info',
 'gq--blog-com.turbopages.org': 'gq-blog.com',
 'master--klass-livejournal-com.turbopages.org': 'master-klass.livejournal.com',
 'xn------7cdejnergd4azigb9bpe2dd0u.turbopages.org': 'xn---7cdejnergd4azigb9bpe2dd0u',
 'bbqcash-com.turbopages.org': 'bbqcash.com',
 'bitlite-ru.turbopages.org': 'bitlite.ru',
 'yola--poisk-ru.turbopages.org': 'yola-poisk.ru',
 'skoviaktualno-livejournal-com.turbopages.org': 'skoviaktualno.livejournal.com',
 'blau--kraehe-livejournal-com.turbopages.org': 'blau-kraehe.livejournal.com',
 'ebelyaev-livejournal-com.turbopages.org': 'ebelyaev.livejournal.com',
 'lenorlux-livejournal-com.turbopages.org': 'lenorlux.livejournal.com',
 'as--elvis-livejournal-com.turbopages.org': 'as-elvis.livejournal.com',
 'ezavi-ru.turbopages.org': 'ezavi.ru',
 'princip1-livejournal-com.turbopages.org': 'princip1.livejournal.com',
 'mne--30-ru.turbopages.org': 'mne-30.ru',
 'je--nny-livejournal-com.turbopages.org': 'je-nny.livejournal.com',
 'nemeckaja--ovcharka-ru.turbopages.org': 'nemeckaja-ovcharka.ru',
 'emigrantov-net.turbopages.org': 'emigrantov.net',
 'timetoboil-ru.turbopages.org': 'timetoboil.ru',
 'medovyidom-ru.turbopages.org': 'medovyidom.ru',
 'bilety--na--poezd-com.turbopages.org': 'bilety-na-poezd.com',
 'poltora--bobra-livejournal-com.turbopages.org': 'poltora-bobra.livejournal.com',
 'diwis-ru.turbopages.org': 'diwis.ru',
 'shkolarukodelia-ru.turbopages.org': 'shkolarukodelia.ru',
 'onshe33-livejournal-com.turbopages.org': 'onshe33.livejournal.com',
 'shtrich--kod-ru.turbopages.org': 'shtrich-kod.ru',
 'goscontract-info.turbopages.org': 'goscontract.info',
 'krasnoyarsk-bezformata-com.turbopages.org': 'krasnoyarsk.bezformata.com',
 'm--tsyganov-livejournal-com.turbopages.org': 'm-tsyganov.livejournal.com',
 'sichovka-ru.turbopages.org': 'sichovka.ru',
 'metarankings-ru.turbopages.org': 'metarankings.ru',
 'newxboxone-ru.turbopages.org': 'newxboxone.ru',
 'altr-livejournal-com.turbopages.org': 'altr.livejournal.com',
 'vashehobbyrf-ru.turbopages.org': 'vashehobbyrf.ru',
 'nokiagid-ru.turbopages.org': 'nokiagid.ru',
 'usas--mdk-livejournal-com.turbopages.org': 'usas-mdk.livejournal.com',
 'nikolaeva-livejournal-com.turbopages.org': 'nikolaeva.livejournal.com',
 'nb16-ru.turbopages.org': 'nb16.ru',
 'xn----7sbldamg8aeopdhp7b.turbopages.org': 'xn--7sbldamg8aeopdhp7b',
 'okna--veka64-ru.turbopages.org': 'okna-veka64.ru',
 'toptigki-livejournal-com.turbopages.org': 'toptigki.livejournal.com',
 'mkuznecov-ru.turbopages.org': 'mkuznecov.ru',
 'anothercity-livejournal-com.turbopages.org': 'anothercity.livejournal.com',
 'lux--stahl-ru.turbopages.org': 'lux-stahl.ru',
 'timelottery-ru.turbopages.org': 'timelottery.ru',
 'milewski--igor-livejournal-com.turbopages.org': 'milewski-igor.livejournal.com',
 'kiberbob2000-livejournal-com.turbopages.org': 'kiberbob2000.livejournal.com',
 'maxkatz-livejournal-com.turbopages.org': 'maxkatz.livejournal.com',
 'isurok-livejournal-com.turbopages.org': 'isurok.livejournal.com',
 'wodakachka-com.turbopages.org': 'wodakachka.com',
 'upravdom--msk-livejournal-com.turbopages.org': 'upravdom-msk.livejournal.com',
 'obacne-ru.turbopages.org': 'obacne.ru',
 'tainyisovetnik-livejournal-com.turbopages.org': 'tainyisovetnik.livejournal.com',
 'warcats-ru.turbopages.org': 'warcats.ru',
 'queyntefantasye-livejournal-com.turbopages.org': 'queyntefantasye.livejournal.com',
 'lavagra-livejournal-com.turbopages.org': 'lavagra.livejournal.com',
 'pashalena-livejournal-com.turbopages.org': 'pashalena.livejournal.com',
 'paruslex-ru.turbopages.org': 'paruslex.ru',
 'chidis-ru.turbopages.org': 'chidis.ru',
 'goincome-ru.turbopages.org': 'goincome.ru',
 'fedor--afanasiev-livejournal-com.turbopages.org': 'fedor-afanasiev.livejournal.com',
 'kibalchish75-livejournal-com.turbopages.org': 'kibalchish75.livejournal.com',
 'allbreakingnews-ru.turbopages.org': 'allbreakingnews.ru',
 'ovideo-ru.turbopages.org': 'ovideo.ru',
 'soberger-ru.turbopages.org': 'soberger.ru',
 'svetafonseca-livejournal-com.turbopages.org': 'svetafonseca.livejournal.com',
 'buki--v--ruki-livejournal-com.turbopages.org': 'buki-v-ruki.livejournal.com',
 'gold-1prime-ru.turbopages.org': 'gold.1prime.ru',
 'calm73-livejournal-com.turbopages.org': 'calm73.livejournal.com',
 'istra--boats-livejournal-com.turbopages.org': 'istra-boats.livejournal.com',
 'rf--passport-ru.turbopages.org': 'rf-passport.ru',
 'usolt-livejournal-com.turbopages.org': 'usolt.livejournal.com',
 'se7en-ws.turbopages.org': 'se7en.ws',
 '1msale-livejournal-com.turbopages.org': '1msale.livejournal.com',
 'vr419-ru.turbopages.org': 'vr419.ru',
 'exspertrieltor-ru.turbopages.org': 'exspertrieltor.ru',
 'zmantika-info.turbopages.org': 'zmantika.info',
 'ladyspecial-ru.turbopages.org': 'ladyspecial.ru',
 'postmodernism-livejournal-com.turbopages.org': 'postmodernism.livejournal.com',
 'crea87-livejournal-com.turbopages.org': 'crea87.livejournal.com',
 'babyplayroom-ru.turbopages.org': 'babyplayroom.ru',
 'gadgetsnews-net.turbopages.org': 'gadgetsnews.net',
 'rusovok-ru.turbopages.org': 'rusovok.ru',
 'kotologia-com.turbopages.org': 'kotologia.com',
 'izhevsk-big--book--med-ru.turbopages.org': 'izhevsk.big-book-med.ru',
 'gribokinfo-com.turbopages.org': 'gribokinfo.com',
 'kostyad-livejournal-com.turbopages.org': 'kostyad.livejournal.com',
 'worldsupp-com.turbopages.org': 'worldsupp.com',
 'znatokfinansov-ru.turbopages.org': 'znatokfinansov.ru',
 'stirkaved-ru.turbopages.org': 'stirkaved.ru',
 'nautro-livejournal-com.turbopages.org': 'nautro.livejournal.com',
 'hvatailopatu-ru.turbopages.org': 'hvatailopatu.ru',
 'd--russia-ru.turbopages.org': 'd-russia.ru',
 'mytischiriamo-ru.turbopages.org': 'mytischiriamo.ru',
 'generator98-ru.turbopages.org': 'generator98.ru',
 'omar--khayam-livejournal-com.turbopages.org': 'omar-khayam.livejournal.com',
 'master--pmg-ru.turbopages.org': 'master-pmg.ru',
 'levashove-ru.turbopages.org': 'levashove.ru',
 'gagarinbank-ru.turbopages.org': 'gagarinbank.ru',
 'anton--agarkov-livejournal-com.turbopages.org': 'anton-agarkov.livejournal.com',
 'v--kosmetikovna-livejournal-com.turbopages.org': 'v-kosmetikovna.livejournal.com',
 'plusfinance-ru.turbopages.org': 'plusfinance.ru',
 '1istochnik-ru.turbopages.org': '1istochnik.ru',
 'navigator--kirov-ru.turbopages.org': 'navigator-kirov.ru',
 'cultprosvet--mag-livejournal-com.turbopages.org': 'cultprosvet-mag.livejournal.com',
 'danil--vinokurov-livejournal-com.turbopages.org': 'danil-vinokurov.livejournal.com',
 'oleg--kozyrev-livejournal-com.turbopages.org': 'oleg-kozyrev.livejournal.com',
 'sonniki-guru.turbopages.org': 'sonniki.guru',
 'alexey--ionov-livejournal-com.turbopages.org': 'alexey-ionov.livejournal.com',
 'allovertheus-ru.turbopages.org': 'allovertheus.ru',
 'promtu-ru.turbopages.org': 'promtu.ru',
 'aauhadullin-ru.turbopages.org': 'aauhadullin.ru',
 'smolensk--news-net.turbopages.org': 'smolensk-news.net',
 'smolensk-bezformata-com.turbopages.org': 'smolensk.bezformata.com',
 'smolnews-ru.turbopages.org': 'smolnews.ru',
 'govoritufa-ru.turbopages.org': 'govoritufa.ru',
 'tech--touch-ru.turbopages.org': 'tech-touch.ru',
 'usa-one.turbopages.org': 'usa.one',
 'prosports-kz.turbopages.org': 'prosports.kz',
 'rulaws-ru.turbopages.org': 'rulaws.ru',
 'masterbanketa-livejournal-com.turbopages.org': 'masterbanketa.livejournal.com',
 'samvguvt-livejournal-com.turbopages.org': 'samvguvt.livejournal.com',
 'elenachebanenko-livejournal-com.turbopages.org': 'elenachebanenko.livejournal.com',
 'crazy--fishing-ru.turbopages.org': 'crazy-fishing.ru',
 'sandra--rimskaya-livejournal-com.turbopages.org': 'sandra-rimskaya.livejournal.com',
 'max--cars-ru.turbopages.org': 'max-cars.ru',
 'bally--36-livejournal-com.turbopages.org': 'bally-36.livejournal.com',
 'ufatime-ru.turbopages.org': 'ufatime.ru',
 'business--notebooks-ru.turbopages.org': 'business-notebooks.ru',
 'dimonvideo-ru.turbopages.org': 'dimonvideo.ru',
 'alvantara-livejournal-com.turbopages.org': 'alvantara.livejournal.com',
 'xpcom-ru.turbopages.org': 'xpcom.ru',
 'wiki--health-ru.turbopages.org': 'wiki-health.ru',
 'ru--psiholog-livejournal-com.turbopages.org': 'ru-psiholog.livejournal.com',
 'newscosmetology-com.turbopages.org': 'newscosmetology.com',
 'vladivostok--map-livejournal-com.turbopages.org': 'vladivostok-map.livejournal.com',
 'nashampure-ru.turbopages.org': 'nashampure.ru',
 'amulet--dzi-ru.turbopages.org': 'amulet-dzi.ru',
 'snorbinka-livejournal-com.turbopages.org': 'snorbinka.livejournal.com',
 'bez--sna-ru.turbopages.org': 'bez-sna.ru',
 'zoostatusefull-livejournal-com.turbopages.org': 'zoostatusefull.livejournal.com',
 'ykristinka-livejournal-com.turbopages.org': 'ykristinka.livejournal.com',
 've--kon-ru.turbopages.org': 've-kon.ru',
 'vmulder-livejournal-com.turbopages.org': 'vmulder.livejournal.com',
 'chekistka63-livejournal-com.turbopages.org': 'chekistka63.livejournal.com',
 'banket--dubrov-ru.turbopages.org': 'banket-dubrov.ru',
 'rostelekom1-ru.turbopages.org': 'rostelekom1.ru',
 'natalyland-ru.turbopages.org': 'natalyland.ru',
 'kamuflyzh-ru.turbopages.org': 'kamuflyzh.ru',
 'taroznanie-ru.turbopages.org': 'taroznanie.ru',
 'russian--money-ru.turbopages.org': 'russian-money.ru',
    'donpress-com.turbopages.org': 'donpress.com',
 'ukranews-com.turbopages.org': 'ukranews.com',
 'rulipozakonu-ru.turbopages.org': 'rulipozakonu.ru',
 'volos--t-livejournal-com.turbopages.org': 'volos-t.livejournal.com',
 'vladivostok--news-net.turbopages.org': 'vladivostok-news.net',
 'vestikavkaza-ru.turbopages.org': 'vestikavkaza.ru',
 'vlad-mk-ru.turbopages.org': 'vlad.mk.ru',
 'metispeople-livejournal-com.turbopages.org': 'metispeople.livejournal.com',
 'fountravel-ru.turbopages.org': 'fountravel.ru',
 'makulatura--list-ru.turbopages.org': 'makulatura-list.ru',
 'my--mask-ru.turbopages.org': 'my-mask.ru',
 'distonija-ru.turbopages.org': 'distonija.ru',
 'moyafirma-com.turbopages.org': 'moyafirma.com',
 'aniko--gas-ru.turbopages.org': 'aniko-gas.ru',
 'domostroydon-ru.turbopages.org': 'domostroydon.ru',
 'd--yatsenko-livejournal-com.turbopages.org': 'd-yatsenko.livejournal.com',
 'arigus-tv.turbopages.org': 'arigus.tv',
 'cameralabs-org.turbopages.org': 'cameralabs.org',
 '4tololo-ru.turbopages.org': '4tololo.ru',
 'irkutsk-bezformata-com.turbopages.org': 'irkutsk.bezformata.com',
 '24-kg.turbopages.org': '24.kg',
 'revmatolog-net.turbopages.org': 'revmatolog.net',
 'samara-domostroyrf-ru.turbopages.org': 'samara.domostroyrf.ru',
 'krivoe--zerkalo-ru.turbopages.org': 'krivoe-zerkalo.ru',
 'ysia-ru.turbopages.org': 'ysia.ru',
 'kid--book--museum-livejournal-com.turbopages.org': 'kid-book-museum.livejournal.com',
 'tumen-mk-ru.turbopages.org': 'tumen.mk.ru',
 'mrt--solikamsk-ru.turbopages.org': 'mrt-solikamsk.ru',
 'mstrok-ru.turbopages.org': 'mstrok.ru',
 'dzgo-ru.turbopages.org': 'dzgo.ru',
 'melt--spb-ru.turbopages.org': 'melt-spb.ru',
 'multivarka-info.turbopages.org': 'multivarka.info',
 'moya--shubka-ru.turbopages.org': 'moya-shubka.ru',
 'kofem-ru.turbopages.org': 'kofem.ru',
 'leopold1984-livejournal-com.turbopages.org': 'leopold1984.livejournal.com',
 'mag-relax-by.turbopages.org': 'mag.relax.by',
 'lipetskmedia-ru.turbopages.org': 'lipetskmedia.ru',
 'mybryansk-ru.turbopages.org': 'mybryansk.ru',
 'mobilegadjet-ru.turbopages.org': 'mobilegadjet.ru',
 'kartridge-org.turbopages.org': 'kartridge.org',
 'tstosterone-ru.turbopages.org': 'tstosterone.ru',
 'bobik-online.turbopages.org': 'bobik.online',
 'natyazhpotolki-ru.turbopages.org': 'natyazhpotolki.ru',
 'lada--avia-ru.turbopages.org': 'lada-avia.ru',
 'ugolovnoe--law-ru.turbopages.org': 'ugolovnoe-law.ru',
 'truesharing-ru.turbopages.org': 'truesharing.ru',
 'jurobzor-livejournal-com.turbopages.org': 'jurobzor.livejournal.com',
 'wins--go-livejournal-com.turbopages.org': 'wins-go.livejournal.com',
 'sexshopers-ru.turbopages.org': 'sexshopers.ru',
 'topavtoclub-ru.turbopages.org': 'topavtoclub.ru',
 'big--blue--heron-livejournal-com.turbopages.org': 'big-blue-heron.livejournal.com',
 'biznes--prost-ru.turbopages.org': 'biznes-prost.ru',
 'kaksekonomit-com.turbopages.org': 'kaksekonomit.com',
 'finansy-name.turbopages.org': 'finansy.name',
 'seyminfo-ru.turbopages.org': 'seyminfo.ru',
 'astrologi18-ru.turbopages.org': 'astrologi18.ru',
 'news--24-ru.turbopages.org': 'news-24.ru',
 'abcdpolk-livejournal-com.turbopages.org': 'abcdpolk.livejournal.com',
 'rci76-ru.turbopages.org': 'rci76.ru',
 'vesogorod-ru.turbopages.org': 'vesogorod.ru',
 'organic--fertil-ru.turbopages.org': 'organic-fertil.ru',
 'au--info-ru.turbopages.org': 'au-info.ru',
 'sasha--bogdanov-livejournal-com.turbopages.org': 'sasha-bogdanov.livejournal.com',
 'vestniksr-ru.turbopages.org': 'vestniksr.ru',
 'toolstver-ru.turbopages.org': 'toolstver.ru',
 'prostudio-ru.turbopages.org': 'prostudio.ru',
 'verybigfish-livejournal-com.turbopages.org': 'verybigfish.livejournal.com',
 'blog--invest-online.turbopages.org': 'blog-invest.online',
 'mygrind-ru.turbopages.org': 'mygrind.ru',
 'stmnsk-ru.turbopages.org': 'stmnsk.ru',
 'calc-ru.turbopages.org': 'calc.ru',
 'castlepeak-ru.turbopages.org': 'castlepeak.ru',
 'marpravda-ru.turbopages.org': 'marpravda.ru',
 'vernuti--tovar-ru.turbopages.org': 'vernuti-tovar.ru',
 'marimedia-ru.turbopages.org': 'marimedia.ru',
 'sindikativantsa-livejournal-com.turbopages.org': 'sindikativantsa.livejournal.com',
 'ivanetsoleg-livejournal-com.turbopages.org': 'ivanetsoleg.livejournal.com',
 'ne63ru-livejournal-com.turbopages.org': 'ne63ru.livejournal.com',
 'katia--lexx-livejournal-com.turbopages.org': 'katia-lexx.livejournal.com',
 'mnogoznaniy-com.turbopages.org': 'mnogoznaniy.com',
 'pristavkatv-ru.turbopages.org': 'pristavkatv.ru',
 'narkologiya--orel-ru.turbopages.org': 'narkologiya-orel.ru',
 'prodavlenie-online.turbopages.org': 'prodavlenie.online',
 'cardio--help-ru.turbopages.org': 'cardio-help.ru',
 'kprfrd-su.turbopages.org': 'kprfrd.su',
 'newsment-ru.turbopages.org': 'newsment.ru',
 'maja--dacha-ru.turbopages.org': 'maja-dacha.ru',
 'bezbukv-ru.turbopages.org': 'bezbukv.ru',
 'provisy-ru.turbopages.org': 'provisy.ru',
 'xn--apteka-103--qoj4c.turbopages.org': 'xn-apteka.103-qoj4c',
 'zm--sochi-livejournal-com.turbopages.org': 'zm-sochi.livejournal.com',
 '4joker-livejournal-com.turbopages.org': '4joker.livejournal.com',
 'rakarskiy-livejournal-com.turbopages.org': 'rakarskiy.livejournal.com',
 'virtual--sim-ru.turbopages.org': 'virtual-sim.ru',
 'dvinatoday-ru.turbopages.org': 'dvinatoday.ru',
 'lgbt--grani-livejournal-com.turbopages.org': 'lgbt-grani.livejournal.com',
 'gasmin--sib-ru.turbopages.org': 'gasmin-sib.ru',
 'yarreg-ru.turbopages.org': 'yarreg.ru',
 'runochka-ru.turbopages.org': 'runochka.ru',
 'finikova--malina-livejournal-com.turbopages.org': 'finikova-malina.livejournal.com',
 'genuspeha-ru.turbopages.org': 'genuspeha.ru',
 'boombate-com.turbopages.org': 'boombate.com',
 'vlast--sovetov-ru.turbopages.org': 'vlast-sovetov.ru',
 'autoflagman-ru.turbopages.org': 'autoflagman.ru',
 'vz--nn-ru.turbopages.org': 'vz-nn.ru',
 'hemofenix-ru.turbopages.org': 'hemofenix.ru',
 'amigo--pets-ru.turbopages.org': 'amigo-pets.ru',
 'belogorsk-bezformata-com.turbopages.org': 'belogorsk.bezformata.com',
 'centr--zanyatosti--naseleniya-ru.turbopages.org': 'centr-zanyatosti-naseleniya.ru',
 'zhkh-club.turbopages.org': 'zhkh.club',
 'rating--card-com.turbopages.org': 'rating-card.com',
 'laesus--de--liro-livejournal-com.turbopages.org': 'laesus-de-liro.livejournal.com',
 'arbuztoday-ru.turbopages.org': 'arbuztoday.ru',
 'building--ooo-ru.turbopages.org': 'building-ooo.ru',
 'odnushka-ru.turbopages.org': 'odnushka.ru',
 'wbcon-ru.turbopages.org': 'wbcon.ru',
 'readovka67-ru.turbopages.org': 'readovka67.ru',
 'prostoprosport-ru.turbopages.org': 'prostoprosport.ru',
 'sn--gazeta-ru.turbopages.org': 'sn-gazeta.ru',
 'smol-kp-ru.turbopages.org': 'smol.kp.ru',
 'creditam-ru.turbopages.org': 'creditam.ru',
 'renovar-ru.turbopages.org': 'renovar.ru',
 'wellnews-ru.turbopages.org': 'wellnews.ru',
 'agro--sales-ru.turbopages.org': 'agro-sales.ru',
 'msk--tinao-livejournal-com.turbopages.org': 'msk-tinao.livejournal.com',
 'livekuban-ru.turbopages.org': 'livekuban.ru',
 'bolit-net.turbopages.org': 'bolit.net',
 'techinsider-ru.turbopages.org': 'techinsider.ru',
 'veritas4-livejournal-com.turbopages.org': 'veritas4.livejournal.com',
 'proglib-io.turbopages.org': 'proglib.io',
 'strir-ru.turbopages.org': 'strir.ru',
 'krasnoyarskmedia-ru.turbopages.org': 'krasnoyarskmedia.ru',
 'night--onlooker-livejournal-com.turbopages.org': 'night-onlooker.livejournal.com',
 'cvarnou-livejournal-com.turbopages.org': 'cvarnou.livejournal.com',
 'advokat--sokolov74-ru.turbopages.org': 'advokat-sokolov74.ru',
 'yelkz-livejournal-com.turbopages.org': 'yelkz.livejournal.com',
 'chemege-ru.turbopages.org': 'chemege.ru',
 'mirogorodov-ru.turbopages.org': 'mirogorodov.ru',
 'vitgid-ru.turbopages.org': 'vitgid.ru',
 'zabor22-ru.turbopages.org': 'zabor22.ru',
 'clinpharm--journal-ru.turbopages.org': 'clinpharm-journal.ru',
 'valkiriarf-livejournal-com.turbopages.org': 'valkiriarf.livejournal.com',
 'homeopat--sam-com.turbopages.org': 'homeopat-sam.com',
 '25school-ru.turbopages.org': '25school.ru',
 'vericon-ru.turbopages.org': 'vericon.ru',
 'ekosad--vsem-ru.turbopages.org': 'ekosad-vsem.ru',
 'mistika-xyz.turbopages.org': 'mistika.xyz',
 '6sotok--dom-com.turbopages.org': '6sotok-dom.com',
 'medicina-dobro--est-com.turbopages.org': 'medicina.dobro-est.com',
 'yantar--apart-ru.turbopages.org': 'yantar-apart.ru',
 'hans--gille-livejournal-com.turbopages.org': 'hans-gille.livejournal.com',
 'soba4nik-pet.turbopages.org': 'soba4nik.pet',
 'happydoctor-ru.turbopages.org': 'happydoctor.ru',
 'ohota-guru.turbopages.org': 'ohota.guru',
 'zavodfoto-livejournal-com.turbopages.org': 'zavodfoto.livejournal.com',
 'pravoved-online.turbopages.org': 'pravoved.online',
 'v--gromov-livejournal-com.turbopages.org': 'v-gromov.livejournal.com',
 'domosedy-com.turbopages.org': 'domosedy.com',
 'dr--dez-ru.turbopages.org': 'dr-dez.ru',
 'kladovayalesa-ru.turbopages.org': 'kladovayalesa.ru',
 'vse--pro--lekarstva-ru.turbopages.org': 'vse-pro-lekarstva.ru',
 'djhooligantk-livejournal-com.turbopages.org': 'djhooligantk.livejournal.com',
 'playmaker24-ru.turbopages.org': 'playmaker24.ru',
 'korobok12-livejournal-com.turbopages.org': 'korobok12.livejournal.com',
 'doktornic-livejournal-com.turbopages.org': 'doktornic.livejournal.com',
 'banya--expert-com.turbopages.org': 'banya-expert.com',
 'platan--metal-ru.turbopages.org': 'platan-metal.ru',
 'dolgi-org.turbopages.org': 'dolgi.org',
 'android--example-ru.turbopages.org': 'android-example.ru',
 'most-tv.turbopages.org': 'most.tv',
 'yar-aif-ru.turbopages.org': 'yar.aif.ru',
 'alex--brass-livejournal-com.turbopages.org': 'alex-brass.livejournal.com',
 '1000bankov-ru.turbopages.org': '1000bankov.ru',
 'alekka4alin2012-livejournal-com.turbopages.org': 'alekka4alin2012.livejournal.com',
 'uborshizzza-livejournal-com.turbopages.org': 'uborshizzza.livejournal.com',
 'dacha-expert.turbopages.org': 'dacha.expert',
 'sorokanews-ru.turbopages.org': 'sorokanews.ru',
 'info--mitsubishi-ru.turbopages.org': 'info-mitsubishi.ru',
 '1000sovetov-ru.turbopages.org': '1000sovetov.ru',
 'marishka57v-livejournal-com.turbopages.org': 'marishka57v.livejournal.com',
 'dosaafvlg--kotovo-ru.turbopages.org': 'dosaafvlg-kotovo.ru',
 'mtz--80-ru.turbopages.org': 'mtz-80.ru',
 'dosgames-ru.turbopages.org': 'dosgames.ru',
 'tveda-ru.turbopages.org': 'tveda.ru',
 'druggist-ru.turbopages.org': 'druggist.ru',
 'bananafishka-livejournal-com.turbopages.org': 'bananafishka.livejournal.com',
 'crr74-ru.turbopages.org': 'crr74.ru',
 'vdiete-ru.turbopages.org': 'vdiete.ru',
 '14bytes-ru.turbopages.org': '14bytes.ru',
 'cleanbin-ru.turbopages.org': 'cleanbin.ru',
 'viskablivanie-ru.turbopages.org': 'viskablivanie.ru',
 'theyoga-livejournal-com.turbopages.org': 'theyoga.livejournal.com',
 'scfh-ru.turbopages.org': 'scfh.ru',
 'truehunter-ru.turbopages.org': 'truehunter.ru',
 'kypavino-ru.turbopages.org': 'kypavino.ru',
 'tent3302-ru.turbopages.org': 'tent3302.ru',
 'i--gazeta-com.turbopages.org': 'i-gazeta.com',
 'droidov-com.turbopages.org': 'droidov.com',
 'orenburg-bezformata-com.turbopages.org': 'orenburg.bezformata.com',
 'remoskop-ru.turbopages.org': 'remoskop.ru',
 'mr--garett-livejournal-com.turbopages.org': 'mr-garett.livejournal.com',
 'homelka-livejournal-com.turbopages.org': 'homelka.livejournal.com',
 'parnas42-ru.turbopages.org': 'parnas42.ru',
 'aleks070565-livejournal-com.turbopages.org': 'aleks070565.livejournal.com',
 'zabivan-livejournal-com.turbopages.org': 'zabivan.livejournal.com',
 'prompriem-ru.turbopages.org': 'prompriem.ru',
 'alex--lw--65-livejournal-com.turbopages.org': 'alex-lw-65.livejournal.com',
 'avtokart-ru.turbopages.org': 'avtokart.ru',
 'obmanu--net-com.turbopages.org': 'obmanu-net.com',
 'upravlenie-org.turbopages.org': 'upravlenie.org',
 'wolcha-ru.turbopages.org': 'wolcha.ru',
 'colonelcassad-livejournal-com.turbopages.org': 'colonelcassad.livejournal.com',
 'yadroid-ru.turbopages.org': 'yadroid.ru',
 'al--lex--ey-livejournal-com.turbopages.org': 'al-lex-ey.livejournal.com',
 'goryachie--photo-net.turbopages.org': 'goryachie-photo.net',
 'ast-mk-ru.turbopages.org': 'ast.mk.ru',
 'penza--press-ru.turbopages.org': 'penza-press.ru',
 'autoparus-by.turbopages.org': 'autoparus.by',
 'sevastopolmedia-ru.turbopages.org': 'sevastopolmedia.ru',
 'zdorovoeoko-ru.turbopages.org': 'zdorovoeoko.ru',
 'u3poccuu-livejournal-com.turbopages.org': 'u3poccuu.livejournal.com',
 'uvelir-info.turbopages.org': 'uvelir.info',
 'tokyostreet-ru.turbopages.org': 'tokyostreet.ru',
 'moirebenok-ua.turbopages.org': 'moirebenok.ua',
 'kopiraitery-ru.turbopages.org': 'kopiraitery.ru',
 'kiz-ru.turbopages.org': 'kiz.ru',
 'huaweihelp-ru.turbopages.org': 'huaweihelp.ru',
 'soldati--russian-ru.turbopages.org': 'soldati-russian.ru',
 'aillarionov-livejournal-com.turbopages.org': 'aillarionov.livejournal.com',
 '1xmatch-com.turbopages.org': '1xmatch.com',
 'otzyvua-net.turbopages.org': 'otzyvua.net',
 'law-ru.turbopages.org': 'law.ru',
 'm--kalashnikov-livejournal-com.turbopages.org': 'm-kalashnikov.livejournal.com',
 'lera--komor-livejournal-com.turbopages.org': 'lera-komor.livejournal.com',
 'zotych7-livejournal-com.turbopages.org': 'zotych7.livejournal.com',
 'newcripto-ru.turbopages.org': 'newcripto.ru',
 'gtrk--vyatka-ru.turbopages.org': 'gtrk-vyatka.ru',
 'blizko-by.turbopages.org': 'blizko.by',
 'endometriy-com.turbopages.org': 'endometriy.com',
 'velife-ru.turbopages.org': 'velife.ru',
 'kreslav-livejournal-com.turbopages.org': 'kreslav.livejournal.com',
 'spoilering-ru.turbopages.org': 'spoilering.ru',
 'bashkort--tele-livejournal-com.turbopages.org': 'bashkort-tele.livejournal.com',
 'psy--files-ru.turbopages.org': 'psy-files.ru',
 'countrel-ru.turbopages.org': 'countrel.ru',
 'promopulse-info.turbopages.org': 'promopulse.info',
 'prouksus-ru.turbopages.org': 'prouksus.ru',
 'skitalets-livejournal-com.turbopages.org': 'skitalets.livejournal.com',
 'pchelgid-ru.turbopages.org': 'pchelgid.ru',
 'mirnotarius-ru.turbopages.org': 'mirnotarius.ru',
 'nvsport-ru.turbopages.org': 'nvsport.ru',
 'kz-tsargrad-tv.turbopages.org': 'kz.tsargrad.tv',
 'md-tsargrad-tv.turbopages.org': 'md.tsargrad.tv',
 'informugra-ru.turbopages.org': 'informugra.ru',
 'tsn24-ru.turbopages.org': 'tsn24.ru',
 'kavkaz-mk-ru.turbopages.org': 'kavkaz.mk.ru',
 'vpravda-ru.turbopages.org': 'vpravda.ru',
 'nn-rbc-ru.turbopages.org': 'nn.rbc.ru',
 'bdsmn-livejournal-com.turbopages.org': 'bdsmn.livejournal.com',
 'nnovgorod-bezformata-com.turbopages.org': 'nnovgorod.bezformata.com',
 'royalcheese-ru.turbopages.org': 'royalcheese.ru',
 'nn-mk-ru.turbopages.org': 'nn.mk.ru',
 'kaluganews-ru.turbopages.org': 'kaluganews.ru',
 'topmekhanik-ru.turbopages.org': 'topmekhanik.ru',
 'zdrav-ru.turbopages.org': 'zdrav.ru',
 'diana--mihailova-livejournal-com.turbopages.org': 'diana-mihailova.livejournal.com',
 'alexgitara-ru.turbopages.org': 'alexgitara.ru',
 'domashniecvety-ru.turbopages.org': 'domashniecvety.ru',
 'if24-ru.turbopages.org': 'if24.ru',
 'pohod365-ru.turbopages.org': 'pohod365.ru',
 'sammedic-ru.turbopages.org': 'sammedic.ru',
 'chr-mk-ru.turbopages.org': 'chr.mk.ru',
 'arenda--ros-ru.turbopages.org': 'arenda-ros.ru',
 'v--mire--sobak-ru.turbopages.org': 'v-mire-sobak.ru',
 'prokopievsk-ru.turbopages.org': 'prokopievsk.ru',
 'saratov-aif-ru.turbopages.org': 'saratov.aif.ru',
 'rt-rbc-ru.turbopages.org': 'rt.rbc.ru',
 'sila--vody-ru.turbopages.org': 'sila-vody.ru',
 'trud-ru.turbopages.org': 'trud.ru',
 'sportpit68-ru.turbopages.org': 'sportpit68.ru',
 'domkrolikov-ru.turbopages.org': 'domkrolikov.ru',
 'moj--kot-ru.turbopages.org': 'moj-kot.ru',
 'legart-livejournal-com.turbopages.org': 'legart.livejournal.com',
 'selosovetov-ru.turbopages.org': 'selosovetov.ru',
 'pobetony-ru.turbopages.org': 'pobetony.ru',
 '2sotki-ru.turbopages.org': '2sotki.ru',
 'sergey--verevkin-livejournal-com.turbopages.org': 'sergey-verevkin.livejournal.com',
 '0gnev-livejournal-com.turbopages.org': '0gnev.livejournal.com',
 'roman--sha-livejournal-com.turbopages.org': 'roman-sha.livejournal.com',
 'marafonec-livejournal-com.turbopages.org': 'marafonec.livejournal.com',
 'mamsila-ru.turbopages.org': 'mamsila.ru',
 'zdr-ru.turbopages.org': 'zdr.ru',
 'ekaterinburg-bezformata-com.turbopages.org': 'ekaterinburg.bezformata.com',
 'bataysk--gorod-ru.turbopages.org': 'bataysk-gorod.ru',
 'anton--polynov-livejournal-com.turbopages.org': 'anton-polynov.livejournal.com',
 'capost-media.turbopages.org': 'capost.media',
 'oskada-ru.turbopages.org': 'oskada.ru',
 'rosinkastorg-ru.turbopages.org': 'rosinkastorg.ru',
 'bepowerback-livejournal-com.turbopages.org': 'bepowerback.livejournal.com',
 'ogorodlegko-ru.turbopages.org': 'ogorodlegko.ru',
 'astronews-ru.turbopages.org': 'astronews.ru',
 'gdeichto-ru.turbopages.org': 'gdeichto.ru',
 'luv-ru.turbopages.org': 'luv.ru',
 'malyshi-livejournal-com.turbopages.org': 'malyshi.livejournal.com',
 'dr--roman--suhov-livejournal-com.turbopages.org': 'dr-roman-suhov.livejournal.com',
 '7freiheit-livejournal-com.turbopages.org': '7freiheit.livejournal.com',
 'kakoy--smysl-ru.turbopages.org': 'kakoy-smysl.ru',
 'salon-ru.turbopages.org': 'salon.ru',
 'moi--finansi-ru.turbopages.org': 'moi-finansi.ru',
 'agronews-com.turbopages.org': 'agronews.com',
 'gazavtomir-ru.turbopages.org': 'gazavtomir.ru',
 'lgotnikam-ru.turbopages.org': 'lgotnikam.ru',
 'po--orlu-livejournal-com.turbopages.org': 'po-orlu.livejournal.com',
 'vseoallergii-ru.turbopages.org': 'vseoallergii.ru',
 'fresh-ru.turbopages.org': 'fresh.ru',
 'orchardo-ru.turbopages.org': 'orchardo.ru',
 'oooevna-ru.turbopages.org': 'oooevna.ru',
 'respimed-ru.turbopages.org': 'respimed.ru',
 'how2get-ru.turbopages.org': 'how2get.ru',
 'lessale-ru.turbopages.org': 'lessale.ru',
 'svalitoday-livejournal-com.turbopages.org': 'svalitoday.livejournal.com',
 'mainavi-ru.turbopages.org': 'mainavi.ru',
 'promkomrostov-ru.turbopages.org': 'promkomrostov.ru',
 'trasyy-livejournal-com.turbopages.org': 'trasyy.livejournal.com',
 'arhi1-ru.turbopages.org': 'arhi1.ru',
 'pravoved-ru.turbopages.org': 'pravoved.ru',
 'kashira-ru.turbopages.org': 'kashira.ru',
 'auto--nota-ru.turbopages.org': 'auto-nota.ru',
 'cinematographua-livejournal-com.turbopages.org': 'cinematographua.livejournal.com',
 'kiri2ll-livejournal-com.turbopages.org': 'kiri2ll.livejournal.com',
 'ufa--town-ru.turbopages.org': 'ufa-town.ru',
 'sekreti--domovodstva-ru.turbopages.org': 'sekreti-domovodstva.ru',
 'zen--mn-ru.turbopages.org': 'zen-mn.ru',
 'anastgal-livejournal-com.turbopages.org': 'anastgal.livejournal.com',
 'untaboo-ru.turbopages.org': 'untaboo.ru',
 '1way--to--english-livejournal-com.turbopages.org': '1way-to-english.livejournal.com',
 'kaliningradfirst-ru.turbopages.org': 'kaliningradfirst.ru',
 'giknutye-ru.turbopages.org': 'giknutye.ru',
 'ahilla-ru.turbopages.org': 'ahilla.ru',
 'greg--butcher-livejournal-com.turbopages.org': 'greg-butcher.livejournal.com',
 'mordovmedia-ru.turbopages.org': 'mordovmedia.ru',
 '101prazdnik-com.turbopages.org': '101prazdnik.com',
 'sovet--podarok-ru.turbopages.org': 'sovet-podarok.ru',
 'podolskriamo-ru.turbopages.org': 'podolskriamo.ru',
 'iprodvinem-com.turbopages.org': 'iprodvinem.com',
 'just--ice-info.turbopages.org': 'just-ice.info',
 'blogkadrovika-ru.turbopages.org': 'blogkadrovika.ru',
 'mariya--mironova-ru.turbopages.org': 'mariya-mironova.ru',
 'memorysafonovo-ru.turbopages.org': 'memorysafonovo.ru',
 'vesti--yamal-ru.turbopages.org': 'vesti-yamal.ru',
 'znakomim--vas-ru.turbopages.org': 'znakomim-vas.ru',
 'diet--tips-ru.turbopages.org': 'diet-tips.ru',
 'volga--kaspiy-ru.turbopages.org': 'volga-kaspiy.ru',
 'academia-interfax-ru.turbopages.org': 'academia.interfax.ru',
 'odessa1824-livejournal-com.turbopages.org': 'odessa1824.livejournal.com',
 'kelebek--tr-livejournal-com.turbopages.org': 'kelebek-tr.livejournal.com',
 'platon2-livejournal-com.turbopages.org': 'platon2.livejournal.com',
 'avtolektron-ru.turbopages.org': 'avtolektron.ru',
 'asbio-ru.turbopages.org': 'asbio.ru',
 'remont--avtovaz-ru.turbopages.org': 'remont-avtovaz.ru',
 'pressadv-ru.turbopages.org': 'pressadv.ru',
 'kamin--expert-ru.turbopages.org': 'kamin-expert.ru',
 'kaspyinfo-ru.turbopages.org': 'kaspyinfo.ru',
 'mospravda-ru.turbopages.org': 'mospravda.ru',
 'vazoriginal-ru.turbopages.org': 'vazoriginal.ru',
 'urklad-ru.turbopages.org': 'urklad.ru',
 'revda--info-ru.turbopages.org': 'revda-info.ru',
 'alex--777--70-livejournal-com.turbopages.org': 'alex-777-70.livejournal.com',
 'beregispinu-ru.turbopages.org': 'beregispinu.ru',
 'new--variant-ru.turbopages.org': 'new-variant.ru',
 '5play-ru.turbopages.org': '5play.ru',
 'jpgazeta-ru.turbopages.org': 'jpgazeta.ru',
 'cicon-ru.turbopages.org': 'cicon.ru',
 'vp43-ru.turbopages.org': 'vp43.ru',
 'vomske-ru.turbopages.org': 'vomske.ru',
 'galik--123-livejournal-com.turbopages.org': 'galik-123.livejournal.com',
 'mcdill-livejournal-com.turbopages.org': 'mcdill.livejournal.com',
 'sajtyzarabotka-ru.turbopages.org': 'sajtyzarabotka.ru',
 'pubg1337-ru.turbopages.org': 'pubg1337.ru',
 'samara-bezformata-com.turbopages.org': 'samara.bezformata.com',
 'current1-livejournal-com.turbopages.org': 'current1.livejournal.com',
 'kinsite-ru.turbopages.org': 'kinsite.ru',
 'doloykurit-ru.turbopages.org': 'doloykurit.ru',
 'pp--vkusno-ru.turbopages.org': 'pp-vkusno.ru',
 'parentchild-ru.turbopages.org': 'parentchild.ru',
 'dailybaby-ru.turbopages.org': 'dailybaby.ru',
 'avtonomnaya--gazifikaciya-ru.turbopages.org': 'avtonomnaya-gazifikaciya.ru',
 'elektrikexpert-ru.turbopages.org': 'elektrikexpert.ru',
 'dachaa-ru.turbopages.org': 'dachaa.ru',
 'kungur-bezformata-com.turbopages.org': 'kungur.bezformata.com',
 'stapravda-ru.turbopages.org': 'stapravda.ru',
 'assadin-livejournal-com.turbopages.org': 'assadin.livejournal.com',
 'parazitdoma-ru.turbopages.org': 'parazitdoma.ru',
 'bannoemesto-com.turbopages.org': 'bannoemesto.com',
 'anle--dent-ru.turbopages.org': 'anle-dent.ru',
 'iarybak-com.turbopages.org': 'iarybak.com',
 'zabnews-ru.turbopages.org': 'zabnews.ru',
 'znaikak-ru.turbopages.org': 'znaikak.ru',
 'starover-livejournal-com.turbopages.org': 'starover.livejournal.com',
 'inkocoin-ru.turbopages.org': 'inkocoin.ru',
 'zamana-info.turbopages.org': 'zamana.info',
 'compgramotnost-ru.turbopages.org': 'compgramotnost.ru',
 'it--like-ru.turbopages.org': 'it-like.ru',
 'magistral--1a-ru.turbopages.org': 'magistral-1a.ru',
 'nn--kalyan-ru.turbopages.org': 'nn-kalyan.ru',
 'bwe-su.turbopages.org': 'bwe.su',
 'bluesmaker-livejournal-com.turbopages.org': 'bluesmaker.livejournal.com',
 'i--kramaren-livejournal-com.turbopages.org': 'i-kramaren.livejournal.com',
 'dinamika--zhizni-ru.turbopages.org': 'dinamika-zhizni.ru',
 'oopslady-ru.turbopages.org': 'oopslady.ru',
 'switki-ru.turbopages.org': 'switki.ru',
 'parilochka-com.turbopages.org': 'parilochka.com',
 'cocktailsbar-livejournal-com.turbopages.org': 'cocktailsbar.livejournal.com',
 'okosmetologii-ru.turbopages.org': 'okosmetologii.ru',
 'feelpassion-ru.turbopages.org': 'feelpassion.ru',
 'ipechen-ru.turbopages.org': 'ipechen.ru',
 'nv--online-info.turbopages.org': 'nv-online.info',
 'sofya1444-livejournal-com.turbopages.org': 'sofya1444.livejournal.com',
 'evilsmoke-ru.turbopages.org': 'evilsmoke.ru',
 'travelq-ru.turbopages.org': 'travelq.ru',
 'militarist--info-ru.turbopages.org': 'militarist-info.ru',
 'yadocent-livejournal-com.turbopages.org': 'yadocent.livejournal.com',
 'repaireasily-ru.turbopages.org': 'repaireasily.ru',
 'maxxbay-livejournal-com.turbopages.org': 'maxxbay.livejournal.com',
 'vopros--pdd-info.turbopages.org': 'vopros-pdd.info',
 'mostszh-ru.turbopages.org': 'mostszh.ru',
 'flackelf-livejournal-com.turbopages.org': 'flackelf.livejournal.com',
 'cheb--live-ru.turbopages.org': 'cheb-live.ru',
 'cheb-mk-ru.turbopages.org': 'cheb.mk.ru',
 'chayguru-info.turbopages.org': 'chayguru.info',
 'simhost-org.turbopages.org': 'simhost.org',
 'beetlestop-ru.turbopages.org': 'beetlestop.ru',
 'fermer--72-ru.turbopages.org': 'fermer-72.ru',
 'koprote-ru.turbopages.org': 'koprote.ru',
 'plekhovlive-livejournal-com.turbopages.org': 'plekhovlive.livejournal.com',
 'neft-media.turbopages.org': 'neft.media',
 'how--info-ru.turbopages.org': 'how-info.ru',
 'svgorko-ru.turbopages.org': 'svgorko.ru',
 'nevestabest-com.turbopages.org': 'nevestabest.com',
 'elsvarkin-ru.turbopages.org': 'elsvarkin.ru',
 'zetsila-ru.turbopages.org': 'zetsila.ru',
 'aqua--hunter-ru.turbopages.org': 'aqua-hunter.ru',
 'geodesist-ru.turbopages.org': 'geodesist.ru',
 'expromtom-ru.turbopages.org': 'expromtom.ru',
 'storedigital-ru.turbopages.org': 'storedigital.ru',
 'podnimibabla-ru.turbopages.org': 'podnimibabla.ru',
 'codernet-ru.turbopages.org': 'codernet.ru',
 'vnarod-livejournal-com.turbopages.org': 'vnarod.livejournal.com',
 'vsepromozg-ru.turbopages.org': 'vsepromozg.ru',
 'tehnowar-ru.turbopages.org': 'tehnowar.ru',
 'chervonec--001-livejournal-com.turbopages.org': 'chervonec-001.livejournal.com',
 'djoker--lj-livejournal-com.turbopages.org': 'djoker-lj.livejournal.com',
 'finmarket-ru.turbopages.org': 'finmarket.ru',
 'roman--smirnov-livejournal-com.turbopages.org': 'roman-smirnov.livejournal.com',
 'ru--travel-livejournal-com.turbopages.org': 'ru-travel.livejournal.com',
 'autotaf-ru.turbopages.org': 'autotaf.ru',
 'astfisher-ru.turbopages.org': 'astfisher.ru',
 'cefiro-ru.turbopages.org': 'cefiro.ru',
 'lifestyleplus-ru.turbopages.org': 'lifestyleplus.ru',
 '1sonnik-com.turbopages.org': '1sonnik.com',
 'quantuz-livejournal-com.turbopages.org': 'quantuz.livejournal.com',
 'fix--note-ru.turbopages.org': 'fix-note.ru',
 'dezhur-com.turbopages.org': 'dezhur.com',
 'ushastyi-livejournal-com.turbopages.org': 'ushastyi.livejournal.com',
 'zarabotatdoma-ru.turbopages.org': 'zarabotatdoma.ru',
 'natrukodel-ru.turbopages.org': 'natrukodel.ru',
 'ryazan-bezformata-com.turbopages.org': 'ryazan.bezformata.com',
 'progorod62-ru.turbopages.org': 'progorod62.ru',
 'p--i--f-livejournal-com.turbopages.org': 'p-i-f.livejournal.com',
 'iga--motor-ru.turbopages.org': 'iga-motor.ru',
 'lv973-livejournal-com.turbopages.org': 'lv973.livejournal.com',
 'aviation21-ru.turbopages.org': 'aviation21.ru',
 'hodynskoe--pole-livejournal-com.turbopages.org': 'hodynskoe-pole.livejournal.com',
 'sintez--audio-livejournal-com.turbopages.org': 'sintez-audio.livejournal.com',
 'gadaniya-me.turbopages.org': 'gadaniya.me',
 'cistitam-net.turbopages.org': 'cistitam.net',
 'attuale-ru.turbopages.org': 'attuale.ru',
 'mediapravo-com.turbopages.org': 'mediapravo.com',
 '100yuristov-com.turbopages.org': '100yuristov.com',
 'nastolkoff-ru.turbopages.org': 'nastolkoff.ru',
 'pro--ugolovnoe-ru.turbopages.org': 'pro-ugolovnoe.ru',
 'vesti22-tv.turbopages.org': 'vesti22.tv',
 'novvedomosti-ru.turbopages.org': 'novvedomosti.ru',
 'vnovgorode-ru.turbopages.org': 'vnovgorode.ru',
 'xn------7cdalnei0czacs5a2ae5a.turbopages.org': 'xn---7cdalnei0czacs5a2ae5a',
 'pasha--popolam-livejournal-com.turbopages.org': 'pasha-popolam.livejournal.com',
 '409zmz-ru.turbopages.org': '409zmz.ru',
 'multivarka--recepti-ru.turbopages.org': 'multivarka-recepti.ru',
 'recepti--vmultivarke-ru.turbopages.org': 'recepti-vmultivarke.ru',
 'kirill--kvs-livejournal-com.turbopages.org': 'kirill-kvs.livejournal.com',
 'mister--office-ru.turbopages.org': 'mister-office.ru',
 'think--head-livejournal-com.turbopages.org': 'think-head.livejournal.com',
 'wisegeek-ru.turbopages.org': 'wisegeek.ru',
 'anna--saharova-livejournal-com.turbopages.org': 'anna-saharova.livejournal.com',
 'hero--wars-ru.turbopages.org': 'hero-wars.ru',
 'katek--kat-livejournal-com.turbopages.org': 'katek-kat.livejournal.com',
 'lapulik-com.turbopages.org': 'lapulik.com',
 'prpr-su.turbopages.org': 'prpr.su',
 '1964l-livejournal-com.turbopages.org': '1964l.livejournal.com',
 'v--r--tsoy-livejournal-com.turbopages.org': 'v-r-tsoy.livejournal.com',
 'gun--spb-com.turbopages.org': 'gun-spb.com',
 'mama--beauty-livejournal-com.turbopages.org': 'mama-beauty.livejournal.com',
 'tsvetyzhizni-ru.turbopages.org': 'tsvetyzhizni.ru',
 'sputnikmts-ru.turbopages.org': 'sputnikmts.ru',
 'notklop-ru.turbopages.org': 'notklop.ru',
 'starcheolog-livejournal-com.turbopages.org': 'starcheolog.livejournal.com',
 'ru--sherwood-livejournal-com.turbopages.org': 'ru-sherwood.livejournal.com',
 'ru--abandoned-livejournal-com.turbopages.org': 'ru-abandoned.livejournal.com',
 'statuirovkoy-ru.turbopages.org': 'statuirovkoy.ru',
 'cosmetism-ru.turbopages.org': 'cosmetism.ru',
 'vmirevolos-ru.turbopages.org': 'vmirevolos.ru',
 'geshtv-ru.turbopages.org': 'geshtv.ru',
 'davidaidelman-livejournal-com.turbopages.org': 'davidaidelman.livejournal.com',
 'varandej-livejournal-com.turbopages.org': 'varandej.livejournal.com',
 'censury-net.turbopages.org': 'censury.net',
 'gotovooo-ru.turbopages.org': 'gotovooo.ru',
 'svoimi--rukami--da-ru.turbopages.org': 'svoimi-rukami-da.ru',
 'newia-ru.turbopages.org': 'newia.ru',
 'gazeta--kozelsk-ru.turbopages.org': 'gazeta-kozelsk.ru',
 'opechkah-ru.turbopages.org': 'opechkah.ru',
 'sheika--matka-ru.turbopages.org': 'sheika-matka.ru',
 'xn----ctbydui2a.turbopages.org': 'xn--ctbydui2a',
 'remontsami-ru.turbopages.org': 'remontsami.ru',
 'revansh--fpg--53-livejournal-com.turbopages.org': 'revansh-fpg-53.livejournal.com',
 'delairukami-ru.turbopages.org': 'delairukami.ru',
 'berlogakarelia-ru.turbopages.org': 'berlogakarelia.ru',
 'bezpolitiki2020-ru.turbopages.org': 'bezpolitiki2020.ru',
 'retail--loyalty-org.turbopages.org': 'retail-loyalty.org',
 'easybizzi39-ru.turbopages.org': 'easybizzi39.ru',
 'sanktpeterburg-bezformata-com.turbopages.org': 'sanktpeterburg.bezformata.com',
 'moygospodin-livejournal-com.turbopages.org': 'moygospodin.livejournal.com',
 'mysummercar-ru.turbopages.org': 'mysummercar.ru',
 'ocrb-ru.turbopages.org': 'ocrb.ru',
 'vedom-ru.turbopages.org': 'vedom.ru',
 'birth--controll-ru.turbopages.org': 'birth-controll.ru',
 'countryscanner-ru.turbopages.org': 'countryscanner.ru',
 'sn-ria-ru.turbopages.org': 'sn.ria.ru',
 '4x4-media.turbopages.org': '4x4.media',
 'besedkibest-ru.turbopages.org': 'besedkibest.ru',
 'tiowiliam-ru.turbopages.org': 'tiowiliam.ru',
 'itblog21-ru.turbopages.org': 'itblog21.ru',
 'studopedia-net.turbopages.org': 'studopedia.net',
 'workinnet-ru.turbopages.org': 'workinnet.ru',
 'itrew-ru.turbopages.org': 'itrew.ru',
 'vtemu-by.turbopages.org': 'vtemu.by',
 'asmotor-ru.turbopages.org': 'asmotor.ru',
 'a--ezdakov-livejournal-com.turbopages.org': 'a-ezdakov.livejournal.com',
 'zarplata--es-com.turbopages.org': 'zarplata-es.com',
 'grib--info-ru.turbopages.org': 'grib-info.ru',
 'dvayarda-ru.turbopages.org': 'dvayarda.ru',
 'bankmira-ru.turbopages.org': 'bankmira.ru',
 'stoptubik-ru.turbopages.org': 'stoptubik.ru',
 'psiho-guru.turbopages.org': 'psiho.guru',
 'radulova-livejournal-com.turbopages.org': 'radulova.livejournal.com',
 'csandriel-livejournal-com.turbopages.org': 'csandriel.livejournal.com',
 'samru-ru.turbopages.org': 'samru.ru',
 'heavyrussia-livejournal-com.turbopages.org': 'heavyrussia.livejournal.com',
 'kinosait24-ru.turbopages.org': 'kinosait24.ru',
 'kemping--na--chernom--more-ru.turbopages.org': 'kemping-na-chernom-more.ru',
 'acmespb-ru.turbopages.org': 'acmespb.ru',
 'paes250-ru.turbopages.org': 'paes250.ru',
 'kursk--izvestia-ru.turbopages.org': 'kursk-izvestia.ru',
 'frequentflyers-ru.turbopages.org': 'frequentflyers.ru',
 'users-livejournal-com.turbopages.org': 'users.livejournal.com',
 'dizajn--gostinoj-ru.turbopages.org': 'dizajn-gostinoj.ru',
 'dekodiz-ru.turbopages.org': 'dekodiz.ru',
 'bur-aif-ru.turbopages.org': 'bur.aif.ru',
 'vkuso-ru.turbopages.org': 'vkuso.ru',
 'tomin-by.turbopages.org': 'tomin.by',
 'pressa40-ru.turbopages.org': 'pressa40.ru',
 'novgorod--tv-ru.turbopages.org': 'novgorod-tv.ru',
 'hollywood--actors-ru.turbopages.org': 'hollywood-actors.ru',
 'tutsvarka-ru.turbopages.org': 'tutsvarka.ru',
 'tehnika--soveti-ru.turbopages.org': 'tehnika-soveti.ru',
 'allergiyanet-ru.turbopages.org': 'allergiyanet.ru',
 'yoga24-info.turbopages.org': 'yoga24.info',
 'fulledu-ru.turbopages.org': 'fulledu.ru',
 'vladikavkaz-bezformata-com.turbopages.org': 'vladikavkaz.bezformata.com',
 'expertsouth-ru.turbopages.org': 'expertsouth.ru',
 'agrovesti-net.turbopages.org': 'agrovesti.net',
 'diabetik-guru.turbopages.org': 'diabetik.guru',
 'mmc--optima-ru.turbopages.org': 'mmc-optima.ru',
 'ivprave-ru.turbopages.org': 'ivprave.ru',
 'prophotos--ru-livejournal-com.turbopages.org': 'prophotos-ru.livejournal.com',
 'poznyaev-ru.turbopages.org': 'poznyaev.ru',
 'gidsnov-com.turbopages.org': 'gidsnov.com',
 'beznasekomyh-ru.turbopages.org': 'beznasekomyh.ru',
 'annataliya-livejournal-com.turbopages.org': 'annataliya.livejournal.com',
 'ul-aif-ru.turbopages.org': 'ul.aif.ru',
 'popados-livejournal-com.turbopages.org': 'popados.livejournal.com',
 'dachnyeotvety-ru.turbopages.org': 'dachnyeotvety.ru',
 'tsvetem-ru.turbopages.org': 'tsvetem.ru',
 'itonga-livejournal-com.turbopages.org': 'itonga.livejournal.com',
 'roomplan-ru.turbopages.org': 'roomplan.ru',
 'krsk-sibnovosti-ru.turbopages.org': 'krsk.sibnovosti.ru',
 'redhood--true-livejournal-com.turbopages.org': 'redhood-true.livejournal.com',
 'omchanin-livejournal-com.turbopages.org': 'omchanin.livejournal.com',
 'xander--jet--li-livejournal-com.turbopages.org': 'xander-jet-li.livejournal.com',
 'smecalkin-ru.turbopages.org': 'smecalkin.ru',
 'rezhemmetall-ru.turbopages.org': 'rezhemmetall.ru',
 'e--utopiya-ru.turbopages.org': 'e-utopiya.ru',
 'blah-ru.turbopages.org': 'blah.ru',
 'taxfree-livejournal-com.turbopages.org': 'taxfree.livejournal.com',
 'kot--genealog-livejournal-com.turbopages.org': 'kot-genealog.livejournal.com',
 'solvaigsamara-livejournal-com.turbopages.org': 'solvaigsamara.livejournal.com',
 'mobidevices-ru.turbopages.org': 'mobidevices.ru',
 'primamedia-ru.turbopages.org': 'primamedia.ru',
 '4trek-ru.turbopages.org': '4trek.ru',
 'delvin--stil-livejournal-com.turbopages.org': 'delvin-stil.livejournal.com',
 'extra--mir-ru.turbopages.org': 'extra-mir.ru',
 'arabian--persian-livejournal-com.turbopages.org': 'arabian-persian.livejournal.com',
 'interface31-ru.turbopages.org': 'interface31.ru',
 'moskvichi-net.turbopages.org': 'moskvichi.net',
 'crazyseo1-livejournal-com.turbopages.org': 'crazyseo1.livejournal.com',
 'proulyanovsk-ru.turbopages.org': 'proulyanovsk.ru',
 'za--dergavy-livejournal-com.turbopages.org': 'za-dergavy.livejournal.com',
 'velofans-ru.turbopages.org': 'velofans.ru',
 'liza-ua.turbopages.org': 'liza.ua',
 'venerbol-ru.turbopages.org': 'venerbol.ru',
 'abvgdee-ru.turbopages.org': 'abvgdee.ru',
 'kuleshovoleg-livejournal-com.turbopages.org': 'kuleshovoleg.livejournal.com',
 'macro--jane-livejournal-com.turbopages.org': 'macro-jane.livejournal.com',
 'kotoholik-com.turbopages.org': 'kotoholik.com',
 'hydro--pnevmo-ru.turbopages.org': 'hydro-pnevmo.ru',
 'npcriz--bigshop-ru.turbopages.org': 'npcriz-bigshop.ru',
 'centrbalans-ru.turbopages.org': 'centrbalans.ru',
 'dietyexpert-ru.turbopages.org': 'dietyexpert.ru',
 'sozero-livejournal-com.turbopages.org': 'sozero.livejournal.com',
 'boris--mavlyutov-livejournal-com.turbopages.org': 'boris-mavlyutov.livejournal.com',
 'carsup-ru.turbopages.org': 'carsup.ru',
 '35media-ru.turbopages.org': '35media.ru',
 'saucyintruder-org.turbopages.org': 'saucyintruder.org',
 'bzem1-ru.turbopages.org': 'bzem1.ru',
 'gorodrabot-ru.turbopages.org': 'gorodrabot.ru',
 'kaksdelatsvoimirukami-ru.turbopages.org': 'kaksdelatsvoimirukami.ru',
 'imbirnost-ru.turbopages.org': 'imbirnost.ru',
 'dog--care-ru.turbopages.org': 'dog-care.ru',
 '1001fermer-ru.turbopages.org': '1001fermer.ru',
 'sadovyexpert-ru.turbopages.org': 'sadovyexpert.ru',
 'zepesh-livejournal-com.turbopages.org': 'zepesh.livejournal.com',
 'citizen--rusfed-livejournal-com.turbopages.org': 'citizen-rusfed.livejournal.com',
 'reissig-livejournal-com.turbopages.org': 'reissig.livejournal.com',
 'xn----7sbaahnwfbbc6awl4aeiimht8a5b8opa.turbopages.org': 'xn--7sbaahnwfbbc6awl4aeiimht8a5b8opa',
 'forex-blog.turbopages.org': 'forex.blog',
 'gosh100-livejournal-com.turbopages.org': 'gosh100.livejournal.com',
 'povar24-info.turbopages.org': 'povar24.info',
 'altermama-ru.turbopages.org': 'altermama.ru',
 'oblok-ru.turbopages.org': 'oblok.ru',
 'cistit911-ru.turbopages.org': 'cistit911.ru',
 'stupidhouse-info.turbopages.org': 'stupidhouse.info',
 'serveradmin-ru.turbopages.org': 'serveradmin.ru',
 'scaner-pro.turbopages.org': 'scaner.pro',
 'lesley82-livejournal-com.turbopages.org': 'lesley82.livejournal.com',
 'kp-md.turbopages.org': 'kp.md',
 'shaon-livejournal-com.turbopages.org': 'shaon.livejournal.com',
 'ochendaje-livejournal-com.turbopages.org': 'ochendaje.livejournal.com',
 'babyps-livejournal-com.turbopages.org': 'babyps.livejournal.com',
 'romveres-livejournal-com.turbopages.org': 'romveres.livejournal.com',
 'vladsv-livejournal-com.turbopages.org': 'vladsv.livejournal.com',
 'alexcheban-livejournal-com.turbopages.org': 'alexcheban.livejournal.com',
 'rustravel-livejournal-com.turbopages.org': 'rustravel.livejournal.com',
 'love1y--fairy-livejournal-com.turbopages.org': 'love1y-fairy.livejournal.com',
 'donni--nina-livejournal-com.turbopages.org': 'donni-nina.livejournal.com',
 'stalic-livejournal-com.turbopages.org': 'stalic.livejournal.com',
 'francis--maks-livejournal-com.turbopages.org': 'francis-maks.livejournal.com',
 'horoshieretseptyi-ru.turbopages.org': 'horoshieretseptyi.ru',
 'brandshop--ru-livejournal-com.turbopages.org': 'brandshop-ru.livejournal.com',
 'znaytovar-ru.turbopages.org': 'znaytovar.ru',
 'xn----7sboapkwwfbp3aw.turbopages.org': 'xn--7sboapkwwfbp3aw',
 'weddingblog-ru.turbopages.org': 'weddingblog.ru',
 'hairs--club-ru.turbopages.org': 'hairs-club.ru',
 'moto163-ru.turbopages.org': 'moto163.ru',
 'spb--guide-livejournal-com.turbopages.org': 'spb-guide.livejournal.com',
 'serguntius-livejournal-com.turbopages.org': 'serguntius.livejournal.com',
 'moneymakerfactory-ru.turbopages.org': 'moneymakerfactory.ru',
 'superplanshet-ru.turbopages.org': 'superplanshet.ru',
 'hi--tech--obzor-ru.turbopages.org': 'hi-tech-obzor.ru',
 'sx888-livejournal-com.turbopages.org': 'sx888.livejournal.com',
 'protop10-ru.turbopages.org': 'protop10.ru',
 'southklad-ru.turbopages.org': 'southklad.ru',
 'soccer-ru.turbopages.org': 'soccer.ru',
 'antifashist-com.turbopages.org': 'antifashist.com',
 'yuliya231971-livejournal-com.turbopages.org': 'yuliya231971.livejournal.com',
 'ruslan63-livejournal-com.turbopages.org': 'ruslan63.livejournal.com',
 'ru--iphone-ru.turbopages.org': 'ru-iphone.ru',
 'crimea-octagon-media.turbopages.org': 'crimea.octagon.media',
 'dobrosos-ru.turbopages.org': 'dobrosos.ru',
 'lepestriny-livejournal-com.turbopages.org': 'lepestriny.livejournal.com',
 'droidnews-ru.turbopages.org': 'droidnews.ru',
 'travinn-livejournal-com.turbopages.org': 'travinn.livejournal.com',
 'general--ivanov1-livejournal-com.turbopages.org': 'general-ivanov1.livejournal.com',
 'natroix-ru.turbopages.org': 'natroix.ru',
 'zalizyaka-livejournal-com.turbopages.org': 'zalizyaka.livejournal.com',
 'puziko-online.turbopages.org': 'puziko.online',
 'bulochka-ru.turbopages.org': 'bulochka.ru',
 'vipwash-ru.turbopages.org': 'vipwash.ru',
 'mensmagazines-ru.turbopages.org': 'mensmagazines.ru',
 'myautohelp-ru.turbopages.org': 'myautohelp.ru',
 'kaskad-tv.turbopages.org': 'kaskad.tv',
 'kgd-ru.turbopages.org': 'kgd.ru',
 'vena45-livejournal-com.turbopages.org': 'vena45.livejournal.com',
 'mettiss-livejournal-com.turbopages.org': 'mettiss.livejournal.com',
 'priroda--znaet-ru.turbopages.org': 'priroda-znaet.ru',
 'big--stars-ru.turbopages.org': 'big-stars.ru',
 'blagoveshensk-bezformata-com.turbopages.org': 'blagoveshensk.bezformata.com',
 'rat--felt-ru.turbopages.org': 'rat-felt.ru',
 'zdorovoeogo-ru.turbopages.org': 'zdorovoeogo.ru',
 'xn-------g4dm4a0abplzc0apkd.turbopages.org': 'xn---.g4dm4a0abplzc0apkd',
 'sb--advice-com.turbopages.org': 'sb-advice.com',
 'gd-ru.turbopages.org': 'gd.ru',
 'ardo--dez-ru.turbopages.org': 'ardo-dez.ru',
 'oren-ru.turbopages.org': 'oren.ru',
 'howoldis-ru.turbopages.org': 'howoldis.ru',
 'chitaitext-ru.turbopages.org': 'chitaitext.ru',
 'mobile--dom-ru.turbopages.org': 'mobile-dom.ru',
 'pomeranskij--shpits-ru.turbopages.org': 'pomeranskij-shpits.ru',
 'delo360-ru.turbopages.org': 'delo360.ru',
 'pnevmatiky-net.turbopages.org': 'pnevmatiky.net',
 'beton--house-com.turbopages.org': 'beton-house.com',
 'alfatep-ru.turbopages.org': 'alfatep.ru',
 'autizmy--net-ru.turbopages.org': 'autizmy-net.ru',
 'a--i--z-livejournal-com.turbopages.org': 'a-i-z.livejournal.com',
 'yebanko-ru.turbopages.org': 'yebanko.ru',
 'rupoem-ru.turbopages.org': 'rupoem.ru',
 'kononov--varvar-livejournal-com.turbopages.org': 'kononov-varvar.livejournal.com',
 'gs-by.turbopages.org': 'gs.by',
 'banya10-ru.turbopages.org': 'banya10.ru',
 'truescooters-ru.turbopages.org': 'truescooters.ru',
 'ladysdog-ru.turbopages.org': 'ladysdog.ru',
 'nutsexpert-ru.turbopages.org': 'nutsexpert.ru',
 'onlinetambov-ru.turbopages.org': 'onlinetambov.ru',
 'nedicom-ru.turbopages.org': 'nedicom.ru',
 'novikovski-livejournal-com.turbopages.org': 'novikovski.livejournal.com',
 'road--movie-livejournal-com.turbopages.org': 'road-movie.livejournal.com',
 'foodclub-ru.turbopages.org': 'foodclub.ru',
 'ren--ar-livejournal-com.turbopages.org': 'ren-ar.livejournal.com',
 'kurgan-ru.turbopages.org': 'kurgan.ru',
 'anlo1967-livejournal-com.turbopages.org': 'anlo1967.livejournal.com',
 's1rus-livejournal-com.turbopages.org': 's1rus.livejournal.com',
 'welady-ru.turbopages.org': 'welady.ru',
 'auto--errors-ru.turbopages.org': 'auto-errors.ru',
 'ladycase-ru.turbopages.org': 'ladycase.ru',
 'camodelkin-ru.turbopages.org': 'camodelkin.ru',
 'tatyirovka-ru.turbopages.org': 'tatyirovka.ru',
 'sbitnevsv-livejournal-com.turbopages.org': 'sbitnevsv.livejournal.com',
 'obzor55-ru.turbopages.org': 'obzor55.ru',
 'liliya--travel-ru.turbopages.org': 'liliya-travel.ru',
 'shumaiev-livejournal-com.turbopages.org': 'shumaiev.livejournal.com',
 'realtshcool-ru.turbopages.org': 'realtshcool.ru',
 'futurita-ru.turbopages.org': 'futurita.ru',
 'otdoxnite-ru.turbopages.org': 'otdoxnite.ru',
 'dezinfecc-ru.turbopages.org': 'dezinfecc.ru',
 'anekdoty--svezhie-ru.turbopages.org': 'anekdoty-svezhie.ru',
 'anekdoty--anekdot-ru.turbopages.org': 'anekdoty-anekdot.ru',
 'vashipitomcy-ru.turbopages.org': 'vashipitomcy.ru',
 'krutomama-ru.turbopages.org': 'krutomama.ru',
 '4apk-ru.turbopages.org': '4apk.ru',
 'autotorque-ru.turbopages.org': 'autotorque.ru',
 'club--mashin-ru.turbopages.org': 'club-mashin.ru',
 'gurukuru-ru.turbopages.org': 'gurukuru.ru',
 'kulinarnayakarusel-ru.turbopages.org': 'kulinarnayakarusel.ru',
 'kiev--best--cake-livejournal-com.turbopages.org': 'kiev-best-cake.livejournal.com',
 'woman--centre-com.turbopages.org': 'woman-centre.com',
 'camalady-ru.turbopages.org': 'camalady.ru',
 'coronavirus--covid-info.turbopages.org': 'coronavirus-covid.info',
 'wikers-ru.turbopages.org': 'wikers.ru',
 'rostov-rbc-ru.turbopages.org': 'rostov.rbc.ru',
 'yacollectioner-ru.turbopages.org': 'yacollectioner.ru',
 'skoda--auto2-ru.turbopages.org': 'skoda-auto2.ru',
 'banknn-ru.turbopages.org': 'banknn.ru',
 'mymind-yoga.turbopages.org': 'mymind.yoga',
 'grib-expert.turbopages.org': 'grib.expert',
 'grodnonews-by.turbopages.org': 'grodnonews.by',
 'ladyline-me.turbopages.org': 'ladyline.me',
 'janet-ru.turbopages.org': 'janet.ru',
 'xn----7sbmqminsb4byd.turbopages.org': 'xn--7sbmqminsb4byd',
 'whatflower-ru.turbopages.org': 'whatflower.ru',
 'retrotruck-ru.turbopages.org': 'retrotruck.ru',
 'tolyatti--tur-ru.turbopages.org': 'tolyatti-tur.ru',
 'svadbavo-ru.turbopages.org': 'svadbavo.ru',
 'rabota--na--kompjutere-ru.turbopages.org': 'rabota-na-kompjutere.ru',
 'erazmys-livejournal-com.turbopages.org': 'erazmys.livejournal.com',
 'norski-livejournal-com.turbopages.org': 'norski.livejournal.com',
 'dfcbkbx-livejournal-com.turbopages.org': 'dfcbkbx.livejournal.com',
 'alexandr3-livejournal-com.turbopages.org': 'alexandr3.livejournal.com',
 'skiings-ru.turbopages.org': 'skiings.ru',
 'ru-motor1-com.turbopages.org': 'ru.motor1.com',
 'loganlogan-ru.turbopages.org': 'loganlogan.ru',
 'mikhailosherov-livejournal-com.turbopages.org': 'mikhailosherov.livejournal.com',
 'dinamikservis-ru.turbopages.org': 'dinamikservis.ru',
 'ladafakt-ru.turbopages.org': 'ladafakt.ru',
 'yar-kp-ru.turbopages.org': 'yar.kp.ru',
 'vesti--yaroslavl-ru.turbopages.org': 'vesti-yaroslavl.ru',
 'leadinlife-info.turbopages.org': 'leadinlife.info',
 'diet--diet-ru.turbopages.org': 'diet-diet.ru',
 'xn------7cdhflcb3a4elglbk9c.turbopages.org': 'xn---7cdhflcb3a4elglbk9c',
 'f1sh1ng-ru.turbopages.org': 'f1sh1ng.ru',
 'opowiastka-livejournal-com.turbopages.org': 'opowiastka.livejournal.com',
 'activefisher-net.turbopages.org': 'activefisher.net',
 'volgamaster34-ru.turbopages.org': 'volgamaster34.ru',
 'al--nikolaeva-livejournal-com.turbopages.org': 'al-nikolaeva.livejournal.com',
 'kam-business--gazeta-ru.turbopages.org': 'kam.business-gazeta.ru',
 'prochepetsk-ru.turbopages.org': 'prochepetsk.ru',
 'tripandme-ru.turbopages.org': 'tripandme.ru',
 'komi-aif-ru.turbopages.org': 'komi.aif.ru',
 'pishipravilno-livejournal-com.turbopages.org': 'pishipravilno.livejournal.com',
 'vcomicse-ru.turbopages.org': 'vcomicse.ru',
 'strangearts-ru.turbopages.org': 'strangearts.ru',
 'adt-livejournal-com.turbopages.org': 'adt.livejournal.com',
 'petropech-ru.turbopages.org': 'petropech.ru',
 'kryshadoma-com.turbopages.org': 'kryshadoma.com',
 'vseokrovle-com.turbopages.org': 'vseokrovle.com',
 'sony--es-livejournal-com.turbopages.org': 'sony-es.livejournal.com',
 'krovlyakrishi-ru.turbopages.org': 'krovlyakrishi.ru',
 'ufastrike-ru.turbopages.org': 'ufastrike.ru',
 'yuliya--maganova-livejournal-com.turbopages.org': 'yuliya-maganova.livejournal.com',
 'ixbt-com.turbopages.org': 'ixbt.com',
 'aeroclima-ru.turbopages.org': 'aeroclima.ru',
 'osa--ira-livejournal-com.turbopages.org': 'osa-ira.livejournal.com',
 'atlantknife-ru.turbopages.org': 'atlantknife.ru',
 'dermatit-net.turbopages.org': 'dermatit.net',
 'terranist2012-livejournal-com.turbopages.org': 'terranist2012.livejournal.com',
 '7dogs-livejournal-com.turbopages.org': '7dogs.livejournal.com',
 'huaweinews-ru.turbopages.org': 'huaweinews.ru',
 'gto--normativy-ru.turbopages.org': 'gto-normativy.ru',
 'dvsvetka-livejournal-com.turbopages.org': 'dvsvetka.livejournal.com',
 't80k-livejournal-com.turbopages.org': 't80k.livejournal.com',
 'tettie-livejournal-com.turbopages.org': 'tettie.livejournal.com',
 'wodloft-ru.turbopages.org': 'wodloft.ru',
 'tivi--2-livejournal-com.turbopages.org': 'tivi-2.livejournal.com',
 '100talismanov-ru.turbopages.org': '100talismanov.ru',
 'mary7-ru.turbopages.org': 'mary7.ru',
 'vistanews-ru.turbopages.org': 'vistanews.ru',
 'varikoz-com.turbopages.org': 'varikoz.com',
 'posudaizkamnya-ru.turbopages.org': 'posudaizkamnya.ru',
 'first--travel-ru.turbopages.org': 'first-travel.ru',
 'dachnikexpert-ru.turbopages.org': 'dachnikexpert.ru',
 'xn----ctbd0bubrgec.turbopages.org': 'xn--ctbd0bubrgec',
 'stojak-ru.turbopages.org': 'stojak.ru',
 'tizer42-ru.turbopages.org': 'tizer42.ru',
 'womanmirror-ru.turbopages.org': 'womanmirror.ru',
 'sib--prostor-ru.turbopages.org': 'sib-prostor.ru',
 'ockey-ru.turbopages.org': 'ockey.ru',
 'vne--berega-ru.turbopages.org': 'vne-berega.ru',
 'moygorod-online.turbopages.org': 'moygorod.online',
 'lawabc-ru.turbopages.org': 'lawabc.ru',
 'larik--malasha-livejournal-com.turbopages.org': 'larik-malasha.livejournal.com',
 'chitatel--zhuzhu-livejournal-com.turbopages.org': 'chitatel-zhuzhu.livejournal.com',
 'profservice24-ru.turbopages.org': 'profservice24.ru',
 'ch-versia-ru.turbopages.org': 'ch.versia.ru',
 'nabchelni-bezformata-com.turbopages.org': 'nabchelni.bezformata.com',
 'reis-zr-ru.turbopages.org': 'reis.zr.ru',
 'ru--naturperfume-livejournal-com.turbopages.org': 'ru-naturperfume.livejournal.com',
 'starfate-ru.turbopages.org': 'starfate.ru',
 'radio1-news.turbopages.org': 'radio1.news',
 'wiseeconomist-ru.turbopages.org': 'wiseeconomist.ru',
 'muravey24-ru.turbopages.org': 'muravey24.ru',
 'mam4-ru.turbopages.org': 'mam4.ru',
 'hommad-ru.turbopages.org': 'hommad.ru',
 'gidsocial-ru.turbopages.org': 'gidsocial.ru',
 'lilmy-livejournal-com.turbopages.org': 'lilmy.livejournal.com',
 'real--vin-com.turbopages.org': 'real-vin.com',
 'tamby-info.turbopages.org': 'tamby.info',
 'my--best--travel-ru.turbopages.org': 'my-best-travel.ru',
 'excellent--moto-ru.turbopages.org': 'excellent-moto.ru',
 'avtoyoutube-ru.turbopages.org': 'avtoyoutube.ru',
 'mebeloptovik-ru.turbopages.org': 'mebeloptovik.ru',
 'stopbeg-livejournal-com.turbopages.org': 'stopbeg.livejournal.com',
 'stolica58-ru.turbopages.org': 'stolica58.ru',
 'useless--faq-livejournal-com.turbopages.org': 'useless-faq.livejournal.com',
 'papilkin-livejournal-com.turbopages.org': 'papilkin.livejournal.com',
 'zkm--v-ru.turbopages.org': 'zkm-v.ru',
 'ukrepit--immunitet-ru.turbopages.org': 'ukrepit-immunitet.ru',
 'brl-mk-ru.turbopages.org': 'brl.mk.ru',
 'altai-aif-ru.turbopages.org': 'altai.aif.ru',
 'ts58-livejournal-com.turbopages.org': 'ts58.livejournal.com',
 'marc--aureli-livejournal-com.turbopages.org': 'marc-aureli.livejournal.com',
 'game--raft-ru.turbopages.org': 'game-raft.ru',
 'premier--agro-ru.turbopages.org': 'premier-agro.ru',
 'zorych-livejournal-com.turbopages.org': 'zorych.livejournal.com',
 'stanspb-livejournal-com.turbopages.org': 'stanspb.livejournal.com',
 'mulino58-ru.turbopages.org': 'mulino58.ru',
 'kakprigotovila-ru.turbopages.org': 'kakprigotovila.ru',
 '101da-ru.turbopages.org': '101da.ru',
 'frenchblogs-ru.turbopages.org': 'frenchblogs.ru',
 'tgrm-su.turbopages.org': 'tgrm.su',
 'letim--v--otpusk-ru.turbopages.org': 'letim-v-otpusk.ru',
 'ladywow-club.turbopages.org': 'ladywow.club',
 'dokladiki-ru.turbopages.org': 'dokladiki.ru',
 'signalochka-ru.turbopages.org': 'signalochka.ru',
 'pricormi-ru.turbopages.org': 'pricormi.ru',
 'kunisan-livejournal-com.turbopages.org': 'kunisan.livejournal.com',
 'nikitskij-livejournal-com.turbopages.org': 'nikitskij.livejournal.com',
 'bazaznaniyst-ru.turbopages.org': 'bazaznaniyst.ru',
 'rem-ninja.turbopages.org': 'rem.ninja',
 'cheek--look-ru.turbopages.org': 'cheek-look.ru',
 'rem--stroitelstvo-ru.turbopages.org': 'rem-stroitelstvo.ru',
 'voron--xak-ru.turbopages.org': 'voron-xak.ru',
 'vesti-kz.turbopages.org': 'vesti.kz',
 'manikyurdizajn-ru.turbopages.org': 'manikyurdizajn.ru',
 'style40plus-ru.turbopages.org': 'style40plus.ru',
 'onlinezayavkanacredit-ru.turbopages.org': 'onlinezayavkanacredit.ru',
 'storylists-ru.turbopages.org': 'storylists.ru',
 'stroimdom44-ru.turbopages.org': 'stroimdom44.ru',
 'wikipet-ru.turbopages.org': 'wikipet.ru',
 'stalers-ru.turbopages.org': 'stalers.ru',
 'vrednopolezno-ru.turbopages.org': 'vrednopolezno.ru',
 'miumau-livejournal-com.turbopages.org': 'miumau.livejournal.com',
 'xn------5cdcacn3aqk2ah0at8ab1b8a.turbopages.org': 'xn---5cdcacn3aqk2ah0at8ab1b8a',
 'voenhronika-livejournal-com.turbopages.org': 'voenhronika.livejournal.com',
 'vonauto-ru.turbopages.org': 'vonauto.ru',
 'investfuture-ru.turbopages.org': 'investfuture.ru',
 'bronhus-com.turbopages.org': 'bronhus.com',
 'avtovesti-com.turbopages.org': 'avtovesti.com',
 'bmpd-livejournal-com.turbopages.org': 'bmpd.livejournal.com',
 'vot--zdorovo-livejournal-com.turbopages.org': 'vot-zdorovo.livejournal.com',
 'mankubus-livejournal-com.turbopages.org': 'mankubus.livejournal.com',
 'animallist-ru.turbopages.org': 'animallist.ru',
 'aboutanimals-ru.turbopages.org': 'aboutanimals.ru',
 'strannov-livejournal-com.turbopages.org': 'strannov.livejournal.com',
 'slazav--news-livejournal-com.turbopages.org': 'slazav-news.livejournal.com',
 'europe--today-ru.turbopages.org': 'europe-today.ru',
 'love--suit-ru.turbopages.org': 'love-suit.ru',
 'glinks-ru.turbopages.org': 'glinks.ru',
 'olga--pro-livejournal-com.turbopages.org': 'olga-pro.livejournal.com',
 'scireg-livejournal-com.turbopages.org': 'scireg.livejournal.com',
 'voxpop--66-livejournal-com.turbopages.org': 'voxpop-66.livejournal.com',
 'classpic-ru.turbopages.org': 'classpic.ru',
 'perm-rbc-ru.turbopages.org': 'perm.rbc.ru',
 'easyknit-ru.turbopages.org': 'easyknit.ru',
 'oper--1974-livejournal-com.turbopages.org': 'oper-1974.livejournal.com',
 'labnbo-ru.turbopages.org': 'labnbo.ru',
 'koteiki-com.turbopages.org': 'koteiki.com',
 'al--lobanov-livejournal-com.turbopages.org': 'al-lobanov.livejournal.com',
 'hairsite-ru.turbopages.org': 'hairsite.ru',
 'da--zdraperma-livejournal-com.turbopages.org': 'da-zdraperma.livejournal.com',
 'sherlyta-livejournal-com.turbopages.org': 'sherlyta.livejournal.com',
 'lenizdat-ru.turbopages.org': 'lenizdat.ru',
 'moscow-regnews-info.turbopages.org': 'moscow.regnews.info',
 'herbomania-livejournal-com.turbopages.org': 'herbomania.livejournal.com',
 'stalhammar-livejournal-com.turbopages.org': 'stalhammar.livejournal.com',
 'clubvolvo-ru.turbopages.org': 'clubvolvo.ru',
 'otdelka--expert-ru.turbopages.org': 'otdelka-expert.ru',
 'interiors--ru-livejournal-com.turbopages.org': 'interiors-ru.livejournal.com',
 'pricheska--strizhka-ru.turbopages.org': 'pricheska-strizhka.ru',
 'orelgrad-ru.turbopages.org': 'orelgrad.ru',
 'greenmarine-ru.turbopages.org': 'greenmarine.ru',
 'kurskcity-ru.turbopages.org': 'kurskcity.ru',
 'kursk-kp-ru.turbopages.org': 'kursk.kp.ru',
 'abgym-ru.turbopages.org': 'abgym.ru',
 'ok--karamel-ru.turbopages.org': 'ok-karamel.ru',
 'killproblem-livejournal-com.turbopages.org': 'killproblem.livejournal.com',
 'saransk-bezformata-com.turbopages.org': 'saransk.bezformata.com',
 'krugovoy-livejournal-com.turbopages.org': 'krugovoy.livejournal.com',
 'dispetcher--gruzoperevozok-biz.turbopages.org': 'dispetcher-gruzoperevozok.biz',
 'lebkatvas-livejournal-com.turbopages.org': 'lebkatvas.livejournal.com',
 'dmitri--obi-livejournal-com.turbopages.org': 'dmitri-obi.livejournal.com',
 'seoseed-ru.turbopages.org': 'seoseed.ru',
 'smokeexpert-ru.turbopages.org': 'smokeexpert.ru',
 'mindal--nails-ru.turbopages.org': 'mindal-nails.ru',
 'snn--mn-ru.turbopages.org': 'snn-mn.ru',
 'samodelkindrug-ru.turbopages.org': 'samodelkindrug.ru',
 'nehrena-ru.turbopages.org': 'nehrena.ru',
 'fermer-ru.turbopages.org': 'fermer.ru',
 'protinkoff-com.turbopages.org': 'protinkoff.com',
 'msk-dixinews-ru.turbopages.org': 'msk.dixinews.ru',
 'olgushka1971-ru.turbopages.org': 'olgushka1971.ru',
 'dgl-ru.turbopages.org': 'dgl.ru',
 'septik-guru.turbopages.org': 'septik.guru',
 'modlex-ru.turbopages.org': 'modlex.ru',
 'lev--balashov-livejournal-com.turbopages.org': 'lev-balashov.livejournal.com',
 'mk--mari-ru.turbopages.org': 'mk-mari.ru',
 'bondostock-ru.turbopages.org': 'bondostock.ru',
 'litl--admin-ru.turbopages.org': 'litl-admin.ru',
 '3quality-ru.turbopages.org': '3quality.ru',
 'gosuslugiinfo-site.turbopages.org': 'gosuslugiinfo.site',
 'komza-ru.turbopages.org': 'komza.ru',
 'dogovor--kupli--prodazhi-com.turbopages.org': 'dogovor-kupli-prodazhi.com',
 'vizazh--2-ru.turbopages.org': 'vizazh-2.ru',
 'poradka-ru.turbopages.org': 'poradka.ru',
 'vjazaniemaster-ru.turbopages.org': 'vjazaniemaster.ru',
 'goodgrunt-ru.turbopages.org': 'goodgrunt.ru',
 'sotkiradosti-ru.turbopages.org': 'sotkiradosti.ru',
 'ofazende-ru.turbopages.org': 'ofazende.ru',
 'vsaduidoma-com.turbopages.org': 'vsaduidoma.com',
 'family--cooking-ru.turbopages.org': 'family-cooking.ru',
 'nsk-rbc-ru.turbopages.org': 'nsk.rbc.ru',
 'moyamamma-ru.turbopages.org': 'moyamamma.ru',
 'cvet--dom-ru.turbopages.org': 'cvet-dom.ru',
 'brandwiki-ru.turbopages.org': 'brandwiki.ru',
 'child--blog-ru.turbopages.org': 'child-blog.ru',
 'ug--plastics-ru.turbopages.org': 'ug-plastics.ru',
 'journalvse-livejournal-com.turbopages.org': 'journalvse.livejournal.com',
 'yuristufa-livejournal-com.turbopages.org': 'yuristufa.livejournal.com',
 'zakon03-ru.turbopages.org': 'zakon03.ru',
 'proaveo-ru.turbopages.org': 'proaveo.ru',
 'shir4ik-livejournal-com.turbopages.org': 'shir4ik.livejournal.com',
 'dmitry--sasin-livejournal-com.turbopages.org': 'dmitry-sasin.livejournal.com',
 'tata--lind-livejournal-com.turbopages.org': 'tata-lind.livejournal.com',
 'blagin--anton-livejournal-com.turbopages.org': 'blagin-anton.livejournal.com',
    'resbash-ru.turbopages.org': 'resbash.ru',
 'mgazeta-com.turbopages.org': 'mgazeta.com',
 'texnotoys-ru.turbopages.org': 'texnotoys.ru',
 'itbusiness-com-ua.turbopages.org': 'itbusiness.com.ua',
 'androidinfo-ru.turbopages.org': 'androidinfo.ru',
 'psylogik-ru.turbopages.org': 'psylogik.ru',
 'myoptimus-net.turbopages.org': 'myoptimus.net',
 'lsvsx-livejournal-com.turbopages.org': 'lsvsx.livejournal.com',
 'puteshestvuy-com.turbopages.org': 'puteshestvuy.com',
 'naglecc-livejournal-com.turbopages.org': 'naglecc.livejournal.com',
 'bygeo-ru.turbopages.org': 'bygeo.ru',
 'psychbook-ru.turbopages.org': 'psychbook.ru',
 'sestratk-livejournal-com.turbopages.org': 'sestratk.livejournal.com',
 'feniks--kafe-ru.turbopages.org': 'feniks-kafe.ru',
 'azglagol-ru.turbopages.org': 'azglagol.ru',
 'uralpress-ru.turbopages.org': 'uralpress.ru',
 'sapojnik-livejournal-com.turbopages.org': 'sapojnik.livejournal.com',
 'ufa-aif-ru.turbopages.org': 'ufa.aif.ru',
 'newsnn-ru.turbopages.org': 'newsnn.ru',
 'oblgazeta-ru.turbopages.org': 'oblgazeta.ru',
 'terrnews-com.turbopages.org': 'terrnews.com',
 'primechaniya-ru.turbopages.org': 'primechaniya.ru',
 'ural-tsargrad-tv.turbopages.org': 'ural.tsargrad.tv',
 'chel-aif-ru.turbopages.org': 'chel.aif.ru',
 'uralweb-ru.turbopages.org': 'uralweb.ru',
 'dostup1-ru.turbopages.org': 'dostup1.ru',
 'ural--meridian-ru.turbopages.org': 'ural-meridian.ru',
 'globalmsk-ru.turbopages.org': 'globalmsk.ru',
 'prufy-ru.turbopages.org': 'prufy.ru',
 'ural-aif-ru.turbopages.org': 'ural.aif.ru',
 'argumentiru-com.turbopages.org': 'argumentiru.com',
 'weekend-rambler-ru.turbopages.org': 'weekend.rambler.ru',
 'news-sarbc-ru.turbopages.org': 'news.sarbc.ru',
 'novostivolgograda-ru.turbopages.org': 'novostivolgograda.ru',
 'saratov-mk-ru.turbopages.org': 'saratov.mk.ru',
 'kursdela-biz.turbopages.org': 'kursdela.biz',
 'prizivniku-com.turbopages.org': 'prizivniku.com',
 'gruzdevv-ru.turbopages.org': 'gruzdevv.ru',
 'pervouralsk-bezformata-com.turbopages.org': 'pervouralsk.bezformata.com',
 'gorodskievesti-ru.turbopages.org': 'gorodskievesti.ru',
 'shabdua-livejournal-com.turbopages.org': 'shabdua.livejournal.com',
 'vgoroden-ru.turbopages.org': 'vgoroden.ru',
 'newsroom24-ru.turbopages.org': 'newsroom24.ru',
 'gipernn-ru.turbopages.org': 'gipernn.ru',
 'polit-info.turbopages.org': 'polit.info',
 'sport-sevastopol-su.turbopages.org': 'sport.sevastopol.su',
 'otkan-ru.turbopages.org': 'otkan.ru',
 'lingvya-ru.turbopages.org': 'lingvya.ru',
 'krugozorro-ru.turbopages.org': 'krugozorro.ru',
 'autotime-net-ua.turbopages.org': 'autotime.net.ua',
 'banyaprofi-ru.turbopages.org': 'banyaprofi.ru',
 'moddam-ru.turbopages.org': 'moddam.ru',
 'gtrk-tv.turbopages.org': 'gtrk.tv',
 'mkset-ru.turbopages.org': 'mkset.ru',
 'buildingclub-ru.turbopages.org': 'buildingclub.ru',
 'newkaliningrad-ru.turbopages.org': 'newkaliningrad.ru',
 'passport--api-ru.turbopages.org': 'passport-api.ru',
 'vladimir-kp-ru.turbopages.org': 'vladimir.kp.ru',
 'suhoy-guru.turbopages.org': 'suhoy.guru',
 'sadovnic-su.turbopages.org': 'sadovnic.su',
 'mr--build-ru.turbopages.org': 'mr-build.ru',
 'miruspehinfo-ru.turbopages.org': 'miruspehinfo.ru',
 'silaznaniya8-ru.turbopages.org': 'silaznaniya8.ru',
 'multivarnik-ru.turbopages.org': 'multivarnik.ru',
 'zaryad--nn-ru.turbopages.org': 'zaryad-nn.ru',
 'rokgame-ru.turbopages.org': 'rokgame.ru',
 'zebra--tv-ru.turbopages.org': 'zebra-tv.ru',
 'cosmo--frost-ru.turbopages.org': 'cosmo-frost.ru',
 'kukuzya-ru.turbopages.org': 'kukuzya.ru',
 'stavropolye-tv.turbopages.org': 'stavropolye.tv',
 '161-ru.turbopages.org': '161.ru',
 'tagilcity-ru.turbopages.org': 'tagilcity.ru',
 'paperpaper-ru.turbopages.org': 'paperpaper.ru',
 'donnews-ru.turbopages.org': 'donnews.ru',
 'dontr-ru.turbopages.org': 'dontr.ru',
 'parents-ru.turbopages.org': 'parents.ru',
 'tvcook-ru.turbopages.org': 'tvcook.ru',
 'textiletrend-ru.turbopages.org': 'textiletrend.ru',
 'irk-kp-ru.turbopages.org': 'irk.kp.ru',
 'medicalinsider-ru.turbopages.org': 'medicalinsider.ru',
 'cityreporter-ru.turbopages.org': 'cityreporter.ru',
 'pravdoiskatel77-livejournal-com.turbopages.org': 'pravdoiskatel77.livejournal.com',
 'vsegda--tvoj-livejournal-com.turbopages.org': 'vsegda-tvoj.livejournal.com',
 'argo--ua-livejournal-com.turbopages.org': 'argo-ua.livejournal.com',
 'konsultiruet--yurist-ru.turbopages.org': 'konsultiruet-yurist.ru',
 'berezinan55-livejournal-com.turbopages.org': 'berezinan55.livejournal.com',
 'ko44-ru.turbopages.org': 'ko44.ru',
 'figaro-space.turbopages.org': 'figaro.space',
 'samara-tsargrad-tv.turbopages.org': 'samara.tsargrad.tv',
 'vestivrn-ru.turbopages.org': 'vestivrn.ru',
 'sdelai--lestnicu-ru.turbopages.org': 'sdelai-lestnicu.ru',
 'kstati-news.turbopages.org': 'kstati.news',
 'podskazki-info.turbopages.org': 'podskazki.info',
 'delus--opyt-ru.turbopages.org': 'delus-opyt.ru',
 'diabetsahar-ru.turbopages.org': 'diabetsahar.ru',
 'bursin-ru.turbopages.org': 'bursin.ru',
 'ivanvdovin-ru.turbopages.org': 'ivanvdovin.ru',
 'psysovet-ru.turbopages.org': 'psysovet.ru',
 'netran-ru.turbopages.org': 'netran.ru',
 'sprintinvest-ru.turbopages.org': 'sprintinvest.ru',
 'ural56-ru.turbopages.org': 'ural56.ru',
 'novotroisk-bezformata-com.turbopages.org': 'novotroisk.bezformata.com',
 'brenik-livejournal-com.turbopages.org': 'brenik.livejournal.com',
 'ecoportal-info.turbopages.org': 'ecoportal.info',
 'xn---com-u4datk7ac5anj5k.turbopages.org': 'xn-.com.u4datk7ac5anj5k',
 'grounde-ru.turbopages.org': 'grounde.ru',
 'rybakexpert-ru.turbopages.org': 'rybakexpert.ru',
 'fishingwiki-ru.turbopages.org': 'fishingwiki.ru',
 'factroom-ru.turbopages.org': 'factroom.ru',
 'bankinform-ru.turbopages.org': 'bankinform.ru',
 'vsapete-com.turbopages.org': 'vsapete.com',
 'xn------5cdcaj3bemn3ctrmqf7a4c.turbopages.org': 'xn---5cdcaj3bemn3ctrmqf7a4c',
 'mirnov-ru.turbopages.org': 'mirnov.ru',
 'progorod33-ru.turbopages.org': 'progorod33.ru',
 'vkusneetut-ru.turbopages.org': 'vkusneetut.ru',
 'chetbor-livejournal-com.turbopages.org': 'chetbor.livejournal.com',
 '9tv-co-il.turbopages.org': '9tv.co.il',
 'cyclearea-ru.turbopages.org': 'cyclearea.ru',
 'kozhamed-com.turbopages.org': 'kozhamed.com',
 'fruitree-ru.turbopages.org': 'fruitree.ru',
 'pochva-net.turbopages.org': 'pochva.net',
 'supernovum-ru.turbopages.org': 'supernovum.ru',
 'geogen--mir-livejournal-com.turbopages.org': 'geogen-mir.livejournal.com',
 'arinalipkina-livejournal-com.turbopages.org': 'arinalipkina.livejournal.com',
 'naim-ru.turbopages.org': 'naim.ru',
 'lifemotivation-online.turbopages.org': 'lifemotivation.online',
 'native--english-ru.turbopages.org': 'native-english.ru',
 'neuromed-online.turbopages.org': 'neuromed.online',
 'dnevnik--znaniy-ru.turbopages.org': 'dnevnik-znaniy.ru',
 'gudi1991-ru.turbopages.org': 'gudi1991.ru',
 'guruspa-ru.turbopages.org': 'guruspa.ru',
 'koolinar-ru.turbopages.org': 'koolinar.ru',
 'tvc-ru.turbopages.org': 'tvc.ru',
 'vlg--media-ru.turbopages.org': 'vlg-media.ru',
 'katusha--2109-livejournal-com.turbopages.org': 'katusha-2109.livejournal.com',
 'supersadovnik-ru.turbopages.org': 'supersadovnik.ru',
 'medialeaks-ru.turbopages.org': 'medialeaks.ru',
 'kevin--seoshnik-ru.turbopages.org': 'kevin-seoshnik.ru',
 'miniskazka-ru.turbopages.org': 'miniskazka.ru',
 'lektsia-com.turbopages.org': 'lektsia.com',
 'consumersjournal-org.turbopages.org': 'consumersjournal.org',
 'zaslonovgrad-ru.turbopages.org': 'zaslonovgrad.ru',
 'cosmeton-ru.turbopages.org': 'cosmeton.ru',
 'producttoday-ru.turbopages.org': 'producttoday.ru',
 'dns--magazin-ru.turbopages.org': 'dns-magazin.ru',
 'trmzk-ru.turbopages.org': 'trmzk.ru',
 'by-tsargrad-tv.turbopages.org': 'by.tsargrad.tv',
 'skbalkon-ru.turbopages.org': 'skbalkon.ru',
 'oreltimes-ru.turbopages.org': 'oreltimes.ru',
 'kulupa-livejournal-com.turbopages.org': 'kulupa.livejournal.com',
 'arch--heritage-livejournal-com.turbopages.org': 'arch-heritage.livejournal.com',
 'uchazdneg-livejournal-com.turbopages.org': 'uchazdneg.livejournal.com',
 'gosuchetnik-ru.turbopages.org': 'gosuchetnik.ru',
 'el--klinika-ru.turbopages.org': 'el-klinika.ru',
 'digitalsquare-ru.turbopages.org': 'digitalsquare.ru',
 'newsru-com.turbopages.org': 'newsru.com',
 'jiyuu-su.turbopages.org': 'jiyuu.su',
 'natural--cosmetology-ru.turbopages.org': 'natural-cosmetology.ru',
 'mastera--rukodeliya-ru.turbopages.org': 'mastera-rukodeliya.ru',
 'tarologiay-ru.turbopages.org': 'tarologiay.ru',
 'mo-tsargrad-tv.turbopages.org': 'mo.tsargrad.tv',
 'obane-info.turbopages.org': 'obane.info',
 'data--vyhoda-com.turbopages.org': 'data-vyhoda.com',
 'vr--land-ru.turbopages.org': 'vr-land.ru',
 'stav-aif-ru.turbopages.org': 'stav.aif.ru',
 'book--cooks-ru.turbopages.org': 'book-cooks.ru',
 'spbinteres-ru.turbopages.org': 'spbinteres.ru',
 'evo--lutio-livejournal-com.turbopages.org': 'evo-lutio.livejournal.com',
 'carsclick-ru.turbopages.org': 'carsclick.ru',
 'ohnadi-livejournal-com.turbopages.org': 'ohnadi.livejournal.com',
 '1000inf-ru.turbopages.org': '1000inf.ru',
 'v--kurse-ru.turbopages.org': 'v-kurse.ru',
 'boozers-ru.turbopages.org': 'boozers.ru',
 'informburo-kz.turbopages.org': 'informburo.kz',
 'obvm-ru.turbopages.org': 'obvm.ru',
 'shoubiz-guru.turbopages.org': 'shoubiz.guru',
 'fitzdrav-com.turbopages.org': 'fitzdrav.com',
 'b--mag-ru.turbopages.org': 'b-mag.ru',
 'ctv-by.turbopages.org': 'ctv.by',
 'belta-by.turbopages.org': 'belta.by',
 'kaliningrad-kp-ru.turbopages.org': 'kaliningrad.kp.ru',
 'prmira-ru.turbopages.org': 'prmira.ru',
 'tver-aif-ru.turbopages.org': 'tver.aif.ru',
 'nauka-tass-ru.turbopages.org': 'nauka.tass.ru',
 'pg11-ru.turbopages.org': 'pg11.ru',
 'techcult-ru.turbopages.org': 'techcult.ru',
 'vrn-aif-ru.turbopages.org': 'vrn.aif.ru',
 'zavtra-ru.turbopages.org': 'zavtra.ru',
 'soulcar-ru.turbopages.org': 'soulcar.ru',
 'protabletpc-ru.turbopages.org': 'protabletpc.ru',
 'mirogid-ru.turbopages.org': 'mirogid.ru',
 'eurosantehnik-ru.turbopages.org': 'eurosantehnik.ru',
 'vygoda-ru.turbopages.org': 'vygoda.ru',
 'svetorusie-livejournal-com.turbopages.org': 'svetorusie.livejournal.com',
 'sdelanounas-ru.turbopages.org': 'sdelanounas.ru',
 'jolly--roger--y-livejournal-com.turbopages.org': 'jolly-roger-y.livejournal.com',
 'agronom-expert.turbopages.org': 'agronom.expert',
 'mebel--expert-info.turbopages.org': 'mebel-expert.info',
 'serp-mk-ru.turbopages.org': 'serp.mk.ru',
 'rus--bel-online.turbopages.org': 'rus-bel.online',
 'hudvuz-livejournal-com.turbopages.org': 'hudvuz.livejournal.com',
 'etokavkaz-ru.turbopages.org': 'etokavkaz.ru',
 'xn--45--5cdal9cxcvb.turbopages.org': 'xn-45-5cdal9cxcvb',
 'aquarium--fish--home-ru.turbopages.org': 'aquarium-fish-home.ru',
 'greenparkomsk-ru.turbopages.org': 'greenparkomsk.ru',
 'mrhvost-com.turbopages.org': 'mrhvost.com',
 'domostroynn-ru.turbopages.org': 'domostroynn.ru',
 'akki--carsh-ru.turbopages.org': 'akki-carsh.ru',
 'gkd-ru.turbopages.org': 'gkd.ru',
 'spectorg-su.turbopages.org': 'spectorg.su',
 'xn----etbgmjbfo6amkv.turbopages.org': 'xn--etbgmjbfo6amkv',
 'nversia-ru.turbopages.org': 'nversia.ru',
 'gazeta--unp-ru.turbopages.org': 'gazeta-unp.ru',
 'saratov-kp-ru.turbopages.org': 'saratov.kp.ru',
 'zdravstvyite-ru.turbopages.org': 'zdravstvyite.ru',
 'bel-ru.turbopages.org': 'bel.ru',
 'mykaleidoscope-ru.turbopages.org': 'mykaleidoscope.ru',
 'modaok-ru.turbopages.org': 'modaok.ru',
 'znat--kak-livejournal-com.turbopages.org': 'znat-kak.livejournal.com',
 'blogobane-ru.turbopages.org': 'blogobane.ru',
 'pohod--vosemvrat-livejournal-com.turbopages.org': 'pohod-vosemvrat.livejournal.com',
 'vostorg--wedding-ru.turbopages.org': 'vostorg-wedding.ru',
 'bw4u-ru.turbopages.org': 'bw4u.ru',
 'protomarius-livejournal-com.turbopages.org': 'protomarius.livejournal.com',
 '4languagetutors-ru.turbopages.org': '4languagetutors.ru',
 'banyaportal-ru.turbopages.org': 'banyaportal.ru',
 'argumentplus-ru.turbopages.org': 'argumentplus.ru',
 'horosho--tam-ru.turbopages.org': 'horosho-tam.ru',
 '47news-ru.turbopages.org': '47news.ru',
 'politpuzzle-ru.turbopages.org': 'politpuzzle.ru',
 'plastic--club-ru.turbopages.org': 'plastic-club.ru',
 'tks-ru.turbopages.org': 'tks.ru',
 'newkuban-ru.turbopages.org': 'newkuban.ru',
 'smehuechek-ru.turbopages.org': 'smehuechek.ru',
 'vmeste--masterim-ru.turbopages.org': 'vmeste-masterim.ru',
 'chestniy--yurist-livejournal-com.turbopages.org': 'chestniy-yurist.livejournal.com',
 'rtiger-com.turbopages.org': 'rtiger.com',
 'econ-wikireading-ru.turbopages.org': 'econ.wikireading.ru',
 'wiselawyer-ru.turbopages.org': 'wiselawyer.ru',
 'washepravo-ru.turbopages.org': 'washepravo.ru',
 'delta--instrument-ru.turbopages.org': 'delta-instrument.ru',
 'autointerior-ru.turbopages.org': 'autointerior.ru',
 'prvadm-ru.turbopages.org': 'prvadm.ru',
 'afanasy-biz.turbopages.org': 'afanasy.biz',
 'oventilyacii-ru.turbopages.org': 'oventilyacii.ru',
 'totamtotut-ru.turbopages.org': 'totamtotut.ru',
 'avtoradio-ru.turbopages.org': 'avtoradio.ru',
 'moelibido-com.turbopages.org': 'moelibido.com',
 'karhu53-livejournal-com.turbopages.org': 'karhu53.livejournal.com',
 'readd-org.turbopages.org': 'readd.org',
 'yogaclass-su.turbopages.org': 'yogaclass.su',
 'comhub-ru.turbopages.org': 'comhub.ru',
 'pg21-ru.turbopages.org': 'pg21.ru',
 'by-tribuna-com.turbopages.org': 'by.tribuna.com',
 'njcar-ru.turbopages.org': 'njcar.ru',
 'oane-ws.turbopages.org': 'oane.ws',
 'sport--interfax-ru.turbopages.org': 'sport-interfax.ru',
 'metronews-ru.turbopages.org': 'metronews.ru',
 'spadilo-ru.turbopages.org': 'spadilo.ru',
 'vetta-tv.turbopages.org': 'vetta.tv',
 'ara--bublik-livejournal-com.turbopages.org': 'ara-bublik.livejournal.com',
 'socketmira-ru.turbopages.org': 'socketmira.ru',
 'ne--veli4ka-livejournal-com.turbopages.org': 'ne-veli4ka.livejournal.com',
 '101million-com.turbopages.org': '101million.com',
 'procvety-net.turbopages.org': 'procvety.net',
 'appleinsider-ru.turbopages.org': 'appleinsider.ru',
 'textilegu-ru.turbopages.org': 'textilegu.ru',
 'aboutdiabetes-ru.turbopages.org': 'aboutdiabetes.ru',
 'class--tour-com.turbopages.org': 'class-tour.com',
 'razvivaem--rebenka-ru.turbopages.org': 'razvivaem-rebenka.ru',
 'world-news--kmv-ru.turbopages.org': 'world.news-kmv.ru',
 'zarabotkionline-ru.turbopages.org': 'zarabotkionline.ru',
 'pg12-ru.turbopages.org': 'pg12.ru',
 'cadelta-ru.turbopages.org': 'cadelta.ru',
 'realnyezarabotki-com.turbopages.org': 'realnyezarabotki.com',
 'ustanovkaos-ru.turbopages.org': 'ustanovkaos.ru',
 'promusor-info.turbopages.org': 'promusor.info',
 'newstracker-ru.turbopages.org': 'newstracker.ru',
 'angelnumber-ru.turbopages.org': 'angelnumber.ru',
 '7sotok-com.turbopages.org': '7sotok.com',
 'my--happyfood-livejournal-com.turbopages.org': 'my-happyfood.livejournal.com',
 '1yar-tv.turbopages.org': '1yar.tv',
 'go31-ru.turbopages.org': 'go31.ru',
 'mk--belgorod-ru.turbopages.org': 'mk-belgorod.ru',
 'bel-aif-ru.turbopages.org': 'bel.aif.ru',
 'mirbelogorya-ru.turbopages.org': 'mirbelogorya.ru',
 'sadgrad-ru.turbopages.org': 'sadgrad.ru',
 'bel-kp-ru.turbopages.org': 'bel.kp.ru',
 'belnovosti-ru.turbopages.org': 'belnovosti.ru',
 'gudvill-com.turbopages.org': 'gudvill.com',
 'abireg-ru.turbopages.org': 'abireg.ru',
 'sevastopol-su.turbopages.org': 'sevastopol.su',
 'mfarma-ru.turbopages.org': 'mfarma.ru',
 'xn----otbfbifgbw4byc2b.turbopages.org': 'xn--otbfbifgbw4byc2b',
 'pitaniedeti-ru.turbopages.org': 'pitaniedeti.ru',
 'i--tc-ru.turbopages.org': 'i-tc.ru',
 'newsler-ru.turbopages.org': 'newsler.ru',
 'vgorodekirove-ru.turbopages.org': 'vgorodekirove.ru',
 'nkclinica-ru.turbopages.org': 'nkclinica.ru',
 'nastalo-ru.turbopages.org': 'nastalo.ru',
 'gavriel--08-livejournal-com.turbopages.org': 'gavriel-08.livejournal.com',
 'leoblagovest-livejournal-com.turbopages.org': 'leoblagovest.livejournal.com',
 'detskie--skazki-net.turbopages.org': 'detskie-skazki.net',
 'druidnoname-livejournal-com.turbopages.org': 'druidnoname.livejournal.com',
 'neuport-livejournal-com.turbopages.org': 'neuport.livejournal.com',
 'gaimorit--stop-ru.turbopages.org': 'gaimorit-stop.ru',
 'paisiypchelnik-livejournal-com.turbopages.org': 'paisiypchelnik.livejournal.com',
 'punkt--a-info.turbopages.org': 'punkt-a.info',
 'freesmi--by-livejournal-com.turbopages.org': 'freesmi-by.livejournal.com',
 'linur2-livejournal-com.turbopages.org': 'linur2.livejournal.com',
 'bracatus-com.turbopages.org': 'bracatus.com',
 'to--priz-livejournal-com.turbopages.org': 'to-priz.livejournal.com',
 'vesti-kg.turbopages.org': 'vesti.kg',
 'panram-ru.turbopages.org': 'panram.ru',
 'gorabbit-ru.turbopages.org': 'gorabbit.ru',
 'malka--lorenz-livejournal-com.turbopages.org': 'malka-lorenz.livejournal.com',
 'digitall--angell-livejournal-com.turbopages.org': 'digitall-angell.livejournal.com',
 'volgasib-ru.turbopages.org': 'volgasib.ru',
 'gol-ru.turbopages.org': 'gol.ru',
 'v102-ru.turbopages.org': 'v102.ru',
 'tvercult-ru.turbopages.org': 'tvercult.ru',
 'eto--shef-livejournal-com.turbopages.org': 'eto-shef.livejournal.com',
 'oneloveweddingexperience-com.turbopages.org': 'oneloveweddingexperience.com',
 'turk--in-ru.turbopages.org': 'turk-in.ru',
 'e--news-su.turbopages.org': 'e-news.su',
 'avto--idea-ru.turbopages.org': 'avto-idea.ru',
 'russos-livejournal-com.turbopages.org': 'russos.livejournal.com',
 'teplica--exp-ru.turbopages.org': 'teplica-exp.ru',
 'zaboryprofi-ru.turbopages.org': 'zaboryprofi.ru',
 'kakpostroitdomic-ru.turbopages.org': 'kakpostroitdomic.ru',
 'int43-ru.turbopages.org': 'int43.ru',
 'trubanet-ru.turbopages.org': 'trubanet.ru',
 'hozsektor-com.turbopages.org': 'hozsektor.com',
 'efanera-ru.turbopages.org': 'efanera.ru',
 'babushkinadacha-ru.turbopages.org': 'babushkinadacha.ru',
 'ikea--club-org.turbopages.org': 'ikea-club.org',
 'ayaznal-ru.turbopages.org': 'ayaznal.ru',
 'ribalych-ru.turbopages.org': 'ribalych.ru',
 'homeenglish-ru.turbopages.org': 'homeenglish.ru',
 'sputnikkurort-ru.turbopages.org': 'sputnikkurort.ru',
 'jv-ru.turbopages.org': 'jv.ru',
 'best--top10-ru.turbopages.org': 'best-top10.ru',
 '1tulatv-ru.turbopages.org': '1tulatv.ru',
 'gotovite-ru.turbopages.org': 'gotovite.ru',
 'lek-103-ua.turbopages.org': 'lek.103.ua',
 'xn----7sbgao7codm0a9fya.turbopages.org': 'xn--7sbgao7codm0a9fya',
 'athunder-livejournal-com.turbopages.org': 'athunder.livejournal.com',
 'bulkin-me.turbopages.org': 'bulkin.me',
 'strana-ua.turbopages.org': 'strana.ua',
 'cher--poisk-ru.turbopages.org': 'cher-poisk.ru',
 'cherinfo-ru.turbopages.org': 'cherinfo.ru',
 'volga-news.turbopages.org': 'volga.news',
 'antinasekom-ru.turbopages.org': 'antinasekom.ru',
 'nn-tsargrad-tv.turbopages.org': 'nn.tsargrad.tv',
 'samara-aif-ru.turbopages.org': 'samara.aif.ru',
 'ap22-ru.turbopages.org': 'ap22.ru',
 'mosaica-ru.turbopages.org': 'mosaica.ru',
 '73online-ru.turbopages.org': '73online.ru',
 'mycary-ru.turbopages.org': 'mycary.ru',
 'prolmet-ru.turbopages.org': 'prolmet.ru',
 'ugra--tv-ru.turbopages.org': 'ugra-tv.ru',
 'eanews-ru.turbopages.org': 'eanews.ru',
 'stop--obman-com.turbopages.org': 'stop-obman.com',
 'fructberry-com.turbopages.org': 'fructberry.com',
 'dslov-ru.turbopages.org': 'dslov.ru',
 'amarok--man-livejournal-com.turbopages.org': 'amarok-man.livejournal.com',
 'stylenews-ru.turbopages.org': 'stylenews.ru',
 'buffett-ru.turbopages.org': 'buffett.ru',
 'herbalpedia-ru.turbopages.org': 'herbalpedia.ru',
 'medcentre-com-ua.turbopages.org': 'medcentre.com.ua',
 'zarplata--online-ru.turbopages.org': 'zarplata-online.ru',
 'vseloto-ru.turbopages.org': 'vseloto.ru',
 'vitas1917-livejournal-com.turbopages.org': 'vitas1917.livejournal.com',
 'popugai--volnistye-ru.turbopages.org': 'popugai-volnistye.ru',
 'cookmaster-online.turbopages.org': 'cookmaster.online',
 'neuro--ural-ru.turbopages.org': 'neuro-ural.ru',
 'yuga-ru.turbopages.org': 'yuga.ru',
 'blank--dogovor--kupli--prodazhi-ru.turbopages.org': 'blank-dogovor-kupli-prodazhi.ru',
 'ugo--kamsk-ru.turbopages.org': 'ugo-kamsk.ru',
 'domstrousam-ru.turbopages.org': 'domstrousam.ru',
 'prajt-livejournal-com.turbopages.org': 'prajt.livejournal.com',
 'philologist-livejournal-com.turbopages.org': 'philologist.livejournal.com',
 'lorises-ru.turbopages.org': 'lorises.ru',
 'portal--mashin-ru.turbopages.org': 'portal-mashin.ru',
 'altimart-ru.turbopages.org': 'altimart.ru',
 'regionfinans-ru.turbopages.org': 'regionfinans.ru',
 'soberi--kubik-ru.turbopages.org': 'soberi-kubik.ru',
 'city--sochi-ru.turbopages.org': 'city-sochi.ru',
 'medboli-ru.turbopages.org': 'medboli.ru',
 'zdravoline-info.turbopages.org': 'zdravoline.info',
 'roomester-ru.turbopages.org': 'roomester.ru',
 'rehouz-info.turbopages.org': 'rehouz.info',
 'irina--chisa-livejournal-com.turbopages.org': 'irina-chisa.livejournal.com',
 'radonezhskij-livejournal-com.turbopages.org': 'radonezhskij.livejournal.com',
 'ani--al-livejournal-com.turbopages.org': 'ani-al.livejournal.com',
 'nasledie-pravda-ru.turbopages.org': 'nasledie.pravda.ru',
 'wiki-risk-ru.turbopages.org': 'wiki.risk.ru',
 'alexio--marziano-livejournal-com.turbopages.org': 'alexio-marziano.livejournal.com',
 'aranwe--permello-livejournal-com.turbopages.org': 'aranwe-permello.livejournal.com',
 'skaramanga--1972-livejournal-com.turbopages.org': 'skaramanga-1972.livejournal.com',
 'tula-kp-ru.turbopages.org': 'tula.kp.ru',
 'liveposts-ru.turbopages.org': 'liveposts.ru',
 'tema-livejournal-com.turbopages.org': 'tema.livejournal.com',
 'topwar-ru.turbopages.org': 'topwar.ru',
 'comp--plus-ru.turbopages.org': 'comp-plus.ru',
 'online812-ru.turbopages.org': 'online812.ru',
 'kak--eto--sdelano-livejournal-com.turbopages.org': 'kak-eto-sdelano.livejournal.com',
 'avtozvuk--info-ru.turbopages.org': 'avtozvuk-info.ru',
 'avtocity365-ru.turbopages.org': 'avtocity365.ru',
 'invlab-ru.turbopages.org': 'invlab.ru',
 'technorama-ru.turbopages.org': 'technorama.ru',
 'sterlegrad-ru.turbopages.org': 'sterlegrad.ru',
 'ivanovo-sminews-ru.turbopages.org': 'ivanovo.sminews.ru',
 'glamusha-ru.turbopages.org': 'glamusha.ru',
 'quangel-livejournal-com.turbopages.org': 'quangel.livejournal.com',
 'beerhead--bar-ru.turbopages.org': 'beerhead-bar.ru',
 'infonetru-com.turbopages.org': 'infonetru.com',
 'larasdvatri123-livejournal-com.turbopages.org': 'larasdvatri123.livejournal.com',
 'zermd-livejournal-com.turbopages.org': 'zermd.livejournal.com',
 'psychologies-ru.turbopages.org': 'psychologies.ru',
 'graf--orlov33-livejournal-com.turbopages.org': 'graf-orlov33.livejournal.com',
 'sergeysmirnovblog-ru.turbopages.org': 'sergeysmirnovblog.ru',
 'bvk-news.turbopages.org': 'bvk.news',
 'antonio--1984-livejournal-com.turbopages.org': 'antonio-1984.livejournal.com',
 'rus--zoo-ru.turbopages.org': 'rus-zoo.ru',
 'zveridoma-ru.turbopages.org': 'zveridoma.ru',
 'biletik-aero.turbopages.org': 'biletik.aero',
 'vn-ru.turbopages.org': 'vn.ru',
 'magadanmedia-ru.turbopages.org': 'magadanmedia.ru',
 'ru-motorsport-com.turbopages.org': 'ru.motorsport.com',
 'gotennis-ru.turbopages.org': 'gotennis.ru',
 'pressball-by.turbopages.org': 'pressball.by',
 'uznai24-ru.turbopages.org': 'uznai24.ru',
 'luchshie--akcii-ru.turbopages.org': 'luchshie-akcii.ru',
 'staysane71-livejournal-com.turbopages.org': 'staysane71.livejournal.com',
 'zdorovyi--stol-ru.turbopages.org': 'zdorovyi-stol.ru',
 'von--hoffmann-livejournal-com.turbopages.org': 'von-hoffmann.livejournal.com',
 'krasota--altaia-livejournal-com.turbopages.org': 'krasota-altaia.livejournal.com',
 'avtogorn-ru.turbopages.org': 'avtogorn.ru',
 'v161514-livejournal-com.turbopages.org': 'v161514.livejournal.com',
 'bryumer-livejournal-com.turbopages.org': 'bryumer.livejournal.com',
 'back--in--ussr-com.turbopages.org': 'back-in-ussr.com',
 'rasteniadacha-ru.turbopages.org': 'rasteniadacha.ru',
 'pro100ogorod-ru.turbopages.org': 'pro100ogorod.ru',
 'agroxxi-ru.turbopages.org': 'agroxxi.ru',
 'moreprodukt-info.turbopages.org': 'moreprodukt.info',
 'ufa-mk-ru.turbopages.org': 'ufa.mk.ru',
 '220-guru.turbopages.org': '220.guru',
 'newsmma-net.turbopages.org': 'newsmma.net',
 'kvartirka-com.turbopages.org': 'kvartirka.com',
 'salsknews-ru.turbopages.org': 'salsknews.ru',
 'lechuguru-com.turbopages.org': 'lechuguru.com',
 'xn----itbfdaacxipwico5b.turbopages.org': 'xn--itbfdaacxipwico5b',
 'domodedovoriamo-ru.turbopages.org': 'domodedovoriamo.ru',
 'ilifia--club-ru.turbopages.org': 'ilifia-club.ru',
 'life--with--cars-ru.turbopages.org': 'life-with-cars.ru',
 'rk--news-com.turbopages.org': 'rk-news.com',
 'avtomotoprof-ru.turbopages.org': 'avtomotoprof.ru',
 'mymsk-online.turbopages.org': 'mymsk.online',
 'newsvo-ru.turbopages.org': 'newsvo.ru',
 'razborka-org.turbopages.org': 'razborka.org',
 '1remontauto-ru.turbopages.org': '1remontauto.ru',
 'pay--day-ru.turbopages.org': 'pay-day.ru',
 'moikoty-ru.turbopages.org': 'moikoty.ru',
 'vlad-aif-ru.turbopages.org': 'vlad.aif.ru',
 'tolkovaniyasnov-ru.turbopages.org': 'tolkovaniyasnov.ru',
 'cook--s-ru.turbopages.org': 'cook-s.ru',
 'lfkplus-ru.turbopages.org': 'lfkplus.ru',
 'wtftime-ru.turbopages.org': 'wtftime.ru',
 'carsharingi-ru.turbopages.org': 'carsharingi.ru',
 'flakonn-com.turbopages.org': 'flakonn.com',
 'dmpokrov-livejournal-com.turbopages.org': 'dmpokrov.livejournal.com',
 'omez-ru.turbopages.org': 'omez.ru',
 'rus-team.turbopages.org': 'rus.team',
 'remstroiblog-ru.turbopages.org': 'remstroiblog.ru',
 'compnovosti-ru.turbopages.org': 'compnovosti.ru',
 'perm-aif-ru.turbopages.org': 'perm.aif.ru',
 'atvmedia-ru.turbopages.org': 'atvmedia.ru',
 'togliatti24-ru.turbopages.org': 'togliatti24.ru',
 'kreditorpro-ru.turbopages.org': 'kreditorpro.ru',
 'cwetochki-ru.turbopages.org': 'cwetochki.ru',
 'stroi--s--ka-ru.turbopages.org': 'stroi-s-ka.ru',
 'vse--pro--laminat-ru.turbopages.org': 'vse-pro-laminat.ru',
 'xn----9sbgsqkhcu7ah.turbopages.org': 'xn--9sbgsqkhcu7ah',
 'spark--welding-ru.turbopages.org': 'spark-welding.ru',
 'apkhleb-ru.turbopages.org': 'apkhleb.ru',
 'forums-rusmedserv-com.turbopages.org': 'forums.rusmedserv.com',
 'amparo--a-livejournal-com.turbopages.org': 'amparo-a.livejournal.com',
 'soltyk-ru.turbopages.org': 'soltyk.ru',
 'yakutiamedia-ru.turbopages.org': 'yakutiamedia.ru',
 'samyiluchshiy-ru.turbopages.org': 'samyiluchshiy.ru',
 'awayne-biz.turbopages.org': 'awayne.biz',
 'doctos-ru.turbopages.org': 'doctos.ru',
 'podolsk-sminews-ru.turbopages.org': 'podolsk.sminews.ru',
 'ecoplanet777-com.turbopages.org': 'ecoplanet777.com',
 'astera-ru.turbopages.org': 'astera.ru',
 'mir--knig-com.turbopages.org': 'mir-knig.com',
 'glebmusic-ru.turbopages.org': 'glebmusic.ru',
 'veved-ru.turbopages.org': 'veved.ru',
 'ladstas-livejournal-com.turbopages.org': 'ladstas.livejournal.com',
 'belonika-livejournal-com.turbopages.org': 'belonika.livejournal.com',
 'reform-by.turbopages.org': 'reform.by',
 'tula-mk-ru.turbopages.org': 'tula.mk.ru',
 'urist-expert.turbopages.org': 'urist.expert',
 'propk-ru.turbopages.org': 'propk.ru',
 'schooldistance-ru.turbopages.org': 'schooldistance.ru',
 'cutur-ru.turbopages.org': 'cutur.ru',
 'krasotka-club.turbopages.org': 'krasotka.club',
 'gtasa--live-ru.turbopages.org': 'gtasa-live.ru',
 'medcentre24-ru.turbopages.org': 'medcentre24.ru',
 'stories--of--success-ru.turbopages.org': 'stories-of-success.ru',
 'yasnonews-ru.turbopages.org': 'yasnonews.ru',
 'griphon-livejournal-com.turbopages.org': 'griphon.livejournal.com',
 'sergey--larenkov-livejournal-com.turbopages.org': 'sergey-larenkov.livejournal.com',
 'abkhazia-livejournal-com.turbopages.org': 'abkhazia.livejournal.com',
 'alex--dars-livejournal-com.turbopages.org': 'alex-dars.livejournal.com',
 'sell--off-livejournal-com.turbopages.org': 'sell-off.livejournal.com',
 'visualhistory-livejournal-com.turbopages.org': 'visualhistory.livejournal.com',
 'starbeak-livejournal-com.turbopages.org': 'starbeak.livejournal.com',
 'dmitryjewski-livejournal-com.turbopages.org': 'dmitryjewski.livejournal.com',
 'don24-ru.turbopages.org': 'don24.ru',
 'confeton-ru.turbopages.org': 'confeton.ru',
 'begzdorov-ru.turbopages.org': 'begzdorov.ru',
 'naturlich--restoran-ru.turbopages.org': 'naturlich-restoran.ru',
 'pskov-bezformata-com.turbopages.org': 'pskov.bezformata.com',
 'volga--day-ru.turbopages.org': 'volga-day.ru',
 'uraloved-ru.turbopages.org': 'uraloved.ru',
 'kupipet-ru.turbopages.org': 'kupipet.ru',
 'msmetall-ru.turbopages.org': 'msmetall.ru',
 'perunica-ru.turbopages.org': 'perunica.ru',
 'online47-ru.turbopages.org': 'online47.ru',
 'scastje--est-ru.turbopages.org': 'scastje-est.ru',
 'gtrkpskov-ru.turbopages.org': 'gtrkpskov.ru',
 'samnadache-ru.turbopages.org': 'samnadache.ru',
 'picturehistory-livejournal-com.turbopages.org': 'picturehistory.livejournal.com',
 'traveller--eu-ru.turbopages.org': 'traveller-eu.ru',
 'pora--valit-livejournal-com.turbopages.org': 'pora-valit.livejournal.com',
 'cryptoplaneta-ru.turbopages.org': 'cryptoplaneta.ru',
 'juan-livejournal-com.turbopages.org': 'juan.livejournal.com',
 'tehnobzor-ru.turbopages.org': 'tehnobzor.ru',
 'drivenn-ru.turbopages.org': 'drivenn.ru',
 'housesitter-ru.turbopages.org': 'housesitter.ru',
 'itfit-pro.turbopages.org': 'itfit.pro',
 'kuhatpodano-ru.turbopages.org': 'kuhatpodano.ru',
 'maria--selyanina-livejournal-com.turbopages.org': 'maria-selyanina.livejournal.com',
 'wi--fi-ru.turbopages.org': 'wi-fi.ru',
 'turplay-ru.turbopages.org': 'turplay.ru',
 'listaj-ru.turbopages.org': 'listaj.ru',
 'fasad--exp-ru.turbopages.org': 'fasad-exp.ru',
 'selhozobzor-ru.turbopages.org': 'selhozobzor.ru',
 'ogorodnikam-info.turbopages.org': 'ogorodnikam.info',
 'mr--ogorodnik-ru.turbopages.org': 'mr-ogorodnik.ru',
 'novocom-org.turbopages.org': 'novocom.org',
 'dacha365-net.turbopages.org': 'dacha365.net',
 'mariapolina-ru.turbopages.org': 'mariapolina.ru',
 'allreleases-ru.turbopages.org': 'allreleases.ru',
 'inha-ru.turbopages.org': 'inha.ru',
 'vkuspo-info.turbopages.org': 'vkuspo.info',
 'infodacha-ru.turbopages.org': 'infodacha.ru',
 'psyteaman-livejournal-com.turbopages.org': 'psyteaman.livejournal.com',
 'tanja--tank-livejournal-com.turbopages.org': 'tanja-tank.livejournal.com',
 'tgram-ru.turbopages.org': 'tgram.ru',
 'udm--info-ru.turbopages.org': 'udm-info.ru',
 'gamergirl-ru.turbopages.org': 'gamergirl.ru',
 'vladimir--krimov-livejournal-com.turbopages.org': 'vladimir-krimov.livejournal.com',
 'varjag2007su-livejournal-com.turbopages.org': 'varjag2007su.livejournal.com',
 'samsebevoin-ru.turbopages.org': 'samsebevoin.ru',
 'restra-net.turbopages.org': 'restra.net',
 'kriminalnn-ru.turbopages.org': 'kriminalnn.ru',
 'paprika--andlife-livejournal-com.turbopages.org': 'paprika-andlife.livejournal.com',
 'yana--igraeva-livejournal-com.turbopages.org': 'yana-igraeva.livejournal.com',
 'lammantin-livejournal-com.turbopages.org': 'lammantin.livejournal.com',
 'game--skills-ru.turbopages.org': 'game-skills.ru',
 'mikhael--mark-livejournal-com.turbopages.org': 'mikhael-mark.livejournal.com',
 'santehnikguru-ru.turbopages.org': 'santehnikguru.ru',
 'diksio-livejournal-com.turbopages.org': 'diksio.livejournal.com',
 'vseneobichnoe-livejournal-com.turbopages.org': 'vseneobichnoe.livejournal.com',
 'vrgames-by.turbopages.org': 'vrgames.by',
 'tvin270584-livejournal-com.turbopages.org': 'tvin270584.livejournal.com',
 'kaljaninfo-ru.turbopages.org': 'kaljaninfo.ru',
 'hidogs-ru.turbopages.org': 'hidogs.ru',
 'progov-ru.turbopages.org': 'progov.ru',
 'vyvoz-org.turbopages.org': 'vyvoz.org',
 'evropa--park-ru.turbopages.org': 'evropa-park.ru',
 'luludmila-ru.turbopages.org': 'luludmila.ru',
 '7sisters-ru.turbopages.org': '7sisters.ru',
 'vkusnosushi-ru.turbopages.org': 'vkusnosushi.ru',
 'itcrumbs-ru.turbopages.org': 'itcrumbs.ru',
 'mama--i--deti-ru.turbopages.org': 'mama-i-deti.ru',
 'spbliders-ru.turbopages.org': 'spbliders.ru',
 'gadalkindom-ru.turbopages.org': 'gadalkindom.ru',
 'moscowtoner-ru.turbopages.org': 'moscowtoner.ru',
 '8war-ru.turbopages.org': '8war.ru',
 'pochini-guru.turbopages.org': 'pochini.guru',
 'le--milady-livejournal-com.turbopages.org': 'le-milady.livejournal.com',
 'trashexpert-ru.turbopages.org': 'trashexpert.ru',
 'grimnir74-livejournal-com.turbopages.org': 'grimnir74.livejournal.com',
 'journal-learnoff-com.turbopages.org': 'journal.learnoff.com',
 'sherycat-livejournal-com.turbopages.org': 'sherycat.livejournal.com',
 'headinsider-net.turbopages.org': 'headinsider.net',
 'livejournal-com.turbopages.org': 'livejournal.com',
 'ros--obrazovanie-ru.turbopages.org': 'ros-obrazovanie.ru',
 'msk-ramsaydiagnostics-ru.turbopages.org': 'msk.ramsaydiagnostics.ru',
 'hamshenci-livejournal-com.turbopages.org': 'hamshenci.livejournal.com',
 'elenaruvel-com.turbopages.org': 'elenaruvel.com',
 'sam--stroitel-com.turbopages.org': 'sam-stroitel.com',
 'iceforge-ru.turbopages.org': 'iceforge.ru',
 'mor77-livejournal-com.turbopages.org': 'mor77.livejournal.com',
 'trof--av-livejournal-com.turbopages.org': 'trof-av.livejournal.com',
 'andrewbek--1974-livejournal-com.turbopages.org': 'andrewbek-1974.livejournal.com',
 'panzer038-livejournal-com.turbopages.org': 'panzer038.livejournal.com',
 'satchel17-livejournal-com.turbopages.org': 'satchel17.livejournal.com',
 'fomasovetnik-livejournal-com.turbopages.org': 'fomasovetnik.livejournal.com',
 'pulman-livejournal-com.turbopages.org': 'pulman.livejournal.com',
 'radio--rhodesia-livejournal-com.turbopages.org': 'radio-rhodesia.livejournal.com',
 'mihalchuk--1974-livejournal-com.turbopages.org': 'mihalchuk-1974.livejournal.com',
 'moskov--tsarstvo-livejournal-com.turbopages.org': 'moskov-tsarstvo.livejournal.com',
 'bar--chk-livejournal-com.turbopages.org': 'bar-chk.livejournal.com',
 'arms--theory-livejournal-com.turbopages.org': 'arms-theory.livejournal.com',
 'arpadhaizy-livejournal-com.turbopages.org': 'arpadhaizy.livejournal.com',
 'bowandtie-ru.turbopages.org': 'bowandtie.ru',
 'droidway-net.turbopages.org': 'droidway.net',
 'kopomko-ru.turbopages.org': 'kopomko.ru',
 'avtodispetcher-ru.turbopages.org': 'avtodispetcher.ru',
 'glavny-tv.turbopages.org': 'glavny.tv',
 'posobie-guru.turbopages.org': 'posobie.guru',
 'on--tattoo-ru.turbopages.org': 'on-tattoo.ru',
 'afrodita--spa-ru.turbopages.org': 'afrodita-spa.ru',
 'finzav-ru.turbopages.org': 'finzav.ru',
 'vrogov-ru.turbopages.org': 'vrogov.ru',
 'evrookna--mos-ru.turbopages.org': 'evrookna-mos.ru',
 'rep-ru.turbopages.org': 'rep.ru',
 'tlum-ru.turbopages.org': 'tlum.ru',
 'kvu-su.turbopages.org': 'kvu.su',
 'bor--odin-livejournal-com.turbopages.org': 'bor-odin.livejournal.com',
 'gorodn-ru.turbopages.org': 'gorodn.ru',
 'rostov-tsargrad-tv.turbopages.org': 'rostov.tsargrad.tv',
 'samors-ru.turbopages.org': 'samors.ru',
 'jaay-ru.turbopages.org': 'jaay.ru',
 'mojmir--online-ru.turbopages.org': 'mojmir-online.ru',
 'vsenapolzu-ru.turbopages.org': 'vsenapolzu.ru',
 'bycars-ru.turbopages.org': 'bycars.ru',
 'onchuka-livejournal-com.turbopages.org': 'onchuka.livejournal.com',
 'jutkoe-ru.turbopages.org': 'jutkoe.ru',
 'balali-livejournal-com.turbopages.org': 'balali.livejournal.com',
 'about--tea-ru.turbopages.org': 'about-tea.ru',
 'tomsk-kp-ru.turbopages.org': 'tomsk.kp.ru',
 'tomsk-mk-ru.turbopages.org': 'tomsk.mk.ru',
 'tomsk-aif-ru.turbopages.org': 'tomsk.aif.ru',
 'ugra--news-ru.turbopages.org': 'ugra-news.ru',
 'news-vtomske-ru.turbopages.org': 'news.vtomske.ru',
 'foodandscience-org.turbopages.org': 'foodandscience.org',
 'prostitutka--ket-livejournal-com.turbopages.org': 'prostitutka-ket.livejournal.com',
 'samaraonline24-ru.turbopages.org': 'samaraonline24.ru',
 'mybagazh-ru.turbopages.org': 'mybagazh.ru',
 'micoff-livejournal-com.turbopages.org': 'micoff.livejournal.com',
 'crimea--news-com.turbopages.org': 'crimea-news.com',
 'mygeografi-ru.turbopages.org': 'mygeografi.ru',
 'xn------7cdogdq8cje1ab8cvf.turbopages.org': 'xn---7cdogdq8cje1ab8cvf',
 'cssrzd-ru.turbopages.org': 'cssrzd.ru',
 'psychospace-ru.turbopages.org': 'psychospace.ru',
 'beguza-ru.turbopages.org': 'beguza.ru',
 'carexpert-ru.turbopages.org': 'carexpert.ru',
 'vlad--burtsev-livejournal-com.turbopages.org': 'vlad-burtsev.livejournal.com',
 '2x2-su.turbopages.org': '2x2.su',
 'amur-kp-ru.turbopages.org': 'amur.kp.ru',
 'gtrkamur-ru.turbopages.org': 'gtrkamur.ru',
 'bulochnikov-livejournal-com.turbopages.org': 'bulochnikov.livejournal.com',
 'faken--magda-livejournal-com.turbopages.org': 'faken-magda.livejournal.com',
 'kino-anews-com.turbopages.org': 'kino.anews.com',
 'coffexpert-ru.turbopages.org': 'coffexpert.ru',
 'ngs42-ru.turbopages.org': 'ngs42.ru',
 'a--malyavin-livejournal-com.turbopages.org': 'a-malyavin.livejournal.com',
 'naked--science-ru.turbopages.org': 'naked-science.ru',
 'kopilpremudrosti-ru.turbopages.org': 'kopilpremudrosti.ru',
 'pravo-team.turbopages.org': 'pravo.team',
 'headlife-ru.turbopages.org': 'headlife.ru',
 'medrussia-org.turbopages.org': 'medrussia.org',
 'pinov-net.turbopages.org': 'pinov.net',
 'combat--dez-ru.turbopages.org': 'combat-dez.ru',
 'babyzzz-ru.turbopages.org': 'babyzzz.ru',
 'iphone--gps-ru.turbopages.org': 'iphone-gps.ru',
 'disgustingmen-com.turbopages.org': 'disgustingmen.com',
 'boeing--is--back-livejournal-com.turbopages.org': 'boeing-is-back.livejournal.com',
 'bezbukashek-ru.turbopages.org': 'bezbukashek.ru',
 'anatoly--gendin-livejournal-com.turbopages.org': 'anatoly-gendin.livejournal.com',
 'roouh-livejournal-com.turbopages.org': 'roouh.livejournal.com',
 'dergachev--va-livejournal-com.turbopages.org': 'dergachev-va.livejournal.com',
 'forpost--sz-ru.turbopages.org': 'forpost-sz.ru',
 'yatrans-ru.turbopages.org': 'yatrans.ru',
 'naschaeda-ru.turbopages.org': 'naschaeda.ru',
 'terrao-livejournal-com.turbopages.org': 'terrao.livejournal.com',
 'a--les--sandro-livejournal-com.turbopages.org': 'a-les-sandro.livejournal.com',
 'russia--today-net.turbopages.org': 'russia-today.net',
 'ooo--interier-ru.turbopages.org': 'ooo-interier.ru',
 'marena99-livejournal-com.turbopages.org': 'marena99.livejournal.com',
 'mastack-ru.turbopages.org': 'mastack.ru',
 'infonotes-ru.turbopages.org': 'infonotes.ru',
 'andreistp-livejournal-com.turbopages.org': 'andreistp.livejournal.com',
 'trubaspec-com.turbopages.org': 'trubaspec.com',
 'planirovkamaster-ru.turbopages.org': 'planirovkamaster.ru',
 'moshekam-livejournal-com.turbopages.org': 'moshekam.livejournal.com',
 'doma--vkusnee-ru.turbopages.org': 'doma-vkusnee.ru',
 'catmolly-com.turbopages.org': 'catmolly.com',
 'minikuhonka-ru.turbopages.org': 'minikuhonka.ru',
 'tvoe--avto-com.turbopages.org': 'tvoe-avto.com',
 'octagon-media.turbopages.org': 'octagon.media',
 'inforuss-info.turbopages.org': 'inforuss.info',
 'bitbat-ru.turbopages.org': 'bitbat.ru',
 'medmaniac-ru.turbopages.org': 'medmaniac.ru',
 'foodgid-net.turbopages.org': 'foodgid.net',
 'postila-ru.turbopages.org': 'postila.ru',
 'internet--technologies-ru.turbopages.org': 'internet-technologies.ru',
 'health--post-ru.turbopages.org': 'health-post.ru',
 'yasno-anews-com.turbopages.org': 'yasno.anews.com',
 'madhunter-ru.turbopages.org': 'madhunter.ru',
 'doktorsemenova-ru.turbopages.org': 'doktorsemenova.ru',
 'philology--pro-livejournal-com.turbopages.org': 'philology-pro.livejournal.com',
 'veterinargid-ru.turbopages.org': 'veterinargid.ru',
 'gormonoff-com.turbopages.org': 'gormonoff.com',
 '21074-ru.turbopages.org': '21074.ru',
 'gameguru-ru.turbopages.org': 'gameguru.ru',
 'steakandgrill-ru.turbopages.org': 'steakandgrill.ru',
 'plitochnikexpert-ru.turbopages.org': 'plitochnikexpert.ru',
 'coop--land-ru.turbopages.org': 'coop-land.ru',
 'minutkoclinic-com.turbopages.org': 'minutkoclinic.com',
 'mel-fm.turbopages.org': 'mel.fm',
 'avianity-ru.turbopages.org': 'avianity.ru',
 'dogzy-ru.turbopages.org': 'dogzy.ru',
 'papyrus--net-livejournal-com.turbopages.org': 'papyrus-net.livejournal.com',
 'sbotvet-com.turbopages.org': 'sbotvet.com',
 'gendilaniya-ru.turbopages.org': 'gendilaniya.ru',
 'penzavzglyad-ru.turbopages.org': 'penzavzglyad.ru',
 'earthwatching-livejournal-com.turbopages.org': 'earthwatching.livejournal.com',
 'prokomputer-ru.turbopages.org': 'prokomputer.ru',
 'andreyex-ru.turbopages.org': 'andreyex.ru',
 'gorno--altaisk-info.turbopages.org': 'gorno-altaisk.info',
 'psylib-org.turbopages.org': 'psylib.org',
 'ast--news-ru.turbopages.org': 'ast-news.ru',
 'dengi--dolg--spb-ru.turbopages.org': 'dengi-dolg-spb.ru',
 '24kredits--spb-ru.turbopages.org': '24kredits-spb.ru',
 'abw-by.turbopages.org': 'abw.by',
 'aloyze-livejournal-com.turbopages.org': 'aloyze.livejournal.com',
 'alexkolm-livejournal-com.turbopages.org': 'alexkolm.livejournal.com',
 'ej--ka-net.turbopages.org': 'ej-ka.net',
 'wotexpress-info.turbopages.org': 'wotexpress.info',
 'xn------cddrilihgo8aokf2d.turbopages.org': 'xn---cddrilihgo8aokf2d',
 'vkussovet-net.turbopages.org': 'vkussovet.net',
 'nakopi--deneg-ru.turbopages.org': 'nakopi-deneg.ru',
 'zdorrov-com.turbopages.org': 'zdorrov.com',
 'ookicter-livejournal-com.turbopages.org': 'ookicter.livejournal.com',
 'music--education-ru.turbopages.org': 'music-education.ru',
 'consilium38-ru.turbopages.org': 'consilium38.ru',
 'alfalady-org.turbopages.org': 'alfalady.org',
 'ezogoros-ru.turbopages.org': 'ezogoros.ru',
 'avtostandart--m24-ru.turbopages.org': 'avtostandart-m24.ru',
 'reytingelektroniki-ru.turbopages.org': 'reytingelektroniki.ru',
 'oleg--borisovich-livejournal-com.turbopages.org': 'oleg-borisovich.livejournal.com',
 'kaluga-aif-ru.turbopages.org': 'kaluga.aif.ru',
 'my19edwin-livejournal-com.turbopages.org': 'my19edwin.livejournal.com',
 'psyfactor-org.turbopages.org': 'psyfactor.org',
 'foto--history-livejournal-com.turbopages.org': 'foto-history.livejournal.com',
 'monitor-net-ru.turbopages.org': 'monitor.net.ru',
 'autostat-ru.turbopages.org': 'autostat.ru',
 'technodlog-ru.turbopages.org': 'technodlog.ru',
 'ludacha-ru.turbopages.org': 'ludacha.ru',
 'photoretsept-ru.turbopages.org': 'photoretsept.ru',
 'zamok62-ru.turbopages.org': 'zamok62.ru',
 'lechimkrohu-ru.turbopages.org': 'lechimkrohu.ru',
 'kidbooms-ru.turbopages.org': 'kidbooms.ru',
 'new--science-ru.turbopages.org': 'new-science.ru',
 'mos--jkh-livejournal-com.turbopages.org': 'mos-jkh.livejournal.com',
 'electricdoma-ru.turbopages.org': 'electricdoma.ru',
 'nikatv-ru.turbopages.org': 'nikatv.ru',
 'krasnogorskriamo-ru.turbopages.org': 'krasnogorskriamo.ru',
 'godbo-livejournal-com.turbopages.org': 'godbo.livejournal.com',
 'saginatus-livejournal-com.turbopages.org': 'saginatus.livejournal.com',
 'd--galaydov-livejournal-com.turbopages.org': 'd-galaydov.livejournal.com',
 'kislovodsk-bezformata-com.turbopages.org': 'kislovodsk.bezformata.com',
 'edo--tokyo-livejournal-com.turbopages.org': 'edo-tokyo.livejournal.com',
 'specpryanosti-ru.turbopages.org': 'specpryanosti.ru',
 'ed--glezin-livejournal-com.turbopages.org': 'ed-glezin.livejournal.com',
 'licenseplates-ru.turbopages.org': 'licenseplates.ru',
 'zvezdaweekly-ru.turbopages.org': 'zvezdaweekly.ru',
 'brutals-ru.turbopages.org': 'brutals.ru',
 'obozvrn-ru.turbopages.org': 'obozvrn.ru',
 'skolko--stoit-ru.turbopages.org': 'skolko-stoit.ru',
 'zipzip03-ru.turbopages.org': 'zipzip03.ru',
 'rzn-mk-ru.turbopages.org': 'rzn.mk.ru',
 'vse--dengy-ru.turbopages.org': 'vse-dengy.ru',
 'itshneg-com.turbopages.org': 'itshneg.com',
 'probanki--info-ru.turbopages.org': 'probanki-info.ru',
 'otoplenie--doma-org.turbopages.org': 'otoplenie-doma.org',
 'fast--wolker-ru.turbopages.org': 'fast-wolker.ru',
 'avtonauka-ru.turbopages.org': 'avtonauka.ru',
 'beautyart-pro.turbopages.org': 'beautyart.pro',
 'sitekid-ru.turbopages.org': 'sitekid.ru',
 'podomu-info.turbopages.org': 'podomu.info',
 'desmyhome-ru.turbopages.org': 'desmyhome.ru',
 'ksk--prestige-ru.turbopages.org': 'ksk-prestige.ru',
 'metaldoors--spb-ru.turbopages.org': 'metaldoors-spb.ru',
 'gadgetpage-ru.turbopages.org': 'gadgetpage.ru',
 'bezopasnik-info.turbopages.org': 'bezopasnik.info',
 '33tura-ru.turbopages.org': '33tura.ru',
 'autogear-ru.turbopages.org': 'autogear.ru',
 'moscowsad-ru.turbopages.org': 'moscowsad.ru',
 'nicedeck-ru.turbopages.org': 'nicedeck.ru',
 'analitic-livejournal-com.turbopages.org': 'analitic.livejournal.com',
 'tvoytarif-ru.turbopages.org': 'tvoytarif.ru',
 'itpotok-ru.turbopages.org': 'itpotok.ru',
 'bankstoday-net.turbopages.org': 'bankstoday.net',
 'app--time-ru.turbopages.org': 'app-time.ru',
 'raz--lyudi-ru.turbopages.org': 'raz-lyudi.ru',
 'dom--isemya-ru.turbopages.org': 'dom-isemya.ru',
 'antennadaily-ru.turbopages.org': 'antennadaily.ru',
 'alice--journal-ru.turbopages.org': 'alice-journal.ru',
 'euroradio-fm.turbopages.org': 'euroradio.fm',
 'wangeliy-livejournal-com.turbopages.org': 'wangeliy.livejournal.com',
 'novye--statusy-ru.turbopages.org': 'novye-statusy.ru',
 'lifesovet-ru.turbopages.org': 'lifesovet.ru',
 'yrin8-livejournal-com.turbopages.org': 'yrin8.livejournal.com',
 'krasnodarmedia-su.turbopages.org': 'krasnodarmedia.su',
 'ru--dark--triad-livejournal-com.turbopages.org': 'ru-dark-triad.livejournal.com',
 'geosts-livejournal-com.turbopages.org': 'geosts.livejournal.com',
 'view--photo-ru.turbopages.org': 'view-photo.ru',
 '62info-ru.turbopages.org': '62info.ru',
 'voronej-bezformata-com.turbopages.org': 'voronej.bezformata.com',
 'wday-ru.turbopages.org': 'wday.ru',
 'vlg-aif-ru.turbopages.org': 'vlg.aif.ru',
 'alexandravent-ru.turbopages.org': 'alexandravent.ru',
 'serg07011972-livejournal-com.turbopages.org': 'serg07011972.livejournal.com',
 'flylowcost-ru.turbopages.org': 'flylowcost.ru',
 'trip--advice-ru.turbopages.org': 'trip-advice.ru',
 'kaliningrad--guide-com.turbopages.org': 'kaliningrad-guide.com',
 'politnavigator-net.turbopages.org': 'politnavigator.net',
 'pro--voinu-ru.turbopages.org': 'pro-voinu.ru',
 'glazexpert-ru.turbopages.org': 'glazexpert.ru',
 'zolotoe--runo--sl-ru.turbopages.org': 'zolotoe-runo-sl.ru',
 'bash-today.turbopages.org': 'bash.today',
 'rukozhopim-ru.turbopages.org': 'rukozhopim.ru',
 'chtoikak-ru.turbopages.org': 'chtoikak.ru',
 'survinat-ru.turbopages.org': 'survinat.ru',
 'porodisobak-ru.turbopages.org': 'porodisobak.ru',
 'xn------7cdaiv2aa1cmydcdt4b.turbopages.org': 'xn---7cdaiv2aa1cmydcdt4b',
 'prozemlu-ru.turbopages.org': 'prozemlu.ru',
 'ygashae--zvezdu-livejournal-com.turbopages.org': 'ygashae-zvezdu.livejournal.com',
 'polit-ru.turbopages.org': 'polit.ru',
 'utro-ru.turbopages.org': 'utro.ru',
 'rostov-aif-ru.turbopages.org': 'rostov.aif.ru',
 'progorod58-ru.turbopages.org': 'progorod58.ru',
 'samstroy-com.turbopages.org': 'samstroy.com',
 'bubblemom-ru.turbopages.org': 'bubblemom.ru',
 'mob--os-ru.turbopages.org': 'mob-os.ru',
 'ruselectronic-com.turbopages.org': 'ruselectronic.com',
 'burforum-ru.turbopages.org': 'burforum.ru',
 'ectrl-ru.turbopages.org': 'ectrl.ru',
 'moykaapelsin-ru.turbopages.org': 'moykaapelsin.ru',
 'lingvana-ru.turbopages.org': 'lingvana.ru',
 'polzablog-ru.turbopages.org': 'polzablog.ru',
 'food--wiki-ru.turbopages.org': 'food-wiki.ru',
 'vremya--sovetov-ru.turbopages.org': 'vremya-sovetov.ru',
 'etopolezno-com.turbopages.org': 'etopolezno.com',
 'izhevsk-mk-ru.turbopages.org': 'izhevsk.mk.ru',
 'sergei--berez-livejournal-com.turbopages.org': 'sergei-berez.livejournal.com',
 'ozon--promokody-ru.turbopages.org': 'ozon-promokody.ru',
 'vinohobby-ru.turbopages.org': 'vinohobby.ru',
 'sdelaysam--svoimirukami-ru.turbopages.org': 'sdelaysam-svoimirukami.ru',
 'omsk-aif-ru.turbopages.org': 'omsk.aif.ru',
 'pyrogi-ru.turbopages.org': 'pyrogi.ru',
 'domovodstvo--kulinariya-ru.turbopages.org': 'domovodstvo-kulinariya.ru',
 'kedem-ru.turbopages.org': 'kedem.ru',
 'michail--shor-livejournal-com.turbopages.org': 'michail-shor.livejournal.com',
 'sergeyurich-livejournal-com.turbopages.org': 'sergeyurich.livejournal.com',
 'rb--ra-org.turbopages.org': 'rb-ra.org',
 'original--present-ru.turbopages.org': 'original-present.ru',
 'mob--mobile-ru.turbopages.org': 'mob-mobile.ru',
 'volosoved-ru.turbopages.org': 'volosoved.ru',
 'kicksharingi-ru.turbopages.org': 'kicksharingi.ru',
 'kirovpravda-ru.turbopages.org': 'kirovpravda.ru',
 'muizre-ru.turbopages.org': 'muizre.ru',
 'autoportret-livejournal-com.turbopages.org': 'autoportret.livejournal.com',
 'myfitnesblog-com.turbopages.org': 'myfitnesblog.com',
 'lastday-club.turbopages.org': 'lastday.club',
 'mitishi-bezformata-com.turbopages.org': 'mitishi.bezformata.com',
 'twnews-ru.turbopages.org': 'twnews.ru',
 'color--harmony-livejournal-com.turbopages.org': 'color-harmony.livejournal.com',
 'disertiki-ru.turbopages.org': 'disertiki.ru',
 'accbook-ru.turbopages.org': 'accbook.ru',
 'kdc-clinic.turbopages.org': 'kdc.clinic',
 'musaget-ru.turbopages.org': 'musaget.ru',
 'bobsoccer-ru.turbopages.org': 'bobsoccer.ru',
 'japanchin-ru.turbopages.org': 'japanchin.ru',
 'spmag-ru.turbopages.org': 'spmag.ru',
 'xn----ctbink3aefbe1b.turbopages.org': 'xn--ctbink3aefbe1b',
 'kinocomedy-livejournal-com.turbopages.org': 'kinocomedy.livejournal.com',
 'kartam47-livejournal-com.turbopages.org': 'kartam47.livejournal.com',
 'crimea-mk-ru.turbopages.org': 'crimea.mk.ru',
 'souzconsalt-com.turbopages.org': 'souzconsalt.com',
 'russialand-livejournal-com.turbopages.org': 'russialand.livejournal.com',
 'skurkys-livejournal-com.turbopages.org': 'skurkys.livejournal.com',
 'mosobl-kp-ru.turbopages.org': 'mosobl.kp.ru',
 'teleport2001-ru.turbopages.org': 'teleport2001.ru',
 'top--knig-ru.turbopages.org': 'top-knig.ru',
 'chita-aif-ru.turbopages.org': 'chita.aif.ru',
 'ekaterinburg-octagon-media.turbopages.org': 'ekaterinburg.octagon.media',
 'klymenko--time-com.turbopages.org': 'klymenko-time.com',
 'agesecrets-ru.turbopages.org': 'agesecrets.ru',
 '30--plus-ru.turbopages.org': '30-plus.ru',
 'irina--kozina-ru.turbopages.org': 'irina-kozina.ru',
 'dohcolonoc-ru.turbopages.org': 'dohcolonoc.ru',
 'balkonoved-ru.turbopages.org': 'balkonoved.ru',
 'chayku-net.turbopages.org': 'chayku.net',
 'mkkaluga-ru.turbopages.org': 'mkkaluga.ru',
 'makchen-ru.turbopages.org': 'makchen.ru',
 'tvoi--uvelirr-ru.turbopages.org': 'tvoi-uvelirr.ru',
 'the--fishing-ru.turbopages.org': 'the-fishing.ru',
 'hozyayushka--online-ru.turbopages.org': 'hozyayushka-online.ru',
 'tytkleva-net.turbopages.org': 'tytkleva.net',
 'willcomfort-ru.turbopages.org': 'willcomfort.ru',
 'ruputeshestvie-ru.turbopages.org': 'ruputeshestvie.ru',
 'b--g-by.turbopages.org': 'b-g.by',
 'date--release-info.turbopages.org': 'date-release.info',
 'martin33-livejournal-com.turbopages.org': 'martin33.livejournal.com',
 'vecherka--spb-ru.turbopages.org': 'vecherka-spb.ru',
 'kerch-fm.turbopages.org': 'kerch.fm',
 'txt--music-ru.turbopages.org': 'txt-music.ru',
 'pilotprof-ru.turbopages.org': 'pilotprof.ru',
 'strannick--ru-livejournal-com.turbopages.org': 'strannick-ru.livejournal.com',
 'burckina--new-livejournal-com.turbopages.org': 'burckina-new.livejournal.com',
 'avto--layn-ru.turbopages.org': 'avto-layn.ru',
 'promokodreg-ru.turbopages.org': 'promokodreg.ru',
 'dralexmd-livejournal-com.turbopages.org': 'dralexmd.livejournal.com',
 'podg--kursy-ru.turbopages.org': 'podg-kursy.ru',
 'marochkin-livejournal-com.turbopages.org': 'marochkin.livejournal.com',
 'nfcwiki-ru.turbopages.org': 'nfcwiki.ru',
 'meladan-livejournal-com.turbopages.org': 'meladan.livejournal.com',
 'vannaitualet-ru.turbopages.org': 'vannaitualet.ru',
 'lavitto-ru.turbopages.org': 'lavitto.ru',
 'nabor--massa-ru.turbopages.org': 'nabor-massa.ru',
 'tehnomir32-ru.turbopages.org': 'tehnomir32.ru',
 'glawtruba-ru.turbopages.org': 'glawtruba.ru',
 'waterok-ru.turbopages.org': 'waterok.ru',
 'activityedu-ru.turbopages.org': 'activityedu.ru',
 'summerhotels-ru.turbopages.org': 'summerhotels.ru',
 'daklinik-ru.turbopages.org': 'daklinik.ru',
 'samosoverhenstvovanie-ru.turbopages.org': 'samosoverhenstvovanie.ru',
 'territoriya--zhenschiny-ru.turbopages.org': 'territoriya-zhenschiny.ru',
 'gogosmart-ru.turbopages.org': 'gogosmart.ru',
 'spid--rub-ru.turbopages.org': 'spid-rub.ru',
 'horoshiy--otzyv-ru.turbopages.org': 'horoshiy-otzyv.ru',
 'kulinarka--tv-ru.turbopages.org': 'kulinarka-tv.ru',
 'pinta--pub-ru.turbopages.org': 'pinta-pub.ru',
 'pronedra-ru.turbopages.org': 'pronedra.ru',
 'poezd-ru.turbopages.org': 'poezd.ru',
 'o--pereezde-ru.turbopages.org': 'o-pereezde.ru',
 'ksonline-ru.turbopages.org': 'ksonline.ru',
 'magic--pendel-ru.turbopages.org': 'magic-pendel.ru',
 'delovoy--kvartal-ru.turbopages.org': 'delovoy-kvartal.ru',
 'sovetysadovodam-ru.turbopages.org': 'sovetysadovodam.ru',
 'otsvetax-ru.turbopages.org': 'otsvetax.ru',
 'irk-today.turbopages.org': 'irk.today',
 'afanarizm-livejournal-com.turbopages.org': 'afanarizm.livejournal.com',
 'teplogalaxy-ru.turbopages.org': 'teplogalaxy.ru',
 'blog-brigada174-ru.turbopages.org': 'blog.brigada174.ru',
 'autopeople-ru.turbopages.org': 'autopeople.ru',
 'technologicus-ru.turbopages.org': 'technologicus.ru',
 'pokatim-ru.turbopages.org': 'pokatim.ru',
 'nibbl-ru.turbopages.org': 'nibbl.ru',
 'tveremiova-livejournal-com.turbopages.org': 'tveremiova.livejournal.com',
 'kitjournal-ru.turbopages.org': 'kitjournal.ru',
 'vnru-ru.turbopages.org': 'vnru.ru',
 'creator--arseny-site.turbopages.org': 'creator-arseny.site',
 'kirov-kp-ru.turbopages.org': 'kirov.kp.ru',
 'lovereport-ru.turbopages.org': 'lovereport.ru',
 'murmansk-kp-ru.turbopages.org': 'murmansk.kp.ru',
 'niasam-ru.turbopages.org': 'niasam.ru',
 'for--travels-ru.turbopages.org': 'for-travels.ru',
 'neauto-ru.turbopages.org': 'neauto.ru',
 'uspeh--woman-com.turbopages.org': 'uspeh-woman.com',
 'samelectric-ru.turbopages.org': 'samelectric.ru',
 'tverigrad-ru.turbopages.org': 'tverigrad.ru',
 'risk-ru.turbopages.org': 'risk.ru',
 'profinance-ru.turbopages.org': 'profinance.ru',
 'max--volna-livejournal-com.turbopages.org': 'max-volna.livejournal.com',
 'wroom-ru.turbopages.org': 'wroom.ru',
 'vrachmedik-ru.turbopages.org': 'vrachmedik.ru',
 'botlist-ru.turbopages.org': 'botlist.ru',
 'm--skazitelnitsa-livejournal-com.turbopages.org': 'm-skazitelnitsa.livejournal.com',
 'mymeizu--online-ru.turbopages.org': 'mymeizu-online.ru',
 'moscow--i--ya-livejournal-com.turbopages.org': 'moscow-i-ya.livejournal.com',
 'stalker--world-ru.turbopages.org': 'stalker-world.ru',
 'natoke-ru.turbopages.org': 'natoke.ru',
 'kudagid-ru.turbopages.org': 'kudagid.ru',
 'unworld-ru.turbopages.org': 'unworld.ru',
 'eponim2008-livejournal-com.turbopages.org': 'eponim2008.livejournal.com',
 'sergeytsvetkov-livejournal-com.turbopages.org': 'sergeytsvetkov.livejournal.com',
 'xn------5cdcgguzbgaeldyo9ab9bftk1d.turbopages.org': 'xn---5cdcgguzbgaeldyo9ab9bftk1d',
 'hueviebin1-livejournal-com.turbopages.org': 'hueviebin1.livejournal.com',
 'ex--press-by.turbopages.org': 'ex-press.by',
 'leonard17-livejournal-com.turbopages.org': 'leonard17.livejournal.com',
    'rg-ru.turbopages.org': 'rg.ru',
 'stroim-guru.turbopages.org': 'stroim.guru',
 'ria-ru.turbopages.org': 'ria.ru',
 'overclockers-ru.turbopages.org': 'overclockers.ru',
 'ukraina-ru.turbopages.org': 'ukraina.ru',
 'rbc-ru.turbopages.org': 'rbc.ru',
 'quote-rbc-ru.turbopages.org': 'quote.rbc.ru',
 'osnmedia-ru.turbopages.org': 'osnmedia.ru',
 'vseprokley-com.turbopages.org': 'vseprokley.com',
 'schci-ru.turbopages.org': 'schci.ru',
 'russian-rt-com.turbopages.org': 'russian.rt.com',
 'kommersant-ru.turbopages.org': 'kommersant.ru',
 'mk-ru.turbopages.org': 'mk.ru',
 'life-ru.turbopages.org': 'life.ru',
 '2bitcoins-ru.turbopages.org': '2bitcoins.ru',
 'regnum-ru.turbopages.org': 'regnum.ru',
 'kuhniclub-ru.turbopages.org': 'kuhniclub.ru',
 'apteka-103-by.turbopages.org': 'apteka.103.by',
 'kuban24-tv.turbopages.org': 'kuban24.tv',
 'nashgorod-ru.turbopages.org': 'nashgorod.ru',
 '3dnews-ru.turbopages.org': '3dnews.ru',
 'tumen-kp-ru.turbopages.org': 'tumen.kp.ru',
 'tass-ru.turbopages.org': 'tass.ru',
 'riafan-ru.turbopages.org': 'riafan.ru',
 'lenta-ru.turbopages.org': 'lenta.ru',
 'tourism-interfax-ru.turbopages.org': 'tourism.interfax.ru',
 'popcornnews-ru.turbopages.org': 'popcornnews.ru',
 'ura-news.turbopages.org': 'ura.news',
 'tmn-aif-ru.turbopages.org': 'tmn.aif.ru',
 '72-ru.turbopages.org': '72.ru',
 'ren-tv.turbopages.org': 'ren.tv',
 'aif-ru.turbopages.org': 'aif.ru',
 'mandarinka-info.turbopages.org': 'mandarinka.info',
 '9111-ru.turbopages.org': '9111.ru',
 'kitchendecorium-ru.turbopages.org': 'kitchendecorium.ru',
 'rtvi-com.turbopages.org': 'rtvi.com',
 'infoniac-ru.turbopages.org': 'infoniac.ru',
 'vsebasni-ru.turbopages.org': 'vsebasni.ru',
 'chudopredki-ru.turbopages.org': 'chudopredki.ru',
 'forbes-ru.turbopages.org': 'forbes.ru',
 'drive2-ru.turbopages.org': 'drive2.ru',
 'minsknews-by.turbopages.org': 'minsknews.by',
 'banki-loans.turbopages.org': 'banki.loans',
 'tvzvezda-ru.turbopages.org': 'tvzvezda.ru',
 'vsevkulinary-ru.turbopages.org': 'vsevkulinary.ru',
 'inosmi-ru.turbopages.org': 'inosmi.ru',
 'anekdot-ru.turbopages.org': 'anekdot.ru',
 'povar-ru.turbopages.org': 'povar.ru',
 'n4k-ru.turbopages.org': 'n4k.ru',
 'gercules-fit.turbopages.org': 'gercules.fit',
 'gazeta-ru.turbopages.org': 'gazeta.ru',
 'philipshero-livejournal-com.turbopages.org': 'philipshero.livejournal.com',
 'fedpress-ru.turbopages.org': 'fedpress.ru',
 'recepty--kulinara-ru.turbopages.org': 'recepty-kulinara.ru',
 'anekdot--anekdoty-ru.turbopages.org': 'anekdot-anekdoty.ru',
 'e1-ru.turbopages.org': 'e1.ru',
 'bytiemoe-ru.turbopages.org': 'bytiemoe.ru',
 'astromeridian-ru.turbopages.org': 'astromeridian.ru',
 'chel-kp-ru.turbopages.org': 'chel.kp.ru',
 'business--class-su.turbopages.org': 'business-class.su',
 'progorod59-ru.turbopages.org': 'progorod59.ru',
 'u--f-ru.turbopages.org': 'u-f.ru',
 'news-rambler-ru.turbopages.org': 'news.rambler.ru',
 'kapital--rus-ru.turbopages.org': 'kapital-rus.ru',
 'vkusnogotovlu-ru.turbopages.org': 'vkusnogotovlu.ru',
 '1tv-ru.turbopages.org': '1tv.ru',
 '7ya-ru.turbopages.org': '7ya.ru',
 'perm-kp-ru.turbopages.org': 'perm.kp.ru',
 'discover24-ru.turbopages.org': 'discover24.ru',
 'magija--vkusa-ru.turbopages.org': 'magija-vkusa.ru',
 'ogorod-ru.turbopages.org': 'ogorod.ru',
 'voennoedelo-com.turbopages.org': 'voennoedelo.com',
 'kulinarenok-ru.turbopages.org': 'kulinarenok.ru',
 'interfax-ru.turbopages.org': 'interfax.ru',
 'svpressa-ru.turbopages.org': 'svpressa.ru',
 'vm-ru.turbopages.org': 'vm.ru',
 'precedent-tv.turbopages.org': 'precedent.tv',
 'sport24-ru.turbopages.org': 'sport24.ru',
 'kp-ru.turbopages.org': 'kp.ru',
 'kulturologia-ru.turbopages.org': 'kulturologia.ru',
 'biographe-ru.turbopages.org': 'biographe.ru',
 'lifehacker-ru.turbopages.org': 'lifehacker.ru',
 'anews-com.turbopages.org': 'anews.com',
 'warbook-club.turbopages.org': 'warbook.club',
 'sport--express-ru.turbopages.org': 'sport-express.ru',
 '24smi-org.turbopages.org': '24smi.org',
 'championat-com.turbopages.org': 'championat.com',
 'dp-ru.turbopages.org': 'dp.ru',
 'stuki--druki-com.turbopages.org': 'stuki-druki.com',
 'ntv-ru.turbopages.org': 'ntv.ru',
 'arbalett-livejournal-com.turbopages.org': 'arbalett.livejournal.com',
 '29-ru.turbopages.org': '29.ru',
 'dela-ru.turbopages.org': 'dela.ru',
 'vz-ru.turbopages.org': 'vz.ru',
 'kras-mk-ru.turbopages.org': 'kras.mk.ru',
 'ngs24-ru.turbopages.org': 'ngs24.ru',
 'poklevka-com.turbopages.org': 'poklevka.com',
 'lena7-ru.turbopages.org': 'lena7.ru',
 '360tv-ru.turbopages.org': '360tv.ru',
 'litbro-ru.turbopages.org': 'litbro.ru',
 '5--tv-ru.turbopages.org': '5-tv.ru',
 'baby-ru.turbopages.org': 'baby.ru',
 'turbopages.org': '',
 'plastica-top.turbopages.org': 'plastica.top',
 'fb-ru.turbopages.org': 'fb.ru',
 'mysekret-ru.turbopages.org': 'mysekret.ru',
 'medaboutme-ru.turbopages.org': 'medaboutme.ru',
 'povarenok-ru.turbopages.org': 'povarenok.ru',
 'clinica--opora-ru.turbopages.org': 'clinica-opora.ru',
 'kylinariya-ru.turbopages.org': 'kylinariya.ru',
 'ogorodguru-com.turbopages.org': 'ogorodguru.com',
 'vilkin-pro.turbopages.org': 'vilkin.pro',
 'nplus1-ru.turbopages.org': 'nplus1.ru',
 'fishki-net.turbopages.org': 'fishki.net',
 'inmoment-ru.turbopages.org': 'inmoment.ru',
 'mir24-tv.turbopages.org': 'mir24.tv',
 'lisa-ru.turbopages.org': 'lisa.ru',
 'matchtv-ru.turbopages.org': 'matchtv.ru',
 'parniplus-com.turbopages.org': 'parniplus.com',
 'auto-onliner-by.turbopages.org': 'auto.onliner.by',
 'gastronom-ru.turbopages.org': 'gastronom.ru',
 'fermerss-ru.turbopages.org': 'fermerss.ru',
 'stroychik-ru.turbopages.org': 'stroychik.ru',
 'drive2-com.turbopages.org': 'drive2.com',
 'gadgetok-ru.turbopages.org': 'gadgetok.ru',
 'eda-ru.turbopages.org': 'eda.ru',
 'zr-ru.turbopages.org': 'zr.ru',
 'krutilvertel-com.turbopages.org': 'krutilvertel.com',
 'investgazeta-ru.turbopages.org': 'investgazeta.ru',
 'lublu--vino-ru.turbopages.org': 'lublu-vino.ru',
 'vesti-ru.turbopages.org': 'vesti.ru',
 'zverovod-info.turbopages.org': 'zverovod.info',
 '32cars-ru.turbopages.org': '32cars.ru',
 'volgograd-kp-ru.turbopages.org': 'volgograd.kp.ru',
 'motor-ru.turbopages.org': 'motor.ru',
 'dm--st-ru.turbopages.org': 'dm-st.ru',
 'otzovikavto-ru.turbopages.org': 'otzovikavto.ru',
 'vitalidrobishev-livejournal-com.turbopages.org': 'vitalidrobishev.livejournal.com',
 'lelik--ch-livejournal-com.turbopages.org': 'lelik-ch.livejournal.com',
 'akademiyavkusa-ru.turbopages.org': 'akademiyavkusa.ru',
 'krymania-ru.turbopages.org': 'krymania.ru',
 'hi--news-ru.turbopages.org': 'hi-news.ru',
 'news-ru.turbopages.org': 'news.ru',
 'rb-ru.turbopages.org': 'rb.ru',
 'versia-ru.turbopages.org': 'versia.ru',
 'vologda--poisk-ru.turbopages.org': 'vologda-poisk.ru',
 '1prime-ru.turbopages.org': '1prime.ru',
 'mag-auto-ru.turbopages.org': 'mag.auto.ru',
 'finance-rambler-ru.turbopages.org': 'finance.rambler.ru',
 'tsargrad-tv.turbopages.org': 'tsargrad.tv',
 'hi--fi-ru.turbopages.org': 'hi-fi.ru',
 'liveresult-ru.turbopages.org': 'liveresult.ru',
 'yaplakal-com.turbopages.org': 'yaplakal.com',
 'pasmi-ru.turbopages.org': 'pasmi.ru',
 'rueconomics-ru.turbopages.org': 'rueconomics.ru',
 'm24-ru.turbopages.org': 'm24.ru',
 'dni-ru.turbopages.org': 'dni.ru',
 'tur--ray-ru.turbopages.org': 'tur-ray.ru',
 'novayagazeta-ru.turbopages.org': 'novayagazeta.ru',
 'zooon-ru.turbopages.org': 'zooon.ru',
 'privetpeople-ru.turbopages.org': 'privetpeople.ru',
 'bookmakers--rank-ru.turbopages.org': 'bookmakers-rank.ru',
 'eg-ru.turbopages.org': 'eg.ru',
 'medportal-ru.turbopages.org': 'medportal.ru',
 'style-rbc-ru.turbopages.org': 'style.rbc.ru',
 'fontanka-ru.turbopages.org': 'fontanka.ru',
 'moika78-ru.turbopages.org': 'moika78.ru',
 '78-ru.turbopages.org': '78.ru',
 'nevnov-ru.turbopages.org': 'nevnov.ru',
 'nord--ursus-livejournal-com.turbopages.org': 'nord-ursus.livejournal.com',
 '53news-ru.turbopages.org': '53news.ru',
 'kinopoduglom-ru.turbopages.org': 'kinopoduglom.ru',
 'bugaga-ru.turbopages.org': 'bugaga.ru',
 'alexmonc-livejournal-com.turbopages.org': 'alexmonc.livejournal.com',
 'avtosotka-ru.turbopages.org': 'avtosotka.ru',
 'kanobu-ru.turbopages.org': 'kanobu.ru',
 'igromania-ru.turbopages.org': 'igromania.ru',
 'rsport-ria-ru.turbopages.org': 'rsport.ria.ru',
 'sovets-net.turbopages.org': 'sovets.net',
 'gazeta-a42-ru.turbopages.org': 'gazeta.a42.ru',
 'psy--magic-org.turbopages.org': 'psy-magic.org',
 'julia--klay-livejournal-com.turbopages.org': 'julia-klay.livejournal.com',
 'izh-kp-ru.turbopages.org': 'izh.kp.ru',
 'xn----7sbgaytzumno.turbopages.org': 'xn--7sbgaytzumno',
 'live--gta-ru.turbopages.org': 'live-gta.ru',
 'argumenti-ru.turbopages.org': 'argumenti.ru',
 'iz-ru.turbopages.org': 'iz.ru',
 'starhit-ru.turbopages.org': 'starhit.ru',
 'vmo24-ru.turbopages.org': 'vmo24.ru',
 'teleprogramma-pro.turbopages.org': 'teleprogramma.pro',
 '7days-ru.turbopages.org': '7days.ru',
 'medikforum-ru.turbopages.org': 'medikforum.ru',
 'pikabu-ru.turbopages.org': 'pikabu.ru',
 'medside-ru.turbopages.org': 'medside.ru',
 'smolenkaspb-ru.turbopages.org': 'smolenkaspb.ru',
 'spletnik-ru.turbopages.org': 'spletnik.ru',
 'dama--plus-ru.turbopages.org': 'dama-plus.ru',
 'mistymag-ru.turbopages.org': 'mistymag.ru',
 'mariantas-ru.turbopages.org': 'mariantas.ru',
 'xn--1000--9veopa1gyah6c.turbopages.org': 'xn-1000-9veopa1gyah6c',
 'mostpress-ru.turbopages.org': 'mostpress.ru',
 'signorina-ru.turbopages.org': 'signorina.ru',
 'zagovormaga-ru.turbopages.org': 'zagovormaga.ru',
 '365calend-ru.turbopages.org': '365calend.ru',
 'kurer--sreda-ru.turbopages.org': 'kurer-sreda.ru',
 'ezo-today.turbopages.org': 'ezo.today',
 'ezoterfr-ru.turbopages.org': 'ezoterfr.ru',
 'art--assorty-ru.turbopages.org': 'art-assorty.ru',
 'woman-rambler-ru.turbopages.org': 'woman.rambler.ru',
 'astroclan-ru.turbopages.org': 'astroclan.ru',
 'kleos-ru.turbopages.org': 'kleos.ru',
 'domznatok-ru.turbopages.org': 'domznatok.ru',
 'plastichno-com.turbopages.org': 'plastichno.com',
 'themaykl-ru.turbopages.org': 'themaykl.ru',
 'v--2022-life.turbopages.org': 'v-2022.life',
 'planeta--mineral-ru.turbopages.org': 'planeta-mineral.ru',
 'raduga-net-ru.turbopages.org': 'raduga.net.ru',
 'pogoda--crimea-ru.turbopages.org': 'pogoda-crimea.ru',
 'ivanovo-bezformata-com.turbopages.org': 'ivanovo.bezformata.com',
 'lovelyhair-org.turbopages.org': 'lovelyhair.org',
 '2022--years-ru.turbopages.org': '2022-years.ru',
 'astrogod-ru.turbopages.org': 'astrogod.ru',
 'womanhappiness-ru.turbopages.org': 'womanhappiness.ru',
 'nameorigin-ru.turbopages.org': 'nameorigin.ru',
 'poisklekarstv-com.turbopages.org': 'poisklekarstv.com',
 'shmk-ru.turbopages.org': 'shmk.ru',
 'car-ru.turbopages.org': 'car.ru',
 'radiokp-ru.turbopages.org': 'radiokp.ru',
 'eadaily-com.turbopages.org': 'eadaily.com',
 'ilyavaliev-livejournal-com.turbopages.org': 'ilyavaliev.livejournal.com',
 'moydom-moscow.turbopages.org': 'moydom.moscow',
 'pasporta-org.turbopages.org': 'pasporta.org',
 'sunhome-ru.turbopages.org': 'sunhome.ru',
 'pishu--pravilno-livejournal-com.turbopages.org': 'pishu-pravilno.livejournal.com',
 'ourbaby-ru.turbopages.org': 'ourbaby.ru',
 'megapteka-ru.turbopages.org': 'megapteka.ru',
 'translated.turbopages.org': 'translated',
 'vbr-ru.turbopages.org': 'vbr.ru',
 'blog-dohcolonoc-ru.turbopages.org': 'blog.dohcolonoc.ru',
 'telegraf-by.turbopages.org': 'telegraf.by',
 'kak--pravilno-net.turbopages.org': 'kak-pravilno.net',
 'my--expert-ru.turbopages.org': 'my-expert.ru',
 'kotletki-online.turbopages.org': 'kotletki.online',
 'b17-ru.turbopages.org': 'b17.ru',
 'blog--recept-ru.turbopages.org': 'blog-recept.ru',
 'ilive-com-ua.turbopages.org': 'ilive.com.ua',
 'enigma--project-ru.turbopages.org': 'enigma-project.ru',
 'nalog--nalog-ru.turbopages.org': 'nalog-nalog.ru',
 'm--strana-ru.turbopages.org': 'm-strana.ru',
 'yugarf-ru.turbopages.org': 'yugarf.ru',
 'lafoy-ru.turbopages.org': 'lafoy.ru',
 'edaturistu-ru.turbopages.org': 'edaturistu.ru',
 'bankiros-ru.turbopages.org': 'bankiros.ru',
 'vasilev--life-ru.turbopages.org': 'vasilev-life.ru',
 'vladtime-ru.turbopages.org': 'vladtime.ru',
 'dacha73-ru.turbopages.org': 'dacha73.ru',
 'ogon-guru.turbopages.org': 'ogon.guru',
 'djurenko-com.turbopages.org': 'djurenko.com',
 'hr--portal-ru.turbopages.org': 'hr-portal.ru',
 'dizajny-guru.turbopages.org': 'dizajny.guru',
 'russo--travel-ru.turbopages.org': 'russo-travel.ru',
 'fedeter-livejournal-com.turbopages.org': 'fedeter.livejournal.com',
 'diabaz--angarsk-ru.turbopages.org': 'diabaz-angarsk.ru',
 'podelunchik-ru.turbopages.org': 'podelunchik.ru',
 'staryiy-livejournal-com.turbopages.org': 'staryiy.livejournal.com',
 'realty-rbc-ru.turbopages.org': 'realty.rbc.ru',
 'sdelaikamin-ru.turbopages.org': 'sdelaikamin.ru',
 'comfortdacha-ru.turbopages.org': 'comfortdacha.ru',
 'heroine-ru.turbopages.org': 'heroine.ru',
 'spasibovsem-ru.turbopages.org': 'spasibovsem.ru',
 'vkusno--gotovit-ru.turbopages.org': 'vkusno-gotovit.ru',
 'okvedkod-ru.turbopages.org': 'okvedkod.ru',
 'ko-ru.turbopages.org': 'ko.ru',
 'krasotaved-ru.turbopages.org': 'krasotaved.ru',
 'legkayaeda-ru.turbopages.org': 'legkayaeda.ru',
 '56orb-ru.turbopages.org': '56orb.ru',
 'ufa-kp-ru.turbopages.org': 'ufa.kp.ru',
 'zachestnyibiznes-ru.turbopages.org': 'zachestnyibiznes.ru',
 'okolomeda-ru.turbopages.org': 'okolomeda.ru',
 'ayzdorov-ru.turbopages.org': 'ayzdorov.ru',
 'xn----7sbahent3a5bc8b8h.turbopages.org': 'xn--7sbahent3a5bc8b8h',
 'ctege-info.turbopages.org': 'ctege.info',
 'nukadeti-ru.turbopages.org': 'nukadeti.ru',
 'datafilm-info.turbopages.org': 'datafilm.info',
 'nation--news-ru.turbopages.org': 'nation-news.ru',
 'augustnews-ru.turbopages.org': 'augustnews.ru',
 'slovaronline-com.turbopages.org': 'slovaronline.com',
 'pravilnoe--pokhudenie-ru.turbopages.org': 'pravilnoe-pokhudenie.ru',
 'tab-103-kz.turbopages.org': 'tab.103.kz',
 'zvukibukvy-ru.turbopages.org': 'zvukibukvy.ru',
 'bestcube-space.turbopages.org': 'bestcube.space',
 'mag-103-by.turbopages.org': 'mag.103.by',
 'chv-aif-ru.turbopages.org': 'chv.aif.ru',
 'gadgets--reviews-com.turbopages.org': 'gadgets-reviews.com',
 'letidor-ru.turbopages.org': 'letidor.ru',
 'yur--gazeta-ru.turbopages.org': 'yur-gazeta.ru',
 'knyazhinskaya-ru.turbopages.org': 'knyazhinskaya.ru',
 'gufo-me.turbopages.org': 'gufo.me',
 'tkac-ru.turbopages.org': 'tkac.ru',
 'proaist-ru.turbopages.org': 'proaist.ru',
 'media--polesye-by.turbopages.org': 'media-polesye.by',
 'sunmag-me.turbopages.org': 'sunmag.me',
 'givotniymir-ru.turbopages.org': 'givotniymir.ru',
 'news-sportbox-ru.turbopages.org': 'news.sportbox.ru',
 'horoscopes-rambler-ru.turbopages.org': 'horoscopes.rambler.ru',
 'passion-ru.turbopages.org': 'passion.ru',
 '1gai-ru.turbopages.org': '1gai.ru',
 'kuban-kp-ru.turbopages.org': 'kuban.kp.ru',
 'moskvichmag-ru.turbopages.org': 'moskvichmag.ru',
 'aif-by.turbopages.org': 'aif.by',
 'botanichka-ru.turbopages.org': 'botanichka.ru',
 'apral-ru.turbopages.org': 'apral.ru',
 'voenhronika-ru.turbopages.org': 'voenhronika.ru',
 'diz--cafe-com.turbopages.org': 'diz-cafe.com',
 'muksun-fm.turbopages.org': 'muksun.fm',
 'asienda-ru.turbopages.org': 'asienda.ru',
 'sudba-info.turbopages.org': 'sudba.info',
 'sarnovosti-ru.turbopages.org': 'sarnovosti.ru',
 'ferra-ru.turbopages.org': 'ferra.ru',
 'stroyguru-com.turbopages.org': 'stroyguru.com',
 'autokuz-ru.turbopages.org': 'autokuz.ru',
 'tvcenter-ru.turbopages.org': 'tvcenter.ru',
 'krasotka-cc.turbopages.org': 'krasotka.cc',
 'fd-ru.turbopages.org': 'fd.ru',
 'travel-rambler-ru.turbopages.org': 'travel.rambler.ru',
 'shraibikus-com.turbopages.org': 'shraibikus.com',
 'spbdnevnik-ru.turbopages.org': 'spbdnevnik.ru',
 'runews24-ru.turbopages.org': 'runews24.ru',
 'kolesa-ru.turbopages.org': 'kolesa.ru',
 'genapilot-ru.turbopages.org': 'genapilot.ru',
 'povolosam-ru.turbopages.org': 'povolosam.ru',
 'stroim--domik-org.turbopages.org': 'stroim-domik.org',
 'profazu-ru.turbopages.org': 'profazu.ru',
 'kost--shirokaya-ru.turbopages.org': 'kost-shirokaya.ru',
 'tonkosti-ru.turbopages.org': 'tonkosti.ru',
 'trudogolikam-ru.turbopages.org': 'trudogolikam.ru',
 'odinelectric-ru.turbopages.org': 'odinelectric.ru',
 'mastergrad-com.turbopages.org': 'mastergrad.com',
 'malls-ru.turbopages.org': 'malls.ru',
 'dyshis-livejournal-com.turbopages.org': 'dyshis.livejournal.com',
 'zdorovieinfo-ru.turbopages.org': 'zdorovieinfo.ru',
 'scanwordhelper-ru.turbopages.org': 'scanwordhelper.ru',
 'expert-ru.turbopages.org': 'expert.ru',
 'nsk-aif-ru.turbopages.org': 'nsk.aif.ru',
 'poleznii--site-ru.turbopages.org': 'poleznii-site.ru',
 'riamo-ru.turbopages.org': 'riamo.ru',
 'sb-by.turbopages.org': 'sb.by',
 'molokook-ru.turbopages.org': 'molokook.ru',
 'vladimirdar-livejournal-com.turbopages.org': 'vladimirdar.livejournal.com',
 'tayga-info.turbopages.org': 'tayga.info',
 'gotovim--doma-ru.turbopages.org': 'gotovim-doma.ru',
 'realt-onliner-by.turbopages.org': 'realt.onliner.by',
 'webznam-ru.turbopages.org': 'webznam.ru',
 'travel--russia-livejournal-com.turbopages.org': 'travel-russia.livejournal.com',
 'ptoday-ru.turbopages.org': 'ptoday.ru',
 'astro--world-ru.turbopages.org': 'astro-world.ru',
 'sergey--gora-livejournal-com.turbopages.org': 'sergey-gora.livejournal.com',
 'playboyrussia-com.turbopages.org': 'playboyrussia.com',
 'dogtricks-ru.turbopages.org': 'dogtricks.ru',
 'azbyka--vkysa-ru.turbopages.org': 'azbyka-vkysa.ru',
 'edimdoma-ru.turbopages.org': 'edimdoma.ru',
 'bobruisk-ru.turbopages.org': 'bobruisk.ru',
 'patee-ru.turbopages.org': 'patee.ru',
 '63-ru.turbopages.org': '63.ru',
 'bragazeta-ru.turbopages.org': 'bragazeta.ru',
 'csn--tv-ru.turbopages.org': 'csn-tv.ru',
 'kino-rambler-ru.turbopages.org': 'kino.rambler.ru',
 'euro--football-ru.turbopages.org': 'euro-football.ru',
 'avchernov-ru.turbopages.org': 'avchernov.ru',
 'nashe-ru.turbopages.org': 'nashe.ru',
 'riastrela-ru.turbopages.org': 'riastrela.ru',
 'spb-mk-ru.turbopages.org': 'spb.mk.ru',
 'politros-com.turbopages.org': 'politros.com',
 'speedme-ru.turbopages.org': 'speedme.ru',
 'newsbryansk-ru.turbopages.org': 'newsbryansk.ru',
 'football-kulichki-net.turbopages.org': 'football.kulichki.net',
 'reproduktor-net.turbopages.org': 'reproduktor.net',
 'ktv--ray-ru.turbopages.org': 'ktv-ray.ru',
 'nsk-kp-ru.turbopages.org': 'nsk.kp.ru',
 'rostov-mk-ru.turbopages.org': 'rostov.mk.ru',
 'bryansktoday-ru.turbopages.org': 'bryansktoday.ru',
 'ngs-ru.turbopages.org': 'ngs.ru',
 'thebell-io.turbopages.org': 'thebell.io',
 'bryansku-ru.turbopages.org': 'bryansku.ru',
 'dailystorm-ru.turbopages.org': 'dailystorm.ru',
 'mkbryansk-ru.turbopages.org': 'mkbryansk.ru',
 'mockva-ru.turbopages.org': 'mockva.ru',
 'samara-kp-ru.turbopages.org': 'samara.kp.ru',
 'sport-rambler-ru.turbopages.org': 'sport.rambler.ru',
 'people-onliner-by.turbopages.org': 'people.onliner.by',
 'rosbalt-ru.turbopages.org': 'rosbalt.ru',
 'nsn-fm.turbopages.org': 'nsn.fm',
 'selo-guru.turbopages.org': 'selo.guru',
 'sportrbc-ru.turbopages.org': 'sportrbc.ru',
 'vedomosti-ru.turbopages.org': 'vedomosti.ru',
 'detki-guru.turbopages.org': 'detki.guru',
 '26--2-ru.turbopages.org': '26-2.ru',
 'spark-ru.turbopages.org': 'spark.ru',
 'profile-ru.turbopages.org': 'profile.ru',
 'sex-missfit-ru.turbopages.org': 'sex.missfit.ru',
 'gorod55-ru.turbopages.org': 'gorod55.ru',
 'posobie--expert-com.turbopages.org': 'posobie-expert.com',
 'newdaynews-ru.turbopages.org': 'newdaynews.ru',
 'ekb-tsargrad-tv.turbopages.org': 'ekb.tsargrad.tv',
 'ng-ru.turbopages.org': 'ng.ru',
 'ianed-ru.turbopages.org': 'ianed.ru',
 'kirov-online.turbopages.org': 'kirov.online',
 'avtonovostidnya-ru.turbopages.org': 'avtonovostidnya.ru',
 'urexpert-online.turbopages.org': 'urexpert.online',
 'wer-ru.turbopages.org': 'wer.ru',
 'osetinskie--avdeevskie-ru.turbopages.org': 'osetinskie-avdeevskie.ru',
 'v--nayke-ru.turbopages.org': 'v-nayke.ru',
 'vokrug-tv.turbopages.org': 'vokrug.tv',
 'newizv-ru.turbopages.org': 'newizv.ru',
 'nashural-ru.turbopages.org': 'nashural.ru',
 'utv-ru.turbopages.org': 'utv.ru',
 'ufa1-ru.turbopages.org': 'ufa1.ru',
 'actualnews-org.turbopages.org': 'actualnews.org',
 '1beton-info.turbopages.org': '1beton.info',
 'trinixy-ru.turbopages.org': 'trinixy.ru',
 'remont--book-com.turbopages.org': 'remont-book.com',
 'pravovoi-center.turbopages.org': 'pravovoi.center',
 'kaluga--poisk-ru.turbopages.org': 'kaluga-poisk.ru',
 'business--gazeta-ru.turbopages.org': 'business-gazeta.ru',
 'spayte-livejournal-com.turbopages.org': 'spayte.livejournal.com',
 'forumhouse-ru.turbopages.org': 'forumhouse.ru',
 'pravda-ru.turbopages.org': 'pravda.ru',
 'ulan-mk-ru.turbopages.org': 'ulan.mk.ru',
 'autotopik-ru.turbopages.org': 'autotopik.ru',
 'vk--sto-by.turbopages.org': 'vk-sto.by',
 'fastmb-ru.turbopages.org': 'fastmb.ru',
 'fix--my--car-ru.turbopages.org': 'fix-my-car.ru',
 'rusfermer-net.turbopages.org': 'rusfermer.net',
 'endoexpert-ru.turbopages.org': 'endoexpert.ru',
 'carpedia-club.turbopages.org': 'carpedia.club',
 'znatoksna-ru.turbopages.org': 'znatoksna.ru',
 'hiromantia-net.turbopages.org': 'hiromantia.net',
 'ss69100-livejournal-com.turbopages.org': 'ss69100.livejournal.com',
 'psyont-livejournal-com.turbopages.org': 'psyont.livejournal.com',
 'alkopona-livejournal-com.turbopages.org': 'alkopona.livejournal.com',
 'syl-ru.turbopages.org': 'syl.ru',
 'be5-biz.turbopages.org': 'be5.biz',
 'paranormal--news-ru.turbopages.org': 'paranormal-news.ru',
 'kakprosto-ru.turbopages.org': 'kakprosto.ru',
 'lovetrue-ru.turbopages.org': 'lovetrue.ru',
 'megaotkrytka-ru.turbopages.org': 'megaotkrytka.ru',
 'b--picture-livejournal-com.turbopages.org': 'b-picture.livejournal.com',
 'brodude-ru.turbopages.org': 'brodude.ru',
 'ridus-ru.turbopages.org': 'ridus.ru',
 'rus--songs-ru.turbopages.org': 'rus-songs.ru',
 'otzyvru-com.turbopages.org': 'otzyvru.com',
 'abnews-ru.turbopages.org': 'abnews.ru',
 'vostokmedia-com.turbopages.org': 'vostokmedia.com',
 'realnoevremya-ru.turbopages.org': 'realnoevremya.ru',
 'anchiktigra-livejournal-com.turbopages.org': 'anchiktigra.livejournal.com',
 'polismed-com.turbopages.org': 'polismed.com',
 'inpearls-ru.turbopages.org': 'inpearls.ru',
 'fintolk-pro.turbopages.org': 'fintolk.pro',
 'pro--personal-ru.turbopages.org': 'pro-personal.ru',
 'vagon--vokzal-ru.turbopages.org': 'vagon-vokzal.ru',
 'trudtk-ru.turbopages.org': 'trudtk.ru',
 'buhguru-com.turbopages.org': 'buhguru.com',
 'prazdniki--na--nosu-com.turbopages.org': 'prazdniki-na-nosu.com',
 'tvsamara-ru.turbopages.org': 'tvsamara.ru',
 'she--win-ru.turbopages.org': 'she-win.ru',
 'blotos-ru.turbopages.org': 'blotos.ru',
 'progorodsamara-ru.turbopages.org': 'progorodsamara.ru',
 'zakonrf-info.turbopages.org': 'zakonrf.info',
 'komiinform-ru.turbopages.org': 'komiinform.ru',
 'poisk--ru-ru.turbopages.org': 'poisk-ru.ru',
 'xn------sddwiscobghkkgw6k.turbopages.org': 'xn---sddwiscobghkkgw6k',
 'zdravotvet-ru.turbopages.org': 'zdravotvet.ru',
 'ctnews-ru.turbopages.org': 'ctnews.ru',
 'freesoft-ru.turbopages.org': 'freesoft.ru',
 'testometrika-com.turbopages.org': 'testometrika.com',
 'businessman-ru.turbopages.org': 'businessman.ru',
 'altapress-ru.turbopages.org': 'altapress.ru',
 'gorsite-ru.turbopages.org': 'gorsite.ru',
 'rberega-info.turbopages.org': 'rberega.info',
 'sib-fm.turbopages.org': 'sib.fm',
 'nsk-tsargrad-tv.turbopages.org': 'nsk.tsargrad.tv',
 'sibkray-ru.turbopages.org': 'sibkray.ru',
 'om1-ru.turbopages.org': 'om1.ru',
 'russianteleweek-ru.turbopages.org': 'russianteleweek.ru',
 'livelib-ru.turbopages.org': 'livelib.ru',
 'rubaltic-ru.turbopages.org': 'rubaltic.ru',
 'vse42-ru.turbopages.org': 'vse42.ru',
 'ryazan-kp-ru.turbopages.org': 'ryazan.kp.ru',
 'novos-mk-ru.turbopages.org': 'novos.mk.ru',
 'atas-info.turbopages.org': 'atas.info',
 'deita-ru.turbopages.org': 'deita.ru',
 'msk-kp-ru.turbopages.org': 'msk.kp.ru',
 'mk--hakasia-ru.turbopages.org': 'mk-hakasia.ru',
 'balakovo24-ru.turbopages.org': 'balakovo24.ru',
 'newkhakasiya-online.turbopages.org': 'newkhakasiya.online',
 'zapad24-ru.turbopages.org': 'zapad24.ru',
 'bashinform-ru.turbopages.org': 'bashinform.ru',
 'infox-ru.turbopages.org': 'infox.ru',
 'chita-ru.turbopages.org': 'chita.ru',
 'fn--volga-ru.turbopages.org': 'fn-volga.ru',
 'glasnarod-ru.turbopages.org': 'glasnarod.ru',
 'rostovgazeta-ru.turbopages.org': 'rostovgazeta.ru',
 'kino--teatr-ru.turbopages.org': 'kino-teatr.ru',
 'sadovnikam-ru.turbopages.org': 'sadovnikam.ru',
 'pskov-aif-ru.turbopages.org': 'pskov.aif.ru',
 'u--mama-ru.turbopages.org': 'u-mama.ru',
 'ferma-expert.turbopages.org': 'ferma.expert',
 'analogi-info.turbopages.org': 'analogi.info',
 'lipetsk-kp-ru.turbopages.org': 'lipetsk.kp.ru',
 'vrn-kp-ru.turbopages.org': 'vrn.kp.ru',
 'seite1-ru.turbopages.org': 'seite1.ru',
 'riavrn-ru.turbopages.org': 'riavrn.ru',
 'rezina48-ru.turbopages.org': 'rezina48.ru',
 'ladaautos-ru.turbopages.org': 'ladaautos.ru',
 'auto-rambler-ru.turbopages.org': 'auto.rambler.ru',
 'usamodelkina-ru.turbopages.org': 'usamodelkina.ru',
 'twokarburators-ru.turbopages.org': 'twokarburators.ru',
 'burninghut-ru.turbopages.org': 'burninghut.ru',
 'pulsplus-ru.turbopages.org': 'pulsplus.ru',
 'iklife-ru.turbopages.org': 'iklife.ru',
 'best--stroy-ru.turbopages.org': 'best-stroy.ru',
 'ichip-ru.turbopages.org': 'ichip.ru',
 'tourister-ru.turbopages.org': 'tourister.ru',
 'ivd-ru.turbopages.org': 'ivd.ru',
 'foodandhealth-ru.turbopages.org': 'foodandhealth.ru',
 'comfort--myhouse-ru.turbopages.org': 'comfort-myhouse.ru',
 'kagdela-ru.turbopages.org': 'kagdela.ru',
 'ladyelena-ru.turbopages.org': 'ladyelena.ru',
 'tvjam-ru.turbopages.org': 'tvjam.ru',
 'vademec-ru.turbopages.org': 'vademec.ru',
 'vibirayigry-ru.turbopages.org': 'vibirayigry.ru',
 'medvisor-ru.turbopages.org': 'medvisor.ru',
 'idealnyi--manikur-ru.turbopages.org': 'idealnyi-manikur.ru',
 'glavbukh-ru.turbopages.org': 'glavbukh.ru',
 'inkazan-ru.turbopages.org': 'inkazan.ru',
 'kudago-com.turbopages.org': 'kudago.com',
 'kostroma-mk-ru.turbopages.org': 'kostroma.mk.ru',
 'kostroma-kp-ru.turbopages.org': 'kostroma.kp.ru',
 'topsapozhok-ru.turbopages.org': 'topsapozhok.ru',
 'smolensk-big--book--med-ru.turbopages.org': 'smolensk.big-book-med.ru',
 'kurkumagid-ru.turbopages.org': 'kurkumagid.ru',
 'razvitie--vospitanie-ru.turbopages.org': 'razvitie-vospitanie.ru',
 'inetgramotnost-ru.turbopages.org': 'inetgramotnost.ru',
 'materinstvo-ru.turbopages.org': 'materinstvo.ru',
 'kogda-org.turbopages.org': 'kogda.org',
 'shenkama-livejournal-com.turbopages.org': 'shenkama.livejournal.com',
 'mk--kuzbass-ru.turbopages.org': 'mk-kuzbass.ru',
 '7mednews-ru.turbopages.org': '7mednews.ru',
 'nn-ru.turbopages.org': 'nn.ru',
 'mtsdtv-ru.turbopages.org': 'mtsdtv.ru',
 'peopletalk-ru.turbopages.org': 'peopletalk.ru',
 'dni24-com.turbopages.org': 'dni24.com',
 'spb-kp-ru.turbopages.org': 'spb.kp.ru',
 'itzine-ru.turbopages.org': 'itzine.ru',
 'tver-kp-ru.turbopages.org': 'tver.kp.ru',
 'tarantas-news.turbopages.org': 'tarantas.news',
 'matveychev--oleg-livejournal-com.turbopages.org': 'matveychev-oleg.livejournal.com',
 'mogura-ru.turbopages.org': 'mogura.ru',
 'vsvoemdome-ru.turbopages.org': 'vsvoemdome.ru',
 'miss--runet-ru.turbopages.org': 'miss-runet.ru',
 'amic-ru.turbopages.org': 'amic.ru',
 'edaplus-info.turbopages.org': 'edaplus.info',
 'hozaika-online.turbopages.org': 'hozaika.online',
 'karamellka-ru.turbopages.org': 'karamellka.ru',
 'swinopes-livejournal-com.turbopages.org': 'swinopes.livejournal.com',
 'agronom-guru.turbopages.org': 'agronom.guru',
 'angelvalentina-livejournal-com.turbopages.org': 'angelvalentina.livejournal.com',
 'happylifestyle-ru.turbopages.org': 'happylifestyle.ru',
 'blagoe21-ru.turbopages.org': 'blagoe21.ru',
 'activestudy-info.turbopages.org': 'activestudy.info',
 'kipmu-ru.turbopages.org': 'kipmu.ru',
 'correctno-ru.turbopages.org': 'correctno.ru',
 'zakonguru-com.turbopages.org': 'zakonguru.com',
 'proslo-ru.turbopages.org': 'proslo.ru',
 'glam--lady-ru.turbopages.org': 'glam-lady.ru',
 'culture-wikireading-ru.turbopages.org': 'culture.wikireading.ru',
 'fito--tea-com.turbopages.org': 'fito-tea.com',
 'apteka24chasa-ru.turbopages.org': 'apteka24chasa.ru',
 'dacha-help.turbopages.org': 'dacha.help',
 'daymam-ru.turbopages.org': 'daymam.ru',
 'easytravelling-ru.turbopages.org': 'easytravelling.ru',
 'moy--povar-ru.turbopages.org': 'moy-povar.ru',
 'ezoterika--ru-ru.turbopages.org': 'ezoterika-ru.ru',
 'immanuilchurch--vladimir-ru.turbopages.org': 'immanuilchurch-vladimir.ru',
 'tech-onliner-by.turbopages.org': 'tech.onliner.by',
 'silaamuleta-ru.turbopages.org': 'silaamuleta.ru',
 '74-ru.turbopages.org': '74.ru',
 'dentoland-com.turbopages.org': 'dentoland.com',
 'vizhuchetko-com.turbopages.org': 'vizhuchetko.com',
 'vegatrend-ru.turbopages.org': 'vegatrend.ru',
 'vkreditbe-ru.turbopages.org': 'vkreditbe.ru',
 'uralpolit-ru.turbopages.org': 'uralpolit.ru',
 'levelself-ru.turbopages.org': 'levelself.ru',
 'newsbash-ru.turbopages.org': 'newsbash.ru',
 '59-ru.turbopages.org': '59.ru',
 'restate-ru.turbopages.org': 'restate.ru',
 'pravo-rg-ru.turbopages.org': 'pravo.rg.ru',
 'tochka-press.turbopages.org': 'tochka.press',
 'fin--az-ru.turbopages.org': 'fin-az.ru',
 'leonuri-livejournal-com.turbopages.org': 'leonuri.livejournal.com',
 'hab-kp-ru.turbopages.org': 'hab.kp.ru',
 'psihomed-com.turbopages.org': 'psihomed.com',
 'amurmedia-ru.turbopages.org': 'amurmedia.ru',
 'syzran--small-ru.turbopages.org': 'syzran-small.ru',
 'sakhalin-kp-ru.turbopages.org': 'sakhalin.kp.ru',
 'realty-ria-ru.turbopages.org': 'realty.ria.ru',
 'vesti--k-ru.turbopages.org': 'vesti-k.ru',
 'sipililo-livejournal-com.turbopages.org': 'sipililo.livejournal.com',
 'intex--press-by.turbopages.org': 'intex-press.by',
 'news-21-by.turbopages.org': 'news.21.by',
 'progorod76-ru.turbopages.org': 'progorod76.ru',
 'penzainform-ru.turbopages.org': 'penzainform.ru',
 'rcycle-net.turbopages.org': 'rcycle.net',
 'organikeda-com.turbopages.org': 'organikeda.com',
 'hotsoul-ru.turbopages.org': 'hotsoul.ru',
 'vcusnyatina-com.turbopages.org': 'vcusnyatina.com',
 'avtovzglyad-ru.turbopages.org': 'avtovzglyad.ru',
 'vladimir-tsargrad-tv.turbopages.org': 'vladimir.tsargrad.tv',
 'tkaner-com.turbopages.org': 'tkaner.com',
 'wmj-ru.turbopages.org': 'wmj.ru',
 'slovodel-com.turbopages.org': 'slovodel.com',
 'spb-tsargrad-tv.turbopages.org': 'spb.tsargrad.tv',
 'wiki--dacha-ru.turbopages.org': 'wiki-dacha.ru',
 'ya62-ru.turbopages.org': 'ya62.ru',
 'rzn-aif-ru.turbopages.org': 'rzn.aif.ru',
 'nakanune-ru.turbopages.org': 'nakanune.ru',
 'ngs55-ru.turbopages.org': 'ngs55.ru',
 'omsk-mk-ru.turbopages.org': 'omsk.mk.ru',
 'newsomsk-ru.turbopages.org': 'newsomsk.ru',
 'omskpress-ru.turbopages.org': 'omskpress.ru',
 'inni-info.turbopages.org': 'inni.info',
 'mur-tv.turbopages.org': 'mur.tv',
 'gorodkirov-ru.turbopages.org': 'gorodkirov.ru',
 'zhenskoe--mnenie-ru.turbopages.org': 'zhenskoe-mnenie.ru',
 'progorod43-ru.turbopages.org': 'progorod43.ru',
 'khngi-ru.turbopages.org': 'khngi.ru',
 'kirov--portal-ru.turbopages.org': 'kirov-portal.ru',
 'setka22-ru.turbopages.org': 'setka22.ru',
 'kubnews-ru.turbopages.org': 'kubnews.ru',
 'progorodnn-ru.turbopages.org': 'progorodnn.ru',
 'n--e--n-ru.turbopages.org': 'n-e-n.ru',
 'crimea-kp-ru.turbopages.org': 'crimea.kp.ru',
 'oren-aif-ru.turbopages.org': 'oren.aif.ru',
 'strana--sovetov-com.turbopages.org': 'strana-sovetov.com',
 'komnatnue--rastenija-com.turbopages.org': 'komnatnue-rastenija.com',
 '101kote-ru.turbopages.org': '101kote.ru',
 'diary-ru.turbopages.org': 'diary.ru',
 'catholic--russia-livejournal-com.turbopages.org': 'catholic-russia.livejournal.com',
 'domibit-ru.turbopages.org': 'domibit.ru',
 'spb-aif-ru.turbopages.org': 'spb.aif.ru',
 'neplohoy-livejournal-com.turbopages.org': 'neplohoy.livejournal.com',
 'sekret-me.turbopages.org': 'sekret.me',
 'infobliz-ru.turbopages.org': 'infobliz.ru',
 'news2star-ru.turbopages.org': 'news2star.ru',
 'vstretim--prazdnik-com.turbopages.org': 'vstretim-prazdnik.com',
 'protatar-ru.turbopages.org': 'protatar.ru',
 'gb21perm-ru.turbopages.org': 'gb21perm.ru',
 'stom-32top-ru.turbopages.org': 'stom.32top.ru',
 'nahybride-ru.turbopages.org': 'nahybride.ru',
 'volg-mk-ru.turbopages.org': 'volg.mk.ru',
 'msknovosti-ru.turbopages.org': 'msknovosti.ru',
 'mir--auto24-ru.turbopages.org': 'mir-auto24.ru',
 'urok-1sept-ru.turbopages.org': 'urok.1sept.ru',
 'banyaspec-com.turbopages.org': 'banyaspec.com',
 'gadinagod-livejournal-com.turbopages.org': 'gadinagod.livejournal.com',
 'kak--sdelano-ru.turbopages.org': 'kak-sdelano.ru',
 'dizain--vannoy-ru.turbopages.org': 'dizain-vannoy.ru',
 'septik27-ru.turbopages.org': 'septik27.ru',
 'creativportal-ru.turbopages.org': 'creativportal.ru',
 'moslenta-ru.turbopages.org': 'moslenta.ru',
 'viasun-ru.turbopages.org': 'viasun.ru',
 'stranamam-ru.turbopages.org': 'stranamam.ru',
 'fictionbook-ru.turbopages.org': 'fictionbook.ru',
 'i2hard-ru.turbopages.org': 'i2hard.ru',
 'topkin-ru.turbopages.org': 'topkin.ru',
 'ug-tsargrad-tv.turbopages.org': 'ug.tsargrad.tv',
 'txt--pesen-ru.turbopages.org': 'txt-pesen.ru',
 '7plus-pro.turbopages.org': '7plus.pro',
 'progorodchelny-ru.turbopages.org': 'progorodchelny.ru',
 'kazan-mk-ru.turbopages.org': 'kazan.mk.ru',
 'trt--tv-ru.turbopages.org': 'trt-tv.ru',
 'kazanfirst-ru.turbopages.org': 'kazanfirst.ru',
 'prokazan-ru.turbopages.org': 'prokazan.ru',
 'intermedia-ru.turbopages.org': 'intermedia.ru',
 'kazan-aif-ru.turbopages.org': 'kazan.aif.ru',
 'kazan-kp-ru.turbopages.org': 'kazan.kp.ru',
 'netpryshi-ru.turbopages.org': 'netpryshi.ru',
 'tehno--gid-net.turbopages.org': 'tehno-gid.net',
 'radiosputnik-ria-ru.turbopages.org': 'radiosputnik.ria.ru',
 'prosto--eto-ru.turbopages.org': 'prosto-eto.ru',
 'magicrituals-ru.turbopages.org': 'magicrituals.ru',
 'burdastyle-ru.turbopages.org': 'burdastyle.ru',
 'chel-mk-ru.turbopages.org': 'chel.mk.ru',
 'enisey-tv.turbopages.org': 'enisey.tv',
 'xn----itbibasdykeh4b2f7b.turbopages.org': 'xn--itbibasdykeh4b2f7b',
 'urozhajnayagryadka-ru.turbopages.org': 'urozhajnayagryadka.ru',
 'doctor-rambler-ru.turbopages.org': 'doctor.rambler.ru',
 'temaved-ru.turbopages.org': 'temaved.ru',
 'instyle-ru.turbopages.org': 'instyle.ru',
 'vselady-club.turbopages.org': 'vselady.club',
 'neboleem-net.turbopages.org': 'neboleem.net',
 'smolnarod-ru.turbopages.org': 'smolnarod.ru',
 'mk--pskov-ru.turbopages.org': 'mk-pskov.ru',
 'starandstar-ru.turbopages.org': 'starandstar.ru',
 'belnovosti-by.turbopages.org': 'belnovosti.by',
 'informing-ru.turbopages.org': 'informing.ru',
 'o--vode-net.turbopages.org': 'o-vode.net',
 'anisima-ru.turbopages.org': 'anisima.ru',
 'smarttvnews-ru.turbopages.org': 'smarttvnews.ru',
 'ndn-info.turbopages.org': 'ndn.info',
 'kemerovo-tsargrad-tv.turbopages.org': 'kemerovo.tsargrad.tv',
 'musizmp3-ru.turbopages.org': 'musizmp3.ru',
 'kopchen-ru.turbopages.org': 'kopchen.ru',
 'sesaga-ru.turbopages.org': 'sesaga.ru',
 'myshows-me.turbopages.org': 'myshows.me',
 'trudohrana-ru.turbopages.org': 'trudohrana.ru',
 'ohotniki-ru.turbopages.org': 'ohotniki.ru',
 'inforeactor-ru.turbopages.org': 'inforeactor.ru',
 'monavista-ru.turbopages.org': 'monavista.ru',
 'newinform-com.turbopages.org': 'newinform.com',
 'vk--smi-ru.turbopages.org': 'vk-smi.ru',
 'readovka-news.turbopages.org': 'readovka.news',
 'autonews-ru.turbopages.org': 'autonews.ru',
 'carsweek-ru.turbopages.org': 'carsweek.ru',
 'info24-ru.turbopages.org': 'info24.ru',
 'military-pravda-ru.turbopages.org': 'military.pravda.ru',
 'neva-today.turbopages.org': 'neva.today',
 'politexpert-net.turbopages.org': 'politexpert.net',
 'topdaynews-ru.turbopages.org': 'topdaynews.ru',
 'live24-ru.turbopages.org': 'live24.ru',
 'ruposters-ru.turbopages.org': 'ruposters.ru',
 'rzn-info.turbopages.org': 'rzn.info',
 '7info-ru.turbopages.org': '7info.ru',
 'rznonline-ru.turbopages.org': 'rznonline.ru',
 'mr--7-ru.turbopages.org': 'mr-7.ru',
 'vkirove-ru.turbopages.org': 'vkirove.ru',
 'portamur-ru.turbopages.org': 'portamur.ru',
 'secretmag-ru.turbopages.org': 'secretmag.ru',
 'chips--journal-ru.turbopages.org': 'chips-journal.ru',
 'openmedia-io.turbopages.org': 'openmedia.io',
 'trends-rbc-ru.turbopages.org': 'trends.rbc.ru',
 'snob-ru.turbopages.org': 'snob.ru',
 'sultan--tv-ru.turbopages.org': 'sultan-tv.ru',
 'daily-afisha-ru.turbopages.org': 'daily.afisha.ru',
 'v1-ru.turbopages.org': 'v1.ru',
 'mainstyles-ru.turbopages.org': 'mainstyles.ru',
 'mednovelty-ru.turbopages.org': 'mednovelty.ru',
 'oren-mk-ru.turbopages.org': 'oren.mk.ru',
 'jeannaskokova-livejournal-com.turbopages.org': 'jeannaskokova.livejournal.com',
 'nashideti-site.turbopages.org': 'nashideti.site',
 'citytraffic-ru.turbopages.org': 'citytraffic.ru',
 'tolksnov-ru.turbopages.org': 'tolksnov.ru',
 'big--rostov-ru.turbopages.org': 'big-rostov.ru',
 'blogfreo-ru.turbopages.org': 'blogfreo.ru',
 'rostov-kp-ru.turbopages.org': 'rostov.kp.ru',
 'sad--fialok-ru.turbopages.org': 'sad-fialok.ru',
 'drive-ru.turbopages.org': 'drive.ru',
 'maminakopilka-ru.turbopages.org': 'maminakopilka.ru',
 'kamchatinfo-com.turbopages.org': 'kamchatinfo.com',
 'alt-kp-ru.turbopages.org': 'alt.kp.ru',
 'ircity-ru.turbopages.org': 'ircity.ru',
 'islamnews-ru.turbopages.org': 'islamnews.ru',
 'clubtk-ru.turbopages.org': 'clubtk.ru',
 'stav26-livejournal-com.turbopages.org': 'stav26.livejournal.com',
 'knigaelektrika-ru.turbopages.org': 'knigaelektrika.ru',
 'samogonpil-ru.turbopages.org': 'samogonpil.ru',
 'militaryequipm-livejournal-com.turbopages.org': 'militaryequipm.livejournal.com',
 'filmpro-ru.turbopages.org': 'filmpro.ru',
 'vesti-ua.turbopages.org': 'vesti.ua',
 'calorizator-ru.turbopages.org': 'calorizator.ru',
 'hyundaicreta-ru.turbopages.org': 'hyundaicreta.ru',
 'komnatnie-com.turbopages.org': 'komnatnie.com',
 'mycharm-ru.turbopages.org': 'mycharm.ru',
 'mirkvartir-ru.turbopages.org': 'mirkvartir.ru',
 'androidinsider-ru.turbopages.org': 'androidinsider.ru',
 'tolknews-ru.turbopages.org': 'tolknews.ru',
 'trip--together-ru.turbopages.org': 'trip-together.ru',
 'productplanet-ru.turbopages.org': 'productplanet.ru',
 'calend-ru.turbopages.org': 'calend.ru',
 'kosarev39-ru.turbopages.org': 'kosarev39.ru',
 'otravamnet-ru.turbopages.org': 'otravamnet.ru',
 'crimea-ria-ru.turbopages.org': 'crimea.ria.ru',
 'italy4-me.turbopages.org': 'italy4.me',
 'cross-expert.turbopages.org': 'cross.expert',
 'zsigns-ru.turbopages.org': 'zsigns.ru',
 'woman-ru.turbopages.org': 'woman.ru',
 'felomena-com.turbopages.org': 'felomena.com',
 'novate-ru.turbopages.org': 'novate.ru',
 'tony-ru.turbopages.org': 'tony.ru',
 'dorogi--ne--dorogi-ru.turbopages.org': 'dorogi-ne-dorogi.ru',
 'enot2290-livejournal-com.turbopages.org': 'enot2290.livejournal.com',
 'dachnii-ru.turbopages.org': 'dachnii.ru',
 'krsk-kp-ru.turbopages.org': 'krsk.kp.ru',
 'turistas-me.turbopages.org': 'turistas.me',
 'mk--smolensk-ru.turbopages.org': 'mk-smolensk.ru',
 'smol-aif-ru.turbopages.org': 'smol.aif.ru',
 'komi-kp-ru.turbopages.org': 'komi.kp.ru',
 'allergolog-online.turbopages.org': 'allergolog.online',
 'bezpuza-ru.turbopages.org': 'bezpuza.ru',
 'dachadecor-ru.turbopages.org': 'dachadecor.ru',
 'lokhony-com.turbopages.org': 'lokhony.com',
 'moninomama-ru.turbopages.org': 'moninomama.ru',
 'page365-ru.turbopages.org': 'page365.ru',
 'esttat-ru.turbopages.org': 'esttat.ru',
 'donetsk-kp-ru.turbopages.org': 'donetsk.kp.ru',
 'kianews24-ru.turbopages.org': 'kianews24.ru',
 'tut--news-ru.turbopages.org': 'tut-news.ru',
 'am-tsargrad-tv.turbopages.org': 'am.tsargrad.tv',
 'yborka-online.turbopages.org': 'yborka.online',
 'topbloger-livejournal-com.turbopages.org': 'topbloger.livejournal.com',
 'psychosearch-ru.turbopages.org': 'psychosearch.ru',
 'paracels66-ru.turbopages.org': 'paracels66.ru',
 'kikonline-ru.turbopages.org': 'kikonline.ru',
 'apple--rent-ru.turbopages.org': 'apple-rent.ru',
 '45-ru.turbopages.org': '45.ru',
 'rozetked-me.turbopages.org': 'rozetked.me',
 'prosto--smak-ru.turbopages.org': 'prosto-smak.ru',
 'nnils-livejournal-com.turbopages.org': 'nnils.livejournal.com',
 'sev-ik-news.turbopages.org': 'sev.ik.news',
 'glagol-press.turbopages.org': 'glagol.press',
 'radion--gz--china-livejournal-com.turbopages.org': 'radion-gz-china.livejournal.com',
 'stav-kp-ru.turbopages.org': 'stav.kp.ru',
 'kavkaz--uzel-eu.turbopages.org': 'kavkaz-uzel.eu',
 'ruspravda-info.turbopages.org': 'ruspravda.info',
 'donbasstoday-ru.turbopages.org': 'donbasstoday.ru',
 'sz--lanna-livejournal-com.turbopages.org': 'sz-lanna.livejournal.com',
 'ua--katarsis-livejournal-com.turbopages.org': 'ua-katarsis.livejournal.com',
 'gorodche-ru.turbopages.org': 'gorodche.ru',
 'klerk-ru.turbopages.org': 'klerk.ru',
 'zensovet-ru.turbopages.org': 'zensovet.ru',
 'vmyatynnet-ru.turbopages.org': 'vmyatynnet.ru',
 'sto--tolyatti-ru.turbopages.org': 'sto-tolyatti.ru',
 'nnov-kp-ru.turbopages.org': 'nnov.kp.ru',
 'nn-aif-ru.turbopages.org': 'nn.aif.ru',
 'morphologyonline-ru.turbopages.org': 'morphologyonline.ru',
 'vremyan-ru.turbopages.org': 'vremyan.ru',
 'yaznay-ru.turbopages.org': 'yaznay.ru',
 'simpleslim-ru.turbopages.org': 'simpleslim.ru',
 'fitnessera-ru.turbopages.org': 'fitnessera.ru',
 'vashynogi-com.turbopages.org': 'vashynogi.com',
 'womfit-com.turbopages.org': 'womfit.com',
 'ignes--fatui-ru.turbopages.org': 'ignes-fatui.ru',
 'alkotraz-ru.turbopages.org': 'alkotraz.ru',
 'tonustela-net.turbopages.org': 'tonustela.net',
 'ural-kp-ru.turbopages.org': 'ural.kp.ru',
 'super-ru.turbopages.org': 'super.ru',
 'artspecter-ru.turbopages.org': 'artspecter.ru',
 'inttershop-com.turbopages.org': 'inttershop.com',
 'sen--semilia-livejournal-com.turbopages.org': 'sen-semilia.livejournal.com',
 'chto--proishodit-ru.turbopages.org': 'chto-proishodit.ru',
 'hozpedia-ru.turbopages.org': 'hozpedia.ru',
 'harmfulgrumpy-livejournal-com.turbopages.org': 'harmfulgrumpy.livejournal.com',
 'dubikvit-livejournal-com.turbopages.org': 'dubikvit.livejournal.com',
 'spetsialny-livejournal-com.turbopages.org': 'spetsialny.livejournal.com',
 'sympaty-net.turbopages.org': 'sympaty.net',
 'aleksei--turchin-livejournal-com.turbopages.org': 'aleksei-turchin.livejournal.com',
 '1rnd-ru.turbopages.org': '1rnd.ru',
 'eaomedia-ru.turbopages.org': 'eaomedia.ru',
 'kuban-aif-ru.turbopages.org': 'kuban.aif.ru',
 'donday-ru.turbopages.org': 'donday.ru',
 'ro-today.turbopages.org': 'ro.today',
 'privet--rostov-ru.turbopages.org': 'privet-rostov.ru',
 'sportkp-ru.turbopages.org': 'sportkp.ru',
 'fomag-ru.turbopages.org': 'fomag.ru',
 'health-7days-ru.turbopages.org': 'health.7days.ru',
 'zagge-ru.turbopages.org': 'zagge.ru',
 'hightech-fm.turbopages.org': 'hightech.fm',
 'vesiskitim-ru.turbopages.org': 'vesiskitim.ru',
 'rifey-ru.turbopages.org': 'rifey.ru',
 'turisti-guru.turbopages.org': 'turisti.guru',
 'mosregion-info.turbopages.org': 'mosregion.info',
 'rus--kostroma-ru.turbopages.org': 'rus-kostroma.ru',
 'dom--pchel-ru.turbopages.org': 'dom-pchel.ru',
 'yar-mk-ru.turbopages.org': 'yar.mk.ru',
 'aqa-ru.turbopages.org': 'aqa.ru',
 'yabloki-site.turbopages.org': 'yabloki.site',
 'sadsezon-com.turbopages.org': 'sadsezon.com',
 'okna--dom-net.turbopages.org': 'okna-dom.net',
 'ivanovonews-ru.turbopages.org': 'ivanovonews.ru',
 '7x7--journal-ru.turbopages.org': '7x7-journal.ru',
 'mikle1-livejournal-com.turbopages.org': 'mikle1.livejournal.com',
 'riamobalashiha-ru.turbopages.org': 'riamobalashiha.ru',
 'ocomp-info.turbopages.org': 'ocomp.info',
 'igor--mikhaylin-livejournal-com.turbopages.org': 'igor-mikhaylin.livejournal.com',
 'elif--kose-livejournal-com.turbopages.org': 'elif-kose.livejournal.com',
 'autoexpert174-ru.turbopages.org': 'autoexpert174.ru',
 'army--today-ru.turbopages.org': 'army-today.ru',
 'cyprusbutterfly-com-cy.turbopages.org': 'cyprusbutterfly.com.cy',
 'centro--pol-ru.turbopages.org': 'centro-pol.ru',
 'pensermen-ru.turbopages.org': 'pensermen.ru',
 'kadara-ru.turbopages.org': 'kadara.ru',
 'tumen-bezformata-com.turbopages.org': 'tumen.bezformata.com',
 'macnoob-ru.turbopages.org': 'macnoob.ru',
 'yablyk-com.turbopages.org': 'yablyk.com',
 'germania-one.turbopages.org': 'germania.one',
 'instagramis-ru.turbopages.org': 'instagramis.ru',
 '76-ru.turbopages.org': '76.ru',
 'liveinorthodoxy-com.turbopages.org': 'liveinorthodoxy.com',
 'smart--lab-ru.turbopages.org': 'smart-lab.ru',
 'driver--news-ru.turbopages.org': 'driver-news.ru',
 'lvrach-ru.turbopages.org': 'lvrach.ru',
 'natpopova-livejournal-com.turbopages.org': 'natpopova.livejournal.com',
 'noginashi-ru.turbopages.org': 'noginashi.ru',
 'womanhit-ru.turbopages.org': 'womanhit.ru',
 'dombusin-livejournal-com.turbopages.org': 'dombusin.livejournal.com',
 'ivethelp-ru.turbopages.org': 'ivethelp.ru',
 'mskgazeta-ru.turbopages.org': 'mskgazeta.ru',
 'kotiko-ru.turbopages.org': 'kotiko.ru',
 'alanspb-ru.turbopages.org': 'alanspb.ru',
 'buhsoft-ru.turbopages.org': 'buhsoft.ru',
 'strop--snab-ru.turbopages.org': 'strop-snab.ru',
 'karelia-news.turbopages.org': 'karelia.news',
 'ptzgovorit-ru.turbopages.org': 'ptzgovorit.ru',
 'alexey--228-livejournal-com.turbopages.org': 'alexey-228.livejournal.com',
 'karelinform-ru.turbopages.org': 'karelinform.ru',
 'kdelo-ru.turbopages.org': 'kdelo.ru',
 'me--frai-livejournal-com.turbopages.org': 'me-frai.livejournal.com',
 'gazeta-spb-ru.turbopages.org': 'gazeta.spb.ru',
 'svetan--56-livejournal-com.turbopages.org': 'svetan-56.livejournal.com',
 'asutpp-ru.turbopages.org': 'asutpp.ru',
 'pobeda26-ru.turbopages.org': 'pobeda26.ru',
 'rk-karelia-ru.turbopages.org': 'rk.karelia.ru',
 'avtoguru-net.turbopages.org': 'avtoguru.net',
 'karel-mk-ru.turbopages.org': 'karel.mk.ru',
 'sportrg-ru.turbopages.org': 'sportrg.ru',
 'rnbee-ru.turbopages.org': 'rnbee.ru',
 'banki-ru.turbopages.org': 'banki.ru',
 'tophookah-info.turbopages.org': 'tophookah.info',
 'ru--sku-livejournal-com.turbopages.org': 'ru-sku.livejournal.com',
 'pro--kurenie-info.turbopages.org': 'pro-kurenie.info',
 'tvoimesyachnye-ru.turbopages.org': 'tvoimesyachnye.ru',
 'gadgetstyle-com-ua.turbopages.org': 'gadgetstyle.com.ua',
 'stroyka--gid-ru.turbopages.org': 'stroyka-gid.ru',
 'weddingfm-ru.turbopages.org': 'weddingfm.ru',
 'elitkot-ru.turbopages.org': 'elitkot.ru',
 'ophthalmocenter-ru.turbopages.org': 'ophthalmocenter.ru',
 'zaks-ru.turbopages.org': 'zaks.ru',
 '12--kanal-ru.turbopages.org': '12-kanal.ru',
 'omsk-kp-ru.turbopages.org': 'omsk.kp.ru',
 'omskinform-ru.turbopages.org': 'omskinform.ru',
 'masse--dzr-ru.turbopages.org': 'masse-dzr.ru',
 'bankfax-ru.turbopages.org': 'bankfax.ru',
 'notabene-info.turbopages.org': 'notabene.info',
 'moe--online-ru.turbopages.org': 'moe-online.ru',
 'sibdepo-ru.turbopages.org': 'sibdepo.ru',
 'knews-kg.turbopages.org': 'knews.kg',
 'vashgorod-ru.turbopages.org': 'vashgorod.ru',
 'fengshui24-ru.turbopages.org': 'fengshui24.ru',
 'devushka-ru.turbopages.org': 'devushka.ru',
 'nacion-ru.turbopages.org': 'nacion.ru',
 'avto--lover-ru.turbopages.org': 'avto-lover.ru',
 'nashasemja-ru.turbopages.org': 'nashasemja.ru',
 'day-ru.turbopages.org': 'day.ru',
 'eva-ru.turbopages.org': 'eva.ru',
 'tiensmed-ru.turbopages.org': 'tiensmed.ru',
 'givnost-ru.turbopages.org': 'givnost.ru',
 'o--prirode-ru.turbopages.org': 'o-prirode.ru',
 'medeponim-ru.turbopages.org': 'medeponim.ru',
 'kuban-mk-ru.turbopages.org': 'kuban.mk.ru',
 'sportmk-ru.turbopages.org': 'sportmk.ru',
 'parrotsworld-ru.turbopages.org': 'parrotsworld.ru',
 'realzarabotok-com.turbopages.org': 'realzarabotok.com',
 'furnishhome-ru.turbopages.org': 'furnishhome.ru',
 'quto-ru.turbopages.org': 'quto.ru',
 '101hairtips-com.turbopages.org': '101hairtips.com',
 'androidsfaq-com.turbopages.org': 'androidsfaq.com',
 'pantv-livejournal-com.turbopages.org': 'pantv.livejournal.com',
 'chr-aif-ru.turbopages.org': 'chr.aif.ru',
 'pipla-ru.turbopages.org': 'pipla.ru',
 'finecooking-ru.turbopages.org': 'finecooking.ru',
 'cpykami-ru.turbopages.org': 'cpykami.ru',
 'ogorodum-ru.turbopages.org': 'ogorodum.ru',
 'zdorovakrasiva-com.turbopages.org': 'zdorovakrasiva.com',
 'obzor174-ru.turbopages.org': 'obzor174.ru',
 'karel-aif-ru.turbopages.org': 'karel.aif.ru',
 'beautyexpert-pro.turbopages.org': 'beautyexpert.pro',
 'dv-kp-ru.turbopages.org': 'dv.kp.ru',
 'vl-aif-ru.turbopages.org': 'vl.aif.ru',
 'hab-aif-ru.turbopages.org': 'hab.aif.ru',
 'nalivay--ka-ru.turbopages.org': 'nalivay-ka.ru',
 'goferma-ru.turbopages.org': 'goferma.ru',
 'zoolandy-ru.turbopages.org': 'zoolandy.ru',
 'bimru-ru.turbopages.org': 'bimru.ru',
 'hab-mk-ru.turbopages.org': 'hab.mk.ru',
 '101-ru.turbopages.org': '101.ru',
 'pozhmashina-ru.turbopages.org': 'pozhmashina.ru',
 'baikalhangkai-ru.turbopages.org': 'baikalhangkai.ru',
 'ivbg-ru.turbopages.org': 'ivbg.ru',
 'wordpresslab-ru.turbopages.org': 'wordpresslab.ru',
 'koreec73-ru.turbopages.org': 'koreec73.ru',
 'hozsekretiki-ru.turbopages.org': 'hozsekretiki.ru',
 'domosite-ru.turbopages.org': 'domosite.ru',
 'setafi-com.turbopages.org': 'setafi.com',
 'femida--ksenia-livejournal-com.turbopages.org': 'femida-ksenia.livejournal.com',
 'furnilux-ru.turbopages.org': 'furnilux.ru',
 'mrkot-com.turbopages.org': 'mrkot.com',
 'blondinchen-livejournal-com.turbopages.org': 'blondinchen.livejournal.com',
 'victorrom-livejournal-com.turbopages.org': 'victorrom.livejournal.com',
 'charsov-livejournal-com.turbopages.org': 'charsov.livejournal.com',
 'varganshik-livejournal-com.turbopages.org': 'varganshik.livejournal.com',
    'forum.qrz.ru': 'qrz.ru',
 'm.qrz.ru': 'qrz.ru',
 'rx4hx.qrz.ru': 'qrz.ru',
 'radiokrot.qrz.ru': 'qrz.ru',
 'ur4nww.qrz.ru': 'qrz.ru',
 'rd7t.qrz.ru': 'qrz.ru',
 'qrz.ru': 'qrz.ru',
 'goryham.qrz.ru': 'qrz.ru',
 'ra4cbh.qrz.ru': 'qrz.ru',
 'ra3ggi.qrz.ru': 'qrz.ru',
 'hamradio.qrz.ru': 'qrz.ru',
 'ru3ga.qrz.ru': 'qrz.ru',
 'rfanat.qrz.ru': 'qrz.ru',
 'ua4lou.qrz.ru': 'qrz.ru',
 'ua6hjq.qrz.ru': 'qrz.ru',
    'mirstroek.ru': 'mirstroek.ru',
 'samara.mirstroek.ru': 'mirstroek.ru',
 'moscow.mirstroek.ru': 'mirstroek.ru',
 'spb.mirstroek.ru': 'mirstroek.ru',
 'chita.mirstroek.ru': 'mirstroek.ru',
 'krasnodar.mirstroek.ru': 'mirstroek.ru',
 'belarus.mirstroek.ru': 'mirstroek.ru',
 'kazakhstan.mirstroek.ru': 'mirstroek.ru',
 'ekaterinburg.mirstroek.ru': 'mirstroek.ru',
 'nizhniynovgorod.mirstroek.ru': 'mirstroek.ru',
 'novosibirsk.mirstroek.ru': 'mirstroek.ru',
 'rostovnadonu.mirstroek.ru': 'mirstroek.ru',
 'nabchelny.mirstroek.ru': 'mirstroek.ru',
 'tomsk.mirstroek.ru': 'mirstroek.ru',
 'cheboksary.mirstroek.ru': 'mirstroek.ru',
    'or.stro18.ru': 'stro18.ru',
 'chelny.stro18.ru': 'stro18.ru',
 'stro18.ru': 'stro18.ru',
 'biisk.stro18.ru': 'stro18.ru',
 'kurgan.stro18.ru': 'stro18.ru',
 'tomsk.stro18.ru': 'stro18.ru',
 'tolyatti.stro18.ru': 'stro18.ru',
 'perm.stro18.ru': 'stro18.ru',
 'tambov.stro18.ru': 'stro18.ru',
 'salavat.stro18.ru': 'stro18.ru',
 'novokuzneck.stro18.ru': 'stro18.ru',
 'nijnevartovsk.stro18.ru': 'stro18.ru',
 'saransk.stro18.ru': 'stro18.ru',
 'tagil.stro18.ru': 'stro18.ru',
 'lipetsk.stro18.ru': 'stro18.ru',
    'gmn44.edu-penza.ru': 'edu-penza.ru',
 'school56.edu-penza.ru': 'edu-penza.ru',
 'ppms.edu-penza.ru': 'edu-penza.ru',
 'pro-26.edu-penza.ru': 'edu-penza.ru',
 'school51.edu-penza.ru': 'edu-penza.ru',
 'zassputnik.edu-penza.ru': 'edu-penza.ru',
 'sch222.edu-penza.ru': 'edu-penza.ru',
 'zemlic.edu-penza.ru': 'edu-penza.ru',
 'pro-57.edu-penza.ru': 'edu-penza.ru',
 'gymnasium42.edu-penza.ru': 'edu-penza.ru',
 'school31.edu-penza.ru': 'edu-penza.ru',
 'dsad99.edu-penza.ru': 'edu-penza.ru',
 'edu-penza.ru': 'edu-penza.ru',
 'school68.edu-penza.ru': 'edu-penza.ru',
 'gnomik124.edu-penza.ru': 'edu-penza.ru',
    'samgtu.ru': 'samgtu.ru',
 'toop.samgtu.ru': 'samgtu.ru',
 'toe.samgtu.ru': 'samgtu.ru',
 'uup.samgtu.ru': 'samgtu.ru',
 'studentlife.samgtu.ru': 'samgtu.ru',
 'economic.samgtu.ru': 'samgtu.ru',
 'oldsamara.samgtu.ru': 'samgtu.ru',
 'ido.samgtu.ru': 'samgtu.ru',
 'nf.samgtu.ru': 'samgtu.ru',
 'filippov.samgtu.ru': 'samgtu.ru',
 'bngs.samgtu.ru': 'samgtu.ru',
 'organic.samgtu.ru': 'samgtu.ru',
 'd21221704.samgtu.ru': 'samgtu.ru',
 'pmanag.samgtu.ru': 'samgtu.ru',
 'es.samgtu.ru': 'samgtu.ru',
    'ekb.rosfirm.ru': 'rosfirm.ru',
 'spb.rosfirm.ru': 'rosfirm.ru',
 'krasnodar.rosfirm.ru': 'rosfirm.ru',
 'cheboksary.rosfirm.ru': 'rosfirm.ru',
 'izhevsk.rosfirm.ru': 'rosfirm.ru',
 'voronezh.rosfirm.ru': 'rosfirm.ru',
 'samara.rosfirm.ru': 'rosfirm.ru',
 'moskva.rosfirm.ru': 'rosfirm.ru',
 'blagoveschensk.rosfirm.ru': 'rosfirm.ru',
 'yaroslavl.rosfirm.ru': 'rosfirm.ru',
 'novgorod.rosfirm.ru': 'rosfirm.ru',
 'kazan.rosfirm.ru': 'rosfirm.ru',
 'ip-karasov-sp.rosfirm.ru': 'rosfirm.ru',
 'tula.rosfirm.ru': 'rosfirm.ru',
 'chelyabinsk.rosfirm.ru': 'rosfirm.ru',
    'tmod.lordfilms.cfd': 'lordfilms.cfd',
 'moi.lordfilms.cfd': 'lordfilms.cfd',
 'bro.lordfilms.cfd': 'lordfilms.cfd',
 'try4.lordfilms.cfd': 'lordfilms.cfd',
 'udf2.lordfilms.cfd': 'lordfilms.cfd',
 'url.lordfilms.cfd': 'lordfilms.cfd',
 'zaz9.lordfilms.cfd': 'lordfilms.cfd',
 'bbot.lordfilms.cfd': 'lordfilms.cfd',
 'vrt2.lordfilms.cfd': 'lordfilms.cfd',
 'dot6.lordfilms.cfd': 'lordfilms.cfd',
 'gui.lordfilms.cfd': 'lordfilms.cfd',
 'ree1.lordfilms.cfd': 'lordfilms.cfd',
 'maz2.lordfilms.cfd': 'lordfilms.cfd',
 'tru2.lordfilms.cfd': 'lordfilms.cfd',
 'urz1.lordfilms.cfd': 'lordfilms.cfd',
    'news.ivest.kz': 'ivest.kz',
 'shushinsk.b2b.ivest.kz': 'ivest.kz',
 'kostanay.b2b.ivest.kz': 'ivest.kz',
 'stepnogorsk.b2b.ivest.kz': 'ivest.kz',
 'petropavlovsk.b2b.ivest.kz': 'ivest.kz',
 'omsk.b2b.ivest.kz': 'ivest.kz',
 'ivest.kz': 'ivest.kz',
 'almaty.b2b.ivest.kz': 'ivest.kz',
 'pavlodar.b2b.ivest.kz': 'ivest.kz',
 'nur-sultan.b2b.ivest.kz': 'ivest.kz',
 'kokshetau.b2b.ivest.kz': 'ivest.kz',
 'culinar.ivest.kz': 'ivest.kz',
 'b2b.ivest.kz': 'ivest.kz',
 'bti.b2b.ivest.kz': 'ivest.kz',
 'balakov.b2b.ivest.kz': 'ivest.kz',
    'msk.dixinews.ru': 'dixinews.ru',
 'spb.dixinews.ru': 'dixinews.ru',
 'omsk.dixinews.ru': 'dixinews.ru',
 'dixinews.ru': 'dixinews.ru',
 'chaik.dixinews.ru': 'dixinews.ru',
 'vrn.dixinews.ru': 'dixinews.ru',
 'chel.dixinews.ru': 'dixinews.ru',
 'rostov.dixinews.ru': 'dixinews.ru',
 'nsk.dixinews.ru': 'dixinews.ru',
 'ufa.dixinews.ru': 'dixinews.ru',
 'samara.dixinews.ru': 'dixinews.ru',
 'krsk.dixinews.ru': 'dixinews.ru',
 'kazan.dixinews.ru': 'dixinews.ru',
 'perm.dixinews.ru': 'dixinews.ru',
 'ekb.dixinews.ru': 'dixinews.ru',
    'lsk.omsu-nnov.ru': 'omsu-nnov.ru',
 'sergach.omsu-nnov.ru': 'omsu-nnov.ru',
 'arz.omsu-nnov.ru': 'omsu-nnov.ru',
 'tns.omsu-nnov.ru': 'omsu-nnov.ru',
 'volodarsk.omsu-nnov.ru': 'omsu-nnov.ru',
 'sokolskoe.omsu-nnov.ru': 'omsu-nnov.ru',
 'lukojanov.omsu-nnov.ru': 'omsu-nnov.ru',
 'pilna.omsu-nnov.ru': 'omsu-nnov.ru',
 'vorotynec.omsu-nnov.ru': 'omsu-nnov.ru',
 'vetluga.omsu-nnov.ru': 'omsu-nnov.ru',
 'navashino.omsu-nnov.ru': 'omsu-nnov.ru',
 'kro.omsu-nnov.ru': 'omsu-nnov.ru',
 'admbbl.omsu-nnov.ru': 'omsu-nnov.ru',
 'spasskoe.omsu-nnov.ru': 'omsu-nnov.ru',
 'gorod.lsk.omsu-nnov.ru': 'omsu-nnov.ru',
    'hd1.2lordserials.xyz': '2lordserials.xyz',
 'hdi.2lordserials.xyz': '2lordserials.xyz',
 'hd12.2lordserials.xyz': '2lordserials.xyz',
 'hd13.2lordserials.xyz': '2lordserials.xyz',
 'hd4.2lordserials.xyz': '2lordserials.xyz',
 'hd7.2lordserials.xyz': '2lordserials.xyz',
 'hd6.2lordserials.xyz': '2lordserials.xyz',
 'hd3.2lordserials.xyz': '2lordserials.xyz',
 'hd2.2lordserials.xyz': '2lordserials.xyz',
 'hd9.2lordserials.xyz': '2lordserials.xyz',
 'hd15.2lordserials.xyz': '2lordserials.xyz',
 'hd5.2lordserials.xyz': '2lordserials.xyz',
 'hd8.2lordserials.xyz': '2lordserials.xyz',
 'hd10.2lordserials.xyz': '2lordserials.xyz',
 'hd14.2lordserials.xyz': '2lordserials.xyz',
    'ad.adriver.ru': 'adriver.ru',
 'edp2.adriver.ru': 'adriver.ru',
 'servers7.adriver.ru': 'adriver.ru',
 'servers6.adriver.ru': 'adriver.ru',
 'mh8.adriver.ru': 'adriver.ru',
 'masterh7.adriver.ru': 'adriver.ru',
 'mh6.adriver.ru': 'adriver.ru',
 'servers3.adriver.ru': 'adriver.ru',
 'servers1.adriver.ru': 'adriver.ru',
 'servers4.adriver.ru': 'adriver.ru',
 'servers2.adriver.ru': 'adriver.ru',
 'masterh5.adriver.ru': 'adriver.ru',
 'web.adriver.ru': 'adriver.ru',
 'adriver.ru': 'adriver.ru',
 'sib2.adriver.ru': 'adriver.ru',
    'm.en.lyrsense.com': 'lyrsense.com',
 'forum.lyrsense.com': 'lyrsense.com',
 'en.lyrsense.com': 'lyrsense.com',
 'm.de.lyrsense.com': 'lyrsense.com',
 'm.es.lyrsense.com': 'lyrsense.com',
 'm.soundtrack.lyrsense.com': 'lyrsense.com',
 'm.fr.lyrsense.com': 'lyrsense.com',
 'm.it.lyrsense.com': 'lyrsense.com',
 'de.lyrsense.com': 'lyrsense.com',
 'lyrsense.com': 'lyrsense.com',
 'm.gb.lyrsense.com': 'lyrsense.com',
 'es.lyrsense.com': 'lyrsense.com',
 'fr.lyrsense.com': 'lyrsense.com',
 'it.lyrsense.com': 'lyrsense.com',
 'gb.lyrsense.com': 'lyrsense.com',
    'tomsk.mestarf.su': 'mestarf.su',
 'samara.mestarf.su': 'mestarf.su',
 'velikie-luki.mestarf.su': 'mestarf.su',
 'sankt-peterburg.mestarf.su': 'mestarf.su',
 'dmitrov.mestarf.su': 'mestarf.su',
 'ufa.mestarf.su': 'mestarf.su',
 'mestarf.su': 'mestarf.su',
 'habarovsk.mestarf.su': 'mestarf.su',
 'krasnodar.mestarf.su': 'mestarf.su',
 'novosibirsk.mestarf.su': 'mestarf.su',
 'saratov.mestarf.su': 'mestarf.su',
 'tyumeny.mestarf.su': 'mestarf.su',
 'petrozavodsk.mestarf.su': 'mestarf.su',
 'nijnyaya-tura.mestarf.su': 'mestarf.su',
 'biysk.mestarf.su': 'mestarf.su',
    'medikateka.ru': 'medikateka.ru',
 'other.medikateka.ru': 'medikateka.ru',
 'spb.medikateka.ru': 'medikateka.ru',
 'perm.medikateka.ru': 'medikateka.ru',
 'ekaterinburg.medikateka.ru': 'medikateka.ru',
 'khabarovsk.medikateka.ru': 'medikateka.ru',
 'kazan.medikateka.ru': 'medikateka.ru',
 'tyumen.medikateka.ru': 'medikateka.ru',
 'omsk.medikateka.ru': 'medikateka.ru',
 'ufa.medikateka.ru': 'medikateka.ru',
 'samara.medikateka.ru': 'medikateka.ru',
 'kaluga.medikateka.ru': 'medikateka.ru',
 'rostov.medikateka.ru': 'medikateka.ru',
 'surgut.medikateka.ru': 'medikateka.ru',
 'nnovgorod.medikateka.ru': 'medikateka.ru',
    'zarinsk.2br.ru': '2br.ru',
 'kumertau.2br.ru': '2br.ru',
 'uhta.2br.ru': '2br.ru',
 'buzuluk.2br.ru': '2br.ru',
 'tujmazy.2br.ru': '2br.ru',
 'vorkuta.2br.ru': '2br.ru',
 'belebej.2br.ru': '2br.ru',
 'bugulma.2br.ru': '2br.ru',
 'chistopol.2br.ru': '2br.ru',
 'ahtubinsk.2br.ru': '2br.ru',
 'chajkovskij.2br.ru': '2br.ru',
 'leninogorsk.2br.ru': '2br.ru',
 'iskitim.2br.ru': '2br.ru',
 'tihoreck.2br.ru': '2br.ru',
 'ishim.2br.ru': '2br.ru',
    'club.cnews.ru': 'cnews.ru',
 'market.cnews.ru': 'cnews.ru',
 'zoom.cnews.ru': 'cnews.ru',
 'innovations.cnews.ru': 'cnews.ru',
 'cnews.ru': 'cnews.ru',
 'events.cnews.ru': 'cnews.ru',
 'safe.cnews.ru': 'cnews.ru',
 'corp.cnews.ru': 'cnews.ru',
 'internet.cnews.ru': 'cnews.ru',
 'ai.cnews.ru': 'cnews.ru',
 'retail.cnews.ru': 'cnews.ru',
 'telecom.cnews.ru': 'cnews.ru',
 'biz.cnews.ru': 'cnews.ru',
 'gov.cnews.ru': 'cnews.ru',
 'mobile.cnews.ru': 'cnews.ru',
    'ulan-ude.motolife24.ru': 'motolife24.ru',
 'motolife24.ru': 'motolife24.ru',
 'spb.motolife24.ru': 'motolife24.ru',
 'cherepovets.motolife24.ru': 'motolife24.ru',
 'vladimir.motolife24.ru': 'motolife24.ru',
 'rostov.motolife24.ru': 'motolife24.ru',
 'novosibirsk.motolife24.ru': 'motolife24.ru',
 'msk.motolife24.ru': 'motolife24.ru',
 'voronezh.motolife24.ru': 'motolife24.ru',
 'chelyabinsk.motolife24.ru': 'motolife24.ru',
 'surgut.motolife24.ru': 'motolife24.ru',
 'sochi.motolife24.ru': 'motolife24.ru',
 'perm.motolife24.ru': 'motolife24.ru',
 'omsk.motolife24.ru': 'motolife24.ru',
 'novorossiysk.motolife24.ru': 'motolife24.ru',
    'ouuo.ru': 'ouuo.ru',
 'ijevsk.ouuo.ru': 'ouuo.ru',
 'perm.ouuo.ru': 'ouuo.ru',
 'novosibirsk.ouuo.ru': 'ouuo.ru',
 'moskva.ouuo.ru': 'ouuo.ru',
 'velikiy-novgorod.ouuo.ru': 'ouuo.ru',
 'stavropol.ouuo.ru': 'ouuo.ru',
 'voronej.ouuo.ru': 'ouuo.ru',
 'tolyatti.ouuo.ru': 'ouuo.ru',
 'rostov-na-donu.ouuo.ru': 'ouuo.ru',
 'sankt-peterburg.ouuo.ru': 'ouuo.ru',
 'pgt-promyshlennaya.ouuo.ru': 'ouuo.ru',
 'stanica-kanevskaya.ouuo.ru': 'ouuo.ru',
 'ekaterinburg.ouuo.ru': 'ouuo.ru',
 'samara.ouuo.ru': 'ouuo.ru',
    'samara.siss.ru': 'siss.ru',
 'siss.ru': 'siss.ru',
 'tyumen.siss.ru': 'siss.ru',
 'nsk.siss.ru': 'siss.ru',
 'novokuzneck.siss.ru': 'siss.ru',
 'hanty-mansiysk.siss.ru': 'siss.ru',
 'krd.siss.ru': 'siss.ru',
 'perm.siss.ru': 'siss.ru',
 'kazan.siss.ru': 'siss.ru',
 'ekaterinburg.siss.ru': 'siss.ru',
 'barnaul.siss.ru': 'siss.ru',
 'msk.siss.ru': 'siss.ru',
 'yoshkar-ola.siss.ru': 'siss.ru',
 'tomsk.siss.ru': 'siss.ru',
 'saratov.siss.ru': 'siss.ru',
    'dyatkovo.sushi-foods.com': 'sushi-foods.com',
 'sushi-foods.com': 'sushi-foods.com',
 'bologoe.sushi-foods.com': 'sushi-foods.com',
 'volochek.sushi-foods.com': 'sushi-foods.com',
 'borovichi.sushi-foods.com': 'sushi-foods.com',
 'lyudinovo.sushi-foods.com': 'sushi-foods.com',
 'kirov.sushi-foods.com': 'sushi-foods.com',
 'nerehta.sushi-foods.com': 'sushi-foods.com',
 'buy.sushi-foods.com': 'sushi-foods.com',
 'ozerniy.sushi-foods.com': 'sushi-foods.com',
 'krasnokamensk.sushi-foods.com': 'sushi-foods.com',
 'torzhok.sushi-foods.com': 'sushi-foods.com',
 'sokol.sushi-foods.com': 'sushi-foods.com',
 'klincy.sushi-foods.com': 'sushi-foods.com',
 'rzhev.sushi-foods.com': 'sushi-foods.com',
    'spb.bestru.ru': 'bestru.ru',
 'ekaterinburg.bestru.ru': 'bestru.ru',
 'bestru.ru': 'bestru.ru',
 'ufa.bestru.ru': 'bestru.ru',
 'belgorod.bestru.ru': 'bestru.ru',
 'tyumen.bestru.ru': 'bestru.ru',
 'kirov.bestru.ru': 'bestru.ru',
 'krasnodar.bestru.ru': 'bestru.ru',
 'novosibirsk.bestru.ru': 'bestru.ru',
 'voronezh.bestru.ru': 'bestru.ru',
 'volgograd.bestru.ru': 'bestru.ru',
 'kazan.bestru.ru': 'bestru.ru',
 'habarovsk.bestru.ru': 'bestru.ru',
 'rostovnadonu.bestru.ru': 'bestru.ru',
 'omsk.bestru.ru': 'bestru.ru',
 'kaluga.bestru.ru': 'bestru.ru',
    'ulyanovsk.dibi.ru': 'dibi.ru',
 'dibi.ru': 'dibi.ru',
 'kazan-116.dibi.ru': 'dibi.ru',
 'saratov.dibi.ru': 'dibi.ru',
 'podolsk.dibi.ru': 'dibi.ru',
 'severodvinsk.dibi.ru': 'dibi.ru',
 'sankt-peterburg.dibi.ru': 'dibi.ru',
 'tolyatti.dibi.ru': 'dibi.ru',
 'novosibirsk.dibi.ru': 'dibi.ru',
 'magnitogorsk.dibi.ru': 'dibi.ru',
 'penza.dibi.ru': 'dibi.ru',
 'cheboksary.dibi.ru': 'dibi.ru',
 'samara.dibi.ru': 'dibi.ru',
 'tver.dibi.ru': 'dibi.ru',
 'kamensk-uralskij.dibi.ru': 'dibi.ru',
 'smolensk.dibi.ru': 'dibi.ru',
    'ppschel.eps74.ru': 'eps74.ru',
 'lenadmin74.eps74.ru': 'eps74.ru',
 'uszn65.eps74.ru': 'eps74.ru',
 'uszn46.eps74.ru': 'eps74.ru',
 'uszn10.eps74.ru': 'eps74.ru',
 'troitsk-rayon.eps74.ru': 'eps74.ru',
 'sosnovka.eps74.ru': 'eps74.ru',
 'u-uralsk.eps74.ru': 'eps74.ru',
 'uszn32.eps74.ru': 'eps74.ru',
 'uszn43.eps74.ru': 'eps74.ru',
 'zags-ch-m.eps74.ru': 'eps74.ru',
 'kcso15.eps74.ru': 'eps74.ru',
 'kcso39.eps74.ru': 'eps74.ru',
 'okt74.eps74.ru': 'eps74.ru',
 'varnaroo.eps74.ru': 'eps74.ru',
 'mbuz-ds6.eps74.ru': 'eps74.ru',
    '2022-17.lordfilm0.com': 'lordfilm0.com',
 '2022-18.lordfilm0.com': 'lordfilm0.com',
 '2022-122.lordfilm0.com': 'lordfilm0.com',
 '2022-77.lordfilm0.com': 'lordfilm0.com',
 '5-online.lordfilm0.com': 'lordfilm0.com',
 '2022-12.lordfilm0.com': 'lordfilm0.com',
 '2022-88.lordfilm0.com': 'lordfilm0.com',
 '3-hd.lordfilm0.com': 'lordfilm0.com',
 '2022-film.lordfilm0.com': 'lordfilm0.com',
 '2022-555.lordfilm0.com': 'lordfilm0.com',
 '2022.lordfilm0.com': 'lordfilm0.com',
 '2022-666.lordfilm0.com': 'lordfilm0.com',
 '2022-online.lordfilm0.com': 'lordfilm0.com',
 '2022-13.lordfilm0.com': 'lordfilm0.com',
 '2022-11.lordfilm0.com': 'lordfilm0.com',
 '2022-hd.lordfilm0.com': 'lordfilm0.com',
    'autoneva.ru': 'autoneva.ru',
 'krim.autoneva.ru': 'autoneva.ru',
 'kuban.autoneva.ru': 'autoneva.ru',
 'nsk.autoneva.ru': 'autoneva.ru',
 'otzyvy.autoneva.ru': 'autoneva.ru',
 'msk.autoneva.ru': 'autoneva.ru',
 'lipetsk.autoneva.ru': 'autoneva.ru',
 'kirov.autoneva.ru': 'autoneva.ru',
 'perm.autoneva.ru': 'autoneva.ru',
 'vrn.autoneva.ru': 'autoneva.ru',
 'belgorod.autoneva.ru': 'autoneva.ru',
 'omsk.autoneva.ru': 'autoneva.ru',
 'altai.autoneva.ru': 'autoneva.ru',
 'ivanovo.autoneva.ru': 'autoneva.ru',
 'vladikavkaz.autoneva.ru': 'autoneva.ru',
 'spb.autoneva.ru': 'autoneva.ru',
    'minzdravrso.ru': 'minzdravrso.ru',
 'dp3.minzdravrso.ru': 'minzdravrso.ru',
 'dp2.minzdravrso.ru': 'minzdravrso.ru',
 'p7.minzdravrso.ru': 'minzdravrso.ru',
 'rdkb.minzdravrso.ru': 'minzdravrso.ru',
 'rod.minzdravrso.ru': 'minzdravrso.ru',
 'rkb.minzdravrso.ru': 'minzdravrso.ru',
 'dcrb.minzdravrso.ru': 'minzdravrso.ru',
 'p4.minzdravrso.ru': 'minzdravrso.ru',
 'rmiac.minzdravrso.ru': 'minzdravrso.ru',
 'rkvd.minzdravrso.ru': 'minzdravrso.ru',
 'dp4.minzdravrso.ru': 'minzdravrso.ru',
 'rnd.minzdravrso.ru': 'minzdravrso.ru',
 'rcozsir.minzdravrso.ru': 'minzdravrso.ru',
 'rsdrf.minzdravrso.ru': 'minzdravrso.ru',
 'rptd.minzdravrso.ru': 'minzdravrso.ru',
    'xn--90aee6admdx.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--80aacf4bwnk3a.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--b1afchn5b.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--80a1bd.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--80atblfjdfd2l.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn----7sbeiia6axumbcqds.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--80agatlhjjbulh.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--90absbknhbvge.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--90ahkico2a6b9d.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--90aode8a.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--80aab6birx.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--80adxhks.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--j1adfn.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn----8sbavnmfmfe8af1c2h.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
 'xn--b1afaslnbn.xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai': 'xn-----6kcabofibag8dqarfelguc4ag84a.xn--p1ai',
    'sudak.reklama-crimea.com': 'reklama-crimea.com',
 'feodosia.reklama-crimea.com': 'reklama-crimea.com',
 'reklama-crimea.com': 'reklama-crimea.com',
 'shelkino.reklama-crimea.com': 'reklama-crimea.com',
 'belogorsk.reklama-crimea.com': 'reklama-crimea.com',
 'simferopol.reklama-crimea.com': 'reklama-crimea.com',
 'kerch.reklama-crimea.com': 'reklama-crimea.com',
 'koktebel.reklama-crimea.com': 'reklama-crimea.com',
 'yalta.reklama-crimea.com': 'reklama-crimea.com',
 'gurzuf.reklama-crimea.com': 'reklama-crimea.com',
 'evpatoria.reklama-crimea.com': 'reklama-crimea.com',
 'partenit.reklama-crimea.com': 'reklama-crimea.com',
 'alushta.reklama-crimea.com': 'reklama-crimea.com',
 'djankoy.reklama-crimea.com': 'reklama-crimea.com',
 'chernomorskoe.reklama-crimea.com': 'reklama-crimea.com',
 'foros.reklama-crimea.com': 'reklama-crimea.com',
    'krd.rubri.co': 'rubri.co',
 'ekb.rubri.co': 'rubri.co',
 'nkz.rubri.co': 'rubri.co',
 'msk.rubri.co': 'rubri.co',
 'nsk.rubri.co': 'rubri.co',
 'mur.rubri.co': 'rubri.co',
 'ufa.rubri.co': 'rubri.co',
 'chl.rubri.co': 'rubri.co',
 'rubri.co': 'rubri.co',
 'prm.rubri.co': 'rubri.co',
 'kzn.rubri.co': 'rubri.co',
 'spb.rubri.co': 'rubri.co',
 'nvr.rubri.co': 'rubri.co',
 'izh.rubri.co': 'rubri.co',
 'kry.rubri.co': 'rubri.co',
 'tom.rubri.co': 'rubri.co',
    'cena-auto.ru': 'cena-auto.ru',
 'kazan.cena-auto.ru': 'cena-auto.ru',
 'kaliningrad.cena-auto.ru': 'cena-auto.ru',
 'krasnodar.cena-auto.ru': 'cena-auto.ru',
 'rostov-na-donu.cena-auto.ru': 'cena-auto.ru',
 'sankt-peterburg.cena-auto.ru': 'cena-auto.ru',
 'moskva.cena-auto.ru': 'cena-auto.ru',
 'mineralnye_vody.cena-auto.ru': 'cena-auto.ru',
 'ufa.cena-auto.ru': 'cena-auto.ru',
 'nijnii_novgorod.cena-auto.ru': 'cena-auto.ru',
 'ekaterinburg.cena-auto.ru': 'cena-auto.ru',
 'stavropol.cena-auto.ru': 'cena-auto.ru',
 'kurgan.cena-auto.ru': 'cena-auto.ru',
 'irkutsk.cena-auto.ru': 'cena-auto.ru',
 'kursk.cena-auto.ru': 'cena-auto.ru',
 'orel.cena-auto.ru': 'cena-auto.ru',
    'xn----etbbhq1abxc1l.xn-----7kcawwn9ahlcqmy6me.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----7sbbatzcfbdd7anvefmf.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbpghgfohdp5a.xn----8sbajh8awipfg.xn----7sbbatzcfbdd7anvefmf.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn-----7kcba3daxhbtdedgbee0a2cxn.xn----8sbajh8awipfg.xn----7sbbatzcfbdd7anvefmf.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbfd0amesfbg.xn----etbqxo1b.xn----7sbbatzcfbdd7anvefmf.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----dtb6adbbbrb.xn-----olchechgwmjieep.xn----8sbablheyvdbzbg3bs4v.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbablheyvdbzbg3bs4v.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbfc0bloy.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbukjzd4j.xn----8sbaack3bw6g2a.xn----8sbbf3bbdjoeb8ad2r.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbbf3bbdjoeb8ad2r.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sb9akbd3dxa.xn-----xlcafemgdrht1gwb.xn----8sbbfalg6bde0ahxlh0u.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbbfalg6bde0ahxlh0u.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----7sbb6acdbdb3alredmd0v.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----7sbbaeh6bcgbh0ctgr1d.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbahna1bf6abieo9q.xn------8cdxodb2agiesp.xn----7sbbaeh6bcgbh0ctgr1d.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----ptbfelddrdu.xn-----clcqpgfhoophn8q.xn----8sbbfalg6bde0ahxlh0u.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
 'xn----8sbjc6blcw3g.xn----etbqxo1b.xn----7sbbatzcfbdd7anvefmf.xn----9sbbbpi8a9bt6f.xn--p1ai': 'xn----9sbbbpi8a9bt6f.xn--p1ai',
    'kyiv.depo.ua': 'depo.ua',
 'vn.depo.ua': 'depo.ua',
 'news.depo.ua': 'depo.ua',
 'dnipro.depo.ua': 'depo.ua',
 'dn.depo.ua': 'depo.ua',
 'herson.depo.ua': 'depo.ua',
 'sector.depo.ua': 'depo.ua',
 'kh.depo.ua': 'depo.ua',
 'khm.depo.ua': 'depo.ua',
 'depo.ua': 'depo.ua',
 'zak.depo.ua': 'depo.ua',
 'odesa.depo.ua': 'depo.ua',
 'poltava.depo.ua': 'depo.ua',
 'zhytomyr.depo.ua': 'depo.ua',
 'lviv.depo.ua': 'depo.ua',
 'zp.depo.ua': 'depo.ua',
 'kr.depo.ua': 'depo.ua',
    'yak.house2you.ru': 'house2you.ru',
 'mo.house2you.ru': 'house2you.ru',
 'ar.house2you.ru': 'house2you.ru',
 'bash.house2you.ru': 'house2you.ru',
 'kurg.house2you.ru': 'house2you.ru',
 'vladimir.house2you.ru': 'house2you.ru',
 'vlad.house2you.ru': 'house2you.ru',
 'pk.house2you.ru': 'house2you.ru',
 'kras.house2you.ru': 'house2you.ru',
 'samara.house2you.ru': 'house2you.ru',
 'no.house2you.ru': 'house2you.ru',
 'len.house2you.ru': 'house2you.ru',
 'house2you.ru': 'house2you.ru',
 'chelyabinsk.house2you.ru': 'house2you.ru',
 'prim.house2you.ru': 'house2you.ru',
 'sk.house2you.ru': 'house2you.ru',
 'tum.house2you.ru': 'house2you.ru',
    'samara.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'ufa.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'perm.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'nizhniy-tagil.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'kaliningrad.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'tula.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'krasnodar.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'yaroslavl.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'tyumen.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'barnaul.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'surgut.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'omsk.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'novosibirsk.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'ekaterinburg.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'sochi.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'tolyatti.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
 'magnitogorsk.drenazhnaya-sistema.ru': 'drenazhnaya-sistema.ru',
    'gde.by': 'gde.by',
 'schuchin.gde.by': 'gde.by',
 'mogilev.gde.by': 'gde.by',
 'gomel.gde.by': 'gde.by',
 'borisov.gde.by': 'gde.by',
 'svetlogorsk.gde.by': 'gde.by',
 'minsk.gde.by': 'gde.by',
 'dubrovno.gde.by': 'gde.by',
 'bobrujsk.gde.by': 'gde.by',
 'rogachev.gde.by': 'gde.by',
 'bereza.gde.by': 'gde.by',
 'logoysk.gde.by': 'gde.by',
 'baranovichi.gde.by': 'gde.by',
 'zhlobin.gde.by': 'gde.by',
 'brest.gde.by': 'gde.by',
 'vitebsk.gde.by': 'gde.by',
 'orsha.gde.by': 'gde.by',
    'ekaterinburg.rabotajob.ru': 'rabotajob.ru',
 'samara.rabotajob.ru': 'rabotajob.ru',
 'rabotajob.ru': 'rabotajob.ru',
 'perm.rabotajob.ru': 'rabotajob.ru',
 'tyumen.rabotajob.ru': 'rabotajob.ru',
 'spb.rabotajob.ru': 'rabotajob.ru',
 'kazan.rabotajob.ru': 'rabotajob.ru',
 'msk.rabotajob.ru': 'rabotajob.ru',
 'novosibirsk.rabotajob.ru': 'rabotajob.ru',
 'saratov.rabotajob.ru': 'rabotajob.ru',
 'vladimir.rabotajob.ru': 'rabotajob.ru',
 'respublika-krym.rabotajob.ru': 'rabotajob.ru',
 'chelyabinsk.rabotajob.ru': 'rabotajob.ru',
 'krasnoyarsk.rabotajob.ru': 'rabotajob.ru',
 'krasnodar.rabotajob.ru': 'rabotajob.ru',
 'rostov.rabotajob.ru': 'rabotajob.ru',
 'ufa.rabotajob.ru': 'rabotajob.ru',
    'kazan.valuta24.ru': 'valuta24.ru',
 'krasnodar.valuta24.ru': 'valuta24.ru',
 'valuta24.ru': 'valuta24.ru',
 'spb.valuta24.ru': 'valuta24.ru',
 'ekb.valuta24.ru': 'valuta24.ru',
 'kaliningrad.valuta24.ru': 'valuta24.ru',
 'cr.valuta24.ru': 'valuta24.ru',
 'chelyabinsk.valuta24.ru': 'valuta24.ru',
 'krasnoyarsk.valuta24.ru': 'valuta24.ru',
 'grozny.valuta24.ru': 'valuta24.ru',
 'syktyvkar.valuta24.ru': 'valuta24.ru',
 'stavropol.valuta24.ru': 'valuta24.ru',
 'barnaul.valuta24.ru': 'valuta24.ru',
 'rostov.valuta24.ru': 'valuta24.ru',
 'orenburg.valuta24.ru': 'valuta24.ru',
 'orel.valuta24.ru': 'valuta24.ru',
 'vladivostok.valuta24.ru': 'valuta24.ru',
    'xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aag1ciek.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----7sbdqaabf2clfe5a7hpcg.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1ajt2aza.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1afkclaggf6a2g.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adio3aejo4j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adi0aoagldk8i.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90agcv2ah.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aaah7b8am.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80ajaublc1bet.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aanmmcj2ahke.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--j1adfn.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aegbkvxddlre.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aaifraljtb8a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----7sb1affqblhff4i.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--j1ahge.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1acdveadkc.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
    'sfo.spr.ru': 'spr.ru',
 'spr.ru': 'spr.ru',
 'pfo.spr.ru': 'spr.ru',
 'yfo.spr.ru': 'spr.ru',
 'ufo.spr.ru': 'spr.ru',
 'dfo.spr.ru': 'spr.ru',
 'szo.spr.ru': 'spr.ru',
 'cfo.spr.ru': 'spr.ru',
 'uzbekistan.spr.ru': 'spr.ru',
 'sko.spr.ru': 'spr.ru',
 'gruzia.spr.ru': 'spr.ru',
 'abkhaziya.spr.ru': 'spr.ru',
 'armenia.spr.ru': 'spr.ru',
 'kirgizia.spr.ru': 'spr.ru',
 'yuznaya-osetiya.spr.ru': 'spr.ru',
 'azerbaidzhan.spr.ru': 'spr.ru',
 'moldova.spr.ru': 'spr.ru',
    'bryansk.harat.ru': 'harat.ru',
 'harat.ru': 'harat.ru',
 'chelyabinsk.harat.ru': 'harat.ru',
 'nsk.harat.ru': 'harat.ru',
 'moskva.harat.ru': 'harat.ru',
 'ekb.harat.ru': 'harat.ru',
 'izhevsk.harat.ru': 'harat.ru',
 'krasnodar.harat.ru': 'harat.ru',
 'voskresensk.harat.ru': 'harat.ru',
 'novokuzneck.harat.ru': 'harat.ru',
 'yoshkar-ola.harat.ru': 'harat.ru',
 'murmansk.harat.ru': 'harat.ru',
 'ivanovo.harat.ru': 'harat.ru',
 'kazan.harat.ru': 'harat.ru',
 'naberezhnie-chelni.harat.ru': 'harat.ru',
 'adler.harat.ru': 'harat.ru',
 'tolyatti.harat.ru': 'harat.ru',
    'sf.tuvsu.ru': 'tuvsu.ru',
 'ippk.tuvsu.ru': 'tuvsu.ru',
 'ido.tuvsu.ru': 'tuvsu.ru',
 'ff.tuvsu.ru': 'tuvsu.ru',
 'ffkis.tuvsu.ru': 'tuvsu.ru',
 'if.tuvsu.ru': 'tuvsu.ru',
 'anketa.tuvsu.ru': 'tuvsu.ru',
 'law.tuvsu.ru': 'tuvsu.ru',
 'itf.tuvsu.ru': 'tuvsu.ru',
 'kpi.tuvsu.ru': 'tuvsu.ru',
 'tuvsu.ru': 'tuvsu.ru',
 'job.tuvsu.ru': 'tuvsu.ru',
 'kpk.tuvsu.ru': 'tuvsu.ru',
 'fmf.tuvsu.ru': 'tuvsu.ru',
 'education.tuvsu.ru': 'tuvsu.ru',
 'egf.tuvsu.ru': 'tuvsu.ru',
 'abiturient.tuvsu.ru': 'tuvsu.ru',
    'yaroslavl.trudko.ru': 'trudko.ru',
 'trudko.ru': 'trudko.ru',
 'vahta.trudko.ru': 'trudko.ru',
 'russia.trudko.ru': 'trudko.ru',
 'stavropol.trudko.ru': 'trudko.ru',
 'salekhard.trudko.ru': 'trudko.ru',
 'krasnodar.trudko.ru': 'trudko.ru',
 'izhevsk.trudko.ru': 'trudko.ru',
 'kazan.trudko.ru': 'trudko.ru',
 'yakutsk.trudko.ru': 'trudko.ru',
 'nn.trudko.ru': 'trudko.ru',
 'krasnoyarsk.trudko.ru': 'trudko.ru',
 'ekaterinburg.trudko.ru': 'trudko.ru',
 'voronezh.trudko.ru': 'trudko.ru',
 'spb.trudko.ru': 'trudko.ru',
 'ufa.trudko.ru': 'trudko.ru',
 'khanty-mansiysk.trudko.ru': 'trudko.ru',
 'perm.trudko.ru': 'trudko.ru',
    't195117.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't113532.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't44180d.dou.obrazovanie33.ru': 'obrazovanie33.ru',
 't570821.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't159822.sch.obrazovanie33.ru': 'obrazovanie33.ru',
 't144626.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't130631.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't706222.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't245814.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't47610v.sch.obrazovanie33.ru': 'obrazovanie33.ru',
 't900528.sch.obrazovanie33.ru': 'obrazovanie33.ru',
 'ypigk.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 't783442.dou.obrazovanie33.ru': 'obrazovanie33.ru',
 't192387.dou.obrazovanie33.ru': 'obrazovanie33.ru',
 'sigk.spo.obrazovanie33.ru': 'obrazovanie33.ru',
 'gim39.sch.obrazovanie33.ru': 'obrazovanie33.ru',
 't880248.dou.obrazovanie33.ru': 'obrazovanie33.ru',
 't29760w.sch.obrazovanie33.ru': 'obrazovanie33.ru',
    'xn--80adxhks.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--h1aecgfmj1g.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80aauks4g.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80ablvtof7b4b.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--e1aohf5d.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80aalwqglfe.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80aai0ag2c.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn----7sbeiia6axumbcqds.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80aag1ciek.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--h1afipca2eya.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--b1amdelgbarlac.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80a1bd.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--90aee6admdx.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80antj7do.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80acgfbsl1azdqr.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--b1afchn5b.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn-----7kcgn5cdbagnnnx.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
 'xn--80atblfjdfd2l.xn--b1ajukhp0cb.xn--p1ai': 'xn--b1ajukhp0cb.xn--p1ai',
    'edu-inform.ru': 'edu-inform.ru',
 'kostroma.edu-inform.ru': 'edu-inform.ru',
 'kazan.edu-inform.ru': 'edu-inform.ru',
 'voronezh.edu-inform.ru': 'edu-inform.ru',
 'arkhangelsk.edu-inform.ru': 'edu-inform.ru',
 'maikop.edu-inform.ru': 'edu-inform.ru',
 'vladikavkaz.edu-inform.ru': 'edu-inform.ru',
 'kaluga.edu-inform.ru': 'edu-inform.ru',
 'vladivostok.edu-inform.ru': 'edu-inform.ru',
 'sevastopol.edu-inform.ru': 'edu-inform.ru',
 'barnaul.edu-inform.ru': 'edu-inform.ru',
 'ekaterinburg.edu-inform.ru': 'edu-inform.ru',
 'murmansk.edu-inform.ru': 'edu-inform.ru',
 'kemerovo.edu-inform.ru': 'edu-inform.ru',
 'volgograd.edu-inform.ru': 'edu-inform.ru',
 'magas.edu-inform.ru': 'edu-inform.ru',
 'sankt-peterburg.edu-inform.ru': 'edu-inform.ru',
 'belgorod.edu-inform.ru': 'edu-inform.ru',
    'kalininsk.sarmo.ru': 'sarmo.ru',
 'rtishevo.sarmo.ru': 'sarmo.ru',
 'rovnoe.sarmo.ru': 'sarmo.ru',
 'duhovnitskoe.sarmo.ru': 'sarmo.ru',
 'turki.sarmo.ru': 'sarmo.ru',
 'dergachi.sarmo.ru': 'sarmo.ru',
 'smosar.sarmo.ru': 'sarmo.ru',
 'romanovka.sarmo.ru': 'sarmo.ru',
 'ekaterinovka.sarmo.ru': 'sarmo.ru',
 'brik.duhovnitskoe.sarmo.ru': 'sarmo.ru',
 'talov.kalininsk.sarmo.ru': 'sarmo.ru',
 'mal.arkadak.sarmo.ru': 'sarmo.ru',
 'rost.arkadak.sarmo.ru': 'sarmo.ru',
 'bkarabulak.sarmo.ru': 'sarmo.ru',
 'ivanteevka.sarmo.ru': 'sarmo.ru',
 'shirok.kalininsk.sarmo.ru': 'sarmo.ru',
 'hvalynsk.sarmo.ru': 'sarmo.ru',
 'sarmo.ru': 'sarmo.ru',
    'kaz.bodio.ru': 'bodio.ru',
 'bodio.ru': 'bodio.ru',
 'sam.bodio.ru': 'bodio.ru',
 'nn.bodio.ru': 'bodio.ru',
 'ekb.bodio.ru': 'bodio.ru',
 'orenburg.bodio.ru': 'bodio.ru',
 'nsk.bodio.ru': 'bodio.ru',
 'rostov.bodio.ru': 'bodio.ru',
 'spb.bodio.ru': 'bodio.ru',
 'voronezh.bodio.ru': 'bodio.ru',
 'chel.bodio.ru': 'bodio.ru',
 'krasnoyarsk.bodio.ru': 'bodio.ru',
 'ufa.bodio.ru': 'bodio.ru',
 'sochi.bodio.ru': 'bodio.ru',
 'perm.bodio.ru': 'bodio.ru',
 'krasnodar.bodio.ru': 'bodio.ru',
 'smolensk.bodio.ru': 'bodio.ru',
 'penza.bodio.ru': 'bodio.ru',
    'postroyforum.ru': 'postroyforum.ru',
 'kotel.postroyforum.ru': 'postroyforum.ru',
 'steni.postroyforum.ru': 'postroyforum.ru',
 'materialy.postroyforum.ru': 'postroyforum.ru',
 'potolok.postroyforum.ru': 'postroyforum.ru',
 'otzyvy.postroyforum.ru': 'postroyforum.ru',
 'kuhni.postroyforum.ru': 'postroyforum.ru',
 'trubi.postroyforum.ru': 'postroyforum.ru',
 'stroitelstvo.postroyforum.ru': 'postroyforum.ru',
 'zemlya.postroyforum.ru': 'postroyforum.ru',
 'garaj.postroyforum.ru': 'postroyforum.ru',
 'kamin.postroyforum.ru': 'postroyforum.ru',
 'dveri.postroyforum.ru': 'postroyforum.ru',
 'karkasnye-doma.postroyforum.ru': 'postroyforum.ru',
 'electro-montazh.postroyforum.ru': 'postroyforum.ru',
 'derevyannye-doma.postroyforum.ru': 'postroyforum.ru',
 'elektroinstrument.postroyforum.ru': 'postroyforum.ru',
 'mebel.postroyforum.ru': 'postroyforum.ru',
    'tyapse.reklama-kuban.com': 'reklama-kuban.com',
 'anapa.reklama-kuban.com': 'reklama-kuban.com',
 'adler.reklama-kuban.com': 'reklama-kuban.com',
 'armavir.reklama-kuban.com': 'reklama-kuban.com',
 'gelendjik.reklama-kuban.com': 'reklama-kuban.com',
 'reklama-kuban.com': 'reklama-kuban.com',
 'krasnodar.reklama-kuban.com': 'reklama-kuban.com',
 'sochi.reklama-kuban.com': 'reklama-kuban.com',
 'yeisk.reklama-kuban.com': 'reklama-kuban.com',
 'novorossiysk.reklama-kuban.com': 'reklama-kuban.com',
 'temryk.reklama-kuban.com': 'reklama-kuban.com',
 'slavyansk-na-kubani.reklama-kuban.com': 'reklama-kuban.com',
 'goryachiy_klych.reklama-kuban.com': 'reklama-kuban.com',
 'labinsk.reklama-kuban.com': 'reklama-kuban.com',
 'timashevsk.reklama-kuban.com': 'reklama-kuban.com',
 'kropotkin.reklama-kuban.com': 'reklama-kuban.com',
 'gylkevichi.reklama-kuban.com': 'reklama-kuban.com',
 'krymsk.reklama-kuban.com': 'reklama-kuban.com',
 'kurganinsk.reklama-kuban.com': 'reklama-kuban.com',
    'drivercan.ru': 'drivercan.ru',
 'epox.drivercan.ru': 'drivercan.ru',
 'ricoh.drivercan.ru': 'drivercan.ru',
 'kyocera.drivercan.ru': 'drivercan.ru',
 'gigabyte.drivercan.ru': 'drivercan.ru',
 'hp.drivercan.ru': 'drivercan.ru',
 'asus.drivercan.ru': 'drivercan.ru',
 'nvidia.drivercan.ru': 'drivercan.ru',
 'toshiba.drivercan.ru': 'drivercan.ru',
 'realtek.drivercan.ru': 'drivercan.ru',
 'acer.drivercan.ru': 'drivercan.ru',
 'canon.drivercan.ru': 'drivercan.ru',
 'lenovo.drivercan.ru': 'drivercan.ru',
 'msi-microstar.drivercan.ru': 'drivercan.ru',
 'tp-link.drivercan.ru': 'drivercan.ru',
 'amd.drivercan.ru': 'drivercan.ru',
 'intel.drivercan.ru': 'drivercan.ru',
 'a4tech.drivercan.ru': 'drivercan.ru',
 'genius.drivercan.ru': 'drivercan.ru',
    'mo.qadviser.ru': 'qadviser.ru',
 'spb.qadviser.ru': 'qadviser.ru',
 'krasnoyarsk.qadviser.ru': 'qadviser.ru',
 'nn.qadviser.ru': 'qadviser.ru',
 'perm.qadviser.ru': 'qadviser.ru',
 'pfo.qadviser.ru': 'qadviser.ru',
 'chelabinsk.qadviser.ru': 'qadviser.ru',
 'nw.qadviser.ru': 'qadviser.ru',
 'nsk.qadviser.ru': 'qadviser.ru',
 'sbr.qadviser.ru': 'qadviser.ru',
 'kr.qadviser.ru': 'qadviser.ru',
 'qadviser.ru': 'qadviser.ru',
 'kgd.qadviser.ru': 'qadviser.ru',
 'kazan.qadviser.ru': 'qadviser.ru',
 'krasnodar.qadviser.ru': 'qadviser.ru',
 'cfo.qadviser.ru': 'qadviser.ru',
 'vld.qadviser.ru': 'qadviser.ru',
 'ufo.qadviser.ru': 'qadviser.ru',
 'dv.qadviser.ru': 'qadviser.ru',
    'aromavape.ru': 'aromavape.ru',
 'balashiha.aromavape.ru': 'aromavape.ru',
 'kazan.aromavape.ru': 'aromavape.ru',
 'habarovsk.aromavape.ru': 'aromavape.ru',
 'samara.aromavape.ru': 'aromavape.ru',
 'yaroslavl.aromavape.ru': 'aromavape.ru',
 'krasnodar.aromavape.ru': 'aromavape.ru',
 'ufa.aromavape.ru': 'aromavape.ru',
 'krasnoyarsk.aromavape.ru': 'aromavape.ru',
 'moskva.aromavape.ru': 'aromavape.ru',
 'tver.aromavape.ru': 'aromavape.ru',
 'lipetsk.aromavape.ru': 'aromavape.ru',
 'saratov.aromavape.ru': 'aromavape.ru',
 'novokuznetsk.aromavape.ru': 'aromavape.ru',
 'kirov.aromavape.ru': 'aromavape.ru',
 'spb.aromavape.ru': 'aromavape.ru',
 'orsk.aromavape.ru': 'aromavape.ru',
 'syktyvkar.aromavape.ru': 'aromavape.ru',
 'perm.aromavape.ru': 'aromavape.ru',
    'kg.vesti.news': 'vesti.news',
 'tj.vesti.news': 'vesti.news',
 'kz.vesti.news': 'vesti.news',
 'uz.vesti.news': 'vesti.news',
 'ua.vesti.news': 'vesti.news',
 'lt.vesti.news': 'vesti.news',
 'ee.vesti.news': 'vesti.news',
 'ca.vesti.news': 'vesti.news',
 'tm.vesti.news': 'vesti.news',
 'md.vesti.news': 'vesti.news',
 'by.vesti.news': 'vesti.news',
 'es.vesti.news': 'vesti.news',
 'il.vesti.news': 'vesti.news',
 'ge.vesti.news': 'vesti.news',
 'uk.vesti.news': 'vesti.news',
 'us.vesti.news': 'vesti.news',
 'az.vesti.news': 'vesti.news',
 'lv.vesti.news': 'vesti.news',
 'vesti.news': 'vesti.news',
    'kaliningrad.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'spb.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'kirov.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'krasnodar.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'novosib.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'rostovnadonu.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'samara.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'volgograd.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'ufa.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'krasnoyarsk.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'garantstroikompleks.ru': 'garantstroikompleks.ru',
 'omsk.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'chel.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'ekb.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'perm.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'tyumen.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'kazan.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'voronezh.garantstroikompleks.ru': 'garantstroikompleks.ru',
 'orenburg.garantstroikompleks.ru': 'garantstroikompleks.ru',
    'saratov.tiberg.ru': 'tiberg.ru',
 'oskol.tiberg.ru': 'tiberg.ru',
 'voron.tiberg.ru': 'tiberg.ru',
 'krasnodar.tiberg.ru': 'tiberg.ru',
 'perm.tiberg.ru': 'tiberg.ru',
 'biysk.tiberg.ru': 'tiberg.ru',
 'tolyatti.tiberg.ru': 'tiberg.ru',
 'penza.tiberg.ru': 'tiberg.ru',
 'ulianovsk.tiberg.ru': 'tiberg.ru',
 'nn.tiberg.ru': 'tiberg.ru',
 'novgorod.tiberg.ru': 'tiberg.ru',
 'surgut.tiberg.ru': 'tiberg.ru',
 'samara.tiberg.ru': 'tiberg.ru',
 'tiberg.ru': 'tiberg.ru',
 'ufa.tiberg.ru': 'tiberg.ru',
 'spb.tiberg.ru': 'tiberg.ru',
 'saransk.tiberg.ru': 'tiberg.ru',
 'orenburg.tiberg.ru': 'tiberg.ru',
 'ola.tiberg.ru': 'tiberg.ru',
    'nizhny-tagil.mjobs.ru': 'mjobs.ru',
 'mjobs.ru': 'mjobs.ru',
 'msk.mjobs.ru': 'mjobs.ru',
 'spb.mjobs.ru': 'mjobs.ru',
 'ekaterinburg.mjobs.ru': 'mjobs.ru',
 'odintsovo.mjobs.ru': 'mjobs.ru',
 'kazan.mjobs.ru': 'mjobs.ru',
 'tolyatti.mjobs.ru': 'mjobs.ru',
 'novosibirsk.mjobs.ru': 'mjobs.ru',
 'krasnoyarsk.mjobs.ru': 'mjobs.ru',
 'krasnodar.mjobs.ru': 'mjobs.ru',
 'cheboksary.mjobs.ru': 'mjobs.ru',
 'yaroslavl.mjobs.ru': 'mjobs.ru',
 'lipetsk.mjobs.ru': 'mjobs.ru',
 'noviy-urengoy.mjobs.ru': 'mjobs.ru',
 'sochi.mjobs.ru': 'mjobs.ru',
 'tomsk.mjobs.ru': 'mjobs.ru',
 'nizhnekamsk.mjobs.ru': 'mjobs.ru',
 'saratov.mjobs.ru': 'mjobs.ru',
 'izhevsk.mjobs.ru': 'mjobs.ru',
    'slavyansk-na-kubani.ya23.ru': 'ya23.ru',
 'eysk.ya23.ru': 'ya23.ru',
 'gulkevichi.ya23.ru': 'ya23.ru',
 'sochi.ya23.ru': 'ya23.ru',
 'gelendzhik.ya23.ru': 'ya23.ru',
 'timashevsk.ya23.ru': 'ya23.ru',
 'novorossiysk.ya23.ru': 'ya23.ru',
 'anapa.ya23.ru': 'ya23.ru',
 'krasnodar.ya23.ru': 'ya23.ru',
 'labinsk.ya23.ru': 'ya23.ru',
 'temryuk.ya23.ru': 'ya23.ru',
 'belorechensk.ya23.ru': 'ya23.ru',
 'tuapse.ya23.ru': 'ya23.ru',
 'kurganinsk.ya23.ru': 'ya23.ru',
 'korenovsk.ya23.ru': 'ya23.ru',
 'apsheronsk.ya23.ru': 'ya23.ru',
 'khadyzhensk.ya23.ru': 'ya23.ru',
 'abinsk.ya23.ru': 'ya23.ru',
 'armavir.ya23.ru': 'ya23.ru',
 'goryachiy-klyuch.ya23.ru': 'ya23.ru',
    'edu.by': 'edu.by',
 'sch115.minsk.edu.by': 'edu.by',
 'sch2.lyahovichi.edu.by': 'edu.by',
 'sch178.minsk.edu.by': 'edu.by',
 'ptk-zht.minsk.edu.by': 'edu.by',
 'goo.pinsk.edu.by': 'edu.by',
 'sch16.pinsk.edu.by': 'edu.by',
 'sch19.baranovichi.edu.by': 'edu.by',
 'gymn21.minsk.edu.by': 'edu.by',
 'sch14.pinsk.edu.by': 'edu.by',
 'dvorec.luninec.edu.by': 'edu.by',
 'sch1.minsk.edu.by': 'edu.by',
 'goo.baranovichi.edu.by': 'edu.by',
 'kroschin.baranovichi.edu.by': 'edu.by',
 'ogorodniki.bereza.edu.by': 'edu.by',
 'sch1.krichev.edu.by': 'edu.by',
 'zavue.minsk.edu.by': 'edu.by',
 'nssh.kostjukovichi.edu.by': 'edu.by',
 'sch61.minsk.edu.by': 'edu.by',
 'sch203.minsk.edu.by': 'edu.by',
    'fis.ru': 'fis.ru',
 'omsk.fis.ru': 'fis.ru',
 'moskva.fis.ru': 'fis.ru',
 '10082331.fis.ru': 'fis.ru',
 '10084999.fis.ru': 'fis.ru',
 'burgeotehnika.fis.ru': 'fis.ru',
 'promsib.fis.ru': 'fis.ru',
 'kuban-iskozh.fis.ru': 'fis.ru',
 'vladivostok.fis.ru': 'fis.ru',
 '10049601.fis.ru': 'fis.ru',
 'mosselprom.fis.ru': 'fis.ru',
 'ukr-himiya.fis.ru': 'fis.ru',
 '10110546.fis.ru': 'fis.ru',
 'chelyabinsk.fis.ru': 'fis.ru',
 'sibirskie-produkty.fis.ru': 'fis.ru',
 'afrgroup-msk.fis.ru': 'fis.ru',
 'chetz.fis.ru': 'fis.ru',
 'sarsun.fis.ru': 'fis.ru',
 'kipr.fis.ru': 'fis.ru',
 'vladpoliteks.fis.ru': 'fis.ru',
    'volzhsky.firma-plast.ru': 'firma-plast.ru',
 'barnaul.firma-plast.ru': 'firma-plast.ru',
 'dzerzhinsk.firma-plast.ru': 'firma-plast.ru',
 'volgograd.firma-plast.ru': 'firma-plast.ru',
 'izh.firma-plast.ru': 'firma-plast.ru',
 'arkhangelsk.firma-plast.ru': 'firma-plast.ru',
 'angarsk.firma-plast.ru': 'firma-plast.ru',
 'krasnoyarsk.firma-plast.ru': 'firma-plast.ru',
 'iola.firma-plast.ru': 'firma-plast.ru',
 'irk.firma-plast.ru': 'firma-plast.ru',
 'balakovo.firma-plast.ru': 'firma-plast.ru',
 'vrn.firma-plast.ru': 'firma-plast.ru',
 'kalinin.firma-plast.ru': 'firma-plast.ru',
 'kirov.firma-plast.ru': 'firma-plast.ru',
 'abakan.firma-plast.ru': 'firma-plast.ru',
 'kemerovo.firma-plast.ru': 'firma-plast.ru',
 'bratsk.firma-plast.ru': 'firma-plast.ru',
 'vladivostok.firma-plast.ru': 'firma-plast.ru',
 'kostroma.firma-plast.ru': 'firma-plast.ru',
 'vnovgorod.firma-plast.ru': 'firma-plast.ru',
    'xn----7sbbhma3bcfp4olb.vuxo7.com': 'vuxo7.com',
 'xn--vovabg--8fg6euat3cpk.vuxo7.com': 'vuxo7.com',
 'xn----8sbemauuaghh3ab4ale2a.vuxo7.com': 'vuxo7.com',
 'xn--i-s-i-sleeping-hummingbird--10c912vzbwa9d8c6ec.vuxo7.com': 'vuxo7.com',
 'xn------6cdcblv1cbthsbiry7ij0h6b.vuxo7.com': 'vuxo7.com',
 'xn-----nlckdteefkqge1a.vuxo7.com': 'vuxo7.com',
 'xn------5cdnna4bdbjsjhcggsm6c5f.vuxo7.com': 'vuxo7.com',
 'xn----7sbqamclbnbgnvpkcy2q.vuxo7.com': 'vuxo7.com',
 'xn------5cdbgobccnb5ajjiflx4ayc0bhc6cjr8j1f.vuxo7.com': 'vuxo7.com',
 '471681634.vuxo7.com': 'vuxo7.com',
 'xn-----6kcbbxqdzxnj.vuxo7.com': 'vuxo7.com',
 'vuxo7.com': 'vuxo7.com',
 'xn----------1fgabafb9blc2dseghiehmkmm6bgb6bdephnah1bb6a3swfsgj.vuxo7.com': 'vuxo7.com',
 'xn----7sbabavf4bhhij4bdh4j.vuxo7.com': 'vuxo7.com',
 'vbots-ru.vuxo7.com': 'vuxo7.com',
 'xn--b1afbahf7ag8ak.vuxo7.com': 'vuxo7.com',
 'xn----8sbwbmb3ab.vuxo7.com': 'vuxo7.com',
 'xn----etbbgaca5ce0df.vuxo7.com': 'vuxo7.com',
 'unknown-artist.vuxo7.com': 'vuxo7.com',
 'xn--80aimjn8ao.vuxo7.com': 'vuxo7.com',
    'stom.32top.ru': '32top.ru',
 '32top.ru': '32top.ru',
 'spb.32top.ru': '32top.ru',
 'orenburg.32top.ru': '32top.ru',
 'tolyatti.32top.ru': '32top.ru',
 'kurgan.32top.ru': '32top.ru',
 'perm.32top.ru': '32top.ru',
 'barnaul.32top.ru': '32top.ru',
 'izhevsk.32top.ru': '32top.ru',
 'omsk.32top.ru': '32top.ru',
 'ekaterinburg.32top.ru': '32top.ru',
 'samara.32top.ru': '32top.ru',
 'voronezh.32top.ru': '32top.ru',
 'forum.32top.ru': '32top.ru',
 'saratov.32top.ru': '32top.ru',
 'chelyabinsk.32top.ru': '32top.ru',
 'tomsk.32top.ru': '32top.ru',
 'penza.32top.ru': '32top.ru',
 'kostroma.32top.ru': '32top.ru',
 'ulan-udeh.32top.ru': '32top.ru',
    'volgograd.avteon.ru': 'avteon.ru',
 'avteon.ru': 'avteon.ru',
 'kaluga-service.avteon.ru': 'avteon.ru',
 'voronezh.avteon.ru': 'avteon.ru',
 'krasnoyarsk.avteon.ru': 'avteon.ru',
 'izhevsk-zapchasti.avteon.ru': 'avteon.ru',
 'maykop.avteon.ru': 'avteon.ru',
 'orel-service.avteon.ru': 'avteon.ru',
 'spb.avteon.ru': 'avteon.ru',
 'seversk.avteon.ru': 'avteon.ru',
 'seversk-service.avteon.ru': 'avteon.ru',
 'samara.avteon.ru': 'avteon.ru',
 'msk.avteon.ru': 'avteon.ru',
 'murmansk-tuning.avteon.ru': 'avteon.ru',
 'murmansk.avteon.ru': 'avteon.ru',
 'krasnodar.avteon.ru': 'avteon.ru',
 'zheleznogorsk.avteon.ru': 'avteon.ru',
 'stavropol.avteon.ru': 'avteon.ru',
 'smolensk.avteon.ru': 'avteon.ru',
 'ulyanovsk.avteon.ru': 'avteon.ru',
    'nsk.365rem.ru': '365rem.ru',
 '365rem.ru': '365rem.ru',
 'ulyanovsk.365rem.ru': '365rem.ru',
 'kurgan.365rem.ru': '365rem.ru',
 'omsk.365rem.ru': '365rem.ru',
 'cheboksari.365rem.ru': '365rem.ru',
 'tagil.365rem.ru': '365rem.ru',
 'barnaul.365rem.ru': '365rem.ru',
 'habarovsk.365rem.ru': '365rem.ru',
 'saratov.365rem.ru': '365rem.ru',
 'kaliningrad.365rem.ru': '365rem.ru',
 'perm.365rem.ru': '365rem.ru',
 'kaluga.365rem.ru': '365rem.ru',
 'orenburg.365rem.ru': '365rem.ru',
 'tumen.365rem.ru': '365rem.ru',
 'kirov.365rem.ru': '365rem.ru',
 'kazan.365rem.ru': '365rem.ru',
 'izhevsk.365rem.ru': '365rem.ru',
 'ufa.365rem.ru': '365rem.ru',
 'smolensk.365rem.ru': '365rem.ru',
 'novokuznetsk.365rem.ru': '365rem.ru',
    'nizhnevartovsk.qlaster.ru': 'qlaster.ru',
 'qlaster.ru': 'qlaster.ru',
 'kamensk.qlaster.ru': 'qlaster.ru',
 'sver.qlaster.ru': 'qlaster.ru',
 'tmn.qlaster.ru': 'qlaster.ru',
 'lk.qlaster.ru': 'qlaster.ru',
 'ufa.qlaster.ru': 'qlaster.ru',
 'tumen.qlaster.ru': 'qlaster.ru',
 'miass.qlaster.ru': 'qlaster.ru',
 'zlatoust.qlaster.ru': 'qlaster.ru',
 'chelyabinsk.qlaster.ru': 'qlaster.ru',
 'chel.qlaster.ru': 'qlaster.ru',
 'n-tagil.qlaster.ru': 'qlaster.ru',
 'magnitogorsk.qlaster.ru': 'qlaster.ru',
 'surgut.qlaster.ru': 'qlaster.ru',
 'aramil.qlaster.ru': 'qlaster.ru',
 'hm.qlaster.ru': 'qlaster.ru',
 'msk.qlaster.ru': 'qlaster.ru',
 'n-urengoy.qlaster.ru': 'qlaster.ru',
 'kemerovo.qlaster.ru': 'qlaster.ru',
 'penza.qlaster.ru': 'qlaster.ru',
    '61.rodina.news': 'rodina.news',
 '32.rodina.news': 'rodina.news',
 '77.rodina.news': 'rodina.news',
 '64.rodina.news': 'rodina.news',
 '31.rodina.news': 'rodina.news',
 '63.rodina.news': 'rodina.news',
 '78.rodina.news': 'rodina.news',
 'rodina.news': 'rodina.news',
 '66.rodina.news': 'rodina.news',
 '34.rodina.news': 'rodina.news',
 '51.rodina.news': 'rodina.news',
 '02.rodina.news': 'rodina.news',
 '69.rodina.news': 'rodina.news',
 '56.rodina.news': 'rodina.news',
 '73.rodina.news': 'rodina.news',
 '54.rodina.news': 'rodina.news',
 '11.rodina.news': 'rodina.news',
 '30.rodina.news': 'rodina.news',
 '10.rodina.news': 'rodina.news',
 '62.rodina.news': 'rodina.news',
 '47.rodina.news': 'rodina.news',
    'obrblag.info': 'obrblag.info',
 'school06.obrblag.info': 'obrblag.info',
 'do01.obrblag.info': 'obrblag.info',
 'ds32.obrblag.info': 'obrblag.info',
 'school27.obrblag.info': 'obrblag.info',
 'school12.obrblag.info': 'obrblag.info',
 'school25.obrblag.info': 'obrblag.info',
 'ds19.obrblag.info': 'obrblag.info',
 'cevd.obrblag.info': 'obrblag.info',
 'ds67.obrblag.info': 'obrblag.info',
 'pg.obrblag.info': 'obrblag.info',
 'school10.obrblag.info': 'obrblag.info',
 'school14.obrblag.info': 'obrblag.info',
 'school16.obrblag.info': 'obrblag.info',
 'school05.obrblag.info': 'obrblag.info',
 'school02.obrblag.info': 'obrblag.info',
 'school28.obrblag.info': 'obrblag.info',
 'school01.obrblag.info': 'obrblag.info',
 'school11.obrblag.info': 'obrblag.info',
 'school22.obrblag.info': 'obrblag.info',
 'school04.obrblag.info': 'obrblag.info',
    'hd16.1-lordserials.xyz': '1-lordserials.xyz',
 'hd1.1-lordserials.xyz': '1-lordserials.xyz',
 'hd5.1-lordserials.xyz': '1-lordserials.xyz',
 'hd23.1-lordserials.xyz': '1-lordserials.xyz',
 'hd.1-lordserials.xyz': '1-lordserials.xyz',
 'hd17.1-lordserials.xyz': '1-lordserials.xyz',
 'hd20.1-lordserials.xyz': '1-lordserials.xyz',
 'hd19.1-lordserials.xyz': '1-lordserials.xyz',
 'hd28.1-lordserials.xyz': '1-lordserials.xyz',
 'hd18.1-lordserials.xyz': '1-lordserials.xyz',
 'hd29.1-lordserials.xyz': '1-lordserials.xyz',
 'hd30.1-lordserials.xyz': '1-lordserials.xyz',
 'hd8.1-lordserials.xyz': '1-lordserials.xyz',
 'hd25.1-lordserials.xyz': '1-lordserials.xyz',
 'hd24.1-lordserials.xyz': '1-lordserials.xyz',
 'hd27.1-lordserials.xyz': '1-lordserials.xyz',
 'hd10.1-lordserials.xyz': '1-lordserials.xyz',
 'hd14.1-lordserials.xyz': '1-lordserials.xyz',
 'hd12.1-lordserials.xyz': '1-lordserials.xyz',
 'hd22.1-lordserials.xyz': '1-lordserials.xyz',
 'hd21.1-lordserials.xyz': '1-lordserials.xyz',
    'fox-fan.tv': 'fox-fan.tv',
 'm.americandad.fox-fan.tv': 'fox-fan.tv',
 'americandad.fox-fan.tv': 'fox-fan.tv',
 'm.familyguy.fox-fan.tv': 'fox-fan.tv',
 'familyguy.fox-fan.tv': 'fox-fan.tv',
 'm.blesstheharts.fox-fan.tv': 'fox-fan.tv',
 'simpsons.fox-fan.tv': 'fox-fan.tv',
 'blesstheharts.fox-fan.tv': 'fox-fan.tv',
 'm.simpsons.fox-fan.tv': 'fox-fan.tv',
 'm.duncanville.fox-fan.tv': 'fox-fan.tv',
 'duncanville.fox-fan.tv': 'fox-fan.tv',
 'm.bobsburgers.fox-fan.tv': 'fox-fan.tv',
 'bobsburgers.fox-fan.tv': 'fox-fan.tv',
 'futurama.fox-fan.tv': 'fox-fan.tv',
 'm.futurama.fox-fan.tv': 'fox-fan.tv',
 'kingofthehill.fox-fan.tv': 'fox-fan.tv',
 'm.bordertown.fox-fan.tv': 'fox-fan.tv',
 'bordertown.fox-fan.tv': 'fox-fan.tv',
 'm.kingofthehill.fox-fan.tv': 'fox-fan.tv',
 'm.clevelandshow.fox-fan.tv': 'fox-fan.tv',
 'clevelandshow.fox-fan.tv': 'fox-fan.tv',
    'tumen.zapisatsya.ru': 'zapisatsya.ru',
 'ufa.zapisatsya.ru': 'zapisatsya.ru',
 'zapisatsya.ru': 'zapisatsya.ru',
 'spb.zapisatsya.ru': 'zapisatsya.ru',
 'krasnodar.zapisatsya.ru': 'zapisatsya.ru',
 'volgograd.zapisatsya.ru': 'zapisatsya.ru',
 'ekb.zapisatsya.ru': 'zapisatsya.ru',
 'nsk.zapisatsya.ru': 'zapisatsya.ru',
 'izhevsk.zapisatsya.ru': 'zapisatsya.ru',
 'sochi.zapisatsya.ru': 'zapisatsya.ru',
 'samara.zapisatsya.ru': 'zapisatsya.ru',
 'omsk.zapisatsya.ru': 'zapisatsya.ru',
 'kazan.zapisatsya.ru': 'zapisatsya.ru',
 'rostov.zapisatsya.ru': 'zapisatsya.ru',
 'tver.zapisatsya.ru': 'zapisatsya.ru',
 'mo.zapisatsya.ru': 'zapisatsya.ru',
 'ryazan.zapisatsya.ru': 'zapisatsya.ru',
 'yaroslavl.zapisatsya.ru': 'zapisatsya.ru',
 'voronezh.zapisatsya.ru': 'zapisatsya.ru',
 'perm.zapisatsya.ru': 'zapisatsya.ru',
 'nn.zapisatsya.ru': 'zapisatsya.ru',
    'len.uszn52.ru': 'uszn52.ru',
 'sorm.uszn52.ru': 'uszn52.ru',
 'priok.uszn52.ru': 'uszn52.ru',
 'bal.uszn52.ru': 'uszn52.ru',
 'dzr.uszn52.ru': 'uszn52.ru',
 'nizh.uszn52.ru': 'uszn52.ru',
 'sem.uszn52.ru': 'uszn52.ru',
 'arz.uszn52.ru': 'uszn52.ru',
 'bgr.uszn52.ru': 'uszn52.ru',
 'vks.uszn52.ru': 'uszn52.ru',
 'kan.uszn52.ru': 'uszn52.ru',
 'ard.uszn52.ru': 'uszn52.ru',
 'kst.uszn52.ru': 'uszn52.ru',
 'msk.uszn52.ru': 'uszn52.ru',
 'bor.uszn52.ru': 'uszn52.ru',
 'arzamasskij.uszn52.ru': 'uszn52.ru',
 'vad.uszn52.ru': 'uszn52.ru',
 'luk.uszn52.ru': 'uszn52.ru',
 'var.uszn52.ru': 'uszn52.ru',
 'pvl.uszn52.ru': 'uszn52.ru',
 'tns.uszn52.ru': 'uszn52.ru',
 'klb.uszn52.ru': 'uszn52.ru',
    'medvezhonok.caduk.ru': 'caduk.ru',
 'teremok16.caduk.ru': 'caduk.ru',
 'crr38.caduk.ru': 'caduk.ru',
 'mdou-118.caduk.ru': 'caduk.ru',
 'rodnik452.caduk.ru': 'caduk.ru',
 'mbdoydc84vlad33.caduk.ru': 'caduk.ru',
 '39frspb.caduk.ru': 'caduk.ru',
 'dou97klspb.caduk.ru': 'caduk.ru',
 'dou-petrovka.caduk.ru': 'caduk.ru',
 'logoped.kam16.caduk.ru': 'caduk.ru',
 '40frspb.caduk.ru': 'caduk.ru',
 'psychology.dou84.caduk.ru': 'caduk.ru',
 'logoped.mbdoydc84vlad33.caduk.ru': 'caduk.ru',
 '25mrspb.caduk.ru': 'caduk.ru',
 'dc26krs.caduk.ru': 'caduk.ru',
 '65kgspb.caduk.ru': 'caduk.ru',
 'rodnichokob.caduk.ru': 'caduk.ru',
 '65frspb.caduk.ru': 'caduk.ru',
 'detsad-46.caduk.ru': 'caduk.ru',
 '124-luchiki.caduk.ru': 'caduk.ru',
 'dou114fr.caduk.ru': 'caduk.ru',
 '34kpspb.caduk.ru': 'caduk.ru',
    'ufa.medcentr.online': 'medcentr.online',
 'nsk.medcentr.online': 'medcentr.online',
 'vrn.medcentr.online': 'medcentr.online',
 'medcentr.online': 'medcentr.online',
 'krd.medcentr.online': 'medcentr.online',
 'tver.medcentr.online': 'medcentr.online',
 'tyumen.medcentr.online': 'medcentr.online',
 'perm.medcentr.online': 'medcentr.online',
 'spb.medcentr.online': 'medcentr.online',
 'rnd.medcentr.online': 'medcentr.online',
 'kazan.medcentr.online': 'medcentr.online',
 'saratov.medcentr.online': 'medcentr.online',
 'sochi.medcentr.online': 'medcentr.online',
 'krsk.medcentr.online': 'medcentr.online',
 'samara.medcentr.online': 'medcentr.online',
 'izh.medcentr.online': 'medcentr.online',
 'vlg.medcentr.online': 'medcentr.online',
 'pnz.medcentr.online': 'medcentr.online',
 'klng.medcentr.online': 'medcentr.online',
 'volgograd.medcentr.online': 'medcentr.online',
 'nn.medcentr.online': 'medcentr.online',
 'yaroslavl.medcentr.online': 'medcentr.online',
    'fn.bmstu.ru': 'bmstu.ru',
 'wwv.bmstu.ru': 'bmstu.ru',
 'bmstu.ru': 'bmstu.ru',
 'cendop.bmstu.ru': 'bmstu.ru',
 'culture.bmstu.ru': 'bmstu.ru',
 'profkom.bmstu.ru': 'bmstu.ru',
 'hoster.bmstu.ru': 'bmstu.ru',
 'eun.bmstu.ru': 'bmstu.ru',
 'energo.bmstu.ru': 'bmstu.ru',
 'edu.bmstu.ru': 'bmstu.ru',
 'start-career.bmstu.ru': 'bmstu.ru',
 'ibm6.bmstu.ru': 'bmstu.ru',
 'ibm4.bmstu.ru': 'bmstu.ru',
 'iu5.bmstu.ru': 'bmstu.ru',
 'fsgn.bmstu.ru': 'bmstu.ru',
 'akf.bmstu.ru': 'bmstu.ru',
 'rk1.bmstu.ru': 'bmstu.ru',
 'ibm.bmstu.ru': 'bmstu.ru',
 'rl2.bmstu.ru': 'bmstu.ru',
 'jantugan.bmstu.ru': 'bmstu.ru',
 'ipf.bmstu.ru': 'bmstu.ru',
 'rk.bmstu.ru': 'bmstu.ru',
    'habarovsk.wedhub.ru': 'wedhub.ru',
 'ivanovo.wedhub.ru': 'wedhub.ru',
 'sanktpeterburg.wedhub.ru': 'wedhub.ru',
 'wedhub.ru': 'wedhub.ru',
 'belgorod.wedhub.ru': 'wedhub.ru',
 'stavropol.wedhub.ru': 'wedhub.ru',
 'tjumen.wedhub.ru': 'wedhub.ru',
 'moskva.wedhub.ru': 'wedhub.ru',
 'omsk.wedhub.ru': 'wedhub.ru',
 'rostovnadonu.wedhub.ru': 'wedhub.ru',
 'ekaterinburg.wedhub.ru': 'wedhub.ru',
 'krasnodar.wedhub.ru': 'wedhub.ru',
 'orenburg.wedhub.ru': 'wedhub.ru',
 'izhevsk.wedhub.ru': 'wedhub.ru',
 'volgograd.wedhub.ru': 'wedhub.ru',
 'naberezhnyechelny.wedhub.ru': 'wedhub.ru',
 'sochi.wedhub.ru': 'wedhub.ru',
 'cheboksary.wedhub.ru': 'wedhub.ru',
 'vladivostok.wedhub.ru': 'wedhub.ru',
 'kostroma.wedhub.ru': 'wedhub.ru',
 'ufa.wedhub.ru': 'wedhub.ru',
 'kazan.wedhub.ru': 'wedhub.ru',
    'oktyabrskiy.dmaps.ru': 'dmaps.ru',
 'kursk.dmaps.ru': 'dmaps.ru',
 'msk.dmaps.ru': 'dmaps.ru',
 'tyumen.dmaps.ru': 'dmaps.ru',
 'pskov.dmaps.ru': 'dmaps.ru',
 'barnaul.dmaps.ru': 'dmaps.ru',
 'cherepovets.dmaps.ru': 'dmaps.ru',
 'krasnodar.dmaps.ru': 'dmaps.ru',
 'sochi.dmaps.ru': 'dmaps.ru',
 'kazan.dmaps.ru': 'dmaps.ru',
 'tambov.dmaps.ru': 'dmaps.ru',
 'taganrog.dmaps.ru': 'dmaps.ru',
 'yaroslavl.dmaps.ru': 'dmaps.ru',
 'colomna.dmaps.ru': 'dmaps.ru',
 'neftekamsk.dmaps.ru': 'dmaps.ru',
 'ussuriisk.dmaps.ru': 'dmaps.ru',
 'velikieluki.dmaps.ru': 'dmaps.ru',
 'samara.dmaps.ru': 'dmaps.ru',
 'izhevsk.dmaps.ru': 'dmaps.ru',
 'chelyabinsk.dmaps.ru': 'dmaps.ru',
 'spb.dmaps.ru': 'dmaps.ru',
 'volgodonsk.dmaps.ru': 'dmaps.ru',
 'ufa.dmaps.ru': 'dmaps.ru',
    'school57.irk.ru': 'irk.ru',
 '1m.irk.ru': 'irk.ru',
 'irk.ru': 'irk.ru',
 'special.1m.irk.ru': 'irk.ru',
 'bct.irk.ru': 'irk.ru',
 'school6.irk.ru': 'irk.ru',
 'school69.irk.ru': 'irk.ru',
 'school77.irk.ru': 'irk.ru',
 'school15.irk.ru': 'irk.ru',
 'itam.irk.ru': 'irk.ru',
 'terrano.irk.ru': 'irk.ru',
 'realty.irk.ru': 'irk.ru',
 'geol.irk.ru': 'irk.ru',
 'avia.rpdu.irk.ru': 'irk.ru',
 'bm.isc.irk.ru': 'irk.ru',
 'rmcraion.irk.ru': 'irk.ru',
 'venera.irk.ru': 'irk.ru',
 'school2.irk.ru': 'irk.ru',
 'school23.irk.ru': 'irk.ru',
 'school31.irk.ru': 'irk.ru',
 'lin.irk.ru': 'irk.ru',
 'school.irk.ru': 'irk.ru',
 'iek.irk.ru': 'irk.ru',
    'uo.armavir.kubannet.ru': 'kubannet.ru',
 'kubannet.ru': 'kubannet.ru',
 'pushkin.kubannet.ru': 'kubannet.ru',
 'school46.kubannet.ru': 'kubannet.ru',
 'gimnaz23.kubannet.ru': 'kubannet.ru',
 'gimnaz7.krim.kubannet.ru': 'kubannet.ru',
 'portal.kubannet.ru': 'kubannet.ru',
 'school4.armavir.kubannet.ru': 'kubannet.ru',
 'school101.kubannet.ru': 'kubannet.ru',
 'school18.kor.kubannet.ru': 'kubannet.ru',
 'school3.kurgan.kubannet.ru': 'kubannet.ru',
 'school5.bel.kubannet.ru': 'kubannet.ru',
 'school16.armavir.kubannet.ru': 'kubannet.ru',
 'uo.bru.kubannet.ru': 'kubannet.ru',
 'neklib.kubannet.ru': 'kubannet.ru',
 'school2.gul.kubannet.ru': 'kubannet.ru',
 'school20.gul.kubannet.ru': 'kubannet.ru',
 'uo.krsm.kubannet.ru': 'kubannet.ru',
 'school1.krp.kubannet.ru': 'kubannet.ru',
 'school16.krp.kubannet.ru': 'kubannet.ru',
 'school15.tim.kubannet.ru': 'kubannet.ru',
 'crtgul.gul.kubannet.ru': 'kubannet.ru',
 'uo.gul.kubannet.ru': 'kubannet.ru',
    'ryazan.po-aptekam.ru': 'po-aptekam.ru',
 'kostroma.po-aptekam.ru': 'po-aptekam.ru',
 'voronezh.po-aptekam.ru': 'po-aptekam.ru',
 'vladimir.po-aptekam.ru': 'po-aptekam.ru',
 'ivanovo.po-aptekam.ru': 'po-aptekam.ru',
 'tula.po-aptekam.ru': 'po-aptekam.ru',
 'arhangelsk.po-aptekam.ru': 'po-aptekam.ru',
 'cherepovets.po-aptekam.ru': 'po-aptekam.ru',
 'yaroslavl.po-aptekam.ru': 'po-aptekam.ru',
 'tver.po-aptekam.ru': 'po-aptekam.ru',
 'vologda.po-aptekam.ru': 'po-aptekam.ru',
 'nnovgorod.po-aptekam.ru': 'po-aptekam.ru',
 'po-aptekam.ru': 'po-aptekam.ru',
 'rostov.po-aptekam.ru': 'po-aptekam.ru',
 'chuvashiya.po-aptekam.ru': 'po-aptekam.ru',
 'lipetsk.po-aptekam.ru': 'po-aptekam.ru',
 'rybinsk.po-aptekam.ru': 'po-aptekam.ru',
 'arzamas.po-aptekam.ru': 'po-aptekam.ru',
 'murmansk.po-aptekam.ru': 'po-aptekam.ru',
 'orel.po-aptekam.ru': 'po-aptekam.ru',
 'nerehta.po-aptekam.ru': 'po-aptekam.ru',
 'tutaev.po-aptekam.ru': 'po-aptekam.ru',
 'kaluga.po-aptekam.ru': 'po-aptekam.ru',
    't.rbc.ru': 'rbc.ru',
 'amp.rbc.ru': 'rbc.ru',
 'cash.rbc.ru': 'rbc.ru',
 'style.rbc.ru': 'rbc.ru',
 'quote.rbc.ru': 'rbc.ru',
 'trends.rbc.ru': 'rbc.ru',
 'ufa.rbc.ru': 'rbc.ru',
 'realty.rbc.ru': 'rbc.ru',
 'pro.rbc.ru': 'rbc.ru',
 'chr.rbc.ru': 'rbc.ru',
 'nsk.rbc.ru': 'rbc.ru',
 'kuban.rbc.ru': 'rbc.ru',
 'rbc.ru': 'rbc.ru',
 'tv.rbc.ru': 'rbc.ru',
 'kaliningrad.rbc.ru': 'rbc.ru',
 'perm.rbc.ru': 'rbc.ru',
 'ekb.rbc.ru': 'rbc.ru',
 'rt.rbc.ru': 'rbc.ru',
 'rostov.rbc.ru': 'rbc.ru',
 'kavkaz.rbc.ru': 'rbc.ru',
 'nn.rbc.ru': 'rbc.ru',
 'vo.rbc.ru': 'rbc.ru',
 'karelia.rbc.ru': 'rbc.ru',
    'zeroevent.ru': 'zeroevent.ru',
 'ufa.zeroevent.ru': 'zeroevent.ru',
 'ekb.zeroevent.ru': 'zeroevent.ru',
 'omsk.zeroevent.ru': 'zeroevent.ru',
 'spb.zeroevent.ru': 'zeroevent.ru',
 'kzn.zeroevent.ru': 'zeroevent.ru',
 'msk.zeroevent.ru': 'zeroevent.ru',
 'vrn.zeroevent.ru': 'zeroevent.ru',
 'volgograd.zeroevent.ru': 'zeroevent.ru',
 'ngs.zeroevent.ru': 'zeroevent.ru',
 'perm.zeroevent.ru': 'zeroevent.ru',
 'tgl.zeroevent.ru': 'zeroevent.ru',
 'izh.zeroevent.ru': 'zeroevent.ru',
 'minsk.zeroevent.ru': 'zeroevent.ru',
 'yar.zeroevent.ru': 'zeroevent.ru',
 'tumen.zeroevent.ru': 'zeroevent.ru',
 'rostov.zeroevent.ru': 'zeroevent.ru',
 'kirov.zeroevent.ru': 'zeroevent.ru',
 'nn.zeroevent.ru': 'zeroevent.ru',
 'saratov.zeroevent.ru': 'zeroevent.ru',
 'samara.zeroevent.ru': 'zeroevent.ru',
 'krd.zeroevent.ru': 'zeroevent.ru',
 'kaliningrad.zeroevent.ru': 'zeroevent.ru',
    'webufa.ru': 'webufa.ru',
 'xn----8sbh9df.webufa.ru': 'webufa.ru',
 'xn----8sbgkb8agqeni.xn-----8kclvgaide0ag4aeuz1q.webufa.ru': 'webufa.ru',
 'xn-----8kcapmvihjqd7aeu2d8g.webufa.ru': 'webufa.ru',
 'xn----8sbaha2cfyfs.xn-----8kci2achgphbdntew0a.webufa.ru': 'webufa.ru',
 'xn----8sbiedfnuvoaxf.xn-----xlcafenfzptm.webufa.ru': 'webufa.ru',
 'xn----dtbfeathce7dgt.xn-----ilctbahghqf2av.webufa.ru': 'webufa.ru',
 'xn----8sbcfd7be1abvy.xn-----xlcafenfzptm.webufa.ru': 'webufa.ru',
 'xn----dtbalaaa2bzappcg4dyi.xn-----flcloacihgt8av9g.webufa.ru': 'webufa.ru',
 'xn----8sbjgzm5aw4aq.xn-----8kciekczjeetj4bza3a6dya.webufa.ru': 'webufa.ru',
 'xn----dtberakja1abq.xn-----xlcafenfzptm.webufa.ru': 'webufa.ru',
 'xn----8sbacet4afiqlbd7a0a.xn-----8kcadyxhbfdhwsx1ao.webufa.ru': 'webufa.ru',
 'xn----jtbfdzub.xn-----clcqashghqprh9f.webufa.ru': 'webufa.ru',
 'xn----btbmaoanbeyfh2afelp2gsdf.xn-----olcapahghhjg4axl2f.webufa.ru': 'webufa.ru',
 'xn----8sbaa1ccie9agg7j.xn-----8kcaa2caidcckjkj0bfuf.webufa.ru': 'webufa.ru',
 'xn----8sbcfo9ckfu.xn-----8kcdgrwihy5aeqx.webufa.ru': 'webufa.ru',
 'xn----8sbflzbsibyc4e.xn-----xlcafenfzptm.webufa.ru': 'webufa.ru',
 'xn----btbewk9ah.xn-----clc2aagfuzfl.webufa.ru': 'webufa.ru',
 'xn----8sbaidn6cnt.xn----8sbh9df.webufa.ru': 'webufa.ru',
 'xn----ptbahijhr0e.xn-----xlcaageboif2av8g.webufa.ru': 'webufa.ru',
 'xn----btbds0bbps.xn-----xlcafenfzptm.webufa.ru': 'webufa.ru',
 'xn----8sbajlsqel0bjme.webufa.ru': 'webufa.ru',
 'xn-----8kci2achgphbdntew0a.webufa.ru': 'webufa.ru',
    'krasnodar.chio-chio.ru': 'chio-chio.ru',
 'chio-chio.ru': 'chio-chio.ru',
 'moscow.chio-chio.ru': 'chio-chio.ru',
 'ufa.chio-chio.ru': 'chio-chio.ru',
 'balashikha.chio-chio.ru': 'chio-chio.ru',
 'irkutsk.chio-chio.ru': 'chio-chio.ru',
 'mytishchi.chio-chio.ru': 'chio-chio.ru',
 'penza.chio-chio.ru': 'chio-chio.ru',
 'vladivostok.chio-chio.ru': 'chio-chio.ru',
 'kursk.chio-chio.ru': 'chio-chio.ru',
 'nizhniy-novgorod.chio-chio.ru': 'chio-chio.ru',
 'omsk.chio-chio.ru': 'chio-chio.ru',
 'naberezhnyye-chelny.chio-chio.ru': 'chio-chio.ru',
 'ekaterinburg.chio-chio.ru': 'chio-chio.ru',
 'blagoveshensk.chio-chio.ru': 'chio-chio.ru',
 'orenburg.chio-chio.ru': 'chio-chio.ru',
 'krasnoyarsk.chio-chio.ru': 'chio-chio.ru',
 'voronezh.chio-chio.ru': 'chio-chio.ru',
 'novosibirsk.chio-chio.ru': 'chio-chio.ru',
 'sankt-peterburg.chio-chio.ru': 'chio-chio.ru',
 'lyubertsy.chio-chio.ru': 'chio-chio.ru',
 'chelyabinsk.chio-chio.ru': 'chio-chio.ru',
 'saratov.chio-chio.ru': 'chio-chio.ru',
 'lipetsk.chio-chio.ru': 'chio-chio.ru',
    'agroserver.ru': 'agroserver.ru',
 'msk.agroserver.ru': 'agroserver.ru',
 'barnaul.agroserver.ru': 'agroserver.ru',
 'bo4ka.agroserver.ru': 'agroserver.ru',
 'spb.agroserver.ru': 'agroserver.ru',
 'stavropol.agroserver.ru': 'agroserver.ru',
 'rnd.agroserver.ru': 'agroserver.ru',
 'nsk.agroserver.ru': 'agroserver.ru',
 'agrofirma.agroserver.ru': 'agroserver.ru',
 'ekb.agroserver.ru': 'agroserver.ru',
 'imperiatrade.agroserver.ru': 'agroserver.ru',
 'postavka.agroserver.ru': 'agroserver.ru',
 'befresh.agroserver.ru': 'agroserver.ru',
 'terrarf.agroserver.ru': 'agroserver.ru',
 'ryazanvegetable.agroserver.ru': 'agroserver.ru',
 'celina22.agroserver.ru': 'agroserver.ru',
 'chelyabinsk.agroserver.ru': 'agroserver.ru',
 'voronezh.agroserver.ru': 'agroserver.ru',
 'zernokorm.agroserver.ru': 'agroserver.ru',
 'halaltrade.agroserver.ru': 'agroserver.ru',
 'stavrgrad.agroserver.ru': 'agroserver.ru',
 'mb.agroserver.ru': 'agroserver.ru',
 'euroeggs.agroserver.ru': 'agroserver.ru',
 'is22.agroserver.ru': 'agroserver.ru',
    'profi-rost.usue.ru': 'usue.ru',
 'indo.usue.ru': 'usue.ru',
 'civilrecht.usue.ru': 'usue.ru',
 'usue.ru': 'usue.ru',
 'sei.usue.ru': 'usue.ru',
 'meu.usue.ru': 'usue.ru',
 'etr.usue.ru': 'usue.ru',
 'fdok.usue.ru': 'usue.ru',
 'cpdo.usue.ru': 'usue.ru',
 'chemistry.usue.ru': 'usue.ru',
 'bi.usue.ru': 'usue.ru',
 'ukpt.usue.ru': 'usue.ru',
 'kimp.usue.ru': 'usue.ru',
 'econopr.usue.ru': 'usue.ru',
 'college.usue.ru': 'usue.ru',
 'ido.usue.ru': 'usue.ru',
 'kcg.usue.ru': 'usue.ru',
 'sport.usue.ru': 'usue.ru',
 'pm.usue.ru': 'usue.ru',
 'men.usue.ru': 'usue.ru',
 'umu.usue.ru': 'usue.ru',
 'publiclaw.usue.ru': 'usue.ru',
 'blc.usue.ru': 'usue.ru',
 'bua.usue.ru': 'usue.ru',
    'nsk.besposrednika.ru': 'besposrednika.ru',
 'krsk.besposrednika.ru': 'besposrednika.ru',
 'rostov-nd.besposrednika.ru': 'besposrednika.ru',
 'kzn.besposrednika.ru': 'besposrednika.ru',
 'ufa.besposrednika.ru': 'besposrednika.ru',
 'spb.besposrednika.ru': 'besposrednika.ru',
 'besposrednika.ru': 'besposrednika.ru',
 'nn.besposrednika.ru': 'besposrednika.ru',
 'msk.besposrednika.ru': 'besposrednika.ru',
 'krd.besposrednika.ru': 'besposrednika.ru',
 'tomsk.besposrednika.ru': 'besposrednika.ru',
 'kem.besposrednika.ru': 'besposrednika.ru',
 'nkz.besposrednika.ru': 'besposrednika.ru',
 'tumen.besposrednika.ru': 'besposrednika.ru',
 'perm.besposrednika.ru': 'besposrednika.ru',
 'omsk.besposrednika.ru': 'besposrednika.ru',
 'hbk.besposrednika.ru': 'besposrednika.ru',
 'chel.besposrednika.ru': 'besposrednika.ru',
 'vrn.besposrednika.ru': 'besposrednika.ru',
 'samara.besposrednika.ru': 'besposrednika.ru',
 'vgg.besposrednika.ru': 'besposrednika.ru',
 'surgut.besposrednika.ru': 'besposrednika.ru',
 'ekb.besposrednika.ru': 'besposrednika.ru',
 'irk.besposrednika.ru': 'besposrednika.ru',
    'telik.top': 'telik.top',
 'chl.telik.top': 'telik.top',
 'nsk.telik.top': 'telik.top',
 'perm.telik.top': 'telik.top',
 'ekt.telik.top': 'telik.top',
 'ufa.telik.top': 'telik.top',
 'tol.telik.top': 'telik.top',
 'vl.telik.top': 'telik.top',
 'hab.telik.top': 'telik.top',
 'krsk.telik.top': 'telik.top',
 'tomsk.telik.top': 'telik.top',
 'kem.telik.top': 'telik.top',
 'ab.telik.top': 'telik.top',
 'izh.telik.top': 'telik.top',
 'omsk.telik.top': 'telik.top',
 'spb.telik.top': 'telik.top',
 'ul.telik.top': 'telik.top',
 'tum.telik.top': 'telik.top',
 'sam.telik.top': 'telik.top',
 'or.telik.top': 'telik.top',
 'ulan.telik.top': 'telik.top',
 'nvk.telik.top': 'telik.top',
 'bar.telik.top': 'telik.top',
 'ykt.telik.top': 'telik.top',
    'detkin-club.ru': 'detkin-club.ru',
 'dav-liski.detkin-club.ru': 'detkin-club.ru',
 '9liski.detkin-club.ru': 'detkin-club.ru',
 'ds81.detkin-club.ru': 'detkin-club.ru',
 'ds77.detkin-club.ru': 'detkin-club.ru',
 'chstrana.detkin-club.ru': 'detkin-club.ru',
 '10liski.detkin-club.ru': 'detkin-club.ru',
 '1liski.detkin-club.ru': 'detkin-club.ru',
 'progim.detkin-club.ru': 'detkin-club.ru',
 'ds25kis.detkin-club.ru': 'detkin-club.ru',
 'zal-liski.detkin-club.ru': 'detkin-club.ru',
 '4liski.detkin-club.ru': 'detkin-club.ru',
 '2liski.detkin-club.ru': 'detkin-club.ru',
 '5liski.detkin-club.ru': 'detkin-club.ru',
 '8liski.detkin-club.ru': 'detkin-club.ru',
 'ds65.detkin-club.ru': 'detkin-club.ru',
 '117kurgan.detkin-club.ru': 'detkin-club.ru',
 'flower.detkin-club.ru': 'detkin-club.ru',
 'raduga.detkin-club.ru': 'detkin-club.ru',
 '72hotkovo.detkin-club.ru': 'detkin-club.ru',
 'kapitoshka.detkin-club.ru': 'detkin-club.ru',
 '8sp.detkin-club.ru': 'detkin-club.ru',
 '56sp.detkin-club.ru': 'detkin-club.ru',
 'gulliver.detkin-club.ru': 'detkin-club.ru',
    'github.io': 'github.io',
 'hammerjs.github.io': 'github.io',
 'blueimp.github.io': 'github.io',
 'mozilla.github.io': 'github.io',
 'stemkoski.github.io': 'github.io',
 'rbaulin.github.io': 'github.io',
 'gijsroge.github.io': 'github.io',
 'microsoft.github.io': 'github.io',
 'cpetry.github.io': 'github.io',
 'flexberry.github.io': 'github.io',
 'world-of-greyhawk.github.io': 'github.io',
 'react-component.github.io': 'github.io',
 'dimik.github.io': 'github.io',
 'rom1504.github.io': 'github.io',
 'gearmobile.github.io': 'github.io',
 'unetbootin.github.io': 'github.io',
 'asvd.github.io': 'github.io',
 'valve.github.io': 'github.io',
 'romeogolf.github.io': 'github.io',
 'mustache.github.io': 'github.io',
 'arogozhnikov.github.io': 'github.io',
 'colah.github.io': 'github.io',
 'forasoft.github.io': 'github.io',
 'karpathy.github.io': 'github.io',
    'tver.relax-info.ru': 'relax-info.ru',
 'relax-info.ru': 'relax-info.ru',
 'ufa.relax-info.ru': 'relax-info.ru',
 'kirov.relax-info.ru': 'relax-info.ru',
 'tolyatti.relax-info.ru': 'relax-info.ru',
 'saratov.relax-info.ru': 'relax-info.ru',
 'syktyvkar.relax-info.ru': 'relax-info.ru',
 'yoshkar-ola.relax-info.ru': 'relax-info.ru',
 'yaroslavl.relax-info.ru': 'relax-info.ru',
 'yakutsk.relax-info.ru': 'relax-info.ru',
 'tomsk.relax-info.ru': 'relax-info.ru',
 'cheboksary.relax-info.ru': 'relax-info.ru',
 'belgorod.relax-info.ru': 'relax-info.ru',
 'abakan.relax-info.ru': 'relax-info.ru',
 'kostroma.relax-info.ru': 'relax-info.ru',
 'engels.relax-info.ru': 'relax-info.ru',
 'magnitogorsk.relax-info.ru': 'relax-info.ru',
 'barnaul.relax-info.ru': 'relax-info.ru',
 'novokuznetsk.relax-info.ru': 'relax-info.ru',
 'vladimir.relax-info.ru': 'relax-info.ru',
 'khabarovsk.relax-info.ru': 'relax-info.ru',
 'voronezh.relax-info.ru': 'relax-info.ru',
 'kursk.relax-info.ru': 'relax-info.ru',
 'kaluga.relax-info.ru': 'relax-info.ru',
 'astrakhan.relax-info.ru': 'relax-info.ru',
    'tizu.ru': 'tizu.ru',
 'mariel.tizu.ru': 'tizu.ru',
 'voronezh.tizu.ru': 'tizu.ru',
 'mskobl.tizu.ru': 'tizu.ru',
 'msk.tizu.ru': 'tizu.ru',
 'perm.tizu.ru': 'tizu.ru',
 'kuzbass.tizu.ru': 'tizu.ru',
 'spb.tizu.ru': 'tizu.ru',
 'krasnodar.tizu.ru': 'tizu.ru',
 'penza.tizu.ru': 'tizu.ru',
 'pskov.tizu.ru': 'tizu.ru',
 'orenburg.tizu.ru': 'tizu.ru',
 'ivanovo.tizu.ru': 'tizu.ru',
 'lipetsk.tizu.ru': 'tizu.ru',
 'samara.tizu.ru': 'tizu.ru',
 'tumen.tizu.ru': 'tizu.ru',
 'vladivostok.tizu.ru': 'tizu.ru',
 'ufa.tizu.ru': 'tizu.ru',
 'tuva.tizu.ru': 'tizu.ru',
 'omsk.tizu.ru': 'tizu.ru',
 'chelyabinsk.tizu.ru': 'tizu.ru',
 'krasnoyarsk.tizu.ru': 'tizu.ru',
 'komi.tizu.ru': 'tizu.ru',
 'orel.tizu.ru': 'tizu.ru',
 'kazan.tizu.ru': 'tizu.ru',
    'domru.ru': 'domru.ru',
 'ryazan.wifi.domru.ru': 'domru.ru',
 'perm.wifi.domru.ru': 'domru.ru',
 'bryansk.wifi.domru.ru': 'domru.ru',
 'izhevsk.wifi.domru.ru': 'domru.ru',
 'yar.wifi.domru.ru': 'domru.ru',
 'penza.wifi.domru.ru': 'domru.ru',
 'samara.wifi.domru.ru': 'domru.ru',
 'chelny.wifi.domru.ru': 'domru.ru',
 'kirov.wifi.domru.ru': 'domru.ru',
 'nn.wifi.domru.ru': 'domru.ru',
 'volgograd.wifi.domru.ru': 'domru.ru',
 'spb.wifi.domru.ru': 'domru.ru',
 'kzn.wifi.domru.ru': 'domru.ru',
 'tmn.wifi.domru.ru': 'domru.ru',
 'yola.wifi.domru.ru': 'domru.ru',
 'ekat.wifi.domru.ru': 'domru.ru',
 'tomsk.wifi.domru.ru': 'domru.ru',
 'oren.wifi.domru.ru': 'domru.ru',
 'omsk.wifi.domru.ru': 'domru.ru',
 'lipetsk.wifi.domru.ru': 'domru.ru',
 'tula.wifi.domru.ru': 'domru.ru',
 'tver.wifi.domru.ru': 'domru.ru',
 'mgn.wifi.domru.ru': 'domru.ru',
 'barnaul.wifi.domru.ru': 'domru.ru',
 'cheb.wifi.domru.ru': 'domru.ru',
    'hd46.magnatov.cc': 'magnatov.cc',
 'hd38.magnatov.cc': 'magnatov.cc',
 'hd8.magnatov.cc': 'magnatov.cc',
 'hd62.magnatov.cc': 'magnatov.cc',
 'hd14.magnatov.cc': 'magnatov.cc',
 'hd24.magnatov.cc': 'magnatov.cc',
 'hd42.magnatov.cc': 'magnatov.cc',
 'hd63.magnatov.cc': 'magnatov.cc',
 'hd6.magnatov.cc': 'magnatov.cc',
 'hd16.magnatov.cc': 'magnatov.cc',
 'hd44.magnatov.cc': 'magnatov.cc',
 'hd59.magnatov.cc': 'magnatov.cc',
 'hd39.magnatov.cc': 'magnatov.cc',
 'hd61.magnatov.cc': 'magnatov.cc',
 'hd9.magnatov.cc': 'magnatov.cc',
 'hd22.magnatov.cc': 'magnatov.cc',
 'hd51.magnatov.cc': 'magnatov.cc',
 'hd55.magnatov.cc': 'magnatov.cc',
 'hd37.magnatov.cc': 'magnatov.cc',
 'hd30.magnatov.cc': 'magnatov.cc',
 'hd34.magnatov.cc': 'magnatov.cc',
 'hd48.magnatov.cc': 'magnatov.cc',
 'hd19.magnatov.cc': 'magnatov.cc',
 'hd65.magnatov.cc': 'magnatov.cc',
 'hd64.magnatov.cc': 'magnatov.cc',
 'hd60.magnatov.cc': 'magnatov.cc',
    'kr.auto2.info': 'auto2.info',
 'bir.auto2.info': 'auto2.info',
 'oren.auto2.info': 'auto2.info',
 'spb.auto2.info': 'auto2.info',
 'penza.auto2.info': 'auto2.info',
 'kamchatka.auto2.info': 'auto2.info',
 'tuva.auto2.info': 'auto2.info',
 'adygeya.auto2.info': 'auto2.info',
 'yar.auto2.info': 'auto2.info',
 'klgd.auto2.info': 'auto2.info',
 'cr.auto2.info': 'auto2.info',
 'ekb.auto2.info': 'auto2.info',
 'vlg.auto2.info': 'auto2.info',
 'ossetia.auto2.info': 'auto2.info',
 'ufa.auto2.info': 'auto2.info',
 'rnd.auto2.info': 'auto2.info',
 'tyumen.auto2.info': 'auto2.info',
 'murm.auto2.info': 'auto2.info',
 'kras.auto2.info': 'auto2.info',
 'khv.auto2.info': 'auto2.info',
 'kzn.auto2.info': 'auto2.info',
 'vl.auto2.info': 'auto2.info',
 'sakh.auto2.info': 'auto2.info',
 'vrn.auto2.info': 'auto2.info',
 'saratov.auto2.info': 'auto2.info',
 'ul.auto2.info': 'auto2.info',
    'ekaterinburg.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'kaliningrad.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'moskva.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'khabarovsk.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'tyumen.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'ostrov.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'nizhniy-novgorod.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'cheboksary.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'samara.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'rostov-na-donu.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'vladikavkaz.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'mozhaysk.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'stavropol.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'oryel.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'penza.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'krasnodar.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'kazan.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'kubinka.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'novokuznetsk.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'perm.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'vladivostok.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'sochi.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'vologda.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'akhtubinsk.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'sestroretsk.spravochnik-rf.ru': 'spravochnik-rf.ru',
 'barnaul.spravochnik-rf.ru': 'spravochnik-rf.ru',
    'xn--80aaa0cvac.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--e1amg1bxa.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80aze9d.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80aauks4g.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn----dtbdeglbi6acdmca3a.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80adahe6afyphh4o.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn----8sbjjjbdewlc7d.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80adbjeltrcnvok.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80atblfjdfd2l.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--j1adfnc.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--j1adhk.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--c1azcgcc.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--b1ag7af7c.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn----ctbbefcvajij3anca5a.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80antj7do.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80acgfbsl1azdqr.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--e1aner7ci.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80aacf4bwnk3a.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80adbv1agb.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn----btbbmf6ajhaeq.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--90aee6admdx.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--h1aliz.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80aai0ag2c.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--80aag1ciek.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
 'xn--b1acdycodcb5a.xn----ctbfgcmoftedf0d.xn--p1ai': 'xn----ctbfgcmoftedf0d.xn--p1ai',
    'moskva.ya77.ru': 'ya77.ru',
 'pushkino.ya77.ru': 'ya77.ru',
 'zheleznodorozhnyy.ya77.ru': 'ya77.ru',
 'shchelkovo.ya77.ru': 'ya77.ru',
 'vidnoe.ya77.ru': 'ya77.ru',
 'noginsk.ya77.ru': 'ya77.ru',
 'balashikha.ya77.ru': 'ya77.ru',
 'domodedovo.ya77.ru': 'ya77.ru',
 'elektrostal.ya77.ru': 'ya77.ru',
 'podolsk.ya77.ru': 'ya77.ru',
 'serpukhov.ya77.ru': 'ya77.ru',
 'dmitrov.ya77.ru': 'ya77.ru',
 'sergiev-posad.ya77.ru': 'ya77.ru',
 'dzerzhinskiy.ya77.ru': 'ya77.ru',
 'mytishchi.ya77.ru': 'ya77.ru',
 'kolomna.ya77.ru': 'ya77.ru',
 'zhukovskiy.ya77.ru': 'ya77.ru',
 'khimki.ya77.ru': 'ya77.ru',
 'lyubertsy.ya77.ru': 'ya77.ru',
 'solnechnogorsk.ya77.ru': 'ya77.ru',
 'zelenograd.ya77.ru': 'ya77.ru',
 'dedovsk.ya77.ru': 'ya77.ru',
 'dolgoprudnyy.ya77.ru': 'ya77.ru',
 'voskresensk.ya77.ru': 'ya77.ru',
 'odintsovo.ya77.ru': 'ya77.ru',
 'zvenigorod.ya77.ru': 'ya77.ru',
    'goup.ru': 'goup.ru',
 'krasnoyarsk.goup.ru': 'goup.ru',
 'samara.goup.ru': 'goup.ru',
 'vladivostok.goup.ru': 'goup.ru',
 'novosib.goup.ru': 'goup.ru',
 'voronezh.goup.ru': 'goup.ru',
 'belgorod.goup.ru': 'goup.ru',
 'joshkar-ola.goup.ru': 'goup.ru',
 'ryazan.goup.ru': 'goup.ru',
 'syktyvkar.goup.ru': 'goup.ru',
 'perm.goup.ru': 'goup.ru',
 'kemerovo.goup.ru': 'goup.ru',
 'kurgan.goup.ru': 'goup.ru',
 'khabarovsk.goup.ru': 'goup.ru',
 'n-novgorod.goup.ru': 'goup.ru',
 'krasnodar.goup.ru': 'goup.ru',
 'kazan.goup.ru': 'goup.ru',
 'rostov.goup.ru': 'goup.ru',
 'volgograd.goup.ru': 'goup.ru',
 'orel.goup.ru': 'goup.ru',
 'kaluga.goup.ru': 'goup.ru',
 'ivanovo.goup.ru': 'goup.ru',
 'irkutsk.goup.ru': 'goup.ru',
 'cheboksary.goup.ru': 'goup.ru',
 'barnaul.goup.ru': 'goup.ru',
 'novgorod.goup.ru': 'goup.ru',
    'n13.kinoroom.link': 'kinoroom.link',
 'n3.kinoroom.link': 'kinoroom.link',
 'n15.kinoroom.link': 'kinoroom.link',
 'ru3.kinoroom.link': 'kinoroom.link',
 'n27.kinoroom.link': 'kinoroom.link',
 'n7.kinoroom.link': 'kinoroom.link',
 'n26.kinoroom.link': 'kinoroom.link',
 'n11.kinoroom.link': 'kinoroom.link',
 'n5.kinoroom.link': 'kinoroom.link',
 'n19.kinoroom.link': 'kinoroom.link',
 'n9.kinoroom.link': 'kinoroom.link',
 'n20.kinoroom.link': 'kinoroom.link',
 'n22.kinoroom.link': 'kinoroom.link',
 'n24.kinoroom.link': 'kinoroom.link',
 'n25.kinoroom.link': 'kinoroom.link',
 'n17.kinoroom.link': 'kinoroom.link',
 'n16.kinoroom.link': 'kinoroom.link',
 'n23.kinoroom.link': 'kinoroom.link',
 'n8.kinoroom.link': 'kinoroom.link',
 'n14.kinoroom.link': 'kinoroom.link',
 'n18.kinoroom.link': 'kinoroom.link',
 'n21.kinoroom.link': 'kinoroom.link',
 'nu.kinoroom.link': 'kinoroom.link',
 'n10.kinoroom.link': 'kinoroom.link',
 'n4.kinoroom.link': 'kinoroom.link',
 'n12.kinoroom.link': 'kinoroom.link',
 'n6.kinoroom.link': 'kinoroom.link',
    'dou40spb.petersburgedu.ru': 'petersburgedu.ru',
 'kpt.petersburgedu.ru': 'petersburgedu.ru',
 'gym652.petersburgedu.ru': 'petersburgedu.ru',
 'sch103.petersburgedu.ru': 'petersburgedu.ru',
 'sch055.petersburgedu.ru': 'petersburgedu.ru',
 'cor584.petersburgedu.ru': 'petersburgedu.ru',
 'crt.petersburgedu.ru': 'petersburgedu.ru',
 'sch072.petersburgedu.ru': 'petersburgedu.ru',
 'gym664.petersburgedu.ru': 'petersburgedu.ru',
 'sch002.petersburgedu.ru': 'petersburgedu.ru',
 'sch075.petersburgedu.ru': 'petersburgedu.ru',
 'sch158.petersburgedu.ru': 'petersburgedu.ru',
 'sch185.petersburgedu.ru': 'petersburgedu.ru',
 'sch145.petersburgedu.ru': 'petersburgedu.ru',
 'dou70spb.petersburgedu.ru': 'petersburgedu.ru',
 'sch127.petersburgedu.ru': 'petersburgedu.ru',
 'sch121.petersburgedu.ru': 'petersburgedu.ru',
 'dou3spb.petersburgedu.ru': 'petersburgedu.ru',
 'sch220.petersburgedu.ru': 'petersburgedu.ru',
 'sch128.petersburgedu.ru': 'petersburgedu.ru',
 'ced001.petersburgedu.ru': 'petersburgedu.ru',
 'cor003krgv.petersburgedu.ru': 'petersburgedu.ru',
 'sch091.petersburgedu.ru': 'petersburgedu.ru',
 'cor025.petersburgedu.ru': 'petersburgedu.ru',
 'sch390.petersburgedu.ru': 'petersburgedu.ru',
 'odo98spb.petersburgedu.ru': 'petersburgedu.ru',
 'sch558.petersburgedu.ru': 'petersburgedu.ru',
    'novosibirsk.vsesrazu.su': 'vsesrazu.su',
 'vsesrazu.su': 'vsesrazu.su',
 'ufa.vsesrazu.su': 'vsesrazu.su',
 'sankt-peterburg.vsesrazu.su': 'vsesrazu.su',
 'krasnoyarsk.vsesrazu.su': 'vsesrazu.su',
 'izhevsk.vsesrazu.su': 'vsesrazu.su',
 'vladivostok.vsesrazu.su': 'vsesrazu.su',
 'kazan.vsesrazu.su': 'vsesrazu.su',
 'habarovsk.vsesrazu.su': 'vsesrazu.su',
 'perm.vsesrazu.su': 'vsesrazu.su',
 'samara.vsesrazu.su': 'vsesrazu.su',
 'nizhniy-novgorod.vsesrazu.su': 'vsesrazu.su',
 'yaroslavl.vsesrazu.su': 'vsesrazu.su',
 'belgorod.vsesrazu.su': 'vsesrazu.su',
 'kirov.vsesrazu.su': 'vsesrazu.su',
 'barnaul.vsesrazu.su': 'vsesrazu.su',
 'tyumen.vsesrazu.su': 'vsesrazu.su',
 'naberezhnie-chelny.vsesrazu.su': 'vsesrazu.su',
 'ekaterinburg.vsesrazu.su': 'vsesrazu.su',
 'tver.vsesrazu.su': 'vsesrazu.su',
 'ivanovo.vsesrazu.su': 'vsesrazu.su',
 'krasnodar.vsesrazu.su': 'vsesrazu.su',
 'rostov-na-donu.vsesrazu.su': 'vsesrazu.su',
 'ulyanovsk.vsesrazu.su': 'vsesrazu.su',
 'cheboksary.vsesrazu.su': 'vsesrazu.su',
 'kemerovo.vsesrazu.su': 'vsesrazu.su',
 'voronezh.vsesrazu.su': 'vsesrazu.su',
    'spb.flado.ru': 'flado.ru',
 'msk.flado.ru': 'flado.ru',
 'novosibirskaya.flado.ru': 'flado.ru',
 'krasnoyarski.flado.ru': 'flado.ru',
 'rostovskaya.flado.ru': 'flado.ru',
 'voronezskaya.flado.ru': 'flado.ru',
 'samarskaya.flado.ru': 'flado.ru',
 'krasnodarski.flado.ru': 'flado.ru',
 'tatarstan.flado.ru': 'flado.ru',
 'primorski.flado.ru': 'flado.ru',
 'yaroslavskaya.flado.ru': 'flado.ru',
 'omskaya.flado.ru': 'flado.ru',
 'saratovskaya.flado.ru': 'flado.ru',
 'bashkortostan.flado.ru': 'flado.ru',
 'belgorodskaya.flado.ru': 'flado.ru',
 'kostromskaya.flado.ru': 'flado.ru',
 'flado.ru': 'flado.ru',
 'pskovskaya.flado.ru': 'flado.ru',
 'vologodskaya.flado.ru': 'flado.ru',
 'dagestan.flado.ru': 'flado.ru',
 'tomskaya.flado.ru': 'flado.ru',
 'kurganskaya.flado.ru': 'flado.ru',
 'chelyabinskaya.flado.ru': 'flado.ru',
 'orenburgskaya.flado.ru': 'flado.ru',
 'altaiski.flado.ru': 'flado.ru',
 'sverdlovskaya.flado.ru': 'flado.ru',
 'krim.flado.ru': 'flado.ru',
 'permski.flado.ru': 'flado.ru',
    'novosibirsk.elektrichki.net': 'elektrichki.net',
 'elektrichki.net': 'elektrichki.net',
 'vladivostok.elektrichki.net': 'elektrichki.net',
 'krasnoyarsk.elektrichki.net': 'elektrichki.net',
 'rostov.elektrichki.net': 'elektrichki.net',
 'kaliningrad.elektrichki.net': 'elektrichki.net',
 'moskva.elektrichki.net': 'elektrichki.net',
 'saratov.elektrichki.net': 'elektrichki.net',
 'ufa.elektrichki.net': 'elektrichki.net',
 'sankt-peterburg.elektrichki.net': 'elektrichki.net',
 'samara.elektrichki.net': 'elektrichki.net',
 'krasnodar.elektrichki.net': 'elektrichki.net',
 'orenburg.elektrichki.net': 'elektrichki.net',
 'ryazan.elektrichki.net': 'elektrichki.net',
 'kazan.elektrichki.net': 'elektrichki.net',
 'voronezh.elektrichki.net': 'elektrichki.net',
 'vladimir.elektrichki.net': 'elektrichki.net',
 'kemerovo.elektrichki.net': 'elektrichki.net',
 'omsk.elektrichki.net': 'elektrichki.net',
 'habarovsk.elektrichki.net': 'elektrichki.net',
 'chelyabinsk.elektrichki.net': 'elektrichki.net',
 'perm.elektrichki.net': 'elektrichki.net',
 'ekaterinburg.elektrichki.net': 'elektrichki.net',
 'tver.elektrichki.net': 'elektrichki.net',
 'stavropol.elektrichki.net': 'elektrichki.net',
 'volgograd.elektrichki.net': 'elektrichki.net',
 'yaroslavl.elektrichki.net': 'elektrichki.net',
 'tyumen.elektrichki.net': 'elektrichki.net',
    'novosibirsk.sprav.co': 'sprav.co',
 'sprav.co': 'sprav.co',
 'omsk.sprav.co': 'sprav.co',
 'ekaterinburg.sprav.co': 'sprav.co',
 'chelyabinsk.sprav.co': 'sprav.co',
 'novorossiysk.sprav.co': 'sprav.co',
 'nabchelny.sprav.co': 'sprav.co',
 'barnaul.sprav.co': 'sprav.co',
 'tomsk.sprav.co': 'sprav.co',
 'pkamchatskiy.sprav.co': 'sprav.co',
 'krasnoyarsk.sprav.co': 'sprav.co',
 'novokuznetsk.sprav.co': 'sprav.co',
 'nizhnevartovsk.sprav.co': 'sprav.co',
 'cheboksary.sprav.co': 'sprav.co',
 'abakan.sprav.co': 'sprav.co',
 'kemerovo.sprav.co': 'sprav.co',
 'krasnodar.sprav.co': 'sprav.co',
 'vladivostok.sprav.co': 'sprav.co',
 'moscow.sprav.co': 'sprav.co',
 'tyumen.sprav.co': 'sprav.co',
 'kurgan.sprav.co': 'sprav.co',
 'kazan.sprav.co': 'sprav.co',
 'armawir.sprav.co': 'sprav.co',
 'samara.sprav.co': 'sprav.co',
 'voronezh.sprav.co': 'sprav.co',
 'khabarovsk.sprav.co': 'sprav.co',
 'vologda.sprav.co': 'sprav.co',
 'miass.sprav.co': 'sprav.co',
    'stvrp.wikiotzyv.org': 'wikiotzyv.org',
 'vlggr.wikiotzyv.org': 'wikiotzyv.org',
 'smr.wikiotzyv.org': 'wikiotzyv.org',
 'ttrstn.wikiotzyv.org': 'wikiotzyv.org',
 'chvsh.wikiotzyv.org': 'wikiotzyv.org',
 'bshkrtstn.wikiotzyv.org': 'wikiotzyv.org',
 'pnzn.wikiotzyv.org': 'wikiotzyv.org',
 'nvsb.wikiotzyv.org': 'wikiotzyv.org',
 'msk.wikiotzyv.org': 'wikiotzyv.org',
 'krsnrk.wikiotzyv.org': 'wikiotzyv.org',
 'krsndr.wikiotzyv.org': 'wikiotzyv.org',
 'prmrsk.wikiotzyv.org': 'wikiotzyv.org',
 'mrl.wikiotzyv.org': 'wikiotzyv.org',
 'lnngr.wikiotzyv.org': 'wikiotzyv.org',
 'astrhn.wikiotzyv.org': 'wikiotzyv.org',
 'srtvsk.wikiotzyv.org': 'wikiotzyv.org',
 'hm.wikiotzyv.org': 'wikiotzyv.org',
 'vldmr.wikiotzyv.org': 'wikiotzyv.org',
 'svrdlv.wikiotzyv.org': 'wikiotzyv.org',
 'tmn.wikiotzyv.org': 'wikiotzyv.org',
 'amr.wikiotzyv.org': 'wikiotzyv.org',
 'chlbn.wikiotzyv.org': 'wikiotzyv.org',
 'tvr.wikiotzyv.org': 'wikiotzyv.org',
 'wikiotzyv.org': 'wikiotzyv.org',
 'nzhgr.wikiotzyv.org': 'wikiotzyv.org',
 'hks.wikiotzyv.org': 'wikiotzyv.org',
 'rstv.wikiotzyv.org': 'wikiotzyv.org',
 'smln.wikiotzyv.org': 'wikiotzyv.org',
    'kirsanov.ikoleso.co': 'ikoleso.co',
 'ikoleso.co': 'ikoleso.co',
 'nignekamsk.ikoleso.co': 'ikoleso.co',
 'moskva.ikoleso.co': 'ikoleso.co',
 'novomoskovsk.ikoleso.co': 'ikoleso.co',
 'blagoveshchensk.ikoleso.co': 'ikoleso.co',
 'kaliningrad.ikoleso.co': 'ikoleso.co',
 'salsk.ikoleso.co': 'ikoleso.co',
 'kamishin.ikoleso.co': 'ikoleso.co',
 'pervouralsk.ikoleso.co': 'ikoleso.co',
 'neftekamsk.ikoleso.co': 'ikoleso.co',
 'korolyov.ikoleso.co': 'ikoleso.co',
 'novokuznetsk.ikoleso.co': 'ikoleso.co',
 'uglich.ikoleso.co': 'ikoleso.co',
 'krasnoyarsk.ikoleso.co': 'ikoleso.co',
 'kansk.ikoleso.co': 'ikoleso.co',
 'elektrogorsk.ikoleso.co': 'ikoleso.co',
 'vladivostok.ikoleso.co': 'ikoleso.co',
 'tuymazi.ikoleso.co': 'ikoleso.co',
 'novosibirsk.ikoleso.co': 'ikoleso.co',
 'shatura.ikoleso.co': 'ikoleso.co',
 'zaprudnya.ikoleso.co': 'ikoleso.co',
 'uhta.ikoleso.co': 'ikoleso.co',
 'barnaul.ikoleso.co': 'ikoleso.co',
 'abakan.ikoleso.co': 'ikoleso.co',
 'aleksandrov.ikoleso.co': 'ikoleso.co',
 'saratov.ikoleso.co': 'ikoleso.co',
 'liski.ikoleso.co': 'ikoleso.co',
    'spb.egent.ru': 'egent.ru',
 'krasnodar.egent.ru': 'egent.ru',
 'balashiha.egent.ru': 'egent.ru',
 'tomsk.egent.ru': 'egent.ru',
 'egent.ru': 'egent.ru',
 'ufa.egent.ru': 'egent.ru',
 'chelyabinsk.egent.ru': 'egent.ru',
 'troitsk.egent.ru': 'egent.ru',
 'belgorod.egent.ru': 'egent.ru',
 'novosibirsk.egent.ru': 'egent.ru',
 'yaroslavl.egent.ru': 'egent.ru',
 'kazan.egent.ru': 'egent.ru',
 'kemerovo.egent.ru': 'egent.ru',
 'nigniy-novgorod.egent.ru': 'egent.ru',
 'krasnoyarsk.egent.ru': 'egent.ru',
 'rostov-na-donu.egent.ru': 'egent.ru',
 'saratov.egent.ru': 'egent.ru',
 'sevastopol.egent.ru': 'egent.ru',
 'voroneg.egent.ru': 'egent.ru',
 'kursk.egent.ru': 'egent.ru',
 'vladimir.egent.ru': 'egent.ru',
 'sochi.egent.ru': 'egent.ru',
 'lipetsk.egent.ru': 'egent.ru',
 'podolsk.egent.ru': 'egent.ru',
 'novorossijsk.egent.ru': 'egent.ru',
 'tyumen.egent.ru': 'egent.ru',
 'kaliningrad.egent.ru': 'egent.ru',
 'ekaterinburg.egent.ru': 'egent.ru',
    'autodmir.ru': 'autodmir.ru',
 'kaluga.autodmir.ru': 'autodmir.ru',
 'chelyabinsk.autodmir.ru': 'autodmir.ru',
 'kursk.autodmir.ru': 'autodmir.ru',
 'ufa.autodmir.ru': 'autodmir.ru',
 'samara.autodmir.ru': 'autodmir.ru',
 'kaliningrad.autodmir.ru': 'autodmir.ru',
 'nn.autodmir.ru': 'autodmir.ru',
 'krasnoyarsk.autodmir.ru': 'autodmir.ru',
 'kemerovo.autodmir.ru': 'autodmir.ru',
 'saratov.autodmir.ru': 'autodmir.ru',
 'omsk.autodmir.ru': 'autodmir.ru',
 'tyumen.autodmir.ru': 'autodmir.ru',
 'izhevsk.autodmir.ru': 'autodmir.ru',
 'ekb.autodmir.ru': 'autodmir.ru',
 'kazan.autodmir.ru': 'autodmir.ru',
 'petrozavodsk.autodmir.ru': 'autodmir.ru',
 'voronezh.autodmir.ru': 'autodmir.ru',
 'lipetsk.autodmir.ru': 'autodmir.ru',
 'orenburg.autodmir.ru': 'autodmir.ru',
 'tomsk.autodmir.ru': 'autodmir.ru',
 'barnaul.autodmir.ru': 'autodmir.ru',
 'stavropol.autodmir.ru': 'autodmir.ru',
 'khanty-mansiysk.autodmir.ru': 'autodmir.ru',
 'vladivostok.autodmir.ru': 'autodmir.ru',
 'vologda.autodmir.ru': 'autodmir.ru',
 'ryazan.autodmir.ru': 'autodmir.ru',
 'volgograd.autodmir.ru': 'autodmir.ru',
    'vorkuta.infrus.ru': 'infrus.ru',
 'novosibirsk.infrus.ru': 'infrus.ru',
 'krasnoyarsk.infrus.ru': 'infrus.ru',
 'ekaterinburg.infrus.ru': 'infrus.ru',
 'vladivostok.infrus.ru': 'infrus.ru',
 'krasnodar.infrus.ru': 'infrus.ru',
 'kaliningrad.infrus.ru': 'infrus.ru',
 'tomsk.infrus.ru': 'infrus.ru',
 'rostov-na-donu.infrus.ru': 'infrus.ru',
 'yoshkar-ola.infrus.ru': 'infrus.ru',
 'velikiy-novgorod.infrus.ru': 'infrus.ru',
 'abakan.infrus.ru': 'infrus.ru',
 'samara.infrus.ru': 'infrus.ru',
 'ufa.infrus.ru': 'infrus.ru',
 'kemerovo.infrus.ru': 'infrus.ru',
 'tyumen.infrus.ru': 'infrus.ru',
 'volgograd.infrus.ru': 'infrus.ru',
 'chelyabinsk.infrus.ru': 'infrus.ru',
 'spb-obl.infrus.ru': 'infrus.ru',
 'omsk.infrus.ru': 'infrus.ru',
 'mo.infrus.ru': 'infrus.ru',
 'spb.infrus.ru': 'infrus.ru',
 'petrozavodsk.infrus.ru': 'infrus.ru',
 'arhangelsk.infrus.ru': 'infrus.ru',
 'perm.infrus.ru': 'infrus.ru',
 'stavropol.infrus.ru': 'infrus.ru',
 'tula.infrus.ru': 'infrus.ru',
 'barnaul.infrus.ru': 'infrus.ru',
 'kaluga.infrus.ru': 'infrus.ru',
    'chelyabinsk.vyboruslug.com': 'vyboruslug.com',
 'vyboruslug.com': 'vyboruslug.com',
 'rubtsovsk.vyboruslug.com': 'vyboruslug.com',
 'pskov.vyboruslug.com': 'vyboruslug.com',
 'barnaul.vyboruslug.com': 'vyboruslug.com',
 'vladimir.vyboruslug.com': 'vyboruslug.com',
 'izhevsk.vyboruslug.com': 'vyboruslug.com',
 'vologda.vyboruslug.com': 'vyboruslug.com',
 'kirov.vyboruslug.com': 'vyboruslug.com',
 'nizhniy-novgorod.vyboruslug.com': 'vyboruslug.com',
 'bryansk.vyboruslug.com': 'vyboruslug.com',
 'ufa.vyboruslug.com': 'vyboruslug.com',
 'perm.vyboruslug.com': 'vyboruslug.com',
 'saransk.vyboruslug.com': 'vyboruslug.com',
 'moskva.vyboruslug.com': 'vyboruslug.com',
 'surgut.vyboruslug.com': 'vyboruslug.com',
 'novosibirsk.vyboruslug.com': 'vyboruslug.com',
 'nizhnekamsk.vyboruslug.com': 'vyboruslug.com',
 'yakutsk.vyboruslug.com': 'vyboruslug.com',
 'nizhnevartovsk.vyboruslug.com': 'vyboruslug.com',
 'orenburg.vyboruslug.com': 'vyboruslug.com',
 'ekaterinburg.vyboruslug.com': 'vyboruslug.com',
 'almetevsk.vyboruslug.com': 'vyboruslug.com',
 'spb.vyboruslug.com': 'vyboruslug.com',
 'kazan.vyboruslug.com': 'vyboruslug.com',
 'kurgan.vyboruslug.com': 'vyboruslug.com',
 'murmansk.vyboruslug.com': 'vyboruslug.com',
 'syktyvkar.vyboruslug.com': 'vyboruslug.com',
 'achinsk.vyboruslug.com': 'vyboruslug.com',
    'kiev.ukrgo.com': 'ukrgo.com',
 'kr.ukrgo.com': 'ukrgo.com',
 'ukrgo.com': 'ukrgo.com',
 'khartsyzk.ukrgo.com': 'ukrgo.com',
 'lugansk.ukrgo.com': 'ukrgo.com',
 'enakievo.ukrgo.com': 'ukrgo.com',
 'berdyansk.ukrgo.com': 'ukrgo.com',
 'nikolaev.ukrgo.com': 'ukrgo.com',
 'uzhgorod.ukrgo.com': 'ukrgo.com',
 'odessa.ukrgo.com': 'ukrgo.com',
 'vinnica.ukrgo.com': 'ukrgo.com',
 'kharkov.ukrgo.com': 'ukrgo.com',
 'melitopol.ukrgo.com': 'ukrgo.com',
 'simferopol.ukrgo.com': 'ukrgo.com',
 'zp.ukrgo.com': 'ukrgo.com',
 'if.ukrgo.com': 'ukrgo.com',
 'donetsk.ukrgo.com': 'ukrgo.com',
 'kherson.ukrgo.com': 'ukrgo.com',
 'chernigov.ukrgo.com': 'ukrgo.com',
 'mariupol.ukrgo.com': 'ukrgo.com',
 'lvov.ukrgo.com': 'ukrgo.com',
 'chernovcy.ukrgo.com': 'ukrgo.com',
 'dp.ukrgo.com': 'ukrgo.com',
 'feodosia.ukrgo.com': 'ukrgo.com',
 'yalta.ukrgo.com': 'ukrgo.com',
 'nm.ukrgo.com': 'ukrgo.com',
 'kerch.ukrgo.com': 'ukrgo.com',
 'lisichansk.ukrgo.com': 'ukrgo.com',
 'pervomaysk.ukrgo.com': 'ukrgo.com',
    'yoshkar-ola.gde-sto.ru': 'gde-sto.ru',
 'miass.gde-sto.ru': 'gde-sto.ru',
 'gde-sto.ru': 'gde-sto.ru',
 'noyabrsk.gde-sto.ru': 'gde-sto.ru',
 'sankt-peterburg.gde-sto.ru': 'gde-sto.ru',
 'novorossiysk.gde-sto.ru': 'gde-sto.ru',
 'abakan.gde-sto.ru': 'gde-sto.ru',
 'ekaterinburg.gde-sto.ru': 'gde-sto.ru',
 'naberezhnyie-chelnyi.gde-sto.ru': 'gde-sto.ru',
 'ufa.gde-sto.ru': 'gde-sto.ru',
 'kirov.gde-sto.ru': 'gde-sto.ru',
 'smolensk.gde-sto.ru': 'gde-sto.ru',
 'barnaul.gde-sto.ru': 'gde-sto.ru',
 'petropavlovsk-kamchatskiy.gde-sto.ru': 'gde-sto.ru',
 'sterlitamak.gde-sto.ru': 'gde-sto.ru',
 'norilsk.gde-sto.ru': 'gde-sto.ru',
 'samara.gde-sto.ru': 'gde-sto.ru',
 'nahodka.gde-sto.ru': 'gde-sto.ru',
 'moskva.gde-sto.ru': 'gde-sto.ru',
 'tyumen.gde-sto.ru': 'gde-sto.ru',
 'biysk.gde-sto.ru': 'gde-sto.ru',
 'stavropol.gde-sto.ru': 'gde-sto.ru',
 'izhevsk.gde-sto.ru': 'gde-sto.ru',
 'penza.gde-sto.ru': 'gde-sto.ru',
 'kamensk-uralskiy.gde-sto.ru': 'gde-sto.ru',
 'tambov.gde-sto.ru': 'gde-sto.ru',
 'almetevsk.gde-sto.ru': 'gde-sto.ru',
 'perm.gde-sto.ru': 'gde-sto.ru',
 'cheboksaryi.gde-sto.ru': 'gde-sto.ru',
    'cataloxy.ru': 'cataloxy.ru',
 'naberezhnie-chelni.cataloxy.ru': 'cataloxy.ru',
 'chita.cataloxy.ru': 'cataloxy.ru',
 'sverdlovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'vladimir.cataloxy.ru': 'cataloxy.ru',
 'kurganinsk.cataloxy.ru': 'cataloxy.ru',
 'barnaul.cataloxy.ru': 'cataloxy.ru',
 'petrozavodsk.cataloxy.ru': 'cataloxy.ru',
 'kaliningrad.cataloxy.ru': 'cataloxy.ru',
 'tver.cataloxy.ru': 'cataloxy.ru',
 'ulyanovsk.cataloxy.ru': 'cataloxy.ru',
 'rostov-na-donu.cataloxy.ru': 'cataloxy.ru',
 'ivanteevka.cataloxy.ru': 'cataloxy.ru',
 'tatarstan-respublika.cataloxy.ru': 'cataloxy.ru',
 'magnitogorsk.cataloxy.ru': 'cataloxy.ru',
 'norilsk.cataloxy.ru': 'cataloxy.ru',
 'sovetsk.cataloxy.ru': 'cataloxy.ru',
 'olenegorsk.cataloxy.ru': 'cataloxy.ru',
 'murmanskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'novorossisk.cataloxy.ru': 'cataloxy.ru',
 'ussuriysk.cataloxy.ru': 'cataloxy.ru',
 'klin.cataloxy.ru': 'cataloxy.ru',
 'chelyabinsk.cataloxy.ru': 'cataloxy.ru',
 'severodvinsk.cataloxy.ru': 'cataloxy.ru',
 'sevastopol.cataloxy.ru': 'cataloxy.ru',
 'elabuga.cataloxy.ru': 'cataloxy.ru',
 'sasovo.cataloxy.ru': 'cataloxy.ru',
    'volgograd.yellmarket.ru': 'yellmarket.ru',
 'ufa.yellmarket.ru': 'yellmarket.ru',
 'tula.yellmarket.ru': 'yellmarket.ru',
 'tyumen.yellmarket.ru': 'yellmarket.ru',
 'komi.yellmarket.ru': 'yellmarket.ru',
 'orenburg.yellmarket.ru': 'yellmarket.ru',
 'chelyabinsk.yellmarket.ru': 'yellmarket.ru',
 'smolensk.yellmarket.ru': 'yellmarket.ru',
 'petersburg.yellmarket.ru': 'yellmarket.ru',
 'petrozavodsk.yellmarket.ru': 'yellmarket.ru',
 'moscow.yellmarket.ru': 'yellmarket.ru',
 'makhachkala.yellmarket.ru': 'yellmarket.ru',
 'cherkessk.yellmarket.ru': 'yellmarket.ru',
 'rostov.yellmarket.ru': 'yellmarket.ru',
 'vladivostok.yellmarket.ru': 'yellmarket.ru',
 'birobidzhan.yellmarket.ru': 'yellmarket.ru',
 'arkhangelsk.yellmarket.ru': 'yellmarket.ru',
 'saratov.yellmarket.ru': 'yellmarket.ru',
 'vladimir.yellmarket.ru': 'yellmarket.ru',
 'moscowob.yellmarket.ru': 'yellmarket.ru',
 'khabarovsk.yellmarket.ru': 'yellmarket.ru',
 'nnovgorod.yellmarket.ru': 'yellmarket.ru',
 'astrakhan.yellmarket.ru': 'yellmarket.ru',
 'kemerovo.yellmarket.ru': 'yellmarket.ru',
 'barnaul.yellmarket.ru': 'yellmarket.ru',
 'izhevsk.yellmarket.ru': 'yellmarket.ru',
 'tambov.yellmarket.ru': 'yellmarket.ru',
 'vologda.yellmarket.ru': 'yellmarket.ru',
 'belgorod.yellmarket.ru': 'yellmarket.ru',
 'perm.yellmarket.ru': 'yellmarket.ru',
    'carsguru.net': 'carsguru.net',
 'spb.carsguru.net': 'carsguru.net',
 'forums.carsguru.net': 'carsguru.net',
 'ekaterinburg.carsguru.net': 'carsguru.net',
 'tyumen.carsguru.net': 'carsguru.net',
 'yerevan.carsguru.net': 'carsguru.net',
 'novosibirsk.carsguru.net': 'carsguru.net',
 'izhevsk.carsguru.net': 'carsguru.net',
 'eysk.carsguru.net': 'carsguru.net',
 'vladivostok.carsguru.net': 'carsguru.net',
 'ufa.carsguru.net': 'carsguru.net',
 'krasnodar.carsguru.net': 'carsguru.net',
 'hanti-mansiysk.carsguru.net': 'carsguru.net',
 'tomsk.carsguru.net': 'carsguru.net',
 'magnitogorsk.carsguru.net': 'carsguru.net',
 'samara.carsguru.net': 'carsguru.net',
 'kirov.carsguru.net': 'carsguru.net',
 'mineralnye-vody.carsguru.net': 'carsguru.net',
 'surgut.carsguru.net': 'carsguru.net',
 'murmansk.carsguru.net': 'carsguru.net',
 'kemerovo.carsguru.net': 'carsguru.net',
 'maykop.carsguru.net': 'carsguru.net',
 'region52.carsguru.net': 'carsguru.net',
 'region42.carsguru.net': 'carsguru.net',
 'kazan.carsguru.net': 'carsguru.net',
 'rostow-na-donu.carsguru.net': 'carsguru.net',
 'habarovsk.carsguru.net': 'carsguru.net',
 'vladikavkaz.carsguru.net': 'carsguru.net',
 'komsomolsk-na-amure.carsguru.net': 'carsguru.net',
 'skoda.carsguru.net': 'carsguru.net',
    'nur.isfirm.ru': 'isfirm.ru',
 'nev.isfirm.ru': 'isfirm.ru',
 'nnv.isfirm.ru': 'isfirm.ru',
 'ekb.isfirm.ru': 'isfirm.ru',
 'isfirm.ru': 'isfirm.ru',
 'brest.isfirm.ru': 'isfirm.ru',
 'rezh.isfirm.ru': 'isfirm.ru',
 'kry.isfirm.ru': 'isfirm.ru',
 'orb.isfirm.ru': 'isfirm.ru',
 'kmr.isfirm.ru': 'isfirm.ru',
 'msk.isfirm.ru': 'isfirm.ru',
 'krd.isfirm.ru': 'isfirm.ru',
 'lobnya.isfirm.ru': 'isfirm.ru',
 'ufa.isfirm.ru': 'isfirm.ru',
 'sam.isfirm.ru': 'isfirm.ru',
 'makiivka.isfirm.ru': 'isfirm.ru',
 'sta.isfirm.ru': 'isfirm.ru',
 'ryz.isfirm.ru': 'isfirm.ru',
 'yar.isfirm.ru': 'isfirm.ru',
 'sht.isfirm.ru': 'isfirm.ru',
 'prokhladny.isfirm.ru': 'isfirm.ru',
 'lukhovitsy.isfirm.ru': 'isfirm.ru',
 'ostashkov.isfirm.ru': 'isfirm.ru',
 'tikhoretsk.isfirm.ru': 'isfirm.ru',
 'ors.isfirm.ru': 'isfirm.ru',
 'rnd.isfirm.ru': 'isfirm.ru',
 'sch.isfirm.ru': 'isfirm.ru',
 'krg.isfirm.ru': 'isfirm.ru',
 'buguruslan.isfirm.ru': 'isfirm.ru',
 'solnechnogorsk.isfirm.ru': 'isfirm.ru',
    'mebel-store.com': 'mebel-store.com',
 'samara.mebel-store.com': 'mebel-store.com',
 'lensk.mebel-store.com': 'mebel-store.com',
 'tuymazy.mebel-store.com': 'mebel-store.com',
 'kurchatov.mebel-store.com': 'mebel-store.com',
 'dyatkovo.mebel-store.com': 'mebel-store.com',
 'mozhga.mebel-store.com': 'mebel-store.com',
 'roslavl.mebel-store.com': 'mebel-store.com',
 'norilsk.mebel-store.com': 'mebel-store.com',
 'nsk.mebel-store.com': 'mebel-store.com',
 'kirov.mebel-store.com': 'mebel-store.com',
 'penza.mebel-store.com': 'mebel-store.com',
 'essentuki.mebel-store.com': 'mebel-store.com',
 'khabarovsk.mebel-store.com': 'mebel-store.com',
 'komsomolsk-na-amure.mebel-store.com': 'mebel-store.com',
 'leningradskaya.mebel-store.com': 'mebel-store.com',
 'novotroitsk.mebel-store.com': 'mebel-store.com',
 'apatity.mebel-store.com': 'mebel-store.com',
 'timashevsk.mebel-store.com': 'mebel-store.com',
 'elets.mebel-store.com': 'mebel-store.com',
 'moscow.mebel-store.com': 'mebel-store.com',
 'belovo.mebel-store.com': 'mebel-store.com',
 'syzran.mebel-store.com': 'mebel-store.com',
 'shadrinsk.mebel-store.com': 'mebel-store.com',
 'volgodonsk.mebel-store.com': 'mebel-store.com',
 'berdsk.mebel-store.com': 'mebel-store.com',
 'shatura.mebel-store.com': 'mebel-store.com',
 'birsk.mebel-store.com': 'mebel-store.com',
 'skhodnya.mebel-store.com': 'mebel-store.com',
 'tyumen.mebel-store.com': 'mebel-store.com',
 'mozhaysk.mebel-store.com': 'mebel-store.com',
    'ufa.fasone.ru': 'fasone.ru',
 'spb.fasone.ru': 'fasone.ru',
 'vladivostok.fasone.ru': 'fasone.ru',
 'yaroslavl.fasone.ru': 'fasone.ru',
 'fasone.ru': 'fasone.ru',
 'kirov.fasone.ru': 'fasone.ru',
 'tyumen.fasone.ru': 'fasone.ru',
 'samara.fasone.ru': 'fasone.ru',
 'vladimir.fasone.ru': 'fasone.ru',
 'nnovgorod.fasone.ru': 'fasone.ru',
 'ekaterinburg.fasone.ru': 'fasone.ru',
 'stavropol.fasone.ru': 'fasone.ru',
 'novosibirsk.fasone.ru': 'fasone.ru',
 'khabarovsk.fasone.ru': 'fasone.ru',
 'kaliningrad.fasone.ru': 'fasone.ru',
 'lipetsk.fasone.ru': 'fasone.ru',
 'krasnodar.fasone.ru': 'fasone.ru',
 'bryansk.fasone.ru': 'fasone.ru',
 'tver.fasone.ru': 'fasone.ru',
 'rostov-na-donu.fasone.ru': 'fasone.ru',
 'astrakhan.fasone.ru': 'fasone.ru',
 'perm.fasone.ru': 'fasone.ru',
 'saratov.fasone.ru': 'fasone.ru',
 'tula.fasone.ru': 'fasone.ru',
 'omsk.fasone.ru': 'fasone.ru',
 'krasnoyarsk.fasone.ru': 'fasone.ru',
 'irkutsk.fasone.ru': 'fasone.ru',
 'orenburg.fasone.ru': 'fasone.ru',
 'voronezh.fasone.ru': 'fasone.ru',
 'smolensk.fasone.ru': 'fasone.ru',
 'chelyabinsk.fasone.ru': 'fasone.ru',
 'ryazan.fasone.ru': 'fasone.ru',
    'xn-----7kcgn5cdbagnnnx.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--90a1af.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--j1adfn.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--b1aqbiftj7e1a.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80ablvtof7b4b.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80aag1ciek.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--90aogkbcz.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--j1aaude4e.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80aauks4g.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80aab6birx.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80adxhks.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80a1bd.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--90aiaifanwy.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--90absbknhbvge.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn----dtbdeglbi6acdmca3a.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80addag2buct.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--h1afipca2eya.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80acgfbsl1azdqr.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--e1aner7ci.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80aai0ag2c.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80aacf4bwnk3a.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80abedla9acxg1b7f.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80atblfjdfd2l.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--90ahkico2a6b9d.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--b1afab7bff7cb.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--e1afhbv7b.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--90aoglmj0e.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--c1azcgcc.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn----7sbnzl7ae5f.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--j1adfnc.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
 'xn--80asdfng.xn--e1aebcaaarunvna5d.xn--p1ai': 'xn--e1aebcaaarunvna5d.xn--p1ai',
    'kirov.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'kerchy.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'ekaterinburg.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'perm.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'kazan.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'yaroslavl.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'spb.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'habarovsk.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'sochi.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'ufa.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'rostov-na-donu.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'omsk.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'msk.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'taganrog.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'barnaul.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'biysk.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'belgorod.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'balakovo.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'abakan.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'samara.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'naberezhnye-chelny.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'yakutsk.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'mahachkale.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'kursk.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'penza.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'surgut.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'kemerovo.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'vladivostok.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'kaliningrad.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'tyumen.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
 'saratov.snyat-kvartiru-bez-posrednikov.ru': 'snyat-kvartiru-bez-posrednikov.ru',
    'wikimapia.org': 'wikimapia.org',
 'astrakhan.wikimapia.org': 'wikimapia.org',
 'moscow.wikimapia.org': 'wikimapia.org',
 'novosibirsk.wikimapia.org': 'wikimapia.org',
 'voronezh.wikimapia.org': 'wikimapia.org',
 'yekaterinburg.wikimapia.org': 'wikimapia.org',
 'dolgoprudnyy.wikimapia.org': 'wikimapia.org',
 'ryazan.wikimapia.org': 'wikimapia.org',
 'perm.wikimapia.org': 'wikimapia.org',
 'taganrog.wikimapia.org': 'wikimapia.org',
 'magnitogorsk.wikimapia.org': 'wikimapia.org',
 'engels.wikimapia.org': 'wikimapia.org',
 'kurgan.wikimapia.org': 'wikimapia.org',
 'lipetsk.wikimapia.org': 'wikimapia.org',
 'kursk.wikimapia.org': 'wikimapia.org',
 'bryansk.wikimapia.org': 'wikimapia.org',
 'krasnoyarsk.wikimapia.org': 'wikimapia.org',
 'tyumen.wikimapia.org': 'wikimapia.org',
 'domodedovo.wikimapia.org': 'wikimapia.org',
 'orenburg.wikimapia.org': 'wikimapia.org',
 'ulyanovsk.wikimapia.org': 'wikimapia.org',
 'elektrostal.wikimapia.org': 'wikimapia.org',
 'kirov.wikimapia.org': 'wikimapia.org',
 'pervouralsk.wikimapia.org': 'wikimapia.org',
 'barnaul.wikimapia.org': 'wikimapia.org',
 'ufa.wikimapia.org': 'wikimapia.org',
 'podolsk.wikimapia.org': 'wikimapia.org',
 'krasnodar.wikimapia.org': 'wikimapia.org',
 'nakhodka.wikimapia.org': 'wikimapia.org',
 'nizhniy-novgorod.wikimapia.org': 'wikimapia.org',
 'kazan.wikimapia.org': 'wikimapia.org',
 'cherepovets.wikimapia.org': 'wikimapia.org',
    'lord-film4zz.lordfilm2.band': 'lordfilm2.band',
 'lord-filmxhd.lordfilm2.band': 'lordfilm2.band',
 'lordfilmx-hd.lordfilm2.band': 'lordfilm2.band',
 'lord-filmixhd.lordfilm2.band': 'lordfilm2.band',
 'lords-filmi.lordfilm2.band': 'lordfilm2.band',
 'lord16.lordfilm2.band': 'lordfilm2.band',
 'lord8.lordfilm2.band': 'lordfilm2.band',
 'lord9.lordfilm2.band': 'lordfilm2.band',
 'lord7.lordfilm2.band': 'lordfilm2.band',
 'lord17.lordfilm2.band': 'lordfilm2.band',
 'lordz-filmx.lordfilm2.band': 'lordfilm2.band',
 'lords-filmx.lordfilm2.band': 'lordfilm2.band',
 'lordz-film.lordfilm2.band': 'lordfilm2.band',
 'lord10.lordfilm2.band': 'lordfilm2.band',
 'lord-film5z.lordfilm2.band': 'lordfilm2.band',
 'lord3.lordfilm2.band': 'lordfilm2.band',
 'lord12.lordfilm2.band': 'lordfilm2.band',
 'lord-filmc4z.lordfilm2.band': 'lordfilm2.band',
 'lord5.lordfilm2.band': 'lordfilm2.band',
 'lords-filmix.lordfilm2.band': 'lordfilm2.band',
 'lord2.lordfilm2.band': 'lordfilm2.band',
 'lord-filmz4z.lordfilm2.band': 'lordfilm2.band',
 'lord6.lordfilm2.band': 'lordfilm2.band',
 'lordz-filmhd.lordfilm2.band': 'lordfilm2.band',
 'lord-filmihd.lordfilm2.band': 'lordfilm2.band',
 'lord14.lordfilm2.band': 'lordfilm2.band',
 'lords-filmxhd.lordfilm2.band': 'lordfilm2.band',
 'lord13.lordfilm2.band': 'lordfilm2.band',
 'lord-film3.lordfilm2.band': 'lordfilm2.band',
 'lord-film5.lordfilm2.band': 'lordfilm2.band',
 'lord15.lordfilm2.band': 'lordfilm2.band',
 'lordz-filmxhd.lordfilm2.band': 'lordfilm2.band',
    'novorossiisk.spravochnikov.ru': 'spravochnikov.ru',
 'kazan.spravochnikov.ru': 'spravochnikov.ru',
 'rostov.spravochnikov.ru': 'spravochnikov.ru',
 'ufa.spravochnikov.ru': 'spravochnikov.ru',
 'moscow.spravochnikov.ru': 'spravochnikov.ru',
 'tumen.spravochnikov.ru': 'spravochnikov.ru',
 'kaluga.spravochnikov.ru': 'spravochnikov.ru',
 'gorno-altaysk.spravochnikov.ru': 'spravochnikov.ru',
 'ryazan.spravochnikov.ru': 'spravochnikov.ru',
 'khabarovsk.spravochnikov.ru': 'spravochnikov.ru',
 'yaroslavl.spravochnikov.ru': 'spravochnikov.ru',
 'tolyatti.spravochnikov.ru': 'spravochnikov.ru',
 'yakutsk.spravochnikov.ru': 'spravochnikov.ru',
 'orenburg.spravochnikov.ru': 'spravochnikov.ru',
 'ivanovo.spravochnikov.ru': 'spravochnikov.ru',
 'lipetsk.spravochnikov.ru': 'spravochnikov.ru',
 'krasnoyarsk.spravochnikov.ru': 'spravochnikov.ru',
 'veliky-novgorod.spravochnikov.ru': 'spravochnikov.ru',
 'naberezhnie-chelny.spravochnikov.ru': 'spravochnikov.ru',
 'blagoveshchensk.spravochnikov.ru': 'spravochnikov.ru',
 'sochi.spravochnikov.ru': 'spravochnikov.ru',
 'krasnodar.spravochnikov.ru': 'spravochnikov.ru',
 'saratov.spravochnikov.ru': 'spravochnikov.ru',
 'samara.spravochnikov.ru': 'spravochnikov.ru',
 'volgograd.spravochnikov.ru': 'spravochnikov.ru',
 'nizhniy-novgorod.spravochnikov.ru': 'spravochnikov.ru',
 'omsk.spravochnikov.ru': 'spravochnikov.ru',
 'sterlitamak.spravochnikov.ru': 'spravochnikov.ru',
 'smolensk.spravochnikov.ru': 'spravochnikov.ru',
 'novosibirsk.spravochnikov.ru': 'spravochnikov.ru',
 'kursk.spravochnikov.ru': 'spravochnikov.ru',
 'arhangelsk.spravochnikov.ru': 'spravochnikov.ru',
    'novosibirsk-auto.nenahodu.ru': 'nenahodu.ru',
 'nenahodu.ru': 'nenahodu.ru',
 'sankt-peterburg-auto.nenahodu.ru': 'nenahodu.ru',
 'zelenograd-auto.nenahodu.ru': 'nenahodu.ru',
 'smolensk-auto.nenahodu.ru': 'nenahodu.ru',
 'moskva-auto.nenahodu.ru': 'nenahodu.ru',
 'balashiha-auto.nenahodu.ru': 'nenahodu.ru',
 'murom-auto.nenahodu.ru': 'nenahodu.ru',
 'kaluga-auto.nenahodu.ru': 'nenahodu.ru',
 'pyatigorsk-auto.nenahodu.ru': 'nenahodu.ru',
 'ramenskoe-auto.nenahodu.ru': 'nenahodu.ru',
 'orenburg-auto.nenahodu.ru': 'nenahodu.ru',
 'tolyatti-auto.nenahodu.ru': 'nenahodu.ru',
 'naberezhnye-chelny-auto.nenahodu.ru': 'nenahodu.ru',
 'belovo-auto.nenahodu.ru': 'nenahodu.ru',
 'samara-auto.nenahodu.ru': 'nenahodu.ru',
 'lipeck-auto.nenahodu.ru': 'nenahodu.ru',
 'krasnodar-auto.nenahodu.ru': 'nenahodu.ru',
 'kemerovo-auto.nenahodu.ru': 'nenahodu.ru',
 'tomsk-auto.nenahodu.ru': 'nenahodu.ru',
 'ufa-auto.nenahodu.ru': 'nenahodu.ru',
 'ulyanovsk-auto.nenahodu.ru': 'nenahodu.ru',
 'kirov-1-auto.nenahodu.ru': 'nenahodu.ru',
 'nizhniy-novgorod-auto.nenahodu.ru': 'nenahodu.ru',
 'anapa-auto.nenahodu.ru': 'nenahodu.ru',
 'ekaterinburg-auto.nenahodu.ru': 'nenahodu.ru',
 'vologda-auto.nenahodu.ru': 'nenahodu.ru',
 'saratov-auto.nenahodu.ru': 'nenahodu.ru',
 'rostov-na-donu-auto.nenahodu.ru': 'nenahodu.ru',
 'cheboksary-auto.nenahodu.ru': 'nenahodu.ru',
 'engels-auto.nenahodu.ru': 'nenahodu.ru',
 'barnaul-auto.nenahodu.ru': 'nenahodu.ru',
    'dhsh-buy.kst.muzkult.ru': 'muzkult.ru',
 'muzkult.ru': 'muzkult.ru',
 'dhsh-filippova.krd.muzkult.ru': 'muzkult.ru',
 'gdk-kansk.krn.muzkult.ru': 'muzkult.ru',
 'kremlin.mari-el.muzkult.ru': 'muzkult.ru',
 'pkio-stupino.mo.muzkult.ru': 'muzkult.ru',
 'kxudu.krd.muzkult.ru': 'muzkult.ru',
 'kdc-okean.shl.muzkult.ru': 'muzkult.ru',
 'mgndmsh3.chel.muzkult.ru': 'muzkult.ru',
 'dshi30.nsk.muzkult.ru': 'muzkult.ru',
 'dshi21.nsk.muzkult.ru': 'muzkult.ru',
 'divartmuseum.krn.muzkult.ru': 'muzkult.ru',
 'dshi.surgut.muzkult.ru': 'muzkult.ru',
 'rkk-schelkovo.mo.muzkult.ru': 'muzkult.ru',
 'novoaltmuzei.altai.muzkult.ru': 'muzkult.ru',
 'garmonia.mo.muzkult.ru': 'muzkult.ru',
 'cbs-satka.chel.muzkult.ru': 'muzkult.ru',
 'kultura-lbn.omsk.muzkult.ru': 'muzkult.ru',
 'rdk-planeta.chel.muzkult.ru': 'muzkult.ru',
 'liskusstv.smr.muzkult.ru': 'muzkult.ru',
 'oblbibl.rnd.muzkult.ru': 'muzkult.ru',
 'rck-kurkino.tls.muzkult.ru': 'muzkult.ru',
 'dmsh4.tom.muzkult.ru': 'muzkult.ru',
 'ncbs.arkh.muzkult.ru': 'muzkult.ru',
 'kupino-lib.nsk.muzkult.ru': 'muzkult.ru',
 'ufadmsh11.bash.muzkult.ru': 'muzkult.ru',
 'mdms.krn.muzkult.ru': 'muzkult.ru',
 'mdkros.chel.muzkult.ru': 'muzkult.ru',
 'dmsh6.perm.muzkult.ru': 'muzkult.ru',
 'dshi-buzuluk.oren.muzkult.ru': 'muzkult.ru',
 'dom-znanij.vlg.muzkult.ru': 'muzkult.ru',
 'dshi-pushkin.oren.muzkult.ru': 'muzkult.ru',
 'tavr-ckd.vl.muzkult.ru': 'muzkult.ru',
    'torrent-film.monna.top': 'monna.top',
 '6c860a10ffb0d492c9cb3f96a688d1436c860a10ffb0d492c9cb3f96a688d21.monna.top': 'monna.top',
 '06211303.monna.top': 'monna.top',
 '19221604.monna.top': 'monna.top',
 'lmystulfpsjuunsnhnuu.monna.top': 'monna.top',
 '03212504.monna.top': 'monna.top',
 '10223105.monna.top': 'monna.top',
 '21222605.monna.top': 'monna.top',
 '13220304.monna.top': 'monna.top',
 '19211808.monna.top': 'monna.top',
 '10211607.monna.top': 'monna.top',
 '16212604.monna.top': 'monna.top',
 '09212706.monna.top': 'monna.top',
 '18220504.monna.top': 'monna.top',
 '10221405.monna.top': 'monna.top',
 '15220904.monna.top': 'monna.top',
 '16212703.monna.top': 'monna.top',
 '10211108.monna.top': 'monna.top',
 '19221105.monna.top': 'monna.top',
 '08210711.monna.top': 'monna.top',
 '07210506.monna.top': 'monna.top',
 '12222405.monna.top': 'monna.top',
 '09222704.monna.top': 'monna.top',
 '08220405.monna.top': 'monna.top',
 '12221505.monna.top': 'monna.top',
 '09220705.monna.top': 'monna.top',
 '12210108.monna.top': 'monna.top',
 '20212907.monna.top': 'monna.top',
 '13220504.monna.top': 'monna.top',
 '16223105.monna.top': 'monna.top',
 '13212606.monna.top': 'monna.top',
 '13211507.monna.top': 'monna.top',
 '05220603.monna.top': 'monna.top',
    'mytishchi.damspravku.ru': 'damspravku.ru',
 'damspravku.ru': 'damspravku.ru',
 'khimki.damspravku.ru': 'damspravku.ru',
 'samara.damspravku.ru': 'damspravku.ru',
 'msk.damspravku.ru': 'damspravku.ru',
 'taganrog.damspravku.ru': 'damspravku.ru',
 'chapaevsk.damspravku.ru': 'damspravku.ru',
 'anapa.damspravku.ru': 'damspravku.ru',
 'kolomna.damspravku.ru': 'damspravku.ru',
 'ufa.damspravku.ru': 'damspravku.ru',
 'dmitrov.damspravku.ru': 'damspravku.ru',
 'briansk.damspravku.ru': 'damspravku.ru',
 'sochi.damspravku.ru': 'damspravku.ru',
 'gelend.damspravku.ru': 'damspravku.ru',
 'domodedovo.damspravku.ru': 'damspravku.ru',
 'nefteiugansk.damspravku.ru': 'damspravku.ru',
 'krasnoturinsk.damspravku.ru': 'damspravku.ru',
 'strezhevoi.damspravku.ru': 'damspravku.ru',
 'neftekamsk.damspravku.ru': 'damspravku.ru',
 'novosib.damspravku.ru': 'damspravku.ru',
 'dolgoprudnyi.damspravku.ru': 'damspravku.ru',
 'armavir.damspravku.ru': 'damspravku.ru',
 'spb.damspravku.ru': 'damspravku.ru',
 'cherkessk.damspravku.ru': 'damspravku.ru',
 'michurinsk.damspravku.ru': 'damspravku.ru',
 'syktyvkar.damspravku.ru': 'damspravku.ru',
 'plavsk.damspravku.ru': 'damspravku.ru',
 'zhigulevsk.damspravku.ru': 'damspravku.ru',
 'ulyanovsk.damspravku.ru': 'damspravku.ru',
 'shakhty.damspravku.ru': 'damspravku.ru',
 'ioshkar-ola.damspravku.ru': 'damspravku.ru',
 'sarapul.damspravku.ru': 'damspravku.ru',
 'furmanov.damspravku.ru': 'damspravku.ru',
    'srg.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'rnd.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'nnv.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'kry.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'krd.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'vrn.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'orl.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'tvr.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'srv.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'tul.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'rzn.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'klu.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'kmr.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'yrs.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'nrs.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'kzn.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'kgn.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'brn.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'iva.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'bla.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'chl.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'tmn.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'smr.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'nsk.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'vlg.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'oms.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'uln.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'vln.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'ast.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'izh.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'ufa.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'kst.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
 'stv.vse-avtoservisy.ru': 'vse-avtoservisy.ru',
    'kurgan.icity.life': 'icity.life',
 'icity.life': 'icity.life',
 'surgut.icity.life': 'icity.life',
 'krasnodar.icity.life': 'icity.life',
 'sochi.icity.life': 'icity.life',
 'nsk.icity.life': 'icity.life',
 'chel.icity.life': 'icity.life',
 'nnv.icity.life': 'icity.life',
 'spb.icity.life': 'icity.life',
 'kemerovo.icity.life': 'icity.life',
 'perm.icity.life': 'icity.life',
 'krasnoyarsk.icity.life': 'icity.life',
 'ufa.icity.life': 'icity.life',
 'kirov.icity.life': 'icity.life',
 'izhevsk.icity.life': 'icity.life',
 'ekb.icity.life': 'icity.life',
 'tyumen.icity.life': 'icity.life',
 'rostov.icity.life': 'icity.life',
 'ryazan.icity.life': 'icity.life',
 'murmansk.icity.life': 'icity.life',
 'habarovsk.icity.life': 'icity.life',
 'irkutsk.icity.life': 'icity.life',
 'omsk.icity.life': 'icity.life',
 'voronezh.icity.life': 'icity.life',
 'penza.icity.life': 'icity.life',
 'orenburg.icity.life': 'icity.life',
 'gelendzhik.icity.life': 'icity.life',
 'tver.icity.life': 'icity.life',
 'bryansk.icity.life': 'icity.life',
 'tolyatti.icity.life': 'icity.life',
 'barnaul.icity.life': 'icity.life',
 'msk.icity.life': 'icity.life',
 'saratov.icity.life': 'icity.life',
 'kaliningrad.icity.life': 'icity.life',
    'voronezh.rusavtobus.ru': 'rusavtobus.ru',
 'volga.rusavtobus.ru': 'rusavtobus.ru',
 'kazan.rusavtobus.ru': 'rusavtobus.ru',
 'kiev.rusavtobus.ru': 'rusavtobus.ru',
 'novosib.rusavtobus.ru': 'rusavtobus.ru',
 'astra.rusavtobus.ru': 'rusavtobus.ru',
 'chelny.rusavtobus.ru': 'rusavtobus.ru',
 'krd.rusavtobus.ru': 'rusavtobus.ru',
 'lipetsk.rusavtobus.ru': 'rusavtobus.ru',
 'krsk.rusavtobus.ru': 'rusavtobus.ru',
 'omsk.rusavtobus.ru': 'rusavtobus.ru',
 'nn.rusavtobus.ru': 'rusavtobus.ru',
 'samara.rusavtobus.ru': 'rusavtobus.ru',
 'chel.rusavtobus.ru': 'rusavtobus.ru',
 'ufa.rusavtobus.ru': 'rusavtobus.ru',
 'minsk.rusavtobus.ru': 'rusavtobus.ru',
 'eka.rusavtobus.ru': 'rusavtobus.ru',
 'spb.rusavtobus.ru': 'rusavtobus.ru',
 'yar.rusavtobus.ru': 'rusavtobus.ru',
 'oren.rusavtobus.ru': 'rusavtobus.ru',
 'astana.rusavtobus.ru': 'rusavtobus.ru',
 'irk.rusavtobus.ru': 'rusavtobus.ru',
 'tumen.rusavtobus.ru': 'rusavtobus.ru',
 'saratov.rusavtobus.ru': 'rusavtobus.ru',
 'sochi.rusavtobus.ru': 'rusavtobus.ru',
 'kostroma.rusavtobus.ru': 'rusavtobus.ru',
 'vlad.rusavtobus.ru': 'rusavtobus.ru',
 'cheb.rusavtobus.ru': 'rusavtobus.ru',
 'tlt.rusavtobus.ru': 'rusavtobus.ru',
 'msk.rusavtobus.ru': 'rusavtobus.ru',
 'khb.rusavtobus.ru': 'rusavtobus.ru',
 'rostov.rusavtobus.ru': 'rusavtobus.ru',
 'perm.rusavtobus.ru': 'rusavtobus.ru',
 'rusavtobus.ru': 'rusavtobus.ru',
    'xn--h1alffa9f.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--b1afih.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--b1axaggg.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80atblfjdfd2l.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--b1ahhecb0azo.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80abmghlx4ajd.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--e1afkclaggf6a2g.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--h1aliz.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--90a1af.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aalwqglfe.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80abedla9acxg1b7f.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--b1afchn5b.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--90ahkico2a6b9d.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn----7sbdqaabf2clfe5a7hpcg.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aaczf9e9c.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aaa0cvac.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80adi0aoagldk8i.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--e1aid1a.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80acgfbsl1azdqr.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80antj7do.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--e1aohf5d.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aaifraljtb8a.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80ae1alafffj1i.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80afmksoji0fc.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--b1alfrj.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aaafckyesce.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--b1agflcndeb5cwb.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80adbhunc2aa3al.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aaxgrpt.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aai0ag2c.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--m1aa.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80aag1ciek.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--h1aeawgfg.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
 'xn--80ajgxi.xn--80abbembcyvesfij3at4loa4ff.xn--p1ai': 'xn--80abbembcyvesfij3at4loa4ff.xn--p1ai',
    'chelyabinsk.tradeis.ru': 'tradeis.ru',
 'abakan.tradeis.ru': 'tradeis.ru',
 'blagoveshchensk.tradeis.ru': 'tradeis.ru',
 'vladivostok.tradeis.ru': 'tradeis.ru',
 'gorno-altaisk.tradeis.ru': 'tradeis.ru',
 'joshkar-ola.tradeis.ru': 'tradeis.ru',
 'astrakhan.tradeis.ru': 'tradeis.ru',
 'bishkek.tradeis.ru': 'tradeis.ru',
 'bryansk.tradeis.ru': 'tradeis.ru',
 'v-novgorod.tradeis.ru': 'tradeis.ru',
 'leninsk-kuznetskiy.tradeis.ru': 'tradeis.ru',
 'kemerovo.tradeis.ru': 'tradeis.ru',
 'krasnoyarsk.tradeis.ru': 'tradeis.ru',
 'kirov.tradeis.ru': 'tradeis.ru',
 'ivanovo.tradeis.ru': 'tradeis.ru',
 'voronezh.tradeis.ru': 'tradeis.ru',
 'kaliningrad.tradeis.ru': 'tradeis.ru',
 'irkutsk.tradeis.ru': 'tradeis.ru',
 'vladimir.tradeis.ru': 'tradeis.ru',
 'miass.tradeis.ru': 'tradeis.ru',
 'volgograd.tradeis.ru': 'tradeis.ru',
 'kazan.tradeis.ru': 'tradeis.ru',
 'nahodka.tradeis.ru': 'tradeis.ru',
 'vologda.tradeis.ru': 'tradeis.ru',
 'arkhangelsk.tradeis.ru': 'tradeis.ru',
 'izhevsk.tradeis.ru': 'tradeis.ru',
 'tradeis.ru': 'tradeis.ru',
 'lipetsk.tradeis.ru': 'tradeis.ru',
 'belgorod.tradeis.ru': 'tradeis.ru',
 'ekaterinburg.tradeis.ru': 'tradeis.ru',
 'krasnodar.tradeis.ru': 'tradeis.ru',
 'kaluga.tradeis.ru': 'tradeis.ru',
 'kursk.tradeis.ru': 'tradeis.ru',
 'nizhnevartovsk.tradeis.ru': 'tradeis.ru',
 'stavropol.tradeis.ru': 'tradeis.ru',
    'nyagan.2map.su': '2map.su',
 'cherepovets.2map.su': '2map.su',
 'vologda.2map.su': '2map.su',
 'kotlas.2map.su': '2map.su',
 'tula.2map.su': '2map.su',
 'orenburg.2map.su': '2map.su',
 'ubk.2map.su': '2map.su',
 'udm.2map.su': '2map.su',
 'novokuznetsk.2map.su': '2map.su',
 'sevastopol.2map.su': '2map.su',
 'voronezh.2map.su': '2map.su',
 'krasnodar.2map.su': '2map.su',
 'grozny.2map.su': '2map.su',
 'evpatoriya.2map.su': '2map.su',
 'rostov.2map.su': '2map.su',
 'krasnoyarsk.2map.su': '2map.su',
 'kerch.2map.su': '2map.su',
 'raduzhny.2map.su': '2map.su',
 'surgut.2map.su': '2map.su',
 'taganrog.2map.su': '2map.su',
 'hm.2map.su': '2map.su',
 'simferopol.2map.su': '2map.su',
 'tyumen.2map.su': '2map.su',
 'anzhero.2map.su': '2map.su',
 'novomoskovsk.2map.su': '2map.su',
 'nefteyugansk.2map.su': '2map.su',
 'asino.2map.su': '2map.su',
 'kemerovo.2map.su': '2map.su',
 'khasavyurt.2map.su': '2map.su',
 'beloretsk.2map.su': '2map.su',
 '2map.su': '2map.su',
 'orsk.2map.su': '2map.su',
 'tomsk.2map.su': '2map.su',
 'nnovgorod.2map.su': '2map.su',
 'nizhnevartovsk.2map.su': '2map.su',
    'legolas.allohastream.com': 'allohastream.com',
 'faramir.allohastream.com': 'allohastream.com',
 'polemic.allohastream.com': 'allohastream.com',
 'morgan.allohastream.com': 'allohastream.com',
 'tree.allohastream.com': 'allohastream.com',
 'beggins.allohastream.com': 'allohastream.com',
 'cusik.allohastream.com': 'allohastream.com',
 'april.allohastream.com': 'allohastream.com',
 'hvitserg.allohastream.com': 'allohastream.com',
 'abraham-erskine.allohastream.com': 'allohastream.com',
 'bike.allohastream.com': 'allohastream.com',
 'art.allohastream.com': 'allohastream.com',
 'aragorn.allohastream.com': 'allohastream.com',
 'abigail-brand.allohastream.com': 'allohastream.com',
 'sign.allohastream.com': 'allohastream.com',
 'marie.allohastream.com': 'allohastream.com',
 'etelstan.allohastream.com': 'allohastream.com',
 'agent-venom.allohastream.com': 'allohastream.com',
 'aslaug.allohastream.com': 'allohastream.com',
 'bear.allohastream.com': 'allohastream.com',
 'absorbing-man.allohastream.com': 'allohastream.com',
 'hogun.allohastream.com': 'allohastream.com',
 'arven.allohastream.com': 'allohastream.com',
 'miyagi.allohastream.com': 'allohastream.com',
 'polyhedron.allohastream.com': 'allohastream.com',
 'fork.allohastream.com': 'allohastream.com',
 'biorn.allohastream.com': 'allohastream.com',
 'ollo.allohastream.com': 'allohastream.com',
 'agent-13.allohastream.com': 'allohastream.com',
 'theft.allohastream.com': 'allohastream.com',
 'abbis.allohastream.com': 'allohastream.com',
 'politicalscientist.allohastream.com': 'allohastream.com',
 'lunior.allohastream.com': 'allohastream.com',
 'synthezoid.allohastream.com': 'allohastream.com',
 'air-walker.allohastream.com': 'allohastream.com',
    'api.delivembd.ws': 'delivembd.ws',
 'api1629624694.delivembd.ws': 'delivembd.ws',
 'api1636982941.delivembd.ws': 'delivembd.ws',
 'api1646816722.delivembd.ws': 'delivembd.ws',
 'apimovie.delivembd.ws': 'delivembd.ws',
 'api1647876146.delivembd.ws': 'delivembd.ws',
 'api1636552081.delivembd.ws': 'delivembd.ws',
 'api1646651114.delivembd.ws': 'delivembd.ws',
 'api1583257802.delivembd.ws': 'delivembd.ws',
 'api1624393077.delivembd.ws': 'delivembd.ws',
 'api1646305450.delivembd.ws': 'delivembd.ws',
 'api1646348402.delivembd.ws': 'delivembd.ws',
 'api1650114944.delivembd.ws': 'delivembd.ws',
 'api1647107188.delivembd.ws': 'delivembd.ws',
 'api1647259911.delivembd.ws': 'delivembd.ws',
 'api1648283233.delivembd.ws': 'delivembd.ws',
 'api1636633387.delivembd.ws': 'delivembd.ws',
 'api1637770669.delivembd.ws': 'delivembd.ws',
 'api1646310551.delivembd.ws': 'delivembd.ws',
 'api1646478540.delivembd.ws': 'delivembd.ws',
 'api1646592812.delivembd.ws': 'delivembd.ws',
 'api1649433701.delivembd.ws': 'delivembd.ws',
 'api1646479690.delivembd.ws': 'delivembd.ws',
 'api1649064909.delivembd.ws': 'delivembd.ws',
 'api1637756658.delivembd.ws': 'delivembd.ws',
 'api1647894095.delivembd.ws': 'delivembd.ws',
 'sp.delivembd.ws': 'delivembd.ws',
 'api1637861210.delivembd.ws': 'delivembd.ws',
 'api1636723160.delivembd.ws': 'delivembd.ws',
 'api1637396823.delivembd.ws': 'delivembd.ws',
 'api1646465786.delivembd.ws': 'delivembd.ws',
 'api1646321641.delivembd.ws': 'delivembd.ws',
 'api1637256906.delivembd.ws': 'delivembd.ws',
 'api1649499145.delivembd.ws': 'delivembd.ws',
 'api1637262676.delivembd.ws': 'delivembd.ws',
    'gooz.ru': 'gooz.ru',
 'spb.gooz.ru': 'gooz.ru',
 'tambov.gooz.ru': 'gooz.ru',
 'ekb.gooz.ru': 'gooz.ru',
 'rostov-na-donu.gooz.ru': 'gooz.ru',
 'petrozavodsk.gooz.ru': 'gooz.ru',
 'khabarovsk.gooz.ru': 'gooz.ru',
 'nn.gooz.ru': 'gooz.ru',
 'krasnodar.gooz.ru': 'gooz.ru',
 'omsk.gooz.ru': 'gooz.ru',
 'dmitrov.gooz.ru': 'gooz.ru',
 'kursk.gooz.ru': 'gooz.ru',
 'tolyatti.gooz.ru': 'gooz.ru',
 'arhangelsk.gooz.ru': 'gooz.ru',
 'kaliningrad.gooz.ru': 'gooz.ru',
 'anapa.gooz.ru': 'gooz.ru',
 'kazan.gooz.ru': 'gooz.ru',
 'samara.gooz.ru': 'gooz.ru',
 'zelenograd.gooz.ru': 'gooz.ru',
 'perm.gooz.ru': 'gooz.ru',
 'krasnoyarsk.gooz.ru': 'gooz.ru',
 'novosibirsk.gooz.ru': 'gooz.ru',
 'kirov.gooz.ru': 'gooz.ru',
 'surgut.gooz.ru': 'gooz.ru',
 'vladikavkaz.gooz.ru': 'gooz.ru',
 'sochi.gooz.ru': 'gooz.ru',
 'gelendzhik.gooz.ru': 'gooz.ru',
 'chelyabinsk.gooz.ru': 'gooz.ru',
 'vladivostok.gooz.ru': 'gooz.ru',
 'bryansk.gooz.ru': 'gooz.ru',
 'tomsk.gooz.ru': 'gooz.ru',
 'voronezh.gooz.ru': 'gooz.ru',
 'tyumen.gooz.ru': 'gooz.ru',
 'naberezhnye-chelny.gooz.ru': 'gooz.ru',
 'ufa.gooz.ru': 'gooz.ru',
    'prihod.ru': 'prihod.ru',
 'support.prihod.ru': 'prihod.ru',
 'hram-troicy.prihod.ru': 'prihod.ru',
 'sosnowka.prihod.ru': 'prihod.ru',
 'svluka.prihod.ru': 'prihod.ru',
 'karacharovohram.prihod.ru': 'prihod.ru',
 'mixarx.prihod.ru': 'prihod.ru',
 'blagovest72.prihod.ru': 'prihod.ru',
 'gornal.prihod.ru': 'prihod.ru',
 'hram-anevskogo.prihod.ru': 'prihod.ru',
 'in.prihod.ru': 'prihod.ru',
 'buzulukjenmonastyr.prihod.ru': 'prihod.ru',
 'pokrov-kaltan.prihod.ru': 'prihod.ru',
 'hramholodovo.prihod.ru': 'prihod.ru',
 'preobragenxram.prihod.ru': 'prihod.ru',
 'nicolaslud.prihod.ru': 'prihod.ru',
 'kirillimefody.prihod.ru': 'prihod.ru',
 'vseh-skorb-radost.prihod.ru': 'prihod.ru',
 'vasilevskoe.prihod.ru': 'prihod.ru',
 'petra-i-pavla.prihod.ru': 'prihod.ru',
 'sobor-ivanovo.prihod.ru': 'prihod.ru',
 'sv-troica.prihod.ru': 'prihod.ru',
 'iliya.prihod.ru': 'prihod.ru',
 'sv-stefan.prihod.ru': 'prihod.ru',
 'vozdvigenie.prihod.ru': 'prihod.ru',
 'ik-hram.prihod.ru': 'prihod.ru',
 'ilynka.prihod.ru': 'prihod.ru',
 'omsk-znamenie.prihod.ru': 'prihod.ru',
 'uspenie-kamishin.prihod.ru': 'prihod.ru',
 'voskresensk.prihod.ru': 'prihod.ru',
 'voskresenskyi.prihod.ru': 'prihod.ru',
 'nevsky.prihod.ru': 'prihod.ru',
 'pokrovpp.prihod.ru': 'prihod.ru',
 'znamenietomsk.prihod.ru': 'prihod.ru',
 'karatuz.prihod.ru': 'prihod.ru',
 'ioannrus.prihod.ru': 'prihod.ru',
    'uosd.spbu.ru': 'spbu.ru',
 'phil.spbu.ru': 'spbu.ru',
 'spbu.ru': 'spbu.ru',
 'cabinet.spbu.ru': 'spbu.ru',
 'chem.spbu.ru': 'spbu.ru',
 'hge.spbu.ru': 'spbu.ru',
 'politology.spbu.ru': 'spbu.ru',
 'sakai.spbu.ru': 'spbu.ru',
 'earth.spbu.ru': 'spbu.ru',
 'sir.spbu.ru': 'spbu.ru',
 'fock.phys.spbu.ru': 'spbu.ru',
 'msk.phil.spbu.ru': 'spbu.ru',
 'conf.spbu.ru': 'spbu.ru',
 'novist.history.spbu.ru': 'spbu.ru',
 'jf.spbu.ru': 'spbu.ru',
 'cufts.library.spbu.ru': 'spbu.ru',
 'vc.spbu.ru': 'spbu.ru',
 'proxy.library.spbu.ru': 'spbu.ru',
 'old.journal.spbu.ru': 'spbu.ru',
 'barsic.spbu.ru': 'spbu.ru',
 'arts.spbu.ru': 'spbu.ru',
 'iberorus.spbu.ru': 'spbu.ru',
 'distolymp2.spbu.ru': 'spbu.ru',
 'centant.spbu.ru': 'spbu.ru',
 '1-line.spbu.ru': 'spbu.ru',
 'medialing.spbu.ru': 'spbu.ru',
 'molbioph.niif.spbu.ru': 'spbu.ru',
 'optics.phys.spbu.ru': 'spbu.ru',
 'folk.spbu.ru': 'spbu.ru',
 'india.spbu.ru': 'spbu.ru',
 'zoology.bio.spbu.ru': 'spbu.ru',
 'cuf.spbu.ru': 'spbu.ru',
 'rki.spbu.ru': 'spbu.ru',
 'genling.spbu.ru': 'spbu.ru',
 'vestnik.spbu.ru': 'spbu.ru',
 'cmr.spbu.ru': 'spbu.ru',
    'tula.spravka.me': 'spravka.me',
 'kazan.spravka.me': 'spravka.me',
 'sochi.spravka.me': 'spravka.me',
 'sankt-peterburg.spravka.me': 'spravka.me',
 'naberezhnie-chelny.spravka.me': 'spravka.me',
 'samara.spravka.me': 'spravka.me',
 'moscow.spravka.me': 'spravka.me',
 'norilsk.spravka.me': 'spravka.me',
 'saratov.spravka.me': 'spravka.me',
 'ivanovo.spravka.me': 'spravka.me',
 'podolsk.spravka.me': 'spravka.me',
 'lipetsk.spravka.me': 'spravka.me',
 'kaliningrad.spravka.me': 'spravka.me',
 'khabarovsk.spravka.me': 'spravka.me',
 'gatchina.spravka.me': 'spravka.me',
 'vysokaya-gora.spravka.me': 'spravka.me',
 'ekaterinburg.spravka.me': 'spravka.me',
 'kemerovo.spravka.me': 'spravka.me',
 'minusinsk.spravka.me': 'spravka.me',
 'abakan.spravka.me': 'spravka.me',
 'ryazan.spravka.me': 'spravka.me',
 'lyubercy.spravka.me': 'spravka.me',
 'nizhnekamsk.spravka.me': 'spravka.me',
 'novorossiisk.spravka.me': 'spravka.me',
 'novosibirsk.spravka.me': 'spravka.me',
 'rostov.spravka.me': 'spravka.me',
 'izhevsk.spravka.me': 'spravka.me',
 'stavropol.spravka.me': 'spravka.me',
 'barnaul.spravka.me': 'spravka.me',
 'ufa.spravka.me': 'spravka.me',
 'spravka.me': 'spravka.me',
 'tumen.spravka.me': 'spravka.me',
 'perm.spravka.me': 'spravka.me',
 'chelyabinsk.spravka.me': 'spravka.me',
 'tolyatti.spravka.me': 'spravka.me',
 'petrozavodsk.spravka.me': 'spravka.me',
 'nizhniy-novgorod.spravka.me': 'spravka.me',
    'keer.su': 'keer.su',
 'feedback.keer.su': 'keer.su',
 'help.keer.su': 'keer.su',
 'noyabrsk.keer.su': 'keer.su',
 'engels.keer.su': 'keer.su',
 'abakan.keer.su': 'keer.su',
 'tula.keer.su': 'keer.su',
 'nizhnekamsk.keer.su': 'keer.su',
 'stavropol.keer.su': 'keer.su',
 'volgograd.keer.su': 'keer.su',
 'saratov.keer.su': 'keer.su',
 'yuzhno-sahalinsk.keer.su': 'keer.su',
 'kaliningrad.keer.su': 'keer.su',
 'vladivostok.keer.su': 'keer.su',
 'novaya-usman.keer.su': 'keer.su',
 'blagoveshchensk.keer.su': 'keer.su',
 'orenburg.keer.su': 'keer.su',
 'biysk.keer.su': 'keer.su',
 'perm.keer.su': 'keer.su',
 'almetevsk.keer.su': 'keer.su',
 'sankt-peterburg.keer.su': 'keer.su',
 'petrozavodsk.keer.su': 'keer.su',
 'chelyabinsk.keer.su': 'keer.su',
 'samara.keer.su': 'keer.su',
 'pskov.keer.su': 'keer.su',
 'shinnyy-centr-robinzon.keer.su': 'keer.su',
 'elizovo.keer.su': 'keer.su',
 'krasnodar.keer.su': 'keer.su',
 'moskva.keer.su': 'keer.su',
 'tobolsk.keer.su': 'keer.su',
 'novorossiysk.keer.su': 'keer.su',
 'anapa.keer.su': 'keer.su',
 'belgorod.keer.su': 'keer.su',
 'dzerzhinsk.keer.su': 'keer.su',
 'pervouralsk.keer.su': 'keer.su',
 'salavat.keer.su': 'keer.su',
 'gelendzhik.keer.su': 'keer.su',
    'orenburg.stankodrom.ru': 'stankodrom.ru',
 'ishimbaj.stankodrom.ru': 'stankodrom.ru',
 'sterlitamak.stankodrom.ru': 'stankodrom.ru',
 'beloretsk.stankodrom.ru': 'stankodrom.ru',
 'stankodrom.ru': 'stankodrom.ru',
 'uchaly.stankodrom.ru': 'stankodrom.ru',
 'ufa.stankodrom.ru': 'stankodrom.ru',
 'murom.stankodrom.ru': 'stankodrom.ru',
 'odintsovo.stankodrom.ru': 'stankodrom.ru',
 'lyubertsy.stankodrom.ru': 'stankodrom.ru',
 'podolsk.stankodrom.ru': 'stankodrom.ru',
 'mytischi.stankodrom.ru': 'stankodrom.ru',
 'moskva.stankodrom.ru': 'stankodrom.ru',
 'stavropol.stankodrom.ru': 'stankodrom.ru',
 'mikhaylovsk.stankodrom.ru': 'stankodrom.ru',
 'majkop.stankodrom.ru': 'stankodrom.ru',
 'krasnodar.stankodrom.ru': 'stankodrom.ru',
 'armavir.stankodrom.ru': 'stankodrom.ru',
 'anapa.stankodrom.ru': 'stankodrom.ru',
 'kropotkin.stankodrom.ru': 'stankodrom.ru',
 'cheboksary.stankodrom.ru': 'stankodrom.ru',
 'gelendzhik.stankodrom.ru': 'stankodrom.ru',
 'simpheropol.stankodrom.ru': 'stankodrom.ru',
 'saratov.stankodrom.ru': 'stankodrom.ru',
 'voronezh.stankodrom.ru': 'stankodrom.ru',
 'sevastopol.stankodrom.ru': 'stankodrom.ru',
 'feodosia.stankodrom.ru': 'stankodrom.ru',
 'samara.stankodrom.ru': 'stankodrom.ru',
 'volgograd.stankodrom.ru': 'stankodrom.ru',
 'serpukhov.stankodrom.ru': 'stankodrom.ru',
 'tula.stankodrom.ru': 'stankodrom.ru',
 'vladimir.stankodrom.ru': 'stankodrom.ru',
 'ekaterinburg.stankodrom.ru': 'stankodrom.ru',
 'leninogorsk.stankodrom.ru': 'stankodrom.ru',
 'nizhnij_novgorod.stankodrom.ru': 'stankodrom.ru',
 'polevskoj.stankodrom.ru': 'stankodrom.ru',
 'kirov.stankodrom.ru': 'stankodrom.ru',
    'st-petersburg.exdex.ru': 'exdex.ru',
 'exdex.ru': 'exdex.ru',
 'sochi.exdex.ru': 'exdex.ru',
 'maikop.exdex.ru': 'exdex.ru',
 'pskov.exdex.ru': 'exdex.ru',
 'moscow.exdex.ru': 'exdex.ru',
 'krasnodar.exdex.ru': 'exdex.ru',
 'bor.exdex.ru': 'exdex.ru',
 'naberezhnie-chelni.exdex.ru': 'exdex.ru',
 'samara.exdex.ru': 'exdex.ru',
 'tolyatti.exdex.ru': 'exdex.ru',
 'saratov.exdex.ru': 'exdex.ru',
 'tula.exdex.ru': 'exdex.ru',
 'evpatoriya.exdex.ru': 'exdex.ru',
 'podolsk.exdex.ru': 'exdex.ru',
 'tuapse.exdex.ru': 'exdex.ru',
 'obninsk.exdex.ru': 'exdex.ru',
 'kaluga.exdex.ru': 'exdex.ru',
 'novorossisk.exdex.ru': 'exdex.ru',
 'cheboksary.exdex.ru': 'exdex.ru',
 'kazan.exdex.ru': 'exdex.ru',
 'temryuk.exdex.ru': 'exdex.ru',
 'vsevolozhsk.exdex.ru': 'exdex.ru',
 'orel.exdex.ru': 'exdex.ru',
 'belgorod.exdex.ru': 'exdex.ru',
 'vladikavkaz.exdex.ru': 'exdex.ru',
 'ekaterinburg.exdex.ru': 'exdex.ru',
 'anapa.exdex.ru': 'exdex.ru',
 'voronezh.exdex.ru': 'exdex.ru',
 'orenburg.exdex.ru': 'exdex.ru',
 'ramenskoe.exdex.ru': 'exdex.ru',
 'kaliningrad.exdex.ru': 'exdex.ru',
 'kislovodsk.exdex.ru': 'exdex.ru',
 'vjatskie-poljany.exdex.ru': 'exdex.ru',
 'lipetsk.exdex.ru': 'exdex.ru',
 'izhevsk.exdex.ru': 'exdex.ru',
 'himki.exdex.ru': 'exdex.ru',
    'krym.lst-group.ru': 'lst-group.ru',
 'taganrog.lst-group.ru': 'lst-group.ru',
 'rostov-na-donu.lst-group.ru': 'lst-group.ru',
 'abhaz.lst-group.ru': 'lst-group.ru',
 'krasnodarskiy.lst-group.ru': 'lst-group.ru',
 'lst-group.ru': 'lst-group.ru',
 'kemerovskaya.lst-group.ru': 'lst-group.ru',
 'altay.lst-group.ru': 'lst-group.ru',
 'krasnodar.lst-group.ru': 'lst-group.ru',
 'nizhniy-novgorod.lst-group.ru': 'lst-group.ru',
 'pyatigorsk.lst-group.ru': 'lst-group.ru',
 'ijevsk.lst-group.ru': 'lst-group.ru',
 'spb.lst-group.ru': 'lst-group.ru',
 'arhangelsk.lst-group.ru': 'lst-group.ru',
 'orenburg.lst-group.ru': 'lst-group.ru',
 'kazan.lst-group.ru': 'lst-group.ru',
 'naberezhnye-chelny.lst-group.ru': 'lst-group.ru',
 'almetievsk.lst-group.ru': 'lst-group.ru',
 'minvody.lst-group.ru': 'lst-group.ru',
 'stavropol.lst-group.ru': 'lst-group.ru',
 'lermontov.lst-group.ru': 'lst-group.ru',
 'sterlitamak.lst-group.ru': 'lst-group.ru',
 'borisoglebsk.lst-group.ru': 'lst-group.ru',
 'ekaterinburg.lst-group.ru': 'lst-group.ru',
 'blagoveshchensk.lst-group.ru': 'lst-group.ru',
 'surgut.lst-group.ru': 'lst-group.ru',
 'habarovsk.lst-group.ru': 'lst-group.ru',
 'hakas.lst-group.ru': 'lst-group.ru',
 'moscow.lst-group.ru': 'lst-group.ru',
 'belgorod.lst-group.ru': 'lst-group.ru',
 'ufa.lst-group.ru': 'lst-group.ru',
 'uzlovaya.lst-group.ru': 'lst-group.ru',
 'novorossiysk.lst-group.ru': 'lst-group.ru',
 'sochi.lst-group.ru': 'lst-group.ru',
 'krasnoyarsk.lst-group.ru': 'lst-group.ru',
 'novosibirsk.lst-group.ru': 'lst-group.ru',
 'tatarstan.lst-group.ru': 'lst-group.ru',
    'ginfo.ru': 'ginfo.ru',
 'tlt.ginfo.ru': 'ginfo.ru',
 'nn.ginfo.ru': 'ginfo.ru',
 'krasnodar.ginfo.ru': 'ginfo.ru',
 'barnaul.ginfo.ru': 'ginfo.ru',
 'saratov.ginfo.ru': 'ginfo.ru',
 'krasnoyarsk.ginfo.ru': 'ginfo.ru',
 'ryazan.ginfo.ru': 'ginfo.ru',
 'voronej.ginfo.ru': 'ginfo.ru',
 'spb.ginfo.ru': 'ginfo.ru',
 'vladivostok.ginfo.ru': 'ginfo.ru',
 'ekb.ginfo.ru': 'ginfo.ru',
 'rnd.ginfo.ru': 'ginfo.ru',
 'ufa.ginfo.ru': 'ginfo.ru',
 'orenburg.ginfo.ru': 'ginfo.ru',
 'lipeck.ginfo.ru': 'ginfo.ru',
 'kazan.ginfo.ru': 'ginfo.ru',
 'kirov.ginfo.ru': 'ginfo.ru',
 'samara.ginfo.ru': 'ginfo.ru',
 'nsk.ginfo.ru': 'ginfo.ru',
 'penza.ginfo.ru': 'ginfo.ru',
 'habarovsk.ginfo.ru': 'ginfo.ru',
 'volgograd.ginfo.ru': 'ginfo.ru',
 'yaroslavl.ginfo.ru': 'ginfo.ru',
 'mahachkala.ginfo.ru': 'ginfo.ru',
 'irkutsk.ginfo.ru': 'ginfo.ru',
 'astrahan.ginfo.ru': 'ginfo.ru',
 'ulyanovsk.ginfo.ru': 'ginfo.ru',
 'omsk.ginfo.ru': 'ginfo.ru',
 'perm.ginfo.ru': 'ginfo.ru',
 'tumen.ginfo.ru': 'ginfo.ru',
 'izhevsk.ginfo.ru': 'ginfo.ru',
 'kaliningrad.ginfo.ru': 'ginfo.ru',
 'novokuzneck.ginfo.ru': 'ginfo.ru',
 'tomsk.ginfo.ru': 'ginfo.ru',
 'chel.ginfo.ru': 'ginfo.ru',
 'chelny.ginfo.ru': 'ginfo.ru',
    'nizhniy-novgorod.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'sterlitamak.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'ufa.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'sochi.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'samara.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'kazan.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'lipetsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'kurgan.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'krasnodar.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'krasnoyarsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'kostroma.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'yuzhno-sakhalinsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'yoshkar-ola.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'moscow.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'kursk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'kaliningrad.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'miass.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'perm.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'pervouralsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'tvoyaspravka.ru': 'tvoyaspravka.ru',
 'ryazan.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'norilsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'biysk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'ekaterinburg.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'stavropol.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'smolensk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'tomsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'novosibirsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'kemerovo.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'izhevsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'naberezhnie-chelny.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'tumen.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'shhelkovo.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'novoaltajsk.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'volgograd.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'saratov.tvoyaspravka.ru': 'tvoyaspravka.ru',
 'sankt-peterburg.tvoyaspravka.ru': 'tvoyaspravka.ru',
    'moskva.gorodrabot.ru': 'gorodrabot.ru',
 'gorodrabot.ru': 'gorodrabot.ru',
 'elabuga.gorodrabot.ru': 'gorodrabot.ru',
 'sankt-peterburg.gorodrabot.ru': 'gorodrabot.ru',
 'sochi.gorodrabot.ru': 'gorodrabot.ru',
 'gatchina.gorodrabot.ru': 'gorodrabot.ru',
 'russia.gorodrabot.ru': 'gorodrabot.ru',
 'krasnodar.gorodrabot.ru': 'gorodrabot.ru',
 'pervouralsk.gorodrabot.ru': 'gorodrabot.ru',
 'cheboksary.gorodrabot.ru': 'gorodrabot.ru',
 'chelyabinsk.gorodrabot.ru': 'gorodrabot.ru',
 'nijni-novgorod.gorodrabot.ru': 'gorodrabot.ru',
 'syktyvkar.gorodrabot.ru': 'gorodrabot.ru',
 'volgodonsk.gorodrabot.ru': 'gorodrabot.ru',
 'samara.gorodrabot.ru': 'gorodrabot.ru',
 'tomsk.gorodrabot.ru': 'gorodrabot.ru',
 'zelenograd.gorodrabot.ru': 'gorodrabot.ru',
 'novosibirsk.gorodrabot.ru': 'gorodrabot.ru',
 'podolsk.gorodrabot.ru': 'gorodrabot.ru',
 'komsomolsk-na-amure.gorodrabot.ru': 'gorodrabot.ru',
 'sergiev-posad.gorodrabot.ru': 'gorodrabot.ru',
 'syzran.gorodrabot.ru': 'gorodrabot.ru',
 'tula.gorodrabot.ru': 'gorodrabot.ru',
 'tolyatti.gorodrabot.ru': 'gorodrabot.ru',
 'vologda.gorodrabot.ru': 'gorodrabot.ru',
 'mineralnie-vodi.gorodrabot.ru': 'gorodrabot.ru',
 'norilsk.gorodrabot.ru': 'gorodrabot.ru',
 'sevastopol.gorodrabot.ru': 'gorodrabot.ru',
 'ozery.gorodrabot.ru': 'gorodrabot.ru',
 'moskovskaya-oblast.gorodrabot.ru': 'gorodrabot.ru',
 'barabinsk.gorodrabot.ru': 'gorodrabot.ru',
 'zelenodolsk.gorodrabot.ru': 'gorodrabot.ru',
 'voskresensk.gorodrabot.ru': 'gorodrabot.ru',
 'yalta.gorodrabot.ru': 'gorodrabot.ru',
 'eisk.gorodrabot.ru': 'gorodrabot.ru',
 'oktyabrski.gorodrabot.ru': 'gorodrabot.ru',
 'kalujskaya-oblast.gorodrabot.ru': 'gorodrabot.ru',
 'krasnaya-polyana.gorodrabot.ru': 'gorodrabot.ru',
    'nizhniy-tagil.dorus.ru': 'dorus.ru',
 'st-petersburg.dorus.ru': 'dorus.ru',
 'novosibirsk.dorus.ru': 'dorus.ru',
 'russia.dorus.ru': 'dorus.ru',
 'zhukovsky.dorus.ru': 'dorus.ru',
 'krasnodar.dorus.ru': 'dorus.ru',
 'sevastopol.dorus.ru': 'dorus.ru',
 'engels.dorus.ru': 'dorus.ru',
 'tver.dorus.ru': 'dorus.ru',
 'biysk.dorus.ru': 'dorus.ru',
 'anapa.dorus.ru': 'dorus.ru',
 'blagoveshchensk.dorus.ru': 'dorus.ru',
 'astrakhan.dorus.ru': 'dorus.ru',
 'abakan.dorus.ru': 'dorus.ru',
 'arkhangelsk.dorus.ru': 'dorus.ru',
 'ufa.dorus.ru': 'dorus.ru',
 'ekaterinburg.dorus.ru': 'dorus.ru',
 'rostov-on-don.dorus.ru': 'dorus.ru',
 'balashikha.dorus.ru': 'dorus.ru',
 'novorossiysk.dorus.ru': 'dorus.ru',
 'omsk.dorus.ru': 'dorus.ru',
 'novokuznetsk.dorus.ru': 'dorus.ru',
 'krasnoyarsk.dorus.ru': 'dorus.ru',
 'taganrog.dorus.ru': 'dorus.ru',
 'stavropol.dorus.ru': 'dorus.ru',
 'kursk.dorus.ru': 'dorus.ru',
 'volgograd.dorus.ru': 'dorus.ru',
 'kaliningrad.dorus.ru': 'dorus.ru',
 'yoshkar-ola.dorus.ru': 'dorus.ru',
 'serpukhov.dorus.ru': 'dorus.ru',
 'ulyanovsk.dorus.ru': 'dorus.ru',
 'barnaul.dorus.ru': 'dorus.ru',
 'belgorod.dorus.ru': 'dorus.ru',
 'smolensk.dorus.ru': 'dorus.ru',
 'cheboksary.dorus.ru': 'dorus.ru',
 'cherepovets.dorus.ru': 'dorus.ru',
 'chelyabinsk.dorus.ru': 'dorus.ru',
 'khabarovsk.dorus.ru': 'dorus.ru',
    'forum.stomatologija.su': 'stomatologija.su',
 'nizhniynovgorod.stomatologija.su': 'stomatologija.su',
 'khabarovsk.stomatologija.su': 'stomatologija.su',
 'krasnoyarsk.stomatologija.su': 'stomatologija.su',
 'krasnodar.stomatologija.su': 'stomatologija.su',
 'moscow.stomatologija.su': 'stomatologija.su',
 'cheboksary.stomatologija.su': 'stomatologija.su',
 'penza.stomatologija.su': 'stomatologija.su',
 'saratov.stomatologija.su': 'stomatologija.su',
 'vologda.stomatologija.su': 'stomatologija.su',
 'kursk.stomatologija.su': 'stomatologija.su',
 'donetsk.stomatologija.su': 'stomatologija.su',
 'kemerovo.stomatologija.su': 'stomatologija.su',
 'ufa.stomatologija.su': 'stomatologija.su',
 'kaliningrad.stomatologija.su': 'stomatologija.su',
 'omsk.stomatologija.su': 'stomatologija.su',
 'kirov.stomatologija.su': 'stomatologija.su',
 'ryazan.stomatologija.su': 'stomatologija.su',
 'sochi.stomatologija.su': 'stomatologija.su',
 'samara.stomatologija.su': 'stomatologija.su',
 'yaroslavl.stomatologija.su': 'stomatologija.su',
 'voronezh.stomatologija.su': 'stomatologija.su',
 'kazan.stomatologija.su': 'stomatologija.su',
 'ekaterinburg.stomatologija.su': 'stomatologija.su',
 'izhevsk.stomatologija.su': 'stomatologija.su',
 'surgut.stomatologija.su': 'stomatologija.su',
 'anapa.stomatologija.su': 'stomatologija.su',
 'ulyanovsk.stomatologija.su': 'stomatologija.su',
 'barnaul.stomatologija.su': 'stomatologija.su',
 'tolyatti.stomatologija.su': 'stomatologija.su',
 'stomatologija.su': 'stomatologija.su',
 'belgorod.stomatologija.su': 'stomatologija.su',
 'stavropol.stomatologija.su': 'stomatologija.su',
 'sanktpeterburg.stomatologija.su': 'stomatologija.su',
 'arhangelsk.stomatologija.su': 'stomatologija.su',
 'astrahan.stomatologija.su': 'stomatologija.su',
 'novosibirsk.stomatologija.su': 'stomatologija.su',
 'tyumen.stomatologija.su': 'stomatologija.su',
    'kazan.raise.ru': 'raise.ru',
 'mos.raise.ru': 'raise.ru',
 'vladivostok.raise.ru': 'raise.ru',
 'pri.raise.ru': 'raise.ru',
 'kemerovo.raise.ru': 'raise.ru',
 'sankt-peterburg.raise.ru': 'raise.ru',
 'raise.ru': 'raise.ru',
 'ekaterinburg.raise.ru': 'raise.ru',
 'chelyabinsk.raise.ru': 'raise.ru',
 'samara.raise.ru': 'raise.ru',
 'all.raise.ru': 'raise.ru',
 'voronezh.raise.ru': 'raise.ru',
 'krasnodar.raise.ru': 'raise.ru',
 'krd.raise.ru': 'raise.ru',
 'omsk.raise.ru': 'raise.ru',
 'rostov-na-donu.raise.ru': 'raise.ru',
 'novosibirsk.raise.ru': 'raise.ru',
 'ufa.raise.ru': 'raise.ru',
 'himki.raise.ru': 'raise.ru',
 'smolensk.raise.ru': 'raise.ru',
 'tmn.raise.ru': 'raise.ru',
 'vladimir.raise.ru': 'raise.ru',
 'orenburg.raise.ru': 'raise.ru',
 'tomsk.raise.ru': 'raise.ru',
 'irk.raise.ru': 'raise.ru',
 'len.raise.ru': 'raise.ru',
 'zab.raise.ru': 'raise.ru',
 'nsk.raise.ru': 'raise.ru',
 'dag.raise.ru': 'raise.ru',
 'alt.raise.ru': 'raise.ru',
 'tom.raise.ru': 'raise.ru',
 'ngr.raise.ru': 'raise.ru',
 'tve.raise.ru': 'raise.ru',
 'kry.raise.ru': 'raise.ru',
 'tul.raise.ru': 'raise.ru',
 'lip.raise.ru': 'raise.ru',
 'me.raise.ru': 'raise.ru',
 'amu.raise.ru': 'raise.ru',
 'orl.raise.ru': 'raise.ru',
    'areviews.ru': 'areviews.ru',
 'samara.areviews.ru': 'areviews.ru',
 'spb.areviews.ru': 'areviews.ru',
 'blagoveschensk.areviews.ru': 'areviews.ru',
 'chelyabinsk.areviews.ru': 'areviews.ru',
 'nizhniy-novgorod.areviews.ru': 'areviews.ru',
 'zlatoust.areviews.ru': 'areviews.ru',
 'joshkar-ola.areviews.ru': 'areviews.ru',
 'novosibirsk.areviews.ru': 'areviews.ru',
 'volgograd.areviews.ru': 'areviews.ru',
 'orenburg.areviews.ru': 'areviews.ru',
 'petrozavodsk.areviews.ru': 'areviews.ru',
 'barnaul.areviews.ru': 'areviews.ru',
 'kirov.areviews.ru': 'areviews.ru',
 'ussurijsk.areviews.ru': 'areviews.ru',
 'rostov-na-donu.areviews.ru': 'areviews.ru',
 'ivanovo.areviews.ru': 'areviews.ru',
 'krasnodar.areviews.ru': 'areviews.ru',
 'voronezh.areviews.ru': 'areviews.ru',
 'arhangelsk.areviews.ru': 'areviews.ru',
 'saratov.areviews.ru': 'areviews.ru',
 'irkutsk.areviews.ru': 'areviews.ru',
 'krasnoyarsk.areviews.ru': 'areviews.ru',
 'ekaterinburg.areviews.ru': 'areviews.ru',
 'bryansk.areviews.ru': 'areviews.ru',
 'ryazan.areviews.ru': 'areviews.ru',
 'lipetsk.areviews.ru': 'areviews.ru',
 'simferopol.areviews.ru': 'areviews.ru',
 'sevastopol.areviews.ru': 'areviews.ru',
 'pyatigorsk.areviews.ru': 'areviews.ru',
 'shahty.areviews.ru': 'areviews.ru',
 'smolensk.areviews.ru': 'areviews.ru',
 'nizhnevartovsk.areviews.ru': 'areviews.ru',
 'zelenograd.areviews.ru': 'areviews.ru',
 'novomoskovsk.areviews.ru': 'areviews.ru',
 'yuzhno-sahalinsk.areviews.ru': 'areviews.ru',
 'chita.areviews.ru': 'areviews.ru',
 'evpatoriya.areviews.ru': 'areviews.ru',
 'hasavyurt.areviews.ru': 'areviews.ru',
 'orsk.areviews.ru': 'areviews.ru',
    'food.inmyroom.ru': 'inmyroom.ru',
 'inmyroom.ru': 'inmyroom.ru',
 'sochi.inmyroom.ru': 'inmyroom.ru',
 'habarovsk.inmyroom.ru': 'inmyroom.ru',
 'vladivostok.inmyroom.ru': 'inmyroom.ru',
 'spb.inmyroom.ru': 'inmyroom.ru',
 'ekaterinburg.inmyroom.ru': 'inmyroom.ru',
 'krasnodar.inmyroom.ru': 'inmyroom.ru',
 'naberezhnye-chelny.inmyroom.ru': 'inmyroom.ru',
 'ulyanovsk.inmyroom.ru': 'inmyroom.ru',
 'voronezh.inmyroom.ru': 'inmyroom.ru',
 'kazan.inmyroom.ru': 'inmyroom.ru',
 'omsk.inmyroom.ru': 'inmyroom.ru',
 'saratov.inmyroom.ru': 'inmyroom.ru',
 'krasnoyarsk.inmyroom.ru': 'inmyroom.ru',
 'lyubercy.inmyroom.ru': 'inmyroom.ru',
 'ryazan.inmyroom.ru': 'inmyroom.ru',
 'zhukovskij.inmyroom.ru': 'inmyroom.ru',
 'ufa.inmyroom.ru': 'inmyroom.ru',
 'novgorod.inmyroom.ru': 'inmyroom.ru',
 'chelyabinsk.inmyroom.ru': 'inmyroom.ru',
 'cheboksary.inmyroom.ru': 'inmyroom.ru',
 'novosibirsk.inmyroom.ru': 'inmyroom.ru',
 'novorossiysk.inmyroom.ru': 'inmyroom.ru',
 'izhevsk.inmyroom.ru': 'inmyroom.ru',
 'ivanovo.inmyroom.ru': 'inmyroom.ru',
 'orenburg.inmyroom.ru': 'inmyroom.ru',
 'rostov.inmyroom.ru': 'inmyroom.ru',
 'barnaul.inmyroom.ru': 'inmyroom.ru',
 'tula.inmyroom.ru': 'inmyroom.ru',
 'surgut.inmyroom.ru': 'inmyroom.ru',
 'saransk.inmyroom.ru': 'inmyroom.ru',
 'kaliningrad.inmyroom.ru': 'inmyroom.ru',
 'domodedovo.inmyroom.ru': 'inmyroom.ru',
 'belgorod.inmyroom.ru': 'inmyroom.ru',
 'novokuznetsk.inmyroom.ru': 'inmyroom.ru',
 'perm.inmyroom.ru': 'inmyroom.ru',
 'kaluga.inmyroom.ru': 'inmyroom.ru',
 'podolsk.inmyroom.ru': 'inmyroom.ru',
 'nizhniy-tagil.inmyroom.ru': 'inmyroom.ru',
    'fabulous.thealloha.club': 'thealloha.club',
 'belyashi.thealloha.club': 'thealloha.club',
 'fascist.thealloha.club': 'thealloha.club',
 'abelite.thealloha.club': 'thealloha.club',
 'fariseism.thealloha.club': 'thealloha.club',
 'aboulia.thealloha.club': 'thealloha.club',
 'blinds.thealloha.club': 'thealloha.club',
 'a-kimbo.thealloha.club': 'thealloha.club',
 'ability.thealloha.club': 'thealloha.club',
 'fad.thealloha.club': 'thealloha.club',
 'fanaberia.thealloha.club': 'thealloha.club',
 'abalone.thealloha.club': 'thealloha.club',
 'a-going.thealloha.club': 'thealloha.club',
 'fat.thealloha.club': 'thealloha.club',
 'blink.thealloha.club': 'thealloha.club',
 'abdomen.thealloha.club': 'thealloha.club',
 'abelian.thealloha.club': 'thealloha.club',
 'banish.thealloha.club': 'thealloha.club',
 'bypass.thealloha.club': 'thealloha.club',
 'busty.thealloha.club': 'thealloha.club',
 'absciss.thealloha.club': 'thealloha.club',
 'badminton.thealloha.club': 'thealloha.club',
 'facetious.thealloha.club': 'thealloha.club',
 'far.thealloha.club': 'thealloha.club',
 'abettee.thealloha.club': 'thealloha.club',
 'farmer.thealloha.club': 'thealloha.club',
 'ablator.thealloha.club': 'thealloha.club',
 'bundle.thealloha.club': 'thealloha.club',
 'abridge.thealloha.club': 'thealloha.club',
 'abolish.thealloha.club': 'thealloha.club',
 'fantasy.thealloha.club': 'thealloha.club',
 'abscond.thealloha.club': 'thealloha.club',
 'abdulov.thealloha.club': 'thealloha.club',
 'falsestart.thealloha.club': 'thealloha.club',
 'abettor.thealloha.club': 'thealloha.club',
 'bob.thealloha.club': 'thealloha.club',
 'abacost.thealloha.club': 'thealloha.club',
 'failing.thealloha.club': 'thealloha.club',
 'bis.thealloha.club': 'thealloha.club',
 'browning.thealloha.club': 'thealloha.club',
 'balabolka.thealloha.club': 'thealloha.club',
    'kasimov.esbd.ru': 'esbd.ru',
 'esbd.ru': 'esbd.ru',
 'glazov.esbd.ru': 'esbd.ru',
 'kotlas.esbd.ru': 'esbd.ru',
 'tinda.esbd.ru': 'esbd.ru',
 'sochi.esbd.ru': 'esbd.ru',
 'ishim.esbd.ru': 'esbd.ru',
 'korsakov.esbd.ru': 'esbd.ru',
 'suzemka.esbd.ru': 'esbd.ru',
 'mesyagutovo.esbd.ru': 'esbd.ru',
 'fryanovo.esbd.ru': 'esbd.ru',
 'smolensk.esbd.ru': 'esbd.ru',
 'matveev-kurgan.esbd.ru': 'esbd.ru',
 'bessonovka.esbd.ru': 'esbd.ru',
 'ekaterinburg.esbd.ru': 'esbd.ru',
 'orel.esbd.ru': 'esbd.ru',
 'kislovodsk.esbd.ru': 'esbd.ru',
 'shodnya.esbd.ru': 'esbd.ru',
 'chernyahovsk.esbd.ru': 'esbd.ru',
 'spassk-dalniy.esbd.ru': 'esbd.ru',
 'verhnyaya-sinyachiha.esbd.ru': 'esbd.ru',
 'buzuluk.esbd.ru': 'esbd.ru',
 'kulebaki.esbd.ru': 'esbd.ru',
 'lobnya.esbd.ru': 'esbd.ru',
 'kansk.esbd.ru': 'esbd.ru',
 'yurga1.esbd.ru': 'esbd.ru',
 'nahodka.esbd.ru': 'esbd.ru',
 'anapa.esbd.ru': 'esbd.ru',
 'krasnodar.esbd.ru': 'esbd.ru',
 'shahti.esbd.ru': 'esbd.ru',
 'stupino.esbd.ru': 'esbd.ru',
 'ust-kishert.esbd.ru': 'esbd.ru',
 'rostov-velikiy.esbd.ru': 'esbd.ru',
 'eysk.esbd.ru': 'esbd.ru',
 'krasnogorsk.esbd.ru': 'esbd.ru',
 'vyazma.esbd.ru': 'esbd.ru',
 'golitsino.esbd.ru': 'esbd.ru',
 'zhukov.esbd.ru': 'esbd.ru',
 'sharya.esbd.ru': 'esbd.ru',
 'evpatoriya.esbd.ru': 'esbd.ru',
 'severomorsk.esbd.ru': 'esbd.ru',
    'msu.ru': 'msu.ru',
 'selena.sai.msu.ru': 'msu.ru',
 'mmmf.msu.ru': 'msu.ru',
 'letopis.msu.ru': 'msu.ru',
 'nuclphys.sinp.msu.ru': 'msu.ru',
 'law.msu.ru': 'msu.ru',
 'krasnovydovo.msu.ru': 'msu.ru',
 'fbm.msu.ru': 'msu.ru',
 'de.bio.msu.ru': 'msu.ru',
 'mc.msu.ru': 'msu.ru',
 'pk.math.msu.ru': 'msu.ru',
 'old.spa.msu.ru': 'msu.ru',
 'herba.msu.ru': 'msu.ru',
 'hist.msu.ru': 'msu.ru',
 'geol.msu.ru': 'msu.ru',
 'poliklinika.mc.msu.ru': 'msu.ru',
 'chem.msu.ru': 'msu.ru',
 'journ.msu.ru': 'msu.ru',
 'fgp.msu.ru': 'msu.ru',
 'teacher.msu.ru': 'msu.ru',
 'sitito.cs.msu.ru': 'msu.ru',
 'esti.msu.ru': 'msu.ru',
 'tipl.philol.msu.ru': 'msu.ru',
 'psy.msu.ru': 'msu.ru',
 'biotech.msu.ru': 'msu.ru',
 'old.law.msu.ru': 'msu.ru',
 'audit.msu.ru': 'msu.ru',
 'quantum.phys.msu.ru': 'msu.ru',
 'geogr.msu.ru': 'msu.ru',
 'hsscm.msu.ru': 'msu.ru',
 'fonetica.philol.msu.ru': 'msu.ru',
 'football.msu.ru': 'msu.ru',
 'irm.msu.ru': 'msu.ru',
 'grep.cs.msu.ru': 'msu.ru',
 'sinp.msu.ru': 'msu.ru',
 'socio.msu.ru': 'msu.ru',
 'open.phys.msu.ru': 'msu.ru',
 'pk.ffl.msu.ru': 'msu.ru',
 'ffl.msu.ru': 'msu.ru',
 'old.polit.msu.ru': 'msu.ru',
 'ctcongress2022.chem.msu.ru': 'msu.ru',
    'video.sibnet.ru': 'sibnet.ru',
 'lib.sibnet.ru': 'sibnet.ru',
 'info.sibnet.ru': 'sibnet.ru',
 'ad1.sibnet.ru': 'sibnet.ru',
 'svoi.sibnet.ru': 'sibnet.ru',
 'mix.sibnet.ru': 'sibnet.ru',
 'soft.sibnet.ru': 'sibnet.ru',
 'photo.sibnet.ru': 'sibnet.ru',
 'forum.sibnet.ru': 'sibnet.ru',
 'getsoft.sibnet.ru': 'sibnet.ru',
 'pressa.sibnet.ru': 'sibnet.ru',
 'test.sibnet.ru': 'sibnet.ru',
 'sibnet.ru': 'sibnet.ru',
 'astro.sibnet.ru': 'sibnet.ru',
 'catalog.sibnet.ru': 'sibnet.ru',
 'mors-novosibirsk.sibnet.ru': 'sibnet.ru',
 'pobedagame.sibnet.ru': 'sibnet.ru',
 'pobeda.sibnet.ru': 'sibnet.ru',
 'link.sibnet.ru': 'sibnet.ru',
 'goroda.sibnet.ru': 'sibnet.ru',
 'smotri.sibnet.ru': 'sibnet.ru',
 'joke.sibnet.ru': 'sibnet.ru',
 'auto.sibnet.ru': 'sibnet.ru',
 'shop.sibnet.ru': 'sibnet.ru',
 'file2.sibnet.ru': 'sibnet.ru',
 'club.sibnet.ru': 'sibnet.ru',
 'help.sibnet.ru': 'sibnet.ru',
 'chess.sibnet.ru': 'sibnet.ru',
 'chat.sibnet.ru': 'sibnet.ru',
 'mail.sibnet.ru': 'sibnet.ru',
 'board.sibnet.ru': 'sibnet.ru',
 'love.sibnet.ru': 'sibnet.ru',
 'top.sibnet.ru': 'sibnet.ru',
 'day.sibnet.ru': 'sibnet.ru',
 'job.sibnet.ru': 'sibnet.ru',
 'more.sibnet.ru': 'sibnet.ru',
 'game.sibnet.ru': 'sibnet.ru',
 'rtday2021.sibnet.ru': 'sibnet.ru',
 'mors-irkutsk.sibnet.ru': 'sibnet.ru',
 'mors-barnaul.sibnet.ru': 'sibnet.ru',
 'rtnew2022.sibnet.ru': 'sibnet.ru',
    'catalog.rusdealers.ru': 'rusdealers.ru',
 'rusdealers.ru': 'rusdealers.ru',
 'izh.rusdealers.ru': 'rusdealers.ru',
 'neft.rusdealers.ru': 'rusdealers.ru',
 'chelny.rusdealers.ru': 'rusdealers.ru',
 'spb.rusdealers.ru': 'rusdealers.ru',
 'smol.rusdealers.ru': 'rusdealers.ru',
 'sykt.rusdealers.ru': 'rusdealers.ru',
 'surg.rusdealers.ru': 'rusdealers.ru',
 'kgrad.rusdealers.ru': 'rusdealers.ru',
 'vlg.rusdealers.ru': 'rusdealers.ru',
 'penza.rusdealers.ru': 'rusdealers.ru',
 'orb.rusdealers.ru': 'rusdealers.ru',
 'irk.rusdealers.ru': 'rusdealers.ru',
 'pskov.rusdealers.ru': 'rusdealers.ru',
 'brk.rusdealers.ru': 'rusdealers.ru',
 'kaluga.rusdealers.ru': 'rusdealers.ru',
 'omsk.rusdealers.ru': 'rusdealers.ru',
 'arh.rusdealers.ru': 'rusdealers.ru',
 'stav.rusdealers.ru': 'rusdealers.ru',
 'rzn.rusdealers.ru': 'rusdealers.ru',
 'kem.rusdealers.ru': 'rusdealers.ru',
 'khv.rusdealers.ru': 'rusdealers.ru',
 'bel.rusdealers.ru': 'rusdealers.ru',
 'kazan.rusdealers.ru': 'rusdealers.ru',
 'kirov.rusdealers.ru': 'rusdealers.ru',
 'ivn.rusdealers.ru': 'rusdealers.ru',
 'vrn.rusdealers.ru': 'rusdealers.ru',
 'nn.rusdealers.ru': 'rusdealers.ru',
 'srk.rusdealers.ru': 'rusdealers.ru',
 'vlad.rusdealers.ru': 'rusdealers.ru',
 'tula.rusdealers.ru': 'rusdealers.ru',
 'nsk.rusdealers.ru': 'rusdealers.ru',
 'sar.rusdealers.ru': 'rusdealers.ru',
 'ekb.rusdealers.ru': 'rusdealers.ru',
 'vl.rusdealers.ru': 'rusdealers.ru',
 'perm.rusdealers.ru': 'rusdealers.ru',
 'sochi.rusdealers.ru': 'rusdealers.ru',
 'barnaul.rusdealers.ru': 'rusdealers.ru',
 'kras.rusdealers.ru': 'rusdealers.ru',
 'ster.rusdealers.ru': 'rusdealers.ru',
    'admsurgut.ru': 'admsurgut.ru',
 'school10.admsurgut.ru': 'admsurgut.ru',
 'tik.admsurgut.ru': 'admsurgut.ru',
 'do.admsurgut.ru': 'admsurgut.ru',
 'school24.admsurgut.ru': 'admsurgut.ru',
 'school29.admsurgut.ru': 'admsurgut.ru',
 'imc.admsurgut.ru': 'admsurgut.ru',
 'maps.admsurgut.ru': 'admsurgut.ru',
 'budget.admsurgut.ru': 'admsurgut.ru',
 'invest.admsurgut.ru': 'admsurgut.ru',
 'lic1.admsurgut.ru': 'admsurgut.ru',
 'school22.admsurgut.ru': 'admsurgut.ru',
 'lic4.admsurgut.ru': 'admsurgut.ru',
 'school12.admsurgut.ru': 'admsurgut.ru',
 'school30.admsurgut.ru': 'admsurgut.ru',
 'school4.admsurgut.ru': 'admsurgut.ru',
 'school1.admsurgut.ru': 'admsurgut.ru',
 'gim2.admsurgut.ru': 'admsurgut.ru',
 'school19.admsurgut.ru': 'admsurgut.ru',
 'sgls.admsurgut.ru': 'admsurgut.ru',
 'school9.admsurgut.ru': 'admsurgut.ru',
 'progim.admsurgut.ru': 'admsurgut.ru',
 'school5.admsurgut.ru': 'admsurgut.ru',
 'school46.admsurgut.ru': 'admsurgut.ru',
 'sun.admsurgut.ru': 'admsurgut.ru',
 'zakupki.admsurgut.ru': 'admsurgut.ru',
 'lic3.admsurgut.ru': 'admsurgut.ru',
 'cdt.admsurgut.ru': 'admsurgut.ru',
 'school13.admsurgut.ru': 'admsurgut.ru',
 'gim3.admsurgut.ru': 'admsurgut.ru',
 'lic2.admsurgut.ru': 'admsurgut.ru',
 'school32.admsurgut.ru': 'admsurgut.ru',
 'school45.admsurgut.ru': 'admsurgut.ru',
 'school31.admsurgut.ru': 'admsurgut.ru',
 'delfin.admsurgut.ru': 'admsurgut.ru',
 'school3.admsurgut.ru': 'admsurgut.ru',
 'school43.admsurgut.ru': 'admsurgut.ru',
 'school27.admsurgut.ru': 'admsurgut.ru',
 'school44.admsurgut.ru': 'admsurgut.ru',
 'school20.admsurgut.ru': 'admsurgut.ru',
 'ososh1.admsurgut.ru': 'admsurgut.ru',
    'rusanalogi.ru': 'rusanalogi.ru',
 'barnaul.rusanalogi.ru': 'rusanalogi.ru',
 'balakovo.rusanalogi.ru': 'rusanalogi.ru',
 'anzhero-sudzhensk.rusanalogi.ru': 'rusanalogi.ru',
 'belgorod.rusanalogi.ru': 'rusanalogi.ru',
 'buguruslan.rusanalogi.ru': 'rusanalogi.ru',
 'batajsk.rusanalogi.ru': 'rusanalogi.ru',
 'artem.rusanalogi.ru': 'rusanalogi.ru',
 'angarsk.rusanalogi.ru': 'rusanalogi.ru',
 'arkhangelsk.rusanalogi.ru': 'rusanalogi.ru',
 'bryansk.rusanalogi.ru': 'rusanalogi.ru',
 'vladikavkaz.rusanalogi.ru': 'rusanalogi.ru',
 'arsenev.rusanalogi.ru': 'rusanalogi.ru',
 'bijsk.rusanalogi.ru': 'rusanalogi.ru',
 'balashov.rusanalogi.ru': 'rusanalogi.ru',
 'ejsk.rusanalogi.ru': 'rusanalogi.ru',
 'krasnodar.rusanalogi.ru': 'rusanalogi.ru',
 'perm.rusanalogi.ru': 'rusanalogi.ru',
 'sochi.rusanalogi.ru': 'rusanalogi.ru',
 'cherkessk.rusanalogi.ru': 'rusanalogi.ru',
 'volgodonsk.rusanalogi.ru': 'rusanalogi.ru',
 'kingisepp.rusanalogi.ru': 'rusanalogi.ru',
 'ishim.rusanalogi.ru': 'rusanalogi.ru',
 'kaliningrad.rusanalogi.ru': 'rusanalogi.ru',
 'kazan.rusanalogi.ru': 'rusanalogi.ru',
 'irkutsk.rusanalogi.ru': 'rusanalogi.ru',
 'zhigulevsk.rusanalogi.ru': 'rusanalogi.ru',
 'ivanovo.rusanalogi.ru': 'rusanalogi.ru',
 'kinel.rusanalogi.ru': 'rusanalogi.ru',
 'kansk.rusanalogi.ru': 'rusanalogi.ru',
 'kamensk-shahtinskij.rusanalogi.ru': 'rusanalogi.ru',
 'zelenograd.rusanalogi.ru': 'rusanalogi.ru',
 'spb.rusanalogi.ru': 'rusanalogi.ru',
 'himki.rusanalogi.ru': 'rusanalogi.ru',
 'shahty.rusanalogi.ru': 'rusanalogi.ru',
 'shelehov.rusanalogi.ru': 'rusanalogi.ru',
 'shadrinsk.rusanalogi.ru': 'rusanalogi.ru',
 'hanty-mansijsk.rusanalogi.ru': 'rusanalogi.ru',
 'cheboksary.rusanalogi.ru': 'rusanalogi.ru',
 'cherepovec.rusanalogi.ru': 'rusanalogi.ru',
 'chelyabinsk.rusanalogi.ru': 'rusanalogi.ru',
 'voskresensk.rusanalogi.ru': 'rusanalogi.ru',
 'novosibirsk.rusanalogi.ru': 'rusanalogi.ru',
    'ru-916.telephonebase.com': 'telephonebase.com',
 'ru-902.telephonebase.com': 'telephonebase.com',
 'ru-381.telephonebase.com': 'telephonebase.com',
 'ru-833.telephonebase.com': 'telephonebase.com',
 'ru-905.telephonebase.com': 'telephonebase.com',
 'ru-383.telephonebase.com': 'telephonebase.com',
 'ru-985.telephonebase.com': 'telephonebase.com',
 'ru-906.telephonebase.com': 'telephonebase.com',
 'ru-911.telephonebase.com': 'telephonebase.com',
 'ru-346.telephonebase.com': 'telephonebase.com',
 'ru-800.telephonebase.com': 'telephonebase.com',
 'ru-967.telephonebase.com': 'telephonebase.com',
 'ru-913.telephonebase.com': 'telephonebase.com',
 'ru-499.telephonebase.com': 'telephonebase.com',
 'ru-952.telephonebase.com': 'telephonebase.com',
 'ru-938.telephonebase.com': 'telephonebase.com',
 'ru-950.telephonebase.com': 'telephonebase.com',
 'ru-342.telephonebase.com': 'telephonebase.com',
 'ru-982.telephonebase.com': 'telephonebase.com',
 'ru-495.telephonebase.com': 'telephonebase.com',
 'ru-965.telephonebase.com': 'telephonebase.com',
 'ru-347.telephonebase.com': 'telephonebase.com',
 'ru-953.telephonebase.com': 'telephonebase.com',
 'ru-343.telephonebase.com': 'telephonebase.com',
 'ru-914.telephonebase.com': 'telephonebase.com',
 'ru-977.telephonebase.com': 'telephonebase.com',
 'ru-917.telephonebase.com': 'telephonebase.com',
 'ru-904.telephonebase.com': 'telephonebase.com',
 'ru-395.telephonebase.com': 'telephonebase.com',
 'ru-962.telephonebase.com': 'telephonebase.com',
 'ru-817.telephonebase.com': 'telephonebase.com',
 'ru-908.telephonebase.com': 'telephonebase.com',
 'ru-961.telephonebase.com': 'telephonebase.com',
 'ru-491.telephonebase.com': 'telephonebase.com',
 'ru-812.telephonebase.com': 'telephonebase.com',
 'ru-863.telephonebase.com': 'telephonebase.com',
 'ru-352.telephonebase.com': 'telephonebase.com',
 'telephonebase.com': 'telephonebase.com',
 'ru-925.telephonebase.com': 'telephonebase.com',
 'ru-963.telephonebase.com': 'telephonebase.com',
 'ru-983.telephonebase.com': 'telephonebase.com',
 'ru-991.telephonebase.com': 'telephonebase.com',
 'ru-846.telephonebase.com': 'telephonebase.com',
    'ru.investing.com': 'investing.com',
 'm.ru.investing.com': 'investing.com',
 'm.investing.com': 'investing.com',
 'investing.com': 'investing.com',
 'au.investing.com': 'investing.com',
 'm.sa.investing.com': 'investing.com',
 'm.fi.investing.com': 'investing.com',
 'cn.investing.com': 'investing.com',
 'm.uk.investing.com': 'investing.com',
 'kr.investing.com': 'investing.com',
 'm.de.investing.com': 'investing.com',
 'sa.investing.com': 'investing.com',
 'de.investing.com': 'investing.com',
 'mx.investing.com': 'investing.com',
 'fr.investing.com': 'investing.com',
 'm.jp.investing.com': 'investing.com',
 'm.cn.investing.com': 'investing.com',
 'm.in.investing.com': 'investing.com',
 'm.fr.investing.com': 'investing.com',
 'pl.investing.com': 'investing.com',
 'm.ca.investing.com': 'investing.com',
 'm.id.investing.com': 'investing.com',
 'm.br.investing.com': 'investing.com',
 'm.au.investing.com': 'investing.com',
 'm.tr.investing.com': 'investing.com',
 'br.investing.com': 'investing.com',
 'se.investing.com': 'investing.com',
 'fi.investing.com': 'investing.com',
 'jp.investing.com': 'investing.com',
 'm.gr.investing.com': 'investing.com',
 'th.investing.com': 'investing.com',
 'nl.investing.com': 'investing.com',
 'it.investing.com': 'investing.com',
 'm.it.investing.com': 'investing.com',
 'vn.investing.com': 'investing.com',
 'm.vn.investing.com': 'investing.com',
 'es.investing.com': 'investing.com',
 'm.es.investing.com': 'investing.com',
 'id.investing.com': 'investing.com',
 'm.kr.investing.com': 'investing.com',
 'il.investing.com': 'investing.com',
 'uk.investing.com': 'investing.com',
 'tr.investing.com': 'investing.com',
    'poisk.metaltorg.ru': 'metaltorg.ru',
 'doska.metaltorg.ru': 'metaltorg.ru',
 'metaltorg.ru': 'metaltorg.ru',
 'moskva.metaltorg.ru': 'metaltorg.ru',
 'truba.metaltorg.ru': 'metaltorg.ru',
 'gostbank.metaltorg.ru': 'metaltorg.ru',
 'podshipniki.metaltorg.ru': 'metaltorg.ru',
 'ferrosplavy.metaltorg.ru': 'metaltorg.ru',
 'samara.metaltorg.ru': 'metaltorg.ru',
 'med.metaltorg.ru': 'metaltorg.ru',
 'ukr.metaltorg.ru': 'metaltorg.ru',
 'kaz.metaltorg.ru': 'metaltorg.ru',
 'turkey.metaltorg.ru': 'metaltorg.ru',
 'bel.metaltorg.ru': 'metaltorg.ru',
 'china.metaltorg.ru': 'metaltorg.ru',
 'poland.metaltorg.ru': 'metaltorg.ru',
 'peterburg.metaltorg.ru': 'metaltorg.ru',
 'provoloka.metaltorg.ru': 'metaltorg.ru',
 'rostov.metaltorg.ru': 'metaltorg.ru',
 'tula.metaltorg.ru': 'metaltorg.ru',
 'armatura.metaltorg.ru': 'metaltorg.ru',
 'ekaterinburg.metaltorg.ru': 'metaltorg.ru',
 'tambov.metaltorg.ru': 'metaltorg.ru',
 'yaroslavl.metaltorg.ru': 'metaltorg.ru',
 'kaliningrad.metaltorg.ru': 'metaltorg.ru',
 'petrozavodsk.metaltorg.ru': 'metaltorg.ru',
 'kaluga.metaltorg.ru': 'metaltorg.ru',
 'cfo.metaltorg.ru': 'metaltorg.ru',
 'orel.metaltorg.ru': 'metaltorg.ru',
 'vologda.metaltorg.ru': 'metaltorg.ru',
 'vladimir.metaltorg.ru': 'metaltorg.ru',
 'lipetsk.metaltorg.ru': 'metaltorg.ru',
 'belgorod.metaltorg.ru': 'metaltorg.ru',
 'ivanovo.metaltorg.ru': 'metaltorg.ru',
 'kostroma.metaltorg.ru': 'metaltorg.ru',
 'bryansk.metaltorg.ru': 'metaltorg.ru',
 'iran.metaltorg.ru': 'metaltorg.ru',
 'tver.metaltorg.ru': 'metaltorg.ru',
 'ryazan.metaltorg.ru': 'metaltorg.ru',
 'smolensk.metaltorg.ru': 'metaltorg.ru',
 'kursk.metaltorg.ru': 'metaltorg.ru',
 'voronezh.metaltorg.ru': 'metaltorg.ru',
 'arkhangelsk.metaltorg.ru': 'metaltorg.ru',
    'dud.allohalive.com': 'allohalive.com',
 'sansa.allohalive.com': 'allohalive.com',
 'sheldon.allohalive.com': 'allohalive.com',
 'aurora.allohalive.com': 'allohalive.com',
 'klipzona.allohalive.com': 'allohalive.com',
 'pope.allohalive.com': 'allohalive.com',
 'best.allohalive.com': 'allohalive.com',
 'targaryen.allohalive.com': 'allohalive.com',
 'they.allohalive.com': 'allohalive.com',
 'karl.allohalive.com': 'allohalive.com',
 'theatrical.allohalive.com': 'allohalive.com',
 'bayas.allohalive.com': 'allohalive.com',
 'lanister.allohalive.com': 'allohalive.com',
 'bob.allohalive.com': 'allohalive.com',
 'therefore.allohalive.com': 'allohalive.com',
 'theoname.allohalive.com': 'allohalive.com',
 'kinchik.allohalive.com': 'allohalive.com',
 'thermal.allohalive.com': 'allohalive.com',
 'thermostat.allohalive.com': 'allohalive.com',
 'theory.allohalive.com': 'allohalive.com',
 'mm.allohalive.com': 'allohalive.com',
 'serseya.allohalive.com': 'allohalive.com',
 'kevin.allohalive.com': 'allohalive.com',
 'theology.allohalive.com': 'allohalive.com',
 'associative.allohalive.com': 'allohalive.com',
 'spinning.allohalive.com': 'allohalive.com',
 'thesaurus.allohalive.com': 'allohalive.com',
 'engineering.allohalive.com': 'allohalive.com',
 'theism.allohalive.com': 'allohalive.com',
 'theatre.allohalive.com': 'allohalive.com',
 'therapy.allohalive.com': 'allohalive.com',
 'alder.allohalive.com': 'allohalive.com',
 'thermometer.allohalive.com': 'allohalive.com',
 'tornado.allohalive.com': 'allohalive.com',
 'torturer.allohalive.com': 'allohalive.com',
 'thermist.allohalive.com': 'allohalive.com',
 'jerry.allohalive.com': 'allohalive.com',
 'lari.allohalive.com': 'allohalive.com',
 'theoretical.allohalive.com': 'allohalive.com',
 'torpedo-nosed.allohalive.com': 'allohalive.com',
 'baha.allohalive.com': 'allohalive.com',
 'tortuousus.allohalive.com': 'allohalive.com',
 'thesis.allohalive.com': 'allohalive.com',
    'avia.tutu.ru': 'tutu.ru',
 'tutu.ru': 'tutu.ru',
 'yaroslavl.tutu.ru': 'tutu.ru',
 'kostroma.tutu.ru': 'tutu.ru',
 'sochi.tutu.ru': 'tutu.ru',
 'novosibirsk.tutu.ru': 'tutu.ru',
 'volgograd.tutu.ru': 'tutu.ru',
 'rostov-na-donu.tutu.ru': 'tutu.ru',
 'kazan.tutu.ru': 'tutu.ru',
 'ivanovo.tutu.ru': 'tutu.ru',
 'ekburg.tutu.ru': 'tutu.ru',
 'voronezh.tutu.ru': 'tutu.ru',
 'kemerovo.tutu.ru': 'tutu.ru',
 'krasnodar.tutu.ru': 'tutu.ru',
 'samara.tutu.ru': 'tutu.ru',
 'bryansk.tutu.ru': 'tutu.ru',
 'stavropol.tutu.ru': 'tutu.ru',
 'tambov.tutu.ru': 'tutu.ru',
 'lipetsk.tutu.ru': 'tutu.ru',
 'krasnoyarsk.tutu.ru': 'tutu.ru',
 'kaluga.tutu.ru': 'tutu.ru',
 'chelyabinsk.tutu.ru': 'tutu.ru',
 'habarovsk.tutu.ru': 'tutu.ru',
 'ufa.tutu.ru': 'tutu.ru',
 'saratov.tutu.ru': 'tutu.ru',
 'barnaul.tutu.ru': 'tutu.ru',
 'kursk.tutu.ru': 'tutu.ru',
 'kaliningrad.tutu.ru': 'tutu.ru',
 'belgorod.tutu.ru': 'tutu.ru',
 'kurgan.tutu.ru': 'tutu.ru',
 'orel.tutu.ru': 'tutu.ru',
 'vologda.tutu.ru': 'tutu.ru',
 'orenburg.tutu.ru': 'tutu.ru',
 'omsk.tutu.ru': 'tutu.ru',
 'ryazan.tutu.ru': 'tutu.ru',
 'penza.tutu.ru': 'tutu.ru',
 'tula.tutu.ru': 'tutu.ru',
 'vladivostok.tutu.ru': 'tutu.ru',
 'crimea.tutu.ru': 'tutu.ru',
 'mahachkala.tutu.ru': 'tutu.ru',
 'irkutsk.tutu.ru': 'tutu.ru',
 'smolensk.tutu.ru': 'tutu.ru',
 'arhangelsk.tutu.ru': 'tutu.ru',
 'ulyanovsk.tutu.ru': 'tutu.ru',
 'abakan.tutu.ru': 'tutu.ru',
    'nagradion.ru': 'nagradion.ru',
 'fflo.nagradion.ru': 'nagradion.ru',
 'sakhfootball.nagradion.ru': 'nagradion.ru',
 'ffrb.nagradion.ru': 'nagradion.ru',
 'pfl-liga.nagradion.ru': 'nagradion.ru',
 'ffrybinsk.nagradion.ru': 'nagradion.ru',
 'goldfootball.nagradion.ru': 'nagradion.ru',
 'mrsss.nagradion.ru': 'nagradion.ru',
 'nnfootball.nagradion.ru': 'nagradion.ru',
 'mipt.nagradion.ru': 'nagradion.ru',
 'odinfutbol.nagradion.ru': 'nagradion.ru',
 'ffgol.nagradion.ru': 'nagradion.ru',
 'boff32.nagradion.ru': 'nagradion.ru',
 'sffu.nagradion.ru': 'nagradion.ru',
 'kkff-kuban.nagradion.ru': 'nagradion.ru',
 'ffgov.nagradion.ru': 'nagradion.ru',
 'ffeggo.nagradion.ru': 'nagradion.ru',
 'mfsprivolg.nagradion.ru': 'nagradion.ru',
 'ffso.nagradion.ru': 'nagradion.ru',
 'ffku.nagradion.ru': 'nagradion.ru',
 'football72.nagradion.ru': 'nagradion.ru',
 'ffge.nagradion.ru': 'nagradion.ru',
 'ffnt.nagradion.ru': 'nagradion.ru',
 'ffpvk.nagradion.ru': 'nagradion.ru',
 'voronezhfootball.nagradion.ru': 'nagradion.ru',
 'ff-rb.nagradion.ru': 'nagradion.ru',
 'mronw.nagradion.ru': 'nagradion.ru',
 'rfs21.nagradion.ru': 'nagradion.ru',
 'football64.nagradion.ru': 'nagradion.ru',
 'ffpp.nagradion.ru': 'nagradion.ru',
 'ooff.nagradion.ru': 'nagradion.ru',
 'voff35.nagradion.ru': 'nagradion.ru',
 'football70.nagradion.ru': 'nagradion.ru',
 'ffgt.nagradion.ru': 'nagradion.ru',
 'odintsovo.nagradion.ru': 'nagradion.ru',
 'ff13rm.nagradion.ru': 'nagradion.ru',
 'ffur.nagradion.ru': 'nagradion.ru',
 'yanaofootball.nagradion.ru': 'nagradion.ru',
 'vladikavkazfootball.nagradion.ru': 'nagradion.ru',
 'ffbgo.nagradion.ru': 'nagradion.ru',
 'ffdgo.nagradion.ru': 'nagradion.ru',
 'ffvrn.nagradion.ru': 'nagradion.ru',
 'ff-19.nagradion.ru': 'nagradion.ru',
 'urffm.nagradion.ru': 'nagradion.ru',
 'fs35.nagradion.ru': 'nagradion.ru',
    'ruso-oge.sdamgia.ru': 'sdamgia.ru',
 'rus-ege.sdamgia.ru': 'sdamgia.ru',
 'math-ege.sdamgia.ru': 'sdamgia.ru',
 'ege.sdamgia.ru': 'sdamgia.ru',
 'math-oge.sdamgia.ru': 'sdamgia.ru',
 'oge.sdamgia.ru': 'sdamgia.ru',
 'rus-oge.sdamgia.ru': 'sdamgia.ru',
 'sdamgia.ru': 'sdamgia.ru',
 'rus6-vpr.sdamgia.ru': 'sdamgia.ru',
 'vpr.sdamgia.ru': 'sdamgia.ru',
 'bio7-vpr.sdamgia.ru': 'sdamgia.ru',
 'chem-ege.sdamgia.ru': 'sdamgia.ru',
 'mathb-ege.sdamgia.ru': 'sdamgia.ru',
 'math4-vpr.sdamgia.ru': 'sdamgia.ru',
 'rus4-vpr.sdamgia.ru': 'sdamgia.ru',
 'phys-ege.sdamgia.ru': 'sdamgia.ru',
 'hist-ege.sdamgia.ru': 'sdamgia.ru',
 'soc-ege.sdamgia.ru': 'sdamgia.ru',
 'hist-oge.sdamgia.ru': 'sdamgia.ru',
 'bio-ege.sdamgia.ru': 'sdamgia.ru',
 'inf-oge.sdamgia.ru': 'sdamgia.ru',
 'en-ege.sdamgia.ru': 'sdamgia.ru',
 'inf-ege.sdamgia.ru': 'sdamgia.ru',
 'en-oge.sdamgia.ru': 'sdamgia.ru',
 'soc-oge.sdamgia.ru': 'sdamgia.ru',
 'geo-ege.sdamgia.ru': 'sdamgia.ru',
 'lit-ege.sdamgia.ru': 'sdamgia.ru',
 'phys-oge.sdamgia.ru': 'sdamgia.ru',
 'phys8-vpr.sdamgia.ru': 'sdamgia.ru',
 'phys11-vpr.sdamgia.ru': 'sdamgia.ru',
 'bio-oge.sdamgia.ru': 'sdamgia.ru',
 'math7-vpr.sdamgia.ru': 'sdamgia.ru',
 'geo-oge.sdamgia.ru': 'sdamgia.ru',
 'chem-oge.sdamgia.ru': 'sdamgia.ru',
 'lit-oge.sdamgia.ru': 'sdamgia.ru',
 'hist11-vpr.sdamgia.ru': 'sdamgia.ru',
 'geo6-vpr.sdamgia.ru': 'sdamgia.ru',
 'bio8-vpr.sdamgia.ru': 'sdamgia.ru',
 'geo7-vpr.sdamgia.ru': 'sdamgia.ru',
 'phys7-vpr.sdamgia.ru': 'sdamgia.ru',
 'sp-ege.sdamgia.ru': 'sdamgia.ru',
 'fr-ege.sdamgia.ru': 'sdamgia.ru',
 'hist7-vpr.sdamgia.ru': 'sdamgia.ru',
 'de-ege.sdamgia.ru': 'sdamgia.ru',
 'fr-oge.sdamgia.ru': 'sdamgia.ru',
    '23met.ru': '23met.ru',
 'spb.23met.ru': '23met.ru',
 'krasnodar.23met.ru': '23met.ru',
 'voronezh.23met.ru': '23met.ru',
 'arhangelsk.23met.ru': '23met.ru',
 'ufa.23met.ru': '23met.ru',
 'vologda.23met.ru': '23met.ru',
 'nn.23met.ru': '23met.ru',
 'nabchelny.23met.ru': '23met.ru',
 'chelyabinsk.23met.ru': '23met.ru',
 'ekb.23met.ru': '23met.ru',
 'novokyzneck.23met.ru': '23met.ru',
 'rostov.23met.ru': '23met.ru',
 'habarovsk.23met.ru': '23met.ru',
 'tumen.23met.ru': '23met.ru',
 'vladimir.23met.ru': '23met.ru',
 'barnayl.23met.ru': '23met.ru',
 'yakytsk.23met.ru': '23met.ru',
 'kirov.23met.ru': '23met.ru',
 'perm.23met.ru': '23met.ru',
 'vladivostok.23met.ru': '23met.ru',
 'samara.23met.ru': '23met.ru',
 'yaroslavl.23met.ru': '23met.ru',
 'nsk.23met.ru': '23met.ru',
 'multicity.23met.ru': '23met.ru',
 'penza.23met.ru': '23met.ru',
 'bryansk.23met.ru': '23met.ru',
 'vladikavkaz.23met.ru': '23met.ru',
 'syrgyt.23met.ru': '23met.ru',
 'kyrsk.23met.ru': '23met.ru',
 'tula.23met.ru': '23met.ru',
 'smolensk.23met.ru': '23met.ru',
 'saransk.23met.ru': '23met.ru',
 'irkytsk.23met.ru': '23met.ru',
 'omsk.23met.ru': '23met.ru',
 'cheboksary.23met.ru': '23met.ru',
 'saratov.23met.ru': '23met.ru',
 'krasnoyarsk.23met.ru': '23met.ru',
 'volgograd.23met.ru': '23met.ru',
 'orel.23met.ru': '23met.ru',
 'kazan.23met.ru': '23met.ru',
 'tambov.23met.ru': '23met.ru',
 'ivanovo.23met.ru': '23met.ru',
 'lipetsk.23met.ru': '23met.ru',
 'mahachkala.23met.ru': '23met.ru',
 'ylianovsk.23met.ru': '23met.ru',
    'ryazhsk.spravmer.ru': 'spravmer.ru',
 'yurga.spravmer.ru': 'spravmer.ru',
 'uchaly.spravmer.ru': 'spravmer.ru',
 'aznakaevo.spravmer.ru': 'spravmer.ru',
 'usinsk.spravmer.ru': 'spravmer.ru',
 'velsk.spravmer.ru': 'spravmer.ru',
 'teberda.spravmer.ru': 'spravmer.ru',
 'stanica-poltavskaya.spravmer.ru': 'spravmer.ru',
 'kurgan.spravmer.ru': 'spravmer.ru',
 'rybinsk.spravmer.ru': 'spravmer.ru',
 'kurganinsk.spravmer.ru': 'spravmer.ru',
 'ekaterinburg.spravmer.ru': 'spravmer.ru',
 'losino-petrovskij.spravmer.ru': 'spravmer.ru',
 'severodvinsk.spravmer.ru': 'spravmer.ru',
 'podolsk.spravmer.ru': 'spravmer.ru',
 'selo-troickoe-2.spravmer.ru': 'spravmer.ru',
 'furmanov.spravmer.ru': 'spravmer.ru',
 'zarechnyj.spravmer.ru': 'spravmer.ru',
 'mirnyj.spravmer.ru': 'spravmer.ru',
 'murom.spravmer.ru': 'spravmer.ru',
 'kaluga.spravmer.ru': 'spravmer.ru',
 'vyborg.spravmer.ru': 'spravmer.ru',
 'solikamsk.spravmer.ru': 'spravmer.ru',
 'mihajlovka.spravmer.ru': 'spravmer.ru',
 'pavlovskij-posad.spravmer.ru': 'spravmer.ru',
 'serov.spravmer.ru': 'spravmer.ru',
 'irbit.spravmer.ru': 'spravmer.ru',
 'karabanovo.spravmer.ru': 'spravmer.ru',
 'noginsk.spravmer.ru': 'spravmer.ru',
 'dzerzhinsk.spravmer.ru': 'spravmer.ru',
 'majkop.spravmer.ru': 'spravmer.ru',
 'shahunya.spravmer.ru': 'spravmer.ru',
 'apsheronsk.spravmer.ru': 'spravmer.ru',
 'kineshma.spravmer.ru': 'spravmer.ru',
 'nevinnomyssk.spravmer.ru': 'spravmer.ru',
 'armavir.spravmer.ru': 'spravmer.ru',
 'dedovsk.spravmer.ru': 'spravmer.ru',
 'samara.spravmer.ru': 'spravmer.ru',
 'tambov.spravmer.ru': 'spravmer.ru',
 'belorechensk.spravmer.ru': 'spravmer.ru',
 'cherepovec.spravmer.ru': 'spravmer.ru',
 'nurlat.spravmer.ru': 'spravmer.ru',
 'buzuluk.spravmer.ru': 'spravmer.ru',
 'anapa.spravmer.ru': 'spravmer.ru',
 'udomlya.spravmer.ru': 'spravmer.ru',
 'ipatovo.spravmer.ru': 'spravmer.ru',
 'rasskazovo.spravmer.ru': 'spravmer.ru',
    'top.novosel.ru': 'novosel.ru',
 'novosel.ru': 'novosel.ru',
 'ulan-ude.novosel.ru': 'novosel.ru',
 'moskva.novosel.ru': 'novosel.ru',
 'anapa.novosel.ru': 'novosel.ru',
 'moskovskaya-oblast.novosel.ru': 'novosel.ru',
 'sankt-peterburg.novosel.ru': 'novosel.ru',
 'novorossiysk.novosel.ru': 'novosel.ru',
 'sochi.novosel.ru': 'novosel.ru',
 'cheboksary.novosel.ru': 'novosel.ru',
 'sevastopol.novosel.ru': 'novosel.ru',
 'mytischi.novosel.ru': 'novosel.ru',
 'krasnodar.novosel.ru': 'novosel.ru',
 'kaliningrad.novosel.ru': 'novosel.ru',
 'sverdlovskaya-oblast.novosel.ru': 'novosel.ru',
 'ekaterinburg.novosel.ru': 'novosel.ru',
 'krasnodarskiy-krai.novosel.ru': 'novosel.ru',
 'krym.novosel.ru': 'novosel.ru',
 'stavropol.novosel.ru': 'novosel.ru',
 'leningradskaya-oblast.novosel.ru': 'novosel.ru',
 'tverskaya-oblast.novosel.ru': 'novosel.ru',
 'tulskaya-oblast.novosel.ru': 'novosel.ru',
 'tatarstan.novosel.ru': 'novosel.ru',
 'krasnogorsk.novosel.ru': 'novosel.ru',
 'vladivostok.novosel.ru': 'novosel.ru',
 'samara.novosel.ru': 'novosel.ru',
 'rostov-na-donu.novosel.ru': 'novosel.ru',
 'rostovskaya-oblast.novosel.ru': 'novosel.ru',
 'ramenskoe.novosel.ru': 'novosel.ru',
 'nizhniy-novgorod.novosel.ru': 'novosel.ru',
 'lubertsy.novosel.ru': 'novosel.ru',
 'kaluzhskaya-oblast.novosel.ru': 'novosel.ru',
 'kareliya.novosel.ru': 'novosel.ru',
 'vladimir.novosel.ru': 'novosel.ru',
 'yakutsk.novosel.ru': 'novosel.ru',
 'altaiskiy-krai.novosel.ru': 'novosel.ru',
 'msk.novosel.ru': 'novosel.ru',
 'az.novosel.ru': 'novosel.ru',
 'podolsk.novosel.ru': 'novosel.ru',
 'novosibirskaya-oblast.novosel.ru': 'novosel.ru',
 'krasnoyarsk.novosel.ru': 'novosel.ru',
 'irkutsk.novosel.ru': 'novosel.ru',
 'novosibirsk.novosel.ru': 'novosel.ru',
 'omsk.novosel.ru': 'novosel.ru',
 'ulyanovsk.novosel.ru': 'novosel.ru',
 'penza.novosel.ru': 'novosel.ru',
 'kirovskaya-oblast.novosel.ru': 'novosel.ru',
 'bashkortostan.novosel.ru': 'novosel.ru',
    'school53.ivedu.ru': 'ivedu.ru',
 'school1.ivedu.ru': 'ivedu.ru',
 'ddt3.ivedu.ru': 'ivedu.ru',
 'school28.ivedu.ru': 'ivedu.ru',
 'crdo.ivedu.ru': 'ivedu.ru',
 'school67.ivedu.ru': 'ivedu.ru',
 'school5.ivedu.ru': 'ivedu.ru',
 'school18.ivedu.ru': 'ivedu.ru',
 'school23.ivedu.ru': 'ivedu.ru',
 'ivedu.ru': 'ivedu.ru',
 'dou56.ivedu.ru': 'ivedu.ru',
 'dou52.ivedu.ru': 'ivedu.ru',
 'dou19.ivedu.ru': 'ivedu.ru',
 'dou21.ivedu.ru': 'ivedu.ru',
 'dou50.ivedu.ru': 'ivedu.ru',
 'school63.ivedu.ru': 'ivedu.ru',
 'school65.ivedu.ru': 'ivedu.ru',
 'school17.ivedu.ru': 'ivedu.ru',
 'school8.ivedu.ru': 'ivedu.ru',
 'school7.ivedu.ru': 'ivedu.ru',
 'school24.ivedu.ru': 'ivedu.ru',
 'school33.ivedu.ru': 'ivedu.ru',
 'school36.ivedu.ru': 'ivedu.ru',
 'licey22.ivedu.ru': 'ivedu.ru',
 'school61.ivedu.ru': 'ivedu.ru',
 'school49.ivedu.ru': 'ivedu.ru',
 'school41.ivedu.ru': 'ivedu.ru',
 'school56.ivedu.ru': 'ivedu.ru',
 'school37.ivedu.ru': 'ivedu.ru',
 'school30.ivedu.ru': 'ivedu.ru',
 'school39.ivedu.ru': 'ivedu.ru',
 'school3.ivedu.ru': 'ivedu.ru',
 'school50.ivedu.ru': 'ivedu.ru',
 'school54.ivedu.ru': 'ivedu.ru',
 'school55.ivedu.ru': 'ivedu.ru',
 'school64.ivedu.ru': 'ivedu.ru',
 'school20.ivedu.ru': 'ivedu.ru',
 'school26.ivedu.ru': 'ivedu.ru',
 'school11.ivedu.ru': 'ivedu.ru',
 'school2.ivedu.ru': 'ivedu.ru',
 'school19.ivedu.ru': 'ivedu.ru',
 'school62.ivedu.ru': 'ivedu.ru',
 'school68.ivedu.ru': 'ivedu.ru',
 'school29.ivedu.ru': 'ivedu.ru',
 'school6.ivedu.ru': 'ivedu.ru',
 'school15.ivedu.ru': 'ivedu.ru',
 'school44.ivedu.ru': 'ivedu.ru',
 'school14.ivedu.ru': 'ivedu.ru',
    'nigniy-novgorod.magazinmayak.ru': 'magazinmayak.ru',
 'magazinmayak.ru': 'magazinmayak.ru',
 'ekaterinburg.magazinmayak.ru': 'magazinmayak.ru',
 'perm.magazinmayak.ru': 'magazinmayak.ru',
 'vologda.magazinmayak.ru': 'magazinmayak.ru',
 'sankt-peterburg.magazinmayak.ru': 'magazinmayak.ru',
 'tomsk.magazinmayak.ru': 'magazinmayak.ru',
 'pskov.magazinmayak.ru': 'magazinmayak.ru',
 'krasnoyarsk.magazinmayak.ru': 'magazinmayak.ru',
 'orenburg.magazinmayak.ru': 'magazinmayak.ru',
 'kaliningrad.magazinmayak.ru': 'magazinmayak.ru',
 'velikie-luki.magazinmayak.ru': 'magazinmayak.ru',
 'novosibirsk.magazinmayak.ru': 'magazinmayak.ru',
 'chelyabinsk.magazinmayak.ru': 'magazinmayak.ru',
 'krasnodar.magazinmayak.ru': 'magazinmayak.ru',
 'tyumen.magazinmayak.ru': 'magazinmayak.ru',
 'tambov.magazinmayak.ru': 'magazinmayak.ru',
 'ufa.magazinmayak.ru': 'magazinmayak.ru',
 'voroneg.magazinmayak.ru': 'magazinmayak.ru',
 'siktivkar.magazinmayak.ru': 'magazinmayak.ru',
 'omsk.magazinmayak.ru': 'magazinmayak.ru',
 'yaroslavl.magazinmayak.ru': 'magazinmayak.ru',
 'orsk.magazinmayak.ru': 'magazinmayak.ru',
 'smolensk.magazinmayak.ru': 'magazinmayak.ru',
 'barnaul.magazinmayak.ru': 'magazinmayak.ru',
 'habarovsk.magazinmayak.ru': 'magazinmayak.ru',
 'samara.magazinmayak.ru': 'magazinmayak.ru',
 'kurgan.magazinmayak.ru': 'magazinmayak.ru',
 'igevsk.magazinmayak.ru': 'magazinmayak.ru',
 'rostov-na-donu.magazinmayak.ru': 'magazinmayak.ru',
 'salavat.magazinmayak.ru': 'magazinmayak.ru',
 'belgorod.magazinmayak.ru': 'magazinmayak.ru',
 'sochi.magazinmayak.ru': 'magazinmayak.ru',
 'vladimir.magazinmayak.ru': 'magazinmayak.ru',
 'kaluga.magazinmayak.ru': 'magazinmayak.ru',
 'taganrog.magazinmayak.ru': 'magazinmayak.ru',
 'oryol.magazinmayak.ru': 'magazinmayak.ru',
 'stariy-oskol.magazinmayak.ru': 'magazinmayak.ru',
 'ryazan.magazinmayak.ru': 'magazinmayak.ru',
 'volgograd.magazinmayak.ru': 'magazinmayak.ru',
 'kursk.magazinmayak.ru': 'magazinmayak.ru',
 'nigniy-tagil.magazinmayak.ru': 'magazinmayak.ru',
 'kazan.magazinmayak.ru': 'magazinmayak.ru',
 'velikiy-novgorod.magazinmayak.ru': 'magazinmayak.ru',
 'noginsk.magazinmayak.ru': 'magazinmayak.ru',
 'saransk.magazinmayak.ru': 'magazinmayak.ru',
 'cheboksari.magazinmayak.ru': 'magazinmayak.ru',
 'novokuznetsk.magazinmayak.ru': 'magazinmayak.ru',
 'lipetsk.magazinmayak.ru': 'magazinmayak.ru',
    'novos.mk.ru': 'mk.ru',
 'hab.mk.ru': 'mk.ru',
 'omsk.mk.ru': 'mk.ru',
 'rzn.mk.ru': 'mk.ru',
 'kuban.mk.ru': 'mk.ru',
 'spb.mk.ru': 'mk.ru',
 'brl.mk.ru': 'mk.ru',
 'ulan.mk.ru': 'mk.ru',
 'volg.mk.ru': 'mk.ru',
 'karel.mk.ru': 'mk.ru',
 'oren.mk.ru': 'mk.ru',
 'chel.mk.ru': 'mk.ru',
 'ufa.mk.ru': 'mk.ru',
 'kavkaz.mk.ru': 'mk.ru',
 'ast.mk.ru': 'mk.ru',
 'kras.mk.ru': 'mk.ru',
 'eburg.mk.ru': 'mk.ru',
 'tomsk.mk.ru': 'mk.ru',
 'izhevsk.mk.ru': 'mk.ru',
 'crimea.mk.ru': 'mk.ru',
 'nn.mk.ru': 'mk.ru',
 'vlad.mk.ru': 'mk.ru',
 'saratov.mk.ru': 'mk.ru',
 'perm.mk.ru': 'mk.ru',
 'vladimir.mk.ru': 'mk.ru',
 'mk.ru': 'mk.ru',
 'kazan.mk.ru': 'mk.ru',
 'serp.mk.ru': 'mk.ru',
 'yar.mk.ru': 'mk.ru',
 'tula.mk.ru': 'mk.ru',
 'mkala.mk.ru': 'mk.ru',
 'cheb.mk.ru': 'mk.ru',
 'tambov.mk.ru': 'mk.ru',
 'sochi.mk.ru': 'mk.ru',
 'kostroma.mk.ru': 'mk.ru',
 'baikal.mk.ru': 'mk.ru',
 'tver.mk.ru': 'mk.ru',
 'ugra.mk.ru': 'mk.ru',
 'vologda.mk.ru': 'mk.ru',
 'murmansk.mk.ru': 'mk.ru',
 'vrn.mk.ru': 'mk.ru',
 'chr.mk.ru': 'mk.ru',
 'rostov.mk.ru': 'mk.ru',
 'arh.mk.ru': 'mk.ru',
 'tumen.mk.ru': 'mk.ru',
 'samara.mk.ru': 'mk.ru',
 'kbr.mk.ru': 'mk.ru',
 'yakutia.mk.ru': 'mk.ru',
 'mrl.mk.ru': 'mk.ru',
    'rostovnadonu.buyreklama.ru': 'buyreklama.ru',
 'kaluga.buyreklama.ru': 'buyreklama.ru',
 'ramenskoe.buyreklama.ru': 'buyreklama.ru',
 'novokuznetsk-42.buyreklama.ru': 'buyreklama.ru',
 'kirov.buyreklama.ru': 'buyreklama.ru',
 '48-lipetsk.buyreklama.ru': 'buyreklama.ru',
 'vologda.buyreklama.ru': 'buyreklama.ru',
 'buyreklama.ru': 'buyreklama.ru',
 'berezniki.buyreklama.ru': 'buyreklama.ru',
 'naberezhnye-chelny-16.buyreklama.ru': 'buyreklama.ru',
 'vladimir.buyreklama.ru': 'buyreklama.ru',
 'blagoveiensk.buyreklama.ru': 'buyreklama.ru',
 'ulyanovsk-73.buyreklama.ru': 'buyreklama.ru',
 'kyzyl.buyreklama.ru': 'buyreklama.ru',
 'yaroslavl-76.buyreklama.ru': 'buyreklama.ru',
 'tver.buyreklama.ru': 'buyreklama.ru',
 'habarovsk27.buyreklama.ru': 'buyreklama.ru',
 'zheleznodorozhnyj.buyreklama.ru': 'buyreklama.ru',
 'salavat.buyreklama.ru': 'buyreklama.ru',
 'shahty.buyreklama.ru': 'buyreklama.ru',
 'miass.buyreklama.ru': 'buyreklama.ru',
 'tolyatti-63.buyreklama.ru': 'buyreklama.ru',
 '36-voronezh.buyreklama.ru': 'buyreklama.ru',
 'nahodka.buyreklama.ru': 'buyreklama.ru',
 'syktyvkar.buyreklama.ru': 'buyreklama.ru',
 'sterlitamak.buyreklama.ru': 'buyreklama.ru',
 'barnaul.buyreklama.ru': 'buyreklama.ru',
 'sergiev-posad.buyreklama.ru': 'buyreklama.ru',
 'elec.buyreklama.ru': 'buyreklama.ru',
 'cherepovec.buyreklama.ru': 'buyreklama.ru',
 'leninsk-kuzneckij.buyreklama.ru': 'buyreklama.ru',
 'balashiha.buyreklama.ru': 'buyreklama.ru',
 'ussurijsk.buyreklama.ru': 'buyreklama.ru',
 'smolensk.buyreklama.ru': 'buyreklama.ru',
 'sarapul.buyreklama.ru': 'buyreklama.ru',
 'majkop.buyreklama.ru': 'buyreklama.ru',
 'tobolsk.buyreklama.ru': 'buyreklama.ru',
 'tula.buyreklama.ru': 'buyreklama.ru',
 'nizhnekamsk.buyreklama.ru': 'buyreklama.ru',
 'taganrog.buyreklama.ru': 'buyreklama.ru',
 'orehovo-zuevo.buyreklama.ru': 'buyreklama.ru',
 'armavir.buyreklama.ru': 'buyreklama.ru',
 'norilsk.buyreklama.ru': 'buyreklama.ru',
 'abakan.buyreklama.ru': 'buyreklama.ru',
 'nojabrsk.buyreklama.ru': 'buyreklama.ru',
 'belogorsk.buyreklama.ru': 'buyreklama.ru',
 'arhangelsk.buyreklama.ru': 'buyreklama.ru',
 'belgorod.buyreklama.ru': 'buyreklama.ru',
 'tikhvin.buyreklama.ru': 'buyreklama.ru',
 'severodvinsk.buyreklama.ru': 'buyreklama.ru',
 'murmansk.buyreklama.ru': 'buyreklama.ru',
    'kartasporta.ru': 'kartasporta.ru',
 'rostov.kartasporta.ru': 'kartasporta.ru',
 'murmansk.kartasporta.ru': 'kartasporta.ru',
 'vladivostok.kartasporta.ru': 'kartasporta.ru',
 'kazan.kartasporta.ru': 'kartasporta.ru',
 'tomsk.kartasporta.ru': 'kartasporta.ru',
 'ufa.kartasporta.ru': 'kartasporta.ru',
 'spb.kartasporta.ru': 'kartasporta.ru',
 'voronezh.kartasporta.ru': 'kartasporta.ru',
 'yaroslavl.kartasporta.ru': 'kartasporta.ru',
 'krasnodar.kartasporta.ru': 'kartasporta.ru',
 'ekaterenburg.kartasporta.ru': 'kartasporta.ru',
 'saransk.kartasporta.ru': 'kartasporta.ru',
 'naberezhnye-chelny.kartasporta.ru': 'kartasporta.ru',
 'kurgan.kartasporta.ru': 'kartasporta.ru',
 'tambov.kartasporta.ru': 'kartasporta.ru',
 'kirov.kartasporta.ru': 'kartasporta.ru',
 'orenburg.kartasporta.ru': 'kartasporta.ru',
 'saratov.kartasporta.ru': 'kartasporta.ru',
 'chelyabinsk.kartasporta.ru': 'kartasporta.ru',
 'tumen.kartasporta.ru': 'kartasporta.ru',
 'stavropol.kartasporta.ru': 'kartasporta.ru',
 'komsomolsk-on-amur.kartasporta.ru': 'kartasporta.ru',
 'izhevsk.kartasporta.ru': 'kartasporta.ru',
 'mytischi.kartasporta.ru': 'kartasporta.ru',
 'novosib.kartasporta.ru': 'kartasporta.ru',
 'abakan.kartasporta.ru': 'kartasporta.ru',
 'habarovsk.kartasporta.ru': 'kartasporta.ru',
 'kaliningrad.kartasporta.ru': 'kartasporta.ru',
 'barnaul.kartasporta.ru': 'kartasporta.ru',
 'lipeck.kartasporta.ru': 'kartasporta.ru',
 'pskov.kartasporta.ru': 'kartasporta.ru',
 'magnitogorsk.kartasporta.ru': 'kartasporta.ru',
 'engels.kartasporta.ru': 'kartasporta.ru',
 'yoshkar-ola.kartasporta.ru': 'kartasporta.ru',
 'sochi.kartasporta.ru': 'kartasporta.ru',
 'omsk.kartasporta.ru': 'kartasporta.ru',
 'perm.kartasporta.ru': 'kartasporta.ru',
 'vologda.kartasporta.ru': 'kartasporta.ru',
 'samara.kartasporta.ru': 'kartasporta.ru',
 'penza.kartasporta.ru': 'kartasporta.ru',
 'krasnoyarsk.kartasporta.ru': 'kartasporta.ru',
 'siktivkar.kartasporta.ru': 'kartasporta.ru',
 'ryazan.kartasporta.ru': 'kartasporta.ru',
 'maykop.kartasporta.ru': 'kartasporta.ru',
 'hanti-mansiysk.kartasporta.ru': 'kartasporta.ru',
 'kemerovo.kartasporta.ru': 'kartasporta.ru',
 'vladimir.kartasporta.ru': 'kartasporta.ru',
 'nnovgorod.kartasporta.ru': 'kartasporta.ru',
 'novorossiysk.kartasporta.ru': 'kartasporta.ru',
 'petrozavodsk.kartasporta.ru': 'kartasporta.ru',
    'spb.tradedir.ru': 'tradedir.ru',
 'info.tradedir.ru': 'tradedir.ru',
 'kemerovo.tradedir.ru': 'tradedir.ru',
 'volgograd.tradedir.ru': 'tradedir.ru',
 'vladimir.tradedir.ru': 'tradedir.ru',
 'cabinet.tradedir.ru': 'tradedir.ru',
 'ekaterinburg.tradedir.ru': 'tradedir.ru',
 'search.tradedir.ru': 'tradedir.ru',
 'nizhnij-novgorod.tradedir.ru': 'tradedir.ru',
 'contracts.tradedir.ru': 'tradedir.ru',
 'novosibirsk.tradedir.ru': 'tradedir.ru',
 'krasnodar.tradedir.ru': 'tradedir.ru',
 'irkutsk.tradedir.ru': 'tradedir.ru',
 'belgorod.tradedir.ru': 'tradedir.ru',
 'moskva.tradedir.ru': 'tradedir.ru',
 'vladivostok.tradedir.ru': 'tradedir.ru',
 'ufa.tradedir.ru': 'tradedir.ru',
 'kaluga.tradedir.ru': 'tradedir.ru',
 'izhevsk.tradedir.ru': 'tradedir.ru',
 'vologda.tradedir.ru': 'tradedir.ru',
 'ryazan.tradedir.ru': 'tradedir.ru',
 'orel.tradedir.ru': 'tradedir.ru',
 'kazan.tradedir.ru': 'tradedir.ru',
 'ulyanovsk.tradedir.ru': 'tradedir.ru',
 'tula.tradedir.ru': 'tradedir.ru',
 'lipeck.tradedir.ru': 'tradedir.ru',
 'barnaul.tradedir.ru': 'tradedir.ru',
 'perm.tradedir.ru': 'tradedir.ru',
 'kursk.tradedir.ru': 'tradedir.ru',
 'orenburg.tradedir.ru': 'tradedir.ru',
 'voronezh.tradedir.ru': 'tradedir.ru',
 'yaroslavl.tradedir.ru': 'tradedir.ru',
 'saratov.tradedir.ru': 'tradedir.ru',
 'tjumen.tradedir.ru': 'tradedir.ru',
 'bryansk.tradedir.ru': 'tradedir.ru',
 'tomsk.tradedir.ru': 'tradedir.ru',
 'stavropol.tradedir.ru': 'tradedir.ru',
 'chelyabinsk.tradedir.ru': 'tradedir.ru',
 'ivanovo.tradedir.ru': 'tradedir.ru',
 'omsk.tradedir.ru': 'tradedir.ru',
 'kirov.tradedir.ru': 'tradedir.ru',
 'krasnoyarsk.tradedir.ru': 'tradedir.ru',
 'rostov-na-donu.tradedir.ru': 'tradedir.ru',
 'simferopol.tradedir.ru': 'tradedir.ru',
 'penza.tradedir.ru': 'tradedir.ru',
 'blagoveschensk.tradedir.ru': 'tradedir.ru',
 'tver.tradedir.ru': 'tradedir.ru',
 'habarovsk.tradedir.ru': 'tradedir.ru',
 'samara.tradedir.ru': 'tradedir.ru',
 'smolensk.tradedir.ru': 'tradedir.ru',
 'cheboksary.tradedir.ru': 'tradedir.ru',
 'tradedir.ru': 'tradedir.ru',
    'novobyt.ru': 'novobyt.ru',
 'lip.novobyt.ru': 'novobyt.ru',
 'krym.novobyt.ru': 'novobyt.ru',
 'bryansk.novobyt.ru': 'novobyt.ru',
 'serg.novobyt.ru': 'novobyt.ru',
 'tumen.novobyt.ru': 'novobyt.ru',
 'blag.novobyt.ru': 'novobyt.ru',
 'rame.novobyt.ru': 'novobyt.ru',
 'bori.novobyt.ru': 'novobyt.ru',
 'pech.novobyt.ru': 'novobyt.ru',
 'belorechensk.novobyt.ru': 'novobyt.ru',
 'uss.novobyt.ru': 'novobyt.ru',
 'vysh.novobyt.ru': 'novobyt.ru',
 'kans.novobyt.ru': 'novobyt.ru',
 'nsk.novobyt.ru': 'novobyt.ru',
 'msk.novobyt.ru': 'novobyt.ru',
 'obni.novobyt.ru': 'novobyt.ru',
 'izh.novobyt.ru': 'novobyt.ru',
 'omsk.novobyt.ru': 'novobyt.ru',
 'perm.novobyt.ru': 'novobyt.ru',
 'kazan.novobyt.ru': 'novobyt.ru',
 'rosl.novobyt.ru': 'novobyt.ru',
 'lysv.novobyt.ru': 'novobyt.ru',
 'brn.novobyt.ru': 'novobyt.ru',
 'hbr.novobyt.ru': 'novobyt.ru',
 'shadrinsk.novobyt.ru': 'novobyt.ru',
 'ufa.novobyt.ru': 'novobyt.ru',
 'tambov.novobyt.ru': 'novobyt.ru',
 'oren.novobyt.ru': 'novobyt.ru',
 'samara.novobyt.ru': 'novobyt.ru',
 'abakan.novobyt.ru': 'novobyt.ru',
 'kbshev.novobyt.ru': 'novobyt.ru',
 'spb.novobyt.ru': 'novobyt.ru',
 'rub.novobyt.ru': 'novobyt.ru',
 'nk.novobyt.ru': 'novobyt.ru',
 'ekat.novobyt.ru': 'novobyt.ru',
 'simf.novobyt.ru': 'novobyt.ru',
 'stav.novobyt.ru': 'novobyt.ru',
 'stk.novobyt.ru': 'novobyt.ru',
 'kln.novobyt.ru': 'novobyt.ru',
 'mich.novobyt.ru': 'novobyt.ru',
 'aksa.novobyt.ru': 'novobyt.ru',
 'altaysk.novobyt.ru': 'novobyt.ru',
 'alex.novobyt.ru': 'novobyt.ru',
 'mirniy.novobyt.ru': 'novobyt.ru',
 'novocheboksarsk.novobyt.ru': 'novobyt.ru',
 'yar.novobyt.ru': 'novobyt.ru',
 'pnz.novobyt.ru': 'novobyt.ru',
 'rz.novobyt.ru': 'novobyt.ru',
 'chel.novobyt.ru': 'novobyt.ru',
 'ga.novobyt.ru': 'novobyt.ru',
 'vrn.novobyt.ru': 'novobyt.ru',
    'other.avtotochki.ru': 'avtotochki.ru',
 'samara.avtotochki.ru': 'avtotochki.ru',
 'avtotochki.ru': 'avtotochki.ru',
 'omsk.avtotochki.ru': 'avtotochki.ru',
 'khabarovsk.avtotochki.ru': 'avtotochki.ru',
 'surgut.avtotochki.ru': 'avtotochki.ru',
 'spb.avtotochki.ru': 'avtotochki.ru',
 'krasnodar.avtotochki.ru': 'avtotochki.ru',
 'rostov.avtotochki.ru': 'avtotochki.ru',
 'kirov.avtotochki.ru': 'avtotochki.ru',
 'krasnoyarsk.avtotochki.ru': 'avtotochki.ru',
 'ryazan.avtotochki.ru': 'avtotochki.ru',
 'orenburg.avtotochki.ru': 'avtotochki.ru',
 'perm.avtotochki.ru': 'avtotochki.ru',
 'tula.avtotochki.ru': 'avtotochki.ru',
 'ufa.avtotochki.ru': 'avtotochki.ru',
 'yuzhnosakhalinsk.avtotochki.ru': 'avtotochki.ru',
 'chelyabinsk.avtotochki.ru': 'avtotochki.ru',
 'ekaterinburg.avtotochki.ru': 'avtotochki.ru',
 'novosibirsk.avtotochki.ru': 'avtotochki.ru',
 'nnovgorod.avtotochki.ru': 'avtotochki.ru',
 'ulanude.avtotochki.ru': 'avtotochki.ru',
 'voronezh.avtotochki.ru': 'avtotochki.ru',
 'vologda.avtotochki.ru': 'avtotochki.ru',
 'stavropol.avtotochki.ru': 'avtotochki.ru',
 'yaroslavl.avtotochki.ru': 'avtotochki.ru',
 'tyumen.avtotochki.ru': 'avtotochki.ru',
 'kaliningrad.avtotochki.ru': 'avtotochki.ru',
 'izhevsk.avtotochki.ru': 'avtotochki.ru',
 'saratov.avtotochki.ru': 'avtotochki.ru',
 'tver.avtotochki.ru': 'avtotochki.ru',
 'kazan.avtotochki.ru': 'avtotochki.ru',
 'penza.avtotochki.ru': 'avtotochki.ru',
 'syktyvkar.avtotochki.ru': 'avtotochki.ru',
 'volgograd.avtotochki.ru': 'avtotochki.ru',
 'ulyanovsk.avtotochki.ru': 'avtotochki.ru',
 'belgorod.avtotochki.ru': 'avtotochki.ru',
 'murmansk.avtotochki.ru': 'avtotochki.ru',
 'tomsk.avtotochki.ru': 'avtotochki.ru',
 'kurgan.avtotochki.ru': 'avtotochki.ru',
 'irkutsk.avtotochki.ru': 'avtotochki.ru',
 'bryansk.avtotochki.ru': 'avtotochki.ru',
 'vladimir.avtotochki.ru': 'avtotochki.ru',
 'lipetsk.avtotochki.ru': 'avtotochki.ru',
 'barnaul.avtotochki.ru': 'avtotochki.ru',
 'cheboksary.avtotochki.ru': 'avtotochki.ru',
 'kaluga.avtotochki.ru': 'avtotochki.ru',
 'arkhangelsk.avtotochki.ru': 'avtotochki.ru',
 'ivanovo.avtotochki.ru': 'avtotochki.ru',
 'kursk.avtotochki.ru': 'avtotochki.ru',
 'kemerovo.avtotochki.ru': 'avtotochki.ru',
 'chita.avtotochki.ru': 'avtotochki.ru',
    'novorossiysk.4kolesa.co': '4kolesa.co',
 'rostov-na-donu.4kolesa.co': '4kolesa.co',
 'bryansk.4kolesa.co': '4kolesa.co',
 'msk.4kolesa.co': '4kolesa.co',
 '4kolesa.co': '4kolesa.co',
 'chelyabinsk.4kolesa.co': '4kolesa.co',
 'ekaterinburg.4kolesa.co': '4kolesa.co',
 'tolyatti.4kolesa.co': '4kolesa.co',
 'nignekamsk.4kolesa.co': '4kolesa.co',
 'ufa.4kolesa.co': '4kolesa.co',
 'novosibirsk.4kolesa.co': '4kolesa.co',
 'tula.4kolesa.co': '4kolesa.co',
 'vladimir.4kolesa.co': '4kolesa.co',
 'spb.4kolesa.co': '4kolesa.co',
 'nigniy-novgorod.4kolesa.co': '4kolesa.co',
 'kaliningrad.4kolesa.co': '4kolesa.co',
 'krasnoyarsk.4kolesa.co': '4kolesa.co',
 'tomsk.4kolesa.co': '4kolesa.co',
 'ryazan.4kolesa.co': '4kolesa.co',
 'stariy-oskol.4kolesa.co': '4kolesa.co',
 'kemerovo.4kolesa.co': '4kolesa.co',
 'kirov.4kolesa.co': '4kolesa.co',
 'pskov.4kolesa.co': '4kolesa.co',
 'orenburg.4kolesa.co': '4kolesa.co',
 'yoshkar-ola.4kolesa.co': '4kolesa.co',
 'blagoveshchensk.4kolesa.co': '4kolesa.co',
 'voroneg.4kolesa.co': '4kolesa.co',
 'samara.4kolesa.co': '4kolesa.co',
 'kaluga.4kolesa.co': '4kolesa.co',
 'neftekamsk.4kolesa.co': '4kolesa.co',
 'saratov.4kolesa.co': '4kolesa.co',
 'siktivkar.4kolesa.co': '4kolesa.co',
 'igevsk.4kolesa.co': '4kolesa.co',
 'magnitogorsk.4kolesa.co': '4kolesa.co',
 'tyumen.4kolesa.co': '4kolesa.co',
 'engels.4kolesa.co': '4kolesa.co',
 'abakan.4kolesa.co': '4kolesa.co',
 'sochi.4kolesa.co': '4kolesa.co',
 'orel.4kolesa.co': '4kolesa.co',
 'novokuznetsk.4kolesa.co': '4kolesa.co',
 'komsomolsk-na-amure.4kolesa.co': '4kolesa.co',
 'kazan.4kolesa.co': '4kolesa.co',
 'krasnodar.4kolesa.co': '4kolesa.co',
 'serpuhov.4kolesa.co': '4kolesa.co',
 'ivanovo.4kolesa.co': '4kolesa.co',
 'astrahan.4kolesa.co': '4kolesa.co',
 'tambov.4kolesa.co': '4kolesa.co',
 'perm.4kolesa.co': '4kolesa.co',
 'barnaul.4kolesa.co': '4kolesa.co',
 'belgorod.4kolesa.co': '4kolesa.co',
 'volgograd.4kolesa.co': '4kolesa.co',
 'armavir.4kolesa.co': '4kolesa.co',
    'neopod.ru': 'neopod.ru',
 'ekb.neopod.ru': 'neopod.ru',
 'nsk.neopod.ru': 'neopod.ru',
 'izhevsk.neopod.ru': 'neopod.ru',
 'tula.neopod.ru': 'neopod.ru',
 'kirov.neopod.ru': 'neopod.ru',
 'sevastopol.neopod.ru': 'neopod.ru',
 'tyumen.neopod.ru': 'neopod.ru',
 'rostov-na-donu.neopod.ru': 'neopod.ru',
 'krasnogorsk.neopod.ru': 'neopod.ru',
 'omsk.neopod.ru': 'neopod.ru',
 'sankt-peterburg.neopod.ru': 'neopod.ru',
 'saratov.neopod.ru': 'neopod.ru',
 'tolyatti.neopod.ru': 'neopod.ru',
 'cheboksary.neopod.ru': 'neopod.ru',
 'perm.neopod.ru': 'neopod.ru',
 'vladivostok.neopod.ru': 'neopod.ru',
 'yaroslavl.neopod.ru': 'neopod.ru',
 'ufa.neopod.ru': 'neopod.ru',
 'krasnoyarsk.neopod.ru': 'neopod.ru',
 'istra.neopod.ru': 'neopod.ru',
 'ulyanovsk.neopod.ru': 'neopod.ru',
 'abakan.neopod.ru': 'neopod.ru',
 'chelyabinsk.neopod.ru': 'neopod.ru',
 'arhangelsk.neopod.ru': 'neopod.ru',
 'barnaul.neopod.ru': 'neopod.ru',
 'magnitogorsk.neopod.ru': 'neopod.ru',
 'astrahan.neopod.ru': 'neopod.ru',
 'kr.neopod.ru': 'neopod.ru',
 'kazan.neopod.ru': 'neopod.ru',
 'syzran.neopod.ru': 'neopod.ru',
 'kurgan.neopod.ru': 'neopod.ru',
 'shchelkovo.neopod.ru': 'neopod.ru',
 'voronezh.neopod.ru': 'neopod.ru',
 'nn.neopod.ru': 'neopod.ru',
 'kaliningrad.neopod.ru': 'neopod.ru',
 'yuzhno-sahalinsk.neopod.ru': 'neopod.ru',
 'lipeck.neopod.ru': 'neopod.ru',
 'penza.neopod.ru': 'neopod.ru',
 'naro-fominsk.neopod.ru': 'neopod.ru',
 'nyagan.neopod.ru': 'neopod.ru',
 'evpatoriya.neopod.ru': 'neopod.ru',
 'chehov.neopod.ru': 'neopod.ru',
 'gorno-altaysk.neopod.ru': 'neopod.ru',
 'vladimir.neopod.ru': 'neopod.ru',
 'vladikavkaz.neopod.ru': 'neopod.ru',
 'uhta.neopod.ru': 'neopod.ru',
 'saransk.neopod.ru': 'neopod.ru',
 'petrozavodsk.neopod.ru': 'neopod.ru',
 'nalchik.neopod.ru': 'neopod.ru',
 'smolensk.neopod.ru': 'neopod.ru',
 'vidnoe.neopod.ru': 'neopod.ru',
 'bryansk.neopod.ru': 'neopod.ru',
    'narule.ru': 'narule.ru',
 'saratov.narule.ru': 'narule.ru',
 'rnd.narule.ru': 'narule.ru',
 'surgut.narule.ru': 'narule.ru',
 'ufa.narule.ru': 'narule.ru',
 'nn.narule.ru': 'narule.ru',
 'samara.narule.ru': 'narule.ru',
 'ijevsk.narule.ru': 'narule.ru',
 'sochi.narule.ru': 'narule.ru',
 'habarovsk.narule.ru': 'narule.ru',
 'kaliningrad.narule.ru': 'narule.ru',
 'vladimir.narule.ru': 'narule.ru',
 'tumen.narule.ru': 'narule.ru',
 'ekb.narule.ru': 'narule.ru',
 'orenburg.narule.ru': 'narule.ru',
 'krasnoyarsk.narule.ru': 'narule.ru',
 'krasnodar.narule.ru': 'narule.ru',
 'barnaul.narule.ru': 'narule.ru',
 'kaluga.narule.ru': 'narule.ru',
 'orel.narule.ru': 'narule.ru',
 'spb.narule.ru': 'narule.ru',
 'kurgan.narule.ru': 'narule.ru',
 'smolensk.narule.ru': 'narule.ru',
 'kemerovo.narule.ru': 'narule.ru',
 'omsk.narule.ru': 'narule.ru',
 'kazan.narule.ru': 'narule.ru',
 'nsk.narule.ru': 'narule.ru',
 'vladivostok.narule.ru': 'narule.ru',
 'rzn.narule.ru': 'narule.ru',
 'perm.narule.ru': 'narule.ru',
 'tver.narule.ru': 'narule.ru',
 'kirov.narule.ru': 'narule.ru',
 'chelny.narule.ru': 'narule.ru',
 'tlt.narule.ru': 'narule.ru',
 'yaroslavl.narule.ru': 'narule.ru',
 'volgograd.narule.ru': 'narule.ru',
 'tomsk.narule.ru': 'narule.ru',
 'chel.narule.ru': 'narule.ru',
 'ulyanovsk.narule.ru': 'narule.ru',
 'murmansk.narule.ru': 'narule.ru',
 'cheboksary.narule.ru': 'narule.ru',
 'stavropol.narule.ru': 'narule.ru',
 'tambov.narule.ru': 'narule.ru',
 'ptz.narule.ru': 'narule.ru',
 'kursk.narule.ru': 'narule.ru',
 'penza.narule.ru': 'narule.ru',
 'cher.narule.ru': 'narule.ru',
 'tula.narule.ru': 'narule.ru',
 'novokuz.narule.ru': 'narule.ru',
 'voronej.narule.ru': 'narule.ru',
 'lipeck.narule.ru': 'narule.ru',
 'syktyvkar.narule.ru': 'narule.ru',
 'bryansk.narule.ru': 'narule.ru',
    'tolyatti.rasprodaga.ru': 'rasprodaga.ru',
 'kemerovo.rasprodaga.ru': 'rasprodaga.ru',
 'ekaterinburg.rasprodaga.ru': 'rasprodaga.ru',
 'chelyabinsk.rasprodaga.ru': 'rasprodaga.ru',
 'tula.rasprodaga.ru': 'rasprodaga.ru',
 'vladivostok.rasprodaga.ru': 'rasprodaga.ru',
 'krasnoyarsk.rasprodaga.ru': 'rasprodaga.ru',
 'novosibirsk.rasprodaga.ru': 'rasprodaga.ru',
 'kirov.rasprodaga.ru': 'rasprodaga.ru',
 'ufa.rasprodaga.ru': 'rasprodaga.ru',
 'yakutsk.rasprodaga.ru': 'rasprodaga.ru',
 'izhevsk.rasprodaga.ru': 'rasprodaga.ru',
 'barnaul.rasprodaga.ru': 'rasprodaga.ru',
 'blagoveshchensk.rasprodaga.ru': 'rasprodaga.ru',
 'sankt-peterburg.rasprodaga.ru': 'rasprodaga.ru',
 'petrozavodsk.rasprodaga.ru': 'rasprodaga.ru',
 'omsk.rasprodaga.ru': 'rasprodaga.ru',
 'krasnodar.rasprodaga.ru': 'rasprodaga.ru',
 'rostov-na-donu.rasprodaga.ru': 'rasprodaga.ru',
 'petropavlovsk.rasprodaga.ru': 'rasprodaga.ru',
 'cheboksary.rasprodaga.ru': 'rasprodaga.ru',
 'kurgan.rasprodaga.ru': 'rasprodaga.ru',
 'kazan.rasprodaga.ru': 'rasprodaga.ru',
 'penza.rasprodaga.ru': 'rasprodaga.ru',
 'ryazan.rasprodaga.ru': 'rasprodaga.ru',
 'habarovsk.rasprodaga.ru': 'rasprodaga.ru',
 'belgorod.rasprodaga.ru': 'rasprodaga.ru',
 'orenburg.rasprodaga.ru': 'rasprodaga.ru',
 'voronezh.rasprodaga.ru': 'rasprodaga.ru',
 'tyumen.rasprodaga.ru': 'rasprodaga.ru',
 'saratov.rasprodaga.ru': 'rasprodaga.ru',
 'sochi.rasprodaga.ru': 'rasprodaga.ru',
 'nabchelny.rasprodaga.ru': 'rasprodaga.ru',
 'samara.rasprodaga.ru': 'rasprodaga.ru',
 'vladimir.rasprodaga.ru': 'rasprodaga.ru',
 'abakan.rasprodaga.ru': 'rasprodaga.ru',
 'smolensk.rasprodaga.ru': 'rasprodaga.ru',
 'vladikavkaz.rasprodaga.ru': 'rasprodaga.ru',
 'kerch.rasprodaga.ru': 'rasprodaga.ru',
 'yaroslavl.rasprodaga.ru': 'rasprodaga.ru',
 'nizhnij-novgorod.rasprodaga.ru': 'rasprodaga.ru',
 'perm.rasprodaga.ru': 'rasprodaga.ru',
 'magadan.rasprodaga.ru': 'rasprodaga.ru',
 'kaluga.rasprodaga.ru': 'rasprodaga.ru',
 'kursk.rasprodaga.ru': 'rasprodaga.ru',
 'murmansk.rasprodaga.ru': 'rasprodaga.ru',
 'saransk.rasprodaga.ru': 'rasprodaga.ru',
 'volgograd.rasprodaga.ru': 'rasprodaga.ru',
 'uljanovsk.rasprodaga.ru': 'rasprodaga.ru',
 'rasprodaga.ru': 'rasprodaga.ru',
 'kaliningrad.rasprodaga.ru': 'rasprodaga.ru',
 'vologda.rasprodaga.ru': 'rasprodaga.ru',
 'yoshkarola.rasprodaga.ru': 'rasprodaga.ru',
 'orel.rasprodaga.ru': 'rasprodaga.ru',
    'abba1982.tourister.ru': 'tourister.ru',
 'tourister.ru': 'tourister.ru',
 'pogoda.tourister.ru': 'tourister.ru',
 'tinka61.tourister.ru': 'tourister.ru',
 'polyakova-tatiana.tourister.ru': 'tourister.ru',
 'vasilets.tourister.ru': 'tourister.ru',
 'tomi-aleks.tourister.ru': 'tourister.ru',
 'leolik.tourister.ru': 'tourister.ru',
 'nomad.tourister.ru': 'tourister.ru',
 'borracho.tourister.ru': 'tourister.ru',
 'lyudmila-klyopova.tourister.ru': 'tourister.ru',
 'tavasa100.tourister.ru': 'tourister.ru',
 'happiness.tourister.ru': 'tourister.ru',
 'tatiana-dudarenko.tourister.ru': 'tourister.ru',
 'valeriiabel.tourister.ru': 'tourister.ru',
 'kvesisskaya.tourister.ru': 'tourister.ru',
 'anna-08.tourister.ru': 'tourister.ru',
 'marsada.tourister.ru': 'tourister.ru',
 'afrosnegka.tourister.ru': 'tourister.ru',
 'margaritka.tourister.ru': 'tourister.ru',
 'sirina99.tourister.ru': 'tourister.ru',
 'vermut-777.tourister.ru': 'tourister.ru',
 'nataliac.tourister.ru': 'tourister.ru',
 'ilmelnik.tourister.ru': 'tourister.ru',
 'yaroslavglukhov.tourister.ru': 'tourister.ru',
 'ege.tourister.ru': 'tourister.ru',
 'serg555.tourister.ru': 'tourister.ru',
 'ena.tourister.ru': 'tourister.ru',
 'annaabrosimova.tourister.ru': 'tourister.ru',
 'acvazul-v2010.tourister.ru': 'tourister.ru',
 'evgeniyamelone.tourister.ru': 'tourister.ru',
 'taushka.tourister.ru': 'tourister.ru',
 'oleg-sarapulov.tourister.ru': 'tourister.ru',
 'asiagate.tourister.ru': 'tourister.ru',
 'evgeniaczech.tourister.ru': 'tourister.ru',
 'marinalarionova.tourister.ru': 'tourister.ru',
 'kosmja.tourister.ru': 'tourister.ru',
 'coshara.tourister.ru': 'tourister.ru',
 'leozman.tourister.ru': 'tourister.ru',
 'grau59.tourister.ru': 'tourister.ru',
 'sheftali.tourister.ru': 'tourister.ru',
 'bujhm54.tourister.ru': 'tourister.ru',
 'solek73.tourister.ru': 'tourister.ru',
 'sonnenschein.tourister.ru': 'tourister.ru',
 'sheboldasik.tourister.ru': 'tourister.ru',
 'pushvalentina.tourister.ru': 'tourister.ru',
 'vladimir-shalaev.tourister.ru': 'tourister.ru',
 'olegka.tourister.ru': 'tourister.ru',
 'kadet370.tourister.ru': 'tourister.ru',
 'vladkonst.tourister.ru': 'tourister.ru',
 'marina5.tourister.ru': 'tourister.ru',
 'odlesya.tourister.ru': 'tourister.ru',
 'swetlyachok.tourister.ru': 'tourister.ru',
 'karela.tourister.ru': 'tourister.ru',
 'liolill.tourister.ru': 'tourister.ru',
 'fanolia.tourister.ru': 'tourister.ru',
    'kp.ru': 'kp.ru',
 'amp.kp.ru': 'kp.ru',
 'amp.spb.kp.ru': 'kp.ru',
 'amp.kuban.kp.ru': 'kp.ru',
 'amp.nsk.kp.ru': 'kp.ru',
 'nnov.kp.ru': 'kp.ru',
 'amp.nnov.kp.ru': 'kp.ru',
 'dv.kp.ru': 'kp.ru',
 'amp.ural.kp.ru': 'kp.ru',
 'krsk.kp.ru': 'kp.ru',
 'pskov.kp.ru': 'kp.ru',
 'izh.kp.ru': 'kp.ru',
 'pda.chel.kp.ru': 'kp.ru',
 'spb.kp.ru': 'kp.ru',
 'msk.kp.ru': 'kp.ru',
 'pda.spb.kp.ru': 'kp.ru',
 'perm.kp.ru': 'kp.ru',
 'kazan.kp.ru': 'kp.ru',
 'hab.kp.ru': 'kp.ru',
 'alt.kp.ru': 'kp.ru',
 'samara.kp.ru': 'kp.ru',
 'pda.hab.kp.ru': 'kp.ru',
 'kuban.kp.ru': 'kp.ru',
 'kirov.kp.ru': 'kp.ru',
 'nsk.kp.ru': 'kp.ru',
 'volgograd.kp.ru': 'kp.ru',
 'omsk.kp.ru': 'kp.ru',
 'pda.stav.kp.ru': 'kp.ru',
 'mosobl.kp.ru': 'kp.ru',
 'vologda.kp.ru': 'kp.ru',
 'rostov.kp.ru': 'kp.ru',
 'ryazan.kp.ru': 'kp.ru',
 'pda.ufa.kp.ru': 'kp.ru',
 'orel.kp.ru': 'kp.ru',
 'yamal.kp.ru': 'kp.ru',
 'tula.kp.ru': 'kp.ru',
 'komi.kp.ru': 'kp.ru',
 'belarus.kp.ru': 'kp.ru',
 'tver.kp.ru': 'kp.ru',
 'kem.kp.ru': 'kp.ru',
 'penza.kp.ru': 'kp.ru',
 'donetsk.kp.ru': 'kp.ru',
 'bryansk.kp.ru': 'kp.ru',
 'pda.samara.kp.ru': 'kp.ru',
 'pda.bryansk.kp.ru': 'kp.ru',
 'lipetsk.kp.ru': 'kp.ru',
 'tumen.kp.ru': 'kp.ru',
 'pda.kazan.kp.ru': 'kp.ru',
 'smol.kp.ru': 'kp.ru',
 'pda.krsk.kp.ru': 'kp.ru',
 'sakhalin.kp.ru': 'kp.ru',
 'tambov.kp.ru': 'kp.ru',
 'kostroma.kp.ru': 'kp.ru',
 'irk.kp.ru': 'kp.ru',
 'murmansk.kp.ru': 'kp.ru',
 'vladimir.kp.ru': 'kp.ru',
    'izap24.ru': 'izap24.ru',
 'samara.izap24.ru': 'izap24.ru',
 'msk.izap24.ru': 'izap24.ru',
 'tomsk.izap24.ru': 'izap24.ru',
 'balashiha.izap24.ru': 'izap24.ru',
 'perm.izap24.ru': 'izap24.ru',
 'krasnodar.izap24.ru': 'izap24.ru',
 'ufa.izap24.ru': 'izap24.ru',
 'chelyabinsk.izap24.ru': 'izap24.ru',
 'nijniy-novgorod.izap24.ru': 'izap24.ru',
 'kazan.izap24.ru': 'izap24.ru',
 'pushkin.izap24.ru': 'izap24.ru',
 'odintsovo.izap24.ru': 'izap24.ru',
 'voronej.izap24.ru': 'izap24.ru',
 'novosibirsk.izap24.ru': 'izap24.ru',
 'rostov-na-donu.izap24.ru': 'izap24.ru',
 'krasnoyarsk.izap24.ru': 'izap24.ru',
 'nijniy-tagil.izap24.ru': 'izap24.ru',
 'naberejnyie-chelnyi.izap24.ru': 'izap24.ru',
 'orenburg.izap24.ru': 'izap24.ru',
 'spb.izap24.ru': 'izap24.ru',
 'pskov.izap24.ru': 'izap24.ru',
 'sergiev-posad.izap24.ru': 'izap24.ru',
 'vladimir.izap24.ru': 'izap24.ru',
 'tula.izap24.ru': 'izap24.ru',
 'ekb.izap24.ru': 'izap24.ru',
 'surgut.izap24.ru': 'izap24.ru',
 'barnaul.izap24.ru': 'izap24.ru',
 'omsk.izap24.ru': 'izap24.ru',
 'habarovsk.izap24.ru': 'izap24.ru',
 'bryansk.izap24.ru': 'izap24.ru',
 'myitischi.izap24.ru': 'izap24.ru',
 'volgograd.izap24.ru': 'izap24.ru',
 'belgorod.izap24.ru': 'izap24.ru',
 'vologda.izap24.ru': 'izap24.ru',
 'himki.izap24.ru': 'izap24.ru',
 'staryiy-oskol.izap24.ru': 'izap24.ru',
 'kaluga.izap24.ru': 'izap24.ru',
 'simferopol.izap24.ru': 'izap24.ru',
 'ivanovo.izap24.ru': 'izap24.ru',
 'petrozavodsk.izap24.ru': 'izap24.ru',
 'sevastopol.izap24.ru': 'izap24.ru',
 'ramenskoe.izap24.ru': 'izap24.ru',
 'podolsk.izap24.ru': 'izap24.ru',
 'lipetsk.izap24.ru': 'izap24.ru',
 'yaroslavl.izap24.ru': 'izap24.ru',
 'tyumen.izap24.ru': 'izap24.ru',
 'sochi.izap24.ru': 'izap24.ru',
 'tambov.izap24.ru': 'izap24.ru',
 'irkutsk.izap24.ru': 'izap24.ru',
 'penza.izap24.ru': 'izap24.ru',
 'lyubertsyi.izap24.ru': 'izap24.ru',
 'kemerovo.izap24.ru': 'izap24.ru',
 'tolyatti.izap24.ru': 'izap24.ru',
 'kaliningrad.izap24.ru': 'izap24.ru',
 'saratov.izap24.ru': 'izap24.ru',
    'sochi.incred.ru': 'incred.ru',
 'izhevsk.incred.ru': 'incred.ru',
 'tolyatti.incred.ru': 'incred.ru',
 'armawir.incred.ru': 'incred.ru',
 'voronezh.incred.ru': 'incred.ru',
 'kzn.incred.ru': 'incred.ru',
 'tobolsk.incred.ru': 'incred.ru',
 'spb.incred.ru': 'incred.ru',
 'ufa.incred.ru': 'incred.ru',
 'stavropol.incred.ru': 'incred.ru',
 'surgut.incred.ru': 'incred.ru',
 'astrakhan.incred.ru': 'incred.ru',
 'krasnodar.incred.ru': 'incred.ru',
 'ivanovo.incred.ru': 'incred.ru',
 'lipetsk.incred.ru': 'incred.ru',
 'orenburg.incred.ru': 'incred.ru',
 'sterlitamak.incred.ru': 'incred.ru',
 'ulyanovsk.incred.ru': 'incred.ru',
 'samara.incred.ru': 'incred.ru',
 'velikiy-novgorod.incred.ru': 'incred.ru',
 'yakutsk.incred.ru': 'incred.ru',
 'novorossiysk.incred.ru': 'incred.ru',
 'blagoveshensk.incred.ru': 'incred.ru',
 'habarovsk.incred.ru': 'incred.ru',
 'perm.incred.ru': 'incred.ru',
 'incred.ru': 'incred.ru',
 'kirov.incred.ru': 'incred.ru',
 'kostroma.incred.ru': 'incred.ru',
 'novokuznetsk.incred.ru': 'incred.ru',
 'volgograd.incred.ru': 'incred.ru',
 'nsk.incred.ru': 'incred.ru',
 'yuzhnosakhalinsk.incred.ru': 'incred.ru',
 'penza.incred.ru': 'incred.ru',
 'barnaul.incred.ru': 'incred.ru',
 'tyumen.incred.ru': 'incred.ru',
 'yaroslavl.incred.ru': 'incred.ru',
 'ekat.incred.ru': 'incred.ru',
 'chel.incred.ru': 'incred.ru',
 'vladivostok.incred.ru': 'incred.ru',
 'krasnoyarsk.incred.ru': 'incred.ru',
 'vologda.incred.ru': 'incred.ru',
 'nahodka.incred.ru': 'incred.ru',
 'saratov.incred.ru': 'incred.ru',
 'cheboksary.incred.ru': 'incred.ru',
 'kemerovo.incred.ru': 'incred.ru',
 'omsk.incred.ru': 'incred.ru',
 'tula.incred.ru': 'incred.ru',
 'p-kamchatskiy.incred.ru': 'incred.ru',
 'noyabrsk.incred.ru': 'incred.ru',
 'syktyvkar.incred.ru': 'incred.ru',
 'abakan.incred.ru': 'incred.ru',
 'rnd.incred.ru': 'incred.ru',
 'kursk.incred.ru': 'incred.ru',
 'nn.incred.ru': 'incred.ru',
 'tomsk.incred.ru': 'incred.ru',
 'biysk.incred.ru': 'incred.ru',
    'pupils.ru': 'pupils.ru',
 'school17.pupils.ru': 'pupils.ru',
 'school2.pupils.ru': 'pupils.ru',
 'ds93.pupils.ru': 'pupils.ru',
 'school9.pupils.ru': 'pupils.ru',
 'gym1.pupils.ru': 'pupils.ru',
 'lic41.pupils.ru': 'pupils.ru',
 'school8.pupils.ru': 'pupils.ru',
 'school18.pupils.ru': 'pupils.ru',
 'ds28.pupils.ru': 'pupils.ru',
 'school62.pupils.ru': 'pupils.ru',
 'school66.pupils.ru': 'pupils.ru',
 'school33.pupils.ru': 'pupils.ru',
 'ds146.pupils.ru': 'pupils.ru',
 'school64.pupils.ru': 'pupils.ru',
 'school22.pupils.ru': 'pupils.ru',
 'school79.pupils.ru': 'pupils.ru',
 'school56.pupils.ru': 'pupils.ru',
 'school38.pupils.ru': 'pupils.ru',
 'school48.pupils.ru': 'pupils.ru',
 'school16.pupils.ru': 'pupils.ru',
 'school13.pupils.ru': 'pupils.ru',
 'school51.pupils.ru': 'pupils.ru',
 'ds124.pupils.ru': 'pupils.ru',
 'school58.pupils.ru': 'pupils.ru',
 'school61.pupils.ru': 'pupils.ru',
 'school73.pupils.ru': 'pupils.ru',
 'ds41.pupils.ru': 'pupils.ru',
 'ds52.pupils.ru': 'pupils.ru',
 'ds42.pupils.ru': 'pupils.ru',
 'ds44.pupils.ru': 'pupils.ru',
 'ds138.pupils.ru': 'pupils.ru',
 'school10.pupils.ru': 'pupils.ru',
 'ds103.pupils.ru': 'pupils.ru',
 'ds2.pupils.ru': 'pupils.ru',
 'school70.pupils.ru': 'pupils.ru',
 'ds8.pupils.ru': 'pupils.ru',
 'school25.pupils.ru': 'pupils.ru',
 'school77.pupils.ru': 'pupils.ru',
 'tl.pupils.ru': 'pupils.ru',
 'ds121.pupils.ru': 'pupils.ru',
 'ds162.pupils.ru': 'pupils.ru',
 'school80.pupils.ru': 'pupils.ru',
 'ds159.pupils.ru': 'pupils.ru',
 'school83.pupils.ru': 'pupils.ru',
 'ds5.pupils.ru': 'pupils.ru',
 'ds67.pupils.ru': 'pupils.ru',
 'school42.pupils.ru': 'pupils.ru',
 'school82.pupils.ru': 'pupils.ru',
 'ds107.pupils.ru': 'pupils.ru',
 'school14.pupils.ru': 'pupils.ru',
 'ds126.pupils.ru': 'pupils.ru',
 'ds114.pupils.ru': 'pupils.ru',
 'ds122.pupils.ru': 'pupils.ru',
 'ds184.pupils.ru': 'pupils.ru',
 'correct.pupils.ru': 'pupils.ru',
 'school59.pupils.ru': 'pupils.ru',
 'school76.pupils.ru': 'pupils.ru',
    'russia.aport.ru': 'aport.ru',
 'spb.aport.ru': 'aport.ru',
 'aport.ru': 'aport.ru',
 'groznyy.aport.ru': 'aport.ru',
 'mytischi.aport.ru': 'aport.ru',
 'kurgan.aport.ru': 'aport.ru',
 'izhevsk.aport.ru': 'aport.ru',
 'kazan.aport.ru': 'aport.ru',
 'lipetsk.aport.ru': 'aport.ru',
 'arkhangelsk.aport.ru': 'aport.ru',
 'novokuznetsk.aport.ru': 'aport.ru',
 'krasnodar.aport.ru': 'aport.ru',
 'krasnoyarsk.aport.ru': 'aport.ru',
 'bryansk.aport.ru': 'aport.ru',
 'ekaterinburg.aport.ru': 'aport.ru',
 'kursk.aport.ru': 'aport.ru',
 'irkutsk.aport.ru': 'aport.ru',
 'blagoveschensk.aport.ru': 'aport.ru',
 'chelny.aport.ru': 'aport.ru',
 'omsk.aport.ru': 'aport.ru',
 'ivanovo.aport.ru': 'aport.ru',
 'kaluga.aport.ru': 'aport.ru',
 'makhachkala.aport.ru': 'aport.ru',
 'kostroma.aport.ru': 'aport.ru',
 'barnaul.aport.ru': 'aport.ru',
 'murmansk.aport.ru': 'aport.ru',
 'novosibirsk.aport.ru': 'aport.ru',
 'nizhniynovgorod.aport.ru': 'aport.ru',
 'belgorod.aport.ru': 'aport.ru',
 'vladivostok.aport.ru': 'aport.ru',
 'kaliningrad.aport.ru': 'aport.ru',
 'kirov.aport.ru': 'aport.ru',
 'voronezh.aport.ru': 'aport.ru',
 'dzerzhinsk.aport.ru': 'aport.ru',
 'volgograd.aport.ru': 'aport.ru',
 'vladimir.aport.ru': 'aport.ru',
 'penza.aport.ru': 'aport.ru',
 'tver.aport.ru': 'aport.ru',
 'ulyanovsk.aport.ru': 'aport.ru',
 'perm.aport.ru': 'aport.ru',
 'rostov-na-donu.aport.ru': 'aport.ru',
 'ryazan.aport.ru': 'aport.ru',
 'tambov.aport.ru': 'aport.ru',
 'ufa.aport.ru': 'aport.ru',
 'stavropol.aport.ru': 'aport.ru',
 'orenburg.aport.ru': 'aport.ru',
 'yakutsk.aport.ru': 'aport.ru',
 'khabarovsk.aport.ru': 'aport.ru',
 'samara.aport.ru': 'aport.ru',
 'saratov.aport.ru': 'aport.ru',
 'yaroslavl.aport.ru': 'aport.ru',
 'yuzhno-sakhalinsk.aport.ru': 'aport.ru',
 'tyumen.aport.ru': 'aport.ru',
 'tomsk.aport.ru': 'aport.ru',
 'chelyabinsk.aport.ru': 'aport.ru',
 'sochi.aport.ru': 'aport.ru',
 'smolensk.aport.ru': 'aport.ru',
 'cheboksary.aport.ru': 'aport.ru',
 'chita.aport.ru': 'aport.ru',
    'spb.aif.ru': 'aif.ru',
 'murmansk.aif.ru': 'aif.ru',
 'aif.ru': 'aif.ru',
 'tver.aif.ru': 'aif.ru',
 'rostov.aif.ru': 'aif.ru',
 'astrakhan.aif.ru': 'aif.ru',
 'bryansk.aif.ru': 'aif.ru',
 'nn.aif.ru': 'aif.ru',
 'stav.aif.ru': 'aif.ru',
 'samara.aif.ru': 'aif.ru',
 'dag.aif.ru': 'aif.ru',
 'ufa.aif.ru': 'aif.ru',
 'kuban.aif.ru': 'aif.ru',
 'vlad.aif.ru': 'aif.ru',
 'kazan.aif.ru': 'aif.ru',
 'nsk.aif.ru': 'aif.ru',
 'vl.aif.ru': 'aif.ru',
 'kuzbass.aif.ru': 'aif.ru',
 'ural.aif.ru': 'aif.ru',
 'tmn.aif.ru': 'aif.ru',
 'perm.aif.ru': 'aif.ru',
 'omsk.aif.ru': 'aif.ru',
 'kamchatka.aif.ru': 'aif.ru',
 'klg.aif.ru': 'aif.ru',
 'oren.aif.ru': 'aif.ru',
 'vrn.aif.ru': 'aif.ru',
 'irk.aif.ru': 'aif.ru',
 'krym.aif.ru': 'aif.ru',
 'karel.aif.ru': 'aif.ru',
 'pskov.aif.ru': 'aif.ru',
 'chv.aif.ru': 'aif.ru',
 'altai.aif.ru': 'aif.ru',
 'tomsk.aif.ru': 'aif.ru',
 'ugra.aif.ru': 'aif.ru',
 'chel.aif.ru': 'aif.ru',
 'rzn.aif.ru': 'aif.ru',
 'udm.aif.ru': 'aif.ru',
 'adigea.aif.ru': 'aif.ru',
 'kirov.aif.ru': 'aif.ru',
 'bel.aif.ru': 'aif.ru',
 'saratov.aif.ru': 'aif.ru',
 'archive.aif.ru': 'aif.ru',
 'hab.aif.ru': 'aif.ru',
 'chr.aif.ru': 'aif.ru',
 'krsk.aif.ru': 'aif.ru',
 'ul.aif.ru': 'aif.ru',
 'penza.aif.ru': 'aif.ru',
 'ivanovo.aif.ru': 'aif.ru',
 'komi.aif.ru': 'aif.ru',
 'smol.aif.ru': 'aif.ru',
 'kaluga.aif.ru': 'aif.ru',
 'yar.aif.ru': 'aif.ru',
 'bur.aif.ru': 'aif.ru',
 'vlg.aif.ru': 'aif.ru',
 'mirtesen.aif.ru': 'aif.ru',
 'yakutia.aif.ru': 'aif.ru',
 'tula.aif.ru': 'aif.ru',
 'yamal.aif.ru': 'aif.ru',
 'chita.aif.ru': 'aif.ru',
 'arh.aif.ru': 'aif.ru',
 'sakhalin.aif.ru': 'aif.ru',
    'kostroma.obam.ru': 'obam.ru',
 'voronezh.obam.ru': 'obam.ru',
 'magadan.obam.ru': 'obam.ru',
 'tolyatti.obam.ru': 'obam.ru',
 'velikie-luki.obam.ru': 'obam.ru',
 'ekaterinburg.obam.ru': 'obam.ru',
 'sochi.obam.ru': 'obam.ru',
 'stavropol.obam.ru': 'obam.ru',
 'obam.ru': 'obam.ru',
 'kirovskaya-oblast-kirov.obam.ru': 'obam.ru',
 'novosibirsk.obam.ru': 'obam.ru',
 'orenburg.obam.ru': 'obam.ru',
 'lipetsk.obam.ru': 'obam.ru',
 'petrozavodsk.obam.ru': 'obam.ru',
 'saratov.obam.ru': 'obam.ru',
 'vladimir.obam.ru': 'obam.ru',
 'amurskaya-oblast-blagoveschensk.obam.ru': 'obam.ru',
 'barnaul.obam.ru': 'obam.ru',
 'saransk.obam.ru': 'obam.ru',
 'volgograd.obam.ru': 'obam.ru',
 'vladivostok.obam.ru': 'obam.ru',
 'abakan.obam.ru': 'obam.ru',
 'neryungri.obam.ru': 'obam.ru',
 'rostov-na-donu.obam.ru': 'obam.ru',
 'cherkessk.obam.ru': 'obam.ru',
 'krasnoyarsk.obam.ru': 'obam.ru',
 'pskov.obam.ru': 'obam.ru',
 'ivanovo.obam.ru': 'obam.ru',
 'maykop.obam.ru': 'obam.ru',
 'kaliningrad.obam.ru': 'obam.ru',
 'surgut.obam.ru': 'obam.ru',
 'belgorod.obam.ru': 'obam.ru',
 'krasnodar.obam.ru': 'obam.ru',
 'arhangelsk.obam.ru': 'obam.ru',
 'irkutsk.obam.ru': 'obam.ru',
 'kurgan.obam.ru': 'obam.ru',
 'kaluga.obam.ru': 'obam.ru',
 'kursk.obam.ru': 'obam.ru',
 'velikiy-novgorod.obam.ru': 'obam.ru',
 'birobidzhan.obam.ru': 'obam.ru',
 'vladikavkaz.obam.ru': 'obam.ru',
 'komsomolsk-na-amure.obam.ru': 'obam.ru',
 'nalchik.obam.ru': 'obam.ru',
 'vologda.obam.ru': 'obam.ru',
 'omsk.obam.ru': 'obam.ru',
 'moskva.obam.ru': 'obam.ru',
 'yoshkar-ola.obam.ru': 'obam.ru',
 'novyy-urengoy.obam.ru': 'obam.ru',
 'novokuznetsk.obam.ru': 'obam.ru',
 'bryansk.obam.ru': 'obam.ru',
 'nizhnevartovsk.obam.ru': 'obam.ru',
 'groznyy.obam.ru': 'obam.ru',
 'mahachkala.obam.ru': 'obam.ru',
 'gorno-altaysk.obam.ru': 'obam.ru',
 'orel.obam.ru': 'obam.ru',
 'kyzyl.obam.ru': 'obam.ru',
 'nizhniy-novgorod.obam.ru': 'obam.ru',
 'nazran.obam.ru': 'obam.ru',
 'kazan.obam.ru': 'obam.ru',
 'kemerovo.obam.ru': 'obam.ru',
 'naberezhnye-chelny.obam.ru': 'obam.ru',
    'xn----8sbeyxgbych3e.ru-an.info': 'ru-an.info',
 'xn--h1akeme.ru-an.info': 'ru-an.info',
 'xn--b1amnebsh.ru-an.info': 'ru-an.info',
 'xn----8sbeybxdibygm.ru-an.info': 'ru-an.info',
 'xn----8sbeboqbpxkb0bn.ru-an.info': 'ru-an.info',
 'xn--80adbj3av3e.ru-an.info': 'ru-an.info',
 'xn----ctbsbazhbctieai.ru-an.info': 'ru-an.info',
 'xn----8sbacdn0a2befbd9ahah.ru-an.info': 'ru-an.info',
 'xn----8sbeylmcibxhm5j.ru-an.info': 'ru-an.info',
 'xn----ctbhoxecbwfl.ru-an.info': 'ru-an.info',
 'xn----ctbsbaa3aibxhck.ru-an.info': 'ru-an.info',
 'xn----8sbeyckrgcbzinq9j.ru-an.info': 'ru-an.info',
 'ru-an.info': 'ru-an.info',
 'xn----ctbsngecbwfl.ru-an.info': 'ru-an.info',
 'xn--80aesfpfapfkv.ru-an.info': 'ru-an.info',
 'xn----ctbbwlldibd3aei7k.ru-an.info': 'ru-an.info',
 'xn----8sbefjqd8ajbqsng.ru-an.info': 'ru-an.info',
 'xn--80ab2ano2b.ru-an.info': 'ru-an.info',
 'xn--80aaf3ajie6ih.ru-an.info': 'ru-an.info',
 'xn----8sbebpgk2befbda1akq.ru-an.info': 'ru-an.info',
 'xn--e1aanfaong5a.ru-an.info': 'ru-an.info',
 'xn----8sbccrpewmkb4aghhx.ru-an.info': 'ru-an.info',
 'xn--e1aaegfefr0b.ru-an.info': 'ru-an.info',
 'xn----8sbeb2a2ahbcaujejf.ru-an.info': 'ru-an.info',
 'xn----ctbbgfqe2bhcb0bn.ru-an.info': 'ru-an.info',
 'xn--d1ahjkjn6b8a.ru-an.info': 'ru-an.info',
 'xn----ctbbwca3adfbdba1aiiai.ru-an.info': 'ru-an.info',
 'xn----btbbyxgbkpci.ru-an.info': 'ru-an.info',
 'xn--80akncbpi8a9b.ru-an.info': 'ru-an.info',
 'xn--b1axfeif.ru-an.info': 'ru-an.info',
 'xn--80aagl7axm6c.ru-an.info': 'ru-an.info',
 'xn----ctbhem2ahbc1afag7k.ru-an.info': 'ru-an.info',
 'xn--b1afic0afhkbk7dze.ru-an.info': 'ru-an.info',
 'xn----8sbccb8acbswdibdb9ahgei.ru-an.info': 'ru-an.info',
 'xn----8sbaagifkw2cidb6bgjh.ru-an.info': 'ru-an.info',
 'xn--b1adnecbybbso4b.ru-an.info': 'ru-an.info',
 'xn----8sbejfprsefbd9ao0e.ru-an.info': 'ru-an.info',
 'xn----8sbaaadeugngjt1cmdb5bnu.ru-an.info': 'ru-an.info',
 'xn----8sbeomda3aegeb3akp.ru-an.info': 'ru-an.info',
 'xn----8sbafg2arfogeb3bgit.ru-an.info': 'ru-an.info',
 'xn--80aaglqim9a4d.ru-an.info': 'ru-an.info',
 'xn----8sbaagd2b0bjbewjcmg.ru-an.info': 'ru-an.info',
 'xn----8sbafc7aqtjbewjo1cxh.ru-an.info': 'ru-an.info',
 'xn----8sbeycseejbflljpw1s.ru-an.info': 'ru-an.info',
 'xn----8sbaaah1bft0aidb6bgjh6c.ru-an.info': 'ru-an.info',
 'коронавирус.ru-an.info': 'ru-an.info',
 'xn----8sbaagoomeav2akdb2cijch.ru-an.info': 'ru-an.info',
 'xn----8sbadcjlbxet5aifb2bdhtf0a.ru-an.info': 'ru-an.info',
 'xn--80ayjbj.ru-an.info': 'ru-an.info',
 'xn--80aafl3ae0gxa.ru-an.info': 'ru-an.info',
 'xn----ctbkkcxcibmmhn.ru-an.info': 'ru-an.info',
 'xn----8sbafbgaj4a6armbdc7ais.ru-an.info': 'ru-an.info',
 'xn----8sbeboqzjjbfamvcdkh5w.ru-an.info': 'ru-an.info',
 'xn----8sbesde7aibeakohp3o0b.ru-an.info': 'ru-an.info',
 'xn----8sbacdumget0ajdb9bhjim.ru-an.info': 'ru-an.info',
 'новости-россии.ru-an.info': 'ru-an.info',
 'xn-----8kcfbgb9abt3ambfbb1agiu.ru-an.info': 'ru-an.info',
 'xn----8sbaf4aujib4ale4o.ru-an.info': 'ru-an.info',
 'xn--e1akicadbj7e.ru-an.info': 'ru-an.info',
 'xn----8sbaf4anbsfebfxdhfdi9w.ru-an.info': 'ru-an.info',
 'xn----8sbccqqdqgpheb6bhi1g6g.ru-an.info': 'ru-an.info',
 'xn----ctbssfbvilk1f.ru-an.info': 'ru-an.info',
    'lit-info.ru': 'lit-info.ru',
 'sholohov.lit-info.ru': 'lit-info.ru',
 'prishvin.lit-info.ru': 'lit-info.ru',
 'tsvetaeva.lit-info.ru': 'lit-info.ru',
 'bulgakov.lit-info.ru': 'lit-info.ru',
 'trediakovskiy.lit-info.ru': 'lit-info.ru',
 'tutchev.lit-info.ru': 'lit-info.ru',
 'zhukovskiy.lit-info.ru': 'lit-info.ru',
 'ostrovskiy.lit-info.ru': 'lit-info.ru',
 'radischev.lit-info.ru': 'lit-info.ru',
 'charskaya.lit-info.ru': 'lit-info.ru',
 'bianki.lit-info.ru': 'lit-info.ru',
 'blok.lit-info.ru': 'lit-info.ru',
 'hodasevich.lit-info.ru': 'lit-info.ru',
 'cherny-sasha.lit-info.ru': 'lit-info.ru',
 'solovyev.lit-info.ru': 'lit-info.ru',
 'tatischev.lit-info.ru': 'lit-info.ru',
 'leskov.lit-info.ru': 'lit-info.ru',
 'griboedov.lit-info.ru': 'lit-info.ru',
 'mamin-sibiryak.lit-info.ru': 'lit-info.ru',
 'karamzin.lit-info.ru': 'lit-info.ru',
 'mandelshtam.lit-info.ru': 'lit-info.ru',
 'saltykov-schedrin.lit-info.ru': 'lit-info.ru',
 'maykov.lit-info.ru': 'lit-info.ru',
 'averchenko.lit-info.ru': 'lit-info.ru',
 'sumarokov.lit-info.ru': 'lit-info.ru',
 'delvig.lit-info.ru': 'lit-info.ru',
 'plescheev.lit-info.ru': 'lit-info.ru',
 'baratynskiy.lit-info.ru': 'lit-info.ru',
 'lomonosov.lit-info.ru': 'lit-info.ru',
 'gnedich.lit-info.ru': 'lit-info.ru',
 'derzhavin.lit-info.ru': 'lit-info.ru',
 'nikitin.lit-info.ru': 'lit-info.ru',
 'zoschenko.lit-info.ru': 'lit-info.ru',
 'severyanin.lit-info.ru': 'lit-info.ru',
 'bryusov.lit-info.ru': 'lit-info.ru',
 'lvov.lit-info.ru': 'lit-info.ru',
 'tolstoy-a-k.lit-info.ru': 'lit-info.ru',
 'chukovskiy.lit-info.ru': 'lit-info.ru',
 'andreev.lit-info.ru': 'lit-info.ru',
 'korolenko.lit-info.ru': 'lit-info.ru',
 'zamyatin.lit-info.ru': 'lit-info.ru',
 'aksakov-k-s.lit-info.ru': 'lit-info.ru',
 'sladkov.lit-info.ru': 'lit-info.ru',
 'krylov.lit-info.ru': 'lit-info.ru',
 'odoevskiy.lit-info.ru': 'lit-info.ru',
 'ivanov.lit-info.ru': 'lit-info.ru',
 'veresaev.lit-info.ru': 'lit-info.ru',
 'merezhkovskiy.lit-info.ru': 'lit-info.ru',
 'annenskiy.lit-info.ru': 'lit-info.ru',
 'larri.lit-info.ru': 'lit-info.ru',
 'charushin.lit-info.ru': 'lit-info.ru',
 'grin.lit-info.ru': 'lit-info.ru',
 'zhitkov.lit-info.ru': 'lit-info.ru',
 'stanyukovich.lit-info.ru': 'lit-info.ru',
 'yazykov.lit-info.ru': 'lit-info.ru',
 'goncharov.lit-info.ru': 'lit-info.ru',
 'chernyshevskiy.lit-info.ru': 'lit-info.ru',
 'vyazemskiy.lit-info.ru': 'lit-info.ru',
 'balmont.lit-info.ru': 'lit-info.ru',
 'klyuev.lit-info.ru': 'lit-info.ru',
 'dobrolyubov.lit-info.ru': 'lit-info.ru',
    'sochi-schools.ru': 'sochi-schools.ru',
 'nosh80.sochi-schools.ru': 'sochi-schools.ru',
 'sport2.sochi-schools.ru': 'sochi-schools.ru',
 '65.sochi-schools.ru': 'sochi-schools.ru',
 'sport1.sochi-schools.ru': 'sochi-schools.ru',
 '57.sochi-schools.ru': 'sochi-schools.ru',
 'v01.sochi-schools.ru': 'sochi-schools.ru',
 '25.sochi-schools.ru': 'sochi-schools.ru',
 'sib.sochi-schools.ru': 'sochi-schools.ru',
 '12.sochi-schools.ru': 'sochi-schools.ru',
 '53.sochi-schools.ru': 'sochi-schools.ru',
 'd140.sochi-schools.ru': 'sochi-schools.ru',
 'gym6.sochi-schools.ru': 'sochi-schools.ru',
 'cvr.sochi-schools.ru': 'sochi-schools.ru',
 'school14.sochi-schools.ru': 'sochi-schools.ru',
 'gymn1.sochi-schools.ru': 'sochi-schools.ru',
 'dou35.sochi-schools.ru': 'sochi-schools.ru',
 'dou76.sochi-schools.ru': 'sochi-schools.ru',
 '82.sochi-schools.ru': 'sochi-schools.ru',
 'gs76.sochi-schools.ru': 'sochi-schools.ru',
 '7.sochi-schools.ru': 'sochi-schools.ru',
 'dou28.sochi-schools.ru': 'sochi-schools.ru',
 'gymn9.sochi-schools.ru': 'sochi-schools.ru',
 'dou9.sochi-schools.ru': 'sochi-schools.ru',
 'd125.sochi-schools.ru': 'sochi-schools.ru',
 'sport4.sochi-schools.ru': 'sochi-schools.ru',
 '2.sochi-schools.ru': 'sochi-schools.ru',
 '99.sochi-schools.ru': 'sochi-schools.ru',
 'nosh85.sochi-schools.ru': 'sochi-schools.ru',
 'int2.sochi-schools.ru': 'sochi-schools.ru',
 'dou126.sochi-schools.ru': 'sochi-schools.ru',
 'ebc.sochi-schools.ru': 'sochi-schools.ru',
 'dou7.sochi-schools.ru': 'sochi-schools.ru',
 '26.sochi-schools.ru': 'sochi-schools.ru',
 'dou67.sochi-schools.ru': 'sochi-schools.ru',
 '100.sochi-schools.ru': 'sochi-schools.ru',
 '66.sochi-schools.ru': 'sochi-schools.ru',
 '48.sochi-schools.ru': 'sochi-schools.ru',
 'dou117.sochi-schools.ru': 'sochi-schools.ru',
 '11.sochi-schools.ru': 'sochi-schools.ru',
 'dou122.sochi-schools.ru': 'sochi-schools.ru',
 '38.sochi-schools.ru': 'sochi-schools.ru',
 'dou4.sochi-schools.ru': 'sochi-schools.ru',
 'gym16.sochi-schools.ru': 'sochi-schools.ru',
 '56.sochi-schools.ru': 'sochi-schools.ru',
 '10.sochi-schools.ru': 'sochi-schools.ru',
 'dou111.sochi-schools.ru': 'sochi-schools.ru',
 'dou56.sochi-schools.ru': 'sochi-schools.ru',
 'gym5.sochi-schools.ru': 'sochi-schools.ru',
 'dou86.sochi-schools.ru': 'sochi-schools.ru',
 'dou93.sochi-schools.ru': 'sochi-schools.ru',
 '85.sochi-schools.ru': 'sochi-schools.ru',
 '55.sochi-schools.ru': 'sochi-schools.ru',
 '90.sochi-schools.ru': 'sochi-schools.ru',
 'dou52.sochi-schools.ru': 'sochi-schools.ru',
 'dou79.sochi-schools.ru': 'sochi-schools.ru',
 'dou40.sochi-schools.ru': 'sochi-schools.ru',
 'dou166.sochi-schools.ru': 'sochi-schools.ru',
 'dou104.sochi-schools.ru': 'sochi-schools.ru',
 'd087.sochi-schools.ru': 'sochi-schools.ru',
 'cdoadler.sochi-schools.ru': 'sochi-schools.ru',
 'prof.sochi-schools.ru': 'sochi-schools.ru',
    'besplatnee.net': 'besplatnee.net',
 'petropavlovsk-kamchatskiy.besplatnee.net': 'besplatnee.net',
 'michurinsk.besplatnee.net': 'besplatnee.net',
 'volgodonsk.besplatnee.net': 'besplatnee.net',
 'millerovo.besplatnee.net': 'besplatnee.net',
 'smolensk.besplatnee.net': 'besplatnee.net',
 'rostov-na-donu.besplatnee.net': 'besplatnee.net',
 'kovrov.besplatnee.net': 'besplatnee.net',
 'ufa.besplatnee.net': 'besplatnee.net',
 'murom.besplatnee.net': 'besplatnee.net',
 'orel.besplatnee.net': 'besplatnee.net',
 'moskva.besplatnee.net': 'besplatnee.net',
 'kazan.besplatnee.net': 'besplatnee.net',
 'novokuznetsk.besplatnee.net': 'besplatnee.net',
 'gelendzhik.besplatnee.net': 'besplatnee.net',
 'vladimir.besplatnee.net': 'besplatnee.net',
 'vladivostok.besplatnee.net': 'besplatnee.net',
 'barnaul.besplatnee.net': 'besplatnee.net',
 'maykop.besplatnee.net': 'besplatnee.net',
 'kurgan.besplatnee.net': 'besplatnee.net',
 'tolyatti.besplatnee.net': 'besplatnee.net',
 'cheboksary.besplatnee.net': 'besplatnee.net',
 'obninsk.besplatnee.net': 'besplatnee.net',
 'arzamas.besplatnee.net': 'besplatnee.net',
 'nalchik.besplatnee.net': 'besplatnee.net',
 'syzran.besplatnee.net': 'besplatnee.net',
 'yaroslavl.besplatnee.net': 'besplatnee.net',
 'bryansk.besplatnee.net': 'besplatnee.net',
 'syktyvkar.besplatnee.net': 'besplatnee.net',
 'odintsovo.besplatnee.net': 'besplatnee.net',
 'blagoveshensk.besplatnee.net': 'besplatnee.net',
 'tula.besplatnee.net': 'besplatnee.net',
 'omsk.besplatnee.net': 'besplatnee.net',
 'orenburg.besplatnee.net': 'besplatnee.net',
 'elets.besplatnee.net': 'besplatnee.net',
 'nefteyugansk.besplatnee.net': 'besplatnee.net',
 'achinsk.besplatnee.net': 'besplatnee.net',
 'orsk.besplatnee.net': 'besplatnee.net',
 'ryazan.besplatnee.net': 'besplatnee.net',
 'anapa.besplatnee.net': 'besplatnee.net',
 'rybinsk.besplatnee.net': 'besplatnee.net',
 'ekaterinburg.besplatnee.net': 'besplatnee.net',
 'arhangelsk.besplatnee.net': 'besplatnee.net',
 'tomsk.besplatnee.net': 'besplatnee.net',
 'kemerovo.besplatnee.net': 'besplatnee.net',
 'komsomolsk-na-amure.besplatnee.net': 'besplatnee.net',
 'krasnoyarsk.besplatnee.net': 'besplatnee.net',
 'biysk.besplatnee.net': 'besplatnee.net',
 'samara.besplatnee.net': 'besplatnee.net',
 'khabarovsk.besplatnee.net': 'besplatnee.net',
 'novomoskovsk.besplatnee.net': 'besplatnee.net',
 'yuzhno-sakhalinsk.besplatnee.net': 'besplatnee.net',
 'cherkessk.besplatnee.net': 'besplatnee.net',
 'saraktash.besplatnee.net': 'besplatnee.net',
 'engels.besplatnee.net': 'besplatnee.net',
 'mozhga.besplatnee.net': 'besplatnee.net',
 'kursk.besplatnee.net': 'besplatnee.net',
 'abakan.besplatnee.net': 'besplatnee.net',
 'magadan.besplatnee.net': 'besplatnee.net',
 'lesosibirsk.besplatnee.net': 'besplatnee.net',
 'chapaevsk.besplatnee.net': 'besplatnee.net',
 'cherepovets.besplatnee.net': 'besplatnee.net',
    'podolsk.firmsbase.ru': 'firmsbase.ru',
 'tomsk.firmsbase.ru': 'firmsbase.ru',
 'moskva.firmsbase.ru': 'firmsbase.ru',
 'sochi.firmsbase.ru': 'firmsbase.ru',
 'syktyvkar.firmsbase.ru': 'firmsbase.ru',
 'sankt-peterburg.firmsbase.ru': 'firmsbase.ru',
 'tula.firmsbase.ru': 'firmsbase.ru',
 'petrozavodsk.firmsbase.ru': 'firmsbase.ru',
 'tambov.firmsbase.ru': 'firmsbase.ru',
 'prokopevsk.firmsbase.ru': 'firmsbase.ru',
 'firmsbase.ru': 'firmsbase.ru',
 'orenburg.firmsbase.ru': 'firmsbase.ru',
 'krasnodar.firmsbase.ru': 'firmsbase.ru',
 'cheljabinsk.firmsbase.ru': 'firmsbase.ru',
 'nizhnekamsk.firmsbase.ru': 'firmsbase.ru',
 'orel.firmsbase.ru': 'firmsbase.ru',
 'jaroslavl.firmsbase.ru': 'firmsbase.ru',
 'rjazan.firmsbase.ru': 'firmsbase.ru',
 'svetlogorsk.firmsbase.ru': 'firmsbase.ru',
 'tjumen.firmsbase.ru': 'firmsbase.ru',
 'novosibirsk.firmsbase.ru': 'firmsbase.ru',
 'krasnojarsk.firmsbase.ru': 'firmsbase.ru',
 'omsk.firmsbase.ru': 'firmsbase.ru',
 'amursk.firmsbase.ru': 'firmsbase.ru',
 'perm.firmsbase.ru': 'firmsbase.ru',
 'magnitogorsk.firmsbase.ru': 'firmsbase.ru',
 'saransk.firmsbase.ru': 'firmsbase.ru',
 'novokuzneck.firmsbase.ru': 'firmsbase.ru',
 'pervouralsk.firmsbase.ru': 'firmsbase.ru',
 'kazan.firmsbase.ru': 'firmsbase.ru',
 'iskitim.firmsbase.ru': 'firmsbase.ru',
 'ekaterinburg.firmsbase.ru': 'firmsbase.ru',
 'belgorod.firmsbase.ru': 'firmsbase.ru',
 'pjatigorsk-kmv.firmsbase.ru': 'firmsbase.ru',
 'naberezhnye-chelny.firmsbase.ru': 'firmsbase.ru',
 'nizhnij-novgorod.firmsbase.ru': 'firmsbase.ru',
 'nahodka.firmsbase.ru': 'firmsbase.ru',
 'izhevsk.firmsbase.ru': 'firmsbase.ru',
 'kaluga.firmsbase.ru': 'firmsbase.ru',
 'nizhnij-tagil.firmsbase.ru': 'firmsbase.ru',
 'korolev.firmsbase.ru': 'firmsbase.ru',
 'samara.firmsbase.ru': 'firmsbase.ru',
 'jakutsk.firmsbase.ru': 'firmsbase.ru',
 'rostov-na-donu.firmsbase.ru': 'firmsbase.ru',
 'armavir.firmsbase.ru': 'firmsbase.ru',
 'novorossijsk.firmsbase.ru': 'firmsbase.ru',
 'tobolsk.firmsbase.ru': 'firmsbase.ru',
 'habarovsk.firmsbase.ru': 'firmsbase.ru',
 'sterlitamak.firmsbase.ru': 'firmsbase.ru',
 'kaliningrad.firmsbase.ru': 'firmsbase.ru',
 'penza.firmsbase.ru': 'firmsbase.ru',
 'petropavlovsk-kamchatskij.firmsbase.ru': 'firmsbase.ru',
 'uljanovsk.firmsbase.ru': 'firmsbase.ru',
 'saratov.firmsbase.ru': 'firmsbase.ru',
 'kolomna.firmsbase.ru': 'firmsbase.ru',
 'tver.firmsbase.ru': 'firmsbase.ru',
 'nefteyugansk.firmsbase.ru': 'firmsbase.ru',
 'mytishchi.firmsbase.ru': 'firmsbase.ru',
 'klimovsk-podolsk-gorodskoy-okrug.firmsbase.ru': 'firmsbase.ru',
 'dzerzhinsk.firmsbase.ru': 'firmsbase.ru',
 'ussurijsk.firmsbase.ru': 'firmsbase.ru',
 'kirov.firmsbase.ru': 'firmsbase.ru',
 'stavropol.firmsbase.ru': 'firmsbase.ru',
    'novgorod.berfi.ru': 'berfi.ru',
 'berfi.ru': 'berfi.ru',
 'tomsk.berfi.ru': 'berfi.ru',
 'petrozavodsk.berfi.ru': 'berfi.ru',
 'spb.berfi.ru': 'berfi.ru',
 'belgorod.berfi.ru': 'berfi.ru',
 'novosibirsk.berfi.ru': 'berfi.ru',
 'ryazan.berfi.ru': 'berfi.ru',
 'omsk.berfi.ru': 'berfi.ru',
 'nn.berfi.ru': 'berfi.ru',
 'ola.berfi.ru': 'berfi.ru',
 'penza.berfi.ru': 'berfi.ru',
 'perm.berfi.ru': 'berfi.ru',
 'vologda.berfi.ru': 'berfi.ru',
 'blagoveshensk.berfi.ru': 'berfi.ru',
 'kaliningrad.berfi.ru': 'berfi.ru',
 'krasnodar.berfi.ru': 'berfi.ru',
 'kurgan.berfi.ru': 'berfi.ru',
 'saratov.berfi.ru': 'berfi.ru',
 'tambov.berfi.ru': 'berfi.ru',
 'barnaul.berfi.ru': 'berfi.ru',
 'tagil.berfi.ru': 'berfi.ru',
 'tyumen.berfi.ru': 'berfi.ru',
 'samara.berfi.ru': 'berfi.ru',
 'bratsk.berfi.ru': 'berfi.ru',
 'rostov.berfi.ru': 'berfi.ru',
 'vladikavkaz.berfi.ru': 'berfi.ru',
 'habarovsk.berfi.ru': 'berfi.ru',
 'anapa.berfi.ru': 'berfi.ru',
 'kemerovo.berfi.ru': 'berfi.ru',
 'krasnoyarsk.berfi.ru': 'berfi.ru',
 'ekb.berfi.ru': 'berfi.ru',
 'voronej.berfi.ru': 'berfi.ru',
 'ufa.berfi.ru': 'berfi.ru',
 'volgograd.berfi.ru': 'berfi.ru',
 'izhevsk.berfi.ru': 'berfi.ru',
 'arzamas.berfi.ru': 'berfi.ru',
 'aleksandrov.berfi.ru': 'berfi.ru',
 'chelyabinsk.berfi.ru': 'berfi.ru',
 'lubercy.berfi.ru': 'berfi.ru',
 'kazan.berfi.ru': 'berfi.ru',
 'cherepovec.berfi.ru': 'berfi.ru',
 'tolyatti.berfi.ru': 'berfi.ru',
 'cheboksary.berfi.ru': 'berfi.ru',
 'orenburg.berfi.ru': 'berfi.ru',
 'armavir.berfi.ru': 'berfi.ru',
 'syktivkar.berfi.ru': 'berfi.ru',
 'vladimir.berfi.ru': 'berfi.ru',
 'berezniki.berfi.ru': 'berfi.ru',
 'yaroslavl.berfi.ru': 'berfi.ru',
 'sochi.berfi.ru': 'berfi.ru',
 'mahachkala.berfi.ru': 'berfi.ru',
 'sterlitamak.berfi.ru': 'berfi.ru',
 'oskol.berfi.ru': 'berfi.ru',
 'balashiha.berfi.ru': 'berfi.ru',
 'chelny.berfi.ru': 'berfi.ru',
 'mytishi.berfi.ru': 'berfi.ru',
 'abakan.berfi.ru': 'berfi.ru',
 'vladivostok.berfi.ru': 'berfi.ru',
 'kirov.berfi.ru': 'berfi.ru',
 'lipeck.berfi.ru': 'berfi.ru',
 'ivanovo.berfi.ru': 'berfi.ru',
 'orel.berfi.ru': 'berfi.ru',
    'kurgan.guidetorussia.ru': 'guidetorussia.ru',
 'guidetorussia.ru': 'guidetorussia.ru',
 'barnaul.guidetorussia.ru': 'guidetorussia.ru',
 'himki.guidetorussia.ru': 'guidetorussia.ru',
 'salavat.guidetorussia.ru': 'guidetorussia.ru',
 'anapa.guidetorussia.ru': 'guidetorussia.ru',
 'prokopevsk.guidetorussia.ru': 'guidetorussia.ru',
 'tver.guidetorussia.ru': 'guidetorussia.ru',
 'moskva.guidetorussia.ru': 'guidetorussia.ru',
 'stavropol.guidetorussia.ru': 'guidetorussia.ru',
 'sankt-peterburg.guidetorussia.ru': 'guidetorussia.ru',
 'novoaltajsk.guidetorussia.ru': 'guidetorussia.ru',
 'samara.guidetorussia.ru': 'guidetorussia.ru',
 'armavir.guidetorussia.ru': 'guidetorussia.ru',
 'volgograd.guidetorussia.ru': 'guidetorussia.ru',
 'vladivostok.guidetorussia.ru': 'guidetorussia.ru',
 'cheboksary.guidetorussia.ru': 'guidetorussia.ru',
 'domodedovo.guidetorussia.ru': 'guidetorussia.ru',
 'krasnodar.guidetorussia.ru': 'guidetorussia.ru',
 'omsk.guidetorussia.ru': 'guidetorussia.ru',
 'engels.guidetorussia.ru': 'guidetorussia.ru',
 'sterlitamak.guidetorussia.ru': 'guidetorussia.ru',
 'kaliningrad.guidetorussia.ru': 'guidetorussia.ru',
 'tobolsk.guidetorussia.ru': 'guidetorussia.ru',
 'nefteyugansk.guidetorussia.ru': 'guidetorussia.ru',
 'pyatigorsk.guidetorussia.ru': 'guidetorussia.ru',
 'nizhnekamsk.guidetorussia.ru': 'guidetorussia.ru',
 'ufa.guidetorussia.ru': 'guidetorussia.ru',
 'komsomolsk-na-amure.guidetorussia.ru': 'guidetorussia.ru',
 'kirov.guidetorussia.ru': 'guidetorussia.ru',
 'saratov.guidetorussia.ru': 'guidetorussia.ru',
 'rostov-na-donu.guidetorussia.ru': 'guidetorussia.ru',
 'murmansk.guidetorussia.ru': 'guidetorussia.ru',
 'yaroslavl.guidetorussia.ru': 'guidetorussia.ru',
 'korolyov.guidetorussia.ru': 'guidetorussia.ru',
 'bijsk.guidetorussia.ru': 'guidetorussia.ru',
 'noyabrsk.guidetorussia.ru': 'guidetorussia.ru',
 'arhangelsk.guidetorussia.ru': 'guidetorussia.ru',
 'izhevsk.guidetorussia.ru': 'guidetorussia.ru',
 'tyumen.guidetorussia.ru': 'guidetorussia.ru',
 'lipeck.guidetorussia.ru': 'guidetorussia.ru',
 'norilsk.guidetorussia.ru': 'guidetorussia.ru',
 'vladimir.guidetorussia.ru': 'guidetorussia.ru',
 'tomsk.guidetorussia.ru': 'guidetorussia.ru',
 'ramenskoe.guidetorussia.ru': 'guidetorussia.ru',
 'novosibirsk.guidetorussia.ru': 'guidetorussia.ru',
 'kazan.guidetorussia.ru': 'guidetorussia.ru',
 'novorossijsk.guidetorussia.ru': 'guidetorussia.ru',
 'nizhnij-novgorod.guidetorussia.ru': 'guidetorussia.ru',
 'pskov.guidetorussia.ru': 'guidetorussia.ru',
 'belovo.guidetorussia.ru': 'guidetorussia.ru',
 'dzerzhinsk.guidetorussia.ru': 'guidetorussia.ru',
 'penza.guidetorussia.ru': 'guidetorussia.ru',
 'naberezhnye-chelny.guidetorussia.ru': 'guidetorussia.ru',
 'ussurijsk.guidetorussia.ru': 'guidetorussia.ru',
 'krasnoyarsk.guidetorussia.ru': 'guidetorussia.ru',
 'yuzhno-sahalinsk.guidetorussia.ru': 'guidetorussia.ru',
 'gelendzhik.guidetorussia.ru': 'guidetorussia.ru',
 'podolsk.guidetorussia.ru': 'guidetorussia.ru',
 'voronezh.guidetorussia.ru': 'guidetorussia.ru',
 'sochi.guidetorussia.ru': 'guidetorussia.ru',
 'orenburg.guidetorussia.ru': 'guidetorussia.ru',
 'tula.guidetorussia.ru': 'guidetorussia.ru',
 'habarovsk.guidetorussia.ru': 'guidetorussia.ru',
 'reutov.guidetorussia.ru': 'guidetorussia.ru',
 'kemerovo.guidetorussia.ru': 'guidetorussia.ru',
    'ivanovo.bebeshka.info': 'bebeshka.info',
 'mordovia.bebeshka.info': 'bebeshka.info',
 'khv.bebeshka.info': 'bebeshka.info',
 'rnd.bebeshka.info': 'bebeshka.info',
 'kzn.bebeshka.info': 'bebeshka.info',
 'kr.bebeshka.info': 'bebeshka.info',
 'kemerovo.bebeshka.info': 'bebeshka.info',
 'blogs.bebeshka.info': 'bebeshka.info',
 'chel.bebeshka.info': 'bebeshka.info',
 'tula.bebeshka.info': 'bebeshka.info',
 'abakan.bebeshka.info': 'bebeshka.info',
 'stv.bebeshka.info': 'bebeshka.info',
 'ufa.bebeshka.info': 'bebeshka.info',
 'perm.bebeshka.info': 'bebeshka.info',
 'samara.bebeshka.info': 'bebeshka.info',
 'kras.bebeshka.info': 'bebeshka.info',
 'dag.bebeshka.info': 'bebeshka.info',
 'udm.bebeshka.info': 'bebeshka.info',
 'ekb.bebeshka.info': 'bebeshka.info',
 'ryazan.bebeshka.info': 'bebeshka.info',
 'komi.bebeshka.info': 'bebeshka.info',
 'nsk.bebeshka.info': 'bebeshka.info',
 'spb.bebeshka.info': 'bebeshka.info',
 'adygeya.bebeshka.info': 'bebeshka.info',
 'vologda.bebeshka.info': 'bebeshka.info',
 'tyumen.bebeshka.info': 'bebeshka.info',
 'brl.bebeshka.info': 'bebeshka.info',
 'irk.bebeshka.info': 'bebeshka.info',
 'mari.bebeshka.info': 'bebeshka.info',
 'kursk.bebeshka.info': 'bebeshka.info',
 'tuva.bebeshka.info': 'bebeshka.info',
 'saratov.bebeshka.info': 'bebeshka.info',
 'tambov.bebeshka.info': 'bebeshka.info',
 'smolensk.bebeshka.info': 'bebeshka.info',
 'oren.bebeshka.info': 'bebeshka.info',
 'kaluga.bebeshka.info': 'bebeshka.info',
 'yar.bebeshka.info': 'bebeshka.info',
 'bel.bebeshka.info': 'bebeshka.info',
 'nnov.bebeshka.info': 'bebeshka.info',
 'bryansk.bebeshka.info': 'bebeshka.info',
 'magadan.bebeshka.info': 'bebeshka.info',
 'vladimir.bebeshka.info': 'bebeshka.info',
 'kostroma.bebeshka.info': 'bebeshka.info',
 'pskov.bebeshka.info': 'bebeshka.info',
 'kurgan.bebeshka.info': 'bebeshka.info',
 'omsk.bebeshka.info': 'bebeshka.info',
 'bebeshka.info': 'bebeshka.info',
 'vrn.bebeshka.info': 'bebeshka.info',
 'klgd.bebeshka.info': 'bebeshka.info',
 'cr.bebeshka.info': 'bebeshka.info',
 'amur.bebeshka.info': 'bebeshka.info',
 'kirov.bebeshka.info': 'bebeshka.info',
 'kchr.bebeshka.info': 'bebeshka.info',
 'orel.bebeshka.info': 'bebeshka.info',
 'vlg.bebeshka.info': 'bebeshka.info',
 'penza.bebeshka.info': 'bebeshka.info',
 'chuvashia.bebeshka.info': 'bebeshka.info',
 'vl.bebeshka.info': 'bebeshka.info',
 'kbr.bebeshka.info': 'bebeshka.info',
 'ul.bebeshka.info': 'bebeshka.info',
 'sakh.bebeshka.info': 'bebeshka.info',
 'lipetsk.bebeshka.info': 'bebeshka.info',
 'astrakhan.bebeshka.info': 'bebeshka.info',
 'arkh.bebeshka.info': 'bebeshka.info',
 'tsk.bebeshka.info': 'bebeshka.info',
 'kamchatka.bebeshka.info': 'bebeshka.info',
    'afy.ru': 'afy.ru',
 'orenburg-region.afy.ru': 'afy.ru',
 'stavropol-region.afy.ru': 'afy.ru',
 'udmurtiya-region.afy.ru': 'afy.ru',
 'krasnodar-region.afy.ru': 'afy.ru',
 'chuvashiya.afy.ru': 'afy.ru',
 'vladimir-region.afy.ru': 'afy.ru',
 'altay-region.afy.ru': 'afy.ru',
 'spb.afy.ru': 'afy.ru',
 'ivanovo-region.afy.ru': 'afy.ru',
 'voronezh-region.afy.ru': 'afy.ru',
 'sverdlov-region.afy.ru': 'afy.ru',
 'tyumen-region.afy.ru': 'afy.ru',
 'rostov-region.afy.ru': 'afy.ru',
 'chita-region.afy.ru': 'afy.ru',
 'saha-yakutiya.afy.ru': 'afy.ru',
 'ingushetiya.afy.ru': 'afy.ru',
 'lipeck-region.afy.ru': 'afy.ru',
 'samara-region.afy.ru': 'afy.ru',
 'tambov-region.afy.ru': 'afy.ru',
 'kaliningrad-region.afy.ru': 'afy.ru',
 'murmansk-region.afy.ru': 'afy.ru',
 'germany.afy.ru': 'afy.ru',
 'tatarstan.afy.ru': 'afy.ru',
 'kirov-region.afy.ru': 'afy.ru',
 'habarovsk-region.afy.ru': 'afy.ru',
 'kemerovo-region.afy.ru': 'afy.ru',
 'yanao.afy.ru': 'afy.ru',
 'krym.afy.ru': 'afy.ru',
 'bashkortostan.afy.ru': 'afy.ru',
 'perm-region.afy.ru': 'afy.ru',
 'omsk-region.afy.ru': 'afy.ru',
 'turkey.afy.ru': 'afy.ru',
 'spain.afy.ru': 'afy.ru',
 'hanty-mansiyskiy-okrug.afy.ru': 'afy.ru',
 'novosib-region.afy.ru': 'afy.ru',
 'nn-region.afy.ru': 'afy.ru',
 'kaluga-region.afy.ru': 'afy.ru',
 'penza-region.afy.ru': 'afy.ru',
 'krasnoyarsk-region.afy.ru': 'afy.ru',
 'chel-region.afy.ru': 'afy.ru',
 'saratov-region.afy.ru': 'afy.ru',
 'tula-region.afy.ru': 'afy.ru',
 'sahalin-region.afy.ru': 'afy.ru',
 'belgorod-region.afy.ru': 'afy.ru',
 'ryazan-region.afy.ru': 'afy.ru',
 'pskov-region.afy.ru': 'afy.ru',
 'arhangelsk-region.afy.ru': 'afy.ru',
 'chechnya.afy.ru': 'afy.ru',
 'mordoviya.afy.ru': 'afy.ru',
 'tomsk-region.afy.ru': 'afy.ru',
 'mariyel.afy.ru': 'afy.ru',
 'primor-region.afy.ru': 'afy.ru',
 'orel-region.afy.ru': 'afy.ru',
 'komi.afy.ru': 'afy.ru',
 'irkutsk-region.afy.ru': 'afy.ru',
 'usa.afy.ru': 'afy.ru',
 'azerbaijan.afy.ru': 'afy.ru',
 'hakasiya.afy.ru': 'afy.ru',
 'severosetiya.afy.ru': 'afy.ru',
 'novgorod-region.afy.ru': 'afy.ru',
 'yaroslav-region.afy.ru': 'afy.ru',
 'vologda-region.afy.ru': 'afy.ru',
 'bryansk-region.afy.ru': 'afy.ru',
 'belarus.afy.ru': 'afy.ru',
 'kursk-region.afy.ru': 'afy.ru',
    'cherepovec.vokzalzhd.ru': 'vokzalzhd.ru',
 'tyumen.vokzalzhd.ru': 'vokzalzhd.ru',
 'kazanskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'ekaterinburg.vokzalzhd.ru': 'vokzalzhd.ru',
 'habarovsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'paveleckiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'vokzalzhd.ru': 'vokzalzhd.ru',
 'simferopol.vokzalzhd.ru': 'vokzalzhd.ru',
 'leningradskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'moskovskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'ladozhskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'chelny.vokzalzhd.ru': 'vokzalzhd.ru',
 'nizhnij-novgorod.vokzalzhd.ru': 'vokzalzhd.ru',
 'nizhnevartovsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'sochi.vokzalzhd.ru': 'vokzalzhd.ru',
 'yaroslavskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'krasnodar.vokzalzhd.ru': 'vokzalzhd.ru',
 'vladivostok.vokzalzhd.ru': 'vokzalzhd.ru',
 'tomsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'kaluga.vokzalzhd.ru': 'vokzalzhd.ru',
 'belorusskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'ufa.vokzalzhd.ru': 'vokzalzhd.ru',
 'ulyanovsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'yaroslavl.vokzalzhd.ru': 'vokzalzhd.ru',
 'volgograd.vokzalzhd.ru': 'vokzalzhd.ru',
 'ivanovo.vokzalzhd.ru': 'vokzalzhd.ru',
 'orel.vokzalzhd.ru': 'vokzalzhd.ru',
 'novocherkassk.vokzalzhd.ru': 'vokzalzhd.ru',
 'vladimir.vokzalzhd.ru': 'vokzalzhd.ru',
 'vitebskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'pskov.vokzalzhd.ru': 'vokzalzhd.ru',
 'kievskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'rostov-na-donu.vokzalzhd.ru': 'vokzalzhd.ru',
 'rybinsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'velikij-novgorod.vokzalzhd.ru': 'vokzalzhd.ru',
 'belgorod.vokzalzhd.ru': 'vokzalzhd.ru',
 'petrozavodsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'barnaul.vokzalzhd.ru': 'vokzalzhd.ru',
 'krasnoyarsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'rizhskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'ishim.vokzalzhd.ru': 'vokzalzhd.ru',
 'chelyabinsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'savelovskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'vologda.vokzalzhd.ru': 'vokzalzhd.ru',
 'minvody.vokzalzhd.ru': 'vokzalzhd.ru',
 'tuapse.vokzalzhd.ru': 'vokzalzhd.ru',
 'novosibirsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'samara.vokzalzhd.ru': 'vokzalzhd.ru',
 'kurskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'zlatoust.vokzalzhd.ru': 'vokzalzhd.ru',
 'kazan.vokzalzhd.ru': 'vokzalzhd.ru',
 'finlyandskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'baltijskiy.vokzalzhd.ru': 'vokzalzhd.ru',
 'mahachkala.vokzalzhd.ru': 'vokzalzhd.ru',
 'izhevsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'adler.vokzalzhd.ru': 'vokzalzhd.ru',
 'novorossijsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'kostroma.vokzalzhd.ru': 'vokzalzhd.ru',
 'ejsk.vokzalzhd.ru': 'vokzalzhd.ru',
 'tihoreck.vokzalzhd.ru': 'vokzalzhd.ru',
 'kurgan.vokzalzhd.ru': 'vokzalzhd.ru',
 'ryazan.vokzalzhd.ru': 'vokzalzhd.ru',
 'kamensk-shahtinskij.vokzalzhd.ru': 'vokzalzhd.ru',
 'stavropol.vokzalzhd.ru': 'vokzalzhd.ru',
 'anapa.vokzalzhd.ru': 'vokzalzhd.ru',
 'rubcovsk.vokzalzhd.ru': 'vokzalzhd.ru',
    'rnd.antiagent.ru': 'antiagent.ru',
 'antiagent.ru': 'antiagent.ru',
 'kzn.antiagent.ru': 'antiagent.ru',
 'krd.antiagent.ru': 'antiagent.ru',
 'spb.antiagent.ru': 'antiagent.ru',
 'lip.antiagent.ru': 'antiagent.ru',
 'kirov.antiagent.ru': 'antiagent.ru',
 'nsk.antiagent.ru': 'antiagent.ru',
 'smr.antiagent.ru': 'antiagent.ru',
 'mgd.antiagent.ru': 'antiagent.ru',
 'chel.antiagent.ru': 'antiagent.ru',
 'msk.antiagent.ru': 'antiagent.ru',
 'bel.antiagent.ru': 'antiagent.ru',
 'prm.antiagent.ru': 'antiagent.ru',
 'ekb.antiagent.ru': 'antiagent.ru',
 'nnov.antiagent.ru': 'antiagent.ru',
 'sochi.antiagent.ru': 'antiagent.ru',
 'izk.antiagent.ru': 'antiagent.ru',
 'ufa.antiagent.ru': 'antiagent.ru',
 'tmn.antiagent.ru': 'antiagent.ru',
 'ptz.antiagent.ru': 'antiagent.ru',
 'khb.antiagent.ru': 'antiagent.ru',
 'galt.antiagent.ru': 'antiagent.ru',
 'vlg.antiagent.ru': 'antiagent.ru',
 'tula.antiagent.ru': 'antiagent.ru',
 'yar.antiagent.ru': 'antiagent.ru',
 'kgn.antiagent.ru': 'antiagent.ru',
 'smo.antiagent.ru': 'antiagent.ru',
 'abk.antiagent.ru': 'antiagent.ru',
 'klg.antiagent.ru': 'antiagent.ru',
 'iva.antiagent.ru': 'antiagent.ru',
 'kursk.antiagent.ru': 'antiagent.ru',
 'sta.antiagent.ru': 'antiagent.ru',
 'rzn.antiagent.ru': 'antiagent.ru',
 'nch.antiagent.ru': 'antiagent.ru',
 'tlt.antiagent.ru': 'antiagent.ru',
 'chb.antiagent.ru': 'antiagent.ru',
 'uln.antiagent.ru': 'antiagent.ru',
 'ntg.antiagent.ru': 'antiagent.ru',
 'orb.antiagent.ru': 'antiagent.ru',
 'irk.antiagent.ru': 'antiagent.ru',
 'penza.antiagent.ru': 'antiagent.ru',
 'krs.antiagent.ru': 'antiagent.ru',
 'simf.antiagent.ru': 'antiagent.ru',
 'ast.antiagent.ru': 'antiagent.ru',
 'vng.antiagent.ru': 'antiagent.ru',
 'tver.antiagent.ru': 'antiagent.ru',
 'sgt.antiagent.ru': 'antiagent.ru',
 'kln.antiagent.ru': 'antiagent.ru',
 'vgg.antiagent.ru': 'antiagent.ru',
 'bal.antiagent.ru': 'antiagent.ru',
 'stk.antiagent.ru': 'antiagent.ru',
 'sar.antiagent.ru': 'antiagent.ru',
 'nkz.antiagent.ru': 'antiagent.ru',
 'mkp.antiagent.ru': 'antiagent.ru',
 'omsk.antiagent.ru': 'antiagent.ru',
 'tmb.antiagent.ru': 'antiagent.ru',
 'mkh.antiagent.ru': 'antiagent.ru',
 'brn.antiagent.ru': 'antiagent.ru',
 'svs.antiagent.ru': 'antiagent.ru',
 'kst.antiagent.ru': 'antiagent.ru',
 'tomsk.antiagent.ru': 'antiagent.ru',
 'chita.antiagent.ru': 'antiagent.ru',
 'vla.antiagent.ru': 'antiagent.ru',
 'srn.antiagent.ru': 'antiagent.ru',
 'mrm.antiagent.ru': 'antiagent.ru',
 'pdl.antiagent.ru': 'antiagent.ru',
    'krasnoyarsk.bankturov.ru': 'bankturov.ru',
 'ekb.bankturov.ru': 'bankturov.ru',
 'hotels.bankturov.ru': 'bankturov.ru',
 'krasnodar.bankturov.ru': 'bankturov.ru',
 'nsk.bankturov.ru': 'bankturov.ru',
 'kazan.bankturov.ru': 'bankturov.ru',
 'rostov.bankturov.ru': 'bankturov.ru',
 'perm.bankturov.ru': 'bankturov.ru',
 'peterburg.bankturov.ru': 'bankturov.ru',
 'omsk.bankturov.ru': 'bankturov.ru',
 'bankturov.ru': 'bankturov.ru',
 'ufa.bankturov.ru': 'bankturov.ru',
 'surgut.bankturov.ru': 'bankturov.ru',
 'barnaul.bankturov.ru': 'bankturov.ru',
 'kurgan.bankturov.ru': 'bankturov.ru',
 'kaliningrad.bankturov.ru': 'bankturov.ru',
 'tyumen.bankturov.ru': 'bankturov.ru',
 'chelyabinsk.bankturov.ru': 'bankturov.ru',
 'kemerovo.bankturov.ru': 'bankturov.ru',
 'rubtsovsk.bankturov.ru': 'bankturov.ru',
 'komsomolsk.bankturov.ru': 'bankturov.ru',
 'khabarovsk.bankturov.ru': 'bankturov.ru',
 'magnitogorsk.bankturov.ru': 'bankturov.ru',
 'samara.bankturov.ru': 'bankturov.ru',
 'tomsk.bankturov.ru': 'bankturov.ru',
 'ulyanovsk.bankturov.ru': 'bankturov.ru',
 'belgorod.bankturov.ru': 'bankturov.ru',
 'novokuznetsk.bankturov.ru': 'bankturov.ru',
 'orenburg.bankturov.ru': 'bankturov.ru',
 'saratov.bankturov.ru': 'bankturov.ru',
 'abakan.bankturov.ru': 'bankturov.ru',
 'cheboksary.bankturov.ru': 'bankturov.ru',
 'vladivostok.bankturov.ru': 'bankturov.ru',
 'izhevsk.bankturov.ru': 'bankturov.ru',
 'chelny.bankturov.ru': 'bankturov.ru',
 'novgorod.bankturov.ru': 'bankturov.ru',
 'blagoveschensk.bankturov.ru': 'bankturov.ru',
 'nizhnevartovsk.bankturov.ru': 'bankturov.ru',
 'ivanovo.bankturov.ru': 'bankturov.ru',
 'volgograd.bankturov.ru': 'bankturov.ru',
 'nefteyugansk.bankturov.ru': 'bankturov.ru',
 'berezniki.bankturov.ru': 'bankturov.ru',
 'tagil.bankturov.ru': 'bankturov.ru',
 'khanty-mansiysk.bankturov.ru': 'bankturov.ru',
 'yaroslavl.bankturov.ru': 'bankturov.ru',
 'pyt-yah.bankturov.ru': 'bankturov.ru',
 'kyzyl.bankturov.ru': 'bankturov.ru',
 'sochi.bankturov.ru': 'bankturov.ru',
 'irkutsk.bankturov.ru': 'bankturov.ru',
 'stavropol.bankturov.ru': 'bankturov.ru',
 'grozny.bankturov.ru': 'bankturov.ru',
 'kamensk-uralskiy.bankturov.ru': 'bankturov.ru',
 'yuzhno-sahalinsk.bankturov.ru': 'bankturov.ru',
 'minvody.bankturov.ru': 'bankturov.ru',
 'urengoy.bankturov.ru': 'bankturov.ru',
 'kirov.bankturov.ru': 'bankturov.ru',
 'angarsk.bankturov.ru': 'bankturov.ru',
 'bryansk.bankturov.ru': 'bankturov.ru',
 'voronezh.bankturov.ru': 'bankturov.ru',
 'ust-ilimsk.bankturov.ru': 'bankturov.ru',
 'tver.bankturov.ru': 'bankturov.ru',
 'leninsk-kuznetskiy.bankturov.ru': 'bankturov.ru',
 'tolyatti.bankturov.ru': 'bankturov.ru',
 'berdsk.bankturov.ru': 'bankturov.ru',
 'ulan-ude.bankturov.ru': 'bankturov.ru',
 'tula.bankturov.ru': 'bankturov.ru',
 'murmansk.bankturov.ru': 'bankturov.ru',
    'academica.ru': 'academica.ru',
 'krasnoyarsk.academica.ru': 'academica.ru',
 'saratov.academica.ru': 'academica.ru',
 'habarovsk.academica.ru': 'academica.ru',
 'omsk.academica.ru': 'academica.ru',
 'chelyabinsk.academica.ru': 'academica.ru',
 'voronezh.academica.ru': 'academica.ru',
 'samara.academica.ru': 'academica.ru',
 'ekb.academica.ru': 'academica.ru',
 'nn.academica.ru': 'academica.ru',
 'gelendzhik.academica.ru': 'academica.ru',
 'tver.academica.ru': 'academica.ru',
 'ryazan.academica.ru': 'academica.ru',
 'kazan.academica.ru': 'academica.ru',
 'rnd.academica.ru': 'academica.ru',
 'orenburg.academica.ru': 'academica.ru',
 'volgograd.academica.ru': 'academica.ru',
 'novosibirsk.academica.ru': 'academica.ru',
 'oryol.academica.ru': 'academica.ru',
 'ufa.academica.ru': 'academica.ru',
 'novorossiysk.academica.ru': 'academica.ru',
 'vladivostok.academica.ru': 'academica.ru',
 'blagoveshchensk.academica.ru': 'academica.ru',
 'msk.academica.ru': 'academica.ru',
 'murmansk.academica.ru': 'academica.ru',
 'yaroslavl.academica.ru': 'academica.ru',
 'izhevsk.academica.ru': 'academica.ru',
 'spb.academica.ru': 'academica.ru',
 'taganrog.academica.ru': 'academica.ru',
 'tula.academica.ru': 'academica.ru',
 'kirov.academica.ru': 'academica.ru',
 'ussuriysk.academica.ru': 'academica.ru',
 'nalchik.academica.ru': 'academica.ru',
 'cherkessk.academica.ru': 'academica.ru',
 'kurgan.academica.ru': 'academica.ru',
 'krasnodar.academica.ru': 'academica.ru',
 'stavropol.academica.ru': 'academica.ru',
 'nizhnevartovsk.academica.ru': 'academica.ru',
 'balakovo.academica.ru': 'academica.ru',
 'barnaul.academica.ru': 'academica.ru',
 'perm.academica.ru': 'academica.ru',
 'armavir.academica.ru': 'academica.ru',
 'abakan.academica.ru': 'academica.ru',
 'kaluga.academica.ru': 'academica.ru',
 'cheboksary.academica.ru': 'academica.ru',
 'tyumen.academica.ru': 'academica.ru',
 'kislovodsk.academica.ru': 'academica.ru',
 'penza.academica.ru': 'academica.ru',
 'cherepovets.academica.ru': 'academica.ru',
 'magadan.academica.ru': 'academica.ru',
 'balashiha.academica.ru': 'academica.ru',
 'smolensk.academica.ru': 'academica.ru',
 'podolsk.academica.ru': 'academica.ru',
 'miass.academica.ru': 'academica.ru',
 'vladimir.academica.ru': 'academica.ru',
 'sochi.academica.ru': 'academica.ru',
 'irkutsk.academica.ru': 'academica.ru',
 'aleksandrov.academica.ru': 'academica.ru',
 'kaliningrad.academica.ru': 'academica.ru',
 'kolomna.academica.ru': 'academica.ru',
 'eysk.academica.ru': 'academica.ru',
 'dimitrovgrad.academica.ru': 'academica.ru',
 'ulyanovsk.academica.ru': 'academica.ru',
 'sarov.academica.ru': 'academica.ru',
 'ivanovo.academica.ru': 'academica.ru',
 'lipetsk.academica.ru': 'academica.ru',
 'hm.academica.ru': 'academica.ru',
    'saratov.big-book-money.ru': 'big-book-money.ru',
 'kolomna.big-book-money.ru': 'big-book-money.ru',
 'tyumen.big-book-money.ru': 'big-book-money.ru',
 'bryansk.big-book-money.ru': 'big-book-money.ru',
 'cheboksary.big-book-money.ru': 'big-book-money.ru',
 'kirov.big-book-money.ru': 'big-book-money.ru',
 'neftekamsk.big-book-money.ru': 'big-book-money.ru',
 'novorossiysk.big-book-money.ru': 'big-book-money.ru',
 'serov.big-book-money.ru': 'big-book-money.ru',
 'moscow.big-book-money.ru': 'big-book-money.ru',
 'zhukovskij.big-book-money.ru': 'big-book-money.ru',
 'surgut.big-book-money.ru': 'big-book-money.ru',
 'maykop.big-book-money.ru': 'big-book-money.ru',
 'petropavlovsk-kamchatskiy.big-book-money.ru': 'big-book-money.ru',
 'kursk.big-book-money.ru': 'big-book-money.ru',
 'krasnodar.big-book-money.ru': 'big-book-money.ru',
 'rostov-na-donu.big-book-money.ru': 'big-book-money.ru',
 'tolyatti.big-book-money.ru': 'big-book-money.ru',
 'izhevsk.big-book-money.ru': 'big-book-money.ru',
 'ufa.big-book-money.ru': 'big-book-money.ru',
 'obninsk.big-book-money.ru': 'big-book-money.ru',
 'ryazan.big-book-money.ru': 'big-book-money.ru',
 'novosibirsk.big-book-money.ru': 'big-book-money.ru',
 'novokuznetsk.big-book-money.ru': 'big-book-money.ru',
 'lipetsk.big-book-money.ru': 'big-book-money.ru',
 'voronezh.big-book-money.ru': 'big-book-money.ru',
 'nevinnomissk.big-book-money.ru': 'big-book-money.ru',
 'tomsk.big-book-money.ru': 'big-book-money.ru',
 'nizhniy-tagil.big-book-money.ru': 'big-book-money.ru',
 'sochi.big-book-money.ru': 'big-book-money.ru',
 'berezniki.big-book-money.ru': 'big-book-money.ru',
 'pyatigorsk.big-book-money.ru': 'big-book-money.ru',
 'khabarovsk.big-book-money.ru': 'big-book-money.ru',
 'engels.big-book-money.ru': 'big-book-money.ru',
 'volgodonsk.big-book-money.ru': 'big-book-money.ru',
 'kaliningrad.big-book-money.ru': 'big-book-money.ru',
 'krasnoyarsk.big-book-money.ru': 'big-book-money.ru',
 'stavropol.big-book-money.ru': 'big-book-money.ru',
 'sterlitamak.big-book-money.ru': 'big-book-money.ru',
 'belgorod.big-book-money.ru': 'big-book-money.ru',
 'vladimir.big-book-money.ru': 'big-book-money.ru',
 'krasnogorsk.big-book-money.ru': 'big-book-money.ru',
 'saransk.big-book-money.ru': 'big-book-money.ru',
 'yeisk.big-book-money.ru': 'big-book-money.ru',
 'almetyevsk.big-book-money.ru': 'big-book-money.ru',
 'smolensk.big-book-money.ru': 'big-book-money.ru',
 'saint-petersburg.big-book-money.ru': 'big-book-money.ru',
 'kaluga.big-book-money.ru': 'big-book-money.ru',
 'kazan.big-book-money.ru': 'big-book-money.ru',
 'armavir.big-book-money.ru': 'big-book-money.ru',
 'rybinsk.big-book-money.ru': 'big-book-money.ru',
 'vladivostok.big-book-money.ru': 'big-book-money.ru',
 'arkhangelsk.big-book-money.ru': 'big-book-money.ru',
 'nakhodka.big-book-money.ru': 'big-book-money.ru',
 'kurgan.big-book-money.ru': 'big-book-money.ru',
 'oryol.big-book-money.ru': 'big-book-money.ru',
 'yekaterinburg.big-book-money.ru': 'big-book-money.ru',
 'staryy-oskol.big-book-money.ru': 'big-book-money.ru',
 'magnitogorsk.big-book-money.ru': 'big-book-money.ru',
 'naberezhnyye-chelny.big-book-money.ru': 'big-book-money.ru',
 'nizhniy-novgorod.big-book-money.ru': 'big-book-money.ru',
 'omsk.big-book-money.ru': 'big-book-money.ru',
 'komsomolsk-na-amure.big-book-money.ru': 'big-book-money.ru',
 'volgograd.big-book-money.ru': 'big-book-money.ru',
 'petrozavodsk.big-book-money.ru': 'big-book-money.ru',
 'taganrog.big-book-money.ru': 'big-book-money.ru',
 'yaroslavl.big-book-money.ru': 'big-book-money.ru',
    'tosno.guiderf.ru': 'guiderf.ru',
 'kazan.guiderf.ru': 'guiderf.ru',
 'ulyanovsk.guiderf.ru': 'guiderf.ru',
 'guiderf.ru': 'guiderf.ru',
 'rezh.guiderf.ru': 'guiderf.ru',
 'buinsk.guiderf.ru': 'guiderf.ru',
 'pavlovo.guiderf.ru': 'guiderf.ru',
 'perm.guiderf.ru': 'guiderf.ru',
 'vyksa.guiderf.ru': 'guiderf.ru',
 'gorodets.guiderf.ru': 'guiderf.ru',
 'nizhniy-novgorod.guiderf.ru': 'guiderf.ru',
 'arzamas.guiderf.ru': 'guiderf.ru',
 'bogdanovich.guiderf.ru': 'guiderf.ru',
 'sankt-peterburg.guiderf.ru': 'guiderf.ru',
 'pyt-yah.guiderf.ru': 'guiderf.ru',
 'sochi.guiderf.ru': 'guiderf.ru',
 'yugorsk.guiderf.ru': 'guiderf.ru',
 'tagil.guiderf.ru': 'guiderf.ru',
 'tayga.guiderf.ru': 'guiderf.ru',
 'moscow.guiderf.ru': 'guiderf.ru',
 'novorossiysk.guiderf.ru': 'guiderf.ru',
 'rostov.guiderf.ru': 'guiderf.ru',
 'cherepovets.guiderf.ru': 'guiderf.ru',
 'usinsk.guiderf.ru': 'guiderf.ru',
 'murmansk.guiderf.ru': 'guiderf.ru',
 'tobolsk.guiderf.ru': 'guiderf.ru',
 'vorkuta.guiderf.ru': 'guiderf.ru',
 'satka.guiderf.ru': 'guiderf.ru',
 'pechora.guiderf.ru': 'guiderf.ru',
 'inta.guiderf.ru': 'guiderf.ru',
 'syktyvkar.guiderf.ru': 'guiderf.ru',
 'sosnogorsk.guiderf.ru': 'guiderf.ru',
 'uhta.guiderf.ru': 'guiderf.ru',
 'dubna.guiderf.ru': 'guiderf.ru',
 'hasan.guiderf.ru': 'guiderf.ru',
 'serpuhov.guiderf.ru': 'guiderf.ru',
 'berezniki.guiderf.ru': 'guiderf.ru',
 'dzerzhinsk.guiderf.ru': 'guiderf.ru',
 'izhevsk.guiderf.ru': 'guiderf.ru',
 'alapaevsk.guiderf.ru': 'guiderf.ru',
 'polevskoy.guiderf.ru': 'guiderf.ru',
 'chel.guiderf.ru': 'guiderf.ru',
 'tomsk.guiderf.ru': 'guiderf.ru',
 'verhnii-ufalei.guiderf.ru': 'guiderf.ru',
 'nighnevartovsk.guiderf.ru': 'guiderf.ru',
 'voronezh.guiderf.ru': 'guiderf.ru',
 'gorno-altaysk.guiderf.ru': 'guiderf.ru',
 'velikie-luky.guiderf.ru': 'guiderf.ru',
 'volhov.guiderf.ru': 'guiderf.ru',
 'novouralsk.guiderf.ru': 'guiderf.ru',
 'serov.guiderf.ru': 'guiderf.ru',
 'ekb.guiderf.ru': 'guiderf.ru',
 'kop.guiderf.ru': 'guiderf.ru',
 'zarechny.guiderf.ru': 'guiderf.ru',
 'sosnoviy-bor.guiderf.ru': 'guiderf.ru',
 'urga.guiderf.ru': 'guiderf.ru',
 'kaliningrad.guiderf.ru': 'guiderf.ru',
 'ufa.guiderf.ru': 'guiderf.ru',
 'ussuriysk.guiderf.ru': 'guiderf.ru',
 'naberezhnie-chelny.guiderf.ru': 'guiderf.ru',
 'elektrostal.guiderf.ru': 'guiderf.ru',
 'khabarovsk.guiderf.ru': 'guiderf.ru',
 'volgograd.guiderf.ru': 'guiderf.ru',
 'kamensk.guiderf.ru': 'guiderf.ru',
 'zelenograd.guiderf.ru': 'guiderf.ru',
 'krasnodar.guiderf.ru': 'guiderf.ru',
 'nurlat.guiderf.ru': 'guiderf.ru',
 'dmitrov.guiderf.ru': 'guiderf.ru',
 'severobaykalsk.guiderf.ru': 'guiderf.ru',
    'nv-rh.cerkov.ru': 'cerkov.ru',
 'vvedenskymon.cerkov.ru': 'cerkov.ru',
 'hramvidiaevo.cerkov.ru': 'cerkov.ru',
 'peskisret.cerkov.ru': 'cerkov.ru',
 'cerkov.ru': 'cerkov.ru',
 'georgievka.cerkov.ru': 'cerkov.ru',
 'missia-penza.cerkov.ru': 'cerkov.ru',
 'pokrov-endov.cerkov.ru': 'cerkov.ru',
 'sv-uspenie.cerkov.ru': 'cerkov.ru',
 'krestovayapustin.cerkov.ru': 'cerkov.ru',
 'troica-tyumen.cerkov.ru': 'cerkov.ru',
 'hram72.cerkov.ru': 'cerkov.ru',
 'matronamosckov.cerkov.ru': 'cerkov.ru',
 'serdobsk-eparh.cerkov.ru': 'cerkov.ru',
 'feodorovskaya.cerkov.ru': 'cerkov.ru',
 'hram-nevsky-kruf.cerkov.ru': 'cerkov.ru',
 'troickiy.cerkov.ru': 'cerkov.ru',
 'bogoyavlensobor.cerkov.ru': 'cerkov.ru',
 'lib.cerkov.ru': 'cerkov.ru',
 'uspenie22.cerkov.ru': 'cerkov.ru',
 'poisk.cerkov.ru': 'cerkov.ru',
 'abalak-monastery.cerkov.ru': 'cerkov.ru',
 'arzblag-sobor.cerkov.ru': 'cerkov.ru',
 'belayakolp.cerkov.ru': 'cerkov.ru',
 'energetik.cerkov.ru': 'cerkov.ru',
 'spb-21344.cerkov.ru': 'cerkov.ru',
 'nascha.cerkov.ru': 'cerkov.ru',
 'sobor-shchelkovo.cerkov.ru': 'cerkov.ru',
 'hram-talovaya.cerkov.ru': 'cerkov.ru',
 'pokrovtulun.cerkov.ru': 'cerkov.ru',
 'hram41.cerkov.ru': 'cerkov.ru',
 'apostol1.cerkov.ru': 'cerkov.ru',
 'saintannakashin.cerkov.ru': 'cerkov.ru',
 'svya-pokrovskiy.cerkov.ru': 'cerkov.ru',
 'uspeniya.cerkov.ru': 'cerkov.ru',
 'soborpenza.cerkov.ru': 'cerkov.ru',
 'inta.cerkov.ru': 'cerkov.ru',
 's-prihod.cerkov.ru': 'cerkov.ru',
 'ikoni.cerkov.ru': 'cerkov.ru',
 'yasenka.cerkov.ru': 'cerkov.ru',
 'sobor-livny.cerkov.ru': 'cerkov.ru',
 'nikola-sov.cerkov.ru': 'cerkov.ru',
 'rossosh-r-b.cerkov.ru': 'cerkov.ru',
 'hramlog.cerkov.ru': 'cerkov.ru',
 'yazikovo.cerkov.ru': 'cerkov.ru',
 'kavkazskaya-hram.cerkov.ru': 'cerkov.ru',
 'voskr-izh.cerkov.ru': 'cerkov.ru',
 'nv-predtecha.cerkov.ru': 'cerkov.ru',
 'nikolangelovo.cerkov.ru': 'cerkov.ru',
 'zelenogorsk-spb.cerkov.ru': 'cerkov.ru',
 'tdu.cerkov.ru': 'cerkov.ru',
 'berezniki.cerkov.ru': 'cerkov.ru',
 'selyatino.cerkov.ru': 'cerkov.ru',
 'serafim-penza.cerkov.ru': 'cerkov.ru',
 'yakov.cerkov.ru': 'cerkov.ru',
 'kursk-eparh.cerkov.ru': 'cerkov.ru',
 'volblag.cerkov.ru': 'cerkov.ru',
 'shadreparh.cerkov.ru': 'cerkov.ru',
 'hram-troitsa.cerkov.ru': 'cerkov.ru',
 'armih.cerkov.ru': 'cerkov.ru',
 'verh-myachkovo.cerkov.ru': 'cerkov.ru',
 'shahtpokrov.cerkov.ru': 'cerkov.ru',
 'arz-palom.cerkov.ru': 'cerkov.ru',
 'aleknew.cerkov.ru': 'cerkov.ru',
 'otradnenskay.cerkov.ru': 'cerkov.ru',
 'nizpen.cerkov.ru': 'cerkov.ru',
 'sergiev-prihod.cerkov.ru': 'cerkov.ru',
 'ortodoxbqt.cerkov.ru': 'cerkov.ru',
 'nikola-obitel.cerkov.ru': 'cerkov.ru',
 'podvorye-arzblag.cerkov.ru': 'cerkov.ru',
 'german73.cerkov.ru': 'cerkov.ru',
 'ankovo.cerkov.ru': 'cerkov.ru',
 'arhmihaila.cerkov.ru': 'cerkov.ru',
 'bmc-hram.cerkov.ru': 'cerkov.ru',
 'hramsuvorov.cerkov.ru': 'cerkov.ru',
 'ntobitel.cerkov.ru': 'cerkov.ru',
 'troitsky.cerkov.ru': 'cerkov.ru',
 'hram-vs.cerkov.ru': 'cerkov.ru',
 'andrejhram.cerkov.ru': 'cerkov.ru',
 'nyagan-hram.cerkov.ru': 'cerkov.ru',
 'dpcentr.cerkov.ru': 'cerkov.ru',
 'troicahram.cerkov.ru': 'cerkov.ru',
 'shelehov.cerkov.ru': 'cerkov.ru',
 'churh-history.cerkov.ru': 'cerkov.ru',
 'panteleimonkuban.cerkov.ru': 'cerkov.ru',
 'kraskovo.cerkov.ru': 'cerkov.ru',
 'hram-pokrova.cerkov.ru': 'cerkov.ru',
 'svyat-sergm.cerkov.ru': 'cerkov.ru',
 'snezhinsk.cerkov.ru': 'cerkov.ru',
 'skaut.cerkov.ru': 'cerkov.ru',
 'tihonzadonski.cerkov.ru': 'cerkov.ru',
 'spasbryansk.cerkov.ru': 'cerkov.ru',
 'beregovoe.cerkov.ru': 'cerkov.ru',
 'selokozino.cerkov.ru': 'cerkov.ru',
 'strelica.cerkov.ru': 'cerkov.ru',
 'sv-troitsa.cerkov.ru': 'cerkov.ru',
 'pip-vdonsk.cerkov.ru': 'cerkov.ru',
 'tver-pervoe-4.cerkov.ru': 'cerkov.ru',
 'smol-stt-app.cerkov.ru': 'cerkov.ru',
 'nusman.cerkov.ru': 'cerkov.ru',
 'smolmerkuriy.cerkov.ru': 'cerkov.ru',
 'trinity29.cerkov.ru': 'cerkov.ru',
 'spass.cerkov.ru': 'cerkov.ru',
 'hram-radosti.cerkov.ru': 'cerkov.ru',
 'hram-katunki.cerkov.ru': 'cerkov.ru',
 'znamenskiy-hram.cerkov.ru': 'cerkov.ru',
    'sm.news': 'sm.news',
 'ekb.sm.news': 'sm.news',
 'samara.sm.news': 'sm.news',
 'izhevsk.sm.news': 'sm.news',
 'kirov.sm.news': 'sm.news',
 'chelyabinsk.sm.news': 'sm.news',
 'crimea.sm.news': 'sm.news',
 'tula.sm.news': 'sm.news',
 'kazan.sm.news': 'sm.news',
 'volgograd.sm.news': 'sm.news',
 'saratov.sm.news': 'sm.news',
 'veliky-novgorod.sm.news': 'sm.news',
 'syktyvkar.sm.news': 'sm.news',
 'ryazan.sm.news': 'sm.news',
 'perm.sm.news': 'sm.news',
 'bryansk.sm.news': 'sm.news',
 'archangelsk.sm.news': 'sm.news',
 'kaliningrad.sm.news': 'sm.news',
 'pskov.sm.news': 'sm.news',
 'lipetsk.sm.news': 'sm.news',
 'voronezh.sm.news': 'sm.news',
 'nijniy-novgorod.sm.news': 'sm.news',
 'mos.sm.news': 'sm.news',
 'magas.sm.news': 'sm.news',
 'orenburg.sm.news': 'sm.news',
 'abakan.sm.news': 'sm.news',
 'astrakhan.sm.news': 'sm.news',
 'maykop.sm.news': 'sm.news',
 'krasnoyarsk.sm.news': 'sm.news',
 'vladivostok.sm.news': 'sm.news',
 'tmn.sm.news': 'sm.news',
 'ufa.sm.news': 'sm.news',
 'barnaul.sm.news': 'sm.news',
 'moscow-oblast.sm.news': 'sm.news',
 'ulyanovsk.sm.news': 'sm.news',
 'saransk.sm.news': 'sm.news',
 'murmansk.sm.news': 'sm.news',
 'cheboksary.sm.news': 'sm.news',
 'vladimir.sm.news': 'sm.news',
 'rostov-on-don.sm.news': 'sm.news',
 'krasnodar.sm.news': 'sm.news',
 'kostroma.sm.news': 'sm.news',
 'anadyr.sm.news': 'sm.news',
 'petrozavodsk.sm.news': 'sm.news',
 'belgorod.sm.news': 'sm.news',
 'chita.sm.news': 'sm.news',
 'mari-el.sm.news': 'sm.news',
 'smolensk.sm.news': 'sm.news',
 'petropavlovsk-kamchatsky.sm.news': 'sm.news',
 'kurgan.sm.news': 'sm.news',
 'cherkessk.sm.news': 'sm.news',
 'novosibirsk.sm.news': 'sm.news',
 'yuzhno-sakhalinsk.sm.news': 'sm.news',
 'tomsk.sm.news': 'sm.news',
 'ulan-ude.sm.news': 'sm.news',
 'ivanovo.sm.news': 'sm.news',
 'stavropol.sm.news': 'sm.news',
 'yaroslavl.sm.news': 'sm.news',
 'kemerovo.sm.news': 'sm.news',
 'yakutsk.sm.news': 'sm.news',
 'kursk.sm.news': 'sm.news',
 'magadan.sm.news': 'sm.news',
 'vladikavkaz.sm.news': 'sm.news',
 'tambov.sm.news': 'sm.news',
 'khabarovsk.sm.news': 'sm.news',
 'elista.sm.news': 'sm.news',
 'saint-petersburg.sm.news': 'sm.news',
 'kyzyl.sm.news': 'sm.news',
 'kaluga.sm.news': 'sm.news',
    'kursk.propartner.ru': 'propartner.ru',
 'krasnoyarsk.propartner.ru': 'propartner.ru',
 'chel.propartner.ru': 'propartner.ru',
 'krasnodar.propartner.ru': 'propartner.ru',
 'moscow.propartner.ru': 'propartner.ru',
 'samara.propartner.ru': 'propartner.ru',
 'nsk.propartner.ru': 'propartner.ru',
 'cheboksary.propartner.ru': 'propartner.ru',
 'kazan.propartner.ru': 'propartner.ru',
 'novorossiysk.propartner.ru': 'propartner.ru',
 'barnaul.propartner.ru': 'propartner.ru',
 'nn.propartner.ru': 'propartner.ru',
 'tatarstan.propartner.ru': 'propartner.ru',
 'podolsk.propartner.ru': 'propartner.ru',
 'spb.propartner.ru': 'propartner.ru',
 'khabarovsk.propartner.ru': 'propartner.ru',
 'saransk.propartner.ru': 'propartner.ru',
 'sochi.propartner.ru': 'propartner.ru',
 'yaroslavl.propartner.ru': 'propartner.ru',
 'ekb.propartner.ru': 'propartner.ru',
 'tomsk.propartner.ru': 'propartner.ru',
 'chelny.propartner.ru': 'propartner.ru',
 'tolyatty.propartner.ru': 'propartner.ru',
 'stavropol.propartner.ru': 'propartner.ru',
 'skt.propartner.ru': 'propartner.ru',
 'magnitogorsk.propartner.ru': 'propartner.ru',
 'stavropol-kray.propartner.ru': 'propartner.ru',
 'mytishy.propartner.ru': 'propartner.ru',
 'izhevsk.propartner.ru': 'propartner.ru',
 'ola.propartner.ru': 'propartner.ru',
 'tyumen.propartner.ru': 'propartner.ru',
 'nt.propartner.ru': 'propartner.ru',
 'muraveinik.propartner.ru': 'propartner.ru',
 'irk.propartner.ru': 'propartner.ru',
 'perm.propartner.ru': 'propartner.ru',
 'orenburg.propartner.ru': 'propartner.ru',
 'vladivostok.propartner.ru': 'propartner.ru',
 'tambov.propartner.ru': 'propartner.ru',
 'ufa.propartner.ru': 'propartner.ru',
 'petropavlovsk.propartner.ru': 'propartner.ru',
 'novokyznetsk.propartner.ru': 'propartner.ru',
 'omsk.propartner.ru': 'propartner.ru',
 'arh.propartner.ru': 'propartner.ru',
 'volgograd.propartner.ru': 'propartner.ru',
 'primorskiy-kray.propartner.ru': 'propartner.ru',
 'surgut.propartner.ru': 'propartner.ru',
 'rostov.propartner.ru': 'propartner.ru',
 'kaluzhskaya-obl.propartner.ru': 'propartner.ru',
 'ivanovo.propartner.ru': 'propartner.ru',
 'saratov.propartner.ru': 'propartner.ru',
 'murmansk.propartner.ru': 'propartner.ru',
 'voronezh.propartner.ru': 'propartner.ru',
 'oboiland.propartner.ru': 'propartner.ru',
 'altayskiy-kray.propartner.ru': 'propartner.ru',
 'abakan.propartner.ru': 'propartner.ru',
 'taganrog.propartner.ru': 'propartner.ru',
 'pyatigorsk.propartner.ru': 'propartner.ru',
 'kaliningrad.propartner.ru': 'propartner.ru',
 'dzerzhinsk.propartner.ru': 'propartner.ru',
 'sakhalinsk.propartner.ru': 'propartner.ru',
 'samara-obl.propartner.ru': 'propartner.ru',
 'pyshma.propartner.ru': 'propartner.ru',
 'ryazan.propartner.ru': 'propartner.ru',
 'sterlitamak.propartner.ru': 'propartner.ru',
 'bryansk.propartner.ru': 'propartner.ru',
 'propartner.ru': 'propartner.ru',
 'belgorod.propartner.ru': 'propartner.ru',
 'orel.propartner.ru': 'propartner.ru',
 'shahty.propartner.ru': 'propartner.ru',
 'novgorod.propartner.ru': 'propartner.ru',
    'perm.promindex.ru': 'promindex.ru',
 'chelyabinsk.promindex.ru': 'promindex.ru',
 'promindex.ru': 'promindex.ru',
 'barnaul.promindex.ru': 'promindex.ru',
 'msk.promindex.ru': 'promindex.ru',
 'krasnodar.promindex.ru': 'promindex.ru',
 'nsk.promindex.ru': 'promindex.ru',
 'tver.promindex.ru': 'promindex.ru',
 'khabarovsk.promindex.ru': 'promindex.ru',
 'spb.promindex.ru': 'promindex.ru',
 'voronezh.promindex.ru': 'promindex.ru',
 'krasnokamsk.promindex.ru': 'promindex.ru',
 'tatarstan.promindex.ru': 'promindex.ru',
 'penza.promindex.ru': 'promindex.ru',
 'rostov-na-donu.promindex.ru': 'promindex.ru',
 'omsk.promindex.ru': 'promindex.ru',
 'kazan.promindex.ru': 'promindex.ru',
 'tumen.promindex.ru': 'promindex.ru',
 'samara.promindex.ru': 'promindex.ru',
 'vladimir.promindex.ru': 'promindex.ru',
 'izhevsk.promindex.ru': 'promindex.ru',
 'ulyanovsk.promindex.ru': 'promindex.ru',
 'taganrog.promindex.ru': 'promindex.ru',
 'nnovgorod.promindex.ru': 'promindex.ru',
 'ekb.promindex.ru': 'promindex.ru',
 'ufa.promindex.ru': 'promindex.ru',
 'naberezhnie-chelni.promindex.ru': 'promindex.ru',
 'irkutsk.promindex.ru': 'promindex.ru',
 'orenburg.promindex.ru': 'promindex.ru',
 'sterlitamak.promindex.ru': 'promindex.ru',
 'novocherkassk.promindex.ru': 'promindex.ru',
 'leningradskaya-oblast.promindex.ru': 'promindex.ru',
 'selyatino.promindex.ru': 'promindex.ru',
 'moskovskaya-oblast.promindex.ru': 'promindex.ru',
 'novokuzneck.promindex.ru': 'promindex.ru',
 'pervouralsk.promindex.ru': 'promindex.ru',
 'krasnoyarsk.promindex.ru': 'promindex.ru',
 'podolsk.promindex.ru': 'promindex.ru',
 'miass.promindex.ru': 'promindex.ru',
 'kirov.promindex.ru': 'promindex.ru',
 'astrakhan.promindex.ru': 'promindex.ru',
 'kemerovo.promindex.ru': 'promindex.ru',
 'sochi.promindex.ru': 'promindex.ru',
 'belgorod.promindex.ru': 'promindex.ru',
 'rostovskaya-oblast.promindex.ru': 'promindex.ru',
 'orel.promindex.ru': 'promindex.ru',
 'arkhangelsk.promindex.ru': 'promindex.ru',
 'surgut.promindex.ru': 'promindex.ru',
 'cheboksari.promindex.ru': 'promindex.ru',
 'tambov.promindex.ru': 'promindex.ru',
 'tomsk.promindex.ru': 'promindex.ru',
 'yaroslavl.promindex.ru': 'promindex.ru',
 'siktivkar.promindex.ru': 'promindex.ru',
 'tolyatti.promindex.ru': 'promindex.ru',
 'vladivostok.promindex.ru': 'promindex.ru',
 'bashkortostan.promindex.ru': 'promindex.ru',
 'volgograd.promindex.ru': 'promindex.ru',
 'neftekamsk.promindex.ru': 'promindex.ru',
 'krasnodarskiy-kray.promindex.ru': 'promindex.ru',
 'kuvandik.promindex.ru': 'promindex.ru',
 'ribinsk.promindex.ru': 'promindex.ru',
 'sevastopol.promindex.ru': 'promindex.ru',
 'nizhegorodskaya-oblast.promindex.ru': 'promindex.ru',
 'chuvashiya.promindex.ru': 'promindex.ru',
 'rostov.promindex.ru': 'promindex.ru',
 'feodosiya.promindex.ru': 'promindex.ru',
 'dzerzhinsk.promindex.ru': 'promindex.ru',
 'ryazan.promindex.ru': 'promindex.ru',
 'kostroma.promindex.ru': 'promindex.ru',
 'stariy-oskol.promindex.ru': 'promindex.ru',
 'kaliningrad.promindex.ru': 'promindex.ru',
    '072107.apad.top': 'apad.top',
 '092204.apad.top': 'apad.top',
 '262103.apad.top': 'apad.top',
 '6c860a10ffb0d492c9cb3f96a688d1436c860a10ffb0d492c9cb3f96a688d20.apad.top': 'apad.top',
 '302106.apad.top': 'apad.top',
 '272106.apad.top': 'apad.top',
 '022011.apad.top': 'apad.top',
 '032011.apad.top': 'apad.top',
 '252202.apad.top': 'apad.top',
 '132108.apad.top': 'apad.top',
 '232107.apad.top': 'apad.top',
 '242203.apad.top': 'apad.top',
 '212203.apad.top': 'apad.top',
 '102107.apad.top': 'apad.top',
 '052107.apad.top': 'apad.top',
 '022107.apad.top': 'apad.top',
 '262106.apad.top': 'apad.top',
 '292203.apad.top': 'apad.top',
 '012108.apad.top': 'apad.top',
 '232106.apad.top': 'apad.top',
 '052203.apad.top': 'apad.top',
 '172106.apad.top': 'apad.top',
 '222108.apad.top': 'apad.top',
 '262108.apad.top': 'apad.top',
 '282205.apad.top': 'apad.top',
 '292107.apad.top': 'apad.top',
 '222106.apad.top': 'apad.top',
 '242106.apad.top': 'apad.top',
 '092108.apad.top': 'apad.top',
 '282203.apad.top': 'apad.top',
 '212106.apad.top': 'apad.top',
 '262107.apad.top': 'apad.top',
 '142107.apad.top': 'apad.top',
 '092107.apad.top': 'apad.top',
 '312203.apad.top': 'apad.top',
 '142106.apad.top': 'apad.top',
 '222204.apad.top': 'apad.top',
 '212103.apad.top': 'apad.top',
 '132107.apad.top': 'apad.top',
 '102108.apad.top': 'apad.top',
 '052204.apad.top': 'apad.top',
 '302108.apad.top': 'apad.top',
 '112205.apad.top': 'apad.top',
 '122107.apad.top': 'apad.top',
 '272204.apad.top': 'apad.top',
 '192108.apad.top': 'apad.top',
 '112204.apad.top': 'apad.top',
 '032110.apad.top': 'apad.top',
 '042107.apad.top': 'apad.top',
 '182106.apad.top': 'apad.top',
 '152106.apad.top': 'apad.top',
 '022108.apad.top': 'apad.top',
 '312107.apad.top': 'apad.top',
 '222107.apad.top': 'apad.top',
 '162101.apad.top': 'apad.top',
 '302204.apad.top': 'apad.top',
 '202112.apad.top': 'apad.top',
 '102011.apad.top': 'apad.top',
 '232203.apad.top': 'apad.top',
 '282106.apad.top': 'apad.top',
 '062108.apad.top': 'apad.top',
 '312205.apad.top': 'apad.top',
 '282103.apad.top': 'apad.top',
 '032107.apad.top': 'apad.top',
 '152102.apad.top': 'apad.top',
 '212204.apad.top': 'apad.top',
 '032108.apad.top': 'apad.top',
 '302107.apad.top': 'apad.top',
 '012107.apad.top': 'apad.top',
 '292106.apad.top': 'apad.top',
 '182107.apad.top': 'apad.top',
    'gmstar.ru': 'gmstar.ru',
 'board.gmstar.ru': 'gmstar.ru',
 'krasnodar.gmstar.ru': 'gmstar.ru',
 'kaliningrad.gmstar.ru': 'gmstar.ru',
 'kaluga.gmstar.ru': 'gmstar.ru',
 'vologda.gmstar.ru': 'gmstar.ru',
 'orel.gmstar.ru': 'gmstar.ru',
 'moskva.gmstar.ru': 'gmstar.ru',
 'chelyaabinsk.gmstar.ru': 'gmstar.ru',
 'samara.gmstar.ru': 'gmstar.ru',
 'irkutsk.gmstar.ru': 'gmstar.ru',
 'kazan.gmstar.ru': 'gmstar.ru',
 'kostroma.gmstar.ru': 'gmstar.ru',
 'vladimir.gmstar.ru': 'gmstar.ru',
 'vladivostok.gmstar.ru': 'gmstar.ru',
 'saratov.gmstar.ru': 'gmstar.ru',
 'velikiy-novgorod.gmstar.ru': 'gmstar.ru',
 'voronezh.gmstar.ru': 'gmstar.ru',
 'belgorod.gmstar.ru': 'gmstar.ru',
 'novosibirsk.gmstar.ru': 'gmstar.ru',
 'kursk.gmstar.ru': 'gmstar.ru',
 'nizhniy-novgorod.gmstar.ru': 'gmstar.ru',
 'kirov.gmstar.ru': 'gmstar.ru',
 'murmansk.gmstar.ru': 'gmstar.ru',
 'orenburg.gmstar.ru': 'gmstar.ru',
 'penza.gmstar.ru': 'gmstar.ru',
 'barnaul.gmstar.ru': 'gmstar.ru',
 'ekaterinburg.gmstar.ru': 'gmstar.ru',
 'lipetsk.gmstar.ru': 'gmstar.ru',
 'krasnoyarsk.gmstar.ru': 'gmstar.ru',
 'sankt-peterburg.gmstar.ru': 'gmstar.ru',
 'volgograd.gmstar.ru': 'gmstar.ru',
 'chita.gmstar.ru': 'gmstar.ru',
 'ulyanovsk.gmstar.ru': 'gmstar.ru',
 'petropavlovsk-kamchatskiy.gmstar.ru': 'gmstar.ru',
 'tomsk.gmstar.ru': 'gmstar.ru',
 'syktyvkar.gmstar.ru': 'gmstar.ru',
 'tula.gmstar.ru': 'gmstar.ru',
 'tver.gmstar.ru': 'gmstar.ru',
 'yaroslavl.gmstar.ru': 'gmstar.ru',
 'ufa.gmstar.ru': 'gmstar.ru',
 'perm.gmstar.ru': 'gmstar.ru',
 'petrozavodsk.gmstar.ru': 'gmstar.ru',
 'habarovsk.gmstar.ru': 'gmstar.ru',
 'rostov-na-donu.gmstar.ru': 'gmstar.ru',
 'smolensk.gmstar.ru': 'gmstar.ru',
 'sahalin.gmstar.ru': 'gmstar.ru',
 'saransk.gmstar.ru': 'gmstar.ru',
 'hanty-mansiysk.gmstar.ru': 'gmstar.ru',
 'tyumen.gmstar.ru': 'gmstar.ru',
 'pavlovskiy-posad.gmstar.ru': 'gmstar.ru',
 'aprelevka.gmstar.ru': 'gmstar.ru',
 'dmitrov.gmstar.ru': 'gmstar.ru',
 'lyubertsy.gmstar.ru': 'gmstar.ru',
 'naro-fominsk.gmstar.ru': 'gmstar.ru',
 'nahabino.gmstar.ru': 'gmstar.ru',
 'odintsovo.gmstar.ru': 'gmstar.ru',
 'podolsk.gmstar.ru': 'gmstar.ru',
 'dedovsk.gmstar.ru': 'gmstar.ru',
 'ivanteevka.gmstar.ru': 'gmstar.ru',
 'klin.gmstar.ru': 'gmstar.ru',
 'voskresensk.gmstar.ru': 'gmstar.ru',
 'domodedovo.gmstar.ru': 'gmstar.ru',
 'korolyov.gmstar.ru': 'gmstar.ru',
 'kolomna.gmstar.ru': 'gmstar.ru',
 'noginsk.gmstar.ru': 'gmstar.ru',
 'taganrog.gmstar.ru': 'gmstar.ru',
 'dzerzhinsk.gmstar.ru': 'gmstar.ru',
 'himki.gmstar.ru': 'gmstar.ru',
 'sochi.gmstar.ru': 'gmstar.ru',
 'izhniy-tagil.gmstar.ru': 'gmstar.ru',
 'kz.gmstar.ru': 'gmstar.ru',
    'seversk.spravkus.com': 'spravkus.com',
 'spravkus.com': 'spravkus.com',
 'neftekamsk.spravkus.com': 'spravkus.com',
 'serpukhov.spravkus.com': 'spravkus.com',
 'spb.spravkus.com': 'spravkus.com',
 'orenburg.spravkus.com': 'spravkus.com',
 'saratov.spravkus.com': 'spravkus.com',
 'nalchik.spravkus.com': 'spravkus.com',
 'velluki.spravkus.com': 'spravkus.com',
 'domodedovo.spravkus.com': 'spravkus.com',
 'lipetsk.spravkus.com': 'spravkus.com',
 'surgut.spravkus.com': 'spravkus.com',
 'stavropol.spravkus.com': 'spravkus.com',
 'kazan.spravkus.com': 'spravkus.com',
 'kirov.spravkus.com': 'spravkus.com',
 'perm.spravkus.com': 'spravkus.com',
 'balashikha.spravkus.com': 'spravkus.com',
 'grozny.spravkus.com': 'spravkus.com',
 'electrostal.spravkus.com': 'spravkus.com',
 'moskva.spravkus.com': 'spravkus.com',
 'rostov-nd.spravkus.com': 'spravkus.com',
 'volgograd.spravkus.com': 'spravkus.com',
 'murmansk.spravkus.com': 'spravkus.com',
 'pskov.spravkus.com': 'spravkus.com',
 'anapa.spravkus.com': 'spravkus.com',
 'krasnodar.spravkus.com': 'spravkus.com',
 'novosibirsk.spravkus.com': 'spravkus.com',
 'vidnoe.spravkus.com': 'spravkus.com',
 'sterlitamak.spravkus.com': 'spravkus.com',
 'gatchina.spravkus.com': 'spravkus.com',
 'almetevsk.spravkus.com': 'spravkus.com',
 'vladikavkaz.spravkus.com': 'spravkus.com',
 'armavir.spravkus.com': 'spravkus.com',
 'kirovochep.spravkus.com': 'spravkus.com',
 'petrozavodsk.spravkus.com': 'spravkus.com',
 'gorno-altaysk.spravkus.com': 'spravkus.com',
 'nnovgorod.spravkus.com': 'spravkus.com',
 'smolensk.spravkus.com': 'spravkus.com',
 'chelyabinsk.spravkus.com': 'spravkus.com',
 'krasnoyarsk.spravkus.com': 'spravkus.com',
 'taganrog.spravkus.com': 'spravkus.com',
 'nakhodka.spravkus.com': 'spravkus.com',
 'khabarovsk.spravkus.com': 'spravkus.com',
 'ntagil.spravkus.com': 'spravkus.com',
 'kurgan.spravkus.com': 'spravkus.com',
 'orsk.spravkus.com': 'spravkus.com',
 'nabchelny.spravkus.com': 'spravkus.com',
 'sochi.spravkus.com': 'spravkus.com',
 'solikamsk.spravkus.com': 'spravkus.com',
 'ishim.spravkus.com': 'spravkus.com',
 'elabuga.spravkus.com': 'spravkus.com',
 'ulyanovsk.spravkus.com': 'spravkus.com',
 'ufa.spravkus.com': 'spravkus.com',
 'syzran.spravkus.com': 'spravkus.com',
 'mezhdurechensk.spravkus.com': 'spravkus.com',
 'donetsk.spravkus.com': 'spravkus.com',
 'kaluga.spravkus.com': 'spravkus.com',
 'tomsk.spravkus.com': 'spravkus.com',
 'chekhov.spravkus.com': 'spravkus.com',
 'tyumen.spravkus.com': 'spravkus.com',
 'izhevsk.spravkus.com': 'spravkus.com',
 'koms-amur.spravkus.com': 'spravkus.com',
 'dimitrovgrad.spravkus.com': 'spravkus.com',
 'nizhnekamsk.spravkus.com': 'spravkus.com',
 'pereslzal.spravkus.com': 'spravkus.com',
 'samara.spravkus.com': 'spravkus.com',
 'barnaul.spravkus.com': 'spravkus.com',
 'volgodonsk.spravkus.com': 'spravkus.com',
 'eburg.spravkus.com': 'spravkus.com',
 'rubtsovsk.spravkus.com': 'spravkus.com',
 'kolomna.spravkus.com': 'spravkus.com',
 'bryansk.spravkus.com': 'spravkus.com',
    'e-burg.ayle.ru': 'ayle.ru',
 'udm.ayle.ru': 'ayle.ru',
 'spb.ayle.ru': 'ayle.ru',
 'kirov.ayle.ru': 'ayle.ru',
 'khakassia.ayle.ru': 'ayle.ru',
 'kurgan.ayle.ru': 'ayle.ru',
 'perm.ayle.ru': 'ayle.ru',
 'ryazan.ayle.ru': 'ayle.ru',
 'tambov.ayle.ru': 'ayle.ru',
 'kras.ayle.ru': 'ayle.ru',
 'rnd.ayle.ru': 'ayle.ru',
 'vladikavkaz.ayle.ru': 'ayle.ru',
 'orenburg.ayle.ru': 'ayle.ru',
 'kaluga.ayle.ru': 'ayle.ru',
 'chel.ayle.ru': 'ayle.ru',
 'bash.ayle.ru': 'ayle.ru',
 'koenig.ayle.ru': 'ayle.ru',
 'samara.ayle.ru': 'ayle.ru',
 'kemerovo.ayle.ru': 'ayle.ru',
 'oryol.ayle.ru': 'ayle.ru',
 'astrakhan.ayle.ru': 'ayle.ru',
 'yaroslavl.ayle.ru': 'ayle.ru',
 'cr.ayle.ru': 'ayle.ru',
 'kuban.ayle.ru': 'ayle.ru',
 'lipetsk.ayle.ru': 'ayle.ru',
 'nsk.ayle.ru': 'ayle.ru',
 'vl.ayle.ru': 'ayle.ru',
 'tver.ayle.ru': 'ayle.ru',
 'ayle.ru': 'ayle.ru',
 'chuvashia.ayle.ru': 'ayle.ru',
 'khv.ayle.ru': 'ayle.ru',
 'smolensk.ayle.ru': 'ayle.ru',
 'murmansk.ayle.ru': 'ayle.ru',
 'kostroma.ayle.ru': 'ayle.ru',
 'nnov.ayle.ru': 'ayle.ru',
 'saratov.ayle.ru': 'ayle.ru',
 'vladimir.ayle.ru': 'ayle.ru',
 'omsk.ayle.ru': 'ayle.ru',
 'tsk.ayle.ru': 'ayle.ru',
 'penza.ayle.ru': 'ayle.ru',
 'ivanovo.ayle.ru': 'ayle.ru',
 'kazan.ayle.ru': 'ayle.ru',
 'kalmykia.ayle.ru': 'ayle.ru',
 'komi.ayle.ru': 'ayle.ru',
 'amur.ayle.ru': 'ayle.ru',
 'adygeya.ayle.ru': 'ayle.ru',
 'stv.ayle.ru': 'ayle.ru',
 'altai.ayle.ru': 'ayle.ru',
 'tula.ayle.ru': 'ayle.ru',
 'kursk.ayle.ru': 'ayle.ru',
 'nalchik.ayle.ru': 'ayle.ru',
 'pskov.ayle.ru': 'ayle.ru',
 'dagestan.ayle.ru': 'ayle.ru',
 'tyumen.ayle.ru': 'ayle.ru',
 'bryansk.ayle.ru': 'ayle.ru',
 'sakhalin.ayle.ru': 'ayle.ru',
 'nov.ayle.ru': 'ayle.ru',
 'belgorod.ayle.ru': 'ayle.ru',
 'vlg.ayle.ru': 'ayle.ru',
 'mari.ayle.ru': 'ayle.ru',
 'vrn.ayle.ru': 'ayle.ru',
 'magadan.ayle.ru': 'ayle.ru',
 'ptz.ayle.ru': 'ayle.ru',
 'bir.ayle.ru': 'ayle.ru',
 'irk.ayle.ru': 'ayle.ru',
 'yamal.ayle.ru': 'ayle.ru',
 'yakutia.ayle.ru': 'ayle.ru',
 'go.ayle.ru': 'ayle.ru',
 'arkhangelsk.ayle.ru': 'ayle.ru',
 'vologda.ayle.ru': 'ayle.ru',
 'simbirsk.ayle.ru': 'ayle.ru',
 'kamchatka.ayle.ru': 'ayle.ru',
 'mordovia.ayle.ru': 'ayle.ru',
    'kurgan.moyaspravka.ru': 'moyaspravka.ru',
 'schelkovo.moyaspravka.ru': 'moyaspravka.ru',
 'moscow.moyaspravka.ru': 'moyaspravka.ru',
 'tuapse.moyaspravka.ru': 'moyaspravka.ru',
 'naberezhnie-chelny.moyaspravka.ru': 'moyaspravka.ru',
 'chelyabinsk.moyaspravka.ru': 'moyaspravka.ru',
 'krasnodar.moyaspravka.ru': 'moyaspravka.ru',
 'saratov.moyaspravka.ru': 'moyaspravka.ru',
 'kugesi.moyaspravka.ru': 'moyaspravka.ru',
 'nizhniy-novgorod.moyaspravka.ru': 'moyaspravka.ru',
 'tolyatti.moyaspravka.ru': 'moyaspravka.ru',
 'petrozavodsk.moyaspravka.ru': 'moyaspravka.ru',
 'orel.moyaspravka.ru': 'moyaspravka.ru',
 'ufa.moyaspravka.ru': 'moyaspravka.ru',
 'novosibirsk.moyaspravka.ru': 'moyaspravka.ru',
 'nizhniy-tagil.moyaspravka.ru': 'moyaspravka.ru',
 'novotitarovskaya.moyaspravka.ru': 'moyaspravka.ru',
 'bryansk.moyaspravka.ru': 'moyaspravka.ru',
 'vladivostok.moyaspravka.ru': 'moyaspravka.ru',
 'sankt-peterburg.moyaspravka.ru': 'moyaspravka.ru',
 'aksay.moyaspravka.ru': 'moyaspravka.ru',
 'schekino.moyaspravka.ru': 'moyaspravka.ru',
 'kaltan.moyaspravka.ru': 'moyaspravka.ru',
 'rostov.moyaspravka.ru': 'moyaspravka.ru',
 'volgograd.moyaspravka.ru': 'moyaspravka.ru',
 'ivanovo.moyaspravka.ru': 'moyaspravka.ru',
 'nijnekamsk.moyaspravka.ru': 'moyaspravka.ru',
 'krasnoyarsk.moyaspravka.ru': 'moyaspravka.ru',
 'izhevsk.moyaspravka.ru': 'moyaspravka.ru',
 'ulyanovsk.moyaspravka.ru': 'moyaspravka.ru',
 'ekaterinburg.moyaspravka.ru': 'moyaspravka.ru',
 'ryazan.moyaspravka.ru': 'moyaspravka.ru',
 'kursk.moyaspravka.ru': 'moyaspravka.ru',
 'syktyvkar.moyaspravka.ru': 'moyaspravka.ru',
 'novorossiisk.moyaspravka.ru': 'moyaspravka.ru',
 'perm.moyaspravka.ru': 'moyaspravka.ru',
 'gorno-altaysk.moyaspravka.ru': 'moyaspravka.ru',
 'vysokaya-gora.moyaspravka.ru': 'moyaspravka.ru',
 'penza.moyaspravka.ru': 'moyaspravka.ru',
 'vologda.moyaspravka.ru': 'moyaspravka.ru',
 'pskov.moyaspravka.ru': 'moyaspravka.ru',
 'tomsk.moyaspravka.ru': 'moyaspravka.ru',
 'omsk.moyaspravka.ru': 'moyaspravka.ru',
 'himki.moyaspravka.ru': 'moyaspravka.ru',
 'podstepki.moyaspravka.ru': 'moyaspravka.ru',
 'sochi.moyaspravka.ru': 'moyaspravka.ru',
 'belgorod.moyaspravka.ru': 'moyaspravka.ru',
 'kemerovo.moyaspravka.ru': 'moyaspravka.ru',
 'ussuriysk.moyaspravka.ru': 'moyaspravka.ru',
 'cheboksary.moyaspravka.ru': 'moyaspravka.ru',
 'kultaevo.moyaspravka.ru': 'moyaspravka.ru',
 'samara.moyaspravka.ru': 'moyaspravka.ru',
 'khabarovsk.moyaspravka.ru': 'moyaspravka.ru',
 'abakan.moyaspravka.ru': 'moyaspravka.ru',
 'severodvinsk.moyaspravka.ru': 'moyaspravka.ru',
 'leninsk-kuznetsky.moyaspravka.ru': 'moyaspravka.ru',
 'tumen.moyaspravka.ru': 'moyaspravka.ru',
 'tambov.moyaspravka.ru': 'moyaspravka.ru',
 'kansk.moyaspravka.ru': 'moyaspravka.ru',
 'blagoveshchensk.moyaspravka.ru': 'moyaspravka.ru',
 'norilsk.moyaspravka.ru': 'moyaspravka.ru',
 'amursk.moyaspravka.ru': 'moyaspravka.ru',
 'zavodoukovsk.moyaspravka.ru': 'moyaspravka.ru',
 'yuzhno-sakhalinsk.moyaspravka.ru': 'moyaspravka.ru',
 'anapa.moyaspravka.ru': 'moyaspravka.ru',
 'berezovka-24.moyaspravka.ru': 'moyaspravka.ru',
 'barnaul.moyaspravka.ru': 'moyaspravka.ru',
 'kazan.moyaspravka.ru': 'moyaspravka.ru',
 'jeleznodorojnyy-28.moyaspravka.ru': 'moyaspravka.ru',
 'yaroslavl.moyaspravka.ru': 'moyaspravka.ru',
 'yakutsk.moyaspravka.ru': 'moyaspravka.ru',
 'korolev.moyaspravka.ru': 'moyaspravka.ru',
 'novokuznetsk.moyaspravka.ru': 'moyaspravka.ru',
    'specteh-yar.speczakaz.info': 'speczakaz.info',
 'specteh-spb.speczakaz.info': 'speczakaz.info',
 'specteh-alt.speczakaz.info': 'speczakaz.info',
 'specteh-stav.speczakaz.info': 'speczakaz.info',
 'specteh-vrn.speczakaz.info': 'speczakaz.info',
 'specteh-sam.speczakaz.info': 'speczakaz.info',
 'specteh-ivanovo.speczakaz.info': 'speczakaz.info',
 'specteh-udmurt.speczakaz.info': 'speczakaz.info',
 'specteh-tver.speczakaz.info': 'speczakaz.info',
 'specteh-hmao.speczakaz.info': 'speczakaz.info',
 'specteh-sakhalin.speczakaz.info': 'speczakaz.info',
 'specteh-khas.speczakaz.info': 'speczakaz.info',
 'specteh-smol.speczakaz.info': 'speczakaz.info',
 'specteh-ykt.speczakaz.info': 'speczakaz.info',
 'specteh-alania.speczakaz.info': 'speczakaz.info',
 'specteh-krs.speczakaz.info': 'speczakaz.info',
 'specteh-rostov.speczakaz.info': 'speczakaz.info',
 'specteh-tula.speczakaz.info': 'speczakaz.info',
 'specteh-novgorod.speczakaz.info': 'speczakaz.info',
 'specteh-svr.speczakaz.info': 'speczakaz.info',
 'specteh-bash.speczakaz.info': 'speczakaz.info',
 'specteh-tat.speczakaz.info': 'speczakaz.info',
 'specteh-tomsk.speczakaz.info': 'speczakaz.info',
 'specteh-khab.speczakaz.info': 'speczakaz.info',
 'uslugi-tyumen.speczakaz.info': 'speczakaz.info',
 'specteh-tuva.speczakaz.info': 'speczakaz.info',
 'specteh-kaluga.speczakaz.info': 'speczakaz.info',
 'specteh-prim.speczakaz.info': 'speczakaz.info',
 'specteh-volgogr.speczakaz.info': 'speczakaz.info',
 'specteh-perm.speczakaz.info': 'speczakaz.info',
 'specteh-krasnodar.speczakaz.info': 'speczakaz.info',
 'specteh-chuvash.speczakaz.info': 'speczakaz.info',
 'uslugi-alania.speczakaz.info': 'speczakaz.info',
 'specteh-kchr.speczakaz.info': 'speczakaz.info',
 'specteh-kbr.speczakaz.info': 'speczakaz.info',
 'specteh-vologda.speczakaz.info': 'speczakaz.info',
 'specteh-kaliningrad.speczakaz.info': 'speczakaz.info',
 'specteh-chel.speczakaz.info': 'speczakaz.info',
 'specteh-msk.speczakaz.info': 'speczakaz.info',
 'specteh-magadan.speczakaz.info': 'speczakaz.info',
 'specteh-dag.speczakaz.info': 'speczakaz.info',
 'specteh-kurg.speczakaz.info': 'speczakaz.info',
 'specteh-kostr.speczakaz.info': 'speczakaz.info',
 'specteh-yanao.speczakaz.info': 'speczakaz.info',
 'specteh-kamchat.speczakaz.info': 'speczakaz.info',
 'specteh-karelia.speczakaz.info': 'speczakaz.info',
 'specteh-kursk.speczakaz.info': 'speczakaz.info',
 'specteh-nsk.speczakaz.info': 'speczakaz.info',
 'specteh-kirov.speczakaz.info': 'speczakaz.info',
 'specteh-rzn.speczakaz.info': 'speczakaz.info',
 'specteh-vladimir.speczakaz.info': 'speczakaz.info',
 'specteh-murman.speczakaz.info': 'speczakaz.info',
 'specteh-pskov.speczakaz.info': 'speczakaz.info',
 'specteh-kem.speczakaz.info': 'speczakaz.info',
 'specteh-tambov.speczakaz.info': 'speczakaz.info',
 'specteh-orb.speczakaz.info': 'speczakaz.info',
 'specteh-omsk.speczakaz.info': 'speczakaz.info',
 'specteh-bel.speczakaz.info': 'speczakaz.info',
 'specteh-tyumen.speczakaz.info': 'speczakaz.info',
 'specteh-nnov.speczakaz.info': 'speczakaz.info',
 'specteh-crimea.speczakaz.info': 'speczakaz.info',
 'specteh-amur.speczakaz.info': 'speczakaz.info',
 'uslugi-krs.speczakaz.info': 'speczakaz.info',
 'specteh-irk.speczakaz.info': 'speczakaz.info',
 'speczakaz.info': 'speczakaz.info',
 'specteh-komi.speczakaz.info': 'speczakaz.info',
 'specteh-penz.speczakaz.info': 'speczakaz.info',
 'specteh-ul.speczakaz.info': 'speczakaz.info',
 'specteh-lip.speczakaz.info': 'speczakaz.info',
 'specteh-adyg.speczakaz.info': 'speczakaz.info',
 'uslugi-vologda.speczakaz.info': 'speczakaz.info',
 'specteh-mordovia.speczakaz.info': 'speczakaz.info',
 'specteh-astr.speczakaz.info': 'speczakaz.info',
    'compumir.ru': 'compumir.ru',
 'msk.compumir.ru': 'compumir.ru',
 'smolensk.compumir.ru': 'compumir.ru',
 'omsk.compumir.ru': 'compumir.ru',
 'habarovsk.compumir.ru': 'compumir.ru',
 'nn.compumir.ru': 'compumir.ru',
 'rostov-na-donu.compumir.ru': 'compumir.ru',
 'perm.compumir.ru': 'compumir.ru',
 'kr.compumir.ru': 'compumir.ru',
 'orenburg.compumir.ru': 'compumir.ru',
 'yaroslavl.compumir.ru': 'compumir.ru',
 'tyumen.compumir.ru': 'compumir.ru',
 'kaliningrad.compumir.ru': 'compumir.ru',
 'nizhniy-tagil.compumir.ru': 'compumir.ru',
 'ekb.compumir.ru': 'compumir.ru',
 'sevastopol.compumir.ru': 'compumir.ru',
 'kazan.compumir.ru': 'compumir.ru',
 'sochi.compumir.ru': 'compumir.ru',
 'nsk.compumir.ru': 'compumir.ru',
 'ivanovo.compumir.ru': 'compumir.ru',
 'surgut.compumir.ru': 'compumir.ru',
 'kemerovo.compumir.ru': 'compumir.ru',
 'lipeck.compumir.ru': 'compumir.ru',
 'elektrostal.compumir.ru': 'compumir.ru',
 'naberezhnye-chelny.compumir.ru': 'compumir.ru',
 'chelyabinsk.compumir.ru': 'compumir.ru',
 'ufa.compumir.ru': 'compumir.ru',
 'vladimir.compumir.ru': 'compumir.ru',
 'tver.compumir.ru': 'compumir.ru',
 'anapa.compumir.ru': 'compumir.ru',
 'krasnoyarsk.compumir.ru': 'compumir.ru',
 'barnaul.compumir.ru': 'compumir.ru',
 'izhevsk.compumir.ru': 'compumir.ru',
 'kurgan.compumir.ru': 'compumir.ru',
 'saratov.compumir.ru': 'compumir.ru',
 'stavropol.compumir.ru': 'compumir.ru',
 'saransk.compumir.ru': 'compumir.ru',
 'astrahan.compumir.ru': 'compumir.ru',
 'ryazan.compumir.ru': 'compumir.ru',
 'kursk.compumir.ru': 'compumir.ru',
 'podolsk.compumir.ru': 'compumir.ru',
 'tula.compumir.ru': 'compumir.ru',
 'tomsk.compumir.ru': 'compumir.ru',
 'syktyvkar.compumir.ru': 'compumir.ru',
 'odincovo.compumir.ru': 'compumir.ru',
 'arhangelsk.compumir.ru': 'compumir.ru',
 'volgograd.compumir.ru': 'compumir.ru',
 'voronezh.compumir.ru': 'compumir.ru',
 'chehov.compumir.ru': 'compumir.ru',
 'engels.compumir.ru': 'compumir.ru',
 'taganrog.compumir.ru': 'compumir.ru',
 'vologda.compumir.ru': 'compumir.ru',
 'belgorod.compumir.ru': 'compumir.ru',
 'magnitogorsk.compumir.ru': 'compumir.ru',
 'ramenskoe.compumir.ru': 'compumir.ru',
 'murmansk.compumir.ru': 'compumir.ru',
 'krasnogorsk.compumir.ru': 'compumir.ru',
 'obninsk.compumir.ru': 'compumir.ru',
 'kostroma.compumir.ru': 'compumir.ru',
 'ulyanovsk.compumir.ru': 'compumir.ru',
 'armavir.compumir.ru': 'compumir.ru',
 'evpatoriya.compumir.ru': 'compumir.ru',
 'kolomna.compumir.ru': 'compumir.ru',
 'feodosiya.compumir.ru': 'compumir.ru',
 'dmitrov.compumir.ru': 'compumir.ru',
 'simferopol.compumir.ru': 'compumir.ru',
 'irkutsk.compumir.ru': 'compumir.ru',
 'kstovo.compumir.ru': 'compumir.ru',
 'lobnya.compumir.ru': 'compumir.ru',
 'orsk.compumir.ru': 'compumir.ru',
 'gatchina.compumir.ru': 'compumir.ru',
 'kamensk-uralskiy.compumir.ru': 'compumir.ru',
 'klin.compumir.ru': 'compumir.ru',
 'yalta.compumir.ru': 'compumir.ru',
    'chelyabinsk.spravkarf.ru': 'spravkarf.ru',
 'tagil.spravkarf.ru': 'spravkarf.ru',
 'moscow.spravkarf.ru': 'spravkarf.ru',
 'kronstadt.spravkarf.ru': 'spravkarf.ru',
 'volhov.spravkarf.ru': 'spravkarf.ru',
 'satka.spravkarf.ru': 'spravkarf.ru',
 'pyt-yah.spravkarf.ru': 'spravkarf.ru',
 'bryansk.spravkarf.ru': 'spravkarf.ru',
 'krasnoyarsk.spravkarf.ru': 'spravkarf.ru',
 'ulyanovsk.spravkarf.ru': 'spravkarf.ru',
 'zelenograd.spravkarf.ru': 'spravkarf.ru',
 'prokopevsk.spravkarf.ru': 'spravkarf.ru',
 'sankt-peterburg.spravkarf.ru': 'spravkarf.ru',
 'khabarovsk.spravkarf.ru': 'spravkarf.ru',
 'vuktyl.spravkarf.ru': 'spravkarf.ru',
 'bogdanovich.spravkarf.ru': 'spravkarf.ru',
 'krasnodar.spravkarf.ru': 'spravkarf.ru',
 'tihoretsk.spravkarf.ru': 'spravkarf.ru',
 'dzerzhinsky.spravkarf.ru': 'spravkarf.ru',
 'asha.spravkarf.ru': 'spravkarf.ru',
 'pervouralsk.spravkarf.ru': 'spravkarf.ru',
 'ozersk.spravkarf.ru': 'spravkarf.ru',
 'shushenskoe.spravkarf.ru': 'spravkarf.ru',
 'novyi-urengoi.spravkarf.ru': 'spravkarf.ru',
 'usinsk.spravkarf.ru': 'spravkarf.ru',
 'sysert.spravkarf.ru': 'spravkarf.ru',
 'krasnoufimsk.spravkarf.ru': 'spravkarf.ru',
 'kazan.spravkarf.ru': 'spravkarf.ru',
 'partizansk.spravkarf.ru': 'spravkarf.ru',
 'krestsy.spravkarf.ru': 'spravkarf.ru',
 'nizhnekamsk.spravkarf.ru': 'spravkarf.ru',
 'uhta.spravkarf.ru': 'spravkarf.ru',
 'samara.spravkarf.ru': 'spravkarf.ru',
 'kolomna.spravkarf.ru': 'spravkarf.ru',
 'velikie-luky.spravkarf.ru': 'spravkarf.ru',
 'novosibirsk.spravkarf.ru': 'spravkarf.ru',
 'tumen.spravkarf.ru': 'spravkarf.ru',
 'magnitogorsk.spravkarf.ru': 'spravkarf.ru',
 'troick.spravkarf.ru': 'spravkarf.ru',
 'barabinsk.spravkarf.ru': 'spravkarf.ru',
 'revda.spravkarf.ru': 'spravkarf.ru',
 'zarechny.spravkarf.ru': 'spravkarf.ru',
 'ufa.spravkarf.ru': 'spravkarf.ru',
 'novouralsk.spravkarf.ru': 'spravkarf.ru',
 'alapaevsk.spravkarf.ru': 'spravkarf.ru',
 'tara.spravkarf.ru': 'spravkarf.ru',
 'klin.spravkarf.ru': 'spravkarf.ru',
 'leninogorsk.spravkarf.ru': 'spravkarf.ru',
 'pechora.spravkarf.ru': 'spravkarf.ru',
 'kemerovo.spravkarf.ru': 'spravkarf.ru',
 'ust-katav.spravkarf.ru': 'spravkarf.ru',
 'yuryuzan.spravkarf.ru': 'spravkarf.ru',
 'gorno-altaysk.spravkarf.ru': 'spravkarf.ru',
 'chehov.spravkarf.ru': 'spravkarf.ru',
 'slavyanka.spravkarf.ru': 'spravkarf.ru',
 'sochi.spravkarf.ru': 'spravkarf.ru',
 'tavricheskoe.spravkarf.ru': 'spravkarf.ru',
 'kansk.spravkarf.ru': 'spravkarf.ru',
 'valuyki.spravkarf.ru': 'spravkarf.ru',
 'tobolsk.spravkarf.ru': 'spravkarf.ru',
 'nighnevartovsk.spravkarf.ru': 'spravkarf.ru',
 'novorossiysk.spravkarf.ru': 'spravkarf.ru',
 'mytischi.spravkarf.ru': 'spravkarf.ru',
 'nizhniy-novgorod.spravkarf.ru': 'spravkarf.ru',
 'volgograd.spravkarf.ru': 'spravkarf.ru',
 'yaroslavl.spravkarf.ru': 'spravkarf.ru',
 'serov.spravkarf.ru': 'spravkarf.ru',
 'petrozavodsk.spravkarf.ru': 'spravkarf.ru',
 'ishim.spravkarf.ru': 'spravkarf.ru',
 'rezh.spravkarf.ru': 'spravkarf.ru',
 'ekaterinburg.spravkarf.ru': 'spravkarf.ru',
 'nefteyugansk.spravkarf.ru': 'spravkarf.ru',
 'strezgevoy.spravkarf.ru': 'spravkarf.ru',
 'kamyshlov.spravkarf.ru': 'spravkarf.ru',
 'ordinskoe.spravkarf.ru': 'spravkarf.ru',
 'kuybyshev.spravkarf.ru': 'spravkarf.ru',
 'mozhaysk.spravkarf.ru': 'spravkarf.ru',
    'ru.taxigator.ru': 'taxigator.ru',
 'taxigator.ru': 'taxigator.ru',
 'taxi-yekaterinburg-sverdlovsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-sochi-krasnodar-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-biysk-altai-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-omsk-omsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-barnaul-altai-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-domodedovo-moscow-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-kiselyovsk-kemerovo-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-svetlogorsk-kaliningrad-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-kaliningrad-kaliningrad-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-moscow-ru.taxigator.ru': 'taxigator.ru',
 'taxi-khabarovsk-khabarovsk-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-orenburg-orenburg-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-orsk-orenburg-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-vnukovo-moscow-ru.taxigator.ru': 'taxigator.ru',
 'taxi-balashikha-moscow-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-st-petersburg-ru.taxigator.ru': 'taxigator.ru',
 'taxi-tula-tula-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-vyborg-leningrad-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-krasnoyarsk-krasnoyarsk-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-kazan-tatarstan-ru.taxigator.ru': 'taxigator.ru',
 'taxi-penza-penza-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-novosibirsk-novosibirsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-novorossiysk-krasnodar-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-vladimir-vladimir-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-anapa-krasnodar-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-adler-krasnodar-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-cheboksary-chuvashia-republic-ru.taxigator.ru': 'taxigator.ru',
 'taxi-strelna-saint-petersburg-ru.taxigator.ru': 'taxigator.ru',
 'taxi-tolyatti-samara-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-syzran-samara-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-samara-samara-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-perm-perm-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-adlerskiy-krasnodarskiy-kray-ru.taxigator.ru': 'taxigator.ru',
 'taxi-blagoveshchensk-amur-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-kovrov-vladimir-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-tver-tver-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-baltiysk-kaliningrad-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-syktyvkar-komi-republic-ru.taxigator.ru': 'taxigator.ru',
 'taxi-bolshoy-sochi-krasnodar-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-ulyanovsk-ulyanovsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-nizhny-novgorod-nizhny-novgorod-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-ruskeala-republic-of-karelia-ru.taxigator.ru': 'taxigator.ru',
 'taxi-nizhneimeretinskaya-bukhta-krasnodar-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-bratsk-irkutsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-belgorod-belgorod-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-ryazan-ryazan-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-null.taxigator.ru': 'taxigator.ru',
 'taxi-kemerovo-kemerovo-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-verkh-tula-novosibirskaya-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-ob-novosibirsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-vladikavkaz-north-ossetiaalania-ru.taxigator.ru': 'taxigator.ru',
 'taxi-volgograd-volgograd-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-veliky-ustyug-vologda-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-veliky-novgorod-novgorod-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-vladivostok-primorsky-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-bor-nizhny-novgorod-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-nakhodka-primorsky-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-knevichi-primorsky-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-voronezh-voronezh-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-oryol-oryol-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-rostov-on-don-rostov-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-pskov-pskov-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-minsk-by.taxigator.ru': 'taxigator.ru',
 'taxi-kirov-kirov-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-lyamtsino-moskovskaya-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-petropavl-kz.taxigator.ru': 'taxigator.ru',
 'taxi-kamensk-shakhtinsky-rostov-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-khimki-moscow-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-yaroslavl-yaroslavl-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-kostroma-kostroma-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-nevinnomyssk-stavropol-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-irkutsk-irkutsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-arkhangelsk-arkhangelsk-oblast-ru.taxigator.ru': 'taxigator.ru',
 'taxi-mineralnye-vody-stavropol-krai-ru.taxigator.ru': 'taxigator.ru',
 'taxi-rzhev-tver-oblast-ru.taxigator.ru': 'taxigator.ru',
    'nalchik.orgsprav.com': 'orgsprav.com',
 'strejevoy.orgsprav.com': 'orgsprav.com',
 'korolev.orgsprav.com': 'orgsprav.com',
 'rostov-na-donu.orgsprav.com': 'orgsprav.com',
 'zelenograd.orgsprav.com': 'orgsprav.com',
 'moskva.orgsprav.com': 'orgsprav.com',
 'birsk.orgsprav.com': 'orgsprav.com',
 'neftekamsk.orgsprav.com': 'orgsprav.com',
 'spb.orgsprav.com': 'orgsprav.com',
 'yoshkar-ola.orgsprav.com': 'orgsprav.com',
 'blagoveschensk.orgsprav.com': 'orgsprav.com',
 'orgsprav.com': 'orgsprav.com',
 'roslavl.orgsprav.com': 'orgsprav.com',
 'voskresensk.orgsprav.com': 'orgsprav.com',
 'klincy.orgsprav.com': 'orgsprav.com',
 'smolensk.orgsprav.com': 'orgsprav.com',
 'nikolaevsk-na-amure.orgsprav.com': 'orgsprav.com',
 'belorechensk.orgsprav.com': 'orgsprav.com',
 'kropotkin.orgsprav.com': 'orgsprav.com',
 'ufa.orgsprav.com': 'orgsprav.com',
 'rasskazovo.orgsprav.com': 'orgsprav.com',
 'tambov.orgsprav.com': 'orgsprav.com',
 'krasnodar.orgsprav.com': 'orgsprav.com',
 'alatyr.orgsprav.com': 'orgsprav.com',
 'shahty.orgsprav.com': 'orgsprav.com',
 'kurgan.orgsprav.com': 'orgsprav.com',
 'magnitogorsk.orgsprav.com': 'orgsprav.com',
 'verhnyaya-pyshma.orgsprav.com': 'orgsprav.com',
 'ishim.orgsprav.com': 'orgsprav.com',
 'novorossiysk.orgsprav.com': 'orgsprav.com',
 'lipeck.orgsprav.com': 'orgsprav.com',
 'rjev.orgsprav.com': 'orgsprav.com',
 'obninsk.orgsprav.com': 'orgsprav.com',
 'habarovsk.orgsprav.com': 'orgsprav.com',
 'alapaevsk.orgsprav.com': 'orgsprav.com',
 'yugorsk.orgsprav.com': 'orgsprav.com',
 'kansk.orgsprav.com': 'orgsprav.com',
 'kineshma.orgsprav.com': 'orgsprav.com',
 'ivanovo.orgsprav.com': 'orgsprav.com',
 'velikie-luki.orgsprav.com': 'orgsprav.com',
 'anapa.orgsprav.com': 'orgsprav.com',
 'tuapse.orgsprav.com': 'orgsprav.com',
 'sevastopol.orgsprav.com': 'orgsprav.com',
 'chelyabinsk.orgsprav.com': 'orgsprav.com',
 'sarapul.orgsprav.com': 'orgsprav.com',
 'vladikavkaz.orgsprav.com': 'orgsprav.com',
 'odincovo.orgsprav.com': 'orgsprav.com',
 'tolyatti.orgsprav.com': 'orgsprav.com',
 'petrovsk.orgsprav.com': 'orgsprav.com',
 'gukovo.orgsprav.com': 'orgsprav.com',
 'omutninsk.orgsprav.com': 'orgsprav.com',
 'nevyansk.orgsprav.com': 'orgsprav.com',
 'abinsk.orgsprav.com': 'orgsprav.com',
 'shuya.orgsprav.com': 'orgsprav.com',
 'kirov.orgsprav.com': 'orgsprav.com',
 'dolgoprudnyy.orgsprav.com': 'orgsprav.com',
 'murmansk.orgsprav.com': 'orgsprav.com',
 'vladivostok.orgsprav.com': 'orgsprav.com',
 'murom.orgsprav.com': 'orgsprav.com',
 'ijevsk.orgsprav.com': 'orgsprav.com',
 'kungur.orgsprav.com': 'orgsprav.com',
 'korenovsk.orgsprav.com': 'orgsprav.com',
 'abakan.orgsprav.com': 'orgsprav.com',
 'semey.orgsprav.com': 'orgsprav.com',
 'kazan.orgsprav.com': 'orgsprav.com',
 'rybinsk.orgsprav.com': 'orgsprav.com',
 'stavropol.orgsprav.com': 'orgsprav.com',
 'nefteyugansk.orgsprav.com': 'orgsprav.com',
 'korkino.orgsprav.com': 'orgsprav.com',
 'maykop.orgsprav.com': 'orgsprav.com',
 'buzuluk.orgsprav.com': 'orgsprav.com',
 'yanaul.orgsprav.com': 'orgsprav.com',
 'morozovsk.orgsprav.com': 'orgsprav.com',
 'zelenogorsk.orgsprav.com': 'orgsprav.com',
 'krasnoyarsk.orgsprav.com': 'orgsprav.com',
 'minsk.orgsprav.com': 'orgsprav.com',
 'komsomolsk-na-amure.orgsprav.com': 'orgsprav.com',
    'sankt-peterburg.stroyportal.ru': 'stroyportal.ru',
 'kaluga.stroyportal.ru': 'stroyportal.ru',
 'tula.stroyportal.ru': 'stroyportal.ru',
 'smolensk.stroyportal.ru': 'stroyportal.ru',
 'blagoveshensk.stroyportal.ru': 'stroyportal.ru',
 'ufa.stroyportal.ru': 'stroyportal.ru',
 'novosibirsk.stroyportal.ru': 'stroyportal.ru',
 'voronezh.stroyportal.ru': 'stroyportal.ru',
 'kurgan.stroyportal.ru': 'stroyportal.ru',
 'krasnoyarsk.stroyportal.ru': 'stroyportal.ru',
 'nizhniy-novgorod.stroyportal.ru': 'stroyportal.ru',
 'birobidzhan.stroyportal.ru': 'stroyportal.ru',
 'kirov.stroyportal.ru': 'stroyportal.ru',
 'rostov-na-donu.stroyportal.ru': 'stroyportal.ru',
 'tver.stroyportal.ru': 'stroyportal.ru',
 'orenburg.stroyportal.ru': 'stroyportal.ru',
 'omsk.stroyportal.ru': 'stroyportal.ru',
 'izhevsk.stroyportal.ru': 'stroyportal.ru',
 'habarovsk.stroyportal.ru': 'stroyportal.ru',
 'ivanovo.stroyportal.ru': 'stroyportal.ru',
 'krasnodar.stroyportal.ru': 'stroyportal.ru',
 'saratov.stroyportal.ru': 'stroyportal.ru',
 'kaliningrad.stroyportal.ru': 'stroyportal.ru',
 'ulyanovsk.stroyportal.ru': 'stroyportal.ru',
 'petropavlovsk-kamchatskiy.stroyportal.ru': 'stroyportal.ru',
 'chelyabinsk.stroyportal.ru': 'stroyportal.ru',
 'volgograd.stroyportal.ru': 'stroyportal.ru',
 'stroyportal.ru': 'stroyportal.ru',
 'yakutsk.stroyportal.ru': 'stroyportal.ru',
 'tyumen.stroyportal.ru': 'stroyportal.ru',
 'yaroslavl.stroyportal.ru': 'stroyportal.ru',
 'gorno-altaysk.stroyportal.ru': 'stroyportal.ru',
 'ryazan.stroyportal.ru': 'stroyportal.ru',
 'pskov.stroyportal.ru': 'stroyportal.ru',
 'bryansk.stroyportal.ru': 'stroyportal.ru',
 'yoshkar-ola.stroyportal.ru': 'stroyportal.ru',
 'lipetsk.stroyportal.ru': 'stroyportal.ru',
 'samara.stroyportal.ru': 'stroyportal.ru',
 'kazan.stroyportal.ru': 'stroyportal.ru',
 'ekaterinburg.stroyportal.ru': 'stroyportal.ru',
 'cheboksary.stroyportal.ru': 'stroyportal.ru',
 'velikiy-novgorod.stroyportal.ru': 'stroyportal.ru',
 'ulan-ude.stroyportal.ru': 'stroyportal.ru',
 'vologda.stroyportal.ru': 'stroyportal.ru',
 'tambov.stroyportal.ru': 'stroyportal.ru',
 'stavropol.stroyportal.ru': 'stroyportal.ru',
 'vladivostok.stroyportal.ru': 'stroyportal.ru',
 'kyzyl.stroyportal.ru': 'stroyportal.ru',
 'murmansk.stroyportal.ru': 'stroyportal.ru',
 'syktyvkar.stroyportal.ru': 'stroyportal.ru',
 'belgorod.stroyportal.ru': 'stroyportal.ru',
 'maykop.stroyportal.ru': 'stroyportal.ru',
 'penza.stroyportal.ru': 'stroyportal.ru',
 'abakan.stroyportal.ru': 'stroyportal.ru',
 'vladimir.stroyportal.ru': 'stroyportal.ru',
 'astrahan.stroyportal.ru': 'stroyportal.ru',
 'tomsk.stroyportal.ru': 'stroyportal.ru',
 'grozny.stroyportal.ru': 'stroyportal.ru',
 'saransk.stroyportal.ru': 'stroyportal.ru',
 'kemerovo.stroyportal.ru': 'stroyportal.ru',
 'perm.stroyportal.ru': 'stroyportal.ru',
 'elista.stroyportal.ru': 'stroyportal.ru',
 'salehard.stroyportal.ru': 'stroyportal.ru',
 'barnaul.stroyportal.ru': 'stroyportal.ru',
 'vladikavkaz.stroyportal.ru': 'stroyportal.ru',
 'kostroma.stroyportal.ru': 'stroyportal.ru',
 'nalchik.stroyportal.ru': 'stroyportal.ru',
 'orel.stroyportal.ru': 'stroyportal.ru',
 'magadan.stroyportal.ru': 'stroyportal.ru',
 'hanty-mansijsk.stroyportal.ru': 'stroyportal.ru',
 'yuzhno-sahalinsk.stroyportal.ru': 'stroyportal.ru',
 'kursk.stroyportal.ru': 'stroyportal.ru',
 'arhangelsk.stroyportal.ru': 'stroyportal.ru',
 'mahachkala.stroyportal.ru': 'stroyportal.ru',
 'chita.stroyportal.ru': 'stroyportal.ru',
 'petrozavodsk.stroyportal.ru': 'stroyportal.ru',
 'cherkessk.stroyportal.ru': 'stroyportal.ru',
 'kz.stroyportal.ru': 'stroyportal.ru',
    'birobidzhan.reginforms.ru': 'reginforms.ru',
 'bratsk.reginforms.ru': 'reginforms.ru',
 'kamensk-uralskiy.reginforms.ru': 'reginforms.ru',
 'almetyevsk.reginforms.ru': 'reginforms.ru',
 'biysk.reginforms.ru': 'reginforms.ru',
 'kaliningrad.reginforms.ru': 'reginforms.ru',
 'ekaterinburg.reginforms.ru': 'reginforms.ru',
 'vladikavkaz.reginforms.ru': 'reginforms.ru',
 'almaty.reginforms.ru': 'reginforms.ru',
 'belgorod.reginforms.ru': 'reginforms.ru',
 'bishkek.reginforms.ru': 'reginforms.ru',
 'barnaul.reginforms.ru': 'reginforms.ru',
 'astrakhan.reginforms.ru': 'reginforms.ru',
 'gomel.reginforms.ru': 'reginforms.ru',
 'kaluga.reginforms.ru': 'reginforms.ru',
 'ivanovo.reginforms.ru': 'reginforms.ru',
 'anadyr.reginforms.ru': 'reginforms.ru',
 'arkhangelsk.reginforms.ru': 'reginforms.ru',
 'grozny.reginforms.ru': 'reginforms.ru',
 'izhevsk.reginforms.ru': 'reginforms.ru',
 'vladivostok.reginforms.ru': 'reginforms.ru',
 'vologda.reginforms.ru': 'reginforms.ru',
 'irkutsk.reginforms.ru': 'reginforms.ru',
 'karaganda.reginforms.ru': 'reginforms.ru',
 'donetsk.reginforms.ru': 'reginforms.ru',
 'volgograd.reginforms.ru': 'reginforms.ru',
 'gorno-altaisk.reginforms.ru': 'reginforms.ru',
 'abakan.reginforms.ru': 'reginforms.ru',
 'voronezh.reginforms.ru': 'reginforms.ru',
 'velikiy-novgorod.reginforms.ru': 'reginforms.ru',
 'joshkar-ola.reginforms.ru': 'reginforms.ru',
 'vladimir.reginforms.ru': 'reginforms.ru',
 'tambov.reginforms.ru': 'reginforms.ru',
 'bryansk.reginforms.ru': 'reginforms.ru',
 'armawir.reginforms.ru': 'reginforms.ru',
 'kazan.reginforms.ru': 'reginforms.ru',
 'reginforms.ru': 'reginforms.ru',
 'yakutsk.reginforms.ru': 'reginforms.ru',
 'makhachkala.reginforms.ru': 'reginforms.ru',
 'novy-urengoy.reginforms.ru': 'reginforms.ru',
 'stavropol.reginforms.ru': 'reginforms.ru',
 'achinsk.reginforms.ru': 'reginforms.ru',
 'petropavlovsk.reginforms.ru': 'reginforms.ru',
 'nizhniynovgorod.reginforms.ru': 'reginforms.ru',
 'samara.reginforms.ru': 'reginforms.ru',
 'murmansk.reginforms.ru': 'reginforms.ru',
 'norilsk.reginforms.ru': 'reginforms.ru',
 'krasnodar.reginforms.ru': 'reginforms.ru',
 'nalchik.reginforms.ru': 'reginforms.ru',
 'ryazan.reginforms.ru': 'reginforms.ru',
 'naberezhnye-chelny.reginforms.ru': 'reginforms.ru',
 'omsk.reginforms.ru': 'reginforms.ru',
 'perm.reginforms.ru': 'reginforms.ru',
 'smolensk.reginforms.ru': 'reginforms.ru',
 'naryan-mar.reginforms.ru': 'reginforms.ru',
 'simferopol.reginforms.ru': 'reginforms.ru',
 'penza.reginforms.ru': 'reginforms.ru',
 'orel.reginforms.ru': 'reginforms.ru',
 'nizhnevartovsk.reginforms.ru': 'reginforms.ru',
 'orehovo-zuevo.reginforms.ru': 'reginforms.ru',
 'saratov.reginforms.ru': 'reginforms.ru',
 'sanktpeterburg.reginforms.ru': 'reginforms.ru',
 'odessa.reginforms.ru': 'reginforms.ru',
 'saransk.reginforms.ru': 'reginforms.ru',
 'noyabrsk.reginforms.ru': 'reginforms.ru',
 'orenburg.reginforms.ru': 'reginforms.ru',
 'petrozavodsk.reginforms.ru': 'reginforms.ru',
 'nahodka.reginforms.ru': 'reginforms.ru',
 'maykop.reginforms.ru': 'reginforms.ru',
 'cheboksary.reginforms.ru': 'reginforms.ru',
 'khabarovsk.reginforms.ru': 'reginforms.ru',
 'moskva.reginforms.ru': 'reginforms.ru',
 'magnitogorsk.reginforms.ru': 'reginforms.ru',
 'novosibirsk.reginforms.ru': 'reginforms.ru',
 'nizhniytagil.reginforms.ru': 'reginforms.ru',
 'novorossiysk.reginforms.ru': 'reginforms.ru',
 'chelyabinsk.reginforms.ru': 'reginforms.ru',
 'yuzhno-sakhalinsk.reginforms.ru': 'reginforms.ru',
 'sochi.reginforms.ru': 'reginforms.ru',
 'obninsk.reginforms.ru': 'reginforms.ru',
    'msk.kupiprodai.ru': 'kupiprodai.ru',
 'kupiprodai.ru': 'kupiprodai.ru',
 'krym.kupiprodai.ru': 'kupiprodai.ru',
 'tyumen.kupiprodai.ru': 'kupiprodai.ru',
 'krasnodar.kupiprodai.ru': 'kupiprodai.ru',
 'tula.kupiprodai.ru': 'kupiprodai.ru',
 'bur.kupiprodai.ru': 'kupiprodai.ru',
 'nsk.kupiprodai.ru': 'kupiprodai.ru',
 'kurgan.kupiprodai.ru': 'kupiprodai.ru',
 'mo.kupiprodai.ru': 'kupiprodai.ru',
 'ufa.kupiprodai.ru': 'kupiprodai.ru',
 'spb.kupiprodai.ru': 'kupiprodai.ru',
 'ek.kupiprodai.ru': 'kupiprodai.ru',
 'sakha.kupiprodai.ru': 'kupiprodai.ru',
 'chel.kupiprodai.ru': 'kupiprodai.ru',
 'lenobl.kupiprodai.ru': 'kupiprodai.ru',
 'yar.kupiprodai.ru': 'kupiprodai.ru',
 'chuvashia.kupiprodai.ru': 'kupiprodai.ru',
 'barnaul.kupiprodai.ru': 'kupiprodai.ru',
 'hmao.kupiprodai.ru': 'kupiprodai.ru',
 'yamal.kupiprodai.ru': 'kupiprodai.ru',
 'krasnoyarsk.kupiprodai.ru': 'kupiprodai.ru',
 'udm.kupiprodai.ru': 'kupiprodai.ru',
 'rostov.kupiprodai.ru': 'kupiprodai.ru',
 'samara.kupiprodai.ru': 'kupiprodai.ru',
 'rzn.kupiprodai.ru': 'kupiprodai.ru',
 'oren.kupiprodai.ru': 'kupiprodai.ru',
 'vol.kupiprodai.ru': 'kupiprodai.ru',
 'tatarstan.kupiprodai.ru': 'kupiprodai.ru',
 'kirov.kupiprodai.ru': 'kupiprodai.ru',
 'khb.kupiprodai.ru': 'kupiprodai.ru',
 'omsk.kupiprodai.ru': 'kupiprodai.ru',
 'murman.kupiprodai.ru': 'kupiprodai.ru',
 'lipetsk.kupiprodai.ru': 'kupiprodai.ru',
 'altai.kupiprodai.ru': 'kupiprodai.ru',
 'blagoveshchensk.kupiprodai.ru': 'kupiprodai.ru',
 'vl.kupiprodai.ru': 'kupiprodai.ru',
 'komi.kupiprodai.ru': 'kupiprodai.ru',
 'tambov.kupiprodai.ru': 'kupiprodai.ru',
 'tomsk.kupiprodai.ru': 'kupiprodai.ru',
 'saratov.kupiprodai.ru': 'kupiprodai.ru',
 'vologda.kupiprodai.ru': 'kupiprodai.ru',
 'irk.kupiprodai.ru': 'kupiprodai.ru',
 'nnov.kupiprodai.ru': 'kupiprodai.ru',
 'pskov.kupiprodai.ru': 'kupiprodai.ru',
 'karelia.kupiprodai.ru': 'kupiprodai.ru',
 'stav.kupiprodai.ru': 'kupiprodai.ru',
 'tver.kupiprodai.ru': 'kupiprodai.ru',
 'penza.kupiprodai.ru': 'kupiprodai.ru',
 'kaliningrad.kupiprodai.ru': 'kupiprodai.ru',
 'perm.kupiprodai.ru': 'kupiprodai.ru',
 'khakas.kupiprodai.ru': 'kupiprodai.ru',
 'bryansk.kupiprodai.ru': 'kupiprodai.ru',
 'voronezh.kupiprodai.ru': 'kupiprodai.ru',
 'smol.kupiprodai.ru': 'kupiprodai.ru',
 'iv.kupiprodai.ru': 'kupiprodai.ru',
 'bel.kupiprodai.ru': 'kupiprodai.ru',
 'kaluga.kupiprodai.ru': 'kupiprodai.ru',
 'nov.kupiprodai.ru': 'kupiprodai.ru',
 'mordovia.kupiprodai.ru': 'kupiprodai.ru',
 'kem.kupiprodai.ru': 'kupiprodai.ru',
 'mari.kupiprodai.ru': 'kupiprodai.ru',
 'kamchatka.kupiprodai.ru': 'kupiprodai.ru',
 'astrakhan.kupiprodai.ru': 'kupiprodai.ru',
 'chita.kupiprodai.ru': 'kupiprodai.ru',
 'ul.kupiprodai.ru': 'kupiprodai.ru',
 'sakhalin.kupiprodai.ru': 'kupiprodai.ru',
 'orel.kupiprodai.ru': 'kupiprodai.ru',
 'chr.kupiprodai.ru': 'kupiprodai.ru',
 'primorsky.kupiprodai.ru': 'kupiprodai.ru',
 'kostroma.kupiprodai.ru': 'kupiprodai.ru',
 'adygheya.kupiprodai.ru': 'kupiprodai.ru',
 'arh.kupiprodai.ru': 'kupiprodai.ru',
 'kursk.kupiprodai.ru': 'kupiprodai.ru',
 'eao.kupiprodai.ru': 'kupiprodai.ru',
 'tuva.kupiprodai.ru': 'kupiprodai.ru',
 'kbr.kupiprodai.ru': 'kupiprodai.ru',
 'magadan.kupiprodai.ru': 'kupiprodai.ru',
 'kalmykia.kupiprodai.ru': 'kupiprodai.ru',
 'osetia.kupiprodai.ru': 'kupiprodai.ru',
    'naydidom.com': 'naydidom.com',
 'novosibirsk.naydidom.com': 'naydidom.com',
 'ufa.naydidom.com': 'naydidom.com',
 'obninsk.naydidom.com': 'naydidom.com',
 'nizhniy-novgorod.naydidom.com': 'naydidom.com',
 'ekaterinburg.naydidom.com': 'naydidom.com',
 'asha.naydidom.com': 'naydidom.com',
 'tyumen.naydidom.com': 'naydidom.com',
 'saratov.naydidom.com': 'naydidom.com',
 'nizhnekamsk.naydidom.com': 'naydidom.com',
 'moskva.naydidom.com': 'naydidom.com',
 'istra.naydidom.com': 'naydidom.com',
 'pervouralsk.naydidom.com': 'naydidom.com',
 'adler.naydidom.com': 'naydidom.com',
 'smolensk.naydidom.com': 'naydidom.com',
 'komsomolsk-na-amure.naydidom.com': 'naydidom.com',
 'penza.naydidom.com': 'naydidom.com',
 'klimovsk.naydidom.com': 'naydidom.com',
 'perm.naydidom.com': 'naydidom.com',
 'krasnodar.naydidom.com': 'naydidom.com',
 'lyubercy.naydidom.com': 'naydidom.com',
 'magadan.naydidom.com': 'naydidom.com',
 'kudrovo2.naydidom.com': 'naydidom.com',
 'evpatorija.naydidom.com': 'naydidom.com',
 'voronezh.naydidom.com': 'naydidom.com',
 'buguruslan.naydidom.com': 'naydidom.com',
 'sochi.naydidom.com': 'naydidom.com',
 'yakutsk.naydidom.com': 'naydidom.com',
 'rostov-na-donu.naydidom.com': 'naydidom.com',
 'taganrog.naydidom.com': 'naydidom.com',
 'saransk.naydidom.com': 'naydidom.com',
 'naberezhnye-chelny.naydidom.com': 'naydidom.com',
 'vidnoe.naydidom.com': 'naydidom.com',
 'omsk.naydidom.com': 'naydidom.com',
 'yaroslavl.naydidom.com': 'naydidom.com',
 'blagoveschensk.naydidom.com': 'naydidom.com',
 'anzhero-sudzhensk.naydidom.com': 'naydidom.com',
 'novoaltaysk.naydidom.com': 'naydidom.com',
 'zvenigorod.naydidom.com': 'naydidom.com',
 'chelyabinsk.naydidom.com': 'naydidom.com',
 'sankt-peterburg.naydidom.com': 'naydidom.com',
 'zheleznodorozhnyy.naydidom.com': 'naydidom.com',
 'barnaul.naydidom.com': 'naydidom.com',
 'tomsk.naydidom.com': 'naydidom.com',
 'volgograd.naydidom.com': 'naydidom.com',
 'belgorod.naydidom.com': 'naydidom.com',
 'novorossiysk.naydidom.com': 'naydidom.com',
 'budennovsk.naydidom.com': 'naydidom.com',
 'habarovsk.naydidom.com': 'naydidom.com',
 'vladimir.naydidom.com': 'naydidom.com',
 'kolomna.naydidom.com': 'naydidom.com',
 'syzran.naydidom.com': 'naydidom.com',
 'arhipo-osipovka.naydidom.com': 'naydidom.com',
 'surgut.naydidom.com': 'naydidom.com',
 'ivanovo.naydidom.com': 'naydidom.com',
 'kirov.naydidom.com': 'naydidom.com',
 'armavir.naydidom.com': 'naydidom.com',
 'samara.naydidom.com': 'naydidom.com',
 'izhevsk.naydidom.com': 'naydidom.com',
 'kaluga.naydidom.com': 'naydidom.com',
 'irkutsk.naydidom.com': 'naydidom.com',
 'slavyansk-na-kubani.naydidom.com': 'naydidom.com',
 'kursk.naydidom.com': 'naydidom.com',
 'kropotkin.naydidom.com': 'naydidom.com',
 'tambov.naydidom.com': 'naydidom.com',
 'kemerovo.naydidom.com': 'naydidom.com',
 'neftekamsk.naydidom.com': 'naydidom.com',
 'vladivostok.naydidom.com': 'naydidom.com',
 'petrozavodsk.naydidom.com': 'naydidom.com',
 'podolsk.naydidom.com': 'naydidom.com',
 'priozersk.naydidom.com': 'naydidom.com',
 'balashiha.naydidom.com': 'naydidom.com',
 'kinel.naydidom.com': 'naydidom.com',
 'pskov.naydidom.com': 'naydidom.com',
 'nahodka.naydidom.com': 'naydidom.com',
 'korenovsk.naydidom.com': 'naydidom.com',
 'kovrov.naydidom.com': 'naydidom.com',
 'stavropol.naydidom.com': 'naydidom.com',
 'kerch.naydidom.com': 'naydidom.com',
 'schelkovo.naydidom.com': 'naydidom.com',
    '062107.wriza.top': 'wriza.top',
 '6c860a10ffb0d492c9cb3f96a688d1436c860a10ffb0d492c9cb3f96a688d15.wriza.top': 'wriza.top',
 '242107.wriza.top': 'wriza.top',
 '232107.wriza.top': 'wriza.top',
 '082203.wriza.top': 'wriza.top',
 '262103.wriza.top': 'wriza.top',
 '072107.wriza.top': 'wriza.top',
 '032108.wriza.top': 'wriza.top',
 '302103.wriza.top': 'wriza.top',
 '252103.wriza.top': 'wriza.top',
 '012105.wriza.top': 'wriza.top',
 '292105.wriza.top': 'wriza.top',
 '272103.wriza.top': 'wriza.top',
 '012104.wriza.top': 'wriza.top',
 '312103.wriza.top': 'wriza.top',
 '022104.wriza.top': 'wriza.top',
 '052108.wriza.top': 'wriza.top',
 '082107.wriza.top': 'wriza.top',
 '092107.wriza.top': 'wriza.top',
 '012108.wriza.top': 'wriza.top',
 '202204.wriza.top': 'wriza.top',
 '292109.wriza.top': 'wriza.top',
 '192204.wriza.top': 'wriza.top',
 '202106.wriza.top': 'wriza.top',
 '112107.wriza.top': 'wriza.top',
 '292203.wriza.top': 'wriza.top',
 '272106.wriza.top': 'wriza.top',
 '182107.wriza.top': 'wriza.top',
 '232106.wriza.top': 'wriza.top',
 '222106.wriza.top': 'wriza.top',
 '122108.wriza.top': 'wriza.top',
 '202107.wriza.top': 'wriza.top',
 '212107.wriza.top': 'wriza.top',
 '192106.wriza.top': 'wriza.top',
 '242106.wriza.top': 'wriza.top',
 '122107.wriza.top': 'wriza.top',
 '162203.wriza.top': 'wriza.top',
 '162107.wriza.top': 'wriza.top',
 '272107.wriza.top': 'wriza.top',
 '042107.wriza.top': 'wriza.top',
 '142108.wriza.top': 'wriza.top',
 '012107.wriza.top': 'wriza.top',
 '072205.wriza.top': 'wriza.top',
 '252107.wriza.top': 'wriza.top',
 '172107.wriza.top': 'wriza.top',
 '162205.wriza.top': 'wriza.top',
 '192203.wriza.top': 'wriza.top',
 '132107.wriza.top': 'wriza.top',
 '252106.wriza.top': 'wriza.top',
 '142107.wriza.top': 'wriza.top',
 '302107.wriza.top': 'wriza.top',
 '262107.wriza.top': 'wriza.top',
 '042106.wriza.top': 'wriza.top',
 '192107.wriza.top': 'wriza.top',
 '222107.wriza.top': 'wriza.top',
 '052107.wriza.top': 'wriza.top',
 '102107.wriza.top': 'wriza.top',
 '212106.wriza.top': 'wriza.top',
 '162106.wriza.top': 'wriza.top',
 '282106.wriza.top': 'wriza.top',
 '312107.wriza.top': 'wriza.top',
 '032107.wriza.top': 'wriza.top',
 '152106.wriza.top': 'wriza.top',
 '232011.wriza.top': 'wriza.top',
 '242201.wriza.top': 'wriza.top',
 '142202.wriza.top': 'wriza.top',
 '112205.wriza.top': 'wriza.top',
 '172106.wriza.top': 'wriza.top',
 '072108.wriza.top': 'wriza.top',
 '122205.wriza.top': 'wriza.top',
 '302106.wriza.top': 'wriza.top',
 '022204.wriza.top': 'wriza.top',
 '092205.wriza.top': 'wriza.top',
 '082205.wriza.top': 'wriza.top',
 '142106.wriza.top': 'wriza.top',
 '042203.wriza.top': 'wriza.top',
 '172108.wriza.top': 'wriza.top',
 '232104.wriza.top': 'wriza.top',
 '272108.wriza.top': 'wriza.top',
 '062203.wriza.top': 'wriza.top',
    'polovinka.org': 'polovinka.org',
 'krasnodar.polovinka.org': 'polovinka.org',
 'nijniy-novgorod.polovinka.org': 'polovinka.org',
 'ufa.polovinka.org': 'polovinka.org',
 'barnaul.polovinka.org': 'polovinka.org',
 'kinder.polovinka.org': 'polovinka.org',
 'zlatoust.polovinka.org': 'polovinka.org',
 'omsk.polovinka.org': 'polovinka.org',
 'samara.polovinka.org': 'polovinka.org',
 'minsk.polovinka.org': 'polovinka.org',
 'ekaterinburg.polovinka.org': 'polovinka.org',
 'chelyabinsk.polovinka.org': 'polovinka.org',
 'ussuriysk.polovinka.org': 'polovinka.org',
 'surgut.polovinka.org': 'polovinka.org',
 'chelno-vershinyi.polovinka.org': 'polovinka.org',
 'tolyatti.polovinka.org': 'polovinka.org',
 'vladivostok.polovinka.org': 'polovinka.org',
 'kemerovo.polovinka.org': 'polovinka.org',
 'orenburg.polovinka.org': 'polovinka.org',
 'krasnoyarsk.polovinka.org': 'polovinka.org',
 'kazan.polovinka.org': 'polovinka.org',
 'irkutsk.polovinka.org': 'polovinka.org',
 'moskva.polovinka.org': 'polovinka.org',
 'mahachkala.polovinka.org': 'polovinka.org',
 'ulyanovsk.polovinka.org': 'polovinka.org',
 'sankt-peterburg.polovinka.org': 'polovinka.org',
 'perm.polovinka.org': 'polovinka.org',
 'ryazan.polovinka.org': 'polovinka.org',
 'habarovsk.polovinka.org': 'polovinka.org',
 'tyumen.polovinka.org': 'polovinka.org',
 'tomsk.polovinka.org': 'polovinka.org',
 'voronej.polovinka.org': 'polovinka.org',
 'volgograd.polovinka.org': 'polovinka.org',
 'naberejnyie-chelnyi.polovinka.org': 'polovinka.org',
 'novokuznetsk.polovinka.org': 'polovinka.org',
 'penza.polovinka.org': 'polovinka.org',
 'saratov.polovinka.org': 'polovinka.org',
 'novosibirsk.polovinka.org': 'polovinka.org',
 'rostov-na-donu.polovinka.org': 'polovinka.org',
 'lipetsk.polovinka.org': 'polovinka.org',
 'iskitim.polovinka.org': 'polovinka.org',
 'norilsk.polovinka.org': 'polovinka.org',
 'leonardo.polovinka.org': 'polovinka.org',
 'kursk.polovinka.org': 'polovinka.org',
 'sergiev-posad.polovinka.org': 'polovinka.org',
 'vologda.polovinka.org': 'polovinka.org',
 'zelenograd.polovinka.org': 'polovinka.org',
 'balashiha.polovinka.org': 'polovinka.org',
 'volokolamsk.polovinka.org': 'polovinka.org',
 'podolsk.polovinka.org': 'polovinka.org',
 'himki.polovinka.org': 'polovinka.org',
 'abinsk.polovinka.org': 'polovinka.org',
 'anapa.polovinka.org': 'polovinka.org',
 'korenovsk.polovinka.org': 'polovinka.org',
 'gelendjik.polovinka.org': 'polovinka.org',
 'adler.polovinka.org': 'polovinka.org',
 'tihoretsk.polovinka.org': 'polovinka.org',
 'eysk.polovinka.org': 'polovinka.org',
 'arhipo-osipovka.polovinka.org': 'polovinka.org',
 'sochi.polovinka.org': 'polovinka.org',
 'karasuk.polovinka.org': 'polovinka.org',
 'toguchin.polovinka.org': 'polovinka.org',
 'chanyi.polovinka.org': 'polovinka.org',
 'belovo.polovinka.org': 'polovinka.org',
 'zainsk.polovinka.org': 'polovinka.org',
 'agryiz.polovinka.org': 'polovinka.org',
 'nijnekamsk.polovinka.org': 'polovinka.org',
 'zelenodolsk.polovinka.org': 'polovinka.org',
 'kukmor.polovinka.org': 'polovinka.org',
 'buinsk.polovinka.org': 'polovinka.org',
 'elabuga.polovinka.org': 'polovinka.org',
 'schelkovo.polovinka.org': 'polovinka.org',
 'alapaevsk.polovinka.org': 'polovinka.org',
 'kamensk-uralskiy.polovinka.org': 'polovinka.org',
 'verhnyaya-salda.polovinka.org': 'polovinka.org',
 'asbest.polovinka.org': 'polovinka.org',
 'nijniy-tagil.polovinka.org': 'polovinka.org',
 'petropavlovsk-kamchatskiy.polovinka.org': 'polovinka.org',
 'nurlat.polovinka.org': 'polovinka.org',
 'tashkent.polovinka.org': 'polovinka.org',
    'ivanovo.vsaunah.ru': 'vsaunah.ru',
 'ijevsk.vsaunah.ru': 'vsaunah.ru',
 'barnaul.vsaunah.ru': 'vsaunah.ru',
 'kostroma.vsaunah.ru': 'vsaunah.ru',
 'tolyatti.vsaunah.ru': 'vsaunah.ru',
 'blagoveshchensk.vsaunah.ru': 'vsaunah.ru',
 'ryazan.vsaunah.ru': 'vsaunah.ru',
 'tomsk.vsaunah.ru': 'vsaunah.ru',
 'sochi.vsaunah.ru': 'vsaunah.ru',
 'perm.vsaunah.ru': 'vsaunah.ru',
 'habarovsk.vsaunah.ru': 'vsaunah.ru',
 'kaliningrad.vsaunah.ru': 'vsaunah.ru',
 'magnitogorsk.vsaunah.ru': 'vsaunah.ru',
 'rostov.vsaunah.ru': 'vsaunah.ru',
 'krasnoyarsk.vsaunah.ru': 'vsaunah.ru',
 'nijnevartovsk.vsaunah.ru': 'vsaunah.ru',
 'omsk.vsaunah.ru': 'vsaunah.ru',
 'belgorod.vsaunah.ru': 'vsaunah.ru',
 'pskov.vsaunah.ru': 'vsaunah.ru',
 'vsaunah.ru': 'vsaunah.ru',
 'kurgan.vsaunah.ru': 'vsaunah.ru',
 'naberezhnyechelny.vsaunah.ru': 'vsaunah.ru',
 'vladimir.vsaunah.ru': 'vsaunah.ru',
 'tyumen.vsaunah.ru': 'vsaunah.ru',
 'orenburg.vsaunah.ru': 'vsaunah.ru',
 'kazan.vsaunah.ru': 'vsaunah.ru',
 'tula.vsaunah.ru': 'vsaunah.ru',
 'kirov.vsaunah.ru': 'vsaunah.ru',
 'noyabrsk.vsaunah.ru': 'vsaunah.ru',
 'sterlitamak.vsaunah.ru': 'vsaunah.ru',
 'petrozavodsk.vsaunah.ru': 'vsaunah.ru',
 'surgut.vsaunah.ru': 'vsaunah.ru',
 'smolensk.vsaunah.ru': 'vsaunah.ru',
 'peterburg.vsaunah.ru': 'vsaunah.ru',
 'yaroslavl.vsaunah.ru': 'vsaunah.ru',
 'samara.vsaunah.ru': 'vsaunah.ru',
 'stavropol.vsaunah.ru': 'vsaunah.ru',
 'saratov.vsaunah.ru': 'vsaunah.ru',
 'krasnogorsk.vsaunah.ru': 'vsaunah.ru',
 'khantymansiysk.vsaunah.ru': 'vsaunah.ru',
 'vladivostok.vsaunah.ru': 'vsaunah.ru',
 'vologda.vsaunah.ru': 'vsaunah.ru',
 'groznyj.vsaunah.ru': 'vsaunah.ru',
 'syktyvkar.vsaunah.ru': 'vsaunah.ru',
 'pervouralsk.vsaunah.ru': 'vsaunah.ru',
 'nsk.vsaunah.ru': 'vsaunah.ru',
 'orsk.vsaunah.ru': 'vsaunah.ru',
 'vladikavkaz.vsaunah.ru': 'vsaunah.ru',
 'tver.vsaunah.ru': 'vsaunah.ru',
 'kursk.vsaunah.ru': 'vsaunah.ru',
 'lipeck.vsaunah.ru': 'vsaunah.ru',
 'novgorod.vsaunah.ru': 'vsaunah.ru',
 'penza.vsaunah.ru': 'vsaunah.ru',
 'irkutsk.vsaunah.ru': 'vsaunah.ru',
 'orel.vsaunah.ru': 'vsaunah.ru',
 'yalta.vsaunah.ru': 'vsaunah.ru',
 'cherepovez.vsaunah.ru': 'vsaunah.ru',
 'yola.vsaunah.ru': 'vsaunah.ru',
 'ulianovsk.vsaunah.ru': 'vsaunah.ru',
 'kaluga.vsaunah.ru': 'vsaunah.ru',
 'bryansk.vsaunah.ru': 'vsaunah.ru',
 'nalchik.vsaunah.ru': 'vsaunah.ru',
 'murmansk.vsaunah.ru': 'vsaunah.ru',
 'nefteyugansk.vsaunah.ru': 'vsaunah.ru',
 'novokuzneck.vsaunah.ru': 'vsaunah.ru',
 'birobidzhan.vsaunah.ru': 'vsaunah.ru',
 'elista.vsaunah.ru': 'vsaunah.ru',
 'chelyabinsk.vsaunah.ru': 'vsaunah.ru',
 'majkop.vsaunah.ru': 'vsaunah.ru',
 'volgograd.vsaunah.ru': 'vsaunah.ru',
 'saransk.vsaunah.ru': 'vsaunah.ru',
 'tobolsk.vsaunah.ru': 'vsaunah.ru',
 'juzhnosahalinsk.vsaunah.ru': 'vsaunah.ru',
 'podolsk.vsaunah.ru': 'vsaunah.ru',
 'yakutsk.vsaunah.ru': 'vsaunah.ru',
 'rybinsk.vsaunah.ru': 'vsaunah.ru',
 'novorossijsk.vsaunah.ru': 'vsaunah.ru',
 'anapa.vsaunah.ru': 'vsaunah.ru',
 'sergievposad.vsaunah.ru': 'vsaunah.ru',
 'sevastopol.vsaunah.ru': 'vsaunah.ru',
 'tambov.vsaunah.ru': 'vsaunah.ru',
 'arkhangelsk.vsaunah.ru': 'vsaunah.ru',
    'novosibirsk.regreestr.com': 'regreestr.com',
 'regreestr.com': 'regreestr.com',
 'moskva.regreestr.com': 'regreestr.com',
 'perm.regreestr.com': 'regreestr.com',
 'spb.regreestr.com': 'regreestr.com',
 'krasnodar.regreestr.com': 'regreestr.com',
 'kazan.regreestr.com': 'regreestr.com',
 'voronezh.regreestr.com': 'regreestr.com',
 'ulyanovsk.regreestr.com': 'regreestr.com',
 'eburg.regreestr.com': 'regreestr.com',
 'krasnoyarsk.regreestr.com': 'regreestr.com',
 'samara.regreestr.com': 'regreestr.com',
 'sochi.regreestr.com': 'regreestr.com',
 'orenburg.regreestr.com': 'regreestr.com',
 'stavropol.regreestr.com': 'regreestr.com',
 'rostov-nd.regreestr.com': 'regreestr.com',
 'vladivostok.regreestr.com': 'regreestr.com',
 'omsk.regreestr.com': 'regreestr.com',
 'ufa.regreestr.com': 'regreestr.com',
 'tver.regreestr.com': 'regreestr.com',
 'electrostal.regreestr.com': 'regreestr.com',
 'volgograd.regreestr.com': 'regreestr.com',
 'bryansk.regreestr.com': 'regreestr.com',
 'nnovgorod.regreestr.com': 'regreestr.com',
 'dmitrov.regreestr.com': 'regreestr.com',
 'petrozavodsk.regreestr.com': 'regreestr.com',
 'almetevsk.regreestr.com': 'regreestr.com',
 'smolensk.regreestr.com': 'regreestr.com',
 'khabarovsk.regreestr.com': 'regreestr.com',
 'surgut.regreestr.com': 'regreestr.com',
 'angarsk.regreestr.com': 'regreestr.com',
 'tyumen.regreestr.com': 'regreestr.com',
 'chelyabinsk.regreestr.com': 'regreestr.com',
 'zlatoust.regreestr.com': 'regreestr.com',
 'kemerovo.regreestr.com': 'regreestr.com',
 'magnitogorsk.regreestr.com': 'regreestr.com',
 'ivanovo.regreestr.com': 'regreestr.com',
 'barnaul.regreestr.com': 'regreestr.com',
 'ulan-ude.regreestr.com': 'regreestr.com',
 'saransk.regreestr.com': 'regreestr.com',
 'cherkessk.regreestr.com': 'regreestr.com',
 'yoshkar-ola.regreestr.com': 'regreestr.com',
 'yakutsk.regreestr.com': 'regreestr.com',
 'ryazan.regreestr.com': 'regreestr.com',
 'arkhangelsk.regreestr.com': 'regreestr.com',
 'makhachkala.regreestr.com': 'regreestr.com',
 'murmansk.regreestr.com': 'regreestr.com',
 'lipetsk.regreestr.com': 'regreestr.com',
 'blagovetshensk.regreestr.com': 'regreestr.com',
 'irkutsk.regreestr.com': 'regreestr.com',
 'nakhodka.regreestr.com': 'regreestr.com',
 'tambov.regreestr.com': 'regreestr.com',
 'vladimir.regreestr.com': 'regreestr.com',
 'yaroslavl.regreestr.com': 'regreestr.com',
 'novoross.regreestr.com': 'regreestr.com',
 'gelendzhik.regreestr.com': 'regreestr.com',
 'saratov.regreestr.com': 'regreestr.com',
 'izhevsk.regreestr.com': 'regreestr.com',
 'kaliningrad.regreestr.com': 'regreestr.com',
 'belgorod.regreestr.com': 'regreestr.com',
 'dubna.regreestr.com': 'regreestr.com',
 'kirov.regreestr.com': 'regreestr.com',
 'penza.regreestr.com': 'regreestr.com',
 'kursk.regreestr.com': 'regreestr.com',
 'kurgan.regreestr.com': 'regreestr.com',
 'krasnogorsk.regreestr.com': 'regreestr.com',
 'mytitshy.regreestr.com': 'regreestr.com',
 'podolsk.regreestr.com': 'regreestr.com',
 'orsk.regreestr.com': 'regreestr.com',
 'kaluga.regreestr.com': 'regreestr.com',
 'pskov.regreestr.com': 'regreestr.com',
 'dzerzhinsk.regreestr.com': 'regreestr.com',
 'nabchelny.regreestr.com': 'regreestr.com',
 'berdsk.regreestr.com': 'regreestr.com',
 'maykop.regreestr.com': 'regreestr.com',
 'nizhnekamsk.regreestr.com': 'regreestr.com',
 'novokuznetsk.regreestr.com': 'regreestr.com',
 'syktyvkar.regreestr.com': 'regreestr.com',
 'cheboksary.regreestr.com': 'regreestr.com',
 'rzhev.regreestr.com': 'regreestr.com',
 'tomsk.regreestr.com': 'regreestr.com',
 'armavir.regreestr.com': 'regreestr.com',
 'anapa.regreestr.com': 'regreestr.com',
    'krasnodar.nashaspravka.ru': 'nashaspravka.ru',
 'omsk.nashaspravka.ru': 'nashaspravka.ru',
 'ufa.nashaspravka.ru': 'nashaspravka.ru',
 'saratov.nashaspravka.ru': 'nashaspravka.ru',
 'kostroma.nashaspravka.ru': 'nashaspravka.ru',
 'bryansk.nashaspravka.ru': 'nashaspravka.ru',
 'syktyvkar.nashaspravka.ru': 'nashaspravka.ru',
 'izhevsk.nashaspravka.ru': 'nashaspravka.ru',
 'moskva.nashaspravka.ru': 'nashaspravka.ru',
 'rostov.nashaspravka.ru': 'nashaspravka.ru',
 'tyumen.nashaspravka.ru': 'nashaspravka.ru',
 'peterburg.nashaspravka.ru': 'nashaspravka.ru',
 'chelyabinsk.nashaspravka.ru': 'nashaspravka.ru',
 'blagoveshchensk.nashaspravka.ru': 'nashaspravka.ru',
 'stavropol.nashaspravka.ru': 'nashaspravka.ru',
 'sochi.nashaspravka.ru': 'nashaspravka.ru',
 'kazan.nashaspravka.ru': 'nashaspravka.ru',
 'habarovsk.nashaspravka.ru': 'nashaspravka.ru',
 'novosibirsk.nashaspravka.ru': 'nashaspravka.ru',
 'belgorod.nashaspravka.ru': 'nashaspravka.ru',
 'nnov.nashaspravka.ru': 'nashaspravka.ru',
 'ekaterinburg.nashaspravka.ru': 'nashaspravka.ru',
 'ulyanovsk.nashaspravka.ru': 'nashaspravka.ru',
 'i-ola.nashaspravka.ru': 'nashaspravka.ru',
 'simferopol.nashaspravka.ru': 'nashaspravka.ru',
 'pskov.nashaspravka.ru': 'nashaspravka.ru',
 'ryazan.nashaspravka.ru': 'nashaspravka.ru',
 'kurgan.nashaspravka.ru': 'nashaspravka.ru',
 'abakan.nashaspravka.ru': 'nashaspravka.ru',
 'vladikavkaz.nashaspravka.ru': 'nashaspravka.ru',
 'kursk.nashaspravka.ru': 'nashaspravka.ru',
 'krasnoyarsk.nashaspravka.ru': 'nashaspravka.ru',
 'tambov.nashaspravka.ru': 'nashaspravka.ru',
 'samara.nashaspravka.ru': 'nashaspravka.ru',
 'saransk.nashaspravka.ru': 'nashaspravka.ru',
 'vladivostok.nashaspravka.ru': 'nashaspravka.ru',
 'vladimir.nashaspravka.ru': 'nashaspravka.ru',
 'ivanovo.nashaspravka.ru': 'nashaspravka.ru',
 'cheboksary.nashaspravka.ru': 'nashaspravka.ru',
 'lipetsk.nashaspravka.ru': 'nashaspravka.ru',
 'petrozavodsk.nashaspravka.ru': 'nashaspravka.ru',
 'tver.nashaspravka.ru': 'nashaspravka.ru',
 'vologda.nashaspravka.ru': 'nashaspravka.ru',
 'orenburg.nashaspravka.ru': 'nashaspravka.ru',
 'penza.nashaspravka.ru': 'nashaspravka.ru',
 'mahachkala.nashaspravka.ru': 'nashaspravka.ru',
 'barnaul.nashaspravka.ru': 'nashaspravka.ru',
 'yaroslavl.nashaspravka.ru': 'nashaspravka.ru',
 'voronezh.nashaspravka.ru': 'nashaspravka.ru',
 'volgograd.nashaspravka.ru': 'nashaspravka.ru',
 'arhangelsk.nashaspravka.ru': 'nashaspravka.ru',
 'perm.nashaspravka.ru': 'nashaspravka.ru',
 'kaliningrad.nashaspravka.ru': 'nashaspravka.ru',
 'orel.nashaspravka.ru': 'nashaspravka.ru',
 'murmansk.nashaspravka.ru': 'nashaspravka.ru',
 'kemerovo.nashaspravka.ru': 'nashaspravka.ru',
 'ulan-ude.nashaspravka.ru': 'nashaspravka.ru',
 'astrahan.nashaspravka.ru': 'nashaspravka.ru',
 'tomsk.nashaspravka.ru': 'nashaspravka.ru',
 'tula.nashaspravka.ru': 'nashaspravka.ru',
 'yuzhno-sahalinsk.nashaspravka.ru': 'nashaspravka.ru',
 'novgorod.nashaspravka.ru': 'nashaspravka.ru',
 'petropavlovsk.nashaspravka.ru': 'nashaspravka.ru',
 'kaluga.nashaspravka.ru': 'nashaspravka.ru',
 'chita.nashaspravka.ru': 'nashaspravka.ru',
 'armavir.nashaspravka.ru': 'nashaspravka.ru',
 'irkutsk.nashaspravka.ru': 'nashaspravka.ru',
 'smolensk.nashaspravka.ru': 'nashaspravka.ru',
 'sevastopol.nashaspravka.ru': 'nashaspravka.ru',
 'nalchik.nashaspravka.ru': 'nashaspravka.ru',
 'toljatti.nashaspravka.ru': 'nashaspravka.ru',
 'nashaspravka.ru': 'nashaspravka.ru',
 'volzhsky.nashaspravka.ru': 'nashaspravka.ru',
 'almetyevsk.nashaspravka.ru': 'nashaspravka.ru',
 'chelny.nashaspravka.ru': 'nashaspravka.ru',
 'engels.nashaspravka.ru': 'nashaspravka.ru',
 'grozny.nashaspravka.ru': 'nashaspravka.ru',
 'birobidzhan.nashaspravka.ru': 'nashaspravka.ru',
 'gornoaltaysk.nashaspravka.ru': 'nashaspravka.ru',
 'sterlitamak.nashaspravka.ru': 'nashaspravka.ru',
 'anapa.nashaspravka.ru': 'nashaspravka.ru',
 'tobolsk.nashaspravka.ru': 'nashaspravka.ru',
 'komsomolsk.nashaspravka.ru': 'nashaspravka.ru',
    'kemerovo.regtorg.ru': 'regtorg.ru',
 'kbk.regtorg.ru': 'regtorg.ru',
 'lipeck.regtorg.ru': 'regtorg.ru',
 'ekaterinburg.regtorg.ru': 'regtorg.ru',
 'samara.regtorg.ru': 'regtorg.ru',
 'ivanovo.regtorg.ru': 'regtorg.ru',
 'smolensk.regtorg.ru': 'regtorg.ru',
 'orenburg.regtorg.ru': 'regtorg.ru',
 'penza.regtorg.ru': 'regtorg.ru',
 'nizhnij-novgorod.regtorg.ru': 'regtorg.ru',
 'volgograd.regtorg.ru': 'regtorg.ru',
 'moskva.regtorg.ru': 'regtorg.ru',
 'kurs.regtorg.ru': 'regtorg.ru',
 'naberezhnye-chelny.regtorg.ru': 'regtorg.ru',
 'cheboksary.regtorg.ru': 'regtorg.ru',
 'podolsk.regtorg.ru': 'regtorg.ru',
 'tomsk.regtorg.ru': 'regtorg.ru',
 'vologda.regtorg.ru': 'regtorg.ru',
 'ryazan.regtorg.ru': 'regtorg.ru',
 'voronezh.regtorg.ru': 'regtorg.ru',
 'tolyatti.regtorg.ru': 'regtorg.ru',
 'sochi.regtorg.ru': 'regtorg.ru',
 'kursk.regtorg.ru': 'regtorg.ru',
 'irkutsk.regtorg.ru': 'regtorg.ru',
 'novokuzneck.regtorg.ru': 'regtorg.ru',
 'taganrog.regtorg.ru': 'regtorg.ru',
 'blagoveschensk.regtorg.ru': 'regtorg.ru',
 'belgorod.regtorg.ru': 'regtorg.ru',
 'tjumen.regtorg.ru': 'regtorg.ru',
 'omsk.regtorg.ru': 'regtorg.ru',
 'balashiha.regtorg.ru': 'regtorg.ru',
 'kazan.regtorg.ru': 'regtorg.ru',
 'izhevsk.regtorg.ru': 'regtorg.ru',
 'spb.regtorg.ru': 'regtorg.ru',
 'chelyabinsk.regtorg.ru': 'regtorg.ru',
 'user.regtorg.ru': 'regtorg.ru',
 'dzerzhinsk.regtorg.ru': 'regtorg.ru',
 'vladimir.regtorg.ru': 'regtorg.ru',
 'engels.regtorg.ru': 'regtorg.ru',
 'tver.regtorg.ru': 'regtorg.ru',
 'kaluga.regtorg.ru': 'regtorg.ru',
 'perm.regtorg.ru': 'regtorg.ru',
 'vladivostok.regtorg.ru': 'regtorg.ru',
 'oktmo.regtorg.ru': 'regtorg.ru',
 'barnaul.regtorg.ru': 'regtorg.ru',
 'rostov-na-donu.regtorg.ru': 'regtorg.ru',
 'orel.regtorg.ru': 'regtorg.ru',
 'stavropol.regtorg.ru': 'regtorg.ru',
 'bryansk.regtorg.ru': 'regtorg.ru',
 'kirov.regtorg.ru': 'regtorg.ru',
 'krasnoyarsk.regtorg.ru': 'regtorg.ru',
 'krasnodar.regtorg.ru': 'regtorg.ru',
 'saratov.regtorg.ru': 'regtorg.ru',
 'novosibirsk.regtorg.ru': 'regtorg.ru',
 'tula.regtorg.ru': 'regtorg.ru',
 'simferopol.regtorg.ru': 'regtorg.ru',
 'volzhskiy.regtorg.ru': 'regtorg.ru',
 'habarovsk.regtorg.ru': 'regtorg.ru',
 'ulyanovsk.regtorg.ru': 'regtorg.ru',
 'ufa.regtorg.ru': 'regtorg.ru',
 'yaroslavl.regtorg.ru': 'regtorg.ru',
 'okato.regtorg.ru': 'regtorg.ru',
 'news.regtorg.ru': 'regtorg.ru',
 'contragent.regtorg.ru': 'regtorg.ru',
 'omac-italy.regtorg.ru': 'regtorg.ru',
 'sibenergo-s285207.regtorg.ru': 'regtorg.ru',
 'petrostroj-s276231.regtorg.ru': 'regtorg.ru',
 'doskipiter.regtorg.ru': 'regtorg.ru',
 'unitor.regtorg.ru': 'regtorg.ru',
 'regtorg.ru': 'regtorg.ru',
 'santegra24.regtorg.ru': 'regtorg.ru',
 'sotsnab.regtorg.ru': 'regtorg.ru',
 'siesta-mebel-opt.regtorg.ru': 'regtorg.ru',
 'resurs-tranzit.regtorg.ru': 'regtorg.ru',
 'opt-torg.regtorg.ru': 'regtorg.ru',
 'kirbaza.regtorg.ru': 'regtorg.ru',
 'dipos-ivanovo.regtorg.ru': 'regtorg.ru',
 'rto-russia.regtorg.ru': 'regtorg.ru',
 'temp-st.regtorg.ru': 'regtorg.ru',
 'orenprokat56.regtorg.ru': 'regtorg.ru',
 'spktechnika.regtorg.ru': 'regtorg.ru',
 'vitahim-sibir.regtorg.ru': 'regtorg.ru',
 'zavod-stenovyh-materialov-porevit-s352970.regtorg.ru': 'regtorg.ru',
 'avtopromagregat-kanavinskij-filial.regtorg.ru': 'regtorg.ru',
    'nizhnevartovsk.allcafe.ru': 'allcafe.ru',
 'tyumen.allcafe.ru': 'allcafe.ru',
 'spb.allcafe.ru': 'allcafe.ru',
 'sochi.allcafe.ru': 'allcafe.ru',
 'allcafe.ru': 'allcafe.ru',
 'achinsk.allcafe.ru': 'allcafe.ru',
 'tomsk.allcafe.ru': 'allcafe.ru',
 'orenburg.allcafe.ru': 'allcafe.ru',
 'bugulma.allcafe.ru': 'allcafe.ru',
 'perm.allcafe.ru': 'allcafe.ru',
 'msk.allcafe.ru': 'allcafe.ru',
 'tmb.allcafe.ru': 'allcafe.ru',
 'saratov.allcafe.ru': 'allcafe.ru',
 'rnd.allcafe.ru': 'allcafe.ru',
 'anapa.allcafe.ru': 'allcafe.ru',
 'gornoaltaysk.allcafe.ru': 'allcafe.ru',
 'ivanovo.allcafe.ru': 'allcafe.ru',
 'novosibirsk.allcafe.ru': 'allcafe.ru',
 'krasnoyarsk.allcafe.ru': 'allcafe.ru',
 'ekburg.allcafe.ru': 'allcafe.ru',
 'saransk.allcafe.ru': 'allcafe.ru',
 'ufa.allcafe.ru': 'allcafe.ru',
 'pskov.allcafe.ru': 'allcafe.ru',
 'magnitogorsk.allcafe.ru': 'allcafe.ru',
 'naberezhnie-chelny.allcafe.ru': 'allcafe.ru',
 'barnaul.allcafe.ru': 'allcafe.ru',
 'habarovsk.allcafe.ru': 'allcafe.ru',
 'kazan.allcafe.ru': 'allcafe.ru',
 'samara.allcafe.ru': 'allcafe.ru',
 'salavat.allcafe.ru': 'allcafe.ru',
 'vladimir.allcafe.ru': 'allcafe.ru',
 'orl.allcafe.ru': 'allcafe.ru',
 'majkop.allcafe.ru': 'allcafe.ru',
 'vrn.allcafe.ru': 'allcafe.ru',
 'astrakhan.allcafe.ru': 'allcafe.ru',
 'mahachkala.allcafe.ru': 'allcafe.ru',
 'chel.allcafe.ru': 'allcafe.ru',
 'vladikavkaz.allcafe.ru': 'allcafe.ru',
 'izhevsk.allcafe.ru': 'allcafe.ru',
 'vlad.allcafe.ru': 'allcafe.ru',
 'komsomolsk-na-amure.allcafe.ru': 'allcafe.ru',
 'nnov.allcafe.ru': 'allcafe.ru',
 'nevinnomyissk.allcafe.ru': 'allcafe.ru',
 'novorossijsk.allcafe.ru': 'allcafe.ru',
 'tula.allcafe.ru': 'allcafe.ru',
 'krasnodar.allcafe.ru': 'allcafe.ru',
 'pyatigorsk.allcafe.ru': 'allcafe.ru',
 'nizhnekamsk.allcafe.ru': 'allcafe.ru',
 'arkhangelsk.allcafe.ru': 'allcafe.ru',
 'kaluga.allcafe.ru': 'allcafe.ru',
 'kstovo.allcafe.ru': 'allcafe.ru',
 'arzamas.allcafe.ru': 'allcafe.ru',
 'omsk.allcafe.ru': 'allcafe.ru',
 'novokuybyishevsk.allcafe.ru': 'allcafe.ru',
 'vnov.allcafe.ru': 'allcafe.ru',
 'kaliningrad.allcafe.ru': 'allcafe.ru',
 'cheboksary.allcafe.ru': 'allcafe.ru',
 'volgograd.allcafe.ru': 'allcafe.ru',
 'volgodonsk.allcafe.ru': 'allcafe.ru',
 'orsk.allcafe.ru': 'allcafe.ru',
 'serpuhov.allcafe.ru': 'allcafe.ru',
 'riazan.allcafe.ru': 'allcafe.ru',
 'petropavlovsk-kamchatskiy.allcafe.ru': 'allcafe.ru',
 'yaroslavl.allcafe.ru': 'allcafe.ru',
 'zlatoust.allcafe.ru': 'allcafe.ru',
 'nalchik.allcafe.ru': 'allcafe.ru',
 'novokuznezk.allcafe.ru': 'allcafe.ru',
 'penza.allcafe.ru': 'allcafe.ru',
 'ulianovsk.allcafe.ru': 'allcafe.ru',
 'bel.allcafe.ru': 'allcafe.ru',
 'kurgan.allcafe.ru': 'allcafe.ru',
 'biysk.allcafe.ru': 'allcafe.ru',
 'cterlitamak.allcafe.ru': 'allcafe.ru',
 'taganrog.allcafe.ru': 'allcafe.ru',
 'abakan.allcafe.ru': 'allcafe.ru',
 'murmansk.allcafe.ru': 'allcafe.ru',
 'azov.allcafe.ru': 'allcafe.ru',
 'yuzhno-sahalinsk.allcafe.ru': 'allcafe.ru',
 'novocherkassk.allcafe.ru': 'allcafe.ru',
 'nizhtagil.allcafe.ru': 'allcafe.ru',
 'vologda.allcafe.ru': 'allcafe.ru',
 'shahtyi.allcafe.ru': 'allcafe.ru',
 'revda.allcafe.ru': 'allcafe.ru',
 'kamensk-uralskiy.allcafe.ru': 'allcafe.ru',
 'stavropol.allcafe.ru': 'allcafe.ru',
    '032107.sinema.top': 'sinema.top',
 '302107.sinema.top': 'sinema.top',
 '242107.sinema.top': 'sinema.top',
 '252105.sinema.top': 'sinema.top',
 'gotorrent.sinema.top': 'sinema.top',
 '092204.sinema.top': 'sinema.top',
 '132108.sinema.top': 'sinema.top',
 '262107.sinema.top': 'sinema.top',
 '032108.sinema.top': 'sinema.top',
 '272012.sinema.top': 'sinema.top',
 '282105.sinema.top': 'sinema.top',
 '302106.sinema.top': 'sinema.top',
 '202107.sinema.top': 'sinema.top',
 '252204.sinema.top': 'sinema.top',
 '112108.sinema.top': 'sinema.top',
 '212106.sinema.top': 'sinema.top',
 '142105.sinema.top': 'sinema.top',
 '062108.sinema.top': 'sinema.top',
 '042107.sinema.top': 'sinema.top',
 '052203.sinema.top': 'sinema.top',
 '032202.sinema.top': 'sinema.top',
 '072111.sinema.top': 'sinema.top',
 '282102.sinema.top': 'sinema.top',
 '012205.sinema.top': 'sinema.top',
 '212205.sinema.top': 'sinema.top',
 '282106.sinema.top': 'sinema.top',
 '272106.sinema.top': 'sinema.top',
 '052107.sinema.top': 'sinema.top',
 '092107.sinema.top': 'sinema.top',
 '112107.sinema.top': 'sinema.top',
 'yatorrent.sinema.top': 'sinema.top',
 '222204.sinema.top': 'sinema.top',
 '242203.sinema.top': 'sinema.top',
 '302104.sinema.top': 'sinema.top',
 '102108.sinema.top': 'sinema.top',
 '222106.sinema.top': 'sinema.top',
 '112204.sinema.top': 'sinema.top',
 '262106.sinema.top': 'sinema.top',
 '312107.sinema.top': 'sinema.top',
 '182106.sinema.top': 'sinema.top',
 '182108.sinema.top': 'sinema.top',
 '022205.sinema.top': 'sinema.top',
 '092108.sinema.top': 'sinema.top',
 '112101.sinema.top': 'sinema.top',
 '232110.sinema.top': 'sinema.top',
 '252107.sinema.top': 'sinema.top',
 '122202.sinema.top': 'sinema.top',
 '142104.sinema.top': 'sinema.top',
 '042108.sinema.top': 'sinema.top',
 '132103.sinema.top': 'sinema.top',
 '062106.sinema.top': 'sinema.top',
 '272107.sinema.top': 'sinema.top',
 '132106.sinema.top': 'sinema.top',
 '292203.sinema.top': 'sinema.top',
 '232106.sinema.top': 'sinema.top',
 '202106.sinema.top': 'sinema.top',
 '252104.sinema.top': 'sinema.top',
 '122107.sinema.top': 'sinema.top',
 '252111.sinema.top': 'sinema.top',
 '162106.sinema.top': 'sinema.top',
 '142205.sinema.top': 'sinema.top',
 '102107.sinema.top': 'sinema.top',
 '042104.sinema.top': 'sinema.top',
 '032106.sinema.top': 'sinema.top',
 '132012.sinema.top': 'sinema.top',
 '162108.sinema.top': 'sinema.top',
 '222107.sinema.top': 'sinema.top',
 '032101.sinema.top': 'sinema.top',
 '182107.sinema.top': 'sinema.top',
 '292106.sinema.top': 'sinema.top',
 '042101.sinema.top': 'sinema.top',
 '062107.sinema.top': 'sinema.top',
 '152108.sinema.top': 'sinema.top',
 '012201.sinema.top': 'sinema.top',
 '092205.sinema.top': 'sinema.top',
 '052105.sinema.top': 'sinema.top',
 '242102.sinema.top': 'sinema.top',
 '102103.sinema.top': 'sinema.top',
 '062105.sinema.top': 'sinema.top',
 '142108.sinema.top': 'sinema.top',
 '152106.sinema.top': 'sinema.top',
 '142106.sinema.top': 'sinema.top',
 '232107.sinema.top': 'sinema.top',
 '132104.sinema.top': 'sinema.top',
 '292107.sinema.top': 'sinema.top',
 '112205.sinema.top': 'sinema.top',
 '022101.sinema.top': 'sinema.top',
    'domodedovo.aeroport.website': 'aeroport.website',
 'novokuzneck.aeroport.website': 'aeroport.website',
 'pulkovo.aeroport.website': 'aeroport.website',
 'vorkuta.aeroport.website': 'aeroport.website',
 'begishevo.aeroport.website': 'aeroport.website',
 'manas.aeroport.website': 'aeroport.website',
 'sheremetyevo.aeroport.website': 'aeroport.website',
 'tynda.aeroport.website': 'aeroport.website',
 'vnukovo.aeroport.website': 'aeroport.website',
 'noyabrsk.aeroport.website': 'aeroport.website',
 'kazan.aeroport.website': 'aeroport.website',
 'noviy-urengoy.aeroport.website': 'aeroport.website',
 'tashkent.aeroport.website': 'aeroport.website',
 'kemerovo.aeroport.website': 'aeroport.website',
 'murmansk.aeroport.website': 'aeroport.website',
 'sokol.aeroport.website': 'aeroport.website',
 'ufa.aeroport.website': 'aeroport.website',
 'yakutsk.aeroport.website': 'aeroport.website',
 'nizhnevartovsk.aeroport.website': 'aeroport.website',
 'nikolaevsk-na-amure.aeroport.website': 'aeroport.website',
 'izhevsk.aeroport.website': 'aeroport.website',
 'emelyanovo.aeroport.website': 'aeroport.website',
 'sharm-el-sheikh.aeroport.website': 'aeroport.website',
 'usinsk.aeroport.website': 'aeroport.website',
 'voronezh.aeroport.website': 'aeroport.website',
 'sochi.aeroport.website': 'aeroport.website',
 'chkalovskij.aeroport.website': 'aeroport.website',
 'gyumri.aeroport.website': 'aeroport.website',
 'ignatevo.aeroport.website': 'aeroport.website',
 'anapa.aeroport.website': 'aeroport.website',
 'gazipasha.aeroport.website': 'aeroport.website',
 'gelendzhik.aeroport.website': 'aeroport.website',
 'koltsovo.aeroport.website': 'aeroport.website',
 'kursk.aeroport.website': 'aeroport.website',
 'polyarnyj.aeroport.website': 'aeroport.website',
 'barataevka.aeroport.website': 'aeroport.website',
 'bratsk.aeroport.website': 'aeroport.website',
 'mirnyj.aeroport.website': 'aeroport.website',
 'sevastopol.aeroport.website': 'aeroport.website',
 'alykel.aeroport.website': 'aeroport.website',
 'dushanbe.aeroport.website': 'aeroport.website',
 'krasnodar.aeroport.website': 'aeroport.website',
 'balandino.aeroport.website': 'aeroport.website',
 'kaluga.aeroport.website': 'aeroport.website',
 'simferopol.aeroport.website': 'aeroport.website',
 'tambov.aeroport.website': 'aeroport.website',
 'arhangelsk.aeroport.website': 'aeroport.website',
 'hrabrovo.aeroport.website': 'aeroport.website',
 'iturup.aeroport.website': 'aeroport.website',
 'astrakhan.aeroport.website': 'aeroport.website',
 'orsk.aeroport.website': 'aeroport.website',
 'groznyj.aeroport.website': 'aeroport.website',
 'talakan.aeroport.website': 'aeroport.website',
 'strigino.aeroport.website': 'aeroport.website',
 'taganrog.aeroport.website': 'aeroport.website',
 'pskov.aeroport.website': 'aeroport.website',
 'syktyvkar.aeroport.website': 'aeroport.website',
 'dubai.aeroport.website': 'aeroport.website',
 'igarka.aeroport.website': 'aeroport.website',
 'kurumoch.aeroport.website': 'aeroport.website',
 'cherepovec.aeroport.website': 'aeroport.website',
 'hurgada.aeroport.website': 'aeroport.website',
 'hanty-mansijsk.aeroport.website': 'aeroport.website',
 'habarovsk.aeroport.website': 'aeroport.website',
 'cheboksary.aeroport.website': 'aeroport.website',
 'tiksi.aeroport.website': 'aeroport.website',
 'varadero.aeroport.website': 'aeroport.website',
 'aeroport.website': 'aeroport.website',
 'osh.aeroport.website': 'aeroport.website',
 'vladikavkaz.aeroport.website': 'aeroport.website',
 'abakan.aeroport.website': 'aeroport.website',
 'neryungri.aeroport.website': 'aeroport.website',
 'zvartnots.aeroport.website': 'aeroport.website',
 'namangan.aeroport.website': 'aeroport.website',
 'platov.aeroport.website': 'aeroport.website',
 'grodno.aeroport.website': 'aeroport.website',
 'komsomolsk-na-amure.aeroport.website': 'aeroport.website',
 'belgorod.aeroport.website': 'aeroport.website',
 'tyumen.aeroport.website': 'aeroport.website',
 'bykovo.aeroport.website': 'aeroport.website',
 'orenburg.aeroport.website': 'aeroport.website',
 'perm.aeroport.website': 'aeroport.website',
 'zhukovskiy.aeroport.website': 'aeroport.website',
 'penza.aeroport.website': 'aeroport.website',
 'petrozavodsk.aeroport.website': 'aeroport.website',
 'kair.aeroport.website': 'aeroport.website',
 'varna.aeroport.website': 'aeroport.website',
 'sabiha-gekchen.aeroport.website': 'aeroport.website',
    'miltor.ru': 'miltor.ru',
 'kirov3.miltor.ru': 'miltor.ru',
 'millerovo.miltor.ru': 'miltor.ru',
 'shumerlya.miltor.ru': 'miltor.ru',
 'bugulma.miltor.ru': 'miltor.ru',
 'rasskazovo.miltor.ru': 'miltor.ru',
 'skopin.miltor.ru': 'miltor.ru',
 'krasnoyarsk.miltor.ru': 'miltor.ru',
 'anapa.miltor.ru': 'miltor.ru',
 'rtishchevo.miltor.ru': 'miltor.ru',
 'volgograd.miltor.ru': 'miltor.ru',
 'beloretsk.miltor.ru': 'miltor.ru',
 'sterlitamak.miltor.ru': 'miltor.ru',
 'novosibirsk.miltor.ru': 'miltor.ru',
 'chapaevsk.miltor.ru': 'miltor.ru',
 'iskitim.miltor.ru': 'miltor.ru',
 'ob.miltor.ru': 'miltor.ru',
 'yuzhnouralsk.miltor.ru': 'miltor.ru',
 'timashevsk.miltor.ru': 'miltor.ru',
 'tuapse.miltor.ru': 'miltor.ru',
 'revda.miltor.ru': 'miltor.ru',
 'belogorsk1.miltor.ru': 'miltor.ru',
 'shatura.miltor.ru': 'miltor.ru',
 'izhevsk.miltor.ru': 'miltor.ru',
 'kurchatov.miltor.ru': 'miltor.ru',
 'tinda.miltor.ru': 'miltor.ru',
 'volsk.miltor.ru': 'miltor.ru',
 'kudimkar.miltor.ru': 'miltor.ru',
 'kazan.miltor.ru': 'miltor.ru',
 'staraya-russa.miltor.ru': 'miltor.ru',
 'volhov.miltor.ru': 'miltor.ru',
 'aleksin.miltor.ru': 'miltor.ru',
 'kolomna.miltor.ru': 'miltor.ru',
 'voronezh1.miltor.ru': 'miltor.ru',
 'magadan.miltor.ru': 'miltor.ru',
 'volokolamsk.miltor.ru': 'miltor.ru',
 'temryuk.miltor.ru': 'miltor.ru',
 'tula.miltor.ru': 'miltor.ru',
 'sizran.miltor.ru': 'miltor.ru',
 'rezh.miltor.ru': 'miltor.ru',
 'siktivkar.miltor.ru': 'miltor.ru',
 'rostov-velikiy.miltor.ru': 'miltor.ru',
 'ivanovo.miltor.ru': 'miltor.ru',
 'svetlograd.miltor.ru': 'miltor.ru',
 'kurganinsk.miltor.ru': 'miltor.ru',
 'kungur.miltor.ru': 'miltor.ru',
 'habarovsk.miltor.ru': 'miltor.ru',
 'tyumen1.miltor.ru': 'miltor.ru',
 'kansk.miltor.ru': 'miltor.ru',
 'abakan.miltor.ru': 'miltor.ru',
 'engels.miltor.ru': 'miltor.ru',
 'uglich.miltor.ru': 'miltor.ru',
 'yurga1.miltor.ru': 'miltor.ru',
 'zelenokumsk.miltor.ru': 'miltor.ru',
 'kanash.miltor.ru': 'miltor.ru',
 'sol-iletsk.miltor.ru': 'miltor.ru',
 'gorno-altaysk.miltor.ru': 'miltor.ru',
 'leninogorsk.miltor.ru': 'miltor.ru',
 'krasnodar.miltor.ru': 'miltor.ru',
 'grozniy.miltor.ru': 'miltor.ru',
 'saransk.miltor.ru': 'miltor.ru',
 'kizlyar.miltor.ru': 'miltor.ru',
 'amursk.miltor.ru': 'miltor.ru',
 'konakovo.miltor.ru': 'miltor.ru',
 'gorodets.miltor.ru': 'miltor.ru',
 'tambov.miltor.ru': 'miltor.ru',
 'vyatskie-polyani.miltor.ru': 'miltor.ru',
 'slavyansk-na-kubani.miltor.ru': 'miltor.ru',
 'asbest.miltor.ru': 'miltor.ru',
 'eysk.miltor.ru': 'miltor.ru',
 'derbent.miltor.ru': 'miltor.ru',
 'bogorodsk2.miltor.ru': 'miltor.ru',
 'lipetsk.miltor.ru': 'miltor.ru',
 'gryazi.miltor.ru': 'miltor.ru',
 'kuybishev.miltor.ru': 'miltor.ru',
 'ishim.miltor.ru': 'miltor.ru',
 'krimsk.miltor.ru': 'miltor.ru',
 'shodnya.miltor.ru': 'miltor.ru',
 'ekaterinburg.miltor.ru': 'miltor.ru',
 'omsk.miltor.ru': 'miltor.ru',
 'penza.miltor.ru': 'miltor.ru',
 'glazov.miltor.ru': 'miltor.ru',
 'roslavl.miltor.ru': 'miltor.ru',
 'novozibkov.miltor.ru': 'miltor.ru',
 'novohopersk.miltor.ru': 'miltor.ru',
 'abinsk.miltor.ru': 'miltor.ru',
 'rostov-na-donu.miltor.ru': 'miltor.ru',
 'opochka.miltor.ru': 'miltor.ru',
 'vladimir.miltor.ru': 'miltor.ru',
    'diamondelectric.ru': 'diamondelectric.ru',
 'sankt-peterburg.diamondelectric.ru': 'diamondelectric.ru',
 'pereslavl-zalesskiy.diamondelectric.ru': 'diamondelectric.ru',
 'pavlovskiy-posad.diamondelectric.ru': 'diamondelectric.ru',
 'saransk.diamondelectric.ru': 'diamondelectric.ru',
 'ufa.diamondelectric.ru': 'diamondelectric.ru',
 'georgievsk.diamondelectric.ru': 'diamondelectric.ru',
 'rostov-na-donu.diamondelectric.ru': 'diamondelectric.ru',
 'naberezhnye-chelny.diamondelectric.ru': 'diamondelectric.ru',
 'barnaul.diamondelectric.ru': 'diamondelectric.ru',
 'sibay.diamondelectric.ru': 'diamondelectric.ru',
 'brody.diamondelectric.ru': 'diamondelectric.ru',
 'severobaykalsk.diamondelectric.ru': 'diamondelectric.ru',
 'voronezh.diamondelectric.ru': 'diamondelectric.ru',
 'nevinnomyssk.diamondelectric.ru': 'diamondelectric.ru',
 'pechora.diamondelectric.ru': 'diamondelectric.ru',
 'sysert.diamondelectric.ru': 'diamondelectric.ru',
 'abakan.diamondelectric.ru': 'diamondelectric.ru',
 'borovichi.diamondelectric.ru': 'diamondelectric.ru',
 'borisoglebsk.diamondelectric.ru': 'diamondelectric.ru',
 'perm.diamondelectric.ru': 'diamondelectric.ru',
 'vzmore.diamondelectric.ru': 'diamondelectric.ru',
 'nizhniy-novgorod.diamondelectric.ru': 'diamondelectric.ru',
 'gukovo.diamondelectric.ru': 'diamondelectric.ru',
 'novosibirsk.diamondelectric.ru': 'diamondelectric.ru',
 'balakovo.diamondelectric.ru': 'diamondelectric.ru',
 'krasnodar.diamondelectric.ru': 'diamondelectric.ru',
 'kazan.diamondelectric.ru': 'diamondelectric.ru',
 'saraktash.diamondelectric.ru': 'diamondelectric.ru',
 'rubezhnoe.diamondelectric.ru': 'diamondelectric.ru',
 'kamensk-uralskiy.diamondelectric.ru': 'diamondelectric.ru',
 'yaroslavl.diamondelectric.ru': 'diamondelectric.ru',
 'berezniki.diamondelectric.ru': 'diamondelectric.ru',
 'tula.diamondelectric.ru': 'diamondelectric.ru',
 'korsakov.diamondelectric.ru': 'diamondelectric.ru',
 'balabanovo.diamondelectric.ru': 'diamondelectric.ru',
 'vladivostok.diamondelectric.ru': 'diamondelectric.ru',
 'marks.diamondelectric.ru': 'diamondelectric.ru',
 'vladimir.diamondelectric.ru': 'diamondelectric.ru',
 'iskitim.diamondelectric.ru': 'diamondelectric.ru',
 'yurga.diamondelectric.ru': 'diamondelectric.ru',
 'ekaterinburg.diamondelectric.ru': 'diamondelectric.ru',
 'yanaul.diamondelectric.ru': 'diamondelectric.ru',
 'anzhero-sudzhensk.diamondelectric.ru': 'diamondelectric.ru',
 'novorossiysk.diamondelectric.ru': 'diamondelectric.ru',
 'sevastopol.diamondelectric.ru': 'diamondelectric.ru',
 'baryshevka.diamondelectric.ru': 'diamondelectric.ru',
 'petrozavodsk.diamondelectric.ru': 'diamondelectric.ru',
 'minusinsk.diamondelectric.ru': 'diamondelectric.ru',
 'velikiy-novgorod.diamondelectric.ru': 'diamondelectric.ru',
 'syzran.diamondelectric.ru': 'diamondelectric.ru',
 'lugansk.diamondelectric.ru': 'diamondelectric.ru',
 'ivanovo.diamondelectric.ru': 'diamondelectric.ru',
 'mineralnye-vody.diamondelectric.ru': 'diamondelectric.ru',
 'belgorod.diamondelectric.ru': 'diamondelectric.ru',
 'velikie-luki.diamondelectric.ru': 'diamondelectric.ru',
 'ovruch.diamondelectric.ru': 'diamondelectric.ru',
 'burshtyn.diamondelectric.ru': 'diamondelectric.ru',
 'izhevsk.diamondelectric.ru': 'diamondelectric.ru',
 'mozhaysk.diamondelectric.ru': 'diamondelectric.ru',
 'cherepovets.diamondelectric.ru': 'diamondelectric.ru',
 'samara.diamondelectric.ru': 'diamondelectric.ru',
 'dolgoprudnyy.diamondelectric.ru': 'diamondelectric.ru',
 'ussuriysk.diamondelectric.ru': 'diamondelectric.ru',
 'luhovitsy.diamondelectric.ru': 'diamondelectric.ru',
 'tver.diamondelectric.ru': 'diamondelectric.ru',
 'vilyuchinsk.diamondelectric.ru': 'diamondelectric.ru',
 'zelenogorsk.diamondelectric.ru': 'diamondelectric.ru',
 'orlovskiy.diamondelectric.ru': 'diamondelectric.ru',
 'boguslav.diamondelectric.ru': 'diamondelectric.ru',
 'tomsk.diamondelectric.ru': 'diamondelectric.ru',
 'timashevsk.diamondelectric.ru': 'diamondelectric.ru',
 'desna.diamondelectric.ru': 'diamondelectric.ru',
 'armavir.diamondelectric.ru': 'diamondelectric.ru',
 'zhukovka.diamondelectric.ru': 'diamondelectric.ru',
 'yakutsk.diamondelectric.ru': 'diamondelectric.ru',
 'chelyabinsk.diamondelectric.ru': 'diamondelectric.ru',
 'syktyvkar.diamondelectric.ru': 'diamondelectric.ru',
 'lisichansk.diamondelectric.ru': 'diamondelectric.ru',
 'kaliningrad.diamondelectric.ru': 'diamondelectric.ru',
 'vyshgorod.diamondelectric.ru': 'diamondelectric.ru',
 'novotroitsk.diamondelectric.ru': 'diamondelectric.ru',
 'obninsk.diamondelectric.ru': 'diamondelectric.ru',
 'gulkevichi.diamondelectric.ru': 'diamondelectric.ru',
 'habarovsk.diamondelectric.ru': 'diamondelectric.ru',
 'shumerlya.diamondelectric.ru': 'diamondelectric.ru',
 'orenburg.diamondelectric.ru': 'diamondelectric.ru',
 'dobryanka.diamondelectric.ru': 'diamondelectric.ru',
 'lebedin.diamondelectric.ru': 'diamondelectric.ru',
    'holme.ru': 'holme.ru',
 'krd.holme.ru': 'holme.ru',
 'vrn.holme.ru': 'holme.ru',
 'ufa.holme.ru': 'holme.ru',
 'orsk.holme.ru': 'holme.ru',
 'perm.holme.ru': 'holme.ru',
 'spb.holme.ru': 'holme.ru',
 'rzn.holme.ru': 'holme.ru',
 'yeisk.holme.ru': 'holme.ru',
 'tumen.holme.ru': 'holme.ru',
 'smr.holme.ru': 'holme.ru',
 'izh.holme.ru': 'holme.ru',
 'chb.holme.ru': 'holme.ru',
 'vlg.holme.ru': 'holme.ru',
 'novokuz.holme.ru': 'holme.ru',
 'ul.holme.ru': 'holme.ru',
 'kursk.holme.ru': 'holme.ru',
 'nk.holme.ru': 'holme.ru',
 'ornb.holme.ru': 'holme.ru',
 'yola.holme.ru': 'holme.ru',
 'dzerzhinsk.holme.ru': 'holme.ru',
 'tagil.holme.ru': 'holme.ru',
 'nabch.holme.ru': 'holme.ru',
 'penza.holme.ru': 'holme.ru',
 'nails.holme.ru': 'holme.ru',
 'rnd.holme.ru': 'holme.ru',
 'novosib.holme.ru': 'holme.ru',
 'barnaul.holme.ru': 'holme.ru',
 'vladikavkaz.holme.ru': 'holme.ru',
 'kzn.holme.ru': 'holme.ru',
 'shahty.holme.ru': 'holme.ru',
 'petrozavodsk.holme.ru': 'holme.ru',
 'surgut.holme.ru': 'holme.ru',
 'saratov.holme.ru': 'holme.ru',
 'lip.holme.ru': 'holme.ru',
 'balashiha.holme.ru': 'holme.ru',
 'ekb.holme.ru': 'holme.ru',
 'podolsk.holme.ru': 'holme.ru',
 'yar.holme.ru': 'holme.ru',
 'sp.holme.ru': 'holme.ru',
 'zhelezno.holme.ru': 'holme.ru',
 'vologda.holme.ru': 'holme.ru',
 'nn.holme.ru': 'holme.ru',
 'himki.holme.ru': 'holme.ru',
 'vladivostok.holme.ru': 'holme.ru',
 'orel.holme.ru': 'holme.ru',
 'chlb.holme.ru': 'holme.ru',
 'kostroma.holme.ru': 'holme.ru',
 'saransk.holme.ru': 'holme.ru',
 'elektrostal.holme.ru': 'holme.ru',
 'vladimir.holme.ru': 'holme.ru',
 'sterlitamak.holme.ru': 'holme.ru',
 'novoros.holme.ru': 'holme.ru',
 'tambov.holme.ru': 'holme.ru',
 'cherep.holme.ru': 'holme.ru',
 'pskov.holme.ru': 'holme.ru',
 'mo.holme.ru': 'holme.ru',
 'tlt.holme.ru': 'holme.ru',
 'kirov.holme.ru': 'holme.ru',
 'murmansk.holme.ru': 'holme.ru',
 'shelkovo.holme.ru': 'holme.ru',
 'klng.holme.ru': 'holme.ru',
 'bryansk.holme.ru': 'holme.ru',
 'sykt.holme.ru': 'holme.ru',
 'stav.holme.ru': 'holme.ru',
 'kaluga.holme.ru': 'holme.ru',
 'taganrog.holme.ru': 'holme.ru',
 'maykop.holme.ru': 'holme.ru',
 'ivanovo.holme.ru': 'holme.ru',
 'miass.holme.ru': 'holme.ru',
 'kurgan.holme.ru': 'holme.ru',
 'moscow.holme.ru': 'holme.ru',
 'zhuk.holme.ru': 'holme.ru',
 'tula.holme.ru': 'holme.ru',
 'arhangelsk.holme.ru': 'holme.ru',
 'gatchina.holme.ru': 'holme.ru',
 'mir.holme.ru': 'holme.ru',
 'tver.holme.ru': 'holme.ru',
 'ast.holme.ru': 'holme.ru',
 'magnit.holme.ru': 'holme.ru',
 'omsk.holme.ru': 'holme.ru',
 'sochi.holme.ru': 'holme.ru',
 'zelenograd.holme.ru': 'holme.ru',
 'krasnogorsk.holme.ru': 'holme.ru',
 'mytishi.holme.ru': 'holme.ru',
 'msk.holme.ru': 'holme.ru',
 'buzuluk.holme.ru': 'holme.ru',
 'smolensk.holme.ru': 'holme.ru',
 'hm.holme.ru': 'holme.ru',
 'kras.holme.ru': 'holme.ru',
    'kaliningrad.zoon.ru': 'zoon.ru',
 'spb.zoon.ru': 'zoon.ru',
 'zoon.ru': 'zoon.ru',
 'krasnoyarsk.zoon.ru': 'zoon.ru',
 'ivanovo.zoon.ru': 'zoon.ru',
 'stavropol.zoon.ru': 'zoon.ru',
 'ekb.zoon.ru': 'zoon.ru',
 'omsk.zoon.ru': 'zoon.ru',
 'nizhnevartovsk.zoon.ru': 'zoon.ru',
 'bryansk.zoon.ru': 'zoon.ru',
 'perm.zoon.ru': 'zoon.ru',
 'chelny.zoon.ru': 'zoon.ru',
 'izhevsk.zoon.ru': 'zoon.ru',
 'krasnodar.zoon.ru': 'zoon.ru',
 'samara.zoon.ru': 'zoon.ru',
 'abakan.zoon.ru': 'zoon.ru',
 'ryazan.zoon.ru': 'zoon.ru',
 'smolensk.zoon.ru': 'zoon.ru',
 'sochi.zoon.ru': 'zoon.ru',
 'chelyabinsk.zoon.ru': 'zoon.ru',
 'tyumen.zoon.ru': 'zoon.ru',
 'rostov.zoon.ru': 'zoon.ru',
 'volgograd.zoon.ru': 'zoon.ru',
 'kurgan.zoon.ru': 'zoon.ru',
 'penza.zoon.ru': 'zoon.ru',
 'nn.zoon.ru': 'zoon.ru',
 'saratov.zoon.ru': 'zoon.ru',
 'ufa.zoon.ru': 'zoon.ru',
 'kemerovo.zoon.ru': 'zoon.ru',
 'khabarovsk.zoon.ru': 'zoon.ru',
 'joshkar-ola.zoon.ru': 'zoon.ru',
 'cheboksary.zoon.ru': 'zoon.ru',
 'vladivostok.zoon.ru': 'zoon.ru',
 'astrakhan.zoon.ru': 'zoon.ru',
 'nsk.zoon.ru': 'zoon.ru',
 'tambov.zoon.ru': 'zoon.ru',
 'kaluga.zoon.ru': 'zoon.ru',
 'tomsk.zoon.ru': 'zoon.ru',
 'blag.zoon.ru': 'zoon.ru',
 'orel.zoon.ru': 'zoon.ru',
 'togliatti.zoon.ru': 'zoon.ru',
 'surgut.zoon.ru': 'zoon.ru',
 'saransk.zoon.ru': 'zoon.ru',
 'yaroslavl.zoon.ru': 'zoon.ru',
 'vologda.zoon.ru': 'zoon.ru',
 'kazan.zoon.ru': 'zoon.ru',
 'lipetsk.zoon.ru': 'zoon.ru',
 'vladikavkaz.zoon.ru': 'zoon.ru',
 'arkhangelsk.zoon.ru': 'zoon.ru',
 'magnitogorsk.zoon.ru': 'zoon.ru',
 'kirov.zoon.ru': 'zoon.ru',
 'sterlitamak.zoon.ru': 'zoon.ru',
 'tagil.zoon.ru': 'zoon.ru',
 'orenburg.zoon.ru': 'zoon.ru',
 'barnaul.zoon.ru': 'zoon.ru',
 'bratsk.zoon.ru': 'zoon.ru',
 'novokuznetsk.zoon.ru': 'zoon.ru',
 'ulan-ude.zoon.ru': 'zoon.ru',
 'tver.zoon.ru': 'zoon.ru',
 'irkutsk.zoon.ru': 'zoon.ru',
 'oskol.zoon.ru': 'zoon.ru',
 'petropavlovsk-kamchatskiy.zoon.ru': 'zoon.ru',
 'pskov.zoon.ru': 'zoon.ru',
 'biysk.zoon.ru': 'zoon.ru',
 'murmansk.zoon.ru': 'zoon.ru',
 'gorno-altaisk.zoon.ru': 'zoon.ru',
 'petrozavodsk.zoon.ru': 'zoon.ru',
 'syktyvkar.zoon.ru': 'zoon.ru',
 'simferopol.zoon.ru': 'zoon.ru',
 'ulyanovsk.zoon.ru': 'zoon.ru',
 'yuzhnosakhalinsk.zoon.ru': 'zoon.ru',
 'novorossiysk.zoon.ru': 'zoon.ru',
 'belgorod.zoon.ru': 'zoon.ru',
 'vladimir.zoon.ru': 'zoon.ru',
 'orsk.zoon.ru': 'zoon.ru',
 'nalchik.zoon.ru': 'zoon.ru',
 'neftekamsk.zoon.ru': 'zoon.ru',
 'taganrog.zoon.ru': 'zoon.ru',
 'chita.zoon.ru': 'zoon.ru',
 'yakutsk.zoon.ru': 'zoon.ru',
 'makhachkala.zoon.ru': 'zoon.ru',
 'maykop.zoon.ru': 'zoon.ru',
 'grozny.zoon.ru': 'zoon.ru',
 'khanty-mansiysk.zoon.ru': 'zoon.ru',
 'nazran.zoon.ru': 'zoon.ru',
 'cherkessk.zoon.ru': 'zoon.ru',
 'shahty.zoon.ru': 'zoon.ru',
 'cherepovets.zoon.ru': 'zoon.ru',
 'salekhard.zoon.ru': 'zoon.ru',
 'anadyr.zoon.ru': 'zoon.ru',
    'tula.big-book-city.ru': 'big-book-city.ru',
 'izhevsk.big-book-city.ru': 'big-book-city.ru',
 'novosibirsk.big-book-city.ru': 'big-book-city.ru',
 'big-book-city.ru': 'big-book-city.ru',
 'petrozavodsk.big-book-city.ru': 'big-book-city.ru',
 'zhukovskij.big-book-city.ru': 'big-book-city.ru',
 'ulyanovsk.big-book-city.ru': 'big-book-city.ru',
 'oktyabrsky.big-book-city.ru': 'big-book-city.ru',
 'tomsk.big-book-city.ru': 'big-book-city.ru',
 'kazan.big-book-city.ru': 'big-book-city.ru',
 'vladimir.big-book-city.ru': 'big-book-city.ru',
 'perm.big-book-city.ru': 'big-book-city.ru',
 'neftekamsk.big-book-city.ru': 'big-book-city.ru',
 'saratov.big-book-city.ru': 'big-book-city.ru',
 'samara.big-book-city.ru': 'big-book-city.ru',
 'balashikha.big-book-city.ru': 'big-book-city.ru',
 'moscow.big-book-city.ru': 'big-book-city.ru',
 'naberezhnyye-chelny.big-book-city.ru': 'big-book-city.ru',
 'cheboksary.big-book-city.ru': 'big-book-city.ru',
 'tver.big-book-city.ru': 'big-book-city.ru',
 'taganrog.big-book-city.ru': 'big-book-city.ru',
 'nizhniy-novgorod.big-book-city.ru': 'big-book-city.ru',
 'smolensk.big-book-city.ru': 'big-book-city.ru',
 'almetyevsk.big-book-city.ru': 'big-book-city.ru',
 'krasnodar.big-book-city.ru': 'big-book-city.ru',
 'khabarovsk.big-book-city.ru': 'big-book-city.ru',
 'pyatigorsk.big-book-city.ru': 'big-book-city.ru',
 'sterlitamak.big-book-city.ru': 'big-book-city.ru',
 'astrakhan.big-book-city.ru': 'big-book-city.ru',
 'sochi.big-book-city.ru': 'big-book-city.ru',
 'cherepovets.big-book-city.ru': 'big-book-city.ru',
 'ufa.big-book-city.ru': 'big-book-city.ru',
 'volgograd.big-book-city.ru': 'big-book-city.ru',
 'vologda.big-book-city.ru': 'big-book-city.ru',
 'chelyabinsk.big-book-city.ru': 'big-book-city.ru',
 'bryansk.big-book-city.ru': 'big-book-city.ru',
 'dzerzhinsk.big-book-city.ru': 'big-book-city.ru',
 'voronezh.big-book-city.ru': 'big-book-city.ru',
 'syktyvkar.big-book-city.ru': 'big-book-city.ru',
 'kirov.big-book-city.ru': 'big-book-city.ru',
 'novorossiysk.big-book-city.ru': 'big-book-city.ru',
 'komsomolsk-na-amure.big-book-city.ru': 'big-book-city.ru',
 'lipetsk.big-book-city.ru': 'big-book-city.ru',
 'penza.big-book-city.ru': 'big-book-city.ru',
 'saint-petersburg.big-book-city.ru': 'big-book-city.ru',
 'armavir.big-book-city.ru': 'big-book-city.ru',
 'novocherkask.big-book-city.ru': 'big-book-city.ru',
 'yeisk.big-book-city.ru': 'big-book-city.ru',
 'nevinnomissk.big-book-city.ru': 'big-book-city.ru',
 'tyumen.big-book-city.ru': 'big-book-city.ru',
 'tolyatti.big-book-city.ru': 'big-book-city.ru',
 'saransk.big-book-city.ru': 'big-book-city.ru',
 'pskov.big-book-city.ru': 'big-book-city.ru',
 'kurgan.big-book-city.ru': 'big-book-city.ru',
 'novomoscowsk.big-book-city.ru': 'big-book-city.ru',
 'orsk.big-book-city.ru': 'big-book-city.ru',
 'yaroslavl.big-book-city.ru': 'big-book-city.ru',
 'berezniki.big-book-city.ru': 'big-book-city.ru',
 'krasnoyarsk.big-book-city.ru': 'big-book-city.ru',
 'magnitogorsk.big-book-city.ru': 'big-book-city.ru',
 'yakutsk.big-book-city.ru': 'big-book-city.ru',
 'kamensk-yralski.big-book-city.ru': 'big-book-city.ru',
 'rostov-na-donu.big-book-city.ru': 'big-book-city.ru',
 'koroljov.big-book-city.ru': 'big-book-city.ru',
 'tobolsk.big-book-city.ru': 'big-book-city.ru',
 'khimki.big-book-city.ru': 'big-book-city.ru',
 'biysk.big-book-city.ru': 'big-book-city.ru',
 'kolomna.big-book-city.ru': 'big-book-city.ru',
 'belgorod.big-book-city.ru': 'big-book-city.ru',
 'velikiy-novgorod.big-book-city.ru': 'big-book-city.ru',
 'shakhty.big-book-city.ru': 'big-book-city.ru',
 'oryol.big-book-city.ru': 'big-book-city.ru',
 'yekaterinburg.big-book-city.ru': 'big-book-city.ru',
 'ljubercy.big-book-city.ru': 'big-book-city.ru',
 'novokuznetsk.big-book-city.ru': 'big-book-city.ru',
 'engels.big-book-city.ru': 'big-book-city.ru',
 'sizran.big-book-city.ru': 'big-book-city.ru',
 'balakovo.big-book-city.ru': 'big-book-city.ru',
 'zlatoust.big-book-city.ru': 'big-book-city.ru',
 'kursk.big-book-city.ru': 'big-book-city.ru',
 'serpuhov.big-book-city.ru': 'big-book-city.ru',
 'maykop.big-book-city.ru': 'big-book-city.ru',
 'surgut.big-book-city.ru': 'big-book-city.ru',
 'ivanovo.big-book-city.ru': 'big-book-city.ru',
 'abakan.big-book-city.ru': 'big-book-city.ru',
 'orenburg.big-book-city.ru': 'big-book-city.ru',
 'pushkino.big-book-city.ru': 'big-book-city.ru',
 'kaliningrad.big-book-city.ru': 'big-book-city.ru',
 'viborg.big-book-city.ru': 'big-book-city.ru',
 'zheleznodorozhnyj.big-book-city.ru': 'big-book-city.ru',
 'vladivostok.big-book-city.ru': 'big-book-city.ru',
 'petropavlovsk-kamchatskiy.big-book-city.ru': 'big-book-city.ru',
 'volgodonsk.big-book-city.ru': 'big-book-city.ru',
 'orehovo-zuevo.big-book-city.ru': 'big-book-city.ru',
 'barnaul.big-book-city.ru': 'big-book-city.ru',
    'proshoper.ru': 'proshoper.ru',
 'petrozavodsk.proshoper.ru': 'proshoper.ru',
 'mos.proshoper.ru': 'proshoper.ru',
 'egorevsk.proshoper.ru': 'proshoper.ru',
 'ufa.proshoper.ru': 'proshoper.ru',
 'novorossijsk.proshoper.ru': 'proshoper.ru',
 'novgorod.proshoper.ru': 'proshoper.ru',
 'moskva.proshoper.ru': 'proshoper.ru',
 'staryi-oskol.proshoper.ru': 'proshoper.ru',
 'tambov.proshoper.ru': 'proshoper.ru',
 'saransk.proshoper.ru': 'proshoper.ru',
 'nizhnii-novgorod.proshoper.ru': 'proshoper.ru',
 'barnaul.proshoper.ru': 'proshoper.ru',
 'ekaterinburg.proshoper.ru': 'proshoper.ru',
 'spb.proshoper.ru': 'proshoper.ru',
 'yoshkar-ola.proshoper.ru': 'proshoper.ru',
 'voronezh.proshoper.ru': 'proshoper.ru',
 'omsk.proshoper.ru': 'proshoper.ru',
 'kazan.proshoper.ru': 'proshoper.ru',
 'nn.proshoper.ru': 'proshoper.ru',
 'tver.proshoper.ru': 'proshoper.ru',
 'rostov.proshoper.ru': 'proshoper.ru',
 'novosibirsk.proshoper.ru': 'proshoper.ru',
 'krasnoyarsk.proshoper.ru': 'proshoper.ru',
 'prokopevsk.proshoper.ru': 'proshoper.ru',
 'kaluga.proshoper.ru': 'proshoper.ru',
 'samara.proshoper.ru': 'proshoper.ru',
 'ulyanovsk.proshoper.ru': 'proshoper.ru',
 'naberezhnye.proshoper.ru': 'proshoper.ru',
 'velikie-luki.proshoper.ru': 'proshoper.ru',
 'almetevsk.proshoper.ru': 'proshoper.ru',
 'vologda.proshoper.ru': 'proshoper.ru',
 'krasnogorsk.proshoper.ru': 'proshoper.ru',
 'penza.proshoper.ru': 'proshoper.ru',
 'bryansk.proshoper.ru': 'proshoper.ru',
 'balashiha.proshoper.ru': 'proshoper.ru',
 'orenburg.proshoper.ru': 'proshoper.ru',
 'ramenskoe.proshoper.ru': 'proshoper.ru',
 'dzerzhinsk.proshoper.ru': 'proshoper.ru',
 'ryazan.proshoper.ru': 'proshoper.ru',
 'mytishi.proshoper.ru': 'proshoper.ru',
 'kolomna.proshoper.ru': 'proshoper.ru',
 'magnitogorsk.proshoper.ru': 'proshoper.ru',
 'murom.proshoper.ru': 'proshoper.ru',
 'kotelniki.proshoper.ru': 'proshoper.ru',
 'lipetsk.proshoper.ru': 'proshoper.ru',
 'yaroslavl.proshoper.ru': 'proshoper.ru',
 'vladimir.proshoper.ru': 'proshoper.ru',
 'tyumen.proshoper.ru': 'proshoper.ru',
 'dmitrov.proshoper.ru': 'proshoper.ru',
 'chelyabinsk.proshoper.ru': 'proshoper.ru',
 'kursk.proshoper.ru': 'proshoper.ru',
 'saratov.proshoper.ru': 'proshoper.ru',
 'cheboksaryi.proshoper.ru': 'proshoper.ru',
 'krasnodar.proshoper.ru': 'proshoper.ru',
 'klin.proshoper.ru': 'proshoper.ru',
 'nizhnij-tagil.proshoper.ru': 'proshoper.ru',
 'smolensk.proshoper.ru': 'proshoper.ru',
 'sochi.proshoper.ru': 'proshoper.ru',
 'orel.proshoper.ru': 'proshoper.ru',
 'podolsk.proshoper.ru': 'proshoper.ru',
 'engels.proshoper.ru': 'proshoper.ru',
 'obninsk.proshoper.ru': 'proshoper.ru',
 'vyiksa.proshoper.ru': 'proshoper.ru',
 'stavropol.proshoper.ru': 'proshoper.ru',
 'lyubercy.proshoper.ru': 'proshoper.ru',
 'himki.proshoper.ru': 'proshoper.ru',
 'sergiev-posad.proshoper.ru': 'proshoper.ru',
 'taganrog.proshoper.ru': 'proshoper.ru',
 'pushkino.proshoper.ru': 'proshoper.ru',
 'ivanovo.proshoper.ru': 'proshoper.ru',
 'voskresensk.proshoper.ru': 'proshoper.ru',
 'zheleznogorsk.proshoper.ru': 'proshoper.ru',
 'korolev.proshoper.ru': 'proshoper.ru',
 'izhevsk.proshoper.ru': 'proshoper.ru',
 'pskov.proshoper.ru': 'proshoper.ru',
 'volgograd.proshoper.ru': 'proshoper.ru',
 'elektrostal.proshoper.ru': 'proshoper.ru',
 'revda.proshoper.ru': 'proshoper.ru',
 'perm.proshoper.ru': 'proshoper.ru',
 'vyborg.proshoper.ru': 'proshoper.ru',
 'odincovo.proshoper.ru': 'proshoper.ru',
 'sterlitamak.proshoper.ru': 'proshoper.ru',
 'serpuhov.proshoper.ru': 'proshoper.ru',
 'surgut.proshoper.ru': 'proshoper.ru',
 'nizhnekamsk.proshoper.ru': 'proshoper.ru',
 'zhukovskij.proshoper.ru': 'proshoper.ru',
 'armavir.proshoper.ru': 'proshoper.ru',
 'tula.proshoper.ru': 'proshoper.ru',
 'vladikavkaz.proshoper.ru': 'proshoper.ru',
 'pyatigorsk.proshoper.ru': 'proshoper.ru',
 'belgorod.proshoper.ru': 'proshoper.ru',
 'elec.proshoper.ru': 'proshoper.ru',
 'kaliningrad.proshoper.ru': 'proshoper.ru',
 'kirov.proshoper.ru': 'proshoper.ru',
 'tolyatti.proshoper.ru': 'proshoper.ru',
 'solnechnogorsk.proshoper.ru': 'proshoper.ru',
    '6657.svetacdn.in': 'svetacdn.in',
 '7224.svetacdn.in': 'svetacdn.in',
 '42.svetacdn.in': 'svetacdn.in',
 '7720.svetacdn.in': 'svetacdn.in',
 '7923.svetacdn.in': 'svetacdn.in',
 '6643.svetacdn.in': 'svetacdn.in',
 '7741.svetacdn.in': 'svetacdn.in',
 '2413.svetacdn.in': 'svetacdn.in',
 '7373.svetacdn.in': 'svetacdn.in',
 '32.svetacdn.in': 'svetacdn.in',
 '18.svetacdn.in': 'svetacdn.in',
 '8209.svetacdn.in': 'svetacdn.in',
 '6556.svetacdn.in': 'svetacdn.in',
 '62.svetacdn.in': 'svetacdn.in',
 '4432.svetacdn.in': 'svetacdn.in',
 '7043.svetacdn.in': 'svetacdn.in',
 '8543.svetacdn.in': 'svetacdn.in',
 '96.svetacdn.in': 'svetacdn.in',
 '2188.svetacdn.in': 'svetacdn.in',
 '63.svetacdn.in': 'svetacdn.in',
 '7851.svetacdn.in': 'svetacdn.in',
 '1.svetacdn.in': 'svetacdn.in',
 '8332.svetacdn.in': 'svetacdn.in',
 '57.svetacdn.in': 'svetacdn.in',
 '92.svetacdn.in': 'svetacdn.in',
 '5091.svetacdn.in': 'svetacdn.in',
 '8422.svetacdn.in': 'svetacdn.in',
 '29.svetacdn.in': 'svetacdn.in',
 '3828.svetacdn.in': 'svetacdn.in',
 '80.svetacdn.in': 'svetacdn.in',
 '2.svetacdn.in': 'svetacdn.in',
 '385.svetacdn.in': 'svetacdn.in',
 '304.svetacdn.in': 'svetacdn.in',
 '35.svetacdn.in': 'svetacdn.in',
 'hdvideopleer.svetacdn.in': 'svetacdn.in',
 '37.svetacdn.in': 'svetacdn.in',
 '60.svetacdn.in': 'svetacdn.in',
 '15.svetacdn.in': 'svetacdn.in',
 '86.svetacdn.in': 'svetacdn.in',
 '59.svetacdn.in': 'svetacdn.in',
 '8965.svetacdn.in': 'svetacdn.in',
 '3343.svetacdn.in': 'svetacdn.in',
 '77.svetacdn.in': 'svetacdn.in',
 '54.svetacdn.in': 'svetacdn.in',
 '24213.svetacdn.in': 'svetacdn.in',
 '64.svetacdn.in': 'svetacdn.in',
 '30.svetacdn.in': 'svetacdn.in',
 '7.svetacdn.in': 'svetacdn.in',
 '47.svetacdn.in': 'svetacdn.in',
 '8.svetacdn.in': 'svetacdn.in',
 '87.svetacdn.in': 'svetacdn.in',
 '39.svetacdn.in': 'svetacdn.in',
 '74.svetacdn.in': 'svetacdn.in',
 '53.svetacdn.in': 'svetacdn.in',
 '70.svetacdn.in': 'svetacdn.in',
 '100.svetacdn.in': 'svetacdn.in',
 '97.svetacdn.in': 'svetacdn.in',
 '27.svetacdn.in': 'svetacdn.in',
 '99.svetacdn.in': 'svetacdn.in',
 '43.svetacdn.in': 'svetacdn.in',
 '4576.svetacdn.in': 'svetacdn.in',
 '20.svetacdn.in': 'svetacdn.in',
 '14.svetacdn.in': 'svetacdn.in',
 '8846.svetacdn.in': 'svetacdn.in',
 '33.svetacdn.in': 'svetacdn.in',
 '18865.svetacdn.in': 'svetacdn.in',
 '664944266443434.svetacdn.in': 'svetacdn.in',
 '6713.svetacdn.in': 'svetacdn.in',
 'kinogo.svetacdn.in': 'svetacdn.in',
 '81.svetacdn.in': 'svetacdn.in',
 '50.svetacdn.in': 'svetacdn.in',
 '23.svetacdn.in': 'svetacdn.in',
 '82.svetacdn.in': 'svetacdn.in',
 '356555.svetacdn.in': 'svetacdn.in',
 '3267.svetacdn.in': 'svetacdn.in',
 '71.svetacdn.in': 'svetacdn.in',
 '52.svetacdn.in': 'svetacdn.in',
 '91.svetacdn.in': 'svetacdn.in',
 '84.svetacdn.in': 'svetacdn.in',
 '26.svetacdn.in': 'svetacdn.in',
 '40.svetacdn.in': 'svetacdn.in',
 '55.svetacdn.in': 'svetacdn.in',
 '25.svetacdn.in': 'svetacdn.in',
 '3.svetacdn.in': 'svetacdn.in',
 '456555.svetacdn.in': 'svetacdn.in',
 '21.svetacdn.in': 'svetacdn.in',
 '72.svetacdn.in': 'svetacdn.in',
 '2010.svetacdn.in': 'svetacdn.in',
 '17.svetacdn.in': 'svetacdn.in',
 '58.svetacdn.in': 'svetacdn.in',
 '666443434.svetacdn.in': 'svetacdn.in',
 '54425.svetacdn.in': 'svetacdn.in',
 '4323.svetacdn.in': 'svetacdn.in',
 '8840773434.svetacdn.in': 'svetacdn.in',
 '0331.svetacdn.in': 'svetacdn.in',
 '4.svetacdn.in': 'svetacdn.in',
 '422140774.svetacdn.in': 'svetacdn.in',
    'kartarf.ru': 'kartarf.ru',
 'perm.kartarf.ru': 'kartarf.ru',
 'langepas.kartarf.ru': 'kartarf.ru',
 'murmansk.kartarf.ru': 'kartarf.ru',
 'labinsk.kartarf.ru': 'kartarf.ru',
 'moskva.kartarf.ru': 'kartarf.ru',
 'saratov.kartarf.ru': 'kartarf.ru',
 'spb.kartarf.ru': 'kartarf.ru',
 'staryy-oskol.kartarf.ru': 'kartarf.ru',
 'sochi.kartarf.ru': 'kartarf.ru',
 'surgut.kartarf.ru': 'kartarf.ru',
 'izhevsk.kartarf.ru': 'kartarf.ru',
 'saransk.kartarf.ru': 'kartarf.ru',
 'nalchik.kartarf.ru': 'kartarf.ru',
 'kazan.kartarf.ru': 'kartarf.ru',
 'shahty.kartarf.ru': 'kartarf.ru',
 'ufa.kartarf.ru': 'kartarf.ru',
 'chelyabinsk.kartarf.ru': 'kartarf.ru',
 'vyshniy-volochek.kartarf.ru': 'kartarf.ru',
 'voronezh.kartarf.ru': 'kartarf.ru',
 'pavlovo.kartarf.ru': 'kartarf.ru',
 'kursk.kartarf.ru': 'kartarf.ru',
 'nizhnevartovsk.kartarf.ru': 'kartarf.ru',
 'velikie-luki.kartarf.ru': 'kartarf.ru',
 'ryazan.kartarf.ru': 'kartarf.ru',
 'tyumen.kartarf.ru': 'kartarf.ru',
 'berdsk.kartarf.ru': 'kartarf.ru',
 'syzran.kartarf.ru': 'kartarf.ru',
 'nahodka.kartarf.ru': 'kartarf.ru',
 'tambov.kartarf.ru': 'kartarf.ru',
 'rostov-na-donu.kartarf.ru': 'kartarf.ru',
 'krasnodar.kartarf.ru': 'kartarf.ru',
 'kaluga.kartarf.ru': 'kartarf.ru',
 'cheboksary.kartarf.ru': 'kartarf.ru',
 'neftekamsk.kartarf.ru': 'kartarf.ru',
 'glazov.kartarf.ru': 'kartarf.ru',
 'orsk.kartarf.ru': 'kartarf.ru',
 'habarovsk.kartarf.ru': 'kartarf.ru',
 'orenburg.kartarf.ru': 'kartarf.ru',
 'almetevsk.kartarf.ru': 'kartarf.ru',
 'nizhnekamsk.kartarf.ru': 'kartarf.ru',
 'ulyanovsk.kartarf.ru': 'kartarf.ru',
 'eysk.kartarf.ru': 'kartarf.ru',
 'nizhniy-tagil.kartarf.ru': 'kartarf.ru',
 'rybinsk.kartarf.ru': 'kartarf.ru',
 'engels.kartarf.ru': 'kartarf.ru',
 'novosibirsk.kartarf.ru': 'kartarf.ru',
 'sterlitamak.kartarf.ru': 'kartarf.ru',
 'samara.kartarf.ru': 'kartarf.ru',
 'ekaterinburg.kartarf.ru': 'kartarf.ru',
 'balakovo.kartarf.ru': 'kartarf.ru',
 'bryansk.kartarf.ru': 'kartarf.ru',
 'novorossiysk.kartarf.ru': 'kartarf.ru',
 'kogalym.kartarf.ru': 'kartarf.ru',
 'vologda.kartarf.ru': 'kartarf.ru',
 'yaroslavl.kartarf.ru': 'kartarf.ru',
 'kudymkar.kartarf.ru': 'kartarf.ru',
 'stavropol.kartarf.ru': 'kartarf.ru',
 'vorkuta.kartarf.ru': 'kartarf.ru',
 'volgograd.kartarf.ru': 'kartarf.ru',
 'krasnoyarsk.kartarf.ru': 'kartarf.ru',
 'shadrinsk.kartarf.ru': 'kartarf.ru',
 'komsomolsk-na-amure.kartarf.ru': 'kartarf.ru',
 'leninsk-kuzneckiy.kartarf.ru': 'kartarf.ru',
 'tver.kartarf.ru': 'kartarf.ru',
 'barnaul.kartarf.ru': 'kartarf.ru',
 'novokubansk.kartarf.ru': 'kartarf.ru',
 'volzhskiy.kartarf.ru': 'kartarf.ru',
 'kungur.kartarf.ru': 'kartarf.ru',
 'apsheronsk.kartarf.ru': 'kartarf.ru',
 'salavat.kartarf.ru': 'kartarf.ru',
 'kamyshin.kartarf.ru': 'kartarf.ru',
 'serov.kartarf.ru': 'kartarf.ru',
 'abakan.kartarf.ru': 'kartarf.ru',
 'naryan-mar.kartarf.ru': 'kartarf.ru',
 'tihoreck.kartarf.ru': 'kartarf.ru',
 'vladivostok.kartarf.ru': 'kartarf.ru',
 'gubkin.kartarf.ru': 'kartarf.ru',
 'ishim.kartarf.ru': 'kartarf.ru',
 'novokuzneck.kartarf.ru': 'kartarf.ru',
 'pskov.kartarf.ru': 'kartarf.ru',
 'kotlas.kartarf.ru': 'kartarf.ru',
 'nizhniy-novgorod.kartarf.ru': 'kartarf.ru',
 'kamensk-uralskiy.kartarf.ru': 'kartarf.ru',
 'kyzyl.kartarf.ru': 'kartarf.ru',
 'penza.kartarf.ru': 'kartarf.ru',
 'ussuriysk.kartarf.ru': 'kartarf.ru',
 'syktyvkar.kartarf.ru': 'kartarf.ru',
 'kurgan.kartarf.ru': 'kartarf.ru',
 'uhta.kartarf.ru': 'kartarf.ru',
 'orel.kartarf.ru': 'kartarf.ru',
 'belgorod.kartarf.ru': 'kartarf.ru',
 'murom.kartarf.ru': 'kartarf.ru',
 'solikamsk.kartarf.ru': 'kartarf.ru',
 'seversk.kartarf.ru': 'kartarf.ru',
 'tomsk.kartarf.ru': 'kartarf.ru',
 'anapa.kartarf.ru': 'kartarf.ru',
 'tobolsk.kartarf.ru': 'kartarf.ru',
    'smolensk.zapster.ru': 'zapster.ru',
 'samara.zapster.ru': 'zapster.ru',
 'ivanovo.zapster.ru': 'zapster.ru',
 'kurgan.zapster.ru': 'zapster.ru',
 'rostov.zapster.ru': 'zapster.ru',
 'blagoveshensk.zapster.ru': 'zapster.ru',
 'novosibirsk.zapster.ru': 'zapster.ru',
 'podolsk.zapster.ru': 'zapster.ru',
 'kaluga.zapster.ru': 'zapster.ru',
 'abakan.zapster.ru': 'zapster.ru',
 'krasnodar.zapster.ru': 'zapster.ru',
 'tyumen.zapster.ru': 'zapster.ru',
 'stavropol.zapster.ru': 'zapster.ru',
 'spb.zapster.ru': 'zapster.ru',
 'kaliningrad.zapster.ru': 'zapster.ru',
 'vladikavkaz.zapster.ru': 'zapster.ru',
 'ufa.zapster.ru': 'zapster.ru',
 'nizhnekamsk.zapster.ru': 'zapster.ru',
 'orel.zapster.ru': 'zapster.ru',
 'tolyatti.zapster.ru': 'zapster.ru',
 'maykop.zapster.ru': 'zapster.ru',
 'izhevsk.zapster.ru': 'zapster.ru',
 'arkhangelsk.zapster.ru': 'zapster.ru',
 'nizhnevartovsk.zapster.ru': 'zapster.ru',
 'novorossiysk.zapster.ru': 'zapster.ru',
 'kirov.zapster.ru': 'zapster.ru',
 'kazan.zapster.ru': 'zapster.ru',
 'kostroma.zapster.ru': 'zapster.ru',
 'khabarovsk.zapster.ru': 'zapster.ru',
 'armavir.zapster.ru': 'zapster.ru',
 'tomsk.zapster.ru': 'zapster.ru',
 'voronezh.zapster.ru': 'zapster.ru',
 'pk.zapster.ru': 'zapster.ru',
 'volgograd.zapster.ru': 'zapster.ru',
 'chelyabinsk.zapster.ru': 'zapster.ru',
 'astrakhan.zapster.ru': 'zapster.ru',
 'krasnoyarsk.zapster.ru': 'zapster.ru',
 'nahodka.zapster.ru': 'zapster.ru',
 'tula.zapster.ru': 'zapster.ru',
 'omsk.zapster.ru': 'zapster.ru',
 'zapster.ru': 'zapster.ru',
 'barnaul.zapster.ru': 'zapster.ru',
 'chelny.zapster.ru': 'zapster.ru',
 'engels.zapster.ru': 'zapster.ru',
 'chita.zapster.ru': 'zapster.ru',
 'ekaterinburg.zapster.ru': 'zapster.ru',
 'sochi.zapster.ru': 'zapster.ru',
 'vladimir.zapster.ru': 'zapster.ru',
 'nn.zapster.ru': 'zapster.ru',
 'saratov.zapster.ru': 'zapster.ru',
 'saransk.zapster.ru': 'zapster.ru',
 'grozny.zapster.ru': 'zapster.ru',
 'pushkino.zapster.ru': 'zapster.ru',
 'perm.zapster.ru': 'zapster.ru',
 'tver.zapster.ru': 'zapster.ru',
 'tagil.zapster.ru': 'zapster.ru',
 'almetevsk.zapster.ru': 'zapster.ru',
 'himki.zapster.ru': 'zapster.ru',
 'bryansk.zapster.ru': 'zapster.ru',
 'kemerovo.zapster.ru': 'zapster.ru',
 'chevoksary.zapster.ru': 'zapster.ru',
 'syzran.zapster.ru': 'zapster.ru',
 'noginsk.zapster.ru': 'zapster.ru',
 'shahty.zapster.ru': 'zapster.ru',
 'yaroslavl.zapster.ru': 'zapster.ru',
 'orenburg.zapster.ru': 'zapster.ru',
 'ryazan.zapster.ru': 'zapster.ru',
 'tambov.zapster.ru': 'zapster.ru',
 'pyatigorsk.zapster.ru': 'zapster.ru',
 'pskov.zapster.ru': 'zapster.ru',
 'serpuhov.zapster.ru': 'zapster.ru',
 'lipetsk.zapster.ru': 'zapster.ru',
 'sterlitamak.zapster.ru': 'zapster.ru',
 'kursk.zapster.ru': 'zapster.ru',
 'ulyanovsk.zapster.ru': 'zapster.ru',
 'ys.zapster.ru': 'zapster.ru',
 'orsk.zapster.ru': 'zapster.ru',
 'belgorod.zapster.ru': 'zapster.ru',
 'taganrog.zapster.ru': 'zapster.ru',
 'krasnogorsk.zapster.ru': 'zapster.ru',
 'birobidzhan.zapster.ru': 'zapster.ru',
 'kolomna.zapster.ru': 'zapster.ru',
 'orehovo-zuevo.zapster.ru': 'zapster.ru',
 'odintsovo.zapster.ru': 'zapster.ru',
 'vladivostok.zapster.ru': 'zapster.ru',
 'yo.zapster.ru': 'zapster.ru',
 'syktyvkar.zapster.ru': 'zapster.ru',
 'penza.zapster.ru': 'zapster.ru',
 'surgut.zapster.ru': 'zapster.ru',
 'yakutsk.zapster.ru': 'zapster.ru',
 'msk.zapster.ru': 'zapster.ru',
 'makhachkala.zapster.ru': 'zapster.ru',
 'petrozavodsk.zapster.ru': 'zapster.ru',
 'neftekamsk.zapster.ru': 'zapster.ru',
 'salavat.zapster.ru': 'zapster.ru',
 'novocherkassk.zapster.ru': 'zapster.ru',
 'balashiha.zapster.ru': 'zapster.ru',
 'irkutsk.zapster.ru': 'zapster.ru',
 'novgorod.zapster.ru': 'zapster.ru',
    '40nog.ru': '40nog.ru',
 'vladivostok.40nog.ru': '40nog.ru',
 'izhevsk.40nog.ru': '40nog.ru',
 'naberezhnye-chelny.40nog.ru': '40nog.ru',
 'tula.40nog.ru': '40nog.ru',
 'smolensk.40nog.ru': '40nog.ru',
 'habarovsk.40nog.ru': '40nog.ru',
 'nsk.40nog.ru': '40nog.ru',
 'penza.40nog.ru': '40nog.ru',
 'arhangelsk.40nog.ru': '40nog.ru',
 'perm.40nog.ru': '40nog.ru',
 'ekb.40nog.ru': '40nog.ru',
 'yaroslavl.40nog.ru': '40nog.ru',
 'tyumen.40nog.ru': '40nog.ru',
 'samara.40nog.ru': '40nog.ru',
 'volgograd.40nog.ru': '40nog.ru',
 'voronezh.40nog.ru': '40nog.ru',
 'novorossiysk.40nog.ru': '40nog.ru',
 'ufa.40nog.ru': '40nog.ru',
 'ivanovo.40nog.ru': '40nog.ru',
 'sochi.40nog.ru': '40nog.ru',
 'saratov.40nog.ru': '40nog.ru',
 'anapa.40nog.ru': '40nog.ru',
 'barnaul.40nog.ru': '40nog.ru',
 'tambov.40nog.ru': '40nog.ru',
 'cheboksary.40nog.ru': '40nog.ru',
 'sankt-peterburg.40nog.ru': '40nog.ru',
 'belgorod.40nog.ru': '40nog.ru',
 'mytishchi.40nog.ru': '40nog.ru',
 'korolyov.40nog.ru': '40nog.ru',
 'krasnoyarsk.40nog.ru': '40nog.ru',
 'elektrostal.40nog.ru': '40nog.ru',
 'nn.40nog.ru': '40nog.ru',
 'podolsk.40nog.ru': '40nog.ru',
 'chelyabinsk.40nog.ru': '40nog.ru',
 'omsk.40nog.ru': '40nog.ru',
 'cherepovec.40nog.ru': '40nog.ru',
 'tver.40nog.ru': '40nog.ru',
 'rostov-na-donu.40nog.ru': '40nog.ru',
 'kemerovo.40nog.ru': '40nog.ru',
 'surgut.40nog.ru': '40nog.ru',
 'bryansk.40nog.ru': '40nog.ru',
 'pskov.40nog.ru': '40nog.ru',
 'ulyanovsk.40nog.ru': '40nog.ru',
 'kr.40nog.ru': '40nog.ru',
 'stupino.40nog.ru': '40nog.ru',
 'kazan.40nog.ru': '40nog.ru',
 'blagoveshchensk.40nog.ru': '40nog.ru',
 'nalchik.40nog.ru': '40nog.ru',
 'kursk.40nog.ru': '40nog.ru',
 'saransk.40nog.ru': '40nog.ru',
 'vologda.40nog.ru': '40nog.ru',
 'petrozavodsk.40nog.ru': '40nog.ru',
 'stavropol.40nog.ru': '40nog.ru',
 'syktyvkar.40nog.ru': '40nog.ru',
 'kstovo.40nog.ru': '40nog.ru',
 'kovrov.40nog.ru': '40nog.ru',
 'kaluga.40nog.ru': '40nog.ru',
 'dzerzhinsk.40nog.ru': '40nog.ru',
 'ryazan.40nog.ru': '40nog.ru',
 'lyubercy.40nog.ru': '40nog.ru',
 'dmitrov.40nog.ru': '40nog.ru',
 'kaliningrad.40nog.ru': '40nog.ru',
 'armavir.40nog.ru': '40nog.ru',
 'murmansk.40nog.ru': '40nog.ru',
 'klin.40nog.ru': '40nog.ru',
 'gelendzhik.40nog.ru': '40nog.ru',
 'orenburg.40nog.ru': '40nog.ru',
 'biysk.40nog.ru': '40nog.ru',
 'vladimir.40nog.ru': '40nog.ru',
 'evpatoriya.40nog.ru': '40nog.ru',
 'chehov.40nog.ru': '40nog.ru',
 'ramenskoe.40nog.ru': '40nog.ru',
 'obninsk.40nog.ru': '40nog.ru',
 'taganrog.40nog.ru': '40nog.ru',
 'zelenograd.40nog.ru': '40nog.ru',
 'kurgan.40nog.ru': '40nog.ru',
 'feodosiya.40nog.ru': '40nog.ru',
 'pervouralsk.40nog.ru': '40nog.ru',
 'engels.40nog.ru': '40nog.ru',
 'novokuzneck.40nog.ru': '40nog.ru',
 'solnechnogorsk.40nog.ru': '40nog.ru',
 'kolomna.40nog.ru': '40nog.ru',
 'sergiev-posad.40nog.ru': '40nog.ru',
 'orel.40nog.ru': '40nog.ru',
 'kirov.40nog.ru': '40nog.ru',
 'himki.40nog.ru': '40nog.ru',
 'lipeck.40nog.ru': '40nog.ru',
 'nizhnevartovsk.40nog.ru': '40nog.ru',
 'pyatigorsk.40nog.ru': '40nog.ru',
 'fryazino.40nog.ru': '40nog.ru',
 'yoshkar-ola.40nog.ru': '40nog.ru',
 'balakovo.40nog.ru': '40nog.ru',
 'staryy-oskol.40nog.ru': '40nog.ru',
 'mahachkala.40nog.ru': '40nog.ru',
 'voskresensk.40nog.ru': '40nog.ru',
 'velikiy-novgorod.40nog.ru': '40nog.ru',
 'tomsk.40nog.ru': '40nog.ru',
 'naro-fominsk.40nog.ru': '40nog.ru',
 'kostroma.40nog.ru': '40nog.ru',
    'murmansk.move.ru': 'move.ru',
 'portugal.move.ru': 'move.ru',
 'samara.move.ru': 'move.ru',
 'perm.move.ru': 'move.ru',
 'smolensk.move.ru': 'move.ru',
 'kamchatka.move.ru': 'move.ru',
 'belgorod.move.ru': 'move.ru',
 'marijel.move.ru': 'move.ru',
 'kostroma.move.ru': 'move.ru',
 'move.ru': 'move.ru',
 'tyumen.move.ru': 'move.ru',
 'orenburg.move.ru': 'move.ru',
 'krasnodar.move.ru': 'move.ru',
 'krim.move.ru': 'move.ru',
 'ufa.move.ru': 'move.ru',
 'sverdlovsk.move.ru': 'move.ru',
 'chelyabinsk.move.ru': 'move.ru',
 'turkey.move.ru': 'move.ru',
 'novosibirsk.move.ru': 'move.ru',
 'krasnoyarsk.move.ru': 'move.ru',
 'udmurtia.move.ru': 'move.ru',
 'spb.move.ru': 'move.ru',
 'tatarstan.move.ru': 'move.ru',
 'vnovgorod.move.ru': 'move.ru',
 'chuvashia.move.ru': 'move.ru',
 'saratov.move.ru': 'move.ru',
 'voronezh.move.ru': 'move.ru',
 'ryazan.move.ru': 'move.ru',
 'kaliningrad.move.ru': 'move.ru',
 'omsk.move.ru': 'move.ru',
 'tambov.move.ru': 'move.ru',
 'kurgan.move.ru': 'move.ru',
 'amursk.move.ru': 'move.ru',
 'kemerov.move.ru': 'move.ru',
 'hmao.move.ru': 'move.ru',
 'orel.move.ru': 'move.ru',
 'komi.move.ru': 'move.ru',
 'hakasiya.move.ru': 'move.ru',
 'kaluga.move.ru': 'move.ru',
 'adygeya.move.ru': 'move.ru',
 'ak.move.ru': 'move.ru',
 'rnd.move.ru': 'move.ru',
 'kareliya.move.ru': 'move.ru',
 'yuzhno-sahalinsk.move.ru': 'move.ru',
 'germany.move.ru': 'move.ru',
 'habarovsk.move.ru': 'move.ru',
 'velikobritania.move.ru': 'move.ru',
 'switzerland.move.ru': 'move.ru',
 'stavropol.move.ru': 'move.ru',
 'ivanovo.move.ru': 'move.ru',
 'dagestan.move.ru': 'move.ru',
 'tomsk.move.ru': 'move.ru',
 'tyva.move.ru': 'move.ru',
 'nn.move.ru': 'move.ru',
 'montenegro.move.ru': 'move.ru',
 'vladivostok.move.ru': 'move.ru',
 'osetiya.move.ru': 'move.ru',
 'irkutsk.move.ru': 'move.ru',
 'ulan-ude.move.ru': 'move.ru',
 'ingushetia.move.ru': 'move.ru',
 'chita.move.ru': 'move.ru',
 'saha.move.ru': 'move.ru',
 'salehard.move.ru': 'move.ru',
 'bryansk.move.ru': 'move.ru',
 'lipetsk.move.ru': 'move.ru',
 'vladimir.move.ru': 'move.ru',
 'yaroslavl.move.ru': 'move.ru',
 'tula.move.ru': 'move.ru',
 'italy.move.ru': 'move.ru',
 'hungary.move.ru': 'move.ru',
 'astrahan.move.ru': 'move.ru',
 'kchr.move.ru': 'move.ru',
 'pskov.move.ru': 'move.ru',
 'kirov.move.ru': 'move.ru',
 'penza.move.ru': 'move.ru',
 'belorussia.move.ru': 'move.ru',
 'mordoviya.move.ru': 'move.ru',
 'tver.move.ru': 'move.ru',
 'ra.move.ru': 'move.ru',
 'kursk.move.ru': 'move.ru',
 'sevastopol.move.ru': 'move.ru',
 'arhangelsk.move.ru': 'move.ru',
 'magadan.move.ru': 'move.ru',
 'kalmykiya.move.ru': 'move.ru',
 'vologda.move.ru': 'move.ru',
 'ulyanovsk.move.ru': 'move.ru',
 'volgograd.move.ru': 'move.ru',
 'spain.move.ru': 'move.ru',
 'cyprus.move.ru': 'move.ru',
 'latvia.move.ru': 'move.ru',
 'czech.move.ru': 'move.ru',
 'bulgaria.move.ru': 'move.ru',
 'gruziya.move.ru': 'move.ru',
 'chechenskaya.move.ru': 'move.ru',
 'kbr.move.ru': 'move.ru',
 'birobidzhan.move.ru': 'move.ru',
 'thailand.move.ru': 'move.ru',
 'oae.move.ru': 'move.ru',
 'greece.move.ru': 'move.ru',
 'france.move.ru': 'move.ru',
    '8.findmerr.com': 'findmerr.com',
 '1.findmerr.com': 'findmerr.com',
 '107.findmerr.com': 'findmerr.com',
 '73.findmerr.com': 'findmerr.com',
 '13.findmerr.com': 'findmerr.com',
 '132.findmerr.com': 'findmerr.com',
 'findmerr.com': 'findmerr.com',
 '81.findmerr.com': 'findmerr.com',
 '26.findmerr.com': 'findmerr.com',
 '54.findmerr.com': 'findmerr.com',
 '56.findmerr.com': 'findmerr.com',
 '4.findmerr.com': 'findmerr.com',
 '128.findmerr.com': 'findmerr.com',
 '53.findmerr.com': 'findmerr.com',
 '161.findmerr.com': 'findmerr.com',
 '144.findmerr.com': 'findmerr.com',
 '7.findmerr.com': 'findmerr.com',
 '2.findmerr.com': 'findmerr.com',
 '6.findmerr.com': 'findmerr.com',
 '170.findmerr.com': 'findmerr.com',
 '9.findmerr.com': 'findmerr.com',
 '18.findmerr.com': 'findmerr.com',
 '25.findmerr.com': 'findmerr.com',
 '12.findmerr.com': 'findmerr.com',
 '87.findmerr.com': 'findmerr.com',
 '29.findmerr.com': 'findmerr.com',
 '72.findmerr.com': 'findmerr.com',
 '27.findmerr.com': 'findmerr.com',
 '55.findmerr.com': 'findmerr.com',
 '249.findmerr.com': 'findmerr.com',
 '201.findmerr.com': 'findmerr.com',
 '187.findmerr.com': 'findmerr.com',
 '96.findmerr.com': 'findmerr.com',
 '117.findmerr.com': 'findmerr.com',
 '36.findmerr.com': 'findmerr.com',
 '95.findmerr.com': 'findmerr.com',
 '182.findmerr.com': 'findmerr.com',
 '69.findmerr.com': 'findmerr.com',
 '5.findmerr.com': 'findmerr.com',
 '66.findmerr.com': 'findmerr.com',
 '10.findmerr.com': 'findmerr.com',
 '34.findmerr.com': 'findmerr.com',
 '37.findmerr.com': 'findmerr.com',
 '208.findmerr.com': 'findmerr.com',
 '130.findmerr.com': 'findmerr.com',
 '88.findmerr.com': 'findmerr.com',
 '60.findmerr.com': 'findmerr.com',
 '204.findmerr.com': 'findmerr.com',
 '57.findmerr.com': 'findmerr.com',
 '211.findmerr.com': 'findmerr.com',
 '175.findmerr.com': 'findmerr.com',
 '104.findmerr.com': 'findmerr.com',
 '67.findmerr.com': 'findmerr.com',
 '71.findmerr.com': 'findmerr.com',
 '76.findmerr.com': 'findmerr.com',
 '58.findmerr.com': 'findmerr.com',
 '20.findmerr.com': 'findmerr.com',
 '206.findmerr.com': 'findmerr.com',
 '16.findmerr.com': 'findmerr.com',
 '196.findmerr.com': 'findmerr.com',
 '21.findmerr.com': 'findmerr.com',
 '65.findmerr.com': 'findmerr.com',
 '61.findmerr.com': 'findmerr.com',
 '92.findmerr.com': 'findmerr.com',
 '90.findmerr.com': 'findmerr.com',
 '75.findmerr.com': 'findmerr.com',
 '11.findmerr.com': 'findmerr.com',
 '49.findmerr.com': 'findmerr.com',
 '189.findmerr.com': 'findmerr.com',
 '160.findmerr.com': 'findmerr.com',
 '121.findmerr.com': 'findmerr.com',
 '193.findmerr.com': 'findmerr.com',
 '79.findmerr.com': 'findmerr.com',
 '78.findmerr.com': 'findmerr.com',
 '181.findmerr.com': 'findmerr.com',
 '3.findmerr.com': 'findmerr.com',
 '70.findmerr.com': 'findmerr.com',
 '62.findmerr.com': 'findmerr.com',
 '100.findmerr.com': 'findmerr.com',
 '39.findmerr.com': 'findmerr.com',
 '77.findmerr.com': 'findmerr.com',
 '31.findmerr.com': 'findmerr.com',
 '93.findmerr.com': 'findmerr.com',
 '98.findmerr.com': 'findmerr.com',
 '84.findmerr.com': 'findmerr.com',
 '24.findmerr.com': 'findmerr.com',
 '74.findmerr.com': 'findmerr.com',
 '97.findmerr.com': 'findmerr.com',
 '28.findmerr.com': 'findmerr.com',
 '115.findmerr.com': 'findmerr.com',
 '221.findmerr.com': 'findmerr.com',
 '143.findmerr.com': 'findmerr.com',
 '35.findmerr.com': 'findmerr.com',
 '33.findmerr.com': 'findmerr.com',
 '180.findmerr.com': 'findmerr.com',
 '154.findmerr.com': 'findmerr.com',
 '45.findmerr.com': 'findmerr.com',
 '173.findmerr.com': 'findmerr.com',
 '89.findmerr.com': 'findmerr.com',
 '80.findmerr.com': 'findmerr.com',
 '38.findmerr.com': 'findmerr.com',
    'sidex.ru': 'sidex.ru',
 'donetsk.sidex.ru': 'sidex.ru',
 'ivanovo.sidex.ru': 'sidex.ru',
 'temryuk.sidex.ru': 'sidex.ru',
 'ryazan.sidex.ru': 'sidex.ru',
 'novosibirsk.sidex.ru': 'sidex.ru',
 'surgut.sidex.ru': 'sidex.ru',
 'kirov.sidex.ru': 'sidex.ru',
 'domodedovo.sidex.ru': 'sidex.ru',
 'salsk.sidex.ru': 'sidex.ru',
 'yakutsk.sidex.ru': 'sidex.ru',
 'arxangelsk.sidex.ru': 'sidex.ru',
 'voronej.sidex.ru': 'sidex.ru',
 'krasnodar.sidex.ru': 'sidex.ru',
 'chernushka.sidex.ru': 'sidex.ru',
 'nijniy-novgorod.sidex.ru': 'sidex.ru',
 'voljskiy.sidex.ru': 'sidex.ru',
 'ekaterinburg.sidex.ru': 'sidex.ru',
 'siyzran.sidex.ru': 'sidex.ru',
 'perm.sidex.ru': 'sidex.ru',
 'sankt-peterburg.sidex.ru': 'sidex.ru',
 'zelenograd.sidex.ru': 'sidex.ru',
 'ximki.sidex.ru': 'sidex.ru',
 'gorno-altaysk.sidex.ru': 'sidex.ru',
 'serpuxov.sidex.ru': 'sidex.ru',
 'murmansk.sidex.ru': 'sidex.ru',
 'berezniki.sidex.ru': 'sidex.ru',
 'voskresensk.sidex.ru': 'sidex.ru',
 'yalta.sidex.ru': 'sidex.ru',
 'novotroitsk.sidex.ru': 'sidex.ru',
 'orenburg.sidex.ru': 'sidex.ru',
 'kazan.sidex.ru': 'sidex.ru',
 'smolensk.sidex.ru': 'sidex.ru',
 'stupino.sidex.ru': 'sidex.ru',
 'yaroslavl.sidex.ru': 'sidex.ru',
 'velikie-luki.sidex.ru': 'sidex.ru',
 'samara.sidex.ru': 'sidex.ru',
 'tomilino.sidex.ru': 'sidex.ru',
 'saransk.sidex.ru': 'sidex.ru',
 'krasnoyarsk.sidex.ru': 'sidex.ru',
 'belgorod.sidex.ru': 'sidex.ru',
 'xabarovsk.sidex.ru': 'sidex.ru',
 'rostov-na-donu.sidex.ru': 'sidex.ru',
 'tomsk.sidex.ru': 'sidex.ru',
 'revda.sidex.ru': 'sidex.ru',
 'nijnevartovsk.sidex.ru': 'sidex.ru',
 'butovo.sidex.ru': 'sidex.ru',
 'buzuluk.sidex.ru': 'sidex.ru',
 'glazov.sidex.ru': 'sidex.ru',
 'borisoglebsk.sidex.ru': 'sidex.ru',
 'shaxtiy.sidex.ru': 'sidex.ru',
 'tolyatti.sidex.ru': 'sidex.ru',
 'penza.sidex.ru': 'sidex.ru',
 'taganrog.sidex.ru': 'sidex.ru',
 'mojaysk.sidex.ru': 'sidex.ru',
 'irkutsk.sidex.ru': 'sidex.ru',
 'gatchina.sidex.ru': 'sidex.ru',
 'pereslavl-zalesskiy.sidex.ru': 'sidex.ru',
 'miass.sidex.ru': 'sidex.ru',
 'slavyansk-na-kubani.sidex.ru': 'sidex.ru',
 'tver.sidex.ru': 'sidex.ru',
 'evpatoriya.sidex.ru': 'sidex.ru',
 'maloyaroslavets.sidex.ru': 'sidex.ru',
 'sergiev-posad.sidex.ru': 'sidex.ru',
 'omsk.sidex.ru': 'sidex.ru',
 'aleksandrov.sidex.ru': 'sidex.ru',
 'krasnogorsk.sidex.ru': 'sidex.ru',
 'cheboksariy.sidex.ru': 'sidex.ru',
 'murino.sidex.ru': 'sidex.ru',
 'obninsk.sidex.ru': 'sidex.ru',
 'petropavlovsk-kamchatskiy.sidex.ru': 'sidex.ru',
 'safonovo4.sidex.ru': 'sidex.ru',
 'velikiy-novgorod.sidex.ru': 'sidex.ru',
 'zelenokumsk.sidex.ru': 'sidex.ru',
 'chusovoy.sidex.ru': 'sidex.ru',
 'ussuriysk.sidex.ru': 'sidex.ru',
 'komsomolsk-na-amure.sidex.ru': 'sidex.ru',
 'adler.sidex.ru': 'sidex.ru',
 'ijevsk.sidex.ru': 'sidex.ru',
 'tyumen.sidex.ru': 'sidex.ru',
 'roslavl.sidex.ru': 'sidex.ru',
 'kamiyshin.sidex.ru': 'sidex.ru',
 'kirov2.sidex.ru': 'sidex.ru',
 'belaya-kalitva.sidex.ru': 'sidex.ru',
 'naberejniye-chelniy.sidex.ru': 'sidex.ru',
 'istra.sidex.ru': 'sidex.ru',
 'kola.sidex.ru': 'sidex.ru',
 'sxodnya.sidex.ru': 'sidex.ru',
 'tiynda.sidex.ru': 'sidex.ru',
 'pyatigorsk.sidex.ru': 'sidex.ru',
 'kansk.sidex.ru': 'sidex.ru',
 'kanevskaya.sidex.ru': 'sidex.ru',
 'yurga.sidex.ru': 'sidex.ru',
 'neftekamsk.sidex.ru': 'sidex.ru',
 'lyubertsiy.sidex.ru': 'sidex.ru',
 'ufa.sidex.ru': 'sidex.ru',
 'novomoskovsk.sidex.ru': 'sidex.ru',
 'viyksa.sidex.ru': 'sidex.ru',
 'novouralsk.sidex.ru': 'sidex.ru',
 'petrozavodsk.sidex.ru': 'sidex.ru',
 'elizovo.sidex.ru': 'sidex.ru',
 'maxachkala.sidex.ru': 'sidex.ru',
    'dcrr-vertelishki.schools.by': 'schools.by',
 'domanovichi.schools.by': 'schools.by',
 'myshanka.schools.by': 'schools.by',
 'schools.by': 'schools.by',
 '15brest.schools.by': 'schools.by',
 'krasnoluki.schools.by': 'schools.by',
 'ddu109grodno.schools.by': 'schools.by',
 'spchoiniki.schools.by': 'schools.by',
 'sad56bobruisk.schools.by': 'schools.by',
 'sv10.schools.by': 'schools.by',
 'ckroir-vileyka.schools.by': 'schools.by',
 'sh2-mozyr.schools.by': 'schools.by',
 'sad2berezovka.schools.by': 'schools.by',
 'spc-besh.schools.by': 'schools.by',
 'sad14molod.schools.by': 'schools.by',
 'korolev-stan.schools.by': 'schools.by',
 'ds40lida.schools.by': 'schools.by',
 'dtdskzl.schools.by': 'schools.by',
 'sad4lida.schools.by': 'schools.by',
 'sad19mozyr.schools.by': 'schools.by',
 'sad29molod.schools.by': 'schools.by',
 'shilovichi.schools.by': 'schools.by',
 'lubushani.schools.by': 'schools.by',
 'sad-leskovka.schools.by': 'schools.by',
 'dvizhki.schools.by': 'schools.by',
 'ddu95grodno.schools.by': 'schools.by',
 '7kalinkovichi.schools.by': 'schools.by',
 'detdom-vitebsk.schools.by': 'schools.by',
 '2bobr.schools.by': 'schools.by',
 'yornovopolotsk.schools.by': 'schools.by',
 'sad24polotsk.schools.by': 'schools.by',
 'spc-shum.schools.by': 'schools.by',
 'pribolovichi.schools.by': 'schools.by',
 'kam-int.schools.by': 'schools.by',
 'sch20grodno.schools.by': 'schools.by',
 'ctdim-zelva.schools.by': 'schools.by',
 'ddu96grodno.schools.by': 'schools.by',
 'zalessye.schools.by': 'schools.by',
 'sad32zhlobin.schools.by': 'schools.by',
 '11borisov.schools.by': 'schools.by',
 '30bobruisk.schools.by': 'schools.by',
 'ludenevichi.schools.by': 'schools.by',
 'voronino.schools.by': 'schools.by',
 'sad-ozerco.schools.by': 'schools.by',
 'sad10vileyka.schools.by': 'schools.by',
 'ddu63grodno.schools.by': 'schools.by',
 '33grodno.schools.by': 'schools.by',
 'petrikov1.schools.by': 'schools.by',
 'petrikov.schools.by': 'schools.by',
 'ddu-koptevka.schools.by': 'schools.by',
 'ds-stsv.schools.by': 'schools.by',
 'bronislav.schools.by': 'schools.by',
 'duravichi.schools.by': 'schools.by',
 'mosty-spc.schools.by': 'schools.by',
 'gymn1borisov.schools.by': 'schools.by',
 'gymn4grodno.schools.by': 'schools.by',
 'sch11grodno.schools.by': 'schools.by',
 '6grodno.schools.by': 'schools.by',
 '62gomel.schools.by': 'schools.by',
 'sh3grodno.schools.by': 'schools.by',
 '31grodno.schools.by': 'schools.by',
 '37grodno.schools.by': 'schools.by',
 'ddu20grodno.schools.by': 'schools.by',
 '38grodno.schools.by': 'schools.by',
 'ddu27grodno.schools.by': 'schools.by',
 '35grodno.schools.by': 'schools.by',
 'ddu23grodno.schools.by': 'schools.by',
 'gymn9grodno.schools.by': 'schools.by',
 'sch5grodno.schools.by': 'schools.by',
 '41grodno.schools.by': 'schools.by',
 'gymn3grodno.schools.by': 'schools.by',
 'sch15grodno.schools.by': 'schools.by',
 'sch12grodno.schools.by': 'schools.by',
 '13grodno.schools.by': 'schools.by',
 'sh26grodno.schools.by': 'schools.by',
 'ddu4grodno.schools.by': 'schools.by',
 'sch39grodno.schools.by': 'schools.by',
 'ddu22grodno.schools.by': 'schools.by',
 'sh34grodno.schools.by': 'schools.by',
 'ddu5grodno.schools.by': 'schools.by',
 'sh19grodno.schools.by': 'schools.by',
 'ddu19grodno.schools.by': 'schools.by',
 'ddu70grodno.schools.by': 'schools.by',
 'upk8brest.schools.by': 'schools.by',
 'ckroir-volkovysk.schools.by': 'schools.by',
 'kamenskaya.schools.by': 'schools.by',
 '7mozyr.schools.by': 'schools.by',
 '7volk.schools.by': 'schools.by',
 'sad81mogilev.schools.by': 'schools.by',
 'sad2smolevichi.schools.by': 'schools.by',
 'ds52brest.schools.by': 'schools.by',
 'sad29mozyr.schools.by': 'schools.by',
 'sad-kopishe.schools.by': 'schools.by',
 'sad7lida.schools.by': 'schools.by',
 '31brest.schools.by': 'schools.by',
 'gimn-elsk.schools.by': 'schools.by',
 'borov.schools.by': 'schools.by',
 'postsh3.schools.by': 'schools.by',
 'trilesino.schools.by': 'schools.by',
 'gromyko.schools.by': 'schools.by',
 'trabysad.schools.by': 'schools.by',
 'orshabsh.schools.by': 'schools.by',
 '8minsk.schools.by': 'schools.by',
    'cherepovets.big-book-edu.ru': 'big-book-edu.ru',
 'chelyabinsk.big-book-edu.ru': 'big-book-edu.ru',
 'saint-petersburg.big-book-edu.ru': 'big-book-edu.ru',
 'novocherkask.big-book-edu.ru': 'big-book-edu.ru',
 'saransk.big-book-edu.ru': 'big-book-edu.ru',
 'cheboksary.big-book-edu.ru': 'big-book-edu.ru',
 'perm.big-book-edu.ru': 'big-book-edu.ru',
 'saratov.big-book-edu.ru': 'big-book-edu.ru',
 'tyumen.big-book-edu.ru': 'big-book-edu.ru',
 'krasnodar.big-book-edu.ru': 'big-book-edu.ru',
 'kazan.big-book-edu.ru': 'big-book-edu.ru',
 'yekaterinburg.big-book-edu.ru': 'big-book-edu.ru',
 'serpuhov.big-book-edu.ru': 'big-book-edu.ru',
 'neftekamsk.big-book-edu.ru': 'big-book-edu.ru',
 'maykop.big-book-edu.ru': 'big-book-edu.ru',
 'khabarovsk.big-book-edu.ru': 'big-book-edu.ru',
 'ufa.big-book-edu.ru': 'big-book-edu.ru',
 'kirov.big-book-edu.ru': 'big-book-edu.ru',
 'mytiwi.big-book-edu.ru': 'big-book-edu.ru',
 'moscow.big-book-edu.ru': 'big-book-edu.ru',
 'sochi.big-book-edu.ru': 'big-book-edu.ru',
 'ivanovo.big-book-edu.ru': 'big-book-edu.ru',
 'sarapyl.big-book-edu.ru': 'big-book-edu.ru',
 'nizhniy-novgorod.big-book-edu.ru': 'big-book-edu.ru',
 'izhevsk.big-book-edu.ru': 'big-book-edu.ru',
 'barnaul.big-book-edu.ru': 'big-book-edu.ru',
 'vladivostok.big-book-edu.ru': 'big-book-edu.ru',
 'naberezhnyye-chelny.big-book-edu.ru': 'big-book-edu.ru',
 'novosibirsk.big-book-edu.ru': 'big-book-edu.ru',
 'vladimir.big-book-edu.ru': 'big-book-edu.ru',
 'belgorod.big-book-edu.ru': 'big-book-edu.ru',
 'pskov.big-book-edu.ru': 'big-book-edu.ru',
 'abakan.big-book-edu.ru': 'big-book-edu.ru',
 'ulyanovsk.big-book-edu.ru': 'big-book-edu.ru',
 'noginsk.big-book-edu.ru': 'big-book-edu.ru',
 'oryol.big-book-edu.ru': 'big-book-edu.ru',
 'tver.big-book-edu.ru': 'big-book-edu.ru',
 'ryazan.big-book-edu.ru': 'big-book-edu.ru',
 'nizhniy-tagil.big-book-edu.ru': 'big-book-edu.ru',
 'volgograd.big-book-edu.ru': 'big-book-edu.ru',
 'samara.big-book-edu.ru': 'big-book-edu.ru',
 'bryansk.big-book-edu.ru': 'big-book-edu.ru',
 'salovat.big-book-edu.ru': 'big-book-edu.ru',
 'kemerovo.big-book-edu.ru': 'big-book-edu.ru',
 'yoshkar-ola.big-book-edu.ru': 'big-book-edu.ru',
 'yaroslavl.big-book-edu.ru': 'big-book-edu.ru',
 'kurgan.big-book-edu.ru': 'big-book-edu.ru',
 'kolomna.big-book-edu.ru': 'big-book-edu.ru',
 'krasnoyarsk.big-book-edu.ru': 'big-book-edu.ru',
 'surgut.big-book-edu.ru': 'big-book-edu.ru',
 'kursk.big-book-edu.ru': 'big-book-edu.ru',
 'tambov.big-book-edu.ru': 'big-book-edu.ru',
 'kovrov.big-book-edu.ru': 'big-book-edu.ru',
 'oktyabrsky.big-book-edu.ru': 'big-book-edu.ru',
 'petrozavodsk.big-book-edu.ru': 'big-book-edu.ru',
 'odincovo.big-book-edu.ru': 'big-book-edu.ru',
 'orenburg.big-book-edu.ru': 'big-book-edu.ru',
 'tula.big-book-edu.ru': 'big-book-edu.ru',
 'pushkino.big-book-edu.ru': 'big-book-edu.ru',
 'taganrog.big-book-edu.ru': 'big-book-edu.ru',
 'rostov-na-donu.big-book-edu.ru': 'big-book-edu.ru',
 'lipetsk.big-book-edu.ru': 'big-book-edu.ru',
 'stavropol.big-book-edu.ru': 'big-book-edu.ru',
 'balashikha.big-book-edu.ru': 'big-book-edu.ru',
 'orehovo-zuevo.big-book-edu.ru': 'big-book-edu.ru',
 'nakhodka.big-book-edu.ru': 'big-book-edu.ru',
 'omsk.big-book-edu.ru': 'big-book-edu.ru',
 'almetyevsk.big-book-edu.ru': 'big-book-edu.ru',
 'komsomolsk-na-amure.big-book-edu.ru': 'big-book-edu.ru',
 'magnitogorsk.big-book-edu.ru': 'big-book-edu.ru',
 'kaliningrad.big-book-edu.ru': 'big-book-edu.ru',
 'zhukovskij.big-book-edu.ru': 'big-book-edu.ru',
 'nizhnevartovsk.big-book-edu.ru': 'big-book-edu.ru',
 'khimki.big-book-edu.ru': 'big-book-edu.ru',
 'penza.big-book-edu.ru': 'big-book-edu.ru',
 'koroljov.big-book-edu.ru': 'big-book-edu.ru',
 'voronezh.big-book-edu.ru': 'big-book-edu.ru',
 'smolensk.big-book-edu.ru': 'big-book-edu.ru',
 'tolyatti.big-book-edu.ru': 'big-book-edu.ru',
 'syktyvkar.big-book-edu.ru': 'big-book-edu.ru',
 'kaluga.big-book-edu.ru': 'big-book-edu.ru',
 'severodvinsk.big-book-edu.ru': 'big-book-edu.ru',
 'kostroma.big-book-edu.ru': 'big-book-edu.ru',
 'sterlitamak.big-book-edu.ru': 'big-book-edu.ru',
 'novokuznetsk.big-book-edu.ru': 'big-book-edu.ru',
 'orsk.big-book-edu.ru': 'big-book-edu.ru',
 'bratsk.big-book-edu.ru': 'big-book-edu.ru',
 'vladikavkaz.big-book-edu.ru': 'big-book-edu.ru',
 'pyatigorsk.big-book-edu.ru': 'big-book-edu.ru',
 'nizhnekamsk.big-book-edu.ru': 'big-book-edu.ru',
 'vologda.big-book-edu.ru': 'big-book-edu.ru',
 'sergiev-posad.big-book-edu.ru': 'big-book-edu.ru',
 'kamensk-yralski.big-book-edu.ru': 'big-book-edu.ru',
 'podolsk.big-book-edu.ru': 'big-book-edu.ru',
 'murmansk.big-book-edu.ru': 'big-book-edu.ru',
 'dzerzhinsk.big-book-edu.ru': 'big-book-edu.ru',
 'groznyy.big-book-edu.ru': 'big-book-edu.ru',
 'obninsk.big-book-edu.ru': 'big-book-edu.ru',
 'pervoyralsk.big-book-edu.ru': 'big-book-edu.ru',
 'jelektrostal.big-book-edu.ru': 'big-book-edu.ru',
 'novorossiysk.big-book-edu.ru': 'big-book-edu.ru',
 'big-book-edu.ru': 'big-book-edu.ru',
 'arkhangelsk.big-book-edu.ru': 'big-book-edu.ru',
 'kislovodsk.big-book-edu.ru': 'big-book-edu.ru',
 'biysk.big-book-edu.ru': 'big-book-edu.ru',
    'careerist.ru': 'careerist.ru',
 'novosibirskaya.careerist.ru': 'careerist.ru',
 'ryazan.careerist.ru': 'careerist.ru',
 'spb.careerist.ru': 'careerist.ru',
 'ryazanskaya.careerist.ru': 'careerist.ru',
 'aldan.careerist.ru': 'careerist.ru',
 'krasnoyarsk.careerist.ru': 'careerist.ru',
 'novocherkassk.careerist.ru': 'careerist.ru',
 'ufa.careerist.ru': 'careerist.ru',
 'saratov.careerist.ru': 'careerist.ru',
 'solnechnogorsk.careerist.ru': 'careerist.ru',
 'novosibirsk.careerist.ru': 'careerist.ru',
 'temryuk.careerist.ru': 'careerist.ru',
 'novorossiysk.careerist.ru': 'careerist.ru',
 'gelendzhik.careerist.ru': 'careerist.ru',
 'krasnodar.careerist.ru': 'careerist.ru',
 'muravlenko.careerist.ru': 'careerist.ru',
 'rostov.careerist.ru': 'careerist.ru',
 'chekhov.careerist.ru': 'careerist.ru',
 'tyumen.careerist.ru': 'careerist.ru',
 'ekaterinburg.careerist.ru': 'careerist.ru',
 'norilsk.careerist.ru': 'careerist.ru',
 'noyabrsk.careerist.ru': 'careerist.ru',
 'vladimir.careerist.ru': 'careerist.ru',
 'kamensk-uralsky.careerist.ru': 'careerist.ru',
 'kazan.careerist.ru': 'careerist.ru',
 'tula.careerist.ru': 'careerist.ru',
 'pervouralsk.careerist.ru': 'careerist.ru',
 'omsk.careerist.ru': 'careerist.ru',
 'nizhegorodskaya.careerist.ru': 'careerist.ru',
 'belgorod.careerist.ru': 'careerist.ru',
 'neryungri.careerist.ru': 'careerist.ru',
 'ivanovo.careerist.ru': 'careerist.ru',
 'belovo.careerist.ru': 'careerist.ru',
 'nnov.careerist.ru': 'careerist.ru',
 'voronezh.careerist.ru': 'careerist.ru',
 'kingisepp.careerist.ru': 'careerist.ru',
 'izhevsk.careerist.ru': 'careerist.ru',
 'elektrostal.careerist.ru': 'careerist.ru',
 'khabarovsk.careerist.ru': 'careerist.ru',
 'lyubertsy.careerist.ru': 'careerist.ru',
 'syktyvkar.careerist.ru': 'careerist.ru',
 'leningradskaya.careerist.ru': 'careerist.ru',
 'neftekamsk.careerist.ru': 'careerist.ru',
 'novokuybyshevsk.careerist.ru': 'careerist.ru',
 'tomsk.careerist.ru': 'careerist.ru',
 'moskovskij.careerist.ru': 'careerist.ru',
 'sochi.careerist.ru': 'careerist.ru',
 'sterlitamak.careerist.ru': 'careerist.ru',
 'tolyatty.careerist.ru': 'careerist.ru',
 'shakhty.careerist.ru': 'careerist.ru',
 'nizhnekamsk.careerist.ru': 'careerist.ru',
 'kaliningrad.careerist.ru': 'careerist.ru',
 'barnaul.careerist.ru': 'careerist.ru',
 'kurgan.careerist.ru': 'careerist.ru',
 'zhukovsky.careerist.ru': 'careerist.ru',
 'usinsk.careerist.ru': 'careerist.ru',
 'kogalym.careerist.ru': 'careerist.ru',
 'naro-fominsk.careerist.ru': 'careerist.ru',
 'uraj.careerist.ru': 'careerist.ru',
 'pskov.careerist.ru': 'careerist.ru',
 'perm.careerist.ru': 'careerist.ru',
 'tchaikovsky.careerist.ru': 'careerist.ru',
 'kursk.careerist.ru': 'careerist.ru',
 'lipetsk.careerist.ru': 'careerist.ru',
 'iskitim.careerist.ru': 'careerist.ru',
 'novyj-urengoy.careerist.ru': 'careerist.ru',
 'nefteyugansk.careerist.ru': 'careerist.ru',
 'samara.careerist.ru': 'careerist.ru',
 'yaroslavl.careerist.ru': 'careerist.ru',
 'ulan-ude.careerist.ru': 'careerist.ru',
 'petergof.careerist.ru': 'careerist.ru',
 'verkhnyaya-pyshma.careerist.ru': 'careerist.ru',
 'cheboksary.careerist.ru': 'careerist.ru',
 'kemerovo.careerist.ru': 'careerist.ru',
 'novgorod.careerist.ru': 'careerist.ru',
 'kolchugino.careerist.ru': 'careerist.ru',
 'volgograd.careerist.ru': 'careerist.ru',
 'blagoveshchensk.careerist.ru': 'careerist.ru',
 'kaluga.careerist.ru': 'careerist.ru',
 'krym.careerist.ru': 'careerist.ru',
 'naberezhnye.careerist.ru': 'careerist.ru',
 'ishim.careerist.ru': 'careerist.ru',
 'podolsk.careerist.ru': 'careerist.ru',
 'kostroma.careerist.ru': 'careerist.ru',
 'nizhnevartovsk.careerist.ru': 'careerist.ru',
 'dzerzhinsk.careerist.ru': 'careerist.ru',
 'balashikha.careerist.ru': 'careerist.ru',
 'nyagan.careerist.ru': 'careerist.ru',
 'polevskoy.careerist.ru': 'careerist.ru',
 'kumertau.careerist.ru': 'careerist.ru',
 'stavropol.careerist.ru': 'careerist.ru',
 'tuapse.careerist.ru': 'careerist.ru',
 'yamalo-nenetskiy.careerist.ru': 'careerist.ru',
 'salekhard.careerist.ru': 'careerist.ru',
 'novokuznetsk.careerist.ru': 'careerist.ru',
 'kirov.careerist.ru': 'careerist.ru',
 'krasnogorsk.careerist.ru': 'careerist.ru',
 'maikop.careerist.ru': 'careerist.ru',
 'cherepovets.careerist.ru': 'careerist.ru',
 'khimki.careerist.ru': 'careerist.ru',
 'domodedovo.careerist.ru': 'careerist.ru',
 'penza.careerist.ru': 'careerist.ru',
 'tambov.careerist.ru': 'careerist.ru',
 'balakovo.careerist.ru': 'careerist.ru',
 'world.careerist.ru': 'careerist.ru',
    'izhevsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'stavropol.adresa-telefony.ru': 'adresa-telefony.ru',
 'novorossiysk.adresa-telefony.ru': 'adresa-telefony.ru',
 'nizhnevartovsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'novosibirsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'nahodka.adresa-telefony.ru': 'adresa-telefony.ru',
 'vologda.adresa-telefony.ru': 'adresa-telefony.ru',
 'berezniki.adresa-telefony.ru': 'adresa-telefony.ru',
 'orenburg.adresa-telefony.ru': 'adresa-telefony.ru',
 'almetevsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'cheboksary.adresa-telefony.ru': 'adresa-telefony.ru',
 'ufa.adresa-telefony.ru': 'adresa-telefony.ru',
 'spb.adresa-telefony.ru': 'adresa-telefony.ru',
 'kostroma.adresa-telefony.ru': 'adresa-telefony.ru',
 'voronezh.adresa-telefony.ru': 'adresa-telefony.ru',
 'ntagil.adresa-telefony.ru': 'adresa-telefony.ru',
 'kursk.adresa-telefony.ru': 'adresa-telefony.ru',
 'kazan.adresa-telefony.ru': 'adresa-telefony.ru',
 'orel.adresa-telefony.ru': 'adresa-telefony.ru',
 'bryansk.adresa-telefony.ru': 'adresa-telefony.ru',
 'volgodonsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'saratov.adresa-telefony.ru': 'adresa-telefony.ru',
 'moscow.adresa-telefony.ru': 'adresa-telefony.ru',
 'kirov.adresa-telefony.ru': 'adresa-telefony.ru',
 'adresa-telefony.ru': 'adresa-telefony.ru',
 'vladikavkaz.adresa-telefony.ru': 'adresa-telefony.ru',
 'barnaul.adresa-telefony.ru': 'adresa-telefony.ru',
 'krasnodar.adresa-telefony.ru': 'adresa-telefony.ru',
 'kurgan.adresa-telefony.ru': 'adresa-telefony.ru',
 'omsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'sochi.adresa-telefony.ru': 'adresa-telefony.ru',
 'novokuznetsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'n-novgorod.adresa-telefony.ru': 'adresa-telefony.ru',
 'noyabrsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'murmansk.adresa-telefony.ru': 'adresa-telefony.ru',
 'tyumen.adresa-telefony.ru': 'adresa-telefony.ru',
 'yoshkarola.adresa-telefony.ru': 'adresa-telefony.ru',
 'podolsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'samara.adresa-telefony.ru': 'adresa-telefony.ru',
 'armawir.adresa-telefony.ru': 'adresa-telefony.ru',
 'tashkent.adresa-telefony.ru': 'adresa-telefony.ru',
 'kaluga.adresa-telefony.ru': 'adresa-telefony.ru',
 'tobolsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'ekaterinburg.adresa-telefony.ru': 'adresa-telefony.ru',
 'magnitogorsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'ussuriysk.adresa-telefony.ru': 'adresa-telefony.ru',
 'korolev.adresa-telefony.ru': 'adresa-telefony.ru',
 'pyatigorsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'ulyanovsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'bataysk.adresa-telefony.ru': 'adresa-telefony.ru',
 'rostov.adresa-telefony.ru': 'adresa-telefony.ru',
 'abakan.adresa-telefony.ru': 'adresa-telefony.ru',
 'sterlitamak.adresa-telefony.ru': 'adresa-telefony.ru',
 'norilsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'neftekamsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'pskov.adresa-telefony.ru': 'adresa-telefony.ru',
 'perm.adresa-telefony.ru': 'adresa-telefony.ru',
 'maykop.adresa-telefony.ru': 'adresa-telefony.ru',
 'chelny.adresa-telefony.ru': 'adresa-telefony.ru',
 'balashiha.adresa-telefony.ru': 'adresa-telefony.ru',
 'surgut.adresa-telefony.ru': 'adresa-telefony.ru',
 'lipetsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'baku.adresa-telefony.ru': 'adresa-telefony.ru',
 'tula.adresa-telefony.ru': 'adresa-telefony.ru',
 'kemerovo.adresa-telefony.ru': 'adresa-telefony.ru',
 'nizhnekamsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'vladivostok.adresa-telefony.ru': 'adresa-telefony.ru',
 'magadan.adresa-telefony.ru': 'adresa-telefony.ru',
 'smolensk.adresa-telefony.ru': 'adresa-telefony.ru',
 'anapa.adresa-telefony.ru': 'adresa-telefony.ru',
 'penza.adresa-telefony.ru': 'adresa-telefony.ru',
 'pushkino.adresa-telefony.ru': 'adresa-telefony.ru',
 'tambov.adresa-telefony.ru': 'adresa-telefony.ru',
 'belgorod.adresa-telefony.ru': 'adresa-telefony.ru',
 'orsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'petrozavodsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'ryazan.adresa-telefony.ru': 'adresa-telefony.ru',
 'yaroslavl.adresa-telefony.ru': 'adresa-telefony.ru',
 'petropavlovsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'kolomna.adresa-telefony.ru': 'adresa-telefony.ru',
 'solnechnogorsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'kyzyl.adresa-telefony.ru': 'adresa-telefony.ru',
 'staroskol.adresa-telefony.ru': 'adresa-telefony.ru',
 'cherepovets.adresa-telefony.ru': 'adresa-telefony.ru',
 'gelendzhik.adresa-telefony.ru': 'adresa-telefony.ru',
 'urengoy.adresa-telefony.ru': 'adresa-telefony.ru',
 'miass.adresa-telefony.ru': 'adresa-telefony.ru',
 'pavposad.adresa-telefony.ru': 'adresa-telefony.ru',
 'murom.adresa-telefony.ru': 'adresa-telefony.ru',
 'grozny.adresa-telefony.ru': 'adresa-telefony.ru',
 'tver.adresa-telefony.ru': 'adresa-telefony.ru',
 'almaty.adresa-telefony.ru': 'adresa-telefony.ru',
 'taganrog.adresa-telefony.ru': 'adresa-telefony.ru',
 'tolyatti.adresa-telefony.ru': 'adresa-telefony.ru',
 'noginsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'biysk.adresa-telefony.ru': 'adresa-telefony.ru',
 'zelenograd.adresa-telefony.ru': 'adresa-telefony.ru',
 'vladimir.adresa-telefony.ru': 'adresa-telefony.ru',
 'tomsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'engels.adresa-telefony.ru': 'adresa-telefony.ru',
 'chita.adresa-telefony.ru': 'adresa-telefony.ru',
 'yakutsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'seversk.adresa-telefony.ru': 'adresa-telefony.ru',
 'dolgoprudny.adresa-telefony.ru': 'adresa-telefony.ru',
 'aksay.adresa-telefony.ru': 'adresa-telefony.ru',
 'georgievsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'kislovodsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'nefteyugansk.adresa-telefony.ru': 'adresa-telefony.ru',
    'dzvr.ru': 'dzvr.ru',
 'rostov.dzvr.ru': 'dzvr.ru',
 'tula.dzvr.ru': 'dzvr.ru',
 'voronez.dzvr.ru': 'dzvr.ru',
 'ladogsky.dzvr.ru': 'dzvr.ru',
 'sochi.dzvr.ru': 'dzvr.ru',
 'kazansky.dzvr.ru': 'dzvr.ru',
 'leningradsky.dzvr.ru': 'dzvr.ru',
 'moskovsky.dzvr.ru': 'dzvr.ru',
 'izevsk.dzvr.ru': 'dzvr.ru',
 'finlyandsky.dzvr.ru': 'dzvr.ru',
 'belgorod.dzvr.ru': 'dzvr.ru',
 'vladivostok.dzvr.ru': 'dzvr.ru',
 'sverdlovsk.dzvr.ru': 'dzvr.ru',
 'abakan.dzvr.ru': 'dzvr.ru',
 'adler.dzvr.ru': 'dzvr.ru',
 'orenburg.dzvr.ru': 'dzvr.ru',
 'tver.dzvr.ru': 'dzvr.ru',
 'anapa.dzvr.ru': 'dzvr.ru',
 'krasnodar-1.dzvr.ru': 'dzvr.ru',
 'rizhsky.dzvr.ru': 'dzvr.ru',
 'ufa.dzvr.ru': 'dzvr.ru',
 'yaroslavsky.dzvr.ru': 'dzvr.ru',
 'kievsky.dzvr.ru': 'dzvr.ru',
 'samara.dzvr.ru': 'dzvr.ru',
 'irkutsk-passazirskiy.dzvr.ru': 'dzvr.ru',
 'novosibirsk-glavnyiy.dzvr.ru': 'dzvr.ru',
 'kursky.dzvr.ru': 'dzvr.ru',
 'yaroslavl-glavnyiy.dzvr.ru': 'dzvr.ru',
 'savelovsky.dzvr.ru': 'dzvr.ru',
 'krasnoyarsk.dzvr.ru': 'dzvr.ru',
 'baltisky.dzvr.ru': 'dzvr.ru',
 'surgut.dzvr.ru': 'dzvr.ru',
 'belorussky.dzvr.ru': 'dzvr.ru',
 'ryazan-2.dzvr.ru': 'dzvr.ru',
 'pskov.dzvr.ru': 'dzvr.ru',
 'dzerzinsk.dzvr.ru': 'dzvr.ru',
 'vladimir.dzvr.ru': 'dzvr.ru',
 'nizhnijnovgorod.dzvr.ru': 'dzvr.ru',
 'paveletsky.dzvr.ru': 'dzvr.ru',
 'novorossiysk.dzvr.ru': 'dzvr.ru',
 'omsk.dzvr.ru': 'dzvr.ru',
 'olimpijskij-park.dzvr.ru': 'dzvr.ru',
 'belogorsk.dzvr.ru': 'dzvr.ru',
 'postcard.dzvr.ru': 'dzvr.ru',
 'ussuriysk.dzvr.ru': 'dzvr.ru',
 'penza-1.dzvr.ru': 'dzvr.ru',
 'kaluga-1.dzvr.ru': 'dzvr.ru',
 'kaliningrad-yuznyiy.dzvr.ru': 'dzvr.ru',
 'vladikavkaz.dzvr.ru': 'dzvr.ru',
 'habarovsk-1.dzvr.ru': 'dzvr.ru',
 'petrozavodsk.dzvr.ru': 'dzvr.ru',
 'volzaltyumen.dzvr.ru': 'dzvr.ru',
 'ivanovo.dzvr.ru': 'dzvr.ru',
 'nizniytagil.dzvr.ru': 'dzvr.ru',
 'yanaul.dzvr.ru': 'dzvr.ru',
 'rossosh.dzvr.ru': 'dzvr.ru',
 'kislovodsk.dzvr.ru': 'dzvr.ru',
 'staryiyoskol.dzvr.ru': 'dzvr.ru',
 'perm-2.dzvr.ru': 'dzvr.ru',
 'vitebsky.dzvr.ru': 'dzvr.ru',
 'barnaul.dzvr.ru': 'dzvr.ru',
 'murmansk.dzvr.ru': 'dzvr.ru',
 'zigulevskoemore.dzvr.ru': 'dzvr.ru',
 'arhangelsk.dzvr.ru': 'dzvr.ru',
 'tomsk.dzvr.ru': 'dzvr.ru',
 'kurgan.dzvr.ru': 'dzvr.ru',
 'vologda.dzvr.ru': 'dzvr.ru',
 'ryazan-1.dzvr.ru': 'dzvr.ru',
 'smolensk.dzvr.ru': 'dzvr.ru',
 'groznyiy.dzvr.ru': 'dzvr.ru',
 'blagoveschensk.dzvr.ru': 'dzvr.ru',
 'tolyatti.dzvr.ru': 'dzvr.ru',
 'michurinsk-uralskiy.dzvr.ru': 'dzvr.ru',
 'saratov-1.dzvr.ru': 'dzvr.ru',
 'pridacha.dzvr.ru': 'dzvr.ru',
 'syiktyivkar.dzvr.ru': 'dzvr.ru',
 'uhta.dzvr.ru': 'dzvr.ru',
 'kirov.dzvr.ru': 'dzvr.ru',
 'komsomolsk.dzvr.ru': 'dzvr.ru',
 'yaroslavl.dzvr.ru': 'dzvr.ru',
 'kazan.dzvr.ru': 'dzvr.ru',
 'vosstanie.dzvr.ru': 'dzvr.ru',
 'mineralnyievodyi.dzvr.ru': 'dzvr.ru',
 'nevinnomyissk.dzvr.ru': 'dzvr.ru',
 'stavropol.dzvr.ru': 'dzvr.ru',
 'tayga.dzvr.ru': 'dzvr.ru',
 'mahachkala.dzvr.ru': 'dzvr.ru',
 'chelyabinsk.dzvr.ru': 'dzvr.ru',
 'maykop.dzvr.ru': 'dzvr.ru',
 'volzskiy.dzvr.ru': 'dzvr.ru',
 'labytnangi.dzvr.ru': 'dzvr.ru',
 'yoshkar-ola.dzvr.ru': 'dzvr.ru',
 'bolotnoe.dzvr.ru': 'dzvr.ru',
 'chapaevsk.dzvr.ru': 'dzvr.ru',
 'kanash.dzvr.ru': 'dzvr.ru',
 'biysk.dzvr.ru': 'dzvr.ru',
 'kansk-eniseyskiy.dzvr.ru': 'dzvr.ru',
 'kostroma.dzvr.ru': 'dzvr.ru',
 'agryiz.dzvr.ru': 'dzvr.ru',
 'pyit-yah.dzvr.ru': 'dzvr.ru',
 'volgograd.dzvr.ru': 'dzvr.ru',
 'pjatigorsk.dzvr.ru': 'dzvr.ru',
 'vyborg.dzvr.ru': 'dzvr.ru',
 'tobolsk.dzvr.ru': 'dzvr.ru',
 'gryazivoronezskie.dzvr.ru': 'dzvr.ru',
 'kotlas.dzvr.ru': 'dzvr.ru',
 'chita-2.dzvr.ru': 'dzvr.ru',
 'kemerovo.dzvr.ru': 'dzvr.ru',
    'tuapse.splar.ru': 'splar.ru',
 'nizhniy-tagil.splar.ru': 'splar.ru',
 'kovrov.splar.ru': 'splar.ru',
 'shadrinsk.splar.ru': 'splar.ru',
 'rostov-na-donu.splar.ru': 'splar.ru',
 'kozelsk.splar.ru': 'splar.ru',
 'cheboksary.splar.ru': 'splar.ru',
 'smolensk.splar.ru': 'splar.ru',
 'ryazan.splar.ru': 'splar.ru',
 'splar.ru': 'splar.ru',
 'tuymazy.splar.ru': 'splar.ru',
 'samara.splar.ru': 'splar.ru',
 'krasnoyarsk.splar.ru': 'splar.ru',
 'abakan.splar.ru': 'splar.ru',
 'khabarovsk.splar.ru': 'splar.ru',
 'novosibirsk.splar.ru': 'splar.ru',
 'chelyabinsk.splar.ru': 'splar.ru',
 'stupino.splar.ru': 'splar.ru',
 'taganrog.splar.ru': 'splar.ru',
 'sankt-peterburg.splar.ru': 'splar.ru',
 'barnaul.splar.ru': 'splar.ru',
 'ekaterinburg.splar.ru': 'splar.ru',
 'tyumen.splar.ru': 'splar.ru',
 'ivanteevka.splar.ru': 'splar.ru',
 'stavropol.splar.ru': 'splar.ru',
 'krasnodar.splar.ru': 'splar.ru',
 'podolsk.splar.ru': 'splar.ru',
 'voronezh.splar.ru': 'splar.ru',
 'almetevsk.splar.ru': 'splar.ru',
 'kazan.splar.ru': 'splar.ru',
 'salavat.splar.ru': 'splar.ru',
 'kostroma.splar.ru': 'splar.ru',
 'bryansk.splar.ru': 'splar.ru',
 'novotroitsk.splar.ru': 'splar.ru',
 'ukhta.splar.ru': 'splar.ru',
 'naberezhnye-chelny.splar.ru': 'splar.ru',
 'asha.splar.ru': 'splar.ru',
 'evpatoriya.splar.ru': 'splar.ru',
 'dzerzhinsk.splar.ru': 'splar.ru',
 'sochi.splar.ru': 'splar.ru',
 'balashikha.splar.ru': 'splar.ru',
 'kamensk-shakhtinskiy.splar.ru': 'splar.ru',
 'tver.splar.ru': 'splar.ru',
 'nizhniy-novgorod.splar.ru': 'splar.ru',
 'glazov.splar.ru': 'splar.ru',
 'omsk.splar.ru': 'splar.ru',
 'kirov.splar.ru': 'splar.ru',
 'kyzyl.splar.ru': 'splar.ru',
 'anapa.splar.ru': 'splar.ru',
 'yakutsk.splar.ru': 'splar.ru',
 'orsk.splar.ru': 'splar.ru',
 'novokuznetsk.splar.ru': 'splar.ru',
 'volgograd.splar.ru': 'splar.ru',
 'arkhangelsk.splar.ru': 'splar.ru',
 'belorechensk.splar.ru': 'splar.ru',
 'vyazniki.splar.ru': 'splar.ru',
 'biysk.splar.ru': 'splar.ru',
 'irkutsk.splar.ru': 'splar.ru',
 'shchelkovo.splar.ru': 'splar.ru',
 'vladikavkaz.splar.ru': 'splar.ru',
 'yaroslavl.splar.ru': 'splar.ru',
 'obninsk.splar.ru': 'splar.ru',
 'balakovo.splar.ru': 'splar.ru',
 'novocheboksarsk.splar.ru': 'splar.ru',
 'ishimbay.splar.ru': 'splar.ru',
 'ulyanovsk.splar.ru': 'splar.ru',
 'simferopol.splar.ru': 'splar.ru',
 'penza.splar.ru': 'splar.ru',
 'khanty-mansiysk.splar.ru': 'splar.ru',
 'kaliningrad.splar.ru': 'splar.ru',
 'tomsk.splar.ru': 'splar.ru',
 'kamensk-uralskiy.splar.ru': 'splar.ru',
 'bataysk.splar.ru': 'splar.ru',
 'pushkino.splar.ru': 'splar.ru',
 'izhevsk.splar.ru': 'splar.ru',
 'gorno-altaysk.splar.ru': 'splar.ru',
 'verkhnyaya-pyshma.splar.ru': 'splar.ru',
 'temryuk.splar.ru': 'splar.ru',
 'kandalaksha.splar.ru': 'splar.ru',
 'mineralnye-vody.splar.ru': 'splar.ru',
 'magnitogorsk.splar.ru': 'splar.ru',
 'berdsk.splar.ru': 'splar.ru',
 'sterlitamak.splar.ru': 'splar.ru',
 'zelenograd.splar.ru': 'splar.ru',
 'budennovsk.splar.ru': 'splar.ru',
 'tula.splar.ru': 'splar.ru',
 'feodosiya.splar.ru': 'splar.ru',
 'krasnogorsk.splar.ru': 'splar.ru',
 'dmitrov.splar.ru': 'splar.ru',
 'belovo.splar.ru': 'splar.ru',
 'zheleznodorozhnyy.splar.ru': 'splar.ru',
 'solikamsk.splar.ru': 'splar.ru',
 'yurga.splar.ru': 'splar.ru',
 'mytishchi.splar.ru': 'splar.ru',
 'zlatoust.splar.ru': 'splar.ru',
 'vyborg.splar.ru': 'splar.ru',
 'tolyatti.splar.ru': 'splar.ru',
 'vologda.splar.ru': 'splar.ru',
 'eysk.splar.ru': 'splar.ru',
 'vorkuta.splar.ru': 'splar.ru',
 'chekhov.splar.ru': 'splar.ru',
 'petropavlovsk-kamchatskiy.splar.ru': 'splar.ru',
 'orenburg.splar.ru': 'splar.ru',
 'meleuz.splar.ru': 'splar.ru',
 'neftekamsk.splar.ru': 'splar.ru',
 'orekhovo-zuevo.splar.ru': 'splar.ru',
 'komsomolsk-na-amure.splar.ru': 'splar.ru',
 'petrozavodsk.splar.ru': 'splar.ru',
 'pskov.splar.ru': 'splar.ru',
 'nizhnevartovsk.splar.ru': 'splar.ru',
    '110km.ru': '110km.ru',
 'forum.110km.ru': '110km.ru',
 'stpetersburg.110km.ru': '110km.ru',
 'moskovskaya-oblast.110km.ru': '110km.ru',
 'moscow.110km.ru': '110km.ru',
 'vladivostok.110km.ru': '110km.ru',
 'leningradskaya-oblast.110km.ru': '110km.ru',
 'ekaterinburg.110km.ru': '110km.ru',
 'sverdlovskaya-oblast.110km.ru': '110km.ru',
 'petrozavodsk.110km.ru': '110km.ru',
 'kaluzhskaya-oblast.110km.ru': '110km.ru',
 'buy.110km.ru': '110km.ru',
 'kemerovskaya-oblast.110km.ru': '110km.ru',
 'kemerovo.110km.ru': '110km.ru',
 'izhevsk.110km.ru': '110km.ru',
 'sterlitamak.110km.ru': '110km.ru',
 'novosibirsk.110km.ru': '110km.ru',
 'ufa.110km.ru': '110km.ru',
 'tatarstan.110km.ru': '110km.ru',
 'kazan.110km.ru': '110km.ru',
 'ryazan.110km.ru': '110km.ru',
 'lipetsk.110km.ru': '110km.ru',
 'samarskaya-oblast.110km.ru': '110km.ru',
 'yoshkar-ola.110km.ru': '110km.ru',
 'habarovsk.110km.ru': '110km.ru',
 'arhangelsk.110km.ru': '110km.ru',
 'volgograd.110km.ru': '110km.ru',
 'volgogradskaya-oblast.110km.ru': '110km.ru',
 'sochi.110km.ru': '110km.ru',
 'syzran.110km.ru': '110km.ru',
 'orsk.110km.ru': '110km.ru',
 'vladimir.110km.ru': '110km.ru',
 'krasnodar.110km.ru': '110km.ru',
 'krasnodarskyy-kray.110km.ru': '110km.ru',
 'penza.110km.ru': '110km.ru',
 'nizhniy-novgorod.110km.ru': '110km.ru',
 'blagoveschensk.110km.ru': '110km.ru',
 'amurskaya-oblast.110km.ru': '110km.ru',
 'armavir.110km.ru': '110km.ru',
 'marnitogorsk.110km.ru': '110km.ru',
 'chelyabinskaya-oblast.110km.ru': '110km.ru',
 'stavropolskiy-kray.110km.ru': '110km.ru',
 'samara.110km.ru': '110km.ru',
 'murmansk.110km.ru': '110km.ru',
 'murmanskaya-oblast.110km.ru': '110km.ru',
 'uhta.110km.ru': '110km.ru',
 'nizhegorodskaya-oblast.110km.ru': '110km.ru',
 'krasnoyarskiy-kray.110km.ru': '110km.ru',
 'krasnoyarsk.110km.ru': '110km.ru',
 'belgorod.110km.ru': '110km.ru',
 'tver.110km.ru': '110km.ru',
 'cheboksary.110km.ru': '110km.ru',
 'novorossiysk.110km.ru': '110km.ru',
 'rostov-na-donu.110km.ru': '110km.ru',
 'vladikavkaz.110km.ru': '110km.ru',
 'novosibirskaya-oblast.110km.ru': '110km.ru',
 'nizhnevartovsk.110km.ru': '110km.ru',
 'tverskaya-oblast.110km.ru': '110km.ru',
 'noyabrsk.110km.ru': '110km.ru',
 'komsomolsk-na-amure.110km.ru': '110km.ru',
 'omsk.110km.ru': '110km.ru',
 'orenburgskaya-oblast.110km.ru': '110km.ru',
 'tyumen.110km.ru': '110km.ru',
 'chelyabinsk.110km.ru': '110km.ru',
 'maykop.110km.ru': '110km.ru',
 'bashkortostan.110km.ru': '110km.ru',
 'udmurtiya.110km.ru': '110km.ru',
 'surgut.110km.ru': '110km.ru',
 'nahodka.110km.ru': '110km.ru',
 'kaliningrad.110km.ru': '110km.ru',
 'ivanovo.110km.ru': '110km.ru',
 'novokuznetsk.110km.ru': '110km.ru',
 'bryansk.110km.ru': '110km.ru',
 'orenburg.110km.ru': '110km.ru',
 'saratovskaya-oblast.110km.ru': '110km.ru',
 'neftekamsk.110km.ru': '110km.ru',
 'tulskaya-oblast.110km.ru': '110km.ru',
 'zlatoust.110km.ru': '110km.ru',
 'shahty.110km.ru': '110km.ru',
 'novocherkassk.110km.ru': '110km.ru',
 'rostovskaya-oblast.110km.ru': '110km.ru',
 'volgodonsk.110km.ru': '110km.ru',
 'taganrog.110km.ru': '110km.ru',
 'yaroslavl.110km.ru': '110km.ru',
 'tomsk.110km.ru': '110km.ru',
 'kareliya.110km.ru': '110km.ru',
 'tambovskaya-oblast.110km.ru': '110km.ru',
 'voronezh.110km.ru': '110km.ru',
 'astrahanskaya-oblast.110km.ru': '110km.ru',
 'astrahan.110km.ru': '110km.ru',
 'tuapse.110km.ru': '110km.ru',
 'hanty-mansiysk.110km.ru': '110km.ru',
 'tolyatti.110km.ru': '110km.ru',
 'kirovskaya-oblast.110km.ru': '110km.ru',
 'norilsk.110km.ru': '110km.ru',
 'tobolsk.110km.ru': '110km.ru',
 'penzenskaya-oblast.110km.ru': '110km.ru',
 'permskiy-kray.110km.ru': '110km.ru',
 'ulyanovsk.110km.ru': '110km.ru',
 'tula.110km.ru': '110km.ru',
 'pskovskaya-oblast.110km.ru': '110km.ru',
 'perm.110km.ru': '110km.ru',
 'abakan.110km.ru': '110km.ru',
 'hakasiya.110km.ru': '110km.ru',
 'kurgan.110km.ru': '110km.ru',
 'kopeysk.110km.ru': '110km.ru',
 'petropavlovsk-kamchatskiy.110km.ru': '110km.ru',
 'voronezhskaya-oblast.110km.ru': '110km.ru',
 'naberezhnye-chelny.110km.ru': '110km.ru',
 'kaluga.110km.ru': '110km.ru',
 'orel.110km.ru': '110km.ru',
 'kabardino-balkariya.110km.ru': '110km.ru',
 'komi.110km.ru': '110km.ru',
    'jobcareer.ru': 'jobcareer.ru',
 'naro-fominsk.jobcareer.ru': 'jobcareer.ru',
 'spb.jobcareer.ru': 'jobcareer.ru',
 'stary-oskol.jobcareer.ru': 'jobcareer.ru',
 'glazov.jobcareer.ru': 'jobcareer.ru',
 'saratov.jobcareer.ru': 'jobcareer.ru',
 'domodedovo.jobcareer.ru': 'jobcareer.ru',
 'zhukovsky.jobcareer.ru': 'jobcareer.ru',
 'solnechnogorsk.jobcareer.ru': 'jobcareer.ru',
 'khabarovsk.jobcareer.ru': 'jobcareer.ru',
 'chelyabinsk.jobcareer.ru': 'jobcareer.ru',
 'polevskoy.jobcareer.ru': 'jobcareer.ru',
 'saransk.jobcareer.ru': 'jobcareer.ru',
 'msk.jobcareer.ru': 'jobcareer.ru',
 'irkutsk.jobcareer.ru': 'jobcareer.ru',
 'kirov.jobcareer.ru': 'jobcareer.ru',
 'ekaterinburg.jobcareer.ru': 'jobcareer.ru',
 'petropavlovsk-kamchatskiy.jobcareer.ru': 'jobcareer.ru',
 'magadan.jobcareer.ru': 'jobcareer.ru',
 'krasnoyarsk.jobcareer.ru': 'jobcareer.ru',
 'samara.jobcareer.ru': 'jobcareer.ru',
 'tyumenskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'tyumen.jobcareer.ru': 'jobcareer.ru',
 'kanash.jobcareer.ru': 'jobcareer.ru',
 'kerch.jobcareer.ru': 'jobcareer.ru',
 'izhevsk.jobcareer.ru': 'jobcareer.ru',
 'balashiha.jobcareer.ru': 'jobcareer.ru',
 'kemerovo.jobcareer.ru': 'jobcareer.ru',
 'dolgoprudny.jobcareer.ru': 'jobcareer.ru',
 'orenburg.jobcareer.ru': 'jobcareer.ru',
 'usinsk.jobcareer.ru': 'jobcareer.ru',
 'kaliningrad.jobcareer.ru': 'jobcareer.ru',
 'kazan.jobcareer.ru': 'jobcareer.ru',
 'perm.jobcareer.ru': 'jobcareer.ru',
 'surgut.jobcareer.ru': 'jobcareer.ru',
 'klin.jobcareer.ru': 'jobcareer.ru',
 'krasnouralsk.jobcareer.ru': 'jobcareer.ru',
 'novoaltaysk.jobcareer.ru': 'jobcareer.ru',
 'vladimir.jobcareer.ru': 'jobcareer.ru',
 'mtsensk.jobcareer.ru': 'jobcareer.ru',
 'pyt-yah.jobcareer.ru': 'jobcareer.ru',
 'simferopol.jobcareer.ru': 'jobcareer.ru',
 'anapa.jobcareer.ru': 'jobcareer.ru',
 'nahodka.jobcareer.ru': 'jobcareer.ru',
 'yermolino.jobcareer.ru': 'jobcareer.ru',
 'novosibirsk.jobcareer.ru': 'jobcareer.ru',
 'miass.jobcareer.ru': 'jobcareer.ru',
 'ufa.jobcareer.ru': 'jobcareer.ru',
 'kurganinsk.jobcareer.ru': 'jobcareer.ru',
 'obninsk.jobcareer.ru': 'jobcareer.ru',
 'tolyatti.jobcareer.ru': 'jobcareer.ru',
 'novotroick.jobcareer.ru': 'jobcareer.ru',
 'krasnodar.jobcareer.ru': 'jobcareer.ru',
 'moskovskaja-oblast.jobcareer.ru': 'jobcareer.ru',
 'svobodnyj.jobcareer.ru': 'jobcareer.ru',
 'lyubertsy.jobcareer.ru': 'jobcareer.ru',
 'pyatigorsk.jobcareer.ru': 'jobcareer.ru',
 'orsk.jobcareer.ru': 'jobcareer.ru',
 'yakutsk.jobcareer.ru': 'jobcareer.ru',
 'sochi.jobcareer.ru': 'jobcareer.ru',
 'pushkino.jobcareer.ru': 'jobcareer.ru',
 'yamalo-neneckij-avtonomnyj-okrug.jobcareer.ru': 'jobcareer.ru',
 'novosibirskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'elektrostal.jobcareer.ru': 'jobcareer.ru',
 'chechenskaya-respublika.jobcareer.ru': 'jobcareer.ru',
 'golitsino.jobcareer.ru': 'jobcareer.ru',
 'tikhoretsk.jobcareer.ru': 'jobcareer.ru',
 'tynda.jobcareer.ru': 'jobcareer.ru',
 'respublika-krym.jobcareer.ru': 'jobcareer.ru',
 'ukhta.jobcareer.ru': 'jobcareer.ru',
 'belogorsk.jobcareer.ru': 'jobcareer.ru',
 'rostov.jobcareer.ru': 'jobcareer.ru',
 'mezhdurechensk.jobcareer.ru': 'jobcareer.ru',
 'astrakhan.jobcareer.ru': 'jobcareer.ru',
 'timashevsk.jobcareer.ru': 'jobcareer.ru',
 'tula.jobcareer.ru': 'jobcareer.ru',
 'yaroslavl.jobcareer.ru': 'jobcareer.ru',
 'komsomolsk-na-amure.jobcareer.ru': 'jobcareer.ru',
 'shahty.jobcareer.ru': 'jobcareer.ru',
 'vyborg.jobcareer.ru': 'jobcareer.ru',
 'kamchatskij-kraj.jobcareer.ru': 'jobcareer.ru',
 'tobolsk.jobcareer.ru': 'jobcareer.ru',
 'tver.jobcareer.ru': 'jobcareer.ru',
 'yaroslavskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'balakovo.jobcareer.ru': 'jobcareer.ru',
 'mytischi.jobcareer.ru': 'jobcareer.ru',
 'staraya-russa.jobcareer.ru': 'jobcareer.ru',
 'nizhnekamsk.jobcareer.ru': 'jobcareer.ru',
 'podolsk.jobcareer.ru': 'jobcareer.ru',
 'armavir.jobcareer.ru': 'jobcareer.ru',
 'kaluzhskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'temryuk.jobcareer.ru': 'jobcareer.ru',
 'murmansk.jobcareer.ru': 'jobcareer.ru',
 'serpuhov.jobcareer.ru': 'jobcareer.ru',
 'norilsk.jobcareer.ru': 'jobcareer.ru',
 'yoshkar-ola.jobcareer.ru': 'jobcareer.ru',
 'volgogradskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'noviy-urengoy.jobcareer.ru': 'jobcareer.ru',
 'naberezhnye-chelny.jobcareer.ru': 'jobcareer.ru',
 'kstovo-nn.jobcareer.ru': 'jobcareer.ru',
 'respublika-saha-yakutiya.jobcareer.ru': 'jobcareer.ru',
 'korolev.jobcareer.ru': 'jobcareer.ru',
 'ulyanovskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'novorossiysk.jobcareer.ru': 'jobcareer.ru',
 'yalta.jobcareer.ru': 'jobcareer.ru',
 'volgograd.jobcareer.ru': 'jobcareer.ru',
 'penza.jobcareer.ru': 'jobcareer.ru',
 'sverdlovskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'salavat.jobcareer.ru': 'jobcareer.ru',
 'zhigulevsk.jobcareer.ru': 'jobcareer.ru',
 'saratovskaya-oblast.jobcareer.ru': 'jobcareer.ru',
 'moskva-i-oblast.jobcareer.ru': 'jobcareer.ru',
 'shushary.jobcareer.ru': 'jobcareer.ru',
 'omsk.jobcareer.ru': 'jobcareer.ru',
    'spb.russianrealty.ru': 'russianrealty.ru',
 'russianrealty.ru': 'russianrealty.ru',
 'belgorodskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'tyumen.russianrealty.ru': 'russianrealty.ru',
 'mo.russianrealty.ru': 'russianrealty.ru',
 'krasnodar-krd.russianrealty.ru': 'russianrealty.ru',
 'saratov.russianrealty.ru': 'russianrealty.ru',
 'solnechnogorsk.russianrealty.ru': 'russianrealty.ru',
 'sertolovo.russianrealty.ru': 'russianrealty.ru',
 'voronezhskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'stavropol.russianrealty.ru': 'russianrealty.ru',
 'perm.russianrealty.ru': 'russianrealty.ru',
 'voronezh.russianrealty.ru': 'russianrealty.ru',
 'furmanov.russianrealty.ru': 'russianrealty.ru',
 'ivanovskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'samara.russianrealty.ru': 'russianrealty.ru',
 'udmurtiya.russianrealty.ru': 'russianrealty.ru',
 'nso.russianrealty.ru': 'russianrealty.ru',
 'ekaterinburg.russianrealty.ru': 'russianrealty.ru',
 'nnov.russianrealty.ru': 'russianrealty.ru',
 'shcherbinka.russianrealty.ru': 'russianrealty.ru',
 'novosibirsk.russianrealty.ru': 'russianrealty.ru',
 'chapaevsk.russianrealty.ru': 'russianrealty.ru',
 'kemerovo.russianrealty.ru': 'russianrealty.ru',
 'pyatigorsk.russianrealty.ru': 'russianrealty.ru',
 'ufa.russianrealty.ru': 'russianrealty.ru',
 'pushkino.russianrealty.ru': 'russianrealty.ru',
 'vladimirskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'cheboksary.russianrealty.ru': 'russianrealty.ru',
 'himki.russianrealty.ru': 'russianrealty.ru',
 'yaroslavl.russianrealty.ru': 'russianrealty.ru',
 'anapa.russianrealty.ru': 'russianrealty.ru',
 'kirov-vyatka.russianrealty.ru': 'russianrealty.ru',
 'sochi.russianrealty.ru': 'russianrealty.ru',
 'tatarstan.russianrealty.ru': 'russianrealty.ru',
 'podolsk.russianrealty.ru': 'russianrealty.ru',
 'ryazanskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'orenburg.russianrealty.ru': 'russianrealty.ru',
 'lyubertsy.russianrealty.ru': 'russianrealty.ru',
 'mytischi.russianrealty.ru': 'russianrealty.ru',
 'zelenograd.russianrealty.ru': 'russianrealty.ru',
 'kurgan.russianrealty.ru': 'russianrealty.ru',
 'lenobl.russianrealty.ru': 'russianrealty.ru',
 'vsevolozhsk.russianrealty.ru': 'russianrealty.ru',
 'kazan.russianrealty.ru': 'russianrealty.ru',
 'ramenskoe.russianrealty.ru': 'russianrealty.ru',
 'krasnoyarsky-kray.russianrealty.ru': 'russianrealty.ru',
 'krasnoyarsk.russianrealty.ru': 'russianrealty.ru',
 'krasnodar.russianrealty.ru': 'russianrealty.ru',
 'nizhny-tagil.russianrealty.ru': 'russianrealty.ru',
 'barnaul.russianrealty.ru': 'russianrealty.ru',
 'tolyatti.russianrealty.ru': 'russianrealty.ru',
 'orsk.russianrealty.ru': 'russianrealty.ru',
 'sevastopol.russianrealty.ru': 'russianrealty.ru',
 'nizhny-novgorod.russianrealty.ru': 'russianrealty.ru',
 'izhevsk.russianrealty.ru': 'russianrealty.ru',
 'nizhnevartovsk.russianrealty.ru': 'russianrealty.ru',
 'chelyabinsk.russianrealty.ru': 'russianrealty.ru',
 'tyumenskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'kgd.russianrealty.ru': 'russianrealty.ru',
 'tulskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'magadanskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'ryazan.russianrealty.ru': 'russianrealty.ru',
 'vidnoe.russianrealty.ru': 'russianrealty.ru',
 'magnitogorsk.russianrealty.ru': 'russianrealty.ru',
 'krasnogorsk.russianrealty.ru': 'russianrealty.ru',
 'saratovskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'samarskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'irk.russianrealty.ru': 'russianrealty.ru',
 'gatchina.russianrealty.ru': 'russianrealty.ru',
 'moskovsky.russianrealty.ru': 'russianrealty.ru',
 'bataysk.russianrealty.ru': 'russianrealty.ru',
 'tverskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'habarovsk.russianrealty.ru': 'russianrealty.ru',
 'fryazino.russianrealty.ru': 'russianrealty.ru',
 'sakhalin.russianrealty.ru': 'russianrealty.ru',
 'uglegorsk.russianrealty.ru': 'russianrealty.ru',
 'chehov-7.russianrealty.ru': 'russianrealty.ru',
 'korolev.russianrealty.ru': 'russianrealty.ru',
 'yubileyny.russianrealty.ru': 'russianrealty.ru',
 'yoshkar-ola.russianrealty.ru': 'russianrealty.ru',
 'kuybyshev.russianrealty.ru': 'russianrealty.ru',
 'stavropolsky-kray.russianrealty.ru': 'russianrealty.ru',
 'surgut.russianrealty.ru': 'russianrealty.ru',
 'schelkovo.russianrealty.ru': 'russianrealty.ru',
 'ivanovo.russianrealty.ru': 'russianrealty.ru',
 'ivanteevka.russianrealty.ru': 'russianrealty.ru',
 'omsk.russianrealty.ru': 'russianrealty.ru',
 'vologda.russianrealty.ru': 'russianrealty.ru',
 'rostov-na-donu.russianrealty.ru': 'russianrealty.ru',
 'gelendzhik.russianrealty.ru': 'russianrealty.ru',
 'balakovo.russianrealty.ru': 'russianrealty.ru',
 'troitsk.russianrealty.ru': 'russianrealty.ru',
 'krym.russianrealty.ru': 'russianrealty.ru',
 'novokuznetsk.russianrealty.ru': 'russianrealty.ru',
 'belgorod.russianrealty.ru': 'russianrealty.ru',
 'beloyarsky.russianrealty.ru': 'russianrealty.ru',
 'vladimir.russianrealty.ru': 'russianrealty.ru',
 'orenburgskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'kursk.russianrealty.ru': 'russianrealty.ru',
 'elektrostal.russianrealty.ru': 'russianrealty.ru',
 'ulyanovsk.russianrealty.ru': 'russianrealty.ru',
 'yakutsk.russianrealty.ru': 'russianrealty.ru',
 'pavlovsky-posad.russianrealty.ru': 'russianrealty.ru',
 'oktyabrsk.russianrealty.ru': 'russianrealty.ru',
 'kotelniki.russianrealty.ru': 'russianrealty.ru',
 'bolshoy-kamen.russianrealty.ru': 'russianrealty.ru',
 'zarechny.russianrealty.ru': 'russianrealty.ru',
 'syzran.russianrealty.ru': 'russianrealty.ru',
 'tomskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'bryansk.russianrealty.ru': 'russianrealty.ru',
 'voskresensk.russianrealty.ru': 'russianrealty.ru',
 'bryanskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'pskovskaya-oblast.russianrealty.ru': 'russianrealty.ru',
 'kolpino.russianrealty.ru': 'russianrealty.ru',
 'kronshtadt.russianrealty.ru': 'russianrealty.ru',
 'tver.russianrealty.ru': 'russianrealty.ru',
 'guryevsk.russianrealty.ru': 'russianrealty.ru',
 'altai.russianrealty.ru': 'russianrealty.ru',
 'tula.russianrealty.ru': 'russianrealty.ru',
 'kaluga.russianrealty.ru': 'russianrealty.ru',
    'naro-fominsk.gorod.guru': 'gorod.guru',
 'gorod.guru': 'gorod.guru',
 'naberezhnyye-chelny.gorod.guru': 'gorod.guru',
 'reutov.gorod.guru': 'gorod.guru',
 'krasnodar.gorod.guru': 'gorod.guru',
 'novosibirsk.gorod.guru': 'gorod.guru',
 'zavodoukovsk.gorod.guru': 'gorod.guru',
 'orenburg.gorod.guru': 'gorod.guru',
 'norilsk.gorod.guru': 'gorod.guru',
 'lomonosov.gorod.guru': 'gorod.guru',
 'syzran.gorod.guru': 'gorod.guru',
 'samara.gorod.guru': 'gorod.guru',
 'yelabuga.gorod.guru': 'gorod.guru',
 'staryy-oskol.gorod.guru': 'gorod.guru',
 'sochi.gorod.guru': 'gorod.guru',
 'izhevsk.gorod.guru': 'gorod.guru',
 'saransk.gorod.guru': 'gorod.guru',
 'krasnoyarsk.gorod.guru': 'gorod.guru',
 'yeysk.gorod.guru': 'gorod.guru',
 'satka.gorod.guru': 'gorod.guru',
 'msk.gorod.guru': 'gorod.guru',
 'omsk.gorod.guru': 'gorod.guru',
 'ivanteyevka.gorod.guru': 'gorod.guru',
 'chehov.gorod.guru': 'gorod.guru',
 'petropavlovsk-kamchatskiy.gorod.guru': 'gorod.guru',
 'novocherkassk.gorod.guru': 'gorod.guru',
 'perm.gorod.guru': 'gorod.guru',
 'kazan.gorod.guru': 'gorod.guru',
 'ufa.gorod.guru': 'gorod.guru',
 'engels.gorod.guru': 'gorod.guru',
 'barnaul.gorod.guru': 'gorod.guru',
 'kursk.gorod.guru': 'gorod.guru',
 'mytishchi.gorod.guru': 'gorod.guru',
 'velikiy-novgorod.gorod.guru': 'gorod.guru',
 'tomsk.gorod.guru': 'gorod.guru',
 'tolyatti.gorod.guru': 'gorod.guru',
 'kirov.gorod.guru': 'gorod.guru',
 'cheboksary.gorod.guru': 'gorod.guru',
 'obninsk.gorod.guru': 'gorod.guru',
 'neftekamsk.gorod.guru': 'gorod.guru',
 'borisoglebsk.gorod.guru': 'gorod.guru',
 'shahty.gorod.guru': 'gorod.guru',
 'svetlogorsk.gorod.guru': 'gorod.guru',
 'abakan.gorod.guru': 'gorod.guru',
 'tyumen.gorod.guru': 'gorod.guru',
 'blagoveshchensk.gorod.guru': 'gorod.guru',
 'novorossiysk.gorod.guru': 'gorod.guru',
 'petrozavodsk.gorod.guru': 'gorod.guru',
 'novokuznetsk.gorod.guru': 'gorod.guru',
 'saratov.gorod.guru': 'gorod.guru',
 'gelendzhik.gorod.guru': 'gorod.guru',
 'maykop.gorod.guru': 'gorod.guru',
 'ulyanovsk.gorod.guru': 'gorod.guru',
 'ramenskoye.gorod.guru': 'gorod.guru',
 'nahodka.gorod.guru': 'gorod.guru',
 'nizhniy-novgorod.gorod.guru': 'gorod.guru',
 'yekaterinburg.gorod.guru': 'gorod.guru',
 'ivanovo.gorod.guru': 'gorod.guru',
 'tobolsk.gorod.guru': 'gorod.guru',
 'almetyevsk.gorod.guru': 'gorod.guru',
 'vologda.gorod.guru': 'gorod.guru',
 'smolensk.gorod.guru': 'gorod.guru',
 'rostov-na-donu.gorod.guru': 'gorod.guru',
 'spb.gorod.guru': 'gorod.guru',
 'lipetsk.gorod.guru': 'gorod.guru',
 'lesnoy.gorod.guru': 'gorod.guru',
 'tambov.gorod.guru': 'gorod.guru',
 'ussuriysk.gorod.guru': 'gorod.guru',
 'leninogorsk.gorod.guru': 'gorod.guru',
 'podolsk.gorod.guru': 'gorod.guru',
 'ryazan.gorod.guru': 'gorod.guru',
 'mineralnyye-vody.gorod.guru': 'gorod.guru',
 'yuzhno-sahalinsk.gorod.guru': 'gorod.guru',
 'penza.gorod.guru': 'gorod.guru',
 'kolomna.gorod.guru': 'gorod.guru',
 'severodvinsk.gorod.guru': 'gorod.guru',
 'zhukovskiy.gorod.guru': 'gorod.guru',
 'murmansk.gorod.guru': 'gorod.guru',
 'komsomolsk-na-amure.gorod.guru': 'gorod.guru',
 'pyatigorsk.gorod.guru': 'gorod.guru',
 'lobnya.gorod.guru': 'gorod.guru',
 'golitsyno.gorod.guru': 'gorod.guru',
 'yoshkar-ola.gorod.guru': 'gorod.guru',
 'magadan.gorod.guru': 'gorod.guru',
 'chita.gorod.guru': 'gorod.guru',
 'pushkino.gorod.guru': 'gorod.guru',
 'pskov.gorod.guru': 'gorod.guru',
 'yalutorovsk.gorod.guru': 'gorod.guru',
 'magnitogorsk.gorod.guru': 'gorod.guru',
 'tver.gorod.guru': 'gorod.guru',
 'tula.gorod.guru': 'gorod.guru',
 'yaroslavl.gorod.guru': 'gorod.guru',
 'noginsk.gorod.guru': 'gorod.guru',
 'abinsk.gorod.guru': 'gorod.guru',
 'kaliningrad.gorod.guru': 'gorod.guru',
 'rybinsk.gorod.guru': 'gorod.guru',
 'balashiha.gorod.guru': 'gorod.guru',
 'volgodonsk.gorod.guru': 'gorod.guru',
 'ruza.gorod.guru': 'gorod.guru',
 'berezniki.gorod.guru': 'gorod.guru',
 'syktyvkar.gorod.guru': 'gorod.guru',
 'slavyansk-na-kubani.gorod.guru': 'gorod.guru',
 'vladivostok.gorod.guru': 'gorod.guru',
 'cherepovets.gorod.guru': 'gorod.guru',
 'budennovsk.gorod.guru': 'gorod.guru',
 'taganrog.gorod.guru': 'gorod.guru',
 'yessentuki.gorod.guru': 'gorod.guru',
 'belgorod.gorod.guru': 'gorod.guru',
 'stavropol.gorod.guru': 'gorod.guru',
 'nizhniy-tagil.gorod.guru': 'gorod.guru',
 'achinsk.gorod.guru': 'gorod.guru',
 'serpuhov.gorod.guru': 'gorod.guru',
 'temryuk.gorod.guru': 'gorod.guru',
 'rzhev.gorod.guru': 'gorod.guru',
 'murom.gorod.guru': 'gorod.guru',
 'kostroma.gorod.guru': 'gorod.guru',
 'dmitrov.gorod.guru': 'gorod.guru',
 'bryansk.gorod.guru': 'gorod.guru',
 'domodedovo.gorod.guru': 'gorod.guru',
 'safonovo.gorod.guru': 'gorod.guru',
 'berdsk.gorod.guru': 'gorod.guru',
 'yakutsk.gorod.guru': 'gorod.guru',
 'volzhskiy.gorod.guru': 'gorod.guru',
    'yoshkar-ola.big-book-avto.ru': 'big-book-avto.ru',
 'krasnodar.big-book-avto.ru': 'big-book-avto.ru',
 'orenburg.big-book-avto.ru': 'big-book-avto.ru',
 'moscow.big-book-avto.ru': 'big-book-avto.ru',
 'voronezh.big-book-avto.ru': 'big-book-avto.ru',
 'omsk.big-book-avto.ru': 'big-book-avto.ru',
 'tula.big-book-avto.ru': 'big-book-avto.ru',
 'viborg.big-book-avto.ru': 'big-book-avto.ru',
 'saint-petersburg.big-book-avto.ru': 'big-book-avto.ru',
 'cheboksary.big-book-avto.ru': 'big-book-avto.ru',
 'ufa.big-book-avto.ru': 'big-book-avto.ru',
 'ljubercy.big-book-avto.ru': 'big-book-avto.ru',
 'balakovo.big-book-avto.ru': 'big-book-avto.ru',
 'ryazan.big-book-avto.ru': 'big-book-avto.ru',
 'samara.big-book-avto.ru': 'big-book-avto.ru',
 'tolyatti.big-book-avto.ru': 'big-book-avto.ru',
 'sochi.big-book-avto.ru': 'big-book-avto.ru',
 'chelyabinsk.big-book-avto.ru': 'big-book-avto.ru',
 'shakhty.big-book-avto.ru': 'big-book-avto.ru',
 'tver.big-book-avto.ru': 'big-book-avto.ru',
 'izhevsk.big-book-avto.ru': 'big-book-avto.ru',
 'yhta.big-book-avto.ru': 'big-book-avto.ru',
 'velikiy-novgorod.big-book-avto.ru': 'big-book-avto.ru',
 'orsk.big-book-avto.ru': 'big-book-avto.ru',
 'nizhnekamsk.big-book-avto.ru': 'big-book-avto.ru',
 'novomoscowsk.big-book-avto.ru': 'big-book-avto.ru',
 'yekaterinburg.big-book-avto.ru': 'big-book-avto.ru',
 'tyumen.big-book-avto.ru': 'big-book-avto.ru',
 'volgograd.big-book-avto.ru': 'big-book-avto.ru',
 'kazan.big-book-avto.ru': 'big-book-avto.ru',
 'saratov.big-book-avto.ru': 'big-book-avto.ru',
 'sergiev-posad.big-book-avto.ru': 'big-book-avto.ru',
 'orehovo-zuevo.big-book-avto.ru': 'big-book-avto.ru',
 'astrakhan.big-book-avto.ru': 'big-book-avto.ru',
 'dzerzhinsk.big-book-avto.ru': 'big-book-avto.ru',
 'surgut.big-book-avto.ru': 'big-book-avto.ru',
 'kurgan.big-book-avto.ru': 'big-book-avto.ru',
 'arkhangelsk.big-book-avto.ru': 'big-book-avto.ru',
 'smolensk.big-book-avto.ru': 'big-book-avto.ru',
 'krasnoyarsk.big-book-avto.ru': 'big-book-avto.ru',
 'sterlitamak.big-book-avto.ru': 'big-book-avto.ru',
 'vladikavkaz.big-book-avto.ru': 'big-book-avto.ru',
 'murmansk.big-book-avto.ru': 'big-book-avto.ru',
 'cherepovets.big-book-avto.ru': 'big-book-avto.ru',
 'naberezhnyye-chelny.big-book-avto.ru': 'big-book-avto.ru',
 'magnitogorsk.big-book-avto.ru': 'big-book-avto.ru',
 'noginsk.big-book-avto.ru': 'big-book-avto.ru',
 'khabarovsk.big-book-avto.ru': 'big-book-avto.ru',
 'kostroma.big-book-avto.ru': 'big-book-avto.ru',
 'tomsk.big-book-avto.ru': 'big-book-avto.ru',
 'nevinnomissk.big-book-avto.ru': 'big-book-avto.ru',
 'komsomolsk-na-amure.big-book-avto.ru': 'big-book-avto.ru',
 'miass.big-book-avto.ru': 'big-book-avto.ru',
 'kolomna.big-book-avto.ru': 'big-book-avto.ru',
 'saransk.big-book-avto.ru': 'big-book-avto.ru',
 'rostov-na-donu.big-book-avto.ru': 'big-book-avto.ru',
 'novorossiysk.big-book-avto.ru': 'big-book-avto.ru',
 'abakan.big-book-avto.ru': 'big-book-avto.ru',
 'stavropol.big-book-avto.ru': 'big-book-avto.ru',
 'podolsk.big-book-avto.ru': 'big-book-avto.ru',
 'balashikha.big-book-avto.ru': 'big-book-avto.ru',
 'odincovo.big-book-avto.ru': 'big-book-avto.ru',
 'pyatigorsk.big-book-avto.ru': 'big-book-avto.ru',
 'almetyevsk.big-book-avto.ru': 'big-book-avto.ru',
 'nalchik.big-book-avto.ru': 'big-book-avto.ru',
 'barnaul.big-book-avto.ru': 'big-book-avto.ru',
 'novosibirsk.big-book-avto.ru': 'big-book-avto.ru',
 'kovrov.big-book-avto.ru': 'big-book-avto.ru',
 'taganrog.big-book-avto.ru': 'big-book-avto.ru',
 'penza.big-book-avto.ru': 'big-book-avto.ru',
 'nefteyugansk.big-book-avto.ru': 'big-book-avto.ru',
 'engels.big-book-avto.ru': 'big-book-avto.ru',
 'sizran.big-book-avto.ru': 'big-book-avto.ru',
 'pskov.big-book-avto.ru': 'big-book-avto.ru',
 'hanty-mansijsk.big-book-avto.ru': 'big-book-avto.ru',
 'kyzyl.big-book-avto.ru': 'big-book-avto.ru',
 'petrozavodsk.big-book-avto.ru': 'big-book-avto.ru',
 'kirov.big-book-avto.ru': 'big-book-avto.ru',
 'maykop.big-book-avto.ru': 'big-book-avto.ru',
 'kamishin.big-book-avto.ru': 'big-book-avto.ru',
 'koroljov.big-book-avto.ru': 'big-book-avto.ru',
 'kaliningrad.big-book-avto.ru': 'big-book-avto.ru',
 'nizhniy-novgorod.big-book-avto.ru': 'big-book-avto.ru',
 'perm.big-book-avto.ru': 'big-book-avto.ru',
 'severodvinsk.big-book-avto.ru': 'big-book-avto.ru',
 'big-book-avto.ru': 'big-book-avto.ru',
 'rybinsk.big-book-avto.ru': 'big-book-avto.ru',
 'serpuhov.big-book-avto.ru': 'big-book-avto.ru',
 'tambov.big-book-avto.ru': 'big-book-avto.ru',
 'yaroslavl.big-book-avto.ru': 'big-book-avto.ru',
 'neftekamsk.big-book-avto.ru': 'big-book-avto.ru',
 'zheleznodorozhnyj.big-book-avto.ru': 'big-book-avto.ru',
 'lipetsk.big-book-avto.ru': 'big-book-avto.ru',
 'belgorod.big-book-avto.ru': 'big-book-avto.ru',
 'artem.big-book-avto.ru': 'big-book-avto.ru',
 'novocherkask.big-book-avto.ru': 'big-book-avto.ru',
 'novokuznetsk.big-book-avto.ru': 'big-book-avto.ru',
 'kursk.big-book-avto.ru': 'big-book-avto.ru',
 'wjolkovo.big-book-avto.ru': 'big-book-avto.ru',
 'ivanovo.big-book-avto.ru': 'big-book-avto.ru',
 'pervoyralsk.big-book-avto.ru': 'big-book-avto.ru',
 'zlatoust.big-book-avto.ru': 'big-book-avto.ru',
 'vologda.big-book-avto.ru': 'big-book-avto.ru',
 'rybtsovsk.big-book-avto.ru': 'big-book-avto.ru',
 'nizhniy-tagil.big-book-avto.ru': 'big-book-avto.ru',
 'serov.big-book-avto.ru': 'big-book-avto.ru',
 'jelektrostal.big-book-avto.ru': 'big-book-avto.ru',
 'oktyabrsky.big-book-avto.ru': 'big-book-avto.ru',
 'vladivostok.big-book-avto.ru': 'big-book-avto.ru',
 'yeisk.big-book-avto.ru': 'big-book-avto.ru',
 'bryansk.big-book-avto.ru': 'big-book-avto.ru',
 'kaluga.big-book-avto.ru': 'big-book-avto.ru',
 'arzamas.big-book-avto.ru': 'big-book-avto.ru',
 'kamensk-yralski.big-book-avto.ru': 'big-book-avto.ru',
 'oryol.big-book-avto.ru': 'big-book-avto.ru',
 'murom.big-book-avto.ru': 'big-book-avto.ru',
 'novocheboksarsk.big-book-avto.ru': 'big-book-avto.ru',
 'syktyvkar.big-book-avto.ru': 'big-book-avto.ru',
 'ulyanovsk.big-book-avto.ru': 'big-book-avto.ru',
 'mytiwi.big-book-avto.ru': 'big-book-avto.ru',
 'blagoveshchensk.big-book-avto.ru': 'big-book-avto.ru',
 'khimki.big-book-avto.ru': 'big-book-avto.ru',
 'ramenskoe.big-book-avto.ru': 'big-book-avto.ru',
 'kemerovo.big-book-avto.ru': 'big-book-avto.ru',
 'biysk.big-book-avto.ru': 'big-book-avto.ru',
 'staryy-oskol.big-book-avto.ru': 'big-book-avto.ru',
    'svetofors.ru': 'svetofors.ru',
 'habarovsk.svetofors.ru': 'svetofors.ru',
 'shchyolkovo.svetofors.ru': 'svetofors.ru',
 'chelyabinsk.svetofors.ru': 'svetofors.ru',
 'engels.svetofors.ru': 'svetofors.ru',
 'novosibirsk.svetofors.ru': 'svetofors.ru',
 'abakan.svetofors.ru': 'svetofors.ru',
 'sankt-peterburg.svetofors.ru': 'svetofors.ru',
 'novorossiysk.svetofors.ru': 'svetofors.ru',
 'igevsk.svetofors.ru': 'svetofors.ru',
 'barnaul.svetofors.ru': 'svetofors.ru',
 'kovrov.svetofors.ru': 'svetofors.ru',
 'salavat.svetofors.ru': 'svetofors.ru',
 'votkinsk.svetofors.ru': 'svetofors.ru',
 'samara.svetofors.ru': 'svetofors.ru',
 'kolomna.svetofors.ru': 'svetofors.ru',
 'vladimir.svetofors.ru': 'svetofors.ru',
 'kaluga.svetofors.ru': 'svetofors.ru',
 'ufa.svetofors.ru': 'svetofors.ru',
 'ekaterinburg.svetofors.ru': 'svetofors.ru',
 'kostroma.svetofors.ru': 'svetofors.ru',
 'hanti-mansiyskiy-ao-surgut.svetofors.ru': 'svetofors.ru',
 'kemerovo.svetofors.ru': 'svetofors.ru',
 'omsk.svetofors.ru': 'svetofors.ru',
 'kazan.svetofors.ru': 'svetofors.ru',
 'odintsovo.svetofors.ru': 'svetofors.ru',
 'tyumen.svetofors.ru': 'svetofors.ru',
 'yaroslavl.svetofors.ru': 'svetofors.ru',
 'tambov.svetofors.ru': 'svetofors.ru',
 'tolyatti.svetofors.ru': 'svetofors.ru',
 'berdsk.svetofors.ru': 'svetofors.ru',
 'kaliningrad.svetofors.ru': 'svetofors.ru',
 'nigniy-tagil.svetofors.ru': 'svetofors.ru',
 'krasnodar.svetofors.ru': 'svetofors.ru',
 'orenburg.svetofors.ru': 'svetofors.ru',
 'ulyanovsk.svetofors.ru': 'svetofors.ru',
 'anapa.svetofors.ru': 'svetofors.ru',
 'ryazan.svetofors.ru': 'svetofors.ru',
 'sterlitamak.svetofors.ru': 'svetofors.ru',
 'miass.svetofors.ru': 'svetofors.ru',
 'kopeysk.svetofors.ru': 'svetofors.ru',
 'nigniy-novgorod.svetofors.ru': 'svetofors.ru',
 'tver.svetofors.ru': 'svetofors.ru',
 'tula.svetofors.ru': 'svetofors.ru',
 'cheboksari.svetofors.ru': 'svetofors.ru',
 'stavropol.svetofors.ru': 'svetofors.ru',
 'rossosh.svetofors.ru': 'svetofors.ru',
 'pskov.svetofors.ru': 'svetofors.ru',
 'noginsk.svetofors.ru': 'svetofors.ru',
 'naberegnie-chelni.svetofors.ru': 'svetofors.ru',
 'omutninsk.svetofors.ru': 'svetofors.ru',
 'saratov.svetofors.ru': 'svetofors.ru',
 'sizran.svetofors.ru': 'svetofors.ru',
 'vladikavkaz.svetofors.ru': 'svetofors.ru',
 'komsomolsk-na-amure.svetofors.ru': 'svetofors.ru',
 'perm.svetofors.ru': 'svetofors.ru',
 'volgodonsk.svetofors.ru': 'svetofors.ru',
 'kursk.svetofors.ru': 'svetofors.ru',
 'tuchkovo.svetofors.ru': 'svetofors.ru',
 'yurga.svetofors.ru': 'svetofors.ru',
 'kirovskaya-oblast-kirov.svetofors.ru': 'svetofors.ru',
 'gigulevsk.svetofors.ru': 'svetofors.ru',
 'ussuriysk.svetofors.ru': 'svetofors.ru',
 'zelenogorsk.svetofors.ru': 'svetofors.ru',
 'tomsk.svetofors.ru': 'svetofors.ru',
 'achinsk.svetofors.ru': 'svetofors.ru',
 'novotroitsk.svetofors.ru': 'svetofors.ru',
 'vologda.svetofors.ru': 'svetofors.ru',
 'primorsko-ahtarsk.svetofors.ru': 'svetofors.ru',
 'himki.svetofors.ru': 'svetofors.ru',
 'murmansk.svetofors.ru': 'svetofors.ru',
 'kashira.svetofors.ru': 'svetofors.ru',
 'siktivkar.svetofors.ru': 'svetofors.ru',
 'novokuznetsk.svetofors.ru': 'svetofors.ru',
 'voroneg.svetofors.ru': 'svetofors.ru',
 'orehovo-zuevo.svetofors.ru': 'svetofors.ru',
 'rostovskaya-oblast-aksay.svetofors.ru': 'svetofors.ru',
 'kurgan.svetofors.ru': 'svetofors.ru',
 'ivanovo.svetofors.ru': 'svetofors.ru',
 'slavyansk-na-kubani.svetofors.ru': 'svetofors.ru',
 'shchekino.svetofors.ru': 'svetofors.ru',
 'moskovskaya-oblast-chehov.svetofors.ru': 'svetofors.ru',
 'shahti.svetofors.ru': 'svetofors.ru',
 'moskva.svetofors.ru': 'svetofors.ru',
 'krasnoyarsk.svetofors.ru': 'svetofors.ru',
 'magnitogorsk.svetofors.ru': 'svetofors.ru',
 'bugulma.svetofors.ru': 'svetofors.ru',
 'temryuk.svetofors.ru': 'svetofors.ru',
 'novoaltaysk.svetofors.ru': 'svetofors.ru',
 'kirishi.svetofors.ru': 'svetofors.ru',
 'elektrostal.svetofors.ru': 'svetofors.ru',
 'shchigri.svetofors.ru': 'svetofors.ru',
 'korolev.svetofors.ru': 'svetofors.ru',
 'glazov.svetofors.ru': 'svetofors.ru',
 'taganrog.svetofors.ru': 'svetofors.ru',
 'tobolsk.svetofors.ru': 'svetofors.ru',
 'lyubertsi.svetofors.ru': 'svetofors.ru',
 'artem.svetofors.ru': 'svetofors.ru',
 'yugno-sahalinsk.svetofors.ru': 'svetofors.ru',
 'velikiy-novgorod.svetofors.ru': 'svetofors.ru',
 'sharya.svetofors.ru': 'svetofors.ru',
 'livni.svetofors.ru': 'svetofors.ru',
 'mahachkala.svetofors.ru': 'svetofors.ru',
 'noviy-urengoy.svetofors.ru': 'svetofors.ru',
 'rostov-na-donu.svetofors.ru': 'svetofors.ru',
 'oryol.svetofors.ru': 'svetofors.ru',
 'buzuluk.svetofors.ru': 'svetofors.ru',
 'sverdlovskaya-oblast-zarechniy.svetofors.ru': 'svetofors.ru',
 'kommunar.svetofors.ru': 'svetofors.ru',
 'yoshkar-ola.svetofors.ru': 'svetofors.ru',
 'zvenigorod.svetofors.ru': 'svetofors.ru',
 'dgubga.svetofors.ru': 'svetofors.ru',
 'uhta.svetofors.ru': 'svetofors.ru',
 'shumiha.svetofors.ru': 'svetofors.ru',
 'nigegorodskaya-oblast-pavlovo.svetofors.ru': 'svetofors.ru',
 'gribki.svetofors.ru': 'svetofors.ru',
 'bryansk.svetofors.ru': 'svetofors.ru',
 'sarapul.svetofors.ru': 'svetofors.ru',
 'leninogorsk.svetofors.ru': 'svetofors.ru',
 'pervouralsk.svetofors.ru': 'svetofors.ru',
 'penza.svetofors.ru': 'svetofors.ru',
 'petrozavodsk.svetofors.ru': 'svetofors.ru',
 'dmitrov.svetofors.ru': 'svetofors.ru',
 'sochi.svetofors.ru': 'svetofors.ru',
 'volgograd.svetofors.ru': 'svetofors.ru',
 'neftekamsk.svetofors.ru': 'svetofors.ru',
 'chistopol.svetofors.ru': 'svetofors.ru',
 'smolensk.svetofors.ru': 'svetofors.ru',
    'yekaterinburg.hipdir.com': 'hipdir.com',
 'hipdir.com': 'hipdir.com',
 'tyumen.hipdir.com': 'hipdir.com',
 'izhevsk.hipdir.com': 'hipdir.com',
 'zhigulyevsk.hipdir.com': 'hipdir.com',
 'perm.hipdir.com': 'hipdir.com',
 'almetyevsk.hipdir.com': 'hipdir.com',
 'spb.hipdir.com': 'hipdir.com',
 'tolyatti.hipdir.com': 'hipdir.com',
 'barnaul.hipdir.com': 'hipdir.com',
 'engels.hipdir.com': 'hipdir.com',
 'yaroslavl.hipdir.com': 'hipdir.com',
 'nizhny-tagil.hipdir.com': 'hipdir.com',
 'ufa.hipdir.com': 'hipdir.com',
 'saratov.hipdir.com': 'hipdir.com',
 'kaluga.hipdir.com': 'hipdir.com',
 'pervouralsk.hipdir.com': 'hipdir.com',
 'sochi.hipdir.com': 'hipdir.com',
 'magnitogorsk.hipdir.com': 'hipdir.com',
 'chelyabinsk.hipdir.com': 'hipdir.com',
 'rostov-on-don.hipdir.com': 'hipdir.com',
 'arkhangelsk.hipdir.com': 'hipdir.com',
 'penza.hipdir.com': 'hipdir.com',
 'oryol.hipdir.com': 'hipdir.com',
 'kurgan.hipdir.com': 'hipdir.com',
 'petropavlovsk-kamchatsky.hipdir.com': 'hipdir.com',
 'tver.hipdir.com': 'hipdir.com',
 'novosibirsk.hipdir.com': 'hipdir.com',
 'samara.hipdir.com': 'hipdir.com',
 'krasnoyarsk.hipdir.com': 'hipdir.com',
 'belgorod.hipdir.com': 'hipdir.com',
 'khabarovsk.hipdir.com': 'hipdir.com',
 'krasnodar.hipdir.com': 'hipdir.com',
 'nizhny-novgorod.hipdir.com': 'hipdir.com',
 'vidnoye.hipdir.com': 'hipdir.com',
 'shchyolkovo.hipdir.com': 'hipdir.com',
 'sterlitamak.hipdir.com': 'hipdir.com',
 'stavropol.hipdir.com': 'hipdir.com',
 'volzhskiy.hipdir.com': 'hipdir.com',
 'korolyov.hipdir.com': 'hipdir.com',
 'anapa.hipdir.com': 'hipdir.com',
 'novokuznetsk.hipdir.com': 'hipdir.com',
 'mineralnye-vody.hipdir.com': 'hipdir.com',
 'tobolsk.hipdir.com': 'hipdir.com',
 'pushkin.hipdir.com': 'hipdir.com',
 'kirov.hipdir.com': 'hipdir.com',
 'armavir.hipdir.com': 'hipdir.com',
 'kazan.hipdir.com': 'hipdir.com',
 'kemerovo.hipdir.com': 'hipdir.com',
 'dolgoprudny.hipdir.com': 'hipdir.com',
 'cheboksary.hipdir.com': 'hipdir.com',
 'syktyvkar.hipdir.com': 'hipdir.com',
 'ulyanovsk.hipdir.com': 'hipdir.com',
 'saransk.hipdir.com': 'hipdir.com',
 'ivanovo.hipdir.com': 'hipdir.com',
 'tomsk.hipdir.com': 'hipdir.com',
 'gatchina.hipdir.com': 'hipdir.com',
 'omsk.hipdir.com': 'hipdir.com',
 'norilsk.hipdir.com': 'hipdir.com',
 'komsomolsk-on-amur.hipdir.com': 'hipdir.com',
 'orenburg.hipdir.com': 'hipdir.com',
 'balashikha.hipdir.com': 'hipdir.com',
 'kaliningrad.hipdir.com': 'hipdir.com',
 'vladivostok.hipdir.com': 'hipdir.com',
 'pskov.hipdir.com': 'hipdir.com',
 'tula.hipdir.com': 'hipdir.com',
 'zelenograd.hipdir.com': 'hipdir.com',
 'podolsk.hipdir.com': 'hipdir.com',
 'domodedovo.hipdir.com': 'hipdir.com',
 'khimki.hipdir.com': 'hipdir.com',
 'pyatigorsk.hipdir.com': 'hipdir.com',
 'tuapse.hipdir.com': 'hipdir.com',
 'vladimir.hipdir.com': 'hipdir.com',
 'miass.hipdir.com': 'hipdir.com',
 'surgut.hipdir.com': 'hipdir.com',
 'naberezhnye-chelny.hipdir.com': 'hipdir.com',
 'berdsk.hipdir.com': 'hipdir.com',
 'salavat.hipdir.com': 'hipdir.com',
 'yuzhno-sakhalinsk.hipdir.com': 'hipdir.com',
 'voronezh.hipdir.com': 'hipdir.com',
 'kursk.hipdir.com': 'hipdir.com',
 'mytishchi.hipdir.com': 'hipdir.com',
 'novorossiysk.hipdir.com': 'hipdir.com',
 'nahodka.hipdir.com': 'hipdir.com',
 'ussurijsk.hipdir.com': 'hipdir.com',
 'krasnogorsk.hipdir.com': 'hipdir.com',
 'yakutsk.hipdir.com': 'hipdir.com',
 'ryazan.hipdir.com': 'hipdir.com',
 'nizhnekamsk.hipdir.com': 'hipdir.com',
 'revda.hipdir.com': 'hipdir.com',
 'abakan.hipdir.com': 'hipdir.com',
 'ulan-ude.hipdir.com': 'hipdir.com',
 'murmansk.hipdir.com': 'hipdir.com',
 'bryansk.hipdir.com': 'hipdir.com',
 'volgograd.hipdir.com': 'hipdir.com',
 'smolensk.hipdir.com': 'hipdir.com',
 'yoshkar-ola.hipdir.com': 'hipdir.com',
 'nizhnevartovsk.hipdir.com': 'hipdir.com',
 'gelendzhik.hipdir.com': 'hipdir.com',
 'biysk.hipdir.com': 'hipdir.com',
 'vologda.hipdir.com': 'hipdir.com',
 'novomoskovsk.hipdir.com': 'hipdir.com',
 'veliky-novgorod.hipdir.com': 'hipdir.com',
 'irkutsk.hipdir.com': 'hipdir.com',
 'petrozavodsk.hipdir.com': 'hipdir.com',
 'astrakhan.hipdir.com': 'hipdir.com',
 'severodvinsk.hipdir.com': 'hipdir.com',
 'lipetsk.hipdir.com': 'hipdir.com',
 'vsevolozhsk.hipdir.com': 'hipdir.com',
 'dzerzhinsk.hipdir.com': 'hipdir.com',
 'yessentuki.hipdir.com': 'hipdir.com',
 'stary-oskol.hipdir.com': 'hipdir.com',
 'lyubertsy.hipdir.com': 'hipdir.com',
 'gorno-altaysk.hipdir.com': 'hipdir.com',
 'pushkino.hipdir.com': 'hipdir.com',
 'odintsovo.hipdir.com': 'hipdir.com',
 'kostroma.hipdir.com': 'hipdir.com',
 'tambov.hipdir.com': 'hipdir.com',
 'reutov.hipdir.com': 'hipdir.com',
 'blagoveshchensk.hipdir.com': 'hipdir.com',
 'zlatoust.hipdir.com': 'hipdir.com',
 'lobnya.hipdir.com': 'hipdir.com',
 'kamensk-uralsky.hipdir.com': 'hipdir.com',
 'kstovo.hipdir.com': 'hipdir.com',
 'suzdal.hipdir.com': 'hipdir.com',
 'peterhof.hipdir.com': 'hipdir.com',
 'chita.hipdir.com': 'hipdir.com',
 'ramenskoye.hipdir.com': 'hipdir.com',
    'respublika-bashkortostan.orgsinfo.ru': 'orgsinfo.ru',
 'petergof.orgsinfo.ru': 'orgsinfo.ru',
 'kaliningrad.orgsinfo.ru': 'orgsinfo.ru',
 'voronezhskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'rostov-na-donu.orgsinfo.ru': 'orgsinfo.ru',
 'ulyanovsk.orgsinfo.ru': 'orgsinfo.ru',
 'sochi.orgsinfo.ru': 'orgsinfo.ru',
 'prokopevsk.orgsinfo.ru': 'orgsinfo.ru',
 'nizhniy-novgorod.orgsinfo.ru': 'orgsinfo.ru',
 'rostovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'orenburg.orgsinfo.ru': 'orgsinfo.ru',
 'stavropol.orgsinfo.ru': 'orgsinfo.ru',
 'kaluzhskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'sankt-peterburg.orgsinfo.ru': 'orgsinfo.ru',
 'orenburgskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'tolyatti.orgsinfo.ru': 'orgsinfo.ru',
 'novosibirsk.orgsinfo.ru': 'orgsinfo.ru',
 'orgsinfo.ru': 'orgsinfo.ru',
 'voronezh.orgsinfo.ru': 'orgsinfo.ru',
 'krasnodarskiy-kray.orgsinfo.ru': 'orgsinfo.ru',
 'moscow.orgsinfo.ru': 'orgsinfo.ru',
 'nalchik.orgsinfo.ru': 'orgsinfo.ru',
 'krasnodar.orgsinfo.ru': 'orgsinfo.ru',
 'smolensk.orgsinfo.ru': 'orgsinfo.ru',
 'kazan.orgsinfo.ru': 'orgsinfo.ru',
 'moskovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'krasnogorsk.orgsinfo.ru': 'orgsinfo.ru',
 'vologda.orgsinfo.ru': 'orgsinfo.ru',
 'tyumen.orgsinfo.ru': 'orgsinfo.ru',
 'ufa.orgsinfo.ru': 'orgsinfo.ru',
 'ekaterinburg.orgsinfo.ru': 'orgsinfo.ru',
 'nahodka.orgsinfo.ru': 'orgsinfo.ru',
 'novokuznetsk.orgsinfo.ru': 'orgsinfo.ru',
 'saratov.orgsinfo.ru': 'orgsinfo.ru',
 'penza.orgsinfo.ru': 'orgsinfo.ru',
 'tobolsk.orgsinfo.ru': 'orgsinfo.ru',
 'nizhegorodskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'sverdlovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'kursk.orgsinfo.ru': 'orgsinfo.ru',
 'hmao.orgsinfo.ru': 'orgsinfo.ru',
 'yaroslavl.orgsinfo.ru': 'orgsinfo.ru',
 'almetevsk.orgsinfo.ru': 'orgsinfo.ru',
 'orsk.orgsinfo.ru': 'orgsinfo.ru',
 'primorskiy-kray.orgsinfo.ru': 'orgsinfo.ru',
 'samarskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'rybinsk.orgsinfo.ru': 'orgsinfo.ru',
 'cheboksary.orgsinfo.ru': 'orgsinfo.ru',
 'barnaul.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-tatarstan.orgsinfo.ru': 'orgsinfo.ru',
 'korolev.orgsinfo.ru': 'orgsinfo.ru',
 'petrozavodsk.orgsinfo.ru': 'orgsinfo.ru',
 'sankt-peterburg-region.orgsinfo.ru': 'orgsinfo.ru',
 'mahachkala.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-dagestan.orgsinfo.ru': 'orgsinfo.ru',
 'karachaevo-cherkesskaya-respublika.orgsinfo.ru': 'orgsinfo.ru',
 'mytishi.orgsinfo.ru': 'orgsinfo.ru',
 'saratovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'lyubertsy.orgsinfo.ru': 'orgsinfo.ru',
 'samara.orgsinfo.ru': 'orgsinfo.ru',
 'saransk.orgsinfo.ru': 'orgsinfo.ru',
 'astrahan.orgsinfo.ru': 'orgsinfo.ru',
 'belgorodskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'habarovsk.orgsinfo.ru': 'orgsinfo.ru',
 'podolsk.orgsinfo.ru': 'orgsinfo.ru',
 'shahty.orgsinfo.ru': 'orgsinfo.ru',
 'staryy-oskol.orgsinfo.ru': 'orgsinfo.ru',
 'elektrostal.orgsinfo.ru': 'orgsinfo.ru',
 'tula.orgsinfo.ru': 'orgsinfo.ru',
 'novosibirskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'engels.orgsinfo.ru': 'orgsinfo.ru',
 'leningradskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-tyva.orgsinfo.ru': 'orgsinfo.ru',
 'vladimirskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'kemerovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'tambov.orgsinfo.ru': 'orgsinfo.ru',
 'habarovskiy-kray.orgsinfo.ru': 'orgsinfo.ru',
 'abakan.orgsinfo.ru': 'orgsinfo.ru',
 'kaliningradskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'irkutsk.orgsinfo.ru': 'orgsinfo.ru',
 'perm.orgsinfo.ru': 'orgsinfo.ru',
 'tver.orgsinfo.ru': 'orgsinfo.ru',
 'severodvinsk.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-hakasiya.orgsinfo.ru': 'orgsinfo.ru',
 'anapa.orgsinfo.ru': 'orgsinfo.ru',
 'krasnoyarsk.orgsinfo.ru': 'orgsinfo.ru',
 'moscow-region.orgsinfo.ru': 'orgsinfo.ru',
 'chelyabinsk.orgsinfo.ru': 'orgsinfo.ru',
 'murmansk.orgsinfo.ru': 'orgsinfo.ru',
 'kirov.orgsinfo.ru': 'orgsinfo.ru',
 'kaluga.orgsinfo.ru': 'orgsinfo.ru',
 'armavir.orgsinfo.ru': 'orgsinfo.ru',
 'krasnoyarskiy-kray.orgsinfo.ru': 'orgsinfo.ru',
 'ryazan.orgsinfo.ru': 'orgsinfo.ru',
 'miass.orgsinfo.ru': 'orgsinfo.ru',
 'stavropolskiy-kray.orgsinfo.ru': 'orgsinfo.ru',
 'omsk.orgsinfo.ru': 'orgsinfo.ru',
 'omskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'udmurtskaya-respublika.orgsinfo.ru': 'orgsinfo.ru',
 'novorossiysk.orgsinfo.ru': 'orgsinfo.ru',
 'orel.orgsinfo.ru': 'orgsinfo.ru',
 'volgograd.orgsinfo.ru': 'orgsinfo.ru',
 'tyumenskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'pyatigorsk.orgsinfo.ru': 'orgsinfo.ru',
 'vladivostok.orgsinfo.ru': 'orgsinfo.ru',
 'penzenskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'cherepovets.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-adygeya.orgsinfo.ru': 'orgsinfo.ru',
 'blagoveshensk.orgsinfo.ru': 'orgsinfo.ru',
 'arhangelskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'ryazanskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'petropavlovsk-kamchatskiy.orgsinfo.ru': 'orgsinfo.ru',
 'yakutsk.orgsinfo.ru': 'orgsinfo.ru',
 'amurskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'ynao.orgsinfo.ru': 'orgsinfo.ru',
 'nizhniy-tagil.orgsinfo.ru': 'orgsinfo.ru',
 'lipetskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'pskovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'hanty-mansiysk.orgsinfo.ru': 'orgsinfo.ru',
 'ussuriysk.orgsinfo.ru': 'orgsinfo.ru',
 'astrahanskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-komi.orgsinfo.ru': 'orgsinfo.ru',
 'salavat.orgsinfo.ru': 'orgsinfo.ru',
 'tambovskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'respublika-kareliya.orgsinfo.ru': 'orgsinfo.ru',
 'tulskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'vladimir.orgsinfo.ru': 'orgsinfo.ru',
 'naberezhnye-chelny.orgsinfo.ru': 'orgsinfo.ru',
 'balashiha.orgsinfo.ru': 'orgsinfo.ru',
 'chelyabinskaya-oblast.orgsinfo.ru': 'orgsinfo.ru',
 'permskiy-kray.orgsinfo.ru': 'orgsinfo.ru',
    'api.cian.ru': 'cian.ru',
 'spb.cian.ru': 'cian.ru',
 'cian.ru': 'cian.ru',
 'lyubertsy.cian.ru': 'cian.ru',
 'reutov.cian.ru': 'cian.ru',
 'balashikha.cian.ru': 'cian.ru',
 'naberezhnye-chelny.cian.ru': 'cian.ru',
 'zvenigorod.cian.ru': 'cian.ru',
 'tyumen.cian.ru': 'cian.ru',
 'bryansk.cian.ru': 'cian.ru',
 'mytishchi.cian.ru': 'cian.ru',
 'vidnoye.cian.ru': 'cian.ru',
 'krym.cian.ru': 'cian.ru',
 'vladivostok.cian.ru': 'cian.ru',
 'ufa.cian.ru': 'cian.ru',
 'samara.cian.ru': 'cian.ru',
 'ryazan.cian.ru': 'cian.ru',
 'pskov.cian.ru': 'cian.ru',
 'smolensk.cian.ru': 'cian.ru',
 'novorossiysk.cian.ru': 'cian.ru',
 'ekb.cian.ru': 'cian.ru',
 'chelyabinsk.cian.ru': 'cian.ru',
 'noginsk.cian.ru': 'cian.ru',
 'omsk.cian.ru': 'cian.ru',
 'novosibirsk.cian.ru': 'cian.ru',
 'krasnodar.cian.ru': 'cian.ru',
 'odintsovo.cian.ru': 'cian.ru',
 'sochi.cian.ru': 'cian.ru',
 'saratov.cian.ru': 'cian.ru',
 'krasnoyarsk.cian.ru': 'cian.ru',
 'kazan.cian.ru': 'cian.ru',
 'volgograd.cian.ru': 'cian.ru',
 'podolsk.cian.ru': 'cian.ru',
 'habarovsk.cian.ru': 'cian.ru',
 'yoshkar-ola.cian.ru': 'cian.ru',
 'nn.cian.ru': 'cian.ru',
 'kostroma.cian.ru': 'cian.ru',
 'tomsk.cian.ru': 'cian.ru',
 'rostov.cian.ru': 'cian.ru',
 'orel.cian.ru': 'cian.ru',
 'belgorod.cian.ru': 'cian.ru',
 'anapa.cian.ru': 'cian.ru',
 'vladimir.cian.ru': 'cian.ru',
 'orenburg.cian.ru': 'cian.ru',
 'naro-fominsk.cian.ru': 'cian.ru',
 'domodedovo.cian.ru': 'cian.ru',
 'chekhov.cian.ru': 'cian.ru',
 'istra.cian.ru': 'cian.ru',
 'ramenskoye.cian.ru': 'cian.ru',
 'izhevsk.cian.ru': 'cian.ru',
 'tula.cian.ru': 'cian.ru',
 'perm.cian.ru': 'cian.ru',
 'yaroslavl.cian.ru': 'cian.ru',
 'elektrostal.cian.ru': 'cian.ru',
 'kaliningrad.cian.ru': 'cian.ru',
 'cheboksary.cian.ru': 'cian.ru',
 'arhangelsk.cian.ru': 'cian.ru',
 'dmitrov.cian.ru': 'cian.ru',
 'aprelevka.cian.ru': 'cian.ru',
 'petrozavodsk.cian.ru': 'cian.ru',
 'krasnogorsk.cian.ru': 'cian.ru',
 'khimki.cian.ru': 'cian.ru',
 'kirov.cian.ru': 'cian.ru',
 'kemerovo.cian.ru': 'cian.ru',
 'vladikavkaz.cian.ru': 'cian.ru',
 'ivanovo.cian.ru': 'cian.ru',
 'hmao.cian.ru': 'cian.ru',
 'abakan.cian.ru': 'cian.ru',
 'korolev.cian.ru': 'cian.ru',
 'zhukovsky.cian.ru': 'cian.ru',
 'yanao.cian.ru': 'cian.ru',
 'gelendzhik.cian.ru': 'cian.ru',
 'kaluga.cian.ru': 'cian.ru',
 'syktyvkar.cian.ru': 'cian.ru',
 'penza.cian.ru': 'cian.ru',
 'yegoryevsk.cian.ru': 'cian.ru',
 'stavropol.cian.ru': 'cian.ru',
 'voronezh.cian.ru': 'cian.ru',
 'yuzhno-sahalinsk.cian.ru': 'cian.ru',
 'kurgan.cian.ru': 'cian.ru',
 'staraya-kupavna.cian.ru': 'cian.ru',
 'surgut.cian.ru': 'cian.ru',
 'astrahan.cian.ru': 'cian.ru',
 'sevastopol.cian.ru': 'cian.ru',
 'maykop.cian.ru': 'cian.ru',
 'novgorod.cian.ru': 'cian.ru',
 'tolyatti.cian.ru': 'cian.ru',
 'irkutsk.cian.ru': 'cian.ru',
 'serpukhov.cian.ru': 'cian.ru',
 'barnaul.cian.ru': 'cian.ru',
 'murmansk.cian.ru': 'cian.ru',
 'klin.cian.ru': 'cian.ru',
 'pushkino.cian.ru': 'cian.ru',
 'yakutsk.cian.ru': 'cian.ru',
 'kolomna.cian.ru': 'cian.ru',
 'orekhovo-zuyevo.cian.ru': 'cian.ru',
 'sergiyev-posad.cian.ru': 'cian.ru',
 'saransk.cian.ru': 'cian.ru',
 'dubna.cian.ru': 'cian.ru',
 'fryazino.cian.ru': 'cian.ru',
 'ulyanovsk.cian.ru': 'cian.ru',
 'kursk.cian.ru': 'cian.ru',
 'ruza.cian.ru': 'cian.ru',
 'tver.cian.ru': 'cian.ru',
 'blagoveschensk.cian.ru': 'cian.ru',
 'lipetsk.cian.ru': 'cian.ru',
 'shchyolkovo.cian.ru': 'cian.ru',
 'tambov.cian.ru': 'cian.ru',
 'mahachkala.cian.ru': 'cian.ru',
 'gorno-altaysk.cian.ru': 'cian.ru',
 'kyzyl.cian.ru': 'cian.ru',
 'losino-petrovskiy.cian.ru': 'cian.ru',
 'chernogolovka.cian.ru': 'cian.ru',
 'dolgoprudny.cian.ru': 'cian.ru',
 'elektrogorsk.cian.ru': 'cian.ru',
 'zaraysk.cian.ru': 'cian.ru',
 'yakhroma.cian.ru': 'cian.ru',
 'voskresensk.cian.ru': 'cian.ru',
 'vologda.cian.ru': 'cian.ru',
 'ulan-ude.cian.ru': 'cian.ru',
 'nazran.cian.ru': 'cian.ru',
 'golitsyno.cian.ru': 'cian.ru',
 'nalchik.cian.ru': 'cian.ru',
 'cherepovec.cian.ru': 'cian.ru',
 'dzerzhinsky.cian.ru': 'cian.ru',
 'petropavlovsk-kamchatskiy.cian.ru': 'cian.ru',
 'cherkessk.cian.ru': 'cian.ru',
 'magadan.cian.ru': 'cian.ru',
 'luhovicy.cian.ru': 'cian.ru',
 'volokolamsk.cian.ru': 'cian.ru',
 'chita.cian.ru': 'cian.ru',
 'service.cian.ru': 'cian.ru',
 'birobidzhan.cian.ru': 'cian.ru',
 'naryan-mar.cian.ru': 'cian.ru',
 'groznyy.cian.ru': 'cian.ru',
    'nevinnomissk.big-book-relax.ru': 'big-book-relax.ru',
 'ryazan.big-book-relax.ru': 'big-book-relax.ru',
 'barnaul.big-book-relax.ru': 'big-book-relax.ru',
 'izhevsk.big-book-relax.ru': 'big-book-relax.ru',
 'kursk.big-book-relax.ru': 'big-book-relax.ru',
 'abakan.big-book-relax.ru': 'big-book-relax.ru',
 'novorossiysk.big-book-relax.ru': 'big-book-relax.ru',
 'tomsk.big-book-relax.ru': 'big-book-relax.ru',
 'kazan.big-book-relax.ru': 'big-book-relax.ru',
 'sterlitamak.big-book-relax.ru': 'big-book-relax.ru',
 'sochi.big-book-relax.ru': 'big-book-relax.ru',
 'nefteyugansk.big-book-relax.ru': 'big-book-relax.ru',
 'rostov-na-donu.big-book-relax.ru': 'big-book-relax.ru',
 'tambov.big-book-relax.ru': 'big-book-relax.ru',
 'saransk.big-book-relax.ru': 'big-book-relax.ru',
 'orenburg.big-book-relax.ru': 'big-book-relax.ru',
 'vladivostok.big-book-relax.ru': 'big-book-relax.ru',
 'moscow.big-book-relax.ru': 'big-book-relax.ru',
 'taganrog.big-book-relax.ru': 'big-book-relax.ru',
 'nizhniy-novgorod.big-book-relax.ru': 'big-book-relax.ru',
 'volgograd.big-book-relax.ru': 'big-book-relax.ru',
 'krasnodar.big-book-relax.ru': 'big-book-relax.ru',
 'saint-petersburg.big-book-relax.ru': 'big-book-relax.ru',
 'tyumen.big-book-relax.ru': 'big-book-relax.ru',
 'ufa.big-book-relax.ru': 'big-book-relax.ru',
 'armavir.big-book-relax.ru': 'big-book-relax.ru',
 'nizhnekamsk.big-book-relax.ru': 'big-book-relax.ru',
 'orehovo-zuevo.big-book-relax.ru': 'big-book-relax.ru',
 'naberezhnyye-chelny.big-book-relax.ru': 'big-book-relax.ru',
 'kirov.big-book-relax.ru': 'big-book-relax.ru',
 'staryy-oskol.big-book-relax.ru': 'big-book-relax.ru',
 'surgut.big-book-relax.ru': 'big-book-relax.ru',
 'tver.big-book-relax.ru': 'big-book-relax.ru',
 'podolsk.big-book-relax.ru': 'big-book-relax.ru',
 'oryol.big-book-relax.ru': 'big-book-relax.ru',
 'nizhnevartovsk.big-book-relax.ru': 'big-book-relax.ru',
 'engels.big-book-relax.ru': 'big-book-relax.ru',
 'tolyatti.big-book-relax.ru': 'big-book-relax.ru',
 'orsk.big-book-relax.ru': 'big-book-relax.ru',
 'vladikavkaz.big-book-relax.ru': 'big-book-relax.ru',
 'nizhniy-tagil.big-book-relax.ru': 'big-book-relax.ru',
 'ivanovo.big-book-relax.ru': 'big-book-relax.ru',
 'kamishin.big-book-relax.ru': 'big-book-relax.ru',
 'kolomna.big-book-relax.ru': 'big-book-relax.ru',
 'norilsk.big-book-relax.ru': 'big-book-relax.ru',
 'smolensk.big-book-relax.ru': 'big-book-relax.ru',
 'pskov.big-book-relax.ru': 'big-book-relax.ru',
 'balashikha.big-book-relax.ru': 'big-book-relax.ru',
 'saratov.big-book-relax.ru': 'big-book-relax.ru',
 'novosibirsk.big-book-relax.ru': 'big-book-relax.ru',
 'pyatigorsk.big-book-relax.ru': 'big-book-relax.ru',
 'belgorod.big-book-relax.ru': 'big-book-relax.ru',
 'dzerzhinsk.big-book-relax.ru': 'big-book-relax.ru',
 'yekaterinburg.big-book-relax.ru': 'big-book-relax.ru',
 'chelyabinsk.big-book-relax.ru': 'big-book-relax.ru',
 'samara.big-book-relax.ru': 'big-book-relax.ru',
 'velikiy-novgorod.big-book-relax.ru': 'big-book-relax.ru',
 'miass.big-book-relax.ru': 'big-book-relax.ru',
 'penza.big-book-relax.ru': 'big-book-relax.ru',
 'vladimir.big-book-relax.ru': 'big-book-relax.ru',
 'yoshkar-ola.big-book-relax.ru': 'big-book-relax.ru',
 'kyzyl.big-book-relax.ru': 'big-book-relax.ru',
 'viborg.big-book-relax.ru': 'big-book-relax.ru',
 'tula.big-book-relax.ru': 'big-book-relax.ru',
 'kurgan.big-book-relax.ru': 'big-book-relax.ru',
 'murmansk.big-book-relax.ru': 'big-book-relax.ru',
 'perm.big-book-relax.ru': 'big-book-relax.ru',
 'kaluga.big-book-relax.ru': 'big-book-relax.ru',
 'hanty-mansijsk.big-book-relax.ru': 'big-book-relax.ru',
 'ulyanovsk.big-book-relax.ru': 'big-book-relax.ru',
 'makhachkala.big-book-relax.ru': 'big-book-relax.ru',
 'sizran.big-book-relax.ru': 'big-book-relax.ru',
 'obninsk.big-book-relax.ru': 'big-book-relax.ru',
 'voronezh.big-book-relax.ru': 'big-book-relax.ru',
 'kaliningrad.big-book-relax.ru': 'big-book-relax.ru',
 'neftekamsk.big-book-relax.ru': 'big-book-relax.ru',
 'volgodonsk.big-book-relax.ru': 'big-book-relax.ru',
 'lipetsk.big-book-relax.ru': 'big-book-relax.ru',
 'bryansk.big-book-relax.ru': 'big-book-relax.ru',
 'vologda.big-book-relax.ru': 'big-book-relax.ru',
 'almetyevsk.big-book-relax.ru': 'big-book-relax.ru',
 'cheboksary.big-book-relax.ru': 'big-book-relax.ru',
 'khabarovsk.big-book-relax.ru': 'big-book-relax.ru',
 'cherepovets.big-book-relax.ru': 'big-book-relax.ru',
 'oktyabrsky.big-book-relax.ru': 'big-book-relax.ru',
 'omsk.big-book-relax.ru': 'big-book-relax.ru',
 'petropavlovsk-kamchatskiy.big-book-relax.ru': 'big-book-relax.ru',
 'berezniki.big-book-relax.ru': 'big-book-relax.ru',
 'pushkino.big-book-relax.ru': 'big-book-relax.ru',
 'kovrov.big-book-relax.ru': 'big-book-relax.ru',
 'jelektrostal.big-book-relax.ru': 'big-book-relax.ru',
 'cherkessk.big-book-relax.ru': 'big-book-relax.ru',
 'groznyy.big-book-relax.ru': 'big-book-relax.ru',
 'arkhangelsk.big-book-relax.ru': 'big-book-relax.ru',
 'novomoscowsk.big-book-relax.ru': 'big-book-relax.ru',
 'wjolkovo.big-book-relax.ru': 'big-book-relax.ru',
 'blagoveshchensk.big-book-relax.ru': 'big-book-relax.ru',
 'yakutsk.big-book-relax.ru': 'big-book-relax.ru',
 'krasnoyarsk.big-book-relax.ru': 'big-book-relax.ru',
 'maykop.big-book-relax.ru': 'big-book-relax.ru',
 'yssyrisk.big-book-relax.ru': 'big-book-relax.ru',
 'arzamas.big-book-relax.ru': 'big-book-relax.ru',
 'balakovo.big-book-relax.ru': 'big-book-relax.ru',
 'petrozavodsk.big-book-relax.ru': 'big-book-relax.ru',
 'yaroslavl.big-book-relax.ru': 'big-book-relax.ru',
 'yhta.big-book-relax.ru': 'big-book-relax.ru',
 'sergiev-posad.big-book-relax.ru': 'big-book-relax.ru',
 'salovat.big-book-relax.ru': 'big-book-relax.ru',
 'kostroma.big-book-relax.ru': 'big-book-relax.ru',
 'serpuhov.big-book-relax.ru': 'big-book-relax.ru',
 'novocherkask.big-book-relax.ru': 'big-book-relax.ru',
 'magnitogorsk.big-book-relax.ru': 'big-book-relax.ru',
 'novy-urengoy.big-book-relax.ru': 'big-book-relax.ru',
 'kemerovo.big-book-relax.ru': 'big-book-relax.ru',
 'yeisk.big-book-relax.ru': 'big-book-relax.ru',
 'syktyvkar.big-book-relax.ru': 'big-book-relax.ru',
 'stavropol.big-book-relax.ru': 'big-book-relax.ru',
 'yelets.big-book-relax.ru': 'big-book-relax.ru',
 'shakhty.big-book-relax.ru': 'big-book-relax.ru',
 'pervoyralsk.big-book-relax.ru': 'big-book-relax.ru',
 'astrakhan.big-book-relax.ru': 'big-book-relax.ru',
 'rybinsk.big-book-relax.ru': 'big-book-relax.ru',
 'zhukovskij.big-book-relax.ru': 'big-book-relax.ru',
 'ramenskoe.big-book-relax.ru': 'big-book-relax.ru',
 'odincovo.big-book-relax.ru': 'big-book-relax.ru',
 'murom.big-book-relax.ru': 'big-book-relax.ru',
 'biysk.big-book-relax.ru': 'big-book-relax.ru',
 'komsomolsk-na-amure.big-book-relax.ru': 'big-book-relax.ru',
 'tobolsk.big-book-relax.ru': 'big-book-relax.ru',
 'zlatoust.big-book-relax.ru': 'big-book-relax.ru',
 'dimitrovgrad.big-book-relax.ru': 'big-book-relax.ru',
 'khimki.big-book-relax.ru': 'big-book-relax.ru',
 'novokuznetsk.big-book-relax.ru': 'big-book-relax.ru',
 'serov.big-book-relax.ru': 'big-book-relax.ru',
 'achinsk.big-book-relax.ru': 'big-book-relax.ru',
 'zheleznodorozhnyj.big-book-relax.ru': 'big-book-relax.ru',
    'postupi.online': 'postupi.online',
 'tyumen.postupi.online': 'postupi.online',
 'krasnoyarsk.postupi.online': 'postupi.online',
 'msk.postupi.online': 'postupi.online',
 'ufa.postupi.online': 'postupi.online',
 'samara.postupi.online': 'postupi.online',
 'kostroma.postupi.online': 'postupi.online',
 'saratov.postupi.online': 'postupi.online',
 'naberezhnye-chelny.postupi.online': 'postupi.online',
 'krasnodar.postupi.online': 'postupi.online',
 'spb.postupi.online': 'postupi.online',
 'vladivostok.postupi.online': 'postupi.online',
 'kazan.postupi.online': 'postupi.online',
 'belgorod.postupi.online': 'postupi.online',
 'vologda.postupi.online': 'postupi.online',
 'saransk.postupi.online': 'postupi.online',
 'perm.postupi.online': 'postupi.online',
 'majkop.postupi.online': 'postupi.online',
 'sterlitamak.postupi.online': 'postupi.online',
 'habarovsk.postupi.online': 'postupi.online',
 'astrahan.postupi.online': 'postupi.online',
 'ekaterinburg.postupi.online': 'postupi.online',
 'himki.postupi.online': 'postupi.online',
 'barnaul.postupi.online': 'postupi.online',
 'chelyabinsk.postupi.online': 'postupi.online',
 'nsk.postupi.online': 'postupi.online',
 'tambov.postupi.online': 'postupi.online',
 'ivanovo.postupi.online': 'postupi.online',
 'ulyanovsk.postupi.online': 'postupi.online',
 'kaliningrad.postupi.online': 'postupi.online',
 'kemerovo.postupi.online': 'postupi.online',
 'ryazan.postupi.online': 'postupi.online',
 'rostov.postupi.online': 'postupi.online',
 'omsk.postupi.online': 'postupi.online',
 'orenburg.postupi.online': 'postupi.online',
 'kirov.postupi.online': 'postupi.online',
 'simferopol.postupi.online': 'postupi.online',
 'lipeck.postupi.online': 'postupi.online',
 'izhevsk.postupi.online': 'postupi.online',
 'novorossijsk.postupi.online': 'postupi.online',
 'nn.postupi.online': 'postupi.online',
 'dolgoprudniy.postupi.online': 'postupi.online',
 'oktyabrskij.postupi.online': 'postupi.online',
 'cheboksary.postupi.online': 'postupi.online',
 'gatchina.postupi.online': 'postupi.online',
 'pskov.postupi.online': 'postupi.online',
 'uhta.postupi.online': 'postupi.online',
 'volgograd.postupi.online': 'postupi.online',
 'murmansk.postupi.online': 'postupi.online',
 'vladimir.postupi.online': 'postupi.online',
 'tula.postupi.online': 'postupi.online',
 'malahovka.postupi.online': 'postupi.online',
 'novocherkassk.postupi.online': 'postupi.online',
 'yaroslavl.postupi.online': 'postupi.online',
 'surgut.postupi.online': 'postupi.online',
 'stavropol.postupi.online': 'postupi.online',
 'stupino.postupi.online': 'postupi.online',
 'orel.postupi.online': 'postupi.online',
 'kursk.postupi.online': 'postupi.online',
 'zelenograd.postupi.online': 'postupi.online',
 'stariy-oskol.postupi.online': 'postupi.online',
 'nizhniy-tagil.postupi.online': 'postupi.online',
 'novokuzneck.postupi.online': 'postupi.online',
 'shadrinsk.postupi.online': 'postupi.online',
 'yalta.postupi.online': 'postupi.online',
 'kaluga.postupi.online': 'postupi.online',
 'yakutsk.postupi.online': 'postupi.online',
 'tver.postupi.online': 'postupi.online',
 'smolensk.postupi.online': 'postupi.online',
 'tomsk.postupi.online': 'postupi.online',
 'petrozavodsk.postupi.online': 'postupi.online',
 'nalchik.postupi.online': 'postupi.online',
 'bryansk.postupi.online': 'postupi.online',
 'hanty-mansiysk.postupi.online': 'postupi.online',
 'sochi.postupi.online': 'postupi.online',
 'voronezh.postupi.online': 'postupi.online',
 'sevastopol.postupi.online': 'postupi.online',
 'anapa.postupi.online': 'postupi.online',
 'orsk.postupi.online': 'postupi.online',
 'tolyatti.postupi.online': 'postupi.online',
 'volzhskij.postupi.online': 'postupi.online',
 'neftekamsk.postupi.online': 'postupi.online',
 'mahachkala.postupi.online': 'postupi.online',
 'kurgan.postupi.online': 'postupi.online',
 'kolomna.postupi.online': 'postupi.online',
 'gorno-altaysk.postupi.online': 'postupi.online',
 'elec.postupi.online': 'postupi.online',
 'dubna.postupi.online': 'postupi.online',
 'dzerzhinsk.postupi.online': 'postupi.online',
 'vladikavkaz.postupi.online': 'postupi.online',
 'ussurijsk.postupi.online': 'postupi.online',
 'elabuga.postupi.online': 'postupi.online',
 'domodedovo.postupi.online': 'postupi.online',
 'sarapul.postupi.online': 'postupi.online',
 'blagoveshhensk.postupi.online': 'postupi.online',
 'orekhovo-zuevo.postupi.online': 'postupi.online',
 'armavir.postupi.online': 'postupi.online',
 'komsomolsk-na-amure.postupi.online': 'postupi.online',
 'petropavlovsk-kamchatskij.postupi.online': 'postupi.online',
 'troick.postupi.online': 'postupi.online',
 'pyatigorsk.postupi.online': 'postupi.online',
 'magadan.postupi.online': 'postupi.online',
 'penza.postupi.online': 'postupi.online',
 'mytishchi.postupi.online': 'postupi.online',
 'magnitogorsk.postupi.online': 'postupi.online',
 'podolsk.postupi.online': 'postupi.online',
 'voskresensk.postupi.online': 'postupi.online',
 'elista.postupi.online': 'postupi.online',
 'joshkar-ola.postupi.online': 'postupi.online',
 'korolyov.postupi.online': 'postupi.online',
 'miass.postupi.online': 'postupi.online',
 'abakan.postupi.online': 'postupi.online',
 'velikiy-novgorod.postupi.online': 'postupi.online',
 'syktyvkar.postupi.online': 'postupi.online',
 'sibaj.postupi.online': 'postupi.online',
 'nizhnevartovsk.postupi.online': 'postupi.online',
 'biysk.postupi.online': 'postupi.online',
 'nizhnekamsk.postupi.online': 'postupi.online',
 'taganrog.postupi.online': 'postupi.online',
 'shahty.postupi.online': 'postupi.online',
 'cherepovec.postupi.online': 'postupi.online',
 'almetevsk.postupi.online': 'postupi.online',
 'tobolsk.postupi.online': 'postupi.online',
 'cherkesk.postupi.online': 'postupi.online',
 'irkutsk.postupi.online': 'postupi.online',
 'obninsk.postupi.online': 'postupi.online',
 'ulan-ude.postupi.online': 'postupi.online',
 'salavat.postupi.online': 'postupi.online',
 'kerch.postupi.online': 'postupi.online',
 'rybinsk.postupi.online': 'postupi.online',
 'birobidzhan.postupi.online': 'postupi.online',
 'arhangelsk.postupi.online': 'postupi.online',
 'odincovo.postupi.online': 'postupi.online',
 'chita.postupi.online': 'postupi.online',
 'nazran.postupi.online': 'postupi.online',
 'sergiev-posad.postupi.online': 'postupi.online',
 'shuya.postupi.online': 'postupi.online',
 'belovo.postupi.online': 'postupi.online',
 'severodvinsk.postupi.online': 'postupi.online',
    'orsk.big-book-med.ru': 'big-book-med.ru',
 'surgut.big-book-med.ru': 'big-book-med.ru',
 'tolyatti.big-book-med.ru': 'big-book-med.ru',
 'novorossiysk.big-book-med.ru': 'big-book-med.ru',
 'tyumen.big-book-med.ru': 'big-book-med.ru',
 'yoshkar-ola.big-book-med.ru': 'big-book-med.ru',
 'shakhty.big-book-med.ru': 'big-book-med.ru',
 'ulyanovsk.big-book-med.ru': 'big-book-med.ru',
 'kursk.big-book-med.ru': 'big-book-med.ru',
 'krasnoyarsk.big-book-med.ru': 'big-book-med.ru',
 'moscow.big-book-med.ru': 'big-book-med.ru',
 'izhevsk.big-book-med.ru': 'big-book-med.ru',
 'yaroslavl.big-book-med.ru': 'big-book-med.ru',
 'neftekamsk.big-book-med.ru': 'big-book-med.ru',
 'salovat.big-book-med.ru': 'big-book-med.ru',
 'bryansk.big-book-med.ru': 'big-book-med.ru',
 'krasnodar.big-book-med.ru': 'big-book-med.ru',
 'ryazan.big-book-med.ru': 'big-book-med.ru',
 'sarapyl.big-book-med.ru': 'big-book-med.ru',
 'novosibirsk.big-book-med.ru': 'big-book-med.ru',
 'yekaterinburg.big-book-med.ru': 'big-book-med.ru',
 'pskov.big-book-med.ru': 'big-book-med.ru',
 'saint-petersburg.big-book-med.ru': 'big-book-med.ru',
 'tver.big-book-med.ru': 'big-book-med.ru',
 'saratov.big-book-med.ru': 'big-book-med.ru',
 'rostov-na-donu.big-book-med.ru': 'big-book-med.ru',
 'orenburg.big-book-med.ru': 'big-book-med.ru',
 'omsk.big-book-med.ru': 'big-book-med.ru',
 'ufa.big-book-med.ru': 'big-book-med.ru',
 'nizhniy-novgorod.big-book-med.ru': 'big-book-med.ru',
 'taganrog.big-book-med.ru': 'big-book-med.ru',
 'sochi.big-book-med.ru': 'big-book-med.ru',
 'pyatigorsk.big-book-med.ru': 'big-book-med.ru',
 'barnaul.big-book-med.ru': 'big-book-med.ru',
 'kaliningrad.big-book-med.ru': 'big-book-med.ru',
 'staryy-oskol.big-book-med.ru': 'big-book-med.ru',
 'yeisk.big-book-med.ru': 'big-book-med.ru',
 'kazan.big-book-med.ru': 'big-book-med.ru',
 'kolomna.big-book-med.ru': 'big-book-med.ru',
 'krasnogorsk.big-book-med.ru': 'big-book-med.ru',
 'khabarovsk.big-book-med.ru': 'big-book-med.ru',
 'vladikavkaz.big-book-med.ru': 'big-book-med.ru',
 'kostroma.big-book-med.ru': 'big-book-med.ru',
 'cherkessk.big-book-med.ru': 'big-book-med.ru',
 'oktyabrsky.big-book-med.ru': 'big-book-med.ru',
 'kovrov.big-book-med.ru': 'big-book-med.ru',
 'penza.big-book-med.ru': 'big-book-med.ru',
 'makhachkala.big-book-med.ru': 'big-book-med.ru',
 'oryol.big-book-med.ru': 'big-book-med.ru',
 'nizhniy-tagil.big-book-med.ru': 'big-book-med.ru',
 'zheleznodorozhnyj.big-book-med.ru': 'big-book-med.ru',
 'engels.big-book-med.ru': 'big-book-med.ru',
 'naberezhnyye-chelny.big-book-med.ru': 'big-book-med.ru',
 'cheboksary.big-book-med.ru': 'big-book-med.ru',
 'tambov.big-book-med.ru': 'big-book-med.ru',
 'lipetsk.big-book-med.ru': 'big-book-med.ru',
 'kurgan.big-book-med.ru': 'big-book-med.ru',
 'murmansk.big-book-med.ru': 'big-book-med.ru',
 'viborg.big-book-med.ru': 'big-book-med.ru',
 'maykop.big-book-med.ru': 'big-book-med.ru',
 'sterlitamak.big-book-med.ru': 'big-book-med.ru',
 'nizhnekamsk.big-book-med.ru': 'big-book-med.ru',
 'podolsk.big-book-med.ru': 'big-book-med.ru',
 'abakan.big-book-med.ru': 'big-book-med.ru',
 'koroljov.big-book-med.ru': 'big-book-med.ru',
 'cherepovets.big-book-med.ru': 'big-book-med.ru',
 'novocheboksarsk.big-book-med.ru': 'big-book-med.ru',
 'arzamas.big-book-med.ru': 'big-book-med.ru',
 'kyzyl.big-book-med.ru': 'big-book-med.ru',
 'petropavlovsk-kamchatskiy.big-book-med.ru': 'big-book-med.ru',
 'stavropol.big-book-med.ru': 'big-book-med.ru',
 'perm.big-book-med.ru': 'big-book-med.ru',
 'vladimir.big-book-med.ru': 'big-book-med.ru',
 'almetyevsk.big-book-med.ru': 'big-book-med.ru',
 'kirov.big-book-med.ru': 'big-book-med.ru',
 'belgorod.big-book-med.ru': 'big-book-med.ru',
 'sizran.big-book-med.ru': 'big-book-med.ru',
 'tomsk.big-book-med.ru': 'big-book-med.ru',
 'groznyy.big-book-med.ru': 'big-book-med.ru',
 'odincovo.big-book-med.ru': 'big-book-med.ru',
 'kaluga.big-book-med.ru': 'big-book-med.ru',
 'novocherkask.big-book-med.ru': 'big-book-med.ru',
 'zhukovskij.big-book-med.ru': 'big-book-med.ru',
 'ivanovo.big-book-med.ru': 'big-book-med.ru',
 'samara.big-book-med.ru': 'big-book-med.ru',
 'severodvinsk.big-book-med.ru': 'big-book-med.ru',
 'sergiev-posad.big-book-med.ru': 'big-book-med.ru',
 'jelektrostal.big-book-med.ru': 'big-book-med.ru',
 'voronezh.big-book-med.ru': 'big-book-med.ru',
 'serpuhov.big-book-med.ru': 'big-book-med.ru',
 'biysk.big-book-med.ru': 'big-book-med.ru',
 'pushkino.big-book-med.ru': 'big-book-med.ru',
 'magnitogorsk.big-book-med.ru': 'big-book-med.ru',
 'armavir.big-book-med.ru': 'big-book-med.ru',
 'nakhodka.big-book-med.ru': 'big-book-med.ru',
 'petrozavodsk.big-book-med.ru': 'big-book-med.ru',
 'hanty-mansijsk.big-book-med.ru': 'big-book-med.ru',
 'chelyabinsk.big-book-med.ru': 'big-book-med.ru',
 'tula.big-book-med.ru': 'big-book-med.ru',
 'rybinsk.big-book-med.ru': 'big-book-med.ru',
 'orehovo-zuevo.big-book-med.ru': 'big-book-med.ru',
 'tobolsk.big-book-med.ru': 'big-book-med.ru',
 'dzerzhinsk.big-book-med.ru': 'big-book-med.ru',
 'saransk.big-book-med.ru': 'big-book-med.ru',
 'elista.big-book-med.ru': 'big-book-med.ru',
 'mytiwi.big-book-med.ru': 'big-book-med.ru',
 'volgodonsk.big-book-med.ru': 'big-book-med.ru',
 'syktyvkar.big-book-med.ru': 'big-book-med.ru',
 'astrakhan.big-book-med.ru': 'big-book-med.ru',
 'obninsk.big-book-med.ru': 'big-book-med.ru',
 'novokuznetsk.big-book-med.ru': 'big-book-med.ru',
 'novoabrsk.big-book-med.ru': 'big-book-med.ru',
 'velikiy-novgorod.big-book-med.ru': 'big-book-med.ru',
 'smolensk.big-book-med.ru': 'big-book-med.ru',
 'novy-urengoy.big-book-med.ru': 'big-book-med.ru',
 'balakovo.big-book-med.ru': 'big-book-med.ru',
 'volzhskiy.big-book-med.ru': 'big-book-med.ru',
 'ramenskoe.big-book-med.ru': 'big-book-med.ru',
 'yelets.big-book-med.ru': 'big-book-med.ru',
 'blagoveshchensk.big-book-med.ru': 'big-book-med.ru',
 'vladivostok.big-book-med.ru': 'big-book-med.ru',
 'vologda.big-book-med.ru': 'big-book-med.ru',
 'nizhnevartovsk.big-book-med.ru': 'big-book-med.ru',
 'balashikha.big-book-med.ru': 'big-book-med.ru',
 'ljubercy.big-book-med.ru': 'big-book-med.ru',
 'artem.big-book-med.ru': 'big-book-med.ru',
 'yssyrisk.big-book-med.ru': 'big-book-med.ru',
 'komsomolsk-na-amure.big-book-med.ru': 'big-book-med.ru',
 'noginsk.big-book-med.ru': 'big-book-med.ru',
 'wjolkovo.big-book-med.ru': 'big-book-med.ru',
 'nevinnomissk.big-book-med.ru': 'big-book-med.ru',
 'kamishin.big-book-med.ru': 'big-book-med.ru',
 'pervoyralsk.big-book-med.ru': 'big-book-med.ru',
 'kemerovo.big-book-med.ru': 'big-book-med.ru',
 'kamensk-yralski.big-book-med.ru': 'big-book-med.ru',
 'khimki.big-book-med.ru': 'big-book-med.ru',
 'nalchik.big-book-med.ru': 'big-book-med.ru',
 'yuzhno-sakhalinsk.big-book-med.ru': 'big-book-med.ru',
 'prokopievsk.big-book-med.ru': 'big-book-med.ru',
 'berezniki.big-book-med.ru': 'big-book-med.ru',
 'volgograd.big-book-med.ru': 'big-book-med.ru',
 'chita.big-book-med.ru': 'big-book-med.ru',
 'miass.big-book-med.ru': 'big-book-med.ru',
 'achinsk.big-book-med.ru': 'big-book-med.ru',
 'irkutsk.big-book-med.ru': 'big-book-med.ru',
 'ulan-ude.big-book-med.ru': 'big-book-med.ru',
 'dimitrovgrad.big-book-med.ru': 'big-book-med.ru',
 'novomoscowsk.big-book-med.ru': 'big-book-med.ru',
 'arkhangelsk.big-book-med.ru': 'big-book-med.ru',
    'moskva.vbr.ru': 'vbr.ru',
 'tumen.vbr.ru': 'vbr.ru',
 'perm.vbr.ru': 'vbr.ru',
 'kaliningrad.vbr.ru': 'vbr.ru',
 'vbr.ru': 'vbr.ru',
 'sankt-peterburg.vbr.ru': 'vbr.ru',
 'serpyhov.vbr.ru': 'vbr.ru',
 'vladimir.vbr.ru': 'vbr.ru',
 'volgograd.vbr.ru': 'vbr.ru',
 'gys-hrystalnii.vbr.ru': 'vbr.ru',
 'sochi.vbr.ru': 'vbr.ru',
 'voronezh.vbr.ru': 'vbr.ru',
 'novosibirsk.vbr.ru': 'vbr.ru',
 'orenburg.vbr.ru': 'vbr.ru',
 'cheljabinsk.vbr.ru': 'vbr.ru',
 'dolgoprydnii.vbr.ru': 'vbr.ru',
 'ufa.vbr.ru': 'vbr.ru',
 'cheboksary.vbr.ru': 'vbr.ru',
 'ijevsk.vbr.ru': 'vbr.ru',
 'ekaterinburg.vbr.ru': 'vbr.ru',
 'krasnodar.vbr.ru': 'vbr.ru',
 'luberci.vbr.ru': 'vbr.ru',
 'nizhnij-novgorod.vbr.ru': 'vbr.ru',
 'maikop.vbr.ru': 'vbr.ru',
 'rostov-na-donu.vbr.ru': 'vbr.ru',
 'engels.vbr.ru': 'vbr.ru',
 'samara.vbr.ru': 'vbr.ru',
 'novorossiisk.vbr.ru': 'vbr.ru',
 'saratov.vbr.ru': 'vbr.ru',
 'rjazan.vbr.ru': 'vbr.ru',
 'korolev.vbr.ru': 'vbr.ru',
 'petrozavodsk.vbr.ru': 'vbr.ru',
 'blagoveshchensk.vbr.ru': 'vbr.ru',
 'hanti-mansiisk.vbr.ru': 'vbr.ru',
 'joshkar-ola.vbr.ru': 'vbr.ru',
 'komsomolsk-na-amyre.vbr.ru': 'vbr.ru',
 'armavir.vbr.ru': 'vbr.ru',
 'jaroslavl.vbr.ru': 'vbr.ru',
 'tolatti.vbr.ru': 'vbr.ru',
 'nijnevartovsk.vbr.ru': 'vbr.ru',
 'barnaul.vbr.ru': 'vbr.ru',
 'habarovsk.vbr.ru': 'vbr.ru',
 'krasnogorsk.vbr.ru': 'vbr.ru',
 'penza.vbr.ru': 'vbr.ru',
 'ivanovo.vbr.ru': 'vbr.ru',
 'tomsk.vbr.ru': 'vbr.ru',
 'orel.vbr.ru': 'vbr.ru',
 'podolsk.vbr.ru': 'vbr.ru',
 'neftekamsk.vbr.ru': 'vbr.ru',
 'kursk.vbr.ru': 'vbr.ru',
 'kirov.vbr.ru': 'vbr.ru',
 'krasnojarsk.vbr.ru': 'vbr.ru',
 'essentyki.vbr.ru': 'vbr.ru',
 'omsk.vbr.ru': 'vbr.ru',
 'tobolsk.vbr.ru': 'vbr.ru',
 'cherkessk.vbr.ru': 'vbr.ru',
 'vladivostok.vbr.ru': 'vbr.ru',
 'naberejnie-celni.vbr.ru': 'vbr.ru',
 'bataisk.vbr.ru': 'vbr.ru',
 'vologda.vbr.ru': 'vbr.ru',
 'velikij-novgorod.vbr.ru': 'vbr.ru',
 'odincovo.vbr.ru': 'vbr.ru',
 'siktivkar.vbr.ru': 'vbr.ru',
 'acinsk.vbr.ru': 'vbr.ru',
 'lipetsk.vbr.ru': 'vbr.ru',
 'kurgan.vbr.ru': 'vbr.ru',
 'abakan.vbr.ru': 'vbr.ru',
 'magnitogorsk.vbr.ru': 'vbr.ru',
 'anapa.vbr.ru': 'vbr.ru',
 'nijnii-tagil.vbr.ru': 'vbr.ru',
 'nevinnomissk.vbr.ru': 'vbr.ru',
 'nijnekamsk.vbr.ru': 'vbr.ru',
 'belgorod.vbr.ru': 'vbr.ru',
 'himki.vbr.ru': 'vbr.ru',
 'stavropol.vbr.ru': 'vbr.ru',
 'taganrog.vbr.ru': 'vbr.ru',
 'kstovo.vbr.ru': 'vbr.ru',
 'yhta.vbr.ru': 'vbr.ru',
 'kaluga.vbr.ru': 'vbr.ru',
 'jeleznodorojnii.vbr.ru': 'vbr.ru',
 'surgut.vbr.ru': 'vbr.ru',
 'arhangelsk.vbr.ru': 'vbr.ru',
 'tambov.vbr.ru': 'vbr.ru',
 'vidnoe.vbr.ru': 'vbr.ru',
 'pskov.vbr.ru': 'vbr.ru',
 'vladikavkaz.vbr.ru': 'vbr.ru',
 'kostroma.vbr.ru': 'vbr.ru',
 'ujno-sahalinsk.vbr.ru': 'vbr.ru',
 'sizran.vbr.ru': 'vbr.ru',
 'salavat.vbr.ru': 'vbr.ru',
 'nalchik.vbr.ru': 'vbr.ru',
 'tver.vbr.ru': 'vbr.ru',
 'ramenskoe.vbr.ru': 'vbr.ru',
 'evpatoria.vbr.ru': 'vbr.ru',
 'brjansk.vbr.ru': 'vbr.ru',
 'derbent.vbr.ru': 'vbr.ru',
 'balasiha.vbr.ru': 'vbr.ru',
 'troick.vbr.ru': 'vbr.ru',
 'biisk.vbr.ru': 'vbr.ru',
 'uljanovsk.vbr.ru': 'vbr.ru',
 'sahti.vbr.ru': 'vbr.ru',
 'simferopol.vbr.ru': 'vbr.ru',
 'sevastopol.vbr.ru': 'vbr.ru',
 'pyskino.vbr.ru': 'vbr.ru',
 'nefteugansk.vbr.ru': 'vbr.ru',
 'kolomna.vbr.ru': 'vbr.ru',
 'kovrov.vbr.ru': 'vbr.ru',
 'ribinsk.vbr.ru': 'vbr.ru',
 'yssyriisk.vbr.ru': 'vbr.ru',
 'klin.vbr.ru': 'vbr.ru',
 'akytsk.vbr.ru': 'vbr.ru',
 'novii-yrengoi.vbr.ru': 'vbr.ru',
 'novokyzneck.vbr.ru': 'vbr.ru',
 'tula.vbr.ru': 'vbr.ru',
 'zlatoyst.vbr.ru': 'vbr.ru',
 'mahachkala.vbr.ru': 'vbr.ru',
 'gorno-altaisk.vbr.ru': 'vbr.ru',
 'domodedovo.vbr.ru': 'vbr.ru',
 'elektrostal.vbr.ru': 'vbr.ru',
 'noginsk.vbr.ru': 'vbr.ru',
 'voljskii.vbr.ru': 'vbr.ru',
 'selkovo.vbr.ru': 'vbr.ru',
 'noabrsk.vbr.ru': 'vbr.ru',
 'saransk.vbr.ru': 'vbr.ru',
 'smolensk.vbr.ru': 'vbr.ru',
 'almetevsk.vbr.ru': 'vbr.ru',
 'obninsk.vbr.ru': 'vbr.ru',
 'solikamsk.vbr.ru': 'vbr.ru',
 'kemerovo.vbr.ru': 'vbr.ru',
 'miass.vbr.ru': 'vbr.ru',
 'zelenograd.vbr.ru': 'vbr.ru',
 'murmansk.vbr.ru': 'vbr.ru',
 'mitisi.vbr.ru': 'vbr.ru',
 'cerepovec.vbr.ru': 'vbr.ru',
 'dzerjinsk.vbr.ru': 'vbr.ru',
 'jykovskii.vbr.ru': 'vbr.ru',
 'salehard.vbr.ru': 'vbr.ru',
 'volgodonsk.vbr.ru': 'vbr.ru',
 'severodvinsk.vbr.ru': 'vbr.ru',
 'starii-oskol.vbr.ru': 'vbr.ru',
 'orsk.vbr.ru': 'vbr.ru',
 'solnecnogorsk.vbr.ru': 'vbr.ru',
 'kamensk-yralskii.vbr.ru': 'vbr.ru',
 'novocerkassk.vbr.ru': 'vbr.ru',
 'kislovodsk.vbr.ru': 'vbr.ru',
 'elista.vbr.ru': 'vbr.ru',
 'pervoyralsk.vbr.ru': 'vbr.ru',
 'birobidjan.vbr.ru': 'vbr.ru',
 'stypino.vbr.ru': 'vbr.ru',
 'magadan.vbr.ru': 'vbr.ru',
    'yakutsk.spravka.city': 'spravka.city',
 'novorossiysk.spravka.city': 'spravka.city',
 'kurgan.spravka.city': 'spravka.city',
 'achinsk.spravka.city': 'spravka.city',
 'blagoveshchensk.spravka.city': 'spravka.city',
 'magnitogorsk.spravka.city': 'spravka.city',
 'pavlovskiy-posad.spravka.city': 'spravka.city',
 'omsk.spravka.city': 'spravka.city',
 'tolyatti.spravka.city': 'spravka.city',
 'sochi.spravka.city': 'spravka.city',
 'revda.spravka.city': 'spravka.city',
 'novosibirsk.spravka.city': 'spravka.city',
 'tula.spravka.city': 'spravka.city',
 'pushkino.spravka.city': 'spravka.city',
 'shcherbinka.spravka.city': 'spravka.city',
 'oktyabrskiy-bashkortostan.spravka.city': 'spravka.city',
 'mahachkala.spravka.city': 'spravka.city',
 'yuzhno-sahalinsk.spravka.city': 'spravka.city',
 'bryansk.spravka.city': 'spravka.city',
 'prokopevsk.spravka.city': 'spravka.city',
 'izhevsk.spravka.city': 'spravka.city',
 'pskov.spravka.city': 'spravka.city',
 'nizhnevartovsk.spravka.city': 'spravka.city',
 'ussuriysk.spravka.city': 'spravka.city',
 'msk.spravka.city': 'spravka.city',
 'spb.spravka.city': 'spravka.city',
 'kemerovo.spravka.city': 'spravka.city',
 'tyumen.spravka.city': 'spravka.city',
 'habarovsk.spravka.city': 'spravka.city',
 'perm.spravka.city': 'spravka.city',
 'salavat.spravka.city': 'spravka.city',
 'odincovo.spravka.city': 'spravka.city',
 'kaliningrad.spravka.city': 'spravka.city',
 'shchelkovo.spravka.city': 'spravka.city',
 'krasnodar.spravka.city': 'spravka.city',
 'dolgoprudnyy.spravka.city': 'spravka.city',
 'nizhniy-novgorod.spravka.city': 'spravka.city',
 'abakan.spravka.city': 'spravka.city',
 'chernogorsk.spravka.city': 'spravka.city',
 'orehovo-zuevo.spravka.city': 'spravka.city',
 'anapa.spravka.city': 'spravka.city',
 'novokuzneck.spravka.city': 'spravka.city',
 'georgievsk.spravka.city': 'spravka.city',
 'dzerzhinsk.spravka.city': 'spravka.city',
 'samara.spravka.city': 'spravka.city',
 'sterlitamak.spravka.city': 'spravka.city',
 'rostov-na-donu.spravka.city': 'spravka.city',
 'tobolsk.spravka.city': 'spravka.city',
 'murmansk.spravka.city': 'spravka.city',
 'biysk.spravka.city': 'spravka.city',
 'ufa.spravka.city': 'spravka.city',
 'voronezh.spravka.city': 'spravka.city',
 'ryazan.spravka.city': 'spravka.city',
 'chehov.spravka.city': 'spravka.city',
 'stavropol.spravka.city': 'spravka.city',
 'kazan.spravka.city': 'spravka.city',
 'novokubansk.spravka.city': 'spravka.city',
 'nahodka.spravka.city': 'spravka.city',
 'tver.spravka.city': 'spravka.city',
 'barnaul.spravka.city': 'spravka.city',
 'chelyabinsk.spravka.city': 'spravka.city',
 'ulyanovsk.spravka.city': 'spravka.city',
 'almetevsk.spravka.city': 'spravka.city',
 'ivanovo.spravka.city': 'spravka.city',
 'iskitim.spravka.city': 'spravka.city',
 'vladivostok.spravka.city': 'spravka.city',
 'amursk.spravka.city': 'spravka.city',
 'komsomolsk-na-amure.spravka.city': 'spravka.city',
 'saransk.spravka.city': 'spravka.city',
 'ekaterinburg.spravka.city': 'spravka.city',
 'saratov.spravka.city': 'spravka.city',
 'surgut.spravka.city': 'spravka.city',
 'penza.spravka.city': 'spravka.city',
 'kostroma.spravka.city': 'spravka.city',
 'yaroslavl.spravka.city': 'spravka.city',
 'gatchina.spravka.city': 'spravka.city',
 'ivanteevka.spravka.city': 'spravka.city',
 'belovo.spravka.city': 'spravka.city',
 'zheleznogorsk.spravka.city': 'spravka.city',
 'tuapse.spravka.city': 'spravka.city',
 'kaluga.spravka.city': 'spravka.city',
 'orenburg.spravka.city': 'spravka.city',
 'moskovskiy.spravka.city': 'spravka.city',
 'petropavlovsk-kamchatskiy.spravka.city': 'spravka.city',
 'kolomna.spravka.city': 'spravka.city',
 'lipeck.spravka.city': 'spravka.city',
 'krasnoyarsk.spravka.city': 'spravka.city',
 'ishimbay.spravka.city': 'spravka.city',
 'voskresensk.spravka.city': 'spravka.city',
 'krasnogorsk.spravka.city': 'spravka.city',
 'vladimir.spravka.city': 'spravka.city',
 'novyy-rogachik.spravka.city': 'spravka.city',
 'tambov.spravka.city': 'spravka.city',
 'cheboksary.spravka.city': 'spravka.city',
 'berdsk.spravka.city': 'spravka.city',
 'armavir.spravka.city': 'spravka.city',
 'tuymazy.spravka.city': 'spravka.city',
 'bataysk.spravka.city': 'spravka.city',
 'volgograd.spravka.city': 'spravka.city',
 'yoshkar-ola.spravka.city': 'spravka.city',
 'berezniki.spravka.city': 'spravka.city',
 'mytishchi.spravka.city': 'spravka.city',
 'vologda.spravka.city': 'spravka.city',
 'orel.spravka.city': 'spravka.city',
 'artem.spravka.city': 'spravka.city',
 'tomsk.spravka.city': 'spravka.city',
 'mineralnye-vody.spravka.city': 'spravka.city',
 'otradnoe.spravka.city': 'spravka.city',
 'kirovsk.spravka.city': 'spravka.city',
 'kirovo-chepeck.spravka.city': 'spravka.city',
 'syzran.spravka.city': 'spravka.city',
 'toguchin.spravka.city': 'spravka.city',
 'kursk.spravka.city': 'spravka.city',
 'naberezhnye-chelny.spravka.city': 'spravka.city',
 'volzhskiy.spravka.city': 'spravka.city',
 'kamensk-uralskiy.spravka.city': 'spravka.city',
 'krymsk.spravka.city': 'spravka.city',
 'novomoskovsk.spravka.city': 'spravka.city',
 'kstovo.spravka.city': 'spravka.city',
 'reutov.spravka.city': 'spravka.city',
 'norilsk.spravka.city': 'spravka.city',
 'smolensk.spravka.city': 'spravka.city',
 'korolev.spravka.city': 'spravka.city',
 'ust-labinsk.spravka.city': 'spravka.city',
 'miass.spravka.city': 'spravka.city',
 'zhukovskiy.spravka.city': 'spravka.city',
 'mihnevo.spravka.city': 'spravka.city',
 'cherepovec.spravka.city': 'spravka.city',
 'belgorod.spravka.city': 'spravka.city',
 'oktyabrsk.spravka.city': 'spravka.city',
 'pionerskiy.spravka.city': 'spravka.city',
 'elabuga.spravka.city': 'spravka.city',
 'nizhniy-tagil.spravka.city': 'spravka.city',
 'podolsk.spravka.city': 'spravka.city',
 'nizhnekamsk.spravka.city': 'spravka.city',
 'severodvinsk.spravka.city': 'spravka.city',
 'irkutsk.spravka.city': 'spravka.city',
 'kaspiysk.spravka.city': 'spravka.city',
 'novocheboksarsk.spravka.city': 'spravka.city',
 'blagoveshchensk-bashkortostan.spravka.city': 'spravka.city',
 'tatishchevo.spravka.city': 'spravka.city',
 'korsakov.spravka.city': 'spravka.city',
 'kolyvan.spravka.city': 'spravka.city',
 'velikiy-novgorod.spravka.city': 'spravka.city',
 'arzamas.spravka.city': 'spravka.city',
 'staryy-oskol.spravka.city': 'spravka.city',
 'ordynskoe.spravka.city': 'spravka.city',
 'labinsk.spravka.city': 'spravka.city',
 'gorno-altaysk.spravka.city': 'spravka.city',
 'gurevsk-kaliningradskay.spravka.city': 'spravka.city',
    'tomsk.doski.ru': 'doski.ru',
 'top.doski.ru': 'doski.ru',
 'sochi.doski.ru': 'doski.ru',
 'doski.ru': 'doski.ru',
 'vse.doski.ru': 'doski.ru',
 'belgorod.doski.ru': 'doski.ru',
 'spb.doski.ru': 'doski.ru',
 'samarskaya-oblast.doski.ru': 'doski.ru',
 'samara.doski.ru': 'doski.ru',
 'cheboksary.doski.ru': 'doski.ru',
 'moskva.doski.ru': 'doski.ru',
 'kaliningrad.doski.ru': 'doski.ru',
 'lipetsk.doski.ru': 'doski.ru',
 'sankt-peterburg.doski.ru': 'doski.ru',
 'tomskaya-oblast.doski.ru': 'doski.ru',
 'krasnoyarsk.doski.ru': 'doski.ru',
 'omsk.doski.ru': 'doski.ru',
 'novosibirskaya-oblast.doski.ru': 'doski.ru',
 'ulan-ude.doski.ru': 'doski.ru',
 'irkutsk.doski.ru': 'doski.ru',
 'novosibirsk.doski.ru': 'doski.ru',
 'altaiskiy-krai.doski.ru': 'doski.ru',
 'irkutskaya-oblast.doski.ru': 'doski.ru',
 'kirov.doski.ru': 'doski.ru',
 'kemerovo.doski.ru': 'doski.ru',
 'nizhniy-novgorod.doski.ru': 'doski.ru',
 'ufa.doski.ru': 'doski.ru',
 'moskovskaya-oblast.doski.ru': 'doski.ru',
 'tatarstan.doski.ru': 'doski.ru',
 'krasnodar.doski.ru': 'doski.ru',
 'penza.doski.ru': 'doski.ru',
 'bryanskaya-oblast.doski.ru': 'doski.ru',
 'sahalinskaya-oblast.doski.ru': 'doski.ru',
 'dmitrov.doski.ru': 'doski.ru',
 'ramenskoe.doski.ru': 'doski.ru',
 'kurgan.doski.ru': 'doski.ru',
 'msk.doski.ru': 'doski.ru',
 'krasnoyarskiy-krai.doski.ru': 'doski.ru',
 'saratov.doski.ru': 'doski.ru',
 'ekaterinburg.doski.ru': 'doski.ru',
 'rostov-na-donu.doski.ru': 'doski.ru',
 'krasnodarskiy-krai.doski.ru': 'doski.ru',
 'chelyabinsk.doski.ru': 'doski.ru',
 'sverdlovskaya-oblast.doski.ru': 'doski.ru',
 'kurganskaya-oblast.doski.ru': 'doski.ru',
 'schelkovo.doski.ru': 'doski.ru',
 'harkov.doski.ru': 'doski.ru',
 'tumenskaya-oblast.doski.ru': 'doski.ru',
 'leningradskaya-oblast.doski.ru': 'doski.ru',
 'odintsovo.doski.ru': 'doski.ru',
 'orenburg.doski.ru': 'doski.ru',
 'bashkortostan.doski.ru': 'doski.ru',
 'kolomna.doski.ru': 'doski.ru',
 'krasnogorsk.doski.ru': 'doski.ru',
 'voskresensk.doski.ru': 'doski.ru',
 'vidnoe.doski.ru': 'doski.ru',
 'chehov.doski.ru': 'doski.ru',
 'noginsk.doski.ru': 'doski.ru',
 'saha-yakutiya.doski.ru': 'doski.ru',
 'mytischi.doski.ru': 'doski.ru',
 'lubertsy.doski.ru': 'doski.ru',
 'pushkino.doski.ru': 'doski.ru',
 'podolsk.doski.ru': 'doski.ru',
 'sergiev-posad.doski.ru': 'doski.ru',
 'komi.doski.ru': 'doski.ru',
 'kareliya.doski.ru': 'doski.ru',
 'syktyvkar.doski.ru': 'doski.ru',
 'kazan.doski.ru': 'doski.ru',
 'voronezh.doski.ru': 'doski.ru',
 'anapa.doski.ru': 'doski.ru',
 'kaliningradskaya-oblast.doski.ru': 'doski.ru',
 'hanty-mansiyskiy-ao.doski.ru': 'doski.ru',
 'saratovskaya-oblast.doski.ru': 'doski.ru',
 'stavropolskiy-krai.doski.ru': 'doski.ru',
 'amurskaya-oblast.doski.ru': 'doski.ru',
 'belgorodskaya-oblast.doski.ru': 'doski.ru',
 'kursk.doski.ru': 'doski.ru',
 'istra.doski.ru': 'doski.ru',
 'kirovskaya-oblast.doski.ru': 'doski.ru',
 'barnaul.doski.ru': 'doski.ru',
 'novorossiysk.doski.ru': 'doski.ru',
 'izhevsk.doski.ru': 'doski.ru',
 'habarovsk.doski.ru': 'doski.ru',
 'solnechnogorsk.doski.ru': 'doski.ru',
 'volgograd.doski.ru': 'doski.ru',
 'tverskaya-oblast.doski.ru': 'doski.ru',
 'pskovskaya-oblast.doski.ru': 'doski.ru',
 'ua.doski.ru': 'doski.ru',
 'vladivostok.doski.ru': 'doski.ru',
 'tumen.doski.ru': 'doski.ru',
 'chelyabinskaya-oblast.doski.ru': 'doski.ru',
 'tula.doski.ru': 'doski.ru',
 'tulskaya-oblast.doski.ru': 'doski.ru',
 'yaroslavl.doski.ru': 'doski.ru',
 'sevastopol.doski.ru': 'doski.ru',
 'adygeya.doski.ru': 'doski.ru',
 'yamalo-nenetskiy-ao.doski.ru': 'doski.ru',
 'perm.doski.ru': 'doski.ru',
 'novgorodskaya-oblast.doski.ru': 'doski.ru',
 'krym.doski.ru': 'doski.ru',
 'kamchatskiy-krai.doski.ru': 'doski.ru',
 'yaroslavskaya-oblast.doski.ru': 'doski.ru',
 'omskaya-oblast.doski.ru': 'doski.ru',
 'rostovskaya-oblast.doski.ru': 'doski.ru',
 'kostroma.doski.ru': 'doski.ru',
 'vladimir.doski.ru': 'doski.ru',
 'vladimirskaya-oblast.doski.ru': 'doski.ru',
 'primorskiy-krai.doski.ru': 'doski.ru',
 'tver.doski.ru': 'doski.ru',
 'uzhno-sahalinsk.doski.ru': 'doski.ru',
 'orlovskaya-oblast.doski.ru': 'doski.ru',
 'kaluga.doski.ru': 'doski.ru',
 'habarovskiy-krai.doski.ru': 'doski.ru',
 'by.doski.ru': 'doski.ru',
 'penzenskaya-oblast.doski.ru': 'doski.ru',
 'nizhegorodskaya-oblast.doski.ru': 'doski.ru',
 'hanty-mansiysk.doski.ru': 'doski.ru',
 'udmurtiya.doski.ru': 'doski.ru',
 'kemerovskaya-oblast.doski.ru': 'doski.ru',
 'pskov.doski.ru': 'doski.ru',
 'yalta.doski.ru': 'doski.ru',
 'petropavlovsk-kamchatskiy.doski.ru': 'doski.ru',
 'ulyanovsk.doski.ru': 'doski.ru',
 'smolenskaya-oblast.doski.ru': 'doski.ru',
 'kerch.doski.ru': 'doski.ru',
 'ivanovo.doski.ru': 'doski.ru',
 'bryansk.doski.ru': 'doski.ru',
 'ulyanovskaya-oblast.doski.ru': 'doski.ru',
 'ioshkar-ola.doski.ru': 'doski.ru',
 'orel.doski.ru': 'doski.ru',
 'mahachkala.doski.ru': 'doski.ru',
 'ryazan.doski.ru': 'doski.ru',
 'kabardino-balkariya.doski.ru': 'doski.ru',
 'stavropol.doski.ru': 'doski.ru',
 'tambov.doski.ru': 'doski.ru',
 'saransk.doski.ru': 'doski.ru',
 'smolensk.doski.ru': 'doski.ru',
 'chechnya.doski.ru': 'doski.ru',
 'alaniya.doski.ru': 'doski.ru',
 'simferopol.doski.ru': 'doski.ru',
 'maikop.doski.ru': 'doski.ru',
 'vladikavkaz.doski.ru': 'doski.ru',
 'ryazanskaya-oblast.doski.ru': 'doski.ru',
 'voronezhskaya-oblast.doski.ru': 'doski.ru',
 'ab.doski.ru': 'doski.ru',
 'arhangelskaya-oblast.doski.ru': 'doski.ru',
 'ivanovskaya-oblast.doski.ru': 'doski.ru',
 'minsk.doski.ru': 'doski.ru',
 'volgogradskaya-oblast.doski.ru': 'doski.ru',
 'kaluzhskaya-oblast.doski.ru': 'doski.ru',
 'blagoveschensk.doski.ru': 'doski.ru',
 'yakutsk.doski.ru': 'doski.ru',
 'kg.doski.ru': 'doski.ru',
 'chuvashiya.doski.ru': 'doski.ru',
 'gelendzhik.doski.ru': 'doski.ru',
 'petrozavodsk.doski.ru': 'doski.ru',
    'krasnodarskyy-kray.restate.ru': 'restate.ru',
 'izhevsk.restate.ru': 'restate.ru',
 'arhangelsk.restate.ru': 'restate.ru',
 'krasnodar.restate.ru': 'restate.ru',
 'saratov.restate.ru': 'restate.ru',
 'tatarstan.restate.ru': 'restate.ru',
 'kamchatskiy-kray.restate.ru': 'restate.ru',
 'restate.ru': 'restate.ru',
 'vladimirskaya-oblast.restate.ru': 'restate.ru',
 'nizhegorodskaya-oblast.restate.ru': 'restate.ru',
 'ulyanovsk.restate.ru': 'restate.ru',
 'krasnoyarsk.restate.ru': 'restate.ru',
 'tolyatti.restate.ru': 'restate.ru',
 'obninsk.restate.ru': 'restate.ru',
 'vladimir.restate.ru': 'restate.ru',
 'novorossiysk.restate.ru': 'restate.ru',
 'ryazan.restate.ru': 'restate.ru',
 'dagestan.restate.ru': 'restate.ru',
 'novosibirsk.restate.ru': 'restate.ru',
 'saha-yakutiya.restate.ru': 'restate.ru',
 'nizhnekamsk.restate.ru': 'restate.ru',
 'kazan.restate.ru': 'restate.ru',
 'sochi.restate.ru': 'restate.ru',
 'samara.restate.ru': 'restate.ru',
 'permskiy-kray.restate.ru': 'restate.ru',
 'perm.restate.ru': 'restate.ru',
 'habarovsk.restate.ru': 'restate.ru',
 'tyumen.restate.ru': 'restate.ru',
 'kaliningrad.restate.ru': 'restate.ru',
 'norilsk.restate.ru': 'restate.ru',
 'bryanskaya-oblast.restate.ru': 'restate.ru',
 'ufa.restate.ru': 'restate.ru',
 'feodosia.restate.ru': 'restate.ru',
 'krym.restate.ru': 'restate.ru',
 'taganrog.restate.ru': 'restate.ru',
 'severodvinsk.restate.ru': 'restate.ru',
 'arhangelskaya-oblast.restate.ru': 'restate.ru',
 'murmansk.restate.ru': 'restate.ru',
 'saransk.restate.ru': 'restate.ru',
 'surgut.restate.ru': 'restate.ru',
 'karachaevo-cherkesiya.restate.ru': 'restate.ru',
 'barnaul.restate.ru': 'restate.ru',
 'voronezh.restate.ru': 'restate.ru',
 'kurskaya-oblast.restate.ru': 'restate.ru',
 'komsomolsk-na-amure.restate.ru': 'restate.ru',
 'naberezhnye-chelny.restate.ru': 'restate.ru',
 'kurgan.restate.ru': 'restate.ru',
 'rostov-na-donu.restate.ru': 'restate.ru',
 'rostovskaya-oblast.restate.ru': 'restate.ru',
 'omsk.restate.ru': 'restate.ru',
 'omskaya-oblast.restate.ru': 'restate.ru',
 'cherepovets.restate.ru': 'restate.ru',
 'vladikavkaz.restate.ru': 'restate.ru',
 'orsk.restate.ru': 'restate.ru',
 'simferopol.restate.ru': 'restate.ru',
 'kislovodsk.restate.ru': 'restate.ru',
 'sverdlovskaya-oblast.restate.ru': 'restate.ru',
 'chelyabinsk.restate.ru': 'restate.ru',
 'pskovskaya-oblast.restate.ru': 'restate.ru',
 'smolensk.restate.ru': 'restate.ru',
 'yalta.restate.ru': 'restate.ru',
 'novokuznetsk.restate.ru': 'restate.ru',
 'tomsk.restate.ru': 'restate.ru',
 'tobolsk.restate.ru': 'restate.ru',
 'gelendzhik.restate.ru': 'restate.ru',
 'armavir.restate.ru': 'restate.ru',
 'ekaterinburg.restate.ru': 'restate.ru',
 'tula.restate.ru': 'restate.ru',
 'tulskaya-oblast.restate.ru': 'restate.ru',
 'pskov.restate.ru': 'restate.ru',
 'respublika-altay.restate.ru': 'restate.ru',
 'ivanovo.restate.ru': 'restate.ru',
 'kineshma.restate.ru': 'restate.ru',
 'ivanovskaya-oblast.restate.ru': 'restate.ru',
 'vladivostok.restate.ru': 'restate.ru',
 'kaluga.restate.ru': 'restate.ru',
 'novocherkassk.restate.ru': 'restate.ru',
 'cheboksary.restate.ru': 'restate.ru',
 'kaliningradskaya-oblast.restate.ru': 'restate.ru',
 'krasnoyarskiy-kray.restate.ru': 'restate.ru',
 'kemerovskaya-oblast.restate.ru': 'restate.ru',
 'chelyabinskaya-oblast.restate.ru': 'restate.ru',
 'buryatiya.restate.ru': 'restate.ru',
 'saratovskaya-oblast.restate.ru': 'restate.ru',
 'habarovskiy-kray.restate.ru': 'restate.ru',
 'kemerovo.restate.ru': 'restate.ru',
 'volgogradskaya-oblast.restate.ru': 'restate.ru',
 'kurganskaya-oblast.restate.ru': 'restate.ru',
 'syktyvkar.restate.ru': 'restate.ru',
 'novosibirskaya-oblast.restate.ru': 'restate.ru',
 'belgorod.restate.ru': 'restate.ru',
 'tyumenskaya-oblast.restate.ru': 'restate.ru',
 'kursk.restate.ru': 'restate.ru',
 'vologda.restate.ru': 'restate.ru',
 'tverskaya-oblast.restate.ru': 'restate.ru',
 'komi.restate.ru': 'restate.ru',
 'kaluzhskaya-oblast.restate.ru': 'restate.ru',
 'tomskaya-oblast.restate.ru': 'restate.ru',
 'mariy-el.restate.ru': 'restate.ru',
 'bashkortostan.restate.ru': 'restate.ru',
 'voronezhskaya-oblast.restate.ru': 'restate.ru',
 'tver.restate.ru': 'restate.ru',
 'udmurtiya.restate.ru': 'restate.ru',
 'irkutskaya-oblast.restate.ru': 'restate.ru',
 'vologodskaya-oblast.restate.ru': 'restate.ru',
 'stavropol.restate.ru': 'restate.ru',
 'samarskaya-oblast.restate.ru': 'restate.ru',
 'zabaykalskiy-kray.restate.ru': 'restate.ru',
 'altayskiy-kray.restate.ru': 'restate.ru',
 'stavropolskiy-kray.restate.ru': 'restate.ru',
 'kostromskaya-oblast.restate.ru': 'restate.ru',
 'penzenskaya-oblast.restate.ru': 'restate.ru',
 'amurskaya-oblast.restate.ru': 'restate.ru',
 'irkutsk.restate.ru': 'restate.ru',
 'chuvashiya.restate.ru': 'restate.ru',
 'eysk.restate.ru': 'restate.ru',
 'anapa.restate.ru': 'restate.ru',
 'smolenskaya-oblast.restate.ru': 'restate.ru',
 'velikiy-novgorod.restate.ru': 'restate.ru',
 'kareliya.restate.ru': 'restate.ru',
 'sortavala.restate.ru': 'restate.ru',
 'petrozavodsk.restate.ru': 'restate.ru',
 'sevastopol.restate.ru': 'restate.ru',
 'penza.restate.ru': 'restate.ru',
 'pyatigorsk.restate.ru': 'restate.ru',
 'yakutsk.restate.ru': 'restate.ru',
 'kerch.restate.ru': 'restate.ru',
 'nizhnevartovsk.restate.ru': 'restate.ru',
 'artem.restate.ru': 'restate.ru',
 'orenburg.restate.ru': 'restate.ru',
 'bratsk.restate.ru': 'restate.ru',
 'tambov.restate.ru': 'restate.ru',
 'yoshkar-ola.restate.ru': 'restate.ru',
 'nizhniy-tagil.restate.ru': 'restate.ru',
 'solikamsk.restate.ru': 'restate.ru',
 'staryy-oskol.restate.ru': 'restate.ru',
 'volgograd.restate.ru': 'restate.ru',
 'miass.restate.ru': 'restate.ru',
 'ulan-ude.restate.ru': 'restate.ru',
 'biysk.restate.ru': 'restate.ru',
 'sterlitamak.restate.ru': 'restate.ru',
 'novyy-urengoy.restate.ru': 'restate.ru',
 'pervouralsk.restate.ru': 'restate.ru',
 'angarsk.restate.ru': 'restate.ru',
 'kamensk-uralskiy.restate.ru': 'restate.ru',
 'seversk.restate.ru': 'restate.ru',
 'neftekamsk.restate.ru': 'restate.ru',
 'syzran.restate.ru': 'restate.ru',
 'volzhsk.restate.ru': 'restate.ru',
 'alushta.restate.ru': 'restate.ru',
 'zlatoust.restate.ru': 'restate.ru',
 'volgodonsk.restate.ru': 'restate.ru',
 'rzhev.restate.ru': 'restate.ru',
 'uhta.restate.ru': 'restate.ru',
 'salavat.restate.ru': 'restate.ru',
 'velikie-luki.restate.ru': 'restate.ru',
 'lipetsk.restate.ru': 'restate.ru',
    'chelyabinsk.compromesso.ru': 'compromesso.ru',
 'nahodka.compromesso.ru': 'compromesso.ru',
 'sterlitamak.compromesso.ru': 'compromesso.ru',
 'kaliningrad.compromesso.ru': 'compromesso.ru',
 'sanktpeterburg.compromesso.ru': 'compromesso.ru',
 'ekaterinburg.compromesso.ru': 'compromesso.ru',
 'samara.compromesso.ru': 'compromesso.ru',
 'neftekamsk.compromesso.ru': 'compromesso.ru',
 'bryansk.compromesso.ru': 'compromesso.ru',
 'krasnoyarsk.compromesso.ru': 'compromesso.ru',
 'novosibirsk.compromesso.ru': 'compromesso.ru',
 'moskva.compromesso.ru': 'compromesso.ru',
 'nizhnevartovsk.compromesso.ru': 'compromesso.ru',
 'voronezh.compromesso.ru': 'compromesso.ru',
 'krasnodar.compromesso.ru': 'compromesso.ru',
 'sochi.compromesso.ru': 'compromesso.ru',
 'novorossiysk.compromesso.ru': 'compromesso.ru',
 'nizhnii-tagil.compromesso.ru': 'compromesso.ru',
 'omsk.compromesso.ru': 'compromesso.ru',
 'ulyanovsk.compromesso.ru': 'compromesso.ru',
 'orenburg.compromesso.ru': 'compromesso.ru',
 'barnaul.compromesso.ru': 'compromesso.ru',
 'khabarovsk.compromesso.ru': 'compromesso.ru',
 'rostov-na-donu.compromesso.ru': 'compromesso.ru',
 'kazan.compromesso.ru': 'compromesso.ru',
 'naberezhnye-chelny.compromesso.ru': 'compromesso.ru',
 'sergiev-posad.compromesso.ru': 'compromesso.ru',
 'simferopol.compromesso.ru': 'compromesso.ru',
 'kirov.compromesso.ru': 'compromesso.ru',
 'tver.compromesso.ru': 'compromesso.ru',
 'almetevsk.compromesso.ru': 'compromesso.ru',
 'petrozavodsk.compromesso.ru': 'compromesso.ru',
 'surgut.compromesso.ru': 'compromesso.ru',
 'balashiha.compromesso.ru': 'compromesso.ru',
 'izhevsk.compromesso.ru': 'compromesso.ru',
 'stary-oskol.compromesso.ru': 'compromesso.ru',
 'cheboksary.compromesso.ru': 'compromesso.ru',
 'velikie-luki.compromesso.ru': 'compromesso.ru',
 'nizhniy-novgorod.compromesso.ru': 'compromesso.ru',
 'saratov.compromesso.ru': 'compromesso.ru',
 'tyumen.compromesso.ru': 'compromesso.ru',
 'fryazino.compromesso.ru': 'compromesso.ru',
 'ufa.compromesso.ru': 'compromesso.ru',
 'volgodonsk.compromesso.ru': 'compromesso.ru',
 'gelendzhik.compromesso.ru': 'compromesso.ru',
 'stavropol.compromesso.ru': 'compromesso.ru',
 'ussuriysk.compromesso.ru': 'compromesso.ru',
 'tambov.compromesso.ru': 'compromesso.ru',
 'kursk.compromesso.ru': 'compromesso.ru',
 'adler.compromesso.ru': 'compromesso.ru',
 'yalta.compromesso.ru': 'compromesso.ru',
 'tolyatti.compromesso.ru': 'compromesso.ru',
 'perm.compromesso.ru': 'compromesso.ru',
 'anapa.compromesso.ru': 'compromesso.ru',
 'abakan.compromesso.ru': 'compromesso.ru',
 'vologda.compromesso.ru': 'compromesso.ru',
 'angarsk.compromesso.ru': 'compromesso.ru',
 'kurgan.compromesso.ru': 'compromesso.ru',
 'groznyy.compromesso.ru': 'compromesso.ru',
 'pskov.compromesso.ru': 'compromesso.ru',
 'murmansk.compromesso.ru': 'compromesso.ru',
 'blagoveshensk.compromesso.ru': 'compromesso.ru',
 'chehov.compromesso.ru': 'compromesso.ru',
 'hanty-mansiysk.compromesso.ru': 'compromesso.ru',
 'maykop.compromesso.ru': 'compromesso.ru',
 'vladimir.compromesso.ru': 'compromesso.ru',
 'kovrov.compromesso.ru': 'compromesso.ru',
 'vladikavkaz.compromesso.ru': 'compromesso.ru',
 'smolensk.compromesso.ru': 'compromesso.ru',
 'stupino.compromesso.ru': 'compromesso.ru',
 'kostroma.compromesso.ru': 'compromesso.ru',
 'pervouralsk.compromesso.ru': 'compromesso.ru',
 'vladivostok.compromesso.ru': 'compromesso.ru',
 'volgograd.compromesso.ru': 'compromesso.ru',
 'nefteyugansk.compromesso.ru': 'compromesso.ru',
 'magnitogorsk.compromesso.ru': 'compromesso.ru',
 'serpuhov.compromesso.ru': 'compromesso.ru',
 'korolev.compromesso.ru': 'compromesso.ru',
 'syktyvkar.compromesso.ru': 'compromesso.ru',
 'salavat.compromesso.ru': 'compromesso.ru',
 'odincovo.compromesso.ru': 'compromesso.ru',
 'tomsk.compromesso.ru': 'compromesso.ru',
 'syzran.compromesso.ru': 'compromesso.ru',
 'pyatigorsk.compromesso.ru': 'compromesso.ru',
 'serov.compromesso.ru': 'compromesso.ru',
 'ryazan.compromesso.ru': 'compromesso.ru',
 'compromesso.ru': 'compromesso.ru',
 'yoshkar-ola.compromesso.ru': 'compromesso.ru',
 'yaroslavl.compromesso.ru': 'compromesso.ru',
 'norilsk.compromesso.ru': 'compromesso.ru',
 'balakovo.compromesso.ru': 'compromesso.ru',
 'yuzhno-sahalinsk.compromesso.ru': 'compromesso.ru',
 'novokuznetsk.compromesso.ru': 'compromesso.ru',
 'engels.compromesso.ru': 'compromesso.ru',
 'tula.compromesso.ru': 'compromesso.ru',
 'komsomolsk-na-amure.compromesso.ru': 'compromesso.ru',
 'eysk.compromesso.ru': 'compromesso.ru',
 'podolsk.compromesso.ru': 'compromesso.ru',
 'nalchik.compromesso.ru': 'compromesso.ru',
 'pushkino.compromesso.ru': 'compromesso.ru',
 'nevinnomyssk.compromesso.ru': 'compromesso.ru',
 'ramenskoe.compromesso.ru': 'compromesso.ru',
 'lyubercy.compromesso.ru': 'compromesso.ru',
 'elektrostal.compromesso.ru': 'compromesso.ru',
 'kaluga.compromesso.ru': 'compromesso.ru',
 'tobolsk.compromesso.ru': 'compromesso.ru',
 'ivanovo.compromesso.ru': 'compromesso.ru',
 'nizhnekamsk.compromesso.ru': 'compromesso.ru',
 'orel.compromesso.ru': 'compromesso.ru',
 'mahachkala.compromesso.ru': 'compromesso.ru',
 'arzamas.compromesso.ru': 'compromesso.ru',
 'arkhangelsk.compromesso.ru': 'compromesso.ru',
 'gorno-altaysk.compromesso.ru': 'compromesso.ru',
 'cherepovec.compromesso.ru': 'compromesso.ru',
 'zelenograd.compromesso.ru': 'compromesso.ru',
 'astrakhan.compromesso.ru': 'compromesso.ru',
 'irkutsk.compromesso.ru': 'compromesso.ru',
 'velikiynovgorod.compromesso.ru': 'compromesso.ru',
 'gatchina.compromesso.ru': 'compromesso.ru',
 'zheleznodorozhnyy.compromesso.ru': 'compromesso.ru',
 'egorevsk.compromesso.ru': 'compromesso.ru',
 'essentuki.compromesso.ru': 'compromesso.ru',
 'berezniki.compromesso.ru': 'compromesso.ru',
 'bratsk.compromesso.ru': 'compromesso.ru',
 'zheleznogorsk.compromesso.ru': 'compromesso.ru',
 'vyborg.compromesso.ru': 'compromesso.ru',
 'vorkuta.compromesso.ru': 'compromesso.ru',
 'glazov.compromesso.ru': 'compromesso.ru',
 'dzerzhinsk.compromesso.ru': 'compromesso.ru',
 'volzhskiy.compromesso.ru': 'compromesso.ru',
 'elets.compromesso.ru': 'compromesso.ru',
 'dubna.compromesso.ru': 'compromesso.ru',
 'dzerzhinskiy.compromesso.ru': 'compromesso.ru',
 'orsk.compromesso.ru': 'compromesso.ru',
 'novyy-urengoy.compromesso.ru': 'compromesso.ru',
 'biysk.compromesso.ru': 'compromesso.ru',
 'domodedovo.compromesso.ru': 'compromesso.ru',
 'zhukovskiy.compromesso.ru': 'compromesso.ru',
 'vidnoe.compromesso.ru': 'compromesso.ru',
 'dimitrovgrad.compromesso.ru': 'compromesso.ru',
 'armavir.compromesso.ru': 'compromesso.ru',
 'belgorod.compromesso.ru': 'compromesso.ru',
 'dolgoprudnyi.compromesso.ru': 'compromesso.ru',
 'birobidjan.compromesso.ru': 'compromesso.ru',
 'dmitrov.compromesso.ru': 'compromesso.ru',
 'achinsk.compromesso.ru': 'compromesso.ru',
 'berdsk.compromesso.ru': 'compromesso.ru',
 'kemerovo.compromesso.ru': 'compromesso.ru',
 'shakhty.compromesso.ru': 'compromesso.ru',
 'novomoskovsk.compromesso.ru': 'compromesso.ru',
 'kamensk-uralskiy.compromesso.ru': 'compromesso.ru',
 'penza.compromesso.ru': 'compromesso.ru',
 'miass.compromesso.ru': 'compromesso.ru',
 'rybinsk.compromesso.ru': 'compromesso.ru',
 'ulan-ude.compromesso.ru': 'compromesso.ru',
 'yakutsk.compromesso.ru': 'compromesso.ru',
 'klin.compromesso.ru': 'compromesso.ru',
 'lipetsk.compromesso.ru': 'compromesso.ru',
 'taganrog.compromesso.ru': 'compromesso.ru',
 'kolomna.compromesso.ru': 'compromesso.ru',
 'krasnogorsk.compromesso.ru': 'compromesso.ru',
    'nabchelny.ktogdeest.com': 'ktogdeest.com',
 'ktogdeest.com': 'ktogdeest.com',
 'cherepovets.ktogdeest.com': 'ktogdeest.com',
 'podolsk.ktogdeest.com': 'ktogdeest.com',
 'rostov-nd.ktogdeest.com': 'ktogdeest.com',
 'novoross.ktogdeest.com': 'ktogdeest.com',
 'spb.ktogdeest.com': 'ktogdeest.com',
 'kostroma.ktogdeest.com': 'ktogdeest.com',
 'korolev.ktogdeest.com': 'ktogdeest.com',
 'murmansk.ktogdeest.com': 'ktogdeest.com',
 'orzuevo.ktogdeest.com': 'ktogdeest.com',
 'solngorsk.ktogdeest.com': 'ktogdeest.com',
 'cheboksary.ktogdeest.com': 'ktogdeest.com',
 'sochi.ktogdeest.com': 'ktogdeest.com',
 'pskov.ktogdeest.com': 'ktogdeest.com',
 'moskva.ktogdeest.com': 'ktogdeest.com',
 'ufa.ktogdeest.com': 'ktogdeest.com',
 'kazan.ktogdeest.com': 'ktogdeest.com',
 'serov.ktogdeest.com': 'ktogdeest.com',
 'khabarovsk.ktogdeest.com': 'ktogdeest.com',
 'gelendzhik.ktogdeest.com': 'ktogdeest.com',
 'zelenogolsk.ktogdeest.com': 'ktogdeest.com',
 'mytitshy.ktogdeest.com': 'ktogdeest.com',
 'gomel.ktogdeest.com': 'ktogdeest.com',
 'nnovgorod.ktogdeest.com': 'ktogdeest.com',
 'novosibirsk.ktogdeest.com': 'ktogdeest.com',
 'aksay.ktogdeest.com': 'ktogdeest.com',
 'odintsovo.ktogdeest.com': 'ktogdeest.com',
 'kiev.ktogdeest.com': 'ktogdeest.com',
 'zelenograd.ktogdeest.com': 'ktogdeest.com',
 'lipetsk.ktogdeest.com': 'ktogdeest.com',
 'vladivostok.ktogdeest.com': 'ktogdeest.com',
 'volgograd.ktogdeest.com': 'ktogdeest.com',
 'shakhty.ktogdeest.com': 'ktogdeest.com',
 'eburg.ktogdeest.com': 'ktogdeest.com',
 'krasnodar.ktogdeest.com': 'ktogdeest.com',
 'novokuznetsk.ktogdeest.com': 'ktogdeest.com',
 'tshelkovo.ktogdeest.com': 'ktogdeest.com',
 'roslavl.ktogdeest.com': 'ktogdeest.com',
 'kaliningrad.ktogdeest.com': 'ktogdeest.com',
 'kovrov.ktogdeest.com': 'ktogdeest.com',
 'syktyvkar.ktogdeest.com': 'ktogdeest.com',
 'neftekamsk.ktogdeest.com': 'ktogdeest.com',
 'nalchik.ktogdeest.com': 'ktogdeest.com',
 'grozny.ktogdeest.com': 'ktogdeest.com',
 'borisoglebsk.ktogdeest.com': 'ktogdeest.com',
 'zhukovsky.ktogdeest.com': 'ktogdeest.com',
 'saratov.ktogdeest.com': 'ktogdeest.com',
 'pervouralsk.ktogdeest.com': 'ktogdeest.com',
 'ulyanovsk.ktogdeest.com': 'ktogdeest.com',
 'perm.ktogdeest.com': 'ktogdeest.com',
 'voronezh.ktogdeest.com': 'ktogdeest.com',
 'kamyshin.ktogdeest.com': 'ktogdeest.com',
 'buzuluk.ktogdeest.com': 'ktogdeest.com',
 'electrostal.ktogdeest.com': 'ktogdeest.com',
 'ruzaevka.ktogdeest.com': 'ktogdeest.com',
 'pushkino.ktogdeest.com': 'ktogdeest.com',
 'tula.ktogdeest.com': 'ktogdeest.com',
 'magadan.ktogdeest.com': 'ktogdeest.com',
 'novocher.ktogdeest.com': 'ktogdeest.com',
 'omsk.ktogdeest.com': 'ktogdeest.com',
 'berezniki.ktogdeest.com': 'ktogdeest.com',
 'timashvsk.ktogdeest.com': 'ktogdeest.com',
 'karpinsk.ktogdeest.com': 'ktogdeest.com',
 'abakan.ktogdeest.com': 'ktogdeest.com',
 'kurgan.ktogdeest.com': 'ktogdeest.com',
 'goryachiy-klyuch.ktogdeest.com': 'ktogdeest.com',
 'rostov.ktogdeest.com': 'ktogdeest.com',
 'lyubertsy.ktogdeest.com': 'ktogdeest.com',
 'kirov.ktogdeest.com': 'ktogdeest.com',
 'barnaul.ktogdeest.com': 'ktogdeest.com',
 'rossosh.ktogdeest.com': 'ktogdeest.com',
 'khimki.ktogdeest.com': 'ktogdeest.com',
 'tobolsk.ktogdeest.com': 'ktogdeest.com',
 'astrakhan.ktogdeest.com': 'ktogdeest.com',
 'zhelezn.ktogdeest.com': 'ktogdeest.com',
 'yaroslavl.ktogdeest.com': 'ktogdeest.com',
 'domodedovo.ktogdeest.com': 'ktogdeest.com',
 'orenburg.ktogdeest.com': 'ktogdeest.com',
 'akhtubinsk.ktogdeest.com': 'ktogdeest.com',
 'miass.ktogdeest.com': 'ktogdeest.com',
 'shadrinsk.ktogdeest.com': 'ktogdeest.com',
 'balashikha.ktogdeest.com': 'ktogdeest.com',
 'balabanovo.ktogdeest.com': 'ktogdeest.com',
 'chelyabinsk.ktogdeest.com': 'ktogdeest.com',
 'tver.ktogdeest.com': 'ktogdeest.com',
 'pyatigorsk.ktogdeest.com': 'ktogdeest.com',
 'temryuk.ktogdeest.com': 'ktogdeest.com',
 'mikhaylovsk26.ktogdeest.com': 'ktogdeest.com',
 'slavyansk-na-kubani.ktogdeest.com': 'ktogdeest.com',
 'povorino.ktogdeest.com': 'ktogdeest.com',
 'naro-fominsk.ktogdeest.com': 'ktogdeest.com',
 'krasnoyarsk.ktogdeest.com': 'ktogdeest.com',
 'tyumen.ktogdeest.com': 'ktogdeest.com',
 'labinsk.ktogdeest.com': 'ktogdeest.com',
 'tolyatti.ktogdeest.com': 'ktogdeest.com',
 'chekhov.ktogdeest.com': 'ktogdeest.com',
 'petrozavodsk.ktogdeest.com': 'ktogdeest.com',
 'fokino25.ktogdeest.com': 'ktogdeest.com',
 'usinsk.ktogdeest.com': 'ktogdeest.com',
 'velluki.ktogdeest.com': 'ktogdeest.com',
 'shatura.ktogdeest.com': 'ktogdeest.com',
 'vologda.ktogdeest.com': 'ktogdeest.com',
 'kirovochep.ktogdeest.com': 'ktogdeest.com',
 'izhevsk.ktogdeest.com': 'ktogdeest.com',
 'stupino.ktogdeest.com': 'ktogdeest.com',
 'poltava.ktogdeest.com': 'ktogdeest.com',
 'surgut.ktogdeest.com': 'ktogdeest.com',
 'alexandrov.ktogdeest.com': 'ktogdeest.com',
 'stavropol.ktogdeest.com': 'ktogdeest.com',
 'nefteyugansk.ktogdeest.com': 'ktogdeest.com',
 'dolgoprudny.ktogdeest.com': 'ktogdeest.com',
 'maykop.ktogdeest.com': 'ktogdeest.com',
 'tuapse.ktogdeest.com': 'ktogdeest.com',
 'volgodonsk.ktogdeest.com': 'ktogdeest.com',
 'zernograd.ktogdeest.com': 'ktogdeest.com',
 'vladimir.ktogdeest.com': 'ktogdeest.com',
 'magnitogorsk.ktogdeest.com': 'ktogdeest.com',
 'volzhsky.ktogdeest.com': 'ktogdeest.com',
 'vnovgorod.ktogdeest.com': 'ktogdeest.com',
 'kopeysk.ktogdeest.com': 'ktogdeest.com',
 'kamshaht.ktogdeest.com': 'ktogdeest.com',
 'kirzhach.ktogdeest.com': 'ktogdeest.com',
 'rybinsk.ktogdeest.com': 'ktogdeest.com',
 'chapaevsk.ktogdeest.com': 'ktogdeest.com',
 'moskovskiy.ktogdeest.com': 'ktogdeest.com',
 'ramenskoe.ktogdeest.com': 'ktogdeest.com',
 'kerch.ktogdeest.com': 'ktogdeest.com',
 'nizhnekamsk.ktogdeest.com': 'ktogdeest.com',
 'smolensk.ktogdeest.com': 'ktogdeest.com',
 'samara.ktogdeest.com': 'ktogdeest.com',
 'balakovo.ktogdeest.com': 'ktogdeest.com',
 'ntagil.ktogdeest.com': 'ktogdeest.com',
 'gatchina.ktogdeest.com': 'ktogdeest.com',
 'pereslzal.ktogdeest.com': 'ktogdeest.com',
 'bryansk.ktogdeest.com': 'ktogdeest.com',
 'novoaleksandrovsk.ktogdeest.com': 'ktogdeest.com',
 'khotkovo.ktogdeest.com': 'ktogdeest.com',
 'kuvandyk.ktogdeest.com': 'ktogdeest.com',
 'syzran.ktogdeest.com': 'ktogdeest.com',
 'kamyshlov.ktogdeest.com': 'ktogdeest.com',
 'simferopol.ktogdeest.com': 'ktogdeest.com',
 'sterlitamak.ktogdeest.com': 'ktogdeest.com',
 'taganrog.ktogdeest.com': 'ktogdeest.com',
 'novocheboksarsk.ktogdeest.com': 'ktogdeest.com',
 'zvenigorod.ktogdeest.com': 'ktogdeest.com',
 'kherson.ktogdeest.com': 'ktogdeest.com',
 'belorechensk.ktogdeest.com': 'ktogdeest.com',
 'zaraysk.ktogdeest.com': 'ktogdeest.com',
 'belgorod.ktogdeest.com': 'ktogdeest.com',
 'ryazan.ktogdeest.com': 'ktogdeest.com',
 'lesosibirsk.ktogdeest.com': 'ktogdeest.com',
 'kislovodsk.ktogdeest.com': 'ktogdeest.com',
 'neftegorsk.ktogdeest.com': 'ktogdeest.com',
 'yalta.ktogdeest.com': 'ktogdeest.com',
 'krasnogorsk.ktogdeest.com': 'ktogdeest.com',
 'nevinms.ktogdeest.com': 'ktogdeest.com',
 'birobidzhan.ktogdeest.com': 'ktogdeest.com',
 'orsk.ktogdeest.com': 'ktogdeest.com',
 'kursk.ktogdeest.com': 'ktogdeest.com',
 'zelenokumsk.ktogdeest.com': 'ktogdeest.com',
 'yanaul.ktogdeest.com': 'ktogdeest.com',
 'kandalaksha.ktogdeest.com': 'ktogdeest.com',
 'buguruslan.ktogdeest.com': 'ktogdeest.com',
    'promportal.su': 'promportal.su',
 'armenia.promportal.su': 'promportal.su',
 'moscow.promportal.su': 'promportal.su',
 'adler.promportal.su': 'promportal.su',
 'millerovo.promportal.su': 'promportal.su',
 'krasnodar.promportal.su': 'promportal.su',
 'tambov.promportal.su': 'promportal.su',
 'tuymazi.promportal.su': 'promportal.su',
 'kaluga.promportal.su': 'promportal.su',
 'kiev.promportal.su': 'promportal.su',
 'tuapse.promportal.su': 'promportal.su',
 'sankt-peterburg.promportal.su': 'promportal.su',
 'kirov.promportal.su': 'promportal.su',
 'sochi.promportal.su': 'promportal.su',
 'ulyanovsk.promportal.su': 'promportal.su',
 'zheleznodorozhniy.promportal.su': 'promportal.su',
 'gruziya.promportal.su': 'promportal.su',
 'rostov-na-donu.promportal.su': 'promportal.su',
 'yamalo-neneckiy-avtonomniy-okrug.promportal.su': 'promportal.su',
 'orel.promportal.su': 'promportal.su',
 'izrail.promportal.su': 'promportal.su',
 'polevskoy.promportal.su': 'promportal.su',
 'respublika-dagestan.promportal.su': 'promportal.su',
 'krasnodarskiy-kray.promportal.su': 'promportal.su',
 'altayskiy-kray.promportal.su': 'promportal.su',
 'kazan.promportal.su': 'promportal.su',
 'lenobl.promportal.su': 'promportal.su',
 'pushkino.promportal.su': 'promportal.su',
 'naro-fominsk.promportal.su': 'promportal.su',
 'kubinka.promportal.su': 'promportal.su',
 'vladivostok.promportal.su': 'promportal.su',
 'murmanskaya-oblast.promportal.su': 'promportal.su',
 'omsk.promportal.su': 'promportal.su',
 'yakutsk.promportal.su': 'promportal.su',
 'dagomys.promportal.su': 'promportal.su',
 'voronezhskaya-oblast.promportal.su': 'promportal.su',
 'slavyansk-na-kubani.promportal.su': 'promportal.su',
 'zheleznogorsk.promportal.su': 'promportal.su',
 'chelyabinsk.promportal.su': 'promportal.su',
 'pskov.promportal.su': 'promportal.su',
 'goryachiy-klyuch.promportal.su': 'promportal.su',
 'habarovsk.promportal.su': 'promportal.su',
 'samara.promportal.su': 'promportal.su',
 'voskresensk.promportal.su': 'promportal.su',
 'novosibirsk.promportal.su': 'promportal.su',
 'tyumen.promportal.su': 'promportal.su',
 'saratovskaya-oblast.promportal.su': 'promportal.su',
 'blagoveschensk.promportal.su': 'promportal.su',
 'pskovskaya-oblast.promportal.su': 'promportal.su',
 'maloyaroslavec.promportal.su': 'promportal.su',
 'ekaterinburg.promportal.su': 'promportal.su',
 'leninogorsk.promportal.su': 'promportal.su',
 'volgograd.promportal.su': 'promportal.su',
 'borisoglebsk.promportal.su': 'promportal.su',
 'kolomna.promportal.su': 'promportal.su',
 'tomsk.promportal.su': 'promportal.su',
 'buyer.promportal.su': 'promportal.su',
 'glazov.promportal.su': 'promportal.su',
 'perm.promportal.su': 'promportal.su',
 'elektrougli.promportal.su': 'promportal.su',
 'borovichi.promportal.su': 'promportal.su',
 'ryazan.promportal.su': 'promportal.su',
 'china.promportal.su': 'promportal.su',
 'ufa.promportal.su': 'promportal.su',
 'belebey.promportal.su': 'promportal.su',
 'kanevskaya.promportal.su': 'promportal.su',
 'shatura.promportal.su': 'promportal.su',
 'kaluzhskaya-oblast.promportal.su': 'promportal.su',
 'gulkevichi.promportal.su': 'promportal.su',
 'ivanovo.promportal.su': 'promportal.su',
 'erevan.promportal.su': 'promportal.su',
 'vladimir.promportal.su': 'promportal.su',
 'orsk.promportal.su': 'promportal.su',
 'kashira.promportal.su': 'promportal.su',
 'volhov.promportal.su': 'promportal.su',
 'primorskiy-kray.promportal.su': 'promportal.su',
 'turkey.promportal.su': 'promportal.su',
 'klin.promportal.su': 'promportal.su',
 'luhovici.promportal.su': 'promportal.su',
 'kimry.promportal.su': 'promportal.su',
 'luga.promportal.su': 'promportal.su',
 'kurgan.promportal.su': 'promportal.su',
 'murom.promportal.su': 'promportal.su',
 'pervouralsk.promportal.su': 'promportal.su',
 'barnaul.promportal.su': 'promportal.su',
 'volgogradskaya-oblast.promportal.su': 'promportal.su',
 'reutov.promportal.su': 'promportal.su',
 'shadrinsk.promportal.su': 'promportal.su',
 'prohladniy.promportal.su': 'promportal.su',
 'neftekamsk.promportal.su': 'promportal.su',
 'ramenskoe.promportal.su': 'promportal.su',
 'aramil.promportal.su': 'promportal.su',
 'elec.promportal.su': 'promportal.su',
 'izhevsk.promportal.su': 'promportal.su',
 'mo.promportal.su': 'promportal.su',
 'yaroslavl.promportal.su': 'promportal.su',
 'bryanskaya-oblast.promportal.su': 'promportal.su',
 'biysk.promportal.su': 'promportal.su',
 'belgorod.promportal.su': 'promportal.su',
 'kaliningrad.promportal.su': 'promportal.su',
 'st-varenikovskaya.promportal.su': 'promportal.su',
 'kursk.promportal.su': 'promportal.su',
 'belogorsk.promportal.su': 'promportal.su',
 'armavir.promportal.su': 'promportal.su',
 'kanash.promportal.su': 'promportal.su',
 'irkutsk.promportal.su': 'promportal.su',
 'sverdlovskaya-oblast.promportal.su': 'promportal.su',
 'pereslavl-zalesskiy.promportal.su': 'promportal.su',
 'yoshkar-ola.promportal.su': 'promportal.su',
 'aleksin.promportal.su': 'promportal.su',
 'tolyatti.promportal.su': 'promportal.su',
 'saratov.promportal.su': 'promportal.su',
 'arzamas.promportal.su': 'promportal.su',
 'hanti-mansiysk.promportal.su': 'promportal.su',
 'sverdlovsk.promportal.su': 'promportal.su',
 'simferopol.promportal.su': 'promportal.su',
 'nizhniy-novgorod.promportal.su': 'promportal.su',
 'voronezh.promportal.su': 'promportal.su',
 'berdsk.promportal.su': 'promportal.su',
 'irbit.promportal.su': 'promportal.su',
 'nahodka.promportal.su': 'promportal.su',
 'smolensk.promportal.su': 'promportal.su',
 'saransk.promportal.su': 'promportal.su',
 'ust-labinsk.promportal.su': 'promportal.su',
 'engels.promportal.su': 'promportal.su',
 'meleuz.promportal.su': 'promportal.su',
 'penza.promportal.su': 'promportal.su',
 'temryuk.promportal.su': 'promportal.su',
 'krasnoyarsk.promportal.su': 'promportal.su',
 'south-korea.promportal.su': 'promportal.su',
 'cheboksari.promportal.su': 'promportal.su',
 'gus-hrustalniy.promportal.su': 'promportal.su',
 'orenburg.promportal.su': 'promportal.su',
 'ussuriysk.promportal.su': 'promportal.su',
 'poland.promportal.su': 'promportal.su',
 'novokuzneck.promportal.su': 'promportal.su',
 'bryansk.promportal.su': 'promportal.su',
 'kemerovo.promportal.su': 'promportal.su',
 'murmansk.promportal.su': 'promportal.su',
 'belorussiya.promportal.su': 'promportal.su',
 'belovo.promportal.su': 'promportal.su',
 'aleksandrov.promportal.su': 'promportal.su',
 'taganrog.promportal.su': 'promportal.su',
 'indiya.promportal.su': 'promportal.su',
 'furmanov.promportal.su': 'promportal.su',
 'dzerzhinsk.promportal.su': 'promportal.su',
 'sevastopol.promportal.su': 'promportal.su',
 'timashevsk.promportal.su': 'promportal.su',
 'salsk.promportal.su': 'promportal.su',
 'nahabino.promportal.su': 'promportal.su',
 'permskaya-oblast.promportal.su': 'promportal.su',
 'strunino.promportal.su': 'promportal.su',
 'achinsk.promportal.su': 'promportal.su',
 'petushki.promportal.su': 'promportal.su',
 'kabardino-balkarskaya-respublika.promportal.su': 'promportal.su',
 'rzhev.promportal.su': 'promportal.su',
 'sterlitamak.promportal.su': 'promportal.su',
 'siktivkar.promportal.su': 'promportal.su',
 'sayanogorsk.promportal.su': 'promportal.su',
 'balashiha.promportal.su': 'promportal.su',
 'nizhnevartovsk.promportal.su': 'promportal.su',
 'spain.promportal.su': 'promportal.su',
 'vyazniki.promportal.su': 'promportal.su',
 'lipeck.promportal.su': 'promportal.su',
 'rostov.promportal.su': 'promportal.su',
 'novocherkassk.promportal.su': 'promportal.su',
 'oktyabrskiy.promportal.su': 'promportal.su',
 'iran.promportal.su': 'promportal.su',
 'shahti.promportal.su': 'promportal.su',
 'himki.promportal.su': 'promportal.su',
 'kovrov.promportal.su': 'promportal.su',
 'lomonosov.promportal.su': 'promportal.su',
 'kirzhach.promportal.su': 'promportal.su',
 'bronnici.promportal.su': 'promportal.su',
 'klinci.promportal.su': 'promportal.su',
 'noviy-urengoy.promportal.su': 'promportal.su',
    'vladivostok.irr.ru': 'irr.ru',
 'tula.irr.ru': 'irr.ru',
 'russia.irr.ru': 'irr.ru',
 'yamalo-nenetskiy-ao.irr.ru': 'irr.ru',
 'sverdlovskaya-obl.irr.ru': 'irr.ru',
 'orenburgskaya-obl.irr.ru': 'irr.ru',
 'krasnodarskiy-kray.irr.ru': 'irr.ru',
 'irr.ru': 'irr.ru',
 'nizhniynovgorod.irr.ru': 'irr.ru',
 'orenburg.irr.ru': 'irr.ru',
 'ryazanskaya-obl.irr.ru': 'irr.ru',
 'omsk.irr.ru': 'irr.ru',
 'perm.irr.ru': 'irr.ru',
 'chelyabinskaya-obl.irr.ru': 'irr.ru',
 'kazan.irr.ru': 'irr.ru',
 'lipetskaya-obl.irr.ru': 'irr.ru',
 'sterlitamak.irr.ru': 'irr.ru',
 'ufa.irr.ru': 'irr.ru',
 'blog.irr.ru': 'irr.ru',
 'krasnodar.irr.ru': 'irr.ru',
 'primorskiy-kray.irr.ru': 'irr.ru',
 'yugra-ao.irr.ru': 'irr.ru',
 'nahodka.irr.ru': 'irr.ru',
 'chuvashskaya-resp.irr.ru': 'irr.ru',
 'belgorod.irr.ru': 'irr.ru',
 'rybinsk.irr.ru': 'irr.ru',
 'chelyabinsk.irr.ru': 'irr.ru',
 'kirov.irr.ru': 'irr.ru',
 'belgorodskaya-obl.irr.ru': 'irr.ru',
 'sochi.irr.ru': 'irr.ru',
 'izhevsk.irr.ru': 'irr.ru',
 'nizhegorodskaya-obl.irr.ru': 'irr.ru',
 'ekaterinburg.irr.ru': 'irr.ru',
 'voronezh.irr.ru': 'irr.ru',
 'tatarstan-resp.irr.ru': 'irr.ru',
 'vladimirskaya-obl.irr.ru': 'irr.ru',
 'volgogradskaya-obl.irr.ru': 'irr.ru',
 'kolomna.irr.ru': 'irr.ru',
 'adygeya-resp.irr.ru': 'irr.ru',
 'kaliningradskaya-obl.irr.ru': 'irr.ru',
 'ryazan.irr.ru': 'irr.ru',
 'novosibirskaya-obl.irr.ru': 'irr.ru',
 'rostovnadonu.irr.ru': 'irr.ru',
 'kemerovo.irr.ru': 'irr.ru',
 'saratovskaya-obl.irr.ru': 'irr.ru',
 'bryanskaya-obl.irr.ru': 'irr.ru',
 'nabchelny.irr.ru': 'irr.ru',
 'ulanude.irr.ru': 'irr.ru',
 'kaliningrad.irr.ru': 'irr.ru',
 'novorossiysk.irr.ru': 'irr.ru',
 'permskiy-kray.irr.ru': 'irr.ru',
 'barnaul.irr.ru': 'irr.ru',
 'ivanovskaya-obl.irr.ru': 'irr.ru',
 'tyumen.irr.ru': 'irr.ru',
 'samara.irr.ru': 'irr.ru',
 'rostovskaya-obl.irr.ru': 'irr.ru',
 'tambovskaya-obl.irr.ru': 'irr.ru',
 'bashkortostan-resp.irr.ru': 'irr.ru',
 'kursk.irr.ru': 'irr.ru',
 'smolenskaya-obl.irr.ru': 'irr.ru',
 'smolensk.irr.ru': 'irr.ru',
 'sevastopol.irr.ru': 'irr.ru',
 'tyumenskaya-obl.irr.ru': 'irr.ru',
 'omskaya-obl.irr.ru': 'irr.ru',
 'orel.irr.ru': 'irr.ru',
 'novosibirsk.irr.ru': 'irr.ru',
 'blagoveschensk.irr.ru': 'irr.ru',
 'udmurtskaya-resp.irr.ru': 'irr.ru',
 'tolyatti.irr.ru': 'irr.ru',
 'norilsk.irr.ru': 'irr.ru',
 'khabarovsk.irr.ru': 'irr.ru',
 'saint-petersburg.irr.ru': 'irr.ru',
 'kurskaya-obl.irr.ru': 'irr.ru',
 'altayskiy-kray.irr.ru': 'irr.ru',
 'biysk.irr.ru': 'irr.ru',
 'ussuriysk.irr.ru': 'irr.ru',
 'ingushetiya-resp.irr.ru': 'irr.ru',
 'samarskaya-obl.irr.ru': 'irr.ru',
 'petrozavodsk.irr.ru': 'irr.ru',
 'nalchik.irr.ru': 'irr.ru',
 'arzamas.irr.ru': 'irr.ru',
 'pskov.irr.ru': 'irr.ru',
 'lipetsk.irr.ru': 'irr.ru',
 'nizhnekamsk.irr.ru': 'irr.ru',
 'vladimir.irr.ru': 'irr.ru',
 'velnovgorod.irr.ru': 'irr.ru',
 'cherepovets.irr.ru': 'irr.ru',
 'kirovskaya-obl.irr.ru': 'irr.ru',
 'kurganskaya-obl.irr.ru': 'irr.ru',
 'krasnoyarsk.irr.ru': 'irr.ru',
 'stavropolskiy-kray.irr.ru': 'irr.ru',
 'komsomolsk.irr.ru': 'irr.ru',
 'habarovskiy-kray.irr.ru': 'irr.ru',
 'novgorodskaya-obl.irr.ru': 'irr.ru',
 'gelendzhik.irr.ru': 'irr.ru',
 'arhangelskaya-obl.irr.ru': 'irr.ru',
 'kaluga.irr.ru': 'irr.ru',
 'alaniya-resp.irr.ru': 'irr.ru',
 'yaroslavl.irr.ru': 'irr.ru',
 'vladikavkaz.irr.ru': 'irr.ru',
 'yaroslavskaya-obl.irr.ru': 'irr.ru',
 'astrahan.irr.ru': 'irr.ru',
 'astrahanskaya-obl.irr.ru': 'irr.ru',
 'yakutsk.irr.ru': 'irr.ru',
 'krasnoyarskiy-kray.irr.ru': 'irr.ru',
 'kemerovskaya-obl.irr.ru': 'irr.ru',
 'magnitogorsk.irr.ru': 'irr.ru',
 'penza.irr.ru': 'irr.ru',
 'tver.irr.ru': 'irr.ru',
 'tverskaya-obl.irr.ru': 'irr.ru',
 'kaluzhskaya-obl.irr.ru': 'irr.ru',
 'yalta.irr.ru': 'irr.ru',
 'ivanovo.irr.ru': 'irr.ru',
 'obninsk.irr.ru': 'irr.ru',
 'kurgan.irr.ru': 'irr.ru',
 'surgut.irr.ru': 'irr.ru',
 'novokuznetsk.irr.ru': 'irr.ru',
 'vologda.irr.ru': 'irr.ru',
 'kabardino-balkarskaya-resp.irr.ru': 'irr.ru',
 'saransk.irr.ru': 'irr.ru',
 'voronezhskaya-obl.irr.ru': 'irr.ru',
 'anapa.irr.ru': 'irr.ru',
 'noyabrsk.irr.ru': 'irr.ru',
 'mordoviya-resp.irr.ru': 'irr.ru',
 'volgograd.irr.ru': 'irr.ru',
 'stavropol.irr.ru': 'irr.ru',
 'tomsk.irr.ru': 'irr.ru',
 'amurskaya-obl.irr.ru': 'irr.ru',
 'cheboksary.irr.ru': 'irr.ru',
 'syzran.irr.ru': 'irr.ru',
 'pskovskaya-obl.irr.ru': 'irr.ru',
 'nijniy-tagil.irr.ru': 'irr.ru',
 'orlovskaya-obl.irr.ru': 'irr.ru',
 'tomskaya-obl.irr.ru': 'irr.ru',
 'bryansk.irr.ru': 'irr.ru',
 'hakasiya-resp.irr.ru': 'irr.ru',
 'syktyvkar.irr.ru': 'irr.ru',
 'vologodskaya-obl.irr.ru': 'irr.ru',
 'kareliya-resp.irr.ru': 'irr.ru',
 'saratov.irr.ru': 'irr.ru',
 'kostroma.irr.ru': 'irr.ru',
 'mariy-el-resp.irr.ru': 'irr.ru',
 'magadan.irr.ru': 'irr.ru',
 'murmanskaya-obl.irr.ru': 'irr.ru',
 'mahachkala.irr.ru': 'irr.ru',
 'kavminvody.irr.ru': 'irr.ru',
 'murmansk.irr.ru': 'irr.ru',
 'yujno-sahalinsk.irr.ru': 'irr.ru',
 'kostromskaya-obl.irr.ru': 'irr.ru',
 'dagestan-resp.irr.ru': 'irr.ru',
 'penzenskaya-obl.irr.ru': 'irr.ru',
 'irkutskaya-obl.irr.ru': 'irr.ru',
 'tulskaya-obl.irr.ru': 'irr.ru',
 'saha-yakutiya-resp.irr.ru': 'irr.ru',
 'sahalinskaya-obl.irr.ru': 'irr.ru',
 'evreyskaya-aobl.irr.ru': 'irr.ru',
 'ulyanovskaya-obl.irr.ru': 'irr.ru',
 'tambov.irr.ru': 'irr.ru',
 'ulyanovsk.irr.ru': 'irr.ru',
 'buryatiya-resp.irr.ru': 'irr.ru',
 'yoshkarola.irr.ru': 'irr.ru',
 'altay-resp.irr.ru': 'irr.ru',
 'alushta.irr.ru': 'irr.ru',
 'krym-resp.irr.ru': 'irr.ru',
 'petropavlovsk-kamchatskiy.irr.ru': 'irr.ru',
 'evpatoriya.irr.ru': 'irr.ru',
 'komi-resp.irr.ru': 'irr.ru',
 'kerch.irr.ru': 'irr.ru',
 'groznyi.irr.ru': 'irr.ru',
 'tyva-resp.irr.ru': 'irr.ru',
 'arhangelsk.irr.ru': 'irr.ru',
 'nenetskiy-ao.irr.ru': 'irr.ru',
 'feodosiya.irr.ru': 'irr.ru',
 'kamchatskiy-kray.irr.ru': 'irr.ru',
 'irkutsk.irr.ru': 'irr.ru',
 'bahchisaray.irr.ru': 'irr.ru',
 'chechenskaya-resp.irr.ru': 'irr.ru',
 'kalmykiya-resp.irr.ru': 'irr.ru',
 'voljskiy.irr.ru': 'irr.ru',
 'zabaykalskiy-kray.irr.ru': 'irr.ru',
 'berezniki.irr.ru': 'irr.ru',
    'perm.zarplata.ru': 'zarplata.ru',
 'nsk.zarplata.ru': 'zarplata.ru',
 'samara.zarplata.ru': 'zarplata.ru',
 'zarplata.ru': 'zarplata.ru',
 'ekb.zarplata.ru': 'zarplata.ru',
 'sverdlovskaya-oblast.zarplata.ru': 'zarplata.ru',
 'tyumen.zarplata.ru': 'zarplata.ru',
 'kolomna.zarplata.ru': 'zarplata.ru',
 'chelyabinsk.zarplata.ru': 'zarplata.ru',
 'habarovsk.zarplata.ru': 'zarplata.ru',
 'spb.zarplata.ru': 'zarplata.ru',
 'novokujbyshevsk.zarplata.ru': 'zarplata.ru',
 'omsk.zarplata.ru': 'zarplata.ru',
 'krs.zarplata.ru': 'zarplata.ru',
 'tomsk.zarplata.ru': 'zarplata.ru',
 'novokuznetsk.zarplata.ru': 'zarplata.ru',
 'vladivostok.zarplata.ru': 'zarplata.ru',
 'bogdanovich.zarplata.ru': 'zarplata.ru',
 'yaroslavl.zarplata.ru': 'zarplata.ru',
 'ryazan.zarplata.ru': 'zarplata.ru',
 'kemerovo.zarplata.ru': 'zarplata.ru',
 'kurgan.zarplata.ru': 'zarplata.ru',
 'krasnodar.zarplata.ru': 'zarplata.ru',
 'ufa.zarplata.ru': 'zarplata.ru',
 'podolsk.zarplata.ru': 'zarplata.ru',
 'pervouralsk.zarplata.ru': 'zarplata.ru',
 'nizhniy-tagil.zarplata.ru': 'zarplata.ru',
 'kaliningrad.zarplata.ru': 'zarplata.ru',
 'arkhangelsk.zarplata.ru': 'zarplata.ru',
 'zheleznogorsk-krasnoyarskij-kray.zarplata.ru': 'zarplata.ru',
 'verhnjaja-pyshma.zarplata.ru': 'zarplata.ru',
 'tver.zarplata.ru': 'zarplata.ru',
 'emanzhelinsk.zarplata.ru': 'zarplata.ru',
 'barnaul.zarplata.ru': 'zarplata.ru',
 'surgut.zarplata.ru': 'zarplata.ru',
 'iskitim.zarplata.ru': 'zarplata.ru',
 'miass.zarplata.ru': 'zarplata.ru',
 'berezovskij.zarplata.ru': 'zarplata.ru',
 'nn.zarplata.ru': 'zarplata.ru',
 'khanty-mansiysk.zarplata.ru': 'zarplata.ru',
 'sterlitamak.zarplata.ru': 'zarplata.ru',
 'krasnokamsk.zarplata.ru': 'zarplata.ru',
 'magadan.zarplata.ru': 'zarplata.ru',
 'astrakhan.zarplata.ru': 'zarplata.ru',
 'rostov.zarplata.ru': 'zarplata.ru',
 'chehov.zarplata.ru': 'zarplata.ru',
 'irkutsk.zarplata.ru': 'zarplata.ru',
 'sredneuralsk.zarplata.ru': 'zarplata.ru',
 'voronezh.zarplata.ru': 'zarplata.ru',
 'belovo.zarplata.ru': 'zarplata.ru',
 'krasnoobsk-nso.zarplata.ru': 'zarplata.ru',
 'vladimir.zarplata.ru': 'zarplata.ru',
 'novouralsk.zarplata.ru': 'zarplata.ru',
 'kamensk-uralskij.zarplata.ru': 'zarplata.ru',
 'obninsk.zarplata.ru': 'zarplata.ru',
 'belgorod.zarplata.ru': 'zarplata.ru',
 'zlatoust.zarplata.ru': 'zarplata.ru',
 'almetevsk.zarplata.ru': 'zarplata.ru',
 'kazan.zarplata.ru': 'zarplata.ru',
 'orenburg.zarplata.ru': 'zarplata.ru',
 'kopejsk.zarplata.ru': 'zarplata.ru',
 'kaluga.zarplata.ru': 'zarplata.ru',
 'abakan.zarplata.ru': 'zarplata.ru',
 'russia.zarplata.ru': 'zarplata.ru',
 'tolyatti.zarplata.ru': 'zarplata.ru',
 'zelenograd.zarplata.ru': 'zarplata.ru',
 'domodedovo.zarplata.ru': 'zarplata.ru',
 'magnitogorsk.zarplata.ru': 'zarplata.ru',
 'minusinsk.zarplata.ru': 'zarplata.ru',
 'norilsk.zarplata.ru': 'zarplata.ru',
 'himki.zarplata.ru': 'zarplata.ru',
 'rubcovsk.zarplata.ru': 'zarplata.ru',
 'berdsk.zarplata.ru': 'zarplata.ru',
 'syzran.zarplata.ru': 'zarplata.ru',
 'ivanovo.zarplata.ru': 'zarplata.ru',
 'novorossiysk.zarplata.ru': 'zarplata.ru',
 'asbest.zarplata.ru': 'zarplata.ru',
 'izhevsk.zarplata.ru': 'zarplata.ru',
 'bijsk.zarplata.ru': 'zarplata.ru',
 'penza.zarplata.ru': 'zarplata.ru',
 'ulyanovsk.zarplata.ru': 'zarplata.ru',
 'sochi.zarplata.ru': 'zarplata.ru',
 'kansk.zarplata.ru': 'zarplata.ru',
 'sysert.zarplata.ru': 'zarplata.ru',
 'nevjansk.zarplata.ru': 'zarplata.ru',
 'chajkovskij.zarplata.ru': 'zarplata.ru',
 'tobolsk.zarplata.ru': 'zarplata.ru',
 'krasnoturinsk.zarplata.ru': 'zarplata.ru',
 'troitsk.zarplata.ru': 'zarplata.ru',
 'mezhdurechensk.zarplata.ru': 'zarplata.ru',
 'elektrostal.zarplata.ru': 'zarplata.ru',
 'volgograd.zarplata.ru': 'zarplata.ru',
 'pskov.zarplata.ru': 'zarplata.ru',
 'nefteyugansk.zarplata.ru': 'zarplata.ru',
 'krasnogorsk.zarplata.ru': 'zarplata.ru',
 'berezniki.zarplata.ru': 'zarplata.ru',
 'uzhur.zarplata.ru': 'zarplata.ru',
 'odincovo.zarplata.ru': 'zarplata.ru',
 'syktyvkar.zarplata.ru': 'zarplata.ru',
 'novyy-urengoy.zarplata.ru': 'zarplata.ru',
 'ivanteevka.zarplata.ru': 'zarplata.ru',
 'lyubercy.zarplata.ru': 'zarplata.ru',
 'tula.zarplata.ru': 'zarplata.ru',
 'gorno-altaisk.zarplata.ru': 'zarplata.ru',
 'trehgornyj.zarplata.ru': 'zarplata.ru',
 'vladikavkaz.zarplata.ru': 'zarplata.ru',
 'nizhnevartovsk.zarplata.ru': 'zarplata.ru',
 'gelendzhik.zarplata.ru': 'zarplata.ru',
 'ob.zarplata.ru': 'zarplata.ru',
 'voskresensk.zarplata.ru': 'zarplata.ru',
 'novoaltajsk.zarplata.ru': 'zarplata.ru',
 'ramenskoe.zarplata.ru': 'zarplata.ru',
 'zhukovskiy.zarplata.ru': 'zarplata.ru',
 'neftekamsk.zarplata.ru': 'zarplata.ru',
 'aramil.zarplata.ru': 'zarplata.ru',
 'ishim.zarplata.ru': 'zarplata.ru',
 'severodvinsk.zarplata.ru': 'zarplata.ru',
 'hmao.zarplata.ru': 'zarplata.ru',
 'anapa.zarplata.ru': 'zarplata.ru',
 'kyzyl.zarplata.ru': 'zarplata.ru',
 'smolensk.zarplata.ru': 'zarplata.ru',
 'usinsk.zarplata.ru': 'zarplata.ru',
 'serov.zarplata.ru': 'zarplata.ru',
 'cheboksary.zarplata.ru': 'zarplata.ru',
 'jalutorovsk.zarplata.ru': 'zarplata.ru',
 'kashira.zarplata.ru': 'zarplata.ru',
 'hakasiya.zarplata.ru': 'zarplata.ru',
 'naro-fominsk.zarplata.ru': 'zarplata.ru',
 'ust-katav.zarplata.ru': 'zarplata.ru',
 'lipetsk.zarplata.ru': 'zarplata.ru',
 'polevskoj.zarplata.ru': 'zarplata.ru',
 'achinsk.zarplata.ru': 'zarplata.ru',
 'mytischi.zarplata.ru': 'zarplata.ru',
 'kirov.zarplata.ru': 'zarplata.ru',
 'tomilino-moskovskaya-oblast.zarplata.ru': 'zarplata.ru',
 'jurga.zarplata.ru': 'zarplata.ru',
 'petropavlovsk-kamchatskij.zarplata.ru': 'zarplata.ru',
 'kogalym.zarplata.ru': 'zarplata.ru',
 'orsk.zarplata.ru': 'zarplata.ru',
 'saratov.zarplata.ru': 'zarplata.ru',
 'komsomolsk-na-amure.zarplata.ru': 'zarplata.ru',
 'kungur.zarplata.ru': 'zarplata.ru',
 'kamyshlov.zarplata.ru': 'zarplata.ru',
 'prokopevsk.zarplata.ru': 'zarplata.ru',
 'revda.zarplata.ru': 'zarplata.ru',
 'omskaya-oblast.zarplata.ru': 'zarplata.ru',
 'dolgoprudnyy.zarplata.ru': 'zarplata.ru',
 'noyabrsk.zarplata.ru': 'zarplata.ru',
 'naberezhnye-chelny.zarplata.ru': 'zarplata.ru',
 'tavda.zarplata.ru': 'zarplata.ru',
 'mariinsk.zarplata.ru': 'zarplata.ru',
 'anzhero-sudzhensk.zarplata.ru': 'zarplata.ru',
 'gatchina.zarplata.ru': 'zarplata.ru',
 'njagan.zarplata.ru': 'zarplata.ru',
 'zheleznodorozhnyy.zarplata.ru': 'zarplata.ru',
 'alapaevsk.zarplata.ru': 'zarplata.ru',
 'zavodoukovsk.zarplata.ru': 'zarplata.ru',
 'chelyabinskaya-oblast.zarplata.ru': 'zarplata.ru',
 'irbit.zarplata.ru': 'zarplata.ru',
 'mihajlovsk-stavropolskij-kray.zarplata.ru': 'zarplata.ru',
 'orel.zarplata.ru': 'zarplata.ru',
 'sosnovoborsk.zarplata.ru': 'zarplata.ru',
 'korolyov.zarplata.ru': 'zarplata.ru',
 'ozersk.zarplata.ru': 'zarplata.ru',
 'bolshoy-istok66.zarplata.ru': 'zarplata.ru',
 'troick.zarplata.ru': 'zarplata.ru',
 'novosibirskaya-oblast.zarplata.ru': 'zarplata.ru',
 'ulan-ude.zarplata.ru': 'zarplata.ru',
 'lesosibirsk.zarplata.ru': 'zarplata.ru',
 'zarinsk.zarplata.ru': 'zarplata.ru',
 'chusovoj.zarplata.ru': 'zarplata.ru',
 'bashkortostan.zarplata.ru': 'zarplata.ru',
 'seversk.zarplata.ru': 'zarplata.ru',
 'balashiha.zarplata.ru': 'zarplata.ru',
 'dolgoderevenskoe-chelyabinskaya-oblast.zarplata.ru': 'zarplata.ru',
 'alejsk.zarplata.ru': 'zarplata.ru',
 'permskij-kray.zarplata.ru': 'zarplata.ru',
 'berezovskij-kemerovskaya-obl.zarplata.ru': 'zarplata.ru',
 'kiselevsk.zarplata.ru': 'zarplata.ru',
 'juzhnouralsk.zarplata.ru': 'zarplata.ru',
 'nizhnjaja-tura.zarplata.ru': 'zarplata.ru',
 'kotlas.zarplata.ru': 'zarplata.ru',
 'suhoj-log.zarplata.ru': 'zarplata.ru',
 'maykop.zarplata.ru': 'zarplata.ru',
    '1000bankov.ru': '1000bankov.ru',
 'naro-fominsk.1000bankov.ru': '1000bankov.ru',
 'novosibirsk.1000bankov.ru': '1000bankov.ru',
 'tomsk.1000bankov.ru': '1000bankov.ru',
 'ufa.1000bankov.ru': '1000bankov.ru',
 'salavat.1000bankov.ru': '1000bankov.ru',
 'sochi.1000bankov.ru': '1000bankov.ru',
 'cherkessk.1000bankov.ru': '1000bankov.ru',
 'domodedovo.1000bankov.ru': '1000bankov.ru',
 'moskva.1000bankov.ru': '1000bankov.ru',
 'kungur.1000bankov.ru': '1000bankov.ru',
 'cherepovec.1000bankov.ru': '1000bankov.ru',
 'belgorod.1000bankov.ru': '1000bankov.ru',
 'vladimir.1000bankov.ru': '1000bankov.ru',
 'eysk.1000bankov.ru': '1000bankov.ru',
 'ekaterinburg.1000bankov.ru': '1000bankov.ru',
 'biysk.1000bankov.ru': '1000bankov.ru',
 'krasnodar.1000bankov.ru': '1000bankov.ru',
 'kogalym.1000bankov.ru': '1000bankov.ru',
 'frolovo.1000bankov.ru': '1000bankov.ru',
 'anapa.1000bankov.ru': '1000bankov.ru',
 'izhevsk.1000bankov.ru': '1000bankov.ru',
 'vladikavkaz.1000bankov.ru': '1000bankov.ru',
 'balashiha.1000bankov.ru': '1000bankov.ru',
 'tolyatti.1000bankov.ru': '1000bankov.ru',
 'nn.1000bankov.ru': '1000bankov.ru',
 'cheboksary.1000bankov.ru': '1000bankov.ru',
 'spb.1000bankov.ru': '1000bankov.ru',
 'neftekamsk.1000bankov.ru': '1000bankov.ru',
 'vladivostok.1000bankov.ru': '1000bankov.ru',
 'bryansk.1000bankov.ru': '1000bankov.ru',
 'arhangelsk.1000bankov.ru': '1000bankov.ru',
 'tula.1000bankov.ru': '1000bankov.ru',
 'kazan.1000bankov.ru': '1000bankov.ru',
 'omsk.1000bankov.ru': '1000bankov.ru',
 'orenburg.1000bankov.ru': '1000bankov.ru',
 'samara.1000bankov.ru': '1000bankov.ru',
 'kstovo.1000bankov.ru': '1000bankov.ru',
 'blagoveschensk.1000bankov.ru': '1000bankov.ru',
 'ryazan.1000bankov.ru': '1000bankov.ru',
 'novovoronezh.1000bankov.ru': '1000bankov.ru',
 'orehovo-zuevo.1000bankov.ru': '1000bankov.ru',
 'schelkovo.1000bankov.ru': '1000bankov.ru',
 'yuzhno-sahalinsk.1000bankov.ru': '1000bankov.ru',
 'hanty-mansiysk.1000bankov.ru': '1000bankov.ru',
 'syzran.1000bankov.ru': '1000bankov.ru',
 'krasnoyarsk.1000bankov.ru': '1000bankov.ru',
 'kurganinsk.1000bankov.ru': '1000bankov.ru',
 'krasnogorsk2.1000bankov.ru': '1000bankov.ru',
 'vologda.1000bankov.ru': '1000bankov.ru',
 'kyzyl.1000bankov.ru': '1000bankov.ru',
 'korkino.1000bankov.ru': '1000bankov.ru',
 'magnitogorsk.1000bankov.ru': '1000bankov.ru',
 'belorechensk.1000bankov.ru': '1000bankov.ru',
 'amursk.1000bankov.ru': '1000bankov.ru',
 'taganrog.1000bankov.ru': '1000bankov.ru',
 'pskov.1000bankov.ru': '1000bankov.ru',
 'kanash.1000bankov.ru': '1000bankov.ru',
 'kaliningrad.1000bankov.ru': '1000bankov.ru',
 'kurgan.1000bankov.ru': '1000bankov.ru',
 'staraya-russa.1000bankov.ru': '1000bankov.ru',
 'petropavlovsk-kamchatskiy.1000bankov.ru': '1000bankov.ru',
 'alushta.1000bankov.ru': '1000bankov.ru',
 'zheleznodorozhnyy.1000bankov.ru': '1000bankov.ru',
 'sysert.1000bankov.ru': '1000bankov.ru',
 'nalchik.1000bankov.ru': '1000bankov.ru',
 'nizhnevartovsk.1000bankov.ru': '1000bankov.ru',
 'kamen-na-obi.1000bankov.ru': '1000bankov.ru',
 'kostroma.1000bankov.ru': '1000bankov.ru',
 'smolensk.1000bankov.ru': '1000bankov.ru',
 'novorossiysk.1000bankov.ru': '1000bankov.ru',
 'podolsk.1000bankov.ru': '1000bankov.ru',
 'velikie-luki.1000bankov.ru': '1000bankov.ru',
 'tyumen.1000bankov.ru': '1000bankov.ru',
 'yakutsk.1000bankov.ru': '1000bankov.ru',
 'yaroslavl.1000bankov.ru': '1000bankov.ru',
 'nevinnomyssk.1000bankov.ru': '1000bankov.ru',
 'tver.1000bankov.ru': '1000bankov.ru',
 'sterlitamak.1000bankov.ru': '1000bankov.ru',
 'sevastopol.1000bankov.ru': '1000bankov.ru',
 'mezhdurechensk.1000bankov.ru': '1000bankov.ru',
 'kaluga.1000bankov.ru': '1000bankov.ru',
 'oktyabrskiy.1000bankov.ru': '1000bankov.ru',
 'rasskazovo.1000bankov.ru': '1000bankov.ru',
 'meleuz.1000bankov.ru': '1000bankov.ru',
 'saratov.1000bankov.ru': '1000bankov.ru',
 'surgut.1000bankov.ru': '1000bankov.ru',
 'birsk.1000bankov.ru': '1000bankov.ru',
 'orel.1000bankov.ru': '1000bankov.ru',
 'novokuzneck.1000bankov.ru': '1000bankov.ru',
 'novouralsk.1000bankov.ru': '1000bankov.ru',
 'artem.1000bankov.ru': '1000bankov.ru',
 'kropotkin.1000bankov.ru': '1000bankov.ru',
 'novotroick.1000bankov.ru': '1000bankov.ru',
 'engels.1000bankov.ru': '1000bankov.ru',
 'slavyansk-na-kubani.1000bankov.ru': '1000bankov.ru',
 'zlatoust.1000bankov.ru': '1000bankov.ru',
 'elabuga.1000bankov.ru': '1000bankov.ru',
 'ishimbay.1000bankov.ru': '1000bankov.ru',
 'staryy-oskol.1000bankov.ru': '1000bankov.ru',
 'gelendzhik.1000bankov.ru': '1000bankov.ru',
 'uglich.1000bankov.ru': '1000bankov.ru',
 'voronezh.1000bankov.ru': '1000bankov.ru',
 'berezniki.1000bankov.ru': '1000bankov.ru',
 'scherbinka.1000bankov.ru': '1000bankov.ru',
 'vyatskie-polyany.1000bankov.ru': '1000bankov.ru',
 'kirov.1000bankov.ru': '1000bankov.ru',
 'prokopevsk.1000bankov.ru': '1000bankov.ru',
 'kopeysk.1000bankov.ru': '1000bankov.ru',
 'serpuhov.1000bankov.ru': '1000bankov.ru',
 'habarovsk.1000bankov.ru': '1000bankov.ru',
 'tuymazy.1000bankov.ru': '1000bankov.ru',
 'temryuk.1000bankov.ru': '1000bankov.ru',
 'severobaykalsk.1000bankov.ru': '1000bankov.ru',
 'murmansk.1000bankov.ru': '1000bankov.ru',
 'orsk.1000bankov.ru': '1000bankov.ru',
 'korenovsk.1000bankov.ru': '1000bankov.ru',
 'nizhnekamsk.1000bankov.ru': '1000bankov.ru',
 'schekino.1000bankov.ru': '1000bankov.ru',
 'nefteyugansk.1000bankov.ru': '1000bankov.ru',
 'kuzneck.1000bankov.ru': '1000bankov.ru',
 'odincovo.1000bankov.ru': '1000bankov.ru',
 'armavir.1000bankov.ru': '1000bankov.ru',
 'abakan.1000bankov.ru': '1000bankov.ru',
 'berdsk.1000bankov.ru': '1000bankov.ru',
 'novokuybyshevsk.1000bankov.ru': '1000bankov.ru',
 'uhta.1000bankov.ru': '1000bankov.ru',
 'vidnoe.1000bankov.ru': '1000bankov.ru',
 'lipeck.1000bankov.ru': '1000bankov.ru',
 'ivanovo.1000bankov.ru': '1000bankov.ru',
 'kovrov.1000bankov.ru': '1000bankov.ru',
 'verhnyaya-pyshma.1000bankov.ru': '1000bankov.ru',
 'barabinsk.1000bankov.ru': '1000bankov.ru',
 'mytischi.1000bankov.ru': '1000bankov.ru',
 'kuybyshev.1000bankov.ru': '1000bankov.ru',
 'usinsk.1000bankov.ru': '1000bankov.ru',
 'maykop.1000bankov.ru': '1000bankov.ru',
 'tynda.1000bankov.ru': '1000bankov.ru',
 'kommunar.1000bankov.ru': '1000bankov.ru',
 'lyubercy.1000bankov.ru': '1000bankov.ru',
 'bugulma.1000bankov.ru': '1000bankov.ru',
 'gatchina.1000bankov.ru': '1000bankov.ru',
 'syktyvkar.1000bankov.ru': '1000bankov.ru',
 'balakovo.1000bankov.ru': '1000bankov.ru',
 'vyksa.1000bankov.ru': '1000bankov.ru',
 'komsomolsk-na-amure.1000bankov.ru': '1000bankov.ru',
 'polevskoy.1000bankov.ru': '1000bankov.ru',
 'kineshma.1000bankov.ru': '1000bankov.ru',
 'gryazi.1000bankov.ru': '1000bankov.ru',
 'noginsk.1000bankov.ru': '1000bankov.ru',
 'kemerovo.1000bankov.ru': '1000bankov.ru',
 'penza.1000bankov.ru': '1000bankov.ru',
 'georgievsk.1000bankov.ru': '1000bankov.ru',
 'tihoreck.1000bankov.ru': '1000bankov.ru',
 'krasnokamsk.1000bankov.ru': '1000bankov.ru',
 'novocheboksarsk.1000bankov.ru': '1000bankov.ru',
 'novoaltaysk.1000bankov.ru': '1000bankov.ru',
 'feodosija.1000bankov.ru': '1000bankov.ru',
 'volhov.1000bankov.ru': '1000bankov.ru',
 'klimovsk.1000bankov.ru': '1000bankov.ru',
 'pervouralsk.1000bankov.ru': '1000bankov.ru',
 'kolchugino.1000bankov.ru': '1000bankov.ru',
 'kamensk-uralskiy.1000bankov.ru': '1000bankov.ru',
 'saransk.1000bankov.ru': '1000bankov.ru',
 'gorohovec.1000bankov.ru': '1000bankov.ru',
 'himki.1000bankov.ru': '1000bankov.ru',
 'pavlovo.1000bankov.ru': '1000bankov.ru',
 'krymsk.1000bankov.ru': '1000bankov.ru',
 'yoshkar-ola.1000bankov.ru': '1000bankov.ru',
 'rybinsk.1000bankov.ru': '1000bankov.ru',
 'ulyanovsk.1000bankov.ru': '1000bankov.ru',
 'mirnyy2.1000bankov.ru': '1000bankov.ru',
 'kursk.1000bankov.ru': '1000bankov.ru',
 'gubkinskiy.1000bankov.ru': '1000bankov.ru',
 'votkinsk.1000bankov.ru': '1000bankov.ru',
 'tambov.1000bankov.ru': '1000bankov.ru',
 'volgograd.1000bankov.ru': '1000bankov.ru',
 'svobodnyy.1000bankov.ru': '1000bankov.ru',
 'velikiy-novgorod.1000bankov.ru': '1000bankov.ru',
 'rubcovsk.1000bankov.ru': '1000bankov.ru',
 'nizhniy-tagil.1000bankov.ru': '1000bankov.ru',
 'obninsk.1000bankov.ru': '1000bankov.ru',
 'millerovo.1000bankov.ru': '1000bankov.ru',
 'serov.1000bankov.ru': '1000bankov.ru',
 'pushkin.1000bankov.ru': '1000bankov.ru',
 'ussuriysk.1000bankov.ru': '1000bankov.ru',
 'troick.1000bankov.ru': '1000bankov.ru',
 'leninogorsk.1000bankov.ru': '1000bankov.ru',
    'mbib.ru': 'mbib.ru',
 'orsk.mbib.ru': 'mbib.ru',
 'dedovsk.mbib.ru': 'mbib.ru',
 'shilka.mbib.ru': 'mbib.ru',
 'sevastopol.mbib.ru': 'mbib.ru',
 'voronezh.mbib.ru': 'mbib.ru',
 'kaliningrad.mbib.ru': 'mbib.ru',
 'moskva.mbib.ru': 'mbib.ru',
 'privolzhsk.mbib.ru': 'mbib.ru',
 'tyumen.mbib.ru': 'mbib.ru',
 'kirovskaya-kirov.mbib.ru': 'mbib.ru',
 'petrozavodsk.mbib.ru': 'mbib.ru',
 'naberezhnye-chelny.mbib.ru': 'mbib.ru',
 'sankt-peterburg.mbib.ru': 'mbib.ru',
 'novosibirsk.mbib.ru': 'mbib.ru',
 'liski.mbib.ru': 'mbib.ru',
 'belorechensk.mbib.ru': 'mbib.ru',
 'tolyatti.mbib.ru': 'mbib.ru',
 'ekaterinburg.mbib.ru': 'mbib.ru',
 'chelyabinsk.mbib.ru': 'mbib.ru',
 'smolensk.mbib.ru': 'mbib.ru',
 'lipetsk.mbib.ru': 'mbib.ru',
 'tver.mbib.ru': 'mbib.ru',
 'kamyshlov.mbib.ru': 'mbib.ru',
 'rostov-na-donu.mbib.ru': 'mbib.ru',
 'vladivostok.mbib.ru': 'mbib.ru',
 'neftekamsk.mbib.ru': 'mbib.ru',
 'krasnodar.mbib.ru': 'mbib.ru',
 'engels.mbib.ru': 'mbib.ru',
 'velikie-luki.mbib.ru': 'mbib.ru',
 'volgograd.mbib.ru': 'mbib.ru',
 'anapa.mbib.ru': 'mbib.ru',
 'serpuhov.mbib.ru': 'mbib.ru',
 'kaluga.mbib.ru': 'mbib.ru',
 'murmansk.mbib.ru': 'mbib.ru',
 'bronnitsy.mbib.ru': 'mbib.ru',
 'svetlograd.mbib.ru': 'mbib.ru',
 'sergiev-posad.mbib.ru': 'mbib.ru',
 'arhangelsk.mbib.ru': 'mbib.ru',
 'ufa.mbib.ru': 'mbib.ru',
 'novorossiysk.mbib.ru': 'mbib.ru',
 'stavropol.mbib.ru': 'mbib.ru',
 'novocheboksarsk.mbib.ru': 'mbib.ru',
 'vladimir.mbib.ru': 'mbib.ru',
 'himki.mbib.ru': 'mbib.ru',
 'norilsk.mbib.ru': 'mbib.ru',
 'voskresensk.mbib.ru': 'mbib.ru',
 'kemerovo.mbib.ru': 'mbib.ru',
 'velikiy-novgorod.mbib.ru': 'mbib.ru',
 'kazan.mbib.ru': 'mbib.ru',
 'sochi.mbib.ru': 'mbib.ru',
 'nizhniy-novgorod.mbib.ru': 'mbib.ru',
 'izhevsk.mbib.ru': 'mbib.ru',
 'aleksin.mbib.ru': 'mbib.ru',
 'amurskaya-blagoveschensk.mbib.ru': 'mbib.ru',
 'amurskaya-belogorsk.mbib.ru': 'mbib.ru',
 'ivanovo.mbib.ru': 'mbib.ru',
 'surgut.mbib.ru': 'mbib.ru',
 'kurganinsk.mbib.ru': 'mbib.ru',
 'noginsk.mbib.ru': 'mbib.ru',
 'chapaevsk.mbib.ru': 'mbib.ru',
 'karasuk.mbib.ru': 'mbib.ru',
 'temryuk.mbib.ru': 'mbib.ru',
 'komsomolsk-na-amure.mbib.ru': 'mbib.ru',
 'penza.mbib.ru': 'mbib.ru',
 'uhta.mbib.ru': 'mbib.ru',
 'orenburg.mbib.ru': 'mbib.ru',
 'samara.mbib.ru': 'mbib.ru',
 'budennovsk.mbib.ru': 'mbib.ru',
 'partizansk.mbib.ru': 'mbib.ru',
 'salsk.mbib.ru': 'mbib.ru',
 'kostroma.mbib.ru': 'mbib.ru',
 'kalach.mbib.ru': 'mbib.ru',
 'almetievsk.mbib.ru': 'mbib.ru',
 'ramenskoe.mbib.ru': 'mbib.ru',
 'shuya.mbib.ru': 'mbib.ru',
 'egorievsk.mbib.ru': 'mbib.ru',
 'saransk.mbib.ru': 'mbib.ru',
 'barnaul.mbib.ru': 'mbib.ru',
 'ust-kut.mbib.ru': 'mbib.ru',
 'oryol.mbib.ru': 'mbib.ru',
 'staryy-oskol.mbib.ru': 'mbib.ru',
 'ostrogozhsk.mbib.ru': 'mbib.ru',
 'chaplygin.mbib.ru': 'mbib.ru',
 'usinsk.mbib.ru': 'mbib.ru',
 'podporozhie.mbib.ru': 'mbib.ru',
 'vyborg.mbib.ru': 'mbib.ru',
 'luga.mbib.ru': 'mbib.ru',
 'pskov.mbib.ru': 'mbib.ru',
 'tihvin.mbib.ru': 'mbib.ru',
 'vyatskie-polyany.mbib.ru': 'mbib.ru',
 'kashira.mbib.ru': 'mbib.ru',
 'krasnoyarsk.mbib.ru': 'mbib.ru',
 'rezh.mbib.ru': 'mbib.ru',
 'sarapul.mbib.ru': 'mbib.ru',
 'podolsk.mbib.ru': 'mbib.ru',
 'furmanov.mbib.ru': 'mbib.ru',
 'pavlovskiy-posad.mbib.ru': 'mbib.ru',
 'zhukov.mbib.ru': 'mbib.ru',
 'korolev.mbib.ru': 'mbib.ru',
 'murom.mbib.ru': 'mbib.ru',
 'mytischi.mbib.ru': 'mbib.ru',
 'salavat.mbib.ru': 'mbib.ru',
 'apsheronsk.mbib.ru': 'mbib.ru',
 'miass.mbib.ru': 'mbib.ru',
 'chelyabinskaya-troitsk.mbib.ru': 'mbib.ru',
 'taganrog.mbib.ru': 'mbib.ru',
 'vidnoe.mbib.ru': 'mbib.ru',
 'lyubertsy.mbib.ru': 'mbib.ru',
 'maykop.mbib.ru': 'mbib.ru',
 'magnitogorsk.mbib.ru': 'mbib.ru',
 'yoshkar-ola.mbib.ru': 'mbib.ru',
 'dimitrovgrad.mbib.ru': 'mbib.ru',
 'sterlitamak.mbib.ru': 'mbib.ru',
 'sudak.mbib.ru': 'mbib.ru',
 'borisoglebsk.mbib.ru': 'mbib.ru',
 'bryansk.mbib.ru': 'mbib.ru',
 'yurga.mbib.ru': 'mbib.ru',
 'anzhero-sudzhensk.mbib.ru': 'mbib.ru',
 'armavir.mbib.ru': 'mbib.ru',
 'gelendzhik.mbib.ru': 'mbib.ru',
 'ussuriysk.mbib.ru': 'mbib.ru',
 'arzamas.mbib.ru': 'mbib.ru',
 'zlatoust.mbib.ru': 'mbib.ru',
 'belaya-kalitva.mbib.ru': 'mbib.ru',
 'gagarin.mbib.ru': 'mbib.ru',
 'habarovsk.mbib.ru': 'mbib.ru',
 'orehovo-zuevo.mbib.ru': 'mbib.ru',
 'labytnangi.mbib.ru': 'mbib.ru',
 'bogoroditsk.mbib.ru': 'mbib.ru',
 'naro-fominsk.mbib.ru': 'mbib.ru',
 'ryazan.mbib.ru': 'mbib.ru',
 'davlekanovo.mbib.ru': 'mbib.ru',
 'perm.mbib.ru': 'mbib.ru',
 'tuymazy.mbib.ru': 'mbib.ru',
 'ahtubinsk.mbib.ru': 'mbib.ru',
 'cheboksary.mbib.ru': 'mbib.ru',
 'luhovitsy.mbib.ru': 'mbib.ru',
 'kolomna.mbib.ru': 'mbib.ru',
 'yaroslavl.mbib.ru': 'mbib.ru',
 'tuapse.mbib.ru': 'mbib.ru',
 'strezhevoy.mbib.ru': 'mbib.ru',
 'megion.mbib.ru': 'mbib.ru',
 'hanty-mansiysk.mbib.ru': 'mbib.ru',
 'hanty-mansiyskiy-raduzhnyy.mbib.ru': 'mbib.ru',
 'nizhnevartovsk.mbib.ru': 'mbib.ru',
 'yugorsk.mbib.ru': 'mbib.ru',
 'astrahan.mbib.ru': 'mbib.ru',
 'timashevsk.mbib.ru': 'mbib.ru',
 'tihoretsk.mbib.ru': 'mbib.ru',
 'volzhskiy.mbib.ru': 'mbib.ru',
 'pervouralsk.mbib.ru': 'mbib.ru',
 'schelkovo.mbib.ru': 'mbib.ru',
 'asbest.mbib.ru': 'mbib.ru',
 'vologda.mbib.ru': 'mbib.ru',
 'vyksa.mbib.ru': 'mbib.ru',
 'tobolsk.mbib.ru': 'mbib.ru',
 'saratov.mbib.ru': 'mbib.ru',
 'uren.mbib.ru': 'mbib.ru',
 'bor.mbib.ru': 'mbib.ru',
 'labinsk.mbib.ru': 'mbib.ru',
 'mineralnye-vody.mbib.ru': 'mbib.ru',
 'abakan.mbib.ru': 'mbib.ru',
 'vladikavkaz.mbib.ru': 'mbib.ru',
 'kurgan.mbib.ru': 'mbib.ru',
 'petropavlovsk-kamchatskiy.mbib.ru': 'mbib.ru',
 'elets.mbib.ru': 'mbib.ru',
 'balashov.mbib.ru': 'mbib.ru',
 'belebey.mbib.ru': 'mbib.ru',
 'belgorod.mbib.ru': 'mbib.ru',
 'nazyvaevsk.mbib.ru': 'mbib.ru',
 'omsk.mbib.ru': 'mbib.ru',
 'buynaksk.mbib.ru': 'mbib.ru',
 'inza.mbib.ru': 'mbib.ru',
 'tynda.mbib.ru': 'mbib.ru',
 'izberbash.mbib.ru': 'mbib.ru',
 'pyatigorsk.mbib.ru': 'mbib.ru',
 'simferopol.mbib.ru': 'mbib.ru',
 'hotkovo.mbib.ru': 'mbib.ru',
 'domodedovo.mbib.ru': 'mbib.ru',
 'salehard.mbib.ru': 'mbib.ru',
 'dzerzhinsk.mbib.ru': 'mbib.ru',
 'chehov.mbib.ru': 'mbib.ru',
 'solikamsk.mbib.ru': 'mbib.ru',
 'cherepovets.mbib.ru': 'mbib.ru',
 'zainsk.mbib.ru': 'mbib.ru',
 'ulyanovsk.mbib.ru': 'mbib.ru',
 'buzuluk.mbib.ru': 'mbib.ru',
 'berezniki.mbib.ru': 'mbib.ru',
 'novyy-urengoy.mbib.ru': 'mbib.ru',
 'tula.mbib.ru': 'mbib.ru',
    'ekaterinburg.kitabi.ru': 'kitabi.ru',
 'vitebsk.kitabi.ru': 'kitabi.ru',
 'chelyabinsk.kitabi.ru': 'kitabi.ru',
 'ufa.kitabi.ru': 'kitabi.ru',
 'rostov-na-donu.kitabi.ru': 'kitabi.ru',
 'volgograd.kitabi.ru': 'kitabi.ru',
 'samara.kitabi.ru': 'kitabi.ru',
 'nizhniy-novgorod.kitabi.ru': 'kitabi.ru',
 'minsk.kitabi.ru': 'kitabi.ru',
 'tula.kitabi.ru': 'kitabi.ru',
 'harkov.kitabi.ru': 'kitabi.ru',
 'kitabi.ru': 'kitabi.ru',
 'sankt-peterburg.kitabi.ru': 'kitabi.ru',
 'brest.kitabi.ru': 'kitabi.ru',
 'tolyatti.kitabi.ru': 'kitabi.ru',
 'moskva.kitabi.ru': 'kitabi.ru',
 'tumen.kitabi.ru': 'kitabi.ru',
 'omsk.kitabi.ru': 'kitabi.ru',
 'krasnoyarsk.kitabi.ru': 'kitabi.ru',
 'perm.kitabi.ru': 'kitabi.ru',
 'almaty.kitabi.ru': 'kitabi.ru',
 'voronezh.kitabi.ru': 'kitabi.ru',
 'smolensk.kitabi.ru': 'kitabi.ru',
 'dnepr.kitabi.ru': 'kitabi.ru',
 'novosibirsk.kitabi.ru': 'kitabi.ru',
 'saratov.kitabi.ru': 'kitabi.ru',
 'kazan.kitabi.ru': 'kitabi.ru',
 'zaporozhe.kitabi.ru': 'kitabi.ru',
 'ryazan.kitabi.ru': 'kitabi.ru',
 'irkutsk.kitabi.ru': 'kitabi.ru',
 'rabota.kitabi.ru': 'kitabi.ru',
 'krasnodar.kitabi.ru': 'kitabi.ru',
 'orenburg.kitabi.ru': 'kitabi.ru',
 'bryansk.kitabi.ru': 'kitabi.ru',
 'berezniki.kitabi.ru': 'kitabi.ru',
 'saransk.kitabi.ru': 'kitabi.ru',
 'vladimir.kitabi.ru': 'kitabi.ru',
 'sochi.kitabi.ru': 'kitabi.ru',
 'otzyvy.kitabi.ru': 'kitabi.ru',
 'tver.kitabi.ru': 'kitabi.ru',
 'ulyanovsk.kitabi.ru': 'kitabi.ru',
 'taganrog.kitabi.ru': 'kitabi.ru',
 'chaykovskiy.kitabi.ru': 'kitabi.ru',
 'solikamsk.kitabi.ru': 'kitabi.ru',
 'podolsk.kitabi.ru': 'kitabi.ru',
 'naberezhnye-chelny.kitabi.ru': 'kitabi.ru',
 'dzerzhinsk.kitabi.ru': 'kitabi.ru',
 'astrahan.kitabi.ru': 'kitabi.ru',
 'kursk.kitabi.ru': 'kitabi.ru',
 'kurgan.kitabi.ru': 'kitabi.ru',
 'nahodka.kitabi.ru': 'kitabi.ru',
 'vladivostok.kitabi.ru': 'kitabi.ru',
 'yoshkar-ola.kitabi.ru': 'kitabi.ru',
 'surgut.kitabi.ru': 'kitabi.ru',
 'eysk.kitabi.ru': 'kitabi.ru',
 'gelendzhik.kitabi.ru': 'kitabi.ru',
 'serpuhov.kitabi.ru': 'kitabi.ru',
 'armavir.kitabi.ru': 'kitabi.ru',
 'neftekamsk.kitabi.ru': 'kitabi.ru',
 'izhevsk.kitabi.ru': 'kitabi.ru',
 'dmitrov.kitabi.ru': 'kitabi.ru',
 'kaluga.kitabi.ru': 'kitabi.ru',
 'berdsk.kitabi.ru': 'kitabi.ru',
 'barnaul.kitabi.ru': 'kitabi.ru',
 'kolomna.kitabi.ru': 'kitabi.ru',
 'dolgoprudnyy.kitabi.ru': 'kitabi.ru',
 'vologda.kitabi.ru': 'kitabi.ru',
 'arzamas.kitabi.ru': 'kitabi.ru',
 'novorossiysk.kitabi.ru': 'kitabi.ru',
 'balahna.kitabi.ru': 'kitabi.ru',
 'gatchina.kitabi.ru': 'kitabi.ru',
 'vyborg.kitabi.ru': 'kitabi.ru',
 'belovo.kitabi.ru': 'kitabi.ru',
 'vyazma.kitabi.ru': 'kitabi.ru',
 'velikie-luki.kitabi.ru': 'kitabi.ru',
 'arhangelsk.kitabi.ru': 'kitabi.ru',
 'volzhskiy.kitabi.ru': 'kitabi.ru',
 'balakovo.kitabi.ru': 'kitabi.ru',
 'anapa.kitabi.ru': 'kitabi.ru',
 'pervouralsk.kitabi.ru': 'kitabi.ru',
 'yaroslavl.kitabi.ru': 'kitabi.ru',
 'balashiha.kitabi.ru': 'kitabi.ru',
 'bugulma.kitabi.ru': 'kitabi.ru',
 'budennovsk.kitabi.ru': 'kitabi.ru',
 'alapaevsk.kitabi.ru': 'kitabi.ru',
 'tambov.kitabi.ru': 'kitabi.ru',
 'buzuluk.kitabi.ru': 'kitabi.ru',
 'korkino.kitabi.ru': 'kitabi.ru',
 'novokuzneck.kitabi.ru': 'kitabi.ru',
 'mahachkala.kitabi.ru': 'kitabi.ru',
 'abakan.kitabi.ru': 'kitabi.ru',
 'habarovsk.kitabi.ru': 'kitabi.ru',
 'stavropol.kitabi.ru': 'kitabi.ru',
 'novocheboksarsk.kitabi.ru': 'kitabi.ru',
 'cheboksary.kitabi.ru': 'kitabi.ru',
 'voskresensk.kitabi.ru': 'kitabi.ru',
 'cherepovec.kitabi.ru': 'kitabi.ru',
 'kirov.kitabi.ru': 'kitabi.ru',
 'kirovo-chepeck.kitabi.ru': 'kitabi.ru',
 'zlatoust.kitabi.ru': 'kitabi.ru',
 'dimitrovgrad.kitabi.ru': 'kitabi.ru',
 'blagoveschensk.kitabi.ru': 'kitabi.ru',
 'kaliningrad.kitabi.ru': 'kitabi.ru',
 'zelenodolsk.kitabi.ru': 'kitabi.ru',
 'ivanovo.kitabi.ru': 'kitabi.ru',
 'achinsk.kitabi.ru': 'kitabi.ru',
 'biysk.kitabi.ru': 'kitabi.ru',
 'gus-hrustalnyy.kitabi.ru': 'kitabi.ru',
 'nizhnekamsk.kitabi.ru': 'kitabi.ru',
 'georgievsk.kitabi.ru': 'kitabi.ru',
 'essentuki.kitabi.ru': 'kitabi.ru',
 'vladikavkaz.kitabi.ru': 'kitabi.ru',
 'sterlitamak.kitabi.ru': 'kitabi.ru',
 'ulan-ude.kitabi.ru': 'kitabi.ru',
 'novomoskovsk.kitabi.ru': 'kitabi.ru',
 'ishim.kitabi.ru': 'kitabi.ru',
 'kislovodsk.kitabi.ru': 'kitabi.ru',
 'pyatigorsk.kitabi.ru': 'kitabi.ru',
 'himki.kitabi.ru': 'kitabi.ru',
 'uhta.kitabi.ru': 'kitabi.ru',
 'rubcovsk.kitabi.ru': 'kitabi.ru',
 'kamyshin.kitabi.ru': 'kitabi.ru',
 'angarsk.kitabi.ru': 'kitabi.ru',
 'glazov.kitabi.ru': 'kitabi.ru',
 'kuzneck.kitabi.ru': 'kitabi.ru',
 'kovrov.kitabi.ru': 'kitabi.ru',
 'novocherkassk.kitabi.ru': 'kitabi.ru',
 'pavlovskiy-posad.kitabi.ru': 'kitabi.ru',
 'tomsk.kitabi.ru': 'kitabi.ru',
 'pskov.kitabi.ru': 'kitabi.ru',
 'prokopevsk.kitabi.ru': 'kitabi.ru',
 'cherkessk.kitabi.ru': 'kitabi.ru',
 'korolev.kitabi.ru': 'kitabi.ru',
 'orsk.kitabi.ru': 'kitabi.ru',
 'kogalym.kitabi.ru': 'kitabi.ru',
 'usole-sibirskoe.kitabi.ru': 'kitabi.ru',
 'hanty-mansiysk.kitabi.ru': 'kitabi.ru',
 'ussuriysk.kitabi.ru': 'kitabi.ru',
 'krasnoufimsk.kitabi.ru': 'kitabi.ru',
 'kemerovo.kitabi.ru': 'kitabi.ru',
 'urga.kitabi.ru': 'kitabi.ru',
 'slavyansk-na-kubani.kitabi.ru': 'kitabi.ru',
 'sorochinsk.kitabi.ru': 'kitabi.ru',
 'yalta.kitabi.ru': 'kitabi.ru',
 'evpatoriya.kitabi.ru': 'kitabi.ru',
 'klin.kitabi.ru': 'kitabi.ru',
 'yakutsk.kitabi.ru': 'kitabi.ru',
 'polevskoy.kitabi.ru': 'kitabi.ru',
 'kasimov.kitabi.ru': 'kitabi.ru',
 'ozersk.kitabi.ru': 'kitabi.ru',
 'oktyabrskiy.kitabi.ru': 'kitabi.ru',
 'uglich.kitabi.ru': 'kitabi.ru',
 'kineshma.kitabi.ru': 'kitabi.ru',
 'uzhno-sahalinsk.kitabi.ru': 'kitabi.ru',
 'simferopol.kitabi.ru': 'kitabi.ru',
 'kostroma.kitabi.ru': 'kitabi.ru',
 'komsomolsk-na-amure.kitabi.ru': 'kitabi.ru',
 'obninsk.kitabi.ru': 'kitabi.ru',
 'kungur.kitabi.ru': 'kitabi.ru',
 'zheleznogorsk.kitabi.ru': 'kitabi.ru',
 'domodedovo.kitabi.ru': 'kitabi.ru',
 'novotroick.kitabi.ru': 'kitabi.ru',
 'murmansk.kitabi.ru': 'kitabi.ru',
 'severodvinsk.kitabi.ru': 'kitabi.ru',
 'maykop.kitabi.ru': 'kitabi.ru',
 'magadan.kitabi.ru': 'kitabi.ru',
 'groznyy.kitabi.ru': 'kitabi.ru',
 'sergiev-posad.kitabi.ru': 'kitabi.ru',
 'vsevolozhsk.kitabi.ru': 'kitabi.ru',
 'shahty.kitabi.ru': 'kitabi.ru',
 'lipeck.kitabi.ru': 'kitabi.ru',
 'magnitogorsk.kitabi.ru': 'kitabi.ru',
 'kashira.kitabi.ru': 'kitabi.ru',
 'kiselevsk.kitabi.ru': 'kitabi.ru',
 'baranovichi.kitabi.ru': 'kitabi.ru',
 'zarechnyy.kitabi.ru': 'kitabi.ru',
 'solnechnogorsk.kitabi.ru': 'kitabi.ru',
 'nalchik.kitabi.ru': 'kitabi.ru',
 'kirishi.kitabi.ru': 'kitabi.ru',
 'schelkovo.kitabi.ru': 'kitabi.ru',
 'elektrostal.kitabi.ru': 'kitabi.ru',
 'lobnya.kitabi.ru': 'kitabi.ru',
 'shatura.kitabi.ru': 'kitabi.ru',
 'luga.kitabi.ru': 'kitabi.ru',
 'petergof.kitabi.ru': 'kitabi.ru',
 'lubercy.kitabi.ru': 'kitabi.ru',
 'serov.kitabi.ru': 'kitabi.ru',
 'norilsk.kitabi.ru': 'kitabi.ru',
 'nizhnevartovsk.kitabi.ru': 'kitabi.ru',
 'chita.kitabi.ru': 'kitabi.ru',
 'nevinnomyssk.kitabi.ru': 'kitabi.ru',
 'rybinsk.kitabi.ru': 'kitabi.ru',
 'salsk.kitabi.ru': 'kitabi.ru',
 'lugansk.kitabi.ru': 'kitabi.ru',
    'svetoforonline.ru': 'svetoforonline.ru',
 'livny.svetoforonline.ru': 'svetoforonline.ru',
 'kondopoga.svetoforonline.ru': 'svetoforonline.ru',
 'msk-luhman.svetoforonline.ru': 'svetoforonline.ru',
 'spb-mineralnaya.svetoforonline.ru': 'svetoforonline.ru',
 'spb-pargolovo.svetoforonline.ru': 'svetoforonline.ru',
 'obninsk.svetoforonline.ru': 'svetoforonline.ru',
 'spb-aleksferm.svetoforonline.ru': 'svetoforonline.ru',
 'pereslavl.svetoforonline.ru': 'svetoforonline.ru',
 'spb-moskovskoe.svetoforonline.ru': 'svetoforonline.ru',
 'serkovo.svetoforonline.ru': 'svetoforonline.ru',
 'msk-salt.svetoforonline.ru': 'svetoforonline.ru',
 'msk-privolnaya.svetoforonline.ru': 'svetoforonline.ru',
 'kovrov.svetoforonline.ru': 'svetoforonline.ru',
 'zhukov.svetoforonline.ru': 'svetoforonline.ru',
 'mozhaysk.svetoforonline.ru': 'svetoforonline.ru',
 'kineshma-baha.svetoforonline.ru': 'svetoforonline.ru',
 'kursk-respubl.svetoforonline.ru': 'svetoforonline.ru',
 'smolensk-gub.svetoforonline.ru': 'svetoforonline.ru',
 'odincovo.svetoforonline.ru': 'svetoforonline.ru',
 'pahra.svetoforonline.ru': 'svetoforonline.ru',
 'rybinsk.svetoforonline.ru': 'svetoforonline.ru',
 'uglich.svetoforonline.ru': 'svetoforonline.ru',
 'pgt-verhovie.svetoforonline.ru': 'svetoforonline.ru',
 'yar-dekabristov.svetoforonline.ru': 'svetoforonline.ru',
 'kuznechiha.svetoforonline.ru': 'svetoforonline.ru',
 'rybinsk-trud.svetoforonline.ru': 'svetoforonline.ru',
 'pravdinskiy.svetoforonline.ru': 'svetoforonline.ru',
 'balashiha2.svetoforonline.ru': 'svetoforonline.ru',
 'st-oskol-meb.svetoforonline.ru': 'svetoforonline.ru',
 'chehov-sad.svetoforonline.ru': 'svetoforonline.ru',
 'voskr-zav.svetoforonline.ru': 'svetoforonline.ru',
 'msk-lerm.svetoforonline.ru': 'svetoforonline.ru',
 'petergof.svetoforonline.ru': 'svetoforonline.ru',
 'kaliningradskaya-oblast.svetoforonline.ru': 'svetoforonline.ru',
 'ryazan-vysvolt.svetoforonline.ru': 'svetoforonline.ru',
 'pehlec.svetoforonline.ru': 'svetoforonline.ru',
 'porhov.svetoforonline.ru': 'svetoforonline.ru',
 'ryazan-keram.svetoforonline.ru': 'svetoforonline.ru',
 'ryazan-druzhba.svetoforonline.ru': 'svetoforonline.ru',
 'vladimir.svetoforonline.ru': 'svetoforonline.ru',
 'biryuch.svetoforonline.ru': 'svetoforonline.ru',
 'belgorod-dekabristov.svetoforonline.ru': 'svetoforonline.ru',
 'schyolkovo.svetoforonline.ru': 'svetoforonline.ru',
 'sosenki.svetoforonline.ru': 'svetoforonline.ru',
 'rossosh.svetoforonline.ru': 'svetoforonline.ru',
 'borisovka.svetoforonline.ru': 'svetoforonline.ru',
 'muchurinsk-lavrova.svetoforonline.ru': 'svetoforonline.ru',
 'apatity.svetoforonline.ru': 'svetoforonline.ru',
 'tula-surikova.svetoforonline.ru': 'svetoforonline.ru',
 'tula-marata.svetoforonline.ru': 'svetoforonline.ru',
 'sosnenskoe.svetoforonline.ru': 'svetoforonline.ru',
 'shilovo.svetoforonline.ru': 'svetoforonline.ru',
 'vichuga.svetoforonline.ru': 'svetoforonline.ru',
 'kandalaksha.svetoforonline.ru': 'svetoforonline.ru',
 'belgorod-magistral.svetoforonline.ru': 'svetoforonline.ru',
 'pervomayskoe.svetoforonline.ru': 'svetoforonline.ru',
 'lebedyan.svetoforonline.ru': 'svetoforonline.ru',
 'noginsk.svetoforonline.ru': 'svetoforonline.ru',
 'smolensk-kiev.svetoforonline.ru': 'svetoforonline.ru',
 'tihvinka.svetoforonline.ru': 'svetoforonline.ru',
 'kolomna-inter.svetoforonline.ru': 'svetoforonline.ru',
 'moskovskaya-oblast.svetoforonline.ru': 'svetoforonline.ru',
 'kolomna-800.svetoforonline.ru': 'svetoforonline.ru',
 'yurovo.svetoforonline.ru': 'svetoforonline.ru',
 'ivanovo.svetoforonline.ru': 'svetoforonline.ru',
 'msk-vernisazhnaya.svetoforonline.ru': 'svetoforonline.ru',
 'mihaylov.svetoforonline.ru': 'svetoforonline.ru',
 'kimovsk.svetoforonline.ru': 'svetoforonline.ru',
 'kirov.svetoforonline.ru': 'svetoforonline.ru',
 'leningradskaya-oblast.svetoforonline.ru': 'svetoforonline.ru',
 'gubkin-bel.svetoforonline.ru': 'svetoforonline.ru',
 'msk-radialnaya.svetoforonline.ru': 'svetoforonline.ru',
 'bulatnikovskoe.svetoforonline.ru': 'svetoforonline.ru',
 'msk-krasnobogat.svetoforonline.ru': 'svetoforonline.ru',
 'petrozavodsk.svetoforonline.ru': 'svetoforonline.ru',
 'petrozavodsk2.svetoforonline.ru': 'svetoforonline.ru',
 'petrozavodsk3.svetoforonline.ru': 'svetoforonline.ru',
 'yar-papanina.svetoforonline.ru': 'svetoforonline.ru',
 'msk-dorog.svetoforonline.ru': 'svetoforonline.ru',
 'kolomna-ozerskoe.svetoforonline.ru': 'svetoforonline.ru',
 'pavlov-posad.svetoforonline.ru': 'svetoforonline.ru',
 'msk-ogorodny.svetoforonline.ru': 'svetoforonline.ru',
 'suvorov.svetoforonline.ru': 'svetoforonline.ru',
 'rzhev-sadovaya.svetoforonline.ru': 'svetoforonline.ru',
 'lyubercy.svetoforonline.ru': 'svetoforonline.ru',
 'pershutino.svetoforonline.ru': 'svetoforonline.ru',
 'klin.svetoforonline.ru': 'svetoforonline.ru',
 'raduzhniy.svetoforonline.ru': 'svetoforonline.ru',
 'tambov-michur.svetoforonline.ru': 'svetoforonline.ru',
 'msk-kol.svetoforonline.ru': 'svetoforonline.ru',
 'lipeck-kovaleva.svetoforonline.ru': 'svetoforonline.ru',
 'stupino-kr.svetoforonline.ru': 'svetoforonline.ru',
 'kaluga-akadem.svetoforonline.ru': 'svetoforonline.ru',
 'msk-desna.svetoforonline.ru': 'svetoforonline.ru',
 'kaluga-grabcevskoe.svetoforonline.ru': 'svetoforonline.ru',
 'kaluga-guryanova.svetoforonline.ru': 'svetoforonline.ru',
 'kaluga-rezvanskaya.svetoforonline.ru': 'svetoforonline.ru',
 'gubkin-yug.svetoforonline.ru': 'svetoforonline.ru',
 'mytishchi-yar.svetoforonline.ru': 'svetoforonline.ru',
 'rasskazovo.svetoforonline.ru': 'svetoforonline.ru',
 'kos-volzhskaya.svetoforonline.ru': 'svetoforonline.ru',
 'msk-varshav.svetoforonline.ru': 'svetoforonline.ru',
 'elec-kostenko.svetoforonline.ru': 'svetoforonline.ru',
 'elec.svetoforonline.ru': 'svetoforonline.ru',
 'kameshkovo.svetoforonline.ru': 'svetoforonline.ru',
 'balashiha.svetoforonline.ru': 'svetoforonline.ru',
 'tver-suvorova.svetoforonline.ru': 'svetoforonline.ru',
 'elizavetino.svetoforonline.ru': 'svetoforonline.ru',
 'kohma.svetoforonline.ru': 'svetoforonline.ru',
 'tekstilprofi.svetoforonline.ru': 'svetoforonline.ru',
 'kirzhach.svetoforonline.ru': 'svetoforonline.ru',
 'tver-shishkova.svetoforonline.ru': 'svetoforonline.ru',
 'podrezkovo.svetoforonline.ru': 'svetoforonline.ru',
 'msk-yar.svetoforonline.ru': 'svetoforonline.ru',
 'podolsk.svetoforonline.ru': 'svetoforonline.ru',
 'orehovo-zuevo.svetoforonline.ru': 'svetoforonline.ru',
 'msk-verhpol.svetoforonline.ru': 'svetoforonline.ru',
 'borisoglebsk.svetoforonline.ru': 'svetoforonline.ru',
 'gribki.svetoforonline.ru': 'svetoforonline.ru',
 'pskovskaya-oblast.svetoforonline.ru': 'svetoforonline.ru',
 'galich.svetoforonline.ru': 'svetoforonline.ru',
 'kursk-novosel.svetoforonline.ru': 'svetoforonline.ru',
 'kursk-litovskaya.svetoforonline.ru': 'svetoforonline.ru',
 'kursk-dmitrova.svetoforonline.ru': 'svetoforonline.ru',
 'bogorodick.svetoforonline.ru': 'svetoforonline.ru',
 'novo-talicy.svetoforonline.ru': 'svetoforonline.ru',
 'shuya.svetoforonline.ru': 'svetoforonline.ru',
 'rodniki.svetoforonline.ru': 'svetoforonline.ru',
 'dyatlovka.svetoforonline.ru': 'svetoforonline.ru',
 'orel-kromskoy.svetoforonline.ru': 'svetoforonline.ru',
 'pavlovsk.svetoforonline.ru': 'svetoforonline.ru',
 'yar-gromova.svetoforonline.ru': 'svetoforonline.ru',
 'msk-nizh.svetoforonline.ru': 'svetoforonline.ru',
 'msk-bor.svetoforonline.ru': 'svetoforonline.ru',
 'yarcevo.svetoforonline.ru': 'svetoforonline.ru',
 'maloyaroslavec.svetoforonline.ru': 'svetoforonline.ru',
 'kolchugino.svetoforonline.ru': 'svetoforonline.ru',
 'tula-venevskoe.svetoforonline.ru': 'svetoforonline.ru',
 'ryazan-ryazanskaya.svetoforonline.ru': 'svetoforonline.ru',
 'povarovo.svetoforonline.ru': 'svetoforonline.ru',
 'domodedovo.svetoforonline.ru': 'svetoforonline.ru',
 'himki.svetoforonline.ru': 'svetoforonline.ru',
 'likino-dulyovo.svetoforonline.ru': 'svetoforonline.ru',
 'tambov-bastionnaya.svetoforonline.ru': 'svetoforonline.ru',
 'ramenskoe.svetoforonline.ru': 'svetoforonline.ru',
 'mcensk.svetoforonline.ru': 'svetoforonline.ru',
 'safonovo.svetoforonline.ru': 'svetoforonline.ru',
 'dankov.svetoforonline.ru': 'svetoforonline.ru',
 'buharovo.svetoforonline.ru': 'svetoforonline.ru',
 'bryansk-lit.svetoforonline.ru': 'svetoforonline.ru',
 'tver-kominterna.svetoforonline.ru': 'svetoforonline.ru',
 'peremerki-big.svetoforonline.ru': 'svetoforonline.ru',
 'vishny-volochek.svetoforonline.ru': 'svetoforonline.ru',
 'lakinsk.svetoforonline.ru': 'svetoforonline.ru',
 'konakovo.svetoforonline.ru': 'svetoforonline.ru',
 'murom.svetoforonline.ru': 'svetoforonline.ru',
 'ryazan-novaya.svetoforonline.ru': 'svetoforonline.ru',
 'michurinsk-sovetskaya.svetoforonline.ru': 'svetoforonline.ru',
 'kos-tkachey.svetoforonline.ru': 'svetoforonline.ru',
 'orehovo-zuevo-urick.svetoforonline.ru': 'svetoforonline.ru',
 'voronezh-dubrovina.svetoforonline.ru': 'svetoforonline.ru',
 'vologodskaya-oblast.svetoforonline.ru': 'svetoforonline.ru',
 'ruza.svetoforonline.ru': 'svetoforonline.ru',
 'krasnoarmeysk.svetoforonline.ru': 'svetoforonline.ru',
 'kineshma-goroh.svetoforonline.ru': 'svetoforonline.ru',
 'sergiev-posad-msk.svetoforonline.ru': 'svetoforonline.ru',
 'nerehta.svetoforonline.ru': 'svetoforonline.ru',
 'dyatkovo.svetoforonline.ru': 'svetoforonline.ru',
 'tula-newmoscow.svetoforonline.ru': 'svetoforonline.ru',
 'zvenigorod.svetoforonline.ru': 'svetoforonline.ru',
 'tula-newtul.svetoforonline.ru': 'svetoforonline.ru',
 'uvarovo.svetoforonline.ru': 'svetoforonline.ru',
 'yar-fedorovskaya.svetoforonline.ru': 'svetoforonline.ru',
 'pogar.svetoforonline.ru': 'svetoforonline.ru',
 'dmitrov.svetoforonline.ru': 'svetoforonline.ru',
 'efremov.svetoforonline.ru': 'svetoforonline.ru',
 'torzhok.svetoforonline.ru': 'svetoforonline.ru',
 'roslavl.svetoforonline.ru': 'svetoforonline.ru',
 'repnoe.svetoforonline.ru': 'svetoforonline.ru',
 'ivanovka.svetoforonline.ru': 'svetoforonline.ru',
 'bezheck.svetoforonline.ru': 'svetoforonline.ru',
 'yurev-polskiy.svetoforonline.ru': 'svetoforonline.ru',
 'spb-hrust.svetoforonline.ru': 'svetoforonline.ru',
 'kursk-agregat.svetoforonline.ru': 'svetoforonline.ru',
 'kursk-kosuhina.svetoforonline.ru': 'svetoforonline.ru',
 'sharya.svetoforonline.ru': 'svetoforonline.ru',
 'liski.svetoforonline.ru': 'svetoforonline.ru',
 'vyazniki.svetoforonline.ru': 'svetoforonline.ru',
 'skopin.svetoforonline.ru': 'svetoforonline.ru',
 'aleksandrov.svetoforonline.ru': 'svetoforonline.ru',
 'novomoskovsk.svetoforonline.ru': 'svetoforonline.ru',
 'donskoy.svetoforonline.ru': 'svetoforonline.ru',
 'kovrov2.svetoforonline.ru': 'svetoforonline.ru',
    'yavitrina.ru': 'yavitrina.ru',
 'krasnoyarsk.yavitrina.ru': 'yavitrina.ru',
 'naro-fominsk.yavitrina.ru': 'yavitrina.ru',
 'msk.yavitrina.ru': 'yavitrina.ru',
 'habarovsk.yavitrina.ru': 'yavitrina.ru',
 'vladivostok.yavitrina.ru': 'yavitrina.ru',
 'samara.yavitrina.ru': 'yavitrina.ru',
 'gorno-altaysk.yavitrina.ru': 'yavitrina.ru',
 'krasnodar.yavitrina.ru': 'yavitrina.ru',
 'belgorod.yavitrina.ru': 'yavitrina.ru',
 'spb.yavitrina.ru': 'yavitrina.ru',
 'yaroslavl.yavitrina.ru': 'yavitrina.ru',
 'perm.yavitrina.ru': 'yavitrina.ru',
 'voskresensk.yavitrina.ru': 'yavitrina.ru',
 'kaliningrad.yavitrina.ru': 'yavitrina.ru',
 'novorossiysk.yavitrina.ru': 'yavitrina.ru',
 'omsk.yavitrina.ru': 'yavitrina.ru',
 'ekaterinburg.yavitrina.ru': 'yavitrina.ru',
 'novosibirsk.yavitrina.ru': 'yavitrina.ru',
 'velikie-luki.yavitrina.ru': 'yavitrina.ru',
 'orenburg.yavitrina.ru': 'yavitrina.ru',
 'tyumen.yavitrina.ru': 'yavitrina.ru',
 'saratov.yavitrina.ru': 'yavitrina.ru',
 'kostroma.yavitrina.ru': 'yavitrina.ru',
 'tolyatti.yavitrina.ru': 'yavitrina.ru',
 'nizhniy-novgorod.yavitrina.ru': 'yavitrina.ru',
 'tambov.yavitrina.ru': 'yavitrina.ru',
 'ob.yavitrina.ru': 'yavitrina.ru',
 'rostov-na-donu.yavitrina.ru': 'yavitrina.ru',
 'barnaul.yavitrina.ru': 'yavitrina.ru',
 'nizhnekamsk.yavitrina.ru': 'yavitrina.ru',
 'pavlovskiy-posad.yavitrina.ru': 'yavitrina.ru',
 'ufa.yavitrina.ru': 'yavitrina.ru',
 'kaluga.yavitrina.ru': 'yavitrina.ru',
 'naberezhnye-chelny.yavitrina.ru': 'yavitrina.ru',
 'tomsk.yavitrina.ru': 'yavitrina.ru',
 'stupino.yavitrina.ru': 'yavitrina.ru',
 'berezniki.yavitrina.ru': 'yavitrina.ru',
 'petropavlovsk-kamchatskiy.yavitrina.ru': 'yavitrina.ru',
 'chelyabinsk.yavitrina.ru': 'yavitrina.ru',
 'essentuki.yavitrina.ru': 'yavitrina.ru',
 'tver.yavitrina.ru': 'yavitrina.ru',
 'anapa.yavitrina.ru': 'yavitrina.ru',
 'sterlitamak.yavitrina.ru': 'yavitrina.ru',
 'ryazan.yavitrina.ru': 'yavitrina.ru',
 'saransk.yavitrina.ru': 'yavitrina.ru',
 'izhevsk.yavitrina.ru': 'yavitrina.ru',
 'kazan.yavitrina.ru': 'yavitrina.ru',
 'voronezh.yavitrina.ru': 'yavitrina.ru',
 'zhukovskiy.yavitrina.ru': 'yavitrina.ru',
 'ivanovo.yavitrina.ru': 'yavitrina.ru',
 'ulyanovsk.yavitrina.ru': 'yavitrina.ru',
 'bryansk.yavitrina.ru': 'yavitrina.ru',
 'nizhniy-tagil.yavitrina.ru': 'yavitrina.ru',
 'eysk.yavitrina.ru': 'yavitrina.ru',
 'kovrov.yavitrina.ru': 'yavitrina.ru',
 'komsomolsk-na-amure.yavitrina.ru': 'yavitrina.ru',
 'astrahan.yavitrina.ru': 'yavitrina.ru',
 'syzran.yavitrina.ru': 'yavitrina.ru',
 'sochi.yavitrina.ru': 'yavitrina.ru',
 'yoshkar-ola.yavitrina.ru': 'yavitrina.ru',
 'pushkino.yavitrina.ru': 'yavitrina.ru',
 'tula.yavitrina.ru': 'yavitrina.ru',
 'blagoveshchensk.yavitrina.ru': 'yavitrina.ru',
 'domodedovo.yavitrina.ru': 'yavitrina.ru',
 'podolsk.yavitrina.ru': 'yavitrina.ru',
 'smolensk.yavitrina.ru': 'yavitrina.ru',
 'balashiha.yavitrina.ru': 'yavitrina.ru',
 'aksay.yavitrina.ru': 'yavitrina.ru',
 'dolgoprudnyy.yavitrina.ru': 'yavitrina.ru',
 'odincovo.yavitrina.ru': 'yavitrina.ru',
 'vyksa.yavitrina.ru': 'yavitrina.ru',
 'fryazino.yavitrina.ru': 'yavitrina.ru',
 'nahodka.yavitrina.ru': 'yavitrina.ru',
 'lipeck.yavitrina.ru': 'yavitrina.ru',
 'vologda.yavitrina.ru': 'yavitrina.ru',
 'mozhaysk.yavitrina.ru': 'yavitrina.ru',
 'engels.yavitrina.ru': 'yavitrina.ru',
 'volokolamsk.yavitrina.ru': 'yavitrina.ru',
 'shchelkovo.yavitrina.ru': 'yavitrina.ru',
 'birobidzhan.yavitrina.ru': 'yavitrina.ru',
 'magnitogorsk.yavitrina.ru': 'yavitrina.ru',
 'maykop.yavitrina.ru': 'yavitrina.ru',
 'abakan.yavitrina.ru': 'yavitrina.ru',
 'shahty.yavitrina.ru': 'yavitrina.ru',
 'ramenskoe.yavitrina.ru': 'yavitrina.ru',
 'vladimir.yavitrina.ru': 'yavitrina.ru',
 'murmansk.yavitrina.ru': 'yavitrina.ru',
 'biysk.yavitrina.ru': 'yavitrina.ru',
 'elec.yavitrina.ru': 'yavitrina.ru',
 'cheboksary.yavitrina.ru': 'yavitrina.ru',
 'korolyov.yavitrina.ru': 'yavitrina.ru',
 'zvenigorod.yavitrina.ru': 'yavitrina.ru',
 'nizhnevartovsk.yavitrina.ru': 'yavitrina.ru',
 'dzerzhinsk.yavitrina.ru': 'yavitrina.ru',
 'salavat.yavitrina.ru': 'yavitrina.ru',
 'istra.yavitrina.ru': 'yavitrina.ru',
 'obninsk.yavitrina.ru': 'yavitrina.ru',
 'gelendzhik.yavitrina.ru': 'yavitrina.ru',
 'stavropol.yavitrina.ru': 'yavitrina.ru',
 'petrozavodsk.yavitrina.ru': 'yavitrina.ru',
 'vorkuta.yavitrina.ru': 'yavitrina.ru',
 'verhnyaya-pyshma.yavitrina.ru': 'yavitrina.ru',
 'revda.yavitrina.ru': 'yavitrina.ru',
 'kirov.yavitrina.ru': 'yavitrina.ru',
 'shatura.yavitrina.ru': 'yavitrina.ru',
 'severodvinsk.yavitrina.ru': 'yavitrina.ru',
 'pervouralsk.yavitrina.ru': 'yavitrina.ru',
 'pskov.yavitrina.ru': 'yavitrina.ru',
 'orsk.yavitrina.ru': 'yavitrina.ru',
 'magadan.yavitrina.ru': 'yavitrina.ru',
 'rybinsk.yavitrina.ru': 'yavitrina.ru',
 'kurgan.yavitrina.ru': 'yavitrina.ru',
 'volhov.yavitrina.ru': 'yavitrina.ru',
 'tobolsk.yavitrina.ru': 'yavitrina.ru',
 'arhangelsk.yavitrina.ru': 'yavitrina.ru',
 'evpatoriya.yavitrina.ru': 'yavitrina.ru',
 'serpuhov.yavitrina.ru': 'yavitrina.ru',
 'penza.yavitrina.ru': 'yavitrina.ru',
 'arzamas.yavitrina.ru': 'yavitrina.ru',
 'mytishchi.yavitrina.ru': 'yavitrina.ru',
 'elektrostal.yavitrina.ru': 'yavitrina.ru',
 'simferopol.yavitrina.ru': 'yavitrina.ru',
 'bor.yavitrina.ru': 'yavitrina.ru',
 'nefteyugansk.yavitrina.ru': 'yavitrina.ru',
 'almetevsk.yavitrina.ru': 'yavitrina.ru',
 'kimry.yavitrina.ru': 'yavitrina.ru',
 'pyatigorsk.yavitrina.ru': 'yavitrina.ru',
 'himki.yavitrina.ru': 'yavitrina.ru',
 'oryol.yavitrina.ru': 'yavitrina.ru',
 'lyubercy.yavitrina.ru': 'yavitrina.ru',
 'kolomna.yavitrina.ru': 'yavitrina.ru',
 'novokuzneck.yavitrina.ru': 'yavitrina.ru',
 'feodosiya.yavitrina.ru': 'yavitrina.ru',
 'kursk.yavitrina.ru': 'yavitrina.ru',
 'surgut.yavitrina.ru': 'yavitrina.ru',
 'tuapse.yavitrina.ru': 'yavitrina.ru',
 'aleksandrov.yavitrina.ru': 'yavitrina.ru',
 'taganrog.yavitrina.ru': 'yavitrina.ru',
 'dimitrovgrad.yavitrina.ru': 'yavitrina.ru',
 'vladikavkaz.yavitrina.ru': 'yavitrina.ru',
 'beryozovskiy.yavitrina.ru': 'yavitrina.ru',
 'vsevolozhsk.yavitrina.ru': 'yavitrina.ru',
 'yakutsk.yavitrina.ru': 'yavitrina.ru',
 'nalchik.yavitrina.ru': 'yavitrina.ru',
 'mezhdurechensk.yavitrina.ru': 'yavitrina.ru',
 'yuzhno-sahalinsk.yavitrina.ru': 'yavitrina.ru',
 'kemerovo.yavitrina.ru': 'yavitrina.ru',
 'kamensk-uralskiy.yavitrina.ru': 'yavitrina.ru',
 'noyabrsk.yavitrina.ru': 'yavitrina.ru',
 'solnechnogorsk.yavitrina.ru': 'yavitrina.ru',
 'kamensk_shahtinskiy.yavitrina.ru': 'yavitrina.ru',
 'sergiev-posad.yavitrina.ru': 'yavitrina.ru',
 'irkutsk.yavitrina.ru': 'yavitrina.ru',
 'velikiy-novgorod.yavitrina.ru': 'yavitrina.ru',
 'egorevsk.yavitrina.ru': 'yavitrina.ru',
 'balakovo.yavitrina.ru': 'yavitrina.ru',
 'staryy-oskol.yavitrina.ru': 'yavitrina.ru',
 'volgograd.yavitrina.ru': 'yavitrina.ru',
 'kashira.yavitrina.ru': 'yavitrina.ru',
 'volgodonsk.yavitrina.ru': 'yavitrina.ru',
 'murom.yavitrina.ru': 'yavitrina.ru',
 'serov.yavitrina.ru': 'yavitrina.ru',
 'groznyy.yavitrina.ru': 'yavitrina.ru',
 'ussuriysk.yavitrina.ru': 'yavitrina.ru',
 'lobnya.yavitrina.ru': 'yavitrina.ru',
 'kopeysk.yavitrina.ru': 'yavitrina.ru',
 'armavir.yavitrina.ru': 'yavitrina.ru',
 'vidnoe.yavitrina.ru': 'yavitrina.ru',
 'chehov.yavitrina.ru': 'yavitrina.ru',
 'dmitrov.yavitrina.ru': 'yavitrina.ru',
 'cherepovec.yavitrina.ru': 'yavitrina.ru',
 'orehovo-zuevo.yavitrina.ru': 'yavitrina.ru',
 'klin.yavitrina.ru': 'yavitrina.ru',
 'nadym.yavitrina.ru': 'yavitrina.ru',
 'miass.yavitrina.ru': 'yavitrina.ru',
 'salsk.yavitrina.ru': 'yavitrina.ru',
 'krasnogorsk.yavitrina.ru': 'yavitrina.ru',
 'mahachkala.yavitrina.ru': 'yavitrina.ru',
 'zelenograd.yavitrina.ru': 'yavitrina.ru',
 'kogalym.yavitrina.ru': 'yavitrina.ru',
 'uhta.yavitrina.ru': 'yavitrina.ru',
 'vyshniy-volochyok.yavitrina.ru': 'yavitrina.ru',
 'luhovicy.yavitrina.ru': 'yavitrina.ru',
 'gatchina.yavitrina.ru': 'yavitrina.ru',
 'buzuluk.yavitrina.ru': 'yavitrina.ru',
 'tihvin.yavitrina.ru': 'yavitrina.ru',
 'syktyvkar.yavitrina.ru': 'yavitrina.ru',
 'usinsk.yavitrina.ru': 'yavitrina.ru',
 'michurinsk.yavitrina.ru': 'yavitrina.ru',
 'dubna.yavitrina.ru': 'yavitrina.ru',
 'aprelevka.yavitrina.ru': 'yavitrina.ru',
 'kirzhach.yavitrina.ru': 'yavitrina.ru',
 'berdsk.yavitrina.ru': 'yavitrina.ru',
 'reutov.yavitrina.ru': 'yavitrina.ru',
    'barnaul.barahla.net': 'barahla.net',
 'ivanovo.barahla.net': 'barahla.net',
 'barahla.net': 'barahla.net',
 'yo.barahla.net': 'barahla.net',
 'kostroma.barahla.net': 'barahla.net',
 'msk.barahla.net': 'barahla.net',
 'orenburg.barahla.net': 'barahla.net',
 'biysk.barahla.net': 'barahla.net',
 'krasnodar.barahla.net': 'barahla.net',
 'kemerovo.barahla.net': 'barahla.net',
 'str.barahla.net': 'barahla.net',
 'tyumen.barahla.net': 'barahla.net',
 'nv.barahla.net': 'barahla.net',
 'yakutsk.barahla.net': 'barahla.net',
 'sakhalin.barahla.net': 'barahla.net',
 'abakan.barahla.net': 'barahla.net',
 'tula.barahla.net': 'barahla.net',
 'rostov.barahla.net': 'barahla.net',
 'rybinsk.barahla.net': 'barahla.net',
 'eysk.barahla.net': 'barahla.net',
 'chelny.barahla.net': 'barahla.net',
 'zelenograd.barahla.net': 'barahla.net',
 'oskol.barahla.net': 'barahla.net',
 'yuzhnouralsk.barahla.net': 'barahla.net',
 'nefteyugansk.barahla.net': 'barahla.net',
 'kopeysk.barahla.net': 'barahla.net',
 'vladimir.barahla.net': 'barahla.net',
 'belgorod.barahla.net': 'barahla.net',
 'ulan-ude.barahla.net': 'barahla.net',
 'mahachkala.barahla.net': 'barahla.net',
 'sochi.barahla.net': 'barahla.net',
 'belorechensk.barahla.net': 'barahla.net',
 'eburg.barahla.net': 'barahla.net',
 'chel.barahla.net': 'barahla.net',
 'ku.barahla.net': 'barahla.net',
 'vologda.barahla.net': 'barahla.net',
 'schelkovo.barahla.net': 'barahla.net',
 'gatchina.barahla.net': 'barahla.net',
 'omsk.barahla.net': 'barahla.net',
 'spb.barahla.net': 'barahla.net',
 'krasnoyarsk.barahla.net': 'barahla.net',
 'magadan.barahla.net': 'barahla.net',
 'gorno-altaysk.barahla.net': 'barahla.net',
 'samara.barahla.net': 'barahla.net',
 'blag.barahla.net': 'barahla.net',
 'kurganinsk.barahla.net': 'barahla.net',
 'nalchik.barahla.net': 'barahla.net',
 'anapa.barahla.net': 'barahla.net',
 'furmanov.barahla.net': 'barahla.net',
 'perm.barahla.net': 'barahla.net',
 'kyzyl.barahla.net': 'barahla.net',
 'asha.barahla.net': 'barahla.net',
 'stavropol.barahla.net': 'barahla.net',
 'novgorod.barahla.net': 'barahla.net',
 'ns.barahla.net': 'barahla.net',
 'norilsk.barahla.net': 'barahla.net',
 'volgodonsk.barahla.net': 'barahla.net',
 'tol.barahla.net': 'barahla.net',
 'kirov.barahla.net': 'barahla.net',
 'balakovo.barahla.net': 'barahla.net',
 'berezniki.barahla.net': 'barahla.net',
 'saratov.barahla.net': 'barahla.net',
 'noyabrsk.barahla.net': 'barahla.net',
 'lipetsk.barahla.net': 'barahla.net',
 'armavir.barahla.net': 'barahla.net',
 'klin.barahla.net': 'barahla.net',
 'dmitrov.barahla.net': 'barahla.net',
 'surgut.barahla.net': 'barahla.net',
 'odintsovo.barahla.net': 'barahla.net',
 'noviyurengoy.barahla.net': 'barahla.net',
 'zhukovskiy.barahla.net': 'barahla.net',
 'podolsk.barahla.net': 'barahla.net',
 'zheleznogorsk.barahla.net': 'barahla.net',
 'izhevsk.barahla.net': 'barahla.net',
 'rzn.barahla.net': 'barahla.net',
 'shahty.barahla.net': 'barahla.net',
 'orel.barahla.net': 'barahla.net',
 'elets.barahla.net': 'barahla.net',
 'ufa.barahla.net': 'barahla.net',
 'tuapse.barahla.net': 'barahla.net',
 'satka.barahla.net': 'barahla.net',
 'novoaltaysk.barahla.net': 'barahla.net',
 'prokopevsk.barahla.net': 'barahla.net',
 'troitsk.barahla.net': 'barahla.net',
 'labinsk.barahla.net': 'barahla.net',
 'volgograd.barahla.net': 'barahla.net',
 'smolensk.barahla.net': 'barahla.net',
 'novotroitsk.barahla.net': 'barahla.net',
 'novokuznetsk.barahla.net': 'barahla.net',
 'belovo.barahla.net': 'barahla.net',
 'novocheb.barahla.net': 'barahla.net',
 'serov.barahla.net': 'barahla.net',
 'tomsk.barahla.net': 'barahla.net',
 'vl.barahla.net': 'barahla.net',
 'kaz.barahla.net': 'barahla.net',
 'pechora.barahla.net': 'barahla.net',
 'gelendzhik.barahla.net': 'barahla.net',
 'magnitogorsk.barahla.net': 'barahla.net',
 'ul.barahla.net': 'barahla.net',
 'timashevsk.barahla.net': 'barahla.net',
 'kms.barahla.net': 'barahla.net',
 'dolgoprudny.barahla.net': 'barahla.net',
 'kineshma.barahla.net': 'barahla.net',
 'balashiha.barahla.net': 'barahla.net',
 'orsk.barahla.net': 'barahla.net',
 'habarovsk.barahla.net': 'barahla.net',
 'mytischi.barahla.net': 'barahla.net',
 'kolomna.barahla.net': 'barahla.net',
 'oktyabrskiy.barahla.net': 'barahla.net',
 'nizhnekamsk.barahla.net': 'barahla.net',
 'tver.barahla.net': 'barahla.net',
 'nn.barahla.net': 'barahla.net',
 'cheb.barahla.net': 'barahla.net',
 'chita.barahla.net': 'barahla.net',
 'arhangelsk.barahla.net': 'barahla.net',
 'kurgan.barahla.net': 'barahla.net',
 'yar.barahla.net': 'barahla.net',
 'syzran.barahla.net': 'barahla.net',
 'pk.barahla.net': 'barahla.net',
 'arzamas.barahla.net': 'barahla.net',
 'engels.barahla.net': 'barahla.net',
 'severodvinsk.barahla.net': 'barahla.net',
 'novokuybyshevsk.barahla.net': 'barahla.net',
 'elektrostal.barahla.net': 'barahla.net',
 'cher.barahla.net': 'barahla.net',
 'kovrov.barahla.net': 'barahla.net',
 'sergiev-posad.barahla.net': 'barahla.net',
 'pskov.barahla.net': 'barahla.net',
 'himki.barahla.net': 'barahla.net',
 'kaliningrad.barahla.net': 'barahla.net',
 'vorkuta.barahla.net': 'barahla.net',
 'berdsk.barahla.net': 'barahla.net',
 'neftekamsk.barahla.net': 'barahla.net',
 'dz.barahla.net': 'barahla.net',
 'tagil.barahla.net': 'barahla.net',
 'murmansk.barahla.net': 'barahla.net',
 'nr.barahla.net': 'barahla.net',
 'kislovodsk.barahla.net': 'barahla.net',
 'voronezh.barahla.net': 'barahla.net',
 'salavat.barahla.net': 'barahla.net',
 'slavyansk-na-kubani.barahla.net': 'barahla.net',
 'saransk.barahla.net': 'barahla.net',
 'zelenogorsk.barahla.net': 'barahla.net',
 'bor.barahla.net': 'barahla.net',
 'irkutsk.barahla.net': 'barahla.net',
 'kropotkin.barahla.net': 'barahla.net',
 'minusinsk.barahla.net': 'barahla.net',
 'oz.barahla.net': 'barahla.net',
 'kaluga.barahla.net': 'barahla.net',
 'ussur.barahla.net': 'barahla.net',
 'syktyvkar.barahla.net': 'barahla.net',
 'penza.barahla.net': 'barahla.net',
 'tambov.barahla.net': 'barahla.net',
 'vladikavkaz.barahla.net': 'barahla.net',
 'maykop.barahla.net': 'barahla.net',
 'hm.barahla.net': 'barahla.net',
 'petrozavodsk.barahla.net': 'barahla.net',
 'pushkino.barahla.net': 'barahla.net',
 'sarapul.barahla.net': 'barahla.net',
 'artem.barahla.net': 'barahla.net',
 'apsheronsk.barahla.net': 'barahla.net',
 'voskresensk.barahla.net': 'barahla.net',
 'tihoretsk.barahla.net': 'barahla.net',
 'leninsk-kuz.barahla.net': 'barahla.net',
 'vyborg.barahla.net': 'barahla.net',
 'glazov.barahla.net': 'barahla.net',
 'korolev.barahla.net': 'barahla.net',
 'bryansk.barahla.net': 'barahla.net',
 'balashov.barahla.net': 'barahla.net',
 'rubtsovsk.barahla.net': 'barahla.net',
 'miass.barahla.net': 'barahla.net',
 'ramenskoe.barahla.net': 'barahla.net',
 'chebarkul.barahla.net': 'barahla.net',
 'krymsk.barahla.net': 'barahla.net',
 'bataysk.barahla.net': 'barahla.net',
 'pervouralsk.barahla.net': 'barahla.net',
 'zheldor.barahla.net': 'barahla.net',
 'nakhodka.barahla.net': 'barahla.net',
 'serpuhov.barahla.net': 'barahla.net',
 'zlatoust.barahla.net': 'barahla.net',
 'obninsk.barahla.net': 'barahla.net',
 'tag.barahla.net': 'barahla.net',
 'teykovo.barahla.net': 'barahla.net',
 'city.barahla.net': 'barahla.net',
 'novoshahtinsk.barahla.net': 'barahla.net',
 'usinsk.barahla.net': 'barahla.net',
 'krasnogorsk.barahla.net': 'barahla.net',
 'achinsk.barahla.net': 'barahla.net',
 'kursk.barahla.net': 'barahla.net',
 'novomoskovsk.barahla.net': 'barahla.net',
 'kansk.barahla.net': 'barahla.net',
 'russia.barahla.net': 'barahla.net',
 'domodedovo.barahla.net': 'barahla.net',
 'nevinka.barahla.net': 'barahla.net',
 'kamyshin.barahla.net': 'barahla.net',
 'novocherkassk.barahla.net': 'barahla.net',
    'vh296.timeweb.ru': 'timeweb.ru',
 'bitrix148.timeweb.ru': 'timeweb.ru',
 'sdq09.timeweb.ru': 'timeweb.ru',
 'vh18.timeweb.ru': 'timeweb.ru',
 'vh32.timeweb.ru': 'timeweb.ru',
 'vh210.timeweb.ru': 'timeweb.ru',
 'vh382.timeweb.ru': 'timeweb.ru',
 'vh380.timeweb.ru': 'timeweb.ru',
 'vh388.timeweb.ru': 'timeweb.ru',
 'vh106.timeweb.ru': 'timeweb.ru',
 'vh70.timeweb.ru': 'timeweb.ru',
 'vh108.timeweb.ru': 'timeweb.ru',
 'vh202.timeweb.ru': 'timeweb.ru',
 'vh84.timeweb.ru': 'timeweb.ru',
 'vh276.timeweb.ru': 'timeweb.ru',
 'bitrix318.timeweb.ru': 'timeweb.ru',
 'vh36.timeweb.ru': 'timeweb.ru',
 'timeweb.ru': 'timeweb.ru',
 'vh294.timeweb.ru': 'timeweb.ru',
 'vh312.timeweb.ru': 'timeweb.ru',
 'vh376.timeweb.ru': 'timeweb.ru',
 'vh310.timeweb.ru': 'timeweb.ru',
 'margerie.timeweb.ru': 'timeweb.ru',
 'bitrix242.timeweb.ru': 'timeweb.ru',
 'vh92.timeweb.ru': 'timeweb.ru',
 'vh120.timeweb.ru': 'timeweb.ru',
 'vh146.timeweb.ru': 'timeweb.ru',
 'vh368.timeweb.ru': 'timeweb.ru',
 'vh50.timeweb.ru': 'timeweb.ru',
 'betsy.timeweb.ru': 'timeweb.ru',
 'vh328.timeweb.ru': 'timeweb.ru',
 'vh268.timeweb.ru': 'timeweb.ru',
 'bitrix328.timeweb.ru': 'timeweb.ru',
 'vh326.timeweb.ru': 'timeweb.ru',
 'patria.timeweb.ru': 'timeweb.ru',
 'bitrix154.timeweb.ru': 'timeweb.ru',
 'vh322.timeweb.ru': 'timeweb.ru',
 'mdp6.timeweb.ru': 'timeweb.ru',
 'vh384.timeweb.ru': 'timeweb.ru',
 'bitrix258.timeweb.ru': 'timeweb.ru',
 'vh110.timeweb.ru': 'timeweb.ru',
 'bitrix370.timeweb.ru': 'timeweb.ru',
 'vh286.timeweb.ru': 'timeweb.ru',
 'vh304.timeweb.ru': 'timeweb.ru',
 'vh366.timeweb.ru': 'timeweb.ru',
 'vh330.timeweb.ru': 'timeweb.ru',
 'vh142.timeweb.ru': 'timeweb.ru',
 'vh100.timeweb.ru': 'timeweb.ru',
 'premium40.timeweb.ru': 'timeweb.ru',
 'vh136.timeweb.ru': 'timeweb.ru',
 'bitrix194.timeweb.ru': 'timeweb.ru',
 'bitrix368.timeweb.ru': 'timeweb.ru',
 'vh180.timeweb.ru': 'timeweb.ru',
 'vh114.timeweb.ru': 'timeweb.ru',
 'vh182.timeweb.ru': 'timeweb.ru',
 'vh168.timeweb.ru': 'timeweb.ru',
 'vh38.timeweb.ru': 'timeweb.ru',
 'angela.timeweb.ru': 'timeweb.ru',
 'vh150.timeweb.ru': 'timeweb.ru',
 'vh338.timeweb.ru': 'timeweb.ru',
 'bitrix234.timeweb.ru': 'timeweb.ru',
 'bitrix146.timeweb.ru': 'timeweb.ru',
 'bitrix378.timeweb.ru': 'timeweb.ru',
 'vh122.timeweb.ru': 'timeweb.ru',
 'vh300.timeweb.ru': 'timeweb.ru',
 'vh134.timeweb.ru': 'timeweb.ru',
 'bitrix196.timeweb.ru': 'timeweb.ru',
 'hegel.timeweb.ru': 'timeweb.ru',
 'grace.timeweb.ru': 'timeweb.ru',
 'vh282.timeweb.ru': 'timeweb.ru',
 'bitrix352.timeweb.ru': 'timeweb.ru',
 'martha.timeweb.ru': 'timeweb.ru',
 'vh154.timeweb.ru': 'timeweb.ru',
 'vh72.timeweb.ru': 'timeweb.ru',
 'vh404.timeweb.ru': 'timeweb.ru',
 'vh362.timeweb.ru': 'timeweb.ru',
 'vh54.timeweb.ru': 'timeweb.ru',
 'bitrix306.timeweb.ru': 'timeweb.ru',
 'deus.timeweb.ru': 'timeweb.ru',
 'vh394.timeweb.ru': 'timeweb.ru',
 'bitrix240.timeweb.ru': 'timeweb.ru',
 'vh190.timeweb.ru': 'timeweb.ru',
 'vh204.timeweb.ru': 'timeweb.ru',
 'ellen.timeweb.ru': 'timeweb.ru',
 'vh342.timeweb.ru': 'timeweb.ru',
 'vh340.timeweb.ru': 'timeweb.ru',
 'vh390.timeweb.ru': 'timeweb.ru',
 'vh194.timeweb.ru': 'timeweb.ru',
 'vh336.timeweb.ru': 'timeweb.ru',
 'bitrix144.timeweb.ru': 'timeweb.ru',
 'bitrix308.timeweb.ru': 'timeweb.ru',
 'vh42.timeweb.ru': 'timeweb.ru',
 'sab01.timeweb.ru': 'timeweb.ru',
 'vh166.timeweb.ru': 'timeweb.ru',
 'vh364.timeweb.ru': 'timeweb.ru',
 'amber.timeweb.ru': 'timeweb.ru',
 'vh66.timeweb.ru': 'timeweb.ru',
 'vh68.timeweb.ru': 'timeweb.ru',
 'vh360.timeweb.ru': 'timeweb.ru',
 'vh396.timeweb.ru': 'timeweb.ru',
 'bitrix142.timeweb.ru': 'timeweb.ru',
 'bitrix170.timeweb.ru': 'timeweb.ru',
 'vh138.timeweb.ru': 'timeweb.ru',
 'libertas.timeweb.ru': 'timeweb.ru',
 'vh156.timeweb.ru': 'timeweb.ru',
 'vh64.timeweb.ru': 'timeweb.ru',
 'vh184.timeweb.ru': 'timeweb.ru',
 'bitrix288.timeweb.ru': 'timeweb.ru',
 'clarus.timeweb.ru': 'timeweb.ru',
 'vh318.timeweb.ru': 'timeweb.ru',
 'bitrix248.timeweb.ru': 'timeweb.ru',
 'vh356.timeweb.ru': 'timeweb.ru',
 'vh198.timeweb.ru': 'timeweb.ru',
 'vh126.timeweb.ru': 'timeweb.ru',
 'bitrix360.timeweb.ru': 'timeweb.ru',
 'vh358.timeweb.ru': 'timeweb.ru',
 'vh118.timeweb.ru': 'timeweb.ru',
 'babs.timeweb.ru': 'timeweb.ru',
 'vh48.timeweb.ru': 'timeweb.ru',
 'vh102.timeweb.ru': 'timeweb.ru',
 'vh332.timeweb.ru': 'timeweb.ru',
 'sartre.timeweb.ru': 'timeweb.ru',
 'vh16.timeweb.ru': 'timeweb.ru',
 'vh402.timeweb.ru': 'timeweb.ru',
 'vh132.timeweb.ru': 'timeweb.ru',
 'bitrix272.timeweb.ru': 'timeweb.ru',
 'ava.timeweb.ru': 'timeweb.ru',
 'vh162.timeweb.ru': 'timeweb.ru',
 'vh144.timeweb.ru': 'timeweb.ru',
 'vh278.timeweb.ru': 'timeweb.ru',
 'vh90.timeweb.ru': 'timeweb.ru',
 'vh112.timeweb.ru': 'timeweb.ru',
 'bitrix376.timeweb.ru': 'timeweb.ru',
 'vh272.timeweb.ru': 'timeweb.ru',
 'beth.timeweb.ru': 'timeweb.ru',
 'vh140.timeweb.ru': 'timeweb.ru',
 'vh386.timeweb.ru': 'timeweb.ru',
 'bitrix270.timeweb.ru': 'timeweb.ru',
 'vh148.timeweb.ru': 'timeweb.ru',
 'bitrix200.timeweb.ru': 'timeweb.ru',
 'vh348.timeweb.ru': 'timeweb.ru',
 'bitrix152.timeweb.ru': 'timeweb.ru',
 'vh104.timeweb.ru': 'timeweb.ru',
 'bitrix174.timeweb.ru': 'timeweb.ru',
 'vh96.timeweb.ru': 'timeweb.ru',
 'fortuna.timeweb.ru': 'timeweb.ru',
 'wilma.timeweb.ru': 'timeweb.ru',
 'shirley.timeweb.ru': 'timeweb.ru',
 'bitrix256.timeweb.ru': 'timeweb.ru',
 'vh130.timeweb.ru': 'timeweb.ru',
 'vh128.timeweb.ru': 'timeweb.ru',
 'bitrix314.timeweb.ru': 'timeweb.ru',
 'vh308.timeweb.ru': 'timeweb.ru',
 'bitrix168.timeweb.ru': 'timeweb.ru',
 'vh284.timeweb.ru': 'timeweb.ru',
 'mariska.timeweb.ru': 'timeweb.ru',
 'vh292.timeweb.ru': 'timeweb.ru',
 'bitrix302.timeweb.ru': 'timeweb.ru',
 'vh76.timeweb.ru': 'timeweb.ru',
 'bitrix312.timeweb.ru': 'timeweb.ru',
 'vh58.timeweb.ru': 'timeweb.ru',
 'vh34.timeweb.ru': 'timeweb.ru',
 'sincerus.timeweb.ru': 'timeweb.ru',
 'bitrix128.timeweb.ru': 'timeweb.ru',
 'bitrix166.timeweb.ru': 'timeweb.ru',
 'bitrix252.timeweb.ru': 'timeweb.ru',
 'belinda.timeweb.ru': 'timeweb.ru',
 'bitrix244.timeweb.ru': 'timeweb.ru',
 'vh116.timeweb.ru': 'timeweb.ru',
 'bitrix374.timeweb.ru': 'timeweb.ru',
 'bitrix364.timeweb.ru': 'timeweb.ru',
 'vh392.timeweb.ru': 'timeweb.ru',
 'bitrix382.timeweb.ru': 'timeweb.ru',
 'vh316.timeweb.ru': 'timeweb.ru',
 'bitrix330.timeweb.ru': 'timeweb.ru',
 'bitrix206.timeweb.ru': 'timeweb.ru',
 'carol.timeweb.ru': 'timeweb.ru',
 'sabrina.timeweb.ru': 'timeweb.ru',
 'vh40.timeweb.ru': 'timeweb.ru',
 'vh288.timeweb.ru': 'timeweb.ru',
 'vh320.timeweb.ru': 'timeweb.ru',
 'vh152.timeweb.ru': 'timeweb.ru',
 'bitrix172.timeweb.ru': 'timeweb.ru',
 'bitrix260.timeweb.ru': 'timeweb.ru',
 'vh80.timeweb.ru': 'timeweb.ru',
 'bitrix280.timeweb.ru': 'timeweb.ru',
 'bitrix390.timeweb.ru': 'timeweb.ru',
 'bitrix264.timeweb.ru': 'timeweb.ru',
 'bitrix384.timeweb.ru': 'timeweb.ru',
 'bitrix388.timeweb.ru': 'timeweb.ru',
 'vh372.timeweb.ru': 'timeweb.ru',
 'vh280.timeweb.ru': 'timeweb.ru',
 'vh178.timeweb.ru': 'timeweb.ru',
 'vh306.timeweb.ru': 'timeweb.ru',
 'bitrix246.timeweb.ru': 'timeweb.ru',
 'vh344.timeweb.ru': 'timeweb.ru',
 'bitrix366.timeweb.ru': 'timeweb.ru',
 'vh20.timeweb.ru': 'timeweb.ru',
    'bizly.ru': 'bizly.ru',
 'strezhevoy.bizly.ru': 'bizly.ru',
 'mihaylovsk.bizly.ru': 'bizly.ru',
 'pavlovskiy-posad.bizly.ru': 'bizly.ru',
 'samara.bizly.ru': 'bizly.ru',
 'novorossiysk.bizly.ru': 'bizly.ru',
 'aksay.bizly.ru': 'bizly.ru',
 'korolev.bizly.ru': 'bizly.ru',
 'noginsk.bizly.ru': 'bizly.ru',
 'orehovo-zuevo.bizly.ru': 'bizly.ru',
 'kurskaya-oblast.bizly.ru': 'bizly.ru',
 'tomsk.bizly.ru': 'bizly.ru',
 'naro-fominsk.bizly.ru': 'bizly.ru',
 'shatura.bizly.ru': 'bizly.ru',
 'habarovsk.bizly.ru': 'bizly.ru',
 'ekaterinburg.bizly.ru': 'bizly.ru',
 'saratov.bizly.ru': 'bizly.ru',
 'tula.bizly.ru': 'bizly.ru',
 'belebey.bizly.ru': 'bizly.ru',
 'bryanskaya-oblast.bizly.ru': 'bizly.ru',
 'kizlyar.bizly.ru': 'bizly.ru',
 'skopin.bizly.ru': 'bizly.ru',
 'ostashkov.bizly.ru': 'bizly.ru',
 'yurga.bizly.ru': 'bizly.ru',
 'podolsk.bizly.ru': 'bizly.ru',
 'chelyabinsk.bizly.ru': 'bizly.ru',
 'ishim.bizly.ru': 'bizly.ru',
 'kolomna.bizly.ru': 'bizly.ru',
 'tyumen.bizly.ru': 'bizly.ru',
 'kronshtadt.bizly.ru': 'bizly.ru',
 'zelenograd.bizly.ru': 'bizly.ru',
 'vorkuta.bizly.ru': 'bizly.ru',
 'yuryuzan.bizly.ru': 'bizly.ru',
 'usinsk.bizly.ru': 'bizly.ru',
 'vladikavkaz.bizly.ru': 'bizly.ru',
 'ufa.bizly.ru': 'bizly.ru',
 'rostov-na-donu.bizly.ru': 'bizly.ru',
 'pskov.bizly.ru': 'bizly.ru',
 'odincovo.bizly.ru': 'bizly.ru',
 'kursk.bizly.ru': 'bizly.ru',
 'krasnodar.bizly.ru': 'bizly.ru',
 'krasnoznamensk.bizly.ru': 'bizly.ru',
 'shadrinsk.bizly.ru': 'bizly.ru',
 'zelenogorsk.bizly.ru': 'bizly.ru',
 'barnaul.bizly.ru': 'bizly.ru',
 'krasnoyarsk.bizly.ru': 'bizly.ru',
 'moskva.bizly.ru': 'bizly.ru',
 'gusev.bizly.ru': 'bizly.ru',
 'novoshahtinsk.bizly.ru': 'bizly.ru',
 'novosibirsk.bizly.ru': 'bizly.ru',
 'voronezh.bizly.ru': 'bizly.ru',
 'sankt-peterburg.bizly.ru': 'bizly.ru',
 'chapaevsk.bizly.ru': 'bizly.ru',
 'nizhniy-novgorod.bizly.ru': 'bizly.ru',
 'moskovskaya-oblast.bizly.ru': 'bizly.ru',
 'leningradskaya-oblast.bizly.ru': 'bizly.ru',
 'schelkovo.bizly.ru': 'bizly.ru',
 'armavir.bizly.ru': 'bizly.ru',
 'nalchik.bizly.ru': 'bizly.ru',
 'surgut.bizly.ru': 'bizly.ru',
 'tuapse.bizly.ru': 'bizly.ru',
 'taganrog.bizly.ru': 'bizly.ru',
 'perm.bizly.ru': 'bizly.ru',
 'tulskaya-oblast.bizly.ru': 'bizly.ru',
 'balashiha.bizly.ru': 'bizly.ru',
 'rubcovsk.bizly.ru': 'bizly.ru',
 'tobolsk.bizly.ru': 'bizly.ru',
 'gatchina.bizly.ru': 'bizly.ru',
 'yahroma.bizly.ru': 'bizly.ru',
 'vladivostok.bizly.ru': 'bizly.ru',
 'temryuk.bizly.ru': 'bizly.ru',
 'svetlograd.bizly.ru': 'bizly.ru',
 'sverdlovskaya-oblast.bizly.ru': 'bizly.ru',
 'volgograd.bizly.ru': 'bizly.ru',
 'smolensk.bizly.ru': 'bizly.ru',
 'siktivkar.bizly.ru': 'bizly.ru',
 'sochi.bizly.ru': 'bizly.ru',
 'kasimov.bizly.ru': 'bizly.ru',
 'shahti.bizly.ru': 'bizly.ru',
 'alatir.bizly.ru': 'bizly.ru',
 'agidel.bizly.ru': 'bizly.ru',
 'timashevsk.bizly.ru': 'bizly.ru',
 'anapa.bizly.ru': 'bizly.ru',
 'kushva.bizly.ru': 'bizly.ru',
 'birobidzhan.bizly.ru': 'bizly.ru',
 'sol-ileck.bizly.ru': 'bizly.ru',
 'ryazan.bizly.ru': 'bizly.ru',
 'miass.bizly.ru': 'bizly.ru',
 'pskovskaya-oblast.bizly.ru': 'bizly.ru',
 'rtischevo.bizly.ru': 'bizly.ru',
 'udmurtskaya-respublika.bizly.ru': 'bizly.ru',
 'solikamsk.bizly.ru': 'bizly.ru',
 'kazan.bizly.ru': 'bizly.ru',
 'priozersk.bizly.ru': 'bizly.ru',
 'primorsko-ahtarsk.bizly.ru': 'bizly.ru',
 'sosnoviy-bor.bizly.ru': 'bizly.ru',
 'yaroslavskaya-oblast.bizly.ru': 'bizly.ru',
 'yaroslavl.bizly.ru': 'bizly.ru',
 'luhovici.bizly.ru': 'bizly.ru',
 'krasnodarskiy-kray.bizly.ru': 'bizly.ru',
 'vladimirskaya-oblast.bizly.ru': 'bizly.ru',
 'orsk.bizly.ru': 'bizly.ru',
 'belorechensk.bizly.ru': 'bizly.ru',
 'satka.bizly.ru': 'bizly.ru',
 'leninogorsk.bizly.ru': 'bizly.ru',
 'kovrov.bizly.ru': 'bizly.ru',
 'zheleznogorsk.bizly.ru': 'bizly.ru',
 'murmansk.bizly.ru': 'bizly.ru',
 'salavat.bizly.ru': 'bizly.ru',
 'volzhskiy.bizly.ru': 'bizly.ru',
 'kirovsk.bizly.ru': 'bizly.ru',
 'vologda.bizly.ru': 'bizly.ru',
 'ivanteevka.bizly.ru': 'bizly.ru',
 'novokuybishevsk.bizly.ru': 'bizly.ru',
 'neftekamsk.bizly.ru': 'bizly.ru',
 'kurganinsk.bizly.ru': 'bizly.ru',
 'novgorodskaya-oblast.bizly.ru': 'bizly.ru',
 'rasskazovo.bizly.ru': 'bizly.ru',
 'pereslavl-zalesskiy.bizly.ru': 'bizly.ru',
 'izhevsk.bizly.ru': 'bizly.ru',
 'kropotkin.bizly.ru': 'bizly.ru',
 'michurinsk.bizly.ru': 'bizly.ru',
 'dubna.bizly.ru': 'bizly.ru',
 'nevinnomissk.bizly.ru': 'bizly.ru',
 'severodvinsk.bizly.ru': 'bizly.ru',
 'gelendzhik.bizly.ru': 'bizly.ru',
 'penza.bizly.ru': 'bizly.ru',
 'velikie-luki.bizly.ru': 'bizly.ru',
 'ishimbay.bizly.ru': 'bizly.ru',
 'kirov.bizly.ru': 'bizly.ru',
 'sayanogorsk.bizly.ru': 'bizly.ru',
 'serpuhov.bizly.ru': 'bizly.ru',
 'voronezhskaya-oblast.bizly.ru': 'bizly.ru',
 'pushkino.bizly.ru': 'bizly.ru',
 'verhnyaya-pishma.bizly.ru': 'bizly.ru',
 'tverskaya-oblast.bizly.ru': 'bizly.ru',
 'novocherkassk.bizly.ru': 'bizly.ru',
 'respublika-adigeya.bizly.ru': 'bizly.ru',
 'vishniy-volochek.bizly.ru': 'bizly.ru',
 'slavyansk-na-kubani.bizly.ru': 'bizly.ru',
 'minusinsk.bizly.ru': 'bizly.ru',
 'kaliningrad.bizly.ru': 'bizly.ru',
 'solnechnogorsk.bizly.ru': 'bizly.ru',
 'dolgoprudniy.bizly.ru': 'bizly.ru',
 'chelyabinskaya-oblast.bizly.ru': 'bizly.ru',
 'stariy-oskol.bizly.ru': 'bizly.ru',
 'salsk.bizly.ru': 'bizly.ru',
 'pechora.bizly.ru': 'bizly.ru',
 'kolpino.bizly.ru': 'bizly.ru',
 'novomichurinsk.bizly.ru': 'bizly.ru',
 'gorno-altaysk.bizly.ru': 'bizly.ru',
 'lugansk.bizly.ru': 'bizly.ru',
 'novocheboksarsk.bizly.ru': 'bizly.ru',
 'bryansk.bizly.ru': 'bizly.ru',
 'kopeysk.bizly.ru': 'bizly.ru',
 'shuya.bizly.ru': 'bizly.ru',
 'chistopol.bizly.ru': 'bizly.ru',
 'yuzhnouralsk.bizly.ru': 'bizly.ru',
 'asbest.bizly.ru': 'bizly.ru',
 'ulyanovsk.bizly.ru': 'bizly.ru',
 'stupino.bizly.ru': 'bizly.ru',
 'vidnoe.bizly.ru': 'bizly.ru',
 'omsk.bizly.ru': 'bizly.ru',
 'orenburg.bizly.ru': 'bizly.ru',
 'sizran.bizly.ru': 'bizly.ru',
 'blagoveschensk.bizly.ru': 'bizly.ru',
 'komsomolsk-na-amure.bizly.ru': 'bizly.ru',
 'artem.bizly.ru': 'bizly.ru',
 'mozhaysk.bizly.ru': 'bizly.ru',
 'respublika-bashkortostan.bizly.ru': 'bizly.ru',
 'magnitogorsk.bizly.ru': 'bizly.ru',
 'cherkessk.bizly.ru': 'bizly.ru',
 'abakan.bizly.ru': 'bizly.ru',
 'segezha.bizly.ru': 'bizly.ru',
 'stavropol.bizly.ru': 'bizly.ru',
 'kanash.bizly.ru': 'bizly.ru',
 'sterlitamak.bizly.ru': 'bizly.ru',
 'ivanovo.bizly.ru': 'bizly.ru',
 'elektrostal.bizly.ru': 'bizly.ru',
 'belousovo.bizly.ru': 'bizly.ru',
 'obninsk.bizly.ru': 'bizly.ru',
 'naberezhnie-chelni.bizly.ru': 'bizly.ru',
 'mitischi.bizly.ru': 'bizly.ru',
 'bronnici.bizly.ru': 'bizly.ru',
 'alapaevsk.bizly.ru': 'bizly.ru',
 'belgorod.bizly.ru': 'bizly.ru',
 'karachaevo-cherkesskaya-respublika.bizly.ru': 'bizly.ru',
 'velikiy-novgorod.bizly.ru': 'bizly.ru',
 'nazarovo.bizly.ru': 'bizly.ru',
 'cheboksari.bizly.ru': 'bizly.ru',
 'volgodonsk.bizly.ru': 'bizly.ru',
 'sarov.bizly.ru': 'bizly.ru',
 'nyagan.bizly.ru': 'bizly.ru',
 'inta.bizly.ru': 'bizly.ru',
 'kashira.bizly.ru': 'bizly.ru',
 'valday.bizly.ru': 'bizly.ru',
 'yalutorovsk.bizly.ru': 'bizly.ru',
 'almetevsk.bizly.ru': 'bizly.ru',
 'karachaevsk.bizly.ru': 'bizly.ru',
 'ostrov.bizly.ru': 'bizly.ru',
 'kurgan.bizly.ru': 'bizly.ru',
 'kiev.bizly.ru': 'bizly.ru',
 'kostroma.bizly.ru': 'bizly.ru',
 'samarskaya-oblast.bizly.ru': 'bizly.ru',
 'yakutsk.bizly.ru': 'bizly.ru',
 'kaluzhskaya-oblast.bizly.ru': 'bizly.ru',
 'noviy-urengoy.bizly.ru': 'bizly.ru',
 'apsheronsk.bizly.ru': 'bizly.ru',
 'dzerzhinsk.bizly.ru': 'bizly.ru',
 'kommunar.bizly.ru': 'bizly.ru',
 'moskovskiy.bizly.ru': 'bizly.ru',
 'korsakov.bizly.ru': 'bizly.ru',
 'novomoskovsk.bizly.ru': 'bizly.ru',
 'sestroreck.bizly.ru': 'bizly.ru',
 'dmitrov.bizly.ru': 'bizly.ru',
 'almati.bizly.ru': 'bizly.ru',
 'liski.bizly.ru': 'bizly.ru',
 'berezniki.bizly.ru': 'bizly.ru',
 'volosovo.bizly.ru': 'bizly.ru',
    'bankiros.ru': 'bankiros.ru',
 'novosibirsk.bankiros.ru': 'bankiros.ru',
 'amursk.bankiros.ru': 'bankiros.ru',
 'rezh.bankiros.ru': 'bankiros.ru',
 'moskva.bankiros.ru': 'bankiros.ru',
 'tumen.bankiros.ru': 'bankiros.ru',
 'samara.bankiros.ru': 'bankiros.ru',
 'omsk.bankiros.ru': 'bankiros.ru',
 'krasnoyarsk.bankiros.ru': 'bankiros.ru',
 'kazan.bankiros.ru': 'bankiros.ru',
 'habarovsk.bankiros.ru': 'bankiros.ru',
 'ivanovo.bankiros.ru': 'bankiros.ru',
 'spb.bankiros.ru': 'bankiros.ru',
 'belorechensk.bankiros.ru': 'bankiros.ru',
 'orel.bankiros.ru': 'bankiros.ru',
 'obninsk.bankiros.ru': 'bankiros.ru',
 'sterlitamak.bankiros.ru': 'bankiros.ru',
 'nizhniy-tagil.bankiros.ru': 'bankiros.ru',
 'novorossiysk.bankiros.ru': 'bankiros.ru',
 'perm.bankiros.ru': 'bankiros.ru',
 'noviy-urengoy.bankiros.ru': 'bankiros.ru',
 'neftekamsk.bankiros.ru': 'bankiros.ru',
 'ekaterinburg.bankiros.ru': 'bankiros.ru',
 'nizhniy-novgorod.bankiros.ru': 'bankiros.ru',
 'korolev.bankiros.ru': 'bankiros.ru',
 'tolyatti.bankiros.ru': 'bankiros.ru',
 'sochi.bankiros.ru': 'bankiros.ru',
 'krasnodar.bankiros.ru': 'bankiros.ru',
 'hasavurt.bankiros.ru': 'bankiros.ru',
 'novokuzneck.bankiros.ru': 'bankiros.ru',
 'ufa.bankiros.ru': 'bankiros.ru',
 'orenburg.bankiros.ru': 'bankiros.ru',
 'bugulma.bankiros.ru': 'bankiros.ru',
 'naberezhnie-chelni.bankiros.ru': 'bankiros.ru',
 'reutov.bankiros.ru': 'bankiros.ru',
 'cheboksari.bankiros.ru': 'bankiros.ru',
 'kursk.bankiros.ru': 'bankiros.ru',
 'vladivostok.bankiros.ru': 'bankiros.ru',
 'cherepovec.bankiros.ru': 'bankiros.ru',
 'respublika-bashkortostan-oktyabrskiy.bankiros.ru': 'bankiros.ru',
 'volgograd.bankiros.ru': 'bankiros.ru',
 'sergiev-posad.bankiros.ru': 'bankiros.ru',
 'lipeck.bankiros.ru': 'bankiros.ru',
 'naro-fominsk.bankiros.ru': 'bankiros.ru',
 'mitischi.bankiros.ru': 'bankiros.ru',
 'nizhnekamsk.bankiros.ru': 'bankiros.ru',
 'velikiy-novgorod.bankiros.ru': 'bankiros.ru',
 'komsomolsk-na-amure.bankiros.ru': 'bankiros.ru',
 'engels.bankiros.ru': 'bankiros.ru',
 'izhevsk.bankiros.ru': 'bankiros.ru',
 'vologda.bankiros.ru': 'bankiros.ru',
 'odincovo.bankiros.ru': 'bankiros.ru',
 'vladimir.bankiros.ru': 'bankiros.ru',
 'noyabrsk.bankiros.ru': 'bankiros.ru',
 'iskitim.bankiros.ru': 'bankiros.ru',
 'penza.bankiros.ru': 'bankiros.ru',
 'kaliningrad.bankiros.ru': 'bankiros.ru',
 'ishimbay.bankiros.ru': 'bankiros.ru',
 'podolsk.bankiros.ru': 'bankiros.ru',
 'chapaevsk.bankiros.ru': 'bankiros.ru',
 'norilsk.bankiros.ru': 'bankiros.ru',
 'kansk.bankiros.ru': 'bankiros.ru',
 'saratov.bankiros.ru': 'bankiros.ru',
 'nizhegorodskaya-oblast-bor.bankiros.ru': 'bankiros.ru',
 'ryazan.bankiros.ru': 'bankiros.ru',
 'tambov.bankiros.ru': 'bankiros.ru',
 'elabuga.bankiros.ru': 'bankiros.ru',
 'moskovskaja-oblast-ivanteevka.bankiros.ru': 'bankiros.ru',
 'balashiha.bankiros.ru': 'bankiros.ru',
 'kirovskaya-oblast-kirov.bankiros.ru': 'bankiros.ru',
 'orehovo-zuevo.bankiros.ru': 'bankiros.ru',
 'kaluga.bankiros.ru': 'bankiros.ru',
 'aleksandrov.bankiros.ru': 'bankiros.ru',
 'stavropol.bankiros.ru': 'bankiros.ru',
 'schelkovo.bankiros.ru': 'bankiros.ru',
 'elec.bankiros.ru': 'bankiros.ru',
 'surgut.bankiros.ru': 'bankiros.ru',
 'aznakaevo.bankiros.ru': 'bankiros.ru',
 'yaroslavl.bankiros.ru': 'bankiros.ru',
 'goryachiy-kluch.bankiros.ru': 'bankiros.ru',
 'pskov.bankiros.ru': 'bankiros.ru',
 'ulyanovsk.bankiros.ru': 'bankiros.ru',
 'tomsk.bankiros.ru': 'bankiros.ru',
 'prokopevsk.bankiros.ru': 'bankiros.ru',
 'volzhskiy.bankiros.ru': 'bankiros.ru',
 'kovrov.bankiros.ru': 'bankiros.ru',
 'murmansk.bankiros.ru': 'bankiros.ru',
 'almetevsk.bankiros.ru': 'bankiros.ru',
 'tuapse.bankiros.ru': 'bankiros.ru',
 'zlatoust.bankiros.ru': 'bankiros.ru',
 'ramenskoe.bankiros.ru': 'bankiros.ru',
 'serov.bankiros.ru': 'bankiros.ru',
 'hanti-mansiysk.bankiros.ru': 'bankiros.ru',
 'domodedovo.bankiros.ru': 'bankiros.ru',
 'doneck.bankiros.ru': 'bankiros.ru',
 'krimsk.bankiros.ru': 'bankiros.ru',
 'zelenograd.bankiros.ru': 'bankiros.ru',
 'nizhnevartovsk.bankiros.ru': 'bankiros.ru',
 'kemerovo.bankiros.ru': 'bankiros.ru',
 'abinsk.bankiros.ru': 'bankiros.ru',
 'liskovo.bankiros.ru': 'bankiros.ru',
 'asbest.bankiros.ru': 'bankiros.ru',
 'taganrog.bankiros.ru': 'bankiros.ru',
 'kumertau.bankiros.ru': 'bankiros.ru',
 'magnitogorsk.bankiros.ru': 'bankiros.ru',
 'stupino.bankiros.ru': 'bankiros.ru',
 'himki.bankiros.ru': 'bankiros.ru',
 'kurgan.bankiros.ru': 'bankiros.ru',
 'arhangelsk.bankiros.ru': 'bankiros.ru',
 'tula.bankiros.ru': 'bankiros.ru',
 'evpatoriya.bankiros.ru': 'bankiros.ru',
 'tobolsk.bankiros.ru': 'bankiros.ru',
 'zhukovskiy.bankiros.ru': 'bankiros.ru',
 'yoshkar-ola.bankiros.ru': 'bankiros.ru',
 'satka.bankiros.ru': 'bankiros.ru',
 'gatchina.bankiros.ru': 'bankiros.ru',
 'staraya-russa.bankiros.ru': 'bankiros.ru',
 'tver.bankiros.ru': 'bankiros.ru',
 'serpuhov.bankiros.ru': 'bankiros.ru',
 'roslavl.bankiros.ru': 'bankiros.ru',
 'biysk.bankiros.ru': 'bankiros.ru',
 'nevinnomissk.bankiros.ru': 'bankiros.ru',
 'salavat.bankiros.ru': 'bankiros.ru',
 'yuzhno-sahalinsk.bankiros.ru': 'bankiros.ru',
 'pervouralsk.bankiros.ru': 'bankiros.ru',
 'simferopol.bankiros.ru': 'bankiros.ru',
 'leninsk-kuzneckiy.bankiros.ru': 'bankiros.ru',
 'shadrinsk.bankiros.ru': 'bankiros.ru',
 'kostroma.bankiros.ru': 'bankiros.ru',
 'uhta.bankiros.ru': 'bankiros.ru',
 'aksay.bankiros.ru': 'bankiros.ru',
 'luberci.bankiros.ru': 'bankiros.ru',
 'berdsk.bankiros.ru': 'bankiros.ru',
 'sarapul.bankiros.ru': 'bankiros.ru',
 'bryansk.bankiros.ru': 'bankiros.ru',
 'primorsko-ahtarsk.bankiros.ru': 'bankiros.ru',
 'ahtubinsk.bankiros.ru': 'bankiros.ru',
 'pyatigorsk.bankiros.ru': 'bankiros.ru',
 'anapa.bankiros.ru': 'bankiros.ru',
 'anzhero-sudzhensk.bankiros.ru': 'bankiros.ru',
 'nahodka.bankiros.ru': 'bankiros.ru',
 'siktivkar.bankiros.ru': 'bankiros.ru',
 'amurskaya-oblast-belogorsk.bankiros.ru': 'bankiros.ru',
 'volgodonsk.bankiros.ru': 'bankiros.ru',
 'kaliningradskaya-oblast-sovetsk.bankiros.ru': 'bankiros.ru',
 'zheleznodorozhniy.bankiros.ru': 'bankiros.ru',
 'novocherkassk.bankiros.ru': 'bankiros.ru',
 'tihoreck.bankiros.ru': 'bankiros.ru',
 'noginsk.bankiros.ru': 'bankiros.ru',
 'ribinsk.bankiros.ru': 'bankiros.ru',
 'kamensk-uralskiy.bankiros.ru': 'bankiros.ru',
 'kirovo-chepeck.bankiros.ru': 'bankiros.ru',
 'maykop.bankiros.ru': 'bankiros.ru',
 'usinsk.bankiros.ru': 'bankiros.ru',
 'velikie-luki.bankiros.ru': 'bankiros.ru',
 'shahti.bankiros.ru': 'bankiros.ru',
 'zeya.bankiros.ru': 'bankiros.ru',
 'uglich.bankiros.ru': 'bankiros.ru',
 'armavir.bankiros.ru': 'bankiros.ru',
 'ussuriysk.bankiros.ru': 'bankiros.ru',
 'glazov.bankiros.ru': 'bankiros.ru',
 'leningradskaya.bankiros.ru': 'bankiros.ru',
 'monchegorsk.bankiros.ru': 'bankiros.ru',
 'kislovodsk.bankiros.ru': 'bankiros.ru',
 'berezniki.bankiros.ru': 'bankiros.ru',
 'spassk-dalniy.bankiros.ru': 'bankiros.ru',
 'korenovsk.bankiros.ru': 'bankiros.ru',
 'moskovskaya-oblast-pushkino.bankiros.ru': 'bankiros.ru',
 'kashira.bankiros.ru': 'bankiros.ru',
 'belgorod.bankiros.ru': 'bankiros.ru',
 'saransk.bankiros.ru': 'bankiros.ru',
 'bataysk.bankiros.ru': 'bankiros.ru',
 'klin.bankiros.ru': 'bankiros.ru',
 'petropavlovsk-kamchatskiy.bankiros.ru': 'bankiros.ru',
 'salehard.bankiros.ru': 'bankiros.ru',
 'nyagan.bankiros.ru': 'bankiros.ru',
 'konakovo.bankiros.ru': 'bankiros.ru',
 'timashevsk.bankiros.ru': 'bankiros.ru',
 'apatiti.bankiros.ru': 'bankiros.ru',
 'dmitrov.bankiros.ru': 'bankiros.ru',
 'michurinsk.bankiros.ru': 'bankiros.ru',
 'zelenodolsk.bankiros.ru': 'bankiros.ru',
 'verhnyaya-pishma.bankiros.ru': 'bankiros.ru',
 'istra.bankiros.ru': 'bankiros.ru',
 'kolomna.bankiros.ru': 'bankiros.ru',
 'revda.bankiros.ru': 'bankiros.ru',
 'mahachkala.bankiros.ru': 'bankiros.ru',
 'severodvinsk.bankiros.ru': 'bankiros.ru',
 'barabinsk.bankiros.ru': 'bankiros.ru',
 'dolgoprudniy.bankiros.ru': 'bankiros.ru',
 'alushta.bankiros.ru': 'bankiros.ru',
 'magadan.bankiros.ru': 'bankiros.ru',
 'amurskaya-oblast-svobodniy.bankiros.ru': 'bankiros.ru',
 'dzerzhinsk.bankiros.ru': 'bankiros.ru',
 'murom.bankiros.ru': 'bankiros.ru',
 'yakutsk.bankiros.ru': 'bankiros.ru',
 'rzhev.bankiros.ru': 'bankiros.ru',
 'egorevsk.bankiros.ru': 'bankiros.ru',
 'georgievsk.bankiros.ru': 'bankiros.ru',
 'temruk.bankiros.ru': 'bankiros.ru',
 'slavyansk-na-kubani.bankiros.ru': 'bankiros.ru',
 'labinsk.bankiros.ru': 'bankiros.ru',
 'moskovskaja-oblast-dubna.bankiros.ru': 'bankiros.ru',
 'krasnogorsk.bankiros.ru': 'bankiros.ru',
 'voskresensk.bankiros.ru': 'bankiros.ru',
 'gus-hrustalniy.bankiros.ru': 'bankiros.ru',
 'mineralnie-vodi.bankiros.ru': 'bankiros.ru',
 'irbit.bankiros.ru': 'bankiros.ru',
 'gelendzhik.bankiros.ru': 'bankiros.ru',
 'gorno-altaysk.bankiros.ru': 'bankiros.ru',
 'tuymazi.bankiros.ru': 'bankiros.ru',
 'sizran.bankiros.ru': 'bankiros.ru',
 'polevskoy.bankiros.ru': 'bankiros.ru',
 'bologoe.bankiros.ru': 'bankiros.ru',
 'krasnouralsk.bankiros.ru': 'bankiros.ru',
 'shuya.bankiros.ru': 'bankiros.ru',
 'vidnoe.bankiros.ru': 'bankiros.ru',
 'mozhaysk.bankiros.ru': 'bankiros.ru',
 'kropotkin.bankiros.ru': 'bankiros.ru',
 'balakovo.bankiros.ru': 'bankiros.ru',
 'sarov.bankiros.ru': 'bankiros.ru',
 'verhniy-ufaley.bankiros.ru': 'bankiros.ru',
    'ukraina.unibo.ru': 'unibo.ru',
 'orenburg.unibo.ru': 'unibo.ru',
 'moscow.unibo.ru': 'unibo.ru',
 'ufa.unibo.ru': 'unibo.ru',
 'smolensk.unibo.ru': 'unibo.ru',
 'nizhniy-novgorod.unibo.ru': 'unibo.ru',
 'mitischi.unibo.ru': 'unibo.ru',
 'yuzhnouralsk.unibo.ru': 'unibo.ru',
 'krasnoyarsk.unibo.ru': 'unibo.ru',
 'pavlovskiy-posad.unibo.ru': 'unibo.ru',
 'mo.unibo.ru': 'unibo.ru',
 'salehard.unibo.ru': 'unibo.ru',
 'udmurtskaya-respublika.unibo.ru': 'unibo.ru',
 'respublika-bashkortostan.unibo.ru': 'unibo.ru',
 'permskaya-oblast.unibo.ru': 'unibo.ru',
 'liskovo.unibo.ru': 'unibo.ru',
 'sverdlovskaya-oblast.unibo.ru': 'unibo.ru',
 'ekaterinburg.unibo.ru': 'unibo.ru',
 'unibo.ru': 'unibo.ru',
 'sochi.unibo.ru': 'unibo.ru',
 'krasnodar.unibo.ru': 'unibo.ru',
 'krasnodarskiy-kray.unibo.ru': 'unibo.ru',
 'voronezh.unibo.ru': 'unibo.ru',
 'sankt-peterburg.unibo.ru': 'unibo.ru',
 'taldom.unibo.ru': 'unibo.ru',
 'novokuzneck.unibo.ru': 'unibo.ru',
 'rostov-na-donu.unibo.ru': 'unibo.ru',
 'volgogradskaya-oblast.unibo.ru': 'unibo.ru',
 'volgograd.unibo.ru': 'unibo.ru',
 'arzamas.unibo.ru': 'unibo.ru',
 'leninogorsk.unibo.ru': 'unibo.ru',
 'lipeck.unibo.ru': 'unibo.ru',
 'magadan.unibo.ru': 'unibo.ru',
 'kiselevsk.unibo.ru': 'unibo.ru',
 'krasnogorsk.unibo.ru': 'unibo.ru',
 'nizhniy-tagil.unibo.ru': 'unibo.ru',
 'tyumen.unibo.ru': 'unibo.ru',
 'cherepovec.unibo.ru': 'unibo.ru',
 'ryazanskaya-oblast.unibo.ru': 'unibo.ru',
 'ryazan.unibo.ru': 'unibo.ru',
 'odincovo.unibo.ru': 'unibo.ru',
 'podolsk.unibo.ru': 'unibo.ru',
 'tolyatti.unibo.ru': 'unibo.ru',
 'taganrog.unibo.ru': 'unibo.ru',
 'anapa.unibo.ru': 'unibo.ru',
 'korenovsk.unibo.ru': 'unibo.ru',
 'strunino.unibo.ru': 'unibo.ru',
 'elec.unibo.ru': 'unibo.ru',
 'gus-hrustalniy.unibo.ru': 'unibo.ru',
 'armavir.unibo.ru': 'unibo.ru',
 'klimovsk.unibo.ru': 'unibo.ru',
 'konakovo.unibo.ru': 'unibo.ru',
 'uvarovo.unibo.ru': 'unibo.ru',
 'novosibirsk.unibo.ru': 'unibo.ru',
 'admin.unibo.ru': 'unibo.ru',
 'tver.unibo.ru': 'unibo.ru',
 'belorechensk.unibo.ru': 'unibo.ru',
 'iran.unibo.ru': 'unibo.ru',
 'perm.unibo.ru': 'unibo.ru',
 'kineshma.unibo.ru': 'unibo.ru',
 'kemerovo.unibo.ru': 'unibo.ru',
 'vidnoe.unibo.ru': 'unibo.ru',
 'erevan.unibo.ru': 'unibo.ru',
 'znamensk.unibo.ru': 'unibo.ru',
 'tihoreck.unibo.ru': 'unibo.ru',
 'tinda.unibo.ru': 'unibo.ru',
 'kudimkar.unibo.ru': 'unibo.ru',
 'oktyabrskiy.unibo.ru': 'unibo.ru',
 'nizhegorodskaya-obl.unibo.ru': 'unibo.ru',
 'belaya-kalitva.unibo.ru': 'unibo.ru',
 'velikiy-novgorod.unibo.ru': 'unibo.ru',
 'buguruslan.unibo.ru': 'unibo.ru',
 'chelyabinsk.unibo.ru': 'unibo.ru',
 'engels.unibo.ru': 'unibo.ru',
 'kirovgrad.unibo.ru': 'unibo.ru',
 'ruzaevka.unibo.ru': 'unibo.ru',
 'konstantinovsk.unibo.ru': 'unibo.ru',
 'artem.unibo.ru': 'unibo.ru',
 'slavgorod.unibo.ru': 'unibo.ru',
 'sergiev-posad.unibo.ru': 'unibo.ru',
 'anadir.unibo.ru': 'unibo.ru',
 'adler.unibo.ru': 'unibo.ru',
 'almetevsk.unibo.ru': 'unibo.ru',
 'pskovskaya-oblast.unibo.ru': 'unibo.ru',
 'balabanovo.unibo.ru': 'unibo.ru',
 'surgut.unibo.ru': 'unibo.ru',
 'luhovici.unibo.ru': 'unibo.ru',
 'himki.unibo.ru': 'unibo.ru',
 'naro-fominsk.unibo.ru': 'unibo.ru',
 'ramenskoe.unibo.ru': 'unibo.ru',
 'vishniy-volochek.unibo.ru': 'unibo.ru',
 'apsheronsk.unibo.ru': 'unibo.ru',
 'saratov.unibo.ru': 'unibo.ru',
 'elektrostal.unibo.ru': 'unibo.ru',
 'balashiha.unibo.ru': 'unibo.ru',
 'nevyansk.unibo.ru': 'unibo.ru',
 'vladikavkaz.unibo.ru': 'unibo.ru',
 'shahti.unibo.ru': 'unibo.ru',
 'rubcovsk.unibo.ru': 'unibo.ru',
 'kolomna.unibo.ru': 'unibo.ru',
 'arhangelsk.unibo.ru': 'unibo.ru',
 'armenia.unibo.ru': 'unibo.ru',
 'ulyanovsk.unibo.ru': 'unibo.ru',
 'timashevsk.unibo.ru': 'unibo.ru',
 'pyatigorsk.unibo.ru': 'unibo.ru',
 'novosibirskaya-oblast.unibo.ru': 'unibo.ru',
 'shebekino.unibo.ru': 'unibo.ru',
 'aprelevka.unibo.ru': 'unibo.ru',
 'china.unibo.ru': 'unibo.ru',
 'naberezhnie-chelni.unibo.ru': 'unibo.ru',
 'chehov.unibo.ru': 'unibo.ru',
 'yuzhno-sahalinsk.unibo.ru': 'unibo.ru',
 'cheboksari.unibo.ru': 'unibo.ru',
 'kazan.unibo.ru': 'unibo.ru',
 'mineralnie-vodi.unibo.ru': 'unibo.ru',
 'minusinsk.unibo.ru': 'unibo.ru',
 'velikie-luki.unibo.ru': 'unibo.ru',
 'mendeleevsk.unibo.ru': 'unibo.ru',
 'primorskiy-kray.unibo.ru': 'unibo.ru',
 'stupino.unibo.ru': 'unibo.ru',
 'chapligin.unibo.ru': 'unibo.ru',
 'novotroick.unibo.ru': 'unibo.ru',
 'kurganinsk.unibo.ru': 'unibo.ru',
 'dolgoprudniy.unibo.ru': 'unibo.ru',
 'yarcevo.unibo.ru': 'unibo.ru',
 'ust-labinsk.unibo.ru': 'unibo.ru',
 'krasnoyarskiy-kray.unibo.ru': 'unibo.ru',
 'gruziya.unibo.ru': 'unibo.ru',
 'belebey.unibo.ru': 'unibo.ru',
 'korolev.unibo.ru': 'unibo.ru',
 'obninsk.unibo.ru': 'unibo.ru',
 'krasnokamensk.unibo.ru': 'unibo.ru',
 'norilsk.unibo.ru': 'unibo.ru',
 'tverskaya-oblast.unibo.ru': 'unibo.ru',
 'kirov.unibo.ru': 'unibo.ru',
 'glazov.unibo.ru': 'unibo.ru',
 'lipeckaya-oblast.unibo.ru': 'unibo.ru',
 'shlisselburg.unibo.ru': 'unibo.ru',
 'lenobl.unibo.ru': 'unibo.ru',
 'penza.unibo.ru': 'unibo.ru',
 'kaluga.unibo.ru': 'unibo.ru',
 'izhevsk.unibo.ru': 'unibo.ru',
 'novocherkassk.unibo.ru': 'unibo.ru',
 'nefteyugansk.unibo.ru': 'unibo.ru',
 'odesskaya-oblastj.unibo.ru': 'unibo.ru',
 'zelenograd.unibo.ru': 'unibo.ru',
 'samara.unibo.ru': 'unibo.ru',
 'zarechniy.unibo.ru': 'unibo.ru',
 'bor.unibo.ru': 'unibo.ru',
 'elabuga.unibo.ru': 'unibo.ru',
 'gelendzhik.unibo.ru': 'unibo.ru',
 'slavyansk-na-kubani.unibo.ru': 'unibo.ru',
 'respublika-hakasiya.unibo.ru': 'unibo.ru',
 'respublika-tatarstan.unibo.ru': 'unibo.ru',
 'saratovskaya-oblast.unibo.ru': 'unibo.ru',
 'miass.unibo.ru': 'unibo.ru',
 'zlatoust.unibo.ru': 'unibo.ru',
 'litkarino.unibo.ru': 'unibo.ru',
 'bryanskaya-oblast.unibo.ru': 'unibo.ru',
 'salavat.unibo.ru': 'unibo.ru',
 'orsk.unibo.ru': 'unibo.ru',
 'sudac.unibo.ru': 'unibo.ru',
 'aksay.unibo.ru': 'unibo.ru',
 'fryazino.unibo.ru': 'unibo.ru',
 'germany.unibo.ru': 'unibo.ru',
 'gatchina.unibo.ru': 'unibo.ru',
 'liski.unibo.ru': 'unibo.ru',
 'goryachiy-klyuch.unibo.ru': 'unibo.ru',
 'tulskaya-oblast.unibo.ru': 'unibo.ru',
 'irbit.unibo.ru': 'unibo.ru',
 'dmitrov.unibo.ru': 'unibo.ru',
 'kemerovskaya-oblast.unibo.ru': 'unibo.ru',
 'minsk.unibo.ru': 'unibo.ru',
 'kurskaya-oblast.unibo.ru': 'unibo.ru',
 'kursk.unibo.ru': 'unibo.ru',
 'zheleznogorsk.unibo.ru': 'unibo.ru',
 'simferopol.unibo.ru': 'unibo.ru',
 'nizhnekamsk.unibo.ru': 'unibo.ru',
 'sarapul.unibo.ru': 'unibo.ru',
 'kerch.unibo.ru': 'unibo.ru',
 'gulkevichi.unibo.ru': 'unibo.ru',
 'domodedovo.unibo.ru': 'unibo.ru',
 'mozhga.unibo.ru': 'unibo.ru',
 'dzhankoy.unibo.ru': 'unibo.ru',
 'balahna.unibo.ru': 'unibo.ru',
 'abinsk.unibo.ru': 'unibo.ru',
 'valday.unibo.ru': 'unibo.ru',
 'novomoskovsk.unibo.ru': 'unibo.ru',
 'lesosibirsk.unibo.ru': 'unibo.ru',
 'stavropol.unibo.ru': 'unibo.ru',
 'kirzhach.unibo.ru': 'unibo.ru',
 'sterlitamak.unibo.ru': 'unibo.ru',
 'ishimbay.unibo.ru': 'unibo.ru',
 'respublika-kareliya.unibo.ru': 'unibo.ru',
 'labinsk.unibo.ru': 'unibo.ru',
 'priozersk.unibo.ru': 'unibo.ru',
 'noginsk.unibo.ru': 'unibo.ru',
 'sayanogorsk.unibo.ru': 'unibo.ru',
 'maykop.unibo.ru': 'unibo.ru',
 'habarovsk.unibo.ru': 'unibo.ru',
 'irkutsk.unibo.ru': 'unibo.ru',
 'novgorodskaya-oblast.unibo.ru': 'unibo.ru',
 'dyatkovo.unibo.ru': 'unibo.ru',
 'bryansk.unibo.ru': 'unibo.ru',
 'balakovo.unibo.ru': 'unibo.ru',
 'sevastopol.unibo.ru': 'unibo.ru',
 'respublika-krim.unibo.ru': 'unibo.ru',
 'naryan-mar.unibo.ru': 'unibo.ru',
 'tuapse.unibo.ru': 'unibo.ru',
 'dubna.unibo.ru': 'unibo.ru',
 'kaliningrad.unibo.ru': 'unibo.ru',
 'belogorsk.unibo.ru': 'unibo.ru',
 'kungur.unibo.ru': 'unibo.ru',
 'belgorod.unibo.ru': 'unibo.ru',
 'berdsk.unibo.ru': 'unibo.ru',
 'yurga.unibo.ru': 'unibo.ru',
 'semikarakorsk.unibo.ru': 'unibo.ru',
 'volosovo.unibo.ru': 'unibo.ru',
 'vitebsk.unibo.ru': 'unibo.ru',
 'kstovo.unibo.ru': 'unibo.ru',
 'visokovsk.unibo.ru': 'unibo.ru',
 'zhukovskiy.unibo.ru': 'unibo.ru',
    'spravka-region.ru': 'spravka-region.ru',
 'zheleznodorozhnyj.spravka-region.ru': 'spravka-region.ru',
 'anzhero-sudzhensk.spravka-region.ru': 'spravka-region.ru',
 'chelyabinsk.spravka-region.ru': 'spravka-region.ru',
 'yanaul.spravka-region.ru': 'spravka-region.ru',
 'dyurtyuli.spravka-region.ru': 'spravka-region.ru',
 'adygejsk.spravka-region.ru': 'spravka-region.ru',
 'semikarakorsk.spravka-region.ru': 'spravka-region.ru',
 'makushino.spravka-region.ru': 'spravka-region.ru',
 'tobolsk.spravka-region.ru': 'spravka-region.ru',
 'izobilnyj.spravka-region.ru': 'spravka-region.ru',
 'asha.spravka-region.ru': 'spravka-region.ru',
 'beslan.spravka-region.ru': 'spravka-region.ru',
 'rezh.spravka-region.ru': 'spravka-region.ru',
 'slavyansk-na-kubani.spravka-region.ru': 'spravka-region.ru',
 'kushnarenkovo.spravka-region.ru': 'spravka-region.ru',
 'temryuk.spravka-region.ru': 'spravka-region.ru',
 'uspenskoe.spravka-region.ru': 'spravka-region.ru',
 'gusev.spravka-region.ru': 'spravka-region.ru',
 'batajsk.spravka-region.ru': 'spravka-region.ru',
 'bazarnye-mataki.spravka-region.ru': 'spravka-region.ru',
 'essentuki.spravka-region.ru': 'spravka-region.ru',
 'vologda.spravka-region.ru': 'spravka-region.ru',
 'nizhnedevick.spravka-region.ru': 'spravka-region.ru',
 'zverevo.spravka-region.ru': 'spravka-region.ru',
 'menzelinsk.spravka-region.ru': 'spravka-region.ru',
 'majkop.spravka-region.ru': 'spravka-region.ru',
 'nizhnekamsk.spravka-region.ru': 'spravka-region.ru',
 'chebarkul.spravka-region.ru': 'spravka-region.ru',
 'bolshoj-kamen.spravka-region.ru': 'spravka-region.ru',
 'zvenigovo.spravka-region.ru': 'spravka-region.ru',
 'svetlogorsk.spravka-region.ru': 'spravka-region.ru',
 'mozhajsk.spravka-region.ru': 'spravka-region.ru',
 'isakly.spravka-region.ru': 'spravka-region.ru',
 'priobe.spravka-region.ru': 'spravka-region.ru',
 'gatchina.spravka-region.ru': 'spravka-region.ru',
 'nikolaevsk.spravka-region.ru': 'spravka-region.ru',
 'chehov.spravka-region.ru': 'spravka-region.ru',
 'dinskaya.spravka-region.ru': 'spravka-region.ru',
 'korenovsk.spravka-region.ru': 'spravka-region.ru',
 'kachkanar.spravka-region.ru': 'spravka-region.ru',
 'georgievsk.spravka-region.ru': 'spravka-region.ru',
 'pushkinskie-gory.spravka-region.ru': 'spravka-region.ru',
 'kazan.spravka-region.ru': 'spravka-region.ru',
 'ulan-ude.spravka-region.ru': 'spravka-region.ru',
 'fryanovo.spravka-region.ru': 'spravka-region.ru',
 'luga.spravka-region.ru': 'spravka-region.ru',
 'neftekamsk.spravka-region.ru': 'spravka-region.ru',
 'beloomut.spravka-region.ru': 'spravka-region.ru',
 'irbejskoe.spravka-region.ru': 'spravka-region.ru',
 'sorochinsk.spravka-region.ru': 'spravka-region.ru',
 'kurganinsk.spravka-region.ru': 'spravka-region.ru',
 'volsk.spravka-region.ru': 'spravka-region.ru',
 'vozhega.spravka-region.ru': 'spravka-region.ru',
 'arhipo-osipovka.spravka-region.ru': 'spravka-region.ru',
 'bykovo.spravka-region.ru': 'spravka-region.ru',
 'vsevolozhsk.spravka-region.ru': 'spravka-region.ru',
 'kotlas.spravka-region.ru': 'spravka-region.ru',
 'dubna.spravka-region.ru': 'spravka-region.ru',
 'voronezh.spravka-region.ru': 'spravka-region.ru',
 'mezhdurechensk.spravka-region.ru': 'spravka-region.ru',
 'pikaljovo.spravka-region.ru': 'spravka-region.ru',
 'kashary.spravka-region.ru': 'spravka-region.ru',
 'ruza.spravka-region.ru': 'spravka-region.ru',
 'vladikavkaz.spravka-region.ru': 'spravka-region.ru',
 'chernyahovsk.spravka-region.ru': 'spravka-region.ru',
 'zhukovskij.spravka-region.ru': 'spravka-region.ru',
 'yuryuzan.spravka-region.ru': 'spravka-region.ru',
 'pyatigorsk.spravka-region.ru': 'spravka-region.ru',
 'anapa.spravka-region.ru': 'spravka-region.ru',
 'zarajsk.spravka-region.ru': 'spravka-region.ru',
 'chishmy.spravka-region.ru': 'spravka-region.ru',
 'ufa.spravka-region.ru': 'spravka-region.ru',
 'orsk.spravka-region.ru': 'spravka-region.ru',
 'kashira.spravka-region.ru': 'spravka-region.ru',
 'tishkovo.spravka-region.ru': 'spravka-region.ru',
 'bogdanovich.spravka-region.ru': 'spravka-region.ru',
 'nizhnij-novgorod.spravka-region.ru': 'spravka-region.ru',
 'tihvin.spravka-region.ru': 'spravka-region.ru',
 'kadom.spravka-region.ru': 'spravka-region.ru',
 'teberda.spravka-region.ru': 'spravka-region.ru',
 'rostov-na-donu.spravka-region.ru': 'spravka-region.ru',
 'chegem-pervyj.spravka-region.ru': 'spravka-region.ru',
 'verhnebakanskij.spravka-region.ru': 'spravka-region.ru',
 'kalininskaya.spravka-region.ru': 'spravka-region.ru',
 'mednogorsk.spravka-region.ru': 'spravka-region.ru',
 'elabuga.spravka-region.ru': 'spravka-region.ru',
 'kirishi.spravka-region.ru': 'spravka-region.ru',
 'ust-dzheguta.spravka-region.ru': 'spravka-region.ru',
 'odoev.spravka-region.ru': 'spravka-region.ru',
 'likino-dulevo.spravka-region.ru': 'spravka-region.ru',
 'mahachkala.spravka-region.ru': 'spravka-region.ru',
 'dimitrovgrad.spravka-region.ru': 'spravka-region.ru',
 'okoneshnikovo.spravka-region.ru': 'spravka-region.ru',
 'langepas.spravka-region.ru': 'spravka-region.ru',
 'raevskij.spravka-region.ru': 'spravka-region.ru',
 'svetlograd.spravka-region.ru': 'spravka-region.ru',
 'samara.spravka-region.ru': 'spravka-region.ru',
 'shodnya.spravka-region.ru': 'spravka-region.ru',
 'solnechnodolsk.spravka-region.ru': 'spravka-region.ru',
 'krasnozavodsk.spravka-region.ru': 'spravka-region.ru',
 'sarapul.spravka-region.ru': 'spravka-region.ru',
 'verhnij-mamon.spravka-region.ru': 'spravka-region.ru',
 'torzhok.spravka-region.ru': 'spravka-region.ru',
 'konakovo.spravka-region.ru': 'spravka-region.ru',
 'emelyanovo.spravka-region.ru': 'spravka-region.ru',
 'labinsk.spravka-region.ru': 'spravka-region.ru',
 'novopavlovsk.spravka-region.ru': 'spravka-region.ru',
 'turinsk.spravka-region.ru': 'spravka-region.ru',
 'fokino.spravka-region.ru': 'spravka-region.ru',
 'ilanskij.spravka-region.ru': 'spravka-region.ru',
 'habarovsk.spravka-region.ru': 'spravka-region.ru',
 'pokrovskoe.spravka-region.ru': 'spravka-region.ru',
 'lyudinovo.spravka-region.ru': 'spravka-region.ru',
 'sosnogorsk.spravka-region.ru': 'spravka-region.ru',
 'mostovskoj.spravka-region.ru': 'spravka-region.ru',
 'saransk.spravka-region.ru': 'spravka-region.ru',
 'kinel.spravka-region.ru': 'spravka-region.ru',
 'kedrovka.spravka-region.ru': 'spravka-region.ru',
 'vyrica.spravka-region.ru': 'spravka-region.ru',
 'belaya-kalitva.spravka-region.ru': 'spravka-region.ru',
 'gornozavodsk.spravka-region.ru': 'spravka-region.ru',
 'vorkuta.spravka-region.ru': 'spravka-region.ru',
 'kotelniki.spravka-region.ru': 'spravka-region.ru',
 'belorechensk.spravka-region.ru': 'spravka-region.ru',
 'magadan.spravka-region.ru': 'spravka-region.ru',
 'klimovsk.spravka-region.ru': 'spravka-region.ru',
 'verhnyaya-pyshma.spravka-region.ru': 'spravka-region.ru',
 'krasnodar.spravka-region.ru': 'spravka-region.ru',
 'ermakovskoe.spravka-region.ru': 'spravka-region.ru',
 'davlekanovo.spravka-region.ru': 'spravka-region.ru',
 'konstantinovsk.spravka-region.ru': 'spravka-region.ru',
 'igra.spravka-region.ru': 'spravka-region.ru',
 'aznakaevo.spravka-region.ru': 'spravka-region.ru',
 'spassk-dalnij.spravka-region.ru': 'spravka-region.ru',
 'sergiev-posad.spravka-region.ru': 'spravka-region.ru',
 'safonovo.spravka-region.ru': 'spravka-region.ru',
 'roslavl.spravka-region.ru': 'spravka-region.ru',
 'gubkinskij.spravka-region.ru': 'spravka-region.ru',
 'bugulma.spravka-region.ru': 'spravka-region.ru',
 'suhinichi.spravka-region.ru': 'spravka-region.ru',
 'lyskovo.spravka-region.ru': 'spravka-region.ru',
 'ivanteevka.spravka-region.ru': 'spravka-region.ru',
 'inozemcevo.spravka-region.ru': 'spravka-region.ru',
 'ozherele.spravka-region.ru': 'spravka-region.ru',
 'tosno.spravka-region.ru': 'spravka-region.ru',
 'timashevsk.spravka-region.ru': 'spravka-region.ru',
 'strugi-krasnye.spravka-region.ru': 'spravka-region.ru',
 'tihoreck.spravka-region.ru': 'spravka-region.ru',
 'nyagan.spravka-region.ru': 'spravka-region.ru',
 'joshkar-ola.spravka-region.ru': 'spravka-region.ru',
 'mcensk.spravka-region.ru': 'spravka-region.ru',
 'tarko-sale.spravka-region.ru': 'spravka-region.ru',
 'kokoshkino.spravka-region.ru': 'spravka-region.ru',
 'saratov.spravka-region.ru': 'spravka-region.ru',
 'kushevskaya.spravka-region.ru': 'spravka-region.ru',
 'ryazan.spravka-region.ru': 'spravka-region.ru',
 'nevel.spravka-region.ru': 'spravka-region.ru',
 'pryazha.spravka-region.ru': 'spravka-region.ru',
 'kopevo.spravka-region.ru': 'spravka-region.ru',
 'rasskazovo.spravka-region.ru': 'spravka-region.ru',
 'yahroma.spravka-region.ru': 'spravka-region.ru',
 'zarechnyj.spravka-region.ru': 'spravka-region.ru',
 'taganrog.spravka-region.ru': 'spravka-region.ru',
 'balabanovo.spravka-region.ru': 'spravka-region.ru',
 'volgograd.spravka-region.ru': 'spravka-region.ru',
 'sherbinka.spravka-region.ru': 'spravka-region.ru',
 'millerovo.spravka-region.ru': 'spravka-region.ru',
 'kolomna.spravka-region.ru': 'spravka-region.ru',
 'ishim.spravka-region.ru': 'spravka-region.ru',
 'kraskovo.spravka-region.ru': 'spravka-region.ru',
 'chapaevsk.spravka-region.ru': 'spravka-region.ru',
 'neftegorsk.spravka-region.ru': 'spravka-region.ru',
 'sharya.spravka-region.ru': 'spravka-region.ru',
 'noginsk.spravka-region.ru': 'spravka-region.ru',
 'kurovskoe.spravka-region.ru': 'spravka-region.ru',
 'bavly.spravka-region.ru': 'spravka-region.ru',
 'derbent.spravka-region.ru': 'spravka-region.ru',
 'efremov.spravka-region.ru': 'spravka-region.ru',
 'afipskij.spravka-region.ru': 'spravka-region.ru',
 'kamennogorsk.spravka-region.ru': 'spravka-region.ru',
 'ekaterinburg.spravka-region.ru': 'spravka-region.ru',
 'hotkovo.spravka-region.ru': 'spravka-region.ru',
 'myski.spravka-region.ru': 'spravka-region.ru',
 'klyavlino.spravka-region.ru': 'spravka-region.ru',
 'lyubinskij.spravka-region.ru': 'spravka-region.ru',
 'dalnerechensk.spravka-region.ru': 'spravka-region.ru',
 'kukmor.spravka-region.ru': 'spravka-region.ru',
 'ershov.spravka-region.ru': 'spravka-region.ru',
 'egorevsk.spravka-region.ru': 'spravka-region.ru',
 'novoshahtinsk.spravka-region.ru': 'spravka-region.ru',
 'solcy.spravka-region.ru': 'spravka-region.ru',
 'pokachi.spravka-region.ru': 'spravka-region.ru',
 'arsenev.spravka-region.ru': 'spravka-region.ru',
 'novokujbyshevsk.spravka-region.ru': 'spravka-region.ru',
 'pavlovskij-posad.spravka-region.ru': 'spravka-region.ru',
 'rodionovo-nesvetajskaya.spravka-region.ru': 'spravka-region.ru',
 'primorsko-ahtarsk.spravka-region.ru': 'spravka-region.ru',
 'alejsk.spravka-region.ru': 'spravka-region.ru',
 'agryz.spravka-region.ru': 'spravka-region.ru',
 'bezhanicy.spravka-region.ru': 'spravka-region.ru',
 'verhnie-tatyshly.spravka-region.ru': 'spravka-region.ru',
 'kaduj.spravka-region.ru': 'spravka-region.ru',
 'diveevo.spravka-region.ru': 'spravka-region.ru',
 'verbilki.spravka-region.ru': 'spravka-region.ru',
 'tatarsk.spravka-region.ru': 'spravka-region.ru',
 'vereya.spravka-region.ru': 'spravka-region.ru',
 'usinsk.spravka-region.ru': 'spravka-region.ru',
 'novotroick.spravka-region.ru': 'spravka-region.ru',
 'uholovo.spravka-region.ru': 'spravka-region.ru',
 'severouralsk.spravka-region.ru': 'spravka-region.ru',
 'nahabino.spravka-region.ru': 'spravka-region.ru',
 'luhovicy.spravka-region.ru': 'spravka-region.ru',
 'balahna.spravka-region.ru': 'spravka-region.ru',
 'novoaleksandrovsk.spravka-region.ru': 'spravka-region.ru',
 'pereyaslavka.spravka-region.ru': 'spravka-region.ru',
 'abinsk.spravka-region.ru': 'spravka-region.ru',
 'shelkino.spravka-region.ru': 'spravka-region.ru',
 'orenburg.spravka-region.ru': 'spravka-region.ru',
 'kandry.spravka-region.ru': 'spravka-region.ru',
 'dombaj.spravka-region.ru': 'spravka-region.ru',
 'kamyshlov.spravka-region.ru': 'spravka-region.ru',
 'penza.spravka-region.ru': 'spravka-region.ru',
 'megion.spravka-region.ru': 'spravka-region.ru',
 'artem.spravka-region.ru': 'spravka-region.ru',
 'dzerzhinsk.spravka-region.ru': 'spravka-region.ru',
 'belogorsk.spravka-region.ru': 'spravka-region.ru',
 'talica.spravka-region.ru': 'spravka-region.ru',
 'tomilino.spravka-region.ru': 'spravka-region.ru',
    'build2last.ru': 'build2last.ru',
 'vyazma.build2last.ru': 'build2last.ru',
 'buguruslan.build2last.ru': 'build2last.ru',
 'idea.build2last.ru': 'build2last.ru',
 'buzuluk.build2last.ru': 'build2last.ru',
 'novotroitsk2.build2last.ru': 'build2last.ru',
 'mednogorsk.build2last.ru': 'build2last.ru',
 'sorochinsk.build2last.ru': 'build2last.ru',
 'orenburg.build2last.ru': 'build2last.ru',
 'shahovskaya.build2last.ru': 'build2last.ru',
 'efremov.build2last.ru': 'build2last.ru',
 'bugulma.build2last.ru': 'build2last.ru',
 'almetevsk.build2last.ru': 'build2last.ru',
 'naberezhnie-chelni.build2last.ru': 'build2last.ru',
 'nizhnekamsk.build2last.ru': 'build2last.ru',
 'kazan.build2last.ru': 'build2last.ru',
 'zelenodolsk.build2last.ru': 'build2last.ru',
 'chishmi.build2last.ru': 'build2last.ru',
 'artemovskiy3.build2last.ru': 'build2last.ru',
 'kungur.build2last.ru': 'build2last.ru',
 'rasskazovo.build2last.ru': 'build2last.ru',
 'skopin.build2last.ru': 'build2last.ru',
 'iglino.build2last.ru': 'build2last.ru',
 'polevskoy.build2last.ru': 'build2last.ru',
 'strugi-krasnie.build2last.ru': 'build2last.ru',
 'pskov.build2last.ru': 'build2last.ru',
 'porhov.build2last.ru': 'build2last.ru',
 'kalyazin.build2last.ru': 'build2last.ru',
 'neftekumsk.build2last.ru': 'build2last.ru',
 'goryachiy-klyuch.build2last.ru': 'build2last.ru',
 'krasnodar.build2last.ru': 'build2last.ru',
 'armavir.build2last.ru': 'build2last.ru',
 'iskitim.build2last.ru': 'build2last.ru',
 'yuzhnouralsk.build2last.ru': 'build2last.ru',
 'abakan.build2last.ru': 'build2last.ru',
 'vladimir.build2last.ru': 'build2last.ru',
 'kovrov.build2last.ru': 'build2last.ru',
 'murom.build2last.ru': 'build2last.ru',
 'aleksandrov1.build2last.ru': 'build2last.ru',
 'komsomolsk-na-amure.build2last.ru': 'build2last.ru',
 'dalnerechensk.build2last.ru': 'build2last.ru',
 'tolyatti.build2last.ru': 'build2last.ru',
 'krimsk.build2last.ru': 'build2last.ru',
 'nitva1.build2last.ru': 'build2last.ru',
 'borovichi.build2last.ru': 'build2last.ru',
 'kropotkin.build2last.ru': 'build2last.ru',
 'vyatskie-polyani.build2last.ru': 'build2last.ru',
 'yakutsk.build2last.ru': 'build2last.ru',
 'novosibirsk.build2last.ru': 'build2last.ru',
 'tuymazi.build2last.ru': 'build2last.ru',
 'asbest.build2last.ru': 'build2last.ru',
 'shatura.build2last.ru': 'build2last.ru',
 'blagoveshchensk-amur.build2last.ru': 'build2last.ru',
 'serebryanie-prudi.build2last.ru': 'build2last.ru',
 'novovyatsk.build2last.ru': 'build2last.ru',
 'slobodskoy1.build2last.ru': 'build2last.ru',
 'kirov.build2last.ru': 'build2last.ru',
 'kirovo-chepetsk.build2last.ru': 'build2last.ru',
 'kotelnich.build2last.ru': 'build2last.ru',
 'berdsk.build2last.ru': 'build2last.ru',
 'kashira.build2last.ru': 'build2last.ru',
 'staraya-russa.build2last.ru': 'build2last.ru',
 'petropavlovsk-kamchatskiy1.build2last.ru': 'build2last.ru',
 'velikie-luki.build2last.ru': 'build2last.ru',
 'svetlograd.build2last.ru': 'build2last.ru',
 'zvenigorod.build2last.ru': 'build2last.ru',
 'turinsk.build2last.ru': 'build2last.ru',
 'kurganinsk.build2last.ru': 'build2last.ru',
 'hanti-mansiysk.build2last.ru': 'build2last.ru',
 'nefteyugansk.build2last.ru': 'build2last.ru',
 'kogalim.build2last.ru': 'build2last.ru',
 'surgut2.build2last.ru': 'build2last.ru',
 'nyagan.build2last.ru': 'build2last.ru',
 'kuvandik.build2last.ru': 'build2last.ru',
 'sisert.build2last.ru': 'build2last.ru',
 'novorossiysk.build2last.ru': 'build2last.ru',
 'sochi.build2last.ru': 'build2last.ru',
 'slavyansk-na-kubani.build2last.ru': 'build2last.ru',
 'tuapse.build2last.ru': 'build2last.ru',
 'kanevskaya1.build2last.ru': 'build2last.ru',
 'sterlitamak.build2last.ru': 'build2last.ru',
 'neftekamsk.build2last.ru': 'build2last.ru',
 'salavat.build2last.ru': 'build2last.ru',
 'beloretsk.build2last.ru': 'build2last.ru',
 'oktyabrskiy10.build2last.ru': 'build2last.ru',
 'furmanov.build2last.ru': 'build2last.ru',
 'stupino.build2last.ru': 'build2last.ru',
 'dorohovo.build2last.ru': 'build2last.ru',
 'kamen-na-obi.build2last.ru': 'build2last.ru',
 'barnaul.build2last.ru': 'build2last.ru',
 'novoaltaysk.build2last.ru': 'build2last.ru',
 'biysk.build2last.ru': 'build2last.ru',
 'zarinsk.build2last.ru': 'build2last.ru',
 'rubtsovsk.build2last.ru': 'build2last.ru',
 'arhangelsk.build2last.ru': 'build2last.ru',
 'kotlas.build2last.ru': 'build2last.ru',
 'severodvinsk.build2last.ru': 'build2last.ru',
 'novodvinsk.build2last.ru': 'build2last.ru',
 'mirniy6.build2last.ru': 'build2last.ru',
 'koryazhma.build2last.ru': 'build2last.ru',
 'novokubansk.build2last.ru': 'build2last.ru',
 'rtishchevo.build2last.ru': 'build2last.ru',
 'kozmodemyansk.build2last.ru': 'build2last.ru',
 'serov.build2last.ru': 'build2last.ru',
 'pervouralsk.build2last.ru': 'build2last.ru',
 'ekaterinburg.build2last.ru': 'build2last.ru',
 'nizhniy-tagil.build2last.ru': 'build2last.ru',
 'nevyansk.build2last.ru': 'build2last.ru',
 'kamensk-uralskiy.build2last.ru': 'build2last.ru',
 'sibay.build2last.ru': 'build2last.ru',
 'zaraysk.build2last.ru': 'build2last.ru',
 'sharya.build2last.ru': 'build2last.ru',
 'habarovsk.build2last.ru': 'build2last.ru',
 'mozhga.build2last.ru': 'build2last.ru',
 'sharkan.build2last.ru': 'build2last.ru',
 'mihaylovsk.build2last.ru': 'build2last.ru',
 'anapa.build2last.ru': 'build2last.ru',
 'temryuk.build2last.ru': 'build2last.ru',
 'novoorsk.build2last.ru': 'build2last.ru',
 'gus-hrustalniy.build2last.ru': 'build2last.ru',
 'nurlat.build2last.ru': 'build2last.ru',
 'afipskiy1.build2last.ru': 'build2last.ru',
 'emanzhelinsk.build2last.ru': 'build2last.ru',
 'yugorsk.build2last.ru': 'build2last.ru',
 'eysk.build2last.ru': 'build2last.ru',
 'poltavskaya.build2last.ru': 'build2last.ru',
 'kineshma.build2last.ru': 'build2last.ru',
 'uhta1.build2last.ru': 'build2last.ru',
 'salehard.build2last.ru': 'build2last.ru',
 'volsk.build2last.ru': 'build2last.ru',
 'magnitogorsk.build2last.ru': 'build2last.ru',
 'ozersk4.build2last.ru': 'build2last.ru',
 'zlatoust.build2last.ru': 'build2last.ru',
 'miass.build2last.ru': 'build2last.ru',
 'kopeysk.build2last.ru': 'build2last.ru',
 'bizhbulyak.build2last.ru': 'build2last.ru',
 'tihoretsk.build2last.ru': 'build2last.ru',
 'gorno-altaysk1.build2last.ru': 'build2last.ru',
 'amursk.build2last.ru': 'build2last.ru',
 'ruzaevka.build2last.ru': 'build2last.ru',
 'verhneyarkeevo.build2last.ru': 'build2last.ru',
 'leninsk-kuznetskiy.build2last.ru': 'build2last.ru',
 'pavlovskiy-posad.build2last.ru': 'build2last.ru',
 'himki.build2last.ru': 'build2last.ru',
 'moskva.build2last.ru': 'build2last.ru',
 'podolsk.build2last.ru': 'build2last.ru',
 'labinsk.build2last.ru': 'build2last.ru',
 'pereslavl-zalesskiy.build2last.ru': 'build2last.ru',
 'konakovo.build2last.ru': 'build2last.ru',
 'lukoyanov.build2last.ru': 'build2last.ru',
 'michurinsk.build2last.ru': 'build2last.ru',
 'tambov.build2last.ru': 'build2last.ru',
 'morshansk.build2last.ru': 'build2last.ru',
 'anzhero-sudzhensk1.build2last.ru': 'build2last.ru',
 'asha.build2last.ru': 'build2last.ru',
 'kolpino.build2last.ru': 'build2last.ru',
 'nolinsk.build2last.ru': 'build2last.ru',
 'belorechensk.build2last.ru': 'build2last.ru',
 'ust-labinsk.build2last.ru': 'build2last.ru',
 'nignegorskiy.build2last.ru': 'build2last.ru',
 'birsk.build2last.ru': 'build2last.ru',
 'roslavl.build2last.ru': 'build2last.ru',
 'kaliningrad.build2last.ru': 'build2last.ru',
 'sizran.build2last.ru': 'build2last.ru',
 'bahchisaray.build2last.ru': 'build2last.ru',
 'satka.build2last.ru': 'build2last.ru',
 'udomlya.build2last.ru': 'build2last.ru',
 'mihnevo.build2last.ru': 'build2last.ru',
 'abinsk.build2last.ru': 'build2last.ru',
 'bogorodsk.build2last.ru': 'build2last.ru',
 'novosergievka.build2last.ru': 'build2last.ru',
 'chapaevsk.build2last.ru': 'build2last.ru',
 'shuya.build2last.ru': 'build2last.ru',
 'chernushka.build2last.ru': 'build2last.ru',
 'pyatigorsk.build2last.ru': 'build2last.ru',
 'mamadish.build2last.ru': 'build2last.ru',
 'ishimbay.build2last.ru': 'build2last.ru',
 'vladivostok.build2last.ru': 'build2last.ru',
 'mihaylov.build2last.ru': 'build2last.ru',
 'lodeynoe-pole1.build2last.ru': 'build2last.ru',
 'leninogorsk.build2last.ru': 'build2last.ru',
 'irbit.build2last.ru': 'build2last.ru',
 'ufa.build2last.ru': 'build2last.ru',
 'kirzhach.build2last.ru': 'build2last.ru',
 'eniseysk.build2last.ru': 'build2last.ru',
 'zavodoukovsk.build2last.ru': 'build2last.ru',
 'stavropol.build2last.ru': 'build2last.ru',
 'luchegorsk.build2last.ru': 'build2last.ru',
 'livni.build2last.ru': 'build2last.ru',
 'kushnarenkovo.build2last.ru': 'build2last.ru',
 'verhnyaya-pishma.build2last.ru': 'build2last.ru',
 'zelenogorsk.build2last.ru': 'build2last.ru',
 'yuzhno-sahalinsk.build2last.ru': 'build2last.ru',
 'belebey.build2last.ru': 'build2last.ru',
 'safonovo.build2last.ru': 'build2last.ru',
 'yartsevo.build2last.ru': 'build2last.ru',
 'smolensk.build2last.ru': 'build2last.ru',
 'glazov.build2last.ru': 'build2last.ru',
 'yurga1.build2last.ru': 'build2last.ru',
 'primorsko-ahtarsk.build2last.ru': 'build2last.ru',
 'serpuhov.build2last.ru': 'build2last.ru',
 'revda.build2last.ru': 'build2last.ru',
 'belie-stolbi.build2last.ru': 'build2last.ru',
 'nevinnomissk.build2last.ru': 'build2last.ru',
 'kislovodsk.build2last.ru': 'build2last.ru',
 'dmitrov.build2last.ru': 'build2last.ru',
 'rostov-na-donu.build2last.ru': 'build2last.ru',
 'chusovoy.build2last.ru': 'build2last.ru',
 'usinsk1.build2last.ru': 'build2last.ru',
 'verhnie-tatishli.build2last.ru': 'build2last.ru',
 'taganrog.build2last.ru': 'build2last.ru',
 'shahti.build2last.ru': 'build2last.ru',
 'belaya-kalitva.build2last.ru': 'build2last.ru',
 'volgodonsk.build2last.ru': 'build2last.ru',
 'novocherkassk.build2last.ru': 'build2last.ru',
 'volokolamsk.build2last.ru': 'build2last.ru',
 'plyussa.build2last.ru': 'build2last.ru',
 'abdulino.build2last.ru': 'build2last.ru',
 'ulan-ude.build2last.ru': 'build2last.ru',
 'severobaykalsk.build2last.ru': 'build2last.ru',
 'kizil.build2last.ru': 'build2last.ru',
 'vichuga.build2last.ru': 'build2last.ru',
 'atkarsk.build2last.ru': 'build2last.ru',
 'ozinki.build2last.ru': 'build2last.ru',
 'sankt-peterburg.build2last.ru': 'build2last.ru',
 'kemerovo.build2last.ru': 'build2last.ru',
 'belovo1.build2last.ru': 'build2last.ru',
 'prokopevsk.build2last.ru': 'build2last.ru',
 'kiselevsk.build2last.ru': 'build2last.ru',
 'novokuznetsk.build2last.ru': 'build2last.ru',
 'achinsk.build2last.ru': 'build2last.ru',
 'talitsa.build2last.ru': 'build2last.ru',
 'artem.build2last.ru': 'build2last.ru',
 'novotitarovskaya1.build2last.ru': 'build2last.ru',
 'spassk-ryazanskiy.build2last.ru': 'build2last.ru',
 'volzhsk.build2last.ru': 'build2last.ru',
 'borskoe.build2last.ru': 'build2last.ru',
 'berezniki1.build2last.ru': 'build2last.ru',
 'lisva.build2last.ru': 'build2last.ru',
 'chaykovskiy.build2last.ru': 'build2last.ru',
 'tinda.build2last.ru': 'build2last.ru',
 'pechori.build2last.ru': 'build2last.ru',
 'nizhnevartovsk.build2last.ru': 'build2last.ru',
 'apsheronsk.build2last.ru': 'build2last.ru',
 'kusa.build2last.ru': 'build2last.ru',
 'aznakaevo.build2last.ru': 'build2last.ru',
 'velikiy-novgorod.build2last.ru': 'build2last.ru',
 'elizovo1.build2last.ru': 'build2last.ru',
 'kansk.build2last.ru': 'build2last.ru',
 'norilsk.build2last.ru': 'build2last.ru',
 'krasnoyarsk.build2last.ru': 'build2last.ru',
 'mozdok.build2last.ru': 'build2last.ru',
 'dalnee-konstantinovo.build2last.ru': 'build2last.ru',
 'zhirnovsk.build2last.ru': 'build2last.ru',
 'elabuga.build2last.ru': 'build2last.ru',
    'perm.pulscen.ru': 'pulscen.ru',
 'tomsk.pulscen.ru': 'pulscen.ru',
 'krasnoyarsk.pulscen.ru': 'pulscen.ru',
 'orenburg.pulscen.ru': 'pulscen.ru',
 'khabarovsk.pulscen.ru': 'pulscen.ru',
 'tambov.pulscen.ru': 'pulscen.ru',
 'ekb.pulscen.ru': 'pulscen.ru',
 'pulscen.ru': 'pulscen.ru',
 'tula.pulscen.ru': 'pulscen.ru',
 'yaroslavl.pulscen.ru': 'pulscen.ru',
 'nsk.pulscen.ru': 'pulscen.ru',
 'krasnodar.pulscen.ru': 'pulscen.ru',
 'vladivostok.pulscen.ru': 'pulscen.ru',
 'ufa.pulscen.ru': 'pulscen.ru',
 'pyshma.pulscen.ru': 'pulscen.ru',
 'msk.pulscen.ru': 'pulscen.ru',
 'himki.pulscen.ru': 'pulscen.ru',
 'mezhdurechensk.pulscen.ru': 'pulscen.ru',
 'samara.pulscen.ru': 'pulscen.ru',
 'balashiha.pulscen.ru': 'pulscen.ru',
 'tyumen.pulscen.ru': 'pulscen.ru',
 'rostov.pulscen.ru': 'pulscen.ru',
 'korolev.pulscen.ru': 'pulscen.ru',
 'kurgan.pulscen.ru': 'pulscen.ru',
 'nn.pulscen.ru': 'pulscen.ru',
 'kemerovo.pulscen.ru': 'pulscen.ru',
 'omsk.pulscen.ru': 'pulscen.ru',
 'kazan.pulscen.ru': 'pulscen.ru',
 'tagil.pulscen.ru': 'pulscen.ru',
 'spb.pulscen.ru': 'pulscen.ru',
 'chel.pulscen.ru': 'pulscen.ru',
 'izhevsk.pulscen.ru': 'pulscen.ru',
 'neftekamsk.pulscen.ru': 'pulscen.ru',
 'surgut.pulscen.ru': 'pulscen.ru',
 'volgograd.pulscen.ru': 'pulscen.ru',
 'novorossiysk.pulscen.ru': 'pulscen.ru',
 'ryazan.pulscen.ru': 'pulscen.ru',
 'schelkovo.pulscen.ru': 'pulscen.ru',
 'zelenograd.pulscen.ru': 'pulscen.ru',
 'shahty.pulscen.ru': 'pulscen.ru',
 'novokuznetsk.pulscen.ru': 'pulscen.ru',
 'kaliningrad.pulscen.ru': 'pulscen.ru',
 'maykop.pulscen.ru': 'pulscen.ru',
 'kirov.pulscen.ru': 'pulscen.ru',
 'berezovskiy.pulscen.ru': 'pulscen.ru',
 'saratov.pulscen.ru': 'pulscen.ru',
 'voronezh.pulscen.ru': 'pulscen.ru',
 'voskresensk.pulscen.ru': 'pulscen.ru',
 'smolensk.pulscen.ru': 'pulscen.ru',
 'barnaul.pulscen.ru': 'pulscen.ru',
 'berezniki.pulscen.ru': 'pulscen.ru',
 'penza.pulscen.ru': 'pulscen.ru',
 'eysk.pulscen.ru': 'pulscen.ru',
 'ivanovo.pulscen.ru': 'pulscen.ru',
 'nahodka.pulscen.ru': 'pulscen.ru',
 'orel.pulscen.ru': 'pulscen.ru',
 'cherepovets.pulscen.ru': 'pulscen.ru',
 'taganrog.pulscen.ru': 'pulscen.ru',
 'volgodonsk.pulscen.ru': 'pulscen.ru',
 'sochi.pulscen.ru': 'pulscen.ru',
 'abakan.pulscen.ru': 'pulscen.ru',
 'promtaransk.pulscen.ru': 'pulscen.ru',
 'oskol.pulscen.ru': 'pulscen.ru',
 'magnitogorsk.pulscen.ru': 'pulscen.ru',
 'engels.pulscen.ru': 'pulscen.ru',
 'sterlitamak.pulscen.ru': 'pulscen.ru',
 'simferopol.pulscen.ru': 'pulscen.ru',
 'pskov.pulscen.ru': 'pulscen.ru',
 'votkinsk.pulscen.ru': 'pulscen.ru',
 'cheboksary.pulscen.ru': 'pulscen.ru',
 'zhd.pulscen.ru': 'pulscen.ru',
 'ola.pulscen.ru': 'pulscen.ru',
 'garant-torg-produkt.pulscen.ru': 'pulscen.ru',
 'noginsk.pulscen.ru': 'pulscen.ru',
 'tolyatty.pulscen.ru': 'pulscen.ru',
 'irk.pulscen.ru': 'pulscen.ru',
 'orsk.pulscen.ru': 'pulscen.ru',
 'stavropol.pulscen.ru': 'pulscen.ru',
 'hanty.pulscen.ru': 'pulscen.ru',
 'dinamo-derevoobrabatyvayuschiy-zavod.pulscen.ru': 'pulscen.ru',
 'bologoenefteprodukt.pulscen.ru': 'pulscen.ru',
 'berdsk.pulscen.ru': 'pulscen.ru',
 'armavir.pulscen.ru': 'pulscen.ru',
 'stupino.pulscen.ru': 'pulscen.ru',
 'n-urengoy.pulscen.ru': 'pulscen.ru',
 'novgorod.pulscen.ru': 'pulscen.ru',
 'kropotkin.pulscen.ru': 'pulscen.ru',
 'komsomolsk.pulscen.ru': 'pulscen.ru',
 'nizhnekamsk.pulscen.ru': 'pulscen.ru',
 'ulianovsk.pulscen.ru': 'pulscen.ru',
 'tobolsk.pulscen.ru': 'pulscen.ru',
 'chelny.pulscen.ru': 'pulscen.ru',
 'saransk.pulscen.ru': 'pulscen.ru',
 'almetievsk.pulscen.ru': 'pulscen.ru',
 'gatchina.pulscen.ru': 'pulscen.ru',
 'lipetsk.pulscen.ru': 'pulscen.ru',
 'narofominsk.pulscen.ru': 'pulscen.ru',
 'kolomna.pulscen.ru': 'pulscen.ru',
 'murmansk.pulscen.ru': 'pulscen.ru',
 'novomoskovsk.pulscen.ru': 'pulscen.ru',
 'ussuriysk.pulscen.ru': 'pulscen.ru',
 'salavat.pulscen.ru': 'pulscen.ru',
 'oktyabrskiy.pulscen.ru': 'pulscen.ru',
 'kyzyl.pulscen.ru': 'pulscen.ru',
 'pervouralsk.pulscen.ru': 'pulscen.ru',
 'kaluga.pulscen.ru': 'pulscen.ru',
 'dzerzhinsk.pulscen.ru': 'pulscen.ru',
 'petrozavodsk.pulscen.ru': 'pulscen.ru',
 'uhta.pulscen.ru': 'pulscen.ru',
 'domodedovo.pulscen.ru': 'pulscen.ru',
 'essentuki.pulscen.ru': 'pulscen.ru',
 'pyatigorsk.pulscen.ru': 'pulscen.ru',
 'petropavlovsk.pulscen.ru': 'pulscen.ru',
 'skt.pulscen.ru': 'pulscen.ru',
 'kislovodsk.pulscen.ru': 'pulscen.ru',
 'kamensk.pulscen.ru': 'pulscen.ru',
 'nizhnevartovsk.pulscen.ru': 'pulscen.ru',
 'vidnoe.pulscen.ru': 'pulscen.ru',
 'sarapul.pulscen.ru': 'pulscen.ru',
 'bryansk.pulscen.ru': 'pulscen.ru',
 'dmitrov.pulscen.ru': 'pulscen.ru',
 'lakinskiy-pivzavod-filial-rudo-akva.pulscen.ru': 'pulscen.ru',
 'syzran.pulscen.ru': 'pulscen.ru',
 'vladimir.pulscen.ru': 'pulscen.ru',
 'blagovechensk.pulscen.ru': 'pulscen.ru',
 'ramenskoe.pulscen.ru': 'pulscen.ru',
 'odintsovo.pulscen.ru': 'pulscen.ru',
 'biysk.pulscen.ru': 'pulscen.ru',
 'glazov.pulscen.ru': 'pulscen.ru',
 'nefteyugansk.pulscen.ru': 'pulscen.ru',
 'elets.pulscen.ru': 'pulscen.ru',
 'belgorod.pulscen.ru': 'pulscen.ru',
 'orehovo-zuevo.pulscen.ru': 'pulscen.ru',
 'sakhalinsk.pulscen.ru': 'pulscen.ru',
 'vologda.pulscen.ru': 'pulscen.ru',
 'v-luki.pulscen.ru': 'pulscen.ru',
 'kamyshin.pulscen.ru': 'pulscen.ru',
 'severodvinsk.pulscen.ru': 'pulscen.ru',
 'elektrostal.pulscen.ru': 'pulscen.ru',
 'zheleznogorsk.pulscen.ru': 'pulscen.ru',
 'xcom-shop.pulscen.ru': 'pulscen.ru',
 'borisoglebsk.pulscen.ru': 'pulscen.ru',
 'top-akb.pulscen.ru': 'pulscen.ru',
 'novouralsk.pulscen.ru': 'pulscen.ru',
 'serpuhov.pulscen.ru': 'pulscen.ru',
 'volzhsky.pulscen.ru': 'pulscen.ru',
 'metako-kozhevenno-mehovoy-zavod.pulscen.ru': 'pulscen.ru',
 'nevinnomyssk.pulscen.ru': 'pulscen.ru',
 'tabletka72-meditsinskiy-internet-portal.pulscen.ru': 'pulscen.ru',
 'novocherkassk.pulscen.ru': 'pulscen.ru',
 'bataysk.pulscen.ru': 'pulscen.ru',
 'miass.pulscen.ru': 'pulscen.ru',
 'yakutsk.pulscen.ru': 'pulscen.ru',
 'krovlya-master-1.pulscen.ru': 'pulscen.ru',
 'remaks-1.pulscen.ru': 'pulscen.ru',
 'ulanude.pulscen.ru': 'pulscen.ru',
 'murom.pulscen.ru': 'pulscen.ru',
 'nalchik.pulscen.ru': 'pulscen.ru',
 'pushkino.pulscen.ru': 'pulscen.ru',
 'kotlas.pulscen.ru': 'pulscen.ru',
 'arh.pulscen.ru': 'pulscen.ru',
 'leninsk-kuz.pulscen.ru': 'pulscen.ru',
 'klin.pulscen.ru': 'pulscen.ru',
 'ugles-td.pulscen.ru': 'pulscen.ru',
 'krovelnyy-tsentr-smts.pulscen.ru': 'pulscen.ru',
 'podolsk.pulscen.ru': 'pulscen.ru',
 'obninsk.pulscen.ru': 'pulscen.ru',
 'seversk.pulscen.ru': 'pulscen.ru',
 'kovrov.pulscen.ru': 'pulscen.ru',
 'angarsk.pulscen.ru': 'pulscen.ru',
 'nmtmed.pulscen.ru': 'pulscen.ru',
 'glavmet.pulscen.ru': 'pulscen.ru',
 'lyubertsy.pulscen.ru': 'pulscen.ru',
 'ostin.pulscen.ru': 'pulscen.ru',
 'rossosh.pulscen.ru': 'pulscen.ru',
 'mytischi.pulscen.ru': 'pulscen.ru',
 'suomi.pulscen.ru': 'pulscen.ru',
 'vyksa.pulscen.ru': 'pulscen.ru',
 'ksm-314.pulscen.ru': 'pulscen.ru',
 'sevastopol.pulscen.ru': 'pulscen.ru',
 'crimea.pulscen.ru': 'pulscen.ru',
 's-posad.pulscen.ru': 'pulscen.ru',
 'novotroitsk.pulscen.ru': 'pulscen.ru',
 'vyborg.pulscen.ru': 'pulscen.ru',
 'tyumenskiy-hlebokombinat.pulscen.ru': 'pulscen.ru',
 'kursk.pulscen.ru': 'pulscen.ru',
 'achinsk.pulscen.ru': 'pulscen.ru',
 'tver.pulscen.ru': 'pulscen.ru',
 'mahachkala.pulscen.ru': 'pulscen.ru',
 'noyabrsk.pulscen.ru': 'pulscen.ru',
 'arzamas.pulscen.ru': 'pulscen.ru',
 'velobike.pulscen.ru': 'pulscen.ru',
 'puls-welltex.pulscen.ru': 'pulscen.ru',
 'autodush.pulscen.ru': 'pulscen.ru',
 'egorievsk.pulscen.ru': 'pulscen.ru',
 'chita.pulscen.ru': 'pulscen.ru',
 'kuzalians.pulscen.ru': 'pulscen.ru',
 'zhukovskiy.pulscen.ru': 'pulscen.ru',
 'kidus.pulscen.ru': 'pulscen.ru',
 'liski.pulscen.ru': 'pulscen.ru',
 'torro.pulscen.ru': 'pulscen.ru',
 'karmanovskiy-rybhoz.pulscen.ru': 'pulscen.ru',
 'apatity.pulscen.ru': 'pulscen.ru',
 'prokopievsk.pulscen.ru': 'pulscen.ru',
 'kzlmk.pulscen.ru': 'pulscen.ru',
 'surgut-pak.pulscen.ru': 'pulscen.ru',
 'ice-coolnn.pulscen.ru': 'pulscen.ru',
 'sosnovyy-bor.pulscen.ru': 'pulscen.ru',
 'korobeynik-3.pulscen.ru': 'pulscen.ru',
 'npo-kriomash.pulscen.ru': 'pulscen.ru',
 'zlatoust.pulscen.ru': 'pulscen.ru',
 'dolgoprudnyy.pulscen.ru': 'pulscen.ru',
 'sto178.pulscen.ru': 'pulscen.ru',
 'stroymarket-gosstroy.pulscen.ru': 'pulscen.ru',
 'kostroma.pulscen.ru': 'pulscen.ru',
 'omikron-9.pulscen.ru': 'pulscen.ru',
 'cccp-gun.pulscen.ru': 'pulscen.ru',
 'kirpichnyy-dvor-19.pulscen.ru': 'pulscen.ru',
 'balakovo.pulscen.ru': 'pulscen.ru',
 'vseinstrumenty.pulscen.ru': 'pulscen.ru',
 'vladikavkaz.pulscen.ru': 'pulscen.ru',
 'alvis-plus.pulscen.ru': 'pulscen.ru',
 'aleana-mebel.pulscen.ru': 'pulscen.ru',
 'pushkin.pulscen.ru': 'pulscen.ru',
 'visstroy.pulscen.ru': 'pulscen.ru',
 'chehov.pulscen.ru': 'pulscen.ru',
 'zelam.pulscen.ru': 'pulscen.ru',
 '7-kubov-1.pulscen.ru': 'pulscen.ru',
 '7-diametrov.pulscen.ru': 'pulscen.ru',
 'chaykovskiy.pulscen.ru': 'pulscen.ru',
 'opttorgsiti.pulscen.ru': 'pulscen.ru',
 'yalta.pulscen.ru': 'pulscen.ru',
 'iwojimaairsoft.pulscen.ru': 'pulscen.ru',
 'vscentr.pulscen.ru': 'pulscen.ru',
 'pervocvet.pulscen.ru': 'pulscen.ru',
 'krasnogorsk.pulscen.ru': 'pulscen.ru',
 'cherkessk.pulscen.ru': 'pulscen.ru',
 'torgkabel-1.pulscen.ru': 'pulscen.ru',
 'dzerzhinskoe-orgsteklo.pulscen.ru': 'pulscen.ru',
 'chistkakanaliz24.pulscen.ru': 'pulscen.ru',
 'vishnevskiy-kolhoz.pulscen.ru': 'pulscen.ru',
 'novokuybyshevsk.pulscen.ru': 'pulscen.ru',
 '3810123.pulscen.ru': 'pulscen.ru',
 'avtoplast-9.pulscen.ru': 'pulscen.ru',
 'minvody.pulscen.ru': 'pulscen.ru',
 'tilipin-sv.pulscen.ru': 'pulscen.ru',
 'sibirskiy-dom-krovli.pulscen.ru': 'pulscen.ru',
 'kstovo.pulscen.ru': 'pulscen.ru',
 'rubtsovsk.pulscen.ru': 'pulscen.ru',
 '4water.pulscen.ru': 'pulscen.ru',
 'putech.pulscen.ru': 'pulscen.ru',
 'solnechnogorsk.pulscen.ru': 'pulscen.ru',
 'vseinstrumentyru-krasn.pulscen.ru': 'pulscen.ru',
 'zbi29arhangelsk.pulscen.ru': 'pulscen.ru',
 'kraust.pulscen.ru': 'pulscen.ru',
 'michurinsk.pulscen.ru': 'pulscen.ru',
 'toplivnaya-twonp.pulscen.ru': 'pulscen.ru',
 'kerch.pulscen.ru': 'pulscen.ru',
 'rezinotehnika-1.pulscen.ru': 'pulscen.ru',
 'lobnya.pulscen.ru': 'pulscen.ru',
 'reutov.pulscen.ru': 'pulscen.ru',
 'rosposuda.pulscen.ru': 'pulscen.ru',
 'iesa56.pulscen.ru': 'pulscen.ru',
 'dynasty-dvru.pulscen.ru': 'pulscen.ru',
    'orenburg.kinoafisha.info': 'kinoafisha.info',
 'tuapse.kinoafisha.info': 'kinoafisha.info',
 'kinoafisha.info': 'kinoafisha.info',
 'grozny.kinoafisha.info': 'kinoafisha.info',
 'msk.kinoafisha.info': 'kinoafisha.info',
 'spb.kinoafisha.info': 'kinoafisha.info',
 'ufa.kinoafisha.info': 'kinoafisha.info',
 'nn.kinoafisha.info': 'kinoafisha.info',
 'izhevsk.kinoafisha.info': 'kinoafisha.info',
 'nerungri.kinoafisha.info': 'kinoafisha.info',
 'anapa.kinoafisha.info': 'kinoafisha.info',
 'vlg.kinoafisha.info': 'kinoafisha.info',
 'kirov.kinoafisha.info': 'kinoafisha.info',
 'tikhoretsk.kinoafisha.info': 'kinoafisha.info',
 'novomoskovsk.kinoafisha.info': 'kinoafisha.info',
 'rnd.kinoafisha.info': 'kinoafisha.info',
 'vologda.kinoafisha.info': 'kinoafisha.info',
 'sochi.kinoafisha.info': 'kinoafisha.info',
 'nsk.kinoafisha.info': 'kinoafisha.info',
 'liskovo.kinoafisha.info': 'kinoafisha.info',
 'kzn.kinoafisha.info': 'kinoafisha.info',
 'saransk.kinoafisha.info': 'kinoafisha.info',
 'astrakhan.kinoafisha.info': 'kinoafisha.info',
 'mytischi.kinoafisha.info': 'kinoafisha.info',
 'perm.kinoafisha.info': 'kinoafisha.info',
 'velnovgorod.kinoafisha.info': 'kinoafisha.info',
 'nijnevartovsk.kinoafisha.info': 'kinoafisha.info',
 'neftekamsk.kinoafisha.info': 'kinoafisha.info',
 'blagoveschensk.kinoafisha.info': 'kinoafisha.info',
 'tmn.kinoafisha.info': 'kinoafisha.info',
 'slavjansk-na-kubani.kinoafisha.info': 'kinoafisha.info',
 'smr.kinoafisha.info': 'kinoafisha.info',
 'cherkessk.kinoafisha.info': 'kinoafisha.info',
 'omsk.kinoafisha.info': 'kinoafisha.info',
 'lesnojgorodok.kinoafisha.info': 'kinoafisha.info',
 'barnaul.kinoafisha.info': 'kinoafisha.info',
 'khabarovsk.kinoafisha.info': 'kinoafisha.info',
 'kemerovo.kinoafisha.info': 'kinoafisha.info',
 'ulyanovsk.kinoafisha.info': 'kinoafisha.info',
 'gorodets.kinoafisha.info': 'kinoafisha.info',
 'zavolzhie.kinoafisha.info': 'kinoafisha.info',
 'stavropol.kinoafisha.info': 'kinoafisha.info',
 'kr.kinoafisha.info': 'kinoafisha.info',
 'chehov.kinoafisha.info': 'kinoafisha.info',
 'balashikha.kinoafisha.info': 'kinoafisha.info',
 'surgut.kinoafisha.info': 'kinoafisha.info',
 'smolensk.kinoafisha.info': 'kinoafisha.info',
 'svobodny.kinoafisha.info': 'kinoafisha.info',
 'kms.kinoafisha.info': 'kinoafisha.info',
 'ekt.kinoafisha.info': 'kinoafisha.info',
 'armavir.kinoafisha.info': 'kinoafisha.info',
 'murmansk.kinoafisha.info': 'kinoafisha.info',
 'nkyz.kinoafisha.info': 'kinoafisha.info',
 'kaluga.kinoafisha.info': 'kinoafisha.info',
 'tolyatti.kinoafisha.info': 'kinoafisha.info',
 'belgorod.kinoafisha.info': 'kinoafisha.info',
 'zadonsk.kinoafisha.info': 'kinoafisha.info',
 'kostroma.kinoafisha.info': 'kinoafisha.info',
 'ntagil.kinoafisha.info': 'kinoafisha.info',
 'severomorsk.kinoafisha.info': 'kinoafisha.info',
 'kropotkin.kinoafisha.info': 'kinoafisha.info',
 'ulan-ude.kinoafisha.info': 'kinoafisha.info',
 'yakutsk.kinoafisha.info': 'kinoafisha.info',
 'maykop.kinoafisha.info': 'kinoafisha.info',
 'anzhero-sudzhensk.kinoafisha.info': 'kinoafisha.info',
 'kz.kinoafisha.info': 'kinoafisha.info',
 'vl.kinoafisha.info': 'kinoafisha.info',
 'kolomna.kinoafisha.info': 'kinoafisha.info',
 'cherepovec.kinoafisha.info': 'kinoafisha.info',
 'tver.kinoafisha.info': 'kinoafisha.info',
 'shakhty.kinoafisha.info': 'kinoafisha.info',
 'lyubertsy.kinoafisha.info': 'kinoafisha.info',
 'kaliningrad.kinoafisha.info': 'kinoafisha.info',
 'yaroslavl.kinoafisha.info': 'kinoafisha.info',
 'lomonosov.kinoafisha.info': 'kinoafisha.info',
 'biysk.kinoafisha.info': 'kinoafisha.info',
 'oskol.kinoafisha.info': 'kinoafisha.info',
 'petrozavodsk.kinoafisha.info': 'kinoafisha.info',
 'mamadish.kinoafisha.info': 'kinoafisha.info',
 'nizhnekamsk.kinoafisha.info': 'kinoafisha.info',
 'vladimir.kinoafisha.info': 'kinoafisha.info',
 'belorechensk.kinoafisha.info': 'kinoafisha.info',
 'taganrog.kinoafisha.info': 'kinoafisha.info',
 'elektrostal.kinoafisha.info': 'kinoafisha.info',
 'korenovsk.kinoafisha.info': 'kinoafisha.info',
 'kuybyshev.kinoafisha.info': 'kinoafisha.info',
 'vladikavkaz.kinoafisha.info': 'kinoafisha.info',
 'ryazan.kinoafisha.info': 'kinoafisha.info',
 'lebedyan.kinoafisha.info': 'kinoafisha.info',
 'nefteyugansk.kinoafisha.info': 'kinoafisha.info',
 'lipetsk.kinoafisha.info': 'kinoafisha.info',
 'temryuk.kinoafisha.info': 'kinoafisha.info',
 'solnechnogorsk.kinoafisha.info': 'kinoafisha.info',
 'serov.kinoafisha.info': 'kinoafisha.info',
 'lt.kinoafisha.info': 'kinoafisha.info',
 'voronezh.kinoafisha.info': 'kinoafisha.info',
 'tugulym.kinoafisha.info': 'kinoafisha.info',
 'podolsk.kinoafisha.info': 'kinoafisha.info',
 'orel.kinoafisha.info': 'kinoafisha.info',
 'yoshkar-ola.kinoafisha.info': 'kinoafisha.info',
 'sterlitamak.kinoafisha.info': 'kinoafisha.info',
 'srt.kinoafisha.info': 'kinoafisha.info',
 'leninogorsk.kinoafisha.info': 'kinoafisha.info',
 'egorievsk.kinoafisha.info': 'kinoafisha.info',
 'kursk.kinoafisha.info': 'kinoafisha.info',
 'buguruslan.kinoafisha.info': 'kinoafisha.info',
 'ershov.kinoafisha.info': 'kinoafisha.info',
 'serpuhov.kinoafisha.info': 'kinoafisha.info',
 'osinovo.kinoafisha.info': 'kinoafisha.info',
 'essentuki.kinoafisha.info': 'kinoafisha.info',
 'kurgan.kinoafisha.info': 'kinoafisha.info',
 'berdsk.kinoafisha.info': 'kinoafisha.info',
 'chistopol.kinoafisha.info': 'kinoafisha.info',
 'zhukovsky.kinoafisha.info': 'kinoafisha.info',
 'magadan.kinoafisha.info': 'kinoafisha.info',
 'shatura.kinoafisha.info': 'kinoafisha.info',
 'stupino.kinoafisha.info': 'kinoafisha.info',
 'ust-ilimsk.kinoafisha.info': 'kinoafisha.info',
 'tomsk.kinoafisha.info': 'kinoafisha.info',
 'mozhga.kinoafisha.info': 'kinoafisha.info',
 'vluki.kinoafisha.info': 'kinoafisha.info',
 'krsk.kinoafisha.info': 'kinoafisha.info',
 'gulkevichi.kinoafisha.info': 'kinoafisha.info',
 'volgodonsk.kinoafisha.info': 'kinoafisha.info',
 'orsk.kinoafisha.info': 'kinoafisha.info',
 'salekhard.kinoafisha.info': 'kinoafisha.info',
 'severodvinsk.kinoafisha.info': 'kinoafisha.info',
 'reutov.kinoafisha.info': 'kinoafisha.info',
 'novouralsk.kinoafisha.info': 'kinoafisha.info',
 'nazarovo.kinoafisha.info': 'kinoafisha.info',
 'roslavl.kinoafisha.info': 'kinoafisha.info',
 'tambov.kinoafisha.info': 'kinoafisha.info',
 'nabchelny.kinoafisha.info': 'kinoafisha.info',
 'mednogorsk.kinoafisha.info': 'kinoafisha.info',
 'yeysk.kinoafisha.info': 'kinoafisha.info',
 'bryansk.kinoafisha.info': 'kinoafisha.info',
 'gubkin.kinoafisha.info': 'kinoafisha.info',
 'zheleznovodsk.kinoafisha.info': 'kinoafisha.info',
 'zhlzngrsk.kinoafisha.info': 'kinoafisha.info',
 'penza.kinoafisha.info': 'kinoafisha.info',
 'dinskaya.kinoafisha.info': 'kinoafisha.info',
 'kuznetsk.kinoafisha.info': 'kinoafisha.info',
 'novocherkassk.kinoafisha.info': 'kinoafisha.info',
 'novoaltaysk.kinoafisha.info': 'kinoafisha.info',
 'siktivkar.kinoafisha.info': 'kinoafisha.info',
 'sortavala.kinoafisha.info': 'kinoafisha.info',
 'zelenodolsk.kinoafisha.info': 'kinoafisha.info',
 'chernogorsk.kinoafisha.info': 'kinoafisha.info',
 'gagra.kinoafisha.info': 'kinoafisha.info',
 'desnogorsk.kinoafisha.info': 'kinoafisha.info',
 'georgiyevsk.kinoafisha.info': 'kinoafisha.info',
 'sarapul.kinoafisha.info': 'kinoafisha.info',
 'shchyolkovo.kinoafisha.info': 'kinoafisha.info',
 'pervouralsk.kinoafisha.info': 'kinoafisha.info',
 'kotlas.kinoafisha.info': 'kinoafisha.info',
 'himki.kinoafisha.info': 'kinoafisha.info',
 'lobna.kinoafisha.info': 'kinoafisha.info',
 'mgn.kinoafisha.info': 'kinoafisha.info',
 'volsk.kinoafisha.info': 'kinoafisha.info',
 'volkhov.kinoafisha.info': 'kinoafisha.info',
 'chel.kinoafisha.info': 'kinoafisha.info',
 'balashov.kinoafisha.info': 'kinoafisha.info',
 'tula.kinoafisha.info': 'kinoafisha.info',
 'amursk.kinoafisha.info': 'kinoafisha.info',
 'dzerzhinsk.kinoafisha.info': 'kinoafisha.info',
 'pyatigorsk.kinoafisha.info': 'kinoafisha.info',
 'rubtsovsk.kinoafisha.info': 'kinoafisha.info',
 'sovgavan.kinoafisha.info': 'kinoafisha.info',
 'tynda.kinoafisha.info': 'kinoafisha.info',
 'pskov.kinoafisha.info': 'kinoafisha.info',
 'abakan.kinoafisha.info': 'kinoafisha.info',
 'inza.kinoafisha.info': 'kinoafisha.info',
 'yuzhno-sakh.kinoafisha.info': 'kinoafisha.info',
 'ust-labinsk.kinoafisha.info': 'kinoafisha.info',
 'gelendzhik.kinoafisha.info': 'kinoafisha.info',
 'bor.kinoafisha.info': 'kinoafisha.info',
 'novorossiysk.kinoafisha.info': 'kinoafisha.info',
 'belogorsk.kinoafisha.info': 'kinoafisha.info',
 'krasnogorsk.kinoafisha.info': 'kinoafisha.info',
 'nalchik.kinoafisha.info': 'kinoafisha.info',
 'dolgoprudny.kinoafisha.info': 'kinoafisha.info',
 'primorsko-ahtarsk.kinoafisha.info': 'kinoafisha.info',
 'engels.kinoafisha.info': 'kinoafisha.info',
 'ussuriysk.kinoafisha.info': 'kinoafisha.info',
 'archangelsk.kinoafisha.info': 'kinoafisha.info',
 'yalutorovsk.kinoafisha.info': 'kinoafisha.info',
 'voskresensk.kinoafisha.info': 'kinoafisha.info',
 'gorno-altaysk.kinoafisha.info': 'kinoafisha.info',
 'iskitim.kinoafisha.info': 'kinoafisha.info',
 'svetlograd.kinoafisha.info': 'kinoafisha.info',
 'kyzyl.kinoafisha.info': 'kinoafisha.info',
 'zlatoust.kinoafisha.info': 'kinoafisha.info',
 'tuymazy.kinoafisha.info': 'kinoafisha.info',
 'sergiyev-posad.kinoafisha.info': 'kinoafisha.info',
 'krasnoznamensk.kinoafisha.info': 'kinoafisha.info',
 'okulovka.kinoafisha.info': 'kinoafisha.info',
 'kumertau.kinoafisha.info': 'kinoafisha.info',
 'gukovo.kinoafisha.info': 'kinoafisha.info',
 'strezhevoy.kinoafisha.info': 'kinoafisha.info',
 'votkinsk.kinoafisha.info': 'kinoafisha.info',
 'petropavlovsk-kamchatsky.kinoafisha.info': 'kinoafisha.info',
 'kotovo.kinoafisha.info': 'kinoafisha.info',
 'cheboksary.kinoafisha.info': 'kinoafisha.info',
 'fryazino.kinoafisha.info': 'kinoafisha.info',
 'sarov.kinoafisha.info': 'kinoafisha.info',
 'almetyevsk.kinoafisha.info': 'kinoafisha.info',
 'gatchina.kinoafisha.info': 'kinoafisha.info',
 'kurganinsk.kinoafisha.info': 'kinoafisha.info',
 'ivanovo.kinoafisha.info': 'kinoafisha.info',
 'timashevsk.kinoafisha.info': 'kinoafisha.info',
 'afipskiy.kinoafisha.info': 'kinoafisha.info',
 'inozemcevo.kinoafisha.info': 'kinoafisha.info',
 'krasniysulin.kinoafisha.info': 'kinoafisha.info',
 'konokovo.kinoafisha.info': 'kinoafisha.info',
 'bolshoykmn.kinoafisha.info': 'kinoafisha.info',
 'odintsovo.kinoafisha.info': 'kinoafisha.info',
 'shebekino.kinoafisha.info': 'kinoafisha.info',
 'pushkino.kinoafisha.info': 'kinoafisha.info',
 'tarko-sale.kinoafisha.info': 'kinoafisha.info',
 'miass.kinoafisha.info': 'kinoafisha.info',
 'krasnokamsk.kinoafisha.info': 'kinoafisha.info',
 'muslumovo.kinoafisha.info': 'kinoafisha.info',
 'kansk.kinoafisha.info': 'kinoafisha.info',
 'kineshma.kinoafisha.info': 'kinoafisha.info',
 'megion.kinoafisha.info': 'kinoafisha.info',
 'mostovskoi.kinoafisha.info': 'kinoafisha.info',
 'yasny.kinoafisha.info': 'kinoafisha.info',
 'rybinsk.kinoafisha.info': 'kinoafisha.info',
 'salavat.kinoafisha.info': 'kinoafisha.info',
 'orlovskiy.kinoafisha.info': 'kinoafisha.info',
 'otradny.kinoafisha.info': 'kinoafisha.info',
 'novayausman.kinoafisha.info': 'kinoafisha.info',
 'kochubeevskoe.kinoafisha.info': 'kinoafisha.info',
 'dolzhanskaya.kinoafisha.info': 'kinoafisha.info',
 'elizovo.kinoafisha.info': 'kinoafisha.info',
 'konakovo.kinoafisha.info': 'kinoafisha.info',
 'chaykovsky.kinoafisha.info': 'kinoafisha.info',
 'zelenograd.kinoafisha.info': 'kinoafisha.info',
 'achinsk.kinoafisha.info': 'kinoafisha.info',
 'lodeynoyepole.kinoafisha.info': 'kinoafisha.info',
 'noginsk.kinoafisha.info': 'kinoafisha.info',
 'chudovo.kinoafisha.info': 'kinoafisha.info',
 'mikhailovsk.kinoafisha.info': 'kinoafisha.info',
 'zeya.kinoafisha.info': 'kinoafisha.info',
 'yelets.kinoafisha.info': 'kinoafisha.info',
 'khanty-mansiysk.kinoafisha.info': 'kinoafisha.info',
 'schekino.kinoafisha.info': 'kinoafisha.info',
 'norilsk.kinoafisha.info': 'kinoafisha.info',
 'borisoglebsk.kinoafisha.info': 'kinoafisha.info',
 'berezniki.kinoafisha.info': 'kinoafisha.info',
 'agryz.kinoafisha.info': 'kinoafisha.info',
 'sokol.kinoafisha.info': 'kinoafisha.info',
 'nizhnyayatura.kinoafisha.info': 'kinoafisha.info',
 'segezha.kinoafisha.info': 'kinoafisha.info',
 'uray.kinoafisha.info': 'kinoafisha.info',
 'kinel.kinoafisha.info': 'kinoafisha.info',
 'arzamas.kinoafisha.info': 'kinoafisha.info',
 'kanash.kinoafisha.info': 'kinoafisha.info',
 'elista.kinoafisha.info': 'kinoafisha.info',
 'azov.kinoafisha.info': 'kinoafisha.info',
 'kashira.kinoafisha.info': 'kinoafisha.info',
 'celtso.kinoafisha.info': 'kinoafisha.info',
 'korolev.kinoafisha.info': 'kinoafisha.info',
 'koryazhma.kinoafisha.info': 'kinoafisha.info',
 'petergof.kinoafisha.info': 'kinoafisha.info',
 'kyshtym.kinoafisha.info': 'kinoafisha.info',
 'alnashi.kinoafisha.info': 'kinoafisha.info',
 'urupinsk.kinoafisha.info': 'kinoafisha.info',
 'zverevo.kinoafisha.info': 'kinoafisha.info',
 'aban.kinoafisha.info': 'kinoafisha.info',
 'yaransk.kinoafisha.info': 'kinoafisha.info',
 'satka.kinoafisha.info': 'kinoafisha.info',
 'volzhskiy.kinoafisha.info': 'kinoafisha.info',
 'istra.kinoafisha.info': 'kinoafisha.info',
 'staroscherb.kinoafisha.info': 'kinoafisha.info',
 'naryan-mar.kinoafisha.info': 'kinoafisha.info',
 'kamensk-shakhtinsky.kinoafisha.info': 'kinoafisha.info',
 'sukhum.kinoafisha.info': 'kinoafisha.info',
 'vidnoe.kinoafisha.info': 'kinoafisha.info',
 'apsheronsk.kinoafisha.info': 'kinoafisha.info',
 'ruzaevka.kinoafisha.info': 'kinoafisha.info',
 'pushkin.kinoafisha.info': 'kinoafisha.info',
 'inta.kinoafisha.info': 'kinoafisha.info',
 'novoaleksandrovsk.kinoafisha.info': 'kinoafisha.info',
 'zvenigorod.kinoafisha.info': 'kinoafisha.info',
 'ishim.kinoafisha.info': 'kinoafisha.info',
 'az.kinoafisha.info': 'kinoafisha.info',
 'dyurtyuli.kinoafisha.info': 'kinoafisha.info',
 'civilsk.kinoafisha.info': 'kinoafisha.info',
 'obninsk.kinoafisha.info': 'kinoafisha.info',
 'pionerskiy.kinoafisha.info': 'kinoafisha.info',
 'kushva.kinoafisha.info': 'kinoafisha.info',
 'yuzhnouralsk.kinoafisha.info': 'kinoafisha.info',
 'behteevka.kinoafisha.info': 'kinoafisha.info',
 'kozlovka.kinoafisha.info': 'kinoafisha.info',
 'prokopjevsk.kinoafisha.info': 'kinoafisha.info',
 'kiselyovsk.kinoafisha.info': 'kinoafisha.info',
 'tavda.kinoafisha.info': 'kinoafisha.info',
 'zimovniki.kinoafisha.info': 'kinoafisha.info',
 'balakovo.kinoafisha.info': 'kinoafisha.info',
 'orekhovo-zuyevo.kinoafisha.info': 'kinoafisha.info',
 'novoshakhtinsk.kinoafisha.info': 'kinoafisha.info',
 'bronnitsy.kinoafisha.info': 'kinoafisha.info',
 'dyatkovo.kinoafisha.info': 'kinoafisha.info',
 'simf.kinoafisha.info': 'kinoafisha.info',
 'akhtubinsk.kinoafisha.info': 'kinoafisha.info',
 'sayanogorsk.kinoafisha.info': 'kinoafisha.info',
 'stroitel.kinoafisha.info': 'kinoafisha.info',
 'tobolsk.kinoafisha.info': 'kinoafisha.info',
    'nsk.nuipogoda.ru': 'nuipogoda.ru',
 'petrozavodsk.nuipogoda.ru': 'nuipogoda.ru',
 'msk.nuipogoda.ru': 'nuipogoda.ru',
 'nuipogoda.ru': 'nuipogoda.ru',
 'hasan.nuipogoda.ru': 'nuipogoda.ru',
 'livadiya-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
 'andreevka-hasanskiy-rayon-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
 'zarubino-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
 'novotroick.nuipogoda.ru': 'nuipogoda.ru',
 'rostov-na-donu.nuipogoda.ru': 'nuipogoda.ru',
 'krasnodar.nuipogoda.ru': 'nuipogoda.ru',
 'tobolsk.nuipogoda.ru': 'nuipogoda.ru',
 'livny.nuipogoda.ru': 'nuipogoda.ru',
 'ruza.nuipogoda.ru': 'nuipogoda.ru',
 'seversk.nuipogoda.ru': 'nuipogoda.ru',
 'tolyatti.nuipogoda.ru': 'nuipogoda.ru',
 'olenevka-rossiya.nuipogoda.ru': 'nuipogoda.ru',
 'izhevsk.nuipogoda.ru': 'nuipogoda.ru',
 'tyumen.nuipogoda.ru': 'nuipogoda.ru',
 'medvezhegorsk.nuipogoda.ru': 'nuipogoda.ru',
 'orsk.nuipogoda.ru': 'nuipogoda.ru',
 'ufa.nuipogoda.ru': 'nuipogoda.ru',
 'sochi.nuipogoda.ru': 'nuipogoda.ru',
 'nkz.nuipogoda.ru': 'nuipogoda.ru',
 'koryazhma.nuipogoda.ru': 'nuipogoda.ru',
 'samara.nuipogoda.ru': 'nuipogoda.ru',
 'hanty-mansiysk.nuipogoda.ru': 'nuipogoda.ru',
 'nizhnevartovsk.nuipogoda.ru': 'nuipogoda.ru',
 'perm.nuipogoda.ru': 'nuipogoda.ru',
 'novorossiysk.nuipogoda.ru': 'nuipogoda.ru',
 'volgograd.nuipogoda.ru': 'nuipogoda.ru',
 'spb.nuipogoda.ru': 'nuipogoda.ru',
 'zlatoust.nuipogoda.ru': 'nuipogoda.ru',
 'tula.nuipogoda.ru': 'nuipogoda.ru',
 'vrangel.nuipogoda.ru': 'nuipogoda.ru',
 'magnitogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'sibay.nuipogoda.ru': 'nuipogoda.ru',
 'reftinskiy.nuipogoda.ru': 'nuipogoda.ru',
 'vladimir.nuipogoda.ru': 'nuipogoda.ru',
 'vladikavkaz.nuipogoda.ru': 'nuipogoda.ru',
 'kazan.nuipogoda.ru': 'nuipogoda.ru',
 'gelendzhik.nuipogoda.ru': 'nuipogoda.ru',
 'kamyshlov.nuipogoda.ru': 'nuipogoda.ru',
 'mahachkala.nuipogoda.ru': 'nuipogoda.ru',
 'tambov.nuipogoda.ru': 'nuipogoda.ru',
 'penza.nuipogoda.ru': 'nuipogoda.ru',
 'svetlogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'tomsk.nuipogoda.ru': 'nuipogoda.ru',
 'tynda.nuipogoda.ru': 'nuipogoda.ru',
 'tver.nuipogoda.ru': 'nuipogoda.ru',
 'habarovsk.nuipogoda.ru': 'nuipogoda.ru',
 'toguchin.nuipogoda.ru': 'nuipogoda.ru',
 'ekaterinburg.nuipogoda.ru': 'nuipogoda.ru',
 'balashiha.nuipogoda.ru': 'nuipogoda.ru',
 'elec.nuipogoda.ru': 'nuipogoda.ru',
 'sevastopol.nuipogoda.ru': 'nuipogoda.ru',
 'syktyvkar.nuipogoda.ru': 'nuipogoda.ru',
 'kansk.nuipogoda.ru': 'nuipogoda.ru',
 'vityaz.nuipogoda.ru': 'nuipogoda.ru',
 'adler.nuipogoda.ru': 'nuipogoda.ru',
 'chelyabinsk.nuipogoda.ru': 'nuipogoda.ru',
 'tuymazy.nuipogoda.ru': 'nuipogoda.ru',
 'kavalerovo.nuipogoda.ru': 'nuipogoda.ru',
 'tashkent.nuipogoda.ru': 'nuipogoda.ru',
 'revda.nuipogoda.ru': 'nuipogoda.ru',
 'stavropol.nuipogoda.ru': 'nuipogoda.ru',
 'vologda.nuipogoda.ru': 'nuipogoda.ru',
 'achinsk.nuipogoda.ru': 'nuipogoda.ru',
 'krasnoyarsk.nuipogoda.ru': 'nuipogoda.ru',
 'anapa.nuipogoda.ru': 'nuipogoda.ru',
 'vardane.nuipogoda.ru': 'nuipogoda.ru',
 'syzran.nuipogoda.ru': 'nuipogoda.ru',
 'simferopol.nuipogoda.ru': 'nuipogoda.ru',
 'talakan-saha.nuipogoda.ru': 'nuipogoda.ru',
 'sarapul.nuipogoda.ru': 'nuipogoda.ru',
 'svetloyar.nuipogoda.ru': 'nuipogoda.ru',
 'olginka-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'zelenogradsk.nuipogoda.ru': 'nuipogoda.ru',
 'mezhdurechensk.nuipogoda.ru': 'nuipogoda.ru',
 'karaganda.nuipogoda.ru': 'nuipogoda.ru',
 'engels.nuipogoda.ru': 'nuipogoda.ru',
 'noyabrsk.nuipogoda.ru': 'nuipogoda.ru',
 'saransk.nuipogoda.ru': 'nuipogoda.ru',
 'smolensk.nuipogoda.ru': 'nuipogoda.ru',
 'belgorod.nuipogoda.ru': 'nuipogoda.ru',
 'severomorsk.nuipogoda.ru': 'nuipogoda.ru',
 'murmansk.nuipogoda.ru': 'nuipogoda.ru',
 'yuzhno-sahalinsk.nuipogoda.ru': 'nuipogoda.ru',
 'salavat.nuipogoda.ru': 'nuipogoda.ru',
 'lobnya.nuipogoda.ru': 'nuipogoda.ru',
 'raduzhniy.nuipogoda.ru': 'nuipogoda.ru',
 'vladivostok.nuipogoda.ru': 'nuipogoda.ru',
 'blagoveschensk.nuipogoda.ru': 'nuipogoda.ru',
 'ulyanovsk.nuipogoda.ru': 'nuipogoda.ru',
 'petropavlovsk-kamchatskiy.nuipogoda.ru': 'nuipogoda.ru',
 'mozhga.nuipogoda.ru': 'nuipogoda.ru',
 'olonec.nuipogoda.ru': 'nuipogoda.ru',
 'omsk.nuipogoda.ru': 'nuipogoda.ru',
 'dzhubga.nuipogoda.ru': 'nuipogoda.ru',
 'orenburg.nuipogoda.ru': 'nuipogoda.ru',
 'mendeleevsk.nuipogoda.ru': 'nuipogoda.ru',
 'pechora.nuipogoda.ru': 'nuipogoda.ru',
 'nefteyugansk.nuipogoda.ru': 'nuipogoda.ru',
 'kolomna.nuipogoda.ru': 'nuipogoda.ru',
 'mirniy-arhangelskaya.nuipogoda.ru': 'nuipogoda.ru',
 'kaliningrad.nuipogoda.ru': 'nuipogoda.ru',
 'surgut.nuipogoda.ru': 'nuipogoda.ru',
 'zelenograd.nuipogoda.ru': 'nuipogoda.ru',
 'volchanec.nuipogoda.ru': 'nuipogoda.ru',
 'koktebel.nuipogoda.ru': 'nuipogoda.ru',
 'taganrog.nuipogoda.ru': 'nuipogoda.ru',
 'nnov.nuipogoda.ru': 'nuipogoda.ru',
 'chudovo.nuipogoda.ru': 'nuipogoda.ru',
 'novokuybishevsk.nuipogoda.ru': 'nuipogoda.ru',
 'saratov.nuipogoda.ru': 'nuipogoda.ru',
 'vityazevo-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'electrostal.nuipogoda.ru': 'nuipogoda.ru',
 'divnomorskoe.nuipogoda.ru': 'nuipogoda.ru',
 'vikulovo.nuipogoda.ru': 'nuipogoda.ru',
 'desnogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'shebekino.nuipogoda.ru': 'nuipogoda.ru',
 'almetevsk.nuipogoda.ru': 'nuipogoda.ru',
 'tayshet.nuipogoda.ru': 'nuipogoda.ru',
 'minusinsk.nuipogoda.ru': 'nuipogoda.ru',
 'shira.nuipogoda.ru': 'nuipogoda.ru',
 'pervouralsk.nuipogoda.ru': 'nuipogoda.ru',
 'sol-ileck.nuipogoda.ru': 'nuipogoda.ru',
 'kolpashevo.nuipogoda.ru': 'nuipogoda.ru',
 'novocheboksarsk.nuipogoda.ru': 'nuipogoda.ru',
 'argazi.nuipogoda.ru': 'nuipogoda.ru',
 'miass.nuipogoda.ru': 'nuipogoda.ru',
 'glazov.nuipogoda.ru': 'nuipogoda.ru',
 'labinsk.nuipogoda.ru': 'nuipogoda.ru',
 'tavda.nuipogoda.ru': 'nuipogoda.ru',
 'ryazan.nuipogoda.ru': 'nuipogoda.ru',
 'mineralnye-vody.nuipogoda.ru': 'nuipogoda.ru',
 'krasnouralsk.nuipogoda.ru': 'nuipogoda.ru',
 'krym-rostovskaya.nuipogoda.ru': 'nuipogoda.ru',
 'pskov.nuipogoda.ru': 'nuipogoda.ru',
 'mamayka-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'pereslavl-zalesskiy.nuipogoda.ru': 'nuipogoda.ru',
 'nizhniy-tagil.nuipogoda.ru': 'nuipogoda.ru',
 'abinsk.nuipogoda.ru': 'nuipogoda.ru',
 'kislovodsk.nuipogoda.ru': 'nuipogoda.ru',
 'orel.nuipogoda.ru': 'nuipogoda.ru',
 'nizhnekamsk.nuipogoda.ru': 'nuipogoda.ru',
 'volkonka-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'chemal.nuipogoda.ru': 'nuipogoda.ru',
 'irkutsk.nuipogoda.ru': 'nuipogoda.ru',
 'zyuratkul.nuipogoda.ru': 'nuipogoda.ru',
 'hyuston.nuipogoda.ru': 'nuipogoda.ru',
 'kungur.nuipogoda.ru': 'nuipogoda.ru',
 'korolev.nuipogoda.ru': 'nuipogoda.ru',
 'voronezh.nuipogoda.ru': 'nuipogoda.ru',
 'yarovoe.nuipogoda.ru': 'nuipogoda.ru',
 'yanaul.nuipogoda.ru': 'nuipogoda.ru',
 'temryuk.nuipogoda.ru': 'nuipogoda.ru',
 'shebalino.nuipogoda.ru': 'nuipogoda.ru',
 'aya-altaiskiy.nuipogoda.ru': 'nuipogoda.ru',
 'baykit.nuipogoda.ru': 'nuipogoda.ru',
 'lipetsk.nuipogoda.ru': 'nuipogoda.ru',
 'kineshma.nuipogoda.ru': 'nuipogoda.ru',
 'berezniki.nuipogoda.ru': 'nuipogoda.ru',
 'neftekamsk.nuipogoda.ru': 'nuipogoda.ru',
 'altay-altaiskiy.nuipogoda.ru': 'nuipogoda.ru',
 'krymsk.nuipogoda.ru': 'nuipogoda.ru',
 'kurgan.nuipogoda.ru': 'nuipogoda.ru',
 'shahty.nuipogoda.ru': 'nuipogoda.ru',
 'taganay-chelyabinskaya.nuipogoda.ru': 'nuipogoda.ru',
 'gatchina.nuipogoda.ru': 'nuipogoda.ru',
 'neryungri.nuipogoda.ru': 'nuipogoda.ru',
 'chaykovskiy.nuipogoda.ru': 'nuipogoda.ru',
 'pyatigorsk.nuipogoda.ru': 'nuipogoda.ru',
 'kropotkin.nuipogoda.ru': 'nuipogoda.ru',
 'beloreck.nuipogoda.ru': 'nuipogoda.ru',
 'uhta.nuipogoda.ru': 'nuipogoda.ru',
 'yalta.nuipogoda.ru': 'nuipogoda.ru',
 'uchaly.nuipogoda.ru': 'nuipogoda.ru',
 'alatyr.nuipogoda.ru': 'nuipogoda.ru',
 'angarsk.nuipogoda.ru': 'nuipogoda.ru',
 'aban.nuipogoda.ru': 'nuipogoda.ru',
 'amga.nuipogoda.ru': 'nuipogoda.ru',
 'ayhal.nuipogoda.ru': 'nuipogoda.ru',
 'anuchino.nuipogoda.ru': 'nuipogoda.ru',
 'abatskoe.nuipogoda.ru': 'nuipogoda.ru',
 'aleysk.nuipogoda.ru': 'nuipogoda.ru',
 'aleksandrovskoe.nuipogoda.ru': 'nuipogoda.ru',
 'aleksandrov.nuipogoda.ru': 'nuipogoda.ru',
 'aldan.nuipogoda.ru': 'nuipogoda.ru',
 'arzamas.nuipogoda.ru': 'nuipogoda.ru',
 'aginskoe-krasnoyarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'aleksandrovskoe-tomskaya.nuipogoda.ru': 'nuipogoda.ru',
 'arzgir.nuipogoda.ru': 'nuipogoda.ru',
 'amazar.nuipogoda.ru': 'nuipogoda.ru',
 'apatity.nuipogoda.ru': 'nuipogoda.ru',
 'alapaevsk.nuipogoda.ru': 'nuipogoda.ru',
 'abakan.nuipogoda.ru': 'nuipogoda.ru',
 'anadyr.nuipogoda.ru': 'nuipogoda.ru',
 'akyar.nuipogoda.ru': 'nuipogoda.ru',
 'aleksandrovsk-sahalinskiy.nuipogoda.ru': 'nuipogoda.ru',
 'anna.nuipogoda.ru': 'nuipogoda.ru',
 'arhyz.nuipogoda.ru': 'nuipogoda.ru',
 'yaroslavl.nuipogoda.ru': 'nuipogoda.ru',
 'berezovo-kemerovskiy-rayon-kemerovskaya.nuipogoda.ru': 'nuipogoda.ru',
 'bryansk.nuipogoda.ru': 'nuipogoda.ru',
 'tuapse.nuipogoda.ru': 'nuipogoda.ru',
 'noviy-urengoy.nuipogoda.ru': 'nuipogoda.ru',
 'rybinsk.nuipogoda.ru': 'nuipogoda.ru',
 'makopse.nuipogoda.ru': 'nuipogoda.ru',
 'kudymkar.nuipogoda.ru': 'nuipogoda.ru',
 'nahodka.nuipogoda.ru': 'nuipogoda.ru',
 'kaluga.nuipogoda.ru': 'nuipogoda.ru',
 'kemerovo.nuipogoda.ru': 'nuipogoda.ru',
 'ust-koksa.nuipogoda.ru': 'nuipogoda.ru',
 'ramenskoe.nuipogoda.ru': 'nuipogoda.ru',
 'ussuriysk.nuipogoda.ru': 'nuipogoda.ru',
 'tazovskiy.nuipogoda.ru': 'nuipogoda.ru',
 'joshkar-ola.nuipogoda.ru': 'nuipogoda.ru',
 'barnaul.nuipogoda.ru': 'nuipogoda.ru',
 'satka.nuipogoda.ru': 'nuipogoda.ru',
 'pavlogradka.nuipogoda.ru': 'nuipogoda.ru',
 'komsomolsk-na-amure.nuipogoda.ru': 'nuipogoda.ru',
 'verhnyaya-salda.nuipogoda.ru': 'nuipogoda.ru',
 'oktyabrskiy.nuipogoda.ru': 'nuipogoda.ru',
 'kirov.nuipogoda.ru': 'nuipogoda.ru',
 'severodvinsk.nuipogoda.ru': 'nuipogoda.ru',
 'leushi.nuipogoda.ru': 'nuipogoda.ru',
 'nogliki.nuipogoda.ru': 'nuipogoda.ru',
 'verhneyarkeevo.nuipogoda.ru': 'nuipogoda.ru',
 'chernushka.nuipogoda.ru': 'nuipogoda.ru',
 'chusovoy.nuipogoda.ru': 'nuipogoda.ru',
 'dus-hol.nuipogoda.ru': 'nuipogoda.ru',
 'himki.nuipogoda.ru': 'nuipogoda.ru',
 'cherkessk.nuipogoda.ru': 'nuipogoda.ru',
 'cherlak.nuipogoda.ru': 'nuipogoda.ru',
 'michurinsk.nuipogoda.ru': 'nuipogoda.ru',
 'liski.nuipogoda.ru': 'nuipogoda.ru',
 'mostovskoy-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'onega.nuipogoda.ru': 'nuipogoda.ru',
 'nizhnyaya-hobza.nuipogoda.ru': 'nuipogoda.ru',
 'nevel.nuipogoda.ru': 'nuipogoda.ru',
 'ershov.nuipogoda.ru': 'nuipogoda.ru',
 'antalya.nuipogoda.ru': 'nuipogoda.ru',
 'ivanovo.nuipogoda.ru': 'nuipogoda.ru',
 'kostroma.nuipogoda.ru': 'nuipogoda.ru',
 'magadan.nuipogoda.ru': 'nuipogoda.ru',
 'birobidzhan.nuipogoda.ru': 'nuipogoda.ru',
 'kyzyl.nuipogoda.ru': 'nuipogoda.ru',
 'yakutsk.nuipogoda.ru': 'nuipogoda.ru',
 'vnukovo.nuipogoda.ru': 'nuipogoda.ru',
 'mirniy.nuipogoda.ru': 'nuipogoda.ru',
 'belokuriha.nuipogoda.ru': 'nuipogoda.ru',
 'sagra.nuipogoda.ru': 'nuipogoda.ru',
 'slavyanka-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
 'elabuga.nuipogoda.ru': 'nuipogoda.ru',
 'bele.nuipogoda.ru': 'nuipogoda.ru',
 'kushva.nuipogoda.ru': 'nuipogoda.ru',
 'manzherok.nuipogoda.ru': 'nuipogoda.ru',
 'belebey.nuipogoda.ru': 'nuipogoda.ru',
 'volgodonsk.nuipogoda.ru': 'nuipogoda.ru',
 'lesozavodsk.nuipogoda.ru': 'nuipogoda.ru',
 'ryazanovka-hasanskiy-rayon-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
 'kirovskiy.nuipogoda.ru': 'nuipogoda.ru',
 'belorechensk.nuipogoda.ru': 'nuipogoda.ru',
 'ostashkov.nuipogoda.ru': 'nuipogoda.ru',
 'lytkarino.nuipogoda.ru': 'nuipogoda.ru',
 'ishimbay.nuipogoda.ru': 'nuipogoda.ru',
 'kamensk-uralskiy.nuipogoda.ru': 'nuipogoda.ru',
 'nevinnomyssk.nuipogoda.ru': 'nuipogoda.ru',
 'poykovskiy.nuipogoda.ru': 'nuipogoda.ru',
 'serpuhov.nuipogoda.ru': 'nuipogoda.ru',
 'armavir.nuipogoda.ru': 'nuipogoda.ru',
 'segezha.nuipogoda.ru': 'nuipogoda.ru',
 'nugush.nuipogoda.ru': 'nuipogoda.ru',
 'meleuz.nuipogoda.ru': 'nuipogoda.ru',
 'tihoreck.nuipogoda.ru': 'nuipogoda.ru',
 'georgievsk.nuipogoda.ru': 'nuipogoda.ru',
 'noginsk.nuipogoda.ru': 'nuipogoda.ru',
 'primorsko-ahtarsk.nuipogoda.ru': 'nuipogoda.ru',
 'civilsk.nuipogoda.ru': 'nuipogoda.ru',
 'podporozhe.nuipogoda.ru': 'nuipogoda.ru',
 'ryazhsk.nuipogoda.ru': 'nuipogoda.ru',
 'kurchatov.nuipogoda.ru': 'nuipogoda.ru',
 'velikie-luki.nuipogoda.ru': 'nuipogoda.ru',
 'yuzhno-morskoy.nuipogoda.ru': 'nuipogoda.ru',
 'severo-eniseyskiy.nuipogoda.ru': 'nuipogoda.ru',
 'tatarstan-agryzskiy-rayon-tatarstan.nuipogoda.ru': 'nuipogoda.ru',
 'svobodniy.nuipogoda.ru': 'nuipogoda.ru',
 'kudrovo-leningradskaya.nuipogoda.ru': 'nuipogoda.ru',
 'kurganskaya.nuipogoda.ru': 'nuipogoda.ru',
 'kuchugury-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'sukko.nuipogoda.ru': 'nuipogoda.ru',
 'sheksna-sheksninskiy-rayon-vologodskaya.nuipogoda.ru': 'nuipogoda.ru',
 'asha.nuipogoda.ru': 'nuipogoda.ru',
 'morshansk.nuipogoda.ru': 'nuipogoda.ru',
 'brodokalmak.nuipogoda.ru': 'nuipogoda.ru',
 'amursk.nuipogoda.ru': 'nuipogoda.ru',
 'eysk.nuipogoda.ru': 'nuipogoda.ru',
 'stariy-oskol.nuipogoda.ru': 'nuipogoda.ru',
 'lermontov.nuipogoda.ru': 'nuipogoda.ru',
 'askino-askinskiy-rayon-bashkortostan.nuipogoda.ru': 'nuipogoda.ru',
 'murmanskaya.nuipogoda.ru': 'nuipogoda.ru',
 'uvildy.nuipogoda.ru': 'nuipogoda.ru',
 'mariinsk.nuipogoda.ru': 'nuipogoda.ru',
 'nikolaevka-simferopolskiy.nuipogoda.ru': 'nuipogoda.ru',
 'kukmor.nuipogoda.ru': 'nuipogoda.ru',
 'kinel.nuipogoda.ru': 'nuipogoda.ru',
 'bronnicy.nuipogoda.ru': 'nuipogoda.ru',
 'volchiha.nuipogoda.ru': 'nuipogoda.ru',
 'kurganinsk.nuipogoda.ru': 'nuipogoda.ru',
 'priobe.nuipogoda.ru': 'nuipogoda.ru',
 'belogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'mayak-balyuzek.nuipogoda.ru': 'nuipogoda.ru',
 'yuryuzan.nuipogoda.ru': 'nuipogoda.ru',
 'kashira.nuipogoda.ru': 'nuipogoda.ru',
 'vishnevogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'birsk.nuipogoda.ru': 'nuipogoda.ru',
 'topki.nuipogoda.ru': 'nuipogoda.ru',
 'gulkevichi.nuipogoda.ru': 'nuipogoda.ru',
 'podolsk.nuipogoda.ru': 'nuipogoda.ru',
 'vdnh.nuipogoda.ru': 'nuipogoda.ru',
 'sahalinskaya.nuipogoda.ru': 'nuipogoda.ru',
 'pangody.nuipogoda.ru': 'nuipogoda.ru',
 'lazarevskoe-krasnodarskiy.nuipogoda.ru': 'nuipogoda.ru',
 'gubaha.nuipogoda.ru': 'nuipogoda.ru',
 'harabali.nuipogoda.ru': 'nuipogoda.ru',
 'bashkortostan-bashkortostan.nuipogoda.ru': 'nuipogoda.ru',
 'sukpay.nuipogoda.ru': 'nuipogoda.ru',
 'maykop.nuipogoda.ru': 'nuipogoda.ru',
 'magdagachi.nuipogoda.ru': 'nuipogoda.ru',
 'novofedorovka-sakskiy.nuipogoda.ru': 'nuipogoda.ru',
 'kiev.nuipogoda.ru': 'nuipogoda.ru',
 'timashevsk.nuipogoda.ru': 'nuipogoda.ru',
 'novocherkassk.nuipogoda.ru': 'nuipogoda.ru',
 'vorkuta.nuipogoda.ru': 'nuipogoda.ru',
 'arhipo-osipovka.nuipogoda.ru': 'nuipogoda.ru',
 'solnechnogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'shadrinsk.nuipogoda.ru': 'nuipogoda.ru',
 'andreevka-yakovlevskiy-rayon-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
 'strezhevoy.nuipogoda.ru': 'nuipogoda.ru',
 'sudak.nuipogoda.ru': 'nuipogoda.ru',
 'aznakaevo.nuipogoda.ru': 'nuipogoda.ru',
 'biysk.nuipogoda.ru': 'nuipogoda.ru',
 'bolshoy-kamen-primorskiy.nuipogoda.ru': 'nuipogoda.ru',
    'krasnodar.blizko.ru': 'blizko.ru',
 'armavir.blizko.ru': 'blizko.ru',
 'saratov.blizko.ru': 'blizko.ru',
 'tyumen.blizko.ru': 'blizko.ru',
 'minvody.blizko.ru': 'blizko.ru',
 'vladivostok.blizko.ru': 'blizko.ru',
 'tula.blizko.ru': 'blizko.ru',
 'volgograd.blizko.ru': 'blizko.ru',
 'kurgan.blizko.ru': 'blizko.ru',
 'samara.blizko.ru': 'blizko.ru',
 'kirov.blizko.ru': 'blizko.ru',
 'blizko.ru': 'blizko.ru',
 'kostroma.blizko.ru': 'blizko.ru',
 'spb.blizko.ru': 'blizko.ru',
 'msk.blizko.ru': 'blizko.ru',
 'rostov.blizko.ru': 'blizko.ru',
 'orel.blizko.ru': 'blizko.ru',
 'ufa.blizko.ru': 'blizko.ru',
 'pervouralsk.blizko.ru': 'blizko.ru',
 'ulyanovsk.blizko.ru': 'blizko.ru',
 'ga.blizko.ru': 'blizko.ru',
 'orsk.blizko.ru': 'blizko.ru',
 'sochi.blizko.ru': 'blizko.ru',
 'nsk.blizko.ru': 'blizko.ru',
 'mezhdurechensk.blizko.ru': 'blizko.ru',
 'yakutsk.blizko.ru': 'blizko.ru',
 'irk.blizko.ru': 'blizko.ru',
 'yaroslavl.blizko.ru': 'blizko.ru',
 'nn.blizko.ru': 'blizko.ru',
 'tolyatty.blizko.ru': 'blizko.ru',
 'chel.blizko.ru': 'blizko.ru',
 'surgut.blizko.ru': 'blizko.ru',
 'dimaster55.blizko.ru': 'blizko.ru',
 'krasnoyarsk.blizko.ru': 'blizko.ru',
 'komsomolsk.blizko.ru': 'blizko.ru',
 'cheboksary.blizko.ru': 'blizko.ru',
 'kaliningrad.blizko.ru': 'blizko.ru',
 'labinsk.blizko.ru': 'blizko.ru',
 'farkop55.blizko.ru': 'blizko.ru',
 'novgorod.blizko.ru': 'blizko.ru',
 'abakan.blizko.ru': 'blizko.ru',
 'blagovechensk.blizko.ru': 'blizko.ru',
 'maykop.blizko.ru': 'blizko.ru',
 'kazan.blizko.ru': 'blizko.ru',
 'tagil.blizko.ru': 'blizko.ru',
 'almetievsk.blizko.ru': 'blizko.ru',
 'perm.blizko.ru': 'blizko.ru',
 'ozon-dacha.blizko.ru': 'blizko.ru',
 'gatchina.blizko.ru': 'blizko.ru',
 'chelny.blizko.ru': 'blizko.ru',
 'novorossiysk.blizko.ru': 'blizko.ru',
 'pyatigorsk.blizko.ru': 'blizko.ru',
 'kolomna.blizko.ru': 'blizko.ru',
 'sposad.blizko.ru': 'blizko.ru',
 'shahty.blizko.ru': 'blizko.ru',
 'omsk.blizko.ru': 'blizko.ru',
 'tomsk.blizko.ru': 'blizko.ru',
 'obninsk.blizko.ru': 'blizko.ru',
 'voskresensk.blizko.ru': 'blizko.ru',
 'bugulma.blizko.ru': 'blizko.ru',
 '14347411.blizko.ru': 'blizko.ru',
 'tuapse.blizko.ru': 'blizko.ru',
 'noginsk.blizko.ru': 'blizko.ru',
 'general-familyru.blizko.ru': 'blizko.ru',
 'izhevsk.blizko.ru': 'blizko.ru',
 'pavlovo.blizko.ru': 'blizko.ru',
 'belgorod.blizko.ru': 'blizko.ru',
 'slavyansk-na-kubani.blizko.ru': 'blizko.ru',
 'madorskiy.blizko.ru': 'blizko.ru',
 'taxi-rnd.blizko.ru': 'blizko.ru',
 'biysk.blizko.ru': 'blizko.ru',
 'novoshahtinsk.blizko.ru': 'blizko.ru',
 'kropotkin.blizko.ru': 'blizko.ru',
 'pavlovsky-posad.blizko.ru': 'blizko.ru',
 'ramenskoe.blizko.ru': 'blizko.ru',
 'tuymazi.blizko.ru': 'blizko.ru',
 'kaluga.blizko.ru': 'blizko.ru',
 'eysk.blizko.ru': 'blizko.ru',
 'artyom.blizko.ru': 'blizko.ru',
 'mytischi.blizko.ru': 'blizko.ru',
 'sakhalinsk.blizko.ru': 'blizko.ru',
 'iskitim.blizko.ru': 'blizko.ru',
 'essentuki.blizko.ru': 'blizko.ru',
 'ivanovo.blizko.ru': 'blizko.ru',
 'kislovodsk.blizko.ru': 'blizko.ru',
 'ryazan.blizko.ru': 'blizko.ru',
 'volgodonsk.blizko.ru': 'blizko.ru',
 'himki.blizko.ru': 'blizko.ru',
 'noyabrsk.blizko.ru': 'blizko.ru',
 'khabarovsk.blizko.ru': 'blizko.ru',
 'sterlitamak.blizko.ru': 'blizko.ru',
 'staroskol.blizko.ru': 'blizko.ru',
 'barnaul.blizko.ru': 'blizko.ru',
 'tihoreck.blizko.ru': 'blizko.ru',
 'neftekamsk.blizko.ru': 'blizko.ru',
 'orenburg.blizko.ru': 'blizko.ru',
 'stavropol.blizko.ru': 'blizko.ru',
 'anapa.blizko.ru': 'blizko.ru',
 'nizhnevartovsk.blizko.ru': 'blizko.ru',
 'cherepovets.blizko.ru': 'blizko.ru',
 'nahodka.blizko.ru': 'blizko.ru',
 'novokuybyshevsk.blizko.ru': 'blizko.ru',
 'stupino.blizko.ru': 'blizko.ru',
 'korolev.blizko.ru': 'blizko.ru',
 'uhta.blizko.ru': 'blizko.ru',
 'ekb.blizko.ru': 'blizko.ru',
 'magnitogorsk.blizko.ru': 'blizko.ru',
 'oboi-dekor.blizko.ru': 'blizko.ru',
 'kyzyl.blizko.ru': 'blizko.ru',
 'astrahan.blizko.ru': 'blizko.ru',
 'dzerzhinsk.blizko.ru': 'blizko.ru',
 'kumertau.blizko.ru': 'blizko.ru',
 'voronezh.blizko.ru': 'blizko.ru',
 'pyshma.blizko.ru': 'blizko.ru',
 'belorechensk.blizko.ru': 'blizko.ru',
 'kamensk.blizko.ru': 'blizko.ru',
 'kovrov.blizko.ru': 'blizko.ru',
 'salsk.blizko.ru': 'blizko.ru',
 'nk.blizko.ru': 'blizko.ru',
 'podolsk.blizko.ru': 'blizko.ru',
 'krymsk.blizko.ru': 'blizko.ru',
 'salavat.blizko.ru': 'blizko.ru',
 'lesnoe-ozero.blizko.ru': 'blizko.ru',
 'syzran.blizko.ru': 'blizko.ru',
 'timashyovsk.blizko.ru': 'blizko.ru',
 'kristall-plast.blizko.ru': 'blizko.ru',
 'solnechnogorsk.blizko.ru': 'blizko.ru',
 'novokuznetsk.blizko.ru': 'blizko.ru',
 'ehlektrostal.blizko.ru': 'blizko.ru',
 'serpuhov.blizko.ru': 'blizko.ru',
 'domino08.blizko.ru': 'blizko.ru',
 'pushkino.blizko.ru': 'blizko.ru',
 'zdoroveplus.blizko.ru': 'blizko.ru',
 'naro-fominsk.blizko.ru': 'blizko.ru',
 'dolgoprudn.blizko.ru': 'blizko.ru',
 'ozon-mebel.blizko.ru': 'blizko.ru',
 'tambov.blizko.ru': 'blizko.ru',
 'novomoskovsk.blizko.ru': 'blizko.ru',
 'taganrog.blizko.ru': 'blizko.ru',
 'skt.blizko.ru': 'blizko.ru',
 'klin.blizko.ru': 'blizko.ru',
 'ussuriysk.blizko.ru': 'blizko.ru',
 'berdsk.blizko.ru': 'blizko.ru',
 'eldorado-tambov.blizko.ru': 'blizko.ru',
 'vorkuta.blizko.ru': 'blizko.ru',
 'gos-auto.blizko.ru': 'blizko.ru',
 'prokopievsk.blizko.ru': 'blizko.ru',
 'oktyabrsky.blizko.ru': 'blizko.ru',
 'gipermarketdom.blizko.ru': 'blizko.ru',
 'berezniki.blizko.ru': 'blizko.ru',
 'orbitasport.blizko.ru': 'blizko.ru',
 'gelendzhik.blizko.ru': 'blizko.ru',
 'penza.blizko.ru': 'blizko.ru',
 'nefteyugansk.blizko.ru': 'blizko.ru',
 'vladikavkaz.blizko.ru': 'blizko.ru',
 'rzd-medicine.blizko.ru': 'blizko.ru',
 'elabuga.blizko.ru': 'blizko.ru',
 'ola.blizko.ru': 'blizko.ru',
 'svobodn.blizko.ru': 'blizko.ru',
 'kansk.blizko.ru': 'blizko.ru',
 'vladimir.blizko.ru': 'blizko.ru',
 'chapaevsk.blizko.ru': 'blizko.ru',
 'engels.blizko.ru': 'blizko.ru',
 'sibay.blizko.ru': 'blizko.ru',
 'shop-end-shou-shop-show.blizko.ru': 'blizko.ru',
 'ozon-auto.blizko.ru': 'blizko.ru',
 'ishim.blizko.ru': 'blizko.ru',
 'serov.blizko.ru': 'blizko.ru',
 'severouralsk.blizko.ru': 'blizko.ru',
 'vita-samara.blizko.ru': 'blizko.ru',
 'tver.blizko.ru': 'blizko.ru',
 'shchyolkovo.blizko.ru': 'blizko.ru',
 'lipetskiy-pereezd.blizko.ru': 'blizko.ru',
 'kemerovo.blizko.ru': 'blizko.ru',
 'vologda.blizko.ru': 'blizko.ru',
 'pskov.blizko.ru': 'blizko.ru',
 'prohladny.blizko.ru': 'blizko.ru',
 'tobolsk.blizko.ru': 'blizko.ru',
 'neryungri.blizko.ru': 'blizko.ru',
 'zelenodolsk.blizko.ru': 'blizko.ru',
 'chistopol.blizko.ru': 'blizko.ru',
 'balashiha.blizko.ru': 'blizko.ru',
 'ivanteevka.blizko.ru': 'blizko.ru',
 'buzuluk.blizko.ru': 'blizko.ru',
 'odincovo.blizko.ru': 'blizko.ru',
 'korzinka.blizko.ru': 'blizko.ru',
 'lesosibirsk.blizko.ru': 'blizko.ru',
 'velikie-luki.blizko.ru': 'blizko.ru',
 'eldorado-kazan.blizko.ru': 'blizko.ru',
 'grozny.blizko.ru': 'blizko.ru',
 '14347415.blizko.ru': 'blizko.ru',
 'zhbk-1.blizko.ru': 'blizko.ru',
 '220-volt-spb.blizko.ru': 'blizko.ru',
 'klimovsk.blizko.ru': 'blizko.ru',
 'sayanogorsk.blizko.ru': 'blizko.ru',
 'borovichi.blizko.ru': 'blizko.ru',
 'severodvinsk.blizko.ru': 'blizko.ru',
 'linoleum-goroda.blizko.ru': 'blizko.ru',
 'nevinnomyssk.blizko.ru': 'blizko.ru',
 'anzhero-sudzhensk.blizko.ru': 'blizko.ru',
 'chekhov.blizko.ru': 'blizko.ru',
 'grace-group.blizko.ru': 'blizko.ru',
 'alfa08.blizko.ru': 'blizko.ru',
 'eldorado-rostov.blizko.ru': 'blizko.ru',
 'ulanude.blizko.ru': 'blizko.ru',
 'bryansk.blizko.ru': 'blizko.ru',
 'orehovo-zuevo.blizko.ru': 'blizko.ru',
 'komfortbtru.blizko.ru': 'blizko.ru',
 'asbest.blizko.ru': 'blizko.ru',
 'art-travel.blizko.ru': 'blizko.ru',
 'stilnyy-pes.blizko.ru': 'blizko.ru',
 'ozon-8.blizko.ru': 'blizko.ru',
 'ozon-3.blizko.ru': 'blizko.ru',
 'sosnovy-bor.blizko.ru': 'blizko.ru',
 'krasnokamsk.blizko.ru': 'blizko.ru',
 'petrozavodsk.blizko.ru': 'blizko.ru',
 'kirishi.blizko.ru': 'blizko.ru',
 'magadan.blizko.ru': 'blizko.ru',
 'azov.blizko.ru': 'blizko.ru',
 'petropavlovsk.blizko.ru': 'blizko.ru',
 'kursk.blizko.ru': 'blizko.ru',
 'smolensk.blizko.ru': 'blizko.ru',
 'imperiaokon63.blizko.ru': 'blizko.ru',
 'evpatoriya.blizko.ru': 'blizko.ru',
 'tihvin.blizko.ru': 'blizko.ru',
 'servis-remontov.blizko.ru': 'blizko.ru',
 'dealmed-4.blizko.ru': 'blizko.ru',
 'sklad-magazin-darombyt.blizko.ru': 'blizko.ru',
 'balashov.blizko.ru': 'blizko.ru',
 'rossosh.blizko.ru': 'blizko.ru',
 'adonis-31.blizko.ru': 'blizko.ru',
 'aleksandrov.blizko.ru': 'blizko.ru',
 'bor.blizko.ru': 'blizko.ru',
 'sedmoenebo.blizko.ru': 'blizko.ru',
 'miass.blizko.ru': 'blizko.ru',
 'kamensk-shaht.blizko.ru': 'blizko.ru',
 'prava.blizko.ru': 'blizko.ru',
 'ishimbay.blizko.ru': 'blizko.ru',
 'roslavl.blizko.ru': 'blizko.ru',
 'livny.blizko.ru': 'blizko.ru',
 'bratsk.blizko.ru': 'blizko.ru',
 'simferopol.blizko.ru': 'blizko.ru',
 'ukskamges.blizko.ru': 'blizko.ru',
 'novocherkassk.blizko.ru': 'blizko.ru',
 'cherkessk.blizko.ru': 'blizko.ru',
 'urmk.blizko.ru': 'blizko.ru',
 'finplaza.blizko.ru': 'blizko.ru',
 'vyshniy-volochek.blizko.ru': 'blizko.ru',
 'murmansk.blizko.ru': 'blizko.ru',
 'michurinsk.blizko.ru': 'blizko.ru',
 'vidnoe.blizko.ru': 'blizko.ru',
 'yurga.blizko.ru': 'blizko.ru',
 'mahachkala.blizko.ru': 'blizko.ru',
 'karamelnye-nozhki.blizko.ru': 'blizko.ru',
 'volzhsk.blizko.ru': 'blizko.ru',
 'dmitrov.blizko.ru': 'blizko.ru',
 'bataysk.blizko.ru': 'blizko.ru',
 'kopeysk.blizko.ru': 'blizko.ru',
 'saransk.blizko.ru': 'blizko.ru',
 'tisbi.blizko.ru': 'blizko.ru',
 'lipetsk.blizko.ru': 'blizko.ru',
 'votkinsk.blizko.ru': 'blizko.ru',
 'leroy-spb.blizko.ru': 'blizko.ru',
 'vformate.blizko.ru': 'blizko.ru',
 'shadrinsk.blizko.ru': 'blizko.ru',
 'leninogorsk.blizko.ru': 'blizko.ru',
 'nalchik.blizko.ru': 'blizko.ru',
 'belebey.blizko.ru': 'blizko.ru',
 'biomed.blizko.ru': 'blizko.ru',
 'revda.blizko.ru': 'blizko.ru',
 'elista.blizko.ru': 'blizko.ru',
 'akulshin-zerno-korma-kombikorm.blizko.ru': 'blizko.ru',
 'tvoedetstvo.blizko.ru': 'blizko.ru',
 'tsar-stol.blizko.ru': 'blizko.ru',
 'egorevsk.blizko.ru': 'blizko.ru',
 'belovo.blizko.ru': 'blizko.ru',
 'tehnosklad-krasnodar.blizko.ru': 'blizko.ru',
 'plastikakazan.blizko.ru': 'blizko.ru',
 'lider-stroyy.blizko.ru': 'blizko.ru',
 'domovenok-yasha.blizko.ru': 'blizko.ru',
 'magazin-evrokomfort.blizko.ru': 'blizko.ru',
 'lyubercy.blizko.ru': 'blizko.ru',
 'leroymerlin-samara.blizko.ru': 'blizko.ru',
 'chaykovsky.blizko.ru': 'blizko.ru',
 'humanities-academy.blizko.ru': 'blizko.ru',
 'rzhev.blizko.ru': 'blizko.ru',
 'eldorado-spb.blizko.ru': 'blizko.ru',
 'td-imperia.blizko.ru': 'blizko.ru',
 'novy-urengoy.blizko.ru': 'blizko.ru',
 'hantymansiysk.blizko.ru': 'blizko.ru',
 'sady-urala.blizko.ru': 'blizko.ru',
 'norilsk.blizko.ru': 'blizko.ru',
 'agroholding.blizko.ru': 'blizko.ru',
 'leroy-tyumen.blizko.ru': 'blizko.ru',
 'arh.blizko.ru': 'blizko.ru',
 'lysva.blizko.ru': 'blizko.ru',
 'baucenter-krasnodar.blizko.ru': 'blizko.ru',
 '220-voltru-rostov-na-donu.blizko.ru': 'blizko.ru',
 'ozon-13.blizko.ru': 'blizko.ru',
 'madonna.blizko.ru': 'blizko.ru',
 'zoovetsnab.blizko.ru': 'blizko.ru',
 'almaz-anapa.blizko.ru': 'blizko.ru',
 'kungur.blizko.ru': 'blizko.ru',
 'korvet-rybinsk.blizko.ru': 'blizko.ru',
 '4lapy.blizko.ru': 'blizko.ru',
 'volsk.blizko.ru': 'blizko.ru',
 'prodamstul.blizko.ru': 'blizko.ru',
 'yuridicheskaya-k-l.blizko.ru': 'blizko.ru',
 'rybinsk.blizko.ru': 'blizko.ru',
 'dubna.blizko.ru': 'blizko.ru',
 'gubkin.blizko.ru': 'blizko.ru',
 'sbt-8.blizko.ru': 'blizko.ru',
 'borisoglebsk.blizko.ru': 'blizko.ru',
 'seversk.blizko.ru': 'blizko.ru',
 'minusinsk.blizko.ru': 'blizko.ru',
 'ty-i-ya-4.blizko.ru': 'blizko.ru',
 'galamart-perm.blizko.ru': 'blizko.ru',
 'srs.blizko.ru': 'blizko.ru',
 'zhigulyovsk.blizko.ru': 'blizko.ru',
 'liski.blizko.ru': 'blizko.ru',
 'solikamsk.blizko.ru': 'blizko.ru',
 'kuzmich24.blizko.ru': 'blizko.ru',
 'tts-megadom.blizko.ru': 'blizko.ru',
 'vyazma.blizko.ru': 'blizko.ru',
 'fryazino.blizko.ru': 'blizko.ru',
 'posudaok.blizko.ru': 'blizko.ru',
 'krasnoturinsk.blizko.ru': 'blizko.ru',
 'nazarovo.blizko.ru': 'blizko.ru',
 'leroy-rostov.blizko.ru': 'blizko.ru',
 'lobnya.blizko.ru': 'blizko.ru',
 'pushkin.blizko.ru': 'blizko.ru',
 'kirovo-chepeck.blizko.ru': 'blizko.ru',
 'eldorado-cheboksary.blizko.ru': 'blizko.ru',
 'glazov.blizko.ru': 'blizko.ru',
 'chernogorsk.blizko.ru': 'blizko.ru',
 'vsevolozhsk.blizko.ru': 'blizko.ru',
 'eao.blizko.ru': 'blizko.ru',
 'meleuz.blizko.ru': 'blizko.ru',
 'start.blizko.ru': 'blizko.ru',
 'eldorado-norilsk.blizko.ru': 'blizko.ru',
 'tehnosklad-stavropol.blizko.ru': 'blizko.ru',
 'glamour-rybinsk.blizko.ru': 'blizko.ru',
 'etalonsalon.blizko.ru': 'blizko.ru',
 'fakel.blizko.ru': 'blizko.ru',
 'udacha-rybaka-1.blizko.ru': 'blizko.ru',
 'novocheboksarsk.blizko.ru': 'blizko.ru',
 'kipo.blizko.ru': 'blizko.ru',
 'nevyansk.blizko.ru': 'blizko.ru',
 'vyksa.blizko.ru': 'blizko.ru',
 'gukovo.blizko.ru': 'blizko.ru',
 'elec.blizko.ru': 'blizko.ru',
 'shuya.blizko.ru': 'blizko.ru',
 'volnyy-veter-2.blizko.ru': 'blizko.ru',
 'nyagan.blizko.ru': 'blizko.ru',
 'georgievsk.blizko.ru': 'blizko.ru',
 'chita.blizko.ru': 'blizko.ru',
 'strana-mebeli.blizko.ru': 'blizko.ru',
 'baza-otdyha-podkova.blizko.ru': 'blizko.ru',
 'eldorado-habarovsk.blizko.ru': 'blizko.ru',
 'chitai-gorod.blizko.ru': 'blizko.ru',
 'sarapul.blizko.ru': 'blizko.ru',
 'megion.blizko.ru': 'blizko.ru',
 'pnevmach.blizko.ru': 'blizko.ru',
 'proday-avto24.blizko.ru': 'blizko.ru',
 'kleopatra.blizko.ru': 'blizko.ru',
 'budyonnovsk.blizko.ru': 'blizko.ru',
 'megaflowers-1.blizko.ru': 'blizko.ru',
 'aleksin.blizko.ru': 'blizko.ru',
 'laboratoriya-medis.blizko.ru': 'blizko.ru',
 'ust-ilimsk.blizko.ru': 'blizko.ru',
 'm-cosmetica.blizko.ru': 'blizko.ru',
 'kolpino.blizko.ru': 'blizko.ru',
 'zhukovsky.blizko.ru': 'blizko.ru',
 'uzlovaya.blizko.ru': 'blizko.ru',
 'mc-aybolit.blizko.ru': 'blizko.ru',
 'podatvsudrf.blizko.ru': 'blizko.ru',
 'eldorado-kaluga.blizko.ru': 'blizko.ru',
 'ugmk-zdorovie.blizko.ru': 'blizko.ru',
 'stroydetal-chelny.blizko.ru': 'blizko.ru',
 '14344991.blizko.ru': 'blizko.ru',
 'etm.blizko.ru': 'blizko.ru',
 'pilim-les.blizko.ru': 'blizko.ru',
 'kvpanorama.blizko.ru': 'blizko.ru',
 'klinika-1.blizko.ru': 'blizko.ru',
 'stophimiy.blizko.ru': 'blizko.ru',
 'krasnogorsk.blizko.ru': 'blizko.ru',
 'bezlimit--premialnyy-sotovyy-operator.blizko.ru': 'blizko.ru',
 'rubcovsk.blizko.ru': 'blizko.ru',
 'kolesa-deko.blizko.ru': 'blizko.ru',
 'buguruslan.blizko.ru': 'blizko.ru',
 'kiselyovsk.blizko.ru': 'blizko.ru',
 'domodedovo.blizko.ru': 'blizko.ru',
 'azhur-mebel.blizko.ru': 'blizko.ru',
 'feodosiya.blizko.ru': 'blizko.ru',
 'stomatologicheskiy-tsentr-n1.blizko.ru': 'blizko.ru',
 'alfatour-ufa.blizko.ru': 'blizko.ru',
 'umkavet.blizko.ru': 'blizko.ru',
 'g-mc.blizko.ru': 'blizko.ru',
 'intelis-torgovoe-oborudovanie.blizko.ru': 'blizko.ru',
 'artlayf.blizko.ru': 'blizko.ru',
 'kamyshin.blizko.ru': 'blizko.ru',
 'pervaya-meditsinskaya-klinika.blizko.ru': 'blizko.ru',
 'komnatnye-tsvety-na-uralmashe.blizko.ru': 'blizko.ru',
 'mebelnaya-baza.blizko.ru': 'blizko.ru',
 'logos23.blizko.ru': 'blizko.ru',
 'volzhsky.blizko.ru': 'blizko.ru',
 'dveri-barnaul.blizko.ru': 'blizko.ru',
 'rssm.blizko.ru': 'blizko.ru',
 'lukomorie-park.blizko.ru': 'blizko.ru',
 'zlatoust.blizko.ru': 'blizko.ru',
 'georazved.blizko.ru': 'blizko.ru',
 'stroytehnologii-treyd.blizko.ru': 'blizko.ru',
 'spb-holodilnik.blizko.ru': 'blizko.ru',
 'cfm.blizko.ru': 'blizko.ru',
 '4water.blizko.ru': 'blizko.ru',
 'posudamart.blizko.ru': 'blizko.ru',
 'derbent.blizko.ru': 'blizko.ru',
 'kolesa-darom2.blizko.ru': 'blizko.ru',
 'komissioner.blizko.ru': 'blizko.ru',
 'rto-russia.blizko.ru': 'blizko.ru',
 'balakovo.blizko.ru': 'blizko.ru',
 'eldorado-krasnodar.blizko.ru': 'blizko.ru',
 'kirovskoe.blizko.ru': 'blizko.ru',
 'reutov.blizko.ru': 'blizko.ru',
 'instrumentru.blizko.ru': 'blizko.ru',
 'eldorado-ufa.blizko.ru': 'blizko.ru',
 'eldorado-surgut.blizko.ru': 'blizko.ru',
 'leninsk-kuz.blizko.ru': 'blizko.ru',
 'kogalym.blizko.ru': 'blizko.ru',
 'kirov-les16.blizko.ru': 'blizko.ru',
 'kuzneck.blizko.ru': 'blizko.ru',
 'novouralsk.blizko.ru': 'blizko.ru',
 'mmttru.blizko.ru': 'blizko.ru',
 'eldorado-yuzhno-sahalinsk.blizko.ru': 'blizko.ru',
 'severomorsk.blizko.ru': 'blizko.ru',
 'm2-anapa.blizko.ru': 'blizko.ru',
 'shchyokino.blizko.ru': 'blizko.ru',
 'vyborg.blizko.ru': 'blizko.ru',
 'hotstroi.blizko.ru': 'blizko.ru',
 'kineshma.blizko.ru': 'blizko.ru',
    'perm.jobfilter.ru': 'jobfilter.ru',
 'jobfilter.ru': 'jobfilter.ru',
 'velikii-novgorod.jobfilter.ru': 'jobfilter.ru',
 'naro-fominsk.jobfilter.ru': 'jobfilter.ru',
 'moskva.jobfilter.ru': 'jobfilter.ru',
 'tolyatti.jobfilter.ru': 'jobfilter.ru',
 'ekaterinburg.jobfilter.ru': 'jobfilter.ru',
 'sergiev-posad.jobfilter.ru': 'jobfilter.ru',
 'orehovo-zuevo.jobfilter.ru': 'jobfilter.ru',
 'ufa.jobfilter.ru': 'jobfilter.ru',
 'kolomna.jobfilter.ru': 'jobfilter.ru',
 'kazan.jobfilter.ru': 'jobfilter.ru',
 'nijnevartovsk.jobfilter.ru': 'jobfilter.ru',
 'novosibirsk.jobfilter.ru': 'jobfilter.ru',
 'kostroma.jobfilter.ru': 'jobfilter.ru',
 'iskitim.jobfilter.ru': 'jobfilter.ru',
 'krasnodar.jobfilter.ru': 'jobfilter.ru',
 'blagoveschensk.jobfilter.ru': 'jobfilter.ru',
 'sankt-peterburg.jobfilter.ru': 'jobfilter.ru',
 'sochi.jobfilter.ru': 'jobfilter.ru',
 'kaluga.jobfilter.ru': 'jobfilter.ru',
 'yaroslavl.jobfilter.ru': 'jobfilter.ru',
 'staryi-oskol.jobfilter.ru': 'jobfilter.ru',
 'aldan.jobfilter.ru': 'jobfilter.ru',
 'tommot.jobfilter.ru': 'jobfilter.ru',
 'kirov.jobfilter.ru': 'jobfilter.ru',
 'murmansk.jobfilter.ru': 'jobfilter.ru',
 'petrozavodsk.jobfilter.ru': 'jobfilter.ru',
 'kirovo-chepeck.jobfilter.ru': 'jobfilter.ru',
 'glazov.jobfilter.ru': 'jobfilter.ru',
 'balashiha.jobfilter.ru': 'jobfilter.ru',
 'jeleznodorojnyi-mo.jobfilter.ru': 'jobfilter.ru',
 'krasnodarskii-krai.jobfilter.ru': 'jobfilter.ru',
 'tumen.jobfilter.ru': 'jobfilter.ru',
 'krasnoyarsk.jobfilter.ru': 'jobfilter.ru',
 'saratov.jobfilter.ru': 'jobfilter.ru',
 'elektrostal.jobfilter.ru': 'jobfilter.ru',
 'moskovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'solnecnogorsk.jobfilter.ru': 'jobfilter.ru',
 'habarovsk.jobfilter.ru': 'jobfilter.ru',
 'amurskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'saha-yakutiya.jobfilter.ru': 'jobfilter.ru',
 'pyt-yah.jobfilter.ru': 'jobfilter.ru',
 'kurgan.jobfilter.ru': 'jobfilter.ru',
 'surgut.jobfilter.ru': 'jobfilter.ru',
 'schelkovo.jobfilter.ru': 'jobfilter.ru',
 'himki.jobfilter.ru': 'jobfilter.ru',
 'kemerovo.jobfilter.ru': 'jobfilter.ru',
 'vladimir.jobfilter.ru': 'jobfilter.ru',
 'vologda.jobfilter.ru': 'jobfilter.ru',
 'orenburg.jobfilter.ru': 'jobfilter.ru',
 'mihailovsk.jobfilter.ru': 'jobfilter.ru',
 'moskva-region.jobfilter.ru': 'jobfilter.ru',
 'vsevolojsk.jobfilter.ru': 'jobfilter.ru',
 'kaliningrad.jobfilter.ru': 'jobfilter.ru',
 'yujno-sahalinsk.jobfilter.ru': 'jobfilter.ru',
 'gagarin.jobfilter.ru': 'jobfilter.ru',
 'dmitrov.jobfilter.ru': 'jobfilter.ru',
 'nijnii-tagil.jobfilter.ru': 'jobfilter.ru',
 'tobolsk.jobfilter.ru': 'jobfilter.ru',
 'ramenskoe.jobfilter.ru': 'jobfilter.ru',
 'elabuga.jobfilter.ru': 'jobfilter.ru',
 'sahalinskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'nahodka.jobfilter.ru': 'jobfilter.ru',
 'lipeck.jobfilter.ru': 'jobfilter.ru',
 'taiga.jobfilter.ru': 'jobfilter.ru',
 'omsk.jobfilter.ru': 'jobfilter.ru',
 'chelyabinsk.jobfilter.ru': 'jobfilter.ru',
 'samara.jobfilter.ru': 'jobfilter.ru',
 'leninogorsk.jobfilter.ru': 'jobfilter.ru',
 'voskresensk.jobfilter.ru': 'jobfilter.ru',
 'taman.jobfilter.ru': 'jobfilter.ru',
 'borisoglebsk.jobfilter.ru': 'jobfilter.ru',
 'tatarstan.jobfilter.ru': 'jobfilter.ru',
 'tambov.jobfilter.ru': 'jobfilter.ru',
 'razan.jobfilter.ru': 'jobfilter.ru',
 'magadan.jobfilter.ru': 'jobfilter.ru',
 'chukotskii-ao.jobfilter.ru': 'jobfilter.ru',
 'susuman.jobfilter.ru': 'jobfilter.ru',
 'gorno-altaisk.jobfilter.ru': 'jobfilter.ru',
 'rostov-na-donu.jobfilter.ru': 'jobfilter.ru',
 'tomsk.jobfilter.ru': 'jobfilter.ru',
 'ishimbai.jobfilter.ru': 'jobfilter.ru',
 'noginsk.jobfilter.ru': 'jobfilter.ru',
 'nefteyugansk.jobfilter.ru': 'jobfilter.ru',
 'ijevsk.jobfilter.ru': 'jobfilter.ru',
 'arhangelsk.jobfilter.ru': 'jobfilter.ru',
 'chehov.jobfilter.ru': 'jobfilter.ru',
 'hanty-mansiisk.jobfilter.ru': 'jobfilter.ru',
 'sabetta.jobfilter.ru': 'jobfilter.ru',
 'egorevsk.jobfilter.ru': 'jobfilter.ru',
 'luga.jobfilter.ru': 'jobfilter.ru',
 'stupino.jobfilter.ru': 'jobfilter.ru',
 'saransk.jobfilter.ru': 'jobfilter.ru',
 'nijnekamsk.jobfilter.ru': 'jobfilter.ru',
 'ulyanovsk.jobfilter.ru': 'jobfilter.ru',
 'chita.jobfilter.ru': 'jobfilter.ru',
 'eisk.jobfilter.ru': 'jobfilter.ru',
 'naberejnye-chelny.jobfilter.ru': 'jobfilter.ru',
 'vorkuta.jobfilter.ru': 'jobfilter.ru',
 'yakutsk.jobfilter.ru': 'jobfilter.ru',
 'zmeinogorsk.jobfilter.ru': 'jobfilter.ru',
 'irkutsk.jobfilter.ru': 'jobfilter.ru',
 'norilsk.jobfilter.ru': 'jobfilter.ru',
 'petergof.jobfilter.ru': 'jobfilter.ru',
 'ioshkar-ola.jobfilter.ru': 'jobfilter.ru',
 'neftekamsk.jobfilter.ru': 'jobfilter.ru',
 'sankt-peterburg-region.jobfilter.ru': 'jobfilter.ru',
 'pushkino.jobfilter.ru': 'jobfilter.ru',
 'mytischi-mo.jobfilter.ru': 'jobfilter.ru',
 'petropavlovsk-kamchatskii.jobfilter.ru': 'jobfilter.ru',
 'groznyi.jobfilter.ru': 'jobfilter.ru',
 'ivanteevka.jobfilter.ru': 'jobfilter.ru',
 'yamalo-neneckii-okrug.jobfilter.ru': 'jobfilter.ru',
 'pavlovskii-posad.jobfilter.ru': 'jobfilter.ru',
 'innopolis.jobfilter.ru': 'jobfilter.ru',
 'syzran.jobfilter.ru': 'jobfilter.ru',
 'belogorsk.jobfilter.ru': 'jobfilter.ru',
 'belgorod.jobfilter.ru': 'jobfilter.ru',
 'cheboksary.jobfilter.ru': 'jobfilter.ru',
 'maikop.jobfilter.ru': 'jobfilter.ru',
 'aleksin.jobfilter.ru': 'jobfilter.ru',
 'yalutorovsk.jobfilter.ru': 'jobfilter.ru',
 'pavlovo.jobfilter.ru': 'jobfilter.ru',
 'jeleznogorsk-ko.jobfilter.ru': 'jobfilter.ru',
 'krasnoyarskii-krai.jobfilter.ru': 'jobfilter.ru',
 'jeleznogorsk.jobfilter.ru': 'jobfilter.ru',
 'klin.jobfilter.ru': 'jobfilter.ru',
 'adygeya.jobfilter.ru': 'jobfilter.ru',
 'tahtamukai.jobfilter.ru': 'jobfilter.ru',
 'salsk.jobfilter.ru': 'jobfilter.ru',
 'orenburgskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'blagoveschensk-bashkortostan.jobfilter.ru': 'jobfilter.ru',
 'astrahan.jobfilter.ru': 'jobfilter.ru',
 'simferopol.jobfilter.ru': 'jobfilter.ru',
 'ryazanskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'aksai.jobfilter.ru': 'jobfilter.ru',
 'sasovo.jobfilter.ru': 'jobfilter.ru',
 'novorossiisk.jobfilter.ru': 'jobfilter.ru',
 'fryazino.jobfilter.ru': 'jobfilter.ru',
 'svobodnyi-ao.jobfilter.ru': 'jobfilter.ru',
 'krasnokamsk.jobfilter.ru': 'jobfilter.ru',
 'tarko-sale.jobfilter.ru': 'jobfilter.ru',
 'prokopevsk.jobfilter.ru': 'jobfilter.ru',
 'tambovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'kursk.jobfilter.ru': 'jobfilter.ru',
 'voronejskaya.jobfilter.ru': 'jobfilter.ru',
 'abakan.jobfilter.ru': 'jobfilter.ru',
 'evpatoriya.jobfilter.ru': 'jobfilter.ru',
 'kanash.jobfilter.ru': 'jobfilter.ru',
 'penza.jobfilter.ru': 'jobfilter.ru',
 'mirnyi-yakutiya.jobfilter.ru': 'jobfilter.ru',
 'uhta.jobfilter.ru': 'jobfilter.ru',
 'bashkortostan.jobfilter.ru': 'jobfilter.ru',
 'anapa.jobfilter.ru': 'jobfilter.ru',
 'volgograd.jobfilter.ru': 'jobfilter.ru',
 'staraya-kupavna.jobfilter.ru': 'jobfilter.ru',
 'taganrog.jobfilter.ru': 'jobfilter.ru',
 'neryungri.jobfilter.ru': 'jobfilter.ru',
 'malmyz-2.jobfilter.ru': 'jobfilter.ru',
 'habarovskii-krai.jobfilter.ru': 'jobfilter.ru',
 'tula.jobfilter.ru': 'jobfilter.ru',
 'volgodonsk.jobfilter.ru': 'jobfilter.ru',
 'salavat.jobfilter.ru': 'jobfilter.ru',
 'dzerjinsk.jobfilter.ru': 'jobfilter.ru',
 'magnitogorsk.jobfilter.ru': 'jobfilter.ru',
 'lobnya.jobfilter.ru': 'jobfilter.ru',
 'vladivostok.jobfilter.ru': 'jobfilter.ru',
 'jukovskii.jobfilter.ru': 'jobfilter.ru',
 'gornopravdinsk.jobfilter.ru': 'jobfilter.ru',
 'vladikavkaz.jobfilter.ru': 'jobfilter.ru',
 'elec.jobfilter.ru': 'jobfilter.ru',
 'strejevoi.jobfilter.ru': 'jobfilter.ru',
 'salym.jobfilter.ru': 'jobfilter.ru',
 'nadym.jobfilter.ru': 'jobfilter.ru',
 'vorsino-kalujskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'dolgoprudnyi.jobfilter.ru': 'jobfilter.ru',
 'sertolovo.jobfilter.ru': 'jobfilter.ru',
 'voronej.jobfilter.ru': 'jobfilter.ru',
 'sheregesh.jobfilter.ru': 'jobfilter.ru',
 'megion.jobfilter.ru': 'jobfilter.ru',
 'novocherkassk.jobfilter.ru': 'jobfilter.ru',
 'stavropol.jobfilter.ru': 'jobfilter.ru',
 'kaliningradskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'udmurtiya.jobfilter.ru': 'jobfilter.ru',
 'pskov.jobfilter.ru': 'jobfilter.ru',
 'sevastopol.jobfilter.ru': 'jobfilter.ru',
 'sredneuralsk.jobfilter.ru': 'jobfilter.ru',
 'vyazma.jobfilter.ru': 'jobfilter.ru',
 'komsomolsk-na-amure.jobfilter.ru': 'jobfilter.ru',
 'tyumenskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'ulyanovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'gubkin.jobfilter.ru': 'jobfilter.ru',
 'kashira.jobfilter.ru': 'jobfilter.ru',
 'gornoe-loo.jobfilter.ru': 'jobfilter.ru',
 'gelendjik.jobfilter.ru': 'jobfilter.ru',
 'samarskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'obninsk.jobfilter.ru': 'jobfilter.ru',
 'lyubercy.jobfilter.ru': 'jobfilter.ru',
 'osinniki.jobfilter.ru': 'jobfilter.ru',
 'kemerovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'kamensk-uralskii.jobfilter.ru': 'jobfilter.ru',
 'kirjach.jobfilter.ru': 'jobfilter.ru',
 'leninsk-kuzneckii.jobfilter.ru': 'jobfilter.ru',
 'smolensk.jobfilter.ru': 'jobfilter.ru',
 'krasnogorsk-mo.jobfilter.ru': 'jobfilter.ru',
 'voljskii.jobfilter.ru': 'jobfilter.ru',
 'krym.jobfilter.ru': 'jobfilter.ru',
 'sofrino.jobfilter.ru': 'jobfilter.ru',
 'cherepovec.jobfilter.ru': 'jobfilter.ru',
 'sterlitamak.jobfilter.ru': 'jobfilter.ru',
 'novyi-urengoi.jobfilter.ru': 'jobfilter.ru',
 'engels.jobfilter.ru': 'jobfilter.ru',
 'belaya-kalitva.jobfilter.ru': 'jobfilter.ru',
 'vyksa.jobfilter.ru': 'jobfilter.ru',
 'belovo.jobfilter.ru': 'jobfilter.ru',
 'barabinsk.jobfilter.ru': 'jobfilter.ru',
 'bobrov.jobfilter.ru': 'jobfilter.ru',
 'bryansk.jobfilter.ru': 'jobfilter.ru',
 'orel.jobfilter.ru': 'jobfilter.ru',
 'ust-labinsk.jobfilter.ru': 'jobfilter.ru',
 'krasnouralsk.jobfilter.ru': 'jobfilter.ru',
 'elektrougli.jobfilter.ru': 'jobfilter.ru',
 'chelyabinskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'novocheboksarsk.jobfilter.ru': 'jobfilter.ru',
 'sayanogorsk.jobfilter.ru': 'jobfilter.ru',
 'murmino.jobfilter.ru': 'jobfilter.ru',
 'kogalym.jobfilter.ru': 'jobfilter.ru',
 'sarapul.jobfilter.ru': 'jobfilter.ru',
 'rjev.jobfilter.ru': 'jobfilter.ru',
 'tver.jobfilter.ru': 'jobfilter.ru',
 'urga.jobfilter.ru': 'jobfilter.ru',
 'odincovo-mo.jobfilter.ru': 'jobfilter.ru',
 'dzerjinskii.jobfilter.ru': 'jobfilter.ru',
 'lytkarino.jobfilter.ru': 'jobfilter.ru',
 'ivanovo.jobfilter.ru': 'jobfilter.ru',
 'zelenograd.jobfilter.ru': 'jobfilter.ru',
 'syktyvkar.jobfilter.ru': 'jobfilter.ru',
 'bronnicy.jobfilter.ru': 'jobfilter.ru',
 'zlatoust.jobfilter.ru': 'jobfilter.ru',
 'krasnoznamensk.jobfilter.ru': 'jobfilter.ru',
 'vyshnii-volochek.jobfilter.ru': 'jobfilter.ru',
 'miass.jobfilter.ru': 'jobfilter.ru',
 'kamensk-shahtinskii.jobfilter.ru': 'jobfilter.ru',
 'buzuluk.jobfilter.ru': 'jobfilter.ru',
 'tihvin.jobfilter.ru': 'jobfilter.ru',
 'ust-luga.jobfilter.ru': 'jobfilter.ru',
 'leningradskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'tuimazy.jobfilter.ru': 'jobfilter.ru',
 'kurganinsk.jobfilter.ru': 'jobfilter.ru',
 'ust-katav.jobfilter.ru': 'jobfilter.ru',
 'orsk.jobfilter.ru': 'jobfilter.ru',
 'timashevsk.jobfilter.ru': 'jobfilter.ru',
 'serebryanye-prudy.jobfilter.ru': 'jobfilter.ru',
 'shahty.jobfilter.ru': 'jobfilter.ru',
 'bolshoe-polpino.jobfilter.ru': 'jobfilter.ru',
 'bavly.jobfilter.ru': 'jobfilter.ru',
 'dimitrovgrad.jobfilter.ru': 'jobfilter.ru',
 'okulovka.jobfilter.ru': 'jobfilter.ru',
 'alekseevka.jobfilter.ru': 'jobfilter.ru',
 'klimovsk.jobfilter.ru': 'jobfilter.ru',
 'ust-kut.jobfilter.ru': 'jobfilter.ru',
 'aleksandrov.jobfilter.ru': 'jobfilter.ru',
 'volovo-lo.jobfilter.ru': 'jobfilter.ru',
 'chebarkul.jobfilter.ru': 'jobfilter.ru',
 'pyatigorsk.jobfilter.ru': 'jobfilter.ru',
 'scherbinka.jobfilter.ru': 'jobfilter.ru',
 'gatchina.jobfilter.ru': 'jobfilter.ru',
 'votkinsk.jobfilter.ru': 'jobfilter.ru',
 'korolev.jobfilter.ru': 'jobfilter.ru',
 'michurinsk.jobfilter.ru': 'jobfilter.ru',
 'vinzili.jobfilter.ru': 'jobfilter.ru',
 'novokuibyshevsk.jobfilter.ru': 'jobfilter.ru',
 'kimry.jobfilter.ru': 'jobfilter.ru',
 'kotelniki-mo.jobfilter.ru': 'jobfilter.ru',
 'saratovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'hvalynsk.jobfilter.ru': 'jobfilter.ru',
 'kurchatov.jobfilter.ru': 'jobfilter.ru',
 'sim.jobfilter.ru': 'jobfilter.ru',
 'tutaev.jobfilter.ru': 'jobfilter.ru',
 'kamchatskii-krai.jobfilter.ru': 'jobfilter.ru',
 'milkovo.jobfilter.ru': 'jobfilter.ru',
 'pereslavl-zalesskii.jobfilter.ru': 'jobfilter.ru',
 'eniseisk.jobfilter.ru': 'jobfilter.ru',
 'serpuhov.jobfilter.ru': 'jobfilter.ru',
 'suksun.jobfilter.ru': 'jobfilter.ru',
 'yalta.jobfilter.ru': 'jobfilter.ru',
 'novomichurinsk.jobfilter.ru': 'jobfilter.ru',
 'gubkinskii.jobfilter.ru': 'jobfilter.ru',
 'ruzaevka.jobfilter.ru': 'jobfilter.ru',
 'hotkovo.jobfilter.ru': 'jobfilter.ru',
 'reutov.jobfilter.ru': 'jobfilter.ru',
 'glinishevo.jobfilter.ru': 'jobfilter.ru',
 'bryanskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'rybinsk.jobfilter.ru': 'jobfilter.ru',
 'kondopoga.jobfilter.ru': 'jobfilter.ru',
 'kareliya.jobfilter.ru': 'jobfilter.ru',
 'revda.jobfilter.ru': 'jobfilter.ru',
 'krasnozavodsk.jobfilter.ru': 'jobfilter.ru',
 'bataisk.jobfilter.ru': 'jobfilter.ru',
 'hanty-mansiiskii-ao.jobfilter.ru': 'jobfilter.ru',
 'tuapse.jobfilter.ru': 'jobfilter.ru',
 'ishim.jobfilter.ru': 'jobfilter.ru',
 'tulskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'gurevsk.jobfilter.ru': 'jobfilter.ru',
 'berezniki.jobfilter.ru': 'jobfilter.ru',
 'velikie-luki.jobfilter.ru': 'jobfilter.ru',
 'mahackala.jobfilter.ru': 'jobfilter.ru',
 'plast.jobfilter.ru': 'jobfilter.ru',
 'zarechnyi-so.jobfilter.ru': 'jobfilter.ru',
 'lyantor.jobfilter.ru': 'jobfilter.ru',
 'salehard.jobfilter.ru': 'jobfilter.ru',
 'verhnyaya-pyshma.jobfilter.ru': 'jobfilter.ru',
 'korenovsk.jobfilter.ru': 'jobfilter.ru',
 'chernogorsk.jobfilter.ru': 'jobfilter.ru',
 'shebekino.jobfilter.ru': 'jobfilter.ru',
 'kropotkin.jobfilter.ru': 'jobfilter.ru',
 'ojerele.jobfilter.ru': 'jobfilter.ru',
 'ozery.jobfilter.ru': 'jobfilter.ru',
 'oktyabrskii-bashkortostan.jobfilter.ru': 'jobfilter.ru',
 'selyatino.jobfilter.ru': 'jobfilter.ru',
 'dyurtyuli.jobfilter.ru': 'jobfilter.ru',
 'udomlya.jobfilter.ru': 'jobfilter.ru',
 'nijnyaya-tura.jobfilter.ru': 'jobfilter.ru',
 'privoljsk.jobfilter.ru': 'jobfilter.ru',
 'yakovlevskoe.jobfilter.ru': 'jobfilter.ru',
 'murmanskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'bykovo-moskovskaa-oblast.jobfilter.ru': 'jobfilter.ru',
 'valuiki.jobfilter.ru': 'jobfilter.ru',
 'belorechensk.jobfilter.ru': 'jobfilter.ru',
 'podstepki.jobfilter.ru': 'jobfilter.ru',
 'berdsk.jobfilter.ru': 'jobfilter.ru',
 'biisk.jobfilter.ru': 'jobfilter.ru',
 'krasnoturinsk.jobfilter.ru': 'jobfilter.ru',
 'maloyaroslavec.jobfilter.ru': 'jobfilter.ru',
 'mojaisk.jobfilter.ru': 'jobfilter.ru',
 'temryuk.jobfilter.ru': 'jobfilter.ru',
 'jigulevsk.jobfilter.ru': 'jobfilter.ru',
 'selco.jobfilter.ru': 'jobfilter.ru',
 'sverdlovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'gramoteino.jobfilter.ru': 'jobfilter.ru',
 'staraya-russa.jobfilter.ru': 'jobfilter.ru',
 'kinel.jobfilter.ru': 'jobfilter.ru',
 'kondrovo.jobfilter.ru': 'jobfilter.ru',
 'kalujskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'zainsk.jobfilter.ru': 'jobfilter.ru',
 'arhipo-osipovka.jobfilter.ru': 'jobfilter.ru',
 'karachev.jobfilter.ru': 'jobfilter.ru',
 'krasnogvardeiskoe-krym.jobfilter.ru': 'jobfilter.ru',
 'armavir.jobfilter.ru': 'jobfilter.ru',
 'komi.jobfilter.ru': 'jobfilter.ru',
 'spas-klepiki.jobfilter.ru': 'jobfilter.ru',
 'belgorodskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'ussuriisk.jobfilter.ru': 'jobfilter.ru',
 'rostovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'achinsk.jobfilter.ru': 'jobfilter.ru',
 'pechora.jobfilter.ru': 'jobfilter.ru',
 'udachnyi.jobfilter.ru': 'jobfilter.ru',
 'severodvinsk.jobfilter.ru': 'jobfilter.ru',
 'novomoskovsk.jobfilter.ru': 'jobfilter.ru',
 'novyi-uoyan.jobfilter.ru': 'jobfilter.ru',
 'lubercy-4.jobfilter.ru': 'jobfilter.ru',
 'teikovo.jobfilter.ru': 'jobfilter.ru',
 'schekino.jobfilter.ru': 'jobfilter.ru',
 'ust-nera.jobfilter.ru': 'jobfilter.ru',
 'ust-ilimsk.jobfilter.ru': 'jobfilter.ru',
 'yanaul.jobfilter.ru': 'jobfilter.ru',
 'livny.jobfilter.ru': 'jobfilter.ru',
 'kuibyshev.jobfilter.ru': 'jobfilter.ru',
 'sysert.jobfilter.ru': 'jobfilter.ru',
 'shushenskoe.jobfilter.ru': 'jobfilter.ru',
 'saraktash.jobfilter.ru': 'jobfilter.ru',
 'kineshma.jobfilter.ru': 'jobfilter.ru',
 'stavropol-skij.jobfilter.ru': 'jobfilter.ru',
 'vatutinki.jobfilter.ru': 'jobfilter.ru',
 'nevinnomyssk.jobfilter.ru': 'jobfilter.ru',
 'vasyurinskaya.jobfilter.ru': 'jobfilter.ru',
 'krestcy.jobfilter.ru': 'jobfilter.ru',
 'novgorodskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'kurskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'obolensk.jobfilter.ru': 'jobfilter.ru',
 'uzlovaya.jobfilter.ru': 'jobfilter.ru',
 'orlovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'angarsk.jobfilter.ru': 'jobfilter.ru',
 'gorodec.jobfilter.ru': 'jobfilter.ru',
 'balahna.jobfilter.ru': 'jobfilter.ru',
 'yanino-1.jobfilter.ru': 'jobfilter.ru',
 'goryachii-klyuch.jobfilter.ru': 'jobfilter.ru',
 'trehgornyi.jobfilter.ru': 'jobfilter.ru',
 'yagodnoe.jobfilter.ru': 'jobfilter.ru',
 'solikamsk.jobfilter.ru': 'jobfilter.ru',
 'polotnyanyi-zavod.jobfilter.ru': 'jobfilter.ru',
 'rebriha.jobfilter.ru': 'jobfilter.ru',
 'noyabrsk.jobfilter.ru': 'jobfilter.ru',
 'novomihailovskii.jobfilter.ru': 'jobfilter.ru',
 'otradnoe-lo.jobfilter.ru': 'jobfilter.ru',
 'bolshie-kolpany.jobfilter.ru': 'jobfilter.ru',
 'valdai.jobfilter.ru': 'jobfilter.ru',
 'kyzyl.jobfilter.ru': 'jobfilter.ru',
 'respublika-tyva.jobfilter.ru': 'jobfilter.ru',
 'novouralsk.jobfilter.ru': 'jobfilter.ru',
 'krekshino.jobfilter.ru': 'jobfilter.ru',
 'tomilino.jobfilter.ru': 'jobfilter.ru',
 'vityazevo.jobfilter.ru': 'jobfilter.ru',
 'olenegorsk.jobfilter.ru': 'jobfilter.ru',
 'novokubansk.jobfilter.ru': 'jobfilter.ru',
 'primorskii-krai.jobfilter.ru': 'jobfilter.ru',
 'kalyazin.jobfilter.ru': 'jobfilter.ru',
 'rej.jobfilter.ru': 'jobfilter.ru',
 'stolbishi.jobfilter.ru': 'jobfilter.ru',
 'hakasiya.jobfilter.ru': 'jobfilter.ru',
 'rubcovsk.jobfilter.ru': 'jobfilter.ru',
 'mishkino-bashkortostan.jobfilter.ru': 'jobfilter.ru',
 'pristen.jobfilter.ru': 'jobfilter.ru',
 'krasnaya-polyana.jobfilter.ru': 'jobfilter.ru',
 'zavodoukovsk.jobfilter.ru': 'jobfilter.ru',
 'bolshoi-kamen.jobfilter.ru': 'jobfilter.ru',
 'magistralnyi.jobfilter.ru': 'jobfilter.ru',
 'nyagan.jobfilter.ru': 'jobfilter.ru',
 'davydovo.jobfilter.ru': 'jobfilter.ru',
 'krymsk.jobfilter.ru': 'jobfilter.ru',
 'obuhovo.jobfilter.ru': 'jobfilter.ru',
 'smolenskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'shatura.jobfilter.ru': 'jobfilter.ru',
 'malahovka.jobfilter.ru': 'jobfilter.ru',
 'kanevskaya.jobfilter.ru': 'jobfilter.ru',
 'bejeck.jobfilter.ru': 'jobfilter.ru',
 'pindushi.jobfilter.ru': 'jobfilter.ru',
 'medvejegorsk.jobfilter.ru': 'jobfilter.ru',
 'volno-nadejdinskoe.jobfilter.ru': 'jobfilter.ru',
 'kastornoe.jobfilter.ru': 'jobfilter.ru',
 'sibai.jobfilter.ru': 'jobfilter.ru',
 'chapaevsk.jobfilter.ru': 'jobfilter.ru',
 'kolpino.jobfilter.ru': 'jobfilter.ru',
 'suhoi-log.jobfilter.ru': 'jobfilter.ru',
 'mineralnye-vody.jobfilter.ru': 'jobfilter.ru',
 'muravlenko.jobfilter.ru': 'jobfilter.ru',
 'nerchinsk.jobfilter.ru': 'jobfilter.ru',
 'zabaikalskii-krai.jobfilter.ru': 'jobfilter.ru',
 'kozelsk.jobfilter.ru': 'jobfilter.ru',
 'baikalovo-so.jobfilter.ru': 'jobfilter.ru',
 'alushta.jobfilter.ru': 'jobfilter.ru',
 'chernomorskoe.jobfilter.ru': 'jobfilter.ru',
 'vidnoe.jobfilter.ru': 'jobfilter.ru',
 'zelenogorsk.jobfilter.ru': 'jobfilter.ru',
 'stroitel-belgorodskaa-oblast.jobfilter.ru': 'jobfilter.ru',
 'nijegorodskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'chernogolovka.jobfilter.ru': 'jobfilter.ru',
 'baranchinskii.jobfilter.ru': 'jobfilter.ru',
 'shuya.jobfilter.ru': 'jobfilter.ru',
 'novohopersk.jobfilter.ru': 'jobfilter.ru',
 'lisva.jobfilter.ru': 'jobfilter.ru',
 'gavrilov-yam.jobfilter.ru': 'jobfilter.ru',
 'volgogradskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'apatity.jobfilter.ru': 'jobfilter.ru',
 'belusha-guba.jobfilter.ru': 'jobfilter.ru',
 'bugulma.jobfilter.ru': 'jobfilter.ru',
 'ivanovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'furmanov.jobfilter.ru': 'jobfilter.ru',
 'lihoslavl.jobfilter.ru': 'jobfilter.ru',
 'tverskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'berezovskii.jobfilter.ru': 'jobfilter.ru',
 'roslavl.jobfilter.ru': 'jobfilter.ru',
 'rtischevo.jobfilter.ru': 'jobfilter.ru',
 'kirovsk.jobfilter.ru': 'jobfilter.ru',
 'serdobsk.jobfilter.ru': 'jobfilter.ru',
 'kirovskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'beloozerskij-moskovskaa-oblast.jobfilter.ru': 'jobfilter.ru',
    'murmansk.vsn.ru': 'vsn.ru',
 'vsn.ru': 'vsn.ru',
 'tobolsk.vsn.ru': 'vsn.ru',
 'domodedovo.vsn.ru': 'vsn.ru',
 'moskva.vsn.ru': 'vsn.ru',
 'kursk.vsn.ru': 'vsn.ru',
 'stroitel.vsn.ru': 'vsn.ru',
 'novotroitsk.vsn.ru': 'vsn.ru',
 'orsk.vsn.ru': 'vsn.ru',
 'rostov-na-donu.vsn.ru': 'vsn.ru',
 'perm.vsn.ru': 'vsn.ru',
 'lyubertsy.vsn.ru': 'vsn.ru',
 'rossosh.vsn.ru': 'vsn.ru',
 'balashiha.vsn.ru': 'vsn.ru',
 'kovdor.vsn.ru': 'vsn.ru',
 'kurgan.vsn.ru': 'vsn.ru',
 'saratov.vsn.ru': 'vsn.ru',
 'krasnodar.vsn.ru': 'vsn.ru',
 'sosnovyy-bor.vsn.ru': 'vsn.ru',
 'kirov.vsn.ru': 'vsn.ru',
 'vladikavkaz.vsn.ru': 'vsn.ru',
 'yuryuzan.vsn.ru': 'vsn.ru',
 'obninsk.vsn.ru': 'vsn.ru',
 'podolsk.vsn.ru': 'vsn.ru',
 'novorossiysk.vsn.ru': 'vsn.ru',
 'yalutorovsk.vsn.ru': 'vsn.ru',
 'mahachkala.vsn.ru': 'vsn.ru',
 'izhevsk.vsn.ru': 'vsn.ru',
 'ekaterinburg.vsn.ru': 'vsn.ru',
 'apsheronsk.vsn.ru': 'vsn.ru',
 'polevskoy.vsn.ru': 'vsn.ru',
 'sevastopol.vsn.ru': 'vsn.ru',
 'sankt-peterburg.vsn.ru': 'vsn.ru',
 'krasnoyarsk.vsn.ru': 'vsn.ru',
 'sergiev-posad.vsn.ru': 'vsn.ru',
 'asha.vsn.ru': 'vsn.ru',
 'zima.vsn.ru': 'vsn.ru',
 'cheremhovo.vsn.ru': 'vsn.ru',
 'murmanskaya-oblast-kirovsk.vsn.ru': 'vsn.ru',
 'nizhnekamsk.vsn.ru': 'vsn.ru',
 'dyatkovo.vsn.ru': 'vsn.ru',
 'smolensk.vsn.ru': 'vsn.ru',
 'ivanovo.vsn.ru': 'vsn.ru',
 'lipetsk.vsn.ru': 'vsn.ru',
 'oktyabrskiy.vsn.ru': 'vsn.ru',
 'habarovsk.vsn.ru': 'vsn.ru',
 'almetievsk.vsn.ru': 'vsn.ru',
 'velikie-luki.vsn.ru': 'vsn.ru',
 'novosibirsk.vsn.ru': 'vsn.ru',
 'voskresensk.vsn.ru': 'vsn.ru',
 'sochi.vsn.ru': 'vsn.ru',
 'elizovo.vsn.ru': 'vsn.ru',
 'petropavlovsk-kamchatskiy.vsn.ru': 'vsn.ru',
 'samara.vsn.ru': 'vsn.ru',
 'orehovo-zuevo.vsn.ru': 'vsn.ru',
 'korolev.vsn.ru': 'vsn.ru',
 'himki.vsn.ru': 'vsn.ru',
 'mytischi.vsn.ru': 'vsn.ru',
 'yalta.vsn.ru': 'vsn.ru',
 'barnaul.vsn.ru': 'vsn.ru',
 'uchaly.vsn.ru': 'vsn.ru',
 'nizhniy-tagil.vsn.ru': 'vsn.ru',
 'norilsk.vsn.ru': 'vsn.ru',
 'novokuznetsk.vsn.ru': 'vsn.ru',
 'kropotkin.vsn.ru': 'vsn.ru',
 'ryazan.vsn.ru': 'vsn.ru',
 'kyzyl.vsn.ru': 'vsn.ru',
 'neftekamsk.vsn.ru': 'vsn.ru',
 'morshansk.vsn.ru': 'vsn.ru',
 'ufa.vsn.ru': 'vsn.ru',
 'zlatoust.vsn.ru': 'vsn.ru',
 'alushta.vsn.ru': 'vsn.ru',
 'tolyatti.vsn.ru': 'vsn.ru',
 'tyumen.vsn.ru': 'vsn.ru',
 'eysk.vsn.ru': 'vsn.ru',
 'abakan.vsn.ru': 'vsn.ru',
 'slavyansk-na-kubani.vsn.ru': 'vsn.ru',
 'armavir.vsn.ru': 'vsn.ru',
 'gorno-altaysk.vsn.ru': 'vsn.ru',
 'chelyabinsk.vsn.ru': 'vsn.ru',
 'belgorod.vsn.ru': 'vsn.ru',
 'penza.vsn.ru': 'vsn.ru',
 'taganrog.vsn.ru': 'vsn.ru',
 'chehov.vsn.ru': 'vsn.ru',
 'kerch.vsn.ru': 'vsn.ru',
 'kachkanar.vsn.ru': 'vsn.ru',
 'voronezh.vsn.ru': 'vsn.ru',
 'omsk.vsn.ru': 'vsn.ru',
 'dzerzhinsk.vsn.ru': 'vsn.ru',
 'krasnoe-selo.vsn.ru': 'vsn.ru',
 'salehard.vsn.ru': 'vsn.ru',
 'petrozavodsk.vsn.ru': 'vsn.ru',
 'vologda.vsn.ru': 'vsn.ru',
 'kazan.vsn.ru': 'vsn.ru',
 'kemerovo.vsn.ru': 'vsn.ru',
 'sverdlovskaya-oblast-zarechnyy.vsn.ru': 'vsn.ru',
 'komsomolsk-na-amure.vsn.ru': 'vsn.ru',
 'miass.vsn.ru': 'vsn.ru',
 'essentuki.vsn.ru': 'vsn.ru',
 'saransk.vsn.ru': 'vsn.ru',
 'nerehta.vsn.ru': 'vsn.ru',
 'krasnouralsk.vsn.ru': 'vsn.ru',
 'minusinsk.vsn.ru': 'vsn.ru',
 'mtsensk.vsn.ru': 'vsn.ru',
 'yaroslavl.vsn.ru': 'vsn.ru',
 'magnitogorsk.vsn.ru': 'vsn.ru',
 'revda.vsn.ru': 'vsn.ru',
 'novokuybyshevsk.vsn.ru': 'vsn.ru',
 'zhigulevsk.vsn.ru': 'vsn.ru',
 'severodvinsk.vsn.ru': 'vsn.ru',
 'pskov.vsn.ru': 'vsn.ru',
 'scherbinka.vsn.ru': 'vsn.ru',
 'asbest.vsn.ru': 'vsn.ru',
 'vyatskie-polyany.vsn.ru': 'vsn.ru',
 'volgodonsk.vsn.ru': 'vsn.ru',
 'yoshkar-ola.vsn.ru': 'vsn.ru',
 'kislovodsk.vsn.ru': 'vsn.ru',
 'luhovitsy.vsn.ru': 'vsn.ru',
 'megion.vsn.ru': 'vsn.ru',
 'orel.vsn.ru': 'vsn.ru',
 'pereslavl-zalesskiy.vsn.ru': 'vsn.ru',
 'svetlogorsk.vsn.ru': 'vsn.ru',
 'uhta.vsn.ru': 'vsn.ru',
 'abinsk.vsn.ru': 'vsn.ru',
 'kaspiysk.vsn.ru': 'vsn.ru',
 'satka.vsn.ru': 'vsn.ru',
 'krasnokamsk.vsn.ru': 'vsn.ru',
 'naro-fominsk.vsn.ru': 'vsn.ru',
 'orenburg.vsn.ru': 'vsn.ru',
 'elabuga.vsn.ru': 'vsn.ru',
 'kopeysk.vsn.ru': 'vsn.ru',
 'cheboksary.vsn.ru': 'vsn.ru',
 'moskovskaya-oblast-krasnoznamensk.vsn.ru': 'vsn.ru',
 'noginsk.vsn.ru': 'vsn.ru',
 'kotelniki.vsn.ru': 'vsn.ru',
 'aprelevka.vsn.ru': 'vsn.ru',
 'elektrostal.vsn.ru': 'vsn.ru',
 'schyolkovo.vsn.ru': 'vsn.ru',
 'ramenskoe.vsn.ru': 'vsn.ru',
 'zaraysk.vsn.ru': 'vsn.ru',
 'solnechnogorsk.vsn.ru': 'vsn.ru',
 'gelendzhik.vsn.ru': 'vsn.ru',
 'belorechensk.vsn.ru': 'vsn.ru',
 'pervouralsk.vsn.ru': 'vsn.ru',
 'dolgoprudnyy.vsn.ru': 'vsn.ru',
 'tuapse.vsn.ru': 'vsn.ru',
 'feodosiya.vsn.ru': 'vsn.ru',
 'sredneuralsk.vsn.ru': 'vsn.ru',
 'magadan.vsn.ru': 'vsn.ru',
 'labinsk.vsn.ru': 'vsn.ru',
 'ishimbay.vsn.ru': 'vsn.ru',
 'krasnogorsk.vsn.ru': 'vsn.ru',
 'pavlovskiy-posad.vsn.ru': 'vsn.ru',
 'reutov.vsn.ru': 'vsn.ru',
 'nalchik.vsn.ru': 'vsn.ru',
 'losino-petrovskiy.vsn.ru': 'vsn.ru',
 'liski.vsn.ru': 'vsn.ru',
 'sterlitamak.vsn.ru': 'vsn.ru',
 'serpuhov.vsn.ru': 'vsn.ru',
 'kovrov.vsn.ru': 'vsn.ru',
 'sysert.vsn.ru': 'vsn.ru',
 'velikiy-novgorod.vsn.ru': 'vsn.ru',
 'tula.vsn.ru': 'vsn.ru',
 'chernyahovsk.vsn.ru': 'vsn.ru',
 'sarapul.vsn.ru': 'vsn.ru',
 'gudermes.vsn.ru': 'vsn.ru',
 'dmitrov.vsn.ru': 'vsn.ru',
 'kashira.vsn.ru': 'vsn.ru',
 'spassk-dalniy.vsn.ru': 'vsn.ru',
 'syzran.vsn.ru': 'vsn.ru',
 'pyt-yah.vsn.ru': 'vsn.ru',
 'berezniki.vsn.ru': 'vsn.ru',
 'gubkin.vsn.ru': 'vsn.ru',
 'svobodnyy.vsn.ru': 'vsn.ru',
 'novocherkassk.vsn.ru': 'vsn.ru',
 'vichuga.vsn.ru': 'vsn.ru',
 'bataysk.vsn.ru': 'vsn.ru',
 'solikamsk.vsn.ru': 'vsn.ru',
 'kungur.vsn.ru': 'vsn.ru',
 'kaliningrad.vsn.ru': 'vsn.ru',
 'partizansk.vsn.ru': 'vsn.ru',
 'budennovsk.vsn.ru': 'vsn.ru',
 'cherkessk.vsn.ru': 'vsn.ru',
 'novoshahtinsk.vsn.ru': 'vsn.ru',
 'tomsk.vsn.ru': 'vsn.ru',
 'goryachiy-klyuch.vsn.ru': 'vsn.ru',
 'lytkarino.vsn.ru': 'vsn.ru',
 'shadrinsk.vsn.ru': 'vsn.ru',
 'kostroma.vsn.ru': 'vsn.ru',
 'laishevo.vsn.ru': 'vsn.ru',
 'anapa.vsn.ru': 'vsn.ru',
 'lobnya.vsn.ru': 'vsn.ru',
 'nizhnevartovsk.vsn.ru': 'vsn.ru',
 'donetsk.vsn.ru': 'vsn.ru',
 'sasovo.vsn.ru': 'vsn.ru',
 'vladimir.vsn.ru': 'vsn.ru',
 'vidnoe.vsn.ru': 'vsn.ru',
 'evpatoriya.vsn.ru': 'vsn.ru',
 'kaluga.vsn.ru': 'vsn.ru',
 'chistopol.vsn.ru': 'vsn.ru',
 'neryungri.vsn.ru': 'vsn.ru',
 'volgograd.vsn.ru': 'vsn.ru',
 'schigry.vsn.ru': 'vsn.ru',
 'narimanov.vsn.ru': 'vsn.ru',
 'glazov.vsn.ru': 'vsn.ru',
 'divnogorsk.vsn.ru': 'vsn.ru',
 'amursk.vsn.ru': 'vsn.ru',
 'georgievsk.vsn.ru': 'vsn.ru',
 'valuyki.vsn.ru': 'vsn.ru',
 'vyksa.vsn.ru': 'vsn.ru',
 'zvenigorod.vsn.ru': 'vsn.ru',
 'ulyanovsk.vsn.ru': 'vsn.ru',
 'kamensk-uralskiy.vsn.ru': 'vsn.ru',
 'sudak.vsn.ru': 'vsn.ru',
 'belomorsk.vsn.ru': 'vsn.ru',
 'dobryanka.vsn.ru': 'vsn.ru',
 'primorsko-ahtarsk.vsn.ru': 'vsn.ru',
 'zelenograd.vsn.ru': 'vsn.ru',
 'tutaev.vsn.ru': 'vsn.ru',
 'kolomna.vsn.ru': 'vsn.ru',
 'balakovo.vsn.ru': 'vsn.ru',
 'simferopol.vsn.ru': 'vsn.ru',
 'staryy-krym.vsn.ru': 'vsn.ru',
 'barabinsk.vsn.ru': 'vsn.ru',
 'kolpino.vsn.ru': 'vsn.ru',
 'aleysk.vsn.ru': 'vsn.ru',
 'bryansk.vsn.ru': 'vsn.ru',
 'sverdlovskaya-oblast-berezovskiy.vsn.ru': 'vsn.ru',
 'syktyvkar.vsn.ru': 'vsn.ru',
 'bashkortostan-blagoveschensk.vsn.ru': 'vsn.ru',
 'vyborg.vsn.ru': 'vsn.ru',
 'istra.vsn.ru': 'vsn.ru',
 'kondrovo.vsn.ru': 'vsn.ru',
 'noyabrsk.vsn.ru': 'vsn.ru',
 'ivanteevka.vsn.ru': 'vsn.ru',
 'yuzhno-sahalinsk.vsn.ru': 'vsn.ru',
 'degtyarsk.vsn.ru': 'vsn.ru',
 'uryupinsk.vsn.ru': 'vsn.ru',
 'stavropol.vsn.ru': 'vsn.ru',
 'elista.vsn.ru': 'vsn.ru',
 'yarovoe.vsn.ru': 'vsn.ru',
 'novodvinsk.vsn.ru': 'vsn.ru',
 'zainsk.vsn.ru': 'vsn.ru',
 'arhangelsk.vsn.ru': 'vsn.ru',
 'pavlovsk.vsn.ru': 'vsn.ru',
 'rezh.vsn.ru': 'vsn.ru',
 'novokubansk.vsn.ru': 'vsn.ru',
 'surgut.vsn.ru': 'vsn.ru',
 'furmanov.vsn.ru': 'vsn.ru',
 'tambov.vsn.ru': 'vsn.ru',
 'uvarovo.vsn.ru': 'vsn.ru',
 'salavat.vsn.ru': 'vsn.ru',
 'achinsk.vsn.ru': 'vsn.ru',
 'nizhniy-novgorod.vsn.ru': 'vsn.ru',
 'birobidzhan.vsn.ru': 'vsn.ru',
 'kuybyshev.vsn.ru': 'vsn.ru',
 'staraya-russa.vsn.ru': 'vsn.ru',
 'ust-dzheguta.vsn.ru': 'vsn.ru',
 'lgov.vsn.ru': 'vsn.ru',
 'muravlenko.vsn.ru': 'vsn.ru',
 'murino.vsn.ru': 'vsn.ru',
 'staryy-oskol.vsn.ru': 'vsn.ru',
 'chelyabinskaya-oblast-troitsk.vsn.ru': 'vsn.ru',
 'buynaksk.vsn.ru': 'vsn.ru',
 'maykop.vsn.ru': 'vsn.ru',
 'yugorsk.vsn.ru': 'vsn.ru',
 'rubtsovsk.vsn.ru': 'vsn.ru',
 'derbent.vsn.ru': 'vsn.ru',
 'irkutsk.vsn.ru': 'vsn.ru',
 'pushkino.vsn.ru': 'vsn.ru',
 'tuymazy.vsn.ru': 'vsn.ru',
 'kandalaksha.vsn.ru': 'vsn.ru',
 'kirovo-chepetsk.vsn.ru': 'vsn.ru',
 'mariinsk.vsn.ru': 'vsn.ru',
 'mineralnye-vody.vsn.ru': 'vsn.ru',
 'nazarovo.vsn.ru': 'vsn.ru',
 'baykalsk.vsn.ru': 'vsn.ru',
 'ulan-ude.vsn.ru': 'vsn.ru',
 'gornyak.vsn.ru': 'vsn.ru',
 'pyatigorsk.vsn.ru': 'vsn.ru',
 'roslavl.vsn.ru': 'vsn.ru',
 'sibay.vsn.ru': 'vsn.ru',
 'zhukovskiy.vsn.ru': 'vsn.ru',
 'groznyy.vsn.ru': 'vsn.ru',
 'krasnoyarskiy-kray-zheleznogorsk.vsn.ru': 'vsn.ru',
 'slavgorod.vsn.ru': 'vsn.ru',
 'kamyshin.vsn.ru': 'vsn.ru',
 'kondopoga.vsn.ru': 'vsn.ru',
 'blagoveschensk.vsn.ru': 'vsn.ru',
 'gulkevichi.vsn.ru': 'vsn.ru',
 'rostov.vsn.ru': 'vsn.ru',
 'isilkul.vsn.ru': 'vsn.ru',
 'pushkin.vsn.ru': 'vsn.ru',
 'alekseevka.vsn.ru': 'vsn.ru',
 'nefteyugansk.vsn.ru': 'vsn.ru',
 'buzuluk.vsn.ru': 'vsn.ru',
 'dankov.vsn.ru': 'vsn.ru',
 'zhukov.vsn.ru': 'vsn.ru',
 'beslan.vsn.ru': 'vsn.ru',
 'kstovo.vsn.ru': 'vsn.ru',
 'kamen-na-obi.vsn.ru': 'vsn.ru',
 'lesnoy.vsn.ru': 'vsn.ru',
 'bor.vsn.ru': 'vsn.ru',
 'bavly.vsn.ru': 'vsn.ru',
 'chita.vsn.ru': 'vsn.ru',
 'prohladnyy.vsn.ru': 'vsn.ru',
 'hanty-mansiysk.vsn.ru': 'vsn.ru',
 'mirnyy.vsn.ru': 'vsn.ru',
 'berdsk.vsn.ru': 'vsn.ru',
 'kansk.vsn.ru': 'vsn.ru',
 'vyazma.vsn.ru': 'vsn.ru',
 'novozybkov.vsn.ru': 'vsn.ru',
 'strezhevoy.vsn.ru': 'vsn.ru',
 'serov.vsn.ru': 'vsn.ru',
 'tulun.vsn.ru': 'vsn.ru',
 'zelenodolsk.vsn.ru': 'vsn.ru',
 'adygeysk.vsn.ru': 'vsn.ru',
 'sankt-peterburg-pavlovsk.vsn.ru': 'vsn.ru',
 'maloyaroslavets.vsn.ru': 'vsn.ru',
 'mezhdurechensk.vsn.ru': 'vsn.ru',
 'temryuk.vsn.ru': 'vsn.ru',
 'biysk.vsn.ru': 'vsn.ru',
 'kudrovo.vsn.ru': 'vsn.ru',
 'nevinnomyssk.vsn.ru': 'vsn.ru',
 'shahty.vsn.ru': 'vsn.ru',
 'naberezhnye-chelny.vsn.ru': 'vsn.ru',
 'volzhskiy.vsn.ru': 'vsn.ru',
 'egorievsk.vsn.ru': 'vsn.ru',
 'novyy-urengoy.vsn.ru': 'vsn.ru',
 'yurga.vsn.ru': 'vsn.ru',
 'votkinsk.vsn.ru': 'vsn.ru',
 'gorodets.vsn.ru': 'vsn.ru',
 'kasimov.vsn.ru': 'vsn.ru',
 'timashevsk.vsn.ru': 'vsn.ru',
 'kurchatov.vsn.ru': 'vsn.ru',
 'chebarkul.vsn.ru': 'vsn.ru',
 'dubna.vsn.ru': 'vsn.ru',
 'dyurtyuli.vsn.ru': 'vsn.ru',
 'karachaevsk.vsn.ru': 'vsn.ru',
 'usinsk.vsn.ru': 'vsn.ru',
 'ertil.vsn.ru': 'vsn.ru',
 'kolchugino.vsn.ru': 'vsn.ru',
 'pohvistnevo.vsn.ru': 'vsn.ru',
 'kameshkovo.vsn.ru': 'vsn.ru',
 'svetlograd.vsn.ru': 'vsn.ru',
 'tver.vsn.ru': 'vsn.ru',
 'rybnoe.vsn.ru': 'vsn.ru',
 'yakutsk.vsn.ru': 'vsn.ru',
 'plast.vsn.ru': 'vsn.ru',
 'kaltan.vsn.ru': 'vsn.ru',
 'kinel.vsn.ru': 'vsn.ru',
 'teykovo.vsn.ru': 'vsn.ru',
 'usolie-sibirskoe.vsn.ru': 'vsn.ru',
 'bogdanovich.vsn.ru': 'vsn.ru',
 'krasnoturinsk.vsn.ru': 'vsn.ru',
 'bolohovo.vsn.ru': 'vsn.ru',
 'kirishi.vsn.ru': 'vsn.ru',
 'hvalynsk.vsn.ru': 'vsn.ru',
 'klintsy.vsn.ru': 'vsn.ru',
 'kozelsk.vsn.ru': 'vsn.ru',
 'mihaylovka.vsn.ru': 'vsn.ru',
 'zhukovka.vsn.ru': 'vsn.ru',
 'ardon.vsn.ru': 'vsn.ru',
 'gatchina.vsn.ru': 'vsn.ru',
 'shali.vsn.ru': 'vsn.ru',
 'engels.vsn.ru': 'vsn.ru',
 'protvino.vsn.ru': 'vsn.ru',
 'pavlovo.vsn.ru': 'vsn.ru',
 'kumertau.vsn.ru': 'vsn.ru',
 'uray.vsn.ru': 'vsn.ru',
 'kataysk.vsn.ru': 'vsn.ru',
 'chusovoy.vsn.ru': 'vsn.ru',
 'krymsk.vsn.ru': 'vsn.ru',
 'safonovo.vsn.ru': 'vsn.ru',
 'severomorsk.vsn.ru': 'vsn.ru',
 'bratsk.vsn.ru': 'vsn.ru',
 'tihoretsk.vsn.ru': 'vsn.ru',
 'beloretsk.vsn.ru': 'vsn.ru',
 'chernushka.vsn.ru': 'vsn.ru',
 'yartsevo.vsn.ru': 'vsn.ru',
 'langepas.vsn.ru': 'vsn.ru',
 'vsevolozhsk.vsn.ru': 'vsn.ru',
 'blagodarnyy.vsn.ru': 'vsn.ru',
 'chapaevsk.vsn.ru': 'vsn.ru',
 'kamyzyak.vsn.ru': 'vsn.ru',
 'osinniki.vsn.ru': 'vsn.ru',
 'puschino.vsn.ru': 'vsn.ru',
 'kuznetsk.vsn.ru': 'vsn.ru',
 'stavropolskiy-kray-sovetskiy-zelenokumsk.vsn.ru': 'vsn.ru',
 'stupino.vsn.ru': 'vsn.ru',
 'odintsovo.vsn.ru': 'vsn.ru',
 'iskitim.vsn.ru': 'vsn.ru',
 'dimitrovgrad.vsn.ru': 'vsn.ru',
 'mihaylovsk.vsn.ru': 'vsn.ru',
 'novouralsk.vsn.ru': 'vsn.ru',
 'shatura.vsn.ru': 'vsn.ru',
 'arzamas.vsn.ru': 'vsn.ru',
 'nizhnyaya-tura.vsn.ru': 'vsn.ru',
 'aznakaevo.vsn.ru': 'vsn.ru',
 'kaluzhskaya-oblast-kirovskiy-kirov.vsn.ru': 'vsn.ru',
 'otradnyy.vsn.ru': 'vsn.ru',
 'schekino.vsn.ru': 'vsn.ru',
 'angarsk.vsn.ru': 'vsn.ru',
 'petrovsk.vsn.ru': 'vsn.ru',
 'sharya.vsn.ru': 'vsn.ru',
 'buturlinovka.vsn.ru': 'vsn.ru',
 'anzhero-sudzhensk.vsn.ru': 'vsn.ru',
 'ershov.vsn.ru': 'vsn.ru',
 'lyudinovo.vsn.ru': 'vsn.ru',
 'seversk.vsn.ru': 'vsn.ru',
 'novomoskovsk.vsn.ru': 'vsn.ru',
 'irbit.vsn.ru': 'vsn.ru',
 'korenovsk.vsn.ru': 'vsn.ru',
 'zarinsk.vsn.ru': 'vsn.ru',
 'kogalym.vsn.ru': 'vsn.ru',
 'baksan.vsn.ru': 'vsn.ru',
 'salsk.vsn.ru': 'vsn.ru',
 'sharypovo.vsn.ru': 'vsn.ru',
 'sortavala.vsn.ru': 'vsn.ru',
 'belokuriha.vsn.ru': 'vsn.ru',
 'lebedyan.vsn.ru': 'vsn.ru',
 'sosnogorsk.vsn.ru': 'vsn.ru',
 'novoaleksandrovsk.vsn.ru': 'vsn.ru',
 'chernogorsk.vsn.ru': 'vsn.ru',
 'ust-katav.vsn.ru': 'vsn.ru',
 'yuzhnouralsk.vsn.ru': 'vsn.ru',
 'ishim.vsn.ru': 'vsn.ru',
 'kurskaya-oblast-zheleznogorsk.vsn.ru': 'vsn.ru',
 'shuya.vsn.ru': 'vsn.ru',
 'sol-iletsk.vsn.ru': 'vsn.ru',
 'karpinsk.vsn.ru': 'vsn.ru',
 'amurskaya-oblast-belogorsk.vsn.ru': 'vsn.ru',
 'chelyabinskaya-oblast-ozersk.vsn.ru': 'vsn.ru',
 'cherepovets.vsn.ru': 'vsn.ru',
 'verhniy-ufaley.vsn.ru': 'vsn.ru',
 'hasavyurt.vsn.ru': 'vsn.ru',
 'serdobsk.vsn.ru': 'vsn.ru',
 'uzlovaya.vsn.ru': 'vsn.ru',
 'murom.vsn.ru': 'vsn.ru',
 'bugulma.vsn.ru': 'vsn.ru',
 'belovo.vsn.ru': 'vsn.ru',
 'alupka.vsn.ru': 'vsn.ru',
 'nizhneudinsk.vsn.ru': 'vsn.ru',
 'borisoglebsk.vsn.ru': 'vsn.ru',
 'kambarka.vsn.ru': 'vsn.ru',
 'kemerovskaya-oblast-berezovskiy.vsn.ru': 'vsn.ru',
 'rybinsk.vsn.ru': 'vsn.ru',
 'penzenskaya-oblast-zarechnyy.vsn.ru': 'vsn.ru',
 'sayanogorsk.vsn.ru': 'vsn.ru',
 'kulebaki.vsn.ru': 'vsn.ru',
 'segezha.vsn.ru': 'vsn.ru',
 'bogoroditsk.vsn.ru': 'vsn.ru',
 'vorkuta.vsn.ru': 'vsn.ru',
 'dedovsk.vsn.ru': 'vsn.ru',
 'baltiysk.vsn.ru': 'vsn.ru',
 'mozdok.vsn.ru': 'vsn.ru',
 'gorohovets.vsn.ru': 'vsn.ru',
 'nahodka.vsn.ru': 'vsn.ru',
 'elnya.vsn.ru': 'vsn.ru',
 'balabanovo.vsn.ru': 'vsn.ru',
 'bogotol.vsn.ru': 'vsn.ru',
 'trehgornyy.vsn.ru': 'vsn.ru',
 'livny.vsn.ru': 'vsn.ru',
 'elets.vsn.ru': 'vsn.ru',
 'verhnyaya-pyshma.vsn.ru': 'vsn.ru',
 'boguchar.vsn.ru': 'vsn.ru',
 'prokopievsk.vsn.ru': 'vsn.ru',
 'aramil.vsn.ru': 'vsn.ru',
 'klin.vsn.ru': 'vsn.ru',
 'nyagan.vsn.ru': 'vsn.ru',
 'chernogolovka.vsn.ru': 'vsn.ru',
 'kineshma.vsn.ru': 'vsn.ru',
    'gde.ru': 'gde.ru',
 'naro-fominsk.gde.ru': 'gde.ru',
 'tyumen.gde.ru': 'gde.ru',
 'vladivostok.gde.ru': 'gde.ru',
 'tula.gde.ru': 'gde.ru',
 'ivanovo.gde.ru': 'gde.ru',
 'moskva.gde.ru': 'gde.ru',
 'yaroslavl.gde.ru': 'gde.ru',
 'kerch.gde.ru': 'gde.ru',
 'krym.gde.ru': 'gde.ru',
 'sevastopol.gde.ru': 'gde.ru',
 'ekaterinburg.gde.ru': 'gde.ru',
 'nurlat.gde.ru': 'gde.ru',
 'rossosh.gde.ru': 'gde.ru',
 'svobodnyy.gde.ru': 'gde.ru',
 'kazan.gde.ru': 'gde.ru',
 'surgut.gde.ru': 'gde.ru',
 'efremov.gde.ru': 'gde.ru',
 'kurganinsk.gde.ru': 'gde.ru',
 'spb.gde.ru': 'gde.ru',
 'nizhniy-tagil.gde.ru': 'gde.ru',
 'rybinsk.gde.ru': 'gde.ru',
 'cherepovets.gde.ru': 'gde.ru',
 'dolgoprudnyy.gde.ru': 'gde.ru',
 'zelenograd.gde.ru': 'gde.ru',
 'staryy-oskol.gde.ru': 'gde.ru',
 'bugulma.gde.ru': 'gde.ru',
 'millerovo.gde.ru': 'gde.ru',
 'baltiysk.gde.ru': 'gde.ru',
 'sochi.gde.ru': 'gde.ru',
 'svetlogorsk.gde.ru': 'gde.ru',
 'troick.gde.ru': 'gde.ru',
 'krasnodar.gde.ru': 'gde.ru',
 'pavlovskiy-posad.gde.ru': 'gde.ru',
 'gzhel.gde.ru': 'gde.ru',
 'elektrogorsk.gde.ru': 'gde.ru',
 'rzn.gde.ru': 'gde.ru',
 'eysk.gde.ru': 'gde.ru',
 'izhevsk.gde.ru': 'gde.ru',
 'oktyabrskiy.gde.ru': 'gde.ru',
 'norilsk.gde.ru': 'gde.ru',
 'tver.gde.ru': 'gde.ru',
 'lev-tolstoy.gde.ru': 'gde.ru',
 'samara.gde.ru': 'gde.ru',
 'tatarsk.gde.ru': 'gde.ru',
 'novyy-urengoy.gde.ru': 'gde.ru',
 'kinel.gde.ru': 'gde.ru',
 'novomoskovsk.gde.ru': 'gde.ru',
 'tul.gde.ru': 'gde.ru',
 'lobnya.gde.ru': 'gde.ru',
 'kaliningrad.gde.ru': 'gde.ru',
 'polevskoy.gde.ru': 'gde.ru',
 'rostov-na-donu.gde.ru': 'gde.ru',
 'zheleznodorozhnyy.gde.ru': 'gde.ru',
 'nahodka.gde.ru': 'gde.ru',
 'volgograd.gde.ru': 'gde.ru',
 'naryan-mar.gde.ru': 'gde.ru',
 'kamyshlov.gde.ru': 'gde.ru',
 'tuymazy.gde.ru': 'gde.ru',
 'kingisepp.gde.ru': 'gde.ru',
 'chelyabinsk.gde.ru': 'gde.ru',
 'beryozovskiy.gde.ru': 'gde.ru',
 'chusovoy.gde.ru': 'gde.ru',
 'tynda.gde.ru': 'gde.ru',
 'saratov.gde.ru': 'gde.ru',
 'izobilnyj.gde.ru': 'gde.ru',
 'novokubansk.gde.ru': 'gde.ru',
 'armavir.gde.ru': 'gde.ru',
 'naberejnyie-chelnyi.gde.ru': 'gde.ru',
 'adler.gde.ru': 'gde.ru',
 'velikie-luki.gde.ru': 'gde.ru',
 'gorno-altajsk.gde.ru': 'gde.ru',
 'kumertau.gde.ru': 'gde.ru',
 'kalach.gde.ru': 'gde.ru',
 'bogdanovich.gde.ru': 'gde.ru',
 'taganrog.gde.ru': 'gde.ru',
 'bataysk.gde.ru': 'gde.ru',
 'lyubercy.gde.ru': 'gde.ru',
 'taldom.gde.ru': 'gde.ru',
 'krasnoyarsk.gde.ru': 'gde.ru',
 'kostroma.gde.ru': 'gde.ru',
 'vladimir.gde.ru': 'gde.ru',
 'novosibirsk.gde.ru': 'gde.ru',
 'chehov.gde.ru': 'gde.ru',
 'belogorsk.gde.ru': 'gde.ru',
 'balakovo.gde.ru': 'gde.ru',
 'petrovsk-zabaykalskiy.gde.ru': 'gde.ru',
 'komsomolsk-na-amure.gde.ru': 'gde.ru',
 'sergiev-posad.gde.ru': 'gde.ru',
 'ufa.gde.ru': 'gde.ru',
 'porhov.gde.ru': 'gde.ru',
 'cheboksary.gde.ru': 'gde.ru',
 'pilna.gde.ru': 'gde.ru',
 'novozybkov.gde.ru': 'gde.ru',
 'lipeck.gde.ru': 'gde.ru',
 'tolyatti.gde.ru': 'gde.ru',
 'bavly.gde.ru': 'gde.ru',
 'elabuga.gde.ru': 'gde.ru',
 'nizhnekamsk.gde.ru': 'gde.ru',
 'nahabino.gde.ru': 'gde.ru',
 'odincovo.gde.ru': 'gde.ru',
 'krasnoznamensk.gde.ru': 'gde.ru',
 'pyt-yah.gde.ru': 'gde.ru',
 'anapa.gde.ru': 'gde.ru',
 'orenburg.gde.ru': 'gde.ru',
 'borskoe.gde.ru': 'gde.ru',
 'chapaevsk.gde.ru': 'gde.ru',
 'velikiy-novgorod.gde.ru': 'gde.ru',
 'artemovskiy.gde.ru': 'gde.ru',
 'mosobl.gde.ru': 'gde.ru',
 'miass.gde.ru': 'gde.ru',
 'nn.gde.ru': 'gde.ru',
 'iskitim.gde.ru': 'gde.ru',
 'omsk.gde.ru': 'gde.ru',
 'novorossiysk.gde.ru': 'gde.ru',
 'kuban.gde.ru': 'gde.ru',
 'nikolskoe.gde.ru': 'gde.ru',
 'engels.gde.ru': 'gde.ru',
 'novopavlovsk.gde.ru': 'gde.ru',
 'sherbinka.gde.ru': 'gde.ru',
 'domodedovo.gde.ru': 'gde.ru',
 'belebey.gde.ru': 'gde.ru',
 'korenovsk.gde.ru': 'gde.ru',
 'spbobl.gde.ru': 'gde.ru',
 'ryazan.gde.ru': 'gde.ru',
 'almetevsk.gde.ru': 'gde.ru',
 'labinsk.gde.ru': 'gde.ru',
 'neryungri.gde.ru': 'gde.ru',
 'chel.gde.ru': 'gde.ru',
 'kansk.gde.ru': 'gde.ru',
 'habarovsk.gde.ru': 'gde.ru',
 'timashevsk.gde.ru': 'gde.ru',
 'elec.gde.ru': 'gde.ru',
 'groznyy.gde.ru': 'gde.ru',
 'nizhnyaya-tavda.gde.ru': 'gde.ru',
 'bessonovka.gde.ru': 'gde.ru',
 'zhukovskiy.gde.ru': 'gde.ru',
 'tomsk.gde.ru': 'gde.ru',
 'voronezh.gde.ru': 'gde.ru',
 'sibay.gde.ru': 'gde.ru',
 'dmitrov.gde.ru': 'gde.ru',
 'salehard.gde.ru': 'gde.ru',
 'novokuznetsk.gde.ru': 'gde.ru',
 'severodvinsk.gde.ru': 'gde.ru',
 'udomlya.gde.ru': 'gde.ru',
 'novotroick.gde.ru': 'gde.ru',
 'orsk.gde.ru': 'gde.ru',
 'alt.gde.ru': 'gde.ru',
 'kushnarenkovo.gde.ru': 'gde.ru',
 'perm.gde.ru': 'gde.ru',
 'nnov.gde.ru': 'gde.ru',
 'kopeysk.gde.ru': 'gde.ru',
 'irbit.gde.ru': 'gde.ru',
 'petropavlovsk-kamchatskiy.gde.ru': 'gde.ru',
 'isilkul.gde.ru': 'gde.ru',
 'serpuhov.gde.ru': 'gde.ru',
 'smr.gde.ru': 'gde.ru',
 'otradnyy.gde.ru': 'gde.ru',
 'semenov.gde.ru': 'gde.ru',
 'chernushka.gde.ru': 'gde.ru',
 'neftekamsk.gde.ru': 'gde.ru',
 'selyatino.gde.ru': 'gde.ru',
 'shatura.gde.ru': 'gde.ru',
 'kemerovo.gde.ru': 'gde.ru',
 'torzhok.gde.ru': 'gde.ru',
 'nizhnevartovsk.gde.ru': 'gde.ru',
 'magnitogorsk.gde.ru': 'gde.ru',
 'osinniki.gde.ru': 'gde.ru',
 'kaltan.gde.ru': 'gde.ru',
 'sestroreck.gde.ru': 'gde.ru',
 'shahty.gde.ru': 'gde.ru',
 'nikolaevsk-na-amure.gde.ru': 'gde.ru',
 'gatchina.gde.ru': 'gde.ru',
 'pavlovskaja.gde.ru': 'gde.ru',
 'tbilisskaya.gde.ru': 'gde.ru',
 'novopokrovskaya.gde.ru': 'gde.ru',
 'abinsk.gde.ru': 'gde.ru',
 'oktyabrskaya.gde.ru': 'gde.ru',
 'sharya.gde.ru': 'gde.ru',
 'slavyanka.gde.ru': 'gde.ru',
 'stupino.gde.ru': 'gde.ru',
 'kolomna.gde.ru': 'gde.ru',
 'berdsk.gde.ru': 'gde.ru',
 'dzerzhinskiy.gde.ru': 'gde.ru',
 'salavat.gde.ru': 'gde.ru',
 'belaya-kalitva.gde.ru': 'gde.ru',
 'krasnousolskiy.gde.ru': 'gde.ru',
 'rodniki.gde.ru': 'gde.ru',
 'arsenev.gde.ru': 'gde.ru',
 'verhoture.gde.ru': 'gde.ru',
 'buguruslan.gde.ru': 'gde.ru',
 'mednogorsk.gde.ru': 'gde.ru',
 'shahunya.gde.ru': 'gde.ru',
 'blagoveshchensk.gde.ru': 'gde.ru',
 'asha.gde.ru': 'gde.ru',
 'ardon.gde.ru': 'gde.ru',
 'kronshtadt.gde.ru': 'gde.ru',
 'temryuk.gde.ru': 'gde.ru',
 'lunino.gde.ru': 'gde.ru',
 'maykop.gde.ru': 'gde.ru',
 'yoshkar-ola.gde.ru': 'gde.ru',
 'elektrostal.gde.ru': 'gde.ru',
 'myski.gde.ru': 'gde.ru',
 'kirov.gde.ru': 'gde.ru',
 'smolensk.gde.ru': 'gde.ru',
 'slavyansk-na-kubani.gde.ru': 'gde.ru',
 'abaza.gde.ru': 'gde.ru',
 'leningradskaja.gde.ru': 'gde.ru',
 'pereslavl-zalesskiy.gde.ru': 'gde.ru',
 'shushenskoe.gde.ru': 'gde.ru',
 'novokuybyshevsk.gde.ru': 'gde.ru',
 'kushevskaya.gde.ru': 'gde.ru',
 'kurgan.gde.ru': 'gde.ru',
 'bagan.gde.ru': 'gde.ru',
 'yuzhno-sahalinsk.gde.ru': 'gde.ru',
 'partizansk.gde.ru': 'gde.ru',
 'petrozavodsk.gde.ru': 'gde.ru',
 'krasnoufimsk.gde.ru': 'gde.ru',
 'ust-kut.gde.ru': 'gde.ru',
 'kushva.gde.ru': 'gde.ru',
 'novoaltaysk.gde.ru': 'gde.ru',
 'nevinnomyssk.gde.ru': 'gde.ru',
 'ramenskoe.gde.ru': 'gde.ru',
 'uzhur.gde.ru': 'gde.ru',
 'shimanovsk.gde.ru': 'gde.ru',
 'nsk.gde.ru': 'gde.ru',
 'serov.gde.ru': 'gde.ru',
 'arti.gde.ru': 'gde.ru',
 'podolsk.gde.ru': 'gde.ru',
 'chernomorskoe.gde.ru': 'gde.ru',
 'vyazniki.gde.ru': 'gde.ru',
 'tihvin.gde.ru': 'gde.ru',
 'buzuluk.gde.ru': 'gde.ru',
 'sorochinsk.gde.ru': 'gde.ru',
 'satka.gde.ru': 'gde.ru',
 'gukovo.gde.ru': 'gde.ru',
 'essentuki.gde.ru': 'gde.ru',
 'kropotkin.gde.ru': 'gde.ru',
 'vyshniy-volochek.gde.ru': 'gde.ru',
 'mozhaysk.gde.ru': 'gde.ru',
 'dzhubga.gde.ru': 'gde.ru',
 'zelenogradsk.gde.ru': 'gde.ru',
 'arzamas.gde.ru': 'gde.ru',
 'sosnovyy-bor.gde.ru': 'gde.ru',
 'nikolaevka.gde.ru': 'gde.ru',
 'djurtjuli.gde.ru': 'gde.ru',
 'minusinsk.gde.ru': 'gde.ru',
 'kolpashevo.gde.ru': 'gde.ru',
 'chemal.gde.ru': 'gde.ru',
 'zarechnyi.gde.ru': 'gde.ru',
 'bryuhoveckaya.gde.ru': 'gde.ru',
 'tobolsk.gde.ru': 'gde.ru',
 'rostobl.gde.ru': 'gde.ru',
 'stavropol.gde.ru': 'gde.ru',
 'cherepanovo.gde.ru': 'gde.ru',
 'aktanysh.gde.ru': 'gde.ru',
 'sakh.gde.ru': 'gde.ru',
 'kamensk-shahtinskiy.gde.ru': 'gde.ru',
 'mariinsk.gde.ru': 'gde.ru',
 'murmansk.gde.ru': 'gde.ru',
 'megion.gde.ru': 'gde.ru',
 'langepas.gde.ru': 'gde.ru',
 'rezh.gde.ru': 'gde.ru',
 'ingushetia.gde.ru': 'gde.ru',
 'nazran.gde.ru': 'gde.ru',
 'klimovsk.gde.ru': 'gde.ru',
 'voskresensk.gde.ru': 'gde.ru',
 'kurovskoe.gde.ru': 'gde.ru',
 'pushino.gde.ru': 'gde.ru',
 'prokopevsk.gde.ru': 'gde.ru',
 'kirzhach.gde.ru': 'gde.ru',
 'novocherkassk.gde.ru': 'gde.ru',
 'kirowsk.gde.ru': 'gde.ru',
 'arhangelsk.gde.ru': 'gde.ru',
 'kislovodsk.gde.ru': 'gde.ru',
 'yakutsk.gde.ru': 'gde.ru',
 'isetskoe.gde.ru': 'gde.ru',
 'kasimov.gde.ru': 'gde.ru',
 'georgievsk.gde.ru': 'gde.ru',
 'budennovsk.gde.ru': 'gde.ru',
 'lakinsk.gde.ru': 'gde.ru',
 'vla.gde.ru': 'gde.ru',
 'yalta.gde.ru': 'gde.ru',
 'mordovo.gde.ru': 'gde.ru',
 'losino-petrovskiy.gde.ru': 'gde.ru',
 'kalininskaya.gde.ru': 'gde.ru',
 'pskov.gde.ru': 'gde.ru',
 'tuapse.gde.ru': 'gde.ru',
 'apsheronsk.gde.ru': 'gde.ru',
 'nefteyugansk.gde.ru': 'gde.ru',
 'gelendzhik.gde.ru': 'gde.ru',
 'pospeliha.gde.ru': 'gde.ru',
 'uchaly.gde.ru': 'gde.ru',
 'prohladnyy.gde.ru': 'gde.ru',
 'furmanov.gde.ru': 'gde.ru',
 'ishimbay.gde.ru': 'gde.ru',
 'noginsk.gde.ru': 'gde.ru',
 'bezheck.gde.ru': 'gde.ru',
 'himki.gde.ru': 'gde.ru',
 'sebezh.gde.ru': 'gde.ru',
 'kamensk-uralskiy.gde.ru': 'gde.ru',
 'udm.gde.ru': 'gde.ru',
 'belgorod.gde.ru': 'gde.ru',
 'mytishi.gde.ru': 'gde.ru',
 'alikovo.gde.ru': 'gde.ru',
 'chuvash.gde.ru': 'gde.ru',
 'orel.gde.ru': 'gde.ru',
 'podporozhe.gde.ru': 'gde.ru',
 'aldan.gde.ru': 'gde.ru',
 'uray.gde.ru': 'gde.ru',
 'shadrinsk.gde.ru': 'gde.ru',
 'saraktash.gde.ru': 'gde.ru',
 'kuvandyk.gde.ru': 'gde.ru',
 'tyulgan.gde.ru': 'gde.ru',
 'novouralsk.gde.ru': 'gde.ru',
 'zernograd.gde.ru': 'gde.ru',
 'tihoreck.gde.ru': 'gde.ru',
 'nizhniy-bestyah.gde.ru': 'gde.ru',
 'levokumskoe.gde.ru': 'gde.ru',
 'ruzaevka.gde.ru': 'gde.ru',
 'saransk.gde.ru': 'gde.ru',
 'vacha.gde.ru': 'gde.ru',
 'pavlovo.gde.ru': 'gde.ru',
 'kungur.gde.ru': 'gde.ru',
 'kirow.gde.ru': 'gde.ru',
 'amursk.gde.ru': 'gde.ru',
 'pechora.gde.ru': 'gde.ru',
 'ussuriysk.gde.ru': 'gde.ru',
 'severskaya.gde.ru': 'gde.ru',
 'maryanskaya.gde.ru': 'gde.ru',
 'dobroe.gde.ru': 'gde.ru',
 'uglegorsk.gde.ru': 'gde.ru',
 'ahtubinsk.gde.ru': 'gde.ru',
 'chebarkul.gde.ru': 'gde.ru',
 'petushki.gde.ru': 'gde.ru',
 'simferopol.gde.ru': 'gde.ru',
 'krymsk.gde.ru': 'gde.ru',
 'pushkin.gde.ru': 'gde.ru',
 'zubcov.gde.ru': 'gde.ru',
 'tosno.gde.ru': 'gde.ru',
 'vyborg.gde.ru': 'gde.ru',
 'kras.gde.ru': 'gde.ru',
 'vanino.gde.ru': 'gde.ru',
 'vyazma.gde.ru': 'gde.ru',
 'sayanogorsk.gde.ru': 'gde.ru',
 'kogalym.gde.ru': 'gde.ru',
 'velikiy-ustyug.gde.ru': 'gde.ru',
 'frolovo.gde.ru': 'gde.ru',
 'revda.gde.ru': 'gde.ru',
 'kaluga.gde.ru': 'gde.ru',
 'belousovo.gde.ru': 'gde.ru',
 'solnechnogorsk.gde.ru': 'gde.ru',
 'blagoveshensk.gde.ru': 'gde.ru',
 'birsk.gde.ru': 'gde.ru',
 'spas-klepiki.gde.ru': 'gde.ru',
 'udachnyy.gde.ru': 'gde.ru',
 'chita.gde.ru': 'gde.ru',
 'dzerzhinsk.gde.ru': 'gde.ru',
 'vyatskie-polyany.gde.ru': 'gde.ru',
 'vladikavkaz.gde.ru': 'gde.ru',
 'kremyonki.gde.ru': 'gde.ru',
 'aleksin.gde.ru': 'gde.ru',
 'proletarsk.gde.ru': 'gde.ru',
 'amur.gde.ru': 'gde.ru',
 'sharkan.gde.ru': 'gde.ru',
 'bakhchisaray.gde.ru': 'gde.ru',
 'pervouralsk.gde.ru': 'gde.ru',
 'kovylkino.gde.ru': 'gde.ru',
 'abakan.gde.ru': 'gde.ru',
 'sharypovo.gde.ru': 'gde.ru',
 'buturlinovka.gde.ru': 'gde.ru',
 'vsevolozhsk.gde.ru': 'gde.ru',
 'karaidel.gde.ru': 'gde.ru',
 'primorsko-ahtarsk.gde.ru': 'gde.ru',
 'vrn.gde.ru': 'gde.ru',
 'iglino.gde.ru': 'gde.ru',
 'degtyarsk.gde.ru': 'gde.ru',
 'kuybyshev.gde.ru': 'gde.ru',
 'mineralnye-vody.gde.ru': 'gde.ru',
 'pyatigorsk.gde.ru': 'gde.ru',
 'vologda.gde.ru': 'gde.ru',
 'kursk.gde.ru': 'gde.ru',
 'bryansk.gde.ru': 'gde.ru',
 'neftegorsk.gde.ru': 'gde.ru',
 'karachev.gde.ru': 'gde.ru',
 'pikalyovo.gde.ru': 'gde.ru',
 'glushkovo.gde.ru': 'gde.ru',
 'spassk-ryazanskiy.gde.ru': 'gde.ru',
 'vlgd.gde.ru': 'gde.ru',
 'aleksandrov.gde.ru': 'gde.ru',
 'usinsk.gde.ru': 'gde.ru',
 'ulan-ude.gde.ru': 'gde.ru',
 'syzran.gde.ru': 'gde.ru',
 'aprelevka.gde.ru': 'gde.ru',
 'novoshahtinsk.gde.ru': 'gde.ru',
 'biysk.gde.ru': 'gde.ru',
 'yuzhnouralsk.gde.ru': 'gde.ru',
 'stavr.gde.ru': 'gde.ru',
 'fryazino.gde.ru': 'gde.ru',
 'syktyvkar.gde.ru': 'gde.ru',
 'beloyarskii.gde.ru': 'gde.ru',
 'luchegorsk.gde.ru': 'gde.ru',
 'verhniy-ufaley.gde.ru': 'gde.ru',
 'beloreck.gde.ru': 'gde.ru',
 'cherdakly.gde.ru': 'gde.ru',
 'uglich.gde.ru': 'gde.ru',
 'hmao.gde.ru': 'gde.ru',
 'raduzhnyi.gde.ru': 'gde.ru',
 'konakovo.gde.ru': 'gde.ru',
 'anzhero-sudzhensk.gde.ru': 'gde.ru',
 'yurga.gde.ru': 'gde.ru',
 'otradnaya.gde.ru': 'gde.ru',
 'sarapul.gde.ru': 'gde.ru',
 'davlekanovo.gde.ru': 'gde.ru',
 'gavrilov-posad.gde.ru': 'gde.ru',
 'kubinka.gde.ru': 'gde.ru',
 'kyzyl.gde.ru': 'gde.ru',
 'chaykovskiy.gde.ru': 'gde.ru',
 'ulyanovsk.gde.ru': 'gde.ru',
 'penza.gde.ru': 'gde.ru',
 'meleuz.gde.ru': 'gde.ru',
 'kalug.gde.ru': 'gde.ru',
 'balabanovo.gde.ru': 'gde.ru',
 'kimry.gde.ru': 'gde.ru',
 'dubna.gde.ru': 'gde.ru',
 'tambov.gde.ru': 'gde.ru',
 'verhnyaya-pyshma.gde.ru': 'gde.ru',
 'bazarniy-karabulak.gde.ru': 'gde.ru',
 'marks.gde.ru': 'gde.ru',
 'pushkino.gde.ru': 'gde.ru',
 'petergof.gde.ru': 'gde.ru',
 'zeya.gde.ru': 'gde.ru',
 'verhnyaya-salda.gde.ru': 'gde.ru',
 'irkutsk.gde.ru': 'gde.ru',
 'poltavskaya.gde.ru': 'gde.ru',
 'kovrov.gde.ru': 'gde.ru',
 'verhneuralsk.gde.ru': 'gde.ru',
 'dinskaja.gde.ru': 'gde.ru',
 'zavolzhe.gde.ru': 'gde.ru',
 'nevyansk.gde.ru': 'gde.ru',
 'bolshoy-kamen.gde.ru': 'gde.ru',
 'ilek.gde.ru': 'gde.ru',
 'rodionovo-nesvetayskaya.gde.ru': 'gde.ru',
 'sovetsk.gde.ru': 'gde.ru',
 'shuya.gde.ru': 'gde.ru',
 'sovetskaya-gavan.gde.ru': 'gde.ru',
 'lesosibirsk.gde.ru': 'gde.ru',
 'nalchik.gde.ru': 'gde.ru',
 'egorevsk.gde.ru': 'gde.ru',
 'kanevskaja.gde.ru': 'gde.ru',
 'kolpino.gde.ru': 'gde.ru',
 'asbest.gde.ru': 'gde.ru',
 'rybnaya-sloboda.gde.ru': 'gde.ru',
 'dedovsk.gde.ru': 'gde.ru',
 'istra.gde.ru': 'gde.ru',
 'kirovo-chepeck.gde.ru': 'gde.ru',
 'vidnoe.gde.ru': 'gde.ru',
 'balashov.gde.ru': 'gde.ru',
 'volhov.gde.ru': 'gde.ru',
 'kireevsk.gde.ru': 'gde.ru',
 'gulkevichi.gde.ru': 'gde.ru',
 'priozersk.gde.ru': 'gde.ru',
 'katav-ivanovsk.gde.ru': 'gde.ru',
 'redkino.gde.ru': 'gde.ru',
 'zlatoust.gde.ru': 'gde.ru',
 'nov.gde.ru': 'gde.ru',
 'zavodoukovsk.gde.ru': 'gde.ru',
 'balahna.gde.ru': 'gde.ru',
 'reutov.gde.ru': 'gde.ru',
 'sokolskoe.gde.ru': 'gde.ru',
 'kozelsk.gde.ru': 'gde.ru',
 'belokuriha.gde.ru': 'gde.ru',
 'dyatkovo.gde.ru': 'gde.ru',
 'ust-kishert.gde.ru': 'gde.ru',
 'sim.gde.ru': 'gde.ru',
 'minyar.gde.ru': 'gde.ru',
 'ekaterinovka.gde.ru': 'gde.ru',
 'lensk.gde.ru': 'gde.ru',
 'ust-labinsk.gde.ru': 'gde.ru',
 'kamyshin.gde.ru': 'gde.ru',
 'rzhaksa.gde.ru': 'gde.ru',
 'kirsanov.gde.ru': 'gde.ru',
 'kstovo.gde.ru': 'gde.ru',
 'severo-eniseyskiy.gde.ru': 'gde.ru',
 'bash.gde.ru': 'gde.ru',
 'tutaev.gde.ru': 'gde.ru',
 'yar.gde.ru': 'gde.ru',
 'rostov.gde.ru': 'gde.ru',
 'ostrogozhsk.gde.ru': 'gde.ru',
 'sysert.gde.ru': 'gde.ru',
 'novomichurinsk.gde.ru': 'gde.ru',
 'dombarovskiy.gde.ru': 'gde.ru',
 'shekino.gde.ru': 'gde.ru',
 'holmsk.gde.ru': 'gde.ru',
 'vysokovsk.gde.ru': 'gde.ru',
 'khv.gde.ru': 'gde.ru',
 'roslavl.gde.ru': 'gde.ru',
 'verhnyaya-toyma.gde.ru': 'gde.ru',
    'superjob.ru': 'superjob.ru',
 'naro-fominsk.superjob.ru': 'superjob.ru',
 'novocheboksarsk.superjob.ru': 'superjob.ru',
 'ekaterinburg.superjob.ru': 'superjob.ru',
 'russia.superjob.ru': 'superjob.ru',
 'kursk.superjob.ru': 'superjob.ru',
 'ufa.superjob.ru': 'superjob.ru',
 'voronezh.superjob.ru': 'superjob.ru',
 'kolomna.superjob.ru': 'superjob.ru',
 'spb.superjob.ru': 'superjob.ru',
 'kirov.superjob.ru': 'superjob.ru',
 'novokuybyshevsk.superjob.ru': 'superjob.ru',
 'ryazan.superjob.ru': 'superjob.ru',
 'nsk.superjob.ru': 'superjob.ru',
 'tyumen.superjob.ru': 'superjob.ru',
 'krasnoyarsk.superjob.ru': 'superjob.ru',
 'tm.superjob.ru': 'superjob.ru',
 'volgograd.superjob.ru': 'superjob.ru',
 'scherbinka.superjob.ru': 'superjob.ru',
 'elektrostal.superjob.ru': 'superjob.ru',
 'rnd.superjob.ru': 'superjob.ru',
 'izhevsk.superjob.ru': 'superjob.ru',
 'krasnodar.superjob.ru': 'superjob.ru',
 'magadan.superjob.ru': 'superjob.ru',
 'kotelnikovo.superjob.ru': 'superjob.ru',
 'lipetsk.superjob.ru': 'superjob.ru',
 'kaluga.superjob.ru': 'superjob.ru',
 'yaroslavl.superjob.ru': 'superjob.ru',
 'segezha.superjob.ru': 'superjob.ru',
 'kemerovo.superjob.ru': 'superjob.ru',
 'vidnoe.superjob.ru': 'superjob.ru',
 'kurgan.superjob.ru': 'superjob.ru',
 'bashkortostan.superjob.ru': 'superjob.ru',
 'podolsk.superjob.ru': 'superjob.ru',
 'norilsk.superjob.ru': 'superjob.ru',
 'habarovsk.superjob.ru': 'superjob.ru',
 'penza.superjob.ru': 'superjob.ru',
 'sevastopol.superjob.ru': 'superjob.ru',
 'nizhny-tagil.superjob.ru': 'superjob.ru',
 'slavyansk-na-kubani.superjob.ru': 'superjob.ru',
 'temryuk.superjob.ru': 'superjob.ru',
 'nn.superjob.ru': 'superjob.ru',
 'cheboksary.superjob.ru': 'superjob.ru',
 'volzhskiy.superjob.ru': 'superjob.ru',
 'omsk.superjob.ru': 'superjob.ru',
 'himki.superjob.ru': 'superjob.ru',
 'kazan.superjob.ru': 'superjob.ru',
 'samara.superjob.ru': 'superjob.ru',
 'serpukhov.superjob.ru': 'superjob.ru',
 'ulyanovsk.superjob.ru': 'superjob.ru',
 'obninsk.superjob.ru': 'superjob.ru',
 'perm.superjob.ru': 'superjob.ru',
 'saratov.superjob.ru': 'superjob.ru',
 'chelyabinsk.superjob.ru': 'superjob.ru',
 'solnechnogorsk.superjob.ru': 'superjob.ru',
 'komsomolsk-na-amure.superjob.ru': 'superjob.ru',
 'dalnerechensk.superjob.ru': 'superjob.ru',
 'blagoveshensk.superjob.ru': 'superjob.ru',
 'krym.superjob.ru': 'superjob.ru',
 'novyi-urengoi.superjob.ru': 'superjob.ru',
 'murmansk.superjob.ru': 'superjob.ru',
 'belogorsk.superjob.ru': 'superjob.ru',
 'tuapse.superjob.ru': 'superjob.ru',
 'belgorod.superjob.ru': 'superjob.ru',
 'orenburg.superjob.ru': 'superjob.ru',
 'pyt-yah.superjob.ru': 'superjob.ru',
 'troitsk.superjob.ru': 'superjob.ru',
 'mo.superjob.ru': 'superjob.ru',
 'kommunarka.superjob.ru': 'superjob.ru',
 'korolev.superjob.ru': 'superjob.ru',
 'sochi.superjob.ru': 'superjob.ru',
 'tomsk.superjob.ru': 'superjob.ru',
 'abakan.superjob.ru': 'superjob.ru',
 'syktyvkar.superjob.ru': 'superjob.ru',
 'mihajlovsk.superjob.ru': 'superjob.ru',
 'magnitogorsk.superjob.ru': 'superjob.ru',
 'sibay.superjob.ru': 'superjob.ru',
 'pechora.superjob.ru': 'superjob.ru',
 'kaliningrad.superjob.ru': 'superjob.ru',
 'novoshakhtinsk.superjob.ru': 'superjob.ru',
 'sertolovo.superjob.ru': 'superjob.ru',
 'usinsk.superjob.ru': 'superjob.ru',
 'tobolsk.superjob.ru': 'superjob.ru',
 'saransk.superjob.ru': 'superjob.ru',
 'cherepovets.superjob.ru': 'superjob.ru',
 'barnaul.superjob.ru': 'superjob.ru',
 'tihoretsk.superjob.ru': 'superjob.ru',
 'petrozavodsk.superjob.ru': 'superjob.ru',
 'schelkovo.superjob.ru': 'superjob.ru',
 'ramenskoe.superjob.ru': 'superjob.ru',
 'miass.superjob.ru': 'superjob.ru',
 'volgodonsk.superjob.ru': 'superjob.ru',
 'maikop.superjob.ru': 'superjob.ru',
 'vladivostok.superjob.ru': 'superjob.ru',
 'chita.superjob.ru': 'superjob.ru',
 'zelenograd.superjob.ru': 'superjob.ru',
 'astrakhan.superjob.ru': 'superjob.ru',
 'balashiha.superjob.ru': 'superjob.ru',
 'irbit.superjob.ru': 'superjob.ru',
 'ulan-ude.superjob.ru': 'superjob.ru',
 'lyubertsy.superjob.ru': 'superjob.ru',
 'balakovo.superjob.ru': 'superjob.ru',
 'neryungri.superjob.ru': 'superjob.ru',
 'sterlitamak.superjob.ru': 'superjob.ru',
 'svobodniy.superjob.ru': 'superjob.ru',
 'velikie-luki.superjob.ru': 'superjob.ru',
 'voskresensk.superjob.ru': 'superjob.ru',
 'vladimir.superjob.ru': 'superjob.ru',
 'zvenigorod.superjob.ru': 'superjob.ru',
 'surgut.superjob.ru': 'superjob.ru',
 'meleuz.superjob.ru': 'superjob.ru',
 'zaraysk.superjob.ru': 'superjob.ru',
 'tsivilsk.superjob.ru': 'superjob.ru',
 'dedovsk.superjob.ru': 'superjob.ru',
 'tambov.superjob.ru': 'superjob.ru',
 'mytischi.superjob.ru': 'superjob.ru',
 'naberezhnye-chelny.superjob.ru': 'superjob.ru',
 'yubilejnyj.superjob.ru': 'superjob.ru',
 'chehov.superjob.ru': 'superjob.ru',
 'khanty-mansiisk.superjob.ru': 'superjob.ru',
 'irkutskaya-oblast.superjob.ru': 'superjob.ru',
 'zheleznodorozhniy.superjob.ru': 'superjob.ru',
 'borodino.superjob.ru': 'superjob.ru',
 'sharypovo.superjob.ru': 'superjob.ru',
 'smolensk.superjob.ru': 'superjob.ru',
 'ivanovo.superjob.ru': 'superjob.ru',
 'kurganinsk.superjob.ru': 'superjob.ru',
 'konakovo.superjob.ru': 'superjob.ru',
 'nizhnekamsk.superjob.ru': 'superjob.ru',
 'novokuznetsk.superjob.ru': 'superjob.ru',
 'tolyatti.superjob.ru': 'superjob.ru',
 'vologda.superjob.ru': 'superjob.ru',
 'nizhnevartovsk.superjob.ru': 'superjob.ru',
 'irkutsk.superjob.ru': 'superjob.ru',
 'yakutsk.superjob.ru': 'superjob.ru',
 'kyzyl.superjob.ru': 'superjob.ru',
 'labinsk.superjob.ru': 'superjob.ru',
 'komi.superjob.ru': 'superjob.ru',
 'lukhovitsy.superjob.ru': 'superjob.ru',
 'grozniy.superjob.ru': 'superjob.ru',
 'zhukovskiy.superjob.ru': 'superjob.ru',
 'tula.superjob.ru': 'superjob.ru',
 'ioshkar-ola.superjob.ru': 'superjob.ru',
 'orel.superjob.ru': 'superjob.ru',
 'kovrov.superjob.ru': 'superjob.ru',
 'rostov.superjob.ru': 'superjob.ru',
 'novocherkassk.superjob.ru': 'superjob.ru',
 'odintsovo.superjob.ru': 'superjob.ru',
 'staraya-kupavna.superjob.ru': 'superjob.ru',
 'tver.superjob.ru': 'superjob.ru',
 'stupino.superjob.ru': 'superjob.ru',
 'nevinnomyssk.superjob.ru': 'superjob.ru',
 'uhta.superjob.ru': 'superjob.ru',
 'am.superjob.ru': 'superjob.ru',
 'pushkino.superjob.ru': 'superjob.ru',
 'yuzhno-sakhalinsk.superjob.ru': 'superjob.ru',
 'domodedovo.superjob.ru': 'superjob.ru',
 'ishimbay.superjob.ru': 'superjob.ru',
 'goryachij-klyuch.superjob.ru': 'superjob.ru',
 'ivanteevka.superjob.ru': 'superjob.ru',
 'kamensk-uralskiy.superjob.ru': 'superjob.ru',
 'syzran.superjob.ru': 'superjob.ru',
 'almetyevsk.superjob.ru': 'superjob.ru',
 'almaty-kz.superjob.ru': 'superjob.ru',
 'pskov.superjob.ru': 'superjob.ru',
 'chernushka.superjob.ru': 'superjob.ru',
 'kronshtadt.superjob.ru': 'superjob.ru',
 'dolgoprudniy.superjob.ru': 'superjob.ru',
 'pavlovo.superjob.ru': 'superjob.ru',
 'zheleznogorsk-kk.superjob.ru': 'superjob.ru',
 'kamyshin.superjob.ru': 'superjob.ru',
 'inta.superjob.ru': 'superjob.ru',
 'yelets.superjob.ru': 'superjob.ru',
 'vnukovo.superjob.ru': 'superjob.ru',
 'kubinka.superjob.ru': 'superjob.ru',
 'kropotkin.superjob.ru': 'superjob.ru',
 'serov.superjob.ru': 'superjob.ru',
 'dzerzhinsk.superjob.ru': 'superjob.ru',
 'koryazhma.superjob.ru': 'superjob.ru',
 'aldan.superjob.ru': 'superjob.ru',
 'aksaj.superjob.ru': 'superjob.ru',
 'orehovo-zuevo.superjob.ru': 'superjob.ru',
 'ust-labinsk.superjob.ru': 'superjob.ru',
 'krasnoturinsk.superjob.ru': 'superjob.ru',
 'vladikavkaz.superjob.ru': 'superjob.ru',
 'egorievsk.superjob.ru': 'superjob.ru',
 'gelendzhik.superjob.ru': 'superjob.ru',
 'vorkuta.superjob.ru': 'superjob.ru',
 'mcensk.superjob.ru': 'superjob.ru',
 'mirny.superjob.ru': 'superjob.ru',
 'novgorod.superjob.ru': 'superjob.ru',
 'noyabrsk.superjob.ru': 'superjob.ru',
 'leningradskaya-oblast.superjob.ru': 'superjob.ru',
 'chernogorsk.superjob.ru': 'superjob.ru',
 'armavir.superjob.ru': 'superjob.ru',
 'noginsk.superjob.ru': 'superjob.ru',
 'venev.superjob.ru': 'superjob.ru',
 'verhnyaya-pishma.superjob.ru': 'superjob.ru',
 'usolje-sibirskoe.superjob.ru': 'superjob.ru',
 'protvino.superjob.ru': 'superjob.ru',
 'zlatoust.superjob.ru': 'superjob.ru',
 'anapa.superjob.ru': 'superjob.ru',
 'lakinsk.superjob.ru': 'superjob.ru',
 'tikhvin.superjob.ru': 'superjob.ru',
 'kirov-kaluga.superjob.ru': 'superjob.ru',
 'krasnogorsk.superjob.ru': 'superjob.ru',
 'mineralnye-vody.superjob.ru': 'superjob.ru',
 'suzdal.superjob.ru': 'superjob.ru',
 'yeisk.superjob.ru': 'superjob.ru',
 'gus-hrustalniy.superjob.ru': 'superjob.ru',
 'bodaybo.superjob.ru': 'superjob.ru',
 'taganrog.superjob.ru': 'superjob.ru',
 'angarsk.superjob.ru': 'superjob.ru',
 'petropavlovsk-kamchatskij.superjob.ru': 'superjob.ru',
 'birobidzhan.superjob.ru': 'superjob.ru',
 'minusinsk.superjob.ru': 'superjob.ru',
 'severobaykalsk.superjob.ru': 'superjob.ru',
 'glazov.superjob.ru': 'superjob.ru',
 'revda.superjob.ru': 'superjob.ru',
 'luga.superjob.ru': 'superjob.ru',
 'arhangelsk.superjob.ru': 'superjob.ru',
 'novorossiysk.superjob.ru': 'superjob.ru',
 'chebarkul.superjob.ru': 'superjob.ru',
 'reutov.superjob.ru': 'superjob.ru',
 'kogalym.superjob.ru': 'superjob.ru',
 'rostovskaya-oblast.superjob.ru': 'superjob.ru',
 'pervouralsk.superjob.ru': 'superjob.ru',
 'ussuriysk.superjob.ru': 'superjob.ru',
 'rybinsk.superjob.ru': 'superjob.ru',
 'kg.superjob.ru': 'superjob.ru',
 'belovo.superjob.ru': 'superjob.ru',
 'vyazniki.superjob.ru': 'superjob.ru',
 'simferopol.superjob.ru': 'superjob.ru',
 'ishim.superjob.ru': 'superjob.ru',
 'gubkinskij.superjob.ru': 'superjob.ru',
 'tulun.superjob.ru': 'superjob.ru',
 'evpatoriya.superjob.ru': 'superjob.ru',
 'nefteyugansk.superjob.ru': 'superjob.ru',
 'purpe.superjob.ru': 'superjob.ru',
 'vsevolozhsk.superjob.ru': 'superjob.ru',
 'vyborg.superjob.ru': 'superjob.ru',
 'kirovsk-spb.superjob.ru': 'superjob.ru',
 'gatchina.superjob.ru': 'superjob.ru',
 'kansk.superjob.ru': 'superjob.ru',
 'pavlovskiy-posad.superjob.ru': 'superjob.ru',
 'tatarstan.superjob.ru': 'superjob.ru',
 'hanty-mansiyskiy-ao-raduzhnyy.superjob.ru': 'superjob.ru',
 'stavropol.superjob.ru': 'superjob.ru',
 'adler.superjob.ru': 'superjob.ru',
 'sredneuralsk.superjob.ru': 'superjob.ru',
 'kaliningradskaya-oblast.superjob.ru': 'superjob.ru',
 'vyazma.superjob.ru': 'superjob.ru',
 'zhigulevsk.superjob.ru': 'superjob.ru',
 'orsk.superjob.ru': 'superjob.ru',
 'kingisepp.superjob.ru': 'superjob.ru',
 'nahodka.superjob.ru': 'superjob.ru',
 'balashov.superjob.ru': 'superjob.ru',
 'anzhero-sudzhensk.superjob.ru': 'superjob.ru',
 'neftekamsk.superjob.ru': 'superjob.ru',
 'likino-dulevo.superjob.ru': 'superjob.ru',
 'biysk.superjob.ru': 'superjob.ru',
 'aleksandrov.superjob.ru': 'superjob.ru',
 'klin.superjob.ru': 'superjob.ru',
 'nyagan.superjob.ru': 'superjob.ru',
 'zelenogorsk-kk.superjob.ru': 'superjob.ru',
 'kamensk-shahtinskiy.superjob.ru': 'superjob.ru',
 'salehard.superjob.ru': 'superjob.ru',
 'sergiev-posad.superjob.ru': 'superjob.ru',
 'amursk.superjob.ru': 'superjob.ru',
 'dmitrov.superjob.ru': 'superjob.ru',
 'nevyansk.superjob.ru': 'superjob.ru',
 'sverdlovskaya-oblast.superjob.ru': 'superjob.ru',
 'barabinsk.superjob.ru': 'superjob.ru',
 'nikolaevsk-na-amure.superjob.ru': 'superjob.ru',
 'gukovo.superjob.ru': 'superjob.ru',
 'budennovsk.superjob.ru': 'superjob.ru',
 'slancy.superjob.ru': 'superjob.ru',
 'dimitrovgrad.superjob.ru': 'superjob.ru',
 'volokolamsk.superjob.ru': 'superjob.ru',
 'balahna.superjob.ru': 'superjob.ru',
 'novotroitsk.superjob.ru': 'superjob.ru',
 'mezhdurechensk.superjob.ru': 'superjob.ru',
 'berezovsky-kemerov.superjob.ru': 'superjob.ru',
 'lobnya.superjob.ru': 'superjob.ru',
 'novoaltaysk.superjob.ru': 'superjob.ru',
 'lytkarino.superjob.ru': 'superjob.ru',
 'gryazi.superjob.ru': 'superjob.ru',
 'donetsk.superjob.ru': 'superjob.ru',
 'murom.superjob.ru': 'superjob.ru',
 'stariy-oskol.superjob.ru': 'superjob.ru',
 'pushkin.superjob.ru': 'superjob.ru',
 'krasnousolskiy.superjob.ru': 'superjob.ru',
 'kostroma.superjob.ru': 'superjob.ru',
 'nizhegorodskaya-oblast.superjob.ru': 'superjob.ru',
 'rtishevo.superjob.ru': 'superjob.ru',
 'leninsk-kuznetskiy.superjob.ru': 'superjob.ru',
 'yurga.superjob.ru': 'superjob.ru',
 'asha.superjob.ru': 'superjob.ru',
 'fryazino.superjob.ru': 'superjob.ru',
 'agryz.superjob.ru': 'superjob.ru',
 'osinniki.superjob.ru': 'superjob.ru',
 'arsenev.superjob.ru': 'superjob.ru',
 'balabanovo.superjob.ru': 'superjob.ru',
 'salavat.superjob.ru': 'superjob.ru',
 'krasnoznamensk-mo.superjob.ru': 'superjob.ru',
 'ujur.superjob.ru': 'superjob.ru',
 'bryansk.superjob.ru': 'superjob.ru',
 'kotelniki.superjob.ru': 'superjob.ru',
 'chernyahovsk.superjob.ru': 'superjob.ru',
 'buzuluk.superjob.ru': 'superjob.ru',
 'pyatigorsk.superjob.ru': 'superjob.ru',
 'zelenodolsk.superjob.ru': 'superjob.ru',
 'sosnovyi-bor.superjob.ru': 'superjob.ru',
 'blagoveschensk-bashkortostan.superjob.ru': 'superjob.ru',
 'sharya.superjob.ru': 'superjob.ru',
 'nakhabino.superjob.ru': 'superjob.ru',
 'korkino.superjob.ru': 'superjob.ru',
 'tuimazy.superjob.ru': 'superjob.ru',
 'achinsk.superjob.ru': 'superjob.ru',
 'seversk.superjob.ru': 'superjob.ru',
 'michurinsk.superjob.ru': 'superjob.ru',
 'sayanogorsk.superjob.ru': 'superjob.ru',
 'shushary.superjob.ru': 'superjob.ru',
 'selyatino.superjob.ru': 'superjob.ru',
 'engels.superjob.ru': 'superjob.ru',
 'bulgakovo.superjob.ru': 'superjob.ru',
 'alapaevsk.superjob.ru': 'superjob.ru',
 'alatyr.superjob.ru': 'superjob.ru',
 'arzamas.superjob.ru': 'superjob.ru',
 'kinel.superjob.ru': 'superjob.ru',
 'istra.superjob.ru': 'superjob.ru',
 'novomoskovsk.superjob.ru': 'superjob.ru',
 'kuznetsk.superjob.ru': 'superjob.ru',
 'langepas.superjob.ru': 'superjob.ru',
 'tomilino.superjob.ru': 'superjob.ru',
 'lodejnoe-pole.superjob.ru': 'superjob.ru',
 'votkinsk.superjob.ru': 'superjob.ru',
 'berezniki.superjob.ru': 'superjob.ru',
 'bataisk.superjob.ru': 'superjob.ru',
 'gorno-altaisk.superjob.ru': 'superjob.ru',
 'shahty.superjob.ru': 'superjob.ru',
 'udomlya.superjob.ru': 'superjob.ru',
 'pereslavl-zalesski.superjob.ru': 'superjob.ru',
 'taldom.superjob.ru': 'superjob.ru',
 'salsk.superjob.ru': 'superjob.ru',
 'kirovgrad.superjob.ru': 'superjob.ru',
 'ozery.superjob.ru': 'superjob.ru',
 'kirovo-chepetsk.superjob.ru': 'superjob.ru',
 'nadym.superjob.ru': 'superjob.ru',
 'ob.superjob.ru': 'superjob.ru',
 'anadyr.superjob.ru': 'superjob.ru',
 'kamenka-penzenskaya-oblast.superjob.ru': 'superjob.ru',
 'nalchik.superjob.ru': 'superjob.ru',
 'dzerzhinskiy.superjob.ru': 'superjob.ru',
 'kstovo.superjob.ru': 'superjob.ru',
 'kondopoga.superjob.ru': 'superjob.ru',
 'semiluki.superjob.ru': 'superjob.ru',
 'torzhok.superjob.ru': 'superjob.ru',
 'pohvistnevo.superjob.ru': 'superjob.ru',
 'shuya.superjob.ru': 'superjob.ru',
 'aznakaevo.superjob.ru': 'superjob.ru',
 'tynda.superjob.ru': 'superjob.ru',
 'krasnokamsk.superjob.ru': 'superjob.ru',
 'strezhevoj.superjob.ru': 'superjob.ru',
 'severodvinsk.superjob.ru': 'superjob.ru',
 'belorechensk.superjob.ru': 'superjob.ru',
 'izobilnyj.superjob.ru': 'superjob.ru',
 'prokopjevsk.superjob.ru': 'superjob.ru',
 'myski.superjob.ru': 'superjob.ru',
 'yugorsk.superjob.ru': 'superjob.ru',
 'severouralsk.superjob.ru': 'superjob.ru',
 'troitsk-chel.superjob.ru': 'superjob.ru',
 'muravlenko.superjob.ru': 'superjob.ru',
 'buguruslan.superjob.ru': 'superjob.ru',
 'korenovsk.superjob.ru': 'superjob.ru',
 'polevskoy.superjob.ru': 'superjob.ru',
 'vyksa.superjob.ru': 'superjob.ru',
 'berdsk.superjob.ru': 'superjob.ru',
 'uglich.superjob.ru': 'superjob.ru',
 'kotlas.superjob.ru': 'superjob.ru',
 'kopeysk.superjob.ru': 'superjob.ru',
 'kashira.superjob.ru': 'superjob.ru',
 'ulyanovskaya-oblast.superjob.ru': 'superjob.ru',
 'volgogradskaya-oblast.superjob.ru': 'superjob.ru',
 'zheleznogorsk.superjob.ru': 'superjob.ru',
 'essentuki.superjob.ru': 'superjob.ru',
 'privolzhsk.superjob.ru': 'superjob.ru',
 'bykovo.superjob.ru': 'superjob.ru',
 'shatura.superjob.ru': 'superjob.ru',
 'belebey.superjob.ru': 'superjob.ru',
 'sobinka.superjob.ru': 'superjob.ru',
 'magdagachi.superjob.ru': 'superjob.ru',
 'zeya.superjob.ru': 'superjob.ru',
 'voronezhskaya-oblast.superjob.ru': 'superjob.ru',
 'sarapul.superjob.ru': 'superjob.ru',
 'verhnyaya-salda.superjob.ru': 'superjob.ru',
 'gagarin.superjob.ru': 'superjob.ru',
 'kumertau.superjob.ru': 'superjob.ru',
 'gusev.superjob.ru': 'superjob.ru',
 'arsk.superjob.ru': 'superjob.ru',
 'satka.superjob.ru': 'superjob.ru',
 'kudrovo.superjob.ru': 'superjob.ru',
 'zainsk.superjob.ru': 'superjob.ru',
 'dubna.superjob.ru': 'superjob.ru',
 'lomonosov.superjob.ru': 'superjob.ru',
 'kanash.superjob.ru': 'superjob.ru',
 'kerch.superjob.ru': 'superjob.ru',
 'tarko-sale.superjob.ru': 'superjob.ru',
 'birsk.superjob.ru': 'superjob.ru',
 'elabuga.superjob.ru': 'superjob.ru',
 'golitsino.superjob.ru': 'superjob.ru',
 'abinsk.superjob.ru': 'superjob.ru',
 'sasovo.superjob.ru': 'superjob.ru',
 'petergof.superjob.ru': 'superjob.ru',
 'rubtsovsk.superjob.ru': 'superjob.ru',
 'krasnoe-selo.superjob.ru': 'superjob.ru',
 'leninogorsk.superjob.ru': 'superjob.ru',
 'timashevsk.superjob.ru': 'superjob.ru',
 'lyantor.superjob.ru': 'superjob.ru',
 'eniseysk.superjob.ru': 'superjob.ru',
 'ust-ilimsk.superjob.ru': 'superjob.ru',
 'kujbyshev.superjob.ru': 'superjob.ru',
 'skopin.superjob.ru': 'superjob.ru',
 'apsheronsk.superjob.ru': 'superjob.ru',
 'kuschevskaya.superjob.ru': 'superjob.ru',
 'puschino.superjob.ru': 'superjob.ru',
 'mihnevo.superjob.ru': 'superjob.ru',
 'bor.superjob.ru': 'superjob.ru',
 'skovorodino.superjob.ru': 'superjob.ru',
 'borisoglebsk.superjob.ru': 'superjob.ru',
 'tulskaya-oblast.superjob.ru': 'superjob.ru',
 'lesosibirsk.superjob.ru': 'superjob.ru',
 'kirzhach.superjob.ru': 'superjob.ru',
 'klintsy.superjob.ru': 'superjob.ru',
 'alekseevka-belgorod.superjob.ru': 'superjob.ru',
 'slavgorod.superjob.ru': 'superjob.ru',
 'bronnitsi.superjob.ru': 'superjob.ru',
 'belaya-kalitva.superjob.ru': 'superjob.ru',
 'yanaul.superjob.ru': 'superjob.ru',
 'artem.superjob.ru': 'superjob.ru',
 'georgievsk.superjob.ru': 'superjob.ru',
 'yalta.superjob.ru': 'superjob.ru',
 'staraya-russa.superjob.ru': 'superjob.ru',
 'naryan-mar.superjob.ru': 'superjob.ru',
 'maloyaroslavets.superjob.ru': 'superjob.ru',
 'kungur.superjob.ru': 'superjob.ru',
 'beloretsk.superjob.ru': 'superjob.ru',
 'kohma.superjob.ru': 'superjob.ru',
 'chusovoj.superjob.ru': 'superjob.ru',
 'tosno.superjob.ru': 'superjob.ru',
 'kolpino.superjob.ru': 'superjob.ru',
 'kolchugino.superjob.ru': 'superjob.ru',
 'uyar.superjob.ru': 'superjob.ru',
 'divnogorsk.superjob.ru': 'superjob.ru',
 'bikin.superjob.ru': 'superjob.ru',
 'gulkevichi.superjob.ru': 'superjob.ru',
 'kasimov.superjob.ru': 'superjob.ru',
 'lesozavodsk.superjob.ru': 'superjob.ru',
 'krymsk.superjob.ru': 'superjob.ru',
 'vanino.superjob.ru': 'superjob.ru',
 'kirishi.superjob.ru': 'superjob.ru',
 'zarechniy.superjob.ru': 'superjob.ru',
 'valdai.superjob.ru': 'superjob.ru',
 'megion.superjob.ru': 'superjob.ru',
 'seryshevo.superjob.ru': 'superjob.ru',
 'kuvandyk.superjob.ru': 'superjob.ru',
 'monino.superjob.ru': 'superjob.ru',
 'krekshino.superjob.ru': 'superjob.ru',
 'morshansk.superjob.ru': 'superjob.ru',
 'yartsevo.superjob.ru': 'superjob.ru',
 'rezh.superjob.ru': 'superjob.ru',
 'murino.superjob.ru': 'superjob.ru',
 'sosnogorsk.superjob.ru': 'superjob.ru',
 'verkhnij-ufalej.superjob.ru': 'superjob.ru',
 'asbest.superjob.ru': 'superjob.ru',
 'verhneyarkeevo.superjob.ru': 'superjob.ru',
 'derbent.superjob.ru': 'superjob.ru',
 'kamyshlov.superjob.ru': 'superjob.ru',
 'putilkovo.superjob.ru': 'superjob.ru',
 'kokoshkino.superjob.ru': 'superjob.ru',
 'davlekanovo.superjob.ru': 'superjob.ru',
 'novozybkov.superjob.ru': 'superjob.ru',
 'lukoyanov.superjob.ru': 'superjob.ru',
 'aprelevka.superjob.ru': 'superjob.ru',
 'bezhetsk.superjob.ru': 'superjob.ru',
 'raduzhnyj-vladimir.superjob.ru': 'superjob.ru',
 'vichuga.superjob.ru': 'superjob.ru',
 'bratsk.superjob.ru': 'superjob.ru',
 'khotkovo.superjob.ru': 'superjob.ru',
 'sarov.superjob.ru': 'superjob.ru',
 'apatity.superjob.ru': 'superjob.ru',
 'ershov.superjob.ru': 'superjob.ru',
 'yaransk.superjob.ru': 'superjob.ru',
 'nikel.superjob.ru': 'superjob.ru',
 'chapaevsk.superjob.ru': 'superjob.ru',
 'roslavl.superjob.ru': 'superjob.ru',
 'ust-kut.superjob.ru': 'superjob.ru',
 'svetlograd.superjob.ru': 'superjob.ru',
 'navashino.superjob.ru': 'superjob.ru',
 'zima.superjob.ru': 'superjob.ru',
 'borskoe.superjob.ru': 'superjob.ru',
 'afanasevo.superjob.ru': 'superjob.ru',
 'nurlat.superjob.ru': 'superjob.ru',
 'solikamsk.superjob.ru': 'superjob.ru',
 'semenov.superjob.ru': 'superjob.ru',
 'rasskazovo.superjob.ru': 'superjob.ru',
 'bogorodsk.superjob.ru': 'superjob.ru',
 'priozersk.superjob.ru': 'superjob.ru',
 'makhachkala.superjob.ru': 'superjob.ru',
 'krasnaya-polyana.superjob.ru': 'superjob.ru',
 'klimovsk.superjob.ru': 'superjob.ru',
 'yuryuzan.superjob.ru': 'superjob.ru',
 'kachkanar.superjob.ru': 'superjob.ru',
 'sergach.superjob.ru': 'superjob.ru',
 'bogdanovich.superjob.ru': 'superjob.ru',
 'volodarsk.superjob.ru': 'superjob.ru',
 'yuzhnouralsk.superjob.ru': 'superjob.ru',
 'ozersk.superjob.ru': 'superjob.ru',
 'zaokskiy.superjob.ru': 'superjob.ru',
 'iskitim.superjob.ru': 'superjob.ru',
 'aleksin.superjob.ru': 'superjob.ru',
 'bugulma.superjob.ru': 'superjob.ru',
 'kostomuksha.superjob.ru': 'superjob.ru',
 'kutchatov.superjob.ru': 'superjob.ru',
 'furmanov.superjob.ru': 'superjob.ru',
 'alushta.superjob.ru': 'superjob.ru',
 'mozhga.superjob.ru': 'superjob.ru',
 'marks.superjob.ru': 'superjob.ru',
 'novouralsk.superjob.ru': 'superjob.ru',
 'holmsk.superjob.ru': 'superjob.ru',
 'beloozerskiy.superjob.ru': 'superjob.ru',
 'elektrogorsk.superjob.ru': 'superjob.ru',
 'tuchkovo.superjob.ru': 'superjob.ru',
    'seversk.spravker.ru': 'spravker.ru',
 'malmyzh.spravker.ru': 'spravker.ru',
 'partizansk.spravker.ru': 'spravker.ru',
 'tuimazy.spravker.ru': 'spravker.ru',
 'mytishchi.spravker.ru': 'spravker.ru',
 'vladivostok.spravker.ru': 'spravker.ru',
 'neftekamsk.spravker.ru': 'spravker.ru',
 'pervouralsk.spravker.ru': 'spravker.ru',
 'orel.spravker.ru': 'spravker.ru',
 'viazma.spravker.ru': 'spravker.ru',
 'elets.spravker.ru': 'spravker.ru',
 'abinsk.spravker.ru': 'spravker.ru',
 'kasimov.spravker.ru': 'spravker.ru',
 'livny.spravker.ru': 'spravker.ru',
 'korolev.spravker.ru': 'spravker.ru',
 'vologda.spravker.ru': 'spravker.ru',
 'chaikovskii.spravker.ru': 'spravker.ru',
 'aldan.spravker.ru': 'spravker.ru',
 'tommot.spravker.ru': 'spravker.ru',
 'maloiaroslavets.spravker.ru': 'spravker.ru',
 'gubkinskii.spravker.ru': 'spravker.ru',
 'reutov.spravker.ru': 'spravker.ru',
 'nizhnevartovsk.spravker.ru': 'spravker.ru',
 'zainsk.spravker.ru': 'spravker.ru',
 'sharia.spravker.ru': 'spravker.ru',
 'kotlas.spravker.ru': 'spravker.ru',
 'msk.spravker.ru': 'spravker.ru',
 'uglich.spravker.ru': 'spravker.ru',
 'belebei.spravker.ru': 'spravker.ru',
 'nurlat.spravker.ru': 'spravker.ru',
 'komsomolsk-na-amure.spravker.ru': 'spravker.ru',
 'chapaevsk.spravker.ru': 'spravker.ru',
 'liubertsy.spravker.ru': 'spravker.ru',
 'vladikavkaz.spravker.ru': 'spravker.ru',
 'desnogorsk.spravker.ru': 'spravker.ru',
 'staraia-russa.spravker.ru': 'spravker.ru',
 'novoros.spravker.ru': 'spravker.ru',
 'anapa.spravker.ru': 'spravker.ru',
 'krasnodar.spravker.ru': 'spravker.ru',
 'gulkevichi.spravker.ru': 'spravker.ru',
 'tumen.spravker.ru': 'spravker.ru',
 'novosib.spravker.ru': 'spravker.ru',
 'rnd.spravker.ru': 'spravker.ru',
 'ekaterinburg.spravker.ru': 'spravker.ru',
 'kaluga.spravker.ru': 'spravker.ru',
 'iasnyi.spravker.ru': 'spravker.ru',
 'serov.spravker.ru': 'spravker.ru',
 'saratov.spravker.ru': 'spravker.ru',
 'vladimir.spravker.ru': 'spravker.ru',
 'balashikha.spravker.ru': 'spravker.ru',
 'tuapse.spravker.ru': 'spravker.ru',
 'ishimbai.spravker.ru': 'spravker.ru',
 'almetevsk.spravker.ru': 'spravker.ru',
 'obninsk.spravker.ru': 'spravker.ru',
 'katav-ivanovsk.spravker.ru': 'spravker.ru',
 'kolomna.spravker.ru': 'spravker.ru',
 'kazan.spravker.ru': 'spravker.ru',
 'serpukhov.spravker.ru': 'spravker.ru',
 'sredneuralsk.spravker.ru': 'spravker.ru',
 'lensk.spravker.ru': 'spravker.ru',
 'sochi.spravker.ru': 'spravker.ru',
 'gatchina.spravker.ru': 'spravker.ru',
 'mineralnye-vody.spravker.ru': 'spravker.ru',
 'viatskie-poliany.spravker.ru': 'spravker.ru',
 'spravker.ru': 'spravker.ru',
 'toliatti.spravker.ru': 'spravker.ru',
 'bogdanovich.spravker.ru': 'spravker.ru',
 'uzlovaia.spravker.ru': 'spravker.ru',
 'surgut.spravker.ru': 'spravker.ru',
 'pereslavl-zalesskiy.spravker.ru': 'spravker.ru',
 'satka.spravker.ru': 'spravker.ru',
 'zlatoust.spravker.ru': 'spravker.ru',
 'kamyshlov.spravker.ru': 'spravker.ru',
 'arsenev.spravker.ru': 'spravker.ru',
 'essentuki.spravker.ru': 'spravker.ru',
 'pugachev.spravker.ru': 'spravker.ru',
 'chelyabinsk.spravker.ru': 'spravker.ru',
 'pechora.spravker.ru': 'spravker.ru',
 'khimki.spravker.ru': 'spravker.ru',
 'adygeisk.spravker.ru': 'spravker.ru',
 'pavlovsk.spravker.ru': 'spravker.ru',
 'eisk.spravker.ru': 'spravker.ru',
 'spb.spravker.ru': 'spravker.ru',
 'rasskazovo.spravker.ru': 'spravker.ru',
 'agryz.spravker.ru': 'spravker.ru',
 'iartsevo.spravker.ru': 'spravker.ru',
 'nefteiugansk.spravker.ru': 'spravker.ru',
 'ukhta.spravker.ru': 'spravker.ru',
 'magnitogorsk.spravker.ru': 'spravker.ru',
 'volgograd.spravker.ru': 'spravker.ru',
 'kursk.spravker.ru': 'spravker.ru',
 'zheleznodorozhnyi.spravker.ru': 'spravker.ru',
 'korenovsk.spravker.ru': 'spravker.ru',
 'budennovsk.spravker.ru': 'spravker.ru',
 'novokubansk.spravker.ru': 'spravker.ru',
 'zelenograd.spravker.ru': 'spravker.ru',
 'volgodonsk.spravker.ru': 'spravker.ru',
 'irbit.spravker.ru': 'spravker.ru',
 'asha.spravker.ru': 'spravker.ru',
 'krasnoufimsk.spravker.ru': 'spravker.ru',
 'abakan.spravker.ru': 'spravker.ru',
 'kumertau.spravker.ru': 'spravker.ru',
 'birsk.spravker.ru': 'spravker.ru',
 'taganrog.spravker.ru': 'spravker.ru',
 'teberda.spravker.ru': 'spravker.ru',
 'rubtsovsk.spravker.ru': 'spravker.ru',
 'zhukovskii.spravker.ru': 'spravker.ru',
 'ramenskoe.spravker.ru': 'spravker.ru',
 'armavir.spravker.ru': 'spravker.ru',
 'ussuriisk.spravker.ru': 'spravker.ru',
 'kozelsk.spravker.ru': 'spravker.ru',
 'belorechensk.spravker.ru': 'spravker.ru',
 'labinsk.spravker.ru': 'spravker.ru',
 'sterlitamak.spravker.ru': 'spravker.ru',
 'ioshkar-ola.spravker.ru': 'spravker.ru',
 'novokuibyshevsk.spravker.ru': 'spravker.ru',
 'liantor.spravker.ru': 'spravker.ru',
 'belgorod.spravker.ru': 'spravker.ru',
 'volzhskii.spravker.ru': 'spravker.ru',
 'temriuk.spravker.ru': 'spravker.ru',
 'narian-mar.spravker.ru': 'spravker.ru',
 'belogorsk.spravker.ru': 'spravker.ru',
 'mikhailovsk.spravker.ru': 'spravker.ru',
 'nizhnekamsk.spravker.ru': 'spravker.ru',
 'krasnoyarsk.spravker.ru': 'spravker.ru',
 'aksai.spravker.ru': 'spravker.ru',
 'novocheboksarsk.spravker.ru': 'spravker.ru',
 'dzerzhinsk.spravker.ru': 'spravker.ru',
 'verkhniaia-pyshma.spravker.ru': 'spravker.ru',
 'suoiarvi.spravker.ru': 'spravker.ru',
 'chelny.spravker.ru': 'spravker.ru',
 'kropotkin.spravker.ru': 'spravker.ru',
 'cheboksary.spravker.ru': 'spravker.ru',
 'luki.spravker.ru': 'spravker.ru',
 'timashevsk.spravker.ru': 'spravker.ru',
 'kamyshin.spravker.ru': 'spravker.ru',
 'bogorodsk.spravker.ru': 'spravker.ru',
 'roslavl.spravker.ru': 'spravker.ru',
 'strezhevoi.spravker.ru': 'spravker.ru',
 'zelenokumsk.spravker.ru': 'spravker.ru',
 'inta.spravker.ru': 'spravker.ru',
 'tobolsk.spravker.ru': 'spravker.ru',
 'velikii-ustiug.spravker.ru': 'spravker.ru',
 'vyksa.spravker.ru': 'spravker.ru',
 'morshansk.spravker.ru': 'spravker.ru',
 'egorevsk.spravker.ru': 'spravker.ru',
 'sarapul.spravker.ru': 'spravker.ru',
 'otradnyi.spravker.ru': 'spravker.ru',
 'vnovgorod.spravker.ru': 'spravker.ru',
 'lakhdenpokhia.spravker.ru': 'spravker.ru',
 'vidnoe.spravker.ru': 'spravker.ru',
 'khvalynsk.spravker.ru': 'spravker.ru',
 'kansk.spravker.ru': 'spravker.ru',
 'kolchugino.spravker.ru': 'spravker.ru',
 'kaliningrad.spravker.ru': 'spravker.ru',
 'domodedovo.spravker.ru': 'spravker.ru',
 'ryazan.spravker.ru': 'spravker.ru',
 'ufa.spravker.ru': 'spravker.ru',
 'salavat.spravker.ru': 'spravker.ru',
 'engels.spravker.ru': 'spravker.ru',
 'barnaul.spravker.ru': 'spravker.ru',
 'kovrov.spravker.ru': 'spravker.ru',
 'pushkino.spravker.ru': 'spravker.ru',
 'shchelkovo.spravker.ru': 'spravker.ru',
 'morozovsk.spravker.ru': 'spravker.ru',
 'troitsk.spravker.ru': 'spravker.ru',
 'shakhty.spravker.ru': 'spravker.ru',
 'nizhniaia-tura.spravker.ru': 'spravker.ru',
 'beslan.spravker.ru': 'spravker.ru',
 'kemerovo.spravker.ru': 'spravker.ru',
 'baltiisk.spravker.ru': 'spravker.ru',
 'elabuga.spravker.ru': 'spravker.ru',
 'malaia-vishera.spravker.ru': 'spravker.ru',
 'maikop.spravker.ru': 'spravker.ru',
 'rezh.spravker.ru': 'spravker.ru',
 'buzuluk.spravker.ru': 'spravker.ru',
 'nakhodka.spravker.ru': 'spravker.ru',
 'meleuz.spravker.ru': 'spravker.ru',
 'tynda.spravker.ru': 'spravker.ru',
 'vyborg.spravker.ru': 'spravker.ru',
 'gubkin.spravker.ru': 'spravker.ru',
 'chebarkul.spravker.ru': 'spravker.ru',
 'slaviansk-na-kubani.spravker.ru': 'spravker.ru',
 'ishim.spravker.ru': 'spravker.ru',
 'ruzaevka.spravker.ru': 'spravker.ru',
 'zhigulevsk.spravker.ru': 'spravker.ru',
 'belovo.spravker.ru': 'spravker.ru',
 'palekh.spravker.ru': 'spravker.ru',
 'voskresensk.spravker.ru': 'spravker.ru',
 'sukhinichi.spravker.ru': 'spravker.ru',
 'kusa.spravker.ru': 'spravker.ru',
 'odintsovo.spravker.ru': 'spravker.ru',
 'korkino.spravker.ru': 'spravker.ru',
 'perm.spravker.ru': 'spravker.ru',
 'tula.spravker.ru': 'spravker.ru',
 'turinsk.spravker.ru': 'spravker.ru',
 'kungur.spravker.ru': 'spravker.ru',
 'tikhvin.spravker.ru': 'spravker.ru',
 'georgievsk.spravker.ru': 'spravker.ru',
 'manturovo.spravker.ru': 'spravker.ru',
 'aleksandrov.spravker.ru': 'spravker.ru',
 'sovetsk.spravker.ru': 'spravker.ru',
 'pskov.spravker.ru': 'spravker.ru',
 'cherepovets.spravker.ru': 'spravker.ru',
 'piatigorsk.spravker.ru': 'spravker.ru',
 'kineshma.spravker.ru': 'spravker.ru',
 'kalachinsk.spravker.ru': 'spravker.ru',
 'gvardeisk.spravker.ru': 'spravker.ru',
 'tver.spravker.ru': 'spravker.ru',
 'vorkuta.spravker.ru': 'spravker.ru',
 'votkinsk.spravker.ru': 'spravker.ru',
 'klin.spravker.ru': 'spravker.ru',
 'makhachkala.spravker.ru': 'spravker.ru',
 'neriungri.spravker.ru': 'spravker.ru',
 'sergiev-posad.spravker.ru': 'spravker.ru',
 'salsk.spravker.ru': 'spravker.ru',
 'iuzhnouralsk.spravker.ru': 'spravker.ru',
 'mendeleevsk.spravker.ru': 'spravker.ru',
 'dedovsk.spravker.ru': 'spravker.ru',
 'znamensk.spravker.ru': 'spravker.ru',
 'zvenigorod.spravker.ru': 'spravker.ru',
 'pyt-iakh.spravker.ru': 'spravker.ru',
 'nalchik.spravker.ru': 'spravker.ru',
 'shadrinsk.spravker.ru': 'spravker.ru',
 'volochek.spravker.ru': 'spravker.ru',
 'h-m.spravker.ru': 'spravker.ru',
 'ua.spravker.ru': 'spravker.ru',
 'tsimliansk.spravker.ru': 'spravker.ru',
 'voronezh.spravker.ru': 'spravker.ru',
 'saransk.spravker.ru': 'spravker.ru',
 'glazov.spravker.ru': 'spravker.ru',
 'petrozavodsk.spravker.ru': 'spravker.ru',
 'pavlovo.spravker.ru': 'spravker.ru',
 'kambarka.spravker.ru': 'spravker.ru',
 'michurinsk.spravker.ru': 'spravker.ru',
 'gus.spravker.ru': 'spravker.ru',
 'bakal.spravker.ru': 'spravker.ru',
 'iakutsk.spravker.ru': 'spravker.ru',
 'dubna.spravker.ru': 'spravker.ru',
 'zernograd.spravker.ru': 'spravker.ru',
 'akhtubinsk.spravker.ru': 'spravker.ru',
 'kamensk-uralskii.spravker.ru': 'spravker.ru',
 'miass.spravker.ru': 'spravker.ru',
 'krymsk.spravker.ru': 'spravker.ru',
 'kurchatov.spravker.ru': 'spravker.ru',
 'okulovka.spravker.ru': 'spravker.ru',
 'groznyi.spravker.ru': 'spravker.ru',
 'zavolzhe.spravker.ru': 'spravker.ru',
 'khadyzhensk.spravker.ru': 'spravker.ru',
 'nerchinsk.spravker.ru': 'spravker.ru',
 'samara.spravker.ru': 'spravker.ru',
 'penza.spravker.ru': 'spravker.ru',
 'staryi-oskol.spravker.ru': 'spravker.ru',
 'lipetsk.spravker.ru': 'spravker.ru',
 'bogoroditsk.spravker.ru': 'spravker.ru',
 'kirov.spravker.ru': 'spravker.ru',
 'aznakaevo.spravker.ru': 'spravker.ru',
 'syzran.spravker.ru': 'spravker.ru',
 'murom.spravker.ru': 'spravker.ru',
 'ob.spravker.ru': 'spravker.ru',
 'kuibyshev.spravker.ru': 'spravker.ru',
 'briansk.spravker.ru': 'spravker.ru',
 'liski.spravker.ru': 'spravker.ru',
 'ivanovo.spravker.ru': 'spravker.ru',
 'gubakha.spravker.ru': 'spravker.ru',
 'svetlogorsk.spravker.ru': 'spravker.ru',
 'istra.spravker.ru': 'spravker.ru',
 'sukhoi-log.spravker.ru': 'spravker.ru',
 'anzhero-sudzhensk.spravker.ru': 'spravker.ru',
 'solnechnogorsk.spravker.ru': 'spravker.ru',
 'baimak.spravker.ru': 'spravker.ru',
 'ianaul.spravker.ru': 'spravker.ru',
 'dolgoprudnyi.spravker.ru': 'spravker.ru',
 'venev.spravker.ru': 'spravker.ru',
 'emanzhelinsk.spravker.ru': 'spravker.ru',
 'sibai.spravker.ru': 'spravker.ru',
 'omutninsk.spravker.ru': 'spravker.ru',
 'prokopevsk.spravker.ru': 'spravker.ru',
 'apsheronsk.spravker.ru': 'spravker.ru',
 'pochep.spravker.ru': 'spravker.ru',
 'sortavala.spravker.ru': 'spravker.ru',
 'medvezhegorsk.spravker.ru': 'spravker.ru',
 'kinel.spravker.ru': 'spravker.ru',
 'novyi-urengoi.spravker.ru': 'spravker.ru',
 'stavropol.spravker.ru': 'spravker.ru',
 'belaia-kalitva.spravker.ru': 'spravker.ru',
 'iarovoe.spravker.ru': 'spravker.ru',
 'kirovo-chepetsk.spravker.ru': 'spravker.ru',
 'rossosh.spravker.ru': 'spravker.ru',
 'balabanovo.spravker.ru': 'spravker.ru',
 'monchegorsk.spravker.ru': 'spravker.ru',
 'urai.spravker.ru': 'spravker.ru',
 'salekhard.spravker.ru': 'spravker.ru',
 'kiselevsk.spravker.ru': 'spravker.ru',
 'minusinsk.spravker.ru': 'spravker.ru',
 'efremov.spravker.ru': 'spravker.ru',
 'rybinsk.spravker.ru': 'spravker.ru',
 'volsk.spravker.ru': 'spravker.ru',
 'shchekino.spravker.ru': 'spravker.ru',
 'sorochinsk.spravker.ru': 'spravker.ru',
 'kopeisk.spravker.ru': 'spravker.ru',
 'sovetskaia-gavan.spravker.ru': 'spravker.ru',
 'kanash.spravker.ru': 'spravker.ru',
 'biisk.spravker.ru': 'spravker.ru',
 'mozhga.spravker.ru': 'spravker.ru',
 'bahchisaraj.spravker.ru': 'spravker.ru',
 'gelend.spravker.ru': 'spravker.ru',
 'sokol.spravker.ru': 'spravker.ru',
 'kaltan.spravker.ru': 'spravker.ru',
 'khabarovsk.spravker.ru': 'spravker.ru',
 'iugorsk.spravker.ru': 'spravker.ru',
 'kyzyl.spravker.ru': 'spravker.ru',
 'karabash.spravker.ru': 'spravker.ru',
 'sol-iletsk.spravker.ru': 'spravker.ru',
 'niagan.spravker.ru': 'spravker.ru',
 'ulyanovsk.spravker.ru': 'spravker.ru',
 'petropavlovsk.spravker.ru': 'spravker.ru',
 'iurev-polskii.spravker.ru': 'spravker.ru',
 'okhansk.spravker.ru': 'spravker.ru',
 'zelenodolsk.spravker.ru': 'spravker.ru',
 'berezniki.spravker.ru': 'spravker.ru',
 'dmitrov.spravker.ru': 'spravker.ru',
 'krasnoarmeisk.spravker.ru': 'spravker.ru',
 'nevinnomyssk.spravker.ru': 'spravker.ru',
 'aleksin.spravker.ru': 'spravker.ru',
 'tikhoretsk.spravker.ru': 'spravker.ru',
 'rybnoe.spravker.ru': 'spravker.ru',
 'karpinsk.spravker.ru': 'spravker.ru',
 'krasnoturinsk.spravker.ru': 'spravker.ru',
 'kimry.spravker.ru': 'spravker.ru',
 'blagoveshensk.spravker.ru': 'spravker.ru',
 'petushki.spravker.ru': 'spravker.ru',
 'labytnangi.spravker.ru': 'spravker.ru',
 'fatezh.spravker.ru': 'spravker.ru',
 'pushkin.spravker.ru': 'spravker.ru',
 'sim.spravker.ru': 'spravker.ru',
 'polevskoi.spravker.ru': 'spravker.ru',
 'iukhnov.spravker.ru': 'spravker.ru',
 'yar.spravker.ru': 'spravker.ru',
 'kyshtym.spravker.ru': 'spravker.ru',
 'shakhunia.spravker.ru': 'spravker.ru',
 'magadan.spravker.ru': 'spravker.ru',
 'usman.spravker.ru': 'spravker.ru',
 'arzamas.spravker.ru': 'spravker.ru',
 'nizhnii-tagil.spravker.ru': 'spravker.ru',
 'orsk.spravker.ru': 'spravker.ru',
 'liudinovo.spravker.ru': 'spravker.ru',
 'krasnouralsk.spravker.ru': 'spravker.ru',
 'semiluki.spravker.ru': 'spravker.ru',
 'kstovo.spravker.ru': 'spravker.ru',
 'spassk-riazanskii.spravker.ru': 'spravker.ru',
 'borisoglebsk.spravker.ru': 'spravker.ru',
 'kasli.spravker.ru': 'spravker.ru',
 'evpatoriya.spravker.ru': 'spravker.ru',
 'abaza.spravker.ru': 'spravker.ru',
 'novgorod.spravker.ru': 'spravker.ru',
 'slavgorod.spravker.ru': 'spravker.ru',
 'kostroma.spravker.ru': 'spravker.ru',
 'smolensk.spravker.ru': 'spravker.ru',
 'kondrovo.spravker.ru': 'spravker.ru',
 'chusovoi.spravker.ru': 'spravker.ru',
 'aktau.spravker.ru': 'spravker.ru',
 'artem.spravker.ru': 'spravker.ru',
 'shuia.spravker.ru': 'spravker.ru',
 'zheleznogorsk.spravker.ru': 'spravker.ru',
 'rtishchevo.spravker.ru': 'spravker.ru',
 'kurgan.spravker.ru': 'spravker.ru',
 'teikovo.spravker.ru': 'spravker.ru',
 'chistopol.spravker.ru': 'spravker.ru',
 'omsk.spravker.ru': 'spravker.ru',
 'neviansk.spravker.ru': 'spravker.ru',
 'syktyvkar.spravker.ru': 'spravker.ru',
 'kondopoga.spravker.ru': 'spravker.ru',
 'segezha.spravker.ru': 'spravker.ru',
 'novouralsk.spravker.ru': 'spravker.ru',
 'kherson.spravker.ru': 'spravker.ru',
 'elista.spravker.ru': 'spravker.ru',
 'bugulma.spravker.ru': 'spravker.ru',
 'kimovsk.spravker.ru': 'spravker.ru',
 'alagir.spravker.ru': 'spravker.ru',
 'noiabrsk.spravker.ru': 'spravker.ru',
 'simferopol.spravker.ru': 'spravker.ru',
 'svetlograd.spravker.ru': 'spravker.ru',
 'zelenogradsk.spravker.ru': 'spravker.ru',
 'klintsy.spravker.ru': 'spravker.ru',
 'ialutorovsk.spravker.ru': 'spravker.ru',
 'achinsk.spravker.ru': 'spravker.ru',
 'serdobsk.spravker.ru': 'spravker.ru',
 'rzhev.spravker.ru': 'spravker.ru',
 'aleisk.spravker.ru': 'spravker.ru',
 'tomsk.spravker.ru': 'spravker.ru',
 'balashov.spravker.ru': 'spravker.ru',
 'amursk.spravker.ru': 'spravker.ru',
 'gorodets.spravker.ru': 'spravker.ru',
 'konstantinovsk.spravker.ru': 'spravker.ru',
 'volzhsk.spravker.ru': 'spravker.ru',
 'kolpashevo.spravker.ru': 'spravker.ru',
 'polysaevo.spravker.ru': 'spravker.ru',
 'leninsk-kuznetskii.spravker.ru': 'spravker.ru',
 'alapaevsk.spravker.ru': 'spravker.ru',
 'ust-katav.spravker.ru': 'spravker.ru',
 'ust-dzheguta.spravker.ru': 'spravker.ru',
 'novovoronezh.spravker.ru': 'spravker.ru',
 'buturlinovka.spravker.ru': 'spravker.ru',
 'izberbash.spravker.ru': 'spravker.ru',
 'terek.spravker.ru': 'spravker.ru',
 'karachaevsk.spravker.ru': 'spravker.ru',
 'zelenogorsk.spravker.ru': 'spravker.ru',
 'rostov.spravker.ru': 'spravker.ru',
 'viazniki.spravker.ru': 'spravker.ru',
 'aramil.spravker.ru': 'spravker.ru',
 'murmansk.spravker.ru': 'spravker.ru',
 'balakovo.spravker.ru': 'spravker.ru',
 'tambov.spravker.ru': 'spravker.ru',
 'mamadysh.spravker.ru': 'spravker.ru',
 'naro-fominsk.spravker.ru': 'spravker.ru',
 'lugansk.spravker.ru': 'spravker.ru',
 'berdsk.spravker.ru': 'spravker.ru',
 'cherkessk.spravker.ru': 'spravker.ru',
 'snezhinsk.spravker.ru': 'spravker.ru',
 'mednogorsk.spravker.ru': 'spravker.ru',
 'kirovgrad.spravker.ru': 'spravker.ru',
 'kislovodsk.spravker.ru': 'spravker.ru',
 'ples.spravker.ru': 'spravker.ru',
 'birobijan.spravker.ru': 'spravker.ru',
 'arhangelsk.spravker.ru': 'spravker.ru',
 'totma.spravker.ru': 'spravker.ru',
 'divnogorsk.spravker.ru': 'spravker.ru',
 'zvenigovo.spravker.ru': 'spravker.ru',
 'novoshakhtinsk.spravker.ru': 'spravker.ru',
 'kandalaksha.spravker.ru': 'spravker.ru',
 'sakhalinsk.spravker.ru': 'spravker.ru',
 'raichikhinsk.spravker.ru': 'spravker.ru',
 'langepas.spravker.ru': 'spravker.ru',
 'severouralsk.spravker.ru': 'spravker.ru',
 'kachkanar.spravker.ru': 'spravker.ru',
 'volokolamsk.spravker.ru': 'spravker.ru',
 'diatkovo.spravker.ru': 'spravker.ru',
 'lebedian.spravker.ru': 'spravker.ru',
 'kushva.spravker.ru': 'spravker.ru',
 'shumerlia.spravker.ru': 'spravker.ru',
 'novomichurinsk.spravker.ru': 'spravker.ru',
 'krasnoslobodsk.spravker.ru': 'spravker.ru',
 'berezovskii.spravker.ru': 'spravker.ru',
 'ust-kut.spravker.ru': 'spravker.ru',
 'astrahan.spravker.ru': 'spravker.ru',
 'alatyr.spravker.ru': 'spravker.ru',
 'talnakh.spravker.ru': 'spravker.ru',
 'kulebaki.spravker.ru': 'spravker.ru',
 'petrovsk.spravker.ru': 'spravker.ru',
 'iasnogorsk.spravker.ru': 'spravker.ru',
 'krasnokamsk.spravker.ru': 'spravker.ru',
 'poshehone.spravker.ru': 'spravker.ru',
 'kamenka.spravker.ru': 'spravker.ru',
 'sevastopol.spravker.ru': 'spravker.ru',
 'dzerzhinskii.spravker.ru': 'spravker.ru',
 'furmanov.spravker.ru': 'spravker.ru',
 'myski.spravker.ru': 'spravker.ru',
 'lysva.spravker.ru': 'spravker.ru',
 'mariinsk.spravker.ru': 'spravker.ru',
 'ivdel.spravker.ru': 'spravker.ru',
 'krasnyi-sulin.spravker.ru': 'spravker.ru',
 'noginsk.spravker.ru': 'spravker.ru',
 'volgorechensk.spravker.ru': 'spravker.ru',
 'iskitim.spravker.ru': 'spravker.ru',
 'kirsanov.spravker.ru': 'spravker.ru',
 'rylsk.spravker.ru': 'spravker.ru',
 'severomorsk.spravker.ru': 'spravker.ru',
 'poronaisk.spravker.ru': 'spravker.ru',
 'sobinka.spravker.ru': 'spravker.ru',
 'kiziliurt.spravker.ru': 'spravker.ru',
 'galich.spravker.ru': 'spravker.ru',
 'zadonsk.spravker.ru': 'spravker.ru',
 'donskoi.spravker.ru': 'spravker.ru',
 'verkhnii-tagil.spravker.ru': 'spravker.ru',
 'verkhniaia-tura.spravker.ru': 'spravker.ru',
 'donetsk.spravker.ru': 'spravker.ru',
 'kokhma.spravker.ru': 'spravker.ru',
 'novozybkov.spravker.ru': 'spravker.ru',
 'skopin.spravker.ru': 'spravker.ru',
 'pokhvistnevo.spravker.ru': 'spravker.ru',
 'novoaltaisk.spravker.ru': 'spravker.ru',
 'kharabali.spravker.ru': 'spravker.ru',
 'tutaev.spravker.ru': 'spravker.ru',
 'shimanovsk.spravker.ru': 'spravker.ru',
 'spas-klepiki.spravker.ru': 'spravker.ru',
 'kamen-na-obi.spravker.ru': 'spravker.ru',
 'norilsk.spravker.ru': 'spravker.ru',
 'osinniki.spravker.ru': 'spravker.ru',
 'apatity.spravker.ru': 'spravker.ru',
 'irkutsk.spravker.ru': 'spravker.ru',
 'buguruslan.spravker.ru': 'spravker.ru',
 'kargat.spravker.ru': 'spravker.ru',
 'asbest.spravker.ru': 'spravker.ru',
 'cherniakhovsk.spravker.ru': 'spravker.ru',
 'nadym.spravker.ru': 'spravker.ru',
 'tavda.spravker.ru': 'spravker.ru',
 'kotelnich.spravker.ru': 'spravker.ru',
 'valdai.spravker.ru': 'spravker.ru',
 'mikhailovka.spravker.ru': 'spravker.ru',
 'vorsma.spravker.ru': 'spravker.ru',
 'gorokhovets.spravker.ru': 'spravker.ru',
 'lakinsk.spravker.ru': 'spravker.ru',
 'kharovsk.spravker.ru': 'spravker.ru',
 'menzelinsk.spravker.ru': 'spravker.ru',
 'novotroitsk.spravker.ru': 'spravker.ru',
 'chaplygin.spravker.ru': 'spravker.ru',
 'dalnerechensk.spravker.ru': 'spravker.ru',
 'sosnovoborsk.spravker.ru': 'spravker.ru',
 'alupka.spravker.ru': 'spravker.ru',
 'borovsk.spravker.ru': 'spravker.ru',
 'lesozavodsk.spravker.ru': 'spravker.ru',
 'kz.spravker.ru': 'spravker.ru',
 'karachev.spravker.ru': 'spravker.ru',
 'millerovo.spravker.ru': 'spravker.ru',
 'chernogorsk.spravker.ru': 'spravker.ru',
 'oktiabrsk.spravker.ru': 'spravker.ru',
 'karasuk.spravker.ru': 'spravker.ru',
 'bodaibo.spravker.ru': 'spravker.ru',
 'tsivilsk.spravker.ru': 'spravker.ru',
 'plavsk.spravker.ru': 'spravker.ru',
 'buinaksk.spravker.ru': 'spravker.ru',
 'bogotol.spravker.ru': 'spravker.ru',
 'semei.spravker.ru': 'spravker.ru',
 'megion.spravker.ru': 'spravker.ru',
 'nazran.spravker.ru': 'spravker.ru',
 'riazhsk.spravker.ru': 'spravker.ru',
 'beloiarskii.spravker.ru': 'spravker.ru',
 'feodosiya.spravker.ru': 'spravker.ru',
 'kuznetsk.spravker.ru': 'spravker.ru',
 'sergach.spravker.ru': 'spravker.ru',
 'alushta.spravker.ru': 'spravker.ru',
 'derbent.spravker.ru': 'spravker.ru',
 'dolinsk.spravker.ru': 'spravker.ru',
 'novoaleksandrovsk.spravker.ru': 'spravker.ru',
 'pokachi.spravker.ru': 'spravker.ru',
 'gorno-altaisk.spravker.ru': 'spravker.ru',
 'vitebsk.spravker.ru': 'spravker.ru',
 'toguchin.spravker.ru': 'spravker.ru',
 'verkhniaia-salda.spravker.ru': 'spravker.ru',
 'dzhankoi.spravker.ru': 'spravker.ru',
    'seversk.jsprav.ru': 'jsprav.ru',
 'agidel.jsprav.ru': 'jsprav.ru',
 'birsk.jsprav.ru': 'jsprav.ru',
 'uglich.jsprav.ru': 'jsprav.ru',
 'sochi.jsprav.ru': 'jsprav.ru',
 'aramil.jsprav.ru': 'jsprav.ru',
 'krasnoznamensk1.jsprav.ru': 'jsprav.ru',
 'donetsk1.jsprav.ru': 'jsprav.ru',
 'ekaterinburg.jsprav.ru': 'jsprav.ru',
 'sarapul.jsprav.ru': 'jsprav.ru',
 'anzhero-sudzhensk.jsprav.ru': 'jsprav.ru',
 'ishim.jsprav.ru': 'jsprav.ru',
 'belorechensk.jsprav.ru': 'jsprav.ru',
 'mirnyij1.jsprav.ru': 'jsprav.ru',
 'kasimov.jsprav.ru': 'jsprav.ru',
 'ryazhsk.jsprav.ru': 'jsprav.ru',
 'dyurtyuli.jsprav.ru': 'jsprav.ru',
 'vidnoe.jsprav.ru': 'jsprav.ru',
 'golitsyino.jsprav.ru': 'jsprav.ru',
 'samara.jsprav.ru': 'jsprav.ru',
 'yaroslavl.jsprav.ru': 'jsprav.ru',
 'krasnodar.jsprav.ru': 'jsprav.ru',
 'orehovo-zuevo.jsprav.ru': 'jsprav.ru',
 'shatura.jsprav.ru': 'jsprav.ru',
 'habarovsk.jsprav.ru': 'jsprav.ru',
 'petropavlovsk.jsprav.ru': 'jsprav.ru',
 'lyubertsyi.jsprav.ru': 'jsprav.ru',
 'bogdanovich.jsprav.ru': 'jsprav.ru',
 'raduzhnyij1.jsprav.ru': 'jsprav.ru',
 'vladivostok.jsprav.ru': 'jsprav.ru',
 'tolyatti.jsprav.ru': 'jsprav.ru',
 'rajchihinsk.jsprav.ru': 'jsprav.ru',
 'valujki.jsprav.ru': 'jsprav.ru',
 'obninsk.jsprav.ru': 'jsprav.ru',
 'tsivilsk.jsprav.ru': 'jsprav.ru',
 'hmelnitskij.jsprav.ru': 'jsprav.ru',
 'bugulma.jsprav.ru': 'jsprav.ru',
 'yurga.jsprav.ru': 'jsprav.ru',
 'nizhnevartovsk.jsprav.ru': 'jsprav.ru',
 'lesozavodsk.jsprav.ru': 'jsprav.ru',
 'serov.jsprav.ru': 'jsprav.ru',
 'belgorod.jsprav.ru': 'jsprav.ru',
 'tatarsk.jsprav.ru': 'jsprav.ru',
 'moskva1.jsprav.ru': 'jsprav.ru',
 'elabuga.jsprav.ru': 'jsprav.ru',
 'domodedovo.jsprav.ru': 'jsprav.ru',
 'verhnyaya-tura.jsprav.ru': 'jsprav.ru',
 'anapa.jsprav.ru': 'jsprav.ru',
 'kirov1.jsprav.ru': 'jsprav.ru',
 'adyigejsk.jsprav.ru': 'jsprav.ru',
 'jsprav.ru': 'jsprav.ru',
 'bologoe.jsprav.ru': 'jsprav.ru',
 'tyumen.jsprav.ru': 'jsprav.ru',
 'novosibirsk.jsprav.ru': 'jsprav.ru',
 'novyij-urengoj.jsprav.ru': 'jsprav.ru',
 'temryuk.jsprav.ru': 'jsprav.ru',
 'ufa.jsprav.ru': 'jsprav.ru',
 'belaya-holunitsa.jsprav.ru': 'jsprav.ru',
 'cherepovets.jsprav.ru': 'jsprav.ru',
 'krasnoufimsk.jsprav.ru': 'jsprav.ru',
 'kinel.jsprav.ru': 'jsprav.ru',
 'artyomovskij.jsprav.ru': 'jsprav.ru',
 'tuapse.jsprav.ru': 'jsprav.ru',
 'gaj.jsprav.ru': 'jsprav.ru',
 'orenburg.jsprav.ru': 'jsprav.ru',
 'neftekamsk.jsprav.ru': 'jsprav.ru',
 'vyazemskij.jsprav.ru': 'jsprav.ru',
 'slavyansk-na-kubani.jsprav.ru': 'jsprav.ru',
 'satka.jsprav.ru': 'jsprav.ru',
 'penza.jsprav.ru': 'jsprav.ru',
 'berdyansk.jsprav.ru': 'jsprav.ru',
 'ivanovo.jsprav.ru': 'jsprav.ru',
 'uhta.jsprav.ru': 'jsprav.ru',
 'nefteyugansk.jsprav.ru': 'jsprav.ru',
 'novorossijsk.jsprav.ru': 'jsprav.ru',
 'saransk.jsprav.ru': 'jsprav.ru',
 'buinsk.jsprav.ru': 'jsprav.ru',
 'rostov-na-donu.jsprav.ru': 'jsprav.ru',
 'ussurijsk.jsprav.ru': 'jsprav.ru',
 'engels.jsprav.ru': 'jsprav.ru',
 'luga.jsprav.ru': 'jsprav.ru',
 'kurovskoe.jsprav.ru': 'jsprav.ru',
 'tujmazyi.jsprav.ru': 'jsprav.ru',
 'vladimir.jsprav.ru': 'jsprav.ru',
 'vyazma.jsprav.ru': 'jsprav.ru',
 'kyizyil.jsprav.ru': 'jsprav.ru',
 'armavir.jsprav.ru': 'jsprav.ru',
 'mahachkala.jsprav.ru': 'jsprav.ru',
 'dubna.jsprav.ru': 'jsprav.ru',
 'nevel.jsprav.ru': 'jsprav.ru',
 'balahna.jsprav.ru': 'jsprav.ru',
 'velikie-luki.jsprav.ru': 'jsprav.ru',
 'kropotkin.jsprav.ru': 'jsprav.ru',
 'irbit.jsprav.ru': 'jsprav.ru',
 'petergof.jsprav.ru': 'jsprav.ru',
 'stupino.jsprav.ru': 'jsprav.ru',
 'svobodnyij.jsprav.ru': 'jsprav.ru',
 'korolyov.jsprav.ru': 'jsprav.ru',
 'naberezhnyie-chelnyi.jsprav.ru': 'jsprav.ru',
 'yartsevo.jsprav.ru': 'jsprav.ru',
 'ejsk.jsprav.ru': 'jsprav.ru',
 'alekseevka.jsprav.ru': 'jsprav.ru',
 'nevinnomyissk.jsprav.ru': 'jsprav.ru',
 'vnukovo.jsprav.ru': 'jsprav.ru',
 'sterlitamak.jsprav.ru': 'jsprav.ru',
 'pechora.jsprav.ru': 'jsprav.ru',
 'salavat.jsprav.ru': 'jsprav.ru',
 'bogorodsk.jsprav.ru': 'jsprav.ru',
 'gus-hrustalnyij.jsprav.ru': 'jsprav.ru',
 'kazan.jsprav.ru': 'jsprav.ru',
 'asha.jsprav.ru': 'jsprav.ru',
 'velsk.jsprav.ru': 'jsprav.ru',
 'leninogorsk.jsprav.ru': 'jsprav.ru',
 'batajsk.jsprav.ru': 'jsprav.ru',
 'vladikavkaz.jsprav.ru': 'jsprav.ru',
 'neryungri.jsprav.ru': 'jsprav.ru',
 'komsomolsk-na-amure.jsprav.ru': 'jsprav.ru',
 'kujbyishev.jsprav.ru': 'jsprav.ru',
 'turinsk.jsprav.ru': 'jsprav.ru',
 'rezh.jsprav.ru': 'jsprav.ru',
 'majkop.jsprav.ru': 'jsprav.ru',
 'shatsk.jsprav.ru': 'jsprav.ru',
 'volgograd.jsprav.ru': 'jsprav.ru',
 'syizran.jsprav.ru': 'jsprav.ru',
 'solnechnogorsk.jsprav.ru': 'jsprav.ru',
 'rubtsovsk.jsprav.ru': 'jsprav.ru',
 'mozhajsk.jsprav.ru': 'jsprav.ru',
 'saratov.jsprav.ru': 'jsprav.ru',
 'taganrog.jsprav.ru': 'jsprav.ru',
 'vyiborg.jsprav.ru': 'jsprav.ru',
 'partizansk.jsprav.ru': 'jsprav.ru',
 'berezniki.jsprav.ru': 'jsprav.ru',
 'gusev.jsprav.ru': 'jsprav.ru',
 'vyitegra.jsprav.ru': 'jsprav.ru',
 'oktyabrskij.jsprav.ru': 'jsprav.ru',
 'voronezh.jsprav.ru': 'jsprav.ru',
 'kamensk-shahtinskij.jsprav.ru': 'jsprav.ru',
 'kogalyim.jsprav.ru': 'jsprav.ru',
 'chapaevsk.jsprav.ru': 'jsprav.ru',
 'zeya.jsprav.ru': 'jsprav.ru',
 'pskov.jsprav.ru': 'jsprav.ru',
 'manturovo.jsprav.ru': 'jsprav.ru',
 'roslavl.jsprav.ru': 'jsprav.ru',
 'ershov.jsprav.ru': 'jsprav.ru',
 'shadrinsk.jsprav.ru': 'jsprav.ru',
 'pavlovskij-posad.jsprav.ru': 'jsprav.ru',
 'beslan.jsprav.ru': 'jsprav.ru',
 'kstovo.jsprav.ru': 'jsprav.ru',
 'usinsk.jsprav.ru': 'jsprav.ru',
 'dmitrov.jsprav.ru': 'jsprav.ru',
 'prokopevsk.jsprav.ru': 'jsprav.ru',
 'yahroma.jsprav.ru': 'jsprav.ru',
 'sol-iletsk.jsprav.ru': 'jsprav.ru',
 'herson.jsprav.ru': 'jsprav.ru',
 'chajkovskij.jsprav.ru': 'jsprav.ru',
 'kirzhach.jsprav.ru': 'jsprav.ru',
 'severouralsk.jsprav.ru': 'jsprav.ru',
 'yuzhnouralsk.jsprav.ru': 'jsprav.ru',
 'nikolaevsk-na-amure.jsprav.ru': 'jsprav.ru',
 'kuvandyik.jsprav.ru': 'jsprav.ru',
 'gudermes.jsprav.ru': 'jsprav.ru',
 'mihajlov.jsprav.ru': 'jsprav.ru',
 'tyinda.jsprav.ru': 'jsprav.ru',
 'znamensk.jsprav.ru': 'jsprav.ru',
 'krasnoyarsk.jsprav.ru': 'jsprav.ru',
 'ulyanovsk.jsprav.ru': 'jsprav.ru',
 'tarusa.jsprav.ru': 'jsprav.ru',
 'livnyi.jsprav.ru': 'jsprav.ru',
 'kramatorsk.jsprav.ru': 'jsprav.ru',
 'chernogorsk.jsprav.ru': 'jsprav.ru',
 'strezhevoj.jsprav.ru': 'jsprav.ru',
 'ostrov.jsprav.ru': 'jsprav.ru',
 'prohladnyij.jsprav.ru': 'jsprav.ru',
 'beloretsk.jsprav.ru': 'jsprav.ru',
 'kansk.jsprav.ru': 'jsprav.ru',
 'tetyushi.jsprav.ru': 'jsprav.ru',
 'enisejsk.jsprav.ru': 'jsprav.ru',
 'labinsk.jsprav.ru': 'jsprav.ru',
 'perm.jsprav.ru': 'jsprav.ru',
 'pavlovo.jsprav.ru': 'jsprav.ru',
 'pyit-yah.jsprav.ru': 'jsprav.ru',
 'udachnyij.jsprav.ru': 'jsprav.ru',
 'nahodka.jsprav.ru': 'jsprav.ru',
 'odintsovo.jsprav.ru': 'jsprav.ru',
 'michurinsk.jsprav.ru': 'jsprav.ru',
 'zlatoust.jsprav.ru': 'jsprav.ru',
 'elektrostal.jsprav.ru': 'jsprav.ru',
 'novotroitsk.jsprav.ru': 'jsprav.ru',
 'opochka.jsprav.ru': 'jsprav.ru',
 'udomlya.jsprav.ru': 'jsprav.ru',
 'konakovo.jsprav.ru': 'jsprav.ru',
 'donskoj.jsprav.ru': 'jsprav.ru',
 'kurgan.jsprav.ru': 'jsprav.ru',
 'ryazan.jsprav.ru': 'jsprav.ru',
 'chelyabinsk.jsprav.ru': 'jsprav.ru',
 'sharya.jsprav.ru': 'jsprav.ru',
 'vorkuta.jsprav.ru': 'jsprav.ru',
 'dankov.jsprav.ru': 'jsprav.ru',
 'ishimbaj.jsprav.ru': 'jsprav.ru',
 'amursk.jsprav.ru': 'jsprav.ru',
 'kovrov.jsprav.ru': 'jsprav.ru',
 'podolsk.jsprav.ru': 'jsprav.ru',
 'essentuki.jsprav.ru': 'jsprav.ru',
 'yanaul.jsprav.ru': 'jsprav.ru',
 'sevastopol1.jsprav.ru': 'jsprav.ru',
 'orsk.jsprav.ru': 'jsprav.ru',
 'osa.jsprav.ru': 'jsprav.ru',
 'gryazovets.jsprav.ru': 'jsprav.ru',
 'gubkin.jsprav.ru': 'jsprav.ru',
 'borisoglebsk.jsprav.ru': 'jsprav.ru',
 'kursk.jsprav.ru': 'jsprav.ru',
 'surgut.jsprav.ru': 'jsprav.ru',
 'galich.jsprav.ru': 'jsprav.ru',
 'kumertau.jsprav.ru': 'jsprav.ru',
 'asbest.jsprav.ru': 'jsprav.ru',
 'korenovsk.jsprav.ru': 'jsprav.ru',
 'kolchugino.jsprav.ru': 'jsprav.ru',
 'gelendzhik.jsprav.ru': 'jsprav.ru',
 'taldyikorgan.jsprav.ru': 'jsprav.ru',
 'yarovoe.jsprav.ru': 'jsprav.ru',
 'lyisva.jsprav.ru': 'jsprav.ru',
 'stavropol.jsprav.ru': 'jsprav.ru',
 'kurchatov.jsprav.ru': 'jsprav.ru',
 'omsk.jsprav.ru': 'jsprav.ru',
 'hantyi-mansijsk.jsprav.ru': 'jsprav.ru',
 'vitebsk.jsprav.ru': 'jsprav.ru',
 'tihvin.jsprav.ru': 'jsprav.ru',
 'lyantor.jsprav.ru': 'jsprav.ru',
 'torzhok.jsprav.ru': 'jsprav.ru',
 'ustyuzhna.jsprav.ru': 'jsprav.ru',
 'topki.jsprav.ru': 'jsprav.ru',
 'timashyovsk.jsprav.ru': 'jsprav.ru',
 'vyishnij-volochyok.jsprav.ru': 'jsprav.ru',
 'kaliningrad.jsprav.ru': 'jsprav.ru',
 'novokubansk.jsprav.ru': 'jsprav.ru',
 'svetlograd.jsprav.ru': 'jsprav.ru',
 'chehov.jsprav.ru': 'jsprav.ru',
 'belogorsk.jsprav.ru': 'jsprav.ru',
 'bogoroditsk.jsprav.ru': 'jsprav.ru',
 'magadan.jsprav.ru': 'jsprav.ru',
 'dzerzhinsk.jsprav.ru': 'jsprav.ru',
 'iskitim.jsprav.ru': 'jsprav.ru',
 'monchegorsk.jsprav.ru': 'jsprav.ru',
 'magnitogorsk.jsprav.ru': 'jsprav.ru',
 'derbent.jsprav.ru': 'jsprav.ru',
 'sim.jsprav.ru': 'jsprav.ru',
 'joshkar-ola.jsprav.ru': 'jsprav.ru',
 'beloyarskij.jsprav.ru': 'jsprav.ru',
 'polevskoj.jsprav.ru': 'jsprav.ru',
 'sovetskaya-gavan.jsprav.ru': 'jsprav.ru',
 'sosnogorsk.jsprav.ru': 'jsprav.ru',
 'miass.jsprav.ru': 'jsprav.ru',
 'kondrovo.jsprav.ru': 'jsprav.ru',
 'berdsk.jsprav.ru': 'jsprav.ru',
 'sayanogorsk.jsprav.ru': 'jsprav.ru',
 'alapaevsk.jsprav.ru': 'jsprav.ru',
 'abakan.jsprav.ru': 'jsprav.ru',
 'elets.jsprav.ru': 'jsprav.ru',
 'oryol.jsprav.ru': 'jsprav.ru',
 'lipetsk.jsprav.ru': 'jsprav.ru',
 'hadyizhensk.jsprav.ru': 'jsprav.ru',
 'millerovo.jsprav.ru': 'jsprav.ru',
 'krasnouralsk.jsprav.ru': 'jsprav.ru',
 'serpuhov.jsprav.ru': 'jsprav.ru',
 'mezhdurechensk.jsprav.ru': 'jsprav.ru',
 'votkinsk.jsprav.ru': 'jsprav.ru',
 'spassk-ryazanskij.jsprav.ru': 'jsprav.ru',
 'kushva.jsprav.ru': 'jsprav.ru',
 'pereslavl-zalesskij.jsprav.ru': 'jsprav.ru',
 'novoaltajsk.jsprav.ru': 'jsprav.ru',
 'cheboksaryi.jsprav.ru': 'jsprav.ru',
 'likino-dulyovo.jsprav.ru': 'jsprav.ru',
 'mednogorsk.jsprav.ru': 'jsprav.ru',
 'novovoronezh.jsprav.ru': 'jsprav.ru',
 'kashira.jsprav.ru': 'jsprav.ru',
 'novoshahtinsk.jsprav.ru': 'jsprav.ru',
 'karachaevsk.jsprav.ru': 'jsprav.ru',
 'donetsk.jsprav.ru': 'jsprav.ru',
 'nazran.jsprav.ru': 'jsprav.ru',
 'cherkessk.jsprav.ru': 'jsprav.ru',
 'hasavyurt.jsprav.ru': 'jsprav.ru',
 'abinsk.jsprav.ru': 'jsprav.ru',
 'shahtyi.jsprav.ru': 'jsprav.ru',
 'novocherkassk.jsprav.ru': 'jsprav.ru',
 'krasnyij-sulin.jsprav.ru': 'jsprav.ru',
 'zverevo.jsprav.ru': 'jsprav.ru',
 'mineralnyie-vodyi.jsprav.ru': 'jsprav.ru',
 'kaspijsk.jsprav.ru': 'jsprav.ru',
 'izberbash.jsprav.ru': 'jsprav.ru',
 'morozovsk.jsprav.ru': 'jsprav.ru',
 'salsk.jsprav.ru': 'jsprav.ru',
 'uralsk.jsprav.ru': 'jsprav.ru',
 'kolomna.jsprav.ru': 'jsprav.ru',
 'volsk.jsprav.ru': 'jsprav.ru',
 'novokujbyishevsk.jsprav.ru': 'jsprav.ru',
 'smorgon.jsprav.ru': 'jsprav.ru',
 'tihoretsk.jsprav.ru': 'jsprav.ru',
 'zelenokumsk.jsprav.ru': 'jsprav.ru',
 'georgievsk.jsprav.ru': 'jsprav.ru',
 'gatchina.jsprav.ru': 'jsprav.ru',
 'staryij-oskol.jsprav.ru': 'jsprav.ru',
 'mtsensk.jsprav.ru': 'jsprav.ru',
 'volokolamsk.jsprav.ru': 'jsprav.ru',
 'novohopyorsk.jsprav.ru': 'jsprav.ru',
 'shuya.jsprav.ru': 'jsprav.ru',
 'svetlogorsk.jsprav.ru': 'jsprav.ru',
 'nevyansk.jsprav.ru': 'jsprav.ru',
 'megion.jsprav.ru': 'jsprav.ru',
 'gdov.jsprav.ru': 'jsprav.ru',
 'chkalovsk.jsprav.ru': 'jsprav.ru',
 'velikij-novgorod.jsprav.ru': 'jsprav.ru',
 'syisert.jsprav.ru': 'jsprav.ru',
 'gavrilov-posad.jsprav.ru': 'jsprav.ru',
 'achinsk.jsprav.ru': 'jsprav.ru',
 'pyatigorsk.jsprav.ru': 'jsprav.ru',
 'nyagan.jsprav.ru': 'jsprav.ru',
 'elizavetinskaya.jsprav.ru': 'jsprav.ru',
 'novouralsk.jsprav.ru': 'jsprav.ru',
 'novokuznetsk.jsprav.ru': 'jsprav.ru',
 'balashiha.jsprav.ru': 'jsprav.ru',
 'valdaj.jsprav.ru': 'jsprav.ru',
 'pikalyovo.jsprav.ru': 'jsprav.ru',
 'scherbinka.jsprav.ru': 'jsprav.ru',
 'solikamsk.jsprav.ru': 'jsprav.ru',
 'lobnya.jsprav.ru': 'jsprav.ru',
 'davlekanovo.jsprav.ru': 'jsprav.ru',
 'novomoskovsk.jsprav.ru': 'jsprav.ru',
 'pokrov.jsprav.ru': 'jsprav.ru',
 'buzuluk.jsprav.ru': 'jsprav.ru',
 'efremov.jsprav.ru': 'jsprav.ru',
 'kotelnich.jsprav.ru': 'jsprav.ru',
 'nizhnij-novgorod.jsprav.ru': 'jsprav.ru',
 'myitischi.jsprav.ru': 'jsprav.ru',
 'yugorsk.jsprav.ru': 'jsprav.ru',
 'kachkanar.jsprav.ru': 'jsprav.ru',
 'arzamas.jsprav.ru': 'jsprav.ru',
 'novomichurinsk.jsprav.ru': 'jsprav.ru',
 'pugachyov.jsprav.ru': 'jsprav.ru',
 'minusinsk.jsprav.ru': 'jsprav.ru',
 'bavlyi.jsprav.ru': 'jsprav.ru',
 'nurlat.jsprav.ru': 'jsprav.ru',
 'gryazi.jsprav.ru': 'jsprav.ru',
 'ust-katav.jsprav.ru': 'jsprav.ru',
 'kaluga.jsprav.ru': 'jsprav.ru',
 'petrozavodsk.jsprav.ru': 'jsprav.ru',
 'istra.jsprav.ru': 'jsprav.ru',
 'primorsko-ahtarsk.jsprav.ru': 'jsprav.ru',
 'naro-fominsk.jsprav.ru': 'jsprav.ru',
 'krasnogorsk.jsprav.ru': 'jsprav.ru',
 'murmansk.jsprav.ru': 'jsprav.ru',
 'schyolkovo.jsprav.ru': 'jsprav.ru',
 'koryazhma.jsprav.ru': 'jsprav.ru',
 'kislovodsk.jsprav.ru': 'jsprav.ru',
 'kommunarka.jsprav.ru': 'jsprav.ru',
 'ust-dzheguta.jsprav.ru': 'jsprav.ru',
 'mamadyish.jsprav.ru': 'jsprav.ru',
 'gulkevichi.jsprav.ru': 'jsprav.ru',
 'tobolsk.jsprav.ru': 'jsprav.ru',
 'chernushka.jsprav.ru': 'jsprav.ru',
 'glazov.jsprav.ru': 'jsprav.ru',
 'emanzhelinsk.jsprav.ru': 'jsprav.ru',
 'kamyishin.jsprav.ru': 'jsprav.ru',
 'talitsa.jsprav.ru': 'jsprav.ru',
 'labyitnangi.jsprav.ru': 'jsprav.ru',
 'nikolaev1.jsprav.ru': 'jsprav.ru',
 'nazarovo.jsprav.ru': 'jsprav.ru',
 'zhukovskij.jsprav.ru': 'jsprav.ru',
 'aleksandrovsk.jsprav.ru': 'jsprav.ru',
 'volzhskij.jsprav.ru': 'jsprav.ru',
 'tambov.jsprav.ru': 'jsprav.ru',
 'nizhnij-tagil.jsprav.ru': 'jsprav.ru',
 'yalta.jsprav.ru': 'jsprav.ru',
 'pohvistnevo.jsprav.ru': 'jsprav.ru',
 'egorevsk.jsprav.ru': 'jsprav.ru',
 'kambarka.jsprav.ru': 'jsprav.ru',
 'dzerzhinskij.jsprav.ru': 'jsprav.ru',
 'kalach-na-donu.jsprav.ru': 'jsprav.ru',
 'langepas.jsprav.ru': 'jsprav.ru',
 'volgodonsk.jsprav.ru': 'jsprav.ru',
 'norilsk.jsprav.ru': 'jsprav.ru',
 'bor.jsprav.ru': 'jsprav.ru',
 'arhangelsk.jsprav.ru': 'jsprav.ru',
 'zarajsk.jsprav.ru': 'jsprav.ru',
 'dimitrovgrad.jsprav.ru': 'jsprav.ru',
 'kudyimkar.jsprav.ru': 'jsprav.ru',
 'mihajlovsk1.jsprav.ru': 'jsprav.ru',
 'nalchik.jsprav.ru': 'jsprav.ru',
 'rudnya.jsprav.ru': 'jsprav.ru',
 'voskresensk.jsprav.ru': 'jsprav.ru',
 'almetevsk.jsprav.ru': 'jsprav.ru',
 'yalutorovsk.jsprav.ru': 'jsprav.ru',
 'apsheronsk.jsprav.ru': 'jsprav.ru',
 'sortavala.jsprav.ru': 'jsprav.ru',
 'severodvinsk.jsprav.ru': 'jsprav.ru',
 'inta.jsprav.ru': 'jsprav.ru',
 'knyaginino.jsprav.ru': 'jsprav.ru',
 'kingisepp.jsprav.ru': 'jsprav.ru',
 'sorochinsk.jsprav.ru': 'jsprav.ru',
 'fryazino.jsprav.ru': 'jsprav.ru',
 'borovichi.jsprav.ru': 'jsprav.ru',
 'belozersk.jsprav.ru': 'jsprav.ru',
 'tver.jsprav.ru': 'jsprav.ru',
 'alma-ata.jsprav.ru': 'jsprav.ru',
 'aldan.jsprav.ru': 'jsprav.ru',
 'maloarhangelsk.jsprav.ru': 'jsprav.ru',
 'bikin.jsprav.ru': 'jsprav.ru',
 'kostroma.jsprav.ru': 'jsprav.ru',
 'smolensk.jsprav.ru': 'jsprav.ru',
 'polyarnyij.jsprav.ru': 'jsprav.ru',
 'lakinsk.jsprav.ru': 'jsprav.ru',
 'dolgoprudnyij.jsprav.ru': 'jsprav.ru',
 'kemerovo.jsprav.ru': 'jsprav.ru',
 'groznyij.jsprav.ru': 'jsprav.ru',
 'zavodoukovsk.jsprav.ru': 'jsprav.ru',
 'poronajsk.jsprav.ru': 'jsprav.ru',
 'kirishi.jsprav.ru': 'jsprav.ru',
 'lyudinovo.jsprav.ru': 'jsprav.ru',
 'aksaj.jsprav.ru': 'jsprav.ru',
 'pushkino.jsprav.ru': 'jsprav.ru',
 'zhabinka.jsprav.ru': 'jsprav.ru',
 'brest.jsprav.ru': 'jsprav.ru',
 'lyitkarino.jsprav.ru': 'jsprav.ru',
 'belebej.jsprav.ru': 'jsprav.ru',
 'noginsk.jsprav.ru': 'jsprav.ru',
 'roshal.jsprav.ru': 'jsprav.ru',
 'tajshet.jsprav.ru': 'jsprav.ru',
 'ostashkov.jsprav.ru': 'jsprav.ru',
 'bryansk.jsprav.ru': 'jsprav.ru',
 'sarov.jsprav.ru': 'jsprav.ru',
 'aleksandrov.jsprav.ru': 'jsprav.ru',
 'lesosibirsk.jsprav.ru': 'jsprav.ru',
 'krasnoturinsk.jsprav.ru': 'jsprav.ru',
 'noyabrsk.jsprav.ru': 'jsprav.ru',
 'belinskij.jsprav.ru': 'jsprav.ru',
 'priozersk.jsprav.ru': 'jsprav.ru',
 'kungur.jsprav.ru': 'jsprav.ru',
 'kozmodemyansk.jsprav.ru': 'jsprav.ru',
 'kondopoga.jsprav.ru': 'jsprav.ru',
 'baltijsk.jsprav.ru': 'jsprav.ru',
 'urzhum.jsprav.ru': 'jsprav.ru',
 'rzhev.jsprav.ru': 'jsprav.ru',
 'gukovo.jsprav.ru': 'jsprav.ru',
 'volzhsk.jsprav.ru': 'jsprav.ru',
 'semyonov.jsprav.ru': 'jsprav.ru',
 'lahdenpohya.jsprav.ru': 'jsprav.ru',
 'vyatskie-polyanyi.jsprav.ru': 'jsprav.ru',
 'murom.jsprav.ru': 'jsprav.ru',
 'mirnyij.jsprav.ru': 'jsprav.ru',
 'tajga.jsprav.ru': 'jsprav.ru',
 'sosenskij.jsprav.ru': 'jsprav.ru',
 'slobodskoj.jsprav.ru': 'jsprav.ru',
 'vyiksa.jsprav.ru': 'jsprav.ru',
 'zelenogorsk1.jsprav.ru': 'jsprav.ru',
 'hotkovo.jsprav.ru': 'jsprav.ru',
 'omutninsk.jsprav.ru': 'jsprav.ru',
 'troitsk.jsprav.ru': 'jsprav.ru',
 'tejkovo.jsprav.ru': 'jsprav.ru',
 'balakovo.jsprav.ru': 'jsprav.ru',
 'alatyir.jsprav.ru': 'jsprav.ru',
 'dyatkovo.jsprav.ru': 'jsprav.ru',
 'uchalyi.jsprav.ru': 'jsprav.ru',
 'aprelevka.jsprav.ru': 'jsprav.ru',
 'krasnyij-kut.jsprav.ru': 'jsprav.ru',
 'usole.jsprav.ru': 'jsprav.ru',
 'kamenka.jsprav.ru': 'jsprav.ru',
 'alushta.jsprav.ru': 'jsprav.ru',
 'lesnoj.jsprav.ru': 'jsprav.ru',
 'kandalaksha.jsprav.ru': 'jsprav.ru',
 'kyishtyim.jsprav.ru': 'jsprav.ru',
 'ozyoryi.jsprav.ru': 'jsprav.ru',
 'bijsk.jsprav.ru': 'jsprav.ru',
 'zarechnyij1.jsprav.ru': 'jsprav.ru',
 'zelenograd.jsprav.ru': 'jsprav.ru',
 'himki.jsprav.ru': 'jsprav.ru',
 'birobidzhan.jsprav.ru': 'jsprav.ru',
 'luhovitsyi.jsprav.ru': 'jsprav.ru',
 'lodejnoe-pole.jsprav.ru': 'jsprav.ru',
 'ivangorod.jsprav.ru': 'jsprav.ru',
 'kurganinsk.jsprav.ru': 'jsprav.ru',
 'zheleznogorsk.jsprav.ru': 'jsprav.ru',
 'sergiev-posad.jsprav.ru': 'jsprav.ru',
 'naryan-mar.jsprav.ru': 'jsprav.ru',
 'kuznetsk.jsprav.ru': 'jsprav.ru',
 'sosnovka.jsprav.ru': 'jsprav.ru',
 'zheleznovodsk.jsprav.ru': 'jsprav.ru',
 'ipatovo.jsprav.ru': 'jsprav.ru',
 'shahtinsk.jsprav.ru': 'jsprav.ru',
 'holmsk.jsprav.ru': 'jsprav.ru',
 'bolgar.jsprav.ru': 'jsprav.ru',
 'tomsk.jsprav.ru': 'jsprav.ru',
 'neftegorsk.jsprav.ru': 'jsprav.ru',
 'klin.jsprav.ru': 'jsprav.ru',
 'kineshma.jsprav.ru': 'jsprav.ru',
 'chistopol.jsprav.ru': 'jsprav.ru',
 'gorohovets.jsprav.ru': 'jsprav.ru',
 'safonovo.jsprav.ru': 'jsprav.ru',
 'bakal.jsprav.ru': 'jsprav.ru',
 'sergach.jsprav.ru': 'jsprav.ru',
 'lensk.jsprav.ru': 'jsprav.ru',
 'kryimsk.jsprav.ru': 'jsprav.ru',
 'tavda.jsprav.ru': 'jsprav.ru',
 'vyazniki.jsprav.ru': 'jsprav.ru',
 'chusovoj.jsprav.ru': 'jsprav.ru',
 'shlisselburg.jsprav.ru': 'jsprav.ru',
 'ramenskoe.jsprav.ru': 'jsprav.ru',
 'belaya-kalitva.jsprav.ru': 'jsprav.ru',
 'rtischevo.jsprav.ru': 'jsprav.ru',
 'marks.jsprav.ru': 'jsprav.ru',
 'zvenigorod.jsprav.ru': 'jsprav.ru',
 'velikij-ustyug.jsprav.ru': 'jsprav.ru',
 'nizhnij-lomov.jsprav.ru': 'jsprav.ru',
 'meleuz.jsprav.ru': 'jsprav.ru',
 'morshansk.jsprav.ru': 'jsprav.ru',
 'venyov.jsprav.ru': 'jsprav.ru',
 'belokuriha.jsprav.ru': 'jsprav.ru',
 'minsk.jsprav.ru': 'jsprav.ru',
 'maloyaroslavets.jsprav.ru': 'jsprav.ru',
 'babaevo.jsprav.ru': 'jsprav.ru',
 'bodajbo.jsprav.ru': 'jsprav.ru',
 'alejsk.jsprav.ru': 'jsprav.ru',
 'yakutsk.jsprav.ru': 'jsprav.ru',
 'vichuga.jsprav.ru': 'jsprav.ru',
 'bahchisaraj.jsprav.ru': 'jsprav.ru',
 'mozhga.jsprav.ru': 'jsprav.ru',
 'dedovsk.jsprav.ru': 'jsprav.ru',
 'skopin.jsprav.ru': 'jsprav.ru',
 'yuzhno-sahalinsk.jsprav.ru': 'jsprav.ru',
 'chita.jsprav.ru': 'jsprav.ru',
 'korkino.jsprav.ru': 'jsprav.ru',
 'reutov.jsprav.ru': 'jsprav.ru',
 'privolzhsk.jsprav.ru': 'jsprav.ru',
 'volgorechensk.jsprav.ru': 'jsprav.ru',
 'blagoveschensk.jsprav.ru': 'jsprav.ru',
 'zavolzhe.jsprav.ru': 'jsprav.ru',
 'lebedyan.jsprav.ru': 'jsprav.ru',
 'ruza.jsprav.ru': 'jsprav.ru',
 'otradnoe.jsprav.ru': 'jsprav.ru',
 'staritsa.jsprav.ru': 'jsprav.ru',
 'vesegonsk.jsprav.ru': 'jsprav.ru',
 'uraj.jsprav.ru': 'jsprav.ru',
 'cherepanovo.jsprav.ru': 'jsprav.ru',
 'rostov.jsprav.ru': 'jsprav.ru',
 'kameshkovo.jsprav.ru': 'jsprav.ru',
 'syiktyivkar.jsprav.ru': 'jsprav.ru',
 'orsha.jsprav.ru': 'jsprav.ru',
 'novopavlovsk.jsprav.ru': 'jsprav.ru',
 'staraya-kupavna.jsprav.ru': 'jsprav.ru',
 'dolinsk.jsprav.ru': 'jsprav.ru',
 'gorodets.jsprav.ru': 'jsprav.ru',
 'agryiz.jsprav.ru': 'jsprav.ru',
 'borovsk.jsprav.ru': 'jsprav.ru',
 'zaozyorsk.jsprav.ru': 'jsprav.ru',
 'ahtubinsk.jsprav.ru': 'jsprav.ru',
 'uzhur.jsprav.ru': 'jsprav.ru',
    'hh.ru': 'hh.ru',
 'spb.hh.ru': 'hh.ru',
 'perm.hh.ru': 'hh.ru',
 'dzerzhinskij.hh.ru': 'hh.ru',
 'novgorod.hh.ru': 'hh.ru',
 'sochi.hh.ru': 'hh.ru',
 'naro-fominsk.hh.ru': 'hh.ru',
 'saratov.hh.ru': 'hh.ru',
 'samara.hh.ru': 'hh.ru',
 'krasnodar.hh.ru': 'hh.ru',
 'naberezhnye.hh.ru': 'hh.ru',
 'ufa.hh.ru': 'hh.ru',
 'novocheboksarsk.hh.ru': 'hh.ru',
 'cheboksary.hh.ru': 'hh.ru',
 'ekaterinburg.hh.ru': 'hh.ru',
 'khabarovsk.hh.ru': 'hh.ru',
 'nizhnevartovsk.hh.ru': 'hh.ru',
 'novosibirsk.hh.ru': 'hh.ru',
 'sevastopol.hh.ru': 'hh.ru',
 'stavropol.hh.ru': 'hh.ru',
 'vladivostok.hh.ru': 'hh.ru',
 'tuymazy.hh.ru': 'hh.ru',
 'tyumen.hh.ru': 'hh.ru',
 'ryazan.hh.ru': 'hh.ru',
 'kolomna.hh.ru': 'hh.ru',
 'obninsk.hh.ru': 'hh.ru',
 'domodedovo.hh.ru': 'hh.ru',
 'izhevsk.hh.ru': 'hh.ru',
 'kirov.hh.ru': 'hh.ru',
 'bolokhovo.hh.ru': 'hh.ru',
 'groznyj.hh.ru': 'hh.ru',
 'chelyabinsk.hh.ru': 'hh.ru',
 'kamensk-uralsky.hh.ru': 'hh.ru',
 'voronezh.hh.ru': 'hh.ru',
 'barnaul.hh.ru': 'hh.ru',
 'rostov.hh.ru': 'hh.ru',
 'aksai.hh.ru': 'hh.ru',
 'lobnya.hh.ru': 'hh.ru',
 'armavir.hh.ru': 'hh.ru',
 'krasnoyarsk.hh.ru': 'hh.ru',
 'chernushka.hh.ru': 'hh.ru',
 'kazan.hh.ru': 'hh.ru',
 'togliatti.hh.ru': 'hh.ru',
 'istria.hh.ru': 'hh.ru',
 'kostroma.hh.ru': 'hh.ru',
 'nn.hh.ru': 'hh.ru',
 'petrozavodsk.hh.ru': 'hh.ru',
 'achinsk.hh.ru': 'hh.ru',
 'nizhnekamsk.hh.ru': 'hh.ru',
 'chishmy.hh.ru': 'hh.ru',
 'orenburg.hh.ru': 'hh.ru',
 'elektrostal.hh.ru': 'hh.ru',
 'serov.hh.ru': 'hh.ru',
 'belorechensk.hh.ru': 'hh.ru',
 'pervouralsk.hh.ru': 'hh.ru',
 'yoshkar-ola.hh.ru': 'hh.ru',
 'vyborg.hh.ru': 'hh.ru',
 'novokuznetsk.hh.ru': 'hh.ru',
 'magadan.hh.ru': 'hh.ru',
 'pskov.hh.ru': 'hh.ru',
 'chita.hh.ru': 'hh.ru',
 'solikamsk.hh.ru': 'hh.ru',
 'belgorod.hh.ru': 'hh.ru',
 'krasnogorsk.hh.ru': 'hh.ru',
 'sterlitamak.hh.ru': 'hh.ru',
 'penza.hh.ru': 'hh.ru',
 'salavat.hh.ru': 'hh.ru',
 'elista.hh.ru': 'hh.ru',
 'lipetsk.hh.ru': 'hh.ru',
 'volgograd.hh.ru': 'hh.ru',
 'irkutsk.hh.ru': 'hh.ru',
 'khimki.hh.ru': 'hh.ru',
 'livny.hh.ru': 'hh.ru',
 'yaroslavl.hh.ru': 'hh.ru',
 'tver.hh.ru': 'hh.ru',
 'vologda.hh.ru': 'hh.ru',
 'oktyabrskij.hh.ru': 'hh.ru',
 'stary-oskol.hh.ru': 'hh.ru',
 'aldan.hh.ru': 'hh.ru',
 'tomsk.hh.ru': 'hh.ru',
 'kemerovo.hh.ru': 'hh.ru',
 'chernogorsk.hh.ru': 'hh.ru',
 'satka.hh.ru': 'hh.ru',
 'prokopyevsk.hh.ru': 'hh.ru',
 'safonovo.hh.ru': 'hh.ru',
 'kurgan.hh.ru': 'hh.ru',
 'murmansk.hh.ru': 'hh.ru',
 'anapa.hh.ru': 'hh.ru',
 'tuapse.hh.ru': 'hh.ru',
 'bronnitsy.hh.ru': 'hh.ru',
 'magnitogorsk.hh.ru': 'hh.ru',
 'volgodonsk.hh.ru': 'hh.ru',
 'chernogolovka.hh.ru': 'hh.ru',
 'klimovsk.hh.ru': 'hh.ru',
 'pyatigorsk.hh.ru': 'hh.ru',
 'kosteryovo.hh.ru': 'hh.ru',
 'norilsk.hh.ru': 'hh.ru',
 'dolgoprudny.hh.ru': 'hh.ru',
 'korkino.hh.ru': 'hh.ru',
 'nizhny-tagil.hh.ru': 'hh.ru',
 'dzhubga.hh.ru': 'hh.ru',
 'neftekamsk.hh.ru': 'hh.ru',
 'novorossiysk.hh.ru': 'hh.ru',
 'slavyansk-na-kubani.hh.ru': 'hh.ru',
 'kaluga.hh.ru': 'hh.ru',
 'omsk.hh.ru': 'hh.ru',
 'bobrov.hh.ru': 'hh.ru',
 'estonia.hh.ru': 'hh.ru',
 'smolensk.hh.ru': 'hh.ru',
 'salekhard.hh.ru': 'hh.ru',
 'serpukhov.hh.ru': 'hh.ru',
 'tula.hh.ru': 'hh.ru',
 'podolsk.hh.ru': 'hh.ru',
 'odintsovo.hh.ru': 'hh.ru',
 'zelenograd.hh.ru': 'hh.ru',
 'michurinsk.hh.ru': 'hh.ru',
 'svobodnyj.hh.ru': 'hh.ru',
 'bryansk.hh.ru': 'hh.ru',
 'almetyevsk.hh.ru': 'hh.ru',
 'srednyaya-ahtuba.hh.ru': 'hh.ru',
 'volzhskiy.hh.ru': 'hh.ru',
 'blagoveschensk.hh.ru': 'hh.ru',
 'vyksa.hh.ru': 'hh.ru',
 'ukhta.hh.ru': 'hh.ru',
 'gatchina.hh.ru': 'hh.ru',
 'komsomolsk-na-amure.hh.ru': 'hh.ru',
 'ulyanovsk.hh.ru': 'hh.ru',
 'zheleznogorsk-krasnoyarsk.hh.ru': 'hh.ru',
 'kursk.hh.ru': 'hh.ru',
 'ramenskoye.hh.ru': 'hh.ru',
 'mytischi.hh.ru': 'hh.ru',
 'voskresensk.hh.ru': 'hh.ru',
 'beloozerskiy.hh.ru': 'hh.ru',
 'kaliningrad.hh.ru': 'hh.ru',
 'verkhnyaya-pyshma.hh.ru': 'hh.ru',
 'vyazma.hh.ru': 'hh.ru',
 'krasnoznamensk.hh.ru': 'hh.ru',
 'priozersk.hh.ru': 'hh.ru',
 'gelendzhik.hh.ru': 'hh.ru',
 'tambov.hh.ru': 'hh.ru',
 'ivanovo.hh.ru': 'hh.ru',
 'kstovo.hh.ru': 'hh.ru',
 'solnechnogorsk.hh.ru': 'hh.ru',
 'pushkino.hh.ru': 'hh.ru',
 'zlatoust.hh.ru': 'hh.ru',
 'baltiysk.hh.ru': 'hh.ru',
 'dalnerechensk.hh.ru': 'hh.ru',
 'kolpino.hh.ru': 'hh.ru',
 'uae.hh.ru': 'hh.ru',
 'germany.hh.ru': 'hh.ru',
 'canada.hh.ru': 'hh.ru',
 'neryungri.hh.ru': 'hh.ru',
 'noyabrsk.hh.ru': 'hh.ru',
 'syktyvkar.hh.ru': 'hh.ru',
 'yurga.hh.ru': 'hh.ru',
 'vladimir.hh.ru': 'hh.ru',
 'orel.hh.ru': 'hh.ru',
 'moskovskij.hh.ru': 'hh.ru',
 'orsk.hh.ru': 'hh.ru',
 'artem.hh.ru': 'hh.ru',
 'stupino.hh.ru': 'hh.ru',
 'troitsk-chelyabinsk.hh.ru': 'hh.ru',
 'troitsk.hh.ru': 'hh.ru',
 'protvino.hh.ru': 'hh.ru',
 'scherbinka.hh.ru': 'hh.ru',
 'iskitim.hh.ru': 'hh.ru',
 'klin.hh.ru': 'hh.ru',
 'rahya.hh.ru': 'hh.ru',
 'yelizovo.hh.ru': 'hh.ru',
 'gulkevichi.hh.ru': 'hh.ru',
 'asha.hh.ru': 'hh.ru',
 'novoshakhtinsk.hh.ru': 'hh.ru',
 'saransk.hh.ru': 'hh.ru',
 'temryuk.hh.ru': 'hh.ru',
 'usinsk.hh.ru': 'hh.ru',
 'kanevskaya.hh.ru': 'hh.ru',
 'adler.hh.ru': 'hh.ru',
 'makhachkala.hh.ru': 'hh.ru',
 'tikhoretsk.hh.ru': 'hh.ru',
 'sortavala.hh.ru': 'hh.ru',
 'schelkovo.hh.ru': 'hh.ru',
 'surgut.hh.ru': 'hh.ru',
 'skovorodino.hh.ru': 'hh.ru',
 'ishimbaj.hh.ru': 'hh.ru',
 'nyagan.hh.ru': 'hh.ru',
 'nefteyugansk.hh.ru': 'hh.ru',
 'tomilino.hh.ru': 'hh.ru',
 'dubna.hh.ru': 'hh.ru',
 'koshki.hh.ru': 'hh.ru',
 'essentuki.hh.ru': 'hh.ru',
 'kovrov.hh.ru': 'hh.ru',
 'labinsk.hh.ru': 'hh.ru',
 'giaginskaya.hh.ru': 'hh.ru',
 'taganrog.hh.ru': 'hh.ru',
 'dubrovka47.hh.ru': 'hh.ru',
 'nakhodka.hh.ru': 'hh.ru',
 'kirovsk-murmansk.hh.ru': 'hh.ru',
 'noginsk.hh.ru': 'hh.ru',
 'abakan.hh.ru': 'hh.ru',
 'shuya.hh.ru': 'hh.ru',
 'astrakhan.hh.ru': 'hh.ru',
 'cyprus.hh.ru': 'hh.ru',
 'vidnoe.hh.ru': 'hh.ru',
 'balashikha.hh.ru': 'hh.ru',
 'orekhovo-zuevo.hh.ru': 'hh.ru',
 'petushki.hh.ru': 'hh.ru',
 'mtsensk.hh.ru': 'hh.ru',
 'azov.hh.ru': 'hh.ru',
 'bykovo50.hh.ru': 'hh.ru',
 'udelnaya.hh.ru': 'hh.ru',
 'zhukovsky.hh.ru': 'hh.ru',
 'sabetta.hh.ru': 'hh.ru',
 'turkey.hh.ru': 'hh.ru',
 'simferopol.hh.ru': 'hh.ru',
 'yakutsk.hh.ru': 'hh.ru',
 'yuzhno-sakhalinsk.hh.ru': 'hh.ru',
 'kirovsk-leningrad.hh.ru': 'hh.ru',
 'nevinnomyssk.hh.ru': 'hh.ru',
 'krasnokamsk.hh.ru': 'hh.ru',
 'mizhhiria.hh.ru': 'hh.ru',
 'bilogorsk.hh.ru': 'hh.ru',
 'mineralnye-vody.hh.ru': 'hh.ru',
 'tsivilsk.hh.ru': 'hh.ru',
 'borisoglebsk.hh.ru': 'hh.ru',
 'zainsk.hh.ru': 'hh.ru',
 'fryazino.hh.ru': 'hh.ru',
 'ilskiy.hh.ru': 'hh.ru',
 'afipskiy.hh.ru': 'hh.ru',
 'murom.hh.ru': 'hh.ru',
 'syzran.hh.ru': 'hh.ru',
 'zheleznodorozhnyj.hh.ru': 'hh.ru',
 'aykino.hh.ru': 'hh.ru',
 'anadyr.hh.ru': 'hh.ru',
 'novyj-urengoy.hh.ru': 'hh.ru',
 'petropavlovsk-kamchatskij.hh.ru': 'hh.ru',
 'minusinsk.hh.ru': 'hh.ru',
 'luga.hh.ru': 'hh.ru',
 'shatura.hh.ru': 'hh.ru',
 'kurchatov.hh.ru': 'hh.ru',
 'sibai.hh.ru': 'hh.ru',
 'staraya-kupavna.hh.ru': 'hh.ru',
 'berezovsky-sverdlovsk.hh.ru': 'hh.ru',
 'chusovoi.hh.ru': 'hh.ru',
 'arkhangelsk.hh.ru': 'hh.ru',
 'dobryanka.hh.ru': 'hh.ru',
 'yubileyniy.hh.ru': 'hh.ru',
 'kinel.hh.ru': 'hh.ru',
 'neftegorsk.hh.ru': 'hh.ru',
 'busuluk.hh.ru': 'hh.ru',
 'otradniy.hh.ru': 'hh.ru',
 'biysk.hh.ru': 'hh.ru',
 'ust-nera.hh.ru': 'hh.ru',
 'kurganinsk.hh.ru': 'hh.ru',
 'cherepovets.hh.ru': 'hh.ru',
 'obuhovo.hh.ru': 'hh.ru',
 'kotlas.hh.ru': 'hh.ru',
 'maikop.hh.ru': 'hh.ru',
 'krymsk.hh.ru': 'hh.ru',
 'aprelevka.hh.ru': 'hh.ru',
 'dalnegorsk.hh.ru': 'hh.ru',
 'zmeinogorsk.hh.ru': 'hh.ru',
 'apatity.hh.ru': 'hh.ru',
 'gorno-altaisk.hh.ru': 'hh.ru',
 'novayausman.hh.ru': 'hh.ru',
 'luhovitsy.hh.ru': 'hh.ru',
 'industrialniy.hh.ru': 'hh.ru',
 'leninogorsk.hh.ru': 'hh.ru',
 'semikarakorsk.hh.ru': 'hh.ru',
 'kislovodsk.hh.ru': 'hh.ru',
 'novocherkassk.hh.ru': 'hh.ru',
 'tlyustenhabl.hh.ru': 'hh.ru',
 'sergievposad.hh.ru': 'hh.ru',
 'liski.hh.ru': 'hh.ru',
 'konakovo.hh.ru': 'hh.ru',
 'svetlogorsk.hh.ru': 'hh.ru',
 'pioneer.hh.ru': 'hh.ru',
 'berezniki.hh.ru': 'hh.ru',
 'gorodische.hh.ru': 'hh.ru',
 'kerch.hh.ru': 'hh.ru',
 'uglich.hh.ru': 'hh.ru',
 'marks.hh.ru': 'hh.ru',
 'menzelinsk.hh.ru': 'hh.ru',
 'nevyansk.hh.ru': 'hh.ru',
 'zapolyarniy.hh.ru': 'hh.ru',
 'ivanteevka.hh.ru': 'hh.ru',
 'tasks.hh.ru': 'hh.ru',
 'karmaskaly.hh.ru': 'hh.ru',
 'zvenigorod.hh.ru': 'hh.ru',
 'bataisk.hh.ru': 'hh.ru',
 'schekino.hh.ru': 'hh.ru',
 'krasny-kut.hh.ru': 'hh.ru',
 'kostomuksha.hh.ru': 'hh.ru',
 'lazarevskoe.hh.ru': 'hh.ru',
 'yablonovskiy01.hh.ru': 'hh.ru',
 'krasnaya-polyana23.hh.ru': 'hh.ru',
 'zeya.hh.ru': 'hh.ru',
 'derbent.hh.ru': 'hh.ru',
 'vladikavkaz.hh.ru': 'hh.ru',
 'sukhinichi.hh.ru': 'hh.ru',
 'golitsino.hh.ru': 'hh.ru',
 'kronshtadt.hh.ru': 'hh.ru',
 'kansk.hh.ru': 'hh.ru',
 'zaraysk.hh.ru': 'hh.ru',
 'pavlovo.hh.ru': 'hh.ru',
 'kamyshin.hh.ru': 'hh.ru',
 'severo-eniseyskiy.hh.ru': 'hh.ru',
 'tarko-sale.hh.ru': 'hh.ru',
 'prikubanskiy.hh.ru': 'hh.ru',
 'novoaltajsk.hh.ru': 'hh.ru',
 'dzerzhinsk.hh.ru': 'hh.ru',
 'yakhroma.hh.ru': 'hh.ru',
 'kubinka.hh.ru': 'hh.ru',
 'seversk.hh.ru': 'hh.ru',
 'pervomayskiy56.hh.ru': 'hh.ru',
 'sokolniki.hh.ru': 'hh.ru',
 'sobinka.hh.ru': 'hh.ru',
 'yevpatoriya.hh.ru': 'hh.ru',
 'bugulma.hh.ru': 'hh.ru',
 'balashov.hh.ru': 'hh.ru',
 'lesnye-polyany.hh.ru': 'hh.ru',
 'krasnoturinsk.hh.ru': 'hh.ru',
 'arzamas.hh.ru': 'hh.ru',
 'sayanogorsk.hh.ru': 'hh.ru',
 'kurilsk.hh.ru': 'hh.ru',
 'zernograd.hh.ru': 'hh.ru',
 'votkinsk.hh.ru': 'hh.ru',
 'arhipo-osipovka.hh.ru': 'hh.ru',
 'yalta.hh.ru': 'hh.ru',
 'vorkuta.hh.ru': 'hh.ru',
 'boguchar.hh.ru': 'hh.ru',
 'severodvinsk.hh.ru': 'hh.ru',
 'zhukov.hh.ru': 'hh.ru',
 'shales.hh.ru': 'hh.ru',
 'schlisselburg.hh.ru': 'hh.ru',
 'kropotkin.hh.ru': 'hh.ru',
 'aysha.hh.ru': 'hh.ru',
 'miass.hh.ru': 'hh.ru',
 'murino.hh.ru': 'hh.ru',
 'czech.hh.ru': 'hh.ru',
 'belogorsk.hh.ru': 'hh.ru',
 'zavolzhie.hh.ru': 'hh.ru',
 'sredneuralsk.hh.ru': 'hh.ru',
 'adygeysk.hh.ru': 'hh.ru',
 'koryazhma.hh.ru': 'hh.ru',
 'ruza.hh.ru': 'hh.ru',
 'ulan-ude.hh.ru': 'hh.ru',
 'novoivanovskoe.hh.ru': 'hh.ru',
 'chapayevsk.hh.ru': 'hh.ru',
 'kolchugino.hh.ru': 'hh.ru',
 'amursk.hh.ru': 'hh.ru',
 'tikhvin.hh.ru': 'hh.ru',
 'roslavl.hh.ru': 'hh.ru',
 'innopolis.hh.ru': 'hh.ru',
 'leninsk-kuznetsk.hh.ru': 'hh.ru',
 'bezenchuk.hh.ru': 'hh.ru',
 'pyt-yah.hh.ru': 'hh.ru',
 'dmitrov.hh.ru': 'hh.ru',
 'tynda.hh.ru': 'hh.ru',
 'chistopol.hh.ru': 'hh.ru',
 'bodaybo.hh.ru': 'hh.ru',
 'bachatskiy.hh.ru': 'hh.ru',
 'thailand.hh.ru': 'hh.ru',
 'glazov.hh.ru': 'hh.ru',
 'nalchik.hh.ru': 'hh.ru',
 'birobidzhan.hh.ru': 'hh.ru',
 'vengerovo.hh.ru': 'hh.ru',
 'chebarkul.hh.ru': 'hh.ru',
 'engels.hh.ru': 'hh.ru',
 'saratovskaya.hh.ru': 'hh.ru',
 'shakhty.hh.ru': 'hh.ru',
 'yelets.hh.ru': 'hh.ru',
 'novotitarovskaya.hh.ru': 'hh.ru',
 'apsheronsk.hh.ru': 'hh.ru',
 'yelabuga.hh.ru': 'hh.ru',
 'bor.hh.ru': 'hh.ru',
 'belovo.hh.ru': 'hh.ru',
 'tazovskiy.hh.ru': 'hh.ru',
 'cherm.hh.ru': 'hh.ru',
 'muravlenko.hh.ru': 'hh.ru',
 'gubkinskij.hh.ru': 'hh.ru',
 'medvezhyi-ozera.hh.ru': 'hh.ru',
 'iglino.hh.ru': 'hh.ru',
 'kultuk.hh.ru': 'hh.ru',
 'pavlovsky-posad.hh.ru': 'hh.ru',
 'raduzhnyj-yugra.hh.ru': 'hh.ru',
 'mezhdurechensk.hh.ru': 'hh.ru',
 'novomoskovsk.hh.ru': 'hh.ru',
 'rubtsovsk.hh.ru': 'hh.ru',
 'khanty-mansiysk.hh.ru': 'hh.ru',
 'taman.hh.ru': 'hh.ru',
 'zarinsk.hh.ru': 'hh.ru',
 'komsomolsk.hh.ru': 'hh.ru',
 'berdsk.hh.ru': 'hh.ru',
 'revda.hh.ru': 'hh.ru',
 'gagarin.hh.ru': 'hh.ru',
 'kameshkovo.hh.ru': 'hh.ru',
 'kamyshlov.hh.ru': 'hh.ru',
 'kingisepp.hh.ru': 'hh.ru',
 'ust-luga.hh.ru': 'hh.ru',
 'tuchkovo.hh.ru': 'hh.ru',
 'rezh.hh.ru': 'hh.ru',
 'nemchinovka.hh.ru': 'hh.ru',
 'sarapul.hh.ru': 'hh.ru',
 'nikel.hh.ru': 'hh.ru',
 'alekseevskoe.hh.ru': 'hh.ru',
 'novokuybyshevsk.hh.ru': 'hh.ru',
 'raevskiy.hh.ru': 'hh.ru',
 'lomonosov.hh.ru': 'hh.ru',
 'petergof.hh.ru': 'hh.ru',
 'nahabino.hh.ru': 'hh.ru',
 'gorodishche.hh.ru': 'hh.ru',
 'gurievsk-kaliningrad.hh.ru': 'hh.ru',
 'ladozhskaya.hh.ru': 'hh.ru',
 'mga.hh.ru': 'hh.ru',
 'stroitel.hh.ru': 'hh.ru',
 'sestroretsk.hh.ru': 'hh.ru',
 'ust-ilimsk.hh.ru': 'hh.ru',
 'lyantor.hh.ru': 'hh.ru',
 'mikhajlovsk-stavropol.hh.ru': 'hh.ru',
 'mozhaisk.hh.ru': 'hh.ru',
 'vyshny-volochok.hh.ru': 'hh.ru',
 'velikie-luki.hh.ru': 'hh.ru',
 'efremov.hh.ru': 'hh.ru',
 'zelenogorsk47.hh.ru': 'hh.ru',
 'stolbishche.hh.ru': 'hh.ru',
 'alexandrov.hh.ru': 'hh.ru',
 'ipatovo.hh.ru': 'hh.ru',
 'dimitrovgrad.hh.ru': 'hh.ru',
 'kashira.hh.ru': 'hh.ru',
 'polazna.hh.ru': 'hh.ru',
 'mirny-yakutiya.hh.ru': 'hh.ru',
 'abinsk.hh.ru': 'hh.ru',
 'sofrino.hh.ru': 'hh.ru',
 'kamensk-shakhtinsky.hh.ru': 'hh.ru',
 'dagomys.hh.ru': 'hh.ru',
 'ruzaevka.hh.ru': 'hh.ru',
 'podporozhye.hh.ru': 'hh.ru',
 'vyazniki.hh.ru': 'hh.ru',
 'china.hh.ru': 'hh.ru',
 'aleksin.hh.ru': 'hh.ru',
 'dinskaya.hh.ru': 'hh.ru',
 'vasyurinskaya.hh.ru': 'hh.ru',
 'mozhga.hh.ru': 'hh.ru',
 'pushkin.hh.ru': 'hh.ru',
 'south-korea.hh.ru': 'hh.ru',
 'barabinsk.hh.ru': 'hh.ru',
 'langepas.hh.ru': 'hh.ru',
 'rostov-velikij.hh.ru': 'hh.ru',
 'altayskoe.hh.ru': 'hh.ru',
 'tchaikovsky.hh.ru': 'hh.ru',
 'usa.hh.ru': 'hh.ru',
 'sosnogorsk.hh.ru': 'hh.ru',
 'zavyalovo18.hh.ru': 'hh.ru',
 'ehstosadok.hh.ru': 'hh.ru',
 'usady.hh.ru': 'hh.ru',
 'krasnoe-selo.hh.ru': 'hh.ru',
 'mendeleevsk.hh.ru': 'hh.ru',
 'shimanovsk.hh.ru': 'hh.ru',
 'gaspra.hh.ru': 'hh.ru',
 'vityazevo.hh.ru': 'hh.ru',
 'koktebel.hh.ru': 'hh.ru',
 'gryazovets.hh.ru': 'hh.ru',
 'buj.hh.ru': 'hh.ru',
 'redkino69.hh.ru': 'hh.ru',
 'yuriev-polskij.hh.ru': 'hh.ru',
 'peresvet.hh.ru': 'hh.ru',
 'best.hh.ru': 'hh.ru',
 'elektrougli.hh.ru': 'hh.ru',
 'sergach.hh.ru': 'hh.ru',
 'kimry.hh.ru': 'hh.ru',
 'spassk-ryazan.hh.ru': 'hh.ru',
 'hor.hh.ru': 'hh.ru',
 'zagoryanskiy.hh.ru': 'hh.ru',
 'svobodniy.hh.ru': 'hh.ru',
 'skopin.hh.ru': 'hh.ru',
 'bangladesh.hh.ru': 'hh.ru',
 'novovoronezh.hh.ru': 'hh.ru',
 'povarovo.hh.ru': 'hh.ru',
 'timashevsk.hh.ru': 'hh.ru',
 'budennovsk.hh.ru': 'hh.ru',
 'dedovsk.hh.ru': 'hh.ru',
 'gryazi.hh.ru': 'hh.ru',
 'nerekhta.hh.ru': 'hh.ru',
 'kabardinka.hh.ru': 'hh.ru',
 'pyshma.hh.ru': 'hh.ru',
 'vyatskiye-polyana.hh.ru': 'hh.ru',
 'sudak.hh.ru': 'hh.ru',
 'lytkarino.hh.ru': 'hh.ru',
 'chernomorskoe.hh.ru': 'hh.ru',
 'raduzhnyj-vladimir.hh.ru': 'hh.ru',
 'rtischevo.hh.ru': 'hh.ru',
 'davydovo.hh.ru': 'hh.ru',
 'arseniev.hh.ru': 'hh.ru',
 'pavlovskaya.hh.ru': 'hh.ru',
 'pavlovskaya-sloboda.hh.ru': 'hh.ru',
 'tatsinskaya.hh.ru': 'hh.ru',
 'zhigulevsk.hh.ru': 'hh.ru',
 'puschino.hh.ru': 'hh.ru',
 'ishim.hh.ru': 'hh.ru',
 'vysokovsk.hh.ru': 'hh.ru',
 'kyzyl.hh.ru': 'hh.ru',
 'birsk.hh.ru': 'hh.ru',
 'pereslavl.hh.ru': 'hh.ru',
 'yartsevo.hh.ru': 'hh.ru',
 'blagodarnyj.hh.ru': 'hh.ru',
 'shadrinsk.hh.ru': 'hh.ru',
 'labytnangi.hh.ru': 'hh.ru',
 'tsiolkovskiy.hh.ru': 'hh.ru',
 'egorlykskaya.hh.ru': 'hh.ru',
 'yalutorovsk.hh.ru': 'hh.ru',
 'balabanovo.hh.ru': 'hh.ru',
 'vorsino.hh.ru': 'hh.ru',
 'tim.hh.ru': 'hh.ru',
 'kotovsk.hh.ru': 'hh.ru',
 'rybinsk.hh.ru': 'hh.ru',
 'reutov.hh.ru': 'hh.ru',
 'uzhur.hh.ru': 'hh.ru',
 'kotelniki.hh.ru': 'hh.ru',
 'ivdel.hh.ru': 'hh.ru',
 'primorsko-akhtarsk.hh.ru': 'hh.ru',
 'kopeysk.hh.ru': 'hh.ru',
 'cherkessk.hh.ru': 'hh.ru',
 'megion.hh.ru': 'hh.ru',
 'kaspiysk.hh.ru': 'hh.ru',
 'korenovsk.hh.ru': 'hh.ru',
 'zapolyarnyj.hh.ru': 'hh.ru',
 'sosnoviy-bor47.hh.ru': 'hh.ru',
 'kirovo-chepetsk.hh.ru': 'hh.ru',
 'armenia.hh.ru': 'hh.ru',
 'krasnaya-gorka.hh.ru': 'hh.ru',
 'isyangulovo.hh.ru': 'hh.ru',
 'zavodoukovsk.hh.ru': 'hh.ru',
 'aramil.hh.ru': 'hh.ru',
 'vlasiha50.hh.ru': 'hh.ru',
 'barviha.hh.ru': 'hh.ru',
 'rodniki.hh.ru': 'hh.ru',
 'krasnogvardeyskoe.hh.ru': 'hh.ru',
 'kushwa.hh.ru': 'hh.ru',
 'alekseevka63.hh.ru': 'hh.ru',
 'alekseevka.hh.ru': 'hh.ru',
 'dankov.hh.ru': 'hh.ru',
 'tosno.hh.ru': 'hh.ru',
 'ussuriysk.hh.ru': 'hh.ru',
 'gorniy75.hh.ru': 'hh.ru',
 'demino.hh.ru': 'hh.ru',
 'novokhopersk.hh.ru': 'hh.ru',
 'belebey.hh.ru': 'hh.ru',
 'aginskoye.hh.ru': 'hh.ru',
 'zelenodolsk.hh.ru': 'hh.ru',
 'sukhoi-log.hh.ru': 'hh.ru',
 'kuznetsk.hh.ru': 'hh.ru',
 'ehnem.hh.ru': 'hh.ru',
 'monchegorsk.hh.ru': 'hh.ru',
 'france.hh.ru': 'hh.ru',
 'kimovsk.hh.ru': 'hh.ru',
 'kartaly.hh.ru': 'hh.ru',
 'zadonsk.hh.ru': 'hh.ru',
 'ust-kut.hh.ru': 'hh.ru',
 'ostrogozhsk.hh.ru': 'hh.ru',
 'vinzili.hh.ru': 'hh.ru',
 'angarsk.hh.ru': 'hh.ru',
 'arhangelskaya.hh.ru': 'hh.ru',
 'udomlya.hh.ru': 'hh.ru',
 'zvenigovo.hh.ru': 'hh.ru',
 'trudovoe.hh.ru': 'hh.ru',
 'sertolovo.hh.ru': 'hh.ru',
 'kogalym.hh.ru': 'hh.ru',
 'lensk.hh.ru': 'hh.ru',
 'nogliki.hh.ru': 'hh.ru',
 'pereyaslavka.hh.ru': 'hh.ru',
 'ola.hh.ru': 'hh.ru',
 'likino-dulyovo.hh.ru': 'hh.ru',
 'koltushi.hh.ru': 'hh.ru',
 'rasskazovo.hh.ru': 'hh.ru',
 'salsk.hh.ru': 'hh.ru',
 'vniissok.hh.ru': 'hh.ru',
 'ryazhsk.hh.ru': 'hh.ru',
 'sim.hh.ru': 'hh.ru',
 'serbia.hh.ru': 'hh.ru',
 'tatarsk.hh.ru': 'hh.ru',
 'urengoy.hh.ru': 'hh.ru',
 'taishet.hh.ru': 'hh.ru',
 'kandry.hh.ru': 'hh.ru',
 'mezhozerniy.hh.ru': 'hh.ru',
 'segezha.hh.ru': 'hh.ru',
 'lyudinovo.hh.ru': 'hh.ru',
 'yuzhniy.hh.ru': 'hh.ru',
 'novokubansk.hh.ru': 'hh.ru',
 'zelenokumsk.hh.ru': 'hh.ru',
 'ob.hh.ru': 'hh.ru',
 'kuibyshev.hh.ru': 'hh.ru',
 'marfino50.hh.ru': 'hh.ru',
 'kommunar.hh.ru': 'hh.ru',
 'kamenka.hh.ru': 'hh.ru',
 'alushta.hh.ru': 'hh.ru',
 'ordynskoe.hh.ru': 'hh.ru',
 'kumertau.hh.ru': 'hh.ru',
 'novomichurinsk.hh.ru': 'hh.ru',
 'konstantinovsk.hh.ru': 'hh.ru',
 'magas.hh.ru': 'hh.ru',
 'hotkovo.hh.ru': 'hh.ru',
 'volokolamsk.hh.ru': 'hh.ru',
 'abrau-dyurso.hh.ru': 'hh.ru',
 'nevel.hh.ru': 'hh.ru',
 'bezhetsk.hh.ru': 'hh.ru',
 'kaltan.hh.ru': 'hh.ru',
 'pokhvistnevo.hh.ru': 'hh.ru',
 'belaya-kalitva.hh.ru': 'hh.ru',
 'medvezhjegorsk.hh.ru': 'hh.ru',
 'mokshan.hh.ru': 'hh.ru',
 'chunskiy.hh.ru': 'hh.ru',
 'revda51.hh.ru': 'hh.ru',
 'kineshma.hh.ru': 'hh.ru',
 'kirovskoe.hh.ru': 'hh.ru',
 'agryz.hh.ru': 'hh.ru',
 'novoomskiy.hh.ru': 'hh.ru',
 'slavgorod.hh.ru': 'hh.ru',
 'zarechnyj.hh.ru': 'hh.ru',
 'zarechny.hh.ru': 'hh.ru',
 'talmenka.hh.ru': 'hh.ru',
 'yeisk.hh.ru': 'hh.ru',
 'borovichi.hh.ru': 'hh.ru',
 'metallostroy.hh.ru': 'hh.ru',
 'finland.hh.ru': 'hh.ru',
 'balaklava.hh.ru': 'hh.ru',
 'morshansk.hh.ru': 'hh.ru',
 'aznakayevo.hh.ru': 'hh.ru',
 'sloboda.hh.ru': 'hh.ru',
 'kungur.hh.ru': 'hh.ru',
 'novotroitsk.hh.ru': 'hh.ru',
 'polevskoy.hh.ru': 'hh.ru',
 'moskalenki.hh.ru': 'hh.ru',
 'ust-labinsk.hh.ru': 'hh.ru',
 'meleuz.hh.ru': 'hh.ru',
 'lesosibirsk.hh.ru': 'hh.ru',
 'okha.hh.ru': 'hh.ru',
 'yuzhnouralsk.hh.ru': 'hh.ru',
 'torzhok.hh.ru': 'hh.ru',
 'priyutovo.hh.ru': 'hh.ru',
 'bogorodsk.hh.ru': 'hh.ru',
 'vilyuchinsk.hh.ru': 'hh.ru',
 'artemovskiy.hh.ru': 'hh.ru',
 'shushary.hh.ru': 'hh.ru',
 'kasimov.hh.ru': 'hh.ru',
 'kholmsk.hh.ru': 'hh.ru',
 'alatyr.hh.ru': 'hh.ru',
 'kukmor.hh.ru': 'hh.ru',
 'kondrovo.hh.ru': 'hh.ru',
 'lysva.hh.ru': 'hh.ru',
 'ivanovskaya.hh.ru': 'hh.ru',
 'privolzhsk.hh.ru': 'hh.ru',
 'kalininsk.hh.ru': 'hh.ru',
 'maloyaroslavets.hh.ru': 'hh.ru',
 'bogotol.hh.ru': 'hh.ru',
 'krasnoufimsk.hh.ru': 'hh.ru',
 'pervomayskoe.hh.ru': 'hh.ru',
 'ilan.hh.ru': 'hh.ru',
 'valuiki.hh.ru': 'hh.ru',
 'buinaksk.hh.ru': 'hh.ru',
 'beliy-yar.hh.ru': 'hh.ru',
 'slavyanka47.hh.ru': 'hh.ru',
 'gusev.hh.ru': 'hh.ru',
 'yanaul.hh.ru': 'hh.ru',
 'vysotsk.hh.ru': 'hh.ru',
 'otradnoe.hh.ru': 'hh.ru',
 'akhtubinsk.hh.ru': 'hh.ru',
 'vorotynsk.hh.ru': 'hh.ru',
 'desnogorsk.hh.ru': 'hh.ru',
 'blagoveschensk-bashkortostan.hh.ru': 'hh.ru',
 'bogdanovich.hh.ru': 'hh.ru',
 'sokol.hh.ru': 'hh.ru',
 'dikson.hh.ru': 'hh.ru',
 'kudrovo.hh.ru': 'hh.ru',
 'koltsovo.hh.ru': 'hh.ru',
 'igarka.hh.ru': 'hh.ru',
 'konstantinovka.hh.ru': 'hh.ru',
 'sirius.hh.ru': 'hh.ru',
 'novouralsk.hh.ru': 'hh.ru',
 'lakinsk.hh.ru': 'hh.ru',
 'ubinskoe.hh.ru': 'hh.ru',
 'kanash.hh.ru': 'hh.ru',
 'zvezdniy-gorodok.hh.ru': 'hh.ru',
 'frolovo.hh.ru': 'hh.ru',
 'ekaterinoslavka.hh.ru': 'hh.ru',
 'spas-klepiki.hh.ru': 'hh.ru',
 'yuruzan.hh.ru': 'hh.ru',
 'starovelichkovskaya.hh.ru': 'hh.ru',
 'otradnoe36.hh.ru': 'hh.ru',
 'kirensk.hh.ru': 'hh.ru',
 'klincy.hh.ru': 'hh.ru',
 'osinovo.hh.ru': 'hh.ru',
 'sharjah.hh.ru': 'hh.ru',
 'gurzuf.hh.ru': 'hh.ru',
 'simeiz.hh.ru': 'hh.ru',
 'petukhovo.hh.ru': 'hh.ru',
 'severomorsk.hh.ru': 'hh.ru',
 'uchaly.hh.ru': 'hh.ru',
 'snezhnogorsk.hh.ru': 'hh.ru',
 'mihaylovka25.hh.ru': 'hh.ru',
 'tsementniy.hh.ru': 'hh.ru',
 'georgievsk.hh.ru': 'hh.ru',
 'nyandoma.hh.ru': 'hh.ru',
 'teykovo.hh.ru': 'hh.ru',
 'fokino-primorje.hh.ru': 'hh.ru',
 'berezovsky-kemerov.hh.ru': 'hh.ru',
 'vyselki.hh.ru': 'hh.ru',
 'mikhajlovsk-sverdlovsk.hh.ru': 'hh.ru',
 'gus-khrustalnyj.hh.ru': 'hh.ru',
 'ustyuzhna.hh.ru': 'hh.ru',
 'sysert.hh.ru': 'hh.ru',
 'uzlovaya.hh.ru': 'hh.ru',
 'poland.hh.ru': 'hh.ru',
 'rossosh.hh.ru': 'hh.ru',
 'novoaleksandrovsk.hh.ru': 'hh.ru',
 'klyuchi41.hh.ru': 'hh.ru',
 'goryachij-klyuch.hh.ru': 'hh.ru',
 'poselok-lenina.hh.ru': 'hh.ru',
 'razumnoe.hh.ru': 'hh.ru',
 'sosnovoborsk.hh.ru': 'hh.ru',
 'agronom.hh.ru': 'hh.ru',
 'buinsk.hh.ru': 'hh.ru',
 'kodinsk.hh.ru': 'hh.ru',
 'mayma.hh.ru': 'hh.ru',
 'balahna.hh.ru': 'hh.ru',
 'naryshkino.hh.ru': 'hh.ru',
 'yugorsk.hh.ru': 'hh.ru',
 'svirsk.hh.ru': 'hh.ru',
 'kamenka47.hh.ru': 'hh.ru',
 'furmanov.hh.ru': 'hh.ru',
 'chelbasskaya.hh.ru': 'hh.ru',
 'kamennogorsk.hh.ru': 'hh.ru',
 'bolshaya-izhora.hh.ru': 'hh.ru',
 'elizavetinskaya.hh.ru': 'hh.ru',
 'shchedrino.hh.ru': 'hh.ru',
 'monino.hh.ru': 'hh.ru',
 'moskovskiy72.hh.ru': 'hh.ru',
 'priladozhskiy.hh.ru': 'hh.ru',
 'gukovo.hh.ru': 'hh.ru',
 'zverevo.hh.ru': 'hh.ru',
 'naryan-mar.hh.ru': 'hh.ru',
 'chernyakhovsk.hh.ru': 'hh.ru',
 'divnogorsk.hh.ru': 'hh.ru',
 'novaya-lyalya.hh.ru': 'hh.ru',
 'turinsk.hh.ru': 'hh.ru',
 'yamburg.hh.ru': 'hh.ru',
 'novaya-chara.hh.ru': 'hh.ru',
 'donetsk-ru.hh.ru': 'hh.ru',
 'tutayev.hh.ru': 'hh.ru',
 'obolensk.hh.ru': 'hh.ru',
 'sovhozniy66.hh.ru': 'hh.ru',
 'norway.hh.ru': 'hh.ru',
 'yeniseisk.hh.ru': 'hh.ru',
 'krasnoyarskiy.hh.ru': 'hh.ru',
 'beloretsk.hh.ru': 'hh.ru',
 'staraya-russa.hh.ru': 'hh.ru',
 'japan.hh.ru': 'hh.ru',
 'seryshevo.hh.ru': 'hh.ru',
 'pevek.hh.ru': 'hh.ru',
 'tahtamukay.hh.ru': 'hh.ru',
 'alapaevsk.hh.ru': 'hh.ru',
 'omsukchan.hh.ru': 'hh.ru',
 'karabanovo.hh.ru': 'hh.ru',
 'baykalovo.hh.ru': 'hh.ru',
 'belousovo.hh.ru': 'hh.ru',
 'kaskara.hh.ru': 'hh.ru',
 'vietnam.hh.ru': 'hh.ru',
 'golubitskaya.hh.ru': 'hh.ru',
 'mamadysh.hh.ru': 'hh.ru',
 'kamen-na-obi.hh.ru': 'hh.ru',
 'zelenogradsk.hh.ru': 'hh.ru',
 'pervomayskiy43.hh.ru': 'hh.ru',
 'nadym.hh.ru': 'hh.ru',
 'volzhskiy63.hh.ru': 'hh.ru',
 'tavda.hh.ru': 'hh.ru',
 'mariinsk.hh.ru': 'hh.ru',
 'yuzhno-kurilsk.hh.ru': 'hh.ru',
 'anzhero-sudzhensk.hh.ru': 'hh.ru',
 'habary.hh.ru': 'hh.ru',
 'starokorsunskaya.hh.ru': 'hh.ru',
 'krym.hh.ru': 'hh.ru',
 'krasnouralsk.hh.ru': 'hh.ru',
 'andreevka.hh.ru': 'hh.ru',
 'kyshtym.hh.ru': 'hh.ru',
 'inta.hh.ru': 'hh.ru',
 'pavlovsk.hh.ru': 'hh.ru',
 'palana.hh.ru': 'hh.ru',
 'lesozavodsk.hh.ru': 'hh.ru',
 'nurlat.hh.ru': 'hh.ru',
 'kandalaksha.hh.ru': 'hh.ru',
 'kamensk.hh.ru': 'hh.ru',
 'kremenki.hh.ru': 'hh.ru',
 'snezhinsk.hh.ru': 'hh.ru',
 'krasnoe.hh.ru': 'hh.ru',
 'mikhailov.hh.ru': 'hh.ru',
 'shebekino.hh.ru': 'hh.ru',
 'divnomorskoe.hh.ru': 'hh.ru',
 'kukushtan.hh.ru': 'hh.ru',
 'verhovye.hh.ru': 'hh.ru',
 'toksovo.hh.ru': 'hh.ru',
 'sovgavan.hh.ru': 'hh.ru',
 'bolshoj-kamen.hh.ru': 'hh.ru',
 'irbit.hh.ru': 'hh.ru',
 'annino.hh.ru': 'hh.ru',
 'volzhsk.hh.ru': 'hh.ru',
 'gorodets.hh.ru': 'hh.ru',
 'ayhal.hh.ru': 'hh.ru',
 'tashtagol.hh.ru': 'hh.ru',
 'kalachinsk.hh.ru': 'hh.ru',
 'karabash16.hh.ru': 'hh.ru',
 'yagodnoe.hh.ru': 'hh.ru',
 'shatsk.hh.ru': 'hh.ru',
 'ozersk-chelyabinsk.hh.ru': 'hh.ru',
 'rebriha.hh.ru': 'hh.ru',
 'vanino.hh.ru': 'hh.ru',
 'sholohovskiy.hh.ru': 'hh.ru',
 'aleysk.hh.ru': 'hh.ru',
 'rybnoe.hh.ru': 'hh.ru',
 'davlekanovo.hh.ru': 'hh.ru',
 'kurkino.hh.ru': 'hh.ru',
 'dolgorukovo.hh.ru': 'hh.ru',
 'krasniy-bor.hh.ru': 'hh.ru',
 'vichuga.hh.ru': 'hh.ru',
 'zhukovka.hh.ru': 'hh.ru',
 'blagoveshchenka.hh.ru': 'hh.ru',
 'volya.hh.ru': 'hh.ru',
 'sarov.hh.ru': 'hh.ru',
 'israel.hh.ru': 'hh.ru',
 'lyubinskiy.hh.ru': 'hh.ru',
 'great-britain.hh.ru': 'hh.ru',
 'nazran.hh.ru': 'hh.ru',
 'bologoe.hh.ru': 'hh.ru',
 'severniy.hh.ru': 'hh.ru',
 'kovylkino.hh.ru': 'hh.ru',
 'gorki-leninskie.hh.ru': 'hh.ru',
 'novomihaylovskiy.hh.ru': 'hh.ru',
 'kotovo.hh.ru': 'hh.ru',
 'neftekumsk.hh.ru': 'hh.ru',
 'borskoe.hh.ru': 'hh.ru',
 'rzhev.hh.ru': 'hh.ru',
 'plesetsk.hh.ru': 'hh.ru',
 'chkalovsk.hh.ru': 'hh.ru',
 'grachevka.hh.ru': 'hh.ru',
 'nolinsk.hh.ru': 'hh.ru',
 'semenov.hh.ru': 'hh.ru',
 'pokachi.hh.ru': 'hh.ru',
 'beloyarsk.hh.ru': 'hh.ru',
 'kozelsk.hh.ru': 'hh.ru',
 'severskaya.hh.ru': 'hh.ru',
 'ahtyrskiy.hh.ru': 'hh.ru',
 'sorochinsk.hh.ru': 'hh.ru',
 'likhoslavl.hh.ru': 'hh.ru',
 'troitskoe22.hh.ru': 'hh.ru',
 'nikolskoe.hh.ru': 'hh.ru',
 'pikalevo.hh.ru': 'hh.ru',
 'asino.hh.ru': 'hh.ru',
 'vorsma.hh.ru': 'hh.ru',
 'palatka.hh.ru': 'hh.ru',
 'lorino.hh.ru': 'hh.ru',
 'bovanenkovo.hh.ru': 'hh.ru',
 'suraz.hh.ru': 'hh.ru',
 'yemanzhelinsk.hh.ru': 'hh.ru',
 'lgov.hh.ru': 'hh.ru',
 'pavlovsk47.hh.ru': 'hh.ru',
 'solnechniy86.hh.ru': 'hh.ru',
 'telmana.hh.ru': 'hh.ru',
 'malaya-purga.hh.ru': 'hh.ru',
 'isilkul.hh.ru': 'hh.ru',
 'asbest.hh.ru': 'hh.ru',
 'alabino.hh.ru': 'hh.ru',
 'selyatino.hh.ru': 'hh.ru',
 'krasny-sulin.hh.ru': 'hh.ru',
 'rudnya34.hh.ru': 'hh.ru',
 'krasniy-yar63.hh.ru': 'hh.ru',
 'lesnoy50.hh.ru': 'hh.ru',
 'ostashkov.hh.ru': 'hh.ru',
 'kiselevsk.hh.ru': 'hh.ru',
 'tavricheskoe.hh.ru': 'hh.ru',
 'karasuk.hh.ru': 'hh.ru',
 'lesnoj.hh.ru': 'hh.ru',
 'nizhnyaya-tura.hh.ru': 'hh.ru',
 'oktyabrsk.hh.ru': 'hh.ru',
 'sovetskiy.hh.ru': 'hh.ru',
    'bezformata.com': 'bezformata.com',
 'adamovka.bezformata.com': 'bezformata.com',
 'mirniy.bezformata.com': 'bezformata.com',
 'bronnici.bezformata.com': 'bezformata.com',
 'gubaha.bezformata.com': 'bezformata.com',
 'kavalerovo.bezformata.com': 'bezformata.com',
 'karatuzskoe.bezformata.com': 'bezformata.com',
 'berezovka.bezformata.com': 'bezformata.com',
 'bessonovka.bezformata.com': 'bezformata.com',
 'krasnoarmeyskoe.bezformata.com': 'bezformata.com',
 'kantemirovka.bezformata.com': 'bezformata.com',
 'muezerskiy.bezformata.com': 'bezformata.com',
 'sharya.bezformata.com': 'bezformata.com',
 'ponazirevo.bezformata.com': 'bezformata.com',
 'safonovo.bezformata.com': 'bezformata.com',
 'sovetskiykrim.bezformata.com': 'bezformata.com',
 'yalta.bezformata.com': 'bezformata.com',
 'ikryanoe.bezformata.com': 'bezformata.com',
 'kletnya.bezformata.com': 'bezformata.com',
 'glazov.bezformata.com': 'bezformata.com',
 'kirensk.bezformata.com': 'bezformata.com',
 'magaramkent.bezformata.com': 'bezformata.com',
 'proletarsk.bezformata.com': 'bezformata.com',
 'karachaevsk.bezformata.com': 'bezformata.com',
 'suzdal.bezformata.com': 'bezformata.com',
 'civilsk.bezformata.com': 'bezformata.com',
 'motigino.bezformata.com': 'bezformata.com',
 'demidovsmol.bezformata.com': 'bezformata.com',
 'irbeyskoe.bezformata.com': 'bezformata.com',
 'krasniykut.bezformata.com': 'bezformata.com',
 'boguchar.bezformata.com': 'bezformata.com',
 'uyar.bezformata.com': 'bezformata.com',
 'kukmor.bezformata.com': 'bezformata.com',
 'tashla.bezformata.com': 'bezformata.com',
 'kamishla.bezformata.com': 'bezformata.com',
 'bezenchuk.bezformata.com': 'bezformata.com',
 'sudislavl.bezformata.com': 'bezformata.com',
 'leninsk.bezformata.com': 'bezformata.com',
 'svetogorsk.bezformata.com': 'bezformata.com',
 'polysaevo.bezformata.com': 'bezformata.com',
 'guryevsk.bezformata.com': 'bezformata.com',
 'spasdemensk.bezformata.com': 'bezformata.com',
 'melenki.bezformata.com': 'bezformata.com',
 'yurevpolskiy.bezformata.com': 'bezformata.com',
 'kirjach.bezformata.com': 'bezformata.com',
 'kosteryovo.bezformata.com': 'bezformata.com',
 'vadinsk.bezformata.com': 'bezformata.com',
 'uhta.bezformata.com': 'bezformata.com',
 'gryazovets.bezformata.com': 'bezformata.com',
 'ustyuzhna.bezformata.com': 'bezformata.com',
 'korsakov.bezformata.com': 'bezformata.com',
 'tomari.bezformata.com': 'bezformata.com',
 'morozova.bezformata.com': 'bezformata.com',
 'antropovo.bezformata.com': 'bezformata.com',
 'bogovarovo.bezformata.com': 'bezformata.com',
 'kaltan.bezformata.com': 'bezformata.com',
 'poreckoe.bezformata.com': 'bezformata.com',
 'nyagan.bezformata.com': 'bezformata.com',
 'pilna.bezformata.com': 'bezformata.com',
 'kstovo.bezformata.com': 'bezformata.com',
 'maloyaz.bezformata.com': 'bezformata.com',
 'verhneyarkeevo.bezformata.com': 'bezformata.com',
 'beloyarskiyugra.bezformata.com': 'bezformata.com',
 'vorobyovka.bezformata.com': 'bezformata.com',
 'ostrogojsk.bezformata.com': 'bezformata.com',
 'virisa.bezformata.com': 'bezformata.com',
 'slanci.bezformata.com': 'bezformata.com',
 'duhovshina.bezformata.com': 'bezformata.com',
 'hislavichi.bezformata.com': 'bezformata.com',
 'kizel.bezformata.com': 'bezformata.com',
 'sokolvologda.bezformata.com': 'bezformata.com',
 'belozersk.bezformata.com': 'bezformata.com',
 'aldan.bezformata.com': 'bezformata.com',
 'lensk.bezformata.com': 'bezformata.com',
 'turuhansk.bezformata.com': 'bezformata.com',
 'argayash.bezformata.com': 'bezformata.com',
 'tatcinskaya.bezformata.com': 'bezformata.com',
 'zuevka.bezformata.com': 'bezformata.com',
 'zatopervomayskiy.bezformata.com': 'bezformata.com',
 'celinnoealtay.bezformata.com': 'bezformata.com',
 'baevo.bezformata.com': 'bezformata.com',
 'balabanovo.bezformata.com': 'bezformata.com',
 'hakurinohabl.bezformata.com': 'bezformata.com',
 'poshehone.bezformata.com': 'bezformata.com',
 'imenipoliniosipenko.bezformata.com': 'bezformata.com',
 'srednayaahtuba.bezformata.com': 'bezformata.com',
 'yurla.bezformata.com': 'bezformata.com',
 'harabali.bezformata.com': 'bezformata.com',
 'enotaevka.bezformata.com': 'bezformata.com',
 'kopyovo.bezformata.com': 'bezformata.com',
 'bogorodskoekrv.bezformata.com': 'bezformata.com',
 'askino.bezformata.com': 'bezformata.com',
 'yanaul.bezformata.com': 'bezformata.com',
 'mezhgore.bezformata.com': 'bezformata.com',
 'volgorechensk.bezformata.com': 'bezformata.com',
 'lisiegori.bezformata.com': 'bezformata.com',
 'bogorodsk.bezformata.com': 'bezformata.com',
 'isheevka.bezformata.com': 'bezformata.com',
 'gukovo.bezformata.com': 'bezformata.com',
 'baryatino.bezformata.com': 'bezformata.com',
 'artemovskiy.bezformata.com': 'bezformata.com',
 'kalyazin.bezformata.com': 'bezformata.com',
 'budennovsk.bezformata.com': 'bezformata.com',
 'novopavlovsk.bezformata.com': 'bezformata.com',
 'novoaleksandrovsk.bezformata.com': 'bezformata.com',
 'semiluki.bezformata.com': 'bezformata.com',
 'kargapole.bezformata.com': 'bezformata.com',
 'aksai.bezformata.com': 'bezformata.com',
 'tcelina.bezformata.com': 'bezformata.com',
 'svobodniy.bezformata.com': 'bezformata.com',
 'onguday.bezformata.com': 'bezformata.com',
 'neftegorsk.bezformata.com': 'bezformata.com',
 'cherlak.bezformata.com': 'bezformata.com',
 'suhinichi.bezformata.com': 'bezformata.com',
 'gavrilovyam.bezformata.com': 'bezformata.com',
 'yablonovskiy.bezformata.com': 'bezformata.com',
 'pokrovsk.bezformata.com': 'bezformata.com',
 'dorogobuzh.bezformata.com': 'bezformata.com',
 'krasniy.bezformata.com': 'bezformata.com',
 'sudogda.bezformata.com': 'bezformata.com',
 'uporovo.bezformata.com': 'bezformata.com',
 'galich.bezformata.com': 'bezformata.com',
 'raychihinsk.bezformata.com': 'bezformata.com',
 'igra.bezformata.com': 'bezformata.com',
 'krasniyyar.bezformata.com': 'bezformata.com',
 'otradnoe.bezformata.com': 'bezformata.com',
 'veshkayma.bezformata.com': 'bezformata.com',
 'zverevo.bezformata.com': 'bezformata.com',
 'rodionovonesvetayskaya.bezformata.com': 'bezformata.com',
 'kortkeros.bezformata.com': 'bezformata.com',
 'surskoe.bezformata.com': 'bezformata.com',
 'nartkala.bezformata.com': 'bezformata.com',
 'zalukokoazhe.bezformata.com': 'bezformata.com',
 'agapovka.bezformata.com': 'bezformata.com',
 'mihaylovskstav.bezformata.com': 'bezformata.com',
 'sharan.bezformata.com': 'bezformata.com',
 'kazachinskoe.bezformata.com': 'bezformata.com',
 'isyangulovo.bezformata.com': 'bezformata.com',
 'selizharovo.bezformata.com': 'bezformata.com',
 'ishimbay.bezformata.com': 'bezformata.com',
 'pallasovka.bezformata.com': 'bezformata.com',
 'kolishley.bezformata.com': 'bezformata.com',
 'raduzhniy.bezformata.com': 'bezformata.com',
 'yurevetc.bezformata.com': 'bezformata.com',
 'savino.bezformata.com': 'bezformata.com',
 'aleysk.bezformata.com': 'bezformata.com',
 'tonkino.bezformata.com': 'bezformata.com',
 'liskovo.bezformata.com': 'bezformata.com',
 'ustkamchatsk.bezformata.com': 'bezformata.com',
 'leninogorsk.bezformata.com': 'bezformata.com',
 'putyatino.bezformata.com': 'bezformata.com',
 'oktyabrskoeost.bezformata.com': 'bezformata.com',
 'chernushka.bezformata.com': 'bezformata.com',
 'klimovsk.bezformata.com': 'bezformata.com',
 'aniva.bezformata.com': 'bezformata.com',
 'aleksandrovsksahalinskiy.bezformata.com': 'bezformata.com',
 'kolpino.bezformata.com': 'bezformata.com',
 'chuchkovo.bezformata.com': 'bezformata.com',
 'shilovo.bezformata.com': 'bezformata.com',
 'ryajsk.bezformata.com': 'bezformata.com',
 'romanovka.bezformata.com': 'bezformata.com',
 'fokinobrsk.bezformata.com': 'bezformata.com',
 'ekaterinoslavka.bezformata.com': 'bezformata.com',
 'dolgorukovo.bezformata.com': 'bezformata.com',
 'urzhum.bezformata.com': 'bezformata.com',
 'ershov.bezformata.com': 'bezformata.com',
 'hvalinsk.bezformata.com': 'bezformata.com',
 'kardimovo.bezformata.com': 'bezformata.com',
 'kotovo.bezformata.com': 'bezformata.com',
 'novosokolniki.bezformata.com': 'bezformata.com',
 'skovorodino.bezformata.com': 'bezformata.com',
 'iglino.bezformata.com': 'bezformata.com',
 'novovoronej.bezformata.com': 'bezformata.com',
 'ternovka.bezformata.com': 'bezformata.com',
 'osinniki.bezformata.com': 'bezformata.com',
 'kozlovka.bezformata.com': 'bezformata.com',
 'petushki.bezformata.com': 'bezformata.com',
 'yartcevo.bezformata.com': 'bezformata.com',
 'peremishl.bezformata.com': 'bezformata.com',
 'beslan.bezformata.com': 'bezformata.com',
 'isetskoe.bezformata.com': 'bezformata.com',
 'burpa.bezformata.com': 'bezformata.com',
 'verhniyuslon.bezformata.com': 'bezformata.com',
 'kambarka.bezformata.com': 'bezformata.com',
 'pachelma.bezformata.com': 'bezformata.com',
 'bolshayamurta.bezformata.com': 'bezformata.com',
 'egorlikskaya.bezformata.com': 'bezformata.com',
 'safakulevo.bezformata.com': 'bezformata.com',
 'narovchat.bezformata.com': 'bezformata.com',
    'stariyoskol.bezformata.com': 'bezformata.com',
 'korocha.bezformata.com': 'bezformata.com',
 'gubkin.bezformata.com': 'bezformata.com',
 'bezformata.com': 'bezformata.com',
 'alekseevka.bezformata.com': 'bezformata.com',
 'shebekino.bezformata.com': 'bezformata.com',
 'veydelevka.bezformata.com': 'bezformata.com',
 'belgorod.bezformata.com': 'bezformata.com',
 'chernyanka.bezformata.com': 'bezformata.com',
 'pestovo.bezformata.com': 'bezformata.com',
 'starayarussa.bezformata.com': 'bezformata.com',
 'velikiynovgorod.bezformata.com': 'bezformata.com',
 'solikamsk.bezformata.com': 'bezformata.com',
 'fyodorovka.bezformata.com': 'bezformata.com',
 'moskva.bezformata.com': 'bezformata.com',
 'blagoveshensk.bezformata.com': 'bezformata.com',
 'kireevsk.bezformata.com': 'bezformata.com',
 'orenburg.bezformata.com': 'bezformata.com',
 'simferopol.bezformata.com': 'bezformata.com',
 'habarovsk.bezformata.com': 'bezformata.com',
 'novosibirsk.bezformata.com': 'bezformata.com',
 'langepas.bezformata.com': 'bezformata.com',
 'hantimansiysk.bezformata.com': 'bezformata.com',
 'nijnevartovsk.bezformata.com': 'bezformata.com',
 'surgut.bezformata.com': 'bezformata.com',
 'uray.bezformata.com': 'bezformata.com',
 'vichuga.bezformata.com': 'bezformata.com',
 'rodniki.bezformata.com': 'bezformata.com',
 'kineshma.bezformata.com': 'bezformata.com',
 'ivanovo.bezformata.com': 'bezformata.com',
 'shuya.bezformata.com': 'bezformata.com',
 'dyatkovo.bezformata.com': 'bezformata.com',
 'rognedino.bezformata.com': 'bezformata.com',
 'bryansk.bezformata.com': 'bezformata.com',
 'bashblagoveshensk.bezformata.com': 'bezformata.com',
 'birsk.bezformata.com': 'bezformata.com',
 'neftekamsk.bezformata.com': 'bezformata.com',
 'oktyabrskiy.bezformata.com': 'bezformata.com',
 'ufa.bezformata.com': 'bezformata.com',
 'petrozavodsk.bezformata.com': 'bezformata.com',
 'siktivkar.bezformata.com': 'bezformata.com',
 'buguruslan.bezformata.com': 'bezformata.com',
 'tyulgan.bezformata.com': 'bezformata.com',
 'orsk.bezformata.com': 'bezformata.com',
 'gai.bezformata.com': 'bezformata.com',
 'totckoe.bezformata.com': 'bezformata.com',
 'rostovnadonu.bezformata.com': 'bezformata.com',
 'roslavl.bezformata.com': 'bezformata.com',
 'smolensk.bezformata.com': 'bezformata.com',
 'sanktpeterburg.bezformata.com': 'bezformata.com',
 'serpuhov.bezformata.com': 'bezformata.com',
 'vologda.bezformata.com': 'bezformata.com',
 'pavlovskiyposad.bezformata.com': 'bezformata.com',
 'pushino.bezformata.com': 'bezformata.com',
 'lenoblast.bezformata.com': 'bezformata.com',
 'luberci.bezformata.com': 'bezformata.com',
 'pushkino.bezformata.com': 'bezformata.com',
 'ozeri.bezformata.com': 'bezformata.com',
 'zelenograd.bezformata.com': 'bezformata.com',
 'yoshkarola.bezformata.com': 'bezformata.com',
 'krasnoufimsk.bezformata.com': 'bezformata.com',
 'serov.bezformata.com': 'bezformata.com',
 'krasnoturinsk.bezformata.com': 'bezformata.com',
 'beryozovskiy.bezformata.com': 'bezformata.com',
 'kachkanar.bezformata.com': 'bezformata.com',
 'ekaterinburg.bezformata.com': 'bezformata.com',
 'krasnodar.bezformata.com': 'bezformata.com',
 'lipeck.bezformata.com': 'bezformata.com',
 'chelyabinsk.bezformata.com': 'bezformata.com',
 'nyazepetrovsk.bezformata.com': 'bezformata.com',
 'miasskoe.bezformata.com': 'bezformata.com',
 'kumertau.bezformata.com': 'bezformata.com',
 'novotroisk.bezformata.com': 'bezformata.com',
 'tuymazi.bezformata.com': 'bezformata.com',
 'agidel.bezformata.com': 'bezformata.com',
 'sterlitamak.bezformata.com': 'bezformata.com',
 'mishkinobash.bezformata.com': 'bezformata.com',
 'kuvandik.bezformata.com': 'bezformata.com',
 'tolbazi.bezformata.com': 'bezformata.com',
 'arhangelsk.bezformata.com': 'bezformata.com',
 'tambov.bezformata.com': 'bezformata.com',
 'narianmar.bezformata.com': 'bezformata.com',
 'mojga.bezformata.com': 'bezformata.com',
 'ijevsk.bezformata.com': 'bezformata.com',
 'selti.bezformata.com': 'bezformata.com',
 'votkinsk.bezformata.com': 'bezformata.com',
 'krasnogorskoe.bezformata.com': 'bezformata.com',
 'balezino.bezformata.com': 'bezformata.com',
 'sarapul.bezformata.com': 'bezformata.com',
 'tuapse.bezformata.com': 'bezformata.com',
 'pitelino.bezformata.com': 'bezformata.com',
 'birobidjan.bezformata.com': 'bezformata.com',
 'kudimkar.bezformata.com': 'bezformata.com',
 'kugesi.bezformata.com': 'bezformata.com',
 'petropavlovskkamchatskiy.bezformata.com': 'bezformata.com',
 'sorochinsk.bezformata.com': 'bezformata.com',
 'omutninsk.bezformata.com': 'bezformata.com',
 'cheboksari.bezformata.com': 'bezformata.com',
 'saratov.bezformata.com': 'bezformata.com',
 'holm.bezformata.com': 'bezformata.com',
 'abakan.bezformata.com': 'bezformata.com',
 'stavropol.bezformata.com': 'bezformata.com',
 'kirov.bezformata.com': 'bezformata.com',
 'kurtamish.bezformata.com': 'bezformata.com',
 'orel.bezformata.com': 'bezformata.com',
 'krasnoyarsk.bezformata.com': 'bezformata.com',
 'novokuzneck.bezformata.com': 'bezformata.com',
 'omsk.bezformata.com': 'bezformata.com',
 'tula.bezformata.com': 'bezformata.com',
 'murmansk.bezformata.com': 'bezformata.com',
 'kostomuksha.bezformata.com': 'bezformata.com',
 'kostroma.bezformata.com': 'bezformata.com',
 'volgograd.bezformata.com': 'bezformata.com',
 'tver.bezformata.com': 'bezformata.com',
 'ryazan.bezformata.com': 'bezformata.com',
 'barnaul.bezformata.com': 'bezformata.com',
 'kazan.bezformata.com': 'bezformata.com',
 'kemerovo.bezformata.com': 'bezformata.com',
 'samara.bezformata.com': 'bezformata.com',
 'tolyatti.bezformata.com': 'bezformata.com',
 'pohvistnevo.bezformata.com': 'bezformata.com',
 'livni.bezformata.com': 'bezformata.com',
 'zlatoust.bezformata.com': 'bezformata.com',
 'pskov.bezformata.com': 'bezformata.com',
 'velikieluki.bezformata.com': 'bezformata.com',
 'prohorovka.bezformata.com': 'bezformata.com',
 'komsomolsknaamure.bezformata.com': 'bezformata.com',
 'chernomorskoe.bezformata.com': 'bezformata.com',
 'gornoaltaysk.bezformata.com': 'bezformata.com',
 'kaluga.bezformata.com': 'bezformata.com',
 'kurskaya.bezformata.com': 'bezformata.com',
 'uliyanovsk.bezformata.com': 'bezformata.com',
 'lesnoy.bezformata.com': 'bezformata.com',
 'nevinnomissk.bezformata.com': 'bezformata.com',
 'saransk.bezformata.com': 'bezformata.com',
 'astrahan.bezformata.com': 'bezformata.com',
 'seversk.bezformata.com': 'bezformata.com',
 'kislovodsk.bezformata.com': 'bezformata.com',
 'vladivostok.bezformata.com': 'bezformata.com',
 'derbent.bezformata.com': 'bezformata.com',
 'yakutsk.bezformata.com': 'bezformata.com',
 'kurgan.bezformata.com': 'bezformata.com',
 'sevastopol.bezformata.com': 'bezformata.com',
 'vladimir.bezformata.com': 'bezformata.com',
 'chapaevsk.bezformata.com': 'bezformata.com',
 'irkutsk.bezformata.com': 'bezformata.com',
 'navlya.bezformata.com': 'bezformata.com',
 'vigonichi.bezformata.com': 'bezformata.com',
 'ustilimsk.bezformata.com': 'bezformata.com',
 'gordeevka.bezformata.com': 'bezformata.com',
 'novozibkov.bezformata.com': 'bezformata.com',
 'sevck.bezformata.com': 'bezformata.com',
 'klimovo.bezformata.com': 'bezformata.com',
 'suzemka.bezformata.com': 'bezformata.com',
 'mglin.bezformata.com': 'bezformata.com',
 'ctarodub.bezformata.com': 'bezformata.com',
 'karachev.bezformata.com': 'bezformata.com',
 'krasnayagora.bezformata.com': 'bezformata.com',
 'dubrovka.bezformata.com': 'bezformata.com',
 'nnovgorod.bezformata.com': 'bezformata.com',
 'brasovo.bezformata.com': 'bezformata.com',
 'komarichi.bezformata.com': 'bezformata.com',
 'voronej.bezformata.com': 'bezformata.com',
 'pochep.bezformata.com': 'bezformata.com',
 'trubchevck.bezformata.com': 'bezformata.com',
 'klinci.bezformata.com': 'bezformata.com',
 'angarsk.bezformata.com': 'bezformata.com',
 'pogar.bezformata.com': 'bezformata.com',
 'trehgorniy.bezformata.com': 'bezformata.com',
 'miass.bezformata.com': 'bezformata.com',
 'emanjelinsk.bezformata.com': 'bezformata.com',
 'magnitogorsk.bezformata.com': 'bezformata.com',
 'etkul.bezformata.com': 'bezformata.com',
 'chesma.bezformata.com': 'bezformata.com',
 'kopeysk.bezformata.com': 'bezformata.com',
 'verhneuralsk.bezformata.com': 'bezformata.com',
 'kartali.bezformata.com': 'bezformata.com',
 'asha.bezformata.com': 'bezformata.com',
 'tinda.bezformata.com': 'bezformata.com',
 'shimanovsk.bezformata.com': 'bezformata.com',
 'vladikavkaz.bezformata.com': 'bezformata.com',
 'tavda.bezformata.com': 'bezformata.com',
 'chita.bezformata.com': 'bezformata.com',
 'ujnosahalinsk.bezformata.com': 'bezformata.com',
 'kaliningrad.bezformata.com': 'bezformata.com',
 'mahachkala.bezformata.com': 'bezformata.com',
 'kursk.bezformata.com': 'bezformata.com',
 'oha.bezformata.com': 'bezformata.com',
 'novoshahtinsk.bezformata.com': 'bezformata.com',
 'ryabinsk.bezformata.com': 'bezformata.com',
 'yaroslavl.bezformata.com': 'bezformata.com',
 'sorsk.bezformata.com': 'bezformata.com',
 'penza.bezformata.com': 'bezformata.com',
 'ntagil.bezformata.com': 'bezformata.com',
 'kamenskuralsk.bezformata.com': 'bezformata.com',
 'bogdanovich.bezformata.com': 'bezformata.com',
 'leningradskaya.bezformata.com': 'bezformata.com',
 'vorcuta.bezformata.com': 'bezformata.com',
 'novomoskovsk.bezformata.com': 'bezformata.com',
 'gelendjik.bezformata.com': 'bezformata.com',
 'nizhnedevitck.bezformata.com': 'bezformata.com',
 'rossosh.bezformata.com': 'bezformata.com',
 'liski.bezformata.com': 'bezformata.com',
 'ramon.bezformata.com': 'bezformata.com',
 'pavlovskvor.bezformata.com': 'bezformata.com',
 'novayausman.bezformata.com': 'bezformata.com',
 'borisoglebsk.bezformata.com': 'bezformata.com',
 'zverinogolovskoe.bezformata.com': 'bezformata.com',
 'shatrovo.bezformata.com': 'bezformata.com',
 'tomsk.bezformata.com': 'bezformata.com',
 'nadim.bezformata.com': 'bezformata.com',
 'shadrinsk.bezformata.com': 'bezformata.com',
 'usinsk.bezformata.com': 'bezformata.com',
 'balakovo.bezformata.com': 'bezformata.com',
 'mishkino.bezformata.com': 'bezformata.com',
 'nabchelni.bezformata.com': 'bezformata.com',
 'ustabakan.bezformata.com': 'bezformata.com',
 'sayanagorsk.bezformata.com': 'bezformata.com',
 'chernogorsk.bezformata.com': 'bezformata.com',
 'syamzha.bezformata.com': 'bezformata.com',
 'velikiyustug.bezformata.com': 'bezformata.com',
 'cherepovec.bezformata.com': 'bezformata.com',
 'perm.bezformata.com': 'bezformata.com',
 'temryuk.bezformata.com': 'bezformata.com',
 'sochi.bezformata.com': 'bezformata.com',
 'kurganinsk.bezformata.com': 'bezformata.com',
 'kanevskaya.bezformata.com': 'bezformata.com',
 'novorossiysk.bezformata.com': 'bezformata.com',
 'usuhchay.bezformata.com': 'bezformata.com',
 'hasavyurt.bezformata.com': 'bezformata.com',
 'novokayakent.bezformata.com': 'bezformata.com',
 'babaurt.bezformata.com': 'bezformata.com',
 'karabudahkent.bezformata.com': 'bezformata.com',
 'levashi.bezformata.com': 'bezformata.com',
 'dinskaya.bezformata.com': 'bezformata.com',
 'belogorsk.bezformata.com': 'bezformata.com',
 'engels.bezformata.com': 'bezformata.com',
 'kalininsk.bezformata.com': 'bezformata.com',
 'balashov.bezformata.com': 'bezformata.com',
 'magadan.bezformata.com': 'bezformata.com',
 'shahunya.bezformata.com': 'bezformata.com',
 'podolsk.bezformata.com': 'bezformata.com',
 'balashiha.bezformata.com': 'bezformata.com',
 'jukovskiy.bezformata.com': 'bezformata.com',
 'dzerjinskiy.bezformata.com': 'bezformata.com',
 'vykza.bezformata.com': 'bezformata.com',
 'korolev.bezformata.com': 'bezformata.com',
 'egoryevsk.bezformata.com': 'bezformata.com',
 'krasnogorsk.bezformata.com': 'bezformata.com',
 'pervomaysk.bezformata.com': 'bezformata.com',
 'dzerginsk.bezformata.com': 'bezformata.com',
 'domodedovo.bezformata.com': 'bezformata.com',
 'sechenovo.bezformata.com': 'bezformata.com',
 'mitishi.bezformata.com': 'bezformata.com',
 'podmoskovye.bezformata.com': 'bezformata.com',
 'shelkovo.bezformata.com': 'bezformata.com',
 'arzamas.bezformata.com': 'bezformata.com',
 'maikop.bezformata.com': 'bezformata.com',
 'rakitnoe.bezformata.com': 'bezformata.com',
 'grayvoron.bezformata.com': 'bezformata.com',
 'michurinsk.bezformata.com': 'bezformata.com',
 'berdsk.bezformata.com': 'bezformata.com',
 'iskitim.bezformata.com': 'bezformata.com',
 'stepnoe.bezformata.com': 'bezformata.com',
 'volsk.bezformata.com': 'bezformata.com',
 'eysk.bezformata.com': 'bezformata.com',
 'tihoreck.bezformata.com': 'bezformata.com',
 'gulkevichi.bezformata.com': 'bezformata.com',
 'apsheronsk.bezformata.com': 'bezformata.com',
 'bokovskaya.bezformata.com': 'bezformata.com',
 'slavyansknakubani.bezformata.com': 'bezformata.com',
 'anapa.bezformata.com': 'bezformata.com',
 'chistopol.bezformata.com': 'bezformata.com',
 'sizran.bezformata.com': 'bezformata.com',
 'stroitel.bezformata.com': 'bezformata.com',
 'viselki.bezformata.com': 'bezformata.com',
 'kropotkin.bezformata.com': 'bezformata.com',
 'ustlabinsk.bezformata.com': 'bezformata.com',
 'chani.bezformata.com': 'bezformata.com',
 'barabinsk.bezformata.com': 'bezformata.com',
 'toguchin.bezformata.com': 'bezformata.com',
 'nahodka.bezformata.com': 'bezformata.com',
 'shigri.bezformata.com': 'bezformata.com',
 'mejdurechenskiy.bezformata.com': 'bezformata.com',
 'uzlovaya.bezformata.com': 'bezformata.com',
 'oktyabrskoe.bezformata.com': 'bezformata.com',
 'labinsk.bezformata.com': 'bezformata.com',
 'hlevnoe.bezformata.com': 'bezformata.com',
 'dankov.bezformata.com': 'bezformata.com',
 'dobrinka.bezformata.com': 'bezformata.com',
 'dobroe.bezformata.com': 'bezformata.com',
 'boksitogorsk.bezformata.com': 'bezformata.com',
 'lebedyan.bezformata.com': 'bezformata.com',
 'lukoyanov.bezformata.com': 'bezformata.com',
 'volodarsk.bezformata.com': 'bezformata.com',
 'ahtubinsk.bezformata.com': 'bezformata.com',
 'dubnatula.bezformata.com': 'bezformata.com',
 'aleksin.bezformata.com': 'bezformata.com',
 'shekino.bezformata.com': 'bezformata.com',
 'suvorov.bezformata.com': 'bezformata.com',
 'viborg.bezformata.com': 'bezformata.com',
 'tumen.bezformata.com': 'bezformata.com',
 'nijnekamsk.bezformata.com': 'bezformata.com',
 'odinsovo.bezformata.com': 'bezformata.com',
 'kodinsk.bezformata.com': 'bezformata.com',
 'polevskoy.bezformata.com': 'bezformata.com',
 'ussuriysk.bezformata.com': 'bezformata.com',
 'suraj.bezformata.com': 'bezformata.com',
 'zadonsk.bezformata.com': 'bezformata.com',
 'severodvinsk.bezformata.com': 'bezformata.com',
 'kotlas.bezformata.com': 'bezformata.com',
 'onega.bezformata.com': 'bezformata.com',
 'krasnoborsk.bezformata.com': 'bezformata.com',
 'mezen.bezformata.com': 'bezformata.com',
 'shenkursk.bezformata.com': 'bezformata.com',
 'pleseck.bezformata.com': 'bezformata.com',
 'konosha.bezformata.com': 'bezformata.com',
 'rubcovsk.bezformata.com': 'bezformata.com',
 'bogotol.bezformata.com': 'bezformata.com',
 'dudinkakrasnoyarsk.bezformata.com': 'bezformata.com',
 'achinsk.bezformata.com': 'bezformata.com',
 'sharipovo.bezformata.com': 'bezformata.com',
 'voskresensk.bezformata.com': 'bezformata.com',
 'eniseysk.bezformata.com': 'bezformata.com',
 'norilsk.bezformata.com': 'bezformata.com',
 'armavir.bezformata.com': 'bezformata.com',
 'peno.bezformata.com': 'bezformata.com',
 'pyatigorsk.bezformata.com': 'bezformata.com',
 'novoselitckoe.bezformata.com': 'bezformata.com',
 'izobilniy.bezformata.com': 'bezformata.com',
 'svetlograd.bezformata.com': 'bezformata.com',
 'belovo.bezformata.com': 'bezformata.com',
 'prokopevsk.bezformata.com': 'bezformata.com',
 'yashkino.bezformata.com': 'bezformata.com',
 'kurkino.bezformata.com': 'bezformata.com',
 'miski.bezformata.com': 'bezformata.com',
 'chern.bezformata.com': 'bezformata.com',
 'bogoroditsk.bezformata.com': 'bezformata.com',
 'mejdurechensk.bezformata.com': 'bezformata.com',
 'odoev.bezformata.com': 'bezformata.com',
 'pestravka.bezformata.com': 'bezformata.com',
 'novokuybishevsk.bezformata.com': 'bezformata.com',
 'holmogori.bezformata.com': 'bezformata.com',
 'bereznik.bezformata.com': 'bezformata.com',
 'sredneuralsk.bezformata.com': 'bezformata.com',
 'berezovskiy.bezformata.com': 'bezformata.com',
 'irbit.bezformata.com': 'bezformata.com',
 'turinsk.bezformata.com': 'bezformata.com',
 'ntura.bezformata.com': 'bezformata.com',
 'zarechniy.bezformata.com': 'bezformata.com',
 'pervouralsk.bezformata.com': 'bezformata.com',
 'kamishlov.bezformata.com': 'bezformata.com',
 'voljskiy.bezformata.com': 'bezformata.com',
 'inta.bezformata.com': 'bezformata.com',
 'ramenskoe.bezformata.com': 'bezformata.com',
 'istra.bezformata.com': 'bezformata.com',
 'cherdin.bezformata.com': 'bezformata.com',
 'verhniylandeh.bezformata.com': 'bezformata.com',
 'teykovo.bezformata.com': 'bezformata.com',
 'buzuluk.bezformata.com': 'bezformata.com',
 'mcensk.bezformata.com': 'bezformata.com',
 'alapaevsk.bezformata.com': 'bezformata.com',
 'novouralsk.bezformata.com': 'bezformata.com',
 'cherkesk.bezformata.com': 'bezformata.com',
 'taishet.bezformata.com': 'bezformata.com',
 'usolesibirskoe.bezformata.com': 'bezformata.com',
 'shumerlya.bezformata.com': 'bezformata.com',
 'kiselevsk.bezformata.com': 'bezformata.com',
 'kolivan.bezformata.com': 'bezformata.com',
 'kuybishev.bezformata.com': 'bezformata.com',
 'usttarka.bezformata.com': 'bezformata.com',
 'ubinskoe.bezformata.com': 'bezformata.com',
 'znamenskoeorlobl.bezformata.com': 'bezformata.com',
 'davlekanovo.bezformata.com': 'bezformata.com',
 'novobelokatay.bezformata.com': 'bezformata.com',
 'azov.bezformata.com': 'bezformata.com',
 'bataysk.bezformata.com': 'bezformata.com',
 'kansk.bezformata.com': 'bezformata.com',
 'balahta.bezformata.com': 'bezformata.com',
 'zeleznogorsk.bezformata.com': 'bezformata.com',
 'cherepanovo.bezformata.com': 'bezformata.com',
 'tatarsk.bezformata.com': 'bezformata.com',
 'ob1.bezformata.com': 'bezformata.com',
 'nalchik.bezformata.com': 'bezformata.com',
 'ustnera.bezformata.com': 'bezformata.com',
 'magas.bezformata.com': 'bezformata.com',
 'kinel.bezformata.com': 'bezformata.com',
 'voljsk.bezformata.com': 'bezformata.com',
 'otradniy.bezformata.com': 'bezformata.com',
 'sisert.bezformata.com': 'bezformata.com',
 'talisa.bezformata.com': 'bezformata.com',
 'artyomovskiy.bezformata.com': 'bezformata.com',
 'rej.bezformata.com': 'bezformata.com',
 'nizhniyingash.bezformata.com': 'bezformata.com',
 'abinsk.bezformata.com': 'bezformata.com',
 'novopokrovskaya.bezformata.com': 'bezformata.com',
 'grozniy.bezformata.com': 'bezformata.com',
 'ishim.bezformata.com': 'bezformata.com',
 'yalutorovsk.bezformata.com': 'bezformata.com',
 'tobolsk.bezformata.com': 'bezformata.com',
 'sortavala.bezformata.com': 'bezformata.com',
 'kondopoga.bezformata.com': 'bezformata.com',
 'habari.bezformata.com': 'bezformata.com',
 'aleksandrov.bezformata.com': 'bezformata.com',
 'pitkyaranta.bezformata.com': 'bezformata.com',
 'himki.bezformata.com': 'bezformata.com',
 'stupino.bezformata.com': 'bezformata.com',
 'taldom.bezformata.com': 'bezformata.com',
 'noginsk.bezformata.com': 'bezformata.com',
 'narofominsk.bezformata.com': 'bezformata.com',
 'ruza.bezformata.com': 'bezformata.com',
 'litkarino.bezformata.com': 'bezformata.com',
 'nerehta.bezformata.com': 'bezformata.com',
 'serebryannieprudi.bezformata.com': 'bezformata.com',
 'buy.bezformata.com': 'bezformata.com',
 'shahovskaya.bezformata.com': 'bezformata.com',
 'dolgoprudniy.bezformata.com': 'bezformata.com',
 'dmitrov.bezformata.com': 'bezformata.com',
 'timashevsk.bezformata.com': 'bezformata.com',
 'novokubansk.bezformata.com': 'bezformata.com',
 'kozelsk.bezformata.com': 'bezformata.com',
 'alushta.bezformata.com': 'bezformata.com',
 'arsenevo.bezformata.com': 'bezformata.com',
 'taganrog.bezformata.com': 'bezformata.com',
 'kamenka.bezformata.com': 'bezformata.com',
 'dimitrovgrad.bezformata.com': 'bezformata.com',
 'neryungri.bezformata.com': 'bezformata.com',
 'kujener.bezformata.com': 'bezformata.com',
 'krimsk.bezformata.com': 'bezformata.com',
 'elista.bezformata.com': 'bezformata.com',
 'kubinka.bezformata.com': 'bezformata.com',
 'kolpashevo.bezformata.com': 'bezformata.com',
 'ilinskoehovanskoe.bezformata.com': 'bezformata.com',
 'jeleznogorsk-ilimskiy.bezformata.com': 'bezformata.com',
 'ust-kut.bezformata.com': 'bezformata.com',
 'bratsk.bezformata.com': 'bezformata.com',
 'gagarin.bezformata.com': 'bezformata.com',
 'uryupinsk.bezformata.com': 'bezformata.com',
 'ruzaevka.bezformata.com': 'bezformata.com',
 'bavli.bezformata.com': 'bezformata.com',
 'mendeleevsk.bezformata.com': 'bezformata.com',
 'nurlat.bezformata.com': 'bezformata.com',
 'ustdoneckiy.bezformata.com': 'bezformata.com',
 'almetevsk.bezformata.com': 'bezformata.com',
 'kovrov.bezformata.com': 'bezformata.com',
 'protvino.bezformata.com': 'bezformata.com',
 'sergievposad.bezformata.com': 'bezformata.com',
 'solnechnogorsk.bezformata.com': 'bezformata.com',
 'luhovici.bezformata.com': 'bezformata.com',
 'shatura.bezformata.com': 'bezformata.com',
 'neman.bezformata.com': 'bezformata.com',
 'svetliy.bezformata.com': 'bezformata.com',
 'starosherbinovskaya.bezformata.com': 'bezformata.com',
 'starominskaya.bezformata.com': 'bezformata.com',
 'bugulma.bezformata.com': 'bezformata.com',
 'ribnoe.bezformata.com': 'bezformata.com',
 'kolomna.bezformata.com': 'bezformata.com',
 'kizilskoe.bezformata.com': 'bezformata.com',
 'losinopetrovskiy.bezformata.com': 'bezformata.com',
 'varna.bezformata.com': 'bezformata.com',
 'kasli.bezformata.com': 'bezformata.com',
 'ujur.bezformata.com': 'bezformata.com',
 'aginskoekrk.bezformata.com': 'bezformata.com',
 'salsk.bezformata.com': 'bezformata.com',
 'ohotsk.bezformata.com': 'bezformata.com',
 'troitckoehab.bezformata.com': 'bezformata.com',
 'unecha.bezformata.com': 'bezformata.com',
 'tara.bezformata.com': 'bezformata.com',
 'novohopyorsk.bezformata.com': 'bezformata.com',
 'kashirskoe.bezformata.com': 'bezformata.com',
 'talovaya.bezformata.com': 'bezformata.com',
 'shahti.bezformata.com': 'bezformata.com',
 'doneck.bezformata.com': 'bezformata.com',
 'vyoshenskaya.bezformata.com': 'bezformata.com',
 'elabuga.bezformata.com': 'bezformata.com',
 'borisovka.bezformata.com': 'bezformata.com',
 'rovenki.bezformata.com': 'bezformata.com',
 'noviyoskol.bezformata.com': 'bezformata.com',
 'valuyki.bezformata.com': 'bezformata.com',
 'biryuch.bezformata.com': 'bezformata.com',
 'ivnya.bezformata.com': 'bezformata.com',
 'maloarchangelsk.bezformata.com': 'bezformata.com',
 'zalegosh.bezformata.com': 'bezformata.com',
 'arkadak.bezformata.com': 'bezformata.com',
 'pugachev.bezformata.com': 'bezformata.com',
 'atkarsk.bezformata.com': 'bezformata.com',
 'kupino.bezformata.com': 'bezformata.com',
 'krasnousolskiy.bezformata.com': 'bezformata.com',
 'ilanskiy.bezformata.com': 'bezformata.com',
 'sosnoviybor.bezformata.com': 'bezformata.com',
 'uvelskiy.bezformata.com': 'bezformata.com',
 'troisk.bezformata.com': 'bezformata.com',
 'sovetskiyugra.bezformata.com': 'bezformata.com',
 'marks.bezformata.com': 'bezformata.com',
 'yasniy.bezformata.com': 'bezformata.com',
 'solyileck.bezformata.com': 'bezformata.com',
 'gurevsk.bezformata.com': 'bezformata.com',
 'ozyorsk.bezformata.com': 'bezformata.com',
 'abdulino.bezformata.com': 'bezformata.com',
 'arhara.bezformata.com': 'bezformata.com',
 'pavlovo.bezformata.com': 'bezformata.com',
 'bolshoemurashkino.bezformata.com': 'bezformata.com',
 'sarov.bezformata.com': 'bezformata.com',
 'manturovo.bezformata.com': 'bezformata.com',
 'georgievskoe.bezformata.com': 'bezformata.com',
 'vohma.bezformata.com': 'bezformata.com',
 'nitva.bezformata.com': 'bezformata.com',
 'sergach.bezformata.com': 'bezformata.com',
 'vad.bezformata.com': 'bezformata.com',
 'severouralsk.bezformata.com': 'bezformata.com',
 'suhoylog.bezformata.com': 'bezformata.com',
 'nizhnyayasalda.bezformata.com': 'bezformata.com',
 'zavitinsk.bezformata.com': 'bezformata.com',
 'goryachiyklyuch.bezformata.com': 'bezformata.com',
 'biysk.bezformata.com': 'bezformata.com',
 'tyajinskiy.bezformata.com': 'bezformata.com',
 'zavodoukovsk.bezformata.com': 'bezformata.com',
 'zmeinogorsk.bezformata.com': 'bezformata.com',
 'tulun.bezformata.com': 'bezformata.com',
 'sovetskoe.bezformata.com': 'bezformata.com',
 'karmaskali.bezformata.com': 'bezformata.com',
 'kumilzhenskaya.bezformata.com': 'bezformata.com',
 'kamishin.bezformata.com': 'bezformata.com',
 'kletskaya.bezformata.com': 'bezformata.com',
 'lesosibirsk.bezformata.com': 'bezformata.com',
 'sosnovoborsk.bezformata.com': 'bezformata.com',
 'privoljsk.bezformata.com': 'bezformata.com',
 'ilovlya.bezformata.com': 'bezformata.com',
 'reutov.bezformata.com': 'bezformata.com',
 'ozersk.bezformata.com': 'bezformata.com',
 'yasnogorsk.bezformata.com': 'bezformata.com',
 'revda.bezformata.com': 'bezformata.com',
 'jigulevsk.bezformata.com': 'bezformata.com',
 'roshal.bezformata.com': 'bezformata.com',
 'kashira.bezformata.com': 'bezformata.com',
 'mojaysk.bezformata.com': 'bezformata.com',
 'volokolamsk.bezformata.com': 'bezformata.com',
 'yaransk.bezformata.com': 'bezformata.com',
 'kalach.bezformata.com': 'bezformata.com',
 'tbilisskaya.bezformata.com': 'bezformata.com',
 'vilyuchinsk.bezformata.com': 'bezformata.com',
 'nazran.bezformata.com': 'bezformata.com',
 'slavyanka.bezformata.com': 'bezformata.com',
 'komsomolsk.bezformata.com': 'bezformata.com',
 'gavrilovposad.bezformata.com': 'bezformata.com',
 'zavoljsk.bezformata.com': 'bezformata.com',
 'krasnoe.bezformata.com': 'bezformata.com',
 'volokonovka.bezformata.com': 'bezformata.com',
 'arsenev.bezformata.com': 'bezformata.com',
 'raevskiy.bezformata.com': 'bezformata.com',
 'kandalaksha.bezformata.com': 'bezformata.com',
 'severomorsk.bezformata.com': 'bezformata.com',
 'venev.bezformata.com': 'bezformata.com',
 'kogalm.bezformata.com': 'bezformata.com',
 'kataysk.bezformata.com': 'bezformata.com',
 'dalmatovo.bezformata.com': 'bezformata.com',
 'belozerskoe.bezformata.com': 'bezformata.com',
 'shuchye.bezformata.com': 'bezformata.com',
 'korkino.bezformata.com': 'bezformata.com',
 'monchegorsk.bezformata.com': 'bezformata.com',
 'chernyahovsk.bezformata.com': 'bezformata.com',
 'pushkin.bezformata.com': 'bezformata.com',
 'ustdjeguta.bezformata.com': 'bezformata.com',
 'melnikovo.bezformata.com': 'bezformata.com',
 'kozhevnikovo.bezformata.com': 'bezformata.com',
 'krivosheino.bezformata.com': 'bezformata.com',
 'usman.bezformata.com': 'bezformata.com',
 'izhma.bezformata.com': 'bezformata.com',
 'dolinsk.bezformata.com': 'bezformata.com',
 'baltiysk.bezformata.com': 'bezformata.com',
 'gushrustalniy.bezformata.com': 'bezformata.com',
 'novosyolovo.bezformata.com': 'bezformata.com',
 'lobnya.bezformata.com': 'bezformata.com',
 'volhov.bezformata.com': 'bezformata.com',
 'gatchina.bezformata.com': 'bezformata.com',
 'ermekeevo.bezformata.com': 'bezformata.com',
 'adigeysk.bezformata.com': 'bezformata.com',
 'tahtamukay.bezformata.com': 'bezformata.com',
 'plast.bezformata.com': 'bezformata.com',
 'oktyabrskoechel.bezformata.com': 'bezformata.com',
 'vufaley.bezformata.com': 'bezformata.com',
 'zaokskiy.bezformata.com': 'bezformata.com',
 'donskoy.bezformata.com': 'bezformata.com',
 'belomorsk.bezformata.com': 'bezformata.com',
 'segeja.bezformata.com': 'bezformata.com',
 'kotelnich.bezformata.com': 'bezformata.com',
 'pizhanka.bezformata.com': 'bezformata.com',
 'belayaholunitsa.bezformata.com': 'bezformata.com',
 'yurya.bezformata.com': 'bezformata.com',
 'bologoe.bezformata.com': 'bezformata.com',
 'slobodskoy.bezformata.com': 'bezformata.com',
 'kyshevskaya.bezformata.com': 'bezformata.com',
 'tevriz.bezformata.com': 'bezformata.com',
 'ulyanovo.bezformata.com': 'bezformata.com',
 'konakovo.bezformata.com': 'bezformata.com',
 'novolakskoe.bezformata.com': 'bezformata.com',
 'neverkino.bezformata.com': 'bezformata.com',
 'krasniyholm.bezformata.com': 'bezformata.com',
 'kingisepp.bezformata.com': 'bezformata.com',
 'elektrostal.bezformata.com': 'bezformata.com',
 'vilgort.bezformata.com': 'bezformata.com',
 'divnogorsk.bezformata.com': 'bezformata.com',
 'kirishi.bezformata.com': 'bezformata.com',
 'kirovsk.bezformata.com': 'bezformata.com',
 'torjok.bezformata.com': 'bezformata.com',
 'nelidovo.bezformata.com': 'bezformata.com',
 'rjev.bezformata.com': 'bezformata.com',
 'kotelniki.bezformata.com': 'bezformata.com',
 'kovdor.bezformata.com': 'bezformata.com',
 'electrogorsk.bezformata.com': 'bezformata.com',
 'lotoshino.bezformata.com': 'bezformata.com',
 'apatiti.bezformata.com': 'bezformata.com',
 'hotkovo.bezformata.com': 'bezformata.com',
 'troiskmsk.bezformata.com': 'bezformata.com',
 'aginskoe.bezformata.com': 'bezformata.com',
 'shemisheyka.bezformata.com': 'bezformata.com',
 'nikolsk.bezformata.com': 'bezformata.com',
 'gorodishepnz.bezformata.com': 'bezformata.com',
 'kuznetsk.bezformata.com': 'bezformata.com',
 'chusovoy.bezformata.com': 'bezformata.com',
 'bolshoykamen.bezformata.com': 'bezformata.com',
 'bor.bezformata.com': 'bezformata.com',
 'suzun.bezformata.com': 'bezformata.com',
 'gornozavodsk.bezformata.com': 'bezformata.com',
 'novocherkassk.bezformata.com': 'bezformata.com',
 'glyadyanskoe.bezformata.com': 'bezformata.com',
 'petuhovo.bezformata.com': 'bezformata.com',
 'ostrovskoe.bezformata.com': 'bezformata.com',
 'aban.bezformata.com': 'bezformata.com',
 'kuragino.bezformata.com': 'bezformata.com',
 'kirillov.bezformata.com': 'bezformata.com',
 'kadui.bezformata.com': 'bezformata.com',
 'vitegra.bezformata.com': 'bezformata.com',
 'elizovo.bezformata.com': 'bezformata.com',
 'aleksandrovskoe.bezformata.com': 'bezformata.com',
 'gheleznovodsk.bezformata.com': 'bezformata.com',
 'falyonki.bezformata.com': 'bezformata.com',
 'krasnogvardeyskoe.bezformata.com': 'bezformata.com',
 'zaraysk.bezformata.com': 'bezformata.com',
 'karabash.bezformata.com': 'bezformata.com',
 'kusa.bezformata.com': 'bezformata.com',
 'zdvinsk.bezformata.com': 'bezformata.com',
 'krasnozyorskoe.bezformata.com': 'bezformata.com',
 'demyansk.bezformata.com': 'bezformata.com',
 'vyazniki.bezformata.com': 'bezformata.com',
 'vereshagino.bezformata.com': 'bezformata.com',
 'zarechniypenza.bezformata.com': 'bezformata.com',
 'lopatino.bezformata.com': 'bezformata.com',
 'akyar.bezformata.com': 'bezformata.com',
 'kalininskaya.bezformata.com': 'bezformata.com',
 'essentuki.bezformata.com': 'bezformata.com',
 'gusev.bezformata.com': 'bezformata.com',
 'kurya.bezformata.com': 'bezformata.com',
 'talmenka.bezformata.com': 'bezformata.com',
 'romanovo.bezformata.com': 'bezformata.com',
 'kosiha.bezformata.com': 'bezformata.com',
 'gornyak.bezformata.com': 'bezformata.com',
 'blagoveshenka.bezformata.com': 'bezformata.com',
 'belokuriha.bezformata.com': 'bezformata.com',
 'novoaltaysk.bezformata.com': 'bezformata.com',
 'slavgorod.bezformata.com': 'bezformata.com',
 'shelaboliha.bezformata.com': 'bezformata.com',
 'okulovka.bezformata.com': 'bezformata.com',
 'luybinskiy.bezformata.com': 'bezformata.com',
 'artem.bezformata.com': 'bezformata.com',
 'lesozavodsk.bezformata.com': 'bezformata.com',
 'pogranichniy.bezformata.com': 'bezformata.com',
 'morozovsk.bezformata.com': 'bezformata.com',
 'konstantinovsk.bezformata.com': 'bezformata.com',
 'petrovsk.bezformata.com': 'bezformata.com',
 'staroeshaygovo.bezformata.com': 'bezformata.com',
 'saki.bezformata.com': 'bezformata.com',
 'kresttci.bezformata.com': 'bezformata.com',
 'makushino.bezformata.com': 'bezformata.com',
 'pavlovskaya.bezformata.com': 'bezformata.com',
 'malgobek.bezformata.com': 'bezformata.com',
 'madzhalis.bezformata.com': 'bezformata.com',
 'berezniki.bezformata.com': 'bezformata.com',
 'yakshurbodya.bezformata.com': 'bezformata.com',
 'ustkulom.bezformata.com': 'bezformata.com',
 'ustkatav.bezformata.com': 'bezformata.com',
 'chebarkul.bezformata.com': 'bezformata.com',
 'bolhov.bezformata.com': 'bezformata.com',
 'soltci.bezformata.com': 'bezformata.com',
 'ardatovmordovia.bezformata.com': 'bezformata.com',
 'anjerosudjensk.bezformata.com': 'bezformata.com',
 'isilkul.bezformata.com': 'bezformata.com',
 'kolosovka.bezformata.com': 'bezformata.com',
 'maryanovka.bezformata.com': 'bezformata.com',
 'rasskazovo.bezformata.com': 'bezformata.com',
 'kirsanov.bezformata.com': 'bezformata.com',
 'muromcevo.bezformata.com': 'bezformata.com',
 'tavricheskoe.bezformata.com': 'bezformata.com',
 'poltavka.bezformata.com': 'bezformata.com',
 'mordovo.bezformata.com': 'bezformata.com',
 'morshansk.bezformata.com': 'bezformata.com',
 'kursavka.bezformata.com': 'bezformata.com',
 'obninsk.bezformata.com': 'bezformata.com',
 'medyn.bezformata.com': 'bezformata.com',
 'lyudinovo.bezformata.com': 'bezformata.com',
 'partizansk.bezformata.com': 'bezformata.com',
 'volnonadezhdinskoe.bezformata.com': 'bezformata.com',
 'dalnerechensk.bezformata.com': 'bezformata.com',
 'spasskdalniy.bezformata.com': 'bezformata.com',
 'beloyarskiy.bezformata.com': 'bezformata.com',
 'asbest.bezformata.com': 'bezformata.com',
 'krasnouralsk.bezformata.com': 'bezformata.com',
 'kirovgrad.bezformata.com': 'bezformata.com',
 'strejevoy.bezformata.com': 'bezformata.com',
 'ordinskoe.bezformata.com': 'bezformata.com',
 'udomlya.bezformata.com': 'bezformata.com',
 'ostashkov.bezformata.com': 'bezformata.com',
 'digora.bezformata.com': 'bezformata.com',
 'mineralnievodi.bezformata.com': 'bezformata.com',
 'buinsk.bezformata.com': 'bezformata.com',
 'kurchatov.bezformata.com': 'bezformata.com',
 'sovetsk.bezformata.com': 'bezformata.com',
 'shelehov.bezformata.com': 'bezformata.com',
 'pionerskiy.bezformata.com': 'bezformata.com',
 'kologriv.bezformata.com': 'bezformata.com',
 'tarko-sale.bezformata.com': 'bezformata.com',
 'zainsk.bezformata.com': 'bezformata.com',
 'anadir.bezformata.com': 'bezformata.com',
 'pospeliha.bezformata.com': 'bezformata.com',
 'zarinsk.bezformata.com': 'bezformata.com',
 'kalmanka.bezformata.com': 'bezformata.com',
 'rodino.bezformata.com': 'bezformata.com',
 'yarovoe.bezformata.com': 'bezformata.com',
 'altayskoe.bezformata.com': 'bezformata.com',
 'tumencevo.bezformata.com': 'bezformata.com',
 'pavlovskaltay.bezformata.com': 'bezformata.com',
 'kizil.bezformata.com': 'bezformata.com',
 'krasnokamsk.bezformata.com': 'bezformata.com',
 'barda.bezformata.com': 'bezformata.com',
 'kohma.bezformata.com': 'bezformata.com',
 'jeleznogorsk.bezformata.com': 'bezformata.com',
 'kshenskiy.bezformata.com': 'bezformata.com',
 'severskaya.bezformata.com': 'bezformata.com',
 'novichiha.bezformata.com': 'bezformata.com',
 'sargatskoe.bezformata.com': 'bezformata.com',
 'amursk.bezformata.com': 'bezformata.com',
 'vorotinec.bezformata.com': 'bezformata.com',
 'solnechniy.bezformata.com': 'bezformata.com',
 'kovernino.bezformata.com': 'bezformata.com',
 'gorodec.bezformata.com': 'bezformata.com',
 'vanino.bezformata.com': 'bezformata.com',
 'spasskoe.bezformata.com': 'bezformata.com',
 'teploe.bezformata.com': 'bezformata.com',
 'plavsk.bezformata.com': 'bezformata.com',
 'arhangelskoe.bezformata.com': 'bezformata.com',
 'vishniyvolochek.bezformata.com': 'bezformata.com',
 'kagalnitckaya.bezformata.com': 'bezformata.com',
 'zimovniki.bezformata.com': 'bezformata.com',
 'kamenolomni.bezformata.com': 'bezformata.com',
 'oblivskaya.bezformata.com': 'bezformata.com',
 'lyambir.bezformata.com': 'bezformata.com',
 'luh.bezformata.com': 'bezformata.com',
 'uglich.bezformata.com': 'bezformata.com',
 'rostov.bezformata.com': 'bezformata.com',
 'kozmodemyansk.bezformata.com': 'bezformata.com',
 'chuhloma.bezformata.com': 'bezformata.com',
 'mostovskoy.bezformata.com': 'bezformata.com',
 'kerch.bezformata.com': 'bezformata.com',
 'murom.bezformata.com': 'bezformata.com',
 'vasiliha.bezformata.com': 'bezformata.com',
 'shushenskoe.bezformata.com': 'bezformata.com',
 'krasnoturansk.bezformata.com': 'bezformata.com',
 'gryazi.bezformata.com': 'bezformata.com',
 'chapligin.bezformata.com': 'bezformata.com',
 'kronshtadt.bezformata.com': 'bezformata.com',
 'tosno.bezformata.com': 'bezformata.com',
 'serishevo.bezformata.com': 'bezformata.com',
 'zeya.bezformata.com': 'bezformata.com',
 'vladimiroaleksandrovskoe.bezformata.com': 'bezformata.com',
 'tihvin.bezformata.com': 'bezformata.com',
 'nyandoma.bezformata.com': 'bezformata.com',
 'oktyabrskiyarh.bezformata.com': 'bezformata.com',
 'golitcino.bezformata.com': 'bezformata.com',
 'kitmanovo.bezformata.com': 'bezformata.com',
 'smolenskoe.bezformata.com': 'bezformata.com',
 'okoneshnikovo.bezformata.com': 'bezformata.com',
 'sedelnikovo.bezformata.com': 'bezformata.com',
 'malayaserdoba.bezformata.com': 'bezformata.com',
 'nazarovo.bezformata.com': 'bezformata.com',
 'pereslavlzalesskiy.bezformata.com': 'bezformata.com',
 'jeleznodorojniy.bezformata.com': 'bezformata.com',
 'mayma.bezformata.com': 'bezformata.com',
 'krasniechetai.bezformata.com': 'bezformata.com',
 'mariinskiyposad.bezformata.com': 'bezformata.com',
 'kanash.bezformata.com': 'bezformata.com',
 'shemursha.bezformata.com': 'bezformata.com',
 'yadrin.bezformata.com': 'bezformata.com',
 'dubovka.bezformata.com': 'bezformata.com',
 'luza.bezformata.com': 'bezformata.com',
 'vyatskiepolyani.bezformata.com': 'bezformata.com',
 'borodino.bezformata.com': 'bezformata.com',
 'shlisselburg.bezformata.com': 'bezformata.com',
 'vsevoljsk.bezformata.com': 'bezformata.com',
 'syasstroy.bezformata.com': 'bezformata.com',
 'prohladniy.bezformata.com': 'bezformata.com',
 'baksan.bezformata.com': 'bezformata.com',
 'bolotnoe.bezformata.com': 'bezformata.com',
 'bakali.bezformata.com': 'bezformata.com',
 'stariypetergof.bezformata.com': 'bezformata.com',
 'belayaglina.bezformata.com': 'bezformata.com',
 'ocher.bezformata.com': 'bezformata.com',
 'krasnovishersk.bezformata.com': 'bezformata.com',
 'osa.bezformata.com': 'bezformata.com',
 'nikolaevkauln.bezformata.com': 'bezformata.com',
 'luga.bezformata.com': 'bezformata.com',
 'rovnoe.bezformata.com': 'bezformata.com',
 'bogorodskoe.bezformata.com': 'bezformata.com',
 'ayan.bezformata.com': 'bezformata.com',
 'pereyaslavka.bezformata.com': 'bezformata.com',
 'sovetskayagavan.bezformata.com': 'bezformata.com',
 'satka.bezformata.com': 'bezformata.com',
 'bikin.bezformata.com': 'bezformata.com',
 'palech.bezformata.com': 'bezformata.com',
 'leninskkuzneckiy.bezformata.com': 'bezformata.com',
 'bredi.bezformata.com': 'bezformata.com',
 'vyazemskiy.bezformata.com': 'bezformata.com',
 'chegdomina.bezformata.com': 'bezformata.com',
 'vurnari.bezformata.com': 'bezformata.com',
 'degtyarsk.bezformata.com': 'bezformata.com',
 'svetliyyar.bezformata.com': 'bezformata.com',
 'sovetskkirov.bezformata.com': 'bezformata.com',
 'primorskoahtarsk.bezformata.com': 'bezformata.com',
 'yurgamish.bezformata.com': 'bezformata.com',
 'fershampenuaz.bezformata.com': 'bezformata.com',
 'korenovsk.bezformata.com': 'bezformata.com',
 'novoorsk.bezformata.com': 'bezformata.com',
 'tatishevo.bezformata.com': 'bezformata.com',
 'ivanteevkasar.bezformata.com': 'bezformata.com',
 'ozinki.bezformata.com': 'bezformata.com',
 'zatosvetliy.bezformata.com': 'bezformata.com',
 'yaya.bezformata.com': 'bezformata.com',
 'kondrovo.bezformata.com': 'bezformata.com',
 'kilmez.bezformata.com': 'bezformata.com',
 'zelenogradsk.bezformata.com': 'bezformata.com',
 'troitskoe.bezformata.com': 'bezformata.com',
 'ujuralsk.bezformata.com': 'bezformata.com',
 'yuryuzan.bezformata.com': 'bezformata.com',
 'kargopori.bezformata.com': 'bezformata.com',
 'dmitriev.bezformata.com': 'bezformata.com',
 'kargopol.bezformata.com': 'bezformata.com',
 'mrakovo.bezformata.com': 'bezformata.com',
 'karaidel.bezformata.com': 'bezformata.com',
 'starosubhangulovo.bezformata.com': 'bezformata.com',
 'pechora.bezformata.com': 'bezformata.com',
 'kashari.bezformata.com': 'bezformata.com',
 'milyutinskaya.bezformata.com': 'bezformata.com',
 'glubokiy.bezformata.com': 'bezformata.com',
 'semikarakorsk.bezformata.com': 'bezformata.com',
 'aramil.bezformata.com': 'bezformata.com',
 'verhoturie.bezformata.com': 'bezformata.com',
 'muchkapskiy.bezformata.com': 'bezformata.com',
 'kizilyurt.bezformata.com': 'bezformata.com',
 'golishmanovo.bezformata.com': 'bezformata.com',
 'semyonov.bezformata.com': 'bezformata.com',
 'kasimov.bezformata.com': 'bezformata.com',
 'nijniylomov.bezformata.com': 'bezformata.com',
 'frolovo.bezformata.com': 'bezformata.com',
 'neya.bezformata.com': 'bezformata.com',
 'navoloki.bezformata.com': 'bezformata.com',
 'pesnyaki.bezformata.com': 'bezformata.com',
 'djankoy.bezformata.com': 'bezformata.com',
 'bodaibo.bezformata.com': 'bezformata.com',
 'yuzha.bezformata.com': 'bezformata.com',
 'podporojye.bezformata.com': 'bezformata.com',
 'zelenodolsk.bezformata.com': 'bezformata.com',
 'arti.bezformata.com': 'bezformata.com',
 'chulim.bezformata.com': 'bezformata.com',
 'serdobsk.bezformata.com': 'bezformata.com',
 'zvenigovo.bezformata.com': 'bezformata.com',
 'mariturek.bezformata.com': 'bezformata.com',
 'ilek.bezformata.com': 'bezformata.com',
 'neftekumsk.bezformata.com': 'bezformata.com',
 'salavat.bezformata.com': 'bezformata.com',
 'iskateley.bezformata.com': 'bezformata.com',
 'olenegorsk.bezformata.com': 'bezformata.com',
 'nolinsk.bezformata.com': 'bezformata.com',
 'sosnogorsk.bezformata.com': 'bezformata.com',
 'emva.bezformata.com': 'bezformata.com',
 'ipatovo.bezformata.com': 'bezformata.com',
 'zima.bezformata.com': 'bezformata.com',
 'kolchugino.bezformata.com': 'bezformata.com',
 'orda.bezformata.com': 'bezformata.com',
 'suhobuzimskoe.bezformata.com': 'bezformata.com',
 'priozersk.bezformata.com': 'bezformata.com',
 'babevo.bezformata.com': 'bezformata.com',
 'vyazma.bezformata.com': 'bezformata.com',
 'sharlik.bezformata.com': 'bezformata.com',
 'znamensk.bezformata.com': 'bezformata.com',
 'aleksandrovka.bezformata.com': 'bezformata.com',
 'urkarah.bezformata.com': 'bezformata.com',
 'sobolevo.bezformata.com': 'bezformata.com',
 'zelenogorsk.bezformata.com': 'bezformata.com',
 'kasumkent.bezformata.com': 'bezformata.com',
 'gunib.bezformata.com': 'bezformata.com',
 'izberbash.bezformata.com': 'bezformata.com',
 'sergokala.bezformata.com': 'bezformata.com',
 'mehelta.bezformata.com': 'bezformata.com',
 'ostrov.bezformata.com': 'bezformata.com',
 'puchej.bezformata.com': 'bezformata.com',
 'zubovapolyana.bezformata.com': 'bezformata.com',
 'bolshoeignatovo.bezformata.com': 'bezformata.com',
 'romodanovo.bezformata.com': 'bezformata.com',
 'kabansk.bezformata.com': 'bezformata.com',
 'gorodovikovsk.bezformata.com': 'bezformata.com',
 'krasnoslobodskmrd.bezformata.com': 'bezformata.com',
 'koryazhma.bezformata.com': 'bezformata.com',
 'bolshieberezniki.bezformata.com': 'bezformata.com',
 'temnikov.bezformata.com': 'bezformata.com',
 'elniki.bezformata.com': 'bezformata.com',
 'giaginskaya.bezformata.com': 'bezformata.com',
 'ustkan.bezformata.com': 'bezformata.com',
 'gudermes.bezformata.com': 'bezformata.com',
 'ponezhukay.bezformata.com': 'bezformata.com',
 'krasnogvardeyskoeadg.bezformata.com': 'bezformata.com',
 'koshehabl.bezformata.com': 'bezformata.com',
 'sobinka.bezformata.com': 'bezformata.com',
 'gorohovec.bezformata.com': 'bezformata.com',
 'celinnoe.bezformata.com': 'bezformata.com',
 'zaozerniy.bezformata.com': 'bezformata.com',
 'krasnogvardeiskoe.bezformata.com': 'bezformata.com',
 'insar.bezformata.com': 'bezformata.com',
 'sayansk.bezformata.com': 'bezformata.com',
 'pryamitcino.bezformata.com': 'bezformata.com',
 'manturovoksk.bezformata.com': 'bezformata.com',
 'kastornoe.bezformata.com': 'bezformata.com',
 'zhukovka.bezformata.com': 'bezformata.com',
 'kimovsk.bezformata.com': 'bezformata.com',
 'buynaksk.bezformata.com': 'bezformata.com',
 'maloyaroslavec.bezformata.com': 'bezformata.com',
 'peschanokopskoe.bezformata.com': 'bezformata.com',
 'millerovo.bezformata.com': 'bezformata.com',
 'georgievsk.bezformata.com': 'bezformata.com',
 'krasnayayaruga.bezformata.com': 'bezformata.com',
 'taseevo.bezformata.com': 'bezformata.com',
 'belogorskkrim.bezformata.com': 'bezformata.com',
 'evpatoria.bezformata.com': 'bezformata.com',
 'alatir.bezformata.com': 'bezformata.com',
 'batirevo.bezformata.com': 'bezformata.com',
 'novocheboksarsk.bezformata.com': 'bezformata.com',
 'alikovo.bezformata.com': 'bezformata.com',
 'ibresi.bezformata.com': 'bezformata.com',
 'jirnovsk.bezformata.com': 'bezformata.com',
 'lermontov.bezformata.com': 'bezformata.com',
 'blagodarniy.bezformata.com': 'bezformata.com',
 'kunashak.bezformata.com': 'bezformata.com',
 'kotovsk.bezformata.com': 'bezformata.com',
 'mokshan.bezformata.com': 'bezformata.com',
 'labitlangi.bezformata.com': 'bezformata.com',
 'kalachinsk.bezformata.com': 'bezformata.com',
 'nazivaevsk.bezformata.com': 'bezformata.com',
 'bruhovetskaya.bezformata.com': 'bezformata.com',
 'verhniekigi.bezformata.com': 'bezformata.com',
 'spasklepiki.bezformata.com': 'bezformata.com',
 'vozhega.bezformata.com': 'bezformata.com',
 'kamizyak.bezformata.com': 'bezformata.com',
 'surovikino.bezformata.com': 'bezformata.com',
 'tabuni.bezformata.com': 'bezformata.com',
 'emelyanovo.bezformata.com': 'bezformata.com',
 'kremyonki.bezformata.com': 'bezformata.com',
 'vetluga.bezformata.com': 'bezformata.com',
 'verhnietatishli.bezformata.com': 'bezformata.com',
 'bograd.bezformata.com': 'bezformata.com',
 'radujniy.bezformata.com': 'bezformata.com',
 'lyantor.bezformata.com': 'bezformata.com',
 'krasnoarmeyskoesmr.bezformata.com': 'bezformata.com',
 'dobryanka.bezformata.com': 'bezformata.com',
 'perevoz.bezformata.com': 'bezformata.com',
 'uren.bezformata.com': 'bezformata.com',
 'nefteyugansk.bezformata.com': 'bezformata.com',
 'arzgir.bezformata.com': 'bezformata.com',
 'kaspiysk.bezformata.com': 'bezformata.com',
 'lunino.bezformata.com': 'bezformata.com',
 'repyovka.bezformata.com': 'bezformata.com',
 'snejinsk.bezformata.com': 'bezformata.com',
 'kameshkovo.bezformata.com': 'bezformata.com',
 'sasovo.bezformata.com': 'bezformata.com',
 'cheremhovo.bezformata.com': 'bezformata.com',
 'mednogorsk.bezformata.com': 'bezformata.com',
 'jukov.bezformata.com': 'bezformata.com',
 'kirovkaluga.bezformata.com': 'bezformata.com',
 'sosnovka.bezformata.com': 'bezformata.com',
 'bahchisaray.bezformata.com': 'bezformata.com',
 'krasnoperekopsk.bezformata.com': 'bezformata.com',
 'feodosiya.bezformata.com': 'bezformata.com',
 'durtuli.bezformata.com': 'bezformata.com',
 'buraevo.bezformata.com': 'bezformata.com',
 'bolshoeboldino.bezformata.com': 'bezformata.com',
 'balahna.bezformata.com': 'bezformata.com',
 'starobaltachevo.bezformata.com': 'bezformata.com',
 'nikoloberyozovka.bezformata.com': 'bezformata.com',
 'beloreck.bezformata.com': 'bezformata.com',
 'mesyagutovo.bezformata.com': 'bezformata.com',
 'sibay.bezformata.com': 'bezformata.com',
 'uchali.bezformata.com': 'bezformata.com',
 'baymak.bezformata.com': 'bezformata.com',
    'gidm.ru': 'gidm.ru',
 'spb.gidm.ru': 'gidm.ru',
 'samara.gidm.ru': 'gidm.ru',
 'krasnodar.gidm.ru': 'gidm.ru',
 'tumen.gidm.ru': 'gidm.ru',
 'hlebisalo.gidm.ru': 'gidm.ru',
 'nizhniy-novgorod.gidm.ru': 'gidm.ru',
 'unost34.gidm.ru': 'gidm.ru',
 'volgograd.gidm.ru': 'gidm.ru',
 'russamovar.gidm.ru': 'gidm.ru',
 'shinok26.gidm.ru': 'gidm.ru',
 'hirurg.gidm.ru': 'gidm.ru',
 'orange.gidm.ru': 'gidm.ru',
 'mayak.gidm.ru': 'gidm.ru',
 'lasvegas.gidm.ru': 'gidm.ru',
 'volzhskiy.gidm.ru': 'gidm.ru',
 'friendly-cgbar.gidm.ru': 'gidm.ru',
 'salsapicante.gidm.ru': 'gidm.ru',
 'moscow.gidm.ru': 'gidm.ru',
 'omsk.gidm.ru': 'gidm.ru',
 'perm.gidm.ru': 'gidm.ru',
 'voronezh.gidm.ru': 'gidm.ru',
 'snezhinka52.gidm.ru': 'gidm.ru',
 'akvamarin13.gidm.ru': 'gidm.ru',
 'rostov-na-donu.gidm.ru': 'gidm.ru',
    'russia.priceok.ru': 'priceok.ru',
 'vladimir.priceok.ru': 'priceok.ru',
 'murmansk.priceok.ru': 'priceok.ru',
 'yoshkar-ola.priceok.ru': 'priceok.ru',
 'blagoveschensk.priceok.ru': 'priceok.ru',
 'ekaterinburg.priceok.ru': 'priceok.ru',
 'kaluga.priceok.ru': 'priceok.ru',
 'voronezh.priceok.ru': 'priceok.ru',
 'makhachkala.priceok.ru': 'priceok.ru',
 'vladivostok.priceok.ru': 'priceok.ru',
 'barnaul.priceok.ru': 'priceok.ru',
 'kursk.priceok.ru': 'priceok.ru',
 'lipetsk.priceok.ru': 'priceok.ru',
 'irkutsk.priceok.ru': 'priceok.ru',
 'yuzhno-sakhalinsk.priceok.ru': 'priceok.ru',
 'izhevsk.priceok.ru': 'priceok.ru',
 'kaliningrad.priceok.ru': 'priceok.ru',
 'kemerovo.priceok.ru': 'priceok.ru',
 'kirov.priceok.ru': 'priceok.ru',
 'omsk.priceok.ru': 'priceok.ru',
 'bryansk.priceok.ru': 'priceok.ru',
 'krasnoyarsk.priceok.ru': 'priceok.ru',
 'groznyy.priceok.ru': 'priceok.ru',
 'nizhniynovgorod.priceok.ru': 'priceok.ru',
 'priceok.ru': 'priceok.ru',
 'ufa.priceok.ru': 'priceok.ru',
    'ifk.sportedu.ru': 'rgufk.ru',
 'press.sportedu.ru': 'rgufk.ru',
 'mkf.sportedu.ru': 'rgufk.ru',
 'ido.sportedu.ru': 'rgufk.ru',
 'rgufk.ru': 'rgufk.ru',
 'la.sportedu.ru': 'rgufk.ru',
 'turism.sportedu.ru': 'rgufk.ru',
 'hi.sportedu.ru': 'rgufk.ru',
 'fs.sportedu.ru': 'rgufk.ru',
 'football.sportedu.ru': 'rgufk.ru',
 'psy.sportedu.ru': 'rgufk.ru',
 'pedagogy.sportedu.ru': 'rgufk.ru',
 'ipo.sportedu.ru': 'rgufk.ru',
 'lfk.sportedu.ru': 'rgufk.ru',
 'campus.sportedu.ru': 'rgufk.ru',
 'theses.sportedu.ru': 'rgufk.ru',
 'practice.sportedu.ru': 'rgufk.ru',
 'magistr.sportedu.ru': 'rgufk.ru',
 'post-graduate.sportedu.ru': 'rgufk.ru',
 'job.sportedu.ru': 'rgufk.ru',
 'upbringing.sportedu.ru': 'rgufk.ru',
 'umu.sportedu.ru': 'rgufk.ru',
 'museum.sportedu.ru': 'rgufk.ru',
 'officially.sportedu.ru': 'rgufk.ru',
 'se.sportedu.ru': 'rgufk.ru',
 'halls.sportedu.ru': 'rgufk.ru',
    'bankiros.ru': 'bankiros.ru',
 'rostov-na-donu.bankiros.ru': 'bankiros.ru',
 'petrozavodsk.bankiros.ru': 'bankiros.ru',
 'elektrostal.bankiros.ru': 'bankiros.ru',
 'chelyabinsk.bankiros.ru': 'bankiros.ru',
 'amurskaya-oblast-blagoveschensk.bankiros.ru': 'bankiros.ru',
 'barnaul.bankiros.ru': 'bankiros.ru',
 'voronezh.bankiros.ru': 'bankiros.ru',
 'smolensk.bankiros.ru': 'bankiros.ru',
 'eysk.bankiros.ru': 'bankiros.ru',
 'novomoskovsk.bankiros.ru': 'bankiros.ru',
 'grozniy.bankiros.ru': 'bankiros.ru',
 'vladikavkaz.bankiros.ru': 'bankiros.ru',
 'kogalim.bankiros.ru': 'bankiros.ru',
 'sayanogorsk.bankiros.ru': 'bankiros.ru',
 'essentuki.bankiros.ru': 'bankiros.ru',
 'novodvinsk.bankiros.ru': 'bankiros.ru',
 'stariy-oskol.bankiros.ru': 'bankiros.ru',
 'abakan.bankiros.ru': 'bankiros.ru',
 'nalchik.bankiros.ru': 'bankiros.ru',
 'nefteugansk.bankiros.ru': 'bankiros.ru',
 'chistopol.bankiros.ru': 'bankiros.ru',
 'viksa.bankiros.ru': 'bankiros.ru',
 'cherkessk.bankiros.ru': 'bankiros.ru',
 'pechora.bankiros.ru': 'bankiros.ru',
 'borisoglebsk.bankiros.ru': 'bankiros.ru',
    'ruplans.ru': 'ruplans.ru',
 'omsk.ruplans.ru': 'ruplans.ru',
 'm.ruplans.ru': 'ruplans.ru',
 'ufa.ruplans.ru': 'ruplans.ru',
 'sochi.ruplans.ru': 'ruplans.ru',
 'krasnoyarsk.ruplans.ru': 'ruplans.ru',
 'kz.ruplans.ru': 'ruplans.ru',
 'taganrog.ruplans.ru': 'ruplans.ru',
 'perm.ruplans.ru': 'ruplans.ru',
 'rostov.ruplans.ru': 'ruplans.ru',
 'nsk.ruplans.ru': 'ruplans.ru',
 'cherepovets.ruplans.ru': 'ruplans.ru',
 'orel.ruplans.ru': 'ruplans.ru',
 'lipetsk.ruplans.ru': 'ruplans.ru',
 'vologda.ruplans.ru': 'ruplans.ru',
 'spb.ruplans.ru': 'ruplans.ru',
 'voronezh.ruplans.ru': 'ruplans.ru',
 'tula.ruplans.ru': 'ruplans.ru',
 'krasnodar.ruplans.ru': 'ruplans.ru',
 'tumen.ruplans.ru': 'ruplans.ru',
 'moscow.ruplans.ru': 'ruplans.ru',
 'abakan.ruplans.ru': 'ruplans.ru',
 'kaliningrad.ruplans.ru': 'ruplans.ru',
 'tver.ruplans.ru': 'ruplans.ru',
 'ryazan.ruplans.ru': 'ruplans.ru',
 'yakutsk.ruplans.ru': 'ruplans.ru',
 'barnaul.ruplans.ru': 'ruplans.ru',
    'krasnoyarsk.tuzlist.ru': 'tuzlist.ru',
 'tuzlist.ru': 'tuzlist.ru',
 'krasnodar.tuzlist.ru': 'tuzlist.ru',
 'novokuznetsk.tuzlist.ru': 'tuzlist.ru',
 'ekaterinburg.tuzlist.ru': 'tuzlist.ru',
 'orenburg.tuzlist.ru': 'tuzlist.ru',
 'sochi.tuzlist.ru': 'tuzlist.ru',
 'moscow.tuzlist.ru': 'tuzlist.ru',
 'nizhniytagil.tuzlist.ru': 'tuzlist.ru',
 'khabarovsk.tuzlist.ru': 'tuzlist.ru',
 'naberezhnyechelny.tuzlist.ru': 'tuzlist.ru',
 'omsk.tuzlist.ru': 'tuzlist.ru',
 'tomsk.tuzlist.ru': 'tuzlist.ru',
 'nizhniynovgorod.tuzlist.ru': 'tuzlist.ru',
 'yoshkarola.tuzlist.ru': 'tuzlist.ru',
 'sanktpeterburg.tuzlist.ru': 'tuzlist.ru',
 'ufa.tuzlist.ru': 'tuzlist.ru',
 'penza.tuzlist.ru': 'tuzlist.ru',
 'barnaul.tuzlist.ru': 'tuzlist.ru',
 'volgograd.tuzlist.ru': 'tuzlist.ru',
 'lesozavodsk.tuzlist.ru': 'tuzlist.ru',
 'sterlitamak.tuzlist.ru': 'tuzlist.ru',
 'neftekamsk.tuzlist.ru': 'tuzlist.ru',
 'novosibirsk.tuzlist.ru': 'tuzlist.ru',
 'belorechensk.tuzlist.ru': 'tuzlist.ru',
 'kazan.tuzlist.ru': 'tuzlist.ru',
 'rostov.tuzlist.ru': 'tuzlist.ru',
 'rostovnadonu.tuzlist.ru': 'tuzlist.ru',
    'brn.en.cx': 'en.cx',
 'yaroslavl.en.cx': 'en.cx',
 'ykt.en.cx': 'en.cx',
 'nrg.en.cx': 'en.cx',
 'dom.en.cx': 'en.cx',
 'chita.en.cx': 'en.cx',
 'blaga.en.cx': 'en.cx',
 'nn.en.cx': 'en.cx',
 'star69.en.cx': 'en.cx',
 'game.en.cx': 'en.cx',
 '59.en.cx': 'en.cx',
 'lpk.en.cx': 'en.cx',
 'tmb.en.cx': 'en.cx',
 'pnz.en.cx': 'en.cx',
 'fest.en.cx': 'en.cx',
 'ka.en.cx': 'en.cx',
 'moscow.en.cx': 'en.cx',
 '102.en.cx': 'en.cx',
 'ufa.en.cx': 'en.cx',
 'chel.en.cx': 'en.cx',
 'oren.en.cx': 'en.cx',
 'nt.en.cx': 'en.cx',
 'ahtubinsk.en.cx': 'en.cx',
 '31.en.cx': 'en.cx',
 '72.en.cx': 'en.cx',
 'udm.en.cx': 'en.cx',
 'msk.en.cx': 'en.cx',
 'orel.en.cx': 'en.cx',
    'miltor.ru': 'miltor.ru',
 'moskva.miltor.ru': 'miltor.ru',
 'mtsensk.miltor.ru': 'miltor.ru',
 'yuryuzan.miltor.ru': 'miltor.ru',
 'kirzhach.miltor.ru': 'miltor.ru',
 'orel.miltor.ru': 'miltor.ru',
 'vichuga.miltor.ru': 'miltor.ru',
 'sankt-peterburg.miltor.ru': 'miltor.ru',
 'magnitogorsk.miltor.ru': 'miltor.ru',
 'sorochinsk.miltor.ru': 'miltor.ru',
 'podolsk.miltor.ru': 'miltor.ru',
 'dmitrov.miltor.ru': 'miltor.ru',
 'aleysk.miltor.ru': 'miltor.ru',
 'buguruslan.miltor.ru': 'miltor.ru',
 'buzuluk.miltor.ru': 'miltor.ru',
 'pugachev.miltor.ru': 'miltor.ru',
 'sergiev-posad.miltor.ru': 'miltor.ru',
 'labinsk.miltor.ru': 'miltor.ru',
 'egorevsk.miltor.ru': 'miltor.ru',
 'tuymazi.miltor.ru': 'miltor.ru',
 'belovo1.miltor.ru': 'miltor.ru',
 'zelenograd.miltor.ru': 'miltor.ru',
 'murom.miltor.ru': 'miltor.ru',
 'goryachiy-klyuch.miltor.ru': 'miltor.ru',
 'minusinsk.miltor.ru': 'miltor.ru',
 'nurlat.miltor.ru': 'miltor.ru',
 'murmansk.miltor.ru': 'miltor.ru',
 'teykovo.miltor.ru': 'miltor.ru',
 'ryazhsk.miltor.ru': 'miltor.ru',
 'sarapul.miltor.ru': 'miltor.ru',
    'bb-4.lordfilm.green': 'zr.lordfilm.fans',
 'gh-15.lordfilm.green': 'zr.lordfilm.fans',
 'rt-11.lordfilm.green': 'zr.lordfilm.fans',
 'zr.lordfilm.fans': 'zr.lordfilm.fans',
 'xx-0.lordfilm.green': 'zr.lordfilm.fans',
 'buu2.lordfilm5.zone': 'zr.lordfilm.fans',
 'b.lordfilms.icu': 'zr.lordfilm.fans',
 'iz-11.lordfilm.green': 'zr.lordfilm.fans',
 'lord-33.lordfilm0.org': 'zr.lordfilm.fans',
 'troy.lordfilms.cfd': 'zr.lordfilm.fans',
 'uid.lordfilms.cfd': 'zr.lordfilm.fans',
 '2022-222.lordfilm0.com': 'zr.lordfilm.fans',
 'tos2.lordfilms.cfd': 'zr.lordfilm.fans',
 'tti9.lordfilm5.zone': 'zr.lordfilm.fans',
 'mag.lordfilms.cfd': 'zr.lordfilm.fans',
 'lord686.lordfilm.fans': 'zr.lordfilm.fans',
 '2022-99.lordfilm0.com': 'zr.lordfilm.fans',
 'erd2.lordfilms.cfd': 'zr.lordfilm.fans',
 '14.lordfilm0.net': 'zr.lordfilm.fans',
 'raz1.lordfilm5.zone': 'zr.lordfilm.fans',
 'rtt1.lordfilm5.zone': 'zr.lordfilm.fans',
 'zaa2.lordfilm5.zone': 'zr.lordfilm.fans',
 'nap2.lordfilms.cfd': 'zr.lordfilm.fans',
 'ooi.lordfilms.cfd': 'zr.lordfilm.fans',
 'ro11.lordfilm5.zone': 'zr.lordfilm.fans',
 'hh7.lordfilm5.zone': 'zr.lordfilm.fans',
 'umm1.lordfilm5.zone': 'zr.lordfilm.fans',
 'err2.lordfilms.cfd': 'zr.lordfilm.fans',
 'ea-21.lordfilm.green': 'zr.lordfilm.fans',
 'zu-18.lordfilm.green': 'zr.lordfilm.fans',
 'tbot.lordfilms.cfd': 'zr.lordfilm.fans',
    'amp.tsargrad.tv': 'tsargrad.tv',
 'tsargrad.tv': 'tsargrad.tv',
 'm.tsargrad.tv': 'tsargrad.tv',
 'm-chita.tsargrad.tv': 'tsargrad.tv',
 'm-ekb.tsargrad.tv': 'tsargrad.tv',
 'm-am.tsargrad.tv': 'tsargrad.tv',
 'spb.tsargrad.tv': 'tsargrad.tv',
 'm-nsk.tsargrad.tv': 'tsargrad.tv',
 'm-samara.tsargrad.tv': 'tsargrad.tv',
 'samara.tsargrad.tv': 'tsargrad.tv',
 'm-spb.tsargrad.tv': 'tsargrad.tv',
 'am.tsargrad.tv': 'tsargrad.tv',
 'kz.tsargrad.tv': 'tsargrad.tv',
 'mo.tsargrad.tv': 'tsargrad.tv',
 'nn.tsargrad.tv': 'tsargrad.tv',
 'm-nn.tsargrad.tv': 'tsargrad.tv',
 'nsk.tsargrad.tv': 'tsargrad.tv',
 'm-ug.tsargrad.tv': 'tsargrad.tv',
 'by.tsargrad.tv': 'tsargrad.tv',
 'ekb.tsargrad.tv': 'tsargrad.tv',
 'ug.tsargrad.tv': 'tsargrad.tv',
 'md.tsargrad.tv': 'tsargrad.tv',
 'm-rostov.tsargrad.tv': 'tsargrad.tv',
 'rostov.tsargrad.tv': 'tsargrad.tv',
 'chita.tsargrad.tv': 'tsargrad.tv',
 'm-kemerovo.tsargrad.tv': 'tsargrad.tv',
 'vladimir.tsargrad.tv': 'tsargrad.tv',
 'm-vladimir.tsargrad.tv': 'tsargrad.tv',
 'm-ural.tsargrad.tv': 'tsargrad.tv',
 'm-kz.tsargrad.tv': 'tsargrad.tv',
 'ural.tsargrad.tv': 'tsargrad.tv',
 'kemerovo.tsargrad.tv': 'tsargrad.tv',
    'm.oceanoflove.ru': 'mamba.ru',
 'm.love.vl.ru': 'mamba.ru',
 'm.couple.swingplaneta.com': 'mamba.ru',
 'm.tamdam.ru': 'mamba.ru',
 'android.love.mail.ru': 'mamba.ru',
 'm.koketochka.ru': 'mamba.ru',
 'm.lowelas.ru': 'mamba.ru',
 'm.flirtru.ru': 'mamba.ru',
 'wap.nymba.ru': 'mamba.ru',
 'm.love.mambo.ru': 'mamba.ru',
 'm.znakomstvagood.ru': 'mamba.ru',
 'm.girlfriend.ru': 'mamba.ru',
 'm.jdu.ru': 'mamba.ru',
 'm.lovesamara.ru': 'mamba.ru',
 'm.lov.ru': 'mamba.ru',
 'm.love.trans-admirer.ru': 'mamba.ru',
 'm.2serdtsa.ru': 'mamba.ru',
 'wap.2serdtsa.ru': 'mamba.ru',
 'beeline.mamba.ru': 'mamba.ru',
 'm.love.1777.ru': 'mamba.ru',
 'm.love.saratov.ru': 'mamba.ru',
 'm.love.vsamare.com': 'mamba.ru',
 'm.omegalove.ru': 'mamba.ru',
 'm.4love.ru': 'mamba.ru',
 'm.holiday.ru': 'mamba.ru',
 'm.gut.ru': 'mamba.ru',
 'm.love.kenigsberg.su': 'mamba.ru',
 'love.dddkursk.ru': 'mamba.ru',
 'm.love.izgr.ru': 'mamba.ru',
 'm.date.ru': 'mamba.ru',
 'm.love.gde.ru': 'mamba.ru',
 'm.mromance.ru': 'mamba.ru',
    'gorjob.ru': 'gorjob.ru',
 'ufa.gorjob.ru': 'gorjob.ru',
 'tmn.gorjob.ru': 'gorjob.ru',
 'samara.gorjob.ru': 'gorjob.ru',
 'tver.gorjob.ru': 'gorjob.ru',
 'krym.gorjob.ru': 'gorjob.ru',
 'chelny.gorjob.ru': 'gorjob.ru',
 'izh.gorjob.ru': 'gorjob.ru',
 'ptz.gorjob.ru': 'gorjob.ru',
 'stav.gorjob.ru': 'gorjob.ru',
 'kzn.gorjob.ru': 'gorjob.ru',
 'rostov.gorjob.ru': 'gorjob.ru',
 'tomsk.gorjob.ru': 'gorjob.ru',
 'grozny.gorjob.ru': 'gorjob.ru',
 'perm.gorjob.ru': 'gorjob.ru',
 'omsk.gorjob.ru': 'gorjob.ru',
 'blag.gorjob.ru': 'gorjob.ru',
 'msk.gorjob.ru': 'gorjob.ru',
 'kras.gorjob.ru': 'gorjob.ru',
 'kaluga.gorjob.ru': 'gorjob.ru',
 'ekb.gorjob.ru': 'gorjob.ru',
 'shahty.gorjob.ru': 'gorjob.ru',
 'mosobl.gorjob.ru': 'gorjob.ru',
 'sochi.gorjob.ru': 'gorjob.ru',
 'lenobl.gorjob.ru': 'gorjob.ru',
 'vologda.gorjob.ru': 'gorjob.ru',
 'novgorod.gorjob.ru': 'gorjob.ru',
 'krd.gorjob.ru': 'gorjob.ru',
 'chel.gorjob.ru': 'gorjob.ru',
 'yanao.gorjob.ru': 'gorjob.ru',
 'tlt.gorjob.ru': 'gorjob.ru',
 'almet.gorjob.ru': 'gorjob.ru',
    '56440696406159.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56440428232847.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56444682240143.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 'xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56531453739151.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56444541403279.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56439392174223.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56446971543695.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '54102216212571.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56913650843791.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56441972916367.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56538252705935.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56441794723983.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56538081132687.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56917505081487.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56561039048847.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56920013734031.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56919913136271.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56561057595535.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56126916788340.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56920100503695.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56908040241295.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56907927060623.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56912389144719.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56442137346191.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56929676296335.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56538034798735.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56446899060879.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56885787820175.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56548426252431.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56440488853647.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56917705359503.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
 '56441814712463.xn---6--2ddnhoea2am.xn--p1ai': 'xn---6--2ddnhoea2am.xn--p1ai',
    'krd.specdispetcher.ru': 'specdispetcher.ru',
 'rzn.specdispetcher.ru': 'specdispetcher.ru',
 'brj.specdispetcher.ru': 'specdispetcher.ru',
 'bash.specdispetcher.ru': 'specdispetcher.ru',
 'krjs.specdispetcher.ru': 'specdispetcher.ru',
 'rost.specdispetcher.ru': 'specdispetcher.ru',
 'orb.specdispetcher.ru': 'specdispetcher.ru',
 'kurg.specdispetcher.ru': 'specdispetcher.ru',
 'klj.specdispetcher.ru': 'specdispetcher.ru',
 'kir.specdispetcher.ru': 'specdispetcher.ru',
 'vor.specdispetcher.ru': 'specdispetcher.ru',
 'kurs.specdispetcher.ru': 'specdispetcher.ru',
 'srt.specdispetcher.ru': 'specdispetcher.ru',
 'nsk.specdispetcher.ru': 'specdispetcher.ru',
 'dag.specdispetcher.ru': 'specdispetcher.ru',
 'nov.specdispetcher.ru': 'specdispetcher.ru',
 'smr.specdispetcher.ru': 'specdispetcher.ru',
 'kom.specdispetcher.ru': 'specdispetcher.ru',
 'tat.specdispetcher.ru': 'specdispetcher.ru',
 'msk.specdispetcher.ru': 'specdispetcher.ru',
 'spb.specdispetcher.ru': 'specdispetcher.ru',
 'vld.specdispetcher.ru': 'specdispetcher.ru',
 'specdispetcher.ru': 'specdispetcher.ru',
 'ulj.specdispetcher.ru': 'specdispetcher.ru',
 'alt.specdispetcher.ru': 'specdispetcher.ru',
 'klgd.specdispetcher.ru': 'specdispetcher.ru',
 'tum.specdispetcher.ru': 'specdispetcher.ru',
 'smol.specdispetcher.ru': 'specdispetcher.ru',
 'vlg.specdispetcher.ru': 'specdispetcher.ru',
 'che.specdispetcher.ru': 'specdispetcher.ru',
 'vol.specdispetcher.ru': 'specdispetcher.ru',
 'psk.specdispetcher.ru': 'specdispetcher.ru',
 'zab.specdispetcher.ru': 'specdispetcher.ru',
 'tvr.specdispetcher.ru': 'specdispetcher.ru',
    'gobuygo.ru': 'gobuygo.ru',
 'kineshma.gobuygo.ru': 'gobuygo.ru',
 'buguruslan.gobuygo.ru': 'gobuygo.ru',
 'temryuk.gobuygo.ru': 'gobuygo.ru',
 'izhevsk.gobuygo.ru': 'gobuygo.ru',
 'komsomolsk-na-amure.gobuygo.ru': 'gobuygo.ru',
 'vladivostok.gobuygo.ru': 'gobuygo.ru',
 'berezniki.gobuygo.ru': 'gobuygo.ru',
 'krymsk.gobuygo.ru': 'gobuygo.ru',
 'prokhladnyy.gobuygo.ru': 'gobuygo.ru',
 'tuapse.gobuygo.ru': 'gobuygo.ru',
 'morozovsk.gobuygo.ru': 'gobuygo.ru',
 'anapa.gobuygo.ru': 'gobuygo.ru',
 'magnitogorsk.gobuygo.ru': 'gobuygo.ru',
 'yujno-sahalinsk.gobuygo.ru': 'gobuygo.ru',
 'novorossiysk.gobuygo.ru': 'gobuygo.ru',
 'yurga.gobuygo.ru': 'gobuygo.ru',
 'safonovo.gobuygo.ru': 'gobuygo.ru',
 'arhangelsk.gobuygo.ru': 'gobuygo.ru',
 'vologda.gobuygo.ru': 'gobuygo.ru',
 'zelenogorsk.gobuygo.ru': 'gobuygo.ru',
 'adler.gobuygo.ru': 'gobuygo.ru',
 'novocherkassk.gobuygo.ru': 'gobuygo.ru',
 'belogorsk.gobuygo.ru': 'gobuygo.ru',
 'eysk.gobuygo.ru': 'gobuygo.ru',
 'petrozavodsk.gobuygo.ru': 'gobuygo.ru',
 'goryachiy-klyuch.gobuygo.ru': 'gobuygo.ru',
 'shakhty.gobuygo.ru': 'gobuygo.ru',
 'saransk.gobuygo.ru': 'gobuygo.ru',
 'sasovo.gobuygo.ru': 'gobuygo.ru',
 'vladikavkaz.gobuygo.ru': 'gobuygo.ru',
 'abinsk.gobuygo.ru': 'gobuygo.ru',
 'orenburg.gobuygo.ru': 'gobuygo.ru',
 'kalachinsk.gobuygo.ru': 'gobuygo.ru',
    'vh88.timeweb.ru': 'timeweb.ru',
 'vh354.timeweb.ru': 'timeweb.ru',
 'vh352.timeweb.ru': 'timeweb.ru',
 'vh164.timeweb.ru': 'timeweb.ru',
 'vh370.timeweb.ru': 'timeweb.ru',
 'vh188.timeweb.ru': 'timeweb.ru',
 'nubes.timeweb.ru': 'timeweb.ru',
 'bitrix286.timeweb.ru': 'timeweb.ru',
 'vh52.timeweb.ru': 'timeweb.ru',
 'vh196.timeweb.ru': 'timeweb.ru',
 'vh314.timeweb.ru': 'timeweb.ru',
 'tmweb.ru': 'timeweb.ru',
 'vh74.timeweb.ru': 'timeweb.ru',
 'vh124.timeweb.ru': 'timeweb.ru',
 'vh266.timeweb.ru': 'timeweb.ru',
 'moesha.timeweb.ru': 'timeweb.ru',
 'vh94.timeweb.ru': 'timeweb.ru',
 'vh374.timeweb.ru': 'timeweb.ru',
 'vh6.timeweb.ru': 'timeweb.ru',
 'vh176.timeweb.ru': 'timeweb.ru',
 'bone020.timeweb.ru': 'timeweb.ru',
 'aurum.timeweb.ru': 'timeweb.ru',
 'vh302.timeweb.ru': 'timeweb.ru',
 'bitrix362.timeweb.ru': 'timeweb.ru',
 'vh86.timeweb.ru': 'timeweb.ru',
 'csite1.timeweb.ru': 'timeweb.ru',
 'vh270.timeweb.ru': 'timeweb.ru',
 'bitrix296.timeweb.ru': 'timeweb.ru',
 'bitrix386.timeweb.ru': 'timeweb.ru',
 'vh298.timeweb.ru': 'timeweb.ru',
 'vh346.timeweb.ru': 'timeweb.ru',
 'vh82.timeweb.ru': 'timeweb.ru',
 'pulcher.timeweb.ru': 'timeweb.ru',
 'bitrix380.timeweb.ru': 'timeweb.ru',
 'bestia.timeweb.ru': 'timeweb.ru',
 'bitrix320.timeweb.ru': 'timeweb.ru',
    'static.wi-fi.ru': 'domru.wi-fi.ru',
 's3.wi-fi.ru': 'domru.wi-fi.ru',
 'wi-fi.ru': 'domru.wi-fi.ru',
 'spb.wi-fi.ru': 'domru.wi-fi.ru',
 'sport-wifi.ru': 'domru.wi-fi.ru',
 'samara.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'cppk.wi-fi.ru': 'domru.wi-fi.ru',
 'perm.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'mcc.wi-fi.ru': 'domru.wi-fi.ru',
 '3.wi-fi.ru': 'domru.wi-fi.ru',
 'aeroexpress.wi-fi.ru': 'domru.wi-fi.ru',
 'ekat.domru.wi-fi.ru': 'domru.wi-fi.ru',
 '2.wi-fi.ru': 'domru.wi-fi.ru',
 'chelny.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'yola.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'vko.wi-fi.ru': 'domru.wi-fi.ru',
 'nn.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'kirov.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'yar.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'kazan.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'saratov.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'mgn.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'rtk.wi-fi.ru': 'domru.wi-fi.ru',
 'msk.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'spb.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'ryazan.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'barnaul.domru.wi-fi.ru': 'domru.wi-fi.ru',
 '1.wi-fi.ru': 'domru.wi-fi.ru',
 'domru.wi-fi.ru': 'domru.wi-fi.ru',
 'oren.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'ulsk.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'volgograd.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'izhevsk.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'tmn.domru.wi-fi.ru': 'domru.wi-fi.ru',
 'tc.wi-fi.ru': 'domru.wi-fi.ru',
 'tver.domru.wi-fi.ru': 'domru.wi-fi.ru',
    'vikulovo.bezformata.com': 'bezformata.com',
 'armizonskoe.bezformata.com': 'bezformata.com',
 'sladkovo.bezformata.com': 'bezformata.com',
 'berdyuzhe.bezformata.com': 'bezformata.com',
 'moshkovo.bezformata.com': 'bezformata.com',
 'makarev.bezformata.com': 'bezformata.com',
 'parfenevo.bezformata.com': 'bezformata.com',
 'krasnoenavolge.bezformata.com': 'bezformata.com',
 'kadiy.bezformata.com': 'bezformata.com',
 'aykino.bezformata.com': 'bezformata.com',
 'soligalich.bezformata.com': 'bezformata.com',
 'partizanskoe.bezformata.com': 'bezformata.com',
 'susanino.bezformata.com': 'bezformata.com',
 'sudak.bezformata.com': 'bezformata.com',
 'amga.bezformata.com': 'bezformata.com',
 'lenino.bezformata.com': 'bezformata.com',
 'totma.bezformata.com': 'bezformata.com',
 'ermolaevo.bezformata.com': 'bezformata.com',
 'morki.bezformata.com': 'bezformata.com',
 'abaza.bezformata.com': 'bezformata.com',
 'dovolnoe.bezformata.com': 'bezformata.com',
 'kalachnadonu.bezformata.com': 'bezformata.com',
 'glinka.bezformata.com': 'bezformata.com',
 'laishevo.bezformata.com': 'bezformata.com',
 'bolgar.bezformata.com': 'bezformata.com',
 'krasnayagorka.bezformata.com': 'bezformata.com',
 'kirgizmiyaki.bezformata.com': 'bezformata.com',
 'ladushkin.bezformata.com': 'bezformata.com',
 'tetyushi.bezformata.com': 'bezformata.com',
 'shatck.bezformata.com': 'bezformata.com',
 'stanovoe.bezformata.com': 'bezformata.com',
 'topki.bezformata.com': 'bezformata.com',
 'nizhnyayaomka.bezformata.com': 'bezformata.com',
 'gorkovskoe.bezformata.com': 'bezformata.com',
 'zavyalovoudm.bezformata.com': 'bezformata.com',
 'kaltasi.bezformata.com': 'bezformata.com',
    'barnaul.jobfilter.ru': 'jobfilter.ru',
 'istra.jobfilter.ru': 'jobfilter.ru',
 'troick.jobfilter.ru': 'jobfilter.ru',
 'novokuzneck.jobfilter.ru': 'jobfilter.ru',
 'niznij-novgorod.jobfilter.ru': 'jobfilter.ru',
 'irkutskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'pokrov.jobfilter.ru': 'jobfilter.ru',
 'magadanskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'moskovskii.jobfilter.ru': 'jobfilter.ru',
 'kstovo.jobfilter.ru': 'jobfilter.ru',
 'almetevsk.jobfilter.ru': 'jobfilter.ru',
 'podolsk.jobfilter.ru': 'jobfilter.ru',
 'jerdevka.jobfilter.ru': 'jobfilter.ru',
 'dubna-mo.jobfilter.ru': 'jobfilter.ru',
 'mcensk.jobfilter.ru': 'jobfilter.ru',
 'pervouralsk.jobfilter.ru': 'jobfilter.ru',
 'permskii-krai.jobfilter.ru': 'jobfilter.ru',
 'usinsk.jobfilter.ru': 'jobfilter.ru',
 'kingisepp.jobfilter.ru': 'jobfilter.ru',
 'adler.jobfilter.ru': 'jobfilter.ru',
 'novoaltaisk.jobfilter.ru': 'jobfilter.ru',
 'kamyshin.jobfilter.ru': 'jobfilter.ru',
 'segeja.jobfilter.ru': 'jobfilter.ru',
 'kumertau.jobfilter.ru': 'jobfilter.ru',
 'lazarevskoe.jobfilter.ru': 'jobfilter.ru',
 'balabanovo.jobfilter.ru': 'jobfilter.ru',
 'yahroma.jobfilter.ru': 'jobfilter.ru',
 'shushary.jobfilter.ru': 'jobfilter.ru',
 'shadrinsk.jobfilter.ru': 'jobfilter.ru',
 'balakovo.jobfilter.ru': 'jobfilter.ru',
 'vyborg.jobfilter.ru': 'jobfilter.ru',
 'novosibirskaya-oblast.jobfilter.ru': 'jobfilter.ru',
 'otradnyi.jobfilter.ru': 'jobfilter.ru',
 'sovetskaya-gavan.jobfilter.ru': 'jobfilter.ru',
 'kuvandyk.jobfilter.ru': 'jobfilter.ru',
 'gvardeiskoe.jobfilter.ru': 'jobfilter.ru',
 'konakovo.jobfilter.ru': 'jobfilter.ru',
    'ekb.infoskidka.ru': 'infoskidka.ru',
 'ivanovo.infoskidka.ru': 'infoskidka.ru',
 'chel.infoskidka.ru': 'infoskidka.ru',
 'nabchelny.infoskidka.ru': 'infoskidka.ru',
 'sochi.infoskidka.ru': 'infoskidka.ru',
 'habarovsk.infoskidka.ru': 'infoskidka.ru',
 'kazan.infoskidka.ru': 'infoskidka.ru',
 'saratov.infoskidka.ru': 'infoskidka.ru',
 'nsk.infoskidka.ru': 'infoskidka.ru',
 'orel.infoskidka.ru': 'infoskidka.ru',
 'vladivostok.infoskidka.ru': 'infoskidka.ru',
 'belgorod.infoskidka.ru': 'infoskidka.ru',
 'volgograd.infoskidka.ru': 'infoskidka.ru',
 'evpatorya.infoskidka.ru': 'infoskidka.ru',
 'tyumen.infoskidka.ru': 'infoskidka.ru',
 'izhevsk.infoskidka.ru': 'infoskidka.ru',
 'penza.infoskidka.ru': 'infoskidka.ru',
 'rostov-na-donu.infoskidka.ru': 'infoskidka.ru',
 'abakan.infoskidka.ru': 'infoskidka.ru',
 'krasnoyarsk.infoskidka.ru': 'infoskidka.ru',
 'ulan-ude.infoskidka.ru': 'infoskidka.ru',
 'simferopol.infoskidka.ru': 'infoskidka.ru',
 'spb.infoskidka.ru': 'infoskidka.ru',
 'maykop.infoskidka.ru': 'infoskidka.ru',
 'samara.infoskidka.ru': 'infoskidka.ru',
 'barnaul.infoskidka.ru': 'infoskidka.ru',
 'syktyvkar.infoskidka.ru': 'infoskidka.ru',
 'kirov.infoskidka.ru': 'infoskidka.ru',
 'stavropol.infoskidka.ru': 'infoskidka.ru',
 'sevastopol.infoskidka.ru': 'infoskidka.ru',
 'voronezh.infoskidka.ru': 'infoskidka.ru',
 'uljanovsk.infoskidka.ru': 'infoskidka.ru',
 'nn.infoskidka.ru': 'infoskidka.ru',
 'saransk.infoskidka.ru': 'infoskidka.ru',
 'smolensk.infoskidka.ru': 'infoskidka.ru',
 'perm.infoskidka.ru': 'infoskidka.ru',
 'tomsk.infoskidka.ru': 'infoskidka.ru',
 'infoskidka.ru': 'infoskidka.ru',
 'cherepovez.infoskidka.ru': 'infoskidka.ru',
    'orenburg.spravker.ru': 'spravker.ru',
 'aprelevka.spravker.ru': 'spravker.ru',
 'kaierkan.spravker.ru': 'spravker.ru',
 'nolinsk.spravker.ru': 'spravker.ru',
 'suzdal.spravker.ru': 'spravker.ru',
 'ust-labinsk.spravker.ru': 'spravker.ru',
 'bor.spravker.ru': 'spravker.ru',
 'izhevsk.spravker.ru': 'spravker.ru',
 'podolsk.spravker.ru': 'spravker.ru',
 'novokuznetsk.spravker.ru': 'spravker.ru',
 'mtsensk.spravker.ru': 'spravker.ru',
 'bataisk.spravker.ru': 'spravker.ru',
 'spravker.ru': 'spravker.ru',
 'solikamsk.spravker.ru': 'spravker.ru',
 'kurganinsk.spravker.ru': 'spravker.ru',
 'kliuch.spravker.ru': 'spravker.ru',
 'uchaly.spravker.ru': 'spravker.ru',
 'bronnitsy.spravker.ru': 'spravker.ru',
 'semikarakorsk.spravker.ru': 'spravker.ru',
 'dobrianka.spravker.ru': 'spravker.ru',
 'lesosibirsk.spravker.ru': 'spravker.ru',
 'saianogorsk.spravker.ru': 'spravker.ru',
 'spassk-dalnii.spravker.ru': 'spravker.ru',
 'sasovo.spravker.ru': 'spravker.ru',
 'elektrostal.spravker.ru': 'spravker.ru',
 'revda.spravker.ru': 'spravker.ru',
 'kuvandyk.spravker.ru': 'spravker.ru',
 'kamensk-shakhtinskii.spravker.ru': 'spravker.ru',
 'kizliar.spravker.ru': 'spravker.ru',
 'ustiuzhna.spravker.ru': 'spravker.ru',
 'zeia.spravker.ru': 'spravker.ru',
 'kubinka.spravker.ru': 'spravker.ru',
 'zheleznovodsk.spravker.ru': 'spravker.ru',
 'kholmsk.spravker.ru': 'spravker.ru',
 'lida.spravker.ru': 'spravker.ru',
 'gavrilov-posad.spravker.ru': 'spravker.ru',
 'zarinsk.spravker.ru': 'spravker.ru',
 'barabinsk.spravker.ru': 'spravker.ru',
 'buinsk.spravker.ru': 'spravker.ru',
 'kogalym.spravker.ru': 'spravker.ru',
    'chernyahovsk.jsprav.ru': 'jsprav.ru',
 'izhevsk.jsprav.ru': 'jsprav.ru',
 'schyokino.jsprav.ru': 'jsprav.ru',
 'budyonnovsk.jsprav.ru': 'jsprav.ru',
 'zelenodolsk.jsprav.ru': 'jsprav.ru',
 'bronnitsyi.jsprav.ru': 'jsprav.ru',
 'novocheboksarsk.jsprav.ru': 'jsprav.ru',
 'zelenogorsk.jsprav.ru': 'jsprav.ru',
 'otradnyij.jsprav.ru': 'jsprav.ru',
 'nizhnekamsk.jsprav.ru': 'jsprav.ru',
 'ust-labinsk.jsprav.ru': 'jsprav.ru',
 'zhukov.jsprav.ru': 'jsprav.ru',
 'liski.jsprav.ru': 'jsprav.ru',
 'rossosh.jsprav.ru': 'jsprav.ru',
 'astrahan.jsprav.ru': 'jsprav.ru',
 'emva.jsprav.ru': 'jsprav.ru',
 'bolshoj-kamen.jsprav.ru': 'jsprav.ru',
 'barnaul.jsprav.ru': 'jsprav.ru',
 'elista.jsprav.ru': 'jsprav.ru',
 'vologda.jsprav.ru': 'jsprav.ru',
 'krasnokamsk.jsprav.ru': 'jsprav.ru',
 'sankt-peterburg1.jsprav.ru': 'jsprav.ru',
 'tosno.jsprav.ru': 'jsprav.ru',
 'aznakaevo.jsprav.ru': 'jsprav.ru',
 'tula.jsprav.ru': 'jsprav.ru',
 'sayansk.jsprav.ru': 'jsprav.ru',
 'surazh.jsprav.ru': 'jsprav.ru',
 'gorno-altajsk.jsprav.ru': 'jsprav.ru',
 'blagoveschensk1.jsprav.ru': 'jsprav.ru',
 'zheleznodorozhnyij.jsprav.ru': 'jsprav.ru',
 'furmanov.jsprav.ru': 'jsprav.ru',
 'nikolaevsk.jsprav.ru': 'jsprav.ru',
 'polyarnyie-zori.jsprav.ru': 'jsprav.ru',
 'kirovsk1.jsprav.ru': 'jsprav.ru',
 'kotelniki.jsprav.ru': 'jsprav.ru',
 'tutaev.jsprav.ru': 'jsprav.ru',
 'slavgorod.jsprav.ru': 'jsprav.ru',
 'ryibinsk.jsprav.ru': 'jsprav.ru',
 'artyom.jsprav.ru': 'jsprav.ru',
 'volhov.jsprav.ru': 'jsprav.ru',
 'harovsk.jsprav.ru': 'jsprav.ru',
    'fandom.com': 'fandom.com',
 'bakemonogatari.fandom.com': 'fandom.com',
 'terraria-game.fandom.com': 'fandom.com',
 'losyash-library.fandom.com': 'fandom.com',
 'mafia.fandom.com': 'fandom.com',
 'nioh.fandom.com': 'fandom.com',
 'mount-and-blade.fandom.com': 'fandom.com',
 'waha.fandom.com': 'fandom.com',
 'boombeach.fandom.com': 'fandom.com',
 'grimdawn.fandom.com': 'fandom.com',
 'transformers.fandom.com': 'fandom.com',
 'the-escapists-rus.fandom.com': 'fandom.com',
 'towerofgod.fandom.com': 'fandom.com',
 'blhx.fandom.com': 'fandom.com',
 'arrowverse.fandom.com': 'fandom.com',
 'cnc.fandom.com': 'fandom.com',
 'iceage.fandom.com': 'fandom.com',
 'rammstein.fandom.com': 'fandom.com',
 'undertale-au.fandom.com': 'fandom.com',
 'crossout.fandom.com': 'fandom.com',
 'lol.fandom.com': 'fandom.com',
 'strangerthings.fandom.com': 'fandom.com',
 'shield-hero.fandom.com': 'fandom.com',
 'madagascar.fandom.com': 'fandom.com',
 'conanexiles.fandom.com': 'fandom.com',
 'theouterworlds.fandom.com': 'fandom.com',
 'honkaiimpact3.fandom.com': 'fandom.com',
 'infinitode-2.fandom.com': 'fandom.com',
 'everlasting-summer.fandom.com': 'fandom.com',
 'grounded.fandom.com': 'fandom.com',
 'papersplease.fandom.com': 'fandom.com',
 'ace-attorney.fandom.com': 'fandom.com',
 'feargame.fandom.com': 'fandom.com',
 'sakura-school-simulator.fandom.com': 'fandom.com',
 'nightinthewoods.fandom.com': 'fandom.com',
 'wingsoffire.fandom.com': 'fandom.com',
 'yaoi.fandom.com': 'fandom.com',
 'the-millionaire-detective-balance-unlimited.fandom.com': 'fandom.com',
 'theloudhouse.fandom.com': 'fandom.com',
 'sosedopedia.fandom.com': 'fandom.com',
 'guardiantales.fandom.com': 'fandom.com',
    'egorievsk.hh.ru': 'hh.ru',
 'korolev.hh.ru': 'hh.ru',
 'volkhov.hh.ru': 'hh.ru',
 'vsevolozhsk.hh.ru': 'hh.ru',
 'pechora.hh.ru': 'hh.ru',
 'zheleznogorsk-kursk.hh.ru': 'hh.ru',
 'tobolsk.hh.ru': 'hh.ru',
 'lyubertsy.hh.ru': 'hh.ru',
 'balakovo.hh.ru': 'hh.ru',
 'staraya.hh.ru': 'hh.ru',
 'kirishi.hh.ru': 'hh.ru',
 'chekhov.hh.ru': 'hh.ru',
 'syria.hh.ru': 'hh.ru',
 'kirzhach.hh.ru': 'hh.ru',
 'morozovsk.hh.ru': 'hh.ru',
 'oktyabrskiy50.hh.ru': 'hh.ru',
 'dyurtyuli.hh.ru': 'hh.ru',
 'bratsk.hh.ru': 'hh.ru',
 'strizhi.hh.ru': 'hh.ru',
 'strezhevoy.hh.ru': 'hh.ru',
 'volsk.hh.ru': 'hh.ru',
 'solnechniy27.hh.ru': 'hh.ru',
 'volgorechensk.hh.ru': 'hh.ru',
 'solovyevsk.hh.ru': 'hh.ru',
 'buturlinovka.hh.ru': 'hh.ru',
 'lermontov.hh.ru': 'hh.ru',
 'lebedyan.hh.ru': 'hh.ru',
 'krivodanovka.hh.ru': 'hh.ru',
 'tarusa.hh.ru': 'hh.ru',
 'kashin.hh.ru': 'hh.ru',
 'kirovgrad.hh.ru': 'hh.ru',
 'gubkin.hh.ru': 'hh.ru',
 'rodniki50.hh.ru': 'hh.ru',
 'pudozh.hh.ru': 'hh.ru',
 'bryuhovetskaya.hh.ru': 'hh.ru',
 'lyskovo.hh.ru': 'hh.ru',
 'pokrovsk.hh.ru': 'hh.ru',
 'zelenogorsk.hh.ru': 'hh.ru',
 'saraktash.hh.ru': 'hh.ru',
 'novoe-devyatkino.hh.ru': 'hh.ru',
 'putilkovo.hh.ru': 'hh.ru',
 'sasovo.hh.ru': 'hh.ru',
    'fis.ru': 'fis.ru',
 'krasnodar.fis.ru': 'fis.ru',
 'novosibirsk.fis.ru': 'fis.ru',
 'nalchik.fis.ru': 'fis.ru',
 'gruziya.fis.ru': 'fis.ru',
 'beryozovskij.fis.ru': 'fis.ru',
 'goryachij-klyuch.fis.ru': 'fis.ru',
 'armeniya.fis.ru': 'fis.ru',
 'tyumen.fis.ru': 'fis.ru',
 'orenburg.fis.ru': 'fis.ru',
 'sochi.fis.ru': 'fis.ru',
 'chernogorsk.fis.ru': 'fis.ru',
 'krasnoyarsk.fis.ru': 'fis.ru',
 'sankt-peterburg.fis.ru': 'fis.ru',
 'izhevsk.fis.ru': 'fis.ru',
 'germaniya.fis.ru': 'fis.ru',
 'makeevka.fis.ru': 'fis.ru',
 'novorossijsk.fis.ru': 'fis.ru',
 'ufa.fis.ru': 'fis.ru',
 'labinsk.fis.ru': 'fis.ru',
 'ekaterinburg.fis.ru': 'fis.ru',
 'rostest74.fis.ru': 'fis.ru',
 'lipeck.fis.ru': 'fis.ru',
 'naberezhnye-chelny.fis.ru': 'fis.ru',
 'kazan.fis.ru': 'fis.ru',
 'belgorod.fis.ru': 'fis.ru',
 'opt-contex.fis.ru': 'fis.ru',
 'dmitrov.fis.ru': 'fis.ru',
 'saratov.fis.ru': 'fis.ru',
 'podolsk.fis.ru': 'fis.ru',
 'cheboksary.fis.ru': 'fis.ru',
 'tadzhikistan.fis.ru': 'fis.ru',
 'nizhnevartovsk.fis.ru': 'fis.ru',
 'rostov-na-donu.fis.ru': 'fis.ru',
 'mytischi.fis.ru': 'fis.ru',
 'samara.fis.ru': 'fis.ru',
 'velikij-novgorod.fis.ru': 'fis.ru',
 'komsomolsk-na-amure.fis.ru': 'fis.ru',
 'izrail.fis.ru': 'fis.ru',
 'china.fis.ru': 'fis.ru',
 'krymsk.fis.ru': 'fis.ru',
 'voronezh.fis.ru': 'fis.ru',
 'ulyanovsk.fis.ru': 'fis.ru',
    'podarki-market.ru': 'podarki-market.ru',
 'yoshkar-ola.podarki-market.ru': 'podarki-market.ru',
 'nizhnevartovsk.podarki-market.ru': 'podarki-market.ru',
 'krasnoyarsk.podarki-market.ru': 'podarki-market.ru',
 'tambov.podarki-market.ru': 'podarki-market.ru',
 'omsk.podarki-market.ru': 'podarki-market.ru',
 'spb.podarki-market.ru': 'podarki-market.ru',
 'rostov-na-donu.podarki-market.ru': 'podarki-market.ru',
 'sochi.podarki-market.ru': 'podarki-market.ru',
 'nnovgorod.podarki-market.ru': 'podarki-market.ru',
 'krasnodar.podarki-market.ru': 'podarki-market.ru',
 'barnaul.podarki-market.ru': 'podarki-market.ru',
 'kirov.podarki-market.ru': 'podarki-market.ru',
 'nsk.podarki-market.ru': 'podarki-market.ru',
 'kurgan.podarki-market.ru': 'podarki-market.ru',
 'kaluga.podarki-market.ru': 'podarki-market.ru',
 'magnitogorsk.podarki-market.ru': 'podarki-market.ru',
 'bratsk.podarki-market.ru': 'podarki-market.ru',
 'lipetsk.podarki-market.ru': 'podarki-market.ru',
 'orel.podarki-market.ru': 'podarki-market.ru',
 'yaroslavl.podarki-market.ru': 'podarki-market.ru',
 'biysk.podarki-market.ru': 'podarki-market.ru',
 'perm.podarki-market.ru': 'podarki-market.ru',
 'nizhnekamsk.podarki-market.ru': 'podarki-market.ru',
 'tula.podarki-market.ru': 'podarki-market.ru',
 'kazan.podarki-market.ru': 'podarki-market.ru',
 'tolyatti.podarki-market.ru': 'podarki-market.ru',
 'ulyanovsk.podarki-market.ru': 'podarki-market.ru',
 'novokuznetsk.podarki-market.ru': 'podarki-market.ru',
 'habarovsk.podarki-market.ru': 'podarki-market.ru',
 'izhevsk.podarki-market.ru': 'podarki-market.ru',
 'saratov.podarki-market.ru': 'podarki-market.ru',
 'ryazan.podarki-market.ru': 'podarki-market.ru',
 'voronezh.podarki-market.ru': 'podarki-market.ru',
 'saransk.podarki-market.ru': 'podarki-market.ru',
 'ufa.podarki-market.ru': 'podarki-market.ru',
 'abakan.podarki-market.ru': 'podarki-market.ru',
 'chelyabinsk.podarki-market.ru': 'podarki-market.ru',
 'kaliningrad.podarki-market.ru': 'podarki-market.ru',
 'tumen.podarki-market.ru': 'podarki-market.ru',
 'novorossiysk.podarki-market.ru': 'podarki-market.ru',
 'samara.podarki-market.ru': 'podarki-market.ru',
 'ntagil.podarki-market.ru': 'podarki-market.ru',
 'cherepovets.podarki-market.ru': 'podarki-market.ru',
 'kostroma.podarki-market.ru': 'podarki-market.ru',
 'simferopol.podarki-market.ru': 'podarki-market.ru',
 'tver.podarki-market.ru': 'podarki-market.ru',
 'ivanovo.podarki-market.ru': 'podarki-market.ru',
 'penza.podarki-market.ru': 'podarki-market.ru',
 'vladivostok.podarki-market.ru': 'podarki-market.ru',
 'volgograd.podarki-market.ru': 'podarki-market.ru',
 'orenburg.podarki-market.ru': 'podarki-market.ru',
 'kemerovo.podarki-market.ru': 'podarki-market.ru',
 'nchelny.podarki-market.ru': 'podarki-market.ru',
 'kursk.podarki-market.ru': 'podarki-market.ru',
 'tomsk.podarki-market.ru': 'podarki-market.ru',
    'rostov-na-donu.ab-club.ru': 'ab-club.ru',
 'ab-club.ru': 'ab-club.ru',
 'orsk.ab-club.ru': 'ab-club.ru',
 'orenburg.ab-club.ru': 'ab-club.ru',
 'kursk.ab-club.ru': 'ab-club.ru',
 'samara.ab-club.ru': 'ab-club.ru',
 'tyumen.ab-club.ru': 'ab-club.ru',
 'rezh.ab-club.ru': 'ab-club.ru',
 'nizhnekamsk.ab-club.ru': 'ab-club.ru',
 'ufa.ab-club.ru': 'ab-club.ru',
 'vladimir.ab-club.ru': 'ab-club.ru',
 'ryazan.ab-club.ru': 'ab-club.ru',
 'salsk.ab-club.ru': 'ab-club.ru',
 'arxangelsk.ab-club.ru': 'ab-club.ru',
 'nizhnij-novgorod.ab-club.ru': 'ab-club.ru',
 'abakan.ab-club.ru': 'ab-club.ru',
 'novorossijsk.ab-club.ru': 'ab-club.ru',
 'krasnodar.ab-club.ru': 'ab-club.ru',
 'ehlektrostal.ab-club.ru': 'ab-club.ru',
 'blagoveshchensk.ab-club.ru': 'ab-club.ru',
 'penza.ab-club.ru': 'ab-club.ru',
 'tomsk.ab-club.ru': 'ab-club.ru',
 'surgut.ab-club.ru': 'ab-club.ru',
 'tolyatti.ab-club.ru': 'ab-club.ru',
 'sterlitamak.ab-club.ru': 'ab-club.ru',
 'barnaul.ab-club.ru': 'ab-club.ru',
 'voronezh.ab-club.ru': 'ab-club.ru',
 'pyatigorsk.ab-club.ru': 'ab-club.ru',
 'tver.ab-club.ru': 'ab-club.ru',
 'petrozavodsk.ab-club.ru': 'ab-club.ru',
 'tambov.ab-club.ru': 'ab-club.ru',
 'xabarovsk.ab-club.ru': 'ab-club.ru',
 'majkop.ab-club.ru': 'ab-club.ru',
 'anapa.ab-club.ru': 'ab-club.ru',
 'spb.ab-club.ru': 'ab-club.ru',
 'lipeck.ab-club.ru': 'ab-club.ru',
 'izhevsk.ab-club.ru': 'ab-club.ru',
 'ishimbaj.ab-club.ru': 'ab-club.ru',
 'perm.ab-club.ru': 'ab-club.ru',
 'kazan.ab-club.ru': 'ab-club.ru',
 'ekaterinburg.ab-club.ru': 'ab-club.ru',
 'magnitogorsk.ab-club.ru': 'ab-club.ru',
 'smolensk.ab-club.ru': 'ab-club.ru',
 'ulyanovsk.ab-club.ru': 'ab-club.ru',
 'miass.ab-club.ru': 'ab-club.ru',
 'minusinsk.ab-club.ru': 'ab-club.ru',
 'petropavlovsk-kamchatskij.ab-club.ru': 'ab-club.ru',
 'salavat.ab-club.ru': 'ab-club.ru',
 'rossosh.ab-club.ru': 'ab-club.ru',
 'krasnoyarsk.ab-club.ru': 'ab-club.ru',
 'kemerovo.ab-club.ru': 'ab-club.ru',
 'chelyabinsk.ab-club.ru': 'ab-club.ru',
 'pskov.ab-club.ru': 'ab-club.ru',
 'arzamas.ab-club.ru': 'ab-club.ru',
 'ehngels.ab-club.ru': 'ab-club.ru',
 'sevastopol.ab-club.ru': 'ab-club.ru',
 'volgograd.ab-club.ru': 'ab-club.ru',
 'novosibirsk.ab-club.ru': 'ab-club.ru',
    'gde.ru': 'gde.ru',
 'skopin.gde.ru': 'gde.ru',
 'dankov.gde.ru': 'gde.ru',
 'sverdlovsk.gde.ru': 'gde.ru',
 'suzdal.gde.ru': 'gde.ru',
 'klin.gde.ru': 'gde.ru',
 'barnaul.gde.ru': 'gde.ru',
 'bizhbulyak.gde.ru': 'gde.ru',
 'luhovicy.gde.ru': 'gde.ru',
 'tcelinnoe.gde.ru': 'gde.ru',
 'krasnoe_selo.gde.ru': 'gde.ru',
 'lodeynoe-pole.gde.ru': 'gde.ru',
 'staraya-kupavna.gde.ru': 'gde.ru',
 'sterlitamak.gde.ru': 'gde.ru',
 'goryachiy-klyuch.gde.ru': 'gde.ru',
 'mcensk.gde.ru': 'gde.ru',
 'salsk.gde.ru': 'gde.ru',
 'balashiha.gde.ru': 'gde.ru',
 'belorechensk.gde.ru': 'gde.ru',
 'birobidzhan.gde.ru': 'gde.ru',
 'slobodskoy.gde.ru': 'gde.ru',
 'hanty-mansiysk.gde.ru': 'gde.ru',
 'nyagan.gde.ru': 'gde.ru',
 'lihoslavl.gde.ru': 'gde.ru',
 'korolyov.gde.ru': 'gde.ru',
 'buraevo.gde.ru': 'gde.ru',
 'obninsk.gde.ru': 'gde.ru',
 'toropec.gde.ru': 'gde.ru',
 'strezhevoy.gde.ru': 'gde.ru',
 'pohvistnevo.gde.ru': 'gde.ru',
 'klyavlino.gde.ru': 'gde.ru',
 'sol-ilechk.gde.ru': 'gde.ru',
 'achinsk.gde.ru': 'gde.ru',
 'taseevo.gde.ru': 'gde.ru',
 'doneck.gde.ru': 'gde.ru',
 'ust-doneckiy.gde.ru': 'gde.ru',
 'topki.gde.ru': 'gde.ru',
 'nizhniy-lomov.gde.ru': 'gde.ru',
 'kalyazin.gde.ru': 'gde.ru',
 'krasnogvardeyskoe.gde.ru': 'gde.ru',
 'prm.gde.ru': 'gde.ru',
 'iva.gde.ru': 'gde.ru',
 'sergach.gde.ru': 'gde.ru',
 'tavda.gde.ru': 'gde.ru',
 'novohopersk.gde.ru': 'gde.ru',
 'chishmy.gde.ru': 'gde.ru',
 'tmn.gde.ru': 'gde.ru',
 'volgodonsk.gde.ru': 'gde.ru',
 'love.gde.ru': 'gde.ru',
 'aramil.gde.ru': 'gde.ru',
 'sladkovo.gde.ru': 'gde.ru',
 'sasovo.gde.ru': 'gde.ru',
 'chekmagush.gde.ru': 'gde.ru',
 'kirillov.gde.ru': 'gde.ru',
 'krasnyy-sulin.gde.ru': 'gde.ru',
 'gryazovec.gde.ru': 'gde.ru',
 'berezniki.gde.ru': 'gde.ru',
 'mozhga.gde.ru': 'gde.ru',
    'mirtesen.ru': 'mirtesen.ru',
 'otlichnoe-mnenie.mirtesen.ru': 'mirtesen.ru',
 'welemudr.mirtesen.ru': 'mirtesen.ru',
 'myhistory.mirtesen.ru': 'mirtesen.ru',
 'agv.mirtesen.ru': 'mirtesen.ru',
 'vmircvetov.mirtesen.ru': 'mirtesen.ru',
 'moi-dom-semiy.mirtesen.ru': 'mirtesen.ru',
 'shp.mirtesen.ru': 'mirtesen.ru',
 'kote.mirtesen.ru': 'mirtesen.ru',
 'knigi.mirtesen.ru': 'mirtesen.ru',
 's30376407575.mirtesen.ru': 'mirtesen.ru',
 'u-f.mirtesen.ru': 'mirtesen.ru',
 '4kolesa.mirtesen.ru': 'mirtesen.ru',
 'viralife.mirtesen.ru': 'mirtesen.ru',
 'newmirror.mirtesen.ru': 'mirtesen.ru',
 'derzhim-formu.mirtesen.ru': 'mirtesen.ru',
 'vcenovoe.mirtesen.ru': 'mirtesen.ru',
 'mt-smi.mirtesen.ru': 'mirtesen.ru',
 's30333126284.mirtesen.ru': 'mirtesen.ru',
 'all-history.mirtesen.ru': 'mirtesen.ru',
 'sputnik.mirtesen.ru': 'mirtesen.ru',
 'internetshkatulka.mirtesen.ru': 'mirtesen.ru',
 '74news.mirtesen.ru': 'mirtesen.ru',
 's30351029799.mirtesen.ru': 'mirtesen.ru',
 'usiki-xvostiki.mirtesen.ru': 'mirtesen.ru',
 'olanola.com': 'mirtesen.ru',
 'kareliyanews.mirtesen.ru': 'mirtesen.ru',
 'experimental.mirtesen.ru': 'mirtesen.ru',
 'rebenok.mirtesen.ru': 'mirtesen.ru',
 's30440007639.mirtesen.ru': 'mirtesen.ru',
 'nauka-mixt.mirtesen.ru': 'mirtesen.ru',
 'vzglad.mirtesen.ru': 'mirtesen.ru',
 'dailysmi.mirtesen.ru': 'mirtesen.ru',
 'topwar.mirtesen.ru': 'mirtesen.ru',
 's30029982448.mirtesen.ru': 'mirtesen.ru',
 'goroskopp.mirtesen.ru': 'mirtesen.ru',
 'supershowbiz.mirtesen.ru': 'mirtesen.ru',
 'vmipegivotnim.mirtesen.ru': 'mirtesen.ru',
 's30434135483.mirtesen.ru': 'mirtesen.ru',
 'zr.mirtesen.ru': 'mirtesen.ru',
 's30171049813.mirtesen.ru': 'mirtesen.ru',
 's30623888749.mirtesen.ru': 'mirtesen.ru',
 'perepost.mirtesen.ru': 'mirtesen.ru',
 'kopbl.mirtesen.ru': 'mirtesen.ru',
 's30607421350.mirtesen.ru': 'mirtesen.ru',
 'vserisyut.mirtesen.ru': 'mirtesen.ru',
 'nashizvyozdy.mirtesen.ru': 'mirtesen.ru',
 'autoversty.mirtesen.ru': 'mirtesen.ru',
 'ourkids.mirtesen.ru': 'mirtesen.ru',
 'infonewz.mirtesen.ru': 'mirtesen.ru',
 's30760677253.mirtesen.ru': 'mirtesen.ru',
 'adindex.mirtesen.ru': 'mirtesen.ru',
 'gp.mirtesen.ru': 'mirtesen.ru',
 'greatpicture.mirtesen.ru': 'mirtesen.ru',
 'sila-rg.mirtesen.ru': 'mirtesen.ru',
 's30542327491.mirtesen.ru': 'mirtesen.ru',
 'musikwar.mirtesen.ru': 'mirtesen.ru',
 'actualno.mirtesen.ru': 'mirtesen.ru',
    'tyumen.mnogonado.net': 'mnogonado.net',
 'izhevsk.mnogonado.net': 'mnogonado.net',
 'perm.mnogonado.net': 'mnogonado.net',
 'kaliningrad.mnogonado.net': 'mnogonado.net',
 'moscow.mnogonado.net': 'mnogonado.net',
 'ufa.mnogonado.net': 'mnogonado.net',
 'kyzyl.mnogonado.net': 'mnogonado.net',
 'samara.mnogonado.net': 'mnogonado.net',
 'novosibirsk.mnogonado.net': 'mnogonado.net',
 'novocherkassk.mnogonado.net': 'mnogonado.net',
 'yeisk.mnogonado.net': 'mnogonado.net',
 'rostov-on-don.mnogonado.net': 'mnogonado.net',
 'oktyabrsky.mnogonado.net': 'mnogonado.net',
 'kirov.mnogonado.net': 'mnogonado.net',
 'shadrinsk.mnogonado.net': 'mnogonado.net',
 'nizhny-tagil.mnogonado.net': 'mnogonado.net',
 'zlatoust.mnogonado.net': 'mnogonado.net',
 'verkhnyayapyshma.mnogonado.net': 'mnogonado.net',
 'smolensk.mnogonado.net': 'mnogonado.net',
 'naberezhnye-chelny.mnogonado.net': 'mnogonado.net',
 'stavropol.mnogonado.net': 'mnogonado.net',
 'petersburg.mnogonado.net': 'mnogonado.net',
 'omsk.mnogonado.net': 'mnogonado.net',
 'chelyabinsk.mnogonado.net': 'mnogonado.net',
 'sochi.mnogonado.net': 'mnogonado.net',
 'adler.mnogonado.net': 'mnogonado.net',
 'ekaterinburg.mnogonado.net': 'mnogonado.net',
 'saratov.mnogonado.net': 'mnogonado.net',
 'penza.mnogonado.net': 'mnogonado.net',
 'volgodonsk.mnogonado.net': 'mnogonado.net',
 'kamyshin.mnogonado.net': 'mnogonado.net',
 'volzhsky.mnogonado.net': 'mnogonado.net',
 'balakovo.mnogonado.net': 'mnogonado.net',
 'nizhny-novgorod.mnogonado.net': 'mnogonado.net',
 'magnitogorsk.mnogonado.net': 'mnogonado.net',
 'kurgan.mnogonado.net': 'mnogonado.net',
 'maikop.mnogonado.net': 'mnogonado.net',
 'lipetsk.mnogonado.net': 'mnogonado.net',
 'belgorod.mnogonado.net': 'mnogonado.net',
 'voronezh.mnogonado.net': 'mnogonado.net',
 'orel.mnogonado.net': 'mnogonado.net',
 'podolsk.mnogonado.net': 'mnogonado.net',
 'kazan.mnogonado.net': 'mnogonado.net',
 'ulianovsk.mnogonado.net': 'mnogonado.net',
 'zhigulevsk.mnogonado.net': 'mnogonado.net',
 'tuapse.mnogonado.net': 'mnogonado.net',
 'kolomna.mnogonado.net': 'mnogonado.net',
 'krymsk.mnogonado.net': 'mnogonado.net',
 'gukovo.mnogonado.net': 'mnogonado.net',
 'khabarovsk.mnogonado.net': 'mnogonado.net',
 'shakhty.mnogonado.net': 'mnogonado.net',
 'vladikavkaz.mnogonado.net': 'mnogonado.net',
 'saransk.mnogonado.net': 'mnogonado.net',
 'nizhnevartovsk.mnogonado.net': 'mnogonado.net',
 'surgut.mnogonado.net': 'mnogonado.net',
 'vladimir.mnogonado.net': 'mnogonado.net',
 'murmansk.mnogonado.net': 'mnogonado.net',
 'rybinsk.mnogonado.net': 'mnogonado.net',
 'nalchik.mnogonado.net': 'mnogonado.net',
 'nevinnomyssk.mnogonado.net': 'mnogonado.net',
 'kislovodsk.mnogonado.net': 'mnogonado.net',
 'ryazan.mnogonado.net': 'mnogonado.net',
    'bezformata.com': 'bezformata.com',
 'chehov.bezformata.com': 'bezformata.com',
 'orehovozuevo.bezformata.com': 'bezformata.com',
 'klin.bezformata.com': 'bezformata.com',
 'belorechensk.bezformata.com': 'bezformata.com',
 'salehard.bezformata.com': 'bezformata.com',
 'poltavskaya.bezformata.com': 'bezformata.com',
 'volgodonsk.bezformata.com': 'bezformata.com',
 'krilovskaya.bezformata.com': 'bezformata.com',
 'otradnaya.bezformata.com': 'bezformata.com',
 'elets.bezformata.com': 'bezformata.com',
 'rilsk.bezformata.com': 'bezformata.com',
 'volovo.bezformata.com': 'bezformata.com',
 'efremov.bezformata.com': 'bezformata.com',
 'yarenska.bezformata.com': 'bezformata.com',
 'velsk1.bezformata.com': 'bezformata.com',
 'dolgoderevenskoe.bezformata.com': 'bezformata.com',
 'vpishma.bezformata.com': 'bezformata.com',
 'zvenigorod.bezformata.com': 'bezformata.com',
 'dubna.bezformata.com': 'bezformata.com',
 'noyabrsk.bezformata.com': 'bezformata.com',
 'lejnevo.bezformata.com': 'bezformata.com',
 'kimri.bezformata.com': 'bezformata.com',
 'malayapurga.bezformata.com': 'bezformata.com',
 'chaykovskiy.bezformata.com': 'bezformata.com',
 'uyrga.bezformata.com': 'bezformata.com',
 'uspenskoe.bezformata.com': 'bezformata.com',
 'chernogolovka.bezformata.com': 'bezformata.com',
 'krasnoarmeysk.bezformata.com': 'bezformata.com',
 'nikolaevsknaamure.bezformata.com': 'bezformata.com',
 'tashtagol.bezformata.com': 'bezformata.com',
 'beryozovskiykmr.bezformata.com': 'bezformata.com',
 'remontnoe.bezformata.com': 'bezformata.com',
 'rtishevo.bezformata.com': 'bezformata.com',
 'ulanude.bezformata.com': 'bezformata.com',
 'minusinsk.bezformata.com': 'bezformata.com',
 'megion.bezformata.com': 'bezformata.com',
 'tonshaevo.bezformata.com': 'bezformata.com',
 'kungur.bezformata.com': 'bezformata.com',
 'kulebaki.bezformata.com': 'bezformata.com',
 'krasnoznamensk.bezformata.com': 'bezformata.com',
 'idrinskoe.bezformata.com': 'bezformata.com',
 'belev.bezformata.com': 'bezformata.com',
 'kushva.bezformata.com': 'bezformata.com',
 'fryazino.bezformata.com': 'bezformata.com',
 'vidnoe.bezformata.com': 'bezformata.com',
 'kirovochepesk.bezformata.com': 'bezformata.com',
 'zernograd.bezformata.com': 'bezformata.com',
 'chekmagush.bezformata.com': 'bezformata.com',
 'aromashevo.bezformata.com': 'bezformata.com',
 'krasniysulin.bezformata.com': 'bezformata.com',
 'bolsheustikinskoe.bezformata.com': 'bezformata.com',
 'andreapol.bezformata.com': 'bezformata.com',
 'bejeck.bezformata.com': 'bezformata.com',
 'bagan.bezformata.com': 'bezformata.com',
 'lomonosov.bezformata.com': 'bezformata.com',
 'toropec.bezformata.com': 'bezformata.com',
 'rameshki.bezformata.com': 'bezformata.com',
 'maksatiha.bezformata.com': 'bezformata.com',
 'navashino.bezformata.com': 'bezformata.com',
 'polyarniezori.bezformata.com': 'bezformata.com',
 'snezhnogorsk.bezformata.com': 'bezformata.com',
 'koygorodok.bezformata.com': 'bezformata.com',
 'lodeynoepole.bezformata.com': 'bezformata.com',
 'tayga.bezformata.com': 'bezformata.com',
 'lihoslavl.bezformata.com': 'bezformata.com',
 'mozdok.bezformata.com': 'bezformata.com',
 'skopin.bezformata.com': 'bezformata.com',
 'grachyovka.bezformata.com': 'bezformata.com',
 'solntcevo.bezformata.com': 'bezformata.com',
 'svetliyoren.bezformata.com': 'bezformata.com',
 'krapivinskiy.bezformata.com': 'bezformata.com',
 'kochenevo.bezformata.com': 'bezformata.com',
 'bayhaak.bezformata.com': 'bezformata.com',
 'bashmakovo.bezformata.com': 'bezformata.com',
 'furmanov.bezformata.com': 'bezformata.com',
 'kirs.bezformata.com': 'bezformata.com',
 'jiryatino.bezformata.com': 'bezformata.com',
 'belayakalitva.bezformata.com': 'bezformata.com',
 'tamala.bezformata.com': 'bezformata.com',
 'pokrovskoerostov.bezformata.com': 'bezformata.com',
 'maiskiy.bezformata.com': 'bezformata.com',
    'news-ks.info': 'howto-news.info',
 'news-z.info': 'howto-news.info',
 'news-evi.net': 'howto-news.info',
 'howto-news.info': 'howto-news.info',
 'news.news-evi.net': 'howto-news.info',
 '234.howto-news.info': 'howto-news.info',
 '453.howto-news.info': 'howto-news.info',
 '488.howto-news.info': 'howto-news.info',
 '457.howto-news.info': 'howto-news.info',
 '389.howto-news.info': 'howto-news.info',
 '167.howto-news.info': 'howto-news.info',
 '173.howto-news.info': 'howto-news.info',
 '171.howto-news.info': 'howto-news.info',
 '209.howto-news.info': 'howto-news.info',
 '233.howto-news.info': 'howto-news.info',
 '500.howto-news.info': 'howto-news.info',
 '101.howto-news.info': 'howto-news.info',
 '206.howto-news.info': 'howto-news.info',
 '505.howto-news.info': 'howto-news.info',
 '449.howto-news.info': 'howto-news.info',
 '491.howto-news.info': 'howto-news.info',
 '480.howto-news.info': 'howto-news.info',
 '10.howto-news.info': 'howto-news.info',
 '219.howto-news.info': 'howto-news.info',
 '212.howto-news.info': 'howto-news.info',
 '495.howto-news.info': 'howto-news.info',
 '223.howto-news.info': 'howto-news.info',
 '190.howto-news.info': 'howto-news.info',
 '199.howto-news.info': 'howto-news.info',
 '119.howto-news.info': 'howto-news.info',
 '186.howto-news.info': 'howto-news.info',
 '236.howto-news.info': 'howto-news.info',
 '461.howto-news.info': 'howto-news.info',
 '11.howto-news.info': 'howto-news.info',
 '215.howto-news.info': 'howto-news.info',
 '235.howto-news.info': 'howto-news.info',
 '189.howto-news.info': 'howto-news.info',
 '118.howto-news.info': 'howto-news.info',
 '484.howto-news.info': 'howto-news.info',
 '214.howto-news.info': 'howto-news.info',
 '217.howto-news.info': 'howto-news.info',
 '456.howto-news.info': 'howto-news.info',
 '490.howto-news.info': 'howto-news.info',
 'xnewsq.info': 'howto-news.info',
 '225.howto-news.info': 'howto-news.info',
 '230.howto-news.info': 'howto-news.info',
 '178.howto-news.info': 'howto-news.info',
 '510.howto-news.info': 'howto-news.info',
 '229.howto-news.info': 'howto-news.info',
 '475.howto-news.info': 'howto-news.info',
 '130.howto-news.info': 'howto-news.info',
 '469.howto-news.info': 'howto-news.info',
 '489.howto-news.info': 'howto-news.info',
 '467.howto-news.info': 'howto-news.info',
 '273.howto-news.info': 'howto-news.info',
 '165.howto-news.info': 'howto-news.info',
 '423.howto-news.info': 'howto-news.info',
 '421.howto-news.info': 'howto-news.info',
 '123.howto-news.info': 'howto-news.info',
 '176.howto-news.info': 'howto-news.info',
 '106.howto-news.info': 'howto-news.info',
 '168.howto-news.info': 'howto-news.info',
 '181.howto-news.info': 'howto-news.info',
 '224.howto-news.info': 'howto-news.info',
 '177.howto-news.info': 'howto-news.info',
 '210.howto-news.info': 'howto-news.info',
 '477.howto-news.info': 'howto-news.info',
 '450.howto-news.info': 'howto-news.info',
 '451.howto-news.info': 'howto-news.info',
 '493.howto-news.info': 'howto-news.info',
 '462.howto-news.info': 'howto-news.info',
 '494.howto-news.info': 'howto-news.info',
 '232.howto-news.info': 'howto-news.info',
 '261.howto-news.info': 'howto-news.info',
 '198.howto-news.info': 'howto-news.info',
 '216.howto-news.info': 'howto-news.info',
 '107.howto-news.info': 'howto-news.info',
 '162.howto-news.info': 'howto-news.info',
 '294.howto-news.info': 'howto-news.info',
 '464.howto-news.info': 'howto-news.info',
 '452.howto-news.info': 'howto-news.info',
 '445.howto-news.info': 'howto-news.info',
 '363.howto-news.info': 'howto-news.info',
 '465.howto-news.info': 'howto-news.info',
 '470.howto-news.info': 'howto-news.info',
 '208.howto-news.info': 'howto-news.info',
 '487.howto-news.info': 'howto-news.info',
 '486.howto-news.info': 'howto-news.info',
 '460.howto-news.info': 'howto-news.info',
 '448.howto-news.info': 'howto-news.info',
 '163.howto-news.info': 'howto-news.info',
    'krasnodar.gruzovik.ru': 'gruzovik.ru',
 'rostovskaya-oblast.gruzovik.ru': 'gruzovik.ru',
 'krasnodarskij-kraj.gruzovik.ru': 'gruzovik.ru',
 'murmansk.gruzovik.ru': 'gruzovik.ru',
 'rostov-na-donu.gruzovik.ru': 'gruzovik.ru',
 'voronezh.gruzovik.ru': 'gruzovik.ru',
 'spb.gruzovik.ru': 'gruzovik.ru',
 'moscow-oblast.gruzovik.ru': 'gruzovik.ru',
 'gruzovik.ru': 'gruzovik.ru',
 'briansk.gruzovik.ru': 'gruzovik.ru',
 'irkutsk.gruzovik.ru': 'gruzovik.ru',
 'belgorod.gruzovik.ru': 'gruzovik.ru',
 'lipetsk.gruzovik.ru': 'gruzovik.ru',
 'voronegskaya-oblast.gruzovik.ru': 'gruzovik.ru',
 'nn.gruzovik.ru': 'gruzovik.ru',
    'odessa.kitabi.ru': 'kitabi.ru',
 'reutov.kitabi.ru': 'kitabi.ru',
 'mytischi.kitabi.ru': 'kitabi.ru',
 'aleksin.kitabi.ru': 'kitabi.ru',
 'belgorod.kitabi.ru': 'kitabi.ru',
 'novouralsk.kitabi.ru': 'kitabi.ru',
 'gagarin.kitabi.ru': 'kitabi.ru',
 'velikiy-novgorod.kitabi.ru': 'kitabi.ru',
 'bataysk.kitabi.ru': 'kitabi.ru',
 'nefteugansk.kitabi.ru': 'kitabi.ru',
 'orel.kitabi.ru': 'kitabi.ru',
 'ust-ilimsk.kitabi.ru': 'kitabi.ru',
 'penza.kitabi.ru': 'kitabi.ru',
 'odincovo.kitabi.ru': 'kitabi.ru',
 'petrozavodsk.kitabi.ru': 'kitabi.ru',
    'naydidom.com': 'naydidom.com',
 'kazan.naydidom.com': 'naydidom.com',
 'orenburg.naydidom.com': 'naydidom.com',
 'anapa.naydidom.com': 'naydidom.com',
 'nahabino.naydidom.com': 'naydidom.com',
 'lihoslavl.naydidom.com': 'naydidom.com',
 'arhangelsk.naydidom.com': 'naydidom.com',
 'krasnogorsk.naydidom.com': 'naydidom.com',
 'yoshkar-ola.naydidom.com': 'naydidom.com',
 'tula.naydidom.com': 'naydidom.com',
 'zelenogradsk.naydidom.com': 'naydidom.com',
 'kaliningrad.naydidom.com': 'naydidom.com',
 'velikie-luki.naydidom.com': 'naydidom.com',
 'lipeck.naydidom.com': 'naydidom.com',
 'ussuriysk.naydidom.com': 'naydidom.com',
    'divanich74.mavmebel.ru': 'mavmebel.ru',
 'magnitka.mavmebel.ru': 'mavmebel.ru',
 'kimry-meb.ru': 'mavmebel.ru',
 'bagira.mavmebel.ru': 'mavmebel.ru',
 'tob.mavmebel.ru': 'mavmebel.ru',
 'iskmeb.mavmebel.ru': 'mavmebel.ru',
 'divan-ivanych.mavmebel.ru': 'mavmebel.ru',
 'laim96.ru': 'mavmebel.ru',
 'pelikanc.mavmebel.ru': 'mavmebel.ru',
 'topaz.mavmebel.ru': 'mavmebel.ru',
 'yardivanchik-pereslavl.ru': 'mavmebel.ru',
 'mebelanastasiya74.ru': 'mavmebel.ru',
 'mebelsofiya.ru': 'mavmebel.ru',
 'driada.mavmebel.ru': 'mavmebel.ru',
 'dmtob.ru': 'mavmebel.ru',
 'mebelvitsa-ru.ru': 'mavmebel.ru',
    'slovaronline.com': 'slovaronline.com',
 'rus-bse.slovaronline.com': 'slovaronline.com',
 'udarenie-i-pravopisanie.slovaronline.com': 'slovaronline.com',
 '556.slovaronline.com': 'slovaronline.com',
 '1564.slovaronline.com': 'slovaronline.com',
 'foreign.slovaronline.com': 'slovaronline.com',
 'sin.slovaronline.com': 'slovaronline.com',
 'rus-medical-terms.slovaronline.com': 'slovaronline.com',
 '1086.slovaronline.com': 'slovaronline.com',
 'rus-big-psyho.slovaronline.com': 'slovaronline.com',
 'rus-orthography-dict.slovaronline.com': 'slovaronline.com',
 'mat2.slovaronline.com': 'slovaronline.com',
 'rus-big-enc-dict.slovaronline.com': 'slovaronline.com',
 'rus-sov-istoria-enc.slovaronline.com': 'slovaronline.com',
 'ru-az-dict.slovaronline.com': 'slovaronline.com',
 'psychological.slovaronline.com': 'slovaronline.com',
    'mebel24you.ru': 'mebel24you.ru',
 'ulianovsk.mebel24you.ru': 'mebel24you.ru',
 'izhevsk.mebel24you.ru': 'mebel24you.ru',
 'toliatti.mebel24you.ru': 'mebel24you.ru',
 'spb.mebel24you.ru': 'mebel24you.ru',
 'barnaul.mebel24you.ru': 'mebel24you.ru',
 'penza.mebel24you.ru': 'mebel24you.ru',
 'orel.mebel24you.ru': 'mebel24you.ru',
 'yoshkar-ola.mebel24you.ru': 'mebel24you.ru',
 'kursk.mebel24you.ru': 'mebel24you.ru',
 'ekaterinburg.mebel24you.ru': 'mebel24you.ru',
 'kerch.mebel24you.ru': 'mebel24you.ru',
 'stavropol.mebel24you.ru': 'mebel24you.ru',
 'novosibirsk.mebel24you.ru': 'mebel24you.ru',
 'smolensk.mebel24you.ru': 'mebel24you.ru',
 'kazan.mebel24you.ru': 'mebel24you.ru',
 'krasnodar.mebel24you.ru': 'mebel24you.ru',
    'chelyabinskaya-oblast.unibo.ru': 'unibo.ru',
 'vladivostok.unibo.ru': 'unibo.ru',
 'volzhskiy.unibo.ru': 'unibo.ru',
 'staraya-russa.unibo.ru': 'unibo.ru',
 'zaraysk.unibo.ru': 'unibo.ru',
 'unibo.ru': 'unibo.ru',
 'belorussiya.unibo.ru': 'unibo.ru',
 'murmansk.unibo.ru': 'unibo.ru',
 'barnaul.unibo.ru': 'unibo.ru',
 'novorossiysk.unibo.ru': 'unibo.ru',
 'novokuybishevsk.unibo.ru': 'unibo.ru',
 'vladimirskaya-oblast.unibo.ru': 'unibo.ru',
 'evpatoria.unibo.ru': 'unibo.ru',
 'yakutsk.unibo.ru': 'unibo.ru',
 'tobolsk.unibo.ru': 'unibo.ru',
 'karpinsk.unibo.ru': 'unibo.ru',
 'novoshahtinsk.unibo.ru': 'unibo.ru',
    'kp.ru': 'kp.ru',
 'ural.kp.ru': 'kp.ru',
 'pda.kp.ru': 'kp.ru',
 'chel.kp.ru': 'kp.ru',
 'crimea.kp.ru': 'kp.ru',
 'pda.kuban.kp.ru': 'kp.ru',
 'ufa.kp.ru': 'kp.ru',
 'saratov.kp.ru': 'kp.ru',
 'vrn.kp.ru': 'kp.ru',
 'kaliningrad.kp.ru': 'kp.ru',
 'pda.msk.kp.ru': 'kp.ru',
 'stav.kp.ru': 'kp.ru',
 'pda.ural.kp.ru': 'kp.ru',
 'pda.rostov.kp.ru': 'kp.ru',
 'bel.kp.ru': 'kp.ru',
 'pda.nsk.kp.ru': 'kp.ru',
 'pda.perm.kp.ru': 'kp.ru',
    'perm.1000bankov.ru': '1000bankov.ru',
 'chelyabinsk.1000bankov.ru': '1000bankov.ru',
 'barnaul.1000bankov.ru': '1000bankov.ru',
 'rostov-na-donu.1000bankov.ru': '1000bankov.ru',
 'chelny.1000bankov.ru': '1000bankov.ru',
 'almetevsk.1000bankov.ru': '1000bankov.ru',
 'petrozavodsk.1000bankov.ru': '1000bankov.ru',
 'stavropol.1000bankov.ru': '1000bankov.ru',
 'korolev.1000bankov.ru': '1000bankov.ru',
 'nyagan.1000bankov.ru': '1000bankov.ru',
 'kasimov.1000bankov.ru': '1000bankov.ru',
 'reutov.1000bankov.ru': '1000bankov.ru',
 'buguruslan.1000bankov.ru': '1000bankov.ru',
 'korsakov.1000bankov.ru': '1000bankov.ru',
 'inta.1000bankov.ru': '1000bankov.ru',
 'pyt-yah.1000bankov.ru': '1000bankov.ru',
 'zelenodolsk.1000bankov.ru': '1000bankov.ru',
 'belovo.1000bankov.ru': '1000bankov.ru',
    'vladimir.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'gluboki-turizm.ru': 'gluboki-turizm.ru',
 'kazan.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'novosibirsk.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'vladikavkaz.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'cheboksary.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'magnitogorsk.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'velikie-luki.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'kirov-kirovskaya-oblast.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'staryj-oskol.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'ekaterinburg.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'vologda.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'lipetsk.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'sankt-peterburg.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'kurgan.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'rostov-na-donu.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'krasnoyarsk.gluboki-turizm.ru': 'gluboki-turizm.ru',
 'penza.gluboki-turizm.ru': 'gluboki-turizm.ru',
    'vseozakone.ru': 'easytip.ru',
 'pravoved-obzor.ru': 'easytip.ru',
 'easytip.ru': 'easytip.ru',
 'samasv.ru': 'easytip.ru',
 'zdorovyisustav.ru': 'easytip.ru',
 'sibezrcs.ru': 'easytip.ru',
 'dom-tiramisu.ru': 'easytip.ru',
 'znai-zakon.ru': 'easytip.ru',
 'beybik.ru': 'easytip.ru',
 'lastenas.ru': 'easytip.ru',
 'catfur.ru': 'easytip.ru',
 'prokazz.ru': 'easytip.ru',
 'duchist.ru': 'easytip.ru',
 'fintif.ru': 'easytip.ru',
 'russkiy-spaniel.ru': 'easytip.ru',
 '2krendelya.ru': 'easytip.ru',
 'lutiks.ru': 'easytip.ru',
 'globussport.ru': 'easytip.ru',
    'kr.remkarta.ru': 'remkarta.ru',
 'spb.remkarta.ru': 'remkarta.ru',
 'udm.remkarta.ru': 'remkarta.ru',
 'ekb.remkarta.ru': 'remkarta.ru',
 'kostroma.remkarta.ru': 'remkarta.ru',
 'vologda.remkarta.ru': 'remkarta.ru',
 'oren.remkarta.ru': 'remkarta.ru',
 'ryazan.remkarta.ru': 'remkarta.ru',
 'yar.remkarta.ru': 'remkarta.ru',
 'mari.remkarta.ru': 'remkarta.ru',
 'komi.remkarta.ru': 'remkarta.ru',
 'khv.remkarta.ru': 'remkarta.ru',
 'perm.remkarta.ru': 'remkarta.ru',
 'karelia.remkarta.ru': 'remkarta.ru',
 'ufa.remkarta.ru': 'remkarta.ru',
 'kras.remkarta.ru': 'remkarta.ru',
 'tsk.remkarta.ru': 'remkarta.ru',
 'samara.remkarta.ru': 'remkarta.ru',
 'vrn.remkarta.ru': 'remkarta.ru',
    'sankt-peterburg-kalyanika.ru': 'kalyanika.ru',
 'krasnoyarsk-kalyanika.ru': 'kalyanika.ru',
 'vladivostok-kalyanika.ru': 'kalyanika.ru',
 'kaliningrad-kalyanika.ru': 'kalyanika.ru',
 'ekaterinburg-kalyanika.ru': 'kalyanika.ru',
 'irkutsk-kalyanika.ru': 'kalyanika.ru',
 'tyumen-kalyanika.ru': 'kalyanika.ru',
 'kursk-kalyanika.ru': 'kalyanika.ru',
 'kazan-kalyanika.ru': 'kalyanika.ru',
 'orenburg-kalyanika.ru': 'kalyanika.ru',
 'samara-kalyanika.ru': 'kalyanika.ru',
 'barnaul-kalyanika.ru': 'kalyanika.ru',
 'volgograd-kalyanika.ru': 'kalyanika.ru',
 'penza-kalyanika.ru': 'kalyanika.ru',
 'habarovsk-kalyanika.ru': 'kalyanika.ru',
 'ulyanovsk-kalyanika.ru': 'kalyanika.ru',
 'stavropol-kalyanika.ru': 'kalyanika.ru',
 'perm-kalyanika.ru': 'kalyanika.ru',
 'izhevsk-kalyanika.ru': 'kalyanika.ru',
 'voronezh-kalyanika.ru': 'kalyanika.ru',
    'pen.selhozproduct.ru': 'selhozproduct.ru',
 'ekb.selhozproduct.ru': 'selhozproduct.ru',
 'bash.selhozproduct.ru': 'selhozproduct.ru',
 'idj.selhozproduct.ru': 'selhozproduct.ru',
 'che.selhozproduct.ru': 'selhozproduct.ru',
 'selhozproduct.ru': 'selhozproduct.ru',
 'klgd.selhozproduct.ru': 'selhozproduct.ru',
 'hab.selhozproduct.ru': 'selhozproduct.ru',
 'smol.selhozproduct.ru': 'selhozproduct.ru',
 'kurs.selhozproduct.ru': 'selhozproduct.ru',
 'orb.selhozproduct.ru': 'selhozproduct.ru',
 'prim.selhozproduct.ru': 'selhozproduct.ru',
 'msk.selhozproduct.ru': 'selhozproduct.ru',
 'iv.selhozproduct.ru': 'selhozproduct.ru',
 'vor.selhozproduct.ru': 'selhozproduct.ru',
 'krd.selhozproduct.ru': 'selhozproduct.ru',
 'smr.selhozproduct.ru': 'selhozproduct.ru',
 'tamb.selhozproduct.ru': 'selhozproduct.ru',
 'nsk.selhozproduct.ru': 'selhozproduct.ru',
 'krjs.selhozproduct.ru': 'selhozproduct.ru',
 'omsk.selhozproduct.ru': 'selhozproduct.ru',
    '2gis.github.io': 'github.io',
 'owlcarousel2.github.io': 'github.io',
 'sachinchoolur.github.io': 'github.io',
 'googleads.github.io': 'github.io',
 'gromo.github.io': 'github.io',
 'mourner.github.io': 'github.io',
 'qiao.github.io': 'github.io',
 'axelofan.github.io': 'github.io',
 'itdelatrisu.github.io': 'github.io',
 'cbonte.github.io': 'github.io',
 'triplepointfive.github.io': 'github.io',
 'romannurik.github.io': 'github.io',
 'npm.github.io': 'github.io',
 'gionkunz.github.io': 'github.io',
 'antirek.github.io': 'github.io',
 'mongodb.github.io': 'github.io',
 'designmodo.github.io': 'github.io',
 'oletus.github.io': 'github.io',
 'davidstutz.github.io': 'github.io',
 'nightscout.github.io': 'github.io',
 'haxiomic.github.io': 'github.io',
    'adygeya.restate.ru': 'restate.ru',
 'belgorodskaya-oblast.restate.ru': 'restate.ru',
 'nizhniy-novgorod.restate.ru': 'restate.ru',
 'restate.ru': 'restate.ru',
 'goryachiy-klyuch.restate.ru': 'restate.ru',
 'pereslavl-zalesskiy.restate.ru': 'restate.ru',
 'kostroma.restate.ru': 'restate.ru',
 'volzhskiy.restate.ru': 'restate.ru',
 'bryansk.restate.ru': 'restate.ru',
 'chita.restate.ru': 'restate.ru',
 'berdsk.restate.ru': 'restate.ru',
 'blagoveschensk.restate.ru': 'restate.ru',
 'marnitogorsk.restate.ru': 'restate.ru',
 'kimry.restate.ru': 'restate.ru',
 'essentuki.restate.ru': 'restate.ru',
 'novocheboksarsk.restate.ru': 'restate.ru',
 'evpatoriya.restate.ru': 'restate.ru',
 'murom.restate.ru': 'restate.ru',
 'kopeysk.restate.ru': 'restate.ru',
 'orel.restate.ru': 'restate.ru',
 'novgorodskaya-oblast.restate.ru': 'restate.ru',
    'au.ru': 'au.ru',
 'krsk.au.ru': 'au.ru',
 'krasnodar.au.ru': 'au.ru',
 'simferopol.au.ru': 'au.ru',
 'ul.au.ru': 'au.ru',
 'rnd.au.ru': 'au.ru',
 'spb.au.ru': 'au.ru',
 'omsk.au.ru': 'au.ru',
 'kem.au.ru': 'au.ru',
 'vologda.au.ru': 'au.ru',
 'chel.au.ru': 'au.ru',
 'stav.au.ru': 'au.ru',
 'ekat.au.ru': 'au.ru',
 'nsk.au.ru': 'au.ru',
 'abakan.au.ru': 'au.ru',
 'nn.au.ru': 'au.ru',
 'ufa.au.ru': 'au.ru',
 'yar.au.ru': 'au.ru',
 'sevastopol.au.ru': 'au.ru',
 'lipetsk.au.ru': 'au.ru',
 'hm.au.ru': 'au.ru',
    'spravka-region.ru': 'spravka-region.ru',
 'shadrinsk.spravka-region.ru': 'spravka-region.ru',
 'kamenolomni.spravka-region.ru': 'spravka-region.ru',
 'balezino.spravka-region.ru': 'spravka-region.ru',
 'koksovyj.spravka-region.ru': 'spravka-region.ru',
 'orehovo-zuevo.spravka-region.ru': 'spravka-region.ru',
 'zaozersk.spravka-region.ru': 'spravka-region.ru',
 'zhigulevsk.spravka-region.ru': 'spravka-region.ru',
 'lyubytino.spravka-region.ru': 'spravka-region.ru',
 'kaliningrad.spravka-region.ru': 'spravka-region.ru',
 'omutninsk.spravka-region.ru': 'spravka-region.ru',
 'kansk.spravka-region.ru': 'spravka-region.ru',
 'kabardinka.spravka-region.ru': 'spravka-region.ru',
 'starosherbinovskaya.spravka-region.ru': 'spravka-region.ru',
 'babayurt.spravka-region.ru': 'spravka-region.ru',
 'dzhubga.spravka-region.ru': 'spravka-region.ru',
 'bujnaksk.spravka-region.ru': 'spravka-region.ru',
 'minusinsk.spravka-region.ru': 'spravka-region.ru',
 'ruzaevka.spravka-region.ru': 'spravka-region.ru',
 'verhnij-baskunchak.spravka-region.ru': 'spravka-region.ru',
 'anna.spravka-region.ru': 'spravka-region.ru',
 'zirgan.spravka-region.ru': 'spravka-region.ru',
    'careerist.ru': 'careerist.ru',
 'engels.careerist.ru': 'careerist.ru',
 'orenburg.careerist.ru': 'careerist.ru',
 'chelyabinsk.careerist.ru': 'careerist.ru',
 'astrakhan.careerist.ru': 'careerist.ru',
 'sibai.careerist.ru': 'careerist.ru',
 'pyatigorsk.careerist.ru': 'careerist.ru',
 'vologda.careerist.ru': 'careerist.ru',
 'sevastopol.careerist.ru': 'careerist.ru',
 'ulyanovsk.careerist.ru': 'careerist.ru',
 'zelenograd.careerist.ru': 'careerist.ru',
 'vsevolozhsk.careerist.ru': 'careerist.ru',
 'yakutsk.careerist.ru': 'careerist.ru',
 'taganrog.careerist.ru': 'careerist.ru',
 'vladivostok.careerist.ru': 'careerist.ru',
 'zvenigorod.careerist.ru': 'careerist.ru',
 'surgut.careerist.ru': 'careerist.ru',
 'yuzhno-sakhalinsk.careerist.ru': 'careerist.ru',
 'odintsovo.careerist.ru': 'careerist.ru',
 'bataisk.careerist.ru': 'careerist.ru',
 'petrozavodsk.careerist.ru': 'careerist.ru',
 'kolomna.careerist.ru': 'careerist.ru',
 'serpukhov.careerist.ru': 'careerist.ru',
    'humansandelves.rolka.me': 'rolka.me',
 'imperiumaeternum.rolka.me': 'rolka.me',
 'miyron.rolka.me': 'rolka.me',
 'fantalesofmarvel.rolbb.me': 'rolka.me',
 'romanceclub.rolka.me': 'rolka.me',
 'totop.rolka.me': 'rolka.me',
 'aenhanse.rolka.me': 'rolka.me',
 'shardsofpower.rolka.me': 'rolka.me',
 'kiri.rolka.me': 'rolka.me',
 'modao.rolka.me': 'rolka.me',
 'beautifulworld.rolka.me': 'rolka.me',
 'siististi.rolbb.me': 'rolka.me',
 'ogamisamadd.rolka.me': 'rolka.me',
 'rainyday.rolbb.me': 'rolka.me',
 'bloodlines.rolka.me': 'rolka.me',
 'mefochka.rolka.me': 'rolka.me',
 'wiedzmin.rolka.me': 'rolka.me',
 'justfriends.rolka.me': 'rolka.me',
 'bohemia.rolka.me': 'rolka.me',
 'modaozushi.rolka.me': 'rolka.me',
 'alteras.rolka.me': 'rolka.me',
 'elinorworld.rolka.me': 'rolka.me',
 'titaniclove.rolka.me': 'rolka.me',
    'v-novgorod.adresa-telefony.ru': 'adresa-telefony.ru',
 'mahachkala.adresa-telefony.ru': 'adresa-telefony.ru',
 'habarovsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'blagoveshensk.adresa-telefony.ru': 'adresa-telefony.ru',
 'komsomolsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'ivanovo.adresa-telefony.ru': 'adresa-telefony.ru',
 'dzerzhinsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'volgograd.adresa-telefony.ru': 'adresa-telefony.ru',
 'novoaltaysk.adresa-telefony.ru': 'adresa-telefony.ru',
 'saransk.adresa-telefony.ru': 'adresa-telefony.ru',
 'bishkek.adresa-telefony.ru': 'adresa-telefony.ru',
 'kaliningrad.adresa-telefony.ru': 'adresa-telefony.ru',
 'syzran.adresa-telefony.ru': 'adresa-telefony.ru',
 'uhta.adresa-telefony.ru': 'adresa-telefony.ru',
 'chelyabinsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'krasnoyarsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'elets.adresa-telefony.ru': 'adresa-telefony.ru',
 'syktyvkar.adresa-telefony.ru': 'adresa-telefony.ru',
 'gornoaltaysk.adresa-telefony.ru': 'adresa-telefony.ru',
 'novocherkassk.adresa-telefony.ru': 'adresa-telefony.ru',
 'lubertsy.adresa-telefony.ru': 'adresa-telefony.ru',
 'uralsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'yuzhno-sahalinsk.adresa-telefony.ru': 'adresa-telefony.ru',
 'adresa-telefony.ru': 'adresa-telefony.ru',
    'xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'krasnodarskii-krai.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'chelyabinskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'stavropolskii-krai.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'chuvashiya-resp.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'sverdlovskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'saratovskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'krasnoyarsk.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'tulskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'tambovskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'tatarstan-resp.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'samarskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'volgogradskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'kursk.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'yaroslavskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'leningradskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'hakasiya-resp.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'rostovskaya-oblast.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'bashkortostan-resp.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'kostromskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'voronezhskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'krasnodar.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'kareliya-resp.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
 'tverskaya-obl.xn--80ajgpcpbhkds4a4g.xn--p1ai': 'xn--80ajgpcpbhkds4a4g.xn--p1ai',
    'nat4-vpr.sdamgia.ru': 'sdamgia.ru',
 'math6-vpr.sdamgia.ru': 'sdamgia.ru',
 'hist8-vpr.sdamgia.ru': 'sdamgia.ru',
 'rus8-vpr.sdamgia.ru': 'sdamgia.ru',
 'hist5-vpr.sdamgia.ru': 'sdamgia.ru',
 'math5-vpr.sdamgia.ru': 'sdamgia.ru',
 'chem8-vpr.sdamgia.ru': 'sdamgia.ru',
 'rus5-vpr.sdamgia.ru': 'sdamgia.ru',
 'bio5-vpr.sdamgia.ru': 'sdamgia.ru',
 'geo8-vpr.sdamgia.ru': 'sdamgia.ru',
 'geo11-vpr.sdamgia.ru': 'sdamgia.ru',
 'chem11-vpr.sdamgia.ru': 'sdamgia.ru',
 'soc7-vpr.sdamgia.ru': 'sdamgia.ru',
 'bio6-vpr.sdamgia.ru': 'sdamgia.ru',
 'bio11-vpr.sdamgia.ru': 'sdamgia.ru',
 'hist6-vpr.sdamgia.ru': 'sdamgia.ru',
 'soc8-vpr.sdamgia.ru': 'sdamgia.ru',
 'rus7-vpr.sdamgia.ru': 'sdamgia.ru',
 'rus2-vpr.sdamgia.ru': 'sdamgia.ru',
 'math8-vpr.sdamgia.ru': 'sdamgia.ru',
 'en7-vpr.sdamgia.ru': 'sdamgia.ru',
 'en11-vpr.sdamgia.ru': 'sdamgia.ru',
 'soc6-vpr.sdamgia.ru': 'sdamgia.ru',
 'phys-vpr.sdamgia.ru': 'sdamgia.ru',
    'nuipogoda.ru': 'nuipogoda.ru',
 'turgoyak.nuipogoda.ru': 'nuipogoda.ru',
 'naberezhnye-chelny.nuipogoda.ru': 'nuipogoda.ru',
 'gorno-altaysk.nuipogoda.ru': 'nuipogoda.ru',
 'parnaya.nuipogoda.ru': 'nuipogoda.ru',
 'cheboksary.nuipogoda.ru': 'nuipogoda.ru',
 'perevoloki-syzranskiy-rayon-samarskaya.nuipogoda.ru': 'nuipogoda.ru',
 'barzas.nuipogoda.ru': 'nuipogoda.ru',
 'sterlitamak.nuipogoda.ru': 'nuipogoda.ru',
 'leninogorsk-tatarstan.nuipogoda.ru': 'nuipogoda.ru',
 'cherepovec.nuipogoda.ru': 'nuipogoda.ru',
 'kursk.nuipogoda.ru': 'nuipogoda.ru',
 'sosnovo.nuipogoda.ru': 'nuipogoda.ru',
 'kandrykul.nuipogoda.ru': 'nuipogoda.ru',
 'izberbash.nuipogoda.ru': 'nuipogoda.ru',
 'zheleznodorozhniy.nuipogoda.ru': 'nuipogoda.ru',
 'harasavey.nuipogoda.ru': 'nuipogoda.ru',
 'artybash.nuipogoda.ru': 'nuipogoda.ru',
 'yurga.nuipogoda.ru': 'nuipogoda.ru',
 'aprelevka.nuipogoda.ru': 'nuipogoda.ru',
 'sayanogorsk.nuipogoda.ru': 'nuipogoda.ru',
 'lyskovo.nuipogoda.ru': 'nuipogoda.ru',
 'dzerzhinsk.nuipogoda.ru': 'nuipogoda.ru',
 'borisoglebsk.nuipogoda.ru': 'nuipogoda.ru',
    'mahachkala.doctora-mne.ru': 'doctora-mne.ru',
 'doctora-mne.ru': 'doctora-mne.ru',
 'kaliningrad.doctora-mne.ru': 'doctora-mne.ru',
 'mozhaisk.doctora-mne.ru': 'doctora-mne.ru',
 'anapa.doctora-mne.ru': 'doctora-mne.ru',
 'orel.doctora-mne.ru': 'doctora-mne.ru',
 'kolomna.doctora-mne.ru': 'doctora-mne.ru',
 'izhevsk.doctora-mne.ru': 'doctora-mne.ru',
 'rasskazovo.doctora-mne.ru': 'doctora-mne.ru',
 'novorossiysk.doctora-mne.ru': 'doctora-mne.ru',
 'n-chelny.doctora-mne.ru': 'doctora-mne.ru',
 'barnaul.doctora-mne.ru': 'doctora-mne.ru',
 'penza.doctora-mne.ru': 'doctora-mne.ru',
 'serpuhov.doctora-mne.ru': 'doctora-mne.ru',
 'kirsanov.doctora-mne.ru': 'doctora-mne.ru',
 'tambov.doctora-mne.ru': 'doctora-mne.ru',
 'bryansk.doctora-mne.ru': 'doctora-mne.ru',
 'krasnodar.doctora-mne.ru': 'doctora-mne.ru',
 'lipetsk.doctora-mne.ru': 'doctora-mne.ru',
 'toliatti.doctora-mne.ru': 'doctora-mne.ru',
 'lubercy.doctora-mne.ru': 'doctora-mne.ru',
 'orenburg.doctora-mne.ru': 'doctora-mne.ru',
 'zheleznodorozhniy.doctora-mne.ru': 'doctora-mne.ru',
 'maikop.doctora-mne.ru': 'doctora-mne.ru',
    'my.deal.by': 'deal.by',
 'deal.by': 'deal.by',
 'minsk.deal.by': 'deal.by',
 'minskobl.deal.by': 'deal.by',
 'smolevichi.deal.by': 'deal.by',
 'mogilev.deal.by': 'deal.by',
 'majinagorka.deal.by': 'deal.by',
 'brest.deal.by': 'deal.by',
 'orsha.deal.by': 'deal.by',
 'lyahovichi.deal.by': 'deal.by',
 'dzerzhinsk.deal.by': 'deal.by',
 'gorki.deal.by': 'deal.by',
 'polotsk.deal.by': 'deal.by',
 'novopolotsk.deal.by': 'deal.by',
 'gomel.deal.by': 'deal.by',
 'pinsk.deal.by': 'deal.by',
 'molodechno.deal.by': 'deal.by',
 'svetlogorsk.deal.by': 'deal.by',
 'zhodino.deal.by': 'deal.by',
 'slutsk.deal.by': 'deal.by',
 'grodnenskaya-obl.deal.by': 'deal.by',
 'grodno.deal.by': 'deal.by',
 'borisov.deal.by': 'deal.by',
 'slonim.deal.by': 'deal.by',
    'moscow.cataloxy.ru': 'cataloxy.ru',
 'mytischi.cataloxy.ru': 'cataloxy.ru',
 'adler.cataloxy.ru': 'cataloxy.ru',
 'cataloxy.ru': 'cataloxy.ru',
 'orel.cataloxy.ru': 'cataloxy.ru',
 'tolyatti.cataloxy.ru': 'cataloxy.ru',
 'nizhni-novgorod.cataloxy.ru': 'cataloxy.ru',
 'tomskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'saransk.cataloxy.ru': 'cataloxy.ru',
 'yaroslavl.cataloxy.ru': 'cataloxy.ru',
 'dyurtyuli.cataloxy.ru': 'cataloxy.ru',
 'stary-oskol.cataloxy.ru': 'cataloxy.ru',
 'labinsk.cataloxy.ru': 'cataloxy.ru',
 'orenburg.cataloxy.ru': 'cataloxy.ru',
 'novosibirsk.cataloxy.ru': 'cataloxy.ru',
 'st-petersburg.cataloxy.ru': 'cataloxy.ru',
 'slavyansk-na-kubani.cataloxy.ru': 'cataloxy.ru',
 'golitsyno.cataloxy.ru': 'cataloxy.ru',
 'saratov.cataloxy.ru': 'cataloxy.ru',
 'zhukovsky.cataloxy.ru': 'cataloxy.ru',
 'krasnoznamensk.cataloxy.ru': 'cataloxy.ru',
 'cheboksary.cataloxy.ru': 'cataloxy.ru',
 'krasnodar.cataloxy.ru': 'cataloxy.ru',
 'krasnogorsk.cataloxy.ru': 'cataloxy.ru',
 'vladivostok.cataloxy.ru': 'cataloxy.ru',
 'electrostal.cataloxy.ru': 'cataloxy.ru',
 'magadan.cataloxy.ru': 'cataloxy.ru',
 'ryazan.cataloxy.ru': 'cataloxy.ru',
 'novokuznetsk.cataloxy.ru': 'cataloxy.ru',
 'vyatskie-polyany.cataloxy.ru': 'cataloxy.ru',
 'krasnoyarsk.cataloxy.ru': 'cataloxy.ru',
 'sochi.cataloxy.ru': 'cataloxy.ru',
 'yuzhno-sakhalinsk.cataloxy.ru': 'cataloxy.ru',
 'orsk.cataloxy.ru': 'cataloxy.ru',
 'ufa.cataloxy.ru': 'cataloxy.ru',
 'aleksandrov.cataloxy.ru': 'cataloxy.ru',
 'sergiev-posad.cataloxy.ru': 'cataloxy.ru',
 'pushkino.cataloxy.ru': 'cataloxy.ru',
 'volgodonsk.cataloxy.ru': 'cataloxy.ru',
 'balashiha.cataloxy.ru': 'cataloxy.ru',
 'kazan.cataloxy.ru': 'cataloxy.ru',
 'taganrog.cataloxy.ru': 'cataloxy.ru',
 'krymsk.cataloxy.ru': 'cataloxy.ru',
 'kemerovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'ishim.cataloxy.ru': 'cataloxy.ru',
 'timashevsk.cataloxy.ru': 'cataloxy.ru',
 'lobnya.cataloxy.ru': 'cataloxy.ru',
 'habarovskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'petropavlovsk-kamchatsky.cataloxy.ru': 'cataloxy.ru',
 'irkutskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'samara.cataloxy.ru': 'cataloxy.ru',
 'tambov.cataloxy.ru': 'cataloxy.ru',
 'ekaterinburg.cataloxy.ru': 'cataloxy.ru',
 'podolsk.cataloxy.ru': 'cataloxy.ru',
 'moskovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'khabarovsk.cataloxy.ru': 'cataloxy.ru',
 'odintsovo.cataloxy.ru': 'cataloxy.ru',
 'frjazino.cataloxy.ru': 'cataloxy.ru',
 'eysk.cataloxy.ru': 'cataloxy.ru',
 'kirov.cataloxy.ru': 'cataloxy.ru',
 'irkutsk.cataloxy.ru': 'cataloxy.ru',
 'ivanovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'novoaltaysk.cataloxy.ru': 'cataloxy.ru',
 'pskov.cataloxy.ru': 'cataloxy.ru',
 'sosnovyy-bor.cataloxy.ru': 'cataloxy.ru',
 'domodedovo.cataloxy.ru': 'cataloxy.ru',
 'tynda.cataloxy.ru': 'cataloxy.ru',
 'dzerzhinsk.cataloxy.ru': 'cataloxy.ru',
 'bryansk.cataloxy.ru': 'cataloxy.ru',
 'elets.cataloxy.ru': 'cataloxy.ru',
 'ryazhsk.cataloxy.ru': 'cataloxy.ru',
 'tula.cataloxy.ru': 'cataloxy.ru',
 'amursk.cataloxy.ru': 'cataloxy.ru',
 'tutaev.cataloxy.ru': 'cataloxy.ru',
 'rossosh.cataloxy.ru': 'cataloxy.ru',
 'shakhty.cataloxy.ru': 'cataloxy.ru',
 'novyy-urengoy.cataloxy.ru': 'cataloxy.ru',
 'vyborg.cataloxy.ru': 'cataloxy.ru',
 'tambovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'nizhnekamsk.cataloxy.ru': 'cataloxy.ru',
 'leningradskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'rostovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'primorskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'nizhnevartovsk.cataloxy.ru': 'cataloxy.ru',
 'rtischevo.cataloxy.ru': 'cataloxy.ru',
 'tobolsk.cataloxy.ru': 'cataloxy.ru',
 'yurga.cataloxy.ru': 'cataloxy.ru',
 'meleuz.cataloxy.ru': 'cataloxy.ru',
 'elektrougli.cataloxy.ru': 'cataloxy.ru',
 'lipetsk.cataloxy.ru': 'cataloxy.ru',
 'abakan.cataloxy.ru': 'cataloxy.ru',
 'balashov.cataloxy.ru': 'cataloxy.ru',
 'kursk.cataloxy.ru': 'cataloxy.ru',
 'elektrogorsk.cataloxy.ru': 'cataloxy.ru',
 'kemerovo.cataloxy.ru': 'cataloxy.ru',
 'komsomolsk-na-amure.cataloxy.ru': 'cataloxy.ru',
 'omsk.cataloxy.ru': 'cataloxy.ru',
 'lesozavodsk.cataloxy.ru': 'cataloxy.ru',
 'surgut.cataloxy.ru': 'cataloxy.ru',
 'armavir.cataloxy.ru': 'cataloxy.ru',
 'simferopol.cataloxy.ru': 'cataloxy.ru',
 'lyubertsy.cataloxy.ru': 'cataloxy.ru',
 'khanti-mansysk.cataloxy.ru': 'cataloxy.ru',
 'belaja-kalitva.cataloxy.ru': 'cataloxy.ru',
 'petushki.cataloxy.ru': 'cataloxy.ru',
 'vladimirskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'pyt-yah.cataloxy.ru': 'cataloxy.ru',
 'korenovsk.cataloxy.ru': 'cataloxy.ru',
 'miass.cataloxy.ru': 'cataloxy.ru',
 'tihoretsk.cataloxy.ru': 'cataloxy.ru',
 'serpukhov.cataloxy.ru': 'cataloxy.ru',
 'dinskaya.cataloxy.ru': 'cataloxy.ru',
 'biisk.cataloxy.ru': 'cataloxy.ru',
 'svobodnyy.cataloxy.ru': 'cataloxy.ru',
 'nyagan.cataloxy.ru': 'cataloxy.ru',
 'maikop.cataloxy.ru': 'cataloxy.ru',
 'stupino.cataloxy.ru': 'cataloxy.ru',
 'volzhsky.cataloxy.ru': 'cataloxy.ru',
 'zelenogorsk-krasnoyarskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'artem.cataloxy.ru': 'cataloxy.ru',
 'novosibirskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'taldom.cataloxy.ru': 'cataloxy.ru',
 'shushary.cataloxy.ru': 'cataloxy.ru',
 'vidnoe.cataloxy.ru': 'cataloxy.ru',
 'noginsk.cataloxy.ru': 'cataloxy.ru',
 'kuvandyk.cataloxy.ru': 'cataloxy.ru',
 'murmansk.cataloxy.ru': 'cataloxy.ru',
 'murom.cataloxy.ru': 'cataloxy.ru',
 'veliky-novgorod.cataloxy.ru': 'cataloxy.ru',
 'nakhodka.cataloxy.ru': 'cataloxy.ru',
 'kogalym.cataloxy.ru': 'cataloxy.ru',
 'korolev.cataloxy.ru': 'cataloxy.ru',
 'minusinsk.cataloxy.ru': 'cataloxy.ru',
 'smolensk.cataloxy.ru': 'cataloxy.ru',
 'nerjungri.cataloxy.ru': 'cataloxy.ru',
 'saha-respublika.cataloxy.ru': 'cataloxy.ru',
 'tomsk.cataloxy.ru': 'cataloxy.ru',
 'saratovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'petrovsk.cataloxy.ru': 'cataloxy.ru',
 'balakovo.cataloxy.ru': 'cataloxy.ru',
 'bryanskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'rumyantsevo.cataloxy.ru': 'cataloxy.ru',
 'luhovitsy.cataloxy.ru': 'cataloxy.ru',
 'temryuk.cataloxy.ru': 'cataloxy.ru',
 'kansk.cataloxy.ru': 'cataloxy.ru',
 'stavropolskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'bashkortostan-respublika.cataloxy.ru': 'cataloxy.ru',
 'novovoronezh.cataloxy.ru': 'cataloxy.ru',
 'komi-respublika.cataloxy.ru': 'cataloxy.ru',
 'sudak.cataloxy.ru': 'cataloxy.ru',
 'usinsk.cataloxy.ru': 'cataloxy.ru',
 'michurinsk.cataloxy.ru': 'cataloxy.ru',
 'novocheboksarsk.cataloxy.ru': 'cataloxy.ru',
 'kamensk-uralsky.cataloxy.ru': 'cataloxy.ru',
 'yubileynyy.cataloxy.ru': 'cataloxy.ru',
 'blagoveschensk.cataloxy.ru': 'cataloxy.ru',
 'kabardino-balkarskaya-respublika.cataloxy.ru': 'cataloxy.ru',
 'gelendzhik.cataloxy.ru': 'cataloxy.ru',
 'vologda.cataloxy.ru': 'cataloxy.ru',
 'chehov.cataloxy.ru': 'cataloxy.ru',
 'tuymazy.cataloxy.ru': 'cataloxy.ru',
 'zheleznodorozhnyy.cataloxy.ru': 'cataloxy.ru',
 'kolomna.cataloxy.ru': 'cataloxy.ru',
 'penza.cataloxy.ru': 'cataloxy.ru',
 'millerovo.cataloxy.ru': 'cataloxy.ru',
 'bataysk.cataloxy.ru': 'cataloxy.ru',
 'zhukov.cataloxy.ru': 'cataloxy.ru',
 'kaluzhskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'vysokovsk.cataloxy.ru': 'cataloxy.ru',
 'voskresensk.cataloxy.ru': 'cataloxy.ru',
 'georgievsk.cataloxy.ru': 'cataloxy.ru',
 'velikie-luki.cataloxy.ru': 'cataloxy.ru',
 'volgograd.cataloxy.ru': 'cataloxy.ru',
 'ioshkar-ola.cataloxy.ru': 'cataloxy.ru',
 'kurgan.cataloxy.ru': 'cataloxy.ru',
 'nefteyugansk.cataloxy.ru': 'cataloxy.ru',
 'aksay.cataloxy.ru': 'cataloxy.ru',
 'belgorod.cataloxy.ru': 'cataloxy.ru',
 'salavat.cataloxy.ru': 'cataloxy.ru',
 'bolshoy-kamen.cataloxy.ru': 'cataloxy.ru',
 'engels.cataloxy.ru': 'cataloxy.ru',
 'egorevsk.cataloxy.ru': 'cataloxy.ru',
 'nalchik.cataloxy.ru': 'cataloxy.ru',
 'nizhni-tagil.cataloxy.ru': 'cataloxy.ru',
 'kostroma.cataloxy.ru': 'cataloxy.ru',
 'buguruslan.cataloxy.ru': 'cataloxy.ru',
 'orenburgskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'perm.cataloxy.ru': 'cataloxy.ru',
 'voronezh.cataloxy.ru': 'cataloxy.ru',
 'izhevsk.cataloxy.ru': 'cataloxy.ru',
 'shadrinsk.cataloxy.ru': 'cataloxy.ru',
 'cherepovets.cataloxy.ru': 'cataloxy.ru',
 'belogorsk.cataloxy.ru': 'cataloxy.ru',
 'amurskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'krasnoyarskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'belorechensk.cataloxy.ru': 'cataloxy.ru',
 'tyumenskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'schelkovo.cataloxy.ru': 'cataloxy.ru',
 'tyumen.cataloxy.ru': 'cataloxy.ru',
 'cheremhovo.cataloxy.ru': 'cataloxy.ru',
 'anapa.cataloxy.ru': 'cataloxy.ru',
 'dmitrov.cataloxy.ru': 'cataloxy.ru',
 'vladikavkaz.cataloxy.ru': 'cataloxy.ru',
 'reutov.cataloxy.ru': 'cataloxy.ru',
 'zarechnyy-sverdlovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'kaluga.cataloxy.ru': 'cataloxy.ru',
 'stavropol.cataloxy.ru': 'cataloxy.ru',
 'rubtsovsk.cataloxy.ru': 'cataloxy.ru',
 'altayskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'krasnodarskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'tulskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'rybinsk.cataloxy.ru': 'cataloxy.ru',
 'mihaylovsk-stavropolskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'makhachkala.cataloxy.ru': 'cataloxy.ru',
 'putilkovo.cataloxy.ru': 'cataloxy.ru',
 'udmurtskaya-respublika.cataloxy.ru': 'cataloxy.ru',
 'chelyabinskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'ivanovo.cataloxy.ru': 'cataloxy.ru',
 'neftekamsk.cataloxy.ru': 'cataloxy.ru',
 'langepas.cataloxy.ru': 'cataloxy.ru',
 'almetevsk.cataloxy.ru': 'cataloxy.ru',
 'dagestan-respublika.cataloxy.ru': 'cataloxy.ru',
 'kizlyar.cataloxy.ru': 'cataloxy.ru',
 'yaroslavskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'samarskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'yakutsk.cataloxy.ru': 'cataloxy.ru',
 'obninsk.cataloxy.ru': 'cataloxy.ru',
 'ryazanskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'siktivkar.cataloxy.ru': 'cataloxy.ru',
 'udomlya.cataloxy.ru': 'cataloxy.ru',
 'tverskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'kostromskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'sarov.cataloxy.ru': 'cataloxy.ru',
 'kerch.cataloxy.ru': 'cataloxy.ru',
 'valday.cataloxy.ru': 'cataloxy.ru',
 'chernushka.cataloxy.ru': 'cataloxy.ru',
 'belebey.cataloxy.ru': 'cataloxy.ru',
 'arzamas.cataloxy.ru': 'cataloxy.ru',
 'novomoskovsk.cataloxy.ru': 'cataloxy.ru',
 'gorno-altaisk.cataloxy.ru': 'cataloxy.ru',
 'penzenskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'zelenodolsk.cataloxy.ru': 'cataloxy.ru',
 'nevinnomyssk.cataloxy.ru': 'cataloxy.ru',
 'kozmodemyansk.cataloxy.ru': 'cataloxy.ru',
 'mariy-el-respublika.cataloxy.ru': 'cataloxy.ru',
 'dolgoprudnyy.cataloxy.ru': 'cataloxy.ru',
 'chernyahovsk.cataloxy.ru': 'cataloxy.ru',
 'chapaevsk.cataloxy.ru': 'cataloxy.ru',
 'kashira.cataloxy.ru': 'cataloxy.ru',
 'volokolamsk.cataloxy.ru': 'cataloxy.ru',
 'tuapse.cataloxy.ru': 'cataloxy.ru',
 'buzuluk.cataloxy.ru': 'cataloxy.ru',
 'novyy-urengoy-yamalo-nenetskiy.cataloxy.ru': 'cataloxy.ru',
 'yamalo-nenetskiy-ao.cataloxy.ru': 'cataloxy.ru',
 'orehovo-zuevo.cataloxy.ru': 'cataloxy.ru',
 'kuybyshev.cataloxy.ru': 'cataloxy.ru',
 'ukhta.cataloxy.ru': 'cataloxy.ru',
 'yahroma.cataloxy.ru': 'cataloxy.ru',
 'zavolzhe.cataloxy.ru': 'cataloxy.ru',
 'sovetsk-kirovskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'bogorodsk.cataloxy.ru': 'cataloxy.ru',
 'myski.cataloxy.ru': 'cataloxy.ru',
 'gubkin.cataloxy.ru': 'cataloxy.ru',
 'oktjabrsky.cataloxy.ru': 'cataloxy.ru',
 'roslavl.cataloxy.ru': 'cataloxy.ru',
 'adygeya-respublika.cataloxy.ru': 'cataloxy.ru',
 'troitsk.cataloxy.ru': 'cataloxy.ru',
 'achinsk.cataloxy.ru': 'cataloxy.ru',
 'shuya.cataloxy.ru': 'cataloxy.ru',
 'yuzhno-uralsk.cataloxy.ru': 'cataloxy.ru',
 'zlatoust.cataloxy.ru': 'cataloxy.ru',
 'smolenskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'murino.cataloxy.ru': 'cataloxy.ru',
 'asha.cataloxy.ru': 'cataloxy.ru',
 'istra.cataloxy.ru': 'cataloxy.ru',
 'ramenskoe.cataloxy.ru': 'cataloxy.ru',
 'pavlovsky-posad.cataloxy.ru': 'cataloxy.ru',
 'kropotkin.cataloxy.ru': 'cataloxy.ru',
 'chistopol.cataloxy.ru': 'cataloxy.ru',
 'hakasiya-respublika.cataloxy.ru': 'cataloxy.ru',
 'votkinsk.cataloxy.ru': 'cataloxy.ru',
 'permskiy-kray.cataloxy.ru': 'cataloxy.ru',
 'noyabrsk.cataloxy.ru': 'cataloxy.ru',
 'sterlitamak.cataloxy.ru': 'cataloxy.ru',
 'solontsy.cataloxy.ru': 'cataloxy.ru',
 'novocherkassk.cataloxy.ru': 'cataloxy.ru',
 'dzerzhinskiy.cataloxy.ru': 'cataloxy.ru',
 'solnechnogorsk.cataloxy.ru': 'cataloxy.ru',
 'omskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'astrakhan.cataloxy.ru': 'cataloxy.ru',
 'kotelniki.cataloxy.ru': 'cataloxy.ru',
 'neftegorsk.cataloxy.ru': 'cataloxy.ru',
 'vyazma.cataloxy.ru': 'cataloxy.ru',
 'kyzyl.cataloxy.ru': 'cataloxy.ru',
 'essentuki.cataloxy.ru': 'cataloxy.ru',
 'angarsk.cataloxy.ru': 'cataloxy.ru',
 'feodosiya.cataloxy.ru': 'cataloxy.ru',
 'voronezhskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'livny.cataloxy.ru': 'cataloxy.ru',
 'novokuybyshevsk.cataloxy.ru': 'cataloxy.ru',
 'uzlovaya.cataloxy.ru': 'cataloxy.ru',
 'lebedyan.cataloxy.ru': 'cataloxy.ru',
 'himki.cataloxy.ru': 'cataloxy.ru',
 'nizhegorodskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'pavlovo.cataloxy.ru': 'cataloxy.ru',
 'abinsk.cataloxy.ru': 'cataloxy.ru',
 'kirzhach.cataloxy.ru': 'cataloxy.ru',
 'vologodskaya-oblast.cataloxy.ru': 'cataloxy.ru',
 'birobidzhan.cataloxy.ru': 'cataloxy.ru',
 'gukovo.cataloxy.ru': 'cataloxy.ru',
 'sharya.cataloxy.ru': 'cataloxy.ru',
 'klintsy.cataloxy.ru': 'cataloxy.ru',
 'vorkuta.cataloxy.ru': 'cataloxy.ru',
 'salsk.cataloxy.ru': 'cataloxy.ru',
'xn--b1amdelgbarlac.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aliz.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aagyrxe.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1agflcndeb5cwb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1afaslnbn.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1afocqbedfq2j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adiweqejcms5i.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----7sbbsrgbccjgn5blf2a0n.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----8sbjjjbdewlc7d.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80agtcngblfff.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1acdycodcb5a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80akatihikn.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aaky0bj.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1adaae4bnbdb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----ctbbjcl9bbben6bu.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aafh7b.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80afebak3arjv.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adjubnhbtma0e.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----8sbedibbx1djfkj.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1admucdb5d.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aaib3cgnl.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aab8af5be.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80atdefdb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1adkeby9a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1af1adeerq.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1ahhecb0azo.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--c1abbuhodcjk7a6g.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adtilddb4ah6a8a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80ablnkrs4e.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90aiqw4a4aq.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aekhj1a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aaab1bq6ewa.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80a0biks.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--d1amckbds1g.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90aodbbvqig6h4a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--c1akgjcdy.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1apdlal5j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aplwa.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80agatlhjjbulh.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1aibcrgq.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80anmnpecg.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80acd2blu.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aecgfmj1g.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aqglu7a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aabg3bexb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--c1aesfx9dc.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--j1aarei.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80ajchmbncc8b.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80a1bd.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1afchn5b.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1aqbiftj7e1a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80addcdua5attdt.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1apcnb5dwc.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80antkd4dl.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--c1azcgcc.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1afipca2eya.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aauks4g.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90a1af.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90abe7bk7d.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1albuvt.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--c1aehbhtrap.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1aaifueubwg.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1adkfu4a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aahvz2a9a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1addjcde.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aaaf5a1c.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1aner7ci.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adbjeltrcnvok.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aqejg8dua.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1aigedeu0a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80afuomr.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--j1adfnc.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80avue.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1aaxbhehbbb5ad.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1alfrj.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aai0ag2c.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80ajpcdqnjgd6ib.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1apmck.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1ag7af7c.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90aode8a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aab6birx.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90acyoijkz.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adbhunc2aa3al.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90abblsferjb2bt1eyb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1aadecsaihi0amca3a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adbv1agb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aa4alnee.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1afhbv7b.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1aar9b.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80antj7do.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aaa0a0avl.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--d1acdchmuvm.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1akbdhdtf.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90aee6admdx.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adde7arb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1afab7bff7cb.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--h1aeawgfg.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aeia0ach7bg1hd.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--d1abagsdcojb8a6j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aacf4bwnk3a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn----7sbbahas4ajfbcgtvdlcr1aggp2j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aezclli6gta.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90aoglmj0e.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--g1afe6db.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--c1ajhpcjhd0j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--b1afkijcdaj5aa2j.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80ae1alafffj1i.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--e1aajhowakj.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adypkng.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80ajgxi.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80aaaa2chc7eta.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn-----7kcbzzahiagihbb4bfe9b2l.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--80adhqaok7a.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90aedc4atap.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
 'xn--90asilg6f.xn--80agbcqdjc3d.xn--p1ai': 'xn--80agbcqdjc3d.xn--p1ai',
'tab.103.kz': 'lek.103.ua',
'm.gazeta.ru': 'gazeta.ru',
'amp.7days.ru': '7days.ru',
'nashenasledie.livejournal.com': 'nashe-nasledie.livejournal.com',
'world.news-kmv.ru': 'informer.news-kmv.ru',
'm.lenta.ru': 'lenta.ru',
'amp.eda.ru': 'eda.ru', 
    'ekaterinburg.octagon.media': 'octagon.media', 
    'ufa.octagon.media': 'ufa.octagon.media', 
    'kuban.newizv.ru': 'newizv.ru', 
    'dev.infox.ru': 'infox.ru', 
    'infox.life': 'infox.ru'}

In [3]:
id_to_submit = pd.read_parquet('submit_2.pqt')
id_to_submit.head()

,user_id
221301,221301
31271,31271
211594,211594
253119,253119
192578,192578


In [4]:
data = pd.read_feather('dataset_full.feather')
data = data[['user_id', 'url_host']]
print(data.shape)

(322899435, 2)


In [5]:
tmp = data.groupby('user_id')['url_host'].agg(list).to_frame().reset_index()
tmp.head()

,user_id,url_host
0,0,"[s0.2mdn.net, avatars.mds.yandex.net, googlead..."
1,1,"[avatars.mds.yandex.net, i.ytimg.com, ads.betw..."
2,2,"[lentaru.media.eagleplatform.com, i.ytimg.com,..."
3,3,"[ad.mail.ru, avatars.mds.yandex.net, mail.yand..."
4,4,"[kp.md, yastatic.net, avatars.mds.yandex.net, ..."


In [6]:
tmp['history_len'] = tmp.url_host.apply(len)
tmp.head()

,user_id,url_host,history_len
0,0,"[s0.2mdn.net, avatars.mds.yandex.net, googlead...",131
1,1,"[avatars.mds.yandex.net, i.ytimg.com, ads.betw...",700
2,2,"[lentaru.media.eagleplatform.com, i.ytimg.com,...",356
3,3,"[ad.mail.ru, avatars.mds.yandex.net, mail.yand...",188
4,4,"[kp.md, yastatic.net, avatars.mds.yandex.net, ...",591


In [15]:
tmp = tmp[tmp.user_id.isin(id_to_submit.user_id.tolist())].reset_index(drop=True)
tmp.head()

,user_id,url_host,history_len
0,6,"[googleads.g.doubleclick.net, googleads.g.doub...",456
1,7,"[googleads.g.doubleclick.net, googleads.g.doub...",461
2,9,"[avatars.mds.yandex.net, avatars.mds.yandex.ne...",315
3,10,"[ad.adriver.ru, avatars.mds.yandex.net, cstati...",1679
4,11,"[adwile.com, avatars.mds.yandex.net, mail.yand...",571


In [10]:
# tmp[tmp.history_len <= 3]['user_id'].tolist()

import pickle

with open('cold_users.pickle', 'wb') as file:
    pickle.dump(tmp[tmp.history_len <= 3]['user_id'].tolist(), file, protocol=4)

In [20]:
4 /122

0.03278688524590164

In [15]:
data.url_host = data.url_host.apply(lambda x: replace_urls.get(x, x))

In [16]:
data.url_host = data.url_host.apply(lambda x: replace_urls.get(x, x))

In [17]:
data = data.drop_duplicates()
print(data.shape)

(30879871, 2)


In [18]:
data = data[data.url_host.isin(df.url_host.tolist())]
print(data.shape)

(21814775, 2)


In [19]:
data.head()

,user_id,url_host
0,45098,adriver.ru
1,45098,apple.com
2,45098,avatars.mds.yandex.net
6,45098,icloud.com
7,45098,m.avito.ru


In [20]:
data = data.groupby('user_id')['url_host'].agg(list).to_frame().reset_index()
data.head()

,user_id,url_host
0,0,"[s0.2mdn.net, avatars.mds.yandex.net, ads.adfo..."
1,1,"[avatars.mds.yandex.net, google.ru, mail.yande..."
2,2,"[online.sberbank.ru, vk.com, ad.mail.ru, sun9-..."
3,3,"[ad.mail.ru, avatars.mds.yandex.net, mail.yand..."
4,4,"[kp.md, yastatic.net, avatars.mds.yandex.net, ..."


In [23]:
data['lst_len'] = data.url_host.apply(lambda x: len(x))
data.head()

,user_id,url_host,lst_len
0,0,"[s0.2mdn.net, avatars.mds.yandex.net, ads.adfo...",36
1,1,"[avatars.mds.yandex.net, google.ru, mail.yande...",86
2,2,"[online.sberbank.ru, vk.com, ad.mail.ru, sun9-...",36
3,3,"[ad.mail.ru, avatars.mds.yandex.net, mail.yand...",29
4,4,"[kp.md, yastatic.net, avatars.mds.yandex.net, ...",80


In [26]:
data = data[data.lst_len > 5]
print(data.shape)
data.head()

(378135, 3)


,user_id,url_host,lst_len
0,0,"[s0.2mdn.net, avatars.mds.yandex.net, ads.adfo...",36
1,1,"[avatars.mds.yandex.net, google.ru, mail.yande...",86
2,2,"[online.sberbank.ru, vk.com, ad.mail.ru, sun9-...",36
3,3,"[ad.mail.ru, avatars.mds.yandex.net, mail.yand...",29
4,4,"[kp.md, yastatic.net, avatars.mds.yandex.net, ...",80


In [21]:
from sklearn import model_selection
import numpy as np

def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["user_id"], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

data = create_folds(data, num_splits=3)
data.head()

,user_id,url_host,kfold
0,381123,"[avatars.mds.yandex.net, sun9-32.userapi.com, ...",0
1,315481,"[avatars.mds.yandex.net, online.sberbank.ru, s...",0
2,413526,"[apple.com, avatars.mds.yandex.net, yastatic.n...",0
3,354321,"[moevideo.biz, online.sberbank.ru, rtb.com.ru,...",0
4,212722,"[ad.mail.ru, google.com, m.avito.ru, icloud.co...",0


In [22]:
def mean_emb(lst):
    lst = np.array([urls.get(el) for el in lst])
    lst_mean = np.mean(lst, axis=0)
    return lst_mean

In [23]:
data0 = data[data.kfold == 0]
data0['usr_url_embed'] = data0.url_host.parallel_apply(mean_emb)

data1 = data[data.kfold == 1]
data1['usr_url_embed'] = data1.url_host.parallel_apply(mean_emb)

data2 = data[data.kfold == 2]
data2['usr_url_embed'] = data2.url_host.parallel_apply(mean_emb)
data2.head()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_4543/1332537146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data0['usr_url_embed'] = data0.url_host.parallel_apply(mean_emb)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_4543/1332537146.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['usr_url_embed'] = data1.url_host.parallel_apply(mean_emb)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/tmp/ipykernel_4543/1332537146.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['usr_url_embed'] = data2.url_host.parallel_apply(mean_emb)


,user_id,url_host,kfold,usr_url_embed
272099,137590,"[vl.ru, yastatic.net, apple.com, avatars.mds.y...",2,"[-0.01949131668367238, -0.04177820794832179, -..."
272107,152099,"[vk.com, whatsapp.com, sun9-5.userapi.com, exc...",2,"[0.0018688239983320014, -0.04695802130219007, ..."
272118,376345,"[ads.adfox.ru, apple.com, avatars.mds.yandex.n...",2,"[-0.013690567450024753, -0.03990274544160523, ..."
272123,133329,"[ad.mail.ru, ads.adfox.ru, apple.com, avatars....",2,"[-0.013264670883949945, -0.04007700033861198, ..."
272124,135227,"[apple.com, avatars.mds.yandex.net, google.com...",2,"[-0.014196794044226338, -0.03771538332600713, ..."


In [24]:
print(data0.shape)
print(data1.shape)
print(data2.shape)
print(data.shape)

(136687, 4)
(136686, 4)
(136686, 4)
(410059, 3)


In [25]:
len(data0.iloc[0].usr_url_embed)

768

In [26]:
data = pd.concat([data0[['user_id', 'usr_url_embed']], data1[['user_id', 'usr_url_embed']], data2[['user_id', 'usr_url_embed']]])#.to_csv('usr_url_embs.csv', index=False)
print(len(data.iloc[0].usr_url_embed))
data.head()

768


,user_id,usr_url_embed
0,381123,"[-0.006143510648388323, -0.04682461610227299, ..."
1,315481,"[-0.001766734010368207, -0.05124972263155012, ..."
2,413526,"[-0.0023375698365271117, -0.037359521325145424..."
3,354321,"[-0.01750756941762595, -0.040705657512612195, ..."
4,212722,"[-0.002633497538164267, -0.04891790957070695, ..."


In [27]:
type(data.iloc[0].usr_url_embed)

numpy.ndarray

In [28]:
# data.to_csv('usr_url_embs.csv', index=False)
data.to_parquet('usr_url_embs.parquet.gzip', compression='gzip')

In [29]:
# data = pd.read_csv('usr_url_embs.csv')
data = pd.read_parquet('usr_url_embs.parquet.gzip')
print(len(data.iloc[0].usr_url_embed))

768


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 410059 entries, 0 to 410058
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        410059 non-null  int64 
 1   usr_url_embed  410059 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.4+ MB


In [46]:
type(data.iloc[0].usr_url_embed)

numpy.ndarray